In [1]:
import re
import requests
import pandas as pd
from lxml import etree
from datetime import datetime,timedelta
import time
import json
import os

# 认识天天基金网的数据接口

参考：

https://blog.csdn.net/yys1164922014/article/details/100098533?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.channel_param

## 基金代码和基金名称

这是所有基金代码、基金名称简称的集合http://fund.eastmoney.com/js/fundcode_search.js

数据量是接近9000条数据。这个可以简单的用个正则表达，re.findall("(.*?)",url_text)匹配一下存在电脑上。然后再弄成excel文件。比较简单。

In [2]:
url = 'http://fund.eastmoney.com/js/fundcode_search.js'
len_per_fund = 5 # 每个基金的信息有5个字段

In [3]:
response = requests.get(url)
url_text = response.content.decode('utf-8')
url_text_list = re.findall('\"(.*?)\"',url_text)
num5 = int(len(url_text_list)/5)

In [4]:
url_text_list

['000001',
 'HXCZHH',
 '华夏成长混合',
 '混合型-偏股',
 'HUAXIACHENGZHANGHUNHE',
 '000002',
 'HXCZHH',
 '华夏成长混合(后端)',
 '混合型-偏股',
 'HUAXIACHENGZHANGHUNHE',
 '000003',
 'ZHKZZZQA',
 '中海可转债债券A',
 '债券型-可转债',
 'ZHONGHAIKEZHUANZHAIZHAIQUANA',
 '000004',
 'ZHKZZZQC',
 '中海可转债债券C',
 '债券型-可转债',
 'ZHONGHAIKEZHUANZHAIZHAIQUANC',
 '000005',
 'JSZQXYDQZQ',
 '嘉实增强信用定期债券',
 '债券型-长债',
 'JIASHIZENGQIANGXINYONGDINGQIZHAIQUAN',
 '000006',
 'XBLDLHCZHHA',
 '西部利得量化成长混合A',
 '混合型-偏股',
 'XIBULIDELIANGHUACHENGZHANGHUNHEA',
 '000008',
 'JSZZ500ETFLJA',
 '嘉实中证500ETF联接A',
 '指数型-股票',
 'JIASHIZHONGZHENG500ETFLIANJIEA',
 '000009',
 'YFDTTLCHBA',
 '易方达天天理财货币A',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIA',
 '000010',
 'YFDTTLCHBB',
 '易方达天天理财货币B',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIB',
 '000011',
 'HXDPJXHHA',
 '华夏大盘精选混合A',
 '混合型-偏股',
 'HUAXIADAPANJINGXUANHUNHEA',
 '000012',
 'HXDPJXHHA',
 '华夏大盘精选混合A(后端)',
 '混合型-偏股',
 'HUAXIADAPANJINGXUANHUNHEA',
 '000013',
 'YFDTTLCHBR',
 '易方达天天理财货币R',
 '货币型',
 'YIFANGDATIANTIANLICAIHUOBIR',
 '000014

In [5]:
# dict(fund_id:fund_name)
w2dict = {}
for i in range(num5):
    tmp = url_text_list[i*len_per_fund:(i+1)*len_per_fund]
    w2dict[tmp[0]] = dict(
                        suoxie=tmp[1],
                        name=tmp[2], 
                        style=tmp[3],
                        pinyin=tmp[4]
                        )
fjson = json.dumps(w2dict)
f = open('fund_info.json','w')
f.write(fjson)
f.close()

In [6]:
with open("基金列表.txt","w") as f:
    
    wlines = []
    for i in range(num5):
        wline = '\t'.join(url_text_list[i*len_per_fund:(i+1)*len_per_fund])
        wlines.append(wline)
    wlines = '\n'.join(wlines)
    f.write(wlines)

In [21]:
dict(fds=121,)

{'fds': 121}

## 基金历史净值接口

url=‘http://fund.eastmoney.com/f10/F10DataApi.aspx?type=lsjz&code=110022&page=10&sdate=2019-01-01&edate=2019-02-13&per=1’
参数情况一目了然。

url变量：

per=1值代表每页的数据量。最多可以有二十行，per=20。设置更高也没用。

code：基金代码

page：当前是第几页，从1开始到pages为止

sdate：开始日期，在最后一页

edate：结束日期，在第一页

显示变量：

records: 在当前条件下，一共有多少条记录

pages：一共有多少页

curpage:当前页码，与url中的page相同


为了代码更加简洁，我设置的每页数据量是1条。这样就更辛苦电脑了。
由于我们这要爬取全部的历史净值，所以要查询到每只基金的起始日期。

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配

2021-07-19	1.0295	1.0295	-0.38%	场内买入	场内卖出	
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

## 查询基金起始日期

http://fund.eastmoney.com/000001.html?spm=search

自己更改中间的基金代码就好
问题是并不是所有基金都能查到起始日期。所以在用re.findall匹配到起止日期的时候，我又加了个IF语句。以防读取不到，产生错误。

In [7]:
def get_start_date(fund_id):
    # 返回日期字符串
    search_url='http://fund.eastmoney.com/'+fund_id+'.html?spm=search'
    # 获取网页内容；设置网页编码，获取起始日期
    response = requests.get(search_url)
    text = response.content.decode('utf-8')
    
    start_date = re.findall('<td><span class="letterSpace01">成 立 日</span>：(.*?)</td>', text)
    # 对无记录的基金代码进行记录，方便后续清除
    if start_date == []:
        start_date = ['无记录'] 
    return start_date[0] # 如果信息正常就会返回开始日期，如果有错误，就会返回无记录。都是字符串的结果

## 根据基金的每一条数据接口的网址，把那条信息抓下来，返回list结果

一条网址的信息是这样：

var apidata={ content:"
净值日期	单位净值	累计净值	日增长率	申购状态	赎回状态	分红送配
2021-07-16	1.0334	1.0334	-0.03%	场内买入	场内卖出	
",records:414,pages:207,curpage:21};

In [8]:
response = requests.get('https://fundf10.eastmoney.com/F10DataApi.aspx?type=lsjz&code=512000&page=21&sdate=2020-01-01&edate=2021-09-13&per=1')
text = response.text
print(text)
text_html = re.findall(',records:(.*?),pages:',text)
#html = etree.HTML(text_html)
print(text_html)



var apidata={ content:"<table class='w782 comm lsjz'><thead><tr><th class='first'>净值日期</th><th>单位净值</th><th>累计净值</th><th>日增长率</th><th>申购状态</th><th>赎回状态</th><th class='tor last'>分红送配</th></tr></thead><tbody><tr><td>2021-08-16</td><td class='tor bold'>1.0422</td><td class='tor bold'>1.0422</td><td class='tor bold red'>0.80%</td><td>场内买入</td><td>场内卖出</td><td class='red unbold'></td></tr></tbody></table>",records:414,pages:414,curpage:21};
['414']


In [9]:
def make_fund_json(fund_info_url='http://fund.eastmoney.com/js/fundcode_search.js',save=False):
    url = fund_info_url
    len_per_fund = 5 # 每个基金的信息有5个字段
    
    response = requests.get(url)
    url_text = response.content.decode('utf-8')
    url_text_list = re.findall('\"(.*?)\"',url_text)
    num5 = int(len(url_text_list)/len_per_fund)
    
    # dict(fund_id:fund_name)
    w2dict = {}
    for i in range(num5):
        tmp = url_text_list[i*len_per_fund:(i+1)*len_per_fund]
        w2dict[tmp[0]] = dict(
                            suoxie=tmp[1],
                            name=tmp[2], 
                            style=tmp[3],
                            pinyin=tmp[4]
                            )
    fjson = json.dumps(w2dict)
    if save:
        f = open('fund_info.json','w')
        f.write(fjson)
        f.close()
    return w2dict

In [10]:
def get_fund_json(fund_json_path='fund_info.json'):
    f = open('fund_info.json','r')
    fjson = f.read()
    fund_dict = json.loads(fjson)
    f.close()
    return fund_dict #"suoxie": "HXCZHH", "name": "\u534e\u590f\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGHUNHE"}, "000002": {"suoxie": "HXCZHH", "name": "\u534e\u590f\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGHUNHE"}, "000003": {"suoxie": "ZHKZZZQA", "name": "\u4e2d\u6d77\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGHAIKEZHUANZHAIZHAIQUANA"}, "000004": {"suoxie": "ZHKZZZQC", "name": "\u4e2d\u6d77\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGHAIKEZHUANZHAIZHAIQUANC"}, "000005": {"suoxie": "JSZQXYDQZQ", "name": "\u5609\u5b9e\u589e\u5f3a\u4fe1\u7528\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZENGQIANGXINYONGDINGQIZHAIQUAN"}, "000006": {"suoxie": "XBLDLHCZHHA", "name": "\u897f\u90e8\u5229\u5f97\u91cf\u5316\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDELIANGHUACHENGZHANGHUNHEA"}, "000008": {"suoxie": "JSZZ500ETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500ETFLIANJIEA"}, "000009": {"suoxie": "YFDTTLCHBA", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u7406\u8d22\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANLICAIHUOBIA"}, "000010": {"suoxie": "YFDTTLCHBB", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u7406\u8d22\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANLICAIHUOBIB"}, "000011": {"suoxie": "HXDPJXHHA", "name": "\u534e\u590f\u5927\u76d8\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIADAPANJINGXUANHUNHEA"}, "000012": {"suoxie": "HXDPJXHHA", "name": "\u534e\u590f\u5927\u76d8\u7cbe\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIADAPANJINGXUANHUNHEA"}, "000013": {"suoxie": "YFDTTLCHBR", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u7406\u8d22\u8d27\u5e01R", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANLICAIHUOBIR"}, "000014": {"suoxie": "HXJLZQ", "name": "\u534e\u590f\u805a\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAJULIZHAIQUAN"}, "000015": {"suoxie": "HXCZZQA", "name": "\u534e\u590f\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIACHUNZHAIZHAIQUANA"}, "000016": {"suoxie": "HXCZZQC", "name": "\u534e\u590f\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIACHUNZHAIZHAIQUANC"}, "000017": {"suoxie": "CTKCXHH", "name": "\u8d22\u901a\u53ef\u6301\u7eed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGKECHIXUHUNHE"}, "000020": {"suoxie": "JSCCPZTZHH", "name": "\u666f\u987a\u957f\u57ce\u54c1\u8d28\u6295\u8d44\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGPINZHITOUZIHUNHE"}, "000021": {"suoxie": "HXYSZZHH", "name": "\u534e\u590f\u4f18\u52bf\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYOUSHIZENGZHANGHUNHE"}, "000024": {"suoxie": "DMSLZQZQA", "name": "\u5927\u6469\u53cc\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOSHUANGLIZENGQIANGZHAIQUANA"}, "000025": {"suoxie": "DMSLZQZQC", "name": "\u5927\u6469\u53cc\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOSHUANGLIZENGQIANGZHAIQUANC"}, "000028": {"suoxie": "HFAXZQ", "name": "\u534e\u5bcc\u5b89\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANXINZHAIQUAN"}, "000029": {"suoxie": "FGHGCLLHPZHHA", "name": "\u5bcc\u56fd\u5b8f\u89c2\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHONGGUANCELUELINGHUOPEIZHIHUNHEA"}, "000030": {"suoxie": "CCHXYXHH", "name": "\u957f\u57ce\u6838\u5fc3\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGHEXINYOUXUANHUNHE"}, "000031": {"suoxie": "HXFXHH", "name": "\u534e\u590f\u590d\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAFUXINGHUNHE"}, "000032": {"suoxie": "YFDXYZZQA", "name": "\u6613\u65b9\u8fbe\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAXINYONGZHAIZHAIQUANA"}, "000033": {"suoxie": "YFDXYZZQC", "name": "\u6613\u65b9\u8fbe\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAXINYONGZHAIZHAIQUANC"}, "000037": {"suoxie": "GFJNZQA", "name": "\u5e7f\u53d1\u666f\u5b81\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGNINGZHAIQUANA"}, "000039": {"suoxie": "NYGZZHH", "name": "\u519c\u94f6\u9ad8\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINGAOZENGZHANGHUNHE"}, "000041": {"suoxie": "HXQQGPQDII", "name": "\u534e\u590f\u5168\u7403\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "HUAXIAQUANQIUGUPIAOQDII"}, "000042": {"suoxie": "CTZZESG100ZSZQA", "name": "\u8d22\u901a\u4e2d\u8bc1ESG100\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZHONGZHENGESG100ZHISHUZENGQIANGA"}, "000043": {"suoxie": "JSMGCZGPRMB", "name": "\u5609\u5b9e\u7f8e\u56fd\u6210\u957f\u80a1\u7968\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "JIASHIMEIGUOCHENGZHANGGUPIAORENMINBI"}, "000044": {"suoxie": "JSMGCZGPMYXH", "name": "\u5609\u5b9e\u7f8e\u56fd\u6210\u957f\u80a1\u7968\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "JIASHIMEIGUOCHENGZHANGGUPIAOMEIYUANXIANHUI"}, "000045": {"suoxie": "GYCYZZQA", "name": "\u5de5\u94f6\u4ea7\u4e1a\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINCHANYEZHAIZHAIQUANA"}, "000046": {"suoxie": "GYCYZZQB", "name": "\u5de5\u94f6\u4ea7\u4e1a\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINCHANYEZHAIZHAIQUANB"}, "000047": {"suoxie": "HXSZZQA", "name": "\u534e\u590f\u53cc\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIASHUANGZHAIZHAIQUANA"}, "000048": {"suoxie": "HXSZZQC", "name": "\u534e\u590f\u53cc\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIASHUANGZHAIZHAIQUANC"}, "000049": {"suoxie": "ZYBPQQZYDQZZS", "name": "\u4e2d\u94f6\u6807\u666e\u5168\u7403\u8d44\u6e90\u7b49\u6743\u91cd\u6307\u6570", "style": "QDII", "pinyin": "ZHONGYINBIAOPUQUANQIUZIYUANDENGQUANZHONGZHISHU"}, "000051": {"suoxie": "HXHS300ETFLJA", "name": "\u534e\u590f\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ETFLIANJIEA"}, "000053": {"suoxie": "PHYCYNDKZQ", "name": "\u9e4f\u534e\u6c38\u8bda\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGCHENGYINIANDINGKAIZHAIQUAN"}, "000054": {"suoxie": "PHSZZLZQ", "name": "\u9e4f\u534e\u53cc\u503a\u589e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUASHUANGZHAIZENGLIZHAIQUAN"}, "000055": {"suoxie": "GFNSDK100MYXHA", "name": "\u5e7f\u53d1\u7eb3\u65af\u8fbe\u514b100\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "GUANGFANASIDAKE100MEIYUANXIANHUIA"}, "000056": {"suoxie": "JXXFSJHH", "name": "\u5efa\u4fe1\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINXIAOFEISHENGJIHUNHE"}, "000057": {"suoxie": "ZYXFZTHH", "name": "\u4e2d\u94f6\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINXIAOFEIZHUTIHUNHE"}, "000058": {"suoxie": "GLAATLHPZHH", "name": "\u56fd\u8054\u5b89\u5b89\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANANTAILINGHUOPEIZHIHUNHE"}, "000059": {"suoxie": "GLAZZYY100A", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u533b\u836f100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGYIYAO100A"}, "000061": {"suoxie": "HXSSHH", "name": "\u534e\u590f\u76db\u4e16\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIASHENGSHIHUNHE"}, "000063": {"suoxie": "CSDZXXZTLHPZHH", "name": "\u957f\u76db\u7535\u5b50\u4fe1\u606f\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGDIANZIXINXIZHUTILINGHUOPEIZHIHUNHE"}, "000064": {"suoxie": "DM18GYDKZ", "name": "\u5927\u646918\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMO18GEYUEDINGKAIZHAI"}, "000065": {"suoxie": "GFJDQDHH", "name": "\u56fd\u5bcc\u7126\u70b9\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUJIAODIANQUDONGHUNHE"}, "000066": {"suoxie": "NAHXHHA", "name": "\u8bfa\u5b89\u9e3f\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANHONGXINHUNHEA"}, "000067": {"suoxie": "MSJYZZYXA", "name": "\u6c11\u751f\u52a0\u94f6\u8f6c\u503a\u4f18\u9009A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "MINSHENGJIAYINZHUANZHAIYOUXUANA"}, "000068": {"suoxie": "MSJYZZYXC", "name": "\u6c11\u751f\u52a0\u94f6\u8f6c\u503a\u4f18\u9009C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "MINSHENGJIAYINZHUANZHAIYOUXUANC"}, "000069": {"suoxie": "GTRYZGDJZQA", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINZHONGGAODENGJIZHAIQUANA"}, "000070": {"suoxie": "GTRYZGDJZQC", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINZHONGGAODENGJIZHAIQUANC"}, "000071": {"suoxie": "HXHSETFLJA", "name": "\u534e\u590f\u6052\u751fETF\u8054\u63a5A", "style": "QDII", "pinyin": "HUAXIAHENGSHENGETFLIANJIEA"}, "000072": {"suoxie": "HAWJHBHH", "name": "\u534e\u5b89\u7a33\u5065\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANWENJIANHUIBAOHUNHE"}, "000073": {"suoxie": "STMGCZDLHH", "name": "\u4e0a\u6295\u6469\u6839\u6210\u957f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENCHENGZHANGDONGLIHUNHE"}, "000074": {"suoxie": "GYXYCZYNDKZA", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAIYINIANDINGKAIZHAIA"}, "000075": {"suoxie": "HXHSETFLJXH", "name": "\u534e\u590f\u6052\u751fETF\u8054\u63a5\u73b0\u6c47", "style": "QDII", "pinyin": "HUAXIAHENGSHENGETFLIANJIEXIANHUI"}, "000076": {"suoxie": "HXHSETFLJXC", "name": "\u534e\u590f\u6052\u751fETF\u8054\u63a5\u73b0\u949e", "style": "QDII", "pinyin": "HUAXIAHENGSHENGETFLIANJIEXIANCHAO"}, "000077": {"suoxie": "GYXYCZYNDKZC", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAIYINIANDINGKAIZHAIC"}, "000078": {"suoxie": "GYXYCZSGYDKZA", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAISANGEYUEDINGKAIZHAIA"}, "000079": {"suoxie": "GYXYCZSGYDKZC", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAISANGEYUEDINGKAIZHAIC"}, "000080": {"suoxie": "TZKZZZQZQA", "name": "\u5929\u6cbb\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "TIANZHIKEZHUANZHAIZENGQIANGZHAIQUANA"}, "000081": {"suoxie": "TZKZZZQZQC", "name": "\u5929\u6cbb\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "TIANZHIKEZHUANZHAIZENGQIANGZHAIQUANC"}, "000082": {"suoxie": "JSYJAEFGP", "name": "\u5609\u5b9e\u7814\u7a76\u963f\u5c14\u6cd5\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIYANJIUAERFAGUPIAO"}, "000083": {"suoxie": "HTFXFHYHH", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUXIAOFEIHANGYEHUNHE"}, "000084": {"suoxie": "BSAYZQA", "name": "\u535a\u65f6\u5b89\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BOSHIANYINGZHAIQUANA"}, "000085": {"suoxie": "BSAYZQC", "name": "\u535a\u65f6\u5b89\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BOSHIANYINGZHAIQUANC"}, "000086": {"suoxie": "NFWL1NDKZA", "name": "\u5357\u65b9\u7a33\u52291\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGWENLI1NIANDINGKAIZHAIA"}, "000087": {"suoxie": "JSZZJBGZETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u91d1\u8fb9\u56fd\u503aETF\u8054\u63a5A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHENGJINBIANGUOZHAIETFLIANJIEA"}, "000088": {"suoxie": "JSZZJBGZETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u91d1\u8fb9\u56fd\u503aETF\u8054\u63a5C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHENGJINBIANGUOZHAIETFLIANJIEC"}, "000089": {"suoxie": "MSJYGDJXYZC", "name": "\u6c11\u751f\u52a0\u94f6\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINGAODENGJIXINYONGZHAIC"}, "000090": {"suoxie": "MSJYGDJXYZA", "name": "\u6c11\u751f\u52a0\u94f6\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINGAODENGJIXINYONGZHAIA"}, "000103": {"suoxie": "GTZGQYJWGSYZ", "name": "\u56fd\u6cf0\u4e2d\u56fd\u4f01\u4e1a\u5883\u5916\u9ad8\u6536\u76ca\u503a", "style": "QDII", "pinyin": "GUOTAIZHONGGUOQIYEJINGWAIGAOSHOUYIZHAI"}, "000104": {"suoxie": "HCWLWJTLZQA", "name": "\u534e\u5bb8\u672a\u6765\u7a33\u5065\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUACHENWEILAIWENJIANTIANLIZHAIQUANA"}, "000105": {"suoxie": "JXAXHBZQA", "name": "\u5efa\u4fe1\u5b89\u5fc3\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINANXINHUIBAOZHAIQUANA"}, "000106": {"suoxie": "JXAXHBZQC", "name": "\u5efa\u4fe1\u5b89\u5fc3\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINANXINHUIBAOZHAIQUANC"}, "000107": {"suoxie": "FGWJZQZQAB", "name": "\u5bcc\u56fd\u7a33\u5065\u589e\u5f3a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOWENJIANZENGQIANGZHAIQUANAB"}, "000108": {"suoxie": "FGWJZQZQAB", "name": "\u5bcc\u56fd\u7a33\u5065\u589e\u5f3a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOWENJIANZENGQIANGZHAIQUANAB"}, "000109": {"suoxie": "FGWJZQZQC", "name": "\u5bcc\u56fd\u7a33\u5065\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOWENJIANZENGQIANGZHAIQUANC"}, "000110": {"suoxie": "JYYAHHA", "name": "\u91d1\u9e70\u5143\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGYUANANHUNHEA"}, "000111": {"suoxie": "YFDCZ1NDKZA", "name": "\u6613\u65b9\u8fbe\u7eaf\u503a1\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDACHUNZHAI1NIANDINGKAIZHAIA"}, "000112": {"suoxie": "YFDCZ1NDKZC", "name": "\u6613\u65b9\u8fbe\u7eaf\u503a1\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDACHUNZHAI1NIANDINGKAIZHAIC"}, "000116": {"suoxie": "JSFYCZDQZQ", "name": "\u5609\u5b9e\u4e30\u76ca\u7eaf\u503a\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIFENGYICHUNZHAIDINGQIZHAIQUAN"}, "000117": {"suoxie": "GFLDPZHH", "name": "\u5e7f\u53d1\u8f6e\u52a8\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFALUNDONGPEIZHIHUNHE"}, "000118": {"suoxie": "GFJXZQA", "name": "\u5e7f\u53d1\u805a\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJUXINZHAIQUANA"}, "000119": {"suoxie": "GFJXZQC", "name": "\u5e7f\u53d1\u805a\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJUXINZHAIQUANC"}, "000120": {"suoxie": "ZYMLZGHH", "name": "\u4e2d\u94f6\u7f8e\u4e3d\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINMEILIZHONGGUOHUNHE"}, "000121": {"suoxie": "HXYFHHA", "name": "\u534e\u590f\u6c38\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGFUHUNHEA"}, "000122": {"suoxie": "HTFSYZZQA", "name": "\u6c47\u6dfb\u5bcc\u5b9e\u4e1a\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUSHIYEZHAIZHAIQUANA"}, "000123": {"suoxie": "HTFSYZZQC", "name": "\u6c47\u6dfb\u5bcc\u5b9e\u4e1a\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUSHIYEZHAIZHAIQUANC"}, "000124": {"suoxie": "HBFWYXHH", "name": "\u534e\u5b9d\u670d\u52a1\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOFUWUYOUXUANHUNHE"}, "000126": {"suoxie": "ZSARHH", "name": "\u62db\u5546\u5b89\u6da6\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANRUNHUNHE"}, "000127": {"suoxie": "NYHYLXHH", "name": "\u519c\u94f6\u884c\u4e1a\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHANGYELINGXIANHUNHE"}, "000128": {"suoxie": "DCJADRZQA", "name": "\u5927\u6210\u666f\u5b89\u77ed\u878d\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGANDUANRONGZHAIQUANA"}, "000129": {"suoxie": "DCJADRZQB", "name": "\u5927\u6210\u666f\u5b89\u77ed\u878d\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGANDUANRONGZHAIQUANB"}, "000130": {"suoxie": "DCJXXYZZQA", "name": "\u5927\u6210\u666f\u5174\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGXINGXINYONGZHAIZHAIQUANA"}, "000131": {"suoxie": "DCJXXYZZQC", "name": "\u5927\u6210\u666f\u5174\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGXINGXINYONGZHAIZHAIQUANC"}, "000134": {"suoxie": "ZXBCJHZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5609\u9e3f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJIAHONGZHAIQUANA"}, "000135": {"suoxie": "ZXBCJHZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5609\u9e3f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJIAHONGZHAIQUANC"}, "000136": {"suoxie": "MSJYCLJXHHA", "name": "\u6c11\u751f\u52a0\u94f6\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINCELUEJINGXUANHUNHEA"}, "000137": {"suoxie": "MSJYSSZLZQA", "name": "\u6c11\u751f\u52a0\u94f6\u5c81\u5c81\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINSUISUIZENGLIZHAIQUANA"}, "000138": {"suoxie": "MSJYSSZLZQC", "name": "\u6c11\u751f\u52a0\u94f6\u5c81\u5c81\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINSUISUIZENGLIZHAIQUANC"}, "000139": {"suoxie": "FGGYQYZZQAB", "name": "\u5bcc\u56fd\u56fd\u6709\u4f01\u4e1a\u503a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOGUOYOUQIYEZHAIZHAIQUANAB"}, "000140": {"suoxie": "FGGYQYZZQAB", "name": "\u5bcc\u56fd\u56fd\u6709\u4f01\u4e1a\u503a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOGUOYOUQIYEZHAIZHAIQUANAB"}, "000141": {"suoxie": "FGGYQYZZQC", "name": "\u5bcc\u56fd\u56fd\u6709\u4f01\u4e1a\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOGUOYOUQIYEZHAIZHAIQUANC"}, "000142": {"suoxie": "RTZQSYZQA", "name": "\u878d\u901a\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGZENGQIANGSHOUYIZHAIQUANA"}, "000143": {"suoxie": "PHSZJLZQA", "name": "\u9e4f\u534e\u53cc\u503a\u52a0\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUASHUANGZHAIJIALIZHAIQUANA"}, "000147": {"suoxie": "YFDGDJXYZZQA", "name": "\u6613\u65b9\u8fbe\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAGAODENGJIXINYONGZHAIZHAIQUANA"}, "000148": {"suoxie": "YFDGDJXYZZQC", "name": "\u6613\u65b9\u8fbe\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAGAODENGJIXINYONGZHAIZHAIQUANC"}, "000149": {"suoxie": "HASZTLZQA", "name": "\u534e\u5b89\u53cc\u503a\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANSHUANGZHAITIANLIZHAIQUANA"}, "000150": {"suoxie": "HASZTLZQC", "name": "\u534e\u5b89\u53cc\u503a\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANSHUANGZHAITIANLIZHAIQUANC"}, "000152": {"suoxie": "DCJXCZZQA", "name": "\u5927\u6210\u666f\u65ed\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGXUCHUNZHAIZHAIQUANA"}, "000153": {"suoxie": "DCJXCZZQC", "name": "\u5927\u6210\u666f\u65ed\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGXUCHUNZHAIZHAIQUANC"}, "000154": {"suoxie": "FGHS300ZSZQA", "name": "\u5bcc\u56fd\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOHUSHEN300ZHISHUZENGQIANGA"}, "000155": {"suoxie": "FGTZTSZTLDHH", "name": "\u5bcc\u56fd\u901a\u80c0\u901a\u7f29\u4e3b\u9898\u8f6e\u52a8\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTONGZHANGTONGSUOZHUTILUNDONGHUNHE"}, "000157": {"suoxie": "FGQQKJHLWQDII", "name": "\u5bcc\u56fd\u5168\u7403\u79d1\u6280\u4e92\u8054\u7f51(QDII)(\u540e\u7aef)", "style": "QDII", "pinyin": "FUGUOQUANQIUKEJIHULIANWANGQDII"}, "000158": {"suoxie": "FGDTHBHH", "name": "\u5bcc\u56fd\u4f4e\u78b3\u73af\u4fdd\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODITANHUANBAOHUNHE"}, "000159": {"suoxie": "FGGXJSCYHH", "name": "\u5bcc\u56fd\u9ad8\u65b0\u6280\u672f\u4ea7\u4e1a\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGAOXINJISHUCHANYEHUNHE"}, "000160": {"suoxie": "FGQHBDKZAB", "name": "\u5bcc\u56fd\u5f3a\u56de\u62a5\u5b9a\u5f00\u503aA/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOQIANGHUIBAODINGKAIZHAIAB"}, "000161": {"suoxie": "FGHGCLLHPZHHA", "name": "\u5bcc\u56fd\u5b8f\u89c2\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHONGGUANCELUELINGHUOPEIZHIHUNHEA"}, "000162": {"suoxie": "FGZGZXPHHQDIIRMB", "name": "\u5bcc\u56fd\u4e2d\u56fd\u4e2d\u5c0f\u76d8\u6df7\u5408(QDII)\u4eba\u6c11\u5e01(\u540e\u7aef)", "style": "QDII", "pinyin": "FUGUOZHONGGUOZHONGXIAOPANHUNHEQDIIRENMINBI"}, "000164": {"suoxie": "FGSZZSETFLJA", "name": "\u5bcc\u56fd\u4e0a\u8bc1\u6307\u6570ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOSHANGZHENGZHISHUETFLIANJIEA"}, "000165": {"suoxie": "GTRYCLJXHH", "name": "\u56fd\u6295\u745e\u94f6\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINCELUEJINGXUANHUNHE"}, "000166": {"suoxie": "ZHXXCYJXHH", "name": "\u4e2d\u6d77\u4fe1\u606f\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIXINXICHANYEJINGXUANHUNHE"}, "000167": {"suoxie": "GFJYLHPZHHA", "name": "\u5e7f\u53d1\u805a\u4f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJUYOULINGHUOPEIZHIHUNHEA"}, "000171": {"suoxie": "YFDYFHBZQ", "name": "\u6613\u65b9\u8fbe\u88d5\u4e30\u56de\u62a5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUFENGHUIBAOZHAIQUAN"}, "000172": {"suoxie": "HTBRLHZQHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u589e\u5f3a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAZENGQIANGHUNHEA"}, "000173": {"suoxie": "HTFML30HH", "name": "\u6c47\u6dfb\u5bcc\u7f8e\u4e3d30\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUMEILI30HUNHE"}, "000174": {"suoxie": "HTFGXZZQA", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u606f\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUGAOXIZHAIZHAIQUANA"}, "000175": {"suoxie": "HTFGXZZQC", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u606f\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUGAOXIZHAIZHAIQUANC"}, "000176": {"suoxie": "JSHS300ZSYJZQ", "name": "\u5609\u5b9e\u6caa\u6df1300\u6307\u6570\u7814\u7a76\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300ZHISHUYANJIUZENGQIANG"}, "000179": {"suoxie": "GFMGFDCZS", "name": "\u5e7f\u53d1\u7f8e\u56fd\u623f\u5730\u4ea7\u6307\u6570", "style": "QDII", "pinyin": "GUANGFAMEIGUOFANGDICHANZHISHU"}, "000180": {"suoxie": "GFMGFDCZSXH", "name": "\u5e7f\u53d1\u7f8e\u56fd\u623f\u5730\u4ea7\u6307\u6570\u73b0\u6c47", "style": "QDII", "pinyin": "GUANGFAMEIGUOFANGDICHANZHISHUXIANHUI"}, "000181": {"suoxie": "JSCCSJJLZQA", "name": "\u666f\u987a\u957f\u57ce\u56db\u5b63\u91d1\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGSIJIJINLIZHAIQUANA"}, "000182": {"suoxie": "JSCCSJJLZQC", "name": "\u666f\u987a\u957f\u57ce\u56db\u5b63\u91d1\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGSIJIJINLIZHAIQUANC"}, "000183": {"suoxie": "JSFYCLDQZQ", "name": "\u5609\u5b9e\u4e30\u76ca\u7b56\u7565\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIFENGYICELUEDINGQIZHAIQUAN"}, "000184": {"suoxie": "GYTFZQA", "name": "\u5de5\u94f6\u6dfb\u798f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANFUZHAIQUANA"}, "000185": {"suoxie": "GYTFZQB", "name": "\u5de5\u94f6\u6dfb\u798f\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANFUZHAIQUANB"}, "000186": {"suoxie": "HTBRJJHZQ", "name": "\u534e\u6cf0\u67cf\u745e\u5b63\u5b63\u7ea2\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIJIJIHONGZHAIQUAN"}, "000187": {"suoxie": "HTBRFSCZZQA", "name": "\u534e\u6cf0\u67cf\u745e\u4e30\u76db\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIFENGSHENGCHUNZHAIZHAIQUANA"}, "000188": {"suoxie": "HTBRFSCZZQC", "name": "\u534e\u6cf0\u67cf\u745e\u4e30\u76db\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIFENGSHENGCHUNZHAIZHAIQUANC"}, "000189": {"suoxie": "YFDFHZQA", "name": "\u6613\u65b9\u8fbe\u4e30\u534e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAFENGHUAZHAIQUANA"}, "000190": {"suoxie": "ZYXHBLHPZHHA", "name": "\u4e2d\u94f6\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINHUIBAOLINGHUOPEIZHIHUNHEA"}, "000191": {"suoxie": "FGXYZZQAB", "name": "\u5bcc\u56fd\u4fe1\u7528\u503a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXINYONGZHAIZHAIQUANAB"}, "000192": {"suoxie": "FGXYZZQC", "name": "\u5bcc\u56fd\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXINYONGZHAIZHAIQUANC"}, "000194": {"suoxie": "YHXYSJHZQA", "name": "\u94f6\u534e\u4fe1\u7528\u56db\u5b63\u7ea2\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGSIJIHONGZHAIQUANA"}, "000195": {"suoxie": "GYCZSYHHA", "name": "\u5de5\u94f6\u6210\u957f\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINCHENGZHANGSHOUYIHUNHEA"}, "000196": {"suoxie": "GYCZSYHHB", "name": "\u5de5\u94f6\u6210\u957f\u6536\u76ca\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINCHENGZHANGSHOUYIHUNHEB"}, "000197": {"suoxie": "FGMBSYYNQCZZQ", "name": "\u5bcc\u56fd\u76ee\u6807\u6536\u76ca\u4e00\u5e74\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOMUBIAOSHOUYIYINIANQICHUNZHAIZHAIQUAN"}, "000198": {"suoxie": "THYEBHB", "name": "\u5929\u5f18\u4f59\u989d\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGYUEBAOHUOBI"}, "000199": {"suoxie": "GTLHCLSYHH", "name": "\u56fd\u6cf0\u91cf\u5316\u7b56\u7565\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAILIANGHUACELUESHOUYIHUNHE"}, "000200": {"suoxie": "BSSSZLYNDKZ", "name": "\u535a\u65f6\u5c81\u5c81\u589e\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISUISUIZENGLIYINIANDINGKAIZHAI"}, "000201": {"suoxie": "NATXYNDKZA", "name": "\u8bfa\u5b89\u6cf0\u946b\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANTAIXINYINIANDINGKAIZHAIA"}, "000203": {"suoxie": "GFRRSYHBA", "name": "\u56fd\u5bcc\u65e5\u65e5\u6536\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOFURIRISHOUYIHUOBIA"}, "000204": {"suoxie": "GFRRSYHBB", "name": "\u56fd\u5bcc\u65e5\u65e5\u6536\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOFURIRISHOUYIHUOBIB"}, "000205": {"suoxie": "YFDTZJXYZZQA", "name": "\u6613\u65b9\u8fbe\u6295\u8d44\u7ea7\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDATOUZIJIXINYONGZHAIZHAIQUANA"}, "000206": {"suoxie": "YFDTZJXYZZQC", "name": "\u6613\u65b9\u8fbe\u6295\u8d44\u7ea7\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDATOUZIJIXINYONGZHAIZHAIQUANC"}, "000207": {"suoxie": "JXSZZQZQA", "name": "\u5efa\u4fe1\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHUANGZHAIZENGQIANGZHAIQUANA"}, "000208": {"suoxie": "JXSZZQZQC", "name": "\u5efa\u4fe1\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHUANGZHAIZENGQIANGZHAIQUANC"}, "000209": {"suoxie": "XCXXCYHHA", "name": "\u4fe1\u8bda\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGXINXINGCHANYEHUNHEA"}, "000210": {"suoxie": "GDXJBHBA", "name": "\u5149\u5927\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAXIANJINBAOHUOBIA"}, "000211": {"suoxie": "GDXJBHBB", "name": "\u5149\u5927\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAXIANJINBAOHUOBIB"}, "000212": {"suoxie": "TXXYDQKFA", "name": "\u6cf0\u4fe1\u946b\u76ca\u5b9a\u671f\u5f00\u653eA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINXINYIDINGQIKAIFANGA"}, "000213": {"suoxie": "TXXYDQKFC", "name": "\u6cf0\u4fe1\u946b\u76ca\u5b9a\u671f\u5f00\u653eC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINXINYIDINGQIKAIFANGC"}, "000214": {"suoxie": "GFCZYXHH", "name": "\u5e7f\u53d1\u6210\u957f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFACHENGZHANGYOUXUANHUNHE"}, "000215": {"suoxie": "GFQSYXLHPZHHA", "name": "\u5e7f\u53d1\u8d8b\u52bf\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAQUSHIYOUXUANLINGHUOPEIZHIHUNHEA"}, "000216": {"suoxie": "HAHJYETFLJA", "name": "\u534e\u5b89\u9ec4\u91d1\u6613ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAANHUANGJINYIETFLIANJIEA"}, "000217": {"suoxie": "HAHJYETFLJC", "name": "\u534e\u5b89\u9ec4\u91d1\u6613ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAANHUANGJINYIETFLIANJIEC"}, "000218": {"suoxie": "GTHJETFLJA", "name": "\u56fd\u6cf0\u9ec4\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUOTAIHUANGJINETFLIANJIEA"}, "000219": {"suoxie": "BSYYHH", "name": "\u535a\u65f6\u88d5\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYUYIHUNHE"}, "000220": {"suoxie": "FGYLBJHYHHA", "name": "\u5bcc\u56fd\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYILIAOBAOJIANHANGYEHUNHEA"}, "000221": {"suoxie": "HTFNNLDKZA", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUNIANNIANLIDINGKAIZHAIA"}, "000222": {"suoxie": "HTFNNLDKZC", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUNIANNIANLIDINGKAIZHAIC"}, "000227": {"suoxie": "HANNHZQA", "name": "\u534e\u5b89\u5e74\u5e74\u7ea2\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANNIANNIANHONGZHAIQUANA"}, "000235": {"suoxie": "NAWGSYYNDKZ", "name": "\u8bfa\u5b89\u7a33\u56fa\u6536\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANWENGUSHOUYIYINIANDINGKAIZHAI"}, "000236": {"suoxie": "GYYYXDQZFZQA", "name": "\u5de5\u94f6\u6708\u6708\u85aa\u5b9a\u671f\u652f\u4ed8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINYUEYUEXINDINGQIZHIFUZHAIQUANA"}, "000239": {"suoxie": "HANNYDKZA", "name": "\u534e\u5b89\u5e74\u5e74\u76c8\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANNIANNIANYINGDINGKAIZHAIA"}, "000240": {"suoxie": "HANNYDKZC", "name": "\u534e\u5b89\u5e74\u5e74\u76c8\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANNIANNIANYINGDINGKAIZHAIC"}, "000241": {"suoxie": "BYHXYSHHC", "name": "\u5b9d\u76c8\u6838\u5fc3\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGHEXINYOUSHIHUNHEC"}, "000242": {"suoxie": "JSCCCLJXLHPZHH", "name": "\u666f\u987a\u957f\u57ce\u7b56\u7565\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGCELUEJINGXUANLINGHUOPEIZHIHUNHE"}, "000244": {"suoxie": "THWLDQKFA", "name": "\u5929\u5f18\u7a33\u5229\u5b9a\u671f\u5f00\u653eA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGWENLIDINGQIKAIFANGA"}, "000245": {"suoxie": "THWLDQKFB", "name": "\u5929\u5f18\u7a33\u5229\u5b9a\u671f\u5f00\u653eB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGWENLIDINGQIKAIFANGB"}, "000246": {"suoxie": "BSYYXDQZFZQ", "name": "\u535a\u65f6\u6708\u6708\u85aa\u5b9a\u671f\u652f\u4ed8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUEYUEXINDINGQIZHIFUZHAIQUAN"}, "000248": {"suoxie": "HTFZZZYXFETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHUYAOXIAOFEIETFLIANJIEA"}, "000251": {"suoxie": "GYJRDCHHA", "name": "\u5de5\u94f6\u91d1\u878d\u5730\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINRONGDICHANHUNHEA"}, "000252": {"suoxie": "JSCCJXXYCZZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u5174\u4fe1\u7528\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGXINGXINYONGCHUNZHAIZHAIQUANA"}, "000253": {"suoxie": "JSCCJXXYCZZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u5174\u4fe1\u7528\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGXINGXINYONGCHUNZHAIZHAIQUANC"}, "000254": {"suoxie": "CCDKZA", "name": "\u957f\u57ce\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGDINGKAIZHAIA"}, "000255": {"suoxie": "CCDKZC", "name": "\u957f\u57ce\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGDINGKAIZHAIC"}, "000259": {"suoxie": "NYQJSYHH", "name": "\u519c\u94f6\u533a\u95f4\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINQUJIANSHOUYIHUNHE"}, "000263": {"suoxie": "GYXXCYHHA", "name": "\u5de5\u94f6\u4fe1\u606f\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINXICHANYEHUNHEA"}, "000264": {"suoxie": "BSNXZZHHA", "name": "\u535a\u65f6\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHINEIXUZENGZHANGHUNHEA"}, "000265": {"suoxie": "YFDHJTL1NDKZA", "name": "\u6613\u65b9\u8fbe\u6052\u4e45\u6dfb\u52291\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGJIUTIANLI1NIANDINGKAIZHAIA"}, "000266": {"suoxie": "YFDHJTL1NDKZC", "name": "\u6613\u65b9\u8fbe\u6052\u4e45\u6dfb\u52291\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGJIUTIANLI1NIANDINGKAIZHAIC"}, "000267": {"suoxie": "GFJLYNDKZA", "name": "\u5e7f\u53d1\u96c6\u5229\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJILIYINIANDINGKAIZHAIA"}, "000268": {"suoxie": "GFJLYNDKZC", "name": "\u5e7f\u53d1\u96c6\u5229\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJILIYINIANDINGKAIZHAIC"}, "000270": {"suoxie": "JXLHPZHH", "name": "\u5efa\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINLINGHUOPEIZHIHUNHE"}, "000271": {"suoxie": "ZYDKZA", "name": "\u4e2d\u90ae\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUDINGKAIZHAIA"}, "000272": {"suoxie": "ZYDKZC", "name": "\u4e2d\u90ae\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUDINGKAIZHAIC"}, "000273": {"suoxie": "HRYDAXLHPZHHA", "name": "\u534e\u6da6\u5143\u5927\u5b89\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARUNYUANDAANXINLINGHUOPEIZHIHUNHEA"}, "000274": {"suoxie": "GFYTZGSYZQDIIA", "name": "\u5e7f\u53d1\u4e9a\u592a\u4e2d\u9ad8\u6536\u76ca\u503a(QDII)A", "style": "QDII", "pinyin": "GUANGFAYATAIZHONGGAOSHOUYIZHAIQDIIA"}, "000275": {"suoxie": "GFYTZGSYZMYXHQDIIA", "name": "\u5e7f\u53d1\u4e9a\u592a\u4e2d\u9ad8\u6536\u76ca\u503a\u7f8e\u5143\u73b0\u6c47(QDII)A", "style": "QDII", "pinyin": "GUANGFAYATAIZHONGGAOSHOUYIZHAIMEIYUANXIANHUIQDIIA"}, "000277": {"suoxie": "BSSYXDQZFZQ", "name": "\u535a\u65f6\u53cc\u6708\u85aa\u5b9a\u671f\u652f\u4ed8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGYUEXINDINGQIZHIFUZHAIQUAN"}, "000279": {"suoxie": "HSHLYXHH", "name": "\u534e\u5546\u7ea2\u5229\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGHONGLIYOUXUANHUNHE"}, "000286": {"suoxie": "YHXYJJHZQA", "name": "\u94f6\u534e\u4fe1\u7528\u5b63\u5b63\u7ea2\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJIJIHONGZHAIQUANA"}, "000289": {"suoxie": "PHFTDKZA", "name": "\u9e4f\u534e\u4e30\u6cf0\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGTAIDINGKAIZHAIA"}, "000290": {"suoxie": "PHQQGSYZQDII", "name": "\u9e4f\u534e\u5168\u7403\u9ad8\u6536\u76ca\u503a(QDII)", "style": "QDII", "pinyin": "PENGHUAQUANQIUGAOSHOUYIZHAIQDII"}, "000294": {"suoxie": "HASTYXHH", "name": "\u534e\u5b89\u751f\u6001\u4f18\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANSHENGTAIYOUXIANHUNHE"}, "000295": {"suoxie": "PHFSDKZA", "name": "\u9e4f\u534e\u4e30\u5b9e\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGSHIDINGKAIZHAIA"}, "000296": {"suoxie": "PHFSDKZB", "name": "\u9e4f\u534e\u4e30\u5b9e\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGSHIDINGKAIZHAIB"}, "000297": {"suoxie": "PHKZZZQA", "name": "\u9e4f\u534e\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PENGHUAKEZHUANZHAIZHAIQUANA"}, "000298": {"suoxie": "ZHCZZQA", "name": "\u4e2d\u6d77\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHAICHUNZHAIZHAIQUANA"}, "000299": {"suoxie": "ZHCZZQC", "name": "\u4e2d\u6d77\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHAICHUNZHAIZHAIQUANC"}, "000300": {"suoxie": "DBDLHBA", "name": "\u5fb7\u90a6\u5fb7\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DEBANGDELIHUOBIA"}, "000301": {"suoxie": "DBDLHBB", "name": "\u5fb7\u90a6\u5fb7\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DEBANGDELIHUOBIB"}, "000305": {"suoxie": "ZYZGDJA", "name": "\u4e2d\u94f6\u4e2d\u9ad8\u7b49\u7ea7A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGGAODENGJIA"}, "000306": {"suoxie": "THHLZQ", "name": "\u5929\u5f18\u5f18\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGHONGLIZHAIQUAN"}, "000307": {"suoxie": "YFDHJETFLJA", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "YIFANGDAHUANGJINETFLIANJIEA"}, "000308": {"suoxie": "JXCXZGHH", "name": "\u5efa\u4fe1\u521b\u65b0\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINCHUANGXINZHONGGUOHUNHE"}, "000309": {"suoxie": "DMPZSHJXGP", "name": "\u5927\u6469\u54c1\u8d28\u751f\u6d3b\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "DAMOPINZHISHENGHUOJINGXUANGUPIAO"}, "000310": {"suoxie": "AXYLXYA", "name": "\u5b89\u4fe1\u6c38\u5229\u4fe1\u7528A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGLIXINYONGA"}, "000311": {"suoxie": "JSCCHS300ZSZQ", "name": "\u666f\u987a\u957f\u57ce\u6caa\u6df1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGHUSHEN300ZHISHUZENGQIANG"}, "000312": {"suoxie": "HAHS300ZQA", "name": "\u534e\u5b89\u6caa\u6df1300\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANHUSHEN300ZENGQIANGA"}, "000313": {"suoxie": "HAHS300ZQC", "name": "\u534e\u5b89\u6caa\u6df1300\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANHUSHEN300ZENGQIANGC"}, "000314": {"suoxie": "ZSRFLHPZHHFQSA", "name": "\u62db\u5546\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGRUIFENGLINGHUOPEIZHIHUNHEFAQISHIA"}, "000319": {"suoxie": "TDTLZQA", "name": "\u6cf0\u8fbe\u6dd8\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDATAOLIZHAIQUANA"}, "000320": {"suoxie": "TDTLZQC", "name": "\u6cf0\u8fbe\u6dd8\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDATAOLIZHAIQUANC"}, "000322": {"suoxie": "NYHLJHZQA", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u6c47\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINHUIZHAIQUANA"}, "000324": {"suoxie": "HRYDXJSYHBA", "name": "\u534e\u6da6\u5143\u5927\u73b0\u91d1\u6536\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARUNYUANDAXIANJINSHOUYIHUOBIA"}, "000325": {"suoxie": "HRYDXJSYHBB", "name": "\u534e\u6da6\u5143\u5927\u73b0\u91d1\u6536\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARUNYUANDAXIANJINSHOUYIHUOBIB"}, "000326": {"suoxie": "NFZXPCZGP", "name": "\u5357\u65b9\u4e2d\u5c0f\u76d8\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGZHONGXIAOPANCHENGZHANGGUPIAO"}, "000327": {"suoxie": "NFXLCHH", "name": "\u5357\u65b9\u65b0\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINLANCHOUHUNHE"}, "000328": {"suoxie": "STMGZXDLLHPZHH", "name": "\u4e0a\u6295\u6469\u6839\u8f6c\u578b\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENZHUANXINGDONGLILINGHUOPEIZHIHUNHE"}, "000329": {"suoxie": "PHFRDKZ", "name": "\u9e4f\u534e\u4e30\u9976\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGRAODINGKAIZHAI"}, "000330": {"suoxie": "HTFXJBHBA", "name": "\u6c47\u6dfb\u5bcc\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUXIANJINBAOHUOBIA"}, "000331": {"suoxie": "ZJHBA", "name": "\u4e2d\u52a0\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGJIAHUOBIA"}, "000332": {"suoxie": "ZJHBC", "name": "\u4e2d\u52a0\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGJIAHUOBIC"}, "000333": {"suoxie": "CCWGSYZQA", "name": "\u957f\u57ce\u7a33\u56fa\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGWENGUSHOUYIZHAIQUANA"}, "000334": {"suoxie": "CCWGSYZQC", "name": "\u957f\u57ce\u7a33\u56fa\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGWENGUSHOUYIZHAIQUANC"}, "000335": {"suoxie": "AXYLXYC", "name": "\u5b89\u4fe1\u6c38\u5229\u4fe1\u7528C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGLIXINYONGC"}, "000336": {"suoxie": "NYYJJXHH", "name": "\u519c\u94f6\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINYANJIUJINGXUANHUNHE"}, "000338": {"suoxie": "PHSZBLZQ", "name": "\u9e4f\u534e\u53cc\u503a\u4fdd\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUASHUANGZHAIBAOLIZHAIQUAN"}, "000339": {"suoxie": "CCYLBJHH", "name": "\u957f\u57ce\u533b\u7597\u4fdd\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYILIAOBAOJIANHUNHE"}, "000341": {"suoxie": "JSXXSCC2QDII", "name": "\u5609\u5b9e\u65b0\u5174\u5e02\u573aC2(QDII)", "style": "QDII", "pinyin": "JIASHIXINXINGSHICHANGC2QDII"}, "000342": {"suoxie": "JSXXSCA1QDII", "name": "\u5609\u5b9e\u65b0\u5174\u5e02\u573aA1(QDII)", "style": "QDII", "pinyin": "JIASHIXINXINGSHICHANGA1QDII"}, "000343": {"suoxie": "HXCFBHBA", "name": "\u534e\u590f\u8d22\u5bcc\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIACAIFUBAOHUOBIA"}, "000345": {"suoxie": "PHFRDKZ", "name": "\u9e4f\u534e\u4e30\u878d\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGRONGDINGKAIZHAI"}, "000346": {"suoxie": "JXAXHB6GYDKA", "name": "\u5efa\u4fe1\u5b89\u5fc3\u56de\u62a56\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINANXINHUIBAO6GEYUEDINGKAIA"}, "000347": {"suoxie": "JXAXHB6GYDKC", "name": "\u5efa\u4fe1\u5b89\u5fc3\u56de\u62a56\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINANXINHUIBAO6GEYUEDINGKAIC"}, "000351": {"suoxie": "GFHFDQZQA", "name": "\u56fd\u5bcc\u6052\u4e30\u5b9a\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUHENGFENGDINGQIZHAIQUANA"}, "000352": {"suoxie": "GFHFDQZQC", "name": "\u56fd\u5bcc\u6052\u4e30\u5b9a\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUHENGFENGDINGQIZHAIQUANC"}, "000354": {"suoxie": "CSCZHZTHH", "name": "\u957f\u76db\u57ce\u9547\u5316\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGCHENGZHENHUAZHUTIHUNHE"}, "000355": {"suoxie": "NFFYXYZQZQA", "name": "\u5357\u65b9\u4e30\u5143\u4fe1\u7528\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGFENGYUANXINYONGZENGQIANGZHAIQUANA"}, "000356": {"suoxie": "NFFYXYZQZQC", "name": "\u5357\u65b9\u4e30\u5143\u4fe1\u7528\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGFENGYUANXINYONGZENGQIANGZHAIQUANC"}, "000359": {"suoxie": "YFDYLCHBA", "name": "\u6613\u65b9\u8fbe\u6613\u7406\u8d22\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAYILICAIHUOBIA"}, "000362": {"suoxie": "GTJXJZYSHHA", "name": "\u56fd\u6cf0\u805a\u4fe1\u4ef7\u503c\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJUXINJIAZHIYOUSHIHUNHEA"}, "000363": {"suoxie": "GTJXJZYSHHC", "name": "\u56fd\u6cf0\u805a\u4fe1\u4ef7\u503c\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJUXINJIAZHIYOUSHIHUNHEC"}, "000366": {"suoxie": "HTFTFTHBA", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u5bcc\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUTIANFUTONGHUOBIA"}, "000367": {"suoxie": "GTAKDQZFHHA", "name": "\u56fd\u6cf0\u5b89\u5eb7\u5b9a\u671f\u652f\u4ed8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIANKANGDINGQIZHIFUHUNHEA"}, "000368": {"suoxie": "HTFHS300AZZS", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u5b89\u4e2d\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300ANZHONGZHISHU"}, "000369": {"suoxie": "GFQQYLBJQDII", "name": "\u5e7f\u53d1\u5168\u7403\u533b\u7597\u4fdd\u5065(QDII)", "style": "QDII", "pinyin": "GUANGFAQUANQIUYILIAOBAOJIANQDII"}, "000370": {"suoxie": "GFQQYLBJMY", "name": "\u5e7f\u53d1\u5168\u7403\u533b\u7597\u4fdd\u5065\u7f8e\u5143", "style": "QDII", "pinyin": "GUANGFAQUANQIUYILIAOBAOJIANMEIYUAN"}, "000371": {"suoxie": "MSJYXJBHBA", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINBAOHUOBIA"}, "000372": {"suoxie": "ZYHLBNDKZ", "name": "\u4e2d\u94f6\u60e0\u5229\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHUILIBANNIANDINGKAIZHAI"}, "000373": {"suoxie": "HAZZXFYYETFLJA", "name": "\u534e\u5b89\u4e2d\u8bc1\u7ec6\u5206\u533b\u836fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXIFENYIYAOETFLIANJIEA"}, "000376": {"suoxie": "HAZZXFYYETFLJC", "name": "\u534e\u5b89\u4e2d\u8bc1\u7ec6\u5206\u533b\u836fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXIFENYIYAOETFLIANJIEC"}, "000377": {"suoxie": "STMGSZZLZQA", "name": "\u4e0a\u6295\u6469\u6839\u53cc\u503a\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUMOGENSHUANGZHAIZENGLIZHAIQUANA"}, "000378": {"suoxie": "STMGSZZLZQC", "name": "\u4e0a\u6295\u6469\u6839\u53cc\u503a\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUMOGENSHUANGZHAIZENGLIZHAIQUANC"}, "000379": {"suoxie": "PARZLHBA", "name": "\u5e73\u5b89\u65e5\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANRIZENGLIHUOBIA"}, "000380": {"suoxie": "JSCCJYHBA", "name": "\u666f\u987a\u957f\u57ce\u666f\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNCHANGCHENGJINGYIHUOBIA"}, "000381": {"suoxie": "JSCCJYHBB", "name": "\u666f\u987a\u957f\u57ce\u666f\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNCHANGCHENGJINGYIHUOBIB"}, "000385": {"suoxie": "JSCCJYSLZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYISHUANGLIZHAIQUANA"}, "000386": {"suoxie": "JSCCJYSLZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYISHUANGLIZHAIQUANC"}, "000389": {"suoxie": "GFTTHHBA", "name": "\u5e7f\u53d1\u5929\u5929\u7ea2\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANTIANHONGHUOBIA"}, "000390": {"suoxie": "HSYSHYHH", "name": "\u534e\u5546\u4f18\u52bf\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGYOUSHIHANGYEHUNHE"}, "000394": {"suoxie": "RTTYDRZQA", "name": "\u878d\u901a\u901a\u6e90\u77ed\u878d\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "RONGTONGTONGYUANDUANRONGZHAIQUANA"}, "000395": {"suoxie": "HTFAXZGZQA", "name": "\u6c47\u6dfb\u5bcc\u5b89\u5fc3\u4e2d\u56fd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUANXINZHONGGUOZHAIQUANA"}, "000396": {"suoxie": "HTFAXZGZQC", "name": "\u6c47\u6dfb\u5bcc\u5b89\u5fc3\u4e2d\u56fd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUANXINZHONGGUOZHAIQUANC"}, "000397": {"suoxie": "HTFQEBHB", "name": "\u6c47\u6dfb\u5bcc\u5168\u989d\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUQUANEBAOHUOBI"}, "000398": {"suoxie": "HFLHPZHH", "name": "\u534e\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFULINGHUOPEIZHIHUNHE"}, "000402": {"suoxie": "GYCZZQA", "name": "\u5de5\u94f6\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINCHUNZHAIZHAIQUANA"}, "000403": {"suoxie": "GYCZZQB", "name": "\u5de5\u94f6\u7eaf\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINCHUNZHAIZHAIQUANB"}, "000404": {"suoxie": "YFDXXCZLHPZ", "name": "\u6613\u65b9\u8fbe\u65b0\u5174\u6210\u957f\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINXINGCHENGZHANGLINGHUOPEIZHI"}, "000406": {"suoxie": "HTFSLZQZQA", "name": "\u6c47\u6dfb\u5bcc\u53cc\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGLIZENGQIANGZHAIQUANA"}, "000407": {"suoxie": "HTFSLZQZQC", "name": "\u6c47\u6dfb\u5bcc\u53cc\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGLIZENGQIANGZHAIQUANC"}, "000408": {"suoxie": "MSJYCZHHHA", "name": "\u6c11\u751f\u52a0\u94f6\u57ce\u9547\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINCHENGZHENHUAHUNHEA"}, "000409": {"suoxie": "PHHBCYGP", "name": "\u9e4f\u534e\u73af\u4fdd\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAHUANBAOCHANYEGUPIAO"}, "000410": {"suoxie": "YMFWLXHH", "name": "\u76ca\u6c11\u670d\u52a1\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIMINFUWULINGXIANHUNHE"}, "000411": {"suoxie": "JSCCYZCZGP", "name": "\u666f\u987a\u957f\u57ce\u4f18\u8d28\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGYOUZHICHENGZHANGGUPIAO"}, "000414": {"suoxie": "JSJDSYCLDQHHA", "name": "\u5609\u5b9e\u7edd\u5bf9\u6536\u76ca\u7b56\u7565\u5b9a\u671f\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "JIASHIJUEDUISHOUYICELUEDINGQIHUNHEA"}, "000415": {"suoxie": "DMTL18GYDKZA", "name": "\u5927\u6469\u6dfb\u522918\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOTIANLI18GEYUEDINGKAIZHAIA"}, "000416": {"suoxie": "DMTL18GYDKZC", "name": "\u5927\u6469\u6dfb\u522918\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOTIANLI18GEYUEDINGKAIZHAIC"}, "000417": {"suoxie": "GLAXJXHH", "name": "\u56fd\u8054\u5b89\u65b0\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANXINJINGXUANHUNHE"}, "000418": {"suoxie": "JSCCCZZXGP", "name": "\u666f\u987a\u957f\u57ce\u6210\u957f\u4e4b\u661f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGCHENGZHANGZHIXINGGUPIAO"}, "000419": {"suoxie": "DMYZXJCZA", "name": "\u5927\u6469\u4f18\u8d28\u4fe1\u4ef7\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOYOUZHIXINJIACHUNZHAIA"}, "000420": {"suoxie": "DMYZXJCZC", "name": "\u5927\u6469\u4f18\u8d28\u4fe1\u4ef7\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOYOUZHIXINJIACHUNZHAIC"}, "000423": {"suoxie": "QHKYSJQDHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANSHIJIANQUDONGHUNHEA"}, "000424": {"suoxie": "CSTLBHBA", "name": "\u957f\u76db\u6dfb\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGSHENGTIANLIBAOHUOBIA"}, "000425": {"suoxie": "CSTLBHBB", "name": "\u957f\u76db\u6dfb\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGSHENGTIANLIBAOHUOBIB"}, "000431": {"suoxie": "PHPPCCHH", "name": "\u9e4f\u534e\u54c1\u724c\u4f20\u627f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAPINPAICHUANCHENGHUNHE"}, "000432": {"suoxie": "ZYYXQYHH", "name": "\u4e2d\u94f6\u4f18\u79c0\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINYOUXIUQIYEHUNHE"}, "000433": {"suoxie": "AXXFYXHHA", "name": "\u5b89\u4fe1\u946b\u53d1\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINFAYOUXUANHUNHEA"}, "000434": {"suoxie": "XHYNBHBA", "name": "\u65b0\u534e\u58f9\u8bfa\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAYINUOBAOHUOBIA"}, "000436": {"suoxie": "YFDYHDKHHFQS", "name": "\u6613\u65b9\u8fbe\u88d5\u60e0\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUHUIDINGKAIHUNHEFAQISHI"}, "000439": {"suoxie": "GJXYHB", "name": "\u56fd\u91d1\u946b\u76c8\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOJINXINYINGHUOBI"}, "000452": {"suoxie": "NFYYBJLHPZHHA", "name": "\u5357\u65b9\u533b\u836f\u4fdd\u5065\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYIYAOBAOJIANLINGHUOPEIZHIHUNHEA"}, "000457": {"suoxie": "STMGHXCZGP", "name": "\u4e0a\u6295\u6469\u6839\u6838\u5fc3\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHEXINCHENGZHANGGUPIAO"}, "000458": {"suoxie": "YDLXHBHHFQS", "name": "\u82f1\u5927\u9886\u5148\u56de\u62a5\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINGDALINGXIANHUIBAOHUNHEFAQISHI"}, "000459": {"suoxie": "YDLXHBHHFQS", "name": "\u82f1\u5927\u9886\u5148\u56de\u62a5\u6df7\u5408\u53d1\u8d77\u5f0f(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINGDALINGXIANHUIBAOHUNHEFAQISHI"}, "000462": {"suoxie": "NYZTLDHH", "name": "\u519c\u94f6\u4e3b\u9898\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINZHUTILUNDONGHUNHE"}, "000463": {"suoxie": "HSSZFLZQA", "name": "\u534e\u5546\u53cc\u503a\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGSHUANGZHAIFENGLIZHAIQUANA"}, "000464": {"suoxie": "JSHQBHB", "name": "\u5609\u5b9e\u6d3b\u671f\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOQIBAOHUOBI"}, "000465": {"suoxie": "JSCCXYXDQZFZQ", "name": "\u666f\u987a\u957f\u57ce\u946b\u6708\u85aa\u5b9a\u671f\u652f\u4ed8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGXINYUEXINDINGQIZHIFUZHAIQUAN"}, "000466": {"suoxie": "RTTRZQAB", "name": "\u878d\u901a\u901a\u745e\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGRUIZHAIQUANAB"}, "000469": {"suoxie": "FGMBQLYNQCZZQ", "name": "\u5bcc\u56fd\u76ee\u6807\u9f50\u5229\u4e00\u5e74\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOMUBIAOQILIYINIANQICHUNZHAIZHAIQUAN"}, "000470": {"suoxie": "FGMBQLYNQCZZQ", "name": "\u5bcc\u56fd\u76ee\u6807\u9f50\u5229\u4e00\u5e74\u671f\u7eaf\u503a\u503a\u5238(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOMUBIAOQILIYINIANQICHUNZHAIZHAIQUAN"}, "000471": {"suoxie": "FGCZFZGP", "name": "\u5bcc\u56fd\u57ce\u9547\u53d1\u5c55\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOCHENGZHENFAZHANGUPIAO"}, "000472": {"suoxie": "FGCZFZGP", "name": "\u5bcc\u56fd\u57ce\u9547\u53d1\u5c55\u80a1\u7968(\u540e\u7aef)", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOCHENGZHENFAZHANGUPIAO"}, "000475": {"suoxie": "GFTTLHBA", "name": "\u5e7f\u53d1\u5929\u5929\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANTIANLIHUOBIA"}, "000476": {"suoxie": "GFTTLHBB", "name": "\u5e7f\u53d1\u5929\u5929\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANTIANLIHUOBIB"}, "000477": {"suoxie": "GFZTLXHH", "name": "\u5e7f\u53d1\u4e3b\u9898\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAZHUTILINGXIANHUNHE"}, "000478": {"suoxie": "JXZZ500ZSZQA", "name": "\u5efa\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG500ZHISHUZENGQIANGA"}, "000480": {"suoxie": "DFHXDLHH", "name": "\u4e1c\u65b9\u7ea2\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGXINDONGLIHUNHE"}, "000481": {"suoxie": "HSSZFLZQC", "name": "\u534e\u5546\u53cc\u503a\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGSHUANGZHAIFENGLIZHAIQUANC"}, "000483": {"suoxie": "XYHBA", "name": "\u946b\u5143\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINYUANHUOBIA"}, "000484": {"suoxie": "XYHBB", "name": "\u946b\u5143\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINYUANHUOBIB"}, "000487": {"suoxie": "JS3GYLCZQA", "name": "\u5609\u5b9e3\u4e2a\u6708\u7406\u8d22\u503a\u5238A", "style": "", "pinyin": "JIASHI3GEYUELICAIZHAIQUANA"}, "000488": {"suoxie": "JS3GYLCZQE", "name": "\u5609\u5b9e3\u4e2a\u6708\u7406\u8d22\u503a\u5238E", "style": "", "pinyin": "JIASHI3GEYUELICAIZHAIQUANE"}, "000489": {"suoxie": "GDSMHLCZA", "name": "\u5149\u5927\u5c81\u672b\u7ea2\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDASUIMOHONGLICHUNZHAIA"}, "000490": {"suoxie": "GDSMHLCZC", "name": "\u5149\u5927\u5c81\u672b\u7ea2\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDASUIMOHONGLICHUNZHAIC"}, "000493": {"suoxie": "NFXJTA", "name": "\u5357\u65b9\u73b0\u91d1\u901aA", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINTONGA"}, "000494": {"suoxie": "NFXJTB", "name": "\u5357\u65b9\u73b0\u91d1\u901aB", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINTONGB"}, "000495": {"suoxie": "NFXJTC", "name": "\u5357\u65b9\u73b0\u91d1\u901aC", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINTONGC"}, "000496": {"suoxie": "CACYJXHHA", "name": "\u957f\u5b89\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANCHANYEJINGXUANHUNHEA"}, "000497": {"suoxie": "CTCZZQA", "name": "\u8d22\u901a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGCHUNZHAIZHAIQUANA"}, "000501": {"suoxie": "HFHF18GYDKZC", "name": "\u534e\u5bcc\u6052\u5bcc18\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUHENGFU18GEYUEDINGKAIZHAIC"}, "000502": {"suoxie": "HFHF18GYDKZA", "name": "\u534e\u5bcc\u6052\u5bcc18\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUHENGFU18GEYUEDINGKAIZHAIA"}, "000503": {"suoxie": "ZXJTJHZDZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u666f\u548c\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINJIANTOUJINGHEZHONGDUANZHAIA"}, "000504": {"suoxie": "ZXJTJHZDZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u666f\u548c\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINJIANTOUJINGHEZHONGDUANZHAIC"}, "000505": {"suoxie": "GSABHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOHUOBIA"}, "000506": {"suoxie": "GSABHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOHUOBIB"}, "000507": {"suoxie": "TDHLHDHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5b8f\u8fbe\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIHONGDAHUNHEA"}, "000508": {"suoxie": "TDHLHDHHB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5b8f\u8fbe\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIHONGDAHUNHEB"}, "000509": {"suoxie": "GFQDZHBA", "name": "\u5e7f\u53d1\u94b1\u888b\u5b50\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAQIANDAIZIHUOBIA"}, "000511": {"suoxie": "GTGCQDLHPZHHA", "name": "\u56fd\u6cf0\u56fd\u7b56\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIGUOCEQUDONGLINGHUOPEIZHIHUNHEA"}, "000512": {"suoxie": "GTHS300ZSZQA", "name": "\u56fd\u6cf0\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHUSHEN300ZHISHUZENGQIANGA"}, "000513": {"suoxie": "FGGDZZHYGPA", "name": "\u5bcc\u56fd\u9ad8\u7aef\u5236\u9020\u884c\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOGAODUANZHIZAOHANGYEGUPIAOA"}, "000514": {"suoxie": "FGGDZZHYGPA", "name": "\u5bcc\u56fd\u9ad8\u7aef\u5236\u9020\u884c\u4e1a\u80a1\u7968A(\u540e\u7aef)", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOGAODUANZHIZAOHANGYEGUPIAOA"}, "000516": {"suoxie": "FGXLYNQDQKFZQXA", "name": "\u5bcc\u56fd\u7965\u5229\u4e00\u5e74\u671f\u5b9a\u671f\u5f00\u653e\u503a\u5238\u578bA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXIANGLIYINIANQIDINGQIKAIFANGZHAIQUANXINGA"}, "000517": {"suoxie": "FGXLYNQDQKFZQXC", "name": "\u5bcc\u56fd\u7965\u5229\u4e00\u5e74\u671f\u5b9a\u671f\u5f00\u653e\u503a\u5238\u578bC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXIANGLIYINIANQIDINGQIKAIFANGZHAIQUANXINGC"}, "000520": {"suoxie": "SYXXJZCZHH", "name": "\u4e0a\u94f6\u65b0\u5174\u4ef7\u503c\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGYINXINXINGJIAZHICHENGZHANGHUNHE"}, "000521": {"suoxie": "NARXDKZQ", "name": "\u8bfa\u5b89\u745e\u946b\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANRUIXINDINGKAIZHAIQUAN"}, "000522": {"suoxie": "HRYDXXCMKJHH", "name": "\u534e\u6da6\u5143\u5927\u4fe1\u606f\u4f20\u5a92\u79d1\u6280\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAXINXICHUANMEIKEJIHUNHE"}, "000523": {"suoxie": "GTRYYLBJHHA", "name": "\u56fd\u6295\u745e\u94f6\u533b\u7597\u4fdd\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINYILIAOBAOJIANHUNHEA"}, "000524": {"suoxie": "STMGMSXQGP", "name": "\u4e0a\u6295\u6469\u6839\u6c11\u751f\u9700\u6c42\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENMINSHENGXUQIUGUPIAO"}, "000526": {"suoxie": "GTNYLHPZHHA", "name": "\u56fd\u6cf0\u6d53\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAINONGYILINGHUOPEIZHIHUNHEA"}, "000527": {"suoxie": "NFXYXLHPZHHA", "name": "\u5357\u65b9\u65b0\u4f18\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGXINYOUXIANGLINGHUOPEIZHIHUNHEA"}, "000528": {"suoxie": "GYXJHBA", "name": "\u5de5\u94f6\u85aa\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINXINJINHUOBIA"}, "000529": {"suoxie": "GFJZYSHHA", "name": "\u5e7f\u53d1\u7ade\u4e89\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJINGZHENGYOUSHIHUNHEA"}, "000530": {"suoxie": "ZSFSWDZZHHA", "name": "\u62db\u5546\u4e30\u76db\u7a33\u5b9a\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGSHENGWENDINGZENGZHANGHUNHEA"}, "000531": {"suoxie": "DWAEFLHPZHHA", "name": "\u4e1c\u5434\u963f\u5c14\u6cd5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUAERFALINGHUOPEIZHIHUNHEA"}, "000532": {"suoxie": "JSCCYSQYHH", "name": "\u666f\u987a\u957f\u57ce\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYOUSHIQIYEHUNHE"}, "000533": {"suoxie": "YYHBA", "name": "\u6c38\u8d62\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YONGYINGHUOBIA"}, "000534": {"suoxie": "CSGDZBHH", "name": "\u957f\u76db\u9ad8\u7aef\u88c5\u5907\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGGAODUANZHUANGBEIHUNHE"}, "000535": {"suoxie": "CSHTHGZBLHPZHH", "name": "\u957f\u76db\u822a\u5929\u6d77\u5de5\u88c5\u5907\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGHANGTIANHAIGONGZHUANGBEILINGHUOPEIZHIHUNHE"}, "000536": {"suoxie": "QHKYKZZZQ", "name": "\u524d\u6d77\u5f00\u6e90\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "QIANHAIKAIYUANKEZHUANZHAIZHAIQUAN"}, "000538": {"suoxie": "NAYSHYHHA", "name": "\u8bfa\u5b89\u4f18\u52bf\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANYOUSHIHANGYEHUNHEA"}, "000539": {"suoxie": "ZYHQBHB", "name": "\u4e2d\u94f6\u6d3b\u671f\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINHUOQIBAOHUOBI"}, "000540": {"suoxie": "GJJTTHBA", "name": "\u56fd\u91d1\u91d1\u817e\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOJINJINTENGTONGHUOBIA"}, "000541": {"suoxie": "HSCXCZHHFQS", "name": "\u534e\u5546\u521b\u65b0\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGCHUANGXINCHENGZHANGHUNHEFAQISHI"}, "000542": {"suoxie": "SYHCBHBA", "name": "\u4e0a\u94f6\u6167\u8d22\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGYINHUICAIBAOHUOBIA"}, "000543": {"suoxie": "SYHCBHBB", "name": "\u4e0a\u94f6\u6167\u8d22\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGYINHUICAIBAOHUOBIB"}, "000545": {"suoxie": "ZYHXJZLLHPZHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u7ade\u4e89\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUHEXINJINGZHENGLILINGHUOPEIZHIHUNHE"}, "000546": {"suoxie": "XYDKZA", "name": "\u5174\u4e1a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEDINGKAIZHAIA"}, "000547": {"suoxie": "JXJKMSHHA", "name": "\u5efa\u4fe1\u5065\u5eb7\u6c11\u751f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINJIANKANGMINSHENGHUNHEA"}, "000548": {"suoxie": "PHJCTHB", "name": "\u9e4f\u534e\u805a\u8d22\u901a\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAJUCAITONGHUOBI"}, "000549": {"suoxie": "HADGXJJGP", "name": "\u534e\u5b89\u5927\u56fd\u65b0\u7ecf\u6d4e\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANDAGUOXINJINGJIGUPIAO"}, "000550": {"suoxie": "GFXDLHH", "name": "\u5e7f\u53d1\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINDONGLIHUNHE"}, "000551": {"suoxie": "XCXFXFHH", "name": "\u4fe1\u8bda\u5e78\u798f\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGXINGFUXIAOFEIHUNHE"}, "000552": {"suoxie": "ZJCZYNA", "name": "\u4e2d\u52a0\u7eaf\u503a\u4e00\u5e74A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAIYINIANA"}, "000553": {"suoxie": "ZJCZYNC", "name": "\u4e2d\u52a0\u7eaf\u503a\u4e00\u5e74C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAIYINIANC"}, "000554": {"suoxie": "ZGMLHPZHH", "name": "\u4e2d\u56fd\u68a6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGGUOMENGLINGHUOPEIZHIHUNHE"}, "000556": {"suoxie": "GTRYXJYLHPZHHA", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINJIYULINGHUOPEIZHIHUNHEA"}, "000557": {"suoxie": "GTRYXJYLHPZHHC", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINJIYULINGHUOPEIZHIHUNHEC"}, "000559": {"suoxie": "NATTBA", "name": "\u8bfa\u5b89\u5929\u5929\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANTIANTIANBAOA"}, "000560": {"suoxie": "NATTBE", "name": "\u8bfa\u5b89\u5929\u5929\u5b9dE", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANTIANTIANBAOE"}, "000561": {"suoxie": "NFQYZQA", "name": "\u5357\u65b9\u542f\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGQIYUANZHAIQUANA"}, "000562": {"suoxie": "NFQYZQC", "name": "\u5357\u65b9\u542f\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGQIYUANZHAIQUANC"}, "000563": {"suoxie": "NFTLZQA", "name": "\u5357\u65b9\u901a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGTONGLIZHAIQUANA"}, "000564": {"suoxie": "NFTLZQC", "name": "\u5357\u65b9\u901a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGTONGLIZHAIQUANC"}, "000566": {"suoxie": "HTBRCXSJHHA", "name": "\u534e\u6cf0\u67cf\u745e\u521b\u65b0\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUICHUANGXINSHENGJIHUNHEA"}, "000567": {"suoxie": "GFJXLHHH", "name": "\u5e7f\u53d1\u805a\u7965\u7075\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJUXIANGLINGHUOHUNHE"}, "000569": {"suoxie": "PHZZBHB", "name": "\u9e4f\u534e\u589e\u503c\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAZENGZHIBAOHUOBI"}, "000572": {"suoxie": "ZYDCLHHA", "name": "\u4e2d\u94f6\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINDUOCELUEHUNHEA"}, "000573": {"suoxie": "THTLHH", "name": "\u5929\u5f18\u901a\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGTONGLIHUNHE"}, "000574": {"suoxie": "BYXJZHHA", "name": "\u5b9d\u76c8\u65b0\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINJIAZHIHUNHEA"}, "000575": {"suoxie": "XQTLBHB", "name": "\u5174\u5168\u6dfb\u5229\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGQUANTIANLIBAOHUOBI"}, "000576": {"suoxie": "ZYHBA", "name": "\u4e2d\u90ae\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYOUHUOBIA"}, "000577": {"suoxie": "AXJZJXGP", "name": "\u5b89\u4fe1\u4ef7\u503c\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINJIAZHIJINGXUANGUPIAO"}, "000578": {"suoxie": "XYHXSYZQZQXFQSA", "name": "\u946b\u5143\u6052\u946b\u6536\u76ca\u589e\u5f3a\u503a\u5238\u578b\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANHENGXINSHOUYIZENGQIANGZHAIQUANXINGFAQISHIA"}, "000579": {"suoxie": "XYHXSYZQZQXFQSC", "name": "\u946b\u5143\u6052\u946b\u6536\u76ca\u589e\u5f3a\u503a\u5238\u578b\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANHENGXINSHOUYIZENGQIANGZHAIQUANXINGFAQISHIC"}, "000580": {"suoxie": "ZYHBB", "name": "\u4e2d\u90ae\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYOUHUOBIB"}, "000581": {"suoxie": "JSHQBHBA", "name": "\u5609\u5b9e\u6d3b\u94b1\u5305\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOQIANBAOHUOBIA"}, "000583": {"suoxie": "JXJFDKZ", "name": "\u6c5f\u4fe1\u805a\u798f\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINJUFUDINGKAIZHAI"}, "000584": {"suoxie": "XHXYLHPZHHC", "name": "\u65b0\u534e\u946b\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINYILINGHUOPEIZHIHUNHEC"}, "000585": {"suoxie": "JSDCTLDQHHA", "name": "\u5609\u5b9e\u5bf9\u51b2\u5957\u5229\u5b9a\u671f\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "JIASHIDUICHONGTAOLIDINGQIHUNHEA"}, "000586": {"suoxie": "JSCCZXCJXGP", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u5c0f\u521b\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGZHONGXIAOCHUANGJINGXUANGUPIAO"}, "000587": {"suoxie": "DCLHPZHH", "name": "\u5927\u6210\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGLINGHUOPEIZHIHUNHE"}, "000588": {"suoxie": "ZSZQBHBA", "name": "\u62db\u5546\u62db\u94b1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOQIANBAOHUOBIA"}, "000589": {"suoxie": "GDYFSJHH", "name": "\u5149\u5927\u94f6\u53d1\u5546\u673a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYINFASHANGJIHUNHE"}, "000590": {"suoxie": "HAXHLHH", "name": "\u534e\u5b89\u65b0\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINHUOLIHUNHE"}, "000591": {"suoxie": "ZYJKSHHH", "name": "\u4e2d\u94f6\u5065\u5eb7\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINJIANKANGSHENGHUOHUNHE"}, "000592": {"suoxie": "JXGGHLGP", "name": "\u5efa\u4fe1\u6539\u9769\u7ea2\u5229\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINGAIGEHONGLIGUPIAO"}, "000593": {"suoxie": "YFDBPXFPZSMYXHA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u6d88\u8d39\u54c1\u6307\u6570\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "YIFANGDABIAOPUXIAOFEIPINZHISHUMEIYUANXIANHUIA"}, "000594": {"suoxie": "DMJQYXGP", "name": "\u5927\u6469\u8fdb\u53d6\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "DAMOJINQUYOUXUANGUPIAO"}, "000595": {"suoxie": "JSTHHH", "name": "\u5609\u5b9e\u6cf0\u548c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHITAIHEHUNHE"}, "000596": {"suoxie": "QHKYZZJGZSA", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u8bc1\u519b\u5de5\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGZHENGJUNGONGZHISHUA"}, "000597": {"suoxie": "ZHJJSYHH", "name": "\u4e2d\u6d77\u79ef\u6781\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIJIJISHOUYIHUNHE"}, "000598": {"suoxie": "CSSTHJHH", "name": "\u957f\u76db\u751f\u6001\u73af\u5883\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGTAIHUANJINGHUNHE"}, "000599": {"suoxie": "XCXJBHB", "name": "\u4fe1\u8bda\u85aa\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGXINJINBAOHUOBI"}, "000600": {"suoxie": "HTFHJBHB", "name": "\u6c47\u6dfb\u5bcc\u548c\u805a\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHEJUBAOHUOBI"}, "000601": {"suoxie": "HBCXYXHH", "name": "\u534e\u5b9d\u521b\u65b0\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOCHUANGXINYOUXUANHUNHE"}, "000602": {"suoxie": "FGAYHBA", "name": "\u5bcc\u56fd\u5b89\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOANYIHUOBIA"}, "000603": {"suoxie": "YFDCXQDLHPZHH", "name": "\u6613\u65b9\u8fbe\u521b\u65b0\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDACHUANGXINQUDONGLINGHUOPEIZHIHUNHE"}, "000604": {"suoxie": "YHDLBHBA", "name": "\u94f6\u534e\u591a\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUADUOLIBAOHUOBIA"}, "000605": {"suoxie": "YHDLBHBB", "name": "\u94f6\u534e\u591a\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUADUOLIBAOHUOBIB"}, "000606": {"suoxie": "THYXZQ", "name": "\u5929\u5f18\u4f18\u9009\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGYOUXUANZHAIQUAN"}, "000607": {"suoxie": "ZSZQBHBB", "name": "\u62db\u5546\u62db\u94b1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOQIANBAOHUOBIB"}, "000609": {"suoxie": "HSXLHHH", "name": "\u534e\u5546\u65b0\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINLIANGHUAHUNHE"}, "000612": {"suoxie": "HBSTZGHH", "name": "\u534e\u5b9d\u751f\u6001\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOSHENGTAIZHONGGUOHUNHE"}, "000613": {"suoxie": "GSABHS300ETFLJ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6caa\u6df1300ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOHUSHEN300ETFLIANJIE"}, "000614": {"suoxie": "HADGDAXLJQDII", "name": "\u534e\u5b89\u5fb7\u56fd(DAX)\u8054\u63a5(QDII)", "style": "QDII", "pinyin": "HUAANDEGUODAXLIANJIEQDII"}, "000615": {"suoxie": "CCGZBHBA", "name": "\u957f\u57ce\u5de5\u8d44\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGGONGZIBAOHUOBIA"}, "000618": {"suoxie": "JSXJBHB", "name": "\u5609\u5b9e\u85aa\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIXINJINBAOHUOBI"}, "000619": {"suoxie": "DFHCYSJHH", "name": "\u4e1c\u65b9\u7ea2\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGCHANYESHENGJIHUNHE"}, "000620": {"suoxie": "YFDXJZLHBA", "name": "\u6613\u65b9\u8fbe\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAXIANJINZENGLIHUOBIA"}, "000621": {"suoxie": "YFDXJZLHBB", "name": "\u6613\u65b9\u8fbe\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAXIANJINZENGLIHUOBIB"}, "000625": {"suoxie": "NATTBB", "name": "\u8bfa\u5b89\u5929\u5929\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANTIANTIANBAOB"}, "000626": {"suoxie": "DCFCBHBA", "name": "\u5927\u6210\u4e30\u8d22\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGFENGCAIBAOHUOBIA"}, "000627": {"suoxie": "DCFCBHBB", "name": "\u5927\u6210\u4e30\u8d22\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGFENGCAIBAOHUOBIB"}, "000628": {"suoxie": "DCGXJSCYGPA", "name": "\u5927\u6210\u9ad8\u65b0\u6280\u672f\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGGAOXINJISHUCHANYEGUPIAOA"}, "000634": {"suoxie": "FGTSLHPZJJ", "name": "\u5bcc\u56fd\u5929\u76db\u7075\u6d3b\u914d\u7f6e\u57fa\u91d1", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOTIANSHENGLINGHUOPEIZHIJIJIN"}, "000635": {"suoxie": "FGTSLHPZJJ", "name": "\u5bcc\u56fd\u5929\u76db\u7075\u6d3b\u914d\u7f6e\u57fa\u91d1(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOTIANSHENGLINGHUOPEIZHIJIJIN"}, "000638": {"suoxie": "FGFQBHBA", "name": "\u5bcc\u56fd\u5bcc\u94b1\u5305\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOFUQIANBAOHUOBIA"}, "000639": {"suoxie": "BYXRHHA", "name": "\u5b9d\u76c8\u7965\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGRUIHUNHEA"}, "000640": {"suoxie": "NALCBHBA", "name": "\u8bfa\u5b89\u7406\u8d22\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANLICAIBAOHUOBIA"}, "000641": {"suoxie": "NALCBHBB", "name": "\u8bfa\u5b89\u7406\u8d22\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANLICAIBAOHUOBIB"}, "000642": {"suoxie": "HTFHBC", "name": "\u6c47\u6dfb\u5bcc\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUOBIC"}, "000644": {"suoxie": "ZSZJBHBA", "name": "\u62db\u5546\u62db\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOJINBAOHUOBIA"}, "000645": {"suoxie": "HXXJBHB", "name": "\u534e\u590f\u85aa\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAXINJINBAOHUOBI"}, "000646": {"suoxie": "HRYDLHYXHHA", "name": "\u534e\u6da6\u5143\u5927\u91cf\u5316\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDALIANGHUAYOUXUANHUNHEA"}, "000647": {"suoxie": "YFDCFKXHBA", "name": "\u6613\u65b9\u8fbe\u8d22\u5bcc\u5feb\u7ebf\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDACAIFUKUAIXIANHUOBIA"}, "000648": {"suoxie": "YFDCFKXHBB", "name": "\u6613\u65b9\u8fbe\u8d22\u5bcc\u5feb\u7ebf\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDACAIFUKUAIXIANHUOBIB"}, "000649": {"suoxie": "CCJXLHPZ", "name": "\u957f\u57ce\u4e45\u946b\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUXINLINGHUOPEIZHI"}, "000650": {"suoxie": "HTFHBD", "name": "\u6c47\u6dfb\u5bcc\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUOBID"}, "000651": {"suoxie": "ZSZJBHBB", "name": "\u62db\u5546\u62db\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOJINBAOHUOBIB"}, "000652": {"suoxie": "BSYLLHPZHHA", "name": "\u535a\u65f6\u88d5\u9686\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYULONGLINGHUOPEIZHIHUNHEA"}, "000654": {"suoxie": "HSXRCYHH", "name": "\u534e\u5546\u65b0\u9510\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINRUICHANYEHUNHE"}, "000655": {"suoxie": "XYWLZQ", "name": "\u946b\u5143\u7a33\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANWENLIZHAIQUAN"}, "000656": {"suoxie": "QHKYHS300ZS", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6df1300\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANHUSHEN300ZHISHU"}, "000657": {"suoxie": "YHHQBHBA", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBIA"}, "000658": {"suoxie": "YHHQBHBB", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBIB"}, "000659": {"suoxie": "YHHQBHBC", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBIC"}, "000660": {"suoxie": "YHHQBHBD", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBID"}, "000661": {"suoxie": "YHHQBHBE", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBIE"}, "000662": {"suoxie": "YHHQBHBF", "name": "\u94f6\u534e\u6d3b\u94b1\u5b9d\u8d27\u5e01F", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOQIANBAOHUOBIF"}, "000663": {"suoxie": "GTRYMLZGHH", "name": "\u56fd\u6295\u745e\u94f6\u7f8e\u4e3d\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINMEILIZHONGGUOHUNHE"}, "000664": {"suoxie": "GLATYHHA", "name": "\u56fd\u8054\u5b89\u901a\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANTONGYINGHUNHEA"}, "000665": {"suoxie": "BSXJSYHBB", "name": "\u535a\u65f6\u73b0\u91d1\u6536\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINSHOUYIHUOBIB"}, "000667": {"suoxie": "GYJDSYHHFQA", "name": "\u5de5\u94f6\u7edd\u5bf9\u6536\u76ca\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GONGYINJUEDUISHOUYIHUNHEFAQIA"}, "000668": {"suoxie": "GSABZXZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNXIANGZHAIQUANA"}, "000669": {"suoxie": "GSABZXZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNXIANGZHAIQUANC"}, "000672": {"suoxie": "GYJDSYHHFQB", "name": "\u5de5\u94f6\u7edd\u5bf9\u6536\u76ca\u6df7\u5408\u53d1\u8d77B", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GONGYINJUEDUISHOUYIHUNHEFAQIB"}, "000673": {"suoxie": "RTSJTLZQLOFC", "name": "\u878d\u901a\u56db\u5b63\u6dfb\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGSIJITIANLIZHAIQUANLOFC"}, "000674": {"suoxie": "ZHZDZZQ", "name": "\u4e2d\u6d77\u4e2d\u77ed\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGHAIZHONGDUANZHAIZHAIQUAN"}, "000677": {"suoxie": "GYXJHB", "name": "\u5de5\u94f6\u73b0\u91d1\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINXIANJINHUOBI"}, "000678": {"suoxie": "HBXJBHBE", "name": "\u534e\u5b9d\u73b0\u91d1\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOXIANJINBAOHUOBIE"}, "000679": {"suoxie": "ZSFLLHPZHHA", "name": "\u62db\u5546\u4e30\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGLILINGHUOPEIZHIHUNHEA"}, "000681": {"suoxie": "XDAYHGJHBA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUIGUANJIAHUOBIA"}, "000682": {"suoxie": "XDAYHGJHBC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7ba1\u5bb6\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUIGUANJIAHUOBIC"}, "000683": {"suoxie": "XDAYHGJHBE", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7ba1\u5bb6\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUIGUANJIAHUOBIE"}, "000684": {"suoxie": "CSYLJKCYLHPZHH", "name": "\u957f\u76db\u517b\u8001\u5065\u5eb7\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGYANGLAOJIANKANGCHANYELINGHUOPEIZHIHUNHE"}, "000686": {"suoxie": "JXJXBHBA", "name": "\u5efa\u4fe1\u5609\u85aa\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINJIAXINBAOHUOBIA"}, "000687": {"suoxie": "NFXJBHBA", "name": "\u5357\u65b9\u85aa\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXINJINBAOHUOBIA"}, "000688": {"suoxie": "JSCCYJJXGP", "name": "\u666f\u987a\u957f\u57ce\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGYANJIUJINGXUANGUPIAO"}, "000689": {"suoxie": "QHKYXJJHHA", "name": "\u524d\u6d77\u5f00\u6e90\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANXINJINGJIHUNHEA"}, "000690": {"suoxie": "QHKYDHYHH", "name": "\u524d\u6d77\u5f00\u6e90\u5927\u6d77\u6d0b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANDAHAIYANGHUNHE"}, "000692": {"suoxie": "HTFSLZQC", "name": "\u6c47\u6dfb\u5bcc\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGLIZHAIQUANC"}, "000693": {"suoxie": "JXXJTLHBA", "name": "\u5efa\u4fe1\u73b0\u91d1\u6dfb\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINTIANLIHUOBIA"}, "000694": {"suoxie": "XYHLZQA", "name": "\u946b\u5143\u9e3f\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHONGLIZHAIQUANA"}, "000696": {"suoxie": "HTFHBHYGP", "name": "\u6c47\u6dfb\u5bcc\u73af\u4fdd\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUHUANBAOHANGYEGUPIAO"}, "000697": {"suoxie": "HTFYDHLGP", "name": "\u6c47\u6dfb\u5bcc\u79fb\u52a8\u4e92\u8054\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUYIDONGHULIANGUPIAO"}, "000698": {"suoxie": "BYKJ30HH", "name": "\u5b9d\u76c8\u79d1\u628030\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGKEJI30HUNHE"}, "000699": {"suoxie": "ZYXQBHB", "name": "\u4e2d\u94f6\u85aa\u94b1\u5305\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINXINQIANBAOHUOBI"}, "000700": {"suoxie": "TDHLHBB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIHUOBIB"}, "000701": {"suoxie": "JSCCJFHBA", "name": "\u666f\u987a\u957f\u57ce\u666f\u4e30\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNCHANGCHENGJINGFENGHUOBIA"}, "000704": {"suoxie": "YFDTTZLHBA", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANZENGLIHUOBIA"}, "000705": {"suoxie": "YFDTTZLHBB", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANZENGLIHUOBIB"}, "000706": {"suoxie": "ZYDCLLHPZHH", "name": "\u4e2d\u90ae\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUDUOCELUELINGHUOPEIZHIHUNHE"}, "000707": {"suoxie": "JSCCJFHBB", "name": "\u666f\u987a\u957f\u57ce\u666f\u4e30\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNCHANGCHENGJINGFENGHUOBIB"}, "000708": {"suoxie": "HAAXHH", "name": "\u534e\u5b89\u5b89\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANXIANGHUNHE"}, "000709": {"suoxie": "HAHCTHB", "name": "\u534e\u5b89\u6c47\u8d22\u901a\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANHUICAITONGHUOBI"}, "000710": {"suoxie": "JYXJBHBA", "name": "\u4ea4\u94f6\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINXIANJINBAOHUOBIA"}, "000711": {"suoxie": "JSYLBJGP", "name": "\u5609\u5b9e\u533b\u7597\u4fdd\u5065\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIYILIAOBAOJIANGUPIAO"}, "000712": {"suoxie": "STMGTTBHBA", "name": "\u4e0a\u6295\u6469\u6839\u5929\u6dfb\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENTIANTIANBAOHUOBIA"}, "000713": {"suoxie": "STMGTTBHBB", "name": "\u4e0a\u6295\u6469\u6839\u5929\u6dfb\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENTIANTIANBAOHUOBIB"}, "000714": {"suoxie": "NAWJHBLHPZHHA", "name": "\u8bfa\u5b89\u7a33\u5065\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANWENJIANHUIBAOLINGHUOPEIZHIHUNHEA"}, "000715": {"suoxie": "MSJYGDJXYZE", "name": "\u6c11\u751f\u52a0\u94f6\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINGAODENGJIXINYONGZHAIE"}, "000716": {"suoxie": "GYXJHBB", "name": "\u5de5\u94f6\u85aa\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINXINJINHUOBIB"}, "000717": {"suoxie": "RTZXSDLLHPZHHA", "name": "\u878d\u901a\u8f6c\u578b\u4e09\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHUANXINGSANDONGLILINGHUOPEIZHIHUNHEA"}, "000718": {"suoxie": "RTZXSDLLHPZHHA", "name": "\u878d\u901a\u8f6c\u578b\u4e09\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHUANXINGSANDONGLILINGHUOPEIZHIHUNHEA"}, "000719": {"suoxie": "NFXJTE", "name": "\u5357\u65b9\u73b0\u91d1\u901aE", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINTONGE"}, "000720": {"suoxie": "NFWL1NDKZC", "name": "\u5357\u65b9\u7a33\u52291\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGWENLI1NIANDINGKAIZHAIC"}, "000721": {"suoxie": "XYHBA", "name": "\u5174\u4e1a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEHUOBIA"}, "000722": {"suoxie": "XYHBB", "name": "\u5174\u4e1a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEHUOBIB"}, "000724": {"suoxie": "DCTLBHBA", "name": "\u5927\u6210\u6dfb\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANLIBAOHUOBIA"}, "000725": {"suoxie": "DCTLBHBB", "name": "\u5927\u6210\u6dfb\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANLIBAOHUOBIB"}, "000726": {"suoxie": "DCTLBHBE", "name": "\u5927\u6210\u6dfb\u5229\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANLIBAOHUOBIE"}, "000727": {"suoxie": "RTJKCYLHPZHHAB", "name": "\u878d\u901a\u5065\u5eb7\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGJIANKANGCHANYELINGHUOPEIZHIHUNHEAB"}, "000728": {"suoxie": "GYMBSYYNDKC", "name": "\u5de5\u94f6\u76ee\u6807\u6536\u76ca\u4e00\u5e74\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINMUBIAOSHOUYIYINIANDINGKAIC"}, "000729": {"suoxie": "JXZXPXFGPA", "name": "\u5efa\u4fe1\u4e2d\u5c0f\u76d8\u5148\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINZHONGXIAOPANXIANFENGGUPIAOA"}, "000730": {"suoxie": "BSXJBHBA", "name": "\u535a\u65f6\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINBAOHUOBIA"}, "000731": {"suoxie": "RTJKCYLHPZHHAB", "name": "\u878d\u901a\u5065\u5eb7\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGJIANKANGCHANYELINGHUOPEIZHIHUNHEAB"}, "000734": {"suoxie": "BSTTZLHBA", "name": "\u535a\u65f6\u5929\u5929\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHITIANTIANZENGLIHUOBIA"}, "000735": {"suoxie": "BSTTZLHBB", "name": "\u535a\u65f6\u5929\u5929\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHITIANTIANZENGLIHUOBIB"}, "000736": {"suoxie": "NAJLZQA", "name": "\u8bfa\u5b89\u805a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANJULIZHAIQUANA"}, "000737": {"suoxie": "NAJLZQC", "name": "\u8bfa\u5b89\u805a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANJULIZHAIQUANC"}, "000739": {"suoxie": "PAXXXFA", "name": "\u5e73\u5b89\u65b0\u946b\u5148\u950bA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINXINXIANFENGA"}, "000740": {"suoxie": "XYHBB", "name": "\u5174\u94f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYINHUOBIB"}, "000741": {"suoxie": "XYHBA", "name": "\u5174\u94f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYINHUOBIA"}, "000742": {"suoxie": "GTXJJLHPZHHA", "name": "\u56fd\u6cf0\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIXINJINGJILINGHUOPEIZHIHUNHEA"}, "000743": {"suoxie": "HTHTSSPYHHFQS", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u4e16\u666e\u76ca\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGSHIPUYIHUNHEFAQISHI"}, "000744": {"suoxie": "BXRFWDSYA", "name": "\u5317\u4fe1\u745e\u4e30\u7a33\u5b9a\u6536\u76caA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BEIXINRUIFENGWENDINGSHOUYIA"}, "000745": {"suoxie": "BXRFWDSYC", "name": "\u5317\u4fe1\u745e\u4e30\u7a33\u5b9a\u6536\u76caC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BEIXINRUIFENGWENDINGSHOUYIC"}, "000746": {"suoxie": "ZSHYJXGPJJ", "name": "\u62db\u5546\u884c\u4e1a\u7cbe\u9009\u80a1\u7968\u57fa\u91d1", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGHANGYEJINGXUANGUPIAOJIJIN"}, "000747": {"suoxie": "GFNXCLHHA", "name": "\u5e7f\u53d1\u9006\u5411\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFANIXIANGCELUEHUNHEA"}, "000748": {"suoxie": "GFHQBHBA", "name": "\u5e7f\u53d1\u6d3b\u671f\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOQIBAOHUOBIA"}, "000750": {"suoxie": "AXXJZLHBA", "name": "\u5b89\u4fe1\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINXIANJINZENGLIHUOBIA"}, "000751": {"suoxie": "JSXXCYGP", "name": "\u5609\u5b9e\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXINXINGCHANYEGUPIAO"}, "000753": {"suoxie": "HBLHDCHHA", "name": "\u534e\u5b9d\u91cf\u5316\u5bf9\u51b2\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUABAOLIANGHUADUICHONGHUNHEA"}, "000754": {"suoxie": "HBLHDCHHC", "name": "\u534e\u5b9d\u91cf\u5316\u5bf9\u51b2\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUABAOLIANGHUADUICHONGHUNHEC"}, "000755": {"suoxie": "FADXXCZHHA", "name": "\u5bcc\u5b89\u8fbe\u65b0\u5174\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAXINXINGCHENGZHANGHUNHEA"}, "000756": {"suoxie": "JXQLXLCGPA", "name": "\u5efa\u4fe1\u6f5c\u529b\u65b0\u84dd\u7b79\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINQIANLIXINLANCHOUGUPIAOA"}, "000757": {"suoxie": "HFZHCSLHPZHH", "name": "\u534e\u5bcc\u667a\u6167\u57ce\u5e02\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUZHIHUICHENGSHILINGHUOPEIZHIHUNHE"}, "000758": {"suoxie": "ZSZQBHBC", "name": "\u62db\u5546\u62db\u94b1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOQIANBAOHUOBIC"}, "000759": {"suoxie": "PACFBHBA", "name": "\u5e73\u5b89\u8d22\u5bcc\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANCAIFUBAOHUOBIA"}, "000760": {"suoxie": "GYCFHBA", "name": "\u5de5\u94f6\u8d22\u5bcc\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINCAIFUHUOBIA"}, "000761": {"suoxie": "GFJKYZSHGP", "name": "\u56fd\u5bcc\u5065\u5eb7\u4f18\u8d28\u751f\u6d3b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOFUJIANKANGYOUZHISHENGHUOGUPIAO"}, "000762": {"suoxie": "HTFJDSYDKHHA", "name": "\u6c47\u6dfb\u5bcc\u7edd\u5bf9\u6536\u76ca\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUITIANFUJUEDUISHOUYIDINGKAIHUNHEA"}, "000763": {"suoxie": "GYXCFLHPZHH", "name": "\u5de5\u94f6\u65b0\u8d22\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINCAIFULINGHUOPEIZHIHUNHE"}, "000764": {"suoxie": "WJHBE", "name": "\u4e07\u5bb6\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAHUOBIE"}, "000767": {"suoxie": "HFGTMALHPZHH", "name": "\u534e\u5bcc\u56fd\u6cf0\u6c11\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUGUOTAIMINANLINGHUOPEIZHIHUNHE"}, "000771": {"suoxie": "NAJXBHBA", "name": "\u8bfa\u5b89\u805a\u946b\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANJUXINBAOHUOBIA"}, "000772": {"suoxie": "JSCCZGHBLHPZHH", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u56fd\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGZHONGGUOHUIBAOLINGHUOPEIZHIHUNHE"}, "000773": {"suoxie": "WJXJBA", "name": "\u4e07\u5bb6\u73b0\u91d1\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAXIANJINBAOA"}, "000778": {"suoxie": "PHXJZZGP", "name": "\u9e4f\u534e\u5148\u8fdb\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAXIANJINZHIZAOGUPIAO"}, "000779": {"suoxie": "NAJXBHBB", "name": "\u8bfa\u5b89\u805a\u946b\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANJUXINBAOHUOBIB"}, "000780": {"suoxie": "PHYLBJGP", "name": "\u9e4f\u534e\u533b\u7597\u4fdd\u5065\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAYILIAOBAOJIANGUPIAO"}, "000783": {"suoxie": "BSJJXCYQA", "name": "\u535a\u65f6\u5b63\u5b63\u4eab\u6301\u6709\u671fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJIJIXIANGCHIYOUQIA"}, "000784": {"suoxie": "BSJJXCYQB", "name": "\u535a\u65f6\u5b63\u5b63\u4eab\u6301\u6709\u671fB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJIJIXIANGCHIYOUQIB"}, "000785": {"suoxie": "HRXJZLHBA", "name": "\u534e\u878d\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARONGXIANJINZENGLIHUOBIA"}, "000786": {"suoxie": "HRXJZLHBB", "name": "\u534e\u878d\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARONGXIANJINZENGLIHUOBIB"}, "000787": {"suoxie": "HRXJZLHBC", "name": "\u534e\u878d\u73b0\u91d1\u589e\u5229\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARONGXIANJINZENGLIHUOBIC"}, "000788": {"suoxie": "QHKYZGCZHH", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u56fd\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZHONGGUOCHENGZHANGHUNHE"}, "000789": {"suoxie": "YFDLBHBA", "name": "\u6613\u65b9\u8fbe\u9f99\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDALONGBAOHUOBIA"}, "000790": {"suoxie": "YFDLBHBB", "name": "\u6613\u65b9\u8fbe\u9f99\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDALONGBAOHUOBIB"}, "000791": {"suoxie": "YHAYZDZSYCYQZQC", "name": "\u94f6\u534e\u5b89\u9890\u4e2d\u77ed\u503a\u53cc\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANYIZHONGDUANZHAISHUANGYUECHIYOUQIZHAIQUANC"}, "000792": {"suoxie": "ZSDQBLGYQLCZQ", "name": "\u62db\u5546\u5b9a\u671f\u5b9d\u516d\u4e2a\u6708\u671f\u7406\u8d22\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGDINGQIBAOLIUGEYUEQILICAIZHAIQUAN"}, "000793": {"suoxie": "GYGDZZGP", "name": "\u5de5\u94f6\u9ad8\u7aef\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINGAODUANZHIZAOGUPIAO"}, "000794": {"suoxie": "BYRFCXHHAB", "name": "\u5b9d\u76c8\u777f\u4e30\u521b\u65b0\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGRUIFENGCHUANGXINHUNHEAB"}, "000795": {"suoxie": "BYRFCXHHAB", "name": "\u5b9d\u76c8\u777f\u4e30\u521b\u65b0\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGRUIFENGCHUANGXINHUNHEAB"}, "000796": {"suoxie": "BYRFCXHHC", "name": "\u5b9d\u76c8\u777f\u4e30\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGRUIFENGCHUANGXINHUNHEC"}, "000797": {"suoxie": "FZFBJXBHB", "name": "\u65b9\u6b63\u5bcc\u90a6\u91d1\u5c0f\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "FANGZHENGFUBANGJINXIAOBAOHUOBI"}, "000799": {"suoxie": "MSJYJYBNDQB", "name": "\u6c11\u751f\u52a0\u94f6\u5bb6\u76c8\u534a\u5e74\u5b9a\u671f\u5b9d", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINJIAYINGBANNIANDINGQIBAO"}, "000800": {"suoxie": "HSWLZTHH", "name": "\u534e\u5546\u672a\u6765\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGWEILAIZHUTIHUNHE"}, "000801": {"suoxie": "ZJCZA", "name": "\u4e2d\u91d1\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINCHUNZHAIA"}, "000802": {"suoxie": "ZJCZC", "name": "\u4e2d\u91d1\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINCHUNZHAIC"}, "000803": {"suoxie": "GYYJJXGP", "name": "\u5de5\u94f6\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYANJIUJINGXUANGUPIAO"}, "000804": {"suoxie": "ZXJTWLHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOUWENLIHUNHEA"}, "000805": {"suoxie": "ZYXJJLHPZHH", "name": "\u4e2d\u94f6\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINJINGJILINGHUOPEIZHIHUNHE"}, "000808": {"suoxie": "ZSZL1GYQLCZQA", "name": "\u62db\u5546\u62db\u52291\u4e2a\u6708\u671f\u7406\u8d22\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGZHAOLI1GEYUEQILICAIZHAIQUANA"}, "000809": {"suoxie": "ZSZL1GYQLCZQB", "name": "\u62db\u5546\u62db\u52291\u4e2a\u6708\u671f\u7406\u8d22\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGZHAOLI1GEYUEQILICAIZHAIQUANB"}, "000810": {"suoxie": "FGSYZQZQA", "name": "\u5bcc\u56fd\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHOUYIZENGQIANGZHAIQUANA"}, "000812": {"suoxie": "FGSYZQZQC", "name": "\u5bcc\u56fd\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHOUYIZENGQIANGZHAIQUANC"}, "000814": {"suoxie": "XYHXCZC", "name": "\u946b\u5143\u5408\u4eab\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHEXIANGCHUNZHAIC"}, "000815": {"suoxie": "XYHXCZA", "name": "\u946b\u5143\u5408\u4eab\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHEXIANGCHUNZHAIA"}, "000816": {"suoxie": "NFLCJJYXHBA", "name": "\u5357\u65b9\u7406\u8d22\u91d1\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGLICAIJINJIAOYIXINGHUOBIA"}, "000817": {"suoxie": "ZYAXHB", "name": "\u4e2d\u94f6\u5b89\u5fc3\u56de\u62a5", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINANXINHUIBAO"}, "000818": {"suoxie": "NATTBC", "name": "\u8bfa\u5b89\u5929\u5929\u5b9dC", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANTIANTIANBAOC"}, "000822": {"suoxie": "DHMLZGLHPZHH", "name": "\u4e1c\u6d77\u7f8e\u4e3d\u4e2d\u56fd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIMEILIZHONGGUOLINGHUOPEIZHIHUNHE"}, "000823": {"suoxie": "YHGDZZYHH", "name": "\u94f6\u534e\u9ad8\u7aef\u5236\u9020\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAGAODUANZHIZAOYEHUNHE"}, "000824": {"suoxie": "YXYFSLA", "name": "\u5706\u4fe1\u6c38\u4e30\u53cc\u5229A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGSHUANGLIA"}, "000825": {"suoxie": "YXYFSLC", "name": "\u5706\u4fe1\u6c38\u4e30\u53cc\u5229C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGSHUANGLIC"}, "000826": {"suoxie": "GFBF100ZSA", "name": "\u5e7f\u53d1\u767e\u53d1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFABAIFA100ZHISHUA"}, "000827": {"suoxie": "GFBF100ZSE", "name": "\u5e7f\u53d1\u767e\u53d1100\u6307\u6570E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFABAIFA100ZHISHUE"}, "000828": {"suoxie": "TDZXJYGPA", "name": "\u6cf0\u8fbe\u8f6c\u578b\u673a\u9047\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAZHUANXINGJIYUGUPIAOA"}, "000829": {"suoxie": "YFDTTFHBA", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u53d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANFAHUOBIA"}, "000830": {"suoxie": "YFDTTFHBB", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u53d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANFAHUOBIB"}, "000831": {"suoxie": "GYYLBJGP", "name": "\u5de5\u94f6\u533b\u7597\u4fdd\u5065\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYILIAOBAOJIANGUPIAO"}, "000832": {"suoxie": "THXJGJHBC", "name": "\u5929\u5f18\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGXIANJINGUANJIAHUOBIC"}, "000833": {"suoxie": "YFDFHCZC", "name": "\u6613\u65b9\u8fbe\u5bcc\u534e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAFUHUACHUNZHAIC"}, "000834": {"suoxie": "DCNSDK100", "name": "\u5927\u6210\u7eb3\u65af\u8fbe\u514b100", "style": "QDII", "pinyin": "DACHENGNASIDAKE100"}, "000835": {"suoxie": "HRYDFSZGA50ZSA", "name": "\u534e\u6da6\u5143\u5927\u5bcc\u65f6\u4e2d\u56fdA50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUARUNYUANDAFUSHIZHONGGUOA50ZHISHUA"}, "000836": {"suoxie": "GTRYQDBHBA", "name": "\u56fd\u6295\u745e\u94f6\u94b1\u591a\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINQIANDUOBAOHUOBIA"}, "000837": {"suoxie": "GTRYQDBHBI", "name": "\u56fd\u6295\u745e\u94f6\u94b1\u591a\u5b9d\u8d27\u5e01I", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINQIANDUOBAOHUOBII"}, "000839": {"suoxie": "STMGCZFLZQA", "name": "\u4e0a\u6295\u6469\u6839\u7eaf\u503a\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENCHUNZHAIFENGLIZHAIQUANA"}, "000840": {"suoxie": "STMGCZFLZQC", "name": "\u4e0a\u6295\u6469\u6839\u7eaf\u503a\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENCHUNZHAIFENGLIZHAIQUANC"}, "000841": {"suoxie": "FGXHBLHPZHHAB", "name": "\u5bcc\u56fd\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINHUIBAOLINGHUOPEIZHIHUNHEAB"}, "000842": {"suoxie": "FGXHBLHPZHHAB", "name": "\u5bcc\u56fd\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINHUIBAOLINGHUOPEIZHIHUNHEAB"}, "000843": {"suoxie": "FGXHBLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINHUIBAOLINGHUOPEIZHIHUNHEC"}, "000844": {"suoxie": "NFJDSY", "name": "\u5357\u65b9\u7edd\u5bf9\u6536\u76ca", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "NANFANGJUEDUISHOUYI"}, "000845": {"suoxie": "GTRYXXXFHH", "name": "\u56fd\u6295\u745e\u94f6\u4fe1\u606f\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINXIXIAOFEIHUNHE"}, "000846": {"suoxie": "ZRHBC", "name": "\u4e2d\u878d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGHUOBIC"}, "000847": {"suoxie": "ZRHBA", "name": "\u4e2d\u878d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGHUOBIA"}, "000848": {"suoxie": "GYTYKXHB", "name": "\u5de5\u94f6\u6dfb\u76ca\u5feb\u7ebf\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINTIANYIKUAIXIANHUOBI"}, "000849": {"suoxie": "HFJXSHCLHHA", "name": "\u6c47\u4e30\u664b\u4fe1\u53cc\u6838\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINSHUANGHECELUEHUNHEA"}, "000850": {"suoxie": "HFJXSHCLHHC", "name": "\u6c47\u4e30\u664b\u4fe1\u53cc\u6838\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINSHUANGHECELUEHUNHEC"}, "000854": {"suoxie": "PHYLCYGP", "name": "\u9e4f\u534e\u517b\u8001\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAYANGLAOCHANYEGUPIAO"}, "000855": {"suoxie": "STMGTTYHBA", "name": "\u4e0a\u6295\u6469\u6839\u5929\u6dfb\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENTIANTIANYINGHUOBIA"}, "000856": {"suoxie": "STMGTTYHBB", "name": "\u4e0a\u6295\u6469\u6839\u5929\u6dfb\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENTIANTIANYINGHUOBIB"}, "000857": {"suoxie": "STMGTTYHBE", "name": "\u4e0a\u6295\u6469\u6839\u5929\u6dfb\u76c8\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENTIANTIANYINGHUOBIE"}, "000858": {"suoxie": "RTTRZQAB", "name": "\u878d\u901a\u901a\u745e\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGRUIZHAIQUANAB"}, "000859": {"suoxie": "RTTRZQC", "name": "\u878d\u901a\u901a\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGRUIZHAIQUANC"}, "000860": {"suoxie": "YHHZLHB", "name": "\u94f6\u534e\u60e0\u589e\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUIZENGLIHUOBI"}, "000861": {"suoxie": "CCHBE", "name": "\u957f\u57ce\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGHUOBIE"}, "000862": {"suoxie": "FGTSHBC", "name": "\u5bcc\u56fd\u5929\u65f6\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOTIANSHIHUOBIC"}, "000863": {"suoxie": "FGTSHBD", "name": "\u5bcc\u56fd\u5929\u65f6\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOTIANSHIHUOBID"}, "000866": {"suoxie": "HBZZGP", "name": "\u534e\u5b9d\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUABAOZHIZAOGUPIAO"}, "000867": {"suoxie": "HBPZSHGP", "name": "\u534e\u5b9d\u54c1\u8d28\u751f\u6d3b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUABAOPINZHISHENGHUOGUPIAO"}, "000868": {"suoxie": "GTRYZLBHBA", "name": "\u56fd\u6295\u745e\u94f6\u589e\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINZENGLIBAOHUOBIA"}, "000869": {"suoxie": "GTRYZLBHBB", "name": "\u56fd\u6295\u745e\u94f6\u589e\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINZENGLIBAOHUOBIB"}, "000870": {"suoxie": "JSXSYHH", "name": "\u5609\u5b9e\u65b0\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINSHOUYIHUNHE"}, "000871": {"suoxie": "BXRFYTBA", "name": "\u5317\u4fe1\u745e\u4e30\u5b9c\u6295\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "BEIXINRUIFENGYITOUBAOA"}, "000872": {"suoxie": "BXRFYTBB", "name": "\u5317\u4fe1\u745e\u4e30\u5b9c\u6295\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "BEIXINRUIFENGYITOUBAOB"}, "000873": {"suoxie": "HAXJBHBA", "name": "\u534e\u5b89\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANXIANJINBAOHUOBIA"}, "000874": {"suoxie": "HAXJBHBB", "name": "\u534e\u5b89\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANXIANJINBAOHUOBIB"}, "000875": {"suoxie": "JXWDDLZQA", "name": "\u5efa\u4fe1\u7a33\u5b9a\u5f97\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINWENDINGDELIZHAIQUANA"}, "000876": {"suoxie": "JXWDDLZQC", "name": "\u5efa\u4fe1\u7a33\u5b9a\u5f97\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINWENDINGDELIZHAIQUANC"}, "000877": {"suoxie": "HTBRLHYXHH", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAYOUXUANHUNHE"}, "000878": {"suoxie": "ZHYYJKCYJXHHA", "name": "\u4e2d\u6d77\u533b\u836f\u5065\u5eb7\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIYIYAOJIANKANGCHANYEJINGXUANHUNHEA"}, "000879": {"suoxie": "ZHYYJKCYJXHHC", "name": "\u4e2d\u6d77\u533b\u836f\u5065\u5eb7\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIYIYAOJIANKANGCHANYEJINGXUANHUNHEC"}, "000880": {"suoxie": "FGYJJXLHPZHH", "name": "\u5bcc\u56fd\u7814\u7a76\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOYANJIUJINGXUANLINGHUOPEIZHIHUNHE"}, "000882": {"suoxie": "ZJXJGJA", "name": "\u4e2d\u91d1\u73b0\u91d1\u7ba1\u5bb6A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGJINXIANJINGUANJIAA"}, "000883": {"suoxie": "ZJXJGJB", "name": "\u4e2d\u91d1\u73b0\u91d1\u7ba1\u5bb6B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGJINXIANJINGUANJIAB"}, "000884": {"suoxie": "MSJYYXGP", "name": "\u6c11\u751f\u52a0\u94f6\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINYOUXUANGUPIAO"}, "000891": {"suoxie": "BSXJBHBB", "name": "\u535a\u65f6\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINBAOHUOBIB"}, "000892": {"suoxie": "JTTBLHPZHHA", "name": "\u4e5d\u6cf0\u5929\u5b9d\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAITIANBAOLINGHUOPEIZHIHUNHEA"}, "000893": {"suoxie": "GYCXDLGP", "name": "\u5de5\u94f6\u521b\u65b0\u52a8\u529b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINCHUANGXINDONGLIGUPIAO"}, "000894": {"suoxie": "ZORDDQKFHHA", "name": "\u4e2d\u6b27\u777f\u8fbe\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURUIDADINGQIKAIFANGHUNHEA"}, "000895": {"suoxie": "GSABXJBHB", "name": "\u56fd\u5bff\u5b89\u4fdd\u85aa\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOXINJINBAOHUOBI"}, "000896": {"suoxie": "XYJXSYZQZA", "name": "\u946b\u5143\u805a\u946b\u6536\u76ca\u589e\u5f3a\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANJUXINSHOUYIZENGQIANGZHAIA"}, "000897": {"suoxie": "XYJXSYZQZC", "name": "\u946b\u5143\u805a\u946b\u6536\u76ca\u589e\u5f3a\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANJUXINSHOUYIZENGQIANGZHAIC"}, "000898": {"suoxie": "HFHWCZZQA", "name": "\u534e\u5bcc\u6052\u7a33\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGWENCHUNZHAIZHAIQUANA"}, "000899": {"suoxie": "HFHWCZZQC", "name": "\u534e\u5bcc\u6052\u7a33\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGWENCHUNZHAIZHAIQUANC"}, "000903": {"suoxie": "XHHQTLHBA", "name": "\u65b0\u534e\u6d3b\u671f\u6dfb\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAHUOQITIANLIHUOBIA"}, "000904": {"suoxie": "YHHBDKHH", "name": "\u94f6\u534e\u56de\u62a5\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHUIBAODINGKAIHUNHE"}, "000905": {"suoxie": "PHAYBHB", "name": "\u9e4f\u534e\u5b89\u76c8\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAANYINGBAOHUOBI"}, "000906": {"suoxie": "GFQQJXGPMYXH", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u80a1\u7968\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANGUPIAOMEIYUANXIANHUI"}, "000907": {"suoxie": "NYHLRJHBA", "name": "\u519c\u94f6\u7ea2\u5229\u65e5\u7ed3\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHONGLIRIJIEHUOBIA"}, "000908": {"suoxie": "NYHLRJHBB", "name": "\u519c\u94f6\u7ea2\u5229\u65e5\u7ed3\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHONGLIRIJIEHUOBIB"}, "000910": {"suoxie": "XYHFCZC", "name": "\u946b\u5143\u5408\u4e30\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHEFENGCHUNZHAIC"}, "000911": {"suoxie": "XYHFCZA", "name": "\u946b\u5143\u5408\u4e30\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHEFENGCHUNZHAIA"}, "000912": {"suoxie": "YDXJBHBA", "name": "\u82f1\u5927\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINGDAXIANJINBAOHUOBIA"}, "000913": {"suoxie": "NYYLBJGP", "name": "\u519c\u94f6\u533b\u7597\u4fdd\u5065\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NONGYINYILIAOBAOJIANGUPIAO"}, "000914": {"suoxie": "ZJCZZQ", "name": "\u4e2d\u52a0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAIZHAIQUAN"}, "000916": {"suoxie": "QHKYGXL100QGP", "name": "\u524d\u6d77\u5f00\u6e90\u80a1\u606f\u7387100\u5f3a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANGUXILV100QIANGGUPIAO"}, "000917": {"suoxie": "JSKXHBA", "name": "\u5609\u5b9e\u5feb\u7ebf\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIKUAIXIANHUOBIA"}, "000920": {"suoxie": "YFDCFKXHBY", "name": "\u6613\u65b9\u8fbe\u8d22\u5bcc\u5feb\u7ebf\u8d27\u5e01Y", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDACAIFUKUAIXIANHUOBIY"}, "000921": {"suoxie": "ZYXJYZHBA", "name": "\u4e2d\u90ae\u73b0\u91d1\u9a7f\u7ad9\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYOUXIANJINYIZHANHUOBIA"}, "000922": {"suoxie": "ZYXJYZHBB", "name": "\u4e2d\u90ae\u73b0\u91d1\u9a7f\u7ad9\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYOUXIANJINYIZHANHUOBIB"}, "000923": {"suoxie": "ZYXJYZHBC", "name": "\u4e2d\u90ae\u73b0\u91d1\u9a7f\u7ad9\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYOUXIANJINYIZHANHUOBIC"}, "000924": {"suoxie": "BYXJZZHHA", "name": "\u5b9d\u76c8\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXIANJINZHIZAOHUNHEA"}, "000925": {"suoxie": "HTFWYZZGPA", "name": "\u6c47\u6dfb\u5bcc\u5916\u5ef6\u589e\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUWAIYANZENGZHANGGUPIAOA"}, "000926": {"suoxie": "ZXJTRXLHPZHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOURUIXINLINGHUOPEIZHIHUNHEA"}, "000927": {"suoxie": "BSDZHYTJXMYXH", "name": "\u535a\u65f6\u5927\u4e2d\u534e\u4e9a\u592a\u7cbe\u9009\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "BOSHIDAZHONGHUAYATAIJINGXUANMEIYUANXIANHUI"}, "000928": {"suoxie": "ZRGQGGHH", "name": "\u4e2d\u878d\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGGUOQIGAIGEHUNHE"}, "000929": {"suoxie": "BSHJD", "name": "\u535a\u65f6\u9ec4\u91d1D", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "BOSHIHUANGJIND"}, "000930": {"suoxie": "BSHJI", "name": "\u535a\u65f6\u9ec4\u91d1I", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "BOSHIHUANGJINI"}, "000931": {"suoxie": "GSABZYXYCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u76ca\u4fe1\u7528\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNYIXINYONGCHUNZHAIZHAIQUAN"}, "000932": {"suoxie": "QHKYRYWJZLHHA", "name": "\u524d\u6d77\u5f00\u6e90\u777f\u8fdc\u7a33\u5065\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANRUIYUANWENJIANZENGLIHUNHEA"}, "000933": {"suoxie": "QHKYRYWJZLHHC", "name": "\u524d\u6d77\u5f00\u6e90\u777f\u8fdc\u7a33\u5065\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANRUIYUANWENJIANZENGLIHUNHEC"}, "000934": {"suoxie": "GFDZHJXHH", "name": "\u56fd\u5bcc\u5927\u4e2d\u534e\u7cbe\u9009\u6df7\u5408", "style": "QDII", "pinyin": "GUOFUDAZHONGHUAJINGXUANHUNHE"}, "000935": {"suoxie": "ZSHJZXCZ", "name": "\u6d59\u5546\u6c47\u91d1\u8f6c\u578b\u6210\u957f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUIJINZHUANXINGCHENGZHANG"}, "000936": {"suoxie": "BSCYXDLHHA", "name": "\u535a\u65f6\u4ea7\u4e1a\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEXINDONGLIHUNHEA"}, "000939": {"suoxie": "ZYYJJXLHPZHHA", "name": "\u4e2d\u94f6\u7814\u7a76\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYANJIUJINGXUANLINGHUOPEIZHIHUNHEA"}, "000940": {"suoxie": "FGZXPJXHH", "name": "\u5bcc\u56fd\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHONGXIAOPANJINGXUANHUNHE"}, "000942": {"suoxie": "GFXXJSLJA", "name": "\u5e7f\u53d1\u4fe1\u606f\u6280\u672f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAXINXIJISHULIANJIEA"}, "000943": {"suoxie": "GYZGDJXYZZQA", "name": "\u5de5\u94f6\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINZHONGGAODENGJIXINYONGZHAIZHAIQUANA"}, "000944": {"suoxie": "GYZGDJXYZZQB", "name": "\u5de5\u94f6\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINZHONGGAODENGJIXINYONGZHAIZHAIQUANB"}, "000945": {"suoxie": "HXYLJKHHA", "name": "\u534e\u590f\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYILIAOJIANKANGHUNHEA"}, "000946": {"suoxie": "HXYLJKHHC", "name": "\u534e\u590f\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYILIAOJIANKANGHUNHEC"}, "000948": {"suoxie": "HXHGTHSETFLJA", "name": "\u534e\u590f\u6caa\u6e2f\u901a\u6052\u751fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUGANGTONGHENGSHENGETFLIANJIEA"}, "000950": {"suoxie": "YFDHS300FYLJA", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u975e\u94f6\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300FEIYINLIANJIEA"}, "000953": {"suoxie": "GTRJLHPZHHA", "name": "\u56fd\u6cf0\u777f\u5409\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRUIJILINGHUOPEIZHIHUNHEA"}, "000954": {"suoxie": "GTRJLHPZHHC", "name": "\u56fd\u6cf0\u777f\u5409\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRUIJILINGHUOPEIZHIHUNHEC"}, "000955": {"suoxie": "NFCYHL", "name": "\u5357\u65b9\u4ea7\u4e1a\u6d3b\u529b", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGCHANYEHUOLI"}, "000960": {"suoxie": "ZSYYJKCYGP", "name": "\u62db\u5546\u533b\u836f\u5065\u5eb7\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGYIYAOJIANKANGCHANYEGUPIAO"}, "000961": {"suoxie": "THHS300ETFLJA", "name": "\u5929\u5f18\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUSHEN300ETFLIANJIEA"}, "000962": {"suoxie": "THZZ500ETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG500ETFLIANJIEA"}, "000963": {"suoxie": "XYDCLHH", "name": "\u5174\u4e1a\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEDUOCELUEHUNHE"}, "000965": {"suoxie": "HFJXXDLHH", "name": "\u6c47\u4e30\u664b\u4fe1\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINXINDONGLIHUNHE"}, "000966": {"suoxie": "ZYHXKJCXLHPZHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u79d1\u6280\u521b\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUHEXINKEJICHUANGXINLINGHUOPEIZHIHUNHE"}, "000967": {"suoxie": "HTBRCXDLHH", "name": "\u534e\u6cf0\u67cf\u745e\u521b\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUICHUANGXINDONGLIHUNHE"}, "000968": {"suoxie": "GFYLZSA", "name": "\u5e7f\u53d1\u517b\u8001\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYANGLAOZHISHUA"}, "000969": {"suoxie": "QHKYDAQHH", "name": "\u524d\u6d77\u5f00\u6e90\u5927\u5b89\u5168\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANDAANQUANHUNHE"}, "000970": {"suoxie": "DFHRYHH", "name": "\u4e1c\u65b9\u7ea2\u777f\u5143\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIYUANHUNHE"}, "000971": {"suoxie": "NAXJJGP", "name": "\u8bfa\u5b89\u65b0\u7ecf\u6d4e\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANXINJINGJIGUPIAO"}, "000972": {"suoxie": "XHWYCLLHPZHH", "name": "\u65b0\u534e\u4e07\u94f6\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAWANYINCELUELINGHUOPEIZHIHUNHE"}, "000973": {"suoxie": "XHZYHBZQ", "name": "\u65b0\u534e\u589e\u76c8\u56de\u62a5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGYINGHUIBAOZHAIQUAN"}, "000974": {"suoxie": "AXXFYYGP", "name": "\u5b89\u4fe1\u6d88\u8d39\u533b\u836f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINXIAOFEIYIYAOGUPIAO"}, "000975": {"suoxie": "MSCIZGAGGJTLJA", "name": "MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MSCIZHONGGUOAGUGUOJITONGLIANJIEA"}, "000976": {"suoxie": "CCXXCYHH", "name": "\u957f\u57ce\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGXINXINGCHANYEHUNHE"}, "000977": {"suoxie": "CCHBZTHH", "name": "\u957f\u57ce\u73af\u4fdd\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGHUANBAOZHUTIHUNHE"}, "000978": {"suoxie": "JSCCLHJXGP", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAJINGXUANGUPIAO"}, "000979": {"suoxie": "JSCCHGSJXGP", "name": "\u666f\u987a\u957f\u57ce\u6caa\u6e2f\u6df1\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGHUGANGSHENJINGXUANGUPIAO"}, "000980": {"suoxie": "HTFTFTHBB", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u5bcc\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUTIANFUTONGHUOBIB"}, "000981": {"suoxie": "BXRFXJTLA", "name": "\u5317\u4fe1\u745e\u4e30\u73b0\u91d1\u6dfb\u5229A", "style": "\u8d27\u5e01\u578b", "pinyin": "BEIXINRUIFENGXIANJINTIANLIA"}, "000982": {"suoxie": "BXRFXJTLB", "name": "\u5317\u4fe1\u745e\u4e30\u73b0\u91d1\u6dfb\u5229B", "style": "\u8d27\u5e01\u578b", "pinyin": "BEIXINRUIFENGXIANJINTIANLIB"}, "000985": {"suoxie": "JSNXCLGP", "name": "\u5609\u5b9e\u9006\u5411\u7b56\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHINIXIANGCELUEGUPIAO"}, "000986": {"suoxie": "TPLHPZ", "name": "\u592a\u5e73\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIPINGLINGHUOPEIZHI"}, "000988": {"suoxie": "JSQQHLWGPRMB", "name": "\u5609\u5b9e\u5168\u7403\u4e92\u8054\u7f51\u80a1\u7968\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "JIASHIQUANQIUHULIANWANGGUPIAORENMINBI"}, "000989": {"suoxie": "JSQQHLWGPMYXH", "name": "\u5609\u5b9e\u5168\u7403\u4e92\u8054\u7f51\u80a1\u7968\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "JIASHIQUANQIUHULIANWANGGUPIAOMEIYUANXIANHUI"}, "000990": {"suoxie": "JSQQHLWGPMYXC", "name": "\u5609\u5b9e\u5168\u7403\u4e92\u8054\u7f51\u80a1\u7968\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "JIASHIQUANQIUHULIANWANGGUPIAOMEIYUANXIANCHAO"}, "000991": {"suoxie": "GYZLZXGPA", "name": "\u5de5\u94f6\u6218\u7565\u8f6c\u578b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINZHANLUEZHUANXINGGUPIAOA"}, "000992": {"suoxie": "GFDCTLDQKFHH", "name": "\u5e7f\u53d1\u5bf9\u51b2\u5957\u5229\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GUANGFADUICHONGTAOLIDINGQIKAIFANGHUNHE"}, "000993": {"suoxie": "HBWJHBHH", "name": "\u534e\u5b9d\u7a33\u5065\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOWENJIANHUIBAOHUNHE"}, "000994": {"suoxie": "JXRYLHPZHHA", "name": "\u5efa\u4fe1\u777f\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINRUIYINGLINGHUOPEIZHIHUNHEA"}, "000995": {"suoxie": "JXRYLHPZHHC", "name": "\u5efa\u4fe1\u777f\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINRUIYINGLINGHUOPEIZHIHUNHEC"}, "000996": {"suoxie": "ZYXDLGPA", "name": "\u4e2d\u94f6\u65b0\u52a8\u529b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINXINDONGLIGUPIAOA"}, "000997": {"suoxie": "NFSYA", "name": "\u5357\u65b9\u53cc\u5143A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGSHUANGYUANA"}, "000998": {"suoxie": "NFSYC", "name": "\u5357\u65b9\u53cc\u5143C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGSHUANGYUANC"}, "001000": {"suoxie": "ZOMRXQDHH", "name": "\u4e2d\u6b27\u660e\u777f\u65b0\u8d77\u70b9\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUMINGRUIXINQIDIANHUNHE"}, "001001": {"suoxie": "HXZQAB", "name": "\u534e\u590f\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHAIQUANAB"}, "001002": {"suoxie": "HXZQAB", "name": "\u534e\u590f\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHAIQUANAB"}, "001003": {"suoxie": "HXZQC", "name": "\u534e\u590f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHAIQUANC"}, "001004": {"suoxie": "XHWJHBHHFQS", "name": "\u65b0\u534e\u7a33\u5065\u56de\u62a5\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAWENJIANHUIBAOHUNHEFAQISHI"}, "001006": {"suoxie": "ZXJTFHHBA", "name": "\u4e2d\u4fe1\u5efa\u6295\u51e4\u51f0\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGXINJIANTOUFENGHUANGHUOBIA"}, "001007": {"suoxie": "GLAXALHPZHH", "name": "\u56fd\u8054\u5b89\u946b\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANXINANLINGHUOPEIZHIHUNHE"}, "001008": {"suoxie": "GYGQGGGP", "name": "\u5de5\u94f6\u56fd\u4f01\u6539\u9769\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINGUOQIGAIGEGUPIAO"}, "001009": {"suoxie": "STMGAQZLGP", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u5168\u6218\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENANQUANZHANLUEGUPIAO"}, "001010": {"suoxie": "YFDZJBHBA", "name": "\u6613\u65b9\u8fbe\u589e\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAZENGJINBAOHUOBIA"}, "001011": {"suoxie": "HXXWZQA", "name": "\u534e\u590f\u5e0c\u671b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAXIWANGZHAIQUANA"}, "001013": {"suoxie": "HXXWZQC", "name": "\u534e\u590f\u5e0c\u671b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAXIWANGZHAIQUANC"}, "001015": {"suoxie": "HXHS300ZSZQA", "name": "\u534e\u590f\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ZHISHUZENGQIANGA"}, "001016": {"suoxie": "HXHS300ZSZQC", "name": "\u534e\u590f\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ZHISHUZENGQIANGC"}, "001017": {"suoxie": "TDGGDLHHA", "name": "\u6cf0\u8fbe\u6539\u9769\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAGAIGEDONGLIHUNHEA"}, "001018": {"suoxie": "YFDXJJHH", "name": "\u6613\u65b9\u8fbe\u65b0\u7ecf\u6d4e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINJINGJIHUNHE"}, "001019": {"suoxie": "XYNNLDKZ", "name": "\u5174\u4e1a\u5e74\u5e74\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYENIANNIANLIDINGKAIZHAI"}, "001021": {"suoxie": "HXYZZGZSA", "name": "\u534e\u590f\u4e9a\u503a\u4e2d\u56fd\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAYAZHAIZHONGGUOZHISHUA"}, "001023": {"suoxie": "HXYZZGZSC", "name": "\u534e\u590f\u4e9a\u503a\u4e2d\u56fd\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAYAZHAIZHONGGUOZHISHUC"}, "001026": {"suoxie": "NALCBHBC", "name": "\u8bfa\u5b89\u7406\u8d22\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANLICAIBAOHUOBIC"}, "001027": {"suoxie": "QHKYZZDNYZSZQ", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u8bc1\u5927\u519c\u4e1a\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGZHENGDANONGYEZHISHUZENGQIANG"}, "001028": {"suoxie": "HAWLWZTGP", "name": "\u534e\u5b89\u7269\u8054\u7f51\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANWULIANWANGZHUTIGUPIAO"}, "001030": {"suoxie": "THYDSHYX", "name": "\u5929\u5f18\u4e91\u7aef\u751f\u6d3b\u4f18\u9009", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGYUNDUANSHENGHUOYOUXUAN"}, "001031": {"suoxie": "HXAKZQA", "name": "\u534e\u590f\u5b89\u5eb7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAANKANGZHAIQUANA"}, "001033": {"suoxie": "HXAKZQC", "name": "\u534e\u590f\u5b89\u5eb7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAANKANGZHAIQUANC"}, "001035": {"suoxie": "ZYHLBNDKZ", "name": "\u4e2d\u94f6\u6052\u5229\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINHENGLIBANNIANDINGKAIZHAI"}, "001036": {"suoxie": "JSQYBGGP", "name": "\u5609\u5b9e\u4f01\u4e1a\u53d8\u9769\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIQIYEBIANGEGUPIAO"}, "001037": {"suoxie": "GTRYRYGGHH", "name": "\u56fd\u6295\u745e\u94f6\u9510\u610f\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUIYIGAIGEHUNHE"}, "001039": {"suoxie": "JSXJZZGP", "name": "\u5609\u5b9e\u5148\u8fdb\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXIANJINZHIZAOGUPIAO"}, "001040": {"suoxie": "XHCLJXGP", "name": "\u65b0\u534e\u7b56\u7565\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINHUACELUEJINGXUANGUPIAO"}, "001042": {"suoxie": "HXLXGP", "name": "\u534e\u590f\u9886\u5148\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIALINGXIANGUPIAO"}, "001043": {"suoxie": "GYMLCZGPA", "name": "\u5de5\u94f6\u7f8e\u4e3d\u57ce\u9547\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINMEILICHENGZHENGUPIAOA"}, "001044": {"suoxie": "JSXXFGP", "name": "\u5609\u5b9e\u65b0\u6d88\u8d39\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXINXIAOFEIGUPIAO"}, "001045": {"suoxie": "HXKZZZQZQA", "name": "\u534e\u590f\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAXIAKEZHUANZHAIZENGQIANGZHAIQUANA"}, "001046": {"suoxie": "HXKZZZQZQI", "name": "\u534e\u590f\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238I", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAXIAKEZHUANZHAIZENGQIANGZHAIQUANI"}, "001047": {"suoxie": "GDGQGGGP", "name": "\u5149\u5927\u56fd\u4f01\u6539\u9769\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDAGUOQIGAIGEGUPIAO"}, "001048": {"suoxie": "FGXXCYGP", "name": "\u5bcc\u56fd\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOXINXINGCHANYEGUPIAO"}, "001050": {"suoxie": "HTFCZDYZLHCLGP", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u591a\u56e0\u5b50\u91cf\u5316\u7b56\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUCHENGZHANGDUOYINZILIANGHUACELUEGUPIAO"}, "001051": {"suoxie": "HXSZ50ETFLJA", "name": "\u534e\u590f\u4e0a\u8bc150ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENG50ETFLIANJIEA"}, "001052": {"suoxie": "HXZZ500ETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ETFLIANJIEA"}, "001053": {"suoxie": "NFCXJJ", "name": "\u5357\u65b9\u521b\u65b0\u7ecf\u6d4e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGCHUANGXINJINGJI"}, "001054": {"suoxie": "GYXJRGPA", "name": "\u5de5\u94f6\u65b0\u91d1\u878d\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXINJINRONGGUPIAOA"}, "001056": {"suoxie": "BXRFJKSH", "name": "\u5317\u4fe1\u745e\u4e30\u5065\u5eb7\u751f\u6d3b", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGJIANKANGSHENGHUO"}, "001057": {"suoxie": "HXLC30TZQA", "name": "\u534e\u590f\u7406\u8d2230\u5929\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIALICAI30TIANZHAIQUANA"}, "001058": {"suoxie": "HXLC30TZQB", "name": "\u534e\u590f\u7406\u8d2230\u5929\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIALICAI30TIANZHAIQUANB"}, "001059": {"suoxie": "ZJJDSY", "name": "\u4e2d\u91d1\u7edd\u5bf9\u6536\u76ca", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ZHONGJINJUEDUISHOUYI"}, "001060": {"suoxie": "QHKYGDZBZZHH", "name": "\u524d\u6d77\u5f00\u6e90\u9ad8\u7aef\u88c5\u5907\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANGAODUANZHUANGBEIZHIZAOHUNHE"}, "001061": {"suoxie": "HXSYZQQDIIA", "name": "\u534e\u590f\u6536\u76ca\u503a\u5238(QDII)A", "style": "QDII", "pinyin": "HUAXIASHOUYIZHAIQUANQDIIA"}, "001063": {"suoxie": "HXSYZQQDIIC", "name": "\u534e\u590f\u6536\u76ca\u503a\u5238(QDII)C", "style": "QDII", "pinyin": "HUAXIASHOUYIZHAIQUANQDIIC"}, "001064": {"suoxie": "GFZZHBETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u73af\u4fddETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGHUANBAOETFLIANJIEA"}, "001065": {"suoxie": "HXHWSYZQXH", "name": "\u534e\u590f\u6d77\u5916\u6536\u76ca\u503a\u5238\u73b0\u6c47", "style": "QDII", "pinyin": "HUAXIAHAIWAISHOUYIZHAIQUANXIANHUI"}, "001066": {"suoxie": "HXHWSYZQXC", "name": "\u534e\u590f\u6d77\u5916\u6536\u76ca\u503a\u5238\u73b0\u949e", "style": "QDII", "pinyin": "HUAXIAHAIWAISHOUYIZHAIQUANXIANCHAO"}, "001067": {"suoxie": "PHHSHHA", "name": "\u9e4f\u534e\u5f18\u76db\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHENGHUNHEA"}, "001068": {"suoxie": "HRXRLHPZHH", "name": "\u534e\u878d\u65b0\u9510\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGXINRUILINGHUOPEIZHIHUNHE"}, "001069": {"suoxie": "HTBRXFCZHH", "name": "\u534e\u6cf0\u67cf\u745e\u6d88\u8d39\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXIAOFEICHENGZHANGHUNHE"}, "001070": {"suoxie": "JXXXCYGPA", "name": "\u5efa\u4fe1\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINXINXICHANYEGUPIAOA"}, "001071": {"suoxie": "HAMTHLWHHA", "name": "\u534e\u5b89\u5a92\u4f53\u4e92\u8054\u7f51\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANMEITIHULIANWANGHUNHEA"}, "001072": {"suoxie": "HAZNZBZTGPA", "name": "\u534e\u5b89\u667a\u80fd\u88c5\u5907\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANZHINENGZHUANGBEIZHUTIGUPIAOA"}, "001073": {"suoxie": "HTBRLHJDSYHH", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u7edd\u5bf9\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUATAIBAIRUILIANGHUAJUEDUISHOUYIHUNHE"}, "001074": {"suoxie": "HTBRLHQDHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAQUDONGHUNHEA"}, "001075": {"suoxie": "BYZXDLHH", "name": "\u5b9d\u76c8\u8f6c\u578b\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGZHUANXINGDONGLIHUNHE"}, "001076": {"suoxie": "YFDGGHLHH", "name": "\u6613\u65b9\u8fbe\u6539\u9769\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGAIGEHONGLIHUNHE"}, "001077": {"suoxie": "HXXJBHBA", "name": "\u534e\u590f\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAXIANJINBAOHUOBIA"}, "001078": {"suoxie": "HXXJBHBB", "name": "\u534e\u590f\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAXIANJINBAOHUOBIB"}, "001086": {"suoxie": "HFHLZQA", "name": "\u534e\u5bcc\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUHENGLIZHAIQUANA"}, "001087": {"suoxie": "HFHLZQC", "name": "\u534e\u5bcc\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUHENGLIZHAIQUANC"}, "001088": {"suoxie": "HBGCDXHH", "name": "\u534e\u5b9d\u56fd\u7b56\u5bfc\u5411\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOGUOCEDAOXIANGHUNHE"}, "001092": {"suoxie": "GFSWKJZSQDII", "name": "\u5e7f\u53d1\u751f\u7269\u79d1\u6280\u6307\u6570(QDII)", "style": "QDII", "pinyin": "GUANGFASHENGWUKEJIZHISHUQDII"}, "001093": {"suoxie": "GFSWKJZSMY", "name": "\u5e7f\u53d1\u751f\u7269\u79d1\u6280\u6307\u6570\u7f8e\u5143", "style": "QDII", "pinyin": "GUANGFASHENGWUKEJIZHISHUMEIYUAN"}, "001094": {"suoxie": "GTRYTLBHBA", "name": "\u56fd\u6295\u745e\u94f6\u6dfb\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINTIANLIBAOHUOBIA"}, "001095": {"suoxie": "GTRYTLBHBB", "name": "\u56fd\u6295\u745e\u94f6\u6dfb\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINTIANLIBAOHUOBIB"}, "001096": {"suoxie": "GSABJBPHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u805a\u5b9d\u76c6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOJUBAOPENHUOBIA"}, "001097": {"suoxie": "HTBRJJYXGP", "name": "\u534e\u6cf0\u67cf\u745e\u79ef\u6781\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUATAIBAIRUIJIJIYOUXUANGUPIAO"}, "001101": {"suoxie": "YHHTYHB", "name": "\u94f6\u534e\u60e0\u6dfb\u76ca\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUITIANYIHUOBI"}, "001102": {"suoxie": "QHKYGJBJYSHHA", "name": "\u524d\u6d77\u5f00\u6e90\u56fd\u5bb6\u6bd4\u8f83\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANGUOJIABIJIAOYOUSHIHUNHEA"}, "001103": {"suoxie": "QHKYGYGM40HH", "name": "\u524d\u6d77\u5f00\u6e90\u5de5\u4e1a\u9769\u547d4.0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANGONGYEGEMING40HUNHE"}, "001104": {"suoxie": "HAXSLZTGPA", "name": "\u534e\u5b89\u65b0\u4e1d\u8def\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANXINSILUZHUTIGUPIAOA"}, "001105": {"suoxie": "XDAYZXCXGP", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u8f6c\u578b\u521b\u65b0\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINDAAOYINZHUANXINGCHUANGXINGUPIAO"}, "001106": {"suoxie": "HSJKSHHH", "name": "\u534e\u5546\u5065\u5eb7\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGJIANKANGSHENGHUOHUNHE"}, "001110": {"suoxie": "ZOJQLHPZHHA", "name": "\u4e2d\u6b27\u747e\u6cc9\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINQUANLINGHUOPEIZHIHUNHEA"}, "001111": {"suoxie": "ZOJQLHPZHHC", "name": "\u4e2d\u6b27\u747e\u6cc9\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINQUANLINGHUOPEIZHIHUNHEC"}, "001112": {"suoxie": "DFHZGYSHH", "name": "\u4e1c\u65b9\u7ea2\u4e2d\u56fd\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGZHONGGUOYOUSHIHUNHE"}, "001113": {"suoxie": "NFDSJ100A", "name": "\u5357\u65b9\u5927\u6570\u636e100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDASHUJU100A"}, "001115": {"suoxie": "GFJAHHA", "name": "\u5e7f\u53d1\u805a\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUANHUNHEA"}, "001116": {"suoxie": "GFJAHHC", "name": "\u5e7f\u53d1\u805a\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUANHUNHEC"}, "001117": {"suoxie": "ZOJXDQKFHHA", "name": "\u4e2d\u6b27\u7cbe\u9009\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINGXUANDINGQIKAIFANGHUNHEA"}, "001118": {"suoxie": "HBSJQDHH", "name": "\u534e\u5b9d\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOSHIJIANQUDONGHUNHE"}, "001120": {"suoxie": "DFRXRDWJHHA", "name": "\u4e1c\u65b9\u777f\u946b\u70ed\u70b9\u6316\u6398\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGRUIXINREDIANWAJUEHUNHEA"}, "001121": {"suoxie": "DFRXRDWJHHC", "name": "\u4e1c\u65b9\u777f\u946b\u70ed\u70b9\u6316\u6398\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGRUIXINREDIANWAJUEHUNHEC"}, "001122": {"suoxie": "PHHLHHA", "name": "\u9e4f\u534e\u5f18\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGLIHUNHEA"}, "001123": {"suoxie": "PHHLHHC", "name": "\u9e4f\u534e\u5f18\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGLIHUNHEC"}, "001124": {"suoxie": "RTZQSYZQC", "name": "\u878d\u901a\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGZENGQIANGSHOUYIZHAIQUANC"}, "001125": {"suoxie": "BSHLWZTLHPZHH", "name": "\u535a\u65f6\u4e92\u8054\u7f51\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHULIANWANGZHUTILINGHUOPEIZHIHUNHE"}, "001126": {"suoxie": "STMGZYZZGP", "name": "\u4e0a\u6295\u6469\u6839\u5353\u8d8a\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENZHUOYUEZHIZAOGUPIAO"}, "001127": {"suoxie": "ZYHGCLHH", "name": "\u4e2d\u94f6\u5b8f\u89c2\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINHONGGUANCELUEHUNHE"}, "001128": {"suoxie": "BYXXCYHHA", "name": "\u5b9d\u76c8\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINXINGCHANYEHUNHEA"}, "001133": {"suoxie": "GFZZQZKXXFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u53ef\u9009\u6d88\u8d39\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIKEXUANXIAOFEILIANJIEA"}, "001134": {"suoxie": "GFTTLHBE", "name": "\u5e7f\u53d1\u5929\u5929\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANTIANLIHUOBIE"}, "001135": {"suoxie": "YMPZSJHH", "name": "\u76ca\u6c11\u54c1\u8d28\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIMINPINZHISHENGJIHUNHE"}, "001136": {"suoxie": "YFDYRHH", "name": "\u6613\u65b9\u8fbe\u88d5\u5982\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAYURUHUNHE"}, "001139": {"suoxie": "HAXDLLHPZHH", "name": "\u534e\u5b89\u65b0\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINDONGLILINGHUOPEIZHIHUNHE"}, "001140": {"suoxie": "GYZHBLHPZHHA", "name": "\u5de5\u94f6\u603b\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINZONGHUIBAOLINGHUOPEIZHIHUNHEA"}, "001141": {"suoxie": "TDHLCYHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u521b\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLICHUANGYINGHUNHEA"}, "001142": {"suoxie": "TDHLCYHHB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u521b\u76c8\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLICHUANGYINGHUNHEB"}, "001143": {"suoxie": "HSLHJQHH", "name": "\u534e\u5546\u91cf\u5316\u8fdb\u53d6\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGLIANGHUAJINQUHUNHE"}, "001144": {"suoxie": "DCHLWSWHH", "name": "\u5927\u6210\u4e92\u8054\u7f51\u601d\u7ef4\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHULIANWANGSIWEIHUNHE"}, "001146": {"suoxie": "ZOJYLHPZHHA", "name": "\u4e2d\u6b27\u747e\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINYUANLINGHUOPEIZHIHUNHEA"}, "001147": {"suoxie": "ZOJYLHPZHHC", "name": "\u4e2d\u6b27\u747e\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINYUANLINGHUOPEIZHIHUNHEC"}, "001148": {"suoxie": "SWLXDCLLHPZHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINDUOCELUELINGHUOPEIZHIHUNHEA"}, "001149": {"suoxie": "HFJXHSLTZSC", "name": "\u6c47\u4e30\u664b\u4fe1\u6052\u751f\u9f99\u5934\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIFENGJINXINHENGSHENGLONGTOUZHISHUC"}, "001150": {"suoxie": "RTHLWCMLHPZHH", "name": "\u878d\u901a\u4e92\u8054\u7f51\u4f20\u5a92\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGHULIANWANGCHUANMEILINGHUOPEIZHIHUNHE"}, "001151": {"suoxie": "RTHLWCMLHPZHH", "name": "\u878d\u901a\u4e92\u8054\u7f51\u4f20\u5a92\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGHULIANWANGCHUANMEILINGHUOPEIZHIHUNHE"}, "001152": {"suoxie": "RTXQYXJJLHPZHH", "name": "\u878d\u901a\u65b0\u533a\u57df\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINQUYUXINJINGJILINGHUOPEIZHIHUNHE"}, "001153": {"suoxie": "RTXQYXJJLHPZHH", "name": "\u878d\u901a\u65b0\u533a\u57df\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINQUYUXINJINGJILINGHUOPEIZHIHUNHE"}, "001154": {"suoxie": "BXRFPAZG", "name": "\u5317\u4fe1\u745e\u4e30\u5e73\u5b89\u4e2d\u56fd", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGPINGANZHONGGUO"}, "001156": {"suoxie": "SWLXXNYQCHH", "name": "\u7533\u4e07\u83f1\u4fe1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINXINNENGYUANQICHEHUNHE"}, "001157": {"suoxie": "GLARQLHPZHH", "name": "\u56fd\u8054\u5b89\u777f\u797a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANRUIQILINGHUOPEIZHIHUNHE"}, "001158": {"suoxie": "GYXCLXNYGP", "name": "\u5de5\u94f6\u65b0\u6750\u6599\u65b0\u80fd\u6e90\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXINCAILIAOXINNENGYUANGUPIAO"}, "001162": {"suoxie": "QHKYYSLCGPA", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u52bf\u84dd\u7b79\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANYOUSHILANCHOUGUPIAOA"}, "001163": {"suoxie": "YHZGM30GP", "name": "\u94f6\u534e\u4e2d\u56fd\u68a630\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAZHONGGUOMENG30GUPIAO"}, "001164": {"suoxie": "ZOQHLHPZHHA", "name": "\u4e2d\u6b27\u742a\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUQIHELINGHUOPEIZHIHUNHEA"}, "001165": {"suoxie": "ZOQHLHPZHHC", "name": "\u4e2d\u6b27\u742a\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUQIHELINGHUOPEIZHIHUNHEC"}, "001166": {"suoxie": "JXHBCYGP", "name": "\u5efa\u4fe1\u73af\u4fdd\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINHUANBAOCHANYEGUPIAO"}, "001167": {"suoxie": "JYKJCXGP", "name": "\u91d1\u9e70\u79d1\u6280\u521b\u65b0\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGKEJICHUANGXINGUPIAO"}, "001170": {"suoxie": "TDHLFXHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u590d\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIFUXINGHUNHE"}, "001171": {"suoxie": "GYYLCYGPA", "name": "\u5de5\u94f6\u517b\u8001\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYANGLAOCHANYEGUPIAOA"}, "001172": {"suoxie": "PHHZHHA", "name": "\u9e4f\u534e\u5f18\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGZEHUNHEA"}, "001173": {"suoxie": "ZOJHLHPZHHA", "name": "\u4e2d\u6b27\u747e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINHELINGHUOPEIZHIHUNHEA"}, "001174": {"suoxie": "ZOJHLHPZHHC", "name": "\u4e2d\u6b27\u747e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINHELINGHUOPEIZHIHUNHEC"}, "001175": {"suoxie": "SZRRTLHBA", "name": "\u5c71\u8bc1\u65e5\u65e5\u6dfb\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANZHENGRIRITIANLIHUOBIA"}, "001176": {"suoxie": "SZRRTLHBB", "name": "\u5c71\u8bc1\u65e5\u65e5\u6dfb\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANZHENGRIRITIANLIHUOBIB"}, "001177": {"suoxie": "SZRRTLHBC", "name": "\u5c71\u8bc1\u65e5\u65e5\u6dfb\u5229\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANZHENGRIRITIANLIHUOBIC"}, "001178": {"suoxie": "QHKYZRZGP", "name": "\u524d\u6d77\u5f00\u6e90\u518d\u878d\u8d44\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANZAIRONGZIGUPIAO"}, "001179": {"suoxie": "DBDJKLHPZHH", "name": "\u5fb7\u90a6\u5927\u5065\u5eb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGDAJIANKANGLINGHUOPEIZHIHUNHE"}, "001180": {"suoxie": "GFYYWSLJA", "name": "\u5e7f\u53d1\u533b\u836f\u536b\u751f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYIYAOWEISHENGLIANJIEA"}, "001181": {"suoxie": "NFGGJY", "name": "\u5357\u65b9\u6539\u9769\u673a\u9047", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGGAIGEJIYU"}, "001182": {"suoxie": "YFDAXHKHH", "name": "\u6613\u65b9\u8fbe\u5b89\u5fc3\u56de\u9988\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAANXINHUIKUIHUNHE"}, "001183": {"suoxie": "NFLTA", "name": "\u5357\u65b9\u5229\u6dd8A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLITAOA"}, "001184": {"suoxie": "YFDXCTLHPZHH", "name": "\u6613\u65b9\u8fbe\u65b0\u5e38\u6001\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINCHANGTAILINGHUOPEIZHIHUNHE"}, "001185": {"suoxie": "AXDTCLHHA", "name": "\u5b89\u4fe1\u52a8\u6001\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINDONGTAICELUEHUNHEA"}, "001186": {"suoxie": "FGWTJKGPA", "name": "\u5bcc\u56fd\u6587\u4f53\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOWENTIJIANKANGGUPIAOA"}, "001187": {"suoxie": "FGWTJKGPA", "name": "\u5bcc\u56fd\u6587\u4f53\u5065\u5eb7\u80a1\u7968A(\u540e\u7aef)", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOWENTIJIANKANGGUPIAOA"}, "001188": {"suoxie": "PHGGHLGP", "name": "\u9e4f\u534e\u6539\u9769\u7ea2\u5229\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAGAIGEHONGLIGUPIAO"}, "001189": {"suoxie": "GFJBHHA", "name": "\u5e7f\u53d1\u805a\u5b9d\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUBAOHUNHEA"}, "001190": {"suoxie": "PHHRHHA", "name": "\u9e4f\u534e\u5f18\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGRUNHUNHEA"}, "001191": {"suoxie": "PHHRHHC", "name": "\u9e4f\u534e\u5f18\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGRUNHUNHEC"}, "001192": {"suoxie": "STMGZHQDHH", "name": "\u4e0a\u6295\u6469\u6839\u6574\u5408\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENZHENGHEQUDONGHUNHE"}, "001193": {"suoxie": "ZJXFSJ", "name": "\u4e2d\u91d1\u6d88\u8d39\u5347\u7ea7", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINXIAOFEISHENGJI"}, "001194": {"suoxie": "JSCCWJHBHHA", "name": "\u666f\u987a\u957f\u57ce\u7a33\u5065\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGWENJIANHUIBAOHUNHEA"}, "001195": {"suoxie": "GYNYCYGP", "name": "\u5de5\u94f6\u519c\u4e1a\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINNONGYECHANYEGUPIAO"}, "001196": {"suoxie": "DFDXLHPZHHA", "name": "\u4e1c\u65b9\u9f0e\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGDINGXINLINGHUOPEIZHIHUNHEA"}, "001197": {"suoxie": "CSZXSJZTLHPZHH", "name": "\u957f\u76db\u8f6c\u578b\u5347\u7ea7\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGZHUANXINGSHENGJIZHUTILINGHUOPEIZHIHUNHE"}, "001198": {"suoxie": "DFHXLHPZHHA", "name": "\u4e1c\u65b9\u60e0\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHUIXINLINGHUOPEIZHIHUNHEA"}, "001199": {"suoxie": "CJHXJLZQA", "name": "\u521b\u91d1\u5408\u4fe1\u805a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUANGJINHEXINJULIZHAIQUANA"}, "001200": {"suoxie": "CJHXJLZQC", "name": "\u521b\u91d1\u5408\u4fe1\u805a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUANGJINHEXINJULIZHAIQUANC"}, "001201": {"suoxie": "SWLXAXHBHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINANXINHUIBAOHUNHEA"}, "001202": {"suoxie": "DFHLXJXHH", "name": "\u4e1c\u65b9\u7ea2\u9886\u5148\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGLINGXIANJINGXUANHUNHE"}, "001203": {"suoxie": "DFHWJJXHHA", "name": "\u4e1c\u65b9\u7ea2\u7a33\u5065\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGWENJIANJINGXUANHUNHEA"}, "001204": {"suoxie": "DFHWJJXHHC", "name": "\u4e1c\u65b9\u7ea2\u7a33\u5065\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGWENJIANJINGXUANHUNHEC"}, "001208": {"suoxie": "NADTJJGPA", "name": "\u8bfa\u5b89\u4f4e\u78b3\u7ecf\u6d4e\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANDITANJINGJIGUPIAOA"}, "001209": {"suoxie": "QHKYYDYLHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4e00\u5e26\u4e00\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYIDAIYILUHUNHEA"}, "001210": {"suoxie": "THHLWHH", "name": "\u5929\u5f18\u4e92\u8054\u7f51\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGHULIANWANGHUNHE"}, "001211": {"suoxie": "ZOGQBHBA", "name": "\u4e2d\u6b27\u6eda\u94b1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUGUNQIANBAOHUOBIA"}, "001212": {"suoxie": "HRYDWJZQA", "name": "\u534e\u6da6\u5143\u5927\u7a33\u5065\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDAWENJIANZHAIQUANA"}, "001213": {"suoxie": "HRYDWJZQC", "name": "\u534e\u6da6\u5143\u5927\u7a33\u5065\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDAWENJIANZHAIQUANC"}, "001214": {"suoxie": "HTBRZZ500ETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENG500ETFLIANJIEA"}, "001215": {"suoxie": "BSHGSYZQYJJA", "name": "\u535a\u65f6\u6caa\u6e2f\u6df1\u4f18\u8d28\u4f01\u4e1a\u57fa\u91d1A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUGANGSHENYOUZHIQIYEJIJINA"}, "001216": {"suoxie": "YFDXSYHHA", "name": "\u6613\u65b9\u8fbe\u65b0\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINSHOUYIHUNHEA"}, "001217": {"suoxie": "YFDXSYHHC", "name": "\u6613\u65b9\u8fbe\u65b0\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINSHOUYIHUNHEC"}, "001218": {"suoxie": "GTRYJXSYHH", "name": "\u56fd\u6295\u745e\u94f6\u7cbe\u9009\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINJINGXUANSHOUYIHUNHE"}, "001219": {"suoxie": "STMGDTDYZHH", "name": "\u4e0a\u6295\u6469\u6839\u52a8\u6001\u591a\u56e0\u5b50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENDONGTAIDUOYINZIHUNHE"}, "001220": {"suoxie": "MSJYYJJXHH", "name": "\u6c11\u751f\u52a0\u94f6\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINYANJIUJINGXUANHUNHE"}, "001222": {"suoxie": "PHWYCZHH", "name": "\u9e4f\u534e\u5916\u5ef6\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAWAIYANCHENGZHANGHUNHE"}, "001223": {"suoxie": "PHWHCMYLGP", "name": "\u9e4f\u534e\u6587\u5316\u4f20\u5a92\u5a31\u4e50\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAWENHUACHUANMEIYULEGUPIAO"}, "001224": {"suoxie": "ZYXSLLHPZHH", "name": "\u4e2d\u90ae\u65b0\u601d\u8def\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUXINSILULINGHUOPEIZHIHUNHE"}, "001225": {"suoxie": "ZYQSJXLHPZHH", "name": "\u4e2d\u90ae\u8d8b\u52bf\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUQUSHIJINGXUANLINGHUOPEIZHIHUNHE"}, "001226": {"suoxie": "ZYWJTLLHPZHH", "name": "\u4e2d\u90ae\u7a33\u5065\u6dfb\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUWENJIANTIANLILINGHUOPEIZHIHUNHE"}, "001227": {"suoxie": "ZYXXCYLHPZHH", "name": "\u4e2d\u90ae\u4fe1\u606f\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUXINXICHANYELINGHUOPEIZHIHUNHE"}, "001228": {"suoxie": "GLAXXLHPZHHA", "name": "\u56fd\u8054\u5b89\u946b\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANXINXIANGLINGHUOPEIZHIHUNHEA"}, "001229": {"suoxie": "DBFXLHPZHHA", "name": "\u5fb7\u90a6\u798f\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGFUXINLINGHUOPEIZHIHUNHEA"}, "001230": {"suoxie": "PHYYKJ", "name": "\u9e4f\u534e\u533b\u836f\u79d1\u6280", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAYIYAOKEJI"}, "001231": {"suoxie": "YHTLLHPZHHA", "name": "\u94f6\u534e\u6cf0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUATAILILINGHUOPEIZHIHUNHEA"}, "001232": {"suoxie": "JHHBA", "name": "\u5609\u5408\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAHEHUOBIA"}, "001233": {"suoxie": "JHHBB", "name": "\u5609\u5408\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAHEHUOBIB"}, "001234": {"suoxie": "GJZYHB", "name": "\u56fd\u91d1\u4f17\u8d62\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOJINZHONGYINGHUOBI"}, "001235": {"suoxie": "ZYGYQYZA", "name": "\u4e2d\u94f6\u56fd\u6709\u4f01\u4e1a\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINGUOYOUQIYEZHAIA"}, "001236": {"suoxie": "BSSLZTGPA", "name": "\u535a\u65f6\u4e1d\u8def\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHISILUZHUTIGUPIAOA"}, "001237": {"suoxie": "BSSZ50ETFLJA", "name": "\u535a\u65f6\u4e0a\u8bc150ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENG50ETFLIANJIEA"}, "001239": {"suoxie": "CSGQGGHH", "name": "\u957f\u76db\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGGUOQIGAIGEHUNHE"}, "001240": {"suoxie": "MSJYXJZLHBD", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u589e\u5229\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINZENGLIHUOBID"}, "001241": {"suoxie": "GSABZZ500ETFLJ", "name": "\u56fd\u5bff\u5b89\u4fdd\u4e2d\u8bc1500ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOZHONGZHENG500ETFLIANJIE"}, "001242": {"suoxie": "BSZZTJDSJ100A", "name": "\u535a\u65f6\u4e2d\u8bc1\u6dd8\u91d1\u5927\u6570\u636e100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGTAOJINDASHUJU100A"}, "001243": {"suoxie": "BSZZTJDSJ100I", "name": "\u535a\u65f6\u4e2d\u8bc1\u6dd8\u91d1\u5927\u6570\u636e100I", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGTAOJINDASHUJU100I"}, "001244": {"suoxie": "HTBRLHZHHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u667a\u6167\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAZHIHUIHUNHEA"}, "001245": {"suoxie": "GYSTHJGPA", "name": "\u5de5\u94f6\u751f\u6001\u73af\u5883\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINSHENGTAIHUANJINGGUPIAOA"}, "001246": {"suoxie": "XYCLDKZ", "name": "\u5174\u94f6\u957f\u4e50\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINCHANGLEDINGKAIZHAI"}, "001247": {"suoxie": "HTBRXLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXINLIHUNHEA"}, "001249": {"suoxie": "YFDXLLHPZHH", "name": "\u6613\u65b9\u8fbe\u65b0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINLILINGHUOPEIZHIHUNHE"}, "001250": {"suoxie": "THXHLHH", "name": "\u5929\u5f18\u65b0\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGXINHUOLIHUNHE"}, "001251": {"suoxie": "THXJGJHBD", "name": "\u5929\u5f18\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGXIANJINGUANJIAHUOBID"}, "001252": {"suoxie": "ZHJQSYHH", "name": "\u4e2d\u6d77\u8fdb\u53d6\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIJINQUSHOUYIHUNHE"}, "001254": {"suoxie": "TDHLXQDHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u8d77\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIXINQIDIANHUNHEA"}, "001255": {"suoxie": "CCGGHLHH", "name": "\u957f\u57ce\u6539\u9769\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGGAIGEHONGLIHUNHE"}, "001256": {"suoxie": "HDYXCZHH", "name": "\u6cd3\u5fb7\u4f18\u9009\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEYOUXUANCHENGZHANGHUNHE"}, "001257": {"suoxie": "XYSYZQZQA", "name": "\u5174\u4e1a\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYESHOUYIZENGQIANGZHAIQUANA"}, "001258": {"suoxie": "XYSYZQZQC", "name": "\u5174\u4e1a\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYESHOUYIZENGQIANGZHAIQUANC"}, "001261": {"suoxie": "ZRXJYHH", "name": "\u4e2d\u878d\u65b0\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINJIYUHUNHE"}, "001265": {"suoxie": "GTXYLHPZHHA", "name": "\u56fd\u6cf0\u5174\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIXINGYILINGHUOPEIZHIHUNHEA"}, "001266": {"suoxie": "GTRYZCHH", "name": "\u56fd\u6295\u745e\u94f6\u62db\u8d22\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINZHAOCAIHUNHE"}, "001267": {"suoxie": "TDHLLCHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLILANCHOUHUNHE"}, "001268": {"suoxie": "FGGJAQZTHHA", "name": "\u5bcc\u56fd\u56fd\u5bb6\u5b89\u5168\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGUOJIAANQUANZHUTIHUNHEA"}, "001270": {"suoxie": "YDLHPZHHXFQSA", "name": "\u82f1\u5927\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u578b\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDALINGHUOPEIZHIHUNHEXINGFAQISHIA"}, "001271": {"suoxie": "YDLHPZHHXFQSB", "name": "\u82f1\u5927\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u578b\u53d1\u8d77\u5f0fB", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDALINGHUOPEIZHIHUNHEXINGFAQISHIB"}, "001272": {"suoxie": "XYJLLHPZHH", "name": "\u5174\u4e1a\u805a\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJULILINGHUOPEIZHIHUNHE"}, "001273": {"suoxie": "MSJYXDLHHA", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINDONGLIHUNHEA"}, "001274": {"suoxie": "MSJYXDLHHD", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u52a8\u529b\u6df7\u5408D", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINDONGLIHUNHED"}, "001275": {"suoxie": "ZYCXYSLHPZHH", "name": "\u4e2d\u90ae\u521b\u65b0\u4f18\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUCHUANGXINYOUSHILINGHUOPEIZHIHUNHE"}, "001276": {"suoxie": "JXXJJLHPZHH", "name": "\u5efa\u4fe1\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINJINGJILINGHUOPEIZHIHUNHE"}, "001277": {"suoxie": "BSGQGGGPA", "name": "\u535a\u65f6\u56fd\u4f01\u6539\u9769\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIGUOQIGAIGEGUPIAOA"}, "001278": {"suoxie": "QHKYQJNYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6e05\u6d01\u80fd\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANQINGJIENENGYUANHUNHEA"}, "001279": {"suoxie": "ZHJJZLHH", "name": "\u4e2d\u6d77\u79ef\u6781\u589e\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIJIJIZENGLIHUNHE"}, "001280": {"suoxie": "YHJLLHPZHHA", "name": "\u94f6\u534e\u805a\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAJULILINGHUOPEIZHIHUNHEA"}, "001281": {"suoxie": "CAXLYXHHA", "name": "\u957f\u5b89\u946b\u5229\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINLIYOUXUANHUNHEA"}, "001282": {"suoxie": "HAXJYLHPZ", "name": "\u534e\u5b89\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINJIYULINGHUOPEIZHI"}, "001283": {"suoxie": "HTHTSJXDLHHA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u91d1\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGJINXINDONGLIHUNHEA"}, "001284": {"suoxie": "HTHTSJXDLHHC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u91d1\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGJINXINDONGLIHUNHEC"}, "001285": {"suoxie": "YFDXXHHI", "name": "\u6613\u65b9\u8fbe\u65b0\u946b\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINXINHUNHEI"}, "001286": {"suoxie": "YFDXXHHE", "name": "\u6613\u65b9\u8fbe\u65b0\u946b\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINXINHUNHEE"}, "001287": {"suoxie": "AXYSZZHHA", "name": "\u5b89\u4fe1\u4f18\u52bf\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINYOUSHIZENGZHANGHUNHEA"}, "001289": {"suoxie": "YHHLLHPZHHA", "name": "\u94f6\u534e\u6c47\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHUILILINGHUOPEIZHIHUNHEA"}, "001291": {"suoxie": "DMLHDCLGP", "name": "\u5927\u6469\u91cf\u5316\u591a\u7b56\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "DAMOLIANGHUADUOCELUEGUPIAO"}, "001294": {"suoxie": "XHZLXXCYLHPZHH", "name": "\u65b0\u534e\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAZHANLUEXINXINGCHANYELINGHUOPEIZHIHUNHE"}, "001296": {"suoxie": "CCYXZLZQA", "name": "\u957f\u57ce\u60a6\u4eab\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGYUEXIANGZENGLIZHAIQUANA"}, "001297": {"suoxie": "PAZHZGHH", "name": "\u5e73\u5b89\u667a\u6167\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANZHIHUIZHONGGUOHUNHE"}, "001298": {"suoxie": "JYMZXXHH", "name": "\u91d1\u9e70\u6c11\u65cf\u65b0\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGMINZUXINXINGHUNHE"}, "001299": {"suoxie": "XYTLZQ", "name": "\u5174\u4e1a\u6dfb\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYETIANLIZHAIQUAN"}, "001300": {"suoxie": "DCRJLHPZHHA", "name": "\u5927\u6210\u777f\u666f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGRUIJINGLINGHUOPEIZHIHUNHEA"}, "001301": {"suoxie": "DCRJLHPZHHC", "name": "\u5927\u6210\u777f\u666f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGRUIJINGLINGHUOPEIZHIHUNHEC"}, "001302": {"suoxie": "QHKYJYZBHHA", "name": "\u524d\u6d77\u5f00\u6e90\u91d1\u94f6\u73e0\u5b9d\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJINYINZHUBAOHUNHEA"}, "001303": {"suoxie": "YHWLLHPZHHA", "name": "\u94f6\u534e\u7a33\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAWENLILINGHUOPEIZHIHUNHEA"}, "001304": {"suoxie": "JXXAHBLHPZHH", "name": "\u5efa\u4fe1\u946b\u5b89\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINANHUIBAOLINGHUOPEIZHIHUNHE"}, "001305": {"suoxie": "JTTFGGHHA", "name": "\u4e5d\u6cf0\u5929\u5bcc\u6539\u9769\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAITIANFUGAIGEHUNHEA"}, "001306": {"suoxie": "ZOYYHHA", "name": "\u4e2d\u6b27\u6c38\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYONGYUHUNHEA"}, "001307": {"suoxie": "ZOYYHHC", "name": "\u4e2d\u6b27\u6c38\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYONGYUHUNHEC"}, "001308": {"suoxie": "BSWFHB", "name": "\u535a\u65f6\u5916\u670d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIWAIFUHUOBI"}, "001309": {"suoxie": "DFHRYDQKFHH", "name": "\u4e1c\u65b9\u7ea2\u777f\u9038\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGRUIYIDINGQIKAIFANGHUNHE"}, "001311": {"suoxie": "HAXHBLHPZHH", "name": "\u534e\u5b89\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINHUIBAOLINGHUOPEIZHIHUNHE"}, "001312": {"suoxie": "HAXYXLHPZHHA", "name": "\u534e\u5b89\u65b0\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINYOUXUANLINGHUOPEIZHIHUNHEA"}, "001313": {"suoxie": "STMGZHHLGP", "name": "\u4e0a\u6295\u6469\u6839\u667a\u6167\u4e92\u8054\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENZHIHUIHULIANGUPIAO"}, "001314": {"suoxie": "YFDXYHHI", "name": "\u6613\u65b9\u8fbe\u65b0\u76ca\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINYIHUNHEI"}, "001315": {"suoxie": "YFDXYHHE", "name": "\u6613\u65b9\u8fbe\u65b0\u76ca\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINYIHUNHEE"}, "001316": {"suoxie": "AXWJZZHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u589e\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINWENJIANZENGZHIHUNHEA"}, "001318": {"suoxie": "DFXCLLHPZHHA", "name": "\u4e1c\u65b9\u65b0\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGXINCELUELINGHUOPEIZHIHUNHEA"}, "001319": {"suoxie": "NYXXCMGP", "name": "\u519c\u94f6\u4fe1\u606f\u4f20\u5a92\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NONGYINXINXICHUANMEIGUPIAO"}, "001320": {"suoxie": "GYFYHBLHPZHHA", "name": "\u5de5\u94f6\u4e30\u76c8\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINFENGYINGHUIBAOLINGHUOPEIZHIHUNHEA"}, "001322": {"suoxie": "DWXQSJZXHH", "name": "\u4e1c\u5434\u65b0\u8d8b\u52bf\u4ef7\u503c\u7ebf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUXINQUSHIJIAZHIXIANHUNHE"}, "001323": {"suoxie": "DWYDHLHHA", "name": "\u4e1c\u5434\u79fb\u52a8\u4e92\u8054\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUYIDONGHULIANHUNHEA"}, "001324": {"suoxie": "HBXJZHH", "name": "\u534e\u5b9d\u65b0\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINJIAZHIHUNHE"}, "001325": {"suoxie": "PHHHHHA", "name": "\u9e4f\u534e\u5f18\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHEHUNHEA"}, "001326": {"suoxie": "PHHHHHC", "name": "\u9e4f\u534e\u5f18\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHEHUNHEC"}, "001327": {"suoxie": "PHHHHHA", "name": "\u9e4f\u534e\u5f18\u534e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHUAHUNHEA"}, "001328": {"suoxie": "PHHHHHC", "name": "\u9e4f\u534e\u5f18\u534e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHUAHUNHEC"}, "001329": {"suoxie": "PHHSHHA", "name": "\u9e4f\u534e\u5f18\u5b9e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHIHUNHEA"}, "001330": {"suoxie": "PHHSHHC", "name": "\u9e4f\u534e\u5f18\u5b9e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHIHUNHEC"}, "001331": {"suoxie": "PHHXHHA", "name": "\u9e4f\u534e\u5f18\u4fe1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGXINHUNHEA"}, "001332": {"suoxie": "PHHXHHC", "name": "\u9e4f\u534e\u5f18\u4fe1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGXINHUNHEC"}, "001334": {"suoxie": "NFLXA", "name": "\u5357\u65b9\u5229\u946bA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIXINA"}, "001335": {"suoxie": "NFLZA", "name": "\u5357\u65b9\u5229\u4f17A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIZHONGA"}, "001336": {"suoxie": "PHHYHHA", "name": "\u9e4f\u534e\u5f18\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGYIHUNHEA"}, "001337": {"suoxie": "PHHYHHC", "name": "\u9e4f\u534e\u5f18\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGYIHUNHEC"}, "001338": {"suoxie": "AXWJZZHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u589e\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINWENJIANZENGZHIHUNHEC"}, "001339": {"suoxie": "XYDXLHPZHH", "name": "\u5174\u94f6\u9f0e\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYINDINGXINLINGHUOPEIZHIHUNHE"}, "001342": {"suoxie": "YFDXXHHA", "name": "\u6613\u65b9\u8fbe\u65b0\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINXIANGHUNHEA"}, "001343": {"suoxie": "YFDXXHHC", "name": "\u6613\u65b9\u8fbe\u65b0\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINXIANGHUNHEC"}, "001344": {"suoxie": "YFDHS300YYLJA", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u533b\u836f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300YIYAOLIANJIEA"}, "001345": {"suoxie": "FGXSYLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINSHOUYILINGHUOPEIZHIHUNHEA"}, "001347": {"suoxie": "FGXSYLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINSHOUYILINGHUOPEIZHIHUNHEC"}, "001349": {"suoxie": "FGGGDLHH", "name": "\u5bcc\u56fd\u6539\u9769\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGAIGEDONGLIHUNHE"}, "001351": {"suoxie": "NAZZ500ZSZQA", "name": "\u8bfa\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANZHONGZHENG500ZHISHUZENGQIANGA"}, "001352": {"suoxie": "MSJYXZLHHA", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u6218\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINZHANLUEHUNHEA"}, "001355": {"suoxie": "GFJTHHA", "name": "\u5e7f\u53d1\u805a\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUTAIHUNHEA"}, "001356": {"suoxie": "GFJTHHC", "name": "\u5e7f\u53d1\u805a\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUTAIHUNHEC"}, "001357": {"suoxie": "HDHFHHA", "name": "\u6cd3\u5fb7\u6cd3\u5bcc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGFUHUNHEA"}, "001358": {"suoxie": "BYXTHHA", "name": "\u5b9d\u76c8\u7965\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGTAIHUNHEA"}, "001359": {"suoxie": "GLATXLHPZHHA", "name": "\u56fd\u8054\u5b89\u6dfb\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANTIANXINLINGHUOPEIZHIHUNHEA"}, "001361": {"suoxie": "JSZZKJCMTX150ETFLJ", "name": "\u666f\u987a\u4e2d\u8bc1\u79d1\u6280\u4f20\u5a92\u901a\u4fe1150ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENGKEJICHUANMEITONGXIN150ETFLIANJIE"}, "001362": {"suoxie": "JSCCLXHBHHA", "name": "\u666f\u987a\u957f\u57ce\u9886\u5148\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGLINGXIANHUIBAOHUNHEA"}, "001363": {"suoxie": "CCJHLHPZHH", "name": "\u957f\u57ce\u4e45\u60e0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUHUILINGHUOPEIZHIHUNHE"}, "001364": {"suoxie": "DCJRLHPZHH", "name": "\u5927\u6210\u666f\u6da6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJINGRUNLINGHUOPEIZHIHUNHE"}, "001365": {"suoxie": "DCZXHBLHPZHH", "name": "\u5927\u6210\u6b63\u5411\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGZHENGXIANGHUIBAOLINGHUOPEIZHIHUNHE"}, "001366": {"suoxie": "JYCYZHHH", "name": "\u91d1\u9e70\u4ea7\u4e1a\u6574\u5408\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGCHANYEZHENGHEHUNHE"}, "001367": {"suoxie": "DBXTLZQA", "name": "\u5fb7\u90a6\u65b0\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGXINTIANLIZHAIQUANA"}, "001368": {"suoxie": "XYWGSYYNLCZQ", "name": "\u5174\u4e1a\u7a33\u56fa\u6536\u76ca\u4e00\u5e74\u7406\u8d22\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEWENGUSHOUYIYINIANLICAIZHAIQUAN"}, "001369": {"suoxie": "XYWGSYLNLCZQ", "name": "\u5174\u4e1a\u7a33\u56fa\u6536\u76ca\u4e24\u5e74\u7406\u8d22\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEWENGUSHOUYILIANGNIANLICAIZHAIQUAN"}, "001370": {"suoxie": "ZYXQSHH", "name": "\u4e2d\u94f6\u65b0\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINQUSHIHUNHE"}, "001371": {"suoxie": "FGHGSJZHHA", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHUGANGSHENJIAZHIHUNHEA"}, "001373": {"suoxie": "YFDXSLLHPZHH", "name": "\u6613\u65b9\u8fbe\u65b0\u4e1d\u8def\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXINSILULINGHUOPEIZHIHUNHE"}, "001374": {"suoxie": "HXXJZLHBB", "name": "\u534e\u590f\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAXIANJINZENGLIHUOBIB"}, "001375": {"suoxie": "JYSAYZJXHHC", "name": "\u91d1\u5143\u987a\u5b89\u4f18\u8d28\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYUANSHUNANYOUZHIJINGXUANHUNHEC"}, "001376": {"suoxie": "HDHFHHC", "name": "\u6cd3\u5fb7\u6cd3\u5bcc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGFUHUNHEC"}, "001379": {"suoxie": "JSCCLXHBHHC", "name": "\u666f\u987a\u957f\u57ce\u9886\u5148\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGLINGXIANHUIBAOHUNHEC"}, "001380": {"suoxie": "PHHSHHC", "name": "\u9e4f\u534e\u5f18\u76db\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHENGHUNHEC"}, "001381": {"suoxie": "PHHZHHC", "name": "\u9e4f\u534e\u5f18\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGZEHUNHEC"}, "001382": {"suoxie": "YFDGQGGHH", "name": "\u6613\u65b9\u8fbe\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGUOQIGAIGEHUNHE"}, "001384": {"suoxie": "DFXSLHHA", "name": "\u4e1c\u65b9\u65b0\u601d\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGXINSILUHUNHEA"}, "001385": {"suoxie": "DFXSLHHC", "name": "\u4e1c\u65b9\u65b0\u601d\u8def\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGXINSILUHUNHEC"}, "001386": {"suoxie": "THHYBHBA", "name": "\u5929\u5f18\u5f18\u8fd0\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGHONGYUNBAOHUOBIA"}, "001387": {"suoxie": "ZRXJJHHA", "name": "\u4e2d\u878d\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINJINGJIHUNHEA"}, "001388": {"suoxie": "ZRXJJHHC", "name": "\u4e2d\u878d\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINJINGJIHUNHEC"}, "001391": {"suoxie": "THHYBHBB", "name": "\u5929\u5f18\u5f18\u8fd0\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGHONGYUNBAOHUOBIB"}, "001392": {"suoxie": "GFJRDCHHA", "name": "\u56fd\u5bcc\u91d1\u878d\u5730\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUJINRONGDICHANHUNHEA"}, "001393": {"suoxie": "GFJRDCHHC", "name": "\u56fd\u5bcc\u91d1\u878d\u5730\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUJINRONGDICHANHUNHEC"}, "001396": {"suoxie": "JXHLWCYSJGP", "name": "\u5efa\u4fe1\u4e92\u8054\u7f51+\u4ea7\u4e1a\u5347\u7ea7\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINHULIANWANGCHANYESHENGJIGUPIAO"}, "001397": {"suoxie": "JXJGZZZSZQ", "name": "\u5efa\u4fe1\u7cbe\u5de5\u5236\u9020\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINJINGGONGZHIZAOZHISHUZENGQIANG"}, "001398": {"suoxie": "HTBRJKSHHH", "name": "\u534e\u6cf0\u67cf\u745e\u5065\u5eb7\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIJIANKANGSHENGHUOHUNHE"}, "001399": {"suoxie": "AXXADLHHA", "name": "\u5b89\u4fe1\u946b\u5b89\u5f97\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINANDELIHUNHEA"}, "001400": {"suoxie": "AXXADLHHC", "name": "\u5b89\u4fe1\u946b\u5b89\u5f97\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINANDELIHUNHEC"}, "001401": {"suoxie": "DBRYHB", "name": "\u5fb7\u90a6\u5982\u610f\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "DEBANGRUYIHUOBI"}, "001402": {"suoxie": "XCXXHHA", "name": "\u4fe1\u8bda\u65b0\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINXUANHUNHEA"}, "001403": {"suoxie": "ZSGQGGZTHHJJ", "name": "\u62db\u5546\u56fd\u4f01\u6539\u9769\u4e3b\u9898\u6df7\u5408\u57fa\u91d1", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGGUOQIGAIGEZHUTIHUNHEJIJIN"}, "001404": {"suoxie": "ZSYDHLWCYGPJJ", "name": "\u62db\u5546\u79fb\u52a8\u4e92\u8054\u7f51\u4ea7\u4e1a\u80a1\u7968\u57fa\u91d1", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGYIDONGHULIANWANGCHANYEGUPIAOJIJIN"}, "001405": {"suoxie": "DFHCLJXHHA", "name": "\u4e1c\u65b9\u7ea2\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGCELUEJINGXUANHUNHEA"}, "001406": {"suoxie": "DFHCLJXHHC", "name": "\u4e1c\u65b9\u7ea2\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGCELUEJINGXUANHUNHEC"}, "001407": {"suoxie": "JSCCWJHBHHC", "name": "\u666f\u987a\u957f\u57ce\u7a33\u5065\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGWENJIANHUIBAOHUNHEC"}, "001409": {"suoxie": "GYHLWJGP", "name": "\u5de5\u94f6\u4e92\u8054\u7f51\u52a0\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINHULIANWANGJIAGUPIAO"}, "001410": {"suoxie": "XDAYXNYCYGP", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u80fd\u6e90\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINDAAOYINXINNENGYUANCHANYEGUPIAO"}, "001411": {"suoxie": "NACXQDHHA", "name": "\u8bfa\u5b89\u521b\u65b0\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANCHUANGXINQUDONGHUNHEA"}, "001412": {"suoxie": "DBXXJZLHPZHHA", "name": "\u5fb7\u90a6\u946b\u661f\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGXINXINGJIAZHILINGHUOPEIZHIHUNHEA"}, "001413": {"suoxie": "ZRXQDHHA", "name": "\u4e2d\u878d\u946b\u8d77\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINQIDIANHUNHEA"}, "001414": {"suoxie": "ZRXQDHHC", "name": "\u4e2d\u878d\u946b\u8d77\u70b9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINQIDIANHUNHEC"}, "001415": {"suoxie": "XCXRHHA", "name": "\u4fe1\u8bda\u65b0\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINRUIHUNHEA"}, "001416": {"suoxie": "JSSJQDGP", "name": "\u5609\u5b9e\u4e8b\u4ef6\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHISHIJIANQUDONGGUPIAO"}, "001417": {"suoxie": "HTFYLFWHHA", "name": "\u6c47\u6dfb\u5bcc\u533b\u7597\u670d\u52a1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYILIAOFUWUHUNHEA"}, "001418": {"suoxie": "TDHLCYHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u521b\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLICHUANGYIHUNHEA"}, "001419": {"suoxie": "TDHLXSLHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u601d\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIXINSILUHUNHEA"}, "001420": {"suoxie": "NFDSJ300A", "name": "\u5357\u65b9\u5927\u6570\u636e300A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDASHUJU300A"}, "001421": {"suoxie": "NFLHCZ", "name": "\u5357\u65b9\u91cf\u5316\u6210\u957f", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGLIANGHUACHENGZHANG"}, "001422": {"suoxie": "JSCCAXHBHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u4eab\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGANXIANGHUIBAOHUNHEA"}, "001423": {"suoxie": "JSCCAXHBHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u4eab\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGANXIANGHUIBAOHUNHEC"}, "001424": {"suoxie": "BSXQDHHA", "name": "\u535a\u65f6\u65b0\u8d77\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINQIDIANHUNHEA"}, "001425": {"suoxie": "BSXQDHHC", "name": "\u535a\u65f6\u65b0\u8d77\u70b9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINQIDIANHUNHEC"}, "001426": {"suoxie": "NFDSJ300C", "name": "\u5357\u65b9\u5927\u6570\u636e300C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDASHUJU300C"}, "001427": {"suoxie": "ZSFZHHA", "name": "\u62db\u5546\u4e30\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGZEHUNHEA"}, "001428": {"suoxie": "GYRXLHPZHHB", "name": "\u5de5\u94f6\u745e\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINRUIXINLINGHUOPEIZHIHUNHEB"}, "001430": {"suoxie": "ZYLXSYLHPZHH", "name": "\u4e2d\u90ae\u4e50\u4eab\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOULEXIANGSHOUYILINGHUOPEIZHIHUNHE"}, "001433": {"suoxie": "YFDRJHH", "name": "\u6613\u65b9\u8fbe\u745e\u666f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIJINGHUNHE"}, "001437": {"suoxie": "YFDRXHHI", "name": "\u6613\u65b9\u8fbe\u745e\u4eab\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXIANGHUNHEI"}, "001438": {"suoxie": "YFDRXHHE", "name": "\u6613\u65b9\u8fbe\u745e\u4eab\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXIANGHUNHEE"}, "001441": {"suoxie": "YFDRXHHI", "name": "\u6613\u65b9\u8fbe\u745e\u4fe1\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXINHUNHEI"}, "001442": {"suoxie": "YFDRXHHE", "name": "\u6613\u65b9\u8fbe\u745e\u4fe1\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXINHUNHEE"}, "001443": {"suoxie": "YFDRXLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXUANLINGHUOPEIZHIHUNHEI"}, "001444": {"suoxie": "YFDRXLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXUANLINGHUOPEIZHIHUNHEE"}, "001445": {"suoxie": "HAGQGGZTLHPZHH", "name": "\u534e\u5b89\u56fd\u4f01\u6539\u9769\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANGUOQIGAIGEZHUTILINGHUOPEIZHIHUNHE"}, "001446": {"suoxie": "ZSFZHHC", "name": "\u62db\u5546\u4e30\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGZEHUNHEC"}, "001447": {"suoxie": "THHLHH", "name": "\u5929\u5f18\u60e0\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGHUILIHUNHE"}, "001448": {"suoxie": "HSSYPHHH", "name": "\u534e\u5546\u53cc\u7ffc\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUASHANGSHUANGYIPINGHENGHUNHE"}, "001449": {"suoxie": "HSSQYXHH", "name": "\u534e\u5546\u53cc\u9a71\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGSHUANGQUYOUXUANHUNHE"}, "001453": {"suoxie": "PHHXHHA", "name": "\u9e4f\u534e\u5f18\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGXINHUNHEA"}, "001454": {"suoxie": "PHHXHHC", "name": "\u9e4f\u534e\u5f18\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGXINHUNHEC"}, "001455": {"suoxie": "JSCCZZ500ETFLJ", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u8bc1500ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGZHONGZHENG500ETFLIANJIE"}, "001457": {"suoxie": "HSXCTHH", "name": "\u534e\u5546\u65b0\u5e38\u6001\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINCHANGTAIHUNHE"}, "001463": {"suoxie": "GDYDYLHH", "name": "\u5149\u5927\u4e00\u5e26\u4e00\u8def\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYIDAIYILUHUNHE"}, "001464": {"suoxie": "GDDXHHA", "name": "\u5149\u5927\u9f0e\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDADINGXINHUNHEA"}, "001466": {"suoxie": "HFYXLHPZHHA", "name": "\u534e\u5bcc\u6c38\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUYONGXINLINGHUOPEIZHIHUNHEA"}, "001467": {"suoxie": "HFYXLHPZHHC", "name": "\u534e\u5bcc\u6c38\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUYONGXINLINGHUOPEIZHIHUNHEC"}, "001468": {"suoxie": "GFGGHH", "name": "\u5e7f\u53d1\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAGAIGEHUNHE"}, "001469": {"suoxie": "GFZZQZJRDCLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u91d1\u878d\u5730\u4ea7\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJINRONGDICHANLIANJIEA"}, "001470": {"suoxie": "RTTXLHPZHH", "name": "\u878d\u901a\u901a\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGTONGXINLINGHUOPEIZHIHUNHE"}, "001471": {"suoxie": "RTXNYLHPZHH", "name": "\u878d\u901a\u65b0\u80fd\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINNENGYUANLINGHUOPEIZHIHUNHE"}, "001472": {"suoxie": "RTXNYLHPZHH", "name": "\u878d\u901a\u65b0\u80fd\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINNENGYUANLINGHUOPEIZHIHUNHE"}, "001473": {"suoxie": "JXDAQZLJXGP", "name": "\u5efa\u4fe1\u5927\u5b89\u5168\u6218\u7565\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINDAANQUANZHANLUEJINGXUANGUPIAO"}, "001474": {"suoxie": "XYFYLHPZ", "name": "\u5174\u94f6\u4e30\u76c8\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYINFENGYINGLINGHUOPEIZHI"}, "001475": {"suoxie": "YFDGFJGHH", "name": "\u6613\u65b9\u8fbe\u56fd\u9632\u519b\u5de5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGUOFANGJUNGONGHUNHE"}, "001476": {"suoxie": "ZYZNZZGPA", "name": "\u4e2d\u94f6\u667a\u80fd\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHINENGZHIZAOGUPIAOA"}, "001477": {"suoxie": "TKXYBHBA", "name": "\u6cf0\u5eb7\u85aa\u610f\u4fdd\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXINYIBAOHUOBIA"}, "001478": {"suoxie": "TKXYBHBB", "name": "\u6cf0\u5eb7\u85aa\u610f\u4fdd\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXINYIBAOHUOBIB"}, "001479": {"suoxie": "ZYFGLDLHPZHH", "name": "\u4e2d\u90ae\u98ce\u683c\u8f6e\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUFENGGELUNDONGLINGHUOPEIZHIHUNHE"}, "001480": {"suoxie": "CTCZYXHH", "name": "\u8d22\u901a\u6210\u957f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGCHENGZHANGYOUXUANHUNHE"}, "001481": {"suoxie": "HBBPYQSYGPMYA", "name": "\u534e\u5b9d\u6807\u666e\u6cb9\u6c14\u4e0a\u6e38\u80a1\u7968\u7f8e\u5143A", "style": "QDII", "pinyin": "HUABAOBIAOPUYOUQISHANGYOUGUPIAOMEIYUANA"}, "001482": {"suoxie": "STMGXXFWGP", "name": "\u4e0a\u6295\u6469\u6839\u65b0\u5174\u670d\u52a1\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENXINXINGFUWUGUPIAO"}, "001484": {"suoxie": "THXJZHH", "name": "\u5929\u5f18\u65b0\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGXINJIAZHIHUNHE"}, "001485": {"suoxie": "HATYHH", "name": "\u534e\u5b89\u6dfb\u9890\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANYIHUNHE"}, "001487": {"suoxie": "BYYSCYHHA", "name": "\u5b9d\u76c8\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGYOUSHICHANYEHUNHEA"}, "001488": {"suoxie": "WJRFLHPZHHA", "name": "\u4e07\u5bb6\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIFENGLINGHUOPEIZHIHUNHEA"}, "001489": {"suoxie": "WJRFLHPZHHC", "name": "\u4e07\u5bb6\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIFENGLINGHUOPEIZHIHUNHEC"}, "001490": {"suoxie": "HTFGQCXGPA", "name": "\u6c47\u6dfb\u5bcc\u56fd\u4f01\u521b\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUGUOQICHUANGXINGUPIAOA"}, "001495": {"suoxie": "DFXJZHHA", "name": "\u4e1c\u65b9\u65b0\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINJIAZHIHUNHEA"}, "001496": {"suoxie": "GYJJ30GP", "name": "\u5de5\u94f6\u805a\u712630\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINJUJIAO30GUPIAO"}, "001497": {"suoxie": "DCYTLYGYGDCYZDZE", "name": "\u5927\u6210\u6708\u6dfb\u5229\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGYUETIANLIYIGEYUEGUNDONGCHIYOUZHONGDUANZHAIE"}, "001498": {"suoxie": "JXXRHBLHPZHH", "name": "\u5efa\u4fe1\u946b\u8363\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINRONGHUIBAOLINGHUOPEIZHIHUNHE"}, "001499": {"suoxie": "GTRYXZZHHA", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINZENGZHANGHUNHEA"}, "001500": {"suoxie": "HDYJHBHH", "name": "\u6cd3\u5fb7\u8fdc\u89c1\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEYUANJIANHUIBAOHUNHE"}, "001503": {"suoxie": "NFLXC", "name": "\u5357\u65b9\u5229\u946bC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIXINC"}, "001504": {"suoxie": "NFLTC", "name": "\u5357\u65b9\u5229\u6dd8C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLITAOC"}, "001505": {"suoxie": "NFLZC", "name": "\u5357\u65b9\u5229\u4f17C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIZHONGC"}, "001506": {"suoxie": "JSCCTHHBHHA", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u548c\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIHEHUIBAOHUNHEA"}, "001507": {"suoxie": "JSCCTHHBHHC", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u548c\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIHEHUIBAOHUNHEC"}, "001508": {"suoxie": "FGXDLLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINDONGLILINGHUOPEIZHIHUNHEA"}, "001510": {"suoxie": "FGXDLLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINDONGLILINGHUOPEIZHIHUNHEC"}, "001511": {"suoxie": "XQXSYDKHH", "name": "\u5174\u5168\u65b0\u89c6\u91ce\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANXINSHIYEDINGKAIHUNHE"}, "001512": {"suoxie": "YFDZZ35NQGZZS", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a3-5\u5e74\u671f\u56fd\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI35NIANQIGUOZHAIZHISHU"}, "001513": {"suoxie": "YFDXXCYHH", "name": "\u6613\u65b9\u8fbe\u4fe1\u606f\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAXINXICHANYEHUNHE"}, "001515": {"suoxie": "PAXXXFC", "name": "\u5e73\u5b89\u65b0\u946b\u5148\u950bC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINXINXIANFENGC"}, "001516": {"suoxie": "DCAHJRZE", "name": "\u5927\u6210\u5b89\u6c47\u91d1\u878d\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGANHUIJINRONGZHAIE"}, "001518": {"suoxie": "WJRX", "name": "\u4e07\u5bb6\u745e\u5174", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIXING"}, "001520": {"suoxie": "GTRYYJJXGP", "name": "\u56fd\u6295\u745e\u94f6\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTOURUIYINYANJIUJINGXUANGUPIAO"}, "001521": {"suoxie": "GSABCZYXGP", "name": "\u56fd\u5bff\u5b89\u4fdd\u6210\u957f\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOSHOUANBAOCHENGZHANGYOUXUANGUPIAO"}, "001522": {"suoxie": "BSXCLLHPZHHA", "name": "\u535a\u65f6\u65b0\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINCELUELINGHUOPEIZHIHUNHEA"}, "001523": {"suoxie": "BSXCLLHPZHHC", "name": "\u535a\u65f6\u65b0\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINCELUELINGHUOPEIZHIHUNHEC"}, "001524": {"suoxie": "HTBRJXHBHH", "name": "\u534e\u6cf0\u67cf\u745e\u7cbe\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIJINGXUANHUIBAOHUNHE"}, "001526": {"suoxie": "XYAXBHBA", "name": "\u946b\u5143\u5b89\u946b\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINYUANANXINBAOHUOBIA"}, "001527": {"suoxie": "XYAXBHBB", "name": "\u946b\u5143\u5b89\u946b\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINYUANANXINBAOHUOBIB"}, "001528": {"suoxie": "NAXJZZGP", "name": "\u8bfa\u5b89\u5148\u8fdb\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANXIANJINZHIZAOGUPIAO"}, "001529": {"suoxie": "THYSB", "name": "\u5929\u5f18\u4e91\u5546\u5b9d", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGYUNSHANGBAO"}, "001530": {"suoxie": "WJRFLHPZHHA", "name": "\u4e07\u5bb6\u745e\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIFULINGHUOPEIZHIHUNHEA"}, "001531": {"suoxie": "ZSAYLHPZHH", "name": "\u62db\u5546\u5b89\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANYILINGHUOPEIZHIHUNHE"}, "001532": {"suoxie": "HAWTJKHHA", "name": "\u534e\u5b89\u6587\u4f53\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANWENTIJIANKANGHUNHEA"}, "001534": {"suoxie": "HBWWHLHH", "name": "\u534e\u5b9d\u4e07\u7269\u4e92\u8054\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOWANWUHULIANHUNHE"}, "001535": {"suoxie": "JSCCGGJYLHPZA", "name": "\u666f\u987a\u957f\u57ce\u6539\u9769\u673a\u9047\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGGAIGEJIYULINGHUOPEIZHIA"}, "001536": {"suoxie": "NFJX", "name": "\u5357\u65b9\u541b\u9009", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJUNXUAN"}, "001537": {"suoxie": "ZJGGHLHH", "name": "\u4e2d\u52a0\u6539\u9769\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAGAIGEHONGLIHUNHE"}, "001538": {"suoxie": "STMGKJQYLHPZHHA", "name": "\u4e0a\u6295\u6469\u6839\u79d1\u6280\u524d\u6cbf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENKEJIQIANYANLINGHUOPEIZHIHUNHEA"}, "001539": {"suoxie": "JSZZJRDCETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u91d1\u878d\u5730\u4ea7ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGJINRONGDICHANETFLIANJIEA"}, "001540": {"suoxie": "ZSHJZXQD", "name": "\u6d59\u5546\u6c47\u91d1\u8f6c\u578b\u9a71\u52a8", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGHUIJINZHUANXINGQUDONG"}, "001541": {"suoxie": "HTFMYXDLGP", "name": "\u6c47\u6dfb\u5bcc\u6c11\u8425\u65b0\u52a8\u529b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUMINYINGXINDONGLIGUPIAO"}, "001542": {"suoxie": "GTHLWGP", "name": "\u56fd\u6cf0\u4e92\u8054\u7f51+\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIHULIANWANGGUPIAO"}, "001543": {"suoxie": "BYXRHHA", "name": "\u5b9d\u76c8\u65b0\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINRUIHUNHEA"}, "001545": {"suoxie": "BSYJCZ3GYDKZ", "name": "\u535a\u65f6\u88d5\u5609\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUJIACHUNZHAI3GEYUEDINGKAIZHAI"}, "001546": {"suoxie": "BSYY3GYDKZ", "name": "\u535a\u65f6\u88d5\u76c83\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUYING3GEYUEDINGKAIZHAI"}, "001547": {"suoxie": "XYJHLHPZHHA", "name": "\u5174\u4e1a\u805a\u60e0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJUHUILINGHUOPEIZHIHUNHEA"}, "001548": {"suoxie": "THSZ50ZSA", "name": "\u5929\u5f18\u4e0a\u8bc150\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGSHANGZHENG50ZHISHUA"}, "001549": {"suoxie": "THSZ50ZSC", "name": "\u5929\u5f18\u4e0a\u8bc150\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGSHANGZHENG50ZHISHUC"}, "001550": {"suoxie": "THZZYY100A", "name": "\u5929\u5f18\u4e2d\u8bc1\u533b\u836f100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYIYAO100A"}, "001551": {"suoxie": "THZZYY100C", "name": "\u5929\u5f18\u4e2d\u8bc1\u533b\u836f100C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYIYAO100C"}, "001552": {"suoxie": "THZZZQBXA", "name": "\u5929\u5f18\u4e2d\u8bc1\u8bc1\u5238\u4fdd\u9669A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGZHENGQUANBAOXIANA"}, "001553": {"suoxie": "THZZZQBXC", "name": "\u5929\u5f18\u4e2d\u8bc1\u8bc1\u5238\u4fdd\u9669C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGZHENGQUANBAOXIANC"}, "001556": {"suoxie": "THZZ500ZSZQA", "name": "\u5929\u5f18\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG500ZHISHUZENGQIANGA"}, "001557": {"suoxie": "THZZ500ZSZQC", "name": "\u5929\u5f18\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG500ZHISHUZENGQIANGC"}, "001558": {"suoxie": "THYLJKHHA", "name": "\u5929\u5f18\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGYILIAOJIANKANGHUNHEA"}, "001559": {"suoxie": "THYLJKHHC", "name": "\u5929\u5f18\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGYILIAOJIANKANGHUNHEC"}, "001562": {"suoxie": "YFDRHLHPZHH", "name": "\u6613\u65b9\u8fbe\u745e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIHELINGHUOPEIZHIHUNHE"}, "001563": {"suoxie": "HFJKWYLHPZHH", "name": "\u534e\u5bcc\u5065\u5eb7\u6587\u5a31\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUJIANKANGWENYULINGHUOPEIZHIHUNHE"}, "001564": {"suoxie": "DFHJDDSJHH", "name": "\u4e1c\u65b9\u7ea2\u4eac\u4e1c\u5927\u6570\u636e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGJINGDONGDASHUJUHUNHE"}, "001566": {"suoxie": "NFLDA", "name": "\u5357\u65b9\u5229\u8fbeA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIDAA"}, "001567": {"suoxie": "NFLDC", "name": "\u5357\u65b9\u5229\u8fbeC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIDAC"}, "001569": {"suoxie": "TXGCQDLHPZHH", "name": "\u6cf0\u4fe1\u56fd\u7b56\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINGUOCEQUDONGLINGHUOPEIZHIHUNHE"}, "001570": {"suoxie": "NFLAA", "name": "\u5357\u65b9\u5229\u5b89A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIANA"}, "001571": {"suoxie": "JHPSA", "name": "\u5609\u5408\u78d0\u77f3A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEPANSHIA"}, "001572": {"suoxie": "JHPSC", "name": "\u5609\u5408\u78d0\u77f3C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEPANSHIC"}, "001574": {"suoxie": "ZHHGHLHH", "name": "\u4e2d\u6d77\u6df7\u6539\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIHUNGAIHONGLIHUNHE"}, "001576": {"suoxie": "GTZNZBGPA", "name": "\u56fd\u6cf0\u667a\u80fd\u88c5\u5907\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIZHINENGZHUANGBEIGUPIAOA"}, "001577": {"suoxie": "JSDJCLGP", "name": "\u5609\u5b9e\u4f4e\u4ef7\u7b56\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIDIJIACELUEGUPIAO"}, "001578": {"suoxie": "BSYRCZZQ", "name": "\u535a\u65f6\u88d5\u745e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYURUICHUNZHAIZHAIQUAN"}, "001579": {"suoxie": "GTDNYGP", "name": "\u56fd\u6cf0\u5927\u519c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIDANONGYEGUPIAO"}, "001580": {"suoxie": "NFLAC", "name": "\u5357\u65b9\u5229\u5b89C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGLIANC"}, "001581": {"suoxie": "HAHGSTJXLHPZHH", "name": "\u534e\u5b89\u6caa\u6e2f\u6df1\u901a\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANHUGANGSHENTONGJINGXUANLINGHUOPEIZHIHUNHE"}, "001583": {"suoxie": "AXXCTGPA", "name": "\u5b89\u4fe1\u65b0\u5e38\u6001\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINXINCHANGTAIGUPIAOA"}, "001584": {"suoxie": "GTRYXHLDKHHA", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u6d3b\u529b\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINXINHUOLIDINGKAIHUNHEA"}, "001585": {"suoxie": "GTRYXHLDKHHC", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u6d3b\u529b\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINXINHUOLIDINGKAIHUNHEC"}, "001588": {"suoxie": "THZZ800ZSA", "name": "\u5929\u5f18\u4e2d\u8bc1800\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG800ZHISHUA"}, "001589": {"suoxie": "THZZ800ZSC", "name": "\u5929\u5f18\u4e2d\u8bc1800\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG800ZHISHUC"}, "001592": {"suoxie": "THCYBETFLJJJA", "name": "\u5929\u5f18\u521b\u4e1a\u677fETF\u8054\u63a5\u57fa\u91d1A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBANETFLIANJIEJIJINA"}, "001593": {"suoxie": "THCYBETFLJJJC", "name": "\u5929\u5f18\u521b\u4e1a\u677fETF\u8054\u63a5\u57fa\u91d1C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBANETFLIANJIEJIJINC"}, "001594": {"suoxie": "THZZYHETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYINHANGETFLIANJIEA"}, "001595": {"suoxie": "THZZYHETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYINHANGETFLIANJIEC"}, "001596": {"suoxie": "XCXZA", "name": "\u4fe1\u8bda\u65b0\u6cfdA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINZEA"}, "001601": {"suoxie": "XYXXSYLHPZHHA", "name": "\u946b\u5143\u946b\u65b0\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINXINSHOUYILINGHUOPEIZHIHUNHEA"}, "001602": {"suoxie": "XYXXSYLHPZHHC", "name": "\u946b\u5143\u946b\u65b0\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINXINSHOUYILINGHUOPEIZHIHUNHEC"}, "001603": {"suoxie": "YFDAYHBHH", "name": "\u6613\u65b9\u8fbe\u5b89\u76c8\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAANYINGHUIBAOHUNHE"}, "001604": {"suoxie": "ZSHJZXSJLHPZHH", "name": "\u6d59\u5546\u6c47\u91d1\u8f6c\u578b\u5347\u7ea7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGHUIJINZHUANXINGSHENGJILINGHUOPEIZHIHUNHE"}, "001605": {"suoxie": "GFHGSCZJXGP", "name": "\u56fd\u5bcc\u6caa\u6e2f\u6df1\u6210\u957f\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOFUHUGANGSHENCHENGZHANGJINGXUANGUPIAO"}, "001606": {"suoxie": "NYGY40HH", "name": "\u519c\u94f6\u5de5\u4e1a4.0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINGONGYE40HUNHE"}, "001607": {"suoxie": "YDCLYXA", "name": "\u82f1\u5927\u7b56\u7565\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDACELUEYOUXUANA"}, "001608": {"suoxie": "YDCLYXC", "name": "\u82f1\u5927\u7b56\u7565\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDACELUEYOUXUANC"}, "001609": {"suoxie": "PAXXHHA", "name": "\u5e73\u5b89\u946b\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINXIANGHUNHEA"}, "001610": {"suoxie": "PAXXHHC", "name": "\u5e73\u5b89\u946b\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINXIANGHUNHEC"}, "001613": {"suoxie": "CCJXHH", "name": "\u957f\u57ce\u4e45\u7965\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUXIANGHUNHE"}, "001614": {"suoxie": "DFQYFZHH", "name": "\u4e1c\u65b9\u533a\u57df\u53d1\u5c55\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGQUYUFAZHANHUNHE"}, "001616": {"suoxie": "JSHBDTGP", "name": "\u5609\u5b9e\u73af\u4fdd\u4f4e\u78b3\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHUANBAODITANGUPIAO"}, "001617": {"suoxie": "THZZDZETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u7535\u5b50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGDIANZIETFLIANJIEA"}, "001618": {"suoxie": "THZZDZETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u7535\u5b50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGDIANZIETFLIANJIEC"}, "001619": {"suoxie": "XYHFDKZ", "name": "\u5174\u94f6\u6c47\u798f\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIFUDINGKAIZHAI"}, "001621": {"suoxie": "GJJTTHBC", "name": "\u56fd\u91d1\u91d1\u817e\u901a\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOJINJINTENGTONGHUOBIC"}, "001623": {"suoxie": "XYGQGGHH", "name": "\u5174\u4e1a\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEGUOQIGAIGEHUNHE"}, "001624": {"suoxie": "XYTTYHBA", "name": "\u5174\u4e1a\u6dfb\u5929\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYETIANTIANYINGHUOBIA"}, "001625": {"suoxie": "XYTTYHBB", "name": "\u5174\u4e1a\u6dfb\u5929\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYETIANTIANYINGHUOBIB"}, "001626": {"suoxie": "GTYQGGGP", "name": "\u56fd\u6cf0\u592e\u4f01\u6539\u9769\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIYANGQIGAIGEGUPIAO"}, "001628": {"suoxie": "ZSTYWHXXGP", "name": "\u62db\u5546\u4f53\u80b2\u6587\u5316\u4f11\u95f2\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGTIYUWENHUAXIUXIANGUPIAO"}, "001629": {"suoxie": "THZZJSJETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u8ba1\u7b97\u673aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGJISUANJIETFLIANJIEA"}, "001630": {"suoxie": "THZZJSJETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u8ba1\u7b97\u673aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGJISUANJIETFLIANJIEC"}, "001631": {"suoxie": "THZZSPYLETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u98df\u54c1\u996e\u6599ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGSHIPINYINLIAOETFLIANJIEA"}, "001632": {"suoxie": "THZZSPYLETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u98df\u54c1\u996e\u6599ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGSHIPINYINLIAOETFLIANJIEC"}, "001633": {"suoxie": "WJRXHHA", "name": "\u4e07\u5bb6\u745e\u7965\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIXIANGHUNHEA"}, "001634": {"suoxie": "WJRXHHC", "name": "\u4e07\u5bb6\u745e\u7965\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIXIANGHUNHEC"}, "001635": {"suoxie": "WJRYLHPZHHA", "name": "\u4e07\u5bb6\u745e\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYILINGHUOPEIZHIHUNHEA"}, "001636": {"suoxie": "WJRYLHPZHHC", "name": "\u4e07\u5bb6\u745e\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYILINGHUOPEIZHIHUNHEC"}, "001637": {"suoxie": "JSLHJXGP", "name": "\u5609\u5b9e\u91cf\u5316\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHILIANGHUAJINGXUANGUPIAO"}, "001638": {"suoxie": "QHKYYSLCGPC", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u52bf\u84dd\u7b79\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANYOUSHILANCHOUGUPIAOC"}, "001641": {"suoxie": "FGJDSYDCLHHA", "name": "\u5bcc\u56fd\u7edd\u5bf9\u6536\u76ca\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "FUGUOJUEDUISHOUYIDUOCELUEHUNHEA"}, "001643": {"suoxie": "HFJXZZXFGPA", "name": "\u6c47\u4e30\u664b\u4fe1\u667a\u9020\u5148\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINZHIZAOXIANFENGGUPIAOA"}, "001644": {"suoxie": "HFJXZZXFGPC", "name": "\u6c47\u4e30\u664b\u4fe1\u667a\u9020\u5148\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINZHIZAOXIANFENGGUPIAOC"}, "001645": {"suoxie": "GTDJKGPA", "name": "\u56fd\u6cf0\u5927\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIDAJIANKANGGUPIAOA"}, "001648": {"suoxie": "GYXJZLHPZHHA", "name": "\u5de5\u94f6\u65b0\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIAZHILINGHUOPEIZHIHUNHEA"}, "001650": {"suoxie": "GYFSHBLHPZHHA", "name": "\u5de5\u94f6\u4e30\u6536\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINFENGSHOUHUIBAOLINGHUOPEIZHIHUNHEA"}, "001651": {"suoxie": "GYXLCGPA", "name": "\u5de5\u94f6\u65b0\u84dd\u7b79\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXINLANCHOUGUPIAOA"}, "001654": {"suoxie": "GLATXLHPZHHC", "name": "\u56fd\u8054\u5b89\u6dfb\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANTIANXINLINGHUOPEIZHIHUNHEC"}, "001656": {"suoxie": "NYZGYSLHPZHH", "name": "\u519c\u94f6\u4e2d\u56fd\u4f18\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINZHONGGUOYOUSHILINGHUOPEIZHIHUNHE"}, "001657": {"suoxie": "CAXFLXHH", "name": "\u957f\u5b89\u946b\u5bcc\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINFULINGXIANHUNHE"}, "001659": {"suoxie": "FADXDLHH", "name": "\u5bcc\u5b89\u8fbe\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAXINDONGLIHUNHE"}, "001660": {"suoxie": "FADHYLDHH", "name": "\u5bcc\u5b89\u8fbe\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAHANGYELUNDONGHUNHE"}, "001661": {"suoxie": "BSXYZCZZQC", "name": "\u535a\u65f6\u4fe1\u7528\u503a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIXINYONGZHAICHUNZHAIZHAIQUANC"}, "001662": {"suoxie": "CJHXHGSJXHH", "name": "\u521b\u91d1\u5408\u4fe1\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINHUGANGSHENJINGXUANHUNHE"}, "001664": {"suoxie": "PAXAHHA", "name": "\u5e73\u5b89\u946b\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINANHUNHEA"}, "001665": {"suoxie": "PAXAHHC", "name": "\u5e73\u5b89\u946b\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINANHUNHEC"}, "001666": {"suoxie": "PHTLBHBA", "name": "\u9e4f\u534e\u6dfb\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUATIANLIBAOHUOBIA"}, "001667": {"suoxie": "NFZXZZHHA", "name": "\u5357\u65b9\u8f6c\u578b\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGZHUANXINGZENGZHANGHUNHEA"}, "001668": {"suoxie": "HTFQQHLHH", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u4e92\u8054\u6df7\u5408", "style": "QDII", "pinyin": "HUITIANFUQUANQIUHULIANHUNHE"}, "001669": {"suoxie": "NAJXBHBC", "name": "\u8bfa\u5b89\u805a\u946b\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANJUXINBAOHUOBIC"}, "001672": {"suoxie": "GSABZHSHGP", "name": "\u56fd\u5bff\u5b89\u4fdd\u667a\u6167\u751f\u6d3b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOSHOUANBAOZHIHUISHENGHUOGUPIAO"}, "001675": {"suoxie": "JXTFA", "name": "\u6c5f\u4fe1\u540c\u798fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANGXINTONGFUA"}, "001676": {"suoxie": "JXTFC", "name": "\u6c5f\u4fe1\u540c\u798fC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANGXINTONGFUC"}, "001677": {"suoxie": "ZYZLXXCYGPA", "name": "\u4e2d\u94f6\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHANLUEXINXINGCHANYEGUPIAOA"}, "001678": {"suoxie": "YDGQGGZTGP", "name": "\u82f1\u5927\u56fd\u4f01\u6539\u9769\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINGDAGUOQIGAIGEZHUTIGUPIAO"}, "001679": {"suoxie": "QHKYZGXQZCHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u56fd\u7a00\u7f3a\u8d44\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZHONGGUOXIQUEZICHANHUNHEA"}, "001681": {"suoxie": "XHJJJZLHPZHH", "name": "\u65b0\u534e\u79ef\u6781\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAJIJIJIAZHILINGHUOPEIZHIHUNHE"}, "001682": {"suoxie": "XHXHBHH", "name": "\u65b0\u534e\u946b\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINHUIBAOHUNHE"}, "001685": {"suoxie": "HTFHGSXJZGP", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6e2f\u6df1\u65b0\u4ef7\u503c\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUHUGANGSHENXINJIAZHIGUPIAO"}, "001686": {"suoxie": "AXXDLHHA", "name": "\u5b89\u4fe1\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINDONGLIHUNHEA"}, "001687": {"suoxie": "AXXDLHHC", "name": "\u5b89\u4fe1\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINDONGLIHUNHEC"}, "001688": {"suoxie": "JSXQDHHA", "name": "\u5609\u5b9e\u65b0\u8d77\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINQIDIANHUNHEA"}, "001691": {"suoxie": "NFXGCZQDII", "name": "\u5357\u65b9\u9999\u6e2f\u6210\u957f(QDII)", "style": "QDII", "pinyin": "NANFANGXIANGGANGCHENGZHANGQDII"}, "001692": {"suoxie": "NFGCDL", "name": "\u5357\u65b9\u56fd\u7b56\u52a8\u529b", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGGUOCEDONGLI"}, "001693": {"suoxie": "ZSZL1GYQLCZQC", "name": "\u62db\u5546\u62db\u52291\u4e2a\u6708\u671f\u7406\u8d22\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGZHAOLI1GEYUEQILICAIZHAIQUANC"}, "001694": {"suoxie": "HAHGSWYZZHH", "name": "\u534e\u5b89\u6caa\u6e2f\u6df1\u5916\u5ef6\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANHUGANGSHENWAIYANZENGZHANGHUNHE"}, "001695": {"suoxie": "HDHYHH", "name": "\u6cd3\u5fb7\u6cd3\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGYEHUNHE"}, "001697": {"suoxie": "DCHFBHBA", "name": "\u5927\u6210\u6052\u4e30\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHENGFENGBAOHUOBIA"}, "001698": {"suoxie": "DCHFBHBB", "name": "\u5927\u6210\u6052\u4e30\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHENGFENGBAOHUOBIB"}, "001699": {"suoxie": "DCHFBHBE", "name": "\u5927\u6210\u6052\u4e30\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHENGFENGBAOHUOBIE"}, "001701": {"suoxie": "ZRCYSJHH", "name": "\u4e2d\u878d\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGCHANYESHENGJIHUNHE"}, "001702": {"suoxie": "DFCXKJHH", "name": "\u4e1c\u65b9\u521b\u65b0\u79d1\u6280\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGCHUANGXINKEJIHUNHE"}, "001703": {"suoxie": "YHHGSZZGPA", "name": "\u94f6\u534e\u6caa\u6e2f\u6df1\u589e\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAHUGANGSHENZENGZHANGGUPIAOA"}, "001704": {"suoxie": "GTRYJBLHPZHH", "name": "\u56fd\u6295\u745e\u94f6\u8fdb\u5b9d\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINJINBAOLINGHUOPEIZHIHUNHE"}, "001705": {"suoxie": "HDZLZXGP", "name": "\u6cd3\u5fb7\u6218\u7565\u8f6c\u578b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HONGDEZHANLUEZHUANXINGGUPIAO"}, "001706": {"suoxie": "NAJJHBHHA", "name": "\u8bfa\u5b89\u79ef\u6781\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANJIJIHUIBAOHUNHEA"}, "001707": {"suoxie": "NAGDZZGPA", "name": "\u8bfa\u5b89\u9ad8\u7aef\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANGAODUANZHIZAOGUPIAOA"}, "001708": {"suoxie": "DXGGJXHH", "name": "\u4e1c\u5174\u6539\u9769\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGGAIGEJINGXUANHUNHE"}, "001709": {"suoxie": "HFWLSJLHPZHH", "name": "\u534e\u5bcc\u7269\u8054\u4e16\u754c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUWULIANSHIJIELINGHUOPEIZHIHUNHE"}, "001710": {"suoxie": "AXXQSHHA", "name": "\u5b89\u4fe1\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINQUSHIHUNHEA"}, "001711": {"suoxie": "AXXQSHHC", "name": "\u5b89\u4fe1\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINQUSHIHUNHEC"}, "001712": {"suoxie": "DFHYSJXHH", "name": "\u4e1c\u65b9\u7ea2\u4f18\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGYOUSHIJINGXUANHUNHE"}, "001714": {"suoxie": "GYWTCYGPA", "name": "\u5de5\u94f6\u6587\u4f53\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINWENTICHANYEGUPIAOA"}, "001715": {"suoxie": "GYXJDHHA", "name": "\u5de5\u94f6\u65b0\u7126\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIAODIANHUNHEA"}, "001716": {"suoxie": "GYXQSLHPZHHA", "name": "\u5de5\u94f6\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINQUSHILINGHUOPEIZHIHUNHEA"}, "001717": {"suoxie": "GYQYYLGPA", "name": "\u5de5\u94f6\u524d\u6cbf\u533b\u7597\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINQIANYANYILIAOGUPIAOA"}, "001718": {"suoxie": "GYWLCYGPA", "name": "\u5de5\u94f6\u7269\u6d41\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINWULIUCHANYEGUPIAOA"}, "001719": {"suoxie": "GYGJZLGP", "name": "\u5de5\u94f6\u56fd\u5bb6\u6218\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINGUOJIAZHANLUEGUPIAO"}, "001720": {"suoxie": "GYXZLHH", "name": "\u5de5\u94f6\u65b0\u589e\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINXINZENGLIHUNHE"}, "001721": {"suoxie": "GYXZYHH", "name": "\u5de5\u94f6\u65b0\u589e\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINXINZENGYIHUNHE"}, "001722": {"suoxie": "GYYHLHH", "name": "\u5de5\u94f6\u94f6\u548c\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINYINHELIHUNHE"}, "001723": {"suoxie": "HSXDLHH", "name": "\u534e\u5546\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGXINDONGLIHUNHE"}, "001724": {"suoxie": "SWLXDCLLHPZHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINDUOCELUELINGHUOPEIZHIHUNHEC"}, "001725": {"suoxie": "HTFGDZZGPA", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u7aef\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUGAODUANZHIZAOGUPIAOA"}, "001726": {"suoxie": "HTFXXXFGP", "name": "\u6c47\u6dfb\u5bcc\u65b0\u5174\u6d88\u8d39\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUXINXINGXIAOFEIGUPIAO"}, "001727": {"suoxie": "SWLXAXHBHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINANXINHUIBAOHUNHEC"}, "001728": {"suoxie": "YHZLXXDKHH", "name": "\u94f6\u534e\u6218\u7565\u65b0\u5174\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAZHANLUEXINXINGDINGKAIHUNHE"}, "001730": {"suoxie": "XYDJK", "name": "\u5174\u94f6\u5927\u5065\u5eb7", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYINDAJIANKANG"}, "001731": {"suoxie": "GFBFDSJJZHHA", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUJIAZHIHUNHEA"}, "001732": {"suoxie": "GFBFDSJJZHHE", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u4ef7\u503c\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUJIAZHIHUNHEE"}, "001733": {"suoxie": "TDHLLHGP", "name": "\u6cf0\u8fbe\u5b8f\u5229\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLILIANGHUAGUPIAO"}, "001734": {"suoxie": "GFBFDSJCZHHA", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUCHENGZHANGHUNHEA"}, "001735": {"suoxie": "GFBFDSJCZHHE", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u6210\u957f\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUCHENGZHANGHUNHEE"}, "001736": {"suoxie": "YXYFYJSH", "name": "\u5706\u4fe1\u6c38\u4e30\u4f18\u52a0\u751f\u6d3b", "style": "\u80a1\u7968\u578b", "pinyin": "YUANXINYONGFENGYOUJIASHENGHUO"}, "001739": {"suoxie": "ZRRAHHEH", "name": "\u4e2d\u878d\u878d\u5b89\u6df7\u5408\u4e8c\u53f7", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGRONGANHUNHEERHAO"}, "001740": {"suoxie": "GDZGZZ2025HH", "name": "\u5149\u5927\u4e2d\u56fd\u5236\u90202025\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAZHONGGUOZHIZAO2025HUNHE"}, "001741": {"suoxie": "GFBFDSJJXHHA", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUJINGXUANHUNHEA"}, "001742": {"suoxie": "GFBFDSJJXHHE", "name": "\u5e7f\u53d1\u767e\u53d1\u5927\u6570\u636e\u7cbe\u9009\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFABAIFADASHUJUJINGXUANHUNHEE"}, "001743": {"suoxie": "NAYXHBHH", "name": "\u8bfa\u5b89\u4f18\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANYOUXUANHUIBAOHUNHE"}, "001744": {"suoxie": "NAJQHBHH", "name": "\u8bfa\u5b89\u8fdb\u53d6\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANJINQUHUIBAOHUNHE"}, "001745": {"suoxie": "YFDRFLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIFULINGHUOPEIZHIHUNHEI"}, "001746": {"suoxie": "YFDRFLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIFULINGHUOPEIZHIHUNHEE"}, "001747": {"suoxie": "YFDRQLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u797a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIQILINGHUOPEIZHIHUNHEI"}, "001748": {"suoxie": "YFDRQLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u797a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIQILINGHUOPEIZHIHUNHEE"}, "001749": {"suoxie": "ZSZGJYGP", "name": "\u62db\u5546\u4e2d\u56fd\u673a\u9047\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGZHONGGUOJIYUGUPIAO"}, "001750": {"suoxie": "JSCCJRSYZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u745e\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGRUISHOUYIZHAIQUANA"}, "001751": {"suoxie": "HSXYZQZQA", "name": "\u534e\u5546\u4fe1\u7528\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGXINYONGZENGQIANGZHAIQUANA"}, "001752": {"suoxie": "HSXYZQZQC", "name": "\u534e\u5546\u4fe1\u7528\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGXINYONGZENGQIANGZHAIQUANC"}, "001753": {"suoxie": "HTCXXXCYHH", "name": "\u7ea2\u571f\u521b\u65b0\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTUCHUANGXINXINXINGCHANYEHUNHE"}, "001755": {"suoxie": "JSXSLHH", "name": "\u5609\u5b9e\u65b0\u601d\u8def\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINSILUHUNHE"}, "001756": {"suoxie": "JSCLYXHH", "name": "\u5609\u5b9e\u7b56\u7565\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHICELUEYOUXUANHUNHE"}, "001758": {"suoxie": "JSYJZQHH", "name": "\u5609\u5b9e\u7814\u7a76\u589e\u5f3a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIYANJIUZENGQIANGHUNHE"}, "001759": {"suoxie": "JSCZZQHH", "name": "\u5609\u5b9e\u6210\u957f\u589e\u5f3a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHICHENGZHANGZENGQIANGHUNHE"}, "001760": {"suoxie": "JSCXCZHH", "name": "\u5609\u5b9e\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHICHUANGXINCHENGZHANGHUNHE"}, "001761": {"suoxie": "GFAHHBHHA", "name": "\u5e7f\u53d1\u5b89\u5b8f\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANHONGHUIBAOHUNHEA"}, "001762": {"suoxie": "GFAHHBHHC", "name": "\u5e7f\u53d1\u5b89\u5b8f\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANHONGHUIBAOHUNHEC"}, "001763": {"suoxie": "GFDCLHH", "name": "\u5e7f\u53d1\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFADUOCELUEHUNHE"}, "001764": {"suoxie": "GFHGSXJYGP", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u65b0\u673a\u9047\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAHUGANGSHENXINJIYUGUPIAO"}, "001765": {"suoxie": "QHKYJXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u5609\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJIAXINHUNHEA"}, "001766": {"suoxie": "STMGYLJKGPA", "name": "\u4e0a\u6295\u6469\u6839\u533b\u7597\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENYILIAOJIANKANGGUPIAOA"}, "001770": {"suoxie": "QHKYJXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u5609\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJIAXINHUNHEC"}, "001775": {"suoxie": "PHHTC", "name": "\u9e4f\u534e\u5f18\u6cf0C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGTAIC"}, "001776": {"suoxie": "ZOXLZQA", "name": "\u4e2d\u6b27\u5174\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINGLIZHAIQUANA"}, "001779": {"suoxie": "ZRWJTLZQ", "name": "\u4e2d\u878d\u7a33\u5065\u6dfb\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGRONGWENJIANTIANLIZHAIQUAN"}, "001780": {"suoxie": "NAGGQSLHPZHH", "name": "\u8bfa\u5b89\u6539\u9769\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANGAIGEQUSHILINGHUOPEIZHIHUNHE"}, "001781": {"suoxie": "JXXDFWYGP", "name": "\u5efa\u4fe1\u73b0\u4ee3\u670d\u52a1\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINXIANDAIFUWUYEGUPIAO"}, "001782": {"suoxie": "JTJYHHA", "name": "\u4e5d\u6cf0\u4e45\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUYIHUNHEA"}, "001783": {"suoxie": "XYHYZQA", "name": "\u5174\u94f6\u5408\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHEYINGZHAIQUANA"}, "001784": {"suoxie": "XYHYZQC", "name": "\u5174\u94f6\u5408\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHEYINGZHAIQUANC"}, "001785": {"suoxie": "MSJYSSZLZQD", "name": "\u6c11\u751f\u52a0\u94f6\u5c81\u5c81\u589e\u5229\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINSUISUIZENGLIZHAIQUAND"}, "001789": {"suoxie": "GTLHSYLHPZHHA", "name": "\u56fd\u6cf0\u91cf\u5316\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAILIANGHUASHOUYILINGHUOPEIZHIHUNHEA"}, "001790": {"suoxie": "GTZNQCGPA", "name": "\u56fd\u6cf0\u667a\u80fd\u6c7d\u8f66\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIZHINENGQICHEGUPIAOA"}, "001791": {"suoxie": "DCJDSYCLHHA", "name": "\u5927\u6210\u7edd\u5bf9\u6536\u76ca\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "DACHENGJUEDUISHOUYICELUEHUNHEA"}, "001792": {"suoxie": "DCJDSYCLHHC", "name": "\u5927\u6210\u7edd\u5bf9\u6536\u76ca\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "DACHENGJUEDUISHOUYICELUEHUNHEC"}, "001794": {"suoxie": "XYCYZQ", "name": "\u5174\u94f6\u671d\u9633\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINCHAOYANGZHAIQUAN"}, "001795": {"suoxie": "STMGWTXXLHPZHH", "name": "\u4e0a\u6295\u6469\u6839\u6587\u4f53\u4f11\u95f2\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENWENTIXIUXIANLINGHUOPEIZHIHUNHE"}, "001796": {"suoxie": "HTFAXZXHHA", "name": "\u6c47\u6dfb\u5bcc\u5b89\u946b\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUANXINZHIXUANHUNHEA"}, "001797": {"suoxie": "HRXLLHPZHH", "name": "\u534e\u878d\u65b0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGXINLILINGHUOPEIZHIHUNHE"}, "001798": {"suoxie": "TKXHBLHPZHHA", "name": "\u6cf0\u5eb7\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGXINHUIBAOLINGHUOPEIZHIHUNHEA"}, "001799": {"suoxie": "TKXHBLHPZHHC", "name": "\u6cf0\u5eb7\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGXINHUIBAOLINGHUOPEIZHIHUNHEC"}, "001800": {"suoxie": "HAXLXLHPZHHA", "name": "\u534e\u5b89\u65b0\u4e50\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINLEXIANGLINGHUOPEIZHIHUNHEA"}, "001801": {"suoxie": "HTFDXHHA", "name": "\u6c47\u6dfb\u5bcc\u8fbe\u6b23\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUDAXINHUNHEA"}, "001802": {"suoxie": "YFDRCHHI", "name": "\u6613\u65b9\u8fbe\u745e\u8d22\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICAIHUNHEI"}, "001803": {"suoxie": "YFDRCHHE", "name": "\u6613\u65b9\u8fbe\u745e\u8d22\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICAIHUNHEE"}, "001806": {"suoxie": "YFDRZLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u667a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIZHILINGHUOPEIZHIHUNHEI"}, "001807": {"suoxie": "YFDRZLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u667a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIZHILINGHUOPEIZHIHUNHEE"}, "001808": {"suoxie": "YHHLWZTLHPZHH", "name": "\u94f6\u534e\u4e92\u8054\u7f51\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHULIANWANGZHUTILINGHUOPEIZHIHUNHE"}, "001809": {"suoxie": "ZXJTZXWLWA", "name": "\u4e2d\u4fe1\u5efa\u6295\u667a\u4fe1\u7269\u8054\u7f51A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOUZHIXINWULIANWANGA"}, "001810": {"suoxie": "ZOQLJZLHPZHHA", "name": "\u4e2d\u6b27\u6f5c\u529b\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUQIANLIJIAZHILINGHUOPEIZHIHUNHEA"}, "001811": {"suoxie": "ZOMRXCTHHA", "name": "\u4e2d\u6b27\u660e\u777f\u65b0\u5e38\u6001\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUMINGRUIXINCHANGTAIHUNHEA"}, "001812": {"suoxie": "JSHBE", "name": "\u5609\u5b9e\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOBIE"}, "001815": {"suoxie": "HTBRJLDLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u6fc0\u52b1\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIJILIDONGLIHUNHEA"}, "001816": {"suoxie": "HTFXRJXHHA", "name": "\u6c47\u6dfb\u5bcc\u65b0\u777f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUXINRUIJINGXUANHUNHEA"}, "001817": {"suoxie": "YFDRXLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXINGLINGHUOPEIZHIHUNHEI"}, "001818": {"suoxie": "YFDRXLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXINGLINGHUOPEIZHIHUNHEE"}, "001819": {"suoxie": "XQWYDKZFQS", "name": "\u5174\u5168\u7a33\u76ca\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANWENYIDINGKAIZHAIFAQISHI"}, "001820": {"suoxie": "XQTTYHBA", "name": "\u5174\u5168\u5929\u6dfb\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGQUANTIANTIANYIHUOBIA"}, "001821": {"suoxie": "XQTTYHBB", "name": "\u5174\u5168\u5929\u6dfb\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGQUANTIANTIANYIHUOBIB"}, "001822": {"suoxie": "HSZNSHLHPZHH", "name": "\u534e\u5546\u667a\u80fd\u751f\u6d3b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGZHINENGSHENGHUOLINGHUOPEIZHIHUNHE"}, "001823": {"suoxie": "GDDXHHC", "name": "\u5149\u5927\u9f0e\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDADINGXINHUNHEC"}, "001824": {"suoxie": "BSHGSCZQY", "name": "\u535a\u65f6\u6caa\u6e2f\u6df1\u6210\u957f\u4f01\u4e1a", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUGANGSHENCHENGZHANGQIYE"}, "001825": {"suoxie": "JXZGZZ2025GPA", "name": "\u5efa\u4fe1\u4e2d\u56fd\u5236\u90202025\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINZHONGGUOZHIZAO2025GUPIAOA"}, "001826": {"suoxie": "GSABZJBHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u589e\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOZENGJINBAOHUOBIA"}, "001827": {"suoxie": "FGYJYXHGSHH", "name": "\u5bcc\u56fd\u7814\u7a76\u4f18\u9009\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOYANJIUYOUXUANHUGANGSHENHUNHE"}, "001829": {"suoxie": "BXRFZGZZ", "name": "\u5317\u4fe1\u745e\u4e30\u4e2d\u56fd\u667a\u9020", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGZHONGGUOZHIZAO"}, "001830": {"suoxie": "RTKJCZLHPZHH", "name": "\u878d\u901a\u8de8\u754c\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGKUAJIECHENGZHANGLINGHUOPEIZHIHUNHE"}, "001831": {"suoxie": "RTKJCZLHPZHH", "name": "\u878d\u901a\u8de8\u754c\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGKUAJIECHENGZHANGLINGHUOPEIZHIHUNHE"}, "001832": {"suoxie": "YFDRHLHPZHH", "name": "\u6613\u65b9\u8fbe\u745e\u6052\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIHENGLINGHUOPEIZHIHUNHE"}, "001834": {"suoxie": "CSZLXXCYHHC", "name": "\u957f\u76db\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGZHANLUEXINXINGCHANYEHUNHEC"}, "001835": {"suoxie": "YFDRXLHPZHHI", "name": "\u6613\u65b9\u8fbe\u745e\u7965\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXIANGLINGHUOPEIZHIHUNHEI"}, "001836": {"suoxie": "YFDRXLHPZHHE", "name": "\u6613\u65b9\u8fbe\u745e\u7965\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIXIANGLINGHUOPEIZHIHUNHEE"}, "001837": {"suoxie": "QHKYHGSLCJXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u84dd\u7b79\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLANCHOUJINGXUANHUNHEA"}, "001838": {"suoxie": "GTRYGJAQHH", "name": "\u56fd\u6295\u745e\u94f6\u56fd\u5bb6\u5b89\u5168\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINGUOJIAANQUANHUNHE"}, "001839": {"suoxie": "JTJXLHPZHH", "name": "\u4e5d\u6cf0\u4e45\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUXINGLINGHUOPEIZHIHUNHE"}, "001842": {"suoxie": "JTRTJHBA", "name": "\u4e5d\u6cf0\u65e5\u6dfb\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIUTAIRITIANJINHUOBIA"}, "001843": {"suoxie": "JTRTJHBB", "name": "\u4e5d\u6cf0\u65e5\u6dfb\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIUTAIRITIANJINHUOBIB"}, "001844": {"suoxie": "JTJYHHC", "name": "\u4e5d\u6cf0\u4e45\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUYIHUNHEC"}, "001849": {"suoxie": "QHKYQSGS100QGP", "name": "\u524d\u6d77\u5f00\u6e90\u5f3a\u52bf\u5171\u8bc6100\u5f3a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANQIANGSHIGONGSHI100QIANGGUPIAO"}, "001850": {"suoxie": "GTAYLHPZHHA", "name": "\u56fd\u6cf0\u5b89\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIANYILINGHUOPEIZHIHUNHEA"}, "001852": {"suoxie": "RTZGF1HLHPZHHAB", "name": "\u878d\u901a\u4e2d\u56fd\u98ce1\u53f7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHONGGUOFENG1HAOLINGHUOPEIZHIHUNHEAB"}, "001853": {"suoxie": "RTZGF1HLHPZHHAB", "name": "\u878d\u901a\u4e2d\u56fd\u98ce1\u53f7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHONGGUOFENG1HAOLINGHUOPEIZHIHUNHEAB"}, "001856": {"suoxie": "YFDHBZTHH", "name": "\u6613\u65b9\u8fbe\u73af\u4fdd\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAHUANBAOZHUTIHUNHE"}, "001857": {"suoxie": "YFDXDFWYHH", "name": "\u6613\u65b9\u8fbe\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAXIANDAIFUWUYEHUNHE"}, "001858": {"suoxie": "JXXLLHPZHH", "name": "\u5efa\u4fe1\u946b\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINLILINGHUOPEIZHIHUNHE"}, "001859": {"suoxie": "DMZZ18GYKFZQA", "name": "\u5927\u6469\u589e\u503c18\u4e2a\u6708\u5f00\u653e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOZENGZHI18GEYUEKAIFANGZHAIQUANA"}, "001860": {"suoxie": "DMZZ18GYKFZQC", "name": "\u5927\u6469\u589e\u503c18\u4e2a\u6708\u5f00\u653e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOZENGZHI18GEYUEKAIFANGZHAIQUANC"}, "001861": {"suoxie": "FADJKRSHHA", "name": "\u5bcc\u5b89\u8fbe\u5065\u5eb7\u4eba\u751f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAJIANKANGRENSHENGHUNHEA"}, "001862": {"suoxie": "DFHSYZQZQA", "name": "\u4e1c\u65b9\u7ea2\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGSHOUYIZENGQIANGZHAIQUANA"}, "001863": {"suoxie": "DFHSYZQZQC", "name": "\u4e1c\u65b9\u7ea2\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGSHOUYIZENGQIANGZHAIQUANC"}, "001864": {"suoxie": "ZHMLCSJHH", "name": "\u4e2d\u6d77\u9b45\u529b\u957f\u4e09\u89d2\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIMEILICHANGSANJIAOHUNHE"}, "001865": {"suoxie": "QHKYSJQDHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANSHIJIANQUDONGHUNHEC"}, "001866": {"suoxie": "BXRFXCZ", "name": "\u5317\u4fe1\u745e\u4e30\u65b0\u6210\u957f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGXINCHENGZHANG"}, "001867": {"suoxie": "NAJXBHBD", "name": "\u8bfa\u5b89\u805a\u946b\u5b9d\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANJUXINBAOHUOBID"}, "001868": {"suoxie": "ZSCYZQC", "name": "\u62db\u5546\u4ea7\u4e1a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGCHANYEZHAIQUANC"}, "001869": {"suoxie": "ZSZZYHHA", "name": "\u62db\u5546\u5236\u9020\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGZHIZAOYEHUNHEA"}, "001870": {"suoxie": "QHKYHBA", "name": "\u524d\u6d77\u5f00\u6e90\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAIKAIYUANHUOBIA"}, "001871": {"suoxie": "QHKYHBB", "name": "\u524d\u6d77\u5f00\u6e90\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAIKAIYUANHUOBIB"}, "001874": {"suoxie": "QHKYHGSJZJXHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENJIAZHIJINGXUANHUNHE"}, "001875": {"suoxie": "QHKYHGSYSJXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u4f18\u52bf\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENYOUSHIJINGXUANHUNHEA"}, "001876": {"suoxie": "PHQQGSYZMYXH", "name": "\u9e4f\u534e\u5168\u7403\u9ad8\u6536\u76ca\u503a\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "PENGHUAQUANQIUGAOSHOUYIZHAIMEIYUANXIANHUI"}, "001877": {"suoxie": "BYGJAQHGSGPA", "name": "\u5b9d\u76c8\u56fd\u5bb6\u5b89\u5168\u6caa\u6e2f\u6df1\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGGUOJIAANQUANHUGANGSHENGUPIAOA"}, "001878": {"suoxie": "JSHGSJXGP", "name": "\u5609\u5b9e\u6caa\u6e2f\u6df1\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHUGANGSHENJINGXUANGUPIAO"}, "001879": {"suoxie": "CCCYBZSZQA", "name": "\u957f\u57ce\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGCHUANGYEBANZHISHUZENGQIANGA"}, "001880": {"suoxie": "CCZGZZLHPZHHA", "name": "\u957f\u57ce\u4e2d\u56fd\u667a\u9020\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGZHONGGUOZHIZAOLINGHUOPEIZHIHUNHEA"}, "001881": {"suoxie": "ZOXQSHHE", "name": "\u4e2d\u6b27\u65b0\u8d8b\u52bf\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINQUSHIHUNHEE"}, "001882": {"suoxie": "ZOJZFXHHE", "name": "\u4e2d\u6b27\u4ef7\u503c\u53d1\u73b0\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAZHIFAXIANHUNHEE"}, "001883": {"suoxie": "ZOXDLHHLOFE", "name": "\u4e2d\u6b27\u65b0\u52a8\u529b\u6df7\u5408(LOF)E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINDONGLIHUNHELOFE"}, "001884": {"suoxie": "ZOHTJXHHE", "name": "\u4e2d\u6b27\u4e92\u901a\u7cbe\u9009\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHUTONGJINGXUANHUNHEE"}, "001885": {"suoxie": "ZOXLCHHE", "name": "\u4e2d\u6b27\u65b0\u84dd\u7b79\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUXINLANCHOUHUNHEE"}, "001886": {"suoxie": "ZOHYCZHHLOFE", "name": "\u4e2d\u6b27\u884c\u4e1a\u6210\u957f\u6df7\u5408(LOF)E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHANGYECHENGZHANGHUNHELOFE"}, "001887": {"suoxie": "ZOJZZXHHE", "name": "\u4e2d\u6b27\u4ef7\u503c\u667a\u9009\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJIAZHIZHIXUANHUNHEE"}, "001888": {"suoxie": "ZOSSCZHHLOFE", "name": "\u4e2d\u6b27\u76db\u4e16\u6210\u957f\u6df7\u5408(LOF)E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUSHENGSHICHENGZHANGHUNHELOFE"}, "001889": {"suoxie": "ZOZQHBZQLOFE", "name": "\u4e2d\u6b27\u589e\u5f3a\u56de\u62a5\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUZENGQIANGHUIBAOZHAIQUANLOFE"}, "001890": {"suoxie": "ZOJXDQKFHHE", "name": "\u4e2d\u6b27\u7cbe\u9009\u5b9a\u671f\u5f00\u653e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINGXUANDINGQIKAIFANGHUNHEE"}, "001891": {"suoxie": "ZOCZYXHHE", "name": "\u4e2d\u6b27\u6210\u957f\u4f18\u9009\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUCHENGZHANGYOUXUANHUNHEE"}, "001892": {"suoxie": "CSXXCZHH", "name": "\u957f\u76db\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGXINXINGCHENGZHANGHUNHE"}, "001893": {"suoxie": "HBXJTYB", "name": "\u534e\u5b9d\u73b0\u91d1\u6dfb\u76caB", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOXIANJINTIANYIB"}, "001894": {"suoxie": "TDHLHQYHBA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d3b\u671f\u53cb\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIHUOQIYOUHUOBIA"}, "001895": {"suoxie": "TDHLHQYHBB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d3b\u671f\u53cb\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIHUOQIYOUHUOBIB"}, "001897": {"suoxie": "JTJSLHXFHHA", "name": "\u4e5d\u6cf0\u4e45\u76db\u91cf\u5316\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUSHENGLIANGHUAXIANFENGHUNHEA"}, "001898": {"suoxie": "YFDDJKHH", "name": "\u6613\u65b9\u8fbe\u5927\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDADAJIANKANGHUNHE"}, "001899": {"suoxie": "DHSHAQ", "name": "\u4e1c\u6d77\u793e\u4f1a\u5b89\u5168", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGHAISHEHUIANQUAN"}, "001900": {"suoxie": "NAJXJZHH", "name": "\u8bfa\u5b89\u7cbe\u9009\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANJINGXUANJIAZHIHUNHE"}, "001901": {"suoxie": "QHKYHGSLXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u9686\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLONGXINHUNHEA"}, "001902": {"suoxie": "QHKYHGSLXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u9686\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLONGXINHUNHEC"}, "001903": {"suoxie": "GDXXHHA", "name": "\u5149\u5927\u6b23\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAXINXINHUNHEA"}, "001904": {"suoxie": "GDXXHHC", "name": "\u5149\u5927\u6b23\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAXINXINHUNHEC"}, "001905": {"suoxie": "HAAYLHPZHHA", "name": "\u534e\u5b89\u5b89\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANYILINGHUOPEIZHIHUNHEA"}, "001906": {"suoxie": "DFH6GYDKZ", "name": "\u4e1c\u65b9\u7ea26\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONG6GEYUEDINGKAIZHAI"}, "001907": {"suoxie": "GTRYJXLHPZHHA", "name": "\u56fd\u6295\u745e\u94f6\u5883\u714a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINJINGXUANLINGHUOPEIZHIHUNHEA"}, "001908": {"suoxie": "GTRYJXLHPZHHC", "name": "\u56fd\u6295\u745e\u94f6\u5883\u714a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINJINGXUANLINGHUOPEIZHIHUNHEC"}, "001909": {"suoxie": "CJHXHBA", "name": "\u521b\u91d1\u5408\u4fe1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHUANGJINHEXINHUOBIA"}, "001910": {"suoxie": "TKXJYHH", "name": "\u6cf0\u5eb7\u65b0\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGXINJIYUHUNHE"}, "001911": {"suoxie": "BSYHCZZQ", "name": "\u535a\u65f6\u88d5\u6052\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUHENGCHUNZHAIZHAIQUAN"}, "001914": {"suoxie": "ZXJTJLHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u805a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOUJULIHUNHEA"}, "001915": {"suoxie": "BYYLJKHGSGP", "name": "\u5b9d\u76c8\u533b\u7597\u5065\u5eb7\u6caa\u6e2f\u6df1\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGYILIAOJIANKANGHUGANGSHENGUPIAO"}, "001916": {"suoxie": "XWTBA", "name": "\u65b0\u6c83\u901a\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "XINWOTONGBAOA"}, "001917": {"suoxie": "ZSLHJXGPA", "name": "\u62db\u5546\u91cf\u5316\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGLIANGHUAJINGXUANGUPIAOA"}, "001918": {"suoxie": "YXYFXLA", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u5229A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGXINGLIA"}, "001919": {"suoxie": "YXYFXLC", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGXINGLIC"}, "001922": {"suoxie": "GTDCLSYHH", "name": "\u56fd\u6cf0\u591a\u7b56\u7565\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIDUOCELUESHOUYIHUNHE"}, "001924": {"suoxie": "HXGQGGHH", "name": "\u534e\u590f\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAGUOQIGAIGEHUNHE"}, "001925": {"suoxie": "XYXTYHBA", "name": "\u5174\u4e1a\u946b\u5929\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEXINTIANYINGHUOBIA"}, "001926": {"suoxie": "XYXTYHBB", "name": "\u5174\u4e1a\u946b\u5929\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEXINTIANYINGHUOBIB"}, "001927": {"suoxie": "HXXFSJLHPZHHA", "name": "\u534e\u590f\u6d88\u8d39\u5347\u7ea7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXIAOFEISHENGJILINGHUOPEIZHIHUNHEA"}, "001928": {"suoxie": "HXXFSJLHPZHHC", "name": "\u534e\u590f\u6d88\u8d39\u5347\u7ea7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXIAOFEISHENGJILINGHUOPEIZHIHUNHEC"}, "001929": {"suoxie": "HXSYBHBA", "name": "\u534e\u590f\u6536\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIASHOUYIBAOHUOBIA"}, "001930": {"suoxie": "HXSYBHBB", "name": "\u534e\u590f\u6536\u76ca\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIASHOUYIBAOHUOBIB"}, "001931": {"suoxie": "GSABXQBHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u946b\u94b1\u5305\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOXINQIANBAOHUOBIA"}, "001932": {"suoxie": "GSABLHYXHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u7075\u6d3b\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOLINGHUOYOUXUANHUNHE"}, "001933": {"suoxie": "HSXXHLHH", "name": "\u534e\u5546\u65b0\u5174\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINXINGHUOLIHUNHE"}, "001937": {"suoxie": "XYXJZLHB", "name": "\u5174\u94f6\u73b0\u91d1\u589e\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYINXIANJINZENGLIHUOBI"}, "001938": {"suoxie": "ZOSDXFGPA", "name": "\u4e2d\u6b27\u65f6\u4ee3\u5148\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUSHIDAIXIANFENGGUPIAOA"}, "001939": {"suoxie": "GDRXHHA", "name": "\u5149\u5927\u777f\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDARUIXINHUNHEA"}, "001940": {"suoxie": "NYXDNYJ", "name": "\u519c\u94f6\u73b0\u4ee3\u519c\u4e1a\u52a0", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINXIANDAINONGYEJIA"}, "001941": {"suoxie": "RTTYDRZQB", "name": "\u878d\u901a\u901a\u6e90\u77ed\u878d\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "RONGTONGTONGYUANDUANRONGZHAIQUANB"}, "001942": {"suoxie": "QHKYHGSHXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u6c47\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENHUIXINHUNHEA"}, "001943": {"suoxie": "QHKYHGSHXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u6c47\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENHUIXINHUNHEC"}, "001945": {"suoxie": "DFHXYZZQA", "name": "\u4e1c\u65b9\u7ea2\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGXINYONGZHAIZHAIQUANA"}, "001946": {"suoxie": "DFHXYZZQC", "name": "\u4e1c\u65b9\u7ea2\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGXINYONGZHAIZHAIQUANC"}, "001950": {"suoxie": "PHFTDKZB", "name": "\u9e4f\u534e\u4e30\u6cf0\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGTAIDINGKAIZHAIB"}, "001951": {"suoxie": "JYGGHLHH", "name": "\u91d1\u9e70\u6539\u9769\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGGAIGEHONGLIHUNHE"}, "001955": {"suoxie": "ZOYLCYHHA", "name": "\u4e2d\u6b27\u517b\u8001\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYANGLAOCHANYEHUNHEA"}, "001956": {"suoxie": "GLAKJDL", "name": "\u56fd\u8054\u5b89\u79d1\u6280\u52a8\u529b", "style": "\u80a1\u7968\u578b", "pinyin": "GUOLIANANKEJIDONGLI"}, "001957": {"suoxie": "JHPTZQA", "name": "\u5609\u5408\u78d0\u901a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAHEPANTONGZHAIQUANA"}, "001958": {"suoxie": "JHPTZQC", "name": "\u5609\u5408\u78d0\u901a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAHEPANTONGZHAIQUANC"}, "001959": {"suoxie": "HSLXHLLHPZHHA", "name": "\u534e\u5546\u4e50\u4eab\u4e92\u8054\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGLEXIANGHULIANLINGHUOPEIZHIHUNHEA"}, "001960": {"suoxie": "XYRY", "name": "\u5174\u94f6\u745e\u76ca", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINRUIYI"}, "001961": {"suoxie": "BSYRCZZQ", "name": "\u535a\u65f6\u88d5\u8363\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYURONGCHUNZHAIZHAIQUAN"}, "001963": {"suoxie": "ZOTXZQ", "name": "\u4e2d\u6b27\u5929\u79a7\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUTIANXIZHAIQUAN"}, "001964": {"suoxie": "NATXYNDKZC", "name": "\u8bfa\u5b89\u6cf0\u946b\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANTAIXINYINIANDINGKAIZHAIC"}, "001965": {"suoxie": "YXYFXYLHPZHHA", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXINGYUANLINGHUOPEIZHIHUNHEA"}, "001966": {"suoxie": "YXYFXYLHPZHHC", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXINGYUANLINGHUOPEIZHIHUNHEC"}, "001967": {"suoxie": "HBZXSJHH", "name": "\u534e\u5b9d\u8f6c\u578b\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOZHUANXINGSHENGJIHUNHE"}, "001968": {"suoxie": "GDZYBNDKZA", "name": "\u5149\u5927\u5c0a\u76c8\u534a\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNYINGBANNIANDINGKAIZHAIA"}, "001969": {"suoxie": "GDZYBNDKZC", "name": "\u5149\u5927\u5c0a\u76c8\u534a\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNYINGBANNIANDINGKAIZHAIC"}, "001970": {"suoxie": "TXXXLHPZHHA", "name": "\u6cf0\u4fe1\u946b\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINXINXUANLINGHUOPEIZHIHUNHEA"}, "001972": {"suoxie": "QHKYHGSZHSHHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u667a\u6167\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENZHIHUISHENGHUOHUNHE"}, "001973": {"suoxie": "GDYQBHBA", "name": "\u5149\u5927\u8000\u94b1\u5305\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAYAOQIANBAOHUOBIA"}, "001974": {"suoxie": "JSCCLHXDLGP", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u65b0\u52a8\u529b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAXINDONGLIGUPIAO"}, "001975": {"suoxie": "JSCCHBYSGP", "name": "\u666f\u987a\u957f\u57ce\u73af\u4fdd\u4f18\u52bf\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGHUANBAOYOUSHIGUPIAO"}, "001976": {"suoxie": "HFTYNDKZC", "name": "\u6d77\u5bcc\u901a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGYINIANDINGKAIZHAIC"}, "001978": {"suoxie": "TXHLWHH", "name": "\u6cf0\u4fe1\u4e92\u8054\u7f51+\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINHULIANWANGHUNHE"}, "001980": {"suoxie": "ZOLHQDHH", "name": "\u4e2d\u6b27\u91cf\u5316\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOULIANGHUAQUDONGHUNHE"}, "001981": {"suoxie": "FGSYBJYXHBA", "name": "\u5bcc\u56fd\u6536\u76ca\u5b9d\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOSHOUYIBAOJIAOYIXINGHUOBIA"}, "001982": {"suoxie": "FGSYBJYXHBB", "name": "\u5bcc\u56fd\u6536\u76ca\u5b9d\u4ea4\u6613\u578b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOSHOUYIBAOJIAOYIXINGHUOBIB"}, "001983": {"suoxie": "ZYDTPZHH", "name": "\u4e2d\u90ae\u4f4e\u78b3\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUDITANPEIZHIHUNHE"}, "001984": {"suoxie": "STMGZGSWYYQDII", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u56fd\u751f\u7269\u533b\u836f(QDII)", "style": "QDII", "pinyin": "SHANGTOUMOGENZHONGGUOSHENGWUYIYAOQDII"}, "001985": {"suoxie": "FGDTXJJHHA", "name": "\u5bcc\u56fd\u4f4e\u78b3\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODITANXINJINGJIHUNHEA"}, "001986": {"suoxie": "QHKYRGZNZTHH", "name": "\u524d\u6d77\u5f00\u6e90\u4eba\u5de5\u667a\u80fd\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANRENGONGZHINENGZHUTIHUNHE"}, "001987": {"suoxie": "DFJYBHB", "name": "\u4e1c\u65b9\u91d1\u5143\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGJINYUANBAOHUOBI"}, "001988": {"suoxie": "NFCYA", "name": "\u5357\u65b9\u7eaf\u5143A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHUNYUANA"}, "001989": {"suoxie": "NFCYC", "name": "\u5357\u65b9\u7eaf\u5143C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHUNYUANC"}, "001990": {"suoxie": "ZOSJWJHHA", "name": "\u4e2d\u6b27\u6570\u636e\u6316\u6398\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUSHUJUWAJUEHUNHEA"}, "001991": {"suoxie": "NYTTLHBA", "name": "\u519c\u94f6\u5929\u5929\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINTIANTIANLIHUOBIA"}, "001992": {"suoxie": "NYTTLHBB", "name": "\u519c\u94f6\u5929\u5929\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINTIANTIANLIHUOBIB"}, "001993": {"suoxie": "BSYTCZZQ", "name": "\u535a\u65f6\u88d5\u6cf0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUTAICHUNZHAIZHAIQUAN"}, "001994": {"suoxie": "HANNHZQC", "name": "\u534e\u5b89\u5e74\u5e74\u7ea2\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANNIANNIANHONGZHAIQUANC"}, "001997": {"suoxie": "GYXQSLHPZHHC", "name": "\u5de5\u94f6\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINQUSHILINGHUOPEIZHIHUNHEC"}, "001998": {"suoxie": "GYXJDHHC", "name": "\u5de5\u94f6\u65b0\u7126\u70b9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIAODIANHUNHEC"}, "002000": {"suoxie": "GYXSLHH", "name": "\u5de5\u94f6\u65b0\u751f\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINXINSHENGLIHUNHE"}, "002001": {"suoxie": "HXHBHHA", "name": "\u534e\u590f\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAHUIBAOHUNHEA"}, "002002": {"suoxie": "HXHBHHA", "name": "\u534e\u590f\u56de\u62a5\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAHUIBAOHUNHEA"}, "002003": {"suoxie": "GYXJYLHPZHHA", "name": "\u5de5\u94f6\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIYULINGHUOPEIZHIHUNHEA"}, "002004": {"suoxie": "GYXJYLHPZHHC", "name": "\u5de5\u94f6\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIYULINGHUOPEIZHIHUNHEC"}, "002005": {"suoxie": "GYXDLHH", "name": "\u5de5\u94f6\u65b0\u5f97\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINXINDELIHUNHE"}, "002006": {"suoxie": "GYXDYHH", "name": "\u5de5\u94f6\u65b0\u5f97\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINXINDEYIHUNHE"}, "002009": {"suoxie": "ZOJTLHPZHHA", "name": "\u4e2d\u6b27\u747e\u901a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINTONGLINGHUOPEIZHIHUNHEA"}, "002010": {"suoxie": "ZOJTLHPZHHC", "name": "\u4e2d\u6b27\u747e\u901a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINTONGLINGHUOPEIZHIHUNHEC"}, "002011": {"suoxie": "HXHLHH", "name": "\u534e\u590f\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHONGLIHUNHE"}, "002012": {"suoxie": "HXHLHH", "name": "\u534e\u590f\u7ea2\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHONGLIHUNHE"}, "002015": {"suoxie": "NFRGA", "name": "\u5357\u65b9\u8363\u5149A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGRONGGUANGA"}, "002016": {"suoxie": "NFRGC", "name": "\u5357\u65b9\u8363\u5149C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGRONGGUANGC"}, "002017": {"suoxie": "ZSRFLHPZHHFQSC", "name": "\u62db\u5546\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGRUIFENGLINGHUOPEIZHIHUNHEFAQISHIC"}, "002018": {"suoxie": "PHHAHHA", "name": "\u9e4f\u534e\u5f18\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGANHUNHEA"}, "002019": {"suoxie": "PHHAHHC", "name": "\u9e4f\u534e\u5f18\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGANHUNHEC"}, "002020": {"suoxie": "GDCXQD", "name": "\u56fd\u90fd\u521b\u65b0\u9a71\u52a8", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUODUCHUANGXINQUDONG"}, "002021": {"suoxie": "HXHBEHHH", "name": "\u534e\u590f\u56de\u62a5\u4e8c\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAHUIBAOERHAOHUNHE"}, "002023": {"suoxie": "HTHTWJHBA", "name": "\u7ea2\u5854\u7ea2\u571f\u7a33\u5065\u56de\u62a5A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUWENJIANHUIBAOA"}, "002024": {"suoxie": "HTHTWJHBC", "name": "\u7ea2\u5854\u7ea2\u571f\u7a33\u5065\u56de\u62a5C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUWENJIANHUIBAOC"}, "002025": {"suoxie": "GFJSHHA", "name": "\u5e7f\u53d1\u805a\u76db\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJUSHENGHUNHEA"}, "002026": {"suoxie": "GFJSHHC", "name": "\u5e7f\u53d1\u805a\u76db\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJUSHENGHUNHEC"}, "002027": {"suoxie": "ZJXXHHA", "name": "\u4e2d\u52a0\u5fc3\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAXINXIANGHUNHEA"}, "002028": {"suoxie": "JTTBLHPZHHC", "name": "\u4e5d\u6cf0\u5929\u5b9d\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAITIANBAOLINGHUOPEIZHIHUNHEC"}, "002029": {"suoxie": "AXDTCLHHC", "name": "\u5b89\u4fe1\u52a8\u6001\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINDONGTAICELUEHUNHEC"}, "002030": {"suoxie": "XCXXHHB", "name": "\u4fe1\u8bda\u65b0\u9009\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINXUANHUNHEB"}, "002031": {"suoxie": "HXCLHH", "name": "\u534e\u590f\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIACELUEHUNHE"}, "002035": {"suoxie": "AXPWZZHHFQC", "name": "\u5b89\u4fe1\u5e73\u7a33\u589e\u957f\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINPINGWENZENGZHANGHUNHEFAQIC"}, "002036": {"suoxie": "AXYSZZHHC", "name": "\u5b89\u4fe1\u4f18\u52bf\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINYOUSHIZENGZHANGHUNHEC"}, "002043": {"suoxie": "TZYJQDHHC", "name": "\u5929\u6cbb\u7814\u7a76\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIYANJIUQUDONGHUNHEC"}, "002046": {"suoxie": "XCXRHHB", "name": "\u4fe1\u8bda\u65b0\u9510\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINRUIHUNHEB"}, "002048": {"suoxie": "BSAY18GYDKZ", "name": "\u535a\u65f6\u5b89\u8a8918\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANYU18GEYUEDINGKAIZHAI"}, "002049": {"suoxie": "RTXJYLHPZHH", "name": "\u878d\u901a\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINJIYULINGHUOPEIZHIHUNHE"}, "002050": {"suoxie": "RTXJYLHPZHH", "name": "\u878d\u901a\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINJIYULINGHUOPEIZHIHUNHE"}, "002051": {"suoxie": "NACXQDHHC", "name": "\u8bfa\u5b89\u521b\u65b0\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANCHUANGXINQUDONGHUNHEC"}, "002052": {"suoxie": "NAWJHBLHPZHHC", "name": "\u8bfa\u5b89\u7a33\u5065\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANWENJIANHUIBAOLINGHUOPEIZHIHUNHEC"}, "002053": {"suoxie": "NAYSHYHHC", "name": "\u8bfa\u5b89\u4f18\u52bf\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANYOUSHIHANGYEHUNHEC"}, "002054": {"suoxie": "ZYXCFHHA", "name": "\u4e2d\u94f6\u65b0\u8d22\u5bcc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINCAIFUHUNHEA"}, "002055": {"suoxie": "GTXYLHPZHHC", "name": "\u56fd\u6cf0\u5174\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIXINGYILINGHUOPEIZHIHUNHEC"}, "002056": {"suoxie": "ZYXCFHHC", "name": "\u4e2d\u94f6\u65b0\u8d22\u5bcc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINCAIFUHUNHEC"}, "002057": {"suoxie": "ZYXJYHHA", "name": "\u4e2d\u94f6\u65b0\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINJIYUHUNHEA"}, "002058": {"suoxie": "ZYXJYHHC", "name": "\u4e2d\u94f6\u65b0\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINJIYUHUNHEC"}, "002059": {"suoxie": "GTNYLHPZHHC", "name": "\u56fd\u6cf0\u6d53\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAINONGYILINGHUOPEIZHIHUNHEC"}, "002060": {"suoxie": "DFXCLLHPZHHC", "name": "\u4e1c\u65b9\u65b0\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGXINCELUELINGHUOPEIZHIHUNHEC"}, "002061": {"suoxie": "GTAKDQZFHHC", "name": "\u56fd\u6cf0\u5b89\u5eb7\u5b9a\u671f\u652f\u4ed8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIANKANGDINGQIZHIFUHUNHEC"}, "002062": {"suoxie": "GTGCQDLHPZHHC", "name": "\u56fd\u6cf0\u56fd\u7b56\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIGUOCEQUDONGLINGHUOPEIZHIHUNHEC"}, "002063": {"suoxie": "GTHS300ZSZQC", "name": "\u56fd\u6cf0\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHUSHEN300ZHISHUZENGQIANGC"}, "002064": {"suoxie": "HFCYSJLHPZHH", "name": "\u534e\u5bcc\u4ea7\u4e1a\u5347\u7ea7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUCHANYESHENGJILINGHUOPEIZHIHUNHE"}, "002065": {"suoxie": "JSCCJSSXSYZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u76db\u53cc\u606f\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGSHENGSHUANGXISHOUYIZHAIQUANA"}, "002066": {"suoxie": "JSCCJSSXSYZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u76db\u53cc\u606f\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGSHENGSHUANGXISHOUYIZHAIQUANC"}, "002067": {"suoxie": "NAJXHBHH", "name": "\u8bfa\u5b89\u7cbe\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANJINGXUANHUIBAOHUNHE"}, "002068": {"suoxie": "DFDCLLHPZHHC", "name": "\u4e1c\u65b9\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGDUOCELUELINGHUOPEIZHIHUNHEC"}, "002071": {"suoxie": "CACYJXHHC", "name": "\u957f\u5b89\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANCHANYEJINGXUANHUNHEC"}, "002072": {"suoxie": "CAXLYXHHC", "name": "\u957f\u5b89\u946b\u5229\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINLIYOUXUANHUNHEC"}, "002073": {"suoxie": "YXYFXRA", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u878dA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGXINGRONGA"}, "002074": {"suoxie": "YXYFXRC", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u878dC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGXINGRONGC"}, "002075": {"suoxie": "GDRXHHC", "name": "\u5149\u5927\u777f\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDARUIXINHUNHEC"}, "002076": {"suoxie": "ZSZZ500ZSZQA", "name": "\u6d59\u5546\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHONGZHENG500ZHISHUZENGQIANGA"}, "002077": {"suoxie": "ZSRTLA", "name": "\u6d59\u5546\u65e5\u6dfb\u5229A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHESHANGRITIANLIA"}, "002078": {"suoxie": "ZSRTLB", "name": "\u6d59\u5546\u65e5\u6dfb\u5229B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHESHANGRITIANLIB"}, "002079": {"suoxie": "QHKYZGXQZCHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u56fd\u7a00\u7f3a\u8d44\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZHONGGUOXIQUEZICHANHUNHEC"}, "002080": {"suoxie": "QHKYYDYLHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4e00\u5e26\u4e00\u8def\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYIDAIYILUHUNHEC"}, "002082": {"suoxie": "HTBRJLDLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u6fc0\u52b1\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIJILIDONGLIHUNHEC"}, "002083": {"suoxie": "XHXDLLHPZHHA", "name": "\u65b0\u534e\u946b\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINDONGLILINGHUOPEIZHIHUNHEA"}, "002084": {"suoxie": "XHXDLLHPZHHC", "name": "\u65b0\u534e\u946b\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINDONGLILINGHUOPEIZHIHUNHEC"}, "002085": {"suoxie": "CSHLWHH", "name": "\u957f\u76db\u4e92\u8054\u7f51+\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGHULIANWANGHUNHE"}, "002086": {"suoxie": "DCJADRZQE", "name": "\u5927\u6210\u666f\u5b89\u77ed\u878d\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGANDUANRONGZHAIQUANE"}, "002087": {"suoxie": "GFXJYHHA", "name": "\u56fd\u5bcc\u65b0\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUXINJIYUHUNHEA"}, "002088": {"suoxie": "GFXJYHHC", "name": "\u56fd\u5bcc\u65b0\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUXINJIYUHUNHEC"}, "002091": {"suoxie": "HTBRXLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXINLIHUNHEC"}, "002095": {"suoxie": "BSXSYA", "name": "\u535a\u65f6\u65b0\u6536\u76caA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINSHOUYIA"}, "002096": {"suoxie": "BSXSYC", "name": "\u535a\u65f6\u65b0\u6536\u76caC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINSHOUYIC"}, "002101": {"suoxie": "CJHXZZJXZQA", "name": "\u521b\u91d1\u5408\u4fe1\u8f6c\u503a\u7cbe\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHUANGJINHEXINZHUANZHAIJINGXUANZHAIQUANA"}, "002102": {"suoxie": "CJHXZZJXZQC", "name": "\u521b\u91d1\u5408\u4fe1\u8f6c\u503a\u7cbe\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHUANGJINHEXINZHUANZHAIJINGXUANZHAIQUANC"}, "002103": {"suoxie": "ZSKTLHPZHH", "name": "\u62db\u5546\u5eb7\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGKANGTAILINGHUOPEIZHIHUNHE"}, "002106": {"suoxie": "DBFXLHPZHHC", "name": "\u5fb7\u90a6\u798f\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGFUXINLINGHUOPEIZHIHUNHEC"}, "002109": {"suoxie": "BSYFCZ3GYDKZ", "name": "\u535a\u65f6\u88d5\u4e30\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUFENGCHUNZHAI3GEYUEDINGKAIZHAI"}, "002111": {"suoxie": "HBXQDHH", "name": "\u534e\u5b9d\u65b0\u8d77\u70b9\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINQIDIANHUNHE"}, "002112": {"suoxie": "DBXXJZLHPZHHC", "name": "\u5fb7\u90a6\u946b\u661f\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGXINXINGJIAZHILINGHUOPEIZHIHUNHEC"}, "002116": {"suoxie": "GFAXHHA", "name": "\u5e7f\u53d1\u5b89\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANXIANGHUNHEA"}, "002117": {"suoxie": "GFAXHHC", "name": "\u5e7f\u53d1\u5b89\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANXIANGHUNHEC"}, "002118": {"suoxie": "GFAYHHA", "name": "\u5e7f\u53d1\u5b89\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANYINGHUNHEA"}, "002119": {"suoxie": "GFAYHHC", "name": "\u5e7f\u53d1\u5b89\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANYINGHUNHEC"}, "002120": {"suoxie": "GFAYHBHHA", "name": "\u5e7f\u53d1\u5b89\u60a6\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANYUEHUIBAOHUNHEA"}, "002121": {"suoxie": "GFHGSXQDGPA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u65b0\u8d77\u70b9\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAHUGANGSHENXINQIDIANGUPIAOA"}, "002123": {"suoxie": "BXRFWYZZ", "name": "\u5317\u4fe1\u745e\u4e30\u5916\u5ef6\u589e\u957f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGWAIYANZENGZHANG"}, "002124": {"suoxie": "GFXXCYHHA", "name": "\u5e7f\u53d1\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINXINGCHANYEHUNHEA"}, "002125": {"suoxie": "GFXXCZHH", "name": "\u5e7f\u53d1\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINXINGCHENGZHANGHUNHE"}, "002128": {"suoxie": "GFXHCZDK", "name": "\u5e7f\u53d1\u946b\u60e0\u7eaf\u503a\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAXINHUICHUNZHAIDINGKAI"}, "002132": {"suoxie": "GFXXHH", "name": "\u5e7f\u53d1\u946b\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINXIANGHUNHE"}, "002133": {"suoxie": "GFXYHH", "name": "\u5e7f\u53d1\u946b\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINYIHUNHE"}, "002134": {"suoxie": "GFXYHHA", "name": "\u5e7f\u53d1\u946b\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINYUHUNHEA"}, "002135": {"suoxie": "GFXYHHA", "name": "\u5e7f\u53d1\u946b\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINYUANHUNHEA"}, "002136": {"suoxie": "GFXYHHC", "name": "\u5e7f\u53d1\u946b\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINYUANHUNHEC"}, "002137": {"suoxie": "NALXLHPZHHA", "name": "\u8bfa\u5b89\u5229\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANLIXINLINGHUOPEIZHIHUNHEA"}, "002138": {"suoxie": "HDYTZQA", "name": "\u6cd3\u5fb7\u88d5\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUTAIZHAIQUANA"}, "002139": {"suoxie": "HDYTZQC", "name": "\u6cd3\u5fb7\u88d5\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUTAIZHAIQUANC"}, "002140": {"suoxie": "BSYCCZZQ", "name": "\u535a\u65f6\u88d5\u8bda\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUCHENGCHUNZHAIZHAIQUAN"}, "002142": {"suoxie": "BSWYZZZTLHPZHH", "name": "\u535a\u65f6\u5916\u5ef6\u589e\u957f\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIWAIYANZENGZHANGZHUTILINGHUOPEIZHIHUNHE"}, "002143": {"suoxie": "BSYK3GYDKZ", "name": "\u535a\u65f6\u88d5\u57643\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUKUN3GEYUEDINGKAIZHAI"}, "002144": {"suoxie": "HAXYXLHPZHHC", "name": "\u534e\u5b89\u65b0\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINYOUXUANLINGHUOPEIZHIHUNHEC"}, "002145": {"suoxie": "NAJXLHPZHH", "name": "\u8bfa\u5b89\u666f\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANJINGXINLINGHUOPEIZHIHUNHE"}, "002146": {"suoxie": "CAXYZQHHA", "name": "\u957f\u5b89\u946b\u76ca\u589e\u5f3a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGANXINYIZENGQIANGHUNHEA"}, "002147": {"suoxie": "CAXYZQHHC", "name": "\u957f\u5b89\u946b\u76ca\u589e\u5f3a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGANXINYIZENGQIANGHUNHEC"}, "002148": {"suoxie": "GSABWHHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u60e0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOWENHUIHUNHE"}, "002149": {"suoxie": "JSXYXHH", "name": "\u5609\u5b9e\u65b0\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINYOUXUANHUNHE"}, "002152": {"suoxie": "HBHXYSHH", "name": "\u534e\u5b9d\u6838\u5fc3\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOHEXINYOUSHIHUNHE"}, "002155": {"suoxie": "GJXRLHPZHH", "name": "\u56fd\u91d1\u946b\u745e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOJINXINRUILINGHUOPEIZHIHUNHE"}, "002156": {"suoxie": "CSSSHHA", "name": "\u957f\u76db\u76db\u4e16\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGSHIHUNHEA"}, "002157": {"suoxie": "CSSSHHC", "name": "\u957f\u76db\u76db\u4e16\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGSHIHUNHEC"}, "002158": {"suoxie": "HTFAXZXHHC", "name": "\u6c47\u6dfb\u5bcc\u5b89\u946b\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUANXINZHIXUANHUNHEC"}, "002159": {"suoxie": "DWGQGGZTLHPZHHA", "name": "\u4e1c\u5434\u56fd\u4f01\u6539\u9769\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUGUOQIGAIGEZHUTILINGHUOPEIZHIHUNHEA"}, "002160": {"suoxie": "NFQDHH", "name": "\u5357\u65b9\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGQUDONGHUNHE"}, "002161": {"suoxie": "YHWWHLLHPZHH", "name": "\u94f6\u534e\u4e07\u7269\u4e92\u8054\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAWANWUHULIANLINGHUOPEIZHIHUNHE"}, "002162": {"suoxie": "DFXJZHHC", "name": "\u4e1c\u65b9\u65b0\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINJIAZHIHUNHEC"}, "002163": {"suoxie": "DFHXLHPZHHC", "name": "\u4e1c\u65b9\u60e0\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHUIXINLINGHUOPEIZHIHUNHEC"}, "002164": {"suoxie": "HTFXRJXHHC", "name": "\u6c47\u6dfb\u5bcc\u65b0\u777f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUXINRUIJINGXUANHUNHEC"}, "002165": {"suoxie": "HTFDXHHC", "name": "\u6c47\u6dfb\u5bcc\u8fbe\u6b23\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUDAXINHUNHEC"}, "002166": {"suoxie": "HXYFHHC", "name": "\u534e\u590f\u6c38\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGFUHUNHEC"}, "002168": {"suoxie": "JSZNQCGP", "name": "\u5609\u5b9e\u667a\u80fd\u6c7d\u8f66\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIZHINENGQICHEGUPIAO"}, "002169": {"suoxie": "YYWYZQ", "name": "\u6c38\u8d62\u7a33\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGWENYIZHAIQUAN"}, "002170": {"suoxie": "DWYDHLHHC", "name": "\u4e1c\u5434\u79fb\u52a8\u4e92\u8054\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUYIDONGHULIANHUNHEC"}, "002172": {"suoxie": "HFTXNXHHC", "name": "\u6d77\u5bcc\u901a\u65b0\u5185\u9700\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINNEIXUHUNHEC"}, "002174": {"suoxie": "DFHLWJHH", "name": "\u4e1c\u65b9\u4e92\u8054\u7f51\u5609\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHULIANWANGJIAHUNHE"}, "002175": {"suoxie": "BSYQCZA", "name": "\u535a\u65f6\u88d5\u4e7e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUQIANCHUNZHAIA"}, "002177": {"suoxie": "XCXZB", "name": "\u4fe1\u8bda\u65b0\u6cfdB", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINZEB"}, "002178": {"suoxie": "JSXQDHHC", "name": "\u5609\u5b9e\u65b0\u8d77\u70b9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINQIDIANHUNHEC"}, "002179": {"suoxie": "HASJQDLHCLHH", "name": "\u534e\u5b89\u4e8b\u4ef6\u9a71\u52a8\u91cf\u5316\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANSHIJIANQUDONGLIANGHUACELUEHUNHE"}, "002181": {"suoxie": "HADAQZTHHA", "name": "\u534e\u5b89\u5927\u5b89\u5168\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANDAANQUANZHUTIHUNHEA"}, "002182": {"suoxie": "DXLHCFHH", "name": "\u4e1c\u5174\u84dd\u6d77\u8d22\u5bcc\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGLANHAICAIFUHUNHE"}, "002183": {"suoxie": "GFTTHHBB", "name": "\u5e7f\u53d1\u5929\u5929\u7ea2\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANTIANHONGHUOBIB"}, "002184": {"suoxie": "HDHLHBA", "name": "\u6cd3\u5fb7\u6cd3\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGDEHONGLIHUOBIA"}, "002185": {"suoxie": "HDHLHBB", "name": "\u6cd3\u5fb7\u6cd3\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGDEHONGLIHUOBIB"}, "002186": {"suoxie": "GLAXXLHPZHHC", "name": "\u56fd\u8054\u5b89\u946b\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANXINXIANGLINGHUOPEIZHIHUNHEC"}, "002188": {"suoxie": "PHFHZQ", "name": "\u9e4f\u534e\u4e30\u534e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGHUAZHAIQUAN"}, "002189": {"suoxie": "NYHLGQGGHH", "name": "\u519c\u94f6\u6c47\u7406\u56fd\u4f01\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINHUILIGUOQIGAIGEHUNHE"}, "002190": {"suoxie": "NYXNYZT", "name": "\u519c\u94f6\u65b0\u80fd\u6e90\u4e3b\u9898", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINXINNENGYUANZHUTI"}, "002192": {"suoxie": "DFDXLHPZHHC", "name": "\u4e1c\u65b9\u9f0e\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGDINGXINLINGHUOPEIZHIHUNHEC"}, "002194": {"suoxie": "BXRFWDZQ", "name": "\u5317\u4fe1\u745e\u4e30\u7a33\u5b9a\u589e\u5f3a", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BEIXINRUIFENGWENDINGZENGQIANG"}, "002195": {"suoxie": "ZYJGXJGLHB", "name": "\u4e2d\u94f6\u673a\u6784\u73b0\u91d1\u7ba1\u7406\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINJIGOUXIANJINGUANLIHUOBI"}, "002196": {"suoxie": "JYJSLXLHPZHHC", "name": "\u91d1\u9e70\u6280\u672f\u9886\u5148\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGJISHULINGXIANLINGHUOPEIZHIHUNHEC"}, "002197": {"suoxie": "GTXCLJZHH", "name": "\u56fd\u6cf0\u946b\u7b56\u7565\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIXINCELUEJIAZHIHUNHE"}, "002198": {"suoxie": "BSYDCZZQ", "name": "\u535a\u65f6\u88d5\u8fbe\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUDACHUNZHAIZHAIQUAN"}, "002199": {"suoxie": "QHKYZZJGZSC", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u8bc1\u519b\u5de5\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGZHENGJUNGONGZHISHUC"}, "002200": {"suoxie": "DCHCHBA", "name": "\u5927\u6210\u6167\u6210\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHUICHENGHUOBIA"}, "002201": {"suoxie": "DCHCHBB", "name": "\u5927\u6210\u6167\u6210\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHUICHENGHUOBIB"}, "002202": {"suoxie": "DCHCHBE", "name": "\u5927\u6210\u6167\u6210\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHUICHENGHUOBIE"}, "002206": {"suoxie": "BSYKCZZQ", "name": "\u535a\u65f6\u88d5\u5eb7\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUKANGCHUNZHAIZHAIQUAN"}, "002207": {"suoxie": "QHKYJYZBHHC", "name": "\u524d\u6d77\u5f00\u6e90\u91d1\u94f6\u73e0\u5b9d\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJINYINZHUBAOHUNHEC"}, "002210": {"suoxie": "CJHXLHDYZGPA", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u591a\u56e0\u5b50\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINLIANGHUADUOYINZIGUPIAOA"}, "002211": {"suoxie": "JSXCFHH", "name": "\u5609\u5b9e\u65b0\u8d22\u5bcc\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINCAIFUHUNHE"}, "002212": {"suoxie": "JSXQHHH", "name": "\u5609\u5b9e\u65b0\u8d77\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINQIHANGHUNHE"}, "002213": {"suoxie": "ZHSXLHPZHH", "name": "\u4e2d\u6d77\u987a\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAISHUNXINLINGHUOPEIZHIHUNHE"}, "002214": {"suoxie": "ZHHGSJZYXHH", "name": "\u4e2d\u6d77\u6caa\u6e2f\u6df1\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIHUGANGSHENJIAZHIYOUXUANHUNHE"}, "002216": {"suoxie": "YFDLHCLA", "name": "\u6613\u65b9\u8fbe\u91cf\u5316\u7b56\u7565A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDALIANGHUACELUEA"}, "002217": {"suoxie": "YFDLHCLC", "name": "\u6613\u65b9\u8fbe\u91cf\u5316\u7b56\u7565C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDALIANGHUACELUEC"}, "002218": {"suoxie": "NFHLDKZ", "name": "\u5357\u65b9\u5f18\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHONGLIDINGKAIZHAI"}, "002219": {"suoxie": "NFHLC", "name": "\u5357\u65b9\u5f18\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHONGLIC"}, "002220": {"suoxie": "NFRLLHPZHH", "name": "\u5357\u65b9\u745e\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGRUILILINGHUOPEIZHIHUNHE"}, "002222": {"suoxie": "JSXQSHH", "name": "\u5609\u5b9e\u65b0\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINQUSHIHUNHE"}, "002224": {"suoxie": "ZYJDSYCLDQKFHH", "name": "\u4e2d\u90ae\u7edd\u5bf9\u6536\u76ca\u7b56\u7565\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ZHONGYOUJUEDUISHOUYICELUEDINGQIKAIFANGHUNHE"}, "002227": {"suoxie": "CCXYXHHA", "name": "\u957f\u57ce\u65b0\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGXINYOUXUANHUNHEA"}, "002228": {"suoxie": "CCXYXHHC", "name": "\u957f\u57ce\u65b0\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGXINYOUXUANHUNHEC"}, "002229": {"suoxie": "HXJJZXGP", "name": "\u534e\u590f\u7ecf\u6d4e\u8f6c\u578b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAJINGJIZHUANXINGGUPIAO"}, "002230": {"suoxie": "HXDZHHHQDII", "name": "\u534e\u590f\u5927\u4e2d\u534e\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "HUAXIADAZHONGHUAHUNHEQDII"}, "002231": {"suoxie": "HXXQSHHA", "name": "\u534e\u590f\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINQUSHIHUNHEA"}, "002232": {"suoxie": "HXXQSHHC", "name": "\u534e\u590f\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINQUSHIHUNHEC"}, "002233": {"suoxie": "GYFSHBLHPZHHC", "name": "\u5de5\u94f6\u4e30\u6536\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINFENGSHOUHUIBAOLINGHUOPEIZHIHUNHEC"}, "002234": {"suoxie": "TXTTSYHBB", "name": "\u6cf0\u4fe1\u5929\u5929\u6536\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIXINTIANTIANSHOUYIHUOBIB"}, "002235": {"suoxie": "TXTTSYHBE", "name": "\u6cf0\u4fe1\u5929\u5929\u6536\u76ca\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIXINTIANTIANSHOUYIHUOBIE"}, "002236": {"suoxie": "DC360HLWDSJ100A", "name": "\u5927\u6210360\u4e92\u8054\u7f51+\u5927\u6570\u636e100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENG360HULIANWANGDASHUJU100A"}, "002243": {"suoxie": "DFJZTHBA", "name": "\u4e1c\u65b9\u91d1\u8bc1\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGJINZHENGTONGHUOBIA"}, "002244": {"suoxie": "JSCCDTKJZTHH", "name": "\u666f\u987a\u957f\u57ce\u4f4e\u78b3\u79d1\u6280\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGDITANKEJIZHUTIHUNHE"}, "002245": {"suoxie": "TKWJZLZQA", "name": "\u6cf0\u5eb7\u7a33\u5065\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIKANGWENJIANZENGLIZHAIQUANA"}, "002246": {"suoxie": "TKWJZLZQC", "name": "\u6cf0\u5eb7\u7a33\u5065\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIKANGWENJIANZENGLIZHAIQUANC"}, "002247": {"suoxie": "XJQHLHHYHBA", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u6d77\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINJIANGQIANHAILIANHEHAIYINGHUOBIA"}, "002248": {"suoxie": "XJQHLHHYHBB", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u6d77\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINJIANGQIANHAILIANHEHAIYINGHUOBIB"}, "002249": {"suoxie": "ZSJYLHPZHH", "name": "\u62db\u5546\u5883\u8fdc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGJINGYUANLINGHUOPEIZHIHUNHE"}, "002251": {"suoxie": "HXJGAQHHA", "name": "\u534e\u590f\u519b\u5de5\u5b89\u5168\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAJUNGONGANQUANHUNHEA"}, "002252": {"suoxie": "RTCZ30LHPZHHA", "name": "\u878d\u901a\u6210\u957f30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGCHENGZHANG30LINGHUOPEIZHIHUNHEA"}, "002253": {"suoxie": "RTCZ30LHPZHHA", "name": "\u878d\u901a\u6210\u957f30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGCHENGZHANG30LINGHUOPEIZHIHUNHEA"}, "002254": {"suoxie": "CXJKCZA", "name": "\u957f\u4fe1\u91d1\u8475\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINJINKUICHUNZHAIA"}, "002255": {"suoxie": "CXJKCZC", "name": "\u957f\u4fe1\u91d1\u8475\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINJINKUICHUNZHAIC"}, "002256": {"suoxie": "JXHYYXHHFQS", "name": "\u91d1\u4fe1\u884c\u4e1a\u4f18\u9009\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINHANGYEYOUXUANHUNHEFAQISHI"}, "002258": {"suoxie": "DCGQGGLHPZHH", "name": "\u5927\u6210\u56fd\u4f01\u6539\u9769\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGGUOQIGAIGELINGHUOPEIZHIHUNHE"}, "002259": {"suoxie": "PHJKHBHH", "name": "\u9e4f\u534e\u5065\u5eb7\u73af\u4fdd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAJIANKANGHUANBAOHUNHE"}, "002260": {"suoxie": "ZXJTTXB", "name": "\u4e2d\u4fe1\u5efa\u6295\u6dfb\u946b\u5b9d", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGXINJIANTOUTIANXINBAO"}, "002261": {"suoxie": "ZYBLHHA", "name": "\u4e2d\u94f6\u5b9d\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINBAOLIHUNHEA"}, "002262": {"suoxie": "ZYBLHHC", "name": "\u4e2d\u94f6\u5b9d\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINBAOLIHUNHEC"}, "002264": {"suoxie": "HXLXJKHH", "name": "\u534e\u590f\u4e50\u4eab\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIALEXIANGJIANKANGHUNHE"}, "002265": {"suoxie": "XYXLDQKFZ", "name": "\u946b\u5143\u5174\u5229\u5b9a\u671f\u5f00\u653e\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANXINGLIDINGQIKAIFANGZHAI"}, "002268": {"suoxie": "XYFLZQ", "name": "\u5174\u4e1a\u4e30\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEFENGLIZHAIQUAN"}, "002269": {"suoxie": "YHDSJLHPZDKHH", "name": "\u94f6\u534e\u5927\u6570\u636e\u7075\u6d3b\u914d\u7f6e\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUADASHUJULINGHUOPEIZHIDINGKAIHUNHE"}, "002270": {"suoxie": "DWAYLHHH", "name": "\u4e1c\u5434\u5b89\u76c8\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUANYINGLIANGHUAHUNHE"}, "002271": {"suoxie": "ZSAHLHPZHH", "name": "\u62db\u5546\u5b89\u5f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANHONGLINGHUOPEIZHIHUNHE"}, "002272": {"suoxie": "XHKJCXZTLHPZHH", "name": "\u65b0\u534e\u79d1\u6280\u521b\u65b0\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAKEJICHUANGXINZHUTILINGHUOPEIZHIHUNHE"}, "002273": {"suoxie": "TDHLCYHHB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u521b\u76ca\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLICHUANGYIHUNHEB"}, "002274": {"suoxie": "ZYCZJLZQA", "name": "\u4e2d\u90ae\u7eaf\u503a\u805a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIJULIZHAIQUANA"}, "002275": {"suoxie": "ZYCZJLZQC", "name": "\u4e2d\u90ae\u7eaf\u503a\u805a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIJULIZHAIQUANC"}, "002276": {"suoxie": "ZYCZHLZQA", "name": "\u4e2d\u90ae\u7eaf\u503a\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOUCHUNZHAIHENGLIZHAIQUANA"}, "002277": {"suoxie": "ZYCZHLZQC", "name": "\u4e2d\u90ae\u7eaf\u503a\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOUCHUNZHAIHENGLIZHAIQUANC"}, "002279": {"suoxie": "ZSHYCZA", "name": "\u6d59\u5546\u60e0\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIYINGCHUNZHAIA"}, "002280": {"suoxie": "HFAXZQ", "name": "\u534e\u5bcc\u5b89\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANXIANGZHAIQUAN"}, "002281": {"suoxie": "JXYLLHPZHH", "name": "\u5efa\u4fe1\u88d5\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINYULILINGHUOPEIZHIHUNHE"}, "002282": {"suoxie": "PAAXLHPZHHA", "name": "\u5e73\u5b89\u5b89\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANXIANGLINGHUOPEIZHIHUNHEA"}, "002286": {"suoxie": "ZYMYZZQRMB", "name": "\u4e2d\u94f6\u7f8e\u5143\u503a\u503a\u5238\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "ZHONGYINMEIYUANZHAIZHAIQUANRENMINBI"}, "002287": {"suoxie": "ZYMYZZQMY", "name": "\u4e2d\u94f6\u7f8e\u5143\u503a\u503a\u5238\u7f8e\u5143", "style": "QDII", "pinyin": "ZHONGYINMEIYUANZHAIZHAIQUANMEIYUAN"}, "002288": {"suoxie": "ZYWJCLHH", "name": "\u4e2d\u94f6\u7a33\u8fdb\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINWENJINCELUEHUNHE"}, "002289": {"suoxie": "HSGGCXGP", "name": "\u534e\u5546\u6539\u9769\u521b\u65b0\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGGAIGECHUANGXINGUPIAO"}, "002291": {"suoxie": "NAAXLHPZHH", "name": "\u8bfa\u5b89\u5b89\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANANXINLINGHUOPEIZHIHUNHE"}, "002292": {"suoxie": "NAYXLHPZHHA", "name": "\u8bfa\u5b89\u76ca\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANYIXINLINGHUOPEIZHIHUNHEA"}, "002293": {"suoxie": "NFYHHH", "name": "\u5357\u65b9\u76ca\u548c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYIHEHUNHE"}, "002295": {"suoxie": "GFWALHPZA", "name": "\u5e7f\u53d1\u7a33\u5b89\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAWENANLINGHUOPEIZHIA"}, "002296": {"suoxie": "CCHYLDHHA", "name": "\u957f\u57ce\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGHANGYELUNDONGHUNHEA"}, "002298": {"suoxie": "ZSZFBHBA", "name": "\u62db\u5546\u62db\u798f\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOFUBAOHUOBIA"}, "002299": {"suoxie": "ZSZFBHBB", "name": "\u62db\u5546\u62db\u798f\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOFUBAOHUOBIB"}, "002300": {"suoxie": "CSYLHYLHPZGP", "name": "\u957f\u76db\u533b\u7597\u884c\u4e1a\u91cf\u5316\u914d\u7f6e\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGSHENGYILIAOHANGYELIANGHUAPEIZHIGUPIAO"}, "002301": {"suoxie": "XYDZZQA", "name": "\u5174\u4e1a\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYEDUANZHAIZHAIQUANA"}, "002302": {"suoxie": "XWTBB", "name": "\u65b0\u6c83\u901a\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "XINWOTONGBAOB"}, "002303": {"suoxie": "JYZHSHHH", "name": "\u91d1\u9e70\u667a\u6167\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGZHIHUISHENGHUOHUNHE"}, "002304": {"suoxie": "PAAXLHPZHHA", "name": "\u5e73\u5b89\u5b89\u5fc3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANXINLINGHUOPEIZHIHUNHEA"}, "002305": {"suoxie": "GDFGLDHHA", "name": "\u5149\u5927\u98ce\u683c\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAFENGGELUNDONGHUNHEA"}, "002307": {"suoxie": "YHDYSYLHPZHH", "name": "\u94f6\u534e\u591a\u5143\u89c6\u91ce\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUADUOYUANSHIYELINGHUOPEIZHIHUNHE"}, "002310": {"suoxie": "CJHXHS300ZSZQA", "name": "\u521b\u91d1\u5408\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINHUSHEN300ZHISHUZENGQIANGA"}, "002311": {"suoxie": "CJHXZZ500ZSZQA", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINZHONGZHENG500ZHISHUZENGQIANGA"}, "002313": {"suoxie": "TDHLXQDHHB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u8d77\u70b9\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIXINQIDIANHUNHEB"}, "002314": {"suoxie": "TDHLXSLHHB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u601d\u8def\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIXINSILUHUNHEB"}, "002315": {"suoxie": "CJHXHS300ZSZQC", "name": "\u521b\u91d1\u5408\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINHUSHEN300ZHISHUZENGQIANGC"}, "002316": {"suoxie": "CJHXZZ500ZSZQC", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINZHONGZHENG500ZHISHUZENGQIANGC"}, "002317": {"suoxie": "ZSRYHH", "name": "\u62db\u5546\u777f\u9038\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIYIHUNHE"}, "002318": {"suoxie": "PHTLJYXHBA", "name": "\u9e4f\u534e\u6dfb\u5229\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUATIANLIJIAOYIXINGHUOBIA"}, "002319": {"suoxie": "DCYDYLLHPZHH", "name": "\u5927\u6210\u4e00\u5e26\u4e00\u8def\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGYIDAIYILULINGHUOPEIZHIHUNHE"}, "002322": {"suoxie": "YHHLLHPZHHC", "name": "\u94f6\u534e\u6c47\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHUILILINGHUOPEIZHIHUNHEC"}, "002323": {"suoxie": "YHWLLHPZHHC", "name": "\u94f6\u534e\u7a33\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAWENLILINGHUOPEIZHIHUNHEC"}, "002324": {"suoxie": "NFRTYA", "name": "\u5357\u65b9\u65e5\u6dfb\u76caA", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGRITIANYIA"}, "002325": {"suoxie": "NFRTYE", "name": "\u5357\u65b9\u65e5\u6dfb\u76caE", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGRITIANYIE"}, "002326": {"suoxie": "YHJLLHPZHHC", "name": "\u94f6\u534e\u805a\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAJULILINGHUOPEIZHIHUNHEC"}, "002328": {"suoxie": "YHTLLHPZHHC", "name": "\u94f6\u534e\u6cf0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUATAILILINGHUOPEIZHIHUNHEC"}, "002330": {"suoxie": "XYJBLHPZHH", "name": "\u5174\u4e1a\u805a\u5b9d\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJUBAOLINGHUOPEIZHIHUNHE"}, "002331": {"suoxie": "TKATHBHH", "name": "\u6cf0\u5eb7\u5b89\u6cf0\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGANTAIHUIBAOHUNHE"}, "002332": {"suoxie": "HFJXHGSA", "name": "\u6c47\u4e30\u664b\u4fe1\u6caa\u6e2f\u6df1A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINHUGANGSHENA"}, "002333": {"suoxie": "HFJXHGSC", "name": "\u6c47\u4e30\u664b\u4fe1\u6caa\u6e2f\u6df1C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINHUGANGSHENC"}, "002334": {"suoxie": "HFJXDPBDGPA", "name": "\u6c47\u4e30\u664b\u4fe1\u5927\u76d8\u6ce2\u52a8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINDAPANBODONGGUPIAOA"}, "002335": {"suoxie": "HFJXDPBDGPC", "name": "\u6c47\u4e30\u664b\u4fe1\u5927\u76d8\u6ce2\u52a8\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINDAPANBODONGGUPIAOC"}, "002336": {"suoxie": "CJHXZXCZ", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u4eab\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNXIANGCHUNZHAI"}, "002337": {"suoxie": "CJHXJAX3GYA", "name": "\u521b\u91d1\u5408\u4fe1\u5b63\u5b89\u946b3\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINJIANXIN3GEYUEA"}, "002338": {"suoxie": "XYYZZLZQA", "name": "\u5174\u4e1a\u4f18\u503a\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYOUZHAIZENGLIZHAIQUANA"}, "002339": {"suoxie": "HFTAYSYHHC", "name": "\u6d77\u5bcc\u901a\u5b89\u9890\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGANYISHOUYIHUNHEC"}, "002340": {"suoxie": "FGJZYSHH", "name": "\u5bcc\u56fd\u4ef7\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIAZHIYOUSHIHUNHE"}, "002341": {"suoxie": "ZSZRCZFQSA", "name": "\u62db\u5546\u62db\u745e\u7eaf\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAORUICHUNZHAIFAQISHIA"}, "002342": {"suoxie": "RTZYZQAB", "name": "\u878d\u901a\u589e\u76ca\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGYIZHAIQUANAB"}, "002343": {"suoxie": "RTZYZQAB", "name": "\u878d\u901a\u589e\u76ca\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGYIZHAIQUANAB"}, "002344": {"suoxie": "RTZYZQC", "name": "\u878d\u901a\u589e\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGYIZHAIQUANC"}, "002345": {"suoxie": "HXGDZZHH", "name": "\u534e\u590f\u9ad8\u7aef\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAGAODUANZHIZAOHUNHE"}, "002350": {"suoxie": "HAAHLHPZHH", "name": "\u534e\u5b89\u5b89\u534e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANHUALINGHUOPEIZHIHUNHE"}, "002351": {"suoxie": "YFDYXHBZQ", "name": "\u6613\u65b9\u8fbe\u88d5\u7965\u56de\u62a5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUXIANGHUIBAOZHAIQUAN"}, "002354": {"suoxie": "BSYTCZ", "name": "\u535a\u65f6\u88d5\u817e\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUTENGCHUNZHAI"}, "002356": {"suoxie": "BSAT18GYDKZA", "name": "\u535a\u65f6\u5b89\u6cf018\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANTAI18GEYUEDINGKAIZHAIA"}, "002357": {"suoxie": "BSAT18GYDKZC", "name": "\u535a\u65f6\u5b89\u6cf018\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANTAI18GEYUEDINGKAIZHAIC"}, "002358": {"suoxie": "GTRYRXA", "name": "\u56fd\u6295\u745e\u94f6\u745e\u7965A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUIXIANGA"}, "002360": {"suoxie": "QHKYQJNYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6e05\u6d01\u80fd\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANQINGJIENENGYUANHUNHEC"}, "002361": {"suoxie": "GFHRZQA", "name": "\u56fd\u5bcc\u6052\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUHENGRUIZHAIQUANA"}, "002362": {"suoxie": "GFHRZQC", "name": "\u56fd\u5bcc\u6052\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUHENGRUIZHAIQUANC"}, "002363": {"suoxie": "HAAKLHPZHHA", "name": "\u534e\u5b89\u5b89\u5eb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANKANGLINGHUOPEIZHIHUNHEA"}, "002364": {"suoxie": "HAAKLHPZHHC", "name": "\u534e\u5b89\u5b89\u5eb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANKANGLINGHUOPEIZHIHUNHEC"}, "002367": {"suoxie": "GLAAWHH", "name": "\u56fd\u8054\u5b89\u5b89\u7a33\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANANWENHUNHE"}, "002376": {"suoxie": "GSABHXCYHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u6838\u5fc3\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOHEXINCHANYEHUNHE"}, "002377": {"suoxie": "JXRYCZA", "name": "\u5efa\u4fe1\u777f\u6021\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIYICHUNZHAIA"}, "002378": {"suoxie": "JXHLLHPZHH", "name": "\u5efa\u4fe1\u5f18\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINHONGLILINGHUOPEIZHIHUNHE"}, "002379": {"suoxie": "GYXGZXPRMB", "name": "\u5de5\u94f6\u9999\u6e2f\u4e2d\u5c0f\u76d8\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GONGYINXIANGGANGZHONGXIAOPANRENMINBI"}, "002380": {"suoxie": "GYXGZXPMY", "name": "\u5de5\u94f6\u9999\u6e2f\u4e2d\u5c0f\u76d8\u7f8e\u5143", "style": "QDII", "pinyin": "GONGYINXIANGGANGZHONGXIAOPANMEIYUAN"}, "002381": {"suoxie": "DHXRA", "name": "\u4e1c\u6d77\u7965\u745eA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGHAIXIANGRUIA"}, "002382": {"suoxie": "DHXRC", "name": "\u4e1c\u6d77\u7965\u745eC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGHAIXIANGRUIC"}, "002383": {"suoxie": "DCQSHBLHPZHH", "name": "\u5927\u6210\u8d8b\u52bf\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGQUSHIHUIBAOLINGHUOPEIZHIHUNHE"}, "002384": {"suoxie": "JTHXFWSJLHPZHH", "name": "\u4e5d\u6cf0\u9e3f\u7965\u670d\u52a1\u5347\u7ea7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIHONGXIANGFUWUSHENGJILINGHUOPEIZHIHUNHE"}, "002385": {"suoxie": "BSHS300ZSC", "name": "\u535a\u65f6\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUSHEN300ZHISHUC"}, "002387": {"suoxie": "GYHGSGPA", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINHUGANGSHENGUPIAOA"}, "002388": {"suoxie": "THYLLHPZHHA", "name": "\u5929\u5f18\u88d5\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGYULILINGHUOPEIZHIHUNHEA"}, "002389": {"suoxie": "ZSADLHPZHHA", "name": "\u62db\u5546\u5b89\u5fb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANDELINGHUOPEIZHIHUNHEA"}, "002390": {"suoxie": "ZSADLHPZHHC", "name": "\u62db\u5546\u5b89\u5fb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANDELINGHUOPEIZHIHUNHEC"}, "002391": {"suoxie": "HAQQMYSYZRMBA", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u6536\u76ca\u503a\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANSHOUYIZHAIRENMINBIA"}, "002392": {"suoxie": "HAQQMYSYZMYXHA", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u6536\u76ca\u503a\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANSHOUYIZHAIMEIYUANXIANHUIA"}, "002393": {"suoxie": "HAQQMYSYZC", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u6536\u76ca\u503aC", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANSHOUYIZHAIC"}, "002395": {"suoxie": "PHFSDKZA", "name": "\u9e4f\u534e\u4e30\u5c1a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGSHANGDINGKAIZHAIA"}, "002396": {"suoxie": "PHFSDKZB", "name": "\u9e4f\u534e\u4e30\u5c1a\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGSHANGDINGKAIZHAIB"}, "002398": {"suoxie": "HAAXHHA", "name": "\u534e\u5b89\u5b89\u79a7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANXIHUNHEA"}, "002399": {"suoxie": "HAAXHHC", "name": "\u534e\u5b89\u5b89\u79a7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANXIHUNHEC"}, "002400": {"suoxie": "NFYZMYZRMBA", "name": "\u5357\u65b9\u4e9a\u6d32\u7f8e\u5143\u503a\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "NANFANGYAZHOUMEIYUANZHAIRENMINBIA"}, "002401": {"suoxie": "NFYZMYZRMBC", "name": "\u5357\u65b9\u4e9a\u6d32\u7f8e\u5143\u503a\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "NANFANGYAZHOUMEIYUANZHAIRENMINBIC"}, "002402": {"suoxie": "NFYZMYZMYXHA", "name": "\u5357\u65b9\u4e9a\u6d32\u7f8e\u5143\u503a\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "NANFANGYAZHOUMEIYUANZHAIMEIYUANXIANHUIA"}, "002403": {"suoxie": "NFYZMYZMYXHC", "name": "\u5357\u65b9\u4e9a\u6d32\u7f8e\u5143\u503a\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "NANFANGYAZHOUMEIYUANZHAIMEIYUANXIANHUIC"}, "002404": {"suoxie": "BSYQCZC", "name": "\u535a\u65f6\u88d5\u4e7e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUQIANCHUNZHAIC"}, "002405": {"suoxie": "GDZGDJZQA", "name": "\u5149\u5927\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAZHONGGAODENGJIZHAIQUANA"}, "002406": {"suoxie": "GDZGDJZQC", "name": "\u5149\u5927\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAZHONGGAODENGJIZHAIQUANC"}, "002407": {"suoxie": "QHKYHYLHPZHH", "name": "\u524d\u6d77\u5f00\u6e90\u6052\u8fdc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHENGYUANLINGHUOPEIZHIHUNHE"}, "002408": {"suoxie": "ZXJTYGHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u533b\u6539\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOUYIGAIHUNHEA"}, "002409": {"suoxie": "HXXHLHHA", "name": "\u534e\u590f\u65b0\u6d3b\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINHUOLIHUNHEA"}, "002410": {"suoxie": "HXXHLHHC", "name": "\u534e\u590f\u65b0\u6d3b\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINHUOLIHUNHEC"}, "002411": {"suoxie": "HXXJYHHA", "name": "\u534e\u590f\u65b0\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJIYUHUNHEA"}, "002412": {"suoxie": "HFAFZQ", "name": "\u534e\u5bcc\u5b89\u798f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANFUZHAIQUAN"}, "002413": {"suoxie": "ZYRLHHA", "name": "\u4e2d\u94f6\u745e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINRUILIHUNHEA"}, "002414": {"suoxie": "ZYRLHHC", "name": "\u4e2d\u94f6\u745e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINRUILIHUNHEC"}, "002415": {"suoxie": "RTTYLHPZHH", "name": "\u878d\u901a\u901a\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGTONGYINGLINGHUOPEIZHIHUNHE"}, "002416": {"suoxie": "ZSFLLHPZHHC", "name": "\u62db\u5546\u4e30\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGLILINGHUOPEIZHIHUNHEC"}, "002417": {"suoxie": "ZSFSWDZZHHC", "name": "\u62db\u5546\u4e30\u76db\u7a33\u5b9a\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGSHENGWENDINGZENGZHANGHUNHEC"}, "002418": {"suoxie": "HTFYXHBLHPZHHC", "name": "\u6c47\u6dfb\u5bcc\u4f18\u9009\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYOUXUANHUIBAOLINGHUOPEIZHIHUNHEC"}, "002419": {"suoxie": "HTFYALHPZHHA", "name": "\u6c47\u6dfb\u5bcc\u76c8\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGANLINGHUOPEIZHIHUNHEA"}, "002420": {"suoxie": "HTFYXHHA", "name": "\u6c47\u6dfb\u5bcc\u76c8\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGXINHUNHEA"}, "002421": {"suoxie": "XHZQZQA", "name": "\u65b0\u534e\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGQIANGZHAIQUANA"}, "002422": {"suoxie": "XHZQZQC", "name": "\u65b0\u534e\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGQIANGZHAIQUANC"}, "002423": {"suoxie": "HBBPMGXFMY", "name": "\u534e\u5b9d\u6807\u666e\u7f8e\u56fd\u6d88\u8d39\u7f8e\u5143", "style": "QDII", "pinyin": "HUABAOBIAOPUMEIGUOXIAOFEIMEIYUAN"}, "002424": {"suoxie": "BSWTYLZTHH", "name": "\u535a\u65f6\u6587\u4f53\u5a31\u4e50\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIWENTIYULEZHUTIHUNHE"}, "002425": {"suoxie": "JYYXHHC", "name": "\u91d1\u9e70\u5143\u79a7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGYUANXIHUNHEC"}, "002426": {"suoxie": "HAQQMYPXZRMBA", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u7968\u606f\u503a\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANPIAOXIZHAIRENMINBIA"}, "002427": {"suoxie": "HAQQMYPXZMYXHA", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u7968\u606f\u503a\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANPIAOXIZHAIMEIYUANXIANHUIA"}, "002429": {"suoxie": "HAQQMYPXZC", "name": "\u534e\u5b89\u5168\u7403\u7f8e\u5143\u7968\u606f\u503aC", "style": "QDII", "pinyin": "HUAANQUANQIUMEIYUANPIAOXIZHAIC"}, "002430": {"suoxie": "ZYFLHHA", "name": "\u4e2d\u94f6\u4e30\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINFENGLIHUNHEA"}, "002431": {"suoxie": "ZYFLHHC", "name": "\u4e2d\u94f6\u4e30\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINFENGLIHUNHEC"}, "002434": {"suoxie": "ZYHLHHA", "name": "\u4e2d\u94f6\u5b8f\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINHONGLIHUNHEA"}, "002435": {"suoxie": "ZYHLHHC", "name": "\u4e2d\u94f6\u5b8f\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINHONGLIHUNHEC"}, "002438": {"suoxie": "CJHXZSCZ", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u76db\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNSHENGCHUNZHAI"}, "002441": {"suoxie": "DBXTLZQC", "name": "\u5fb7\u90a6\u65b0\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGXINTIANLIZHAIQUANC"}, "002442": {"suoxie": "XYHLZQ", "name": "\u946b\u5143\u6c47\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHUILIZHAIQUAN"}, "002443": {"suoxie": "QHKYHGSLTJXHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u9f99\u5934\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLONGTOUJINGXUANHUNHE"}, "002445": {"suoxie": "XYFTZQ", "name": "\u5174\u4e1a\u4e30\u6cf0\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEFENGTAIZHAIQUAN"}, "002446": {"suoxie": "GFLXLHPZHHA", "name": "\u5e7f\u53d1\u5229\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFALIXINLINGHUOPEIZHIHUNHEA"}, "002447": {"suoxie": "BSYACZDKZFQS", "name": "\u535a\u65f6\u88d5\u5b89\u7eaf\u503a\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUANCHUNZHAIDINGKAIZHAIFAQISHI"}, "002448": {"suoxie": "JXHF", "name": "\u6c5f\u4fe1\u6c47\u798f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINHUIFU"}, "002449": {"suoxie": "MSJYLHZGHH", "name": "\u6c11\u751f\u52a0\u94f6\u91cf\u5316\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINLIANGHUAZHONGGUOHUNHE"}, "002450": {"suoxie": "PARXWYHHA", "name": "\u5e73\u5b89\u777f\u4eab\u6587\u5a31\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANRUIXIANGWENYUHUNHEA"}, "002451": {"suoxie": "PARXWYHHC", "name": "\u5e73\u5b89\u777f\u4eab\u6587\u5a31\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANRUIXIANGWENYUHUNHEC"}, "002452": {"suoxie": "MSJYHXDKZ", "name": "\u6c11\u751f\u52a0\u94f6\u548c\u946b\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHEXINDINGKAIZHAI"}, "002453": {"suoxie": "JTJWLHPZHHA", "name": "\u4e5d\u6cf0\u4e45\u7a33\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUWENLINGHUOPEIZHIHUNHEA"}, "002454": {"suoxie": "JTJWLHPZHHC", "name": "\u4e5d\u6cf0\u4e45\u7a33\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUWENLINGHUOPEIZHIHUNHEC"}, "002455": {"suoxie": "MSJYXXHH", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u559c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINXIHUNHE"}, "002456": {"suoxie": "ZSAYLHPZHHA", "name": "\u62db\u5546\u5b89\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANYUANLINGHUOPEIZHIHUNHEA"}, "002457": {"suoxie": "ZSAYLHPZHHC", "name": "\u62db\u5546\u5b89\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANYUANLINGHUOPEIZHIHUNHEC"}, "002458": {"suoxie": "GTMLCLSYHH", "name": "\u56fd\u6cf0\u6c11\u5229\u7b56\u7565\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINLICELUESHOUYIHUNHE"}, "002459": {"suoxie": "HXDLZQFQSA", "name": "\u534e\u590f\u9f0e\u5229\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGLIZHAIQUANFAQISHIA"}, "002460": {"suoxie": "HXDLZQFQSC", "name": "\u534e\u590f\u9f0e\u5229\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGLIZHAIQUANFAQISHIC"}, "002461": {"suoxie": "ZYZLHHA", "name": "\u4e2d\u94f6\u73cd\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENLIHUNHEA"}, "002462": {"suoxie": "ZYZLHHC", "name": "\u4e2d\u94f6\u73cd\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENLIHUNHEC"}, "002466": {"suoxie": "BSYXCZ", "name": "\u535a\u65f6\u88d5\u65b0\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUXINCHUNZHAI"}, "002469": {"suoxie": "HTBRJYHBB", "name": "\u534e\u6cf0\u67cf\u745e\u4ea4\u6613\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUIJIAOYIHUOBIB"}, "002472": {"suoxie": "GDXJFWYHHA", "name": "\u5149\u5927\u5148\u8fdb\u670d\u52a1\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAXIANJINFUWUYEHUNHEA"}, "002474": {"suoxie": "ZYRXZQZ", "name": "\u4e2d\u90ae\u777f\u4fe1\u589e\u5f3a\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOURUIXINZENGQIANGZHAI"}, "002475": {"suoxie": "ZYRLZQZQ", "name": "\u4e2d\u90ae\u777f\u5229\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOURUILIZENGQIANGZHAIQUAN"}, "002476": {"suoxie": "BSAR18GYDKZA", "name": "\u535a\u65f6\u5b89\u745e18\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANRUI18GEYUEDINGKAIZHAIA"}, "002477": {"suoxie": "BSAR18GYDKZC", "name": "\u535a\u65f6\u5b89\u745e18\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANRUI18GEYUEDINGKAIZHAIC"}, "002482": {"suoxie": "BYHLWHGSHH", "name": "\u5b9d\u76c8\u4e92\u8054\u7f51\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGHULIANWANGHUGANGSHENHUNHE"}, "002483": {"suoxie": "FGTLDKZFQS", "name": "\u5bcc\u56fd\u6cf0\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTAILIDINGKAIZHAIFAQISHI"}, "002485": {"suoxie": "GLATYHHC", "name": "\u56fd\u8054\u5b89\u901a\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANTONGYINGHUNHEC"}, "002486": {"suoxie": "SYHTLZQ", "name": "\u4e0a\u94f6\u6167\u6dfb\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUITIANLIZHAIQUAN"}, "002487": {"suoxie": "HTFWTLDKZA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u6dfb\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUWENTIANLIDINGKAIZHAIA"}, "002488": {"suoxie": "HTFWTLDKZC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u6dfb\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUWENTIANLIDINGKAIZHAIC"}, "002489": {"suoxie": "GTMFCLJZHH", "name": "\u56fd\u6cf0\u6c11\u798f\u7b56\u7565\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINFUCELUEJIAZHIHUNHE"}, "002490": {"suoxie": "JYYQXYZ", "name": "\u91d1\u9e70\u5143\u797a\u4fe1\u7528\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGYUANQIXINYONGZHAI"}, "002491": {"suoxie": "YHTYDKZ", "name": "\u94f6\u534e\u6dfb\u76ca\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUATIANYIDINGKAIZHAI"}, "002492": {"suoxie": "GYYYXDQZFZQC", "name": "\u5de5\u94f6\u6708\u6708\u85aa\u5b9a\u671f\u652f\u4ed8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINYUEYUEXINDINGQIZHIFUZHAIQUANC"}, "002494": {"suoxie": "XYJYHHA", "name": "\u5174\u4e1a\u805a\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUYINGHUNHEA"}, "002495": {"suoxie": "QHKYLHYXA", "name": "\u524d\u6d77\u5f00\u6e90\u91cf\u5316\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANLIANGHUAYOUXUANA"}, "002496": {"suoxie": "QHKYLHYXC", "name": "\u524d\u6d77\u5f00\u6e90\u91cf\u5316\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANLIANGHUAYOUXUANC"}, "002497": {"suoxie": "DFSSLHPZHHA", "name": "\u4e1c\u65b9\u76db\u4e16\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGSHENGSHILINGHUOPEIZHIHUNHEA"}, "002498": {"suoxie": "XYJXLHPZHHA", "name": "\u5174\u4e1a\u805a\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJUXINLINGHUOPEIZHIHUNHEA"}, "002501": {"suoxie": "YHYJZQ", "name": "\u94f6\u534e\u8fdc\u666f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHUAYUANJINGZHAIQUAN"}, "002502": {"suoxie": "ZYTLHHA", "name": "\u4e2d\u94f6\u817e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINTENGLIHUNHEA"}, "002503": {"suoxie": "ZYTLHHC", "name": "\u4e2d\u94f6\u817e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINTENGLIHUNHEC"}, "002504": {"suoxie": "PHJDLHPZHHA", "name": "\u9e4f\u534e\u91d1\u9f0e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAJINDINGLINGHUOPEIZHIHUNHEA"}, "002505": {"suoxie": "PHJDLHPZHHC", "name": "\u9e4f\u534e\u91d1\u9f0e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAJINDINGLINGHUOPEIZHIHUNHEC"}, "002507": {"suoxie": "XYDKZC", "name": "\u5174\u4e1a\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEDINGKAIZHAIC"}, "002510": {"suoxie": "SWLXZZ500ZSZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENG500ZHISHUZENGQIANGA"}, "002512": {"suoxie": "CCJRHH", "name": "\u957f\u57ce\u4e45\u6da6\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIURUNHUNHE"}, "002513": {"suoxie": "JYYAHHC", "name": "\u91d1\u9e70\u5143\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGYUANANHUNHEC"}, "002514": {"suoxie": "ZSFYHHA", "name": "\u62db\u5546\u4e30\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGYIHUNHEA"}, "002515": {"suoxie": "ZSFYHHC", "name": "\u62db\u5546\u4e30\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGYIHUNHEC"}, "002518": {"suoxie": "MSJYXFHHA", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINFUHUNHEA"}, "002519": {"suoxie": "BSYJCZ", "name": "\u535a\u65f6\u88d5\u666f\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUJINGCHUNZHAI"}, "002520": {"suoxie": "ZSZRCZFQSC", "name": "\u62db\u5546\u62db\u745e\u7eaf\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAORUICHUNZHAIFAQISHIC"}, "002521": {"suoxie": "YYSLZQA", "name": "\u6c38\u8d62\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGSHUANGLIZHAIQUANA"}, "002522": {"suoxie": "YYSLZQC", "name": "\u6c38\u8d62\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGSHUANGLIZHAIQUANC"}, "002523": {"suoxie": "GDHLCZ", "name": "\u5149\u5927\u6052\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAHENGLICHUNZHAI"}, "002524": {"suoxie": "XYFYZQ", "name": "\u5174\u4e1a\u798f\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEFUYIZHAIQUAN"}, "002528": {"suoxie": "TKAYCZA", "name": "\u6cf0\u5eb7\u5b89\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANYICHUNZHAIA"}, "002529": {"suoxie": "TKAYCZC", "name": "\u6cf0\u5eb7\u5b89\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANYICHUNZHAIC"}, "002533": {"suoxie": "ZJXXHHC", "name": "\u4e2d\u52a0\u5fc3\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAXINXIANGHUNHEC"}, "002534": {"suoxie": "HAWGSYZQA", "name": "\u534e\u5b89\u7a33\u56fa\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANWENGUSHOUYIZHAIQUANA"}, "002535": {"suoxie": "ZYXLHHA", "name": "\u4e2d\u94f6\u946b\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINLIHUNHEA"}, "002536": {"suoxie": "ZYXLHHC", "name": "\u4e2d\u94f6\u946b\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINLIHUNHEC"}, "002537": {"suoxie": "PAAYLHPZHHA", "name": "\u5e73\u5b89\u5b89\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANYINGLINGHUOPEIZHIHUNHEA"}, "002542": {"suoxie": "CCJDHH", "name": "\u957f\u57ce\u4e45\u9f0e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUDINGHUNHE"}, "002543": {"suoxie": "CCJYHHA", "name": "\u957f\u57ce\u4e45\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUYIHUNHEA"}, "002544": {"suoxie": "CCJYHHC", "name": "\u957f\u57ce\u4e45\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUYIHUNHEC"}, "002545": {"suoxie": "DFYLHPZHH", "name": "\u4e1c\u65b9\u5cb3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGYUELINGHUOPEIZHIHUNHE"}, "002546": {"suoxie": "TKXYBHBE", "name": "\u6cf0\u5eb7\u85aa\u610f\u4fdd\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXINYIBAOHUOBIE"}, "002547": {"suoxie": "MSJYYLFWHH", "name": "\u6c11\u751f\u52a0\u94f6\u517b\u8001\u670d\u52a1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINYANGLAOFUWUHUNHE"}, "002548": {"suoxie": "JSWRCZZQ", "name": "\u5609\u5b9e\u7a33\u745e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENRUICHUNZHAIZHAIQUAN"}, "002549": {"suoxie": "JSWXCZZQA", "name": "\u5609\u5b9e\u7a33\u7965\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENXIANGCHUNZHAIZHAIQUANA"}, "002550": {"suoxie": "JSWRZQ", "name": "\u5609\u5b9e\u7a33\u8363\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENRONGZHAIQUAN"}, "002552": {"suoxie": "HXHLDKZ", "name": "\u534e\u590f\u6052\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAHENGLIDINGKAIZHAI"}, "002553": {"suoxie": "BSCYCZHHC", "name": "\u535a\u65f6\u521b\u4e1a\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGYECHENGZHANGHUNHEC"}, "002555": {"suoxie": "BSHGSYZQYJJC", "name": "\u535a\u65f6\u6caa\u6e2f\u6df1\u4f18\u8d28\u4f01\u4e1a\u57fa\u91d1C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUGANGSHENYOUZHIQIYEJIJINC"}, "002556": {"suoxie": "BSSLZTGPC", "name": "\u535a\u65f6\u4e1d\u8def\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHISILUZHUTIGUPIAOC"}, "002558": {"suoxie": "BSXRHHA", "name": "\u535a\u65f6\u946b\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINRUIHUNHEA"}, "002559": {"suoxie": "BSXRHHC", "name": "\u535a\u65f6\u946b\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINRUIHUNHEC"}, "002560": {"suoxie": "NAHXLHPZHH", "name": "\u8bfa\u5b89\u548c\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANHEXINLINGHUOPEIZHIHUNHE"}, "002561": {"suoxie": "DWAXLHHH", "name": "\u4e1c\u5434\u5b89\u946b\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUANXINLIANGHUAHUNHE"}, "002562": {"suoxie": "HDHYLHHH", "name": "\u6cd3\u5fb7\u6cd3\u76ca\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEHONGYILIANGHUAHUNHE"}, "002563": {"suoxie": "HDHHHH", "name": "\u6cd3\u5fb7\u6cd3\u6c47\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGHUIHUNHE"}, "002564": {"suoxie": "XWTYLHPZHH", "name": "\u65b0\u6c83\u901a\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINWOTONGYINGLINGHUOPEIZHIHUNHE"}, "002567": {"suoxie": "DCGJAQZTHH", "name": "\u5927\u6210\u56fd\u5bb6\u5b89\u5168\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGGUOJIAANQUANZHUTIHUNHE"}, "002568": {"suoxie": "BSYFCZ", "name": "\u535a\u65f6\u88d5\u53d1\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUFACHUNZHAI"}, "002569": {"suoxie": "BSYHCZZQ", "name": "\u535a\u65f6\u88d5\u5f18\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUHONGCHUNZHAIZHAIQUAN"}, "002573": {"suoxie": "JXHLLHPZHH", "name": "\u5efa\u4fe1\u6c47\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINHUILILINGHUOPEIZHIHUNHE"}, "002574": {"suoxie": "ZSRQHHA", "name": "\u62db\u5546\u745e\u5e86\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGRUIQINGHUNHEA"}, "002577": {"suoxie": "NFXXHH", "name": "\u5357\u65b9\u65b0\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGXINXINGHUNHE"}, "002578": {"suoxie": "BSYQCZZQ", "name": "\u535a\u65f6\u88d5\u6cc9\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUQUANCHUNZHAIZHAIQUAN"}, "002580": {"suoxie": "TXXXLHPZHHC", "name": "\u6cf0\u4fe1\u946b\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINXINXUANLINGHUOPEIZHIHUNHEC"}, "002581": {"suoxie": "ZSFKHHA", "name": "\u62db\u5546\u4e30\u51ef\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGKAIHUNHEA"}, "002582": {"suoxie": "ZSFKHHC", "name": "\u62db\u5546\u4e30\u51ef\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGKAIHUNHEC"}, "002583": {"suoxie": "TXHYJXHHC", "name": "\u6cf0\u4fe1\u884c\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINHANGYEJINGXUANHUNHEC"}, "002584": {"suoxie": "FADCYLHPZHH", "name": "\u5bcc\u5b89\u8fbe\u957f\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDACHANGYINGLINGHUOPEIZHIHUNHE"}, "002585": {"suoxie": "JXXLLHPZHH", "name": "\u5efa\u4fe1\u5174\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINGLILINGHUOPEIZHIHUNHE"}, "002586": {"suoxie": "JYTLXYZZQA", "name": "\u91d1\u9e70\u6dfb\u5229\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANLIXINYONGZHAIZHAIQUANA"}, "002587": {"suoxie": "JYTLXYZZQC", "name": "\u91d1\u9e70\u6dfb\u5229\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANLIXINYONGZHAIZHAIQUANC"}, "002591": {"suoxie": "ZOXYZLZQLOFE", "name": "\u4e2d\u6b27\u4fe1\u7528\u589e\u5229\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINYONGZENGLIZHAIQUANLOFE"}, "002592": {"suoxie": "ZOCZZQLOFE", "name": "\u4e2d\u6b27\u7eaf\u503a\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUCHUNZHAIZHAIQUANLOFE"}, "002593": {"suoxie": "FGMLZGHHA", "name": "\u5bcc\u56fd\u7f8e\u4e3d\u4e2d\u56fd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOMEILIZHONGGUOHUNHEA"}, "002594": {"suoxie": "GYXDFWYHH", "name": "\u5de5\u94f6\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXIANDAIFUWUYEHUNHE"}, "002595": {"suoxie": "BSGY40ZTGP", "name": "\u535a\u65f6\u5de5\u4e1a4.0\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIGONGYE40ZHUTIGUPIAO"}, "002597": {"suoxie": "XYCZDLHH", "name": "\u5174\u4e1a\u6210\u957f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYECHENGZHANGDONGLIHUNHE"}, "002598": {"suoxie": "PAXFJXHHA", "name": "\u5e73\u5b89\u6d88\u8d39\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANXIAOFEIJINGXUANHUNHEA"}, "002599": {"suoxie": "PAXFJXHHC", "name": "\u5e73\u5b89\u6d88\u8d39\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANXIAOFEIJINGXUANHUNHEC"}, "002600": {"suoxie": "YFDYJTL6GYDKZ", "name": "\u6613\u65b9\u8fbe\u88d5\u666f\u6dfb\u52296\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAYUJINGTIANLI6GEYUEDINGKAIZHAI"}, "002601": {"suoxie": "ZYZQJZJXHH", "name": "\u4e2d\u94f6\u8bc1\u5238\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANJIAZHIJINGXUANHUNHE"}, "002602": {"suoxie": "YFDFHHH", "name": "\u6613\u65b9\u8fbe\u4e30\u60e0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAFENGHUIHUNHE"}, "002603": {"suoxie": "GYRFBNDKZFQS", "name": "\u5de5\u94f6\u745e\u4e30\u534a\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIFENGBANNIANDINGKAIZHAIFAQISHI"}, "002604": {"suoxie": "HXXQDHHA", "name": "\u534e\u590f\u65b0\u8d77\u70b9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINQIDIANHUNHEA"}, "002605": {"suoxie": "RTXXFLHPZHH", "name": "\u878d\u901a\u65b0\u6d88\u8d39\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINXIAOFEILINGHUOPEIZHIHUNHE"}, "002606": {"suoxie": "RTXXFLHPZHH", "name": "\u878d\u901a\u65b0\u6d88\u8d39\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINXIAOFEILINGHUOPEIZHIHUNHE"}, "002610": {"suoxie": "BSHJETFLJA", "name": "\u535a\u65f6\u9ec4\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "BOSHIHUANGJINETFLIANJIEA"}, "002611": {"suoxie": "BSHJETFLJC", "name": "\u535a\u65f6\u9ec4\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "BOSHIHUANGJINETFLIANJIEC"}, "002612": {"suoxie": "RTTHHHAB", "name": "\u878d\u901a\u901a\u6167\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGTONGHUIHUNHEAB"}, "002613": {"suoxie": "RTTHHHAB", "name": "\u878d\u901a\u901a\u6167\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGTONGHUIHUNHEAB"}, "002614": {"suoxie": "ZYYLHHA", "name": "\u4e2d\u94f6\u9890\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIHUNHEA"}, "002615": {"suoxie": "ZYYLHHC", "name": "\u4e2d\u94f6\u9890\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIHUNHEC"}, "002616": {"suoxie": "ZYYLHHA", "name": "\u4e2d\u94f6\u76ca\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIHUNHEA"}, "002617": {"suoxie": "ZYYLHHC", "name": "\u4e2d\u94f6\u76ca\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIHUNHEC"}, "002618": {"suoxie": "ZYYLHHA", "name": "\u4e2d\u94f6\u88d5\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYULIHUNHEA"}, "002619": {"suoxie": "ZYYLHHC", "name": "\u4e2d\u94f6\u88d5\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYULIHUNHEC"}, "002620": {"suoxie": "ZYWLXLCHH", "name": "\u4e2d\u90ae\u672a\u6765\u65b0\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUWEILAIXINLANCHOUHUNHE"}, "002621": {"suoxie": "ZOXFZTGPA", "name": "\u4e2d\u6b27\u6d88\u8d39\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUXIAOFEIZHUTIGUPIAOA"}, "002622": {"suoxie": "GFWYHHA", "name": "\u5e7f\u53d1\u7a33\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAWENYUHUNHEA"}, "002624": {"suoxie": "GFYQJXHHA", "name": "\u5e7f\u53d1\u4f18\u4f01\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAYOUQIJINGXUANHUNHEA"}, "002625": {"suoxie": "BSAY6GY", "name": "\u535a\u65f6\u5b89\u60216\u4e2a\u6708", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANYI6GEYUE"}, "002628": {"suoxie": "ZSABLHPZHHA", "name": "\u62db\u5546\u5b89\u535a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANBOLINGHUOPEIZHIHUNHEA"}, "002629": {"suoxie": "ZSABLHPZHHC", "name": "\u62db\u5546\u5b89\u535a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANBOLINGHUOPEIZHIHUNHEC"}, "002630": {"suoxie": "JXRFLHPZHHA", "name": "\u6c5f\u4fe1\u745e\u798f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANGXINRUIFULINGHUOPEIZHIHUNHEA"}, "002631": {"suoxie": "JXRFLHPZHHC", "name": "\u6c5f\u4fe1\u745e\u798f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANGXINRUIFULINGHUOPEIZHIHUNHEC"}, "002632": {"suoxie": "XYSZZQZQA", "name": "\u946b\u5143\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANSHUANGZHAIZENGQIANGZHAIQUANA"}, "002633": {"suoxie": "XYSZZQZQC", "name": "\u946b\u5143\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINYUANSHUANGZHAIZENGQIANGZHAIQUANC"}, "002634": {"suoxie": "HBWLZDHHA", "name": "\u534e\u5b9d\u672a\u6765\u4e3b\u5bfc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOWEILAIZHUDAOHUNHEA"}, "002635": {"suoxie": "RTZXZQ", "name": "\u878d\u901a\u589e\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGXINZHAIQUAN"}, "002636": {"suoxie": "GFJYZQA", "name": "\u5e7f\u53d1\u96c6\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUZHAIQUANA"}, "002637": {"suoxie": "GFJYZQC", "name": "\u5e7f\u53d1\u96c6\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUZHAIQUANC"}, "002638": {"suoxie": "XYTRZQ", "name": "\u5174\u4e1a\u5929\u878d\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYETIANRONGZHAIQUAN"}, "002639": {"suoxie": "THJZJXLHPZHH", "name": "\u5929\u5f18\u4ef7\u503c\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGJIAZHIJINGXUANLINGHUOPEIZHIHUNHE"}, "002640": {"suoxie": "ZXJTRYHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u6ea2\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOURUIYIHUNHEA"}, "002644": {"suoxie": "DCJRZQA", "name": "\u5927\u6210\u666f\u8363\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGRONGZHAIQUANA"}, "002645": {"suoxie": "DCJRZQC", "name": "\u5927\u6210\u666f\u8363\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGRONGZHAIQUANC"}, "002646": {"suoxie": "ZKWTHBA", "name": "\u4e2d\u79d1\u6c83\u571f\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGKEWOTUHUOBIA"}, "002647": {"suoxie": "ZKWTHBB", "name": "\u4e2d\u79d1\u6c83\u571f\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGKEWOTUHUOBIB"}, "002649": {"suoxie": "MSZZ2025LHPZHH", "name": "\u6c11\u751f\u667a\u90202025\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGZHIZAO2025LINGHUOPEIZHIHUNHE"}, "002650": {"suoxie": "DFHWTLCZA", "name": "\u4e1c\u65b9\u7ea2\u7a33\u6dfb\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGWENTIANLICHUNZHAIA"}, "002651": {"suoxie": "DFHHLZQA", "name": "\u4e1c\u65b9\u7ea2\u6c47\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGHUILIZHAIQUANA"}, "002652": {"suoxie": "DFHHLZQC", "name": "\u4e1c\u65b9\u7ea2\u6c47\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGHUILIZHAIQUANC"}, "002653": {"suoxie": "TKHGSJXHH", "name": "\u6cf0\u5eb7\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGHUGANGSHENJINGXUANHUNHE"}, "002656": {"suoxie": "NFCYBETFLJA", "name": "\u5357\u65b9\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGCHUANGYEBANETFLIANJIEA"}, "002657": {"suoxie": "ZSAYLHPZHHA", "name": "\u62db\u5546\u5b89\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANYULINGHUOPEIZHIHUNHEA"}, "002658": {"suoxie": "ZSAYLHPZHHC", "name": "\u62db\u5546\u5b89\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANYULINGHUOPEIZHIHUNHEC"}, "002659": {"suoxie": "XYZZ13NZJZA", "name": "\u5174\u4e1a\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHAI13NIANZHENGJINZHAIA"}, "002660": {"suoxie": "XYJYHHA", "name": "\u5174\u4e1a\u805a\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUYUANHUNHEA"}, "002661": {"suoxie": "XYTXZQ", "name": "\u5174\u4e1a\u5929\u79a7\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYETIANXIZHAIQUAN"}, "002662": {"suoxie": "QHKYHGSDXFHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u5927\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENDAXIAOFEIHUNHEA"}, "002663": {"suoxie": "QHKYHGSDXFHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u5927\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENDAXIAOFEIHUNHEC"}, "002664": {"suoxie": "WJRHLHPZHHA", "name": "\u4e07\u5bb6\u745e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIHELINGHUOPEIZHIHUNHEA"}, "002665": {"suoxie": "WJRHLHPZHHC", "name": "\u4e07\u5bb6\u745e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIHELINGHUOPEIZHIHUNHEC"}, "002666": {"suoxie": "QHKYHGSCXCZHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENCHUANGXINCHENGZHANGHUNHEA"}, "002667": {"suoxie": "QHKYHGSCXCZHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENCHUANGXINCHENGZHANGHUNHEC"}, "002668": {"suoxie": "XYJFHHA", "name": "\u5174\u4e1a\u805a\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUFENGHUNHEA"}, "002669": {"suoxie": "HSWZCXHH", "name": "\u534e\u5546\u4e07\u4f17\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGWANZHONGCHUANGXINHUNHE"}, "002670": {"suoxie": "WJHS300ZSZQA", "name": "\u4e07\u5bb6\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAHUSHEN300ZHISHUZENGQIANGA"}, "002671": {"suoxie": "WJHS300ZSZQC", "name": "\u4e07\u5bb6\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAHUSHEN300ZHISHUZENGQIANGC"}, "002672": {"suoxie": "NDHBA", "name": "\u8bfa\u5fb7\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NUODEHUOBIA"}, "002673": {"suoxie": "NDHBB", "name": "\u8bfa\u5fb7\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NUODEHUOBIB"}, "002679": {"suoxie": "GYAYHBA", "name": "\u5de5\u94f6\u5b89\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINANYINGHUOBIA"}, "002680": {"suoxie": "GYAYHBB", "name": "\u5de5\u94f6\u5b89\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINANYINGHUOBIB"}, "002681": {"suoxie": "JYYHLHPZHHA", "name": "\u91d1\u9e70\u5143\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGYUANHELINGHUOPEIZHIHUNHEA"}, "002682": {"suoxie": "JYYHLHPZHHC", "name": "\u91d1\u9e70\u5143\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGYUANHELINGHUOPEIZHIHUNHEC"}, "002683": {"suoxie": "MSJYQYKJLHPZHH", "name": "\u6c11\u751f\u52a0\u94f6\u524d\u6cbf\u79d1\u6280\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINQIANYANKEJILINGHUOPEIZHIHUNHE"}, "002684": {"suoxie": "MSJYXACZA", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u5b89\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINANCHUNZHAIA"}, "002685": {"suoxie": "ZOFHHGSHHA", "name": "\u4e2d\u6b27\u4e30\u6cd3\u6caa\u6e2f\u6df1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUFENGHONGHUGANGSHENHUNHEA"}, "002686": {"suoxie": "ZOFHHGSHHC", "name": "\u4e2d\u6b27\u4e30\u6cd3\u6caa\u6e2f\u6df1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUFENGHONGHUGANGSHENHUNHEC"}, "002688": {"suoxie": "HTHTCYDKZA", "name": "\u7ea2\u5854\u7ea2\u571f\u957f\u76ca\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTAHONGTUCHANGYIDINGKAIZHAIA"}, "002689": {"suoxie": "HTHTCYDKZC", "name": "\u7ea2\u5854\u7ea2\u571f\u957f\u76ca\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTAHONGTUCHANGYIDINGKAIZHAIC"}, "002690": {"suoxie": "QHKYHZHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6052\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANHENGZEHUNHEA"}, "002691": {"suoxie": "QHKYHZHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6052\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANHENGZEHUNHEC"}, "002692": {"suoxie": "FGCXKJHHA", "name": "\u5bcc\u56fd\u521b\u65b0\u79d1\u6280\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHUANGXINKEJIHUNHEA"}, "002697": {"suoxie": "ZOXFZTGPC", "name": "\u4e2d\u6b27\u6d88\u8d39\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUXIAOFEIZHUTIGUPIAOC"}, "002698": {"suoxie": "BSYLCZZQ", "name": "\u535a\u65f6\u88d5\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYULICHUNZHAIZHAIQUAN"}, "002701": {"suoxie": "DFHHYZQA", "name": "\u4e1c\u65b9\u7ea2\u6c47\u9633\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGHUIYANGZHAIQUANA"}, "002702": {"suoxie": "DFHHYZQC", "name": "\u4e1c\u65b9\u7ea2\u6c47\u9633\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGHUIYANGZHAIQUANC"}, "002703": {"suoxie": "CCJYLHPZHHA", "name": "\u957f\u57ce\u4e45\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUYUANLINGHUOPEIZHIHUNHEA"}, "002704": {"suoxie": "DBRXZQA", "name": "\u5fb7\u90a6\u9510\u5174\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIXINGZHAIQUANA"}, "002705": {"suoxie": "DBRXZQC", "name": "\u5fb7\u90a6\u9510\u5174\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIXINGZHAIQUANC"}, "002707": {"suoxie": "DMKJLXHHA", "name": "\u5927\u6469\u79d1\u6280\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DAMOKEJILINGXIANHUNHEA"}, "002708": {"suoxie": "DMJKCYHHA", "name": "\u5927\u6469\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOJIANKANGCHANYEHUNHEA"}, "002709": {"suoxie": "HTHTRRBHBA", "name": "\u7ea2\u5854\u7ea2\u571f\u4eba\u4eba\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTAHONGTURENRENBAOHUOBIA"}, "002710": {"suoxie": "HTHTRRBHBB", "name": "\u7ea2\u5854\u7ea2\u571f\u4eba\u4eba\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTAHONGTURENRENBAOHUOBIB"}, "002711": {"suoxie": "GFJFZQA", "name": "\u5e7f\u53d1\u96c6\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIFENGZHAIQUANA"}, "002712": {"suoxie": "GFJFZQC", "name": "\u5e7f\u53d1\u96c6\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIFENGZHAIQUANC"}, "002714": {"suoxie": "PHJCHH", "name": "\u9e4f\u534e\u91d1\u57ce\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAJINCHENGHUNHE"}, "002716": {"suoxie": "BSYTDKZA", "name": "\u535a\u65f6\u88d5\u901a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUTONGDINGKAIZHAIA"}, "002717": {"suoxie": "HTHTSLLHPZA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u9686\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGLONGLINGHUOPEIZHIA"}, "002718": {"suoxie": "HTHTSLLHPZC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u9686\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGLONGLINGHUOPEIZHIC"}, "002719": {"suoxie": "RTZXSGYDKZ", "name": "\u878d\u901a\u589e\u7965\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGZENGXIANGSANGEYUEDINGKAIZHAI"}, "002720": {"suoxie": "GSABZLZQHBZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5229\u589e\u5f3a\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOSHOUANBAOZUNLIZENGQIANGHUIBAOZHAIQUANA"}, "002721": {"suoxie": "GSABZLZQHBZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5229\u589e\u5f3a\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOSHOUANBAOZUNLIZENGQIANGHUIBAOZHAIQUANC"}, "002722": {"suoxie": "GYCFHBB", "name": "\u5de5\u94f6\u8d22\u5bcc\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINCAIFUHUOBIB"}, "002723": {"suoxie": "JXQFA", "name": "\u6c5f\u4fe1\u797a\u798fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANGXINQIFUA"}, "002724": {"suoxie": "JXQFC", "name": "\u6c5f\u4fe1\u797a\u798fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANGXINQIFUC"}, "002728": {"suoxie": "HFYXLHPZHHA", "name": "\u534e\u5bcc\u76ca\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUYIXINLINGHUOPEIZHIHUNHEA"}, "002729": {"suoxie": "HFYXLHPZHHC", "name": "\u534e\u5bcc\u76ca\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUYIXINLINGHUOPEIZHIHUNHEC"}, "002730": {"suoxie": "HFHXLHPZHHA", "name": "\u534e\u5bcc\u534e\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUHUAXINLINGHUOPEIZHIHUNHEA"}, "002731": {"suoxie": "HFHXLHPZHHC", "name": "\u534e\u5bcc\u534e\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUHUAXINLINGHUOPEIZHIHUNHEC"}, "002732": {"suoxie": "CSHGSHH", "name": "\u957f\u76db\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGHUGANGSHENHUNHE"}, "002733": {"suoxie": "SYHYLHB", "name": "\u4e0a\u94f6\u6167\u76c8\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGYINHUIYINGLIHUOBI"}, "002734": {"suoxie": "HDYRCZZQA", "name": "\u6cd3\u5fb7\u88d5\u8363\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYURONGCHUNZHAIZHAIQUANA"}, "002735": {"suoxie": "HDYRCZZQC", "name": "\u6cd3\u5fb7\u88d5\u8363\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYURONGCHUNZHAIZHAIQUANC"}, "002736": {"suoxie": "HDYHCZZQA", "name": "\u6cd3\u5fb7\u88d5\u548c\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUHECHUNZHAIZHAIQUANA"}, "002737": {"suoxie": "HDYHCZZQC", "name": "\u6cd3\u5fb7\u88d5\u548c\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUHECHUNZHAIZHAIQUANC"}, "002738": {"suoxie": "HDYKZQA", "name": "\u6cd3\u5fb7\u88d5\u5eb7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUKANGZHAIQUANA"}, "002739": {"suoxie": "HDYKZQC", "name": "\u6cd3\u5fb7\u88d5\u5eb7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUKANGZHAIQUANC"}, "002740": {"suoxie": "HDYZYNDKZQA", "name": "\u6cd3\u5fb7\u88d5\u6cfd\u4e00\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUZEYINIANDINGKAIZHAIQUANA"}, "002741": {"suoxie": "HDYZYNDKZQC", "name": "\u6cd3\u5fb7\u88d5\u6cfd\u4e00\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUZEYINIANDINGKAIZHAIQUANC"}, "002742": {"suoxie": "HDYXZQA", "name": "\u6cd3\u5fb7\u88d5\u7965\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUXIANGZHAIQUANA"}, "002743": {"suoxie": "HDYXZQC", "name": "\u6cd3\u5fb7\u88d5\u7965\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGDEYUXIANGZHAIQUANC"}, "002745": {"suoxie": "BXRFFLHH", "name": "\u5317\u4fe1\u745e\u4e30\u4e30\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BEIXINRUIFENGFENGLIHUNHE"}, "002746": {"suoxie": "HTFDCLDKHH", "name": "\u6c47\u6dfb\u5bcc\u591a\u7b56\u7565\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUDUOCELUEDINGKAIHUNHE"}, "002747": {"suoxie": "ZOHBC", "name": "\u4e2d\u6b27\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUHUOBIC"}, "002748": {"suoxie": "ZOHBD", "name": "\u4e2d\u6b27\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUHUOBID"}, "002749": {"suoxie": "JSWSZQ", "name": "\u5609\u5b9e\u7a33\u76db\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENSHENGZHAIQUAN"}, "002750": {"suoxie": "GYTXSNLCZQ", "name": "\u5de5\u94f6\u6cf0\u4eab\u4e09\u5e74\u7406\u8d22\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINTAIXIANGSANNIANLICAIZHAIQUAN"}, "002753": {"suoxie": "JXJXBHBB", "name": "\u5efa\u4fe1\u5609\u85aa\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINJIAXINBAOHUOBIB"}, "002754": {"suoxie": "BSYCCZ", "name": "\u535a\u65f6\u88d5\u521b\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUCHUANGCHUNZHAI"}, "002755": {"suoxie": "BSYSCZZQ", "name": "\u535a\u65f6\u88d5\u76db\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUSHENGCHUNZHAIZHAIQUAN"}, "002756": {"suoxie": "ZSZX3GYDKA", "name": "\u62db\u5546\u62db\u51743\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXING3GEYUEDINGKAIA"}, "002757": {"suoxie": "ZSZX3GYDKC", "name": "\u62db\u5546\u62db\u51743\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXING3GEYUEDINGKAIC"}, "002758": {"suoxie": "JXXJZLHBA", "name": "\u5efa\u4fe1\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINZENGLIHUOBIA"}, "002759": {"suoxie": "DXAYBA", "name": "\u4e1c\u5174\u5b89\u76c8\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGXINGANYINGBAOA"}, "002760": {"suoxie": "DXAYBB", "name": "\u4e1c\u5174\u5b89\u76c8\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGXINGANYINGBAOB"}, "002765": {"suoxie": "XHSLZQA", "name": "\u65b0\u534e\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUASHUANGLIZHAIQUANA"}, "002766": {"suoxie": "XHSLZQC", "name": "\u65b0\u534e\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUASHUANGLIZHAIQUANC"}, "002767": {"suoxie": "TKHTHBHH", "name": "\u6cf0\u5eb7\u5b8f\u6cf0\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGHONGTAIHUIBAOHUNHE"}, "002768": {"suoxie": "HAAJLHPZHH", "name": "\u534e\u5b89\u5b89\u8fdb\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANJINLINGHUOPEIZHIHUNHE"}, "002769": {"suoxie": "XYDZZQC", "name": "\u5174\u4e1a\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYEDUANZHAIZHAIQUANC"}, "002770": {"suoxie": "AXXHBHHA", "name": "\u5b89\u4fe1\u65b0\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINHUIBAOHUNHEA"}, "002771": {"suoxie": "AXXHBHHC", "name": "\u5b89\u4fe1\u65b0\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINHUIBAOHUNHEC"}, "002772": {"suoxie": "GDCYXDLHH", "name": "\u5149\u5927\u4ea7\u4e1a\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDACHANYEXINDONGLIHUNHE"}, "002773": {"suoxie": "GDMXHHA", "name": "\u5149\u5927\u94ed\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAMINGXINHUNHEA"}, "002774": {"suoxie": "GDMXHHC", "name": "\u5149\u5927\u94ed\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAMINGXINHUNHEC"}, "002775": {"suoxie": "BSJXCZZQ", "name": "\u535a\u65f6\u666f\u5174\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJINGXINGCHUNZHAIZHAIQUAN"}, "002776": {"suoxie": "ZSARHHA", "name": "\u62db\u5546\u5b89\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANRONGHUNHEA"}, "002777": {"suoxie": "ZSARHHC", "name": "\u62db\u5546\u5b89\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANRONGHUNHEC"}, "002778": {"suoxie": "XJQHLHXSLHHA", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u65b0\u601d\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINJIANGQIANHAILIANHEXINSILUHUNHEA"}, "002779": {"suoxie": "XJQHLHXSLHHC", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u65b0\u601d\u8def\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINJIANGQIANHAILIANHEXINSILUHUNHEC"}, "002780": {"suoxie": "QHLHHXHHA", "name": "\u524d\u6d77\u8054\u5408\u6cd3\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEHONGXINHUNHEA"}, "002781": {"suoxie": "BSJR6GYDKZ", "name": "\u535a\u65f6\u805a\u745e6\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJURUI6GEYUEDINGKAIZHAI"}, "002782": {"suoxie": "FGXLDKZFQS", "name": "\u5bcc\u56fd\u7965\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOXIANGLIDINGKAIZHAIFAQISHI"}, "002783": {"suoxie": "DFHJZJXHHA", "name": "\u4e1c\u65b9\u7ea2\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJIAZHIJINGXUANHUNHEA"}, "002784": {"suoxie": "DFHJZJXHHC", "name": "\u4e1c\u65b9\u7ea2\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJIAZHIJINGXUANHUNHEC"}, "002785": {"suoxie": "ZRRYSLZQA", "name": "\u4e2d\u878d\u878d\u88d5\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGRONGRONGYUSHUANGLIZHAIQUANA"}, "002786": {"suoxie": "ZRRYSLZQC", "name": "\u4e2d\u878d\u878d\u88d5\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGRONGRONGYUSHUANGLIZHAIQUANC"}, "002788": {"suoxie": "RTXJBHBA", "name": "\u878d\u901a\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGXIANJINBAOHUOBIA"}, "002789": {"suoxie": "CSTXLHPZA", "name": "\u957f\u76db\u540c\u4eab\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGTONGXIANGLINGHUOPEIZHIA"}, "002790": {"suoxie": "CSTXLHPZC", "name": "\u957f\u76db\u540c\u4eab\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGTONGXIANGLINGHUOPEIZHIC"}, "002792": {"suoxie": "JSCCSYHBHHA", "name": "\u666f\u987a\u957f\u57ce\u987a\u76ca\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGSHUNYIHUIBAOHUNHEA"}, "002793": {"suoxie": "JSCCSYHBHHC", "name": "\u666f\u987a\u957f\u57ce\u987a\u76ca\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGSHUNYIHUIBAOHUNHEC"}, "002794": {"suoxie": "THYLZQE", "name": "\u5929\u5f18\u6c38\u5229\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGYONGLIZHAIQUANE"}, "002795": {"suoxie": "PAHYCZA", "name": "\u5e73\u5b89\u60e0\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIYINGCHUNZHAIA"}, "002796": {"suoxie": "JSCCJYSLZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u76c8\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYINGSHUANGLIZHAIQUANA"}, "002797": {"suoxie": "JSCCJYSLZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u76c8\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYINGSHUANGLIZHAIQUANC"}, "002801": {"suoxie": "HDHXHH", "name": "\u6cd3\u5fb7\u6cd3\u4fe1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGXINHUNHE"}, "002802": {"suoxie": "GFDCDSJJXHHA", "name": "\u5e7f\u53d1\u4e1c\u8d22\u5927\u6570\u636e\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFADONGCAIDASHUJUJINGXUANHUNHEA"}, "002803": {"suoxie": "DFHHGSHH", "name": "\u4e1c\u65b9\u7ea2\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGHUGANGSHENHUNHE"}, "002804": {"suoxie": "HTBRLHDC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u5bf9\u51b2", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUATAIBAIRUILIANGHUADUICHONG"}, "002805": {"suoxie": "ZSHJJLYNDKZA", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u5229\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGHUIJINJULIYINIANDINGKAIZHAIA"}, "002806": {"suoxie": "ZSHJJLYNDKZC", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u5229\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGHUIJINJULIYINIANDINGKAIZHAIC"}, "002807": {"suoxie": "RTTAZQ", "name": "\u878d\u901a\u901a\u5b89\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGANZHAIQUAN"}, "002808": {"suoxie": "HDYSLHHH", "name": "\u6cd3\u5fb7\u4f18\u52bf\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEYOUSHILINGHANGHUNHE"}, "002810": {"suoxie": "JXZXCXCZHHFQS", "name": "\u91d1\u4fe1\u8f6c\u578b\u521b\u65b0\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINZHUANXINGCHUANGXINCHENGZHANGHUNHEFAQISHI"}, "002811": {"suoxie": "BSYSCZZQ", "name": "\u535a\u65f6\u88d5\u987a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUSHUNCHUNZHAIZHAIQUAN"}, "002812": {"suoxie": "BSYTDKZC", "name": "\u535a\u65f6\u88d5\u901a\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUTONGDINGKAIZHAIC"}, "002813": {"suoxie": "BSYTHHA", "name": "\u535a\u65f6\u9890\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIYITAIHUNHEA"}, "002814": {"suoxie": "BSYTHHC", "name": "\u535a\u65f6\u9890\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIYITAIHUNHEC"}, "002817": {"suoxie": "ZSZHCZA", "name": "\u62db\u5546\u62db\u6052\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHENGCHUNZHAIA"}, "002818": {"suoxie": "ZSZHCZC", "name": "\u62db\u5546\u62db\u6052\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHENGCHUNZHAIC"}, "002819": {"suoxie": "ZSFMHHA", "name": "\u62db\u5546\u4e30\u7f8e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGMEIHUNHEA"}, "002820": {"suoxie": "ZSFMHHC", "name": "\u62db\u5546\u4e30\u7f8e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGMEIHUNHEC"}, "002825": {"suoxie": "RTTHZQ", "name": "\u878d\u901a\u901a\u548c\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGHEZHAIQUAN"}, "002826": {"suoxie": "ZYYLBNDKZ", "name": "\u4e2d\u94f6\u6c38\u5229\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINYONGLIBANNIANDINGKAIZHAI"}, "002828": {"suoxie": "NFXJZLHBE", "name": "\u5357\u65b9\u73b0\u91d1\u589e\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINZENGLIHUOBIE"}, "002829": {"suoxie": "NFXJZLHBF", "name": "\u5357\u65b9\u73b0\u91d1\u589e\u5229\u8d27\u5e01F", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINZENGLIHUOBIF"}, "002830": {"suoxie": "ZSHFDKZ", "name": "\u6d59\u5546\u60e0\u4e30\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIFENGDINGKAIZHAI"}, "002832": {"suoxie": "GYHXCZZQ", "name": "\u5de5\u94f6\u6052\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINHENGXIANGCHUNZHAIZHAIQUAN"}, "002833": {"suoxie": "HXXJXHHA", "name": "\u534e\u590f\u65b0\u9526\u7ee3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINXIUHUNHEA"}, "002834": {"suoxie": "HXXJXHHC", "name": "\u534e\u590f\u65b0\u9526\u7ee3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINXIUHUNHEC"}, "002837": {"suoxie": "HXWGJXHHA", "name": "\u534e\u590f\u7f51\u8d2d\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAWANGGOUJINGXUANHUNHEA"}, "002838": {"suoxie": "HXXJCHHA", "name": "\u534e\u590f\u65b0\u9526\u7a0b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINCHENGHUNHEA"}, "002839": {"suoxie": "HXXJCHHC", "name": "\u534e\u590f\u65b0\u9526\u7a0b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINCHENGHUNHEC"}, "002844": {"suoxie": "JYDYCLHH", "name": "\u91d1\u9e70\u591a\u5143\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGDUOYUANCELUEHUNHE"}, "002846": {"suoxie": "HDHHHH", "name": "\u6cd3\u5fb7\u6cd3\u534e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEHONGHUAHUNHE"}, "002847": {"suoxie": "THXJGJHBE", "name": "\u5929\u5f18\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGXIANJINGUANJIAHUOBIE"}, "002849": {"suoxie": "JXZNZG2025HH", "name": "\u91d1\u4fe1\u667a\u80fd\u4e2d\u56fd2025\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINZHINENGZHONGGUO2025HUNHE"}, "002851": {"suoxie": "NFPZYXLHPZHHA", "name": "\u5357\u65b9\u54c1\u8d28\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGPINZHIYOUXUANLINGHUOPEIZHIHUNHEA"}, "002852": {"suoxie": "ZSCFBJYXHBA", "name": "\u62db\u5546\u8d22\u5bcc\u5b9d\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGCAIFUBAOJIAOYIXINGHUOBIA"}, "002855": {"suoxie": "BSXJBHBC", "name": "\u535a\u65f6\u73b0\u91d1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINBAOHUOBIC"}, "002858": {"suoxie": "CXFPCZYNDKZA", "name": "\u957f\u4fe1\u5bcc\u5e73\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUPINGCHUNZHAIYINIANDINGKAIZHAIA"}, "002859": {"suoxie": "CXFPCZYNDKZC", "name": "\u957f\u4fe1\u5bcc\u5e73\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUPINGCHUNZHAIYINIANDINGKAIZHAIC"}, "002860": {"suoxie": "QHKYHGSXJYHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u65b0\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENXINJIYUHUNHE"}, "002861": {"suoxie": "GYZNZZGP", "name": "\u5de5\u94f6\u667a\u80fd\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINZHINENGZHIZAOGUPIAO"}, "002862": {"suoxie": "JXLHJXHH", "name": "\u91d1\u4fe1\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINLIANGHUAJINGXUANHUNHE"}, "002863": {"suoxie": "JXSZCZHHFQS", "name": "\u91d1\u4fe1\u6df1\u5733\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINSHENZHENCHENGZHANGHUNHEFAQISHI"}, "002864": {"suoxie": "GFAZDZZQA", "name": "\u5e7f\u53d1\u5b89\u6cfd\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAANZEDUANZHAIZHAIQUANA"}, "002865": {"suoxie": "GFAZDZZQC", "name": "\u5e7f\u53d1\u5b89\u6cfd\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAANZEDUANZHAIZHAIQUANC"}, "002866": {"suoxie": "XHFYHBZQ", "name": "\u65b0\u534e\u4e30\u76c8\u56de\u62a5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAFENGYINGHUIBAOZHAIQUAN"}, "002868": {"suoxie": "PHFMZQ", "name": "\u9e4f\u534e\u4e30\u8302\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGMAOZHAIQUAN"}, "002869": {"suoxie": "RTTYDKZ", "name": "\u878d\u901a\u901a\u88d5\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGYUDINGKAIZHAI"}, "002870": {"suoxie": "XYZYWNDKZ", "name": "\u5174\u4e1a\u589e\u76ca\u4e94\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZENGYIWUNIANDINGKAIZHAI"}, "002871": {"suoxie": "HXZSJZCZA", "name": "\u534e\u590f\u667a\u80dc\u4ef7\u503c\u6210\u957fA", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAZHISHENGJIAZHICHENGZHANGA"}, "002872": {"suoxie": "HXZSJZCZC", "name": "\u534e\u590f\u667a\u80dc\u4ef7\u503c\u6210\u957fC", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAZHISHENGJIAZHICHENGZHANGC"}, "002877": {"suoxie": "HXDZHXYZA", "name": "\u534e\u590f\u5927\u4e2d\u534e\u4fe1\u7528\u503aA", "style": "QDII", "pinyin": "HUAXIADAZHONGHUAXINYONGZHAIA"}, "002878": {"suoxie": "HXDZHXYZMYXHA", "name": "\u534e\u590f\u5927\u4e2d\u534e\u4fe1\u7528\u503a\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "HUAXIADAZHONGHUAXINYONGZHAIMEIYUANXIANHUIA"}, "002879": {"suoxie": "HXDZHXYZMYXCA", "name": "\u534e\u590f\u5927\u4e2d\u534e\u4fe1\u7528\u503a\u7f8e\u5143\u73b0\u949eA", "style": "QDII", "pinyin": "HUAXIADAZHONGHUAXINYONGZHAIMEIYUANXIANCHAOA"}, "002880": {"suoxie": "HXDZHXYZC", "name": "\u534e\u590f\u5927\u4e2d\u534e\u4fe1\u7528\u503aC", "style": "QDII", "pinyin": "HUAXIADAZHONGHUAXINYONGZHAIC"}, "002881": {"suoxie": "ZJFRCZZQA", "name": "\u4e2d\u52a0\u4e30\u6da6\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGRUNCHUNZHAIZHAIQUANA"}, "002882": {"suoxie": "ZJFRCZZQC", "name": "\u4e2d\u52a0\u4e30\u6da6\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGRUNCHUNZHAIZHAIQUANC"}, "002883": {"suoxie": "HRYDXJTHBA", "name": "\u534e\u6da6\u5143\u5927\u73b0\u91d1\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARUNYUANDAXIANJINTONGHUOBIA"}, "002884": {"suoxie": "HRYDXJTHBB", "name": "\u534e\u6da6\u5143\u5927\u73b0\u91d1\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUARUNYUANDAXIANJINTONGHUOBIB"}, "002885": {"suoxie": "DMWZCXHHA", "name": "\u5927\u6469\u4e07\u4f17\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DAMOWANZHONGCHUANGXINHUNHEA"}, "002889": {"suoxie": "JYTLBHBA", "name": "\u4ea4\u94f6\u5929\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANLIBAOHUOBIA"}, "002890": {"suoxie": "JYTLBHBE", "name": "\u4ea4\u94f6\u5929\u5229\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANLIBAOHUOBIE"}, "002891": {"suoxie": "HXYDHLHHRMB", "name": "\u534e\u590f\u79fb\u52a8\u4e92\u8054\u6df7\u5408\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "HUAXIAYIDONGHULIANHUNHERENMINBI"}, "002892": {"suoxie": "HXYDHLHHMYXH", "name": "\u534e\u590f\u79fb\u52a8\u4e92\u8054\u6df7\u5408\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "HUAXIAYIDONGHULIANHUNHEMEIYUANXIANHUI"}, "002893": {"suoxie": "HXYDHLHHMYXC", "name": "\u534e\u590f\u79fb\u52a8\u4e92\u8054\u6df7\u5408\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "HUAXIAYIDONGHULIANHUNHEMEIYUANXIANCHAO"}, "002894": {"suoxie": "HXTLHBA", "name": "\u534e\u590f\u5929\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIATIANLIHUOBIA"}, "002895": {"suoxie": "HXTLHBB", "name": "\u534e\u590f\u5929\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIATIANLIHUOBIB"}, "002898": {"suoxie": "FGLNQLCZQA", "name": "\u5bcc\u56fd\u4e24\u5e74\u671f\u7406\u8d22\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOLIANGNIANQILICAIZHAIQUANA"}, "002899": {"suoxie": "FGLNQLCZQC", "name": "\u5bcc\u56fd\u4e24\u5e74\u671f\u7406\u8d22\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOLIANGNIANQILICAIZHAIQUANC"}, "002900": {"suoxie": "NFZZ500XXJSLJA", "name": "\u5357\u65b9\u4e2d\u8bc1500\u4fe1\u606f\u6280\u672f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500XINXIJISHULIANJIEA"}, "002901": {"suoxie": "CTZGJJSYZQA", "name": "\u8d22\u901a\u8d44\u7ba1\u79ef\u6781\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGZIGUANJIJISHOUYIZHAIQUANA"}, "002902": {"suoxie": "CTZGJJSYZQC", "name": "\u8d22\u901a\u8d44\u7ba1\u79ef\u6781\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGZIGUANJIJISHOUYIZHAIQUANC"}, "002903": {"suoxie": "GFZZ500ETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ETFLIANJIEC"}, "002904": {"suoxie": "BSARYNDKFQSZQA", "name": "\u535a\u65f6\u5b89\u4ec1\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANRENYINIANDINGKAIFAQISHIZHAIQUANA"}, "002905": {"suoxie": "BSARYNDKFQSZQC", "name": "\u535a\u65f6\u5b89\u4ec1\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANRENYINIANDINGKAIFAQISHIZHAIQUANC"}, "002906": {"suoxie": "NFZZ500LHZQA", "name": "\u5357\u65b9\u4e2d\u8bc1500\u91cf\u5316\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500LIANGHUAZENGQIANGA"}, "002907": {"suoxie": "NFZZ500LHZQC", "name": "\u5357\u65b9\u4e2d\u8bc1500\u91cf\u5316\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500LIANGHUAZENGQIANGC"}, "002908": {"suoxie": "FGRLDKHHFQS", "name": "\u5bcc\u56fd\u777f\u5229\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUORUILIDINGKAIHUNHEFAQISHI"}, "002909": {"suoxie": "ZSHXCZ", "name": "\u6d59\u5546\u60e0\u4eab\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIXIANGCHUNZHAI"}, "002910": {"suoxie": "YFDGGGGHH", "name": "\u6613\u65b9\u8fbe\u4f9b\u7ed9\u6539\u9769\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAGONGGEIGAIGEHUNHE"}, "002912": {"suoxie": "XYWTYHBA", "name": "\u5174\u4e1a\u7a33\u5929\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEWENTIANYINGHUOBIA"}, "002915": {"suoxie": "XYYLZQ", "name": "\u946b\u5143\u88d5\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANYULIZHAIQUAN"}, "002917": {"suoxie": "JSHQBHBE", "name": "\u5609\u5b9e\u6d3b\u94b1\u5305\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOQIANBAOHUOBIE"}, "002918": {"suoxie": "JYXJBHBE", "name": "\u4ea4\u94f6\u73b0\u91d1\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINXIANJINBAOHUOBIE"}, "002919": {"suoxie": "DWZHYLLHHHA", "name": "\u4e1c\u5434\u667a\u6167\u533b\u7597\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUZHIHUIYILIAOLIANGHUAHUNHEA"}, "002920": {"suoxie": "ZODZZQA", "name": "\u4e2d\u6b27\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGOUDUANZHAIZHAIQUANA"}, "002923": {"suoxie": "XYJHLHPZHHC", "name": "\u5174\u4e1a\u805a\u60e0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJUHUILINGHUOPEIZHIHUNHEC"}, "002924": {"suoxie": "HSRXDKZ", "name": "\u534e\u5546\u745e\u946b\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGRUIXINDINGKAIZHAI"}, "002925": {"suoxie": "GFJYZQA", "name": "\u5e7f\u53d1\u96c6\u6e90\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUANZHAIQUANA"}, "002926": {"suoxie": "GFJYZQC", "name": "\u5e7f\u53d1\u96c6\u6e90\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUANZHAIQUANC"}, "002927": {"suoxie": "CSSHCZA", "name": "\u957f\u76db\u76db\u548c\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGHECHUNZHAIA"}, "002928": {"suoxie": "CSSHCZC", "name": "\u957f\u76db\u76db\u548c\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGHECHUNZHAIC"}, "002929": {"suoxie": "BSJYCZZQ", "name": "\u535a\u65f6\u805a\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJUYINGCHUNZHAIZHAIQUAN"}, "002930": {"suoxie": "BSJRCZZQ", "name": "\u535a\u65f6\u805a\u6da6\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJURUNCHUNZHAIZHAIQUAN"}, "002932": {"suoxie": "YXYFQHSYA", "name": "\u5706\u4fe1\u6c38\u4e30\u5f3a\u5316\u6536\u76caA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YUANXINYONGFENGQIANGHUASHOUYIA"}, "002933": {"suoxie": "YXYFQHSYC", "name": "\u5706\u4fe1\u6c38\u4e30\u5f3a\u5316\u6536\u76caC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YUANXINYONGFENGQIANGHUASHOUYIC"}, "002934": {"suoxie": "TKHTHBHHA", "name": "\u6cf0\u5eb7\u6052\u6cf0\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGHENGTAIHUIBAOHUNHEA"}, "002935": {"suoxie": "TKHTHBHHC", "name": "\u6cf0\u5eb7\u6052\u6cf0\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGHENGTAIHUIBAOHUNHEC"}, "002936": {"suoxie": "HXWLHBA", "name": "\u534e\u590f\u6c83\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAWOLIHUOBIA"}, "002937": {"suoxie": "HXWLHBB", "name": "\u534e\u590f\u6c83\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAWOLIHUOBIB"}, "002938": {"suoxie": "ZYZQJKCYHH", "name": "\u4e2d\u94f6\u8bc1\u5238\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANJIANKANGCHANYEHUNHE"}, "002939": {"suoxie": "GFCXSJHH", "name": "\u5e7f\u53d1\u521b\u65b0\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFACHUANGXINSHENGJIHUNHE"}, "002943": {"suoxie": "GFDYZHH", "name": "\u5e7f\u53d1\u591a\u56e0\u5b50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFADUOYINZIHUNHE"}, "002945": {"suoxie": "DCSSJXLHPZHH", "name": "\u5927\u6210\u76db\u4e16\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGSHENGSHIJINGXUANLINGHUOPEIZHIHUNHE"}, "002946": {"suoxie": "DCJSYNDKZA", "name": "\u5927\u6210\u666f\u76db\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGSHENGYINIANDINGKAIZHAIA"}, "002947": {"suoxie": "DCJSYNDKZC", "name": "\u5927\u6210\u666f\u76db\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGJINGSHENGYINIANDINGKAIZHAIC"}, "002952": {"suoxie": "JXDYZLHGP", "name": "\u5efa\u4fe1\u591a\u56e0\u5b50\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINDUOYINZILIANGHUAGUPIAO"}, "002955": {"suoxie": "RTXQSLHPZHH", "name": "\u878d\u901a\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINQUSHILINGHUOPEIZHIHUNHE"}, "002956": {"suoxie": "RTXQSLHPZHH", "name": "\u878d\u901a\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINQUSHILINGHUOPEIZHIHUNHE"}, "002957": {"suoxie": "CTCTBHBA", "name": "\u8d22\u901a\u8d22\u901a\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CAITONGCAITONGBAOHUOBIA"}, "002958": {"suoxie": "CTCTBHBB", "name": "\u8d22\u901a\u8d22\u901a\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CAITONGCAITONGBAOHUOBIB"}, "002959": {"suoxie": "HTFYTHH", "name": "\u6c47\u6dfb\u5bcc\u76c8\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGTAIHUNHE"}, "002960": {"suoxie": "BSHLHBB", "name": "\u535a\u65f6\u5408\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHELIHUOBIB"}, "002961": {"suoxie": "ZOSLZQA", "name": "\u4e2d\u6b27\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUSHUANGLIZHAIQUANA"}, "002962": {"suoxie": "ZOSLZQC", "name": "\u4e2d\u6b27\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUSHUANGLIZHAIQUANC"}, "002963": {"suoxie": "YFDHJETFLJC", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "YIFANGDAHUANGJINETFLIANJIEC"}, "002964": {"suoxie": "GTRYSXDK", "name": "\u56fd\u6295\u745e\u94f6\u987a\u946b\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNXINDINGKAI"}, "002965": {"suoxie": "ZHHJZQSYZQA", "name": "\u4e2d\u6d77\u5408\u5609\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHAIHEJIAZENGQIANGSHOUYIZHAIQUANA"}, "002966": {"suoxie": "ZHHJZQSYZQC", "name": "\u4e2d\u6d77\u5408\u5609\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHAIHEJIAZENGQIANGSHOUYIZHAIQUANC"}, "002967": {"suoxie": "ZSDSJZXXFHH", "name": "\u6d59\u5546\u5927\u6570\u636e\u667a\u9009\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGDASHUJUZHIXUANXIAOFEIHUNHE"}, "002969": {"suoxie": "YFDFHZQ", "name": "\u6613\u65b9\u8fbe\u4e30\u548c\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAFENGHEZHAIQUAN"}, "002970": {"suoxie": "BSYACZZQ", "name": "\u535a\u65f6\u88d5\u6602\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUANGCHUNZHAIZHAIQUAN"}, "002971": {"suoxie": "QHKYDAZQA", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u5b89\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGANZHAIQUANA"}, "002972": {"suoxie": "QHKYDAZQC", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u5b89\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGANZHAIQUANC"}, "002974": {"suoxie": "GFXXJSLJC", "name": "\u5e7f\u53d1\u4fe1\u606f\u6280\u672f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAXINXIJISHULIANJIEC"}, "002977": {"suoxie": "GFZZQZKXXFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u53ef\u9009\u6d88\u8d39\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIKEXUANXIAOFEILIANJIEC"}, "002978": {"suoxie": "GFYYWSLJC", "name": "\u5e7f\u53d1\u533b\u836f\u536b\u751f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYIYAOWEISHENGLIANJIEC"}, "002979": {"suoxie": "GFZZQZJRDCLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u91d1\u878d\u5730\u4ea7\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJINRONGDICHANLIANJIEC"}, "002980": {"suoxie": "HXCXQYGP", "name": "\u534e\u590f\u521b\u65b0\u524d\u6cbf\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIACHUANGXINQIANYANGUPIAO"}, "002982": {"suoxie": "GFYLZSC", "name": "\u5e7f\u53d1\u517b\u8001\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYANGLAOZHISHUC"}, "002983": {"suoxie": "CXGFJGLHHHA", "name": "\u957f\u4fe1\u56fd\u9632\u519b\u5de5\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINGUOFANGJUNGONGLIANGHUAHUNHEA"}, "002984": {"suoxie": "GFZZHBETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u73af\u4fddETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGHUANBAOETFLIANJIEC"}, "002985": {"suoxie": "ZYJJHDKZ", "name": "\u4e2d\u94f6\u5b63\u5b63\u7ea2\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINJIJIHONGDINGKAIZHAI"}, "002986": {"suoxie": "TKFYZQ", "name": "\u6cf0\u5eb7\u4e30\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIKANGFENGYINGZHAIQUAN"}, "002987": {"suoxie": "GFHS300ETFLJC", "name": "\u5e7f\u53d1\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ETFLIANJIEC"}, "002988": {"suoxie": "PADXZQ", "name": "\u5e73\u5b89\u9f0e\u4fe1\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANDINGXINZHAIQUAN"}, "002989": {"suoxie": "RTTQYJJXHH", "name": "\u878d\u901a\u901a\u4e7e\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGTONGQIANYANJIUJINGXUANHUNHE"}, "002991": {"suoxie": "JSWXCZZQ", "name": "\u5609\u5b9e\u7a33\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENXINCHUNZHAIZHAIQUAN"}, "002994": {"suoxie": "ZSZYCZA", "name": "\u62db\u5546\u62db\u88d5\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYUCHUNZHAIA"}, "002995": {"suoxie": "ZSZYCZC", "name": "\u62db\u5546\u62db\u88d5\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYUCHUNZHAIC"}, "002996": {"suoxie": "CXWJCZZQA", "name": "\u957f\u4fe1\u7a33\u5065\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENJIANCHUNZHAIZHAIQUANA"}, "002997": {"suoxie": "GYRXCZZQ", "name": "\u5de5\u94f6\u745e\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIXIANGCHUNZHAIZHAIQUAN"}, "003002": {"suoxie": "GJJDZDZZQ", "name": "\u56fd\u91d1\u53ca\u7b2c\u4e2d\u77ed\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINJIDIZHONGDUANZHAIZHAIQUAN"}, "003003": {"suoxie": "HXXJZLHBAE", "name": "\u534e\u590f\u73b0\u91d1\u589e\u5229\u8d27\u5e01A/E", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAXIANJINZENGLIHUOBIAE"}, "003004": {"suoxie": "ZSRXDKHH", "name": "\u62db\u5546\u777f\u7965\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIXIANGDINGKAIHUNHE"}, "003009": {"suoxie": "ZRYZZDZA", "name": "\u4e2d\u878d\u76c8\u6cfd\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGRONGYINGZEZHONGDUANZHAIA"}, "003010": {"suoxie": "ZRYZZDZC", "name": "\u4e2d\u878d\u76c8\u6cfd\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGRONGYINGZEZHONGDUANZHAIC"}, "003013": {"suoxie": "ZRHTCZA", "name": "\u4e2d\u878d\u6052\u6cf0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGTAICHUNZHAIA"}, "003014": {"suoxie": "ZRHTCZC", "name": "\u4e2d\u878d\u6052\u6cf0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGTAICHUNZHAIC"}, "003015": {"suoxie": "ZJHS300A", "name": "\u4e2d\u91d1\u6caa\u6df1300A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINHUSHEN300A"}, "003016": {"suoxie": "ZJZZ500A", "name": "\u4e2d\u91d1\u4e2d\u8bc1500A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINZHONGZHENG500A"}, "003017": {"suoxie": "GFZZJGETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u519b\u5de5ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJUNGONGETFLIANJIEA"}, "003022": {"suoxie": "JXXJTYHBA", "name": "\u5efa\u4fe1\u73b0\u91d1\u6dfb\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINTIANYIHUOBIA"}, "003023": {"suoxie": "BSJFCZZQ", "name": "\u535a\u65f6\u666f\u53d1\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJINGFACHUNZHAIZHAIQUAN"}, "003024": {"suoxie": "PAHJDKZA", "name": "\u5e73\u5b89\u60e0\u91d1\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIJINDINGKAIZHAIA"}, "003025": {"suoxie": "XHHLHBHH", "name": "\u65b0\u534e\u7ea2\u5229\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAHONGLIHUIBAOHUNHE"}, "003026": {"suoxie": "AXXJZHHA", "name": "\u5b89\u4fe1\u65b0\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINJIAZHIHUNHEA"}, "003027": {"suoxie": "AXXJZHHC", "name": "\u5b89\u4fe1\u65b0\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINJIAZHIHUNHEC"}, "003028": {"suoxie": "AXXYXHHA", "name": "\u5b89\u4fe1\u65b0\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINYOUXUANHUNHEA"}, "003029": {"suoxie": "AXXYXHHC", "name": "\u5b89\u4fe1\u65b0\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINYOUXUANHUNHEC"}, "003030": {"suoxie": "AXXMBHHA", "name": "\u5b89\u4fe1\u65b0\u76ee\u6807\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINMUBIAOHUNHEA"}, "003031": {"suoxie": "AXXMBHHC", "name": "\u5b89\u4fe1\u65b0\u76ee\u6807\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINMUBIAOHUNHEC"}, "003032": {"suoxie": "PAYLJKHH", "name": "\u5e73\u5b89\u533b\u7597\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANYILIAOJIANKANGHUNHE"}, "003034": {"suoxie": "PAJYXHBA", "name": "\u5e73\u5b89\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANJIAOYIXINGHUOBIA"}, "003037": {"suoxie": "GFJRZQA", "name": "\u5e7f\u53d1\u96c6\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIRUIZHAIQUANA"}, "003038": {"suoxie": "GFJRZQC", "name": "\u5e7f\u53d1\u96c6\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIRUIZHAIQUANC"}, "003039": {"suoxie": "GFJFCZA", "name": "\u5e7f\u53d1\u96c6\u5bcc\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJIFUCHUNZHAIA"}, "003040": {"suoxie": "GFJFCZC", "name": "\u5e7f\u53d1\u96c6\u5bcc\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJIFUCHUNZHAIC"}, "003041": {"suoxie": "XYDLZQ", "name": "\u946b\u5143\u5f97\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANDELIZHAIQUAN"}, "003042": {"suoxie": "JYHQTHBA", "name": "\u4ea4\u94f6\u6d3b\u671f\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINHUOQITONGHUOBIA"}, "003043": {"suoxie": "JYHQTHBE", "name": "\u4ea4\u94f6\u6d3b\u671f\u901a\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINHUOQITONGHUOBIE"}, "003044": {"suoxie": "DFHZLJXHHA", "name": "\u4e1c\u65b9\u7ea2\u6218\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHANLUEJINGXUANHUNHEA"}, "003045": {"suoxie": "DFHZLJXHHC", "name": "\u4e1c\u65b9\u7ea2\u6218\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHANLUEJINGXUANHUNHEC"}, "003050": {"suoxie": "NYHLJFYNDKZ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u4e30\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINFENGYINIANDINGKAIZHAI"}, "003051": {"suoxie": "NYHLJLYNDKZ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINLIYINIANDINGKAIZHAI"}, "003053": {"suoxie": "JSWTYLGPA", "name": "\u5609\u5b9e\u6587\u4f53\u5a31\u4e50\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIWENTIYULEGUPIAOA"}, "003054": {"suoxie": "JSWTYLGPC", "name": "\u5609\u5b9e\u6587\u4f53\u5a31\u4e50\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIWENTIYULEGUPIAOC"}, "003056": {"suoxie": "JSWZCZZQ", "name": "\u5609\u5b9e\u7a33\u6cfd\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENZECHUNZHAIZHAIQUAN"}, "003059": {"suoxie": "CXXLBNDKHHA", "name": "\u957f\u4fe1\u5148\u5229\u534a\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINXIANLIBANNIANDINGKAIHUNHEA"}, "003062": {"suoxie": "YHTLHHA", "name": "\u94f6\u534e\u901a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUATONGLIHUNHEA"}, "003063": {"suoxie": "YHTLHHC", "name": "\u94f6\u534e\u901a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUATONGLIHUNHEC"}, "003065": {"suoxie": "GDZF18GYDKZA", "name": "\u5149\u5927\u5c0a\u5bcc18\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNFU18GEYUEDINGKAIZHAIA"}, "003066": {"suoxie": "GDZF18GYDKZC", "name": "\u5149\u5927\u5c0a\u5bcc18\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNFU18GEYUEDINGKAIZHAIC"}, "003069": {"suoxie": "GDCYBLHYXA", "name": "\u5149\u5927\u521b\u4e1a\u677f\u91cf\u5316\u4f18\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDACHUANGYEBANLIANGHUAYOUXUANA"}, "003070": {"suoxie": "GDCYBLHYXC", "name": "\u5149\u5927\u521b\u4e1a\u677f\u91cf\u5316\u4f18\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDACHUANGYEBANLIANGHUAYOUXUANC"}, "003071": {"suoxie": "ZRRXCZA", "name": "\u4e2d\u878d\u777f\u7965\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIXIANGCHUNZHAIA"}, "003072": {"suoxie": "ZRRXCZC", "name": "\u4e2d\u878d\u777f\u7965\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIXIANGCHUNZHAIC"}, "003073": {"suoxie": "TDHLHLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6c47\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIHUILIZHAIQUANA"}, "003074": {"suoxie": "TDHLHLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6c47\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIHUILIZHAIQUANC"}, "003075": {"suoxie": "ZRHBE", "name": "\u4e2d\u878d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGHUOBIE"}, "003078": {"suoxie": "TKAHCZZQA", "name": "\u6cf0\u5eb7\u5b89\u60e0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANHUICHUNZHAIZHAIQUANA"}, "003081": {"suoxie": "ZR13NZGDJXYZA", "name": "\u4e2d\u878d1-3\u5e74\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONG13NIANZHONGGAODENGJIXINYONGZHAIA"}, "003082": {"suoxie": "ZR13NZGDJXYZC", "name": "\u4e2d\u878d1-3\u5e74\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONG13NIANZHONGGAODENGJIXINYONGZHAIC"}, "003092": {"suoxie": "HSFLZQDKZA", "name": "\u534e\u5546\u4e30\u5229\u589e\u5f3a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGFENGLIZENGQIANGDINGKAIZHAIA"}, "003093": {"suoxie": "HSFLZQDKZC", "name": "\u534e\u5546\u4e30\u5229\u589e\u5f3a\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGFENGLIZENGQIANGDINGKAIZHAIC"}, "003095": {"suoxie": "ZOYLJKHHA", "name": "\u4e2d\u6b27\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYILIAOJIANKANGHUNHEA"}, "003096": {"suoxie": "ZOYLJKHHC", "name": "\u4e2d\u6b27\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYILIAOJIANKANGHUNHEC"}, "003102": {"suoxie": "CSSYCZA", "name": "\u957f\u76db\u76db\u88d5\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGYUCHUNZHAIA"}, "003103": {"suoxie": "CSSYCZC", "name": "\u957f\u76db\u76db\u88d5\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGYUCHUNZHAIC"}, "003105": {"suoxie": "GDYXHHA", "name": "\u5149\u5927\u6c38\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAYONGXINHUNHEA"}, "003106": {"suoxie": "GDYXHHC", "name": "\u5149\u5927\u6c38\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAYONGXINHUNHEC"}, "003107": {"suoxie": "GDAQZQA", "name": "\u5149\u5927\u5b89\u797a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANQIZHAIQUANA"}, "003108": {"suoxie": "GDAQZQC", "name": "\u5149\u5927\u5b89\u797a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANQIZHAIQUANC"}, "003109": {"suoxie": "GDAHZQA", "name": "\u5149\u5927\u5b89\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANHEZHAIQUANA"}, "003110": {"suoxie": "GDAHZQC", "name": "\u5149\u5927\u5b89\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANHEZHAIQUANC"}, "003115": {"suoxie": "GDCXHHA", "name": "\u5149\u5927\u8bda\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDACHENGXINHUNHEA"}, "003116": {"suoxie": "GDCXHHC", "name": "\u5149\u5927\u8bda\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDACHENGXINHUNHEC"}, "003117": {"suoxie": "GDJXHHA", "name": "\u5149\u5927\u5409\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAJIXINHUNHEA"}, "003118": {"suoxie": "GDJXHHC", "name": "\u5149\u5927\u5409\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAJIXINHUNHEC"}, "003119": {"suoxie": "BSXYHHA", "name": "\u535a\u65f6\u946b\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINYUANHUNHEA"}, "003120": {"suoxie": "BSXYHHC", "name": "\u535a\u65f6\u946b\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINYUANHUNHEC"}, "003121": {"suoxie": "ZXBCWLZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENLIZHAIQUANA"}, "003123": {"suoxie": "TZXLCZZQA", "name": "\u5929\u6cbb\u946b\u5229\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANZHIXINLICHUNZHAIZHAIQUANA"}, "003124": {"suoxie": "TZXLCZZQC", "name": "\u5929\u6cbb\u946b\u5229\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANZHIXINLICHUNZHAIZHAIQUANC"}, "003125": {"suoxie": "ZKWTWXCZJXHHA", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u946b\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWOXINCHENGZHANGJINGXUANHUNHEA"}, "003126": {"suoxie": "CXYJHHA", "name": "\u957f\u4fe1\u6613\u8fdb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINYIJINHUNHEA"}, "003127": {"suoxie": "CXYJHHC", "name": "\u957f\u4fe1\u6613\u8fdb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINYIJINHUNHEC"}, "003130": {"suoxie": "ZXBCWLZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENLIZHAIQUANC"}, "003131": {"suoxie": "GSABQGZZHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u5f3a\u56fd\u667a\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOQIANGGUOZHIZAOHUNHE"}, "003132": {"suoxie": "DBXHBLHPZHH", "name": "\u5fb7\u90a6\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGXINHUIBAOLINGHUOPEIZHIHUNHE"}, "003133": {"suoxie": "YFDYXZQA", "name": "\u6613\u65b9\u8fbe\u88d5\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUXINZHAIQUANA"}, "003134": {"suoxie": "YFDYXZQC", "name": "\u6613\u65b9\u8fbe\u88d5\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUXINZHAIQUANC"}, "003135": {"suoxie": "JYSAFYZQ", "name": "\u91d1\u5143\u987a\u5b89\u6ca3\u6979\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYUANSHUNANFENGYINGZHAIQUAN"}, "003142": {"suoxie": "PHHDHHA", "name": "\u9e4f\u534e\u5f18\u8fbe\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGDAHUNHEA"}, "003143": {"suoxie": "PHHDHHC", "name": "\u9e4f\u534e\u5f18\u8fbe\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGDAHUNHEC"}, "003144": {"suoxie": "HBXJYHHLOFC", "name": "\u534e\u5b9d\u65b0\u673a\u9047\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINJIYUHUNHELOFC"}, "003145": {"suoxie": "ZRJZYS", "name": "\u4e2d\u878d\u7ade\u4e89\u4f18\u52bf", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGRONGJINGZHENGYOUSHI"}, "003146": {"suoxie": "RTTYZQ", "name": "\u878d\u901a\u901a\u4f18\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGYOUZHAIQUAN"}, "003147": {"suoxie": "DCDTLHHH", "name": "\u5927\u6210\u52a8\u6001\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGDONGTAILIANGHUAHUNHE"}, "003152": {"suoxie": "HFTXLHPZHHA", "name": "\u534e\u5bcc\u5929\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUTIANXINLINGHUOPEIZHIHUNHEA"}, "003153": {"suoxie": "HFTXLHPZHHC", "name": "\u534e\u5bcc\u5929\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUTIANXINLINGHUOPEIZHIHUNHEC"}, "003154": {"suoxie": "HBXHLHH", "name": "\u534e\u5b9d\u65b0\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINHUOLIHUNHE"}, "003155": {"suoxie": "ZJFSCZZQ", "name": "\u4e2d\u52a0\u4e30\u5c1a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGSHANGCHUNZHAIZHAIQUAN"}, "003156": {"suoxie": "ZSZYCZA", "name": "\u62db\u5546\u62db\u60a6\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYUECHUNZHAIA"}, "003157": {"suoxie": "ZSZYCZC", "name": "\u62db\u5546\u62db\u60a6\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYUECHUNZHAIC"}, "003159": {"suoxie": "WJHR18GYDKZA", "name": "\u4e07\u5bb6\u6052\u745e18\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAHENGRUI18GEYUEDINGKAIZHAIA"}, "003160": {"suoxie": "WJHR18GYDKZC", "name": "\u4e07\u5bb6\u6052\u745e18\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAHENGRUI18GEYUEDINGKAIZHAIC"}, "003161": {"suoxie": "NFATHHA", "name": "\u5357\u65b9\u5b89\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANTAIHUNHEA"}, "003162": {"suoxie": "BSFNCZZQ", "name": "\u535a\u65f6\u5bcc\u5b81\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUNINGCHUNZHAIZHAIQUAN"}, "003163": {"suoxie": "JYTY3GYDKZ", "name": "\u91d1\u9e70\u6dfb\u76ca3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANYI3GEYUEDINGKAIZHAI"}, "003164": {"suoxie": "JXXJTLHBB", "name": "\u5efa\u4fe1\u73b0\u91d1\u6dfb\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINTIANLIHUOBIB"}, "003165": {"suoxie": "PHHJHHAL", "name": "\u9e4f\u534e\u5f18\u5609\u6df7\u5408A\u7c7b", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGJIAHUNHEALEI"}, "003166": {"suoxie": "PHHJHHCL", "name": "\u9e4f\u534e\u5f18\u5609\u6df7\u5408C\u7c7b", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGJIAHUNHECLEI"}, "003167": {"suoxie": "QHKYDRZQA", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGRUIZHAIQUANA"}, "003168": {"suoxie": "QHKYDRZQC", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGRUIZHAIQUANC"}, "003169": {"suoxie": "CSSHHHA", "name": "\u957f\u76db\u76db\u8f89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGSHENGHUIHUNHEA"}, "003170": {"suoxie": "CSSHHHC", "name": "\u957f\u76db\u76db\u8f89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGSHENGHUIHUNHEC"}, "003171": {"suoxie": "XDAYHLCHB", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7406\u8d22\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUILICAIHUOBI"}, "003173": {"suoxie": "MSJYXACZC", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u5b89\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINANCHUNZHAIC"}, "003174": {"suoxie": "QHLHTHCZA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u60e0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANHUICHUNZHAIA"}, "003175": {"suoxie": "HTBRDCLHH", "name": "\u534e\u6cf0\u67cf\u745e\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIDUOCELUEHUNHE"}, "003176": {"suoxie": "DBJYZQA", "name": "\u5fb7\u90a6\u666f\u9890\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGJINGYIZHAIQUANA"}, "003177": {"suoxie": "DBJYZQC", "name": "\u5fb7\u90a6\u666f\u9890\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGJINGYIZHAIQUANC"}, "003179": {"suoxie": "SXYLDKZ", "name": "\u5c71\u897f\u88d5\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANXIYULIDINGKAIZHAI"}, "003180": {"suoxie": "QHLHTLZQA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAILIANHETIANLIZHAIQUANA"}, "003181": {"suoxie": "QHLHTLZQC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAILIANHETIANLIZHAIQUANC"}, "003182": {"suoxie": "HFHXHHA", "name": "\u534e\u5bcc\u5f18\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUHONGXINHUNHEA"}, "003183": {"suoxie": "HFHXHHC", "name": "\u534e\u5bcc\u5f18\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUHONGXINHUNHEC"}, "003184": {"suoxie": "CTZZESG100ZSZQC", "name": "\u8d22\u901a\u4e2d\u8bc1ESG100\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZHONGZHENGESG100ZHISHUZENGQIANGC"}, "003185": {"suoxie": "JXHBB", "name": "\u5efa\u4fe1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINHUOBIB"}, "003186": {"suoxie": "PHXADQKFHH", "name": "\u9e4f\u534e\u5174\u5b89\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAXINGANDINGQIKAIFANGHUNHE"}, "003187": {"suoxie": "JSAYHH", "name": "\u5609\u5b9e\u5b89\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIANYIHUNHE"}, "003188": {"suoxie": "BSJYCZZQA", "name": "\u535a\u65f6\u805a\u6e90\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJUYUANCHUNZHAIZHAIQUANA"}, "003189": {"suoxie": "HTFBXLHPZHHA", "name": "\u6c47\u6dfb\u5bcc\u4fdd\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUBAOXINLINGHUOPEIZHIHUNHEA"}, "003190": {"suoxie": "CJHXXFZTGPA", "name": "\u521b\u91d1\u5408\u4fe1\u6d88\u8d39\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXIAOFEIZHUTIGUPIAOA"}, "003191": {"suoxie": "CJHXXFZTGPC", "name": "\u521b\u91d1\u5408\u4fe1\u6d88\u8d39\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXIAOFEIZHUTIGUPIAOC"}, "003192": {"suoxie": "CJHXZFCZ", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u4e30\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNFENGCHUNZHAI"}, "003193": {"suoxie": "CJHXZZCZZQA", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u667a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNZHICHUNZHAIZHAIQUANA"}, "003194": {"suoxie": "HTFZZSHGQETFLJ", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e0a\u6d77\u56fd\u4f01ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGSHANGHAIGUOQIETFLIANJIE"}, "003195": {"suoxie": "GDYLCZA", "name": "\u5149\u5927\u6c38\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAYONGLICHUNZHAIA"}, "003196": {"suoxie": "GDYLCZC", "name": "\u5149\u5927\u6c38\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAYONGLICHUNZHAIC"}, "003197": {"suoxie": "GDACZQA", "name": "\u5149\u5927\u5b89\u8bda\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANCHENGZHAIQUANA"}, "003198": {"suoxie": "GDACZQC", "name": "\u5149\u5927\u5b89\u8bda\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANCHENGZHAIQUANC"}, "003199": {"suoxie": "CSSQYNZQA", "name": "\u957f\u76db\u76db\u742a\u4e00\u5e74\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGQIYINIANZHAIQUANA"}, "003200": {"suoxie": "CSSQYNZQC", "name": "\u957f\u76db\u76db\u742a\u4e00\u5e74\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGQIYINIANZHAIQUANC"}, "003204": {"suoxie": "CTSYZQZQC", "name": "\u8d22\u901a\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGSHOUYIZENGQIANGZHAIQUANC"}, "003205": {"suoxie": "CTKZZZQC", "name": "\u8d22\u901a\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CAITONGKEZHUANZHAIZHAIQUANC"}, "003206": {"suoxie": "BSHXHB", "name": "\u535a\u65f6\u5408\u946b\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEXINHUOBI"}, "003207": {"suoxie": "BSFFCZ", "name": "\u535a\u65f6\u5bcc\u53d1\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUFACHUNZHAI"}, "003209": {"suoxie": "PHFDZQ", "name": "\u9e4f\u534e\u4e30\u8fbe\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGDAZHAIQUAN"}, "003210": {"suoxie": "BSZZCZZQ", "name": "\u535a\u65f6\u667a\u81fb\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHIZHENCHUNZHAIZHAIQUAN"}, "003213": {"suoxie": "ZYYXDQKFZQFQS", "name": "\u4e2d\u94f6\u60a6\u4eab\u5b9a\u671f\u5f00\u653e\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINYUEXIANGDINGQIKAIFANGZHAIQUANFAQISHI"}, "003214": {"suoxie": "YFDFHCZZQ", "name": "\u6613\u65b9\u8fbe\u5bcc\u60e0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAFUHUICHUNZHAIZHAIQUAN"}, "003218": {"suoxie": "QHKYXHZQA", "name": "\u524d\u6d77\u5f00\u6e90\u7965\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANXIANGHEZHAIQUANA"}, "003219": {"suoxie": "QHKYXHZQC", "name": "\u524d\u6d77\u5f00\u6e90\u7965\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANXIANGHEZHAIQUANC"}, "003220": {"suoxie": "ZSHLCZZQ", "name": "\u6d59\u5546\u60e0\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUILICHUNZHAIZHAIQUAN"}, "003221": {"suoxie": "XHFLZQA", "name": "\u65b0\u534e\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAFENGLIZHAIQUANA"}, "003222": {"suoxie": "XHFLZQC", "name": "\u65b0\u534e\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAFENGLIZHAIQUANC"}, "003223": {"suoxie": "GFJFCZ", "name": "\u5e7f\u53d1\u666f\u4e30\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGFENGCHUNZHAI"}, "003226": {"suoxie": "XCWJZQA", "name": "\u4fe1\u8bda\u7a33\u5065\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENJIANZHAIQUANA"}, "003227": {"suoxie": "XCWJZQC", "name": "\u4fe1\u8bda\u7a33\u5065\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENJIANZHAIQUANC"}, "003228": {"suoxie": "PYASRRXHBA", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIXINHUOBIA"}, "003229": {"suoxie": "PYASRRXHBB", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIXINHUOBIB"}, "003230": {"suoxie": "CJHXYLBJGPA", "name": "\u521b\u91d1\u5408\u4fe1\u533b\u7597\u4fdd\u5065\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYILIAOBAOJIANGUPIAOA"}, "003231": {"suoxie": "CJHXYLBJGPC", "name": "\u521b\u91d1\u5408\u4fe1\u533b\u7597\u4fdd\u5065\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYILIAOBAOJIANGUPIAOC"}, "003232": {"suoxie": "CJHXJRDCA", "name": "\u521b\u91d1\u5408\u4fe1\u91d1\u878d\u5730\u4ea7A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINJINRONGDICHANA"}, "003233": {"suoxie": "CJHXJRDCC", "name": "\u521b\u91d1\u5408\u4fe1\u91d1\u878d\u5730\u4ea7C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINJINRONGDICHANC"}, "003234": {"suoxie": "XCZLHHA", "name": "\u4fe1\u8bda\u81f3\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHILIHUNHEA"}, "003235": {"suoxie": "XCZLHHC", "name": "\u4fe1\u8bda\u81f3\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHILIHUNHEC"}, "003238": {"suoxie": "XHWYZZZTLHPZHH", "name": "\u65b0\u534e\u5916\u5ef6\u589e\u957f\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAWAIYANZENGZHANGZHUTILINGHUOPEIZHIHUNHE"}, "003239": {"suoxie": "BSAQ6GYDKZA", "name": "\u535a\u65f6\u5b89\u797a6\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANQI6GEYUEDINGKAIZHAIA"}, "003240": {"suoxie": "BSAQ6GYDKZC", "name": "\u535a\u65f6\u5b89\u797a6\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANQI6GEYUEDINGKAIZHAIC"}, "003241": {"suoxie": "CJHXLHFXHHA", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u53d1\u73b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINLIANGHUAFAXIANHUNHEA"}, "003242": {"suoxie": "CJHXLHFXHHC", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u53d1\u73b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINLIANGHUAFAXIANHUNHEC"}, "003243": {"suoxie": "STMGZGSJRMB", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u56fd\u4e16\u7eaa\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "SHANGTOUMOGENZHONGGUOSHIJIRENMINBI"}, "003244": {"suoxie": "STMGZGSJMYXC", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u56fd\u4e16\u7eaa\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "SHANGTOUMOGENZHONGGUOSHIJIMEIYUANXIANCHAO"}, "003245": {"suoxie": "STMGZGSJMYXH", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u56fd\u4e16\u7eaa\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "SHANGTOUMOGENZHONGGUOSHIJIMEIYUANXIANHUI"}, "003246": {"suoxie": "HTBRTTBHBA", "name": "\u534e\u6cf0\u67cf\u745e\u5929\u6dfb\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUITIANTIANBAOHUOBIA"}, "003252": {"suoxie": "DCTYJYXHBA", "name": "\u5927\u6210\u6dfb\u76ca\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANYIJIAOYIXINGHUOBIA"}, "003253": {"suoxie": "DCTYJYXHBB", "name": "\u5927\u6210\u6dfb\u76ca\u4ea4\u6613\u578b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANYIJIAOYIXINGHUOBIB"}, "003254": {"suoxie": "QHKYDYZQA", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGYUZHAIQUANA"}, "003255": {"suoxie": "QHKYDYZQC", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANDINGYUZHAIQUANC"}, "003258": {"suoxie": "BSFXCZZQA", "name": "\u535a\u65f6\u5bcc\u7965\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUXIANGCHUNZHAIZHAIQUANA"}, "003259": {"suoxie": "BSJLCZZQ", "name": "\u535a\u65f6\u805a\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJULICHUNZHAIZHAIQUAN"}, "003260": {"suoxie": "BSLFCZ", "name": "\u535a\u65f6\u5229\u53d1\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHILIFACHUNZHAI"}, "003264": {"suoxie": "XHHQTLHBB", "name": "\u65b0\u534e\u6d3b\u671f\u6dfb\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAHUOQITIANLIHUOBIB"}, "003265": {"suoxie": "ZSZKCZA", "name": "\u62db\u5546\u62db\u5764\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOKUNCHUNZHAIA"}, "003266": {"suoxie": "ZSZKCZC", "name": "\u62db\u5546\u62db\u5764\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOKUNCHUNZHAIC"}, "003267": {"suoxie": "XHYNBHBB", "name": "\u65b0\u534e\u58f9\u8bfa\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAYINUOBAOHUOBIB"}, "003268": {"suoxie": "BSYCCZ", "name": "\u535a\u65f6\u60a6\u695a\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUECHUCHUNZHAI"}, "003269": {"suoxie": "ZSZQ3GYDKZA", "name": "\u62db\u5546\u62db\u4e7e3\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOQIAN3GEYUEDINGKAIZHAIA"}, "003270": {"suoxie": "ZSZQ3GYDKZC", "name": "\u62db\u5546\u62db\u4e7e3\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOQIAN3GEYUEDINGKAIZHAIC"}, "003273": {"suoxie": "AXYFDKZQA", "name": "\u5b89\u4fe1\u6c38\u4e30\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGFENGDINGKAIZHAIQUANA"}, "003274": {"suoxie": "AXYFDKZQC", "name": "\u5b89\u4fe1\u6c38\u4e30\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGFENGDINGKAIZHAIQUANC"}, "003275": {"suoxie": "GLATLZZZA", "name": "\u56fd\u8054\u5b89\u6dfb\u5229\u589e\u957f\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANTIANLIZENGZHANGZHAIA"}, "003276": {"suoxie": "GLATLZZZC", "name": "\u56fd\u8054\u5b89\u6dfb\u5229\u589e\u957f\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANTIANLIZENGZHANGZHAIC"}, "003277": {"suoxie": "XCWRZQA", "name": "\u4fe1\u8bda\u7a33\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENRUIZHAIQUANA"}, "003278": {"suoxie": "XCWRZQC", "name": "\u4fe1\u8bda\u7a33\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENRUIZHAIQUANC"}, "003279": {"suoxie": "RTHGSZHSHLHPZHH", "name": "\u878d\u901a\u6caa\u6e2f\u6df1\u667a\u6167\u751f\u6d3b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGHUGANGSHENZHIHUISHENGHUOLINGHUOPEIZHIHUNHE"}, "003280": {"suoxie": "PHFHZQ", "name": "\u9e4f\u534e\u4e30\u6052\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGHENGZHAIQUAN"}, "003281": {"suoxie": "GFHQBHBB", "name": "\u5e7f\u53d1\u6d3b\u671f\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOQIBAOHUOBIB"}, "003282": {"suoxie": "XCZYLHPZHHA", "name": "\u4fe1\u8bda\u81f3\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIYULINGHUOPEIZHIHUNHEA"}, "003283": {"suoxie": "XCZYLHPZHHC", "name": "\u4fe1\u8bda\u81f3\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIYULINGHUOPEIZHIHUNHEC"}, "003284": {"suoxie": "ZYYYJKHH", "name": "\u4e2d\u90ae\u533b\u836f\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUYIYAOJIANKANGHUNHE"}, "003285": {"suoxie": "GSABAKCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u5eb7\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANKANGCHUNZHAIZHAIQUAN"}, "003286": {"suoxie": "PAHXCZA", "name": "\u5e73\u5b89\u60e0\u4eab\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXIANGCHUNZHAIA"}, "003287": {"suoxie": "ZXBCWYA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u76caA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENYIA"}, "003288": {"suoxie": "ZXBCWYC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u76caC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENYIC"}, "003289": {"suoxie": "CJHXZTCZZQ", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u6cf0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNTAICHUNZHAIZHAIQUAN"}, "003290": {"suoxie": "CCJWZQA", "name": "\u957f\u57ce\u4e45\u7a33\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIUWENZHAIQUANA"}, "003291": {"suoxie": "XDAYJKZGHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5065\u5eb7\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINJIANKANGZHONGGUOHUNHE"}, "003292": {"suoxie": "JSYSCZHH", "name": "\u5609\u5b9e\u4f18\u52bf\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIYOUSHICHENGZHANGHUNHE"}, "003293": {"suoxie": "YFDKRHH", "name": "\u6613\u65b9\u8fbe\u79d1\u745e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAKERUIHUNHE"}, "003295": {"suoxie": "NFAYHHA", "name": "\u5357\u65b9\u5b89\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANYUHUNHEA"}, "003297": {"suoxie": "ZSSZZQZQLOFE", "name": "\u62db\u5546\u53cc\u503a\u589e\u5f3a\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGSHUANGZHAIZENGQIANGZHAIQUANLOFE"}, "003298": {"suoxie": "JSWLCYGPA", "name": "\u5609\u5b9e\u7269\u6d41\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIWULIUCHANYEGUPIAOA"}, "003299": {"suoxie": "JSWLCYGPC", "name": "\u5609\u5b9e\u7269\u6d41\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIWULIUCHANYEGUPIAOC"}, "003300": {"suoxie": "HXYHHH", "name": "\u534e\u590f\u5706\u548c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAYUANHEHUNHE"}, "003301": {"suoxie": "HXDRZQA", "name": "\u534e\u590f\u9f0e\u878d\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGRONGZHAIQUANA"}, "003302": {"suoxie": "HXDRZQC", "name": "\u534e\u590f\u9f0e\u878d\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGRONGZHAIQUANC"}, "003304": {"suoxie": "QHKYHGSHXZYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u6838\u5fc3\u8d44\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENHEXINZIYUANHUNHEA"}, "003305": {"suoxie": "QHKYHGSHXZYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u6838\u5fc3\u8d44\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENHEXINZIYUANHUNHEC"}, "003308": {"suoxie": "ZXJTRLA", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u5229A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOURUILIA"}, "003309": {"suoxie": "XYQYYNDKZA", "name": "\u5174\u4e1a\u542f\u5143\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEQIYUANYINIANDINGKAIZHAIA"}, "003310": {"suoxie": "XYQYYNDKZC", "name": "\u5174\u4e1a\u542f\u5143\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEQIYUANYINIANDINGKAIZHAIC"}, "003313": {"suoxie": "ZYRXDKZQ", "name": "\u4e2d\u94f6\u777f\u4eab\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINRUIXIANGDINGKAIZHAIQUAN"}, "003314": {"suoxie": "ZSHNCZZQ", "name": "\u6d59\u5546\u60e0\u5357\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUINANCHUNZHAIZHAIQUAN"}, "003315": {"suoxie": "JSCCZCXJRZ", "name": "\u666f\u987a\u957f\u57ce\u653f\u7b56\u6027\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGZHENGCEXINGJINRONGZHAI"}, "003316": {"suoxie": "ZYZQXJGJHBA", "name": "\u4e2d\u94f6\u8bc1\u5238\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINZHENGQUANXIANJINGUANJIAHUOBIA"}, "003317": {"suoxie": "ZYZQXJGJHBB", "name": "\u4e2d\u94f6\u8bc1\u5238\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINZHENGQUANXIANJINGUANJIAHUOBIB"}, "003318": {"suoxie": "JSZZ500HYZXDBD", "name": "\u666f\u987a\u4e2d\u8bc1500\u884c\u4e1a\u4e2d\u6027\u4f4e\u6ce2\u52a8", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENG500HANGYEZHONGXINGDIBODONG"}, "003321": {"suoxie": "YFDYYCLRMB", "name": "\u6613\u65b9\u8fbe\u539f\u6cb9C\u7c7b\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "YIFANGDAYUANYOUCLEIRENMINBI"}, "003322": {"suoxie": "YFDYYALMYH", "name": "\u6613\u65b9\u8fbe\u539f\u6cb9A\u7c7b\u7f8e\u5143\u6c47", "style": "QDII", "pinyin": "YIFANGDAYUANYOUALEIMEIYUANHUI"}, "003323": {"suoxie": "YFDYYCLMYH", "name": "\u6613\u65b9\u8fbe\u539f\u6cb9C\u7c7b\u7f8e\u5143\u6c47", "style": "QDII", "pinyin": "YIFANGDAYUANYOUCLEIMEIYUANHUI"}, "003324": {"suoxie": "DFYX18GYDKZA", "name": "\u4e1c\u65b9\u6c38\u517418\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGXING18GEYUEDINGKAIZHAIA"}, "003325": {"suoxie": "DFYX18GYDKZC", "name": "\u4e1c\u65b9\u6c38\u517418\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGXING18GEYUEDINGKAIZHAIC"}, "003327": {"suoxie": "WJXJCZA", "name": "\u4e07\u5bb6\u946b\u749f\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINJINGCHUNZHAIA"}, "003328": {"suoxie": "WJXJCZC", "name": "\u4e07\u5bb6\u946b\u749f\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINJINGCHUNZHAIC"}, "003329": {"suoxie": "WJXACZZQA", "name": "\u4e07\u5bb6\u946b\u5b89\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINANCHUNZHAIZHAIQUANA"}, "003330": {"suoxie": "WJXACZZQC", "name": "\u4e07\u5bb6\u946b\u5b89\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINANCHUNZHAIZHAIQUANC"}, "003331": {"suoxie": "BSLZDKHH", "name": "\u535a\u65f6\u4e50\u81fb\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHILEZHENDINGKAIHUNHE"}, "003333": {"suoxie": "TXZXCZHH", "name": "\u6cf0\u4fe1\u667a\u9009\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINZHIXUANCHENGZHANGHUNHE"}, "003336": {"suoxie": "CJSYZQZQ", "name": "\u957f\u6c5f\u6536\u76ca\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGSHOUYIZENGQIANGZHAIQUAN"}, "003337": {"suoxie": "NFYYDKZQFQ", "name": "\u5357\u65b9\u9890\u5143\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGYIYUANDINGKAIZHAIQUANFAQI"}, "003338": {"suoxie": "NFYYDKZQFQSC", "name": "\u5357\u65b9\u9890\u5143\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGYIYUANDINGKAIZHAIQUANFAQISHIC"}, "003341": {"suoxie": "GYRY18GYDKZ", "name": "\u5de5\u94f6\u745e\u76c818\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINRUIYING18GEYUEDINGKAIZHAI"}, "003343": {"suoxie": "PHHHLHPZHHA", "name": "\u9e4f\u534e\u5f18\u60e0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHUILINGHUOPEIZHIHUNHEA"}, "003344": {"suoxie": "PHHHLHPZHHC", "name": "\u9e4f\u534e\u5f18\u60e0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGHUILINGHUOPEIZHIHUNHEC"}, "003345": {"suoxie": "AXXCZHHA", "name": "\u5b89\u4fe1\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINCHENGZHANGHUNHEA"}, "003346": {"suoxie": "AXXCZHHC", "name": "\u5b89\u4fe1\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINCHENGZHANGHUNHEC"}, "003349": {"suoxie": "CXWYCZ", "name": "\u957f\u4fe1\u7a33\u76ca\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENYICHUNZHAI"}, "003357": {"suoxie": "JSWXCZZQC", "name": "\u5609\u5b9e\u7a33\u7965\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENXIANGCHUNZHAIZHAIQUANC"}, "003358": {"suoxie": "YFDZZ710NGKZA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI710NIANGUOKAIZHAIA"}, "003359": {"suoxie": "DC360HLWDSJ100C", "name": "\u5927\u6210360\u4e92\u8054\u7f51+\u5927\u6570\u636e100C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENG360HULIANWANGDASHUJU100C"}, "003360": {"suoxie": "QHKYRHZQA", "name": "\u524d\u6d77\u5f00\u6e90\u745e\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANRUIHEZHAIQUANA"}, "003361": {"suoxie": "QHKYRHZQC", "name": "\u524d\u6d77\u5f00\u6e90\u745e\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANRUIHEZHAIQUANC"}, "003363": {"suoxie": "CJLXHBA", "name": "\u957f\u6c5f\u4e50\u4eab\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGJIANGLEXIANGHUOBIA"}, "003364": {"suoxie": "CJLXHBB", "name": "\u957f\u6c5f\u4e50\u4eab\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGJIANGLEXIANGHUOBIB"}, "003365": {"suoxie": "CJLXHBC", "name": "\u957f\u6c5f\u4e50\u4eab\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGJIANGLEXIANGHUOBIC"}, "003366": {"suoxie": "ZSHJZZZXCZZS", "name": "\u6d59\u5546\u6c47\u91d1\u4e2d\u8bc1\u8f6c\u578b\u6210\u957f\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGHUIJINZHONGZHENGZHUANXINGCHENGZHANGZHISHU"}, "003373": {"suoxie": "DCJLLHPZHHA", "name": "\u5927\u6210\u666f\u7984\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJINGLULINGHUOPEIZHIHUNHEA"}, "003374": {"suoxie": "DCJLLHPZHHC", "name": "\u5927\u6210\u666f\u7984\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJINGLULINGHUOPEIZHIHUNHEC"}, "003376": {"suoxie": "GFZZ710NGKZZSA", "name": "\u5e7f\u53d1\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI710NIANGUOKAIZHAIZHISHUA"}, "003377": {"suoxie": "GFZZ710NGKZZSC", "name": "\u5e7f\u53d1\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI710NIANGUOKAIZHAIZHISHUC"}, "003378": {"suoxie": "TKCLYXHH", "name": "\u6cf0\u5eb7\u7b56\u7565\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGCELUEYOUXUANHUNHE"}, "003379": {"suoxie": "XCZXHHA", "name": "\u4fe1\u8bda\u81f3\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIXUANHUNHEA"}, "003380": {"suoxie": "XCZXHHC", "name": "\u4fe1\u8bda\u81f3\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIXUANHUNHEC"}, "003382": {"suoxie": "MSJYXXZQA", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINXIANGZHAIQUANA"}, "003383": {"suoxie": "MSJYXXZQC", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINXIANGZHAIQUANC"}, "003384": {"suoxie": "JYTYCZZQA", "name": "\u91d1\u9e70\u6dfb\u76c8\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANYINGCHUNZHAIZHAIQUANA"}, "003385": {"suoxie": "GYQQMYZARMB", "name": "\u5de5\u94f6\u5168\u7403\u7f8e\u5143\u503aA\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GONGYINQUANQIUMEIYUANZHAIARENMINBI"}, "003386": {"suoxie": "GYQQMYZAMYXH", "name": "\u5de5\u94f6\u5168\u7403\u7f8e\u5143\u503aA\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "GONGYINQUANQIUMEIYUANZHAIAMEIYUANXIANHUI"}, "003387": {"suoxie": "GYQQMYZC", "name": "\u5de5\u94f6\u5168\u7403\u7f8e\u5143\u503aC", "style": "QDII", "pinyin": "GONGYINQUANQIUMEIYUANZHAIC"}, "003388": {"suoxie": "ZSZYBHBA", "name": "\u62db\u5546\u62db\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOYIBAOHUOBIA"}, "003389": {"suoxie": "ZSZYBHBB", "name": "\u62db\u5546\u62db\u76ca\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOYIBAOHUOBIB"}, "003390": {"suoxie": "JXYNDK", "name": "\u6c5f\u4fe1\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINYINIANDINGKAI"}, "003391": {"suoxie": "JXTTYHBA", "name": "\u5efa\u4fe1\u5929\u6dfb\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINTIANTIANYIHUOBIA"}, "003392": {"suoxie": "JXTTYHBB", "name": "\u5efa\u4fe1\u5929\u6dfb\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINTIANTIANYIHUOBIB"}, "003393": {"suoxie": "JXTTYHBC", "name": "\u5efa\u4fe1\u5929\u6dfb\u76ca\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINTIANTIANYIHUOBIC"}, "003394": {"suoxie": "JXHAYNDKZ", "name": "\u5efa\u4fe1\u6052\u5b89\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINHENGANYINIANDINGKAIZHAI"}, "003395": {"suoxie": "AXZXCZ", "name": "\u5b89\u4fe1\u5c0a\u4eab\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZUNXIANGCHUNZHAI"}, "003396": {"suoxie": "DFHYXHLHH", "name": "\u4e1c\u65b9\u7ea2\u4f18\u4eab\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGYOUXIANGHONGLIHUNHE"}, "003397": {"suoxie": "YHTYWHLHPZHH", "name": "\u94f6\u534e\u4f53\u80b2\u6587\u5316\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUATIYUWENHUALINGHUOPEIZHIHUNHE"}, "003398": {"suoxie": "TPRRJHBA", "name": "\u592a\u5e73\u65e5\u65e5\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIPINGRIRIJINHUOBIA"}, "003399": {"suoxie": "TPRRJHBB", "name": "\u592a\u5e73\u65e5\u65e5\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIPINGRIRIJINHUOBIB"}, "003400": {"suoxie": "JXHRYNDKZ", "name": "\u5efa\u4fe1\u6052\u745e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINHENGRUIYINIANDINGKAIZHAI"}, "003401": {"suoxie": "GYKZZZQ", "name": "\u5de5\u94f6\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GONGYINKEZHUANZHAIZHAIQUAN"}, "003402": {"suoxie": "AXHQBHBA", "name": "\u5b89\u4fe1\u6d3b\u671f\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINHUOQIBAOHUOBIA"}, "003403": {"suoxie": "HSRFDZA", "name": "\u534e\u5546\u745e\u4e30\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUASHANGRUIFENGDUANZHAIA"}, "003406": {"suoxie": "NFDYDKZQ", "name": "\u5357\u65b9\u591a\u5143\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGDUOYUANDINGKAIZHAIQUAN"}, "003407": {"suoxie": "JSCCJTFLCZA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u4e30\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIFENGLICHUNZHAIA"}, "003408": {"suoxie": "JSCCJTFLCZC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u4e30\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIFENGLICHUNZHAIC"}, "003411": {"suoxie": "PHHKLHPZHHA", "name": "\u9e4f\u534e\u5f18\u5eb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGKANGLINGHUOPEIZHIHUNHEA"}, "003412": {"suoxie": "PHHKLHPZHHC", "name": "\u9e4f\u534e\u5f18\u5eb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGKANGLINGHUOPEIZHIHUNHEC"}, "003413": {"suoxie": "HTBRXJJHGSHH", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u7ecf\u6d4e\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXINJINGJIHUGANGSHENHUNHE"}, "003416": {"suoxie": "ZSCJDSJGPA", "name": "\u62db\u5546\u8d22\u7ecf\u5927\u6570\u636e\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGCAIJINGDASHUJUGUPIAOA"}, "003417": {"suoxie": "ZJFZCZZQ", "name": "\u4e2d\u52a0\u4e30\u6cfd\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGZECHUNZHAIZHAIQUAN"}, "003418": {"suoxie": "HRYDRXZQA", "name": "\u534e\u6da6\u5143\u5927\u6da6\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNXINZHAIQUANA"}, "003422": {"suoxie": "GSABTLHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u6dfb\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOTIANLIHUOBIA"}, "003423": {"suoxie": "GSABTLHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u6dfb\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOTIANLIHUOBIB"}, "003424": {"suoxie": "JXHFCZ", "name": "\u6c5f\u4fe1\u6d2a\u798f\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINHONGFUCHUNZHAI"}, "003425": {"suoxie": "JXTFA", "name": "\u6c5f\u4fe1\u6dfb\u798fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINTIANFUA"}, "003426": {"suoxie": "JXTFC", "name": "\u6c5f\u4fe1\u6dfb\u798fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANGXINTIANFUC"}, "003427": {"suoxie": "JXHYYNDKZ", "name": "\u5efa\u4fe1\u6052\u8fdc\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINHENGYUANYINIANDINGKAIZHAI"}, "003428": {"suoxie": "ZJFYYNDKZ", "name": "\u4e2d\u52a0\u4e30\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGYINGYINIANDINGKAIZHAI"}, "003429": {"suoxie": "ZZXYZGDJXYZZS", "name": "\u4e2d\u8bc1\u5174\u4e1a\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGZHENGXINGYEZHONGGAODENGJIXINYONGZHAIZHISHU"}, "003430": {"suoxie": "XYZDZC", "name": "\u5174\u4e1a\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYEZHONGDUANZHAIC"}, "003431": {"suoxie": "XYZDZA", "name": "\u5174\u4e1a\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYEZHONGDUANZHAIA"}, "003432": {"suoxie": "XCZRLHPZA", "name": "\u4fe1\u8bda\u81f3\u745e\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIRUILINGHUOPEIZHIA"}, "003433": {"suoxie": "XCZRLHPZC", "name": "\u4fe1\u8bda\u81f3\u745e\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHIRUILINGHUOPEIZHIC"}, "003434": {"suoxie": "BSXZLHPZHHA", "name": "\u535a\u65f6\u946b\u6cfd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINZELINGHUOPEIZHIHUNHEA"}, "003435": {"suoxie": "BSXZLHPZHHC", "name": "\u535a\u65f6\u946b\u6cfd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINZELINGHUOPEIZHIHUNHEC"}, "003438": {"suoxie": "ZSZYCZA", "name": "\u62db\u5546\u62db\u6021\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYICHUNZHAIA"}, "003439": {"suoxie": "ZSZYCZC", "name": "\u62db\u5546\u62db\u6021\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYICHUNZHAIC"}, "003440": {"suoxie": "ZSZXCZA", "name": "\u62db\u5546\u62db\u4eab\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXIANGCHUNZHAIA"}, "003441": {"suoxie": "ZSZXCZC", "name": "\u62db\u5546\u62db\u4eab\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXIANGCHUNZHAIC"}, "003442": {"suoxie": "ZSZH3GYDQKFZQA", "name": "\u62db\u5546\u62db\u60e03\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHUI3GEYUEDINGQIKAIFANGZHAIQUANA"}, "003443": {"suoxie": "ZSZH3GYDQKFZQC", "name": "\u62db\u5546\u62db\u60e03\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHUI3GEYUEDINGQIKAIFANGZHAIQUANC"}, "003445": {"suoxie": "ZJFXCZZQ", "name": "\u4e2d\u52a0\u4e30\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGXIANGCHUNZHAIZHAIQUAN"}, "003446": {"suoxie": "YDRXA", "name": "\u82f1\u5927\u777f\u946bA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDARUIXINA"}, "003447": {"suoxie": "YDRXC", "name": "\u82f1\u5927\u777f\u946bC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDARUIXINC"}, "003448": {"suoxie": "ZSZHCZA", "name": "\u62db\u5546\u62db\u534e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHUACHUNZHAIA"}, "003449": {"suoxie": "ZSZHCZC", "name": "\u62db\u5546\u62db\u534e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHUACHUNZHAIC"}, "003450": {"suoxie": "ZSZXDKZA", "name": "\u62db\u5546\u62db\u4fe1\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXINDINGKAIZHAIA"}, "003451": {"suoxie": "ZSZXDKZC", "name": "\u62db\u5546\u62db\u4fe1\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXINDINGKAIZHAIC"}, "003452": {"suoxie": "ZSZSCZA", "name": "\u62db\u5546\u62db\u76db\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOSHENGCHUNZHAIA"}, "003453": {"suoxie": "ZSZSCZC", "name": "\u62db\u5546\u62db\u76db\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOSHENGCHUNZHAIC"}, "003454": {"suoxie": "ZSZTCZA", "name": "\u62db\u5546\u62db\u901a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOTONGCHUNZHAIA"}, "003455": {"suoxie": "ZSZTCZC", "name": "\u62db\u5546\u62db\u901a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOTONGCHUNZHAIC"}, "003456": {"suoxie": "XDAYXMBHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u76ee\u6807\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINXINMUBIAOHUNHE"}, "003457": {"suoxie": "GTRTCZZQ", "name": "\u56fd\u6cf0\u6da6\u6cf0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUNTAICHUNZHAIZHAIQUAN"}, "003458": {"suoxie": "JSWHZQA", "name": "\u5609\u5b9e\u7a33\u5b8f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENHONGZHAIQUANA"}, "003459": {"suoxie": "JSWHZQC", "name": "\u5609\u5b9e\u7a33\u5b8f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENHONGZHAIQUANC"}, "003460": {"suoxie": "JSXJB", "name": "\u5609\u5b9e\u73b0\u91d1\u5b9d", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIXIANJINBAO"}, "003461": {"suoxie": "JSWYCZZQA", "name": "\u5609\u5b9e\u7a33\u5143\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENYUANCHUNZHAIZHAIQUANA"}, "003465": {"suoxie": "PAJGJHBA", "name": "\u5e73\u5b89\u91d1\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANJINGUANJIAHUOBIA"}, "003467": {"suoxie": "FRHBA", "name": "\u5bcc\u8363\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FURONGHUOBIA"}, "003468": {"suoxie": "FRHBB", "name": "\u5bcc\u8363\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FURONGHUOBIB"}, "003471": {"suoxie": "QHLHTX3GYKZA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u946b3\u4e2a\u6708\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAILIANHETIANXIN3GEYUEKAIZHAIA"}, "003472": {"suoxie": "QHLHTX3GYKZC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u946b3\u4e2a\u6708\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAILIANHETIANXIN3GEYUEKAIZHAIC"}, "003473": {"suoxie": "NFTTLHBA", "name": "\u5357\u65b9\u5929\u5929\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGTIANTIANLIHUOBIA"}, "003474": {"suoxie": "NFTTLHBB", "name": "\u5357\u65b9\u5929\u5929\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGTIANTIANLIHUOBIB"}, "003475": {"suoxie": "QHLHHS300ZSA", "name": "\u524d\u6d77\u8054\u5408\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAILIANHEHUSHEN300ZHISHUA"}, "003476": {"suoxie": "NFAYHH", "name": "\u5357\u65b9\u5b89\u9890\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANYIHUNHE"}, "003478": {"suoxie": "MSJYTYBHBA", "name": "\u6c11\u751f\u52a0\u94f6\u817e\u5143\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINTENGYUANBAOHUOBIA"}, "003479": {"suoxie": "CTZGXGJHBA", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CAITONGZIGUANXINGUANJIAHUOBIA"}, "003480": {"suoxie": "CTZGXGJHBB", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u7ba1\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CAITONGZIGUANXINGUANJIAHUOBIB"}, "003481": {"suoxie": "GDYQBHBB", "name": "\u5149\u5927\u8000\u94b1\u5305\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAYAOQIANBAOHUOBIB"}, "003482": {"suoxie": "JYTXBHBA", "name": "\u4ea4\u94f6\u5929\u946b\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANXINBAOHUOBIA"}, "003483": {"suoxie": "JYTXBHBE", "name": "\u4ea4\u94f6\u5929\u946b\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANXINBAOHUOBIE"}, "003484": {"suoxie": "JYXYHHA", "name": "\u91d1\u9e70\u946b\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGXINYIHUNHEA"}, "003485": {"suoxie": "JYXYHHC", "name": "\u91d1\u9e70\u946b\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGXINYIHUNHEC"}, "003486": {"suoxie": "PAHLCZA", "name": "\u5e73\u5b89\u60e0\u9686\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUILONGCHUNZHAIA"}, "003487": {"suoxie": "PAHRCZ", "name": "\u5e73\u5b89\u60e0\u878d\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIRONGCHUNZHAI"}, "003493": {"suoxie": "SWLXAXYXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINANXINYOUXUANHUNHEA"}, "003494": {"suoxie": "FGTHCZHHC", "name": "\u5bcc\u56fd\u5929\u60e0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHUICHENGZHANGHUNHEC"}, "003495": {"suoxie": "PHHSLHPZHHA", "name": "\u9e4f\u534e\u5f18\u5c1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHANGLINGHUOPEIZHIHUNHEA"}, "003496": {"suoxie": "PHHSLHPZHHC", "name": "\u9e4f\u534e\u5f18\u5c1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGSHANGLINGHUOPEIZHIHUNHEC"}, "003498": {"suoxie": "QHLHTHCZA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u548c\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANHECHUNZHAIA"}, "003499": {"suoxie": "QHLHTHCZC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u548c\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANHECHUNZHAIC"}, "003500": {"suoxie": "XYJLZQ", "name": "\u946b\u5143\u805a\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANJULIZHAIQUAN"}, "003501": {"suoxie": "TDRZWJHHA", "name": "\u6cf0\u8fbe\u777f\u667a\u7a33\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDARUIZHIWENJIANHUNHEA"}, "003502": {"suoxie": "JYXRHHA", "name": "\u91d1\u9e70\u946b\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGXINRUIHUNHEA"}, "003503": {"suoxie": "JYXRHHC", "name": "\u91d1\u9e70\u946b\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGXINRUIHUNHEC"}, "003504": {"suoxie": "JSCCJYFLZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYIFENGLIZHAIQUANA"}, "003505": {"suoxie": "JSCCJYFLZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYIFENGLIZHAIQUANC"}, "003510": {"suoxie": "CSKZZZQA", "name": "\u957f\u76db\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGSHENGKEZHUANZHAIZHAIQUANA"}, "003511": {"suoxie": "CSKZZZQC", "name": "\u957f\u76db\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGSHENGKEZHUANZHAIZHAIQUANC"}, "003512": {"suoxie": "SWLXAXYXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINANXINYOUXUANHUNHEC"}, "003513": {"suoxie": "ZYXFSJ", "name": "\u4e2d\u90ae\u6d88\u8d39\u5347\u7ea7", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUXIAOFEISHENGJI"}, "003514": {"suoxie": "GSABAXCZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u4eab\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANXIANGCHUNZHAI"}, "003515": {"suoxie": "GTLSBHB", "name": "\u56fd\u6cf0\u5229\u662f\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAILISHIBAOHUOBI"}, "003516": {"suoxie": "GTRADCLLHPZHHA", "name": "\u56fd\u6cf0\u878d\u5b89\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRONGANDUOCELUELINGHUOPEIZHIHUNHEA"}, "003517": {"suoxie": "GTRLCZZQ", "name": "\u56fd\u6cf0\u6da6\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUNLICHUNZHAIZHAIQUAN"}, "003518": {"suoxie": "WJXRCZA", "name": "\u4e07\u5bb6\u946b\u745e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINRUICHUNZHAIA"}, "003519": {"suoxie": "WJXRCZE", "name": "\u4e07\u5bb6\u946b\u745e\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINRUICHUNZHAIE"}, "003520": {"suoxie": "WJ13NZJZCZA", "name": "\u4e07\u5bb61-3\u5e74\u653f\u91d1\u503a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIA13NIANZHENGJINZHAICHUNZHAIA"}, "003521": {"suoxie": "WJ13NZJZCZC", "name": "\u4e07\u5bb61-3\u5e74\u653f\u91d1\u503a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIA13NIANZHENGJINZHAICHUNZHAIC"}, "003525": {"suoxie": "XYXJSYHB", "name": "\u5174\u94f6\u73b0\u91d1\u6536\u76ca\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYINXIANJINSHOUYIHUOBI"}, "003526": {"suoxie": "NYHLJSCZ3GYDKZ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u7a57\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINSUICHUNZHAI3GEYUEDINGKAIZHAI"}, "003527": {"suoxie": "PHFTZQ", "name": "\u9e4f\u534e\u4e30\u817e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGTENGZHAIQUAN"}, "003528": {"suoxie": "HTFCTLDKZA", "name": "\u6c47\u6dfb\u5bcc\u957f\u6dfb\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUCHANGTIANLIDINGKAIZHAIA"}, "003529": {"suoxie": "HTFCTLDKZC", "name": "\u6c47\u6dfb\u5bcc\u957f\u6dfb\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUCHANGTIANLIDINGKAIZHAIC"}, "003532": {"suoxie": "HTFXLDKZQA", "name": "\u6c47\u6dfb\u5bcc\u946b\u5229\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINLIDINGKAIZHAIQUANA"}, "003533": {"suoxie": "HTFXLDKZQC", "name": "\u6c47\u6dfb\u5bcc\u946b\u5229\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINLIDINGKAIZHAIQUANC"}, "003534": {"suoxie": "PYASRRFHBA", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u4e30\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIFENGHUOBIA"}, "003535": {"suoxie": "PYASRRFHBB", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u4e30\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIFENGHUOBIB"}, "003536": {"suoxie": "PYASRRFHBD", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u4e30\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIFENGHUOBID"}, "003537": {"suoxie": "ZSZLBHBA", "name": "\u62db\u5546\u62db\u5229\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOLIBAOHUOBIA"}, "003538": {"suoxie": "ZSZLBHBB", "name": "\u62db\u5546\u62db\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOLIBAOHUOBIB"}, "003539": {"suoxie": "AXXJZLHBB", "name": "\u5b89\u4fe1\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINXIANJINZENGLIHUOBIB"}, "003542": {"suoxie": "CTCZZQC", "name": "\u8d22\u901a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGCHUNZHAIZHAIQUANC"}, "003545": {"suoxie": "DXXLZQA", "name": "\u4e1c\u5174\u5174\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGXINGXINGLIZHAIQUANA"}, "003547": {"suoxie": "PHFLZQ", "name": "\u9e4f\u534e\u4e30\u7984\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGLUZHAIQUAN"}, "003548": {"suoxie": "TDHLHS300ZSZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAHONGLIHUSHEN300ZHISHUZENGQIANGC"}, "003549": {"suoxie": "ZSHYCZ", "name": "\u6d59\u5546\u60e0\u88d5\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIYUCHUNZHAI"}, "003550": {"suoxie": "TDGGDLHHC", "name": "\u6cf0\u8fbe\u6539\u9769\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAGAIGEDONGLIHUNHEC"}, "003561": {"suoxie": "NDCZJXA", "name": "\u8bfa\u5fb7\u6210\u957f\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODECHENGZHANGJINGXUANA"}, "003562": {"suoxie": "NDCZJXC", "name": "\u8bfa\u5fb7\u6210\u957f\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODECHENGZHANGJINGXUANC"}, "003564": {"suoxie": "BSAC3GYDKZA", "name": "\u535a\u65f6\u5b89\u8bda3\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANCHENG3GEYUEDINGKAIZHAIA"}, "003565": {"suoxie": "BSAC3GYDKZC", "name": "\u535a\u65f6\u5b89\u8bda3\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANCHENG3GEYUEDINGKAIZHAIC"}, "003566": {"suoxie": "BSZXCZZQA", "name": "\u535a\u65f6\u81fb\u9009\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHENXUANCHUNZHAIZHAIQUANA"}, "003567": {"suoxie": "HXHYJQHH", "name": "\u534e\u590f\u884c\u4e1a\u666f\u6c14\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHANGYEJINGQIHUNHE"}, "003568": {"suoxie": "PAHLCZ", "name": "\u5e73\u5b89\u60e0\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUILICHUNZHAI"}, "003569": {"suoxie": "ZSZFCZA", "name": "\u62db\u5546\u62db\u4e30\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOFENGCHUNZHAIA"}, "003570": {"suoxie": "ZSZFCZC", "name": "\u62db\u5546\u62db\u4e30\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOFENGCHUNZHAIC"}, "003571": {"suoxie": "ZSZQCZA", "name": "\u62db\u5546\u62db\u742a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOQICHUNZHAIA"}, "003572": {"suoxie": "ZSZQCZC", "name": "\u62db\u5546\u62db\u742a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOQICHUNZHAIC"}, "003573": {"suoxie": "ZXJTWYDKZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u88d5\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENYUDINGKAIZHAIA"}, "003574": {"suoxie": "DCHLCZ", "name": "\u5927\u6210\u60e0\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUILICHUNZHAI"}, "003578": {"suoxie": "ZJZZ500C", "name": "\u4e2d\u91d1\u4e2d\u8bc1500C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINZHONGZHENG500C"}, "003579": {"suoxie": "ZJHS300C", "name": "\u4e2d\u91d1\u6caa\u6df1300C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINHUSHEN300C"}, "003580": {"suoxie": "TKHGSJZYXHH", "name": "\u6cf0\u5eb7\u6caa\u6e2f\u6df1\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIKANGHUGANGSHENJIAZHIYOUXUANHUNHE"}, "003581": {"suoxie": "QHLHGMJKHHA", "name": "\u524d\u6d77\u8054\u5408\u56fd\u6c11\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEGUOMINJIANKANGHUNHEA"}, "003582": {"suoxie": "ZJLHDCLHH", "name": "\u4e2d\u91d1\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINLIANGHUADUOCELUEHUNHE"}, "003583": {"suoxie": "JXWDXLZQA", "name": "\u5efa\u4fe1\u7a33\u5b9a\u946b\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINWENDINGXINLIZHAIQUANA"}, "003584": {"suoxie": "JXWDXLZQC", "name": "\u5efa\u4fe1\u7a33\u5b9a\u946b\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINWENDINGXINLIZHAIQUANC"}, "003585": {"suoxie": "XFXJBHB", "name": "\u5148\u950b\u73b0\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "XIANFENGXIANJINBAOHUOBI"}, "003586": {"suoxie": "XFJYHHA", "name": "\u5148\u950b\u7cbe\u4e00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJINGYIHUNHEA"}, "003587": {"suoxie": "XFJYHHC", "name": "\u5148\u950b\u7cbe\u4e00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJINGYIHUNHEC"}, "003588": {"suoxie": "DWZXBHBA", "name": "\u4e1c\u5434\u589e\u946b\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGWUZENGXINBAOHUOBIA"}, "003589": {"suoxie": "DWZXBHBB", "name": "\u4e1c\u5434\u589e\u946b\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGWUZENGXINBAOHUOBIB"}, "003590": {"suoxie": "JXRFCZZQ", "name": "\u5efa\u4fe1\u777f\u5bcc\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIFUCHUNZHAIZHAIQUAN"}, "003591": {"suoxie": "HTBRXLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u4eab\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXIANGLIHUNHEA"}, "003592": {"suoxie": "HTBRXLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u4eab\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXIANGLIHUNHEC"}, "003593": {"suoxie": "GTJQHYLHPZHH", "name": "\u56fd\u6cf0\u666f\u6c14\u884c\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJINGQIHANGYELINGHUOPEIZHIHUNHE"}, "003594": {"suoxie": "CSSCLHPZHHA", "name": "\u957f\u76db\u76db\u5d07\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGCHONGLINGHUOPEIZHIHUNHEA"}, "003595": {"suoxie": "CSSCLHPZHHC", "name": "\u957f\u76db\u76db\u5d07\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGCHONGLINGHUOPEIZHIHUNHEC"}, "003598": {"suoxie": "HSRFLHPZHHA", "name": "\u534e\u5546\u6da6\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGRUNFENGLINGHUOPEIZHIHUNHEA"}, "003601": {"suoxie": "SWLXAXJXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINANXINJINGXUANHUNHEA"}, "003602": {"suoxie": "SWLXAXJXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINANXINJINGXUANHUNHEC"}, "003603": {"suoxie": "JSCCTAHBHHA", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u5b89\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIANHUIBAOHUNHEA"}, "003604": {"suoxie": "JSCCTAHBHHC", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u5b89\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIANHUIBAOHUNHEC"}, "003605": {"suoxie": "JSCCJTHLDKZA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u6c47\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIHUILIDINGKAIZHAIA"}, "003606": {"suoxie": "HFTQQSYZQMY", "name": "\u6d77\u5bcc\u901a\u5168\u7403\u6536\u76ca\u503a\u5238\u7f8e\u5143", "style": "QDII", "pinyin": "HAIFUTONGQUANQIUSHOUYIZHAIQUANMEIYUAN"}, "003607": {"suoxie": "BSFYCZZQ", "name": "\u535a\u65f6\u5bcc\u76ca\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYICHUNZHAIZHAIQUAN"}, "003612": {"suoxie": "NFZYZQA", "name": "\u5357\u65b9\u5353\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGZHUOYUANZHAIQUANA"}, "003613": {"suoxie": "NFZYZQC", "name": "\u5357\u65b9\u5353\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGZHUOYUANZHAIQUANC"}, "003614": {"suoxie": "XCJRZQA", "name": "\u4fe1\u8bda\u666f\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGJINGRUIZHAIQUANA"}, "003615": {"suoxie": "XCJRZQC", "name": "\u4fe1\u8bda\u666f\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGJINGRUIZHAIQUANC"}, "003618": {"suoxie": "ZSZWCZA", "name": "\u62db\u5546\u62db\u65fa\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOWANGCHUNZHAIA"}, "003619": {"suoxie": "ZSZWCZC", "name": "\u62db\u5546\u62db\u65fa\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOWANGCHUNZHAIC"}, "003624": {"suoxie": "CJHXZYGPFQSA", "name": "\u521b\u91d1\u5408\u4fe1\u8d44\u6e90\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINZIYUANGUPIAOFAQISHIA"}, "003625": {"suoxie": "CJHXZYGPFQSC", "name": "\u521b\u91d1\u5408\u4fe1\u8d44\u6e90\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINZIYUANGUPIAOFAQISHIC"}, "003626": {"suoxie": "PAXLHHA", "name": "\u5e73\u5b89\u946b\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINLIHUNHEA"}, "003628": {"suoxie": "XYSYZQZQ", "name": "\u5174\u94f6\u6536\u76ca\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYINSHOUYIZENGQIANGZHAIQUAN"}, "003629": {"suoxie": "STMGQQDYPZRMB", "name": "\u4e0a\u6295\u6469\u6839\u5168\u7403\u591a\u5143\u914d\u7f6e\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "SHANGTOUMOGENQUANQIUDUOYUANPEIZHIRENMINBI"}, "003630": {"suoxie": "STMGQQDYPZMYXC", "name": "\u4e0a\u6295\u6469\u6839\u5168\u7403\u591a\u5143\u914d\u7f6e\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "SHANGTOUMOGENQUANQIUDUOYUANPEIZHIMEIYUANXIANCHAO"}, "003631": {"suoxie": "STMGQQDYPZMYXH", "name": "\u4e0a\u6295\u6469\u6839\u5168\u7403\u591a\u5143\u914d\u7f6e\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "SHANGTOUMOGENQUANQIUDUOYUANPEIZHIMEIYUANXIANHUI"}, "003634": {"suoxie": "JSNYCYGP", "name": "\u5609\u5b9e\u519c\u4e1a\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHINONGYECHANYEGUPIAO"}, "003637": {"suoxie": "AXYXZQZQA", "name": "\u5b89\u4fe1\u6c38\u946b\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINYONGXINZENGQIANGZHAIQUANA"}, "003638": {"suoxie": "AXYXZQZQC", "name": "\u5b89\u4fe1\u6c38\u946b\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINYONGXINZENGQIANGZHAIQUANC"}, "003640": {"suoxie": "XYYFZQ", "name": "\u5174\u4e1a\u88d5\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYUFENGZHAIQUAN"}, "003641": {"suoxie": "CSSFLHPZHHA", "name": "\u957f\u76db\u76db\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGFENGLINGHUOPEIZHIHUNHEA"}, "003642": {"suoxie": "CSSFLHPZHHC", "name": "\u957f\u76db\u76db\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGSHENGFENGLINGHUOPEIZHIHUNHEC"}, "003646": {"suoxie": "CJHXZZ1000ZSZQA", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINZHONGZHENG1000ZHISHUZENGQIANGA"}, "003647": {"suoxie": "CJHXZZ1000ZSZQC", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINZHONGZHENG1000ZHISHUZENGQIANGC"}, "003648": {"suoxie": "RTTQZQ", "name": "\u878d\u901a\u901a\u797a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGQIZHAIQUAN"}, "003650": {"suoxie": "RTTRZQ", "name": "\u878d\u901a\u901a\u6da6\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGRUNZHAIQUAN"}, "003651": {"suoxie": "BSFDCZ6GYDKZ", "name": "\u535a\u65f6\u4e30\u8fbe\u7eaf\u503a6\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFENGDACHUNZHAI6GEYUEDINGKAIZHAI"}, "003655": {"suoxie": "XDAYXCFHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u8d22\u5bcc\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINXINCAIFUHUNHE"}, "003656": {"suoxie": "MSJYXYCZA", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u5143\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINYUANCHUNZHAIA"}, "003657": {"suoxie": "MSJYXYCZC", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u5143\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINYUANCHUNZHAIC"}, "003658": {"suoxie": "CSLHDCLLHPZHH", "name": "\u957f\u76db\u91cf\u5316\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGLIANGHUADUOCELUELINGHUOPEIZHIHUNHE"}, "003659": {"suoxie": "SXCLLHPZHH", "name": "\u5c71\u897f\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANXICELUELINGHUOPEIZHIHUNHE"}, "003660": {"suoxie": "ZJCZLNZQA", "name": "\u4e2d\u52a0\u7eaf\u503a\u4e24\u5e74\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAILIANGNIANZHAIQUANA"}, "003661": {"suoxie": "ZJCZLNZQC", "name": "\u4e2d\u52a0\u7eaf\u503a\u4e24\u5e74\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAILIANGNIANZHAIQUANC"}, "003662": {"suoxie": "PHYSYNDKZ", "name": "\u9e4f\u534e\u6c38\u76db\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGSHENGYINIANDINGKAIZHAI"}, "003663": {"suoxie": "PHXTDQKFHH", "name": "\u9e4f\u534e\u5174\u6cf0\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAXINGTAIDINGQIKAIFANGHUNHE"}, "003664": {"suoxie": "XWTLCZA", "name": "\u65b0\u6c83\u901a\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINWOTONGLICHUNZHAIA"}, "003665": {"suoxie": "XWTLCZC", "name": "\u65b0\u6c83\u901a\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINWOTONGLICHUNZHAIC"}, "003668": {"suoxie": "DFHYXCZA", "name": "\u4e1c\u65b9\u7ea2\u76ca\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGYIXINCHUNZHAIA"}, "003669": {"suoxie": "DFHYXCZC", "name": "\u4e1c\u65b9\u7ea2\u76ca\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGYIXINCHUNZHAIC"}, "003670": {"suoxie": "ZRWLWZTLHPZHH", "name": "\u4e2d\u878d\u7269\u8054\u7f51\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGWULIANWANGZHUTILINGHUOPEIZHIHUNHE"}, "003671": {"suoxie": "XYYHZQ", "name": "\u5174\u4e1a\u88d5\u6052\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYUHENGZHAIQUAN"}, "003672": {"suoxie": "XYYHZQ", "name": "\u5174\u4e1a\u88d5\u534e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYUHUAZHAIQUAN"}, "003673": {"suoxie": "ZJFYCZZQ", "name": "\u4e2d\u52a0\u4e30\u88d5\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAFENGYUCHUNZHAIZHAIQUAN"}, "003674": {"suoxie": "RTTXZQ", "name": "\u878d\u901a\u901a\u73ba\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGXIZHAIQUAN"}, "003678": {"suoxie": "ZRXJZLHBA", "name": "\u4e2d\u878d\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGXIANJINZENGLIHUOBIA"}, "003679": {"suoxie": "ZRXJZLHBC", "name": "\u4e2d\u878d\u73b0\u91d1\u589e\u5229\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGXIANJINZENGLIHUOBIC"}, "003680": {"suoxie": "HRYDSXZQA", "name": "\u534e\u6da6\u5143\u5927\u53cc\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUARUNYUANDASHUANGXINZHAIQUANA"}, "003681": {"suoxie": "JXRXCZZQ", "name": "\u5efa\u4fe1\u777f\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIXIANGCHUNZHAIZHAIQUAN"}, "003682": {"suoxie": "BSAHYNDKZFQSA", "name": "\u535a\u65f6\u5b89\u5f18\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANHONGYINIANDINGKAIZHAIFAQISHIA"}, "003683": {"suoxie": "BSAHYNDKZFQSC", "name": "\u535a\u65f6\u5b89\u5f18\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANHONGYINIANDINGKAIZHAIFAQISHIC"}, "003684": {"suoxie": "HAFRHHA", "name": "\u6c47\u5b89\u4e30\u878d\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGRONGHUNHEA"}, "003685": {"suoxie": "HAFRHHC", "name": "\u6c47\u5b89\u4e30\u878d\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGRONGHUNHEC"}, "003692": {"suoxie": "DCJSLHPZHHA", "name": "\u5927\u6210\u666f\u5c1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJINGSHANGLINGHUOPEIZHIHUNHEA"}, "003693": {"suoxie": "DCJSLHPZHHC", "name": "\u5927\u6210\u666f\u5c1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJINGSHANGLINGHUOPEIZHIHUNHEC"}, "003696": {"suoxie": "GTRXDKZFQS", "name": "\u56fd\u6cf0\u6da6\u946b\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUNXINDINGKAIZHAIFAQISHI"}, "003697": {"suoxie": "HXRPTSHH", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u76db\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAISHENGHUNHE"}, "003703": {"suoxie": "BSFXCZ", "name": "\u535a\u65f6\u5bcc\u946b\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUXINCHUNZHAI"}, "003704": {"suoxie": "GDSJQDHH", "name": "\u5149\u5927\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDASHIJIANQUDONGHUNHE"}, "003708": {"suoxie": "BSMFCZA", "name": "\u535a\u65f6\u6c11\u4e30\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIMINFENGCHUNZHAIA"}, "003709": {"suoxie": "BSMFCZC", "name": "\u535a\u65f6\u6c11\u4e30\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIMINFENGCHUNZHAIC"}, "003711": {"suoxie": "TDHLJYBHBA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4eac\u5143\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIJINGYUANBAOHUOBIA"}, "003712": {"suoxie": "TDHLJYBHBB", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4eac\u5143\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIJINGYUANBAOHUOBIB"}, "003713": {"suoxie": "YDRSA", "name": "\u82f1\u5927\u777f\u76dbA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDARUISHENGA"}, "003714": {"suoxie": "YDRSC", "name": "\u82f1\u5927\u777f\u76dbC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINGDARUISHENGC"}, "003715": {"suoxie": "BYXFZTHH", "name": "\u5b9d\u76c8\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXIAOFEIZHUTIHUNHE"}, "003717": {"suoxie": "ZYLHJXHHA", "name": "\u4e2d\u94f6\u91cf\u5316\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINLIANGHUAJINGXUANHUNHEA"}, "003718": {"suoxie": "YFDBP500ZSMYHA", "name": "\u6613\u65b9\u8fbe\u6807\u666e500\u6307\u6570\u7f8e\u5143\u6c47A", "style": "QDII", "pinyin": "YIFANGDABIAOPU500ZHISHUMEIYUANHUIA"}, "003719": {"suoxie": "YFDBPYLBJMYHA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u533b\u7597\u4fdd\u5065\u7f8e\u5143\u6c47A", "style": "QDII", "pinyin": "YIFANGDABIAOPUYILIAOBAOJIANMEIYUANHUIA"}, "003720": {"suoxie": "YFDBPSWKJMYHA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u751f\u7269\u79d1\u6280\u7f8e\u5143\u6c47A", "style": "QDII", "pinyin": "YIFANGDABIAOPUSHENGWUKEJIMEIYUANHUIA"}, "003721": {"suoxie": "YBPXXKJMYHA", "name": "\u6613\u6807\u666e\u4fe1\u606f\u79d1\u6280\u7f8e\u5143\u6c47A", "style": "QDII", "pinyin": "YIBIAOPUXINXIKEJIMEIYUANHUIA"}, "003722": {"suoxie": "YFDNSDK100MYHA", "name": "\u6613\u65b9\u8fbe\u7eb3\u65af\u8fbe\u514b100\u7f8e\u5143\u6c47A", "style": "QDII", "pinyin": "YIFANGDANASIDAKE100MEIYUANHUIA"}, "003723": {"suoxie": "HRYDSXZQC", "name": "\u534e\u6da6\u5143\u5927\u53cc\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUARUNYUANDASHUANGXINZHAIQUANC"}, "003728": {"suoxie": "RTTCZQ", "name": "\u878d\u901a\u901a\u5bb8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGCHENZHAIQUAN"}, "003730": {"suoxie": "BSFHCZZQ", "name": "\u535a\u65f6\u5bcc\u534e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUHUACHUNZHAIZHAIQUAN"}, "003733": {"suoxie": "JYTYCZZQA", "name": "\u91d1\u9e70\u6dfb\u88d5\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANYUCHUNZHAIZHAIQUANA"}, "003734": {"suoxie": "WJRYLHPZHHA", "name": "\u4e07\u5bb6\u745e\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYINGLINGHUOPEIZHIHUNHEA"}, "003735": {"suoxie": "WJRYLHPZHHC", "name": "\u4e07\u5bb6\u745e\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYINGLINGHUOPEIZHIHUNHEC"}, "003739": {"suoxie": "XHXHLHPZHH", "name": "\u65b0\u534e\u946b\u5f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINHONGLINGHUOPEIZHIHUNHE"}, "003741": {"suoxie": "PHFYZQ", "name": "\u9e4f\u534e\u4e30\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGYINGZHAIQUAN"}, "003742": {"suoxie": "HAJHCZZQA", "name": "\u6c47\u5b89\u5609\u6c47\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIAHUICHUNZHAIZHAIQUANA"}, "003745": {"suoxie": "GFDYXXGP", "name": "\u5e7f\u53d1\u591a\u5143\u65b0\u5174\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFADUOYUANXINXINGGUPIAO"}, "003746": {"suoxie": "GFHR3GYDKZQ", "name": "\u5e7f\u53d1\u6c47\u745e3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIRUI3GEYUEDINGKAIZHAIQUAN"}, "003747": {"suoxie": "WJXXCZA", "name": "\u4e07\u5bb6\u946b\u4eab\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINXIANGCHUNZHAIA"}, "003748": {"suoxie": "WJXXCZC", "name": "\u4e07\u5bb6\u946b\u4eab\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINXIANGCHUNZHAIC"}, "003749": {"suoxie": "CJHXXSYHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINXINSHOUYIHUNHEA"}, "003750": {"suoxie": "CJHXXSYHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINXINSHOUYIHUNHEC"}, "003751": {"suoxie": "WJRLHH", "name": "\u4e07\u5bb6\u745e\u9686\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIARUILONGHUNHE"}, "003752": {"suoxie": "GYRYHBA", "name": "\u5de5\u94f6\u5982\u610f\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINRUYIHUOBIA"}, "003753": {"suoxie": "GYRYHBB", "name": "\u5de5\u94f6\u5982\u610f\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINRUYIHUOBIB"}, "003754": {"suoxie": "GTPYHHA", "name": "\u56fd\u6cf0\u666e\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIPUYIHUNHEA"}, "003755": {"suoxie": "GTPYHHC", "name": "\u56fd\u6cf0\u666e\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIPUYIHUNHEC"}, "003760": {"suoxie": "GTZZ500ZSZQA", "name": "\u56fd\u6cf0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG500ZHISHUZENGQIANGA"}, "003761": {"suoxie": "GTZZ500ZSZQC", "name": "\u56fd\u6cf0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG500ZHISHUZENGQIANGC"}, "003765": {"suoxie": "GFCYBETFLJA", "name": "\u5e7f\u53d1\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFACHUANGYEBANETFLIANJIEA"}, "003766": {"suoxie": "GFCYBETFLJC", "name": "\u5e7f\u53d1\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFACHUANGYEBANETFLIANJIEC"}, "003767": {"suoxie": "TDHLCLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u7eaf\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLICHUNLIZHAIQUANA"}, "003768": {"suoxie": "TDHLCLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u7eaf\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLICHUNLIZHAIQUANC"}, "003769": {"suoxie": "ZYPZSHHH", "name": "\u4e2d\u94f6\u54c1\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINPINZHISHENGHUOHUNHE"}, "003770": {"suoxie": "ZYFQDQKFZQ", "name": "\u4e2d\u94f6\u4e30\u5e86\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGQINGDINGQIKAIFANGZHAIQUAN"}, "003776": {"suoxie": "NFXLDKZA", "name": "\u5357\u65b9\u5ba3\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXUANLIDINGKAIZHAIA"}, "003777": {"suoxie": "NFXLDKZC", "name": "\u5357\u65b9\u5ba3\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXUANLIDINGKAIZHAIC"}, "003780": {"suoxie": "PHXYDQKFHH", "name": "\u9e4f\u534e\u5174\u60a6\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAXINGYUEDINGQIKAIFANGHUNHE"}, "003787": {"suoxie": "FZFBHLCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u60e0\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHUILICHUNZHAIA"}, "003788": {"suoxie": "FZFBHLCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u60e0\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHUILICHUNZHAIC"}, "003792": {"suoxie": "MSJYXJBHBC", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINBAOHUOBIC"}, "003793": {"suoxie": "TDHLYLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6ea2\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIYILIZHAIQUANA"}, "003794": {"suoxie": "TDHLYLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6ea2\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIYILIZHAIQUANC"}, "003795": {"suoxie": "FZFBRLCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u777f\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGRUILICHUNZHAIA"}, "003796": {"suoxie": "FZFBRLCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u777f\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGRUILICHUNZHAIC"}, "003797": {"suoxie": "HAXRLHHA", "name": "\u534e\u5b89\u65b0\u745e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINRUILIHUNHEA"}, "003798": {"suoxie": "HAXRLHHC", "name": "\u534e\u5b89\u65b0\u745e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINRUILIHUNHEC"}, "003799": {"suoxie": "HAXTLLHPZHHA", "name": "\u534e\u5b89\u65b0\u6cf0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINTAILILINGHUOPEIZHIHUNHEA"}, "003800": {"suoxie": "HAXTLLHPZHHC", "name": "\u534e\u5b89\u65b0\u6cf0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINTAILILINGHUOPEIZHIHUNHEC"}, "003803": {"suoxie": "HAXFLLHPZHHA", "name": "\u534e\u5b89\u65b0\u4e30\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINFENGLILINGHUOPEIZHIHUNHEA"}, "003804": {"suoxie": "HAXFLLHPZHHC", "name": "\u534e\u5b89\u65b0\u4e30\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINFENGLILINGHUOPEIZHIHUNHEC"}, "003805": {"suoxie": "HAXHLHHA", "name": "\u534e\u5b89\u65b0\u6052\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINHENGLIHUNHEA"}, "003806": {"suoxie": "HAXHLHHC", "name": "\u534e\u5b89\u65b0\u6052\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINHENGLIHUNHEC"}, "003809": {"suoxie": "ZSZSCZA", "name": "\u62db\u5546\u62db\u987a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOSHUNCHUNZHAIA"}, "003810": {"suoxie": "ZSZSCZC", "name": "\u62db\u5546\u62db\u987a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOSHUNCHUNZHAIC"}, "003811": {"suoxie": "ZJJLA", "name": "\u4e2d\u91d1\u91d1\u5229A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINLIA"}, "003812": {"suoxie": "ZJJLC", "name": "\u4e2d\u91d1\u91d1\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINLIC"}, "003813": {"suoxie": "TKJT3YDKHH", "name": "\u6cf0\u5eb7\u91d1\u6cf03\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGJINTAI3YUEDINGKAIHUNHE"}, "003816": {"suoxie": "YHRLB", "name": "\u94f6\u534e\u65e5\u5229B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUARILIB"}, "003819": {"suoxie": "GFJHCZ", "name": "\u5e7f\u53d1\u666f\u534e\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGHUACHUNZHAI"}, "003822": {"suoxie": "ZXJTLHHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u8f6e\u6362\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULUNHUANHUNHEA"}, "003823": {"suoxie": "ZXJTLHHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u8f6e\u6362\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULUNHUANHUNHEC"}, "003824": {"suoxie": "THXLZQA", "name": "\u5929\u5f18\u4fe1\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINLIZHAIQUANA"}, "003825": {"suoxie": "THXLZQC", "name": "\u5929\u5f18\u4fe1\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINLIZHAIQUANC"}, "003828": {"suoxie": "PHXHDQKFHH", "name": "\u9e4f\u534e\u5174\u60e0\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAXINGHUIDINGQIKAIFANGHUNHE"}, "003831": {"suoxie": "JXXRHBLHPZHH", "name": "\u5efa\u4fe1\u946b\u745e\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINRUIHUIBAOLINGHUOPEIZHIHUNHE"}, "003832": {"suoxie": "ZYFRDQKFZQ", "name": "\u4e2d\u94f6\u4e30\u6da6\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGRUNDINGQIKAIFANGZHAIQUAN"}, "003834": {"suoxie": "HXNYGXGPA", "name": "\u534e\u590f\u80fd\u6e90\u9769\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIANENGYUANGEXINGUPIAOA"}, "003835": {"suoxie": "PHHSGXXCZHH", "name": "\u9e4f\u534e\u6caa\u6df1\u6e2f\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHUSHENGANGXINXINGCHENGZHANGHUNHE"}, "003837": {"suoxie": "DFZXCZZQA", "name": "\u4e1c\u65b9\u81fb\u4eab\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENXIANGCHUNZHAIZHAIQUANA"}, "003838": {"suoxie": "DFZXCZZQC", "name": "\u4e1c\u65b9\u81fb\u4eab\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENXIANGCHUNZHAIZHAIQUANC"}, "003839": {"suoxie": "YFDRTLHPZHHA", "name": "\u6613\u65b9\u8fbe\u745e\u901a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUITONGLINGHUOPEIZHIHUNHEA"}, "003840": {"suoxie": "YFDRTLHPZHHC", "name": "\u6613\u65b9\u8fbe\u745e\u901a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUITONGLINGHUOPEIZHIHUNHEC"}, "003841": {"suoxie": "DCHYDKCZZQ", "name": "\u5927\u6210\u60e0\u88d5\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIYUDINGKAICHUNZHAIZHAIQUAN"}, "003842": {"suoxie": "ZYJTLHPZHHA", "name": "\u4e2d\u90ae\u666f\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUJINGTAILINGHUOPEIZHIHUNHEA"}, "003843": {"suoxie": "ZYJTLHPZHHC", "name": "\u4e2d\u90ae\u666f\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUJINGTAILINGHUOPEIZHIHUNHEC"}, "003845": {"suoxie": "HAFHLHPZHHA", "name": "\u6c47\u5b89\u4e30\u6052\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGHENGLINGHUOPEIZHIHUNHEA"}, "003846": {"suoxie": "HAFHLHPZHHC", "name": "\u6c47\u5b89\u4e30\u6052\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGHENGLINGHUOPEIZHIHUNHEC"}, "003847": {"suoxie": "HADFZQ", "name": "\u534e\u5b89\u9f0e\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANDINGFENGZHAIQUAN"}, "003848": {"suoxie": "ZYGLLHPZHHA", "name": "\u4e2d\u94f6\u5e7f\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINGUANGLILINGHUOPEIZHIHUNHEA"}, "003849": {"suoxie": "ZYGLLHPZHHC", "name": "\u4e2d\u94f6\u5e7f\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINGUANGLILINGHUOPEIZHIHUNHEC"}, "003850": {"suoxie": "ZYJLLHPZHHA", "name": "\u4e2d\u94f6\u9526\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINJINLILINGHUOPEIZHIHUNHEA"}, "003851": {"suoxie": "ZYJLLHPZHHC", "name": "\u4e2d\u94f6\u9526\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINJINLILINGHUOPEIZHIHUNHEC"}, "003853": {"suoxie": "JYXXCYGPA", "name": "\u91d1\u9e70\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGXINXICHANYEGUPIAOA"}, "003854": {"suoxie": "HAFHHHA", "name": "\u6c47\u5b89\u4e30\u534e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGHUAHUNHEA"}, "003855": {"suoxie": "HAFHHHC", "name": "\u6c47\u5b89\u4e30\u534e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGHUAHUNHEC"}, "003857": {"suoxie": "QHKYZQYXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u5468\u671f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZHOUQIYOUXUANHUNHEA"}, "003858": {"suoxie": "QHKYZQYXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u5468\u671f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZHOUQIYOUXUANHUNHEC"}, "003859": {"suoxie": "ZSZXCZA", "name": "\u62db\u5546\u62db\u65ed\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXUCHUNZHAIA"}, "003860": {"suoxie": "ZSZXCZC", "name": "\u62db\u5546\u62db\u65ed\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXUCHUNZHAIC"}, "003861": {"suoxie": "ZSXFHHA", "name": "\u62db\u5546\u5174\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGXINGFUHUNHEA"}, "003862": {"suoxie": "ZSXFHHC", "name": "\u62db\u5546\u5174\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGXINGFUHUNHEC"}, "003863": {"suoxie": "ZSZXCZA", "name": "\u62db\u5546\u62db\u7965\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXIANGCHUNZHAIA"}, "003864": {"suoxie": "ZSZXCZC", "name": "\u62db\u5546\u62db\u7965\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXIANGCHUNZHAIC"}, "003865": {"suoxie": "CJHXLHDYZGPC", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u591a\u56e0\u5b50\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINLIANGHUADUOYINZIGUPIAOC"}, "003866": {"suoxie": "BSFCCZZQ", "name": "\u535a\u65f6\u5bcc\u8bda\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUCHENGCHUNZHAIZHAIQUAN"}, "003867": {"suoxie": "ZSZJCZA", "name": "\u62db\u5546\u62db\u666f\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOJINGCHUNZHAIA"}, "003868": {"suoxie": "ZSZJCZC", "name": "\u62db\u5546\u62db\u666f\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOJINGCHUNZHAIC"}, "003869": {"suoxie": "CXWSCZ", "name": "\u957f\u4fe1\u7a33\u52bf\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENSHICHUNZHAI"}, "003871": {"suoxie": "HTBRTTBHBB", "name": "\u534e\u6cf0\u67cf\u745e\u5929\u6dfb\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUITIANTIANBAOHUOBIB"}, "003874": {"suoxie": "ZSRTJHBA", "name": "\u6d59\u5546\u65e5\u6dfb\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHESHANGRITIANJINHUOBIA"}, "003875": {"suoxie": "ZSRTJHBB", "name": "\u6d59\u5546\u65e5\u6dfb\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHESHANGRITIANJINHUOBIB"}, "003876": {"suoxie": "HBHS300ZSZQA", "name": "\u534e\u5b9d\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOHUSHEN300ZHISHUZENGQIANGA"}, "003877": {"suoxie": "FGJLWJPZHHA", "name": "\u5bcc\u56fd\u4e45\u5229\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOJIULIWENJIANPEIZHIHUNHEA"}, "003878": {"suoxie": "FGJLWJPZHHC", "name": "\u5bcc\u56fd\u4e45\u5229\u7a33\u5065\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOJIULIWENJIANPEIZHIHUNHEC"}, "003879": {"suoxie": "JS6GYLCZQA", "name": "\u5609\u5b9e6\u4e2a\u6708\u7406\u8d22\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHI6GEYUELICAIZHAIQUANA"}, "003880": {"suoxie": "JSWJ", "name": "\u5609\u5b9e\u7a33\u9a8f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENJUN"}, "003882": {"suoxie": "YFDRHHHA", "name": "\u6613\u65b9\u8fbe\u745e\u5f18\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIHONGHUNHEA"}, "003883": {"suoxie": "YFDRHHHC", "name": "\u6613\u65b9\u8fbe\u745e\u5f18\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIHONGHUNHEC"}, "003884": {"suoxie": "HAHS300ZSZQA", "name": "\u6c47\u5b89\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANHUSHEN300ZHISHUZENGQIANGA"}, "003885": {"suoxie": "HAHS300ZSZQC", "name": "\u6c47\u5b89\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANHUSHEN300ZHISHUZENGQIANGC"}, "003886": {"suoxie": "HAFLHHA", "name": "\u6c47\u5b89\u4e30\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGLIHUNHEA"}, "003887": {"suoxie": "HAFLHHC", "name": "\u6c47\u5b89\u4e30\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGLIHUNHEC"}, "003888": {"suoxie": "HAJYCZZQ", "name": "\u6c47\u5b89\u5609\u6e90\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIAYUANCHUNZHAIZHAIQUAN"}, "003889": {"suoxie": "HAFZHHA", "name": "\u6c47\u5b89\u4e30\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGZEHUNHEA"}, "003890": {"suoxie": "HAFZHHC", "name": "\u6c47\u5b89\u4e30\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGZEHUNHEC"}, "003891": {"suoxie": "HAJYCZZQ", "name": "\u6c47\u5b89\u5609\u88d5\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIAYUCHUNZHAIZHAIQUAN"}, "003898": {"suoxie": "YYFYZQ", "name": "\u6c38\u8d62\u4e30\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGFENGYIZHAIQUAN"}, "003900": {"suoxie": "JYRXDKHH", "name": "\u4ea4\u94f6\u745e\u946b\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINRUIXINDINGKAIHUNHE"}, "003922": {"suoxie": "CSSKCZZQA", "name": "\u957f\u76db\u76db\u5eb7\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGKANGCHUNZHAIZHAIQUANA"}, "003923": {"suoxie": "CSSKCZZQC", "name": "\u957f\u76db\u76db\u5eb7\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGKANGCHUNZHAIZHAIQUANC"}, "003926": {"suoxie": "ZRHXCZA", "name": "\u4e2d\u878d\u6052\u4fe1\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGXINCHUNZHAIA"}, "003927": {"suoxie": "ZRHXCZC", "name": "\u4e2d\u878d\u6052\u4fe1\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGXINCHUNZHAIC"}, "003928": {"suoxie": "QHLHYXCZA", "name": "\u524d\u6d77\u8054\u5408\u6c38\u5174\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGXINGCHUNZHAIA"}, "003929": {"suoxie": "ZYZQAJZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u8fdb\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANJINZHAIQUANA"}, "003930": {"suoxie": "ZYZQAJZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u8fdb\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANJINZHAIQUANC"}, "003938": {"suoxie": "NFRZHHA", "name": "\u5357\u65b9\u8363\u5c0a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGRONGZUNHUNHEA"}, "003939": {"suoxie": "NFRZHHC", "name": "\u5357\u65b9\u8363\u5c0a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGRONGZUNHUNHEC"}, "003940": {"suoxie": "YHSSJXLHPZHHFQSA", "name": "\u94f6\u534e\u76db\u4e16\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUASHENGSHIJINGXUANLINGHUOPEIZHIHUNHEFAQISHIA"}, "003949": {"suoxie": "XQWTZQA", "name": "\u5174\u5168\u7a33\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANWENTAIZHAIQUANA"}, "003950": {"suoxie": "BSXRHHA", "name": "\u535a\u65f6\u946b\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINRUNHUNHEA"}, "003951": {"suoxie": "BSXRHHC", "name": "\u535a\u65f6\u946b\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINRUNHUNHEC"}, "003952": {"suoxie": "XYJR6GYDKZA", "name": "\u5174\u4e1a\u5609\u745e6\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIARUI6GEYUEDINGKAIZHAIA"}, "003953": {"suoxie": "XYJR6GYDKZC", "name": "\u5174\u4e1a\u5609\u745e6\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIARUI6GEYUEDINGKAIZHAIC"}, "003955": {"suoxie": "GTMFHBDKHH", "name": "\u56fd\u6cf0\u6c11\u4e30\u56de\u62a5\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINFENGHUIBAODINGKAIHUNHE"}, "003956": {"suoxie": "NFCYZXGP", "name": "\u5357\u65b9\u4ea7\u4e1a\u667a\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGCHANYEZHIXUANGUPIAO"}, "003957": {"suoxie": "AXLHJXHS300ZQA", "name": "\u5b89\u4fe1\u91cf\u5316\u7cbe\u9009\u6caa\u6df1300\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINLIANGHUAJINGXUANHUSHEN300ZENGQIANGA"}, "003958": {"suoxie": "AXLHJXHS300ZQC", "name": "\u5b89\u4fe1\u91cf\u5316\u7cbe\u9009\u6caa\u6df1300\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINLIANGHUAJINGXUANHUSHEN300ZENGQIANGC"}, "003961": {"suoxie": "YFDRCLHPZHHA", "name": "\u6613\u65b9\u8fbe\u745e\u7a0b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICHENGLINGHUOPEIZHIHUNHEA"}, "003962": {"suoxie": "YFDRCLHPZHHC", "name": "\u6613\u65b9\u8fbe\u745e\u7a0b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICHENGLINGHUOPEIZHIHUNHEC"}, "003963": {"suoxie": "BSHXCZDKZ", "name": "\u535a\u65f6\u6167\u9009\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIHUIXUANCHUNZHAIDINGKAIZHAI"}, "003966": {"suoxie": "ZYRLHHA", "name": "\u4e2d\u94f6\u6da6\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINRUNLIHUNHEA"}, "003967": {"suoxie": "ZYRLHHC", "name": "\u4e2d\u94f6\u6da6\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINRUNLIHUNHEC"}, "003968": {"suoxie": "JYTYBHBA", "name": "\u4ea4\u94f6\u5929\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANYIBAOHUOBIA"}, "003969": {"suoxie": "JYTYBHBE", "name": "\u4ea4\u94f6\u5929\u76ca\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINTIANYIBAOHUOBIE"}, "003972": {"suoxie": "GFMYZDKZRMB", "name": "\u56fd\u5bcc\u7f8e\u5143\u503a\u5b9a\u5f00\u503a\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GUOFUMEIYUANZHAIDINGKAIZHAIRENMINBI"}, "003973": {"suoxie": "GFMYZDKZMYXH", "name": "\u56fd\u5bcc\u7f8e\u5143\u503a\u5b9a\u5f00\u503a\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "GUOFUMEIYUANZHAIDINGKAIZHAIMEIYUANXIANHUI"}, "003978": {"suoxie": "ZXJTWXA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u7965A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENXIANGA"}, "003979": {"suoxie": "ZXJTWXC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u7965C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENXIANGC"}, "003980": {"suoxie": "ZYZQRYLHPZHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u745e\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANRUIYILINGHUOPEIZHIHUNHEA"}, "003981": {"suoxie": "ZYZQRYLHPZHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u745e\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANRUIYILINGHUOPEIZHIHUNHEC"}, "003983": {"suoxie": "PHFHZQ", "name": "\u9e4f\u534e\u4e30\u60e0\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGHUIZHAIQUAN"}, "003984": {"suoxie": "JSXNYXCLGPA", "name": "\u5609\u5b9e\u65b0\u80fd\u6e90\u65b0\u6750\u6599\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXINNENGYUANXINCAILIAOGUPIAOA"}, "003985": {"suoxie": "JSXNYXCLGPC", "name": "\u5609\u5b9e\u65b0\u80fd\u6e90\u65b0\u6750\u6599\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXINNENGYUANXINCAILIAOGUPIAOC"}, "003986": {"suoxie": "SWLXZZ500ZSYXZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1500\u6307\u6570\u4f18\u9009\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENG500ZHISHUYOUXUANZENGQIANGA"}, "003993": {"suoxie": "QHKYHGSHXQDHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u6838\u5fc3\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENHEXINQUDONGHUNHE"}, "003994": {"suoxie": "HFHBB", "name": "\u534e\u5bcc\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUHUOBIB"}, "003997": {"suoxie": "HDTLHBA", "name": "\u6cd3\u5fb7\u6dfb\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGDETIANLIHUOBIA"}, "003998": {"suoxie": "HDTLHBB", "name": "\u6cd3\u5fb7\u6dfb\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGDETIANLIHUOBIB"}, "003999": {"suoxie": "FRFXCZ", "name": "\u5bcc\u8363\u5bcc\u7965\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUXIANGCHUNZHAI"}, "004001": {"suoxie": "TDHLHLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIHENGLIZHAIQUANA"}, "004002": {"suoxie": "TDHLHLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIHENGLIZHAIQUANC"}, "004005": {"suoxie": "DFMFHBYAHHA", "name": "\u4e1c\u65b9\u6c11\u4e30\u56de\u62a5\u8d62\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGMINFENGHUIBAOYINGANHUNHEA"}, "004006": {"suoxie": "DFMFHBYAHHC", "name": "\u4e1c\u65b9\u6c11\u4e30\u56de\u62a5\u8d62\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGMINFENGHUIBAOYINGANHUNHEC"}, "004008": {"suoxie": "ZRXSLHHA", "name": "\u4e2d\u878d\u946b\u601d\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINSILUHUNHEA"}, "004009": {"suoxie": "ZRXSLHHC", "name": "\u4e2d\u878d\u946b\u601d\u8def\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINSILUHUNHEC"}, "004010": {"suoxie": "HTBRDLLHPZHHA", "name": "\u534e\u6cf0\u67cf\u745e\u9f0e\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIDINGLILINGHUOPEIZHIHUNHEA"}, "004011": {"suoxie": "HTBRDLLHPZHHC", "name": "\u534e\u6cf0\u67cf\u745e\u9f0e\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIDINGLILINGHUOPEIZHIHUNHEC"}, "004020": {"suoxie": "GFJXCZ", "name": "\u5e7f\u53d1\u666f\u7965\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGXIANGCHUNZHAI"}, "004021": {"suoxie": "GFHFYNDQZQA", "name": "\u5e7f\u53d1\u6c47\u5bcc\u4e00\u5e74\u5b9a\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIFUYINIANDINGQIZHAIQUANA"}, "004022": {"suoxie": "GFHFYNDQZQC", "name": "\u5e7f\u53d1\u6c47\u5bcc\u4e00\u5e74\u5b9a\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIFUYINIANDINGQIZHAIQUANC"}, "004024": {"suoxie": "HTBXZCYNDKZ", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u8bda\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNCHENGYINIANDINGKAIZHAI"}, "004025": {"suoxie": "RTSYZQZQA", "name": "\u878d\u901a\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGSHOUYIZENGQIANGZHAIQUANA"}, "004026": {"suoxie": "RTSYZQZQC", "name": "\u878d\u901a\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGSHOUYIZENGQIANGZHAIQUANC"}, "004027": {"suoxie": "GFJYCZA", "name": "\u5e7f\u53d1\u666f\u6e90\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGYUANCHUNZHAIA"}, "004028": {"suoxie": "GFJYCZC", "name": "\u5e7f\u53d1\u666f\u6e90\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGYUANCHUNZHAIC"}, "004030": {"suoxie": "JSFA6GYDQZQ", "name": "\u5609\u5b9e\u4e30\u5b896\u4e2a\u6708\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIFENGAN6GEYUEDINGQIZHAIQUAN"}, "004031": {"suoxie": "XYTLSGYDKZ", "name": "\u946b\u5143\u6dfb\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "XINYUANTIANLISANGEYUEDINGKAIZHAI"}, "004032": {"suoxie": "GYFCBNDKZQ", "name": "\u5de5\u94f6\u4e30\u6df3\u534a\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINFENGCHUNBANNIANDINGKAIZHAIQUAN"}, "004033": {"suoxie": "JYTRCZZQA", "name": "\u91d1\u9e70\u6dfb\u8363\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANRONGCHUNZHAIZHAIQUANA"}, "004038": {"suoxie": "ZYFXDKZ", "name": "\u4e2d\u94f6\u5bcc\u4eab\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFUXIANGDINGKAIZHAI"}, "004039": {"suoxie": "ZOJTHB", "name": "\u4e2d\u6b27\u9a8f\u6cf0\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUJUNTAIHUOBI"}, "004040": {"suoxie": "JYYLJKCYA", "name": "\u91d1\u9e70\u533b\u7597\u5065\u5eb7\u4ea7\u4e1aA", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGYILIAOJIANKANGCHANYEA"}, "004041": {"suoxie": "JYYLJKCYC", "name": "\u91d1\u9e70\u533b\u7597\u5065\u5eb7\u4ea7\u4e1aC", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGYILIAOJIANKANGCHANYEC"}, "004042": {"suoxie": "HXDMZQA", "name": "\u534e\u590f\u9f0e\u8302\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGMAOZHAIQUANA"}, "004043": {"suoxie": "HXDMZQC", "name": "\u534e\u590f\u9f0e\u8302\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGMAOZHAIQUANC"}, "004044": {"suoxie": "JYZXDLHH", "name": "\u91d1\u9e70\u8f6c\u578b\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGZHUANXINGDONGLIHUNHE"}, "004045": {"suoxie": "JYTRDKZ", "name": "\u91d1\u9e70\u6dfb\u6da6\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANRUNDINGKAIZHAI"}, "004046": {"suoxie": "HXXJSHHA", "name": "\u534e\u590f\u65b0\u9526\u987a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINSHUNHUNHEA"}, "004047": {"suoxie": "HXXJSHHC", "name": "\u534e\u590f\u65b0\u9526\u987a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINSHUNHUNHEC"}, "004048": {"suoxie": "HXXJHHHA", "name": "\u534e\u590f\u65b0\u9526\u6c47\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINHUIHUNHEA"}, "004049": {"suoxie": "HXXJHHHC", "name": "\u534e\u590f\u65b0\u9526\u6c47\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINHUIHUNHEC"}, "004050": {"suoxie": "HXXJSHHA", "name": "\u534e\u590f\u65b0\u9526\u5347\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINSHENGHUNHEA"}, "004051": {"suoxie": "HXXJSHHC", "name": "\u534e\u590f\u65b0\u9526\u5347\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJINSHENGHUNHEC"}, "004052": {"suoxie": "HXDZZQA", "name": "\u534e\u590f\u9f0e\u667a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGZHIZHAIQUANA"}, "004053": {"suoxie": "HXDZZQC", "name": "\u534e\u590f\u9f0e\u667a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGZHIZHAIQUANC"}, "004056": {"suoxie": "HXHLHBA", "name": "\u534e\u590f\u60e0\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAHUILIHUOBIA"}, "004059": {"suoxie": "XYZLZQ", "name": "\u946b\u5143\u62db\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHAOLIZHAIQUAN"}, "004060": {"suoxie": "BSXSHBB", "name": "\u535a\u65f6\u5174\u76db\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXINGSHENGHUOBIB"}, "004061": {"suoxie": "HXDLZQA", "name": "\u534e\u590f\u9f0e\u9686\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLONGZHAIQUANA"}, "004062": {"suoxie": "HXDLZQC", "name": "\u534e\u590f\u9f0e\u9686\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLONGZHAIQUANC"}, "004063": {"suoxie": "HXHRZQ", "name": "\u534e\u590f\u6052\u878d\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAHENGRONGZHAIQUAN"}, "004066": {"suoxie": "JSWXCZZQ", "name": "\u5609\u5b9e\u7a33\u7199\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENXICHUNZHAIZHAIQUAN"}, "004069": {"suoxie": "NFZZQZZQGSETFLJA", "name": "\u5357\u65b9\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "004070": {"suoxie": "NFZZQZZQGSETFLJC", "name": "\u5357\u65b9\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "004072": {"suoxie": "JYSAJTBHBA", "name": "\u91d1\u5143\u987a\u5b89\u91d1\u901a\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYUANSHUNANJINTONGBAOHUOBIA"}, "004073": {"suoxie": "JYSAJTBHBB", "name": "\u91d1\u5143\u987a\u5b89\u91d1\u901a\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYUANSHUNANJINTONGBAOHUOBIB"}, "004075": {"suoxie": "JYYYCXGPA", "name": "\u4ea4\u94f6\u533b\u836f\u521b\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIAOYINYIYAOCHUANGXINGUPIAOA"}, "004076": {"suoxie": "GLARYCZHH", "name": "\u56fd\u8054\u5b89\u9510\u610f\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANRUIYICHENGZHANGHUNHE"}, "004077": {"suoxie": "JXMFHBA", "name": "\u91d1\u4fe1\u6c11\u53d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINXINMINFAHUOBIA"}, "004078": {"suoxie": "JXMFHBB", "name": "\u91d1\u4fe1\u6c11\u53d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINXINMINFAHUOBIB"}, "004079": {"suoxie": "WJXFCZA", "name": "\u4e07\u5bb6\u946b\u4e30\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINFENGCHUNZHAIA"}, "004080": {"suoxie": "WJXFCZC", "name": "\u4e07\u5bb6\u946b\u4e30\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINFENGCHUNZHAIC"}, "004081": {"suoxie": "GLAXQHHA", "name": "\u56fd\u8054\u5b89\u946b\u4e7e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINQIANHUNHEA"}, "004082": {"suoxie": "GLAXQHHC", "name": "\u56fd\u8054\u5b89\u946b\u4e7e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINQIANHUNHEC"}, "004083": {"suoxie": "GLAXLHHA", "name": "\u56fd\u8054\u5b89\u946b\u9686\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINLONGHUNHEA"}, "004084": {"suoxie": "GLAXLHHC", "name": "\u56fd\u8054\u5b89\u946b\u9686\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINLONGHUNHEC"}, "004087": {"suoxie": "YHTRDQKFZQ", "name": "\u94f6\u534e\u6dfb\u6da6\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUATIANRUNDINGQIKAIFANGZHAIQUAN"}, "004089": {"suoxie": "HTFXRZQA", "name": "\u6c47\u6dfb\u5bcc\u946b\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINRUIZHAIQUANA"}, "004090": {"suoxie": "HTFXRZQC", "name": "\u6c47\u6dfb\u5bcc\u946b\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINRUIZHAIQUANC"}, "004091": {"suoxie": "BSHGSJZYXA", "name": "\u535a\u65f6\u6caa\u6e2f\u6df1\u4ef7\u503c\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUGANGSHENJIAZHIYOUXUANA"}, "004092": {"suoxie": "BSHGSJZYXC", "name": "\u535a\u65f6\u6caa\u6e2f\u6df1\u4ef7\u503c\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUGANGSHENJIAZHIYOUXUANC"}, "004093": {"suoxie": "JYSAASZQA", "name": "\u91d1\u5143\u987a\u5b89\u6849\u76db\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYUANSHUNANANSHENGZHAIQUANA"}, "004097": {"suoxie": "NYHLRRXJYXHBA", "name": "\u519c\u94f6\u6c47\u7406\u65e5\u65e5\u946b\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHUILIRIRIXINJIAOYIXINGHUOBIA"}, "004098": {"suoxie": "QHKYGGTGXL50Q", "name": "\u524d\u6d77\u5f00\u6e90\u6e2f\u80a1\u901a\u80a1\u606f\u738750\u5f3a", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANGANGGUTONGGUXILV50QIANG"}, "004099": {"suoxie": "QHKYHGSJQHYJXHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u666f\u6c14\u884c\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENJINGQIHANGYEJINGXUANHUNHE"}, "004100": {"suoxie": "PHAYZQHH", "name": "\u9e4f\u534e\u5b89\u76ca\u589e\u5f3a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYIZENGQIANGHUNHE"}, "004101": {"suoxie": "GTMAZYCZA", "name": "\u56fd\u6cf0\u6c11\u5b89\u589e\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIMINANZENGYICHUNZHAIA"}, "004102": {"suoxie": "XCWYZQA", "name": "\u4fe1\u8bda\u7a33\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENYUEZHAIQUANA"}, "004103": {"suoxie": "XCWYZQC", "name": "\u4fe1\u8bda\u7a33\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENYUEZHAIQUANC"}, "004104": {"suoxie": "XCWXA", "name": "\u4fe1\u8bda\u7a33\u946bA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENXINA"}, "004105": {"suoxie": "XCWXC", "name": "\u4fe1\u8bda\u7a33\u946bC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENXINC"}, "004106": {"suoxie": "ZXBCWFA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u4e30A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENFENGA"}, "004107": {"suoxie": "ZXBCWFC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u4e30C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENFENGC"}, "004108": {"suoxie": "XCWTA", "name": "\u4fe1\u8bda\u7a33\u6cf0A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENTAIA"}, "004109": {"suoxie": "XCWTC", "name": "\u4fe1\u8bda\u7a33\u6cf0C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGWENTAIC"}, "004112": {"suoxie": "CJHXGQHLHH", "name": "\u521b\u91d1\u5408\u4fe1\u56fd\u4f01\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINGUOQIHUOLIHUNHE"}, "004117": {"suoxie": "DCHXCZZQ", "name": "\u5927\u6210\u60e0\u7965\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIXIANGCHUNZHAIZHAIQUAN"}, "004118": {"suoxie": "BSYPCZZQ", "name": "\u535a\u65f6\u88d5\u9e4f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIYUPENGCHUNZHAIZHAIQUAN"}, "004119": {"suoxie": "GFCXQDLHPZHH", "name": "\u5e7f\u53d1\u521b\u65b0\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFACHUANGXINQUDONGLINGHUOPEIZHIHUNHE"}, "004120": {"suoxie": "GFAXHB", "name": "\u56fd\u5bcc\u5b89\u4eab\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOFUANXIANGHUOBI"}, "004121": {"suoxie": "XYXJTL", "name": "\u5174\u94f6\u73b0\u91d1\u6dfb\u5229", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYINXIANJINTIANLI"}, "004122": {"suoxie": "XYCYSGYDKZ", "name": "\u5174\u94f6\u957f\u76ca\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINCHANGYISANGEYUEDINGKAIZHAI"}, "004123": {"suoxie": "XYCYSGYDKZ", "name": "\u5174\u94f6\u957f\u76c8\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINCHANGYINGSANGEYUEDINGKAIZHAI"}, "004124": {"suoxie": "MSJYXSCZ", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u5347\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINSHENGCHUNZHAI"}, "004126": {"suoxie": "PYASWJZLZQA", "name": "\u6d66\u94f6\u5b89\u76db\u7a33\u5065\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGWENJIANZENGLIZHAIQUANA"}, "004127": {"suoxie": "PHFKZQ", "name": "\u9e4f\u534e\u4e30\u5eb7\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGKANGZHAIQUAN"}, "004128": {"suoxie": "QHLHYLHHA", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u9686\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGLONGHUNHEA"}, "004129": {"suoxie": "GLAXHHHA", "name": "\u56fd\u8054\u5b89\u946b\u6c47\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINHUIHUNHEA"}, "004130": {"suoxie": "GLAXHHHC", "name": "\u56fd\u8054\u5b89\u946b\u6c47\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINHUIHUNHEC"}, "004131": {"suoxie": "GLAXFHHA", "name": "\u56fd\u8054\u5b89\u946b\u53d1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINFAHUNHEA"}, "004132": {"suoxie": "GLAXFHHC", "name": "\u56fd\u8054\u5b89\u946b\u53d1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINFAHUNHEC"}, "004133": {"suoxie": "ZHHXBHB", "name": "\u4e2d\u822a\u822a\u884c\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGHANGHANGXINGBAOHUOBI"}, "004134": {"suoxie": "NYJA18GYDKZ", "name": "\u519c\u94f6\u91d1\u5b8918\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINJINAN18GEYUEDINGKAIZHAI"}, "004135": {"suoxie": "SWLXLHCZHH", "name": "\u7533\u4e07\u83f1\u4fe1\u91cf\u5316\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANLINGXINLIANGHUACHENGZHANGHUNHE"}, "004136": {"suoxie": "BSMZCZZQ", "name": "\u535a\u65f6\u6c11\u6cfd\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIMINZECHUNZHAIZHAIQUAN"}, "004137": {"suoxie": "BSHHHBB", "name": "\u535a\u65f6\u5408\u60e0\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEHUIHUOBIB"}, "004138": {"suoxie": "SYXDHH", "name": "\u4e0a\u94f6\u946b\u8fbe\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGYINXINDAHUNHE"}, "004139": {"suoxie": "ZYJMRHLHPZHH", "name": "\u4e2d\u90ae\u519b\u6c11\u878d\u5408\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUJUNMINRONGHELINGHUOPEIZHIHUNHE"}, "004140": {"suoxie": "XYFXZQ", "name": "\u5174\u4e1a\u798f\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEFUXINZHAIQUAN"}, "004141": {"suoxie": "XYRF6GYDKZ", "name": "\u5174\u4e1a\u745e\u4e306\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYERUIFENG6GEYUEDINGKAIZHAI"}, "004142": {"suoxie": "ZSSHLHHHA", "name": "\u62db\u5546\u76db\u5408\u7075\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGSHENGHELINGHUOHUNHEA"}, "004143": {"suoxie": "ZSSHLHHHC", "name": "\u62db\u5546\u76db\u5408\u7075\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGSHENGHELINGHUOHUNHEC"}, "004144": {"suoxie": "STAFHBA", "name": "\u4e0a\u6295\u5b89\u4e30\u56de\u62a5A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUANFENGHUIBAOA"}, "004145": {"suoxie": "STAFHBC", "name": "\u4e0a\u6295\u5b89\u4e30\u56de\u62a5C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUANFENGHUIBAOC"}, "004148": {"suoxie": "YXYFDCL", "name": "\u5706\u4fe1\u6c38\u4e30\u591a\u7b56\u7565", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGDUOCELUE"}, "004149": {"suoxie": "BSXHHHA", "name": "\u535a\u65f6\u946b\u60e0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINHUIHUNHEA"}, "004150": {"suoxie": "BSXHHHC", "name": "\u535a\u65f6\u946b\u60e0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINHUIHUNHEC"}, "004151": {"suoxie": "XFRTLA", "name": "\u5148\u950b\u65e5\u6dfb\u5229A", "style": "\u8d27\u5e01\u578b", "pinyin": "XIANFENGRITIANLIA"}, "004152": {"suoxie": "XFRTLB", "name": "\u5148\u950b\u65e5\u6dfb\u5229B", "style": "\u8d27\u5e01\u578b", "pinyin": "XIANFENGRITIANLIB"}, "004153": {"suoxie": "XCXYHBLHPZHHA", "name": "\u4fe1\u8bda\u65b0\u60a6\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINYUEHUIBAOLINGHUOPEIZHIHUNHEA"}, "004154": {"suoxie": "XCXYHBLHPZHHB", "name": "\u4fe1\u8bda\u65b0\u60a6\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINYUEHUIBAOLINGHUOPEIZHIHUNHEB"}, "004155": {"suoxie": "ZXBCZTZDZZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u6cf0\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINBAOCHENGZHITAIZHONGDUANZHAIZHAIQUANA"}, "004156": {"suoxie": "ZXBCZTZDZZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u6cf0\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINBAOCHENGZHITAIZHONGDUANZHAIZHAIQUANC"}, "004157": {"suoxie": "XCZCLHPZHHA", "name": "\u4fe1\u8bda\u81f3\u8bda\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHICHENGLINGHUOPEIZHIHUNHEA"}, "004158": {"suoxie": "XCZCLHPZHHB", "name": "\u4fe1\u8bda\u81f3\u8bda\u7075\u6d3b\u914d\u7f6e\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGZHICHENGLINGHUOPEIZHIHUNHEB"}, "004166": {"suoxie": "DFJZWJLHPZHHA", "name": "\u4e1c\u65b9\u4ef7\u503c\u6316\u6398\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGJIAZHIWAJUELINGHUOPEIZHIHUNHEA"}, "004167": {"suoxie": "AXHQBHBB", "name": "\u5b89\u4fe1\u6d3b\u671f\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINHUOQIBAOHUOBIB"}, "004168": {"suoxie": "BSFJCZZQ", "name": "\u535a\u65f6\u5bcc\u5609\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUJIACHUNZHAIZHAIQUAN"}, "004169": {"suoxie": "WJXJZLHBA", "name": "\u4e07\u5bb6\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAXIANJINZENGLIHUOBIA"}, "004170": {"suoxie": "WJXJZLHBB", "name": "\u4e07\u5bb6\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAXIANJINZENGLIHUOBIB"}, "004173": {"suoxie": "JSZYBHB", "name": "\u5609\u5b9e\u589e\u76ca\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIZENGYIBAOHUOBI"}, "004175": {"suoxie": "BSXTHHA", "name": "\u535a\u65f6\u946b\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINTAIHUNHEA"}, "004176": {"suoxie": "BSXTHHC", "name": "\u535a\u65f6\u946b\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIXINTAIHUNHEC"}, "004178": {"suoxie": "YXYFFRHBA", "name": "\u5706\u4fe1\u6c38\u4e30\u4e30\u6da6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YUANXINYONGFENGFENGRUNHUOBIA"}, "004179": {"suoxie": "YXYFFRHBB", "name": "\u5706\u4fe1\u6c38\u4e30\u4e30\u6da6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YUANXINYONGFENGFENGRUNHUOBIB"}, "004180": {"suoxie": "NFHYDKZ", "name": "\u5357\u65b9\u5b8f\u5143\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHONGYUANDINGKAIZHAI"}, "004181": {"suoxie": "NFHYDKZQC", "name": "\u5357\u65b9\u5b8f\u5143\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHONGYUANDINGKAIZHAIQUANC"}, "004183": {"suoxie": "FGCYSJHHA", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHANYESHENGJIHUNHEA"}, "004185": {"suoxie": "JXZLHBA", "name": "\u6c5f\u4fe1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANGXINZENGLIHUOBIA"}, "004186": {"suoxie": "JXZLHBB", "name": "\u6c5f\u4fe1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANGXINZENGLIHUOBIB"}, "004189": {"suoxie": "HSXFHYGP", "name": "\u534e\u5546\u6d88\u8d39\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGXIAOFEIHANGYEGUPIAO"}, "004190": {"suoxie": "ZSHS300ZSZQA", "name": "\u62db\u5546\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300ZHISHUZENGQIANGA"}, "004191": {"suoxie": "ZSHS300ZSZQC", "name": "\u62db\u5546\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300ZHISHUZENGQIANGC"}, "004192": {"suoxie": "ZSZZ500ZSA", "name": "\u62db\u5546\u4e2d\u8bc1500\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG500ZHISHUA"}, "004193": {"suoxie": "ZSZZ500ZSC", "name": "\u62db\u5546\u4e2d\u8bc1500\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG500ZHISHUC"}, "004194": {"suoxie": "ZSZZ1000ZSZQA", "name": "\u62db\u5546\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG1000ZHISHUZENGQIANGA"}, "004195": {"suoxie": "ZSZZ1000ZSZQC", "name": "\u62db\u5546\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG1000ZHISHUZENGQIANGC"}, "004196": {"suoxie": "HDYXYNDKZQA", "name": "\u6cd3\u5fb7\u88d5\u946b\u4e00\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUXINYINIANDINGKAIZHAIQUANA"}, "004197": {"suoxie": "HDYXYNDKZQC", "name": "\u6cd3\u5fb7\u88d5\u946b\u4e00\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYUXINYINIANDINGKAIZHAIQUANC"}, "004198": {"suoxie": "HFTYHBA", "name": "\u534e\u5bcc\u5929\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUTIANYIHUOBIA"}, "004199": {"suoxie": "HFTYHBB", "name": "\u534e\u5bcc\u5929\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUTIANYIHUOBIB"}, "004200": {"suoxie": "BSFRCZZQA", "name": "\u535a\u65f6\u5bcc\u745e\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFURUICHUNZHAIZHAIQUANA"}, "004201": {"suoxie": "HXCFBHBB", "name": "\u534e\u590f\u8d22\u5bcc\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIACAIFUBAOHUOBIB"}, "004202": {"suoxie": "HXRPTXHH", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAIXINGHUNHE"}, "004205": {"suoxie": "DFZZCYLHPZHH", "name": "\u4e1c\u65b9\u652f\u67f1\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGZHIZHUCHANYELINGHUOPEIZHIHUNHE"}, "004206": {"suoxie": "HSYHLHPZHH", "name": "\u534e\u5546\u5143\u4ea8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGYUANHENGLINGHUOPEIZHIHUNHE"}, "004209": {"suoxie": "DCZHLHDCLHH", "name": "\u5927\u6210\u667a\u60e0\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGZHIHUILIANGHUADUOCELUEHUNHE"}, "004210": {"suoxie": "QHKYHBE", "name": "\u524d\u6d77\u5f00\u6e90\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAIKAIYUANHUOBIE"}, "004211": {"suoxie": "JYZQYXHH", "name": "\u91d1\u9e70\u5468\u671f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGZHOUQIYOUXUANHUNHE"}, "004212": {"suoxie": "ZRLHZXHHA", "name": "\u4e2d\u878d\u91cf\u5316\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGLIANGHUAZHIXUANHUNHEA"}, "004216": {"suoxie": "XYARHBA", "name": "\u5174\u4e1a\u5b89\u6da6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEANRUNHUOBIA"}, "004217": {"suoxie": "XYARHBB", "name": "\u5174\u4e1a\u5b89\u6da6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEANRUNHUOBIB"}, "004218": {"suoxie": "QHKYYHHHA", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYUHEHUNHEA"}, "004220": {"suoxie": "CXCZYHZQC", "name": "\u957f\u4fe1\u7eaf\u503a\u58f9\u53f7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINCHUNZHAIYIHAOZHAIQUANC"}, "004221": {"suoxie": "CXLHXFHHC", "name": "\u957f\u4fe1\u91cf\u5316\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINLIANGHUAXIANFENGHUNHEC"}, "004222": {"suoxie": "JXMWZQA", "name": "\u91d1\u4fe1\u6c11\u65fa\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINXINMINWANGZHAIQUANA"}, "004223": {"suoxie": "JXDCLJXLHPZ", "name": "\u91d1\u4fe1\u591a\u7b56\u7565\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINDUOCELUEJINGXUANLINGHUOPEIZHI"}, "004224": {"suoxie": "NFJGGGLHPZHHA", "name": "\u5357\u65b9\u519b\u5de5\u6539\u9769\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJUNGONGGAIGELINGHUOPEIZHIHUNHEA"}, "004225": {"suoxie": "GSABWCHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u8bda\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENCHENGHUNHEA"}, "004226": {"suoxie": "GSABWCHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u8bda\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENCHENGHUNHEC"}, "004227": {"suoxie": "TXXLHHA", "name": "\u6cf0\u4fe1\u946b\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIXINXINLIHUNHEA"}, "004228": {"suoxie": "TXXLHHC", "name": "\u6cf0\u4fe1\u946b\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIXINXINLIHUNHEC"}, "004230": {"suoxie": "YYTYZQ", "name": "\u6c38\u8d62\u6dfb\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTIANYIZHAIQUAN"}, "004231": {"suoxie": "ZOHYCZHHLOFC", "name": "\u4e2d\u6b27\u884c\u4e1a\u6210\u957f\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHANGYECHENGZHANGHUNHELOFC"}, "004232": {"suoxie": "ZOJZFXHHC", "name": "\u4e2d\u6b27\u4ef7\u503c\u53d1\u73b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAZHIFAXIANHUNHEC"}, "004233": {"suoxie": "ZOSSCZHHLOFC", "name": "\u4e2d\u6b27\u76db\u4e16\u6210\u957f\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUSHENGSHICHENGZHANGHUNHELOFC"}, "004234": {"suoxie": "ZOSJWJHHC", "name": "\u4e2d\u6b27\u6570\u636e\u6316\u6398\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUSHUJUWAJUEHUNHEC"}, "004235": {"suoxie": "ZOJZZXHHC", "name": "\u4e2d\u6b27\u4ef7\u503c\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJIAZHIZHIXUANHUNHEC"}, "004236": {"suoxie": "ZOXDLHHLOFC", "name": "\u4e2d\u6b27\u65b0\u52a8\u529b\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINDONGLIHUNHELOFC"}, "004237": {"suoxie": "ZOXLCHHC", "name": "\u4e2d\u6b27\u65b0\u84dd\u7b79\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUXINLANCHOUHUNHEC"}, "004238": {"suoxie": "YYRYZQ", "name": "\u6c38\u8d62\u745e\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRUIYIZHAIQUAN"}, "004241": {"suoxie": "ZOSDXFGPC", "name": "\u4e2d\u6b27\u65f6\u4ee3\u5148\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUSHIDAIXIANFENGGUPIAOC"}, "004242": {"suoxie": "XYWKSNDKZQ", "name": "\u5174\u4e1a\u7a33\u5eb7\u4e09\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEWENKANGSANNIANDINGKAIZHAIQUAN"}, "004243": {"suoxie": "GFDQSSYZSRMBC", "name": "\u5e7f\u53d1\u9053\u743c\u65af\u77f3\u6cb9\u6307\u6570\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "GUANGFADAOQIONGSISHIYOUZHISHURENMINBIC"}, "004244": {"suoxie": "DFZQYXLHPZHH", "name": "\u4e1c\u65b9\u5468\u671f\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGZHOUQIYOUXUANLINGHUOPEIZHIHUNHE"}, "004246": {"suoxie": "DBRQZQA", "name": "\u5fb7\u90a6\u9510\u4e7e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIQIANZHAIQUANA"}, "004247": {"suoxie": "DBRQZQC", "name": "\u5fb7\u90a6\u9510\u4e7e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIQIANZHAIQUANC"}, "004249": {"suoxie": "AXZGZZ2025HH", "name": "\u5b89\u4fe1\u4e2d\u56fd\u5236\u90202025\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINZHONGGUOZHIZAO2025HUNHE"}, "004250": {"suoxie": "YHLHYXHH", "name": "\u94f6\u6cb3\u91cf\u5316\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELIANGHUAYOUXUANHUNHE"}, "004251": {"suoxie": "HXHLHBB", "name": "\u534e\u590f\u60e0\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAHUILIHUOBIB"}, "004252": {"suoxie": "GTAYLHPZHHC", "name": "\u56fd\u6cf0\u5b89\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIANYILINGHUOPEIZHIHUNHEC"}, "004253": {"suoxie": "GTHJETFLJC", "name": "\u56fd\u6cf0\u9ec4\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUOTAIHUANGJINETFLIANJIEC"}, "004254": {"suoxie": "MSJYHXYNDKZA", "name": "\u6c11\u751f\u52a0\u94f6\u6c47\u946b\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHUIXINYINIANDINGKAIZHAIA"}, "004255": {"suoxie": "MSJYHXYNDKZC", "name": "\u6c11\u751f\u52a0\u94f6\u6c47\u946b\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHUIXINYINIANDINGKAIZHAIC"}, "004256": {"suoxie": "MSJYHXYNDKZD", "name": "\u6c11\u751f\u52a0\u94f6\u6c47\u946b\u4e00\u5e74\u5b9a\u5f00\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHUIXINYINIANDINGKAIZHAID"}, "004258": {"suoxie": "GSABWJHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5609\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENJIAHUNHEA"}, "004259": {"suoxie": "GSABWJHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5609\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENJIAHUNHEC"}, "004260": {"suoxie": "DBWYZZLHPZHH", "name": "\u5fb7\u90a6\u7a33\u76c8\u589e\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGWENYINGZENGZHANGLINGHUOPEIZHIHUNHE"}, "004261": {"suoxie": "ZSZXBHBA", "name": "\u62db\u5546\u62db\u79a7\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOXIBAOHUOBIA"}, "004262": {"suoxie": "ZSZXBHBB", "name": "\u62db\u5546\u62db\u79a7\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGZHAOXIBAOHUOBIB"}, "004263": {"suoxie": "HAHGSJHLHPZHH", "name": "\u534e\u5b89\u6caa\u6e2f\u6df1\u673a\u4f1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANHUGANGSHENJIHUILINGHUOPEIZHIHUNHE"}, "004264": {"suoxie": "HFTRHCZ", "name": "\u6d77\u5bcc\u901a\u745e\u5408\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIHECHUNZHAI"}, "004265": {"suoxie": "JYMFHBDQKFHH", "name": "\u91d1\u9e70\u6c11\u4e30\u56de\u62a5\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGMINFENGHUIBAODINGQIKAIFANGHUNHE"}, "004266": {"suoxie": "ZSHGSKJCXHHA", "name": "\u62db\u5546\u6caa\u6e2f\u6df1\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGHUGANGSHENKEJICHUANGXINHUNHEA"}, "004267": {"suoxie": "JYCJZLZQLOFE", "name": "\u91d1\u9e70\u6301\u4e45\u589e\u5229\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYINGCHIJIUZENGLIZHAIQUANLOFE"}, "004270": {"suoxie": "HTFMFHBHHA", "name": "\u6c47\u6dfb\u5bcc\u6c11\u4e30\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUMINFENGHUIBAOHUNHEA"}, "004271": {"suoxie": "HTFMFHBHHC", "name": "\u6c47\u6dfb\u5bcc\u6c11\u4e30\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUMINFENGHUIBAOHUNHEC"}, "004274": {"suoxie": "PYAHHBDKHHA", "name": "\u6d66\u94f6\u5b89\u6052\u56de\u62a5\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANHENGHUIBAODINGKAIHUNHEA"}, "004275": {"suoxie": "PYAHHBDKHHC", "name": "\u6d66\u94f6\u5b89\u6052\u56de\u62a5\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANHENGHUIBAODINGKAIHUNHEC"}, "004276": {"suoxie": "PYAHHBDKHHA", "name": "\u6d66\u94f6\u5b89\u548c\u56de\u62a5\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANHEHUIBAODINGKAIHUNHEA"}, "004277": {"suoxie": "PYAHHBDKHHC", "name": "\u6d66\u94f6\u5b89\u548c\u56de\u62a5\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANHEHUIBAODINGKAIHUNHEC"}, "004278": {"suoxie": "DFHZYHGSDKHH", "name": "\u4e1c\u65b9\u7ea2\u667a\u9038\u6caa\u6e2f\u6df1\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHIYIHUGANGSHENDINGKAIHUNHE"}, "004279": {"suoxie": "GSABWRHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENRONGHUNHEA"}, "004280": {"suoxie": "GSABWRHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENRONGHUNHEC"}, "004282": {"suoxie": "BSXRHBB", "name": "\u535a\u65f6\u5174\u8363\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXINGRONGHUOBIB"}, "004284": {"suoxie": "HBXYXDKHH", "name": "\u534e\u5b9d\u65b0\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINYOUXUANDINGKAIHUNHE"}, "004285": {"suoxie": "HFTYHBA", "name": "\u534e\u5bcc\u5929\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUTIANYINGHUOBIA"}, "004286": {"suoxie": "HFTYHBB", "name": "\u534e\u5bcc\u5929\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUTIANYINGHUOBIB"}, "004292": {"suoxie": "PHHSGHLWGP", "name": "\u9e4f\u534e\u6caa\u6df1\u6e2f\u4e92\u8054\u7f51\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAHUSHENGANGHULIANWANGGUPIAO"}, "004301": {"suoxie": "GSABWXHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u4fe1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENXINHUNHEA"}, "004302": {"suoxie": "GSABWXHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u4fe1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENXINHUNHEC"}, "004307": {"suoxie": "BSFYCZZQ", "name": "\u535a\u65f6\u5bcc\u5143\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYUANCHUNZHAIZHAIQUAN"}, "004314": {"suoxie": "QHKYHGSXYJA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u65b0\u786c\u4ef6A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENXINYINGJIANA"}, "004315": {"suoxie": "QHKYHGSXYJC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u65b0\u786c\u4ef6C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENXINYINGJIANC"}, "004316": {"suoxie": "QHKYHGSYXA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u88d5\u946bA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENYUXINA"}, "004317": {"suoxie": "QHKYHGSYXC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u88d5\u946bC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENYUXINC"}, "004318": {"suoxie": "GSABZYYHHBZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u88d5\u4f18\u5316\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOSHOUANBAOZUNYUYOUHUAHUIBAOZHAIQUANA"}, "004319": {"suoxie": "GSABZYYHHBZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u88d5\u4f18\u5316\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOSHOUANBAOZUNYUYOUHUAHUIBAOZHAIQUANC"}, "004320": {"suoxie": "QHKYHGSLXSH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u4e50\u4eab\u751f\u6d3b", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLEXIANGSHENGHUO"}, "004321": {"suoxie": "QHKYHGSQGCYHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u5f3a\u56fd\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENQIANGGUOCHANYEHUNHE"}, "004322": {"suoxie": "CJHXZLCZZQA", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u9686\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNLONGCHUNZHAIZHAIQUANA"}, "004330": {"suoxie": "TPRRXHBA", "name": "\u592a\u5e73\u65e5\u65e5\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIPINGRIRIXINHUOBIA"}, "004331": {"suoxie": "TPRRXHBB", "name": "\u592a\u5e73\u65e5\u65e5\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIPINGRIRIXINHUOBIB"}, "004332": {"suoxie": "HSHGSXXCYJXHH", "name": "\u6052\u751f\u6caa\u6e2f\u6df1\u65b0\u5174\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGSHENGHUGANGSHENXINXINGCHANYEJINGXUANHUNHE"}, "004333": {"suoxie": "JYYSZQXFQSE", "name": "\u91d1\u9e70\u5143\u76db\u503a\u5238\u578b\u53d1\u8d77\u5f0fE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGYUANSHENGZHAIQUANXINGFAQISHIE"}, "004334": {"suoxie": "BSGLCZ3GYDK", "name": "\u535a\u65f6\u5e7f\u5229\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIGUANGLICHUNZHAI3GEYUEDINGKAI"}, "004335": {"suoxie": "HBXFYLHPZHH", "name": "\u534e\u5b9d\u65b0\u98de\u8dc3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINFEIYUELINGHUOPEIZHIHUNHE"}, "004340": {"suoxie": "TKXTHBHGSHH", "name": "\u6cf0\u5eb7\u5174\u6cf0\u56de\u62a5\u6caa\u6e2f\u6df1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGXINGTAIHUIBAOHUGANGSHENHUNHE"}, "004341": {"suoxie": "NYJDKJHH", "name": "\u519c\u94f6\u5c16\u7aef\u79d1\u6280\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINJIANDUANKEJIHUNHE"}, "004342": {"suoxie": "NFHS300LJJJC", "name": "\u5357\u65b9\u6caa\u6df1300\u8054\u63a5\u57fa\u91d1C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300LIANJIEJIJINC"}, "004343": {"suoxie": "NFCYBETFLJC", "name": "\u5357\u65b9\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGCHUANGYEBANETFLIANJIEC"}, "004344": {"suoxie": "NFDSJ100C", "name": "\u5357\u65b9\u5927\u6570\u636e100C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDASHUJU100C"}, "004345": {"suoxie": "NFSZCFETFLJC", "name": "\u5357\u65b9\u6df1\u8bc1\u6210\u4efdETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHENZHENGCHENGFENETFLIANJIEC"}, "004346": {"suoxie": "NFXKETFLJC", "name": "\u5357\u65b9\u5c0f\u5eb7ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGXIAOKANGETFLIANJIEC"}, "004347": {"suoxie": "NFZZ500XXJSLJC", "name": "\u5357\u65b9\u4e2d\u8bc1500\u4fe1\u606f\u6280\u672f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500XINXIJISHULIANJIEC"}, "004348": {"suoxie": "NFZZ500ETFLJC", "name": "\u5357\u65b9\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500ETFLIANJIEC"}, "004350": {"suoxie": "HFJXJZXFGP", "name": "\u6c47\u4e30\u664b\u4fe1\u4ef7\u503c\u5148\u950b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINJIAZHIXIANFENGGUPIAO"}, "004351": {"suoxie": "HFJXZSJQYFZHH", "name": "\u6c47\u4e30\u664b\u4fe1\u73e0\u4e09\u89d2\u533a\u57df\u53d1\u5c55\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINZHUSANJIAOQUYUFAZHANHUNHE"}, "004352": {"suoxie": "BXRFYJJX", "name": "\u5317\u4fe1\u745e\u4e30\u7814\u7a76\u7cbe\u9009", "style": "\u80a1\u7968\u578b", "pinyin": "BEIXINRUIFENGYANJIUJINGXUAN"}, "004353": {"suoxie": "JSXTHDQHH", "name": "\u5609\u5b9e\u65b0\u6dfb\u534e\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTIANHUADINGQIHUNHE"}, "004355": {"suoxie": "JSFHLHPZHH", "name": "\u5609\u5b9e\u4e30\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIFENGHELINGHUOPEIZHIHUNHE"}, "004356": {"suoxie": "JS6GYLCZQE", "name": "\u5609\u5b9e6\u4e2a\u6708\u7406\u8d22\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHI6GEYUELICAIZHAIQUANE"}, "004357": {"suoxie": "NFZHHH", "name": "\u5357\u65b9\u667a\u6167\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGZHIHUIHUNHE"}, "004359": {"suoxie": "CJHXLHHXHHA", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u6838\u5fc3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINLIANGHUAHEXINHUNHEA"}, "004360": {"suoxie": "CJHXLHHXHHC", "name": "\u521b\u91d1\u5408\u4fe1\u91cf\u5316\u6838\u5fc3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINLIANGHUAHEXINHUNHEC"}, "004361": {"suoxie": "STMGATHBHHA", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u901a\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUMOGENANTONGHUIBAOHUNHEA"}, "004362": {"suoxie": "STMGATHBHHC", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u901a\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUMOGENANTONGHUIBAOHUNHEC"}, "004366": {"suoxie": "BSHXCZZQA", "name": "\u535a\u65f6\u6c47\u4eab\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIHUIXIANGCHUNZHAIZHAIQUANA"}, "004367": {"suoxie": "BSHXCZZQC", "name": "\u535a\u65f6\u6c47\u4eab\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIHUIXIANGCHUNZHAIZHAIQUANC"}, "004368": {"suoxie": "QHKYJCBA", "name": "\u524d\u6d77\u5f00\u6e90\u805a\u8d22\u5b9dA", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAIKAIYUANJUCAIBAOA"}, "004369": {"suoxie": "QHKYJCBB", "name": "\u524d\u6d77\u5f00\u6e90\u805a\u8d22\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAIKAIYUANJUCAIBAOB"}, "004372": {"suoxie": "JYZYHBA", "name": "\u91d1\u9e70\u589e\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYINGZENGYIHUOBIA"}, "004373": {"suoxie": "JYZYHBB", "name": "\u91d1\u9e70\u589e\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYINGZENGYIHUOBIB"}, "004374": {"suoxie": "HTBXJNFHHFQA", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u4e30\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJINIANFENGHUNHEFAQIA"}, "004375": {"suoxie": "HTBXJNFHHFQC", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u4e30\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJINIANFENGHUNHEFAQIC"}, "004386": {"suoxie": "GFHA18GYDKZA", "name": "\u5e7f\u53d1\u6c47\u5b8918\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIAN18GEYUEDINGKAIZHAIA"}, "004387": {"suoxie": "GFHA18GYDKZC", "name": "\u5e7f\u53d1\u6c47\u5b8918\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIAN18GEYUEDINGKAIZHAIC"}, "004388": {"suoxie": "PHFXZQ", "name": "\u9e4f\u534e\u4e30\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGXIANGZHAIQUAN"}, "004389": {"suoxie": "DCHMCZZQ", "name": "\u5927\u6210\u60e0\u660e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIMINGCHUNZHAIZHAIQUAN"}, "004390": {"suoxie": "PAZXCXHHA", "name": "\u5e73\u5b89\u8f6c\u578b\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANZHUANXINGCHUANGXINHUNHEA"}, "004391": {"suoxie": "PAZXCXHHC", "name": "\u5e73\u5b89\u8f6c\u578b\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANZHUANXINGCHUANGXINHUNHEC"}, "004393": {"suoxie": "AXHZCXLHPZHH", "name": "\u5b89\u4fe1\u5408\u4f5c\u521b\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINHEZUOCHUANGXINLINGHUOPEIZHIHUNHE"}, "004394": {"suoxie": "HTBRLHCYHH", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u521b\u4f18\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUACHUANGYOUHUNHE"}, "004397": {"suoxie": "CSXXAQLHCLHH", "name": "\u957f\u76db\u4fe1\u606f\u5b89\u5168\u91cf\u5316\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGXINXIANQUANLIANGHUACELUEHUNHE"}, "004398": {"suoxie": "RTXJBHBB", "name": "\u878d\u901a\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGXIANJINBAOHUOBIB"}, "004399": {"suoxie": "RTHCBHBE", "name": "\u878d\u901a\u6c47\u8d22\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGHUICAIBAOHUOBIE"}, "004400": {"suoxie": "JXMXZQA", "name": "\u91d1\u4fe1\u6c11\u5174\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINXINMINXINGZHAIQUANA"}, "004401": {"suoxie": "JXMXZQC", "name": "\u91d1\u4fe1\u6c11\u5174\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINXINMINXINGZHAIQUANC"}, "004402": {"suoxie": "JXMWZQC", "name": "\u91d1\u4fe1\u6c11\u65fa\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINXINMINWANGZHAIQUANC"}, "004403": {"suoxie": "PAGXJXHGSA", "name": "\u5e73\u5b89\u80a1\u606f\u7cbe\u9009\u6caa\u6e2f\u6df1A", "style": "\u80a1\u7968\u578b", "pinyin": "PINGANGUXIJINGXUANHUGANGSHENA"}, "004404": {"suoxie": "PAGXJXHGSC", "name": "\u5e73\u5b89\u80a1\u606f\u7cbe\u9009\u6caa\u6e2f\u6df1C", "style": "\u80a1\u7968\u578b", "pinyin": "PINGANGUXIJINGXUANHUGANGSHENC"}, "004405": {"suoxie": "GSABWSHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5bff\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENSHOUHUNHEA"}, "004406": {"suoxie": "GSABWSHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5bff\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENSHOUHUNHEC"}, "004407": {"suoxie": "ZSSZXF80ETFLJC", "name": "\u62db\u5546\u4e0a\u8bc1\u6d88\u8d3980ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHANGZHENGXIAOFEI80ETFLIANJIEC"}, "004408": {"suoxie": "ZSSZ100ZSC", "name": "\u62db\u5546\u6df1\u8bc1100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENG100ZHISHUC"}, "004409": {"suoxie": "ZSSZTMT50ETFLJC", "name": "\u62db\u5546\u6df1\u8bc1TMT50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENGTMT50ETFLIANJIEC"}, "004410": {"suoxie": "ZSYSCJ50ZSC", "name": "\u62db\u5546\u592e\u89c6\u8d22\u7ecf50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGYANGSHICAIJING50ZHISHUC"}, "004413": {"suoxie": "JXMFHBHH", "name": "\u5efa\u4fe1\u6c11\u4e30\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINMINFENGHUIBAOHUNHE"}, "004417": {"suoxie": "XQHBB", "name": "\u5174\u5168\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGQUANHUOBIB"}, "004419": {"suoxie": "HTFMYZZQRMBA", "name": "\u6c47\u6dfb\u5bcc\u7f8e\u5143\u503a\u503a\u5238\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUITIANFUMEIYUANZHAIZHAIQUANRENMINBIA"}, "004420": {"suoxie": "HTFMYZZQRMBC", "name": "\u6c47\u6dfb\u5bcc\u7f8e\u5143\u503a\u503a\u5238\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "HUITIANFUMEIYUANZHAIZHAIQUANRENMINBIC"}, "004421": {"suoxie": "HTFMYZZQMYXHA", "name": "\u6c47\u6dfb\u5bcc\u7f8e\u5143\u503a\u503a\u5238\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "HUITIANFUMEIYUANZHAIZHAIQUANMEIYUANXIANHUIA"}, "004422": {"suoxie": "HTFMYZZQMYXHC", "name": "\u6c47\u6dfb\u5bcc\u7f8e\u5143\u503a\u503a\u5238\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "HUITIANFUMEIYUANZHAIZHAIQUANMEIYUANXIANHUIC"}, "004423": {"suoxie": "HSYJJXLHPZ", "name": "\u534e\u5546\u7814\u7a76\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGYANJIUJINGXUANLINGHUOPEIZHI"}, "004424": {"suoxie": "HTFWTYLHH", "name": "\u6c47\u6dfb\u5bcc\u6587\u4f53\u5a31\u4e50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUWENTIYULEHUNHE"}, "004427": {"suoxie": "JYZLZQZQA", "name": "\u4ea4\u94f6\u589e\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGLIZENGQIANGZHAIQUANA"}, "004428": {"suoxie": "JYZLZQZQC", "name": "\u4ea4\u94f6\u589e\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGLIZENGQIANGZHAIQUANC"}, "004432": {"suoxie": "NFYSJSETFLJA", "name": "\u5357\u65b9\u6709\u8272\u91d1\u5c5eETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYOUSEJINSHUETFLIANJIEA"}, "004433": {"suoxie": "NFYSJSETFLJC", "name": "\u5357\u65b9\u6709\u8272\u91d1\u5c5eETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYOUSEJINSHUETFLIANJIEC"}, "004434": {"suoxie": "BSNXTZHHA", "name": "\u535a\u65f6\u9006\u5411\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHINIXIANGTOUZIHUNHEA"}, "004435": {"suoxie": "BSNXTZHHC", "name": "\u535a\u65f6\u9006\u5411\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHINIXIANGTOUZIHUNHEC"}, "004436": {"suoxie": "HTFNNTDKHHA", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u6cf0\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUNIANNIANTAIDINGKAIHUNHEA"}, "004437": {"suoxie": "HTFNNTDKHHC", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u6cf0\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUNIANNIANTAIDINGKAIHUNHEC"}, "004438": {"suoxie": "PHYADQKFZQ", "name": "\u9e4f\u534e\u6c38\u5b89\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGANDINGQIKAIFANGZHAIQUAN"}, "004441": {"suoxie": "FRFXCZ", "name": "\u5bcc\u8363\u5bcc\u5174\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUXINGCHUNZHAI"}, "004442": {"suoxie": "ZOKYHHA", "name": "\u4e2d\u6b27\u5eb7\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUKANGYUHUNHEA"}, "004446": {"suoxie": "NFRNA", "name": "\u5357\u65b9\u8363\u5e74A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGRONGNIANA"}, "004447": {"suoxie": "NFRNC", "name": "\u5357\u65b9\u8363\u5e74C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGRONGNIANC"}, "004448": {"suoxie": "BSHZHBLHPZHH", "name": "\u535a\u65f6\u6c47\u667a\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIZHIHUIBAOLINGHUOPEIZHIHUNHE"}, "004449": {"suoxie": "SYHZLHB", "name": "\u4e0a\u94f6\u6167\u589e\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGYINHUIZENGLIHUOBI"}, "004450": {"suoxie": "JSQYKJHGSGP", "name": "\u5609\u5b9e\u524d\u6cbf\u79d1\u6280\u6caa\u6e2f\u6df1\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIQIANYANKEJIHUGANGSHENGUPIAO"}, "004451": {"suoxie": "HTFSXTLZA", "name": "\u6c47\u6dfb\u5bcc\u53cc\u946b\u6dfb\u5229\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGXINTIANLIZHAIA"}, "004452": {"suoxie": "HTFSXTLZC", "name": "\u6c47\u6dfb\u5bcc\u53cc\u946b\u6dfb\u5229\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGXINTIANLIZHAIC"}, "004453": {"suoxie": "QHKYYXA", "name": "\u524d\u6d77\u5f00\u6e90\u76c8\u946bA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYINGXINA"}, "004454": {"suoxie": "QHKYYXC", "name": "\u524d\u6d77\u5f00\u6e90\u76c8\u946bC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYINGXINC"}, "004455": {"suoxie": "ZOKYHHC", "name": "\u4e2d\u6b27\u5eb7\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUKANGYUHUNHEC"}, "004456": {"suoxie": "XYXFXQSLHPZ", "name": "\u5174\u94f6\u6d88\u8d39\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYINXIAOFEIXINQUSHILINGHUOPEIZHI"}, "004457": {"suoxie": "GDDCLZX18GYHH", "name": "\u5149\u5927\u591a\u7b56\u7565\u667a\u900918\u4e2a\u6708\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDADUOCELUEZHIXUAN18GEYUEHUNHE"}, "004458": {"suoxie": "BSHYCZZQ", "name": "\u535a\u65f6\u534e\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIHUAYINGCHUNZHAIZHAIQUAN"}, "004459": {"suoxie": "XYRLDQKFZQ", "name": "\u946b\u5143\u745e\u5229\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANRUILIDINGQIKAIFANGZHAIQUAN"}, "004463": {"suoxie": "PHFYZQ", "name": "\u9e4f\u534e\u4e30\u7389\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGYUZHAIQUAN"}, "004464": {"suoxie": "WJJS9GYDKZA", "name": "\u4e07\u5bb6\u7396\u76db9\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAJIUSHENG9GEYUEDINGKAIZHAIA"}, "004465": {"suoxie": "WJJS9GYDKZC", "name": "\u4e07\u5bb6\u7396\u76db9\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAJIUSHENG9GEYUEDINGKAIZHAIC"}, "004469": {"suoxie": "HTFXYDKZFQSA", "name": "\u6c47\u6dfb\u5bcc\u946b\u76ca\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINYIDINGKAIZHAIFAQISHIA"}, "004470": {"suoxie": "HTFXYDKZFQSC", "name": "\u6c47\u6dfb\u5bcc\u946b\u76ca\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINYIDINGKAIZHAIFAQISHIC"}, "004475": {"suoxie": "HTBRFLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u5bcc\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIFULIHUNHEA"}, "004476": {"suoxie": "JSCCHGSLXKJ", "name": "\u666f\u987a\u957f\u57ce\u6caa\u6e2f\u6df1\u9886\u5148\u79d1\u6280", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGHUGANGSHENLINGXIANKEJI"}, "004477": {"suoxie": "JSHGSHBHH", "name": "\u5609\u5b9e\u6caa\u6e2f\u6df1\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIHUGANGSHENHUIBAOHUNHE"}, "004479": {"suoxie": "BSFHCZZQ", "name": "\u535a\u65f6\u5bcc\u548c\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUHECHUNZHAIZHAIQUAN"}, "004480": {"suoxie": "HBZHCYHH", "name": "\u534e\u5b9d\u667a\u6167\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOZHIHUICHANYEHUNHE"}, "004481": {"suoxie": "HBDSCYHHA", "name": "\u534e\u5b9d\u7b2c\u4e09\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAODISANCHANYEHUNHEA"}, "004484": {"suoxie": "TDHLYJGPA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e1a\u7ee9\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIYEJIGUPIAOA"}, "004485": {"suoxie": "TDHLYJGPC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e1a\u7ee9\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIYEJIGUPIAOC"}, "004486": {"suoxie": "JSWYZQ", "name": "\u5609\u5b9e\u7a33\u6021\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENYIZHAIQUAN"}, "004488": {"suoxie": "JSFSZGA50ETFLJA", "name": "\u5609\u5b9e\u5bcc\u65f6\u4e2d\u56fdA50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIFUSHIZHONGGUOA50ETFLIANJIEA"}, "004493": {"suoxie": "HTBXHBA", "name": "\u534e\u6cf0\u4fdd\u5174\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAOXINGHUOBIA"}, "004494": {"suoxie": "HTBXHBB", "name": "\u534e\u6cf0\u4fdd\u5174\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAOXINGHUOBIB"}, "004495": {"suoxie": "BSLHPHHH", "name": "\u535a\u65f6\u91cf\u5316\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHILIANGHUAPINGHENGHUNHE"}, "004496": {"suoxie": "QHKYDYCLHHA", "name": "\u524d\u6d77\u5f00\u6e90\u591a\u5143\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANDUOYUANCELUEHUNHEA"}, "004497": {"suoxie": "QHKYDYCLHHC", "name": "\u524d\u6d77\u5f00\u6e90\u591a\u5143\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANDUOYUANCELUEHUNHEC"}, "004498": {"suoxie": "PHFYZQ", "name": "\u9e4f\u534e\u4e30\u6e90\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGYUANZHAIQUAN"}, "004499": {"suoxie": "PHFRZQ", "name": "\u9e4f\u534e\u4e30\u745e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGRUIZHAIQUAN"}, "004501": {"suoxie": "JSXJTLHB", "name": "\u5609\u5b9e\u73b0\u91d1\u6dfb\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIXIANJINTIANLIHUOBI"}, "004502": {"suoxie": "ZYRYBHBA", "name": "\u4e2d\u94f6\u5982\u610f\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINRUYIBAOHUOBIA"}, "004503": {"suoxie": "PHYTDQKFZQ", "name": "\u9e4f\u534e\u6c38\u6cf0\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGTAIDINGQIKAIFANGZHAIQUAN"}, "004504": {"suoxie": "PHYZ18GYDKZ", "name": "\u9e4f\u534e\u6c38\u6cfd18\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGZE18GEYUEDINGKAIZHAI"}, "004505": {"suoxie": "BSXXXFZTHHA", "name": "\u535a\u65f6\u65b0\u5174\u6d88\u8d39\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINXINGXIAOFEIZHUTIHUNHEA"}, "004510": {"suoxie": "JTJSLHXFHHC", "name": "\u4e5d\u6cf0\u4e45\u76db\u91cf\u5316\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIJIUSHENGLIANGHUAXIANFENGHUNHEC"}, "004512": {"suoxie": "HFTHS300ZSZQC", "name": "\u6d77\u5bcc\u901a\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGHUSHEN300ZHISHUZENGQIANGC"}, "004513": {"suoxie": "HFTHS300ZSZQA", "name": "\u6d77\u5bcc\u901a\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGHUSHEN300ZHISHUZENGQIANGA"}, "004517": {"suoxie": "NFAKHH", "name": "\u5357\u65b9\u5b89\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANKANGHUNHE"}, "004521": {"suoxie": "AXGY40LHPZHHA", "name": "\u5b89\u4fe1\u5de5\u4e1a4.0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINGONGYE40LINGHUOPEIZHIHUNHEA"}, "004522": {"suoxie": "AXGY40LHPZHHC", "name": "\u5b89\u4fe1\u5de5\u4e1a4.0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINGONGYE40LINGHUOPEIZHIHUNHEC"}, "004532": {"suoxie": "MSJYZZGGTZSA", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENGGANGGUTONGZHISHUA"}, "004533": {"suoxie": "MSJYZZGGTZSC", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENGGANGGUTONGZHISHUC"}, "004534": {"suoxie": "HTFNNYDKHHA", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u76ca\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUNIANNIANYIDINGKAIHUNHEA"}, "004535": {"suoxie": "HTFNNYDKHHC", "name": "\u6c47\u6dfb\u5bcc\u5e74\u5e74\u76ca\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUNIANNIANYIDINGKAIHUNHEC"}, "004536": {"suoxie": "JSZXQYLHHLHH", "name": "\u5609\u5b9e\u4e2d\u5c0f\u4f01\u4e1a\u91cf\u5316\u6d3b\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIZHONGXIAOQIYELIANGHUAHUOLIHUNHE"}, "004544": {"suoxie": "JSWHCZZQA", "name": "\u5609\u5b9e\u7a33\u534e\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENHUACHUNZHAIZHAIQUANA"}, "004545": {"suoxie": "YYTTLHBA", "name": "\u6c38\u8d62\u5929\u5929\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YONGYINGTIANTIANLIHUOBIA"}, "004546": {"suoxie": "JXLHYXDKHH", "name": "\u5efa\u4fe1\u91cf\u5316\u4f18\u4eab\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINLIANGHUAYOUXIANGDINGKAIHUNHE"}, "004547": {"suoxie": "HXWDSLZQA", "name": "\u534e\u590f\u7a33\u5b9a\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAWENDINGSHUANGLIZHAIQUANA"}, "004548": {"suoxie": "ZYZGDJC", "name": "\u4e2d\u94f6\u4e2d\u9ad8\u7b49\u7ea7C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGGAODENGJIC"}, "004549": {"suoxie": "FADXFZTHH", "name": "\u5bcc\u5b89\u8fbe\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAXIAOFEIZHUTIHUNHE"}, "004553": {"suoxie": "ZXJTFHHBB", "name": "\u4e2d\u4fe1\u5efa\u6295\u51e4\u51f0\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGXINJIANTOUFENGHUANGHUOBIB"}, "004555": {"suoxie": "NFHYA", "name": "\u5357\u65b9\u548c\u5143A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHEYUANA"}, "004556": {"suoxie": "NFHYC", "name": "\u5357\u65b9\u548c\u5143C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHEYUANC"}, "004557": {"suoxie": "BXRFDFLHPZHH", "name": "\u5317\u4fe1\u745e\u4e30\u9f0e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGDINGFENGLINGHUOPEIZHIHUNHE"}, "004558": {"suoxie": "HAFYLHPZHHA", "name": "\u6c47\u5b89\u4e30\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGYULINGHUOPEIZHIHUNHEA"}, "004559": {"suoxie": "HAFYLHPZHHC", "name": "\u6c47\u5b89\u4e30\u88d5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGYULINGHUOPEIZHIHUNHEC"}, "004560": {"suoxie": "HAFYHHA", "name": "\u6c47\u5b89\u4e30\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGYIHUNHEA"}, "004561": {"suoxie": "HAFYHHC", "name": "\u6c47\u5b89\u4e30\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANFENGYIHUNHEC"}, "004564": {"suoxie": "BXRFDLZQA", "name": "\u5317\u4fe1\u745e\u4e30\u9f0e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BEIXINRUIFENGDINGLIZHAIQUANA"}, "004567": {"suoxie": "XHAXHZ39GYDKZA", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u6cfd39\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIZE39GEYUEDINGKAIZHAIA"}, "004568": {"suoxie": "CCGZBHBB", "name": "\u957f\u57ce\u5de5\u8d44\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGGONGZIBAOHUOBIB"}, "004569": {"suoxie": "ZSZZYHHC", "name": "\u62db\u5546\u5236\u9020\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGZHIZAOYEHUNHEC"}, "004570": {"suoxie": "JXJSBHB", "name": "\u5efa\u4fe1\u7ed3\u7b97\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINJIESUANBAOHUOBI"}, "004571": {"suoxie": "WJJRZQA", "name": "\u4e07\u5bb6\u5bb6\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIAJIARUIZHAIQUANA"}, "004572": {"suoxie": "WJJRZQC", "name": "\u4e07\u5bb6\u5bb6\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIAJIARUIZHAIQUANC"}, "004573": {"suoxie": "XHXTLHPZHH", "name": "\u65b0\u534e\u946b\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINTAILINGHUOPEIZHIHUNHE"}, "004576": {"suoxie": "XHHYLHLHPZHH", "name": "\u65b0\u534e\u6052\u76ca\u91cf\u5316\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAHENGYILIANGHUALINGHUOPEIZHIHUNHE"}, "004585": {"suoxie": "PYHLZQA", "name": "\u9e4f\u626c\u6c47\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGHUILIZHAIQUANA"}, "004586": {"suoxie": "PYHLZQC", "name": "\u9e4f\u626c\u6c47\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGHUILIZHAIQUANC"}, "004589": {"suoxie": "MSJYTYBHBB", "name": "\u6c11\u751f\u52a0\u94f6\u817e\u5143\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINTENGYUANBAOHUOBIB"}, "004596": {"suoxie": "ZKWTWAZDLLA", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u5b89\u4e2d\u77ed\u5229\u7387A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGKEWOTUWOANZHONGDUANLILVA"}, "004597": {"suoxie": "NFYHETFLJA", "name": "\u5357\u65b9\u94f6\u884cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYINHANGETFLIANJIEA"}, "004598": {"suoxie": "NFYHETFLJC", "name": "\u5357\u65b9\u94f6\u884cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYINHANGETFLIANJIEC"}, "004601": {"suoxie": "BSFTCZZQ", "name": "\u535a\u65f6\u5bcc\u817e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUTENGCHUNZHAIZHAIQUAN"}, "004602": {"suoxie": "QHKYRHZQA", "name": "\u524d\u6d77\u5f00\u6e90\u6da6\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANRUNHEZHAIQUANA"}, "004603": {"suoxie": "QHKYRHZQC", "name": "\u524d\u6d77\u5f00\u6e90\u6da6\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANRUNHEZHAIQUANC"}, "004604": {"suoxie": "FGXHLLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u6d3b\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINHUOLILINGHUOPEIZHIHUNHEA"}, "004605": {"suoxie": "FGXHLLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u6d3b\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINHUOLILINGHUOPEIZHIHUNHEC"}, "004606": {"suoxie": "STMGYXDYZGP", "name": "\u4e0a\u6295\u6469\u6839\u4f18\u9009\u591a\u56e0\u5b50\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENYOUXUANDUOYINZIGUPIAO"}, "004607": {"suoxie": "CXLSYNDKHH", "name": "\u957f\u4fe1\u5229\u5c1a\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINLISHANGYINIANDINGKAIHUNHE"}, "004608": {"suoxie": "CXLXLHPZHHA", "name": "\u957f\u4fe1\u4e50\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLEXINLINGHUOPEIZHIHUNHEA"}, "004609": {"suoxie": "CXLXLHPZHHC", "name": "\u957f\u4fe1\u4e50\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLEXINLINGHUOPEIZHIHUNHEC"}, "004612": {"suoxie": "YHXYXDKHHA", "name": "\u94f6\u6cb3\u946b\u6708\u4eab\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEXINYUEXIANGDINGKAIHUNHEA"}, "004613": {"suoxie": "YHXYXDKHHC", "name": "\u94f6\u6cb3\u946b\u6708\u4eab\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEXINYUEXIANGDINGKAIHUNHEC"}, "004614": {"suoxie": "PYLZZQA", "name": "\u9e4f\u626c\u5229\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGLIZEZHAIQUANA"}, "004615": {"suoxie": "PYLZZQC", "name": "\u9e4f\u626c\u5229\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGLIZEZHAIQUANC"}, "004616": {"suoxie": "ZODZXXCYHGSGPA", "name": "\u4e2d\u6b27\u7535\u5b50\u4fe1\u606f\u4ea7\u4e1a\u6caa\u6e2f\u6df1\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUDIANZIXINXICHANYEHUGANGSHENGUPIAOA"}, "004617": {"suoxie": "JXXWHBLHPZHHA", "name": "\u5efa\u4fe1\u946b\u7a33\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINWENHUIBAOLINGHUOPEIZHIHUNHEA"}, "004618": {"suoxie": "JXXWHBLHPZHHC", "name": "\u5efa\u4fe1\u946b\u7a33\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINWENHUIBAOLINGHUOPEIZHIHUNHEC"}, "004629": {"suoxie": "GSABARCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u745e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANRUICHUNZHAIZHAIQUAN"}, "004630": {"suoxie": "PAHXDKZ", "name": "\u5e73\u5b89\u5408\u4fe1\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEXINDINGKAIZHAI"}, "004632": {"suoxie": "PAHYDKZFQS", "name": "\u5e73\u5b89\u5408\u610f\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEYIDINGKAIZHAIFAQISHI"}, "004634": {"suoxie": "QHLHYTHHA", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u6d9b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGTAOHUNHEA"}, "004635": {"suoxie": "ZXJTRLC", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u5229C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOURUILIC"}, "004636": {"suoxie": "ZXJTZXWLWC", "name": "\u4e2d\u4fe1\u5efa\u6295\u667a\u4fe1\u7269\u8054\u7f51C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOUZHIXINWULIANWANGC"}, "004637": {"suoxie": "HXDXZQA", "name": "\u534e\u590f\u9f0e\u5174\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXINGZHAIQUANA"}, "004638": {"suoxie": "HXDXZQC", "name": "\u534e\u590f\u9f0e\u5174\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXINGZHAIQUANC"}, "004639": {"suoxie": "HXHHYNDKZQ", "name": "\u534e\u590f\u6052\u6167\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAHENGHUIYINIANDINGKAIZHAIQUAN"}, "004640": {"suoxie": "HXJNHBGP", "name": "\u534e\u590f\u8282\u80fd\u73af\u4fdd\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAJIENENGHUANBAOGUPIAO"}, "004641": {"suoxie": "WJLHRXHH", "name": "\u4e07\u5bb6\u91cf\u5316\u777f\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIALIANGHUARUIXUANHUNHE"}, "004642": {"suoxie": "NFFDCETFLJA", "name": "\u5357\u65b9\u623f\u5730\u4ea7ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGFANGDICHANETFLIANJIEA"}, "004643": {"suoxie": "NFFDCETFLJC", "name": "\u5357\u65b9\u623f\u5730\u4ea7ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGFANGDICHANETFLIANJIEC"}, "004647": {"suoxie": "XHDLZQA", "name": "\u65b0\u534e\u9f0e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUADINGLIZHAIQUANA"}, "004648": {"suoxie": "NFARHH", "name": "\u5357\u65b9\u5b89\u777f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANRUIHUNHE"}, "004651": {"suoxie": "CXLFZQE", "name": "\u957f\u4fe1\u5229\u4e30\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFENGZHAIQUANE"}, "004652": {"suoxie": "JXXLHBLHPZHHA", "name": "\u5efa\u4fe1\u946b\u5229\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINLIHUIBAOLINGHUOPEIZHIHUNHEA"}, "004653": {"suoxie": "JXXLHBLHPZHHC", "name": "\u5efa\u4fe1\u946b\u5229\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINLIHUIBAOLINGHUOPEIZHIHUNHEC"}, "004655": {"suoxie": "HTFXHDKZQA", "name": "\u6c47\u6dfb\u5bcc\u946b\u6c47\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINHUIDINGKAIZHAIQUANA"}, "004656": {"suoxie": "HTFXHDKZQC", "name": "\u6c47\u6dfb\u5bcc\u946b\u6c47\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINHUIDINGKAIZHAIQUANC"}, "004657": {"suoxie": "JYMFSYHHA", "name": "\u91d1\u9e70\u6c11\u5bcc\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGMINFUSHOUYIHUNHEA"}, "004658": {"suoxie": "JYMFSYHHC", "name": "\u91d1\u9e70\u6c11\u5bcc\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGMINFUSHOUYIHUNHEC"}, "004666": {"suoxie": "CCJJCXCZHHA", "name": "\u957f\u57ce\u4e45\u5609\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUJIACHUANGXINCHENGZHANGHUNHEA"}, "004667": {"suoxie": "ZSZCTLCZQA", "name": "\u62db\u5546\u62db\u8d22\u901a\u7406\u8d22\u503a\u5238A", "style": "", "pinyin": "ZHAOSHANGZHAOCAITONGLICAIZHAIQUANA"}, "004668": {"suoxie": "JXXZHBLHPZHHA", "name": "\u5efa\u4fe1\u946b\u6cfd\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINZEHUIBAOLINGHUOPEIZHIHUNHEA"}, "004669": {"suoxie": "JXXZHBLHPZHHC", "name": "\u5efa\u4fe1\u946b\u6cfd\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINXINZEHUIBAOLINGHUOPEIZHIHUNHEC"}, "004671": {"suoxie": "ZRHXCZ", "name": "\u4e2d\u878d\u6838\u5fc3\u6210\u957f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGHEXINCHENGZHANG"}, "004672": {"suoxie": "HXDZZQA", "name": "\u534e\u590f\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIADUANZHAIZHAIQUANA"}, "004673": {"suoxie": "HXDZZQC", "name": "\u534e\u590f\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIADUANZHAIZHAIQUANC"}, "004674": {"suoxie": "FGXJYLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINJIYULINGHUOPEIZHIHUNHEA"}, "004675": {"suoxie": "FGXJYLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u673a\u9047\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINJIYULINGHUOPEIZHIHUNHEC"}, "004676": {"suoxie": "ZXJTRXLHPZHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOURUIXINLINGHUOPEIZHIHUNHEC"}, "004677": {"suoxie": "BSZLXXCYHH", "name": "\u535a\u65f6\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIZHANLUEXINXINGCHANYEHUNHE"}, "004680": {"suoxie": "QHKYYRHHA", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYURUIHUNHEA"}, "004681": {"suoxie": "WJAHCZYNDKZA", "name": "\u4e07\u5bb6\u5b89\u5f18\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAANHONGCHUNZHAIYINIANDINGKAIZHAIA"}, "004682": {"suoxie": "WJAHCZYNDKZC", "name": "\u4e07\u5bb6\u5b89\u5f18\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAANHONGCHUNZHAIYINIANDINGKAIZHAIC"}, "004683": {"suoxie": "JXGDYLGP", "name": "\u5efa\u4fe1\u9ad8\u7aef\u533b\u7597\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINGAODUANYILIAOGUPIAO"}, "004684": {"suoxie": "PHYYBHBA", "name": "\u9e4f\u534e\u76c8\u4f59\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAYINGYUBAOHUOBIA"}, "004685": {"suoxie": "JYSAYQLHPZHH", "name": "\u91d1\u5143\u987a\u5b89\u5143\u542f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYUANSHUNANYUANQILINGHUOPEIZHIHUNHE"}, "004686": {"suoxie": "HXYJJXGP", "name": "\u534e\u590f\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAYANJIUJINGXUANGUPIAO"}, "004687": {"suoxie": "HTFXHJXHHA", "name": "\u6c47\u6dfb\u5bcc\u7199\u548c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUXIHEJINGXUANHUNHEA"}, "004688": {"suoxie": "HTFXHJXHHC", "name": "\u6c47\u6dfb\u5bcc\u7199\u548c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUXIHEJINGXUANHUNHEC"}, "004689": {"suoxie": "BSFQCZZQ", "name": "\u535a\u65f6\u4e30\u5e86\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFENGQINGCHUNZHAIZHAIQUAN"}, "004693": {"suoxie": "QHLHYJHHA", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u96bd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGJUANHUNHEA"}, "004694": {"suoxie": "THCLJXHHA", "name": "\u5929\u5f18\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGCELUEJINGXUANHUNHEA"}, "004695": {"suoxie": "DXWLJZHHA", "name": "\u4e1c\u5174\u672a\u6765\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGWEILAIJIAZHIHUNHEA"}, "004696": {"suoxie": "DXLHYXHH", "name": "\u4e1c\u5174\u91cf\u5316\u4f18\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGLIANGHUAYOUXIANGHUNHE"}, "004698": {"suoxie": "BSJGZTGPA", "name": "\u535a\u65f6\u519b\u5de5\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIJUNGONGZHUTIGUPIAOA"}, "004699": {"suoxie": "QHLHHYHBA", "name": "\u524d\u6d77\u8054\u5408\u6c47\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAILIANHEHUIYINGHUOBIA"}, "004700": {"suoxie": "QHLHHYHBB", "name": "\u524d\u6d77\u8054\u5408\u6c47\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "QIANHAILIANHEHUIYINGHUOBIB"}, "004701": {"suoxie": "PHYYBHBB", "name": "\u9e4f\u534e\u76c8\u4f59\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAYINGYUBAOHUOBIB"}, "004702": {"suoxie": "NFJRZTLHPZHHA", "name": "\u5357\u65b9\u91d1\u878d\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJINRONGZHUTILINGHUOPEIZHIHUNHEA"}, "004703": {"suoxie": "NFXSHH", "name": "\u5357\u65b9\u5174\u76db\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGXINGSHENGHUNHE"}, "004705": {"suoxie": "NFXYZQA", "name": "\u5357\u65b9\u7965\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXIANGYUANZHAIQUANA"}, "004706": {"suoxie": "NFXYZQC", "name": "\u5357\u65b9\u7965\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXIANGYUANZHAIQUANC"}, "004707": {"suoxie": "JSCCRCHHA", "name": "\u666f\u987a\u957f\u57ce\u777f\u6210\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGRUICHENGHUNHEA"}, "004708": {"suoxie": "HTHTSSYNDKZA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5546\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTAHONGTUSHENGSHANGYINIANDINGKAIZHAIA"}, "004709": {"suoxie": "HTHTSSYNDKZC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5546\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTAHONGTUSHENGSHANGYINIANDINGKAIZHAIC"}, "004710": {"suoxie": "MSJYPCHHA", "name": "\u6c11\u751f\u52a0\u94f6\u9e4f\u7a0b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINPENGCHENGHUNHEA"}, "004716": {"suoxie": "XCLHAEFGPA", "name": "\u4fe1\u8bda\u91cf\u5316\u963f\u5c14\u6cd5\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "XINCHENGLIANGHUAAERFAGUPIAOA"}, "004717": {"suoxie": "WJTTBHBA", "name": "\u4e07\u5bb6\u5929\u6dfb\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIATIANTIANBAOHUOBIA"}, "004718": {"suoxie": "WJTTBHBB", "name": "\u4e07\u5bb6\u5929\u6dfb\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIATIANTIANBAOHUOBIB"}, "004719": {"suoxie": "JSCCRCHHC", "name": "\u666f\u987a\u957f\u57ce\u777f\u6210\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGRUICHENGHUNHEC"}, "004720": {"suoxie": "HXRPTMHHA", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u8302\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAIMAOHUNHEA"}, "004721": {"suoxie": "HXRPTMHHC", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u8302\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAIMAOHUNHEC"}, "004722": {"suoxie": "ZYFHDKZQ", "name": "\u4e2d\u94f6\u4e30\u548c\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGHEDINGKAIZHAIQUAN"}, "004723": {"suoxie": "ZYFSDKZ", "name": "\u4e2d\u94f6\u4e30\u5b9e\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGSHIDINGKAIZHAI"}, "004724": {"suoxie": "XFJYLHPZHHA", "name": "\u5148\u950b\u805a\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJUYUANLINGHUOPEIZHIHUNHEA"}, "004725": {"suoxie": "XFJYLHPZHHC", "name": "\u5148\u950b\u805a\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJUYUANLINGHUOPEIZHIHUNHEC"}, "004726": {"suoxie": "XFJYA", "name": "\u5148\u950b\u805a\u4f18A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJUYOUA"}, "004727": {"suoxie": "XFJYC", "name": "\u5148\u950b\u805a\u4f18C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJUYOUC"}, "004728": {"suoxie": "ZOJTZQA", "name": "\u4e2d\u6b27\u747e\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUJINTAIZHAIQUANA"}, "004729": {"suoxie": "ZOJTZQC", "name": "\u4e2d\u6b27\u747e\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUJINTAIZHAIQUANC"}, "004730": {"suoxie": "JXLHSJQDGP", "name": "\u5efa\u4fe1\u91cf\u5316\u4e8b\u4ef6\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINLIANGHUASHIJIANQUDONGGUPIAO"}, "004731": {"suoxie": "WJRYLHPZHHA", "name": "\u4e07\u5bb6\u745e\u5c27\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYAOLINGHUOPEIZHIHUNHEA"}, "004732": {"suoxie": "WJRYLHPZHHC", "name": "\u4e07\u5bb6\u745e\u5c27\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIYAOLINGHUOPEIZHIHUNHEC"}, "004734": {"suoxie": "ZOJLLHPZHHA", "name": "\u4e2d\u6b27\u747e\u7075\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINLINGLINGHUOPEIZHIHUNHEA"}, "004735": {"suoxie": "ZOJLLHPZHHC", "name": "\u4e2d\u6b27\u747e\u7075\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJINLINGLINGHUOPEIZHIHUNHEC"}, "004736": {"suoxie": "FGDLCZSGYDKZ", "name": "\u5bcc\u56fd\u9f0e\u5229\u7eaf\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUODINGLICHUNZHAISANGEYUEDINGKAIZHAI"}, "004737": {"suoxie": "FGXYXLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u4f18\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINYOUXIANGLINGHUOPEIZHIHUNHEA"}, "004738": {"suoxie": "STMGALHBA", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u9686\u56de\u62a5A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUMOGENANLONGHUIBAOA"}, "004739": {"suoxie": "STMGALHBC", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u9686\u56de\u62a5C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUMOGENANLONGHUIBAOC"}, "004740": {"suoxie": "ZORFLHPZHHC", "name": "\u4e2d\u6b27\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOURUIFENGLINGHUOPEIZHIHUNHEC"}, "004742": {"suoxie": "YFDSZ100ETFLJC", "name": "\u6613\u65b9\u8fbe\u6df1\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHENZHENG100ETFLIANJIEC"}, "004743": {"suoxie": "YFDSZZPETFLJC", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc1\u4e2d\u76d8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENGZHONGPANETFLIANJIEC"}, "004744": {"suoxie": "YFDCYBETFLJC", "name": "\u6613\u65b9\u8fbe\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDACHUANGYEBANETFLIANJIEC"}, "004745": {"suoxie": "CSCXQDLHPZHH", "name": "\u957f\u76db\u521b\u65b0\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGCHUANGXINQUDONGLINGHUOPEIZHIHUNHE"}, "004746": {"suoxie": "YFDSZ50ZQC", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ZENGQIANGC"}, "004747": {"suoxie": "FGXYXLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u4f18\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINYOUXIANGLINGHUOPEIZHIHUNHEC"}, "004748": {"suoxie": "THCLJXHHC", "name": "\u5929\u5f18\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGCELUEJINGXUANHUNHEC"}, "004749": {"suoxie": "HTZJTTJJYXHBB", "name": "\u534e\u6cf0\u7d2b\u91d1\u5929\u5929\u91d1\u4ea4\u6613\u578b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIZIJINTIANTIANJINJIAOYIXINGHUOBIB"}, "004750": {"suoxie": "GFXHLHPZHHA", "name": "\u5e7f\u53d1\u946b\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINHELINGHUOPEIZHIHUNHEA"}, "004751": {"suoxie": "GFXHLHPZHHC", "name": "\u5e7f\u53d1\u946b\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINHELINGHUOPEIZHIHUNHEC"}, "004752": {"suoxie": "GFZZCMETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u4f20\u5a92ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANMEIETFLIANJIEA"}, "004753": {"suoxie": "GFZZCMETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u4f20\u5a92ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANMEIETFLIANJIEC"}, "004756": {"suoxie": "GSABWJHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5409\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENJIHUNHEA"}, "004757": {"suoxie": "GSABWJHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5409\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENJIHUNHEC"}, "004760": {"suoxie": "GSABWRHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENRUIHUNHEA"}, "004761": {"suoxie": "GSABWRHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENRUIHUNHEC"}, "004763": {"suoxie": "ZKWTWJHHA", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u5609\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWOJIAHUNHEA"}, "004764": {"suoxie": "ZKWTWJHHC", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u5609\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWOJIAHUNHEC"}, "004767": {"suoxie": "ZYZXZQ", "name": "\u4e2d\u94f6\u667a\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHIXIANGZHAIQUAN"}, "004769": {"suoxie": "SWLXJZYXHH", "name": "\u7533\u4e07\u83f1\u4fe1\u4ef7\u503c\u4f18\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINJIAZHIYOUXIANHUNHE"}, "004770": {"suoxie": "HFTTYHBA", "name": "\u6d77\u5bcc\u901a\u6dfb\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGTIANYIHUOBIA"}, "004771": {"suoxie": "HFTTYHBB", "name": "\u6d77\u5bcc\u901a\u6dfb\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGTIANYIHUOBIB"}, "004772": {"suoxie": "GSABWTYNDKHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u6cf0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENTAIYINIANDINGKAIHUNHEA"}, "004773": {"suoxie": "GSABWTYNDKHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u6cf0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENTAIYINIANDINGKAIHUNHEC"}, "004774": {"suoxie": "HTFTFJXHH", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u798f\u5409\u7965\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUTIANFUJIXIANGHUNHE"}, "004775": {"suoxie": "JSXTZDQHH", "name": "\u5609\u5b9e\u65b0\u6dfb\u6cfd\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTIANZEDINGQIHUNHE"}, "004776": {"suoxie": "PHJYBHB", "name": "\u9e4f\u534e\u91d1\u5143\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAJINYUANBAOHUOBI"}, "004780": {"suoxie": "ZSZLYNLCZQ", "name": "\u62db\u5546\u62db\u5229\u4e00\u5e74\u7406\u8d22\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGZHAOLIYINIANLICAIZHAIQUAN"}, "004781": {"suoxie": "TXSZZLZQA", "name": "\u6cf0\u4fe1\u53cc\u503a\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINSHUANGZHAIZENGLIZHAIQUANA"}, "004782": {"suoxie": "TXSZZLZQC", "name": "\u6cf0\u4fe1\u53cc\u503a\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINSHUANGZHAIZENGLIZHAIQUANC"}, "004783": {"suoxie": "ZRLHZXHHC", "name": "\u4e2d\u878d\u91cf\u5316\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGLIANGHUAZHIXUANHUNHEC"}, "004784": {"suoxie": "ZSWJYXGP", "name": "\u62db\u5546\u7a33\u5065\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGWENJIANYOUXUANGUPIAO"}, "004786": {"suoxie": "BHHJHTJHBA", "name": "\u6e24\u6d77\u6c47\u91d1\u6c47\u6dfb\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOHAIHUIJINHUITIANJINHUOBIA"}, "004787": {"suoxie": "BHHJHTJHBB", "name": "\u6e24\u6d77\u6c47\u91d1\u6c47\u6dfb\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOHAIHUIJINHUITIANJINHUOBIB"}, "004788": {"suoxie": "FRHS300ZSZQA", "name": "\u5bcc\u8363\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FURONGHUSHEN300ZHISHUZENGQIANGA"}, "004789": {"suoxie": "FRHS300ZSZQC", "name": "\u5bcc\u8363\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FURONGHUSHEN300ZHISHUZENGQIANGC"}, "004790": {"suoxie": "FRZZ500ZSA", "name": "\u5bcc\u8363\u4e2d\u8bc1500\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FURONGZHONGZHENG500ZHISHUA"}, "004791": {"suoxie": "FRZZ500ZSC", "name": "\u5bcc\u8363\u4e2d\u8bc1500\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FURONGZHONGZHENG500ZHISHUC"}, "004792": {"suoxie": "FRFQZQA", "name": "\u5bcc\u8363\u5bcc\u4e7e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FURONGFUQIANZHAIQUANA"}, "004793": {"suoxie": "FRFQZQC", "name": "\u5bcc\u8363\u5bcc\u4e7e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FURONGFUQIANZHAIQUANC"}, "004794": {"suoxie": "FRFXHHA", "name": "\u5bcc\u8363\u798f\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FURONGFUXINHUNHEA"}, "004795": {"suoxie": "FRFXHHC", "name": "\u5bcc\u8363\u798f\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FURONGFUXINHUNHEC"}, "004796": {"suoxie": "GFQDZHBE", "name": "\u5e7f\u53d1\u94b1\u888b\u5b50\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAQIANDAIZIHUOBIE"}, "004797": {"suoxie": "GSABASCZ3GYDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u76db\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANSHENGCHUNZHAI3GEYUEDINGKAIZHAI"}, "004800": {"suoxie": "PYASSTDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u901a\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGTONGDINGKAIZHAIQUAN"}, "004801": {"suoxie": "PYASAJHBDKHHA", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u4e45\u56de\u62a5\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANJIUHUIBAODINGKAIHUNHEA"}, "004802": {"suoxie": "PYASAJHBDKHHC", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u4e45\u56de\u62a5\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANJIUHUIBAODINGKAIHUNHEC"}, "004805": {"suoxie": "CXXFJXLHGPA", "name": "\u957f\u4fe1\u6d88\u8d39\u7cbe\u9009\u91cf\u5316\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINXIAOFEIJINGXUANLIANGHUAGUPIAOA"}, "004806": {"suoxie": "CXXJLNDKHH", "name": "\u957f\u4fe1\u5148\u673a\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINXIANJILIANGNIANDINGKAIHUNHE"}, "004807": {"suoxie": "ZYZQAHZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u5f18\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHENGQUANANHONGZHAIQUANA"}, "004808": {"suoxie": "ZYZQAHZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u5f18\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHENGQUANANHONGZHAIQUANC"}, "004809": {"suoxie": "QHLHRFHHA", "name": "\u524d\u6d77\u8054\u5408\u6da6\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHERUNFENGHUNHEA"}, "004811": {"suoxie": "WJXJBB", "name": "\u4e07\u5bb6\u73b0\u91d1\u5b9dB", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAXIANJINBAOB"}, "004812": {"suoxie": "ZOXJZZGPA", "name": "\u4e2d\u6b27\u5148\u8fdb\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUXIANJINZHIZAOGUPIAOA"}, "004813": {"suoxie": "ZOXJZZGPC", "name": "\u4e2d\u6b27\u5148\u8fdb\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUXIANJINZHIZAOGUPIAOC"}, "004814": {"suoxie": "ZOHLYXHHA", "name": "\u4e2d\u6b27\u7ea2\u5229\u4f18\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUHONGLIYOUXIANGHUNHEA"}, "004815": {"suoxie": "ZOHLYXHHC", "name": "\u4e2d\u6b27\u7ea2\u5229\u4f18\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUHONGLIYOUXIANGHUNHEC"}, "004818": {"suoxie": "GSABMBCLHHFQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u76ee\u6807\u7b56\u7565\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOMUBIAOCELUEHUNHEFAQIA"}, "004819": {"suoxie": "GSABMBCLHHFQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u76ee\u6807\u7b56\u7565\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOMUBIAOCELUEHUNHEFAQIC"}, "004821": {"suoxie": "GSABAJCZBNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u5409\u7eaf\u503a\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANJICHUNZHAIBANNIANDINGKAIZHAI"}, "004823": {"suoxie": "STAYHBA", "name": "\u4e0a\u6295\u5b89\u88d5\u56de\u62a5A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUANYUHUIBAOA"}, "004824": {"suoxie": "STAYHBC", "name": "\u4e0a\u6295\u5b89\u88d5\u56de\u62a5C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUANYUHUIBAOC"}, "004825": {"suoxie": "PAHZCZ", "name": "\u5e73\u5b89\u60e0\u6cfd\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIZECHUNZHAI"}, "004826": {"suoxie": "PAHYCZ", "name": "\u5e73\u5b89\u60e0\u60a6\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIYUECHUNZHAI"}, "004827": {"suoxie": "PAZDZZQA", "name": "\u5e73\u5b89\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANZHONGDUANZHAIZHAIQUANA"}, "004828": {"suoxie": "PAZDZZQC", "name": "\u5e73\u5b89\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANZHONGDUANZHAIZHAIQUANC"}, "004829": {"suoxie": "BXRFXRLP", "name": "\u5317\u4fe1\u745e\u4e30\u5174\u745e\u7075\u914d", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGXINGRUILINGPEI"}, "004831": {"suoxie": "HTFXZDKZA", "name": "\u6c47\u6dfb\u5bcc\u946b\u6cfd\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINZEDINGKAIZHAIA"}, "004832": {"suoxie": "HTFXZDKZC", "name": "\u6c47\u6dfb\u5bcc\u946b\u6cfd\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINZEDINGKAIZHAIC"}, "004833": {"suoxie": "XFJLHHA", "name": "\u5148\u950b\u805a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJULIHUNHEA"}, "004834": {"suoxie": "XFJLHHC", "name": "\u5148\u950b\u805a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGJULIHUNHEC"}, "004836": {"suoxie": "ZRXJZHHA", "name": "\u4e2d\u878d\u946b\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINJIAZHIHUNHEA"}, "004837": {"suoxie": "ZRXJZHHC", "name": "\u4e2d\u878d\u946b\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGXINJIAZHIHUNHEC"}, "004838": {"suoxie": "XDAYAYCZ", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5b89\u76ca\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINDAAOYINANYICHUNZHAI"}, "004839": {"suoxie": "YHAYZDZSYCYQZQA", "name": "\u94f6\u534e\u5b89\u9890\u4e2d\u77ed\u503a\u53cc\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANYIZHONGDUANZHAISHUANGYUECHIYOUQIZHAIQUANA"}, "004840": {"suoxie": "DXPPJXHHA", "name": "\u4e1c\u5174\u54c1\u724c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGPINPAIJINGXUANHUNHEA"}, "004841": {"suoxie": "BSHHHBA", "name": "\u535a\u65f6\u5408\u60e0\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEHUIHUOBIA"}, "004844": {"suoxie": "ZYLXDQKFZQ", "name": "\u4e2d\u94f6\u5229\u4eab\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINLIXIANGDINGQIKAIFANGZHAIQUAN"}, "004845": {"suoxie": "NHRYHHFQA", "name": "\u5357\u534e\u745e\u76c8\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANHUARUIYINGHUNHEFAQIA"}, "004846": {"suoxie": "NHRYHHFQC", "name": "\u5357\u534e\u745e\u76c8\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANHUARUIYINGHUNHEFAQIC"}, "004848": {"suoxie": "ZORHDKHH", "name": "\u4e2d\u6b27\u777f\u6cd3\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOURUIHONGDINGKAIHUNHE"}, "004849": {"suoxie": "XCHBE", "name": "\u4fe1\u8bda\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGHUOBIE"}, "004851": {"suoxie": "GFYLBJGPA", "name": "\u5e7f\u53d1\u533b\u7597\u4fdd\u5065\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYILIAOBAOJIANGUPIAOA"}, "004852": {"suoxie": "GFJZHBHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJIAZHIHUIBAOHUNHEA"}, "004853": {"suoxie": "GFJZHBHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJIAZHIHUIBAOHUNHEC"}, "004854": {"suoxie": "GFZZQZQCZSA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u6c7d\u8f66\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIQICHEZHISHUA"}, "004855": {"suoxie": "GFZZQZQCZSC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u6c7d\u8f66\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIQICHEZHISHUC"}, "004856": {"suoxie": "GFZZQZJZCLZSA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJIANZHUCAILIAOZHISHUA"}, "004857": {"suoxie": "GFZZQZJZCLZSC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJIANZHUCAILIAOZHISHUC"}, "004858": {"suoxie": "CXLHDCLGPC", "name": "\u957f\u4fe1\u91cf\u5316\u591a\u7b56\u7565\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINLIANGHUADUOCELUEGUPIAOC"}, "004859": {"suoxie": "TKNNHCZYNZQ", "name": "\u6cf0\u5eb7\u5e74\u5e74\u7ea2\u7eaf\u503a\u4e00\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGNIANNIANHONGCHUNZHAIYINIANZHAIQUAN"}, "004860": {"suoxie": "HTZJLQBHB", "name": "\u534e\u6cf0\u7d2b\u91d1\u96f6\u94b1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIZIJINLINGQIANBAOHUOBI"}, "004861": {"suoxie": "TKXJGJHBA", "name": "\u6cf0\u5eb7\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXIANJINGUANJIAHUOBIA"}, "004862": {"suoxie": "TKXJGJHBB", "name": "\u6cf0\u5eb7\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXIANJINGUANJIAHUOBIB"}, "004863": {"suoxie": "TKXJGJHBC", "name": "\u6cf0\u5eb7\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXIANJINGUANJIAHUOBIC"}, "004864": {"suoxie": "TKXJGJHBD", "name": "\u6cf0\u5eb7\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIKANGXIANJINGUANJIAHUOBID"}, "004865": {"suoxie": "GLHBA", "name": "\u683c\u6797\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GELINHUOBIA"}, "004866": {"suoxie": "GLHBB", "name": "\u683c\u6797\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GELINHUOBIB"}, "004868": {"suoxie": "JYGXYHHH", "name": "\u4ea4\u94f6\u80a1\u606f\u4f18\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINGUXIYOUHUAHUNHE"}, "004869": {"suoxie": "ZRRRYJYXHBB", "name": "\u4e2d\u878d\u65e5\u65e5\u76c8\u4ea4\u6613\u578b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGRIRIYINGJIAOYIXINGHUOBIB"}, "004870": {"suoxie": "RTCYBZSC", "name": "\u878d\u901a\u521b\u4e1a\u677f\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGCHUANGYEBANZHISHUC"}, "004871": {"suoxie": "ZYJRDCHHA", "name": "\u4e2d\u94f6\u91d1\u878d\u5730\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINJINRONGDICHANHUNHEA"}, "004874": {"suoxie": "RTJC100ZSC", "name": "\u878d\u901a\u5de8\u6f6e100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGJUCHAO100ZHISHUC"}, "004875": {"suoxie": "RTSZCFZSC", "name": "\u878d\u901a\u6df1\u8bc1\u6210\u4efd\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENGCHENGFENZHISHUC"}, "004876": {"suoxie": "RTSZ100ZSC", "name": "\u878d\u901a\u6df1\u8bc1100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENG100ZHISHUC"}, "004877": {"suoxie": "HTFQQYLHHRMB", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u533b\u7597\u6df7\u5408\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "HUITIANFUQUANQIUYILIAOHUNHERENMINBI"}, "004878": {"suoxie": "HTFQQYLHHMYXH", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u533b\u7597\u6df7\u5408\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "HUITIANFUQUANQIUYILIAOHUNHEMEIYUANXIANHUI"}, "004879": {"suoxie": "HTFQQYLHHMYXC", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u533b\u7597\u6df7\u5408\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "HUITIANFUQUANQIUYILIAOHUNHEMEIYUANXIANCHAO"}, "004881": {"suoxie": "ZYLHJZHHA", "name": "\u4e2d\u94f6\u91cf\u5316\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINLIANGHUAJIAZHIHUNHEA"}, "004882": {"suoxie": "ZYFRDQKFZQ", "name": "\u4e2d\u94f6\u4e30\u8363\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGRONGDINGQIKAIFANGZHAIQUAN"}, "004885": {"suoxie": "CXXYZQ", "name": "\u957f\u4fe1\u5148\u4f18\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINXIANYOUZHAIQUAN"}, "004887": {"suoxie": "CXWTSGYDKZFQS", "name": "\u957f\u4fe1\u7a33\u901a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENTONGSANGEYUEDINGKAIZHAIFAQISHI"}, "004888": {"suoxie": "CTZGXYHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u9038\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINYIHUNHEA"}, "004889": {"suoxie": "CTZGXYHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u9038\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINYIHUNHEC"}, "004890": {"suoxie": "ZYJKWYHH", "name": "\u4e2d\u90ae\u5065\u5eb7\u6587\u5a31\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUJIANKANGWENYUHUNHE"}, "004891": {"suoxie": "HRYDCZJXA", "name": "\u534e\u6da6\u5143\u5927\u6210\u957f\u7cbe\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "HUARUNYUANDACHENGZHANGJINGXUANA"}, "004892": {"suoxie": "HRYDCZJXC", "name": "\u534e\u6da6\u5143\u5927\u6210\u957f\u7cbe\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "HUARUNYUANDACHENGZHANGJINGXUANC"}, "004893": {"suoxie": "HRYDRZZQA", "name": "\u534e\u6da6\u5143\u5927\u6da6\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNZEZHAIQUANA"}, "004894": {"suoxie": "HRYDRZZQC", "name": "\u534e\u6da6\u5143\u5927\u6da6\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNZEZHAIQUANC"}, "004895": {"suoxie": "HSXALHHH", "name": "\u534e\u5546\u946b\u5b89\u7075\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINANLINGHUOHUNHE"}, "004896": {"suoxie": "PHXXBHBA", "name": "\u9e4f\u534e\u5174\u946b\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAXINGXINBAOHUOBIA"}, "004897": {"suoxie": "CAHYCZZQA", "name": "\u957f\u5b89\u6cd3\u6e90\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGANHONGYUANCHUNZHAIZHAIQUANA"}, "004898": {"suoxie": "CAHYCZZQC", "name": "\u957f\u5b89\u6cd3\u6e90\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGANHONGYUANCHUNZHAIZHAIQUANC"}, "004899": {"suoxie": "ZYXXDQKFZQ", "name": "\u4e2d\u94f6\u4fe1\u4eab\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINXINXIANGDINGQIKAIFANGZHAIQUAN"}, "004900": {"suoxie": "CTZGXRHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINRUIHUNHEA"}, "004901": {"suoxie": "CTZGXRHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u946b\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINRUIHUNHEC"}, "004902": {"suoxie": "FGFLZQZQ", "name": "\u5bcc\u56fd\u4e30\u5229\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOFENGLIZENGQIANGZHAIQUAN"}, "004903": {"suoxie": "RBHBA", "name": "\u4eba\u4fdd\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "RENBAOHUOBIA"}, "004904": {"suoxie": "RBHBB", "name": "\u4eba\u4fdd\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "RENBAOHUOBIB"}, "004905": {"suoxie": "HTBRSWYYHHA", "name": "\u534e\u6cf0\u67cf\u745e\u751f\u7269\u533b\u836f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUISHENGWUYIYAOHUNHEA"}, "004907": {"suoxie": "CAHFZDZZQA", "name": "\u957f\u5b89\u6cd3\u6ca3\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGANHONGFENGZHONGDUANZHAIZHAIQUANA"}, "004908": {"suoxie": "CAHFZDZZQC", "name": "\u957f\u5b89\u6cd3\u6ca3\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGANHONG*ZHONGDUANZHAIZHA"}, "004910": {"suoxie": "ZJYXCZZQ", "name": "\u4e2d\u52a0\u9890\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIXIANGCHUNZHAIZHAIQUAN"}, "004911": {"suoxie": "ZJCZDKZQA", "name": "\u4e2d\u52a0\u7eaf\u503a\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAIDINGKAIZHAIQUANA"}, "004912": {"suoxie": "ZJCZDKZQC", "name": "\u4e2d\u52a0\u7eaf\u503a\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIACHUNZHAIDINGKAIZHAIQUANC"}, "004913": {"suoxie": "ZYZQJRHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u805a\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANJURUIHUNHEA"}, "004914": {"suoxie": "ZYZQJRHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u805a\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANJURUIHUNHEC"}, "004916": {"suoxie": "JSXTFDQHH", "name": "\u5609\u5b9e\u65b0\u6dfb\u4e30\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTIANFENGDINGQIHUNHE"}, "004917": {"suoxie": "ZYZQXRHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u7965\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANXIANGRUIHUNHEA"}, "004918": {"suoxie": "ZYZQXRHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u7965\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANXIANGRUIHUNHEC"}, "004919": {"suoxie": "XQXTDQKFZQ", "name": "\u5174\u5168\u5174\u6cf0\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANXINGTAIDINGQIKAIFANGZHAIQUAN"}, "004920": {"suoxie": "FGHLCZFQSA", "name": "\u5bcc\u56fd\u6cd3\u5229\u7eaf\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHONGLICHUNZHAIFAQISHIA"}, "004921": {"suoxie": "HXDRSGYDKZA", "name": "\u534e\u590f\u9f0e\u745e\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGRUISANGEYUEDINGKAIZHAIA"}, "004922": {"suoxie": "HXDRSGYDKZC", "name": "\u534e\u590f\u9f0e\u745e\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGRUISANGEYUEDINGKAIZHAIC"}, "004923": {"suoxie": "HXDXSGYDKZA", "name": "\u534e\u590f\u9f0e\u7965\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXIANGSANGEYUEDINGKAIZHAIA"}, "004924": {"suoxie": "HXDXSGYDKZC", "name": "\u534e\u590f\u9f0e\u7965\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXIANGSANGEYUEDINGKAIZHAIC"}, "004925": {"suoxie": "CXDTHBHYLHGPA", "name": "\u957f\u4fe1\u4f4e\u78b3\u73af\u4fdd\u884c\u4e1a\u91cf\u5316\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINDITANHUANBAOHANGYELIANGHUAGUPIAOA"}, "004926": {"suoxie": "ZHJMRHJXA", "name": "\u4e2d\u822a\u519b\u6c11\u878d\u5408\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHANGJUNMINRONGHEJINGXUANA"}, "004927": {"suoxie": "ZHJMRHJXC", "name": "\u4e2d\u822a\u519b\u6c11\u878d\u5408\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHANGJUNMINRONGHEJINGXUANC"}, "004928": {"suoxie": "HRYDXXHHA", "name": "\u534e\u6da6\u5143\u5927\u6b23\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUARUNYUANDAXINXIANGHUNHEA"}, "004929": {"suoxie": "HRYDXXHHC", "name": "\u534e\u6da6\u5143\u5927\u6b23\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUARUNYUANDAXINXIANGHUNHEC"}, "004930": {"suoxie": "HRYDJZYXA", "name": "\u534e\u6da6\u5143\u5927\u4ef7\u503c\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAJIAZHIYOUXUANA"}, "004931": {"suoxie": "HRYDJZYXC", "name": "\u534e\u6da6\u5143\u5927\u4ef7\u503c\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAJIAZHIYOUXUANC"}, "004932": {"suoxie": "ZSFTLHHHA", "name": "\u62db\u5546\u4e30\u62d3\u7075\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGTUOLINGHUOHUNHEA"}, "004933": {"suoxie": "ZSFTLHHHC", "name": "\u62db\u5546\u4e30\u62d3\u7075\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGTUOLINGHUOHUNHEC"}, "004934": {"suoxie": "YXYFXFSJ", "name": "\u5706\u4fe1\u6c38\u4e30\u6d88\u8d39\u5347\u7ea7", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXIAOFEISHENGJI"}, "004936": {"suoxie": "ZHHGJXHHA", "name": "\u4e2d\u822a\u6df7\u6539\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHANGHUNGAIJINGXUANHUNHEA"}, "004937": {"suoxie": "ZHHGJXHHC", "name": "\u4e2d\u822a\u6df7\u6539\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHANGHUNGAIJINGXUANHUNHEC"}, "004938": {"suoxie": "ZOGQBHBB", "name": "\u4e2d\u6b27\u6eda\u94b1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUGUNQIANBAOHUOBIB"}, "004939": {"suoxie": "ZOGQBHBC", "name": "\u4e2d\u6b27\u6eda\u94b1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUGUNQIANBAOHUOBIC"}, "004940": {"suoxie": "ZJJXCZYNDKA", "name": "\u4e2d\u52a0\u805a\u946b\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJUXINCHUNZHAIYINIANDINGKAIA"}, "004941": {"suoxie": "ZJJXCZYNDKC", "name": "\u4e2d\u52a0\u805a\u946b\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJUXINCHUNZHAIYINIANDINGKAIC"}, "004942": {"suoxie": "GLBYLHPZA", "name": "\u683c\u6797\u4f2f\u5143\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINBOYUANLINGHUOPEIZHIA"}, "004943": {"suoxie": "GLBYLHPZC", "name": "\u683c\u6797\u4f2f\u5143\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINBOYUANLINGHUOPEIZHIC"}, "004944": {"suoxie": "XYXQSLHPZHHA", "name": "\u946b\u5143\u946b\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINQUSHILINGHUOPEIZHIHUNHEA"}, "004945": {"suoxie": "CXZZ500ZSZQA", "name": "\u957f\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGXINZHONGZHENG500ZHISHUZENGQIANGA"}, "004946": {"suoxie": "HTFYRHHA", "name": "\u6c47\u6dfb\u5bcc\u76c8\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUYINGRUNHUNHEA"}, "004947": {"suoxie": "HTFYRHHC", "name": "\u6c47\u6dfb\u5bcc\u76c8\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUYINGRUNHUNHEC"}, "004948": {"suoxie": "XYXQSLHPZHHC", "name": "\u946b\u5143\u946b\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINQUSHILINGHUOPEIZHIHUNHEC"}, "004951": {"suoxie": "SWLXJZYLHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u4ef7\u503c\u4f18\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINJIAZHIYOULIHUNHEA"}, "004952": {"suoxie": "XQHYZQA", "name": "\u5174\u5168\u6052\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGQUANHENGYIZHAIQUANA"}, "004953": {"suoxie": "XQHYZQC", "name": "\u5174\u5168\u6052\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGQUANHENGYIZHAIQUANC"}, "004954": {"suoxie": "ZYZQZGDJZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANZHONGGAODENGJIZHAIQUANA"}, "004955": {"suoxie": "ZYZQZGDJZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANZHONGGAODENGJIZHAIQUANC"}, "004956": {"suoxie": "ZYZQAYZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u8a89\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYUZHAIQUANA"}, "004957": {"suoxie": "ZYZQAYZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u8a89\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYUZHAIQUANC"}, "004958": {"suoxie": "YXYFYXSH", "name": "\u5706\u4fe1\u6c38\u4e30\u4f18\u4eab\u751f\u6d3b", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGYOUXIANGSHENGHUO"}, "004959": {"suoxie": "YXYFYYSHHH", "name": "\u5706\u4fe1\u6c38\u4e30\u4f18\u60a6\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYOUYUESHENGHUOHUNHE"}, "004960": {"suoxie": "PAHTDKZ", "name": "\u5e73\u5b89\u5408\u6cf0\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHETAIDINGKAIZHAI"}, "004965": {"suoxie": "HDZYHHA", "name": "\u6cd3\u5fb7\u81f4\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDEZHIYUANHUNHEA"}, "004966": {"suoxie": "HDZYHHC", "name": "\u6cd3\u5fb7\u81f4\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDEZHIYUANHUNHEC"}, "004967": {"suoxie": "HTCXHBA", "name": "\u7ea2\u571f\u521b\u65b0\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTUCHUANGXINHUOBIA"}, "004968": {"suoxie": "HTCXHBB", "name": "\u7ea2\u571f\u521b\u65b0\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTUCHUANGXINHUOBIB"}, "004970": {"suoxie": "NFTTBHBA", "name": "\u5357\u65b9\u5929\u5929\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGTIANTIANBAOHUOBIA"}, "004971": {"suoxie": "NFTTBHBB", "name": "\u5357\u65b9\u5929\u5929\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGTIANTIANBAOHUOBIB"}, "004972": {"suoxie": "CCSYBHBA", "name": "\u957f\u57ce\u6536\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGSHOUYIBAOHUOBIA"}, "004973": {"suoxie": "CCSYBHBB", "name": "\u957f\u57ce\u6536\u76ca\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGSHOUYIBAOHUOBIB"}, "004975": {"suoxie": "JYHYLHPZHHA", "name": "\u4ea4\u94f6\u6052\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINHENGYILINGHUOPEIZHIHUNHEA"}, "004976": {"suoxie": "HRYDJTHHA", "name": "\u534e\u6da6\u5143\u5927\u666f\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUARUNYUANDAJINGTAIHUNHEA"}, "004977": {"suoxie": "HRYDJTHHC", "name": "\u534e\u6da6\u5143\u5927\u666f\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUARUNYUANDAJINGTAIHUNHEC"}, "004978": {"suoxie": "FGJLSGYDKZ", "name": "\u5bcc\u56fd\u805a\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOJULISANGEYUEDINGKAIZHAI"}, "004979": {"suoxie": "HXDNSGYDKZA", "name": "\u534e\u590f\u9f0e\u8bfa\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGNUOSANGEYUEDINGKAIZHAIA"}, "004980": {"suoxie": "HXDNSGYDKZC", "name": "\u534e\u590f\u9f0e\u8bfa\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGNUOSANGEYUEDINGKAIZHAIC"}, "004981": {"suoxie": "XHXRXZDZA", "name": "\u65b0\u534e\u946b\u65e5\u4eab\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINHUAXINRIXIANGZHONGDUANZHAIA"}, "004982": {"suoxie": "XHAXDYDKHH", "name": "\u65b0\u534e\u5b89\u4eab\u591a\u88d5\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAANXIANGDUOYUDINGKAIHUNHE"}, "004983": {"suoxie": "PYXJTLHBA", "name": "\u9e4f\u626c\u73b0\u91d1\u901a\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGYANGXIANJINTONGLIHUOBIA"}, "004984": {"suoxie": "PYXJTLHBB", "name": "\u9e4f\u626c\u73b0\u91d1\u901a\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGYANGXIANJINTONGLIHUOBIB"}, "004985": {"suoxie": "BSHJHB", "name": "\u535a\u65f6\u5408\u6676\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEJINGHUOBI"}, "004986": {"suoxie": "PHCLHBHH", "name": "\u9e4f\u534e\u7b56\u7565\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUACELUEHUIBAOHUNHE"}, "004987": {"suoxie": "NDXXLHPZHH", "name": "\u8bfa\u5fb7\u65b0\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXINXIANGLINGHUOPEIZHIHUNHE"}, "004988": {"suoxie": "RBSLA", "name": "\u4eba\u4fdd\u53cc\u5229A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RENBAOSHUANGLIA"}, "004989": {"suoxie": "RBSLC", "name": "\u4eba\u4fdd\u53cc\u5229C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RENBAOSHUANGLIC"}, "004993": {"suoxie": "ZOKZZZQA", "name": "\u4e2d\u6b27\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGOUKEZHUANZHAIZHAIQUANA"}, "004994": {"suoxie": "ZOKZZZQC", "name": "\u4e2d\u6b27\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGOUKEZHUANZHAIZHAIQUANC"}, "004995": {"suoxie": "GFPPXFGPFQSA", "name": "\u5e7f\u53d1\u54c1\u724c\u6d88\u8d39\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAPINPAIXIAOFEIGUPIAOFAQISHIA"}, "004996": {"suoxie": "GFHSZXGZSC", "name": "\u5e7f\u53d1\u6052\u751f\u4e2d\u578b\u80a1\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHENGSHENGZHONGXINGGUZHISHUC"}, "004997": {"suoxie": "GFGDZZGPA", "name": "\u5e7f\u53d1\u9ad8\u7aef\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAGAODUANZHIZAOGUPIAOA"}, "004998": {"suoxie": "CXQQZQRMB", "name": "\u957f\u4fe1\u5168\u7403\u503a\u5238\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "CHANGXINQUANQIUZHAIQUANRENMINBI"}, "004999": {"suoxie": "CXQQZQMY", "name": "\u957f\u4fe1\u5168\u7403\u503a\u5238\u7f8e\u5143", "style": "QDII", "pinyin": "CHANGXINQUANQIUZHAIQUANMEIYUAN"}, "005000": {"suoxie": "TKQLLHJZJXHHA", "name": "\u6cf0\u5eb7\u6cc9\u6797\u91cf\u5316\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGQUANLINLIANGHUAJIAZHIJINGXUANHUNHEA"}, "005001": {"suoxie": "JYCXCZZTHH", "name": "\u4ea4\u94f6\u6301\u7eed\u6210\u957f\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHIXUCHENGZHANGZHUTIHUNHE"}, "005004": {"suoxie": "JYPZSJHHA", "name": "\u4ea4\u94f6\u54c1\u8d28\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINPINZHISHENGJIHUNHEA"}, "005005": {"suoxie": "ZJJZA", "name": "\u4e2d\u91d1\u91d1\u6cfdA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINJINZEA"}, "005006": {"suoxie": "ZJJZC", "name": "\u4e2d\u91d1\u91d1\u6cfdC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINJINZEC"}, "005008": {"suoxie": "DFHHYZQZ", "name": "\u4e1c\u65b9\u7ea2\u6c47\u9633\u503a\u5238Z", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGHUIYANGZHAIQUANZ"}, "005009": {"suoxie": "SWLXHYLDGP", "name": "\u7533\u4e07\u83f1\u4fe1\u884c\u4e1a\u8f6e\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINHANGYELUNDONGGUPIAO"}, "005010": {"suoxie": "JYTRZDZA", "name": "\u91d1\u9e70\u6dfb\u745e\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINYINGTIANRUIZHONGDUANZHAIA"}, "005011": {"suoxie": "JYTRZDZC", "name": "\u91d1\u9e70\u6dfb\u745e\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINYINGTIANRUIZHONGDUANZHAIC"}, "005014": {"suoxie": "TKJTHBHHA", "name": "\u6cf0\u5eb7\u666f\u6cf0\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGJINGTAIHUIBAOHUNHEA"}, "005015": {"suoxie": "TKJTHBHHC", "name": "\u6cf0\u5eb7\u666f\u6cf0\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGJINGTAIHUIBAOHUNHEC"}, "005018": {"suoxie": "GJMFHB6GYDKHH", "name": "\u56fd\u91d1\u6c11\u4e30\u56de\u62a56\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOJINMINFENGHUIBAO6GEYUEDINGKAIHUNHE"}, "005019": {"suoxie": "GTRYHT6GYZQ", "name": "\u56fd\u6295\u745e\u94f6\u548c\u6cf06\u4e2a\u6708\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINHETAI6GEYUEZHAIQUAN"}, "005020": {"suoxie": "XCZHJA", "name": "\u4fe1\u8bda\u667a\u60e0\u91d1A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGZHIHUIJINA"}, "005021": {"suoxie": "BHHJLHHYHH", "name": "\u6e24\u6d77\u6c47\u91d1\u91cf\u5316\u6c47\u76c8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOHAIHUIJINLIANGHUAHUIYINGHUNHE"}, "005024": {"suoxie": "NFXLDKZQ", "name": "\u5357\u65b9\u5174\u5229\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXINGLIDINGKAIZHAIQUAN"}, "005025": {"suoxie": "JYFYSYZQC", "name": "\u4ea4\u94f6\u4e30\u76c8\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGYINGSHOUYIZHAIQUANC"}, "005027": {"suoxie": "GDDCLYXDKHH", "name": "\u5149\u5927\u591a\u7b56\u7565\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDADUOCELUEYOUXUANDINGKAIHUNHE"}, "005028": {"suoxie": "PHYJJXLHPZHH", "name": "\u9e4f\u534e\u7814\u7a76\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAYANJIUJINGXUANLINGHUOPEIZHIHUNHE"}, "005029": {"suoxie": "ZYCYJXHH", "name": "\u4e2d\u94f6\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCHANYEJINGXUANHUNHE"}, "005035": {"suoxie": "YHXXKJLHGPFQSA", "name": "\u94f6\u534e\u4fe1\u606f\u79d1\u6280\u91cf\u5316\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAXINXIKEJILIANGHUAGUPIAOFAQISHIA"}, "005036": {"suoxie": "YHXXKJLHGPFQSC", "name": "\u94f6\u534e\u4fe1\u606f\u79d1\u6280\u91cf\u5316\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAXINXIKEJILIANGHUAGUPIAOFAQISHIC"}, "005037": {"suoxie": "YHXNYXCLA", "name": "\u94f6\u534e\u65b0\u80fd\u6e90\u65b0\u6750\u6599A", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAXINNENGYUANXINCAILIAOA"}, "005038": {"suoxie": "YHXNYXCLC", "name": "\u94f6\u534e\u65b0\u80fd\u6e90\u65b0\u6750\u6599C", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAXINNENGYUANXINCAILIAOC"}, "005039": {"suoxie": "PYJXHHA", "name": "\u9e4f\u626c\u666f\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGXINGHUNHEA"}, "005040": {"suoxie": "PYJXHHC", "name": "\u9e4f\u626c\u666f\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGXINGHUNHEC"}, "005041": {"suoxie": "RBYJJXHHA", "name": "\u4eba\u4fdd\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOYANJIUJINGXUANHUNHEA"}, "005042": {"suoxie": "RBYJJXHHC", "name": "\u4eba\u4fdd\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOYANJIUJINGXUANHUNHEC"}, "005043": {"suoxie": "GSABJKKXHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5065\u5eb7\u79d1\u5b66\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOJIANKANGKEXUEHUNHEA"}, "005044": {"suoxie": "GSABJKKXHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5065\u5eb7\u79d1\u5b66\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOJIANKANGKEXUEHUNHEC"}, "005047": {"suoxie": "NHRYCZA", "name": "\u5357\u534e\u745e\u626c\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUIYANGCHUNZHAIA"}, "005048": {"suoxie": "NHRYCZC", "name": "\u5357\u534e\u745e\u626c\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUIYANGCHUNZHAIC"}, "005049": {"suoxie": "CAXWJZHHA", "name": "\u957f\u5b89\u946b\u65fa\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINWANGJIAZHIHUNHEA"}, "005050": {"suoxie": "CAXWJZHHC", "name": "\u957f\u5b89\u946b\u65fa\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINWANGJIAZHIHUNHEC"}, "005051": {"suoxie": "STBPGGTDBHLZSA", "name": "\u4e0a\u6295\u6807\u666e\u6e2f\u80a1\u901a\u4f4e\u6ce2\u7ea2\u5229\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUBIAOPUGANGGUTONGDIBOHONGLIZHISHUA"}, "005052": {"suoxie": "STBPGGTDBHLZSC", "name": "\u4e0a\u6295\u6807\u666e\u6e2f\u80a1\u901a\u4f4e\u6ce2\u7ea2\u5229\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUBIAOPUGANGGUTONGDIBOHONGLIZHISHUC"}, "005053": {"suoxie": "YHLHJZHHA", "name": "\u94f6\u6cb3\u91cf\u5316\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELIANGHUAJIAZHIHUNHEA"}, "005054": {"suoxie": "TKRKCZZQ", "name": "\u6cf0\u5eb7\u745e\u5764\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGRUIKUNCHUNZHAIZHAIQUAN"}, "005055": {"suoxie": "HTBRLHAEFA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u963f\u5c14\u6cd5A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAAERFAA"}, "005056": {"suoxie": "DFHHBA", "name": "\u4e1c\u65b9\u7ea2\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGHONGHUOBIA"}, "005057": {"suoxie": "DFHHBB", "name": "\u4e1c\u65b9\u7ea2\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGHONGHUOBIB"}, "005058": {"suoxie": "DFHHBE", "name": "\u4e1c\u65b9\u7ea2\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGHONGHUOBIE"}, "005059": {"suoxie": "NFAFHHA", "name": "\u5357\u65b9\u5b89\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANFUHUNHEA"}, "005062": {"suoxie": "BSZZ500ZSZQA", "name": "\u535a\u65f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG500ZHISHUZENGQIANGA"}, "005063": {"suoxie": "GFZZQZJYDQZSA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u5bb6\u7528\u7535\u5668\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJIAYONGDIANQIZHISHUA"}, "005064": {"suoxie": "GFZZQZJYDQZSC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u5bb6\u7528\u7535\u5668\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJIAYONGDIANQIZHISHUC"}, "005065": {"suoxie": "ZJXJGJC", "name": "\u4e2d\u91d1\u73b0\u91d1\u7ba1\u5bb6C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGJINXIANJINGUANJIAC"}, "005067": {"suoxie": "RTNXCLLHPZHHA", "name": "\u878d\u901a\u9006\u5411\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGNIXIANGCELUELINGHUOPEIZHIHUNHEA"}, "005068": {"suoxie": "CXFHCZYNDKZA", "name": "\u957f\u4fe1\u5bcc\u6d77\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUHAICHUNZHAIYINIANDINGKAIZHAIA"}, "005069": {"suoxie": "CXFMCZYNDKZA", "name": "\u957f\u4fe1\u5bcc\u6c11\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUMINCHUNZHAIYINIANDINGKAIZHAIA"}, "005070": {"suoxie": "CJLFCZ", "name": "\u957f\u6c5f\u4e50\u4e30\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGLEFENGCHUNZHAI"}, "005072": {"suoxie": "ZYFJDQKFZQ", "name": "\u4e2d\u94f6\u4e30\u8fdb\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGJINDINGQIKAIFANGZHAIQUAN"}, "005073": {"suoxie": "YYYYZQA", "name": "\u6c38\u8d62\u6c38\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYONGYIZHAIQUANA"}, "005074": {"suoxie": "YYYYZQC", "name": "\u6c38\u8d62\u6c38\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYONGYIZHAIQUANC"}, "005075": {"suoxie": "FGYJLHJXHH", "name": "\u5bcc\u56fd\u7814\u7a76\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYANJIULIANGHUAJINGXUANHUNHE"}, "005076": {"suoxie": "CJHXYXHBLHPZHH", "name": "\u521b\u91d1\u5408\u4fe1\u4f18\u9009\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINYOUXUANHUIBAOLINGHUOPEIZHIHUNHE"}, "005077": {"suoxie": "PAHYDKZ", "name": "\u5e73\u5b89\u5408\u97f5\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEYUNDINGKAIZHAI"}, "005078": {"suoxie": "FGBLZQZQ", "name": "\u5bcc\u56fd\u5b9d\u5229\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOBAOLIZENGQIANGZHAIQUAN"}, "005079": {"suoxie": "XYXRXDZA", "name": "\u5174\u94f6\u946b\u65e5\u4eab\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINXINRIXIANGDUANZHAIA"}, "005080": {"suoxie": "HFTLHDYZHHC", "name": "\u6d77\u5bcc\u901a\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGLIANGHUADUOYINZIHUNHEC"}, "005081": {"suoxie": "HFTLHDYZHHA", "name": "\u6d77\u5bcc\u901a\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGLIANGHUADUOYINZIHUNHEA"}, "005082": {"suoxie": "NDLHLCZQHHA", "name": "\u8bfa\u5fb7\u91cf\u5316\u84dd\u7b79\u589e\u5f3a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODELIANGHUALANCHOUZENGQIANGHUNHEA"}, "005083": {"suoxie": "NDLHLCZQHHC", "name": "\u8bfa\u5fb7\u91cf\u5316\u84dd\u7b79\u589e\u5f3a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODELIANGHUALANCHOUZENGQIANGHUNHEC"}, "005088": {"suoxie": "JSXTHDQHHA", "name": "\u5609\u5b9e\u65b0\u6dfb\u8f89\u5b9a\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINTIANHUIDINGQIHUNHEA"}, "005089": {"suoxie": "JSXTHDQHHC", "name": "\u5609\u5b9e\u65b0\u6dfb\u8f89\u5b9a\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIXINTIANHUIDINGQIHUNHEC"}, "005090": {"suoxie": "JHRJHHFQSA", "name": "\u5609\u5408\u777f\u91d1\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHERUIJINHUNHEFAQISHIA"}, "005091": {"suoxie": "JHRJHHFQSC", "name": "\u5609\u5408\u777f\u91d1\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHERUIJINHUNHEFAQISHIC"}, "005092": {"suoxie": "GFHBC", "name": "\u5e7f\u53d1\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOBIC"}, "005094": {"suoxie": "WJZXHH", "name": "\u4e07\u5bb6\u81fb\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHENXUANHUNHE"}, "005095": {"suoxie": "GTLHCZYXHHA", "name": "\u56fd\u6cf0\u91cf\u5316\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAILIANGHUACHENGZHANGYOUXUANHUNHEA"}, "005096": {"suoxie": "GTLHCZYXHHC", "name": "\u56fd\u6cf0\u91cf\u5316\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAILIANGHUACHENGZHANGYOUXUANHUNHEC"}, "005097": {"suoxie": "YFDXJZLHBC", "name": "\u6613\u65b9\u8fbe\u73b0\u91d1\u589e\u5229\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAXIANJINZENGLIHUOBIC"}, "005098": {"suoxie": "YFDLBHBC", "name": "\u6613\u65b9\u8fbe\u9f99\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDALONGBAOHUOBIC"}, "005099": {"suoxie": "YFDFHCZA", "name": "\u6613\u65b9\u8fbe\u5bcc\u534e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAFUHUACHUNZHAIA"}, "005102": {"suoxie": "GYHS300ETFLJA", "name": "\u5de5\u94f6\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUSHEN300ETFLIANJIEA"}, "005103": {"suoxie": "GYHS300ETFLJC", "name": "\u5de5\u94f6\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUSHEN300ETFLIANJIEC"}, "005104": {"suoxie": "FRFKHHA", "name": "\u5bcc\u8363\u798f\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUKANGHUNHEA"}, "005105": {"suoxie": "FRFKHHC", "name": "\u5bcc\u8363\u798f\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUKANGHUNHEC"}, "005106": {"suoxie": "YHNYCYGPFQSA", "name": "\u94f6\u534e\u519c\u4e1a\u4ea7\u4e1a\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUANONGYECHANYEGUPIAOFAQISHIA"}, "005107": {"suoxie": "GFTLHBETFB", "name": "\u5e7f\u53d1\u6dfb\u5229\u8d27\u5e01ETFB", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANLIHUOBIETFB"}, "005108": {"suoxie": "YXYFSLYX", "name": "\u5706\u4fe1\u6c38\u4e30\u53cc\u5229\u4f18\u9009", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGSHUANGLIYOUXUAN"}, "005109": {"suoxie": "HADCLHHA", "name": "\u6c47\u5b89\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANDUOCELUEHUNHEA"}, "005110": {"suoxie": "HADCLHHC", "name": "\u6c47\u5b89\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANDUOCELUEHUNHEC"}, "005111": {"suoxie": "TKQLLHJZJXHHC", "name": "\u6cf0\u5eb7\u6cc9\u6797\u91cf\u5316\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGQUANLINLIANGHUAJIAZHIJINGXUANHUNHEC"}, "005112": {"suoxie": "YHZZQZYYWS", "name": "\u94f6\u534e\u4e2d\u8bc1\u5168\u6307\u533b\u836f\u536b\u751f", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGQUANZHIYIYAOWEISHENG"}, "005113": {"suoxie": "PAHS300ZSLHA", "name": "\u5e73\u5b89\u6caa\u6df1300\u6307\u6570\u91cf\u5316A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANHUSHEN300ZHISHULIANGHUAA"}, "005114": {"suoxie": "PAHS300ZSLHC", "name": "\u5e73\u5b89\u6caa\u6df1300\u6307\u6570\u91cf\u5316C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANHUSHEN300ZHISHULIANGHUAC"}, "005117": {"suoxie": "JXJZJXHHA", "name": "\u91d1\u4fe1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINJIAZHIJINGXUANHUNHEA"}, "005118": {"suoxie": "JXJZJXHHC", "name": "\u91d1\u4fe1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINJIAZHIJINGXUANHUNHEC"}, "005119": {"suoxie": "YHZHNZJZLHPZ", "name": "\u94f6\u534e\u667a\u835f\u5185\u5728\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAZHIHUINEIZAIJIAZHILINGHUOPEIZHI"}, "005120": {"suoxie": "STMGLHDYZHH", "name": "\u4e0a\u6295\u6469\u6839\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENLIANGHUADUOYINZIHUNHE"}, "005121": {"suoxie": "FGXLZQZQ", "name": "\u5bcc\u56fd\u5174\u5229\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOXINGLIZENGQIANGZHAIQUAN"}, "005122": {"suoxie": "YFDTTLCHBC", "name": "\u6613\u65b9\u8fbe\u5929\u5929\u7406\u8d22\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDATIANTIANLICAIHUOBIC"}, "005123": {"suoxie": "NFYXFHHHA", "name": "\u5357\u65b9\u4f18\u4eab\u5206\u7ea2\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYOUXIANGFENHONGHUNHEA"}, "005124": {"suoxie": "YFDHYDKZQFQS", "name": "\u6613\u65b9\u8fbe\u6052\u76ca\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGYIDINGKAIZHAIQUANFAQISHI"}, "005125": {"suoxie": "HBBPZGAGHLJHZSC", "name": "\u534e\u5b9d\u6807\u666e\u4e2d\u56fdA\u80a1\u7ea2\u5229\u673a\u4f1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOBIAOPUZHONGGUOAGUHONGLIJIHUIZHISHUC"}, "005126": {"suoxie": "YHLHWJHH", "name": "\u94f6\u6cb3\u91cf\u5316\u7a33\u8fdb\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELIANGHUAWENJINHUNHE"}, "005127": {"suoxie": "PAHZDKZ", "name": "\u5e73\u5b89\u5408\u6b63\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEZHENGDINGKAIZHAI"}, "005128": {"suoxie": "HXYKTFHH", "name": "\u534e\u590f\u6c38\u5eb7\u6dfb\u798f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGKANGTIANFUHUNHE"}, "005131": {"suoxie": "HFTYXHBA", "name": "\u6d77\u5bcc\u901a\u76c8\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGYINGXINHUOBIA"}, "005132": {"suoxie": "HFTYXHBB", "name": "\u6d77\u5bcc\u901a\u76c8\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGYINGXINHUOBIB"}, "005134": {"suoxie": "CXCJTHBA", "name": "\u957f\u4fe1\u957f\u91d1\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGXINCHANGJINTONGHUOBIA"}, "005135": {"suoxie": "CXCJTHBB", "name": "\u957f\u4fe1\u957f\u91d1\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGXINCHANGJINTONGHUOBIB"}, "005136": {"suoxie": "HAXFSHHH", "name": "\u534e\u5b89\u5e78\u798f\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANXINGFUSHENGHUOHUNHE"}, "005137": {"suoxie": "CXHS300ZSZQA", "name": "\u957f\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGXINHUSHEN300ZHISHUZENGQIANGA"}, "005138": {"suoxie": "QHKYHFZQA", "name": "\u524d\u6d77\u5f00\u6e90\u5f18\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANHONGFENGZHAIQUANA"}, "005139": {"suoxie": "QHKYHFZQC", "name": "\u524d\u6d77\u5f00\u6e90\u5f18\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANHONGFENGZHAIQUANC"}, "005140": {"suoxie": "HXRPTRHHA", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAIRONGHUNHEA"}, "005141": {"suoxie": "HXRPTRHHC", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAIRONGHUNHEC"}, "005142": {"suoxie": "ZRHGSDXFZTA", "name": "\u4e2d\u878d\u6caa\u6e2f\u6df1\u5927\u6d88\u8d39\u4e3b\u9898A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGHUGANGSHENDAXIAOFEIZHUTIA"}, "005143": {"suoxie": "ZRHGSDXFZTC", "name": "\u4e2d\u878d\u6caa\u6e2f\u6df1\u5927\u6d88\u8d39\u4e3b\u9898C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGRONGHUGANGSHENDAXIAOFEIZHUTIC"}, "005144": {"suoxie": "DWYYZQA", "name": "\u4e1c\u5434\u4f18\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGWUYOUYIZHAIQUANA"}, "005145": {"suoxie": "DWYYZQC", "name": "\u4e1c\u5434\u4f18\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGWUYOUYIZHAIQUANC"}, "005146": {"suoxie": "XYFRLHPZHH", "name": "\u5174\u94f6\u4e30\u6da6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYINFENGRUNLINGHUOPEIZHIHUNHE"}, "005148": {"suoxie": "XHHQTLHBE", "name": "\u65b0\u534e\u6d3b\u671f\u6dfb\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAHUOQITIANLIHUOBIE"}, "005150": {"suoxie": "HTCXYCHBA", "name": "\u7ea2\u571f\u521b\u65b0\u4f18\u6df3\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTUCHUANGXINYOUCHUNHUOBIA"}, "005151": {"suoxie": "HTCXYCHBB", "name": "\u7ea2\u571f\u521b\u65b0\u4f18\u6df3\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HONGTUCHUANGXINYOUCHUNHUOBIB"}, "005152": {"suoxie": "NYHS300ZSC", "name": "\u519c\u94f6\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NONGYINHUSHEN300ZHISHUC"}, "005153": {"suoxie": "NYHLRRXJYXHBC", "name": "\u519c\u94f6\u6c47\u7406\u65e5\u65e5\u946b\u4ea4\u6613\u578b\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHUILIRIRIXINJIAOYIXINGHUOBIC"}, "005156": {"suoxie": "JSLHZCPZHHA", "name": "\u5609\u5b9e\u9886\u822a\u8d44\u4ea7\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHILINGHANGZICHANPEIZHIHUNHEA"}, "005157": {"suoxie": "JSLHZCPZHHC", "name": "\u5609\u5b9e\u9886\u822a\u8d44\u4ea7\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHILINGHANGZICHANPEIZHIHUNHEC"}, "005158": {"suoxie": "CJLYDKZFQS", "name": "\u957f\u6c5f\u4e50\u76c8\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGLEYINGDINGKAIZHAIFAQISHI"}, "005159": {"suoxie": "HTBXZHZQA", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u5408\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNHEZHAIQUANA"}, "005160": {"suoxie": "HTBXZHZQC", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u5408\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNHEZHAIQUANC"}, "005161": {"suoxie": "HSSYCYGP", "name": "\u534e\u5546\u4e0a\u6e38\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGSHANGYOUCHANYEGUPIAO"}, "005162": {"suoxie": "ZYRYBHBB", "name": "\u4e2d\u94f6\u5982\u610f\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINRUYIBAOHUOBIB"}, "005164": {"suoxie": "FRFJHHA", "name": "\u5bcc\u8363\u798f\u9526\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUJINHUNHEA"}, "005165": {"suoxie": "FRFJHHC", "name": "\u5bcc\u8363\u798f\u9526\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUJINHUNHEC"}, "005166": {"suoxie": "JSRHLHDQHH", "name": "\u5609\u5b9e\u6da6\u548c\u91cf\u5316\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIRUNHELIANGHUADINGQIHUNHE"}, "005167": {"suoxie": "JSRZLHDQHH", "name": "\u5609\u5b9e\u6da6\u6cfd\u91cf\u5316\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIRUNZELIANGHUADINGQIHUNHE"}, "005169": {"suoxie": "HTBXCLJXA", "name": "\u534e\u6cf0\u4fdd\u5174\u7b56\u7565\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAOXINGCELUEJINGXUANA"}, "005170": {"suoxie": "HTBXCLJXC", "name": "\u534e\u6cf0\u4fdd\u5174\u7b56\u7565\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAOXINGCELUEJINGXUANC"}, "005171": {"suoxie": "FGJLCZZQ", "name": "\u5bcc\u56fd\u666f\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOJINGLICHUNZHAIZHAIQUAN"}, "005172": {"suoxie": "TKAYCZ3YDKZQ", "name": "\u6cf0\u5eb7\u5b89\u60a6\u7eaf\u503a3\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANYUECHUNZHAI3YUEDINGKAIZHAIQUAN"}, "005173": {"suoxie": "FRFAZQA", "name": "\u5bcc\u8363\u5bcc\u5b89\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FURONGFUANZHAIQUANA"}, "005174": {"suoxie": "FRFAZQC", "name": "\u5bcc\u8363\u5bcc\u5b89\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FURONGFUANZHAIQUANC"}, "005175": {"suoxie": "GSABXFXLHHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u6d88\u8d39\u65b0\u84dd\u6d77\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOXIAOFEIXINLANHAIHUNHE"}, "005176": {"suoxie": "FGJZYLHH", "name": "\u5bcc\u56fd\u7cbe\u51c6\u533b\u7597\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOJINGZHUNYILIAOHUNHE"}, "005177": {"suoxie": "HXRPTLHHA", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAILIHUNHEA"}, "005178": {"suoxie": "HXRPTLHHC", "name": "\u534e\u590f\u777f\u78d0\u6cf0\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIARUIPANTAILIHUNHEC"}, "005179": {"suoxie": "XDAYXQDDQKFHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u8d77\u70b9\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINXINQIDIANDINGQIKAIFANGHUNHEA"}, "005185": {"suoxie": "GTZHSYDQKFZQ", "name": "\u56fd\u6cf0\u62db\u60e0\u6536\u76ca\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIZHAOHUISHOUYIDINGQIKAIFANGZHAIQUAN"}, "005186": {"suoxie": "CAXXHHA", "name": "\u957f\u5b89\u946b\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINXINGHUNHEA"}, "005187": {"suoxie": "CAXXHHC", "name": "\u957f\u5b89\u946b\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINXINGHUNHEC"}, "005188": {"suoxie": "HFTLHQFGPC", "name": "\u6d77\u5bcc\u901a\u91cf\u5316\u524d\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGLIANGHUAQIANFENGGUPIAOC"}, "005189": {"suoxie": "HFTLHQFGPA", "name": "\u6d77\u5bcc\u901a\u91cf\u5316\u524d\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGLIANGHUAQIANFENGGUPIAOA"}, "005190": {"suoxie": "DFHHGSHH", "name": "\u4e1c\u65b9\u7ea2\u6caa\u6e2f\u6df1\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGHUGANGSHENHUNHE"}, "005193": {"suoxie": "BXRFDLZQC", "name": "\u5317\u4fe1\u745e\u4e30\u9f0e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BEIXINRUIFENGDINGLIZHAIQUANC"}, "005194": {"suoxie": "NFTTLHBE", "name": "\u5357\u65b9\u5929\u5929\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGTIANTIANLIHUOBIE"}, "005195": {"suoxie": "BSHFHBA", "name": "\u535a\u65f6\u5408\u4e30\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEFENGHUOBIA"}, "005196": {"suoxie": "BSHFHBB", "name": "\u535a\u65f6\u5408\u4e30\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHEFENGHUOBIB"}, "005197": {"suoxie": "GYHGSJXHHA", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINHUGANGSHENJINGXUANHUNHEA"}, "005198": {"suoxie": "GYHGSJXHHC", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINHUGANGSHENJINGXUANHUNHEC"}, "005199": {"suoxie": "SYJJLHB", "name": "\u4e0a\u94f6\u805a\u52a0\u5229\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGYINJUJIALIHUOBI"}, "005200": {"suoxie": "PYASPRCZA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u745e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPURUICHUNZHAIA"}, "005201": {"suoxie": "PYASPRCZC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u745e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPURUICHUNZHAIC"}, "005202": {"suoxie": "XYWTYHBB", "name": "\u5174\u4e1a\u7a33\u5929\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGYEWENTIANYINGHUOBIB"}, "005206": {"suoxie": "NFYXCZHHC", "name": "\u5357\u65b9\u4f18\u9009\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYOUXUANCHENGZHANGHUNHEC"}, "005207": {"suoxie": "NFGDZBLHPZHHC", "name": "\u5357\u65b9\u9ad8\u7aef\u88c5\u5907\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGGAODUANZHUANGBEILINGHUOPEIZHIHUNHEC"}, "005208": {"suoxie": "GSABAYCZBNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u88d5\u7eaf\u503a\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANYUCHUNZHAIBANNIANDINGKAIZHAI"}, "005209": {"suoxie": "DWSSJGPA", "name": "\u4e1c\u5434\u53cc\u4e09\u89d2\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUSHUANGSANJIAOGUPIAOA"}, "005210": {"suoxie": "DWSSJGPC", "name": "\u4e1c\u5434\u53cc\u4e09\u89d2\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUSHUANGSANJIAOGUPIAOC"}, "005211": {"suoxie": "YHZHHH", "name": "\u94f6\u6cb3\u667a\u6167\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEZHIHUIHUNHE"}, "005212": {"suoxie": "HAWYZQ", "name": "\u6c47\u5b89\u7a33\u88d5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANWENYUZHAIQUAN"}, "005213": {"suoxie": "HXDWSGYDKZA", "name": "\u534e\u590f\u9f0e\u65fa\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGWANGSANGEYUEDINGKAIZHAIA"}, "005214": {"suoxie": "HXDWSGYDKZC", "name": "\u534e\u590f\u9f0e\u65fa\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGWANGSANGEYUEDINGKAIZHAIC"}, "005215": {"suoxie": "NFQTHCLFOFA", "name": "\u5357\u65b9\u5168\u5929\u5019\u7b56\u7565(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGQUANTIANHOUCELUEFOFA"}, "005216": {"suoxie": "NFQTHCLFOFC", "name": "\u5357\u65b9\u5168\u5929\u5019\u7b56\u7565(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGQUANTIANHOUCELUEFOFC"}, "005217": {"suoxie": "JXFZATHHFOF", "name": "\u5efa\u4fe1\u798f\u6cfd\u5b89\u6cf0\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINFUZEANTAIHUNHEFOF"}, "005218": {"suoxie": "HXJHFOFA", "name": "\u534e\u590f\u805a\u60e0(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUHUIFOFA"}, "005219": {"suoxie": "HXJHFOFC", "name": "\u534e\u590f\u805a\u60e0(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUHUIFOFC"}, "005220": {"suoxie": "HFTJYJXHHFOF", "name": "\u6d77\u5bcc\u901a\u805a\u4f18\u7cbe\u9009\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGJUYOUJINGXUANHUNHEFOF"}, "005221": {"suoxie": "TDHLQNHHFOFA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5168\u80fd\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIQUANNENGHUNHEFOFA"}, "005222": {"suoxie": "TDHLQNHHFOFC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5168\u80fd\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIQUANNENGHUNHEFOFC"}, "005223": {"suoxie": "GFZZJJGCETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u57fa\u5efa\u5de5\u7a0bETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJIJIANGONGCHENGETFLIANJIEA"}, "005224": {"suoxie": "GFZZJJGCETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u57fa\u5efa\u5de5\u7a0bETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJIJIANGONGCHENGETFLIANJIEC"}, "005225": {"suoxie": "GFLHDYZHH", "name": "\u5e7f\u53d1\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFALIANGHUADUOYINZIHUNHE"}, "005226": {"suoxie": "SXGGJXLHPZHH", "name": "\u5c71\u897f\u6539\u9769\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANXIGAIGEJINGXUANLINGHUOPEIZHIHUNHE"}, "005228": {"suoxie": "HTFGGTZZCZ", "name": "\u6c47\u6dfb\u5bcc\u6e2f\u80a1\u901a\u4e13\u6ce8\u6210\u957f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUGANGGUTONGZHUANZHUCHENGZHANG"}, "005229": {"suoxie": "JSFSZGA50ETFLJC", "name": "\u5609\u5b9e\u5bcc\u65f6\u4e2d\u56fdA50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIFUSHIZHONGGUOA50ETFLIANJIEC"}, "005230": {"suoxie": "CSHBB", "name": "\u957f\u76db\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGSHENGHUOBIB"}, "005231": {"suoxie": "HTHTSTHHXFQSA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u901a\u6df7\u5408\u578b\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGTONGHUNHEXINGFAQISHIA"}, "005232": {"suoxie": "HTHTSTHHXFQSC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u901a\u6df7\u5408\u578b\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGTONGHUNHEXINGFAQISHIC"}, "005233": {"suoxie": "GFRYLXHHA", "name": "\u5e7f\u53d1\u777f\u6bc5\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYILINGXIANHUNHEA"}, "005234": {"suoxie": "GFHJ3GYDKZ", "name": "\u5e7f\u53d1\u6c47\u54093\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIJI3GEYUEDINGKAIZHAI"}, "005235": {"suoxie": "YHSPYLLHGPFQSA", "name": "\u94f6\u534e\u98df\u54c1\u996e\u6599\u91cf\u5316\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUASHIPINYINLIAOLIANGHUAGUPIAOFAQISHIA"}, "005236": {"suoxie": "YHSPYLLHGPFQSC", "name": "\u94f6\u534e\u98df\u54c1\u996e\u6599\u91cf\u5316\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUASHIPINYINLIAOLIANGHUAGUPIAOFAQISHIC"}, "005237": {"suoxie": "YHYLJKLHYXA", "name": "\u94f6\u534e\u533b\u7597\u5065\u5eb7\u91cf\u5316\u4f18\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAYILIAOJIANKANGLIANGHUAYOUXUANA"}, "005238": {"suoxie": "YHYLJKLHYXC", "name": "\u94f6\u534e\u533b\u7597\u5065\u5eb7\u91cf\u5316\u4f18\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAYILIAOJIANKANGLIANGHUAYOUXUANC"}, "005241": {"suoxie": "ZOSDZHHHA", "name": "\u4e2d\u6b27\u65f6\u4ee3\u667a\u6167\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUSHIDAIZHIHUIHUNHEA"}, "005242": {"suoxie": "ZOSDZHHHC", "name": "\u4e2d\u6b27\u65f6\u4ee3\u667a\u6167\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUSHIDAIZHIHUIHUNHEC"}, "005243": {"suoxie": "RTZGGNZQ", "name": "\u878d\u901a\u4e2d\u56fd\u6982\u5ff5\u503a\u5238", "style": "QDII", "pinyin": "RONGTONGZHONGGUOGAINIANZHAIQUAN"}, "005244": {"suoxie": "GTJYJZLHPZHHA", "name": "\u56fd\u6cf0\u805a\u4f18\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJUYOUJIAZHILINGHUOPEIZHIHUNHEA"}, "005245": {"suoxie": "GTJYJZLHPZHHC", "name": "\u56fd\u6cf0\u805a\u4f18\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJUYOUJIAZHILINGHUOPEIZHIHUNHEC"}, "005246": {"suoxie": "GTKZZZQ", "name": "\u56fd\u6cf0\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUOTAIKEZHUANZHAIZHAIQUAN"}, "005247": {"suoxie": "GDLHJXHH", "name": "\u56fd\u90fd\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUODOULIANGHUAJINGXUANHUNHE"}, "005248": {"suoxie": "XHHS300ZSZQA", "name": "\u65b0\u534e\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAHUSHEN300ZHISHUZENGQIANGA"}, "005250": {"suoxie": "YHGZYSHH", "name": "\u94f6\u534e\u4f30\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAGUZHIYOUSHIHUNHE"}, "005251": {"suoxie": "YHDYDLLHPZHH", "name": "\u94f6\u534e\u591a\u5143\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUADUOYUANDONGLILINGHUOPEIZHIHUNHE"}, "005252": {"suoxie": "ZHTRDKHH", "name": "\u4e2d\u6d77\u6dfb\u745e\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHAITIANRUIDINGKAIHUNHE"}, "005253": {"suoxie": "GTHBB", "name": "\u56fd\u6cf0\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAIHUOBIB"}, "005255": {"suoxie": "PYASGGTLHHHA", "name": "\u6d66\u94f6\u5b89\u76db\u6e2f\u80a1\u901a\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGGANGGUTONGLIANGHUAHUNHEA"}, "005258": {"suoxie": "JSCCLHPHHH", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAPINGHENGHUNHE"}, "005259": {"suoxie": "JXLTQYGP", "name": "\u5efa\u4fe1\u9f99\u5934\u4f01\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINLONGTOUQIYEGUPIAO"}, "005260": {"suoxie": "YHWJZLLHPZHHA", "name": "\u94f6\u534e\u7a33\u5065\u589e\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAWENJIANZENGLILINGHUOPEIZHIHUNHEA"}, "005261": {"suoxie": "YHWJZLLHPZHHC", "name": "\u94f6\u534e\u7a33\u5065\u589e\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAWENJIANZENGLILINGHUOPEIZHIHUNHEC"}, "005262": {"suoxie": "XYXXLHPZHHA", "name": "\u946b\u5143\u6b23\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINXIANGLINGHUOPEIZHIHUNHEA"}, "005263": {"suoxie": "XYXXLHPZHHC", "name": "\u946b\u5143\u6b23\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANXINXIANGLINGHUOPEIZHIHUNHEC"}, "005264": {"suoxie": "GDDCLHH", "name": "\u56fd\u90fd\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUODUDUOCELUEHUNHE"}, "005265": {"suoxie": "BSHZHBHHA", "name": "\u535a\u65f6\u539a\u6cfd\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHOUZEHUIBAOHUNHEA"}, "005266": {"suoxie": "BSHZHBHHC", "name": "\u535a\u65f6\u539a\u6cfd\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHOUZEHUIBAOHUNHEC"}, "005267": {"suoxie": "JSJZJXGP", "name": "\u5609\u5b9e\u4ef7\u503c\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIJIAZHIJINGXUANGUPIAO"}, "005268": {"suoxie": "PHYSQY", "name": "\u9e4f\u534e\u4f18\u52bf\u4f01\u4e1a", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAYOUSHIQIYE"}, "005269": {"suoxie": "HTBRGGTLHHH", "name": "\u534e\u6cf0\u67cf\u745e\u6e2f\u80a1\u901a\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIGANGGUTONGLIANGHUAHUNHE"}, "005270": {"suoxie": "TPGGHLJXHH", "name": "\u592a\u5e73\u6539\u9769\u7ea2\u5229\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIPINGGAIGEHONGLIJINGXUANHUNHE"}, "005271": {"suoxie": "AXHLZQZQA", "name": "\u5b89\u4fe1\u6052\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINHENGLIZENGQIANGZHAIQUANA"}, "005272": {"suoxie": "AXHLZQZQC", "name": "\u5b89\u4fe1\u6052\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINHENGLIZENGQIANGZHAIQUANC"}, "005273": {"suoxie": "HSKZZZQA", "name": "\u534e\u5546\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUASHANGKEZHUANZHAIZHAIQUANA"}, "005274": {"suoxie": "ZYJFHBHH", "name": "\u4e2d\u94f6\u666f\u798f\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINJINGFUHUIBAOHUNHE"}, "005275": {"suoxie": "ZOCXCZLHPZHHA", "name": "\u4e2d\u6b27\u521b\u65b0\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUCHUANGXINCHENGZHANGLINGHUOPEIZHIHUNHEA"}, "005276": {"suoxie": "ZOCXCZLHPZHHC", "name": "\u4e2d\u6b27\u521b\u65b0\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUCHUANGXINCHENGZHANGLINGHUOPEIZHIHUNHEC"}, "005277": {"suoxie": "HFTRFDKZQ", "name": "\u6d77\u5bcc\u901a\u878d\u4e30\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRONGFENGDINGKAIZHAIQUAN"}, "005280": {"suoxie": "AXWJAEFDKHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u963f\u5c14\u6cd5\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ANXINWENJIANAERFADINGKAIHUNHEA"}, "005281": {"suoxie": "ZKWTZXSJHH", "name": "\u4e2d\u79d1\u6c83\u571f\u8f6c\u578b\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUZHUANXINGSHENGJIHUNHE"}, "005284": {"suoxie": "HSKZZZQC", "name": "\u534e\u5546\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUASHANGKEZHUANZHAIZHAIQUANC"}, "005286": {"suoxie": "YHSFDQKFZQFQS", "name": "\u94f6\u534e\u5c81\u4e30\u5b9a\u671f\u5f00\u653e\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUASUIFENGDINGQIKAIFANGZHAIQUANFAQISHI"}, "005287": {"suoxie": "HFTCYBZQC", "name": "\u6d77\u5bcc\u901a\u521b\u4e1a\u677f\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGCHUANGYEBANZENGQIANGC"}, "005288": {"suoxie": "HFTCYBZQA", "name": "\u6d77\u5bcc\u901a\u521b\u4e1a\u677f\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGCHUANGYEBANZENGQIANGA"}, "005289": {"suoxie": "RTTHSGYDKZ", "name": "\u878d\u901a\u901a\u660a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGHAOSANGEYUEDINGKAIZHAI"}, "005290": {"suoxie": "NDXSLHPZHHA", "name": "\u8bfa\u5fb7\u65b0\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXINSHENGLINGHUOPEIZHIHUNHEA"}, "005291": {"suoxie": "HFXYHHHA", "name": "\u534e\u5bcc\u661f\u7389\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAFUXINGYUHENGHUNHEA"}, "005292": {"suoxie": "HFXYHHHC", "name": "\u534e\u5bcc\u661f\u7389\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAFUXINGYUHENGHUNHEC"}, "005293": {"suoxie": "NDXW", "name": "\u8bfa\u5fb7\u65b0\u65fa", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXINWANG"}, "005294": {"suoxie": "NDXYLHPZHH", "name": "\u8bfa\u5fb7\u65b0\u5b9c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXINYILINGHUOPEIZHIHUNHE"}, "005295": {"suoxie": "NDTFLHPZHH", "name": "\u8bfa\u5fb7\u5929\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODETIANFULINGHUOPEIZHIHUNHE"}, "005296": {"suoxie": "NHFCHHA", "name": "\u5357\u534e\u4e30\u6df3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANHUAFENGCHUNHUNHEA"}, "005297": {"suoxie": "NHFCHHC", "name": "\u5357\u534e\u4e30\u6df3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANHUAFENGCHUNHUNHEC"}, "005299": {"suoxie": "WJCZYXHHA", "name": "\u4e07\u5bb6\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIACHENGZHANGYOUXUANHUNHEA"}, "005300": {"suoxie": "WJCZYXHHC", "name": "\u4e07\u5bb6\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIACHENGZHANGYOUXUANHUNHEC"}, "005301": {"suoxie": "QHKYHZZQFQSA", "name": "\u524d\u6d77\u5f00\u6e90\u5f18\u6cfd\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANHONGZEZHAIQUANFAQISHIA"}, "005302": {"suoxie": "QHKYHZZQFQSC", "name": "\u524d\u6d77\u5f00\u6e90\u5f18\u6cfd\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "QIANHAIKAIYUANHONGZEZHAIQUANFAQISHIC"}, "005303": {"suoxie": "JSYYJKGPA", "name": "\u5609\u5b9e\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIYIYAOJIANKANGGUPIAOA"}, "005304": {"suoxie": "JSYYJKGPC", "name": "\u5609\u5b9e\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIYIYAOJIANKANGGUPIAOC"}, "005305": {"suoxie": "CXHLHHA", "name": "\u957f\u4fe1\u5408\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINHELIHUNHEA"}, "005306": {"suoxie": "CXHLHHC", "name": "\u957f\u4fe1\u5408\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINHELIHUNHEC"}, "005307": {"suoxie": "CTZGHDCZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8fbe\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGDACHUNZHAIA"}, "005308": {"suoxie": "CTZGHDCZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8fbe\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGDACHUNZHAIC"}, "005309": {"suoxie": "ZYZQHJDQKFZQ", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u5609\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIJIADINGQIKAIFANGZHAIQUAN"}, "005310": {"suoxie": "GFDZXXCMGPA", "name": "\u5e7f\u53d1\u7535\u5b50\u4fe1\u606f\u4f20\u5a92\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFADIANZIXINXICHUANMEIGUPIAOA"}, "005311": {"suoxie": "WJJJXDNHHA", "name": "\u4e07\u5bb6\u7ecf\u6d4e\u65b0\u52a8\u80fd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJINGJIXINDONGNENGHUNHEA"}, "005312": {"suoxie": "WJJJXDNHHC", "name": "\u4e07\u5bb6\u7ecf\u6d4e\u65b0\u52a8\u80fd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJINGJIXINDONGNENGHUNHEC"}, "005313": {"suoxie": "WJZZ1000ZSZQA", "name": "\u4e07\u5bb6\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAZHONGZHENG1000ZHISHUZENGQIANGA"}, "005314": {"suoxie": "WJZZ1000ZSZQC", "name": "\u4e07\u5bb6\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAZHONGZHENG1000ZHISHUZENGQIANGC"}, "005315": {"suoxie": "TDHLJL3GYDKZA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4ea4\u52293\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIJIAOLI3GEYUEDINGKAIZHAIA"}, "005316": {"suoxie": "TDHLJL3GYDKZC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4ea4\u52293\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIJIAOLI3GEYUEDINGKAIZHAIC"}, "005317": {"suoxie": "WJRSLHPZHHA", "name": "\u4e07\u5bb6\u745e\u821c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUISHUNLINGHUOPEIZHIHUNHEA"}, "005318": {"suoxie": "WJRSLHPZHHC", "name": "\u4e07\u5bb6\u745e\u821c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUISHUNLINGHUOPEIZHIHUNHEC"}, "005321": {"suoxie": "ZYZQHYDQKFZQ", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u5b87\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIYUDINGQIKAIFANGZHAIQUAN"}, "005322": {"suoxie": "ZYFXDQKFZQ", "name": "\u4e2d\u94f6\u4e30\u79a7\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFENGXIDINGQIKAIFANGZHAIQUAN"}, "005323": {"suoxie": "QHKYZXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u6cfd\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZEXINHUNHEA"}, "005324": {"suoxie": "QHKYZXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6cfd\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANZEXINHUNHEC"}, "005325": {"suoxie": "JSCCTHHBHHA", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u6052\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIHENGHUIBAOHUNHEA"}, "005326": {"suoxie": "JSCCTHHBHHC", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u6052\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGTAIHENGHUIBAOHUNHEC"}, "005327": {"suoxie": "JSCCJTWLDKZA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u7a33\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIWENLIDINGKAIZHAIA"}, "005328": {"suoxie": "QHKYJZCLGP", "name": "\u524d\u6d77\u5f00\u6e90\u4ef7\u503c\u7b56\u7565\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANJIAZHICELUEGUPIAO"}, "005329": {"suoxie": "HTFMAZYDKHHA", "name": "\u6c47\u6dfb\u5bcc\u6c11\u5b89\u589e\u76ca\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUMINANZENGYIDINGKAIHUNHEA"}, "005330": {"suoxie": "HTFMAZYDKHHC", "name": "\u6c47\u6dfb\u5bcc\u6c11\u5b89\u589e\u76ca\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUMINANZENGYIDINGKAIHUNHEC"}, "005331": {"suoxie": "YMYSAXHH", "name": "\u76ca\u6c11\u4f18\u52bf\u5b89\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIMINYOUSHIANXIANGHUNHE"}, "005335": {"suoxie": "ZSQJXFHHA", "name": "\u6d59\u5546\u5168\u666f\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGQUANJINGXIAOFEIHUNHEA"}, "005336": {"suoxie": "ZJYHDKZQFQSA", "name": "\u4e2d\u52a0\u9890\u6167\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIHUIDINGKAIZHAIQUANFAQISHIA"}, "005337": {"suoxie": "ZJYHDKZQFQSC", "name": "\u4e2d\u52a0\u9890\u6167\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIHUIDINGKAIZHAIQUANFAQISHIC"}, "005338": {"suoxie": "XY3GYDKZQ", "name": "\u5174\u4e1a3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYE3GEYUEDINGKAIZHAIQUAN"}, "005340": {"suoxie": "XY6GYDKZQ", "name": "\u5174\u4e1a6\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYE6GEYUEDINGKAIZHAIQUAN"}, "005341": {"suoxie": "CAYTHHA", "name": "\u957f\u5b89\u88d5\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUTAIHUNHEA"}, "005342": {"suoxie": "CAYTHHC", "name": "\u957f\u5b89\u88d5\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUTAIHUNHEC"}, "005343": {"suoxie": "CAYSLHPZHHA", "name": "\u957f\u5b89\u88d5\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUSHENGLINGHUOPEIZHIHUNHEA"}, "005344": {"suoxie": "CAYSLHPZHHC", "name": "\u957f\u5b89\u88d5\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUSHENGLINGHUOPEIZHIHUNHEC"}, "005345": {"suoxie": "CAHRCZZQA", "name": "\u957f\u5b89\u6cd3\u6da6\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGANHONGRUNCHUNZHAIZHAIQUANA"}, "005346": {"suoxie": "CAHRCZZQC", "name": "\u957f\u5b89\u6cd3\u6da6\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGANHONGRUNCHUNZHAIZHAIQUANC"}, "005347": {"suoxie": "NDLHYX6GYCYQHH", "name": "\u8bfa\u5fb7\u91cf\u5316\u4f18\u90096\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODELIANGHUAYOUXUAN6GEYUECHIYOUQIHUNHE"}, "005350": {"suoxie": "NDDZZQA", "name": "\u8bfa\u5fb7\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NUODEDUANZHAIZHAIQUANA"}, "005351": {"suoxie": "HTFHYZHHH", "name": "\u6c47\u6dfb\u5bcc\u884c\u4e1a\u6574\u5408\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHANGYEZHENGHEHUNHE"}, "005352": {"suoxie": "PYJTCZHHA", "name": "\u9e4f\u626c\u666f\u6cf0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGJINGTAICHENGZHANGHUNHEA"}, "005353": {"suoxie": "PYJTCZHHC", "name": "\u9e4f\u626c\u666f\u6cf0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGJINGTAICHENGZHANGHUNHEC"}, "005354": {"suoxie": "FGHGSHYJXHHA", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u884c\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHUGANGSHENHANGYEJINGXUANHUNHEA"}, "005357": {"suoxie": "FGGQGGLHPZHH", "name": "\u5bcc\u56fd\u56fd\u4f01\u6539\u9769\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOGUOQIGAIGELINGHUOPEIZHIHUNHE"}, "005358": {"suoxie": "DFAEFJXHHA", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGAERFAJINGXUANHUNHEA"}, "005359": {"suoxie": "DFAEFJXHHC", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGAERFAJINGXUANHUNHEC"}, "005360": {"suoxie": "HAZCLDHH", "name": "\u6c47\u5b89\u8d44\u4ea7\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANZICHANLUNDONGHUNHE"}, "005361": {"suoxie": "ZRJSDQKFZQ", "name": "\u4e2d\u878d\u805a\u5546\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUSHANGDINGQIKAIFANGZHAIQUAN"}, "005362": {"suoxie": "ZYZQAYZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6e90\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYUANZHAIQUANA"}, "005363": {"suoxie": "ZYZQAYZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6e90\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYUANZHAIQUANC"}, "005364": {"suoxie": "HXDSSGYDKZA", "name": "\u534e\u590f\u9f0e\u987a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGSHUNSANGEYUEDINGKAIZHAIA"}, "005365": {"suoxie": "HXDSSGYDKZC", "name": "\u534e\u590f\u9f0e\u987a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGSHUNSANGEYUEDINGKAIZHAIC"}, "005366": {"suoxie": "STMGFRZQA", "name": "\u4e0a\u6295\u6469\u6839\u4e30\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENFENGRUIZHAIQUANA"}, "005367": {"suoxie": "STMGFRZQC", "name": "\u4e0a\u6295\u6469\u6839\u4e30\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENFENGRUIZHAIQUANC"}, "005368": {"suoxie": "FGQJNYCYHHA", "name": "\u5bcc\u56fd\u6e05\u6d01\u80fd\u6e90\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOQINGJIENENGYUANCHANYEHUNHEA"}, "005369": {"suoxie": "FGZLCZDKZQ", "name": "\u5bcc\u56fd\u81fb\u5229\u7eaf\u503a\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHENLICHUNZHAIDINGKAIZHAIQUAN"}, "005371": {"suoxie": "ZJXYHHA", "name": "\u4e2d\u52a0\u5fc3\u60a6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAXINYUEHUNHEA"}, "005372": {"suoxie": "ZJXYHHC", "name": "\u4e2d\u52a0\u5fc3\u60a6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAXINYUEHUNHEC"}, "005373": {"suoxie": "ZJZJLHPZHHA", "name": "\u4e2d\u52a0\u7d2b\u91d1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAZIJINLINGHUOPEIZHIHUNHEA"}, "005374": {"suoxie": "ZJZJLHPZHHC", "name": "\u4e2d\u52a0\u7d2b\u91d1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAZIJINLINGHUOPEIZHIHUNHEC"}, "005375": {"suoxie": "JXRHCZDKZ", "name": "\u5efa\u4fe1\u777f\u548c\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIHECHUNZHAIDINGKAIZHAI"}, "005376": {"suoxie": "BXRFHFLHPZ", "name": "\u5317\u4fe1\u745e\u4e30\u534e\u4e30\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGHUAFENGLINGHUOPEIZHI"}, "005377": {"suoxie": "HADRDKZFQS", "name": "\u534e\u5b89\u9f0e\u745e\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANDINGRUIDINGKAIZHAIFAQISHI"}, "005378": {"suoxie": "QHLHHYDKZQ", "name": "\u524d\u6d77\u8054\u5408\u6cd3\u5143\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEHONGYUANDINGKAIZHAIQUAN"}, "005379": {"suoxie": "HTFJZCZDKHH", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u521b\u9020\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHICHUANGZAODINGKAIHUNHE"}, "005381": {"suoxie": "TKRLLHDCLHHA", "name": "\u6cf0\u5eb7\u777f\u5229\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGRUILILIANGHUADUOCELUEHUNHEA"}, "005382": {"suoxie": "TKRLLHDCLHHC", "name": "\u6cf0\u5eb7\u777f\u5229\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGRUILILIANGHUADUOCELUEHUNHEC"}, "005383": {"suoxie": "FGLSCZYNDKZ", "name": "\u5bcc\u56fd\u7eff\u8272\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOLVSECHUNZHAIYINIANDINGKAIZHAI"}, "005384": {"suoxie": "YHMY3GYDKZQ", "name": "\u94f6\u6cb3\u94ed\u5fc63\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEMINGYI3GEYUEDINGKAIZHAIQUAN"}, "005386": {"suoxie": "YHRDLHPZHHA", "name": "\u94f6\u6cb3\u777f\u8fbe\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHERUIDALINGHUOPEIZHIHUNHEA"}, "005387": {"suoxie": "YHRDLHPZHHC", "name": "\u94f6\u6cb3\u777f\u8fbe\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHERUIDALINGHUOPEIZHIHUNHEC"}, "005388": {"suoxie": "XYAH3GYDKZ", "name": "\u5174\u4e1a\u5b89\u5f183\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEANHONG3GEYUEDINGKAIZHAI"}, "005390": {"suoxie": "GYCYBETFLJA", "name": "\u5de5\u94f6\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINCHUANGYEBANETFLIANJIEA"}, "005391": {"suoxie": "GYCYBETFLJC", "name": "\u5de5\u94f6\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINCHUANGYEBANETFLIANJIEC"}, "005392": {"suoxie": "CXJZLCLNDKA", "name": "\u957f\u4fe1\u4ef7\u503c\u84dd\u7b79\u4e24\u5e74\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINJIAZHILANCHOULIANGNIANDINGKAIA"}, "005393": {"suoxie": "NFZL3GYDKZ", "name": "\u5357\u65b9\u5353\u52293\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHUOLI3GEYUEDINGKAIZHAI"}, "005394": {"suoxie": "NFZL3GYDKZC", "name": "\u5357\u65b9\u5353\u52293\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHUOLI3GEYUEDINGKAIZHAIC"}, "005395": {"suoxie": "HDZYHBHH", "name": "\u6cd3\u5fb7\u81fb\u8fdc\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEZHENYUANHUIBAOHUNHE"}, "005396": {"suoxie": "ZJFSHH", "name": "\u4e2d\u91d1\u4e30\u7855\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINFENGSHUOHUNHE"}, "005397": {"suoxie": "NFAYHH", "name": "\u5357\u65b9\u5b89\u517b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANYANGHUNHE"}, "005398": {"suoxie": "PYCYYNDQKFZ", "name": "\u9e4f\u626c\u6df3\u4f18\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNYOUYINIANDINGQIKAIFANGZHAI"}, "005399": {"suoxie": "CXLHJZQDHHA", "name": "\u957f\u4fe1\u91cf\u5316\u4ef7\u503c\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINLIANGHUAJIAZHIQUDONGHUNHEA"}, "005400": {"suoxie": "WJQLJZLHPZHHA", "name": "\u4e07\u5bb6\u6f5c\u529b\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAQIANLIJIAZHILINGHUOPEIZHIHUNHEA"}, "005401": {"suoxie": "WJQLJZLHPZHHC", "name": "\u4e07\u5bb6\u6f5c\u529b\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAQIANLIJIAZHILINGHUOPEIZHIHUNHEC"}, "005402": {"suoxie": "GFZYYXGPA", "name": "\u5e7f\u53d1\u8d44\u6e90\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAZIYUANYOUXUANGUPIAOA"}, "005405": {"suoxie": "ZJJXLHLCA", "name": "\u4e2d\u91d1\u91d1\u5e8f\u91cf\u5316\u84dd\u7b79A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINJINXULIANGHUALANCHOUA"}, "005406": {"suoxie": "ZJJXLHLCC", "name": "\u4e2d\u91d1\u91d1\u5e8f\u91cf\u5316\u84dd\u7b79C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINJINXULIANGHUALANCHOUC"}, "005407": {"suoxie": "HXDTLGYDKZA", "name": "\u534e\u590f\u9f0e\u6cf0\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGTAILIUGEYUEDINGKAIZHAIA"}, "005408": {"suoxie": "HXDTLGYDKZC", "name": "\u534e\u590f\u9f0e\u6cf0\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGTAILIUGEYUEDINGKAIZHAIC"}, "005409": {"suoxie": "HTBRXXCYHHA", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIXINXINGCHANYEHUNHEA"}, "005410": {"suoxie": "HTFXSDKZA", "name": "\u6c47\u6dfb\u5bcc\u946b\u76db\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINSHENGDINGKAIZHAIA"}, "005411": {"suoxie": "HTFXSDKZC", "name": "\u6c47\u6dfb\u5bcc\u946b\u76db\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINSHENGDINGKAIZHAIC"}, "005412": {"suoxie": "JXMCHHA", "name": "\u91d1\u4fe1\u6c11\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINMINCHANGHUNHEA"}, "005413": {"suoxie": "JXMCHHC", "name": "\u91d1\u4fe1\u6c11\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINMINCHANGHUNHEC"}, "005416": {"suoxie": "PHZHDQKFHHA", "name": "\u9e4f\u534e\u5c0a\u60e0\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZUNHUIDINGQIKAIFANGHUNHEA"}, "005417": {"suoxie": "PHZHDQKFHHC", "name": "\u9e4f\u534e\u5c0a\u60e0\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZUNHUIDINGQIKAIFANGHUNHEC"}, "005419": {"suoxie": "ZOJRZQA", "name": "\u4e2d\u6b27\u805a\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUJURUIZHAIQUANA"}, "005420": {"suoxie": "ZOJRZQC", "name": "\u4e2d\u6b27\u805a\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUJURUIZHAIQUANC"}, "005421": {"suoxie": "ZOJZLHPZHH", "name": "\u4e2d\u6b27\u5609\u6cfd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJIAZELINGHUOPEIZHIHUNHE"}, "005425": {"suoxie": "MSJYRT3GYDKZ", "name": "\u6c11\u751f\u52a0\u94f6\u777f\u901a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINRUITONG3GEYUEDINGKAIZHAI"}, "005426": {"suoxie": "GDZFCZDKZ", "name": "\u5149\u5927\u5c0a\u4e30\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNFENGCHUNZHAIDINGKAIZHAI"}, "005427": {"suoxie": "BHHJHZL3GYDK", "name": "\u6e24\u6d77\u6c47\u91d1\u6c47\u589e\u52293\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOHAIHUIJINHUIZENGLI3GEYUEDINGKAI"}, "005428": {"suoxie": "BHHJHTY3GYDK", "name": "\u6e24\u6d77\u6c47\u91d1\u6c47\u6dfb\u76ca3\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOHAIHUIJINHUITIANYI3GEYUEDINGKAI"}, "005429": {"suoxie": "BHHJRXHHA", "name": "\u6e24\u6d77\u6c47\u91d1\u777f\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOHAIHUIJINRUIXUANHUNHEA"}, "005430": {"suoxie": "BHHJRXHHC", "name": "\u6e24\u6d77\u6c47\u91d1\u777f\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOHAIHUIJINRUIXUANHUNHEC"}, "005431": {"suoxie": "SYJZFDQKFZQ", "name": "\u4e0a\u94f6\u805a\u589e\u5bcc\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUZENGFUDINGQIKAIFANGZHAIQUAN"}, "005432": {"suoxie": "SYJHYSGYDKZ", "name": "\u4e0a\u94f6\u805a\u9e3f\u76ca\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUHONGYISANGEYUEDINGKAIZHAI"}, "005433": {"suoxie": "SWLXYYXFGP", "name": "\u7533\u4e07\u83f1\u4fe1\u533b\u836f\u5148\u950b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINYIYAOXIANFENGGUPIAO"}, "005434": {"suoxie": "PHRTHH", "name": "\u9e4f\u534e\u777f\u6295\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUARUITOUHUNHE"}, "005435": {"suoxie": "GTRYSYDKZ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u94f6\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNYINDINGKAIZHAI"}, "005436": {"suoxie": "YXYFXRDKZ", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u745e\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGXINGRUIDINGKAIZHAI"}, "005437": {"suoxie": "YFDYBZNLHCLA", "name": "\u6613\u65b9\u8fbe\u6613\u767e\u667a\u80fd\u91cf\u5316\u7b56\u7565A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAYIBAIZHINENGLIANGHUACELUEA"}, "005438": {"suoxie": "YFDYBZNLHCLC", "name": "\u6613\u65b9\u8fbe\u6613\u767e\u667a\u80fd\u91cf\u5316\u7b56\u7565C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAYIBAIZHINENGLIANGHUACELUEC"}, "005439": {"suoxie": "YFDHADKZFQS", "name": "\u6613\u65b9\u8fbe\u6052\u5b89\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGANDINGKAIZHAIFAQISHI"}, "005442": {"suoxie": "XYAH6GYDKZ", "name": "\u5174\u4e1a\u5b89\u548c6\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEANHE6GEYUEDINGKAIZHAI"}, "005443": {"suoxie": "GJLHDCLHH", "name": "\u56fd\u91d1\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOJINLIANGHUADUOCELUEHUNHE"}, "005444": {"suoxie": "GDDCLJXHH", "name": "\u5149\u5927\u591a\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDADUOCELUEJINGXUANHUNHE"}, "005445": {"suoxie": "HBJZFXHH", "name": "\u534e\u5b9d\u4ef7\u503c\u53d1\u73b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOJIAZHIFAXIANHUNHE"}, "005446": {"suoxie": "XYGLDKZFQS", "name": "\u946b\u5143\u5e7f\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANGUANGLIDINGKAIZHAIFAQISHI"}, "005448": {"suoxie": "NALCSXA", "name": "\u8bfa\u5b89\u8054\u521b\u987a\u946bA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANLIANCHUANGSHUNXINA"}, "005449": {"suoxie": "HXHYLTHH", "name": "\u534e\u590f\u884c\u4e1a\u9f99\u5934\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHANGYELONGTOUHUNHE"}, "005450": {"suoxie": "HXWSLHPZHH", "name": "\u534e\u590f\u7a33\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAWENSHENGLINGHUOPEIZHIHUNHE"}, "005451": {"suoxie": "PYSLZQA", "name": "\u9e4f\u626c\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGSHUANGLIZHAIQUANA"}, "005452": {"suoxie": "PYSLZQC", "name": "\u9e4f\u626c\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGSHUANGLIZHAIQUANC"}, "005453": {"suoxie": "QHKYYLJKA", "name": "\u524d\u6d77\u5f00\u6e90\u533b\u7597\u5065\u5eb7A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYILIAOJIANKANGA"}, "005454": {"suoxie": "QHKYYLJKC", "name": "\u524d\u6d77\u5f00\u6e90\u533b\u7597\u5065\u5eb7C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANYILIAOJIANKANGC"}, "005455": {"suoxie": "JXRFCZDQKFZQ", "name": "\u5efa\u4fe1\u777f\u4e30\u7eaf\u503a\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIFENGCHUNZHAIDINGQIKAIFANGZHAIQUAN"}, "005457": {"suoxie": "JSCCLHXPGP", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u5c0f\u76d8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAXIAOPANGUPIAO"}, "005459": {"suoxie": "YHJYLHPZHHA", "name": "\u94f6\u6cb3\u5609\u8c0a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJIAYILINGHUOPEIZHIHUNHEA"}, "005460": {"suoxie": "YHJYLHPZHHC", "name": "\u94f6\u6cb3\u5609\u8c0a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJIAYILINGHUOPEIZHIHUNHEC"}, "005461": {"suoxie": "NFXYZZ", "name": "\u5357\u65b9\u5e0c\u5143\u8f6c\u503a", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "NANFANGXIYUANZHUANZHAI"}, "005462": {"suoxie": "BSFY3GYDKZ", "name": "\u535a\u65f6\u5bcc\u4e1a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYE3GEYUEDINGKAIZHAI"}, "005465": {"suoxie": "HTZJZHDKZQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u60e0\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHIHUIDINGKAIZHAIQUANA"}, "005466": {"suoxie": "HTZJZHDKZQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u60e0\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHIHUIDINGKAIZHAIQUANC"}, "005467": {"suoxie": "HTZJZYZQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHIYINGZHAIQUANA"}, "005468": {"suoxie": "HTZJZYZQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHIYINGZHAIQUANC"}, "005469": {"suoxie": "NFZLDKZQ", "name": "\u5357\u65b9\u6d59\u5229\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHELIDINGKAIZHAIQUAN"}, "005470": {"suoxie": "NFQLDKZ", "name": "\u5357\u65b9\u4e7e\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGQIANLIDINGKAIZHAI"}, "005471": {"suoxie": "ZSZCTLCZQC", "name": "\u62db\u5546\u62db\u8d22\u901a\u7406\u8d22\u503a\u5238C", "style": "", "pinyin": "ZHAOSHANGZHAOCAITONGLICAIZHAIQUANC"}, "005472": {"suoxie": "FGJZQDLHPZHHA", "name": "\u5bcc\u56fd\u4ef7\u503c\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOJIAZHIQUDONGLINGHUOPEIZHIHUNHEA"}, "005473": {"suoxie": "FGJZQDLHPZHHC", "name": "\u5bcc\u56fd\u4ef7\u503c\u9a71\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOJIAZHIQUDONGLINGHUOPEIZHIHUNHEC"}, "005474": {"suoxie": "TKJHYXHHA", "name": "\u6cf0\u5eb7\u5747\u8861\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGJUNHENGYOUXUANHUNHEA"}, "005475": {"suoxie": "TKJHYXHHC", "name": "\u6cf0\u5eb7\u5747\u8861\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGJUNHENGYOUXUANHUNHEC"}, "005476": {"suoxie": "NFFLDKZ", "name": "\u5357\u65b9\u6daa\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGFULIDINGKAIZHAI"}, "005477": {"suoxie": "CAXXLHPZHHA", "name": "\u957f\u5b89\u946b\u79a7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINXILINGHUOPEIZHIHUNHEA"}, "005478": {"suoxie": "CAXXLHPZHHC", "name": "\u957f\u5b89\u946b\u79a7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINXILINGHUOPEIZHIHUNHEC"}, "005479": {"suoxie": "AXYTDKZFQS", "name": "\u5b89\u4fe1\u6c38\u6cf0\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINYONGTAIDINGKAIZHAIFAQISHI"}, "005480": {"suoxie": "NALCSXC", "name": "\u8bfa\u5b89\u8054\u521b\u987a\u946bC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANLIANCHUANGSHUNXINC"}, "005481": {"suoxie": "YHRTLHPZHH", "name": "\u94f6\u534e\u745e\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUARUITAILINGHUOPEIZHIHUNHE"}, "005482": {"suoxie": "BSCXQDHHA", "name": "\u535a\u65f6\u521b\u65b0\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHUANGXINQUDONGHUNHEA"}, "005483": {"suoxie": "BSCXQDHHC", "name": "\u535a\u65f6\u521b\u65b0\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHUANGXINQUDONGHUNHEC"}, "005485": {"suoxie": "HFTHFDKZQ", "name": "\u6d77\u5bcc\u901a\u6052\u4e30\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGHENGFENGDINGKAIZHAIQUAN"}, "005486": {"suoxie": "PALHJXHHA", "name": "\u5e73\u5b89\u91cf\u5316\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANLIANGHUAJINGXUANHUNHEA"}, "005487": {"suoxie": "PALHJXHHC", "name": "\u5e73\u5b89\u91cf\u5316\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANLIANGHUAJINGXUANHUNHEC"}, "005488": {"suoxie": "THZXDKZFQS", "name": "\u5929\u5f18\u5c0a\u4eab\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGZUNXIANGDINGKAIZHAIFAQISHI"}, "005489": {"suoxie": "ZJHYLHPZHHA", "name": "\u4e2d\u91d1\u8861\u4f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINHENGYOULINGHUOPEIZHIHUNHEA"}, "005490": {"suoxie": "ZJHYLHPZHHC", "name": "\u4e2d\u91d1\u8861\u4f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINHENGYOULINGHUOPEIZHIHUNHEC"}, "005491": {"suoxie": "XQHYHHLOFC", "name": "\u5174\u5168\u5408\u5b9c\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANHEYIHUNHELOFC"}, "005492": {"suoxie": "NYHLYJQDHH", "name": "\u519c\u94f6\u6c47\u7406\u7814\u7a76\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINHUILIYANJIUQUDONGHUNHE"}, "005493": {"suoxie": "XYJZJXHHA", "name": "\u946b\u5143\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANJIAZHIJINGXUANHUNHEA"}, "005494": {"suoxie": "XYJZJXHHC", "name": "\u946b\u5143\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANJIAZHIJINGXUANHUNHEC"}, "005495": {"suoxie": "CJHXKJCZGPA", "name": "\u521b\u91d1\u5408\u4fe1\u79d1\u6280\u6210\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINKEJICHENGZHANGGUPIAOA"}, "005496": {"suoxie": "CJHXKJCZGPC", "name": "\u521b\u91d1\u5408\u4fe1\u79d1\u6280\u6210\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINKEJICHENGZHANGGUPIAOC"}, "005497": {"suoxie": "XYYLZQ", "name": "\u946b\u5143\u6c38\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANYONGLIZHAIQUAN"}, "005498": {"suoxie": "YHJJCZHHA", "name": "\u94f6\u534e\u79ef\u6781\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJIJICHENGZHANGHUNHEA"}, "005501": {"suoxie": "HAAYBNDKZ", "name": "\u534e\u5b89\u5b89\u9038\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANYIBANNIANDINGKAIZHAI"}, "005502": {"suoxie": "HTZJZNLHGPFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u80fd\u91cf\u5316\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "HUATAIZIJINZHINENGLIANGHUAGUPIAOFAQIA"}, "005503": {"suoxie": "HTFLC60TZQE", "name": "\u6c47\u6dfb\u5bcc\u7406\u8d2260\u5929\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFULICAI60TIANZHAIQUANE"}, "005504": {"suoxie": "TFHGSDPJZHHA", "name": "\u6dfb\u5bcc\u6caa\u6e2f\u6df1\u5927\u76d8\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANFUHUGANGSHENDAPANJIAZHIHUNHEA"}, "005505": {"suoxie": "QHKYZYGPA", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u836f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANZHONGYAOGUPIAOA"}, "005506": {"suoxie": "QHKYZYGPC", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u836f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANZHONGYAOGUPIAOC"}, "005507": {"suoxie": "YYFLZQA", "name": "\u6c38\u8d62\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGFENGLIZHAIQUANA"}, "005508": {"suoxie": "YYFLZQC", "name": "\u6c38\u8d62\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGFENGLIZHAIQUANC"}, "005513": {"suoxie": "NHRHZDZZQA", "name": "\u5357\u534e\u745e\u6052\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANHUARUIHENGZHONGDUANZHAIZHAIQUANA"}, "005514": {"suoxie": "NHRHZDZZQC", "name": "\u5357\u534e\u745e\u6052\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANHUARUIHENGZHONGDUANZHAIZHAIQUANC"}, "005517": {"suoxie": "FGXQSLHPZHHA", "name": "\u5bcc\u56fd\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINQUSHILINGHUOPEIZHIHUNHEA"}, "005518": {"suoxie": "FGXQSLHPZHHC", "name": "\u5bcc\u56fd\u65b0\u8d8b\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOXINQUSHILINGHUOPEIZHIHUNHEC"}, "005519": {"suoxie": "YHHGHLLHPZHHFQS", "name": "\u94f6\u534e\u6df7\u6539\u7ea2\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHUNGAIHONGLILINGHUOPEIZHIHUNHEFAQISHI"}, "005520": {"suoxie": "GTRYCXYLHH", "name": "\u56fd\u6295\u745e\u94f6\u521b\u65b0\u533b\u7597\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINCHUANGXINYILIAOHUNHE"}, "005521": {"suoxie": "HAHLJXHH", "name": "\u534e\u5b89\u7ea2\u5229\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHONGLIJINGXUANHUNHE"}, "005522": {"suoxie": "HTBXJNFDKHH", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u798f\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAOXINGJINIANFUDINGKAIHUNHE"}, "005523": {"suoxie": "TKYNHHA", "name": "\u6cf0\u5eb7\u9890\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGYINIANHUNHEA"}, "005524": {"suoxie": "TKYNHHC", "name": "\u6cf0\u5eb7\u9890\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGYINIANHUNHEC"}, "005525": {"suoxie": "GYRXDKFQSZQ", "name": "\u5de5\u94f6\u745e\u7965\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIXIANGDINGKAIFAQISHIZHAIQUAN"}, "005526": {"suoxie": "GYXSDXFHH", "name": "\u5de5\u94f6\u65b0\u751f\u4ee3\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINSHENGDAIXIAOFEIHUNHE"}, "005529": {"suoxie": "YHHMDKZQ", "name": "\u94f6\u534e\u534e\u8302\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAHUAMAODINGKAIZHAIQUAN"}, "005530": {"suoxie": "HTFHS300ZSZQA", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300ZHISHUZENGQIANGA"}, "005531": {"suoxie": "HAAYZQA", "name": "\u534e\u5b89\u5b89\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANYUEZHAIQUANA"}, "005532": {"suoxie": "HAAYZQC", "name": "\u534e\u5b89\u5b89\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANYUEZHAIQUANC"}, "005534": {"suoxie": "HXXSDHHRMBQDII", "name": "\u534e\u590f\u65b0\u65f6\u4ee3\u6df7\u5408\u4eba\u6c11\u5e01(QDII)", "style": "QDII", "pinyin": "HUAXIAXINSHIDAIHUNHERENMINBIQDII"}, "005535": {"suoxie": "TXJZYXHH", "name": "\u6cf0\u4fe1\u7ade\u4e89\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINJINGZHENGYOUXUANHUNHE"}, "005536": {"suoxie": "BHHJLHCZHHFQ", "name": "\u6e24\u6d77\u6c47\u91d1\u91cf\u5316\u6210\u957f\u6df7\u5408\u53d1\u8d77", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOHAIHUIJINLIANGHUACHENGZHANGHUNHEFAQI"}, "005537": {"suoxie": "ZHXQHLHPZHHA", "name": "\u4e2d\u822a\u65b0\u8d77\u822a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHANGXINQIHANGLINGHUOPEIZHIHUNHEA"}, "005538": {"suoxie": "ZHXQHLHPZHHC", "name": "\u4e2d\u822a\u65b0\u8d77\u822a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHANGXINQIHANGLINGHUOPEIZHIHUNHEC"}, "005541": {"suoxie": "QHKYSXHHA", "name": "\u524d\u6d77\u5f00\u6e90\u76db\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANSHENGXINHUNHEA"}, "005542": {"suoxie": "QHKYSXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u76db\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANSHENGXINHUNHEC"}, "005543": {"suoxie": "YHXCLHPZHHA", "name": "\u94f6\u534e\u5fc3\u8bda\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINCHENGLINGHUOPEIZHIHUNHEA"}, "005544": {"suoxie": "YHRHLHPZHH", "name": "\u94f6\u534e\u745e\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUARUIHELINGHUOPEIZHIHUNHE"}, "005545": {"suoxie": "ZYGGHLLHPZHH", "name": "\u4e2d\u94f6\u6539\u9769\u7ea2\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINGAIGEHONGLILINGHUOPEIZHIHUNHE"}, "005547": {"suoxie": "NAYDDKZ", "name": "\u8bfa\u5b89\u5706\u9f0e\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANYUANDINGDINGKAIZHAI"}, "005548": {"suoxie": "NAXXDKZFQS", "name": "\u8bfa\u5b89\u946b\u4eab\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANXINXIANGDINGKAIZHAIFAQISHI"}, "005549": {"suoxie": "FGCZYXSNDKHH", "name": "\u5bcc\u56fd\u6210\u957f\u4f18\u9009\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOCHENGZHANGYOUXUANSANNIANDINGKAIHUNHE"}, "005550": {"suoxie": "HACZYXHHA", "name": "\u6c47\u5b89\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANCHENGZHANGYOUXUANHUNHEA"}, "005551": {"suoxie": "HACZYXHHC", "name": "\u6c47\u5b89\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANCHENGZHANGYOUXUANHUNHEC"}, "005552": {"suoxie": "GFXQSHHA", "name": "\u56fd\u5bcc\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUXINQUSHIHUNHEA"}, "005553": {"suoxie": "GFXQSHHC", "name": "\u56fd\u5bcc\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUXINQUSHIHUNHEC"}, "005554": {"suoxie": "NFHGLJA", "name": "\u5357\u65b9H\u80a1\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHGULIANJIEA"}, "005555": {"suoxie": "NFHGLJC", "name": "\u5357\u65b9H\u80a1\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHGULIANJIEC"}, "005556": {"suoxie": "HAYHDKZFQS", "name": "\u6c47\u5b89\u88d5\u534e\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANYUHUADINGKAIZHAIFAQISHI"}, "005561": {"suoxie": "CJHXHLDBDA", "name": "\u521b\u91d1\u5408\u4fe1\u7ea2\u5229\u4f4e\u6ce2\u52a8A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINHONGLIDIBODONGA"}, "005562": {"suoxie": "CJHXHLDBDC", "name": "\u521b\u91d1\u5408\u4fe1\u7ea2\u5229\u4f4e\u6ce2\u52a8C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHUANGJINHEXINHONGLIDIBODONGC"}, "005569": {"suoxie": "ZRZXHLGPA", "name": "\u4e2d\u878d\u667a\u9009\u7ea2\u5229\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGRONGZHIXUANHONGLIGUPIAOA"}, "005570": {"suoxie": "ZRZXHLGPC", "name": "\u4e2d\u878d\u667a\u9009\u7ea2\u5229\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGRONGZHIXUANHONGLIGUPIAOC"}, "005571": {"suoxie": "ZYZQXNYHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u65b0\u80fd\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANXINNENGYUANHUNHEA"}, "005572": {"suoxie": "ZYZQXNYHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u65b0\u80fd\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANXINNENGYUANHUNHEC"}, "005573": {"suoxie": "DWYXCZZQA", "name": "\u4e1c\u5434\u60a6\u79c0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGWUYUEXIUCHUNZHAIZHAIQUANA"}, "005574": {"suoxie": "DWYXCZZQC", "name": "\u4e1c\u5434\u60a6\u79c0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGWUYUEXIUCHUNZHAIZHAIQUANC"}, "005575": {"suoxie": "CXWXSGYDKZFQS", "name": "\u957f\u4fe1\u7a33\u946b\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENXINSANGEYUEDINGKAIZHAIFAQISHI"}, "005576": {"suoxie": "HTBRXJRDCHH", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u91d1\u878d\u5730\u4ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIXINJINRONGDICHANHUNHE"}, "005577": {"suoxie": "JYFSSYZQA", "name": "\u4ea4\u94f6\u4e30\u665f\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGSHENGSHOUYIZHAIQUANA"}, "005578": {"suoxie": "JYFSSYZQC", "name": "\u4ea4\u94f6\u4e30\u665f\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGSHENGSHOUYIZHAIQUANC"}, "005579": {"suoxie": "GDSLZQA", "name": "\u5149\u5927\u665f\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDASHENGLIZHAIQUANA"}, "005580": {"suoxie": "GDSLZQC", "name": "\u5149\u5927\u665f\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDASHENGLIZHAIQUANC"}, "005583": {"suoxie": "YFDGGTHLHH", "name": "\u6613\u65b9\u8fbe\u6e2f\u80a1\u901a\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAGANGGUTONGHONGLIHUNHE"}, "005585": {"suoxie": "YHWTYLHH", "name": "\u94f6\u6cb3\u6587\u4f53\u5a31\u4e50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEWENTIYULEHUNHE"}, "005587": {"suoxie": "AXBJYSHH", "name": "\u5b89\u4fe1\u6bd4\u8f83\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINBIJIAOYOUSHIHUNHE"}, "005588": {"suoxie": "CAYTHHA", "name": "\u957f\u5b89\u88d5\u817e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUTENGHUNHEA"}, "005589": {"suoxie": "CXQYJXDKHH", "name": "\u957f\u4fe1\u4f01\u4e1a\u7cbe\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINQIYEJINGXUANDINGKAIHUNHE"}, "005590": {"suoxie": "HTFXYDKZA", "name": "\u6c47\u6dfb\u5bcc\u946b\u6c38\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINYONGDINGKAIZHAIA"}, "005591": {"suoxie": "HTFXYDKZC", "name": "\u6c47\u6dfb\u5bcc\u946b\u6c38\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINYONGDINGKAIZHAIC"}, "005592": {"suoxie": "CAYTHHC", "name": "\u957f\u5b89\u88d5\u817e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYUTENGHUNHEC"}, "005593": {"suoxie": "STCXSYMSHH", "name": "\u4e0a\u6295\u521b\u65b0\u5546\u4e1a\u6a21\u5f0f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUCHUANGXINSHANGYEMOSHIHUNHE"}, "005594": {"suoxie": "ZSTR3GYDKZA", "name": "\u62db\u5546\u6dfb\u6da63\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRUN3GEYUEDINGKAIZHAIA"}, "005595": {"suoxie": "ZSTR3GYDKZC", "name": "\u62db\u5546\u6dfb\u6da63\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRUN3GEYUEDINGKAIZHAIC"}, "005596": {"suoxie": "JXZLJXLHPZHHA", "name": "\u5efa\u4fe1\u6218\u7565\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINZHANLUEJINGXUANLINGHUOPEIZHIHUNHEA"}, "005597": {"suoxie": "JXZLJXLHPZHHC", "name": "\u5efa\u4fe1\u6218\u7565\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINZHANLUEJINGXUANLINGHUOPEIZHIHUNHEC"}, "005598": {"suoxie": "GFZXPJXHHA", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZHONGXIAOPANJINGXUANHUNHEA"}, "005599": {"suoxie": "HALHYXLHPZA", "name": "\u6c47\u5b89\u91cf\u5316\u4f18\u9009\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANLIANGHUAYOUXUANLINGHUOPEIZHIA"}, "005600": {"suoxie": "HALHYXLHPZC", "name": "\u6c47\u5b89\u91cf\u5316\u4f18\u9009\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUIANLIANGHUAYOUXUANLINGHUOPEIZHIC"}, "005601": {"suoxie": "HAZDZZQA", "name": "\u6c47\u5b89\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANZHONGDUANZHAIZHAIQUANA"}, "005602": {"suoxie": "HAZDZZQC", "name": "\u6c47\u5b89\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANZHONGDUANZHAIZHAIQUANC"}, "005606": {"suoxie": "ZSZH6GYDKZFQS", "name": "\u62db\u5546\u62db\u9e3f6\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHONG6GEYUEDINGKAIZHAIFAQISHI"}, "005607": {"suoxie": "HBZZ500ZQA", "name": "\u534e\u5b9d\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG500ZENGQIANGA"}, "005608": {"suoxie": "HBZZ500ZQC", "name": "\u534e\u5b9d\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG500ZENGQIANGC"}, "005609": {"suoxie": "FGJGZTHHA", "name": "\u5bcc\u56fd\u519b\u5de5\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNGONGZHUTIHUNHEA"}, "005610": {"suoxie": "ZYTXDQKFZQ", "name": "\u4e2d\u94f6\u6cf0\u4eab\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINTAIXIANGDINGQIKAIFANGZHAIQUAN"}, "005611": {"suoxie": "ZYZQHXDKZ", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u4eab\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIXIANGDINGKAIZHAI"}, "005612": {"suoxie": "JSHXYSGPFQS", "name": "\u5609\u5b9e\u6838\u5fc3\u4f18\u52bf\u80a1\u7968\u53d1\u8d77\u5f0f", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHEXINYOUSHIGUPIAOFAQISHI"}, "005613": {"suoxie": "STFSFDSCREITS", "name": "\u4e0a\u6295\u5bcc\u65f6\u53d1\u8fbe\u5e02\u573aREITs", "style": "QDII", "pinyin": "SHANGTOUFUSHIFADASHICHANGREITS"}, "005614": {"suoxie": "STFSFDSCREITSMC", "name": "\u4e0a\u6295\u5bcc\u65f6\u53d1\u8fbe\u5e02\u573aREITs\u7f8e\u949e", "style": "QDII", "pinyin": "SHANGTOUFUSHIFADASHICHANGREITSMEICHAO"}, "005615": {"suoxie": "STFSFDSCREITSMH", "name": "\u4e0a\u6295\u5bcc\u65f6\u53d1\u8fbe\u5e02\u573aREITs\u7f8e\u6c47", "style": "QDII", "pinyin": "SHANGTOUFUSHIFADASHICHANGREITSMEIHUI"}, "005616": {"suoxie": "DFLHCZLHPZHH", "name": "\u4e1c\u65b9\u91cf\u5316\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGLIANGHUACHENGZHANGLINGHUOPEIZHIHUNHE"}, "005617": {"suoxie": "ZXJX3GYDKZ", "name": "\u4e2d\u4fe1\u5609\u946b3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIAXIN3GEYUEDINGKAIZHAI"}, "005618": {"suoxie": "RTHLJHZTJXHHA", "name": "\u878d\u901a\u7ea2\u5229\u673a\u4f1a\u4e3b\u9898\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGHONGLIJIHUIZHUTIJINGXUANHUNHEA"}, "005619": {"suoxie": "RTHLJHZTJXHHC", "name": "\u878d\u901a\u7ea2\u5229\u673a\u4f1a\u4e3b\u9898\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGHONGLIJIHUIZHUTIJINGXUANHUNHEC"}, "005620": {"suoxie": "ZOPZXFGPA", "name": "\u4e2d\u6b27\u54c1\u8d28\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUPINZHIXIAOFEIGUPIAOA"}, "005621": {"suoxie": "ZOPZXFGPC", "name": "\u4e2d\u6b27\u54c1\u8d28\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUPINZHIXIAOFEIGUPIAOC"}, "005622": {"suoxie": "BSFA3GYDKZ", "name": "\u535a\u65f6\u5bcc\u5b893\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUAN3GEYUEDINGKAIZHAI"}, "005623": {"suoxie": "GFZZ13NNFZZSA", "name": "\u5e7f\u53d1\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI13NIANNONGFAZHAIZHISHUA"}, "005624": {"suoxie": "GFZZ13NNFZZSC", "name": "\u5e7f\u53d1\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI13NIANNONGFAZHAIZHISHUC"}, "005625": {"suoxie": "NHRXDQKFZQ", "name": "\u5357\u534e\u745e\u946b\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUIXINDINGQIKAIFANGZHAIQUAN"}, "005626": {"suoxie": "FGZZYYZTZSZQC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u533b\u836f\u4e3b\u9898\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIYAOZHUTIZHISHUZENGQIANGC"}, "005627": {"suoxie": "FGZZGDZZZSZQXLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u9ad8\u7aef\u5236\u9020\u6307\u6570\u589e\u5f3a\u578b(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGAODUANZHIZAOZHISHUZENGQIANGXINGLOFC"}, "005628": {"suoxie": "HAQSDLGPA", "name": "\u6c47\u5b89\u8d8b\u52bf\u52a8\u529b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIANQUSHIDONGLIGUPIAOA"}, "005629": {"suoxie": "HAQSDLGPC", "name": "\u6c47\u5b89\u8d8b\u52bf\u52a8\u529b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIANQUSHIDONGLIGUPIAOC"}, "005630": {"suoxie": "HAYJJXHHA", "name": "\u534e\u5b89\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUJINGXUANHUNHEA"}, "005631": {"suoxie": "BSFQ3GYDKZ", "name": "\u535a\u65f6\u5bcc\u4e7e3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUQIAN3GEYUEDINGKAIZHAI"}, "005632": {"suoxie": "PHLHXFHH", "name": "\u9e4f\u534e\u91cf\u5316\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUALIANGHUAXIANFENGHUNHE"}, "005633": {"suoxie": "JXZZ500ZSZQC", "name": "\u5efa\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG500ZHISHUZENGQIANGC"}, "005634": {"suoxie": "HAHYLTHH", "name": "\u6c47\u5b89\u884c\u4e1a\u9f99\u5934\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHANGYELONGTOUHUNHE"}, "005635": {"suoxie": "BSLHDCLGPA", "name": "\u535a\u65f6\u91cf\u5316\u591a\u7b56\u7565\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHILIANGHUADUOCELUEGUPIAOA"}, "005636": {"suoxie": "BSLHDCLGPC", "name": "\u535a\u65f6\u91cf\u5316\u591a\u7b56\u7565\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHILIANGHUADUOCELUEGUPIAOC"}, "005637": {"suoxie": "ZRJYDQKFZQ", "name": "\u4e2d\u878d\u805a\u4e1a\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUYEDINGQIKAIFANGZHAIQUAN"}, "005638": {"suoxie": "NYHLLHZHDLHH", "name": "\u519c\u94f6\u6c47\u7406\u91cf\u5316\u667a\u6167\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILILIANGHUAZHIHUIDONGLIHUNHE"}, "005639": {"suoxie": "PA300ETFLJA", "name": "\u5e73\u5b89300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGAN300ETFLIANJIEA"}, "005640": {"suoxie": "PA300ETFLJC", "name": "\u5e73\u5b89300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGAN300ETFLIANJIEC"}, "005641": {"suoxie": "GTRYSY6GYDKZ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u6e906\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNYUAN6GEYUEDINGKAIZHAI"}, "005642": {"suoxie": "PYJSA", "name": "\u9e4f\u626c\u666f\u5347A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGJINGSHENGA"}, "005643": {"suoxie": "PYJSC", "name": "\u9e4f\u626c\u666f\u5347C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGJINGSHENGC"}, "005644": {"suoxie": "GFHGSLTHH", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u9f99\u5934\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENLONGTOUHUNHE"}, "005645": {"suoxie": "HTBXZXDKZ", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u4fe1\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNXINDINGKAIZHAI"}, "005646": {"suoxie": "ZHHGSDCLLHPZHH", "name": "\u4e2d\u6d77\u6caa\u6e2f\u6df1\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIHUGANGSHENDUOCELUELINGHUOPEIZHIHUNHE"}, "005647": {"suoxie": "GFHJDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u4f73\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIJIADINGQIKAIFANGZHAIQUAN"}, "005648": {"suoxie": "ZSTQ3GYDKZA", "name": "\u62db\u5546\u6dfb\u742a3\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANQI3GEYUEDINGKAIZHAIA"}, "005649": {"suoxie": "ZSTQ3GYDKZC", "name": "\u62db\u5546\u6dfb\u742a3\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANQI3GEYUEDINGKAIZHAIC"}, "005650": {"suoxie": "WJLHTSHHA", "name": "\u4e07\u5bb6\u91cf\u5316\u540c\u987a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIALIANGHUATONGSHUNHUNHEA"}, "005651": {"suoxie": "WJLHTSHHC", "name": "\u4e07\u5bb6\u91cf\u5316\u540c\u987a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIALIANGHUATONGSHUNHUNHEC"}, "005652": {"suoxie": "GFTYHHA", "name": "\u56fd\u5bcc\u5929\u9890\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOFUTIANYIHUNHEA"}, "005653": {"suoxie": "GFTYHHC", "name": "\u56fd\u5bcc\u5929\u9890\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOFUTIANYIHUNHEC"}, "005654": {"suoxie": "THYXDKZQ", "name": "\u5929\u5f18\u60a6\u4eab\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGYUEXIANGDINGKAIZHAIQUAN"}, "005655": {"suoxie": "NAZXDKZQ", "name": "\u8bfa\u5b89\u6d59\u4eab\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANZHEXIANGDINGKAIZHAIQUAN"}, "005656": {"suoxie": "GDAZZQA", "name": "\u5149\u5927\u5b89\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANZEZHAIQUANA"}, "005657": {"suoxie": "GDAZZQC", "name": "\u5149\u5927\u5b89\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANZEZHAIQUANC"}, "005658": {"suoxie": "HXHS300ETFLJC", "name": "\u534e\u590f\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ETFLIANJIEC"}, "005659": {"suoxie": "NFHZETFLJC", "name": "\u5357\u65b9\u6052\u6307ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHENGZHIETFLIANJIEC"}, "005660": {"suoxie": "JSZYJXGPA", "name": "\u5609\u5b9e\u8d44\u6e90\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIZIYUANJINGXUANGUPIAOA"}, "005661": {"suoxie": "JSZYJXGPC", "name": "\u5609\u5b9e\u8d44\u6e90\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIZIYUANJINGXUANGUPIAOC"}, "005662": {"suoxie": "JSJRJXGPA", "name": "\u5609\u5b9e\u91d1\u878d\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIJINRONGJINGXUANGUPIAOA"}, "005663": {"suoxie": "JSJRJXGPC", "name": "\u5609\u5b9e\u91d1\u878d\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIJINRONGJINGXUANGUPIAOC"}, "005664": {"suoxie": "PYJXHHA", "name": "\u9e4f\u626c\u666f\u6b23\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGXINHUNHEA"}, "005665": {"suoxie": "PYJXHHC", "name": "\u9e4f\u626c\u666f\u6b23\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGXINHUNHEC"}, "005666": {"suoxie": "SYHJYZQ", "name": "\u4e0a\u94f6\u6167\u4f73\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIJIAYINGZHAIQUAN"}, "005667": {"suoxie": "YFDFCCZ", "name": "\u6613\u65b9\u8fbe\u5bcc\u8d22\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAFUCAICHUNZHAI"}, "005668": {"suoxie": "RTXNYQCZTJXHHA", "name": "\u878d\u901a\u65b0\u80fd\u6e90\u6c7d\u8f66\u4e3b\u9898\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINNENGYUANQICHEZHUTIJINGXUANHUNHEA"}, "005669": {"suoxie": "QHKYGYSYGP", "name": "\u524d\u6d77\u5f00\u6e90\u516c\u7528\u4e8b\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANGONGYONGSHIYEGUPIAO"}, "005670": {"suoxie": "JSZXDKZFQS", "name": "\u5609\u5b9e\u81f4\u5174\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIXINGDINGKAIZHAIFAQISHI"}, "005671": {"suoxie": "QHLHYJYXHHA", "name": "\u524d\u6d77\u8054\u5408\u7814\u7a76\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYANJIUYOUXUANHUNHEA"}, "005672": {"suoxie": "QHLHYJYXHHC", "name": "\u524d\u6d77\u8054\u5408\u7814\u7a76\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYANJIUYOUXUANHUNHEC"}, "005674": {"suoxie": "NDXFSJHH", "name": "\u8bfa\u5fb7\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXIAOFEISHENGJIHUNHE"}, "005675": {"suoxie": "YFDHSGQETFLJC", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u56fd\u4f01ETF\u8054\u63a5C", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGGUOQIETFLIANJIEC"}, "005676": {"suoxie": "YFDBPXFPZSC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u6d88\u8d39\u54c1\u6307\u6570C", "style": "QDII", "pinyin": "YIFANGDABIAOPUXIAOFEIPINZHISHUC"}, "005677": {"suoxie": "AXYSDKZQ", "name": "\u5b89\u4fe1\u6c38\u76db\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGSHENGDINGKAIZHAIQUAN"}, "005678": {"suoxie": "AXZXTYZQA", "name": "\u5b89\u4fe1\u5c0a\u4eab\u6dfb\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZUNXIANGTIANYIZHAIQUANA"}, "005679": {"suoxie": "CTXS6GYDK", "name": "\u8d22\u901a\u946b\u76db6\u4e2a\u6708\u5b9a\u5f00", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGXINSHENG6GEYUEDINGKAI"}, "005680": {"suoxie": "CTZGJZCZHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGZIGUANJIAZHICHENGZHANGHUNHEA"}, "005681": {"suoxie": "CTZGJZCZHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGZIGUANJIAZHICHENGZHANGHUNHEC"}, "005682": {"suoxie": "CTZGXFJXHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u6d88\u8d39\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGZIGUANXIAOFEIJINGXUANHUNHEA"}, "005683": {"suoxie": "GSABHXLHPZHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u534e\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOHUAXINGLINGHUOPEIZHIHUNHE"}, "005684": {"suoxie": "CTHR12GYDKZA", "name": "\u8d22\u901a\u9e3f\u777f12\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGHONGRUI12GEYUEDINGKAIZHAIA"}, "005685": {"suoxie": "CTHR12GYDKZC", "name": "\u8d22\u901a\u9e3f\u777f12\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGHONGRUI12GEYUEDINGKAIZHAIC"}, "005686": {"suoxie": "CTRX12GYDK", "name": "\u8d22\u901a\u745e\u4eab12\u4e2a\u6708\u5b9a\u5f00", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGRUIXIANG12GEYUEDINGKAI"}, "005689": {"suoxie": "ZYYLBJHHA", "name": "\u4e2d\u94f6\u533b\u7597\u4fdd\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIAOBAOJIANHUNHEA"}, "005690": {"suoxie": "ZYAXZQ", "name": "\u4e2d\u94f6\u5b89\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINANXIANGZHAIQUAN"}, "005691": {"suoxie": "NFZZ100ZSC", "name": "\u5357\u65b9\u4e2d\u8bc1100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG100ZHISHUC"}, "005693": {"suoxie": "GFZZJGETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u519b\u5de5ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJUNGONGETFLIANJIEC"}, "005695": {"suoxie": "HARMLNDKHHA", "name": "\u534e\u5b89\u777f\u660e\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANRUIMINGLIANGNIANDINGKAIHUNHEA"}, "005696": {"suoxie": "HARMLNDKHHC", "name": "\u534e\u5b89\u777f\u660e\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANRUIMINGLIANGNIANDINGKAIHUNHEC"}, "005698": {"suoxie": "HXQQKJXFHH", "name": "\u534e\u590f\u5168\u7403\u79d1\u6280\u5148\u950b\u6df7\u5408", "style": "QDII", "pinyin": "HUAXIAQUANQIUKEJIXIANFENGHUNHE"}, "005699": {"suoxie": "GYXJJRMB", "name": "\u5de5\u94f6\u65b0\u7ecf\u6d4e\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GONGYINXINJINGJIRENMINBI"}, "005700": {"suoxie": "GYXJJMY", "name": "\u5de5\u94f6\u65b0\u7ecf\u6d4e\u7f8e\u5143", "style": "QDII", "pinyin": "GONGYINXINJINGJIMEIYUAN"}, "005701": {"suoxie": "STMGXGJXGGT", "name": "\u4e0a\u6295\u6469\u6839\u9999\u6e2f\u7cbe\u9009\u6e2f\u80a1\u901a", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENXIANGGANGJINGXUANGANGGUTONG"}, "005702": {"suoxie": "HSQHGGTGGXZS", "name": "\u6052\u751f\u524d\u6d77\u6e2f\u80a1\u901a\u9ad8\u80a1\u606f\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HENGSHENGQIANHAIGANGGUTONGGAOGUXIZHISHU"}, "005703": {"suoxie": "YYZYZQA", "name": "\u6c38\u8d62\u589e\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZENGYIZHAIQUANA"}, "005704": {"suoxie": "YYZYZQC", "name": "\u6c38\u8d62\u589e\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZENGYIZHAIQUANC"}, "005705": {"suoxie": "YYHYZQ", "name": "\u6c38\u8d62\u6052\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHENGYIZHAIQUAN"}, "005706": {"suoxie": "XYLTSYPHHH", "name": "\u5174\u4e1a\u9f99\u817e\u53cc\u76ca\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGYELONGTENGSHUANGYIPINGHENGHUNHE"}, "005707": {"suoxie": "FGGGTLHJXGPA", "name": "\u5bcc\u56fd\u6e2f\u80a1\u901a\u91cf\u5316\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOGANGGUTONGLIANGHUAJINGXUANGUPIAOA"}, "005708": {"suoxie": "GLAYJCZHH", "name": "\u56fd\u8054\u5b89\u8fdc\u89c1\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANYUANJIANCHENGZHANGHUNHE"}, "005709": {"suoxie": "HADYZQA", "name": "\u534e\u5b89\u9f0e\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANDINGYIZHAIQUANA"}, "005710": {"suoxie": "XYJR3GYDKZ", "name": "\u5174\u4e1a\u5609\u6da63\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIARUN3GEYUEDINGKAIZHAI"}, "005711": {"suoxie": "YYHTLLHPZHH", "name": "\u6c38\u8d62\u60e0\u6dfb\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGHUITIANLILINGHUOPEIZHIHUNHE"}, "005712": {"suoxie": "XQXTDQKFZQ", "name": "\u5174\u5168\u7965\u6cf0\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANXIANGTAIDINGQIKAIFANGZHAIQUAN"}, "005713": {"suoxie": "ZRJJHDKZA", "name": "\u4e2d\u878d\u5b63\u5b63\u7ea2\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJIJIHONGDINGKAIZHAIA"}, "005714": {"suoxie": "ZRJJHDKZC", "name": "\u4e2d\u878d\u5b63\u5b63\u7ea2\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJIJIHONGDINGKAIZHAIC"}, "005717": {"suoxie": "XYJYZQA", "name": "\u5174\u4e1a\u673a\u9047\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYEJIYUZHAIQUANA"}, "005718": {"suoxie": "CXFRLNDKZQA", "name": "\u957f\u4fe1\u5bcc\u745e\u4e24\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFURUILIANGNIANDINGKAIZHAIQUANA"}, "005719": {"suoxie": "ZSZCDKZFQS", "name": "\u62db\u5546\u62db\u8bda\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOCHENGDINGKAIZHAIFAQISHI"}, "005720": {"suoxie": "QHKYQSDKZA", "name": "\u524d\u6d77\u5f00\u6e90\u4e7e\u76db\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANQIANSHENGDINGKAIZHAIA"}, "005721": {"suoxie": "QHKYQSDKZC", "name": "\u524d\u6d77\u5f00\u6e90\u4e7e\u76db\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANQIANSHENGDINGKAIZHAIC"}, "005722": {"suoxie": "QHLHHRDKZQ", "name": "\u524d\u6d77\u8054\u5408\u6cd3\u745e\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEHONGRUIDINGKAIZHAIQUAN"}, "005723": {"suoxie": "ZRJA3GYDKZ", "name": "\u4e2d\u878d\u805a\u5b893\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUAN3GEYUEDINGKAIZHAI"}, "005725": {"suoxie": "GTRYHZZDZA", "name": "\u56fd\u6295\u745e\u94f6\u6052\u6cfd\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTOURUIYINHENGZEZHONGDUANZHAIA"}, "005726": {"suoxie": "GTJZJXLHPZHHA", "name": "\u56fd\u6cf0\u4ef7\u503c\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIAZHIJINGXUANLINGHUOPEIZHIHUNHEA"}, "005727": {"suoxie": "JSZC400ETFLJC", "name": "\u5609\u5b9e\u4e2d\u521b400ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGCHUANG400ETFLIANJIEC"}, "005728": {"suoxie": "HBLSZTHHA", "name": "\u534e\u5b9d\u7eff\u8272\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOLVSEZHUTIHUNHEA"}, "005729": {"suoxie": "NFRGZNHH", "name": "\u5357\u65b9\u4eba\u5de5\u667a\u80fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGRENGONGZHINENGHUNHE"}, "005730": {"suoxie": "GTJYYSJXHHA", "name": "\u56fd\u6cf0\u6c5f\u6e90\u4f18\u52bf\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIANGYUANYOUSHIJINGXUANHUNHEA"}, "005731": {"suoxie": "CTRZ6GYDKZ", "name": "\u8d22\u901a\u777f\u667a6\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGRUIZHI6GEYUEDINGKAIZHAI"}, "005732": {"suoxie": "FGZXCZLHPZHH", "name": "\u5bcc\u56fd\u81fb\u9009\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOZHENXUANCHENGZHANGLINGHUOPEIZHIHUNHE"}, "005733": {"suoxie": "HXSZ50ETFLJC", "name": "\u534e\u590f\u4e0a\u8bc150ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENG50ETFLIANJIEC"}, "005734": {"suoxie": "HXHGTHSETFLJC", "name": "\u534e\u590f\u6caa\u6e2f\u901a\u6052\u751fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUGANGTONGHENGSHENGETFLIANJIEC"}, "005735": {"suoxie": "MSCIZGAGGJTLJC", "name": "MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MSCIZHONGGUOAGUGUOJITONGLIANJIEC"}, "005736": {"suoxie": "ZOXHZQ", "name": "\u4e2d\u6b27\u5174\u534e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINGHUAZHAIQUAN"}, "005737": {"suoxie": "BSSZ50ETFLJC", "name": "\u535a\u65f6\u4e0a\u8bc150ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENG50ETFLIANJIEC"}, "005738": {"suoxie": "CCZNCYLHPZHH", "name": "\u957f\u57ce\u667a\u80fd\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGZHINENGCHANYELINGHUOPEIZHIHUNHE"}, "005739": {"suoxie": "FGZXJYHH", "name": "\u5bcc\u56fd\u8f6c\u578b\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHUANXINGJIYUHUNHE"}, "005740": {"suoxie": "YFDHXDQKFZQ", "name": "\u6613\u65b9\u8fbe\u6052\u4fe1\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGXINDINGQIKAIFANGZHAIQUAN"}, "005741": {"suoxie": "NFJXHHA", "name": "\u5357\u65b9\u541b\u4fe1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJUNXINHUNHEA"}, "005742": {"suoxie": "NFCAYXHH", "name": "\u5357\u65b9\u6210\u5b89\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGCHENGANYOUXUANHUNHE"}, "005743": {"suoxie": "CAYLHHA", "name": "\u957f\u5b89\u88d5\u9686\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYULONGHUNHEA"}, "005744": {"suoxie": "CAYLHHC", "name": "\u957f\u5b89\u88d5\u9686\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANYULONGHUNHEC"}, "005745": {"suoxie": "GFHKDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u5eb7\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIKANGDINGQIKAIFANGZHAIQUAN"}, "005746": {"suoxie": "GTJLJZDKHH", "name": "\u56fd\u6cf0\u805a\u5229\u4ef7\u503c\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJULIJIAZHIDINGKAIHUNHE"}, "005749": {"suoxie": "YHTF3GYDKZQ", "name": "\u94f6\u6cb3\u5ead\u82b33\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHETINGFANG3GEYUEDINGKAIZHAIQUAN"}, "005750": {"suoxie": "PASZTYZQA", "name": "\u5e73\u5b89\u53cc\u503a\u6dfb\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANSHUANGZHAITIANYIZHAIQUANA"}, "005751": {"suoxie": "PASZTYZQC", "name": "\u5e73\u5b89\u53cc\u503a\u6dfb\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANSHUANGZHAITIANYIZHAIQUANC"}, "005752": {"suoxie": "JYTSDKZQ", "name": "\u91d1\u9e70\u6dfb\u76db\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANSHENGDINGKAIZHAIQUAN"}, "005753": {"suoxie": "TDHLJLZQ", "name": "\u6cf0\u8fbe\u5b8f\u5229\u91d1\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIJINLIZHAIQUAN"}, "005754": {"suoxie": "PADZA", "name": "\u5e73\u5b89\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANDUANZHAIA"}, "005755": {"suoxie": "PADZC", "name": "\u5e73\u5b89\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANDUANZHAIC"}, "005756": {"suoxie": "PADZE", "name": "\u5e73\u5b89\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANDUANZHAIE"}, "005758": {"suoxie": "ZRLHJXFOFA", "name": "\u4e2d\u878d\u91cf\u5316\u7cbe\u9009(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGLIANGHUAJINGXUANFOFA"}, "005759": {"suoxie": "ZRLHJXFOFC", "name": "\u4e2d\u878d\u91cf\u5316\u7cbe\u9009(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGLIANGHUAJINGXUANFOFC"}, "005760": {"suoxie": "FGZQYSHHA", "name": "\u5bcc\u56fd\u5468\u671f\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHOUQIYOUSHIHUNHEA"}, "005761": {"suoxie": "ZSMSCIZGAGGJETFLJA", "name": "\u62db\u5546MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGMSCIZHONGGUOAGUGUOJIETFLIANJIEA"}, "005762": {"suoxie": "ZSMSCIZGAGGJETFLJC", "name": "\u62db\u5546MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGMSCIZHONGGUOAGUGUOJIETFLIANJIEC"}, "005763": {"suoxie": "ZODZXXCYHGSGPC", "name": "\u4e2d\u6b27\u7535\u5b50\u4fe1\u606f\u4ea7\u4e1a\u6caa\u6e2f\u6df1\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUDIANZIXINXICHANYEHUGANGSHENGUPIAOC"}, "005764": {"suoxie": "ZOQLJZLHPZHHC", "name": "\u4e2d\u6b27\u6f5c\u529b\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUQIANLIJIAZHILINGHUOPEIZHIHUNHEC"}, "005765": {"suoxie": "ZOMRXCTHHC", "name": "\u4e2d\u6b27\u660e\u777f\u65b0\u5e38\u6001\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUMINGRUIXINCHANGTAIHUNHEC"}, "005766": {"suoxie": "PAHRDKZ", "name": "\u5e73\u5b89\u5408\u745e\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHERUIDINGKAIZHAI"}, "005770": {"suoxie": "XDAYZZHGSGGXJX", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u9ad8\u80a1\u606f\u7cbe\u9009", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINDAAOYINZHONGZHENGHUGANGSHENGAOGUXIJINGXUAN"}, "005771": {"suoxie": "YHKZZZQ", "name": "\u94f6\u534e\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "YINHUAKEZHUANZHAIZHAIQUAN"}, "005772": {"suoxie": "GYRJDKFQSZQ", "name": "\u5de5\u94f6\u745e\u666f\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIJINGDINGKAIFAQISHIZHAIQUAN"}, "005774": {"suoxie": "HXCYSJHH", "name": "\u534e\u590f\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHANYESHENGJIHUNHE"}, "005775": {"suoxie": "ZJZXDLHHA", "name": "\u4e2d\u52a0\u8f6c\u578b\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAZHUANXINGDONGLIHUNHEA"}, "005776": {"suoxie": "ZJZXDLHHC", "name": "\u4e2d\u52a0\u8f6c\u578b\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJIAZHUANXINGDONGLIHUNHEC"}, "005777": {"suoxie": "GFKJDLGP", "name": "\u5e7f\u53d1\u79d1\u6280\u52a8\u529b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAKEJIDONGLIGUPIAO"}, "005778": {"suoxie": "GFHYCZDKZ", "name": "\u5e7f\u53d1\u6c47\u5143\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIYUANCHUNZHAIDINGKAIZHAI"}, "005779": {"suoxie": "XYCLDKZ", "name": "\u946b\u5143\u5e38\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANCHANGLIDINGKAIZHAI"}, "005780": {"suoxie": "XYZLDKZFQS", "name": "\u946b\u5143\u589e\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZENGLIDINGKAIZHAIFAQISHI"}, "005781": {"suoxie": "HFFR3GYDKZ", "name": "\u534e\u5bcc\u5bcc\u745e3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUFURUI3GEYUEDINGKAIZHAI"}, "005782": {"suoxie": "CJHXHYCZYNDKZA", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u76ca\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIYICHUNZHAIYINIANDINGKAIZHAIA"}, "005783": {"suoxie": "CJHXHYCZYNDKZC", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u76ca\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIYICHUNZHAIYINIANDINGKAIZHAIC"}, "005784": {"suoxie": "CJHXHYLGYDKZA", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u8a89\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIYULIUGEYUEDINGKAIZHAIA"}, "005785": {"suoxie": "CJHXHYLGYDKZC", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u8a89\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIYULIUGEYUEDINGKAIZHAIC"}, "005786": {"suoxie": "ZYCZHLDKZ", "name": "\u4e2d\u90ae\u7eaf\u503a\u6c47\u5229\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIHUILIDINGKAIZHAI"}, "005787": {"suoxie": "ZOXQSHHC", "name": "\u4e2d\u6b27\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINQUSHIHUNHEC"}, "005788": {"suoxie": "NFMSCIZGAGLJA", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA\u80a1\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOAGULIANJIEA"}, "005789": {"suoxie": "NFMSCIZGAGLJC", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA\u80a1\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOAGULIANJIEC"}, "005790": {"suoxie": "YHJX3GYDKZ", "name": "\u94f6\u6cb3\u666f\u884c3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJINGXING3GEYUEDINGKAIZHAI"}, "005791": {"suoxie": "HXDFSGYDKZA", "name": "\u534e\u590f\u9f0e\u798f\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGFUSANGEYUEDINGKAIZHAIA"}, "005792": {"suoxie": "HXDFSGYDKZC", "name": "\u534e\u590f\u9f0e\u798f\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGFUSANGEYUEDINGKAIZHAIC"}, "005793": {"suoxie": "HFKZZZQ", "name": "\u534e\u5bcc\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAFUKEZHUANZHAIZHAIQUAN"}, "005794": {"suoxie": "YHXYLHPZHHA", "name": "\u94f6\u534e\u5fc3\u6021\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINYILINGHUOPEIZHIHUNHEA"}, "005795": {"suoxie": "BSZZ500ZSZQC", "name": "\u535a\u65f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG500ZHISHUZENGQIANGC"}, "005801": {"suoxie": "GYYDJJMY", "name": "\u5de5\u94f6\u5370\u5ea6\u57fa\u91d1\u7f8e\u5143", "style": "QDII", "pinyin": "GONGYINYINDUJIJINMEIYUAN"}, "005802": {"suoxie": "HTFZNZZGP", "name": "\u6c47\u6dfb\u5bcc\u667a\u80fd\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUZHINENGZHIZAOGUPIAO"}, "005805": {"suoxie": "HTBRYLJKA", "name": "\u534e\u6cf0\u67cf\u745e\u533b\u7597\u5065\u5eb7A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYILIAOJIANKANGA"}, "005809": {"suoxie": "QHKYYYFOF", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u6e90(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYUYUANFOF"}, "005810": {"suoxie": "NFRXYNHHA", "name": "\u5357\u65b9\u745e\u7965\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGRUIXIANGYINIANHUNHEA"}, "005811": {"suoxie": "NFRXYNHHC", "name": "\u5357\u65b9\u745e\u7965\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGRUIXIANGYINIANHUNHEC"}, "005812": {"suoxie": "PHCYJX", "name": "\u9e4f\u534e\u4ea7\u4e1a\u7cbe\u9009", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUACHANYEJINGXUAN"}, "005813": {"suoxie": "HACESGGTETFLJA", "name": "\u534e\u5b89CES\u6e2f\u80a1\u901aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCESGANGGUTONGETFLIANJIEA"}, "005814": {"suoxie": "HACESGGTETFLJC", "name": "\u534e\u5b89CES\u6e2f\u80a1\u901aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCESGANGGUTONGETFLIANJIEC"}, "005815": {"suoxie": "NYHLRXLHPZHH", "name": "\u519c\u94f6\u6c47\u7406\u777f\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NONGYINHUILIRUIXUANLINGHUOPEIZHIHUNHE"}, "005816": {"suoxie": "GTNHDQKFZQ", "name": "\u56fd\u6cf0\u519c\u60e0\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAINONGHUIDINGQIKAIFANGZHAIQUAN"}, "005817": {"suoxie": "JYSAFSDKZ", "name": "\u91d1\u5143\u987a\u5b89\u6ca3\u987a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANFENGSHUNDINGKAIZHAI"}, "005818": {"suoxie": "JYSAFTDKZFQS", "name": "\u91d1\u5143\u987a\u5b89\u6ca3\u6cf0\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANFENGTAIDINGKAIZHAIFAQISHI"}, "005819": {"suoxie": "GTYSHYHH", "name": "\u56fd\u6cf0\u4f18\u52bf\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIYOUSHIHANGYEHUNHE"}, "005820": {"suoxie": "BSFXCZ3GYDKZFQS", "name": "\u535a\u65f6\u5bcc\u5174\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUXINGCHUNZHAI3GEYUEDINGKAIZHAIFAQISHI"}, "005821": {"suoxie": "WJXJYLTQYHHA", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u9f99\u5934\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINJIYULONGTOUQIYEHUNHEA"}, "005823": {"suoxie": "TKYXHHA", "name": "\u6cf0\u5eb7\u9890\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGYIXIANGHUNHEA"}, "005824": {"suoxie": "TKYXHHC", "name": "\u6cf0\u5eb7\u9890\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGYIXIANGHUNHEC"}, "005825": {"suoxie": "SWLXZNQDGP", "name": "\u7533\u4e07\u83f1\u4fe1\u667a\u80fd\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINZHINENGQUDONGGUPIAO"}, "005826": {"suoxie": "HXQLJXGP", "name": "\u534e\u590f\u6f5c\u9f99\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAQIANLONGJINGXUANGUPIAO"}, "005827": {"suoxie": "YFDLCJXHH", "name": "\u6613\u65b9\u8fbe\u84dd\u7b79\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDALANCHOUJINGXUANHUNHE"}, "005828": {"suoxie": "CJLYDKZ", "name": "\u957f\u6c5f\u4e50\u8d8a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGLEYUEDINGKAIZHAI"}, "005829": {"suoxie": "JXMSCILJA", "name": "\u5efa\u4fe1MSCI\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINMSCILIANJIEA"}, "005830": {"suoxie": "JXMSCILJC", "name": "\u5efa\u4fe1MSCI\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINMSCILIANJIEC"}, "005831": {"suoxie": "PHZY3GYDKZ", "name": "\u9e4f\u534e\u5c0a\u60a63\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNYUE3GEYUEDINGKAIZHAI"}, "005832": {"suoxie": "JSMSCIZGAGGJTLJ", "name": "\u666f\u987aMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNMSCIZHONGGUOAGUGUOJITONGLIANJIE"}, "005833": {"suoxie": "GYHLYXHHA", "name": "\u5de5\u94f6\u7ea2\u5229\u4f18\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINHONGLIYOUXIANGHUNHEA"}, "005834": {"suoxie": "GYHLYXHHC", "name": "\u5de5\u94f6\u7ea2\u5229\u4f18\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINHONGLIYOUXIANGHUNHEC"}, "005836": {"suoxie": "CJHXTYSJHDKZQA", "name": "\u521b\u91d1\u5408\u4fe1\u6cf0\u76c8\u53cc\u5b63\u7ea2\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINTAIYINGSHUANGJIHONGDINGKAIZHAIQUANA"}, "005837": {"suoxie": "CJHXTYSJHDKZQC", "name": "\u521b\u91d1\u5408\u4fe1\u6cf0\u76c8\u53cc\u5b63\u7ea2\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINTAIYINGSHUANGJIHONGDINGKAIZHAIQUANC"}, "005838": {"suoxie": "CJHXZZ13NZJZA", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZHONGZHAI13NIANZHENGJINZHAIA"}, "005839": {"suoxie": "CJHXZZ13NZJZC", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZHONGZHAI13NIANZHENGJINZHAIC"}, "005840": {"suoxie": "FGCYQDHH", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHANYEQUDONGHUNHE"}, "005841": {"suoxie": "FGZLCZDKZ", "name": "\u5bcc\u56fd\u5c0a\u5229\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZUNLICHUNZHAIDINGKAIZHAI"}, "005842": {"suoxie": "HFTHFDKZQ", "name": "\u6d77\u5bcc\u901a\u5f18\u4e30\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGHONGFENGDINGKAIZHAIQUAN"}, "005843": {"suoxie": "JYSAFQZQ", "name": "\u91d1\u5143\u987a\u5b89\u6ca3\u6cc9\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYUANSHUNANFENGQUANZHAIQUAN"}, "005844": {"suoxie": "DFRGZNZTHH", "name": "\u4e1c\u65b9\u4eba\u5de5\u667a\u80fd\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGRENGONGZHINENGZHUTIHUNHE"}, "005845": {"suoxie": "CCJRCZDK", "name": "\u957f\u57ce\u4e45\u8363\u7eaf\u503a\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIURONGCHUNZHAIDINGKAI"}, "005846": {"suoxie": "BYYTCZZQA", "name": "\u5b9d\u76c8\u76c8\u6cf0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGTAICHUNZHAIZHAIQUANA"}, "005847": {"suoxie": "FGHGSYJQDHHXA", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u4e1a\u7ee9\u9a71\u52a8\u6df7\u5408\u578bA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHUGANGSHENYEJIQUDONGHUNHEXINGA"}, "005848": {"suoxie": "YHYLHHFQS", "name": "\u94f6\u534e\u88d5\u5229\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAYULIHUNHEFAQISHI"}, "005849": {"suoxie": "XYHLDKZFQS", "name": "\u946b\u5143\u5408\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHELIDINGKAIZHAIFAQISHI"}, "005850": {"suoxie": "CTLHJZYXHH", "name": "\u8d22\u901a\u91cf\u5316\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGLIANGHUAJIAZHIYOUXUANHUNHE"}, "005851": {"suoxie": "CTXSYLHPZHHA", "name": "\u8d22\u901a\u65b0\u89c6\u91ce\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGXINSHIYELINGHUOPEIZHIHUNHEA"}, "005852": {"suoxie": "ZYTLZQFQC", "name": "\u4e2d\u94f6\u6dfb\u5229\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINTIANLIZHAIQUANFAQIC"}, "005853": {"suoxie": "CTJLCZZQ", "name": "\u8d22\u901a\u805a\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGJULICHUNZHAIZHAIQUAN"}, "005854": {"suoxie": "CTHLCZ", "name": "\u8d22\u901a\u6c47\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGHUILICHUNZHAI"}, "005855": {"suoxie": "ZKWTWRHHA", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWORUIHUNHEA"}, "005856": {"suoxie": "ZKWTWRHHC", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWORUIHUNHEC"}, "005857": {"suoxie": "HTFXCDKZA", "name": "\u6c47\u6dfb\u5bcc\u946b\u6210\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINCHENGDINGKAIZHAIA"}, "005858": {"suoxie": "HTFXCDKZC", "name": "\u6c47\u6dfb\u5bcc\u946b\u6210\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINCHENGDINGKAIZHAIC"}, "005862": {"suoxie": "HXDLSGYDKZQA", "name": "\u534e\u590f\u9f0e\u7984\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLUSANGEYUEDINGKAIZHAIQUANA"}, "005863": {"suoxie": "HXDLSGYDKZQC", "name": "\u534e\u590f\u9f0e\u7984\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLUSANGEYUEDINGKAIZHAIQUANC"}, "005864": {"suoxie": "GTRYSDCZZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u8fbe\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNDACHUNZHAIZHAIQUAN"}, "005865": {"suoxie": "PYASLHDCLHHA", "name": "\u6d66\u94f6\u5b89\u76db\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGLIANGHUADUOCELUEHUNHEA"}, "005866": {"suoxie": "PYASLHDCLHHC", "name": "\u6d66\u94f6\u5b89\u76db\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGLIANGHUADUOCELUEHUNHEC"}, "005867": {"suoxie": "GTHS300ZSC", "name": "\u56fd\u6cf0\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHUSHEN300ZHISHUC"}, "005868": {"suoxie": "PAMSCIZGAGETFLJA", "name": "\u5e73\u5b89MSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANMSCIZHONGGUOAGUETFLIANJIEA"}, "005869": {"suoxie": "PAMSCIZGAGETFLJC", "name": "\u5e73\u5b89MSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANMSCIZHONGGUOAGUETFLIANJIEC"}, "005870": {"suoxie": "PHHS300ZSZQ", "name": "\u9e4f\u534e\u6caa\u6df1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAHUSHEN300ZHISHUZENGQIANG"}, "005871": {"suoxie": "THRXDKZ", "name": "\u5929\u5f18\u8363\u4eab\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGRONGXIANGDINGKAIZHAI"}, "005872": {"suoxie": "TPHLCZ", "name": "\u592a\u5e73\u6052\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGLICHUNZHAI"}, "005873": {"suoxie": "JXCYBETFLJA", "name": "\u5efa\u4fe1\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINCHUANGYEBANETFLIANJIEA"}, "005874": {"suoxie": "JXCYBETFLJC", "name": "\u5efa\u4fe1\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINCHUANGYEBANETFLIANJIEC"}, "005875": {"suoxie": "YFDZPCZHH", "name": "\u6613\u65b9\u8fbe\u4e2d\u76d8\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAZHONGPANCHENGZHANGHUNHE"}, "005876": {"suoxie": "YFDXZZLHHA", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANZENGLIHUNHEA"}, "005877": {"suoxie": "YFDXZZLHHC", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANZENGLIHUNHEC"}, "005878": {"suoxie": "BSCYXDLHHC", "name": "\u535a\u65f6\u4ea7\u4e1a\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEXINDONGLIHUNHEC"}, "005879": {"suoxie": "ZJYXDKZQ", "name": "\u4e2d\u52a0\u9890\u5174\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIXINGDINGKAIZHAIQUAN"}, "005880": {"suoxie": "JXSZ50ETFFQLJA", "name": "\u5efa\u4fe1\u4e0a\u8bc150ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHANGZHENG50ETFFAQILIANJIEA"}, "005881": {"suoxie": "JXSZ50ETFFQLJC", "name": "\u5efa\u4fe1\u4e0a\u8bc150ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHANGZHENG50ETFFAQILIANJIEC"}, "005882": {"suoxie": "CTZGHDCZE", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8fbe\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGDACHUNZHAIE"}, "005884": {"suoxie": "PAHYDKZ", "name": "\u5e73\u5b89\u5408\u60a6\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEYUEDINGKAIZHAI"}, "005885": {"suoxie": "JYXXCYGPC", "name": "\u91d1\u9e70\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGXINXICHANYEGUPIAOC"}, "005886": {"suoxie": "HXDPZQA", "name": "\u534e\u590f\u9f0e\u6c9b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGPEIZHAIQUANA"}, "005887": {"suoxie": "HXDPZQC", "name": "\u534e\u590f\u9f0e\u6c9b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGPEIZHAIQUANC"}, "005888": {"suoxie": "HXXXXFHHA", "name": "\u534e\u590f\u65b0\u5174\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINXINGXIAOFEIHUNHEA"}, "005889": {"suoxie": "HXXXXFHHC", "name": "\u534e\u590f\u65b0\u5174\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINXINGXIAOFEIHUNHEC"}, "005890": {"suoxie": "XFBYCZA", "name": "\u5148\u950b\u535a\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANFENGBOYINGCHUNZHAIA"}, "005891": {"suoxie": "XFBYCZC", "name": "\u5148\u950b\u535a\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANFENGBOYINGCHUNZHAIC"}, "005892": {"suoxie": "XFHYCZA", "name": "\u5148\u950b\u6c47\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANFENGHUIYINGCHUNZHAIA"}, "005893": {"suoxie": "XFHYCZC", "name": "\u5148\u950b\u6c47\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANFENGHUIYINGCHUNZHAIC"}, "005894": {"suoxie": "HXYSJXGP", "name": "\u534e\u590f\u4f18\u52bf\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAYOUSHIJINGXUANGUPIAO"}, "005895": {"suoxie": "PAHFDKZ", "name": "\u5e73\u5b89\u5408\u4e30\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEFENGDINGKAIZHAI"}, "005896": {"suoxie": "PAHHDKZ", "name": "\u5e73\u5b89\u5408\u6167\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEHUIDINGKAIZHAI"}, "005897": {"suoxie": "PAHYDKZ", "name": "\u5e73\u5b89\u5408\u9896\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEYINGDINGKAIZHAI"}, "005898": {"suoxie": "PYASSZDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u6cfd\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGZEDINGKAIZHAIQUAN"}, "005901": {"suoxie": "NAHLLHPZHHA", "name": "\u8bfa\u5b89\u6c47\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANHUILILINGHUOPEIZHIHUNHEA"}, "005902": {"suoxie": "NAHLLHPZHHC", "name": "\u8bfa\u5b89\u6c47\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANHUILILINGHUOPEIZHIHUNHEC"}, "005903": {"suoxie": "TDHLJYHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u7ee9\u4f18\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIJIYOUHUNHE"}, "005904": {"suoxie": "HTBXCZYXA", "name": "\u534e\u6cf0\u4fdd\u5174\u6210\u957f\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGCHENGZHANGYOUXUANA"}, "005905": {"suoxie": "HTBXCZYXC", "name": "\u534e\u6cf0\u4fdd\u5174\u6210\u957f\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGCHENGZHANGYOUXUANC"}, "005906": {"suoxie": "ZSFMLHHHFQSA", "name": "\u62db\u5546\u4e30\u8302\u7075\u6d3b\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGMAOLINGHUOHUNHEFAQISHIA"}, "005907": {"suoxie": "ZSFMLHHHFQSC", "name": "\u62db\u5546\u4e30\u8302\u7075\u6d3b\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGMAOLINGHUOHUNHEFAQISHIC"}, "005908": {"suoxie": "HTBXZLZQA", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAOXINGZUNLIZHAIQUANA"}, "005909": {"suoxie": "HTBXZLZQC", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAOXINGZUNLIZHAIQUANC"}, "005910": {"suoxie": "GFLTYXHH", "name": "\u5e7f\u53d1\u9f99\u5934\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFALONGTOUYOUXUANHUNHE"}, "005911": {"suoxie": "GFSQSJHHA", "name": "\u5e7f\u53d1\u53cc\u64ce\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHUANGQINGSHENGJIHUNHEA"}, "005914": {"suoxie": "JSCCZNSHHH", "name": "\u666f\u987a\u957f\u57ce\u667a\u80fd\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGZHINENGSHENGHUOHUNHE"}, "005917": {"suoxie": "GFHY3GYDKZ", "name": "\u5e7f\u53d1\u6c47\u8a893\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIYU3GEYUEDINGKAIZHAI"}, "005918": {"suoxie": "THHS300ETFLJC", "name": "\u5929\u5f18\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUSHEN300ETFLIANJIEC"}, "005919": {"suoxie": "THZZ500ETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG500ETFLIANJIEC"}, "005920": {"suoxie": "FGYLCZZQ", "name": "\u5bcc\u56fd\u9890\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOYILICHUNZHAIZHAIQUAN"}, "005921": {"suoxie": "NYHLJX3GYDKZ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u946b3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINXIN3GEYUEDINGKAIZHAI"}, "005925": {"suoxie": "JXFZYTHHFOFA", "name": "\u5efa\u4fe1\u798f\u6cfd\u88d5\u6cf0\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINFUZEYUTAIHUNHEFOFA"}, "005926": {"suoxie": "JXFZYTHHFOFC", "name": "\u5efa\u4fe1\u798f\u6cfd\u88d5\u6cf0\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINFUZEYUTAIHUNHEFOFC"}, "005927": {"suoxie": "CJHXXNYQCGPA", "name": "\u521b\u91d1\u5408\u4fe1\u65b0\u80fd\u6e90\u6c7d\u8f66\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINNENGYUANQICHEGUPIAOA"}, "005928": {"suoxie": "CJHXXNYQCGPC", "name": "\u521b\u91d1\u5408\u4fe1\u65b0\u80fd\u6e90\u6c7d\u8f66\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINNENGYUANQICHEGUPIAOC"}, "005931": {"suoxie": "ZRHYCZA", "name": "\u4e2d\u878d\u6052\u88d5\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYUCHUNZHAIA"}, "005932": {"suoxie": "ZRHYCZC", "name": "\u4e2d\u878d\u6052\u88d5\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYUCHUNZHAIC"}, "005933": {"suoxie": "QHLHXJZZHHA", "name": "\u524d\u6d77\u8054\u5408\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEXIANJINZHIZAOHUNHEA"}, "005934": {"suoxie": "QHLHXJZZHHC", "name": "\u524d\u6d77\u8054\u5408\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEXIANJINZHIZAOHUNHEC"}, "005935": {"suoxie": "QHLHRFHHC", "name": "\u524d\u6d77\u8054\u5408\u6da6\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHERUNFENGHUNHEC"}, "005936": {"suoxie": "SWLXATHLCZA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u60e0\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIHUILICHUNZHAIA"}, "005937": {"suoxie": "GYJXJRDCHHA", "name": "\u5de5\u94f6\u7cbe\u9009\u91d1\u878d\u5730\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINGXUANJINRONGDICHANHUNHEA"}, "005938": {"suoxie": "GYJXJRDCHHC", "name": "\u5de5\u94f6\u7cbe\u9009\u91d1\u878d\u5730\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINGXUANJINRONGDICHANHUNHEC"}, "005939": {"suoxie": "GYXNYQCHHA", "name": "\u5de5\u94f6\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINNENGYUANQICHEHUNHEA"}, "005940": {"suoxie": "GYXNYQCHHC", "name": "\u5de5\u94f6\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINNENGYUANQICHEHUNHEC"}, "005943": {"suoxie": "GYJFHHA", "name": "\u5de5\u94f6\u805a\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUFUHUNHEA"}, "005944": {"suoxie": "GYJFHHC", "name": "\u5de5\u94f6\u805a\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUFUHUNHEC"}, "005945": {"suoxie": "GYKZZYXZQA", "name": "\u5de5\u94f6\u53ef\u8f6c\u503a\u4f18\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GONGYINKEZHUANZHAIYOUXUANZHAIQUANA"}, "005946": {"suoxie": "GYKZZYXZQC", "name": "\u5de5\u94f6\u53ef\u8f6c\u503a\u4f18\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GONGYINKEZHUANZHAIYOUXUANZHAIQUANC"}, "005947": {"suoxie": "DBMYJQLHHHA", "name": "\u5fb7\u90a6\u6c11\u88d5\u8fdb\u53d6\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGMINYUJINQULIANGHUAHUNHEA"}, "005948": {"suoxie": "DBMYJQLHHHC", "name": "\u5fb7\u90a6\u6c11\u88d5\u8fdb\u53d6\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGMINYUJINQULIANGHUAHUNHEC"}, "005949": {"suoxie": "XYHYLDHHA", "name": "\u946b\u5143\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANHANGYELUNDONGHUNHEA"}, "005950": {"suoxie": "XYHYLDHHC", "name": "\u946b\u5143\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINYUANHANGYELUNDONGHUNHEC"}, "005951": {"suoxie": "MSJYHYCZA", "name": "\u6c11\u751f\u52a0\u94f6\u6052\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHENGYICHUNZHAIA"}, "005952": {"suoxie": "MSJYHYCZC", "name": "\u6c11\u751f\u52a0\u94f6\u6052\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHENGYICHUNZHAIC"}, "005953": {"suoxie": "RBZXXDLHHA", "name": "\u4eba\u4fdd\u8f6c\u578b\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RENBAOZHUANXINGXINDONGLIHUNHEA"}, "005954": {"suoxie": "RBZXXDLHHC", "name": "\u4eba\u4fdd\u8f6c\u578b\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RENBAOZHUANXINGXINDONGLIHUNHEC"}, "005955": {"suoxie": "YFDXZTLHHA", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANTIANLIHUNHEA"}, "005956": {"suoxie": "YFDXZTLHHC", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANTIANLIHUNHEC"}, "005957": {"suoxie": "HXJFWJHHFQSFOFA", "name": "\u534e\u590f\u805a\u4e30\u7a33\u5065\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUFENGWENJIANHUNHEFAQISHIFOFA"}, "005958": {"suoxie": "HXJFWJHHFQSFOFC", "name": "\u534e\u590f\u805a\u4e30\u7a33\u5065\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUFENGWENJIANHUNHEFAQISHIFOFC"}, "005959": {"suoxie": "CTXSYLHPZHHC", "name": "\u8d22\u901a\u65b0\u89c6\u91ce\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGXINSHIYELINGHUOPEIZHIHUNHEC"}, "005960": {"suoxie": "BSLHJZGPA", "name": "\u535a\u65f6\u91cf\u5316\u4ef7\u503c\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHILIANGHUAJIAZHIGUPIAOA"}, "005961": {"suoxie": "BSLHJZGPC", "name": "\u535a\u65f6\u91cf\u5316\u4ef7\u503c\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHILIANGHUAJIAZHIGUPIAOC"}, "005962": {"suoxie": "BYRGZNGPA", "name": "\u5b9d\u76c8\u4eba\u5de5\u667a\u80fd\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGRENGONGZHINENGGUPIAOA"}, "005963": {"suoxie": "BYRGZNGPC", "name": "\u5b9d\u76c8\u4eba\u5de5\u667a\u80fd\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGRENGONGZHINENGGUPIAOC"}, "005964": {"suoxie": "ZOACDKZFQS", "name": "\u4e2d\u6b27\u5b89\u8d22\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUANCAIDINGKAIZHAIFAQISHI"}, "005965": {"suoxie": "AXZZ500ZSZQA", "name": "\u5b89\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINZHONGZHENG500ZHISHUZENGQIANGA"}, "005966": {"suoxie": "AXZZ500ZSZQC", "name": "\u5b89\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINZHONGZHENG500ZHISHUZENGQIANGC"}, "005967": {"suoxie": "PHCXQDHH", "name": "\u9e4f\u534e\u521b\u65b0\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINQUDONGHUNHE"}, "005968": {"suoxie": "CJHXGYZQGPA", "name": "\u521b\u91d1\u5408\u4fe1\u5de5\u4e1a\u5468\u671f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGONGYEZHOUQIGUPIAOA"}, "005969": {"suoxie": "CJHXGYZQGPC", "name": "\u521b\u91d1\u5408\u4fe1\u5de5\u4e1a\u5468\u671f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGONGYEZHOUQIGUPIAOC"}, "005970": {"suoxie": "GTXFYXGP", "name": "\u56fd\u6cf0\u6d88\u8d39\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIXIAOFEIYOUXUANGUPIAO"}, "005971": {"suoxie": "PAHJZQ", "name": "\u5e73\u5b89\u60e0\u9526\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIJINZHAIQUAN"}, "005972": {"suoxie": "JYYRCZZQA", "name": "\u4ea4\u94f6\u88d5\u5982\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYURUCHUNZHAIZHAIQUANA"}, "005973": {"suoxie": "JYYRCZZQC", "name": "\u4ea4\u94f6\u88d5\u5982\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYURUCHUNZHAIZHAIQUANC"}, "005974": {"suoxie": "DFHPZJXHHA", "name": "\u4e1c\u65b9\u7ea2\u914d\u7f6e\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGPEIZHIJINGXUANHUNHEA"}, "005975": {"suoxie": "DFHPZJXHHC", "name": "\u4e1c\u65b9\u7ea2\u914d\u7f6e\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGPEIZHIJINGXUANHUNHEC"}, "005976": {"suoxie": "CXWJZCPZFOF", "name": "\u957f\u4fe1\u7a33\u8fdb\u8d44\u4ea7\u914d\u7f6e(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJINZICHANPEIZHIFOF"}, "005977": {"suoxie": "ZXBCZXHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINBAOCHENGZHIXINGHUNHEA"}, "005978": {"suoxie": "ZXBCZXHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINBAOCHENGZHIXINGHUNHEC"}, "005979": {"suoxie": "NFHSDZCFOFA", "name": "\u5357\u65b9\u5408\u987a\u591a\u8d44\u4ea7(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHESHUNDUOZICHANFOFA"}, "005980": {"suoxie": "NFHSDZCFOFC", "name": "\u5357\u65b9\u5408\u987a\u591a\u8d44\u4ea7(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHESHUNDUOZICHANFOFC"}, "005983": {"suoxie": "STMGHXJXGPA", "name": "\u4e0a\u6295\u6469\u6839\u6838\u5fc3\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHEXINJINGXUANGUPIAOA"}, "005984": {"suoxie": "XYJHHHA", "name": "\u5174\u4e1a\u805a\u534e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUHUAHUNHEA"}, "005985": {"suoxie": "XYJHHHC", "name": "\u5174\u4e1a\u805a\u534e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUHUAHUNHEC"}, "005988": {"suoxie": "XYCZ6GYDKZA", "name": "\u5174\u4e1a\u7eaf\u503a6\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYECHUNZHAI6GEYUEDINGKAIZHAIA"}, "005989": {"suoxie": "XYCZ6GYDKZC", "name": "\u5174\u4e1a\u7eaf\u503a6\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYECHUNZHAI6GEYUEDINGKAIZHAIC"}, "005990": {"suoxie": "SWLXATHLCZC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u60e0\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIHUILICHUNZHAIC"}, "005991": {"suoxie": "CXLFZQA", "name": "\u957f\u4fe1\u5229\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFENGZHAIQUANA"}, "005992": {"suoxie": "GDCDZA", "name": "\u5149\u5927\u8d85\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGDACHAODUANZHAIA"}, "005993": {"suoxie": "GDCDZC", "name": "\u5149\u5927\u8d85\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGDACHAODUANZHAIC"}, "005994": {"suoxie": "GTZZ500ZSLHZQA", "name": "\u56fd\u6295\u4e2d\u8bc1500\u6307\u6570\u91cf\u5316\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOUZHONGZHENG500ZHISHULIANGHUAZENGQIANGA"}, "005995": {"suoxie": "GTRYSHZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u6cd3\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNHONGZHAIQUAN"}, "005996": {"suoxie": "GTRYSCCZZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u660c\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNCHANGCHUNZHAIZHAIQUAN"}, "005997": {"suoxie": "THYLLHPZHHC", "name": "\u5929\u5f18\u88d5\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGYULILINGHUOPEIZHIHUNHEC"}, "005998": {"suoxie": "JSSZJBM120LJC", "name": "\u5609\u5b9e\u6df1\u8bc1\u57fa\u672c\u9762120\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHISHENZHENGJIBENMIAN120LIANJIEC"}, "005999": {"suoxie": "JSZZJRDCETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u91d1\u878d\u5730\u4ea7ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGJINRONGDICHANETFLIANJIEC"}, "006002": {"suoxie": "GYYYJKGPA", "name": "\u5de5\u94f6\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYIYAOJIANKANGGUPIAOA"}, "006003": {"suoxie": "GYYYJKGPC", "name": "\u5de5\u94f6\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYIYAOJIANKANGGUPIAOC"}, "006004": {"suoxie": "GYTXYNDKZQ", "name": "\u5de5\u94f6\u6dfb\u7965\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANXIANGYINIANDINGKAIZHAIQUAN"}, "006005": {"suoxie": "NADLHHA", "name": "\u8bfa\u5b89\u9f0e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NUOANDINGLIHUNHEA"}, "006006": {"suoxie": "NADLHHC", "name": "\u8bfa\u5b89\u9f0e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NUOANDINGLIHUNHEC"}, "006007": {"suoxie": "NAJJPZHHA", "name": "\u8bfa\u5b89\u79ef\u6781\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANJIJIPEIZHIHUNHEA"}, "006008": {"suoxie": "NAJJPZHHC", "name": "\u8bfa\u5b89\u79ef\u6781\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANJIJIPEIZHIHUNHEC"}, "006009": {"suoxie": "GRRYLHPZHHA", "name": "\u56fd\u878d\u878d\u94f6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGYINLINGHUOPEIZHIHUNHEA"}, "006010": {"suoxie": "GRRYLHPZHHC", "name": "\u56fd\u878d\u878d\u94f6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGYINLINGHUOPEIZHIHUNHEC"}, "006011": {"suoxie": "ZXBCWHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u9e3fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENHONGA"}, "006012": {"suoxie": "ZXBCWHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u9e3fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENHONGC"}, "006013": {"suoxie": "YFDXZZLHHA", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u62db\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANZHAOLIHUNHEA"}, "006014": {"suoxie": "YFDXZZLHHC", "name": "\u6613\u65b9\u8fbe\u946b\u8f6c\u62db\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAXINZHUANZHAOLIHUNHEC"}, "006015": {"suoxie": "HAXYSJHZQC", "name": "\u534e\u5b89\u4fe1\u7528\u56db\u5b63\u7ea2\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINYONGSIJIHONGZHAIQUANC"}, "006016": {"suoxie": "PAHAZQ", "name": "\u5e73\u5b89\u60e0\u5b89\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIANZHAIQUAN"}, "006020": {"suoxie": "GFHS300ZSZQA", "name": "\u5e7f\u53d1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ZHISHUZENGQIANGA"}, "006021": {"suoxie": "GFHS300ZSZQC", "name": "\u5e7f\u53d1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ZHISHUZENGQIANGC"}, "006022": {"suoxie": "FGDPJZLHJXHHA", "name": "\u5bcc\u56fd\u5927\u76d8\u4ef7\u503c\u91cf\u5316\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODAPANJIAZHILIANGHUAJINGXUANHUNHEA"}, "006023": {"suoxie": "BYJFLNDKZQA", "name": "\u5b9d\u76c8\u805a\u4e30\u4e24\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGJUFENGLIANGNIANDINGKAIZHAIQUANA"}, "006024": {"suoxie": "BYJFLNDKZQC", "name": "\u5b9d\u76c8\u805a\u4e30\u4e24\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGJUFENGLIANGNIANDINGKAIZHAIQUANC"}, "006025": {"suoxie": "NAYHPZHH", "name": "\u8bfa\u5b89\u4f18\u5316\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANYOUHUAPEIZHIHUNHE"}, "006026": {"suoxie": "DWDTCZZQA", "name": "\u4e1c\u5434\u9f0e\u6cf0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGWUDINGTAICHUNZHAIZHAIQUANA"}, "006027": {"suoxie": "GTRYSXZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u7965\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNXIANGZHAIQUAN"}, "006029": {"suoxie": "PHZXDKZFQS", "name": "\u9e4f\u534e\u5c0a\u4eab\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNXIANGDINGKAIZHAIFAQISHI"}, "006030": {"suoxie": "NFCYZZA", "name": "\u5357\u65b9\u660c\u5143\u8f6c\u503aA", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "NANFANGCHANGYUANZHUANZHAIA"}, "006031": {"suoxie": "NFCYC", "name": "\u5357\u65b9\u660c\u5143C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "NANFANGCHANGYUANC"}, "006032": {"suoxie": "CJHXHZSGYDKZQ", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u6cfd\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIZESANGEYUEDINGKAIZHAIQUAN"}, "006034": {"suoxie": "FGMSCIZGAGGJT", "name": "\u5bcc\u56fdMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOMSCIZHONGGUOAGUGUOJITONG"}, "006035": {"suoxie": "ZRHHCZA", "name": "\u4e2d\u878d\u6052\u60e0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGHUICHUNZHAIA"}, "006036": {"suoxie": "ZRHHCZC", "name": "\u4e2d\u878d\u6052\u60e0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGHUICHUNZHAIC"}, "006037": {"suoxie": "GTRHCZZQ", "name": "\u56fd\u6cf0\u745e\u548c\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUIHECHUNZHAIZHAIQUAN"}, "006038": {"suoxie": "DCJHHHC", "name": "\u5927\u6210\u666f\u6052\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGHENGHUNHEC"}, "006039": {"suoxie": "GFGZYSHH", "name": "\u56fd\u5bcc\u4f30\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUGUZHIYOUSHIHUNHE"}, "006040": {"suoxie": "AXYRDKZQ", "name": "\u5b89\u4fe1\u6c38\u745e\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGRUIDINGKAIZHAIQUAN"}, "006042": {"suoxie": "STMGSRHHFOFA", "name": "\u4e0a\u6295\u6469\u6839\u5c1a\u777f\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENSHANGRUIHUNHEFOFA"}, "006043": {"suoxie": "YYHYZQA", "name": "\u6c38\u8d62\u60e0\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHUIYIZHAIQUANA"}, "006044": {"suoxie": "YYHYZQC", "name": "\u6c38\u8d62\u60e0\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHUIYIZHAIQUANC"}, "006045": {"suoxie": "CCJRSGYDKZFQS", "name": "\u957f\u57ce\u4e45\u745e\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIURUISANGEYUEDINGKAIZHAIFAQISHI"}, "006047": {"suoxie": "CXWJCZZQE", "name": "\u957f\u4fe1\u7a33\u5065\u7eaf\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENJIANCHUNZHAIZHAIQUANE"}, "006048": {"suoxie": "CCZZ500ZSZQA", "name": "\u957f\u57ce\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGZHONGZHENG500ZHISHUZENGQIANGA"}, "006049": {"suoxie": "HYYJJXHHAB", "name": "\u6052\u8d8a\u7814\u7a76\u7cbe\u9009\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYANJIUJINGXUANHUNHEAB"}, "006050": {"suoxie": "HYYJJXHHAB", "name": "\u6052\u8d8a\u7814\u7a76\u7cbe\u9009\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYANJIUJINGXUANHUNHEAB"}, "006051": {"suoxie": "PYHXJZLHPZA", "name": "\u9e4f\u626c\u6838\u5fc3\u4ef7\u503c\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGHEXINJIAZHILINGHUOPEIZHIA"}, "006052": {"suoxie": "PYHXJZLHPZC", "name": "\u9e4f\u626c\u6838\u5fc3\u4ef7\u503c\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGHEXINJIAZHILINGHUOPEIZHIC"}, "006053": {"suoxie": "ZHRJ3GYDKA", "name": "\u4e2d\u822a\u745e\u666f3\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIJING3GEYUEDINGKAIA"}, "006054": {"suoxie": "ZHRJ3GYDKC", "name": "\u4e2d\u822a\u745e\u666f3\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIJING3GEYUEDINGKAIC"}, "006055": {"suoxie": "PYCHZQ", "name": "\u9e4f\u626c\u6df3\u5408\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNHEZHAIQUAN"}, "006057": {"suoxie": "PHFHZQLOFC", "name": "\u9e4f\u534e\u4e30\u548c\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAFENGHEZHAIQUANLOFC"}, "006058": {"suoxie": "MSJYXXCZHH", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINXINGCHENGZHANGHUNHE"}, "006059": {"suoxie": "PYHLZQA", "name": "\u9e4f\u626c\u6cd3\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGHONGLIZHAIQUANA"}, "006060": {"suoxie": "PYHLZQC", "name": "\u9e4f\u626c\u6cd3\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGHONGLIZHAIQUANC"}, "006061": {"suoxie": "HTCXZQSYZQA", "name": "\u7ea2\u571f\u521b\u65b0\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTUCHUANGXINZENGQIANGSHOUYIZHAIQUANA"}, "006063": {"suoxie": "JSCCMSCIZGAGGJT", "name": "\u666f\u987a\u957f\u57ceMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGMSCIZHONGGUOAGUGUOJITONG"}, "006064": {"suoxie": "HTCXZQSYZQC", "name": "\u7ea2\u571f\u521b\u65b0\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HONGTUCHUANGXINZENGQIANGSHOUYIZHAIQUANC"}, "006065": {"suoxie": "JSCCJTWLDKZC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u7a33\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIWENLIDINGKAIZHAIC"}, "006066": {"suoxie": "ZJYRCZZQA", "name": "\u4e2d\u52a0\u9890\u777f\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIRUICHUNZHAIZHAIQUANA"}, "006067": {"suoxie": "ZJYRCZZQC", "name": "\u4e2d\u52a0\u9890\u777f\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIRUICHUNZHAIZHAIQUANC"}, "006068": {"suoxie": "ZJYXCZZQA", "name": "\u4e2d\u52a0\u9890\u4fe1\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIXINCHUNZHAIZHAIQUANA"}, "006069": {"suoxie": "ZJYXCZZQC", "name": "\u4e2d\u52a0\u9890\u4fe1\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIXINCHUNZHAIZHAIQUANC"}, "006070": {"suoxie": "YHWFZQ", "name": "\u94f6\u6cb3\u6c83\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEWOFENGZHAIQUAN"}, "006071": {"suoxie": "YHRJZQA", "name": "\u94f6\u6cb3\u777f\u5609\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHERUIJIAZHAIQUANA"}, "006072": {"suoxie": "MSJYCXCZHHA", "name": "\u6c11\u751f\u52a0\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINCHUANGXINCHENGZHANGHUNHEA"}, "006073": {"suoxie": "RBXRZDZZQA", "name": "\u4eba\u4fdd\u946b\u745e\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "RENBAOXINRUIZHONGDUANZHAIZHAIQUANA"}, "006074": {"suoxie": "RBXRZDZZQC", "name": "\u4eba\u4fdd\u946b\u745e\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "RENBAOXINRUIZHONGDUANZHAIZHAIQUANC"}, "006075": {"suoxie": "BSBP500ETFLJC", "name": "\u535a\u65f6\u6807\u666e500ETF\u8054\u63a5C", "style": "QDII", "pinyin": "BOSHIBIAOPU500ETFLIANJIEC"}, "006076": {"suoxie": "CJHXHLCDZZQA", "name": "\u521b\u91d1\u5408\u4fe1\u6052\u5229\u8d85\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINHENGLICHAODUANZHAIZHAIQUANA"}, "006077": {"suoxie": "CJHXHLCDZZQC", "name": "\u521b\u91d1\u5408\u4fe1\u6052\u5229\u8d85\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINHENGLICHAODUANZHAIZHAIQUANC"}, "006080": {"suoxie": "HFTDZCMGPC", "name": "\u6d77\u5bcc\u901a\u7535\u5b50\u4f20\u5a92\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGDIANZICHUANMEIGUPIAOC"}, "006081": {"suoxie": "HFTDZCMGPA", "name": "\u6d77\u5bcc\u901a\u7535\u5b50\u4f20\u5a92\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGDIANZICHUANMEIGUPIAOA"}, "006082": {"suoxie": "XYQLYNDKZA", "name": "\u946b\u5143\u5168\u5229\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANQUANLIYINIANDINGKAIZHAIA"}, "006083": {"suoxie": "XYQLYNDKZC", "name": "\u946b\u5143\u5168\u5229\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANQUANLIYINIANDINGKAIZHAIC"}, "006084": {"suoxie": "RTYJYXHH", "name": "\u878d\u901a\u7814\u7a76\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGYANJIUYOUXUANHUNHE"}, "006085": {"suoxie": "WJXJYJZQDC", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u4ef7\u503c\u9a71\u52a8C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINJIYUJIAZHIQUDONGC"}, "006086": {"suoxie": "YHRFDKZQ", "name": "\u94f6\u6cb3\u777f\u4e30\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHERUIFENGDINGKAIZHAIQUAN"}, "006087": {"suoxie": "HTBRZZ500ETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENG500ETFLIANJIEC"}, "006088": {"suoxie": "YYRYZQA", "name": "\u6c38\u8d62\u6da6\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRUNYIZHAIQUANA"}, "006089": {"suoxie": "YYRYZQC", "name": "\u6c38\u8d62\u6da6\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRUNYIZHAIQUANC"}, "006092": {"suoxie": "YYRYZQA", "name": "\u6c38\u8d62\u8363\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRONGYIZHAIQUANA"}, "006093": {"suoxie": "YYRYZQC", "name": "\u6c38\u8d62\u8363\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRONGYIZHAIQUANC"}, "006094": {"suoxie": "YYTYZQA", "name": "\u6c38\u8d62\u6cf0\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTAIYIZHAIQUANA"}, "006095": {"suoxie": "YYTYZQC", "name": "\u6c38\u8d62\u6cf0\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTAIYIZHAIQUANC"}, "006096": {"suoxie": "ZJZJ6GYDKZ", "name": "\u4e2d\u91d1\u6d59\u91d16\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINZHEJIN6GEYUEDINGKAIZHAI"}, "006097": {"suoxie": "PAGDJZA", "name": "\u5e73\u5b89\u9ad8\u7b49\u7ea7\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANGAODENGJIZHAIA"}, "006098": {"suoxie": "HBQSETFLJA", "name": "\u534e\u5b9d\u5238\u5546ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOQUANSHANGETFLIANJIEA"}, "006099": {"suoxie": "TDHLZLZQ", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6cfd\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIZELIZHAIQUAN"}, "006100": {"suoxie": "PAYSCYHHA", "name": "\u5e73\u5b89\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANYOUSHICHANYEHUNHEA"}, "006101": {"suoxie": "PAYSCYHHC", "name": "\u5e73\u5b89\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANYOUSHICHANYEHUNHEC"}, "006102": {"suoxie": "ZSFLZQZQ", "name": "\u6d59\u5546\u4e30\u5229\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGFENGLIZENGQIANGZHAIQUAN"}, "006104": {"suoxie": "HTBRLHZHHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u667a\u6167\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAZHIHUIHUNHEC"}, "006105": {"suoxie": "TDHLYDQDII", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5370\u5ea6(QDII)", "style": "QDII", "pinyin": "TAIDAHONGLIYINDUQDII"}, "006106": {"suoxie": "JSCCLHGGTGP", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u6e2f\u80a1\u901a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAGANGGUTONGGUPIAO"}, "006107": {"suoxie": "ZSTL6GYDKZFQSA", "name": "\u62db\u5546\u6dfb\u52296\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANLI6GEYUEDINGKAIZHAIFAQISHIA"}, "006108": {"suoxie": "ZSTL6GYDKZFQSC", "name": "\u62db\u5546\u6dfb\u52296\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANLI6GEYUEDINGKAIZHAIFAQISHIC"}, "006109": {"suoxie": "FRJZJXHHA", "name": "\u5bcc\u8363\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FURONGJIAZHIJINGXUANHUNHEA"}, "006110": {"suoxie": "FRJZJXHHC", "name": "\u5bcc\u8363\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FURONGJIAZHIJINGXUANHUNHEC"}, "006111": {"suoxie": "TKHS3YDKHH", "name": "\u6cf0\u5eb7\u5f18\u5b9e3\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGHONGSHI3YUEDINGKAIHUNHE"}, "006112": {"suoxie": "YFDHHDKZ", "name": "\u6613\u65b9\u8fbe\u6052\u60e0\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGHUIDINGKAIZHAI"}, "006113": {"suoxie": "HTFCXYYHH", "name": "\u6c47\u6dfb\u5bcc\u521b\u65b0\u533b\u836f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHUANGXINYIYAOHUNHE"}, "006114": {"suoxie": "RBXLZQA", "name": "\u4eba\u4fdd\u946b\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RENBAOXINLIZHAIQUANA"}, "006115": {"suoxie": "RBXLZQC", "name": "\u4eba\u4fdd\u946b\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RENBAOXINLIZHAIQUANC"}, "006116": {"suoxie": "GTFQCZZQ", "name": "\u56fd\u6cf0\u4e30\u797a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIFENGQICHUNZHAIZHAIQUAN"}, "006119": {"suoxie": "YHZZYQJGTZETFLJ", "name": "\u94f6\u534e\u4e2d\u8bc1\u592e\u4f01\u7ed3\u6784\u8c03\u6574ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGYANGQIJIEGOUTIAOZHENGETFLIANJIE"}, "006120": {"suoxie": "ZRJMDQKFZQ", "name": "\u4e2d\u878d\u805a\u660e\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUMINGDINGQIKAIFANGZHAIQUAN"}, "006121": {"suoxie": "HASHQDHHA", "name": "\u534e\u5b89\u53cc\u6838\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANSHUANGHEQUDONGHUNHEA"}, "006122": {"suoxie": "HADTSHHH", "name": "\u534e\u5b89\u4f4e\u78b3\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANDITANSHENGHUOHUNHE"}, "006123": {"suoxie": "ZRGGXHHA", "name": "\u4e2d\u878d\u9ad8\u80a1\u606f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGGAOGUXIHUNHEA"}, "006124": {"suoxie": "ZRGGXHHC", "name": "\u4e2d\u878d\u9ad8\u80a1\u606f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGGAOGUXIHUNHEC"}, "006127": {"suoxie": "HBXGZXC", "name": "\u534e\u5b9d\u9999\u6e2f\u4e2d\u5c0fC", "style": "QDII", "pinyin": "HUABAOXIANGGANGZHONGXIAOC"}, "006128": {"suoxie": "YHHMSHHH", "name": "\u94f6\u6cb3\u548c\u7f8e\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEHEMEISHENGHUOHUNHE"}, "006129": {"suoxie": "HAZZ500DBETFLJA", "name": "\u534e\u5b89\u4e2d\u8bc1500\u4f4e\u6ce2ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENG500DIBOETFLIANJIEA"}, "006130": {"suoxie": "HAZZ500DBETFLJC", "name": "\u534e\u5b89\u4e2d\u8bc1500\u4f4e\u6ce2ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENG500DIBOETFLIANJIEC"}, "006131": {"suoxie": "HTBRHS300ETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIHUSHEN300ETFLIANJIEC"}, "006132": {"suoxie": "WJZZYSHHA", "name": "\u4e07\u5bb6\u667a\u9020\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHIZAOYOUSHIHUNHEA"}, "006133": {"suoxie": "WJZZYSHHC", "name": "\u4e07\u5bb6\u667a\u9020\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHIZAOYOUSHIHUNHEC"}, "006134": {"suoxie": "FGJRZZQX", "name": "\u5bcc\u56fd\u91d1\u878d\u503a\u503a\u5238\u578b", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOJINRONGZHAIZHAIQUANXING"}, "006135": {"suoxie": "CJLXDKZ", "name": "\u957f\u6c5f\u4e50\u946b\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGLEXINDINGKAIZHAI"}, "006136": {"suoxie": "GFGZYSHHA", "name": "\u5e7f\u53d1\u4f30\u503c\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGUZHIYOUSHIHUNHEA"}, "006137": {"suoxie": "GFHLDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u7acb\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUILIDINGQIKAIFANGZHAIQUAN"}, "006138": {"suoxie": "GLAJZYXGP", "name": "\u56fd\u8054\u5b89\u4ef7\u503c\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOLIANANJIAZHIYOUXUANGUPIAO"}, "006140": {"suoxie": "GFJJZQA", "name": "\u5e7f\u53d1\u96c6\u5609\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIJIAZHAIQUANA"}, "006141": {"suoxie": "GFJJZQC", "name": "\u5e7f\u53d1\u96c6\u5609\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIJIAZHAIQUANC"}, "006142": {"suoxie": "XYCLDQKFZQ", "name": "\u946b\u5143\u6df3\u5229\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANCHUNLIDINGQIKAIFANGZHAIQUAN"}, "006143": {"suoxie": "HSQHZZZLCZA", "name": "\u6052\u751f\u524d\u6d77\u4e2d\u8bc1\u8d28\u91cf\u6210\u957fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HENGSHENGQIANHAIZHONGZHENGZHILIANGCHENGZHANGA"}, "006144": {"suoxie": "HSQHZZZLCZC", "name": "\u6052\u751f\u524d\u6d77\u4e2d\u8bc1\u8d28\u91cf\u6210\u957fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HENGSHENGQIANHAIZHONGZHENGZHILIANGCHENGZHANGC"}, "006145": {"suoxie": "QHKYDXZQA", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u6b23\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANDINGXINZHAIQUANA"}, "006146": {"suoxie": "QHKYDXZQC", "name": "\u524d\u6d77\u5f00\u6e90\u9f0e\u6b23\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANDINGXINZHAIQUANC"}, "006147": {"suoxie": "BYRYKZZZQA", "name": "\u5b9d\u76c8\u878d\u6e90\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "BAOYINGRONGYUANKEZHUANZHAIZHAIQUANA"}, "006148": {"suoxie": "BYRYKZZZQC", "name": "\u5b9d\u76c8\u878d\u6e90\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "BAOYINGRONGYUANKEZHUANZHAIZHAIQUANC"}, "006149": {"suoxie": "NFYY", "name": "\u5357\u65b9\u8d62\u5143", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGYINGYUAN"}, "006150": {"suoxie": "ZSTLLNZQ", "name": "\u62db\u5546\u6dfb\u5229\u4e24\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANLILIANGNIANZHAIQUAN"}, "006151": {"suoxie": "NFJYZQ", "name": "\u5357\u65b9\u4ea4\u5143\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJIAOYUANZHAIQUAN"}, "006152": {"suoxie": "GLAZXCZA", "name": "\u56fd\u8054\u5b89\u589e\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGXINCHUNZHAIA"}, "006153": {"suoxie": "GLAZXCZC", "name": "\u56fd\u8054\u5b89\u589e\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGXINCHUNZHAIC"}, "006154": {"suoxie": "HAZZXFHHA", "name": "\u534e\u5b89\u5236\u9020\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHIZAOXIANFENGHUNHEA"}, "006157": {"suoxie": "CTLHHXYXHH", "name": "\u8d22\u901a\u91cf\u5316\u6838\u5fc3\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGLIANGHUAHEXINYOUXUANHUNHE"}, "006158": {"suoxie": "BSRXHBHHA", "name": "\u535a\u65f6\u8363\u4eab\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGXIANGHUIBAOHUNHEA"}, "006159": {"suoxie": "BSRXHBHHC", "name": "\u535a\u65f6\u8363\u4eab\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGXIANGHUIBAOHUNHEC"}, "006160": {"suoxie": "BDQHHHA", "name": "\u535a\u9053\u542f\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOQIHANGHUNHEA"}, "006161": {"suoxie": "BDQHHHC", "name": "\u535a\u9053\u542f\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOQIHANGHUNHEC"}, "006162": {"suoxie": "CTZGJJSYZQE", "name": "\u8d22\u901a\u8d44\u7ba1\u79ef\u6781\u6536\u76ca\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGZIGUANJIJISHOUYIZHAIQUANE"}, "006163": {"suoxie": "RTZHDKZQFQS", "name": "\u878d\u901a\u589e\u8f89\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGZENGHUIDINGKAIZHAIQUANFAQISHI"}, "006165": {"suoxie": "JXZZ1000ZSZQA", "name": "\u5efa\u4fe1\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG1000ZHISHUZENGQIANGA"}, "006166": {"suoxie": "JXZZ1000ZSZQC", "name": "\u5efa\u4fe1\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG1000ZHISHUZENGQIANGC"}, "006167": {"suoxie": "DBLXSHHHA", "name": "\u5fb7\u90a6\u4e50\u4eab\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGLEXIANGSHENGHUOHUNHEA"}, "006168": {"suoxie": "DBLXSHHHC", "name": "\u5fb7\u90a6\u4e50\u4eab\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGLEXIANGSHENGHUOHUNHEC"}, "006169": {"suoxie": "GYRFCZZQA", "name": "\u5de5\u94f6\u745e\u798f\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIFUCHUNZHAIZHAIQUANA"}, "006170": {"suoxie": "GYRFCZZQC", "name": "\u5de5\u94f6\u745e\u798f\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIFUCHUNZHAIZHAIQUANC"}, "006171": {"suoxie": "PYCLDQKFZQ", "name": "\u9e4f\u626c\u6df3\u5229\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNLIDINGQIKAIFANGZHAIQUAN"}, "006172": {"suoxie": "WJXYCZA", "name": "\u4e07\u5bb6\u946b\u60a6\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINYUECHUNZHAIA"}, "006173": {"suoxie": "WJXYCZC", "name": "\u4e07\u5bb6\u946b\u60a6\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINYUECHUNZHAIC"}, "006174": {"suoxie": "CXWYSGYDKZ", "name": "\u957f\u4fe1\u7a33\u88d5\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENYUSANGEYUEDINGKAIZHAI"}, "006175": {"suoxie": "WJJLHHA", "name": "\u4e07\u5bb6\u805a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAJULIHUNHEA"}, "006176": {"suoxie": "WJJLHHC", "name": "\u4e07\u5bb6\u805a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAJULIHUNHEC"}, "006177": {"suoxie": "ZXBCWDA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u8fbeA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENDAA"}, "006178": {"suoxie": "ZXBCWDC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7a33\u8fbeC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGWENDAC"}, "006179": {"suoxie": "FGPZSHHHA", "name": "\u5bcc\u56fd\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOPINZHISHENGHUOHUNHEA"}, "006180": {"suoxie": "ZJYHCZZQ", "name": "\u4e2d\u52a0\u9890\u5408\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIHECHUNZHAIZHAIQUAN"}, "006181": {"suoxie": "GLBRLHPZA", "name": "\u683c\u6797\u4f2f\u9510\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINBORUILINGHUOPEIZHIA"}, "006182": {"suoxie": "GLBRLHPZC", "name": "\u683c\u6797\u4f2f\u9510\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINBORUILINGHUOPEIZHIC"}, "006183": {"suoxie": "NFZY", "name": "\u5357\u65b9\u6cfd\u5143", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZEYUAN"}, "006184": {"suoxie": "GLHXCZA", "name": "\u683c\u6797\u6cd3\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGXINCHUNZHAIA"}, "006185": {"suoxie": "GLHXCZC", "name": "\u683c\u6797\u6cd3\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGXINCHUNZHAIC"}, "006186": {"suoxie": "YYYYZQA", "name": "\u6c38\u8d62\u76c8\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYINGYIZHAIQUANA"}, "006187": {"suoxie": "YYYYZQC", "name": "\u6c38\u8d62\u76c8\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYINGYIZHAIQUANC"}, "006188": {"suoxie": "HTBXZYDK", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u9890\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNYIDINGKAI"}, "006190": {"suoxie": "QHKYYRHHC", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYURUIHUNHEC"}, "006191": {"suoxie": "HXDTZQA", "name": "\u534e\u590f\u9f0e\u901a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGTONGZHAIQUANA"}, "006192": {"suoxie": "HXDTZQC", "name": "\u534e\u590f\u9f0e\u901a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGTONGZHAIQUANC"}, "006193": {"suoxie": "XYHXZCA", "name": "\u946b\u5143\u6838\u5fc3\u8d44\u4ea7A", "style": "\u80a1\u7968\u578b", "pinyin": "XINYUANHEXINZICHANA"}, "006194": {"suoxie": "XYHXZCC", "name": "\u946b\u5143\u6838\u5fc3\u8d44\u4ea7C", "style": "\u80a1\u7968\u578b", "pinyin": "XINYUANHEXINZICHANC"}, "006195": {"suoxie": "GJLHDYZ", "name": "\u56fd\u91d1\u91cf\u5316\u591a\u56e0\u5b50", "style": "\u80a1\u7968\u578b", "pinyin": "GUOJINLIANGHUADUOYINZI"}, "006196": {"suoxie": "HXZZYQETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u592e\u4f01ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYANGQIETFLIANJIEA"}, "006197": {"suoxie": "HXZZYQETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u592e\u4f01ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYANGQIETFLIANJIEC"}, "006198": {"suoxie": "CSLTSHQDHH", "name": "\u957f\u76db\u9f99\u5934\u53cc\u6838\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGLONGTOUSHUANGHEQUDONGHUNHE"}, "006199": {"suoxie": "CSTJYJJXHH", "name": "\u957f\u76db\u540c\u9526\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGTONGJINYANJIUJINGXUANHUNHE"}, "006201": {"suoxie": "JSCCLHXFHH", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGLIANGHUAXIANFENGHUNHE"}, "006202": {"suoxie": "JYHXZCHH", "name": "\u4ea4\u94f6\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINHEXINZICHANHUNHE"}, "006203": {"suoxie": "XJQHLHYQCZA", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u6cf3\u797a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINJIANGQIANHAILIANHEYONGQICHUNZHAIA"}, "006204": {"suoxie": "XJQHLHYQCZC", "name": "\u65b0\u7586\u524d\u6d77\u8054\u5408\u6cf3\u797a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINJIANGQIANHAILIANHEYONGQICHUNZHAIC"}, "006205": {"suoxie": "HTFHGSYSDK", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6e2f\u6df1\u4f18\u52bf\u5b9a\u5f00", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHUGANGSHENYOUSHIDINGKAI"}, "006206": {"suoxie": "RTZYZQ", "name": "\u878d\u901a\u589e\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGYUEZHAIQUAN"}, "006207": {"suoxie": "TKYTZQA", "name": "\u6cf0\u5eb7\u88d5\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIKANGYUTAIZHAIQUANA"}, "006208": {"suoxie": "TKYTZQC", "name": "\u6cf0\u5eb7\u88d5\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIKANGYUTAIZHAIQUANC"}, "006209": {"suoxie": "ZXBCXLCHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u65b0\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINBAOCHENGXINLANCHOUHUNHE"}, "006210": {"suoxie": "DFZBCZZQA", "name": "\u4e1c\u65b9\u81fb\u5b9d\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENBAOCHUNZHAIZHAIQUANA"}, "006211": {"suoxie": "DFZBCZZQC", "name": "\u4e1c\u65b9\u81fb\u5b9d\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENBAOCHUNZHAIZHAIQUANC"}, "006212": {"suoxie": "DFZXCZZQA", "name": "\u4e1c\u65b9\u81fb\u9009\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENXUANCHUNZHAIZHAIQUANA"}, "006213": {"suoxie": "DFZXCZZQC", "name": "\u4e1c\u65b9\u81fb\u9009\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENXUANCHUNZHAIZHAIQUANC"}, "006214": {"suoxie": "PA500ETFLJA", "name": "\u5e73\u5b89500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGAN500ETFLIANJIEA"}, "006215": {"suoxie": "PA500ETFLJC", "name": "\u5e73\u5b89500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGAN500ETFLIANJIEC"}, "006216": {"suoxie": "QHKYJZCZHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJIAZHICHENGZHANGHUNHEA"}, "006217": {"suoxie": "QHKYJZCZHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANJIAZHICHENGZHANGHUNHEC"}, "006218": {"suoxie": "FGSWYYKJHHA", "name": "\u5bcc\u56fd\u751f\u7269\u533b\u836f\u79d1\u6280\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOSHENGWUYIYAOKEJIHUNHEA"}, "006219": {"suoxie": "HFTDFDKZQ", "name": "\u6d77\u5bcc\u901a\u9f0e\u4e30\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGDINGFENGDINGKAIZHAIQUAN"}, "006220": {"suoxie": "GYSZ50ETFLJA", "name": "\u5de5\u94f6\u4e0a\u8bc150ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHANGZHENG50ETFLIANJIEA"}, "006221": {"suoxie": "GYSZ50ETFLJC", "name": "\u5de5\u94f6\u4e0a\u8bc150ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHANGZHENG50ETFLIANJIEC"}, "006222": {"suoxie": "PAHXZQ", "name": "\u5e73\u5b89\u60e0\u5174\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXINGZHAIQUAN"}, "006223": {"suoxie": "JYCXCZHH", "name": "\u4ea4\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHUANGXINCHENGZHANGHUNHE"}, "006224": {"suoxie": "ZYZZ35NQNFHZQ", "name": "\u4e2d\u94f6\u4e2d\u503a3-5\u5e74\u671f\u519c\u53d1\u884c\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGZHAI35NIANQINONGFAHANGZHAIQUAN"}, "006225": {"suoxie": "RBLHHHA", "name": "\u4eba\u4fdd\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOLIANGHUAHUNHEA"}, "006226": {"suoxie": "RBLHHHC", "name": "\u4eba\u4fdd\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOLIANGHUAHUNHEC"}, "006227": {"suoxie": "HBKJXFHHA", "name": "\u534e\u5b9d\u79d1\u6280\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOKEJIXIANFENGHUNHEA"}, "006228": {"suoxie": "ZOYLCXGPA", "name": "\u4e2d\u6b27\u533b\u7597\u521b\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUYILIAOCHUANGXINGUPIAOA"}, "006229": {"suoxie": "ZOYLCXGPC", "name": "\u4e2d\u6b27\u533b\u7597\u521b\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGOUYILIAOCHUANGXINGUPIAOC"}, "006230": {"suoxie": "PHYJQDHH", "name": "\u9e4f\u534e\u7814\u7a76\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYANJIUQUDONGHUNHE"}, "006231": {"suoxie": "GRRJHHA", "name": "\u56fd\u878d\u878d\u541b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGJUNHUNHEA"}, "006232": {"suoxie": "GRRJHHC", "name": "\u56fd\u878d\u878d\u541b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGJUNHUNHEC"}, "006233": {"suoxie": "WJQCXQSHHA", "name": "\u4e07\u5bb6\u6c7d\u8f66\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAQICHEXINQUSHIHUNHEA"}, "006234": {"suoxie": "WJQCXQSHHC", "name": "\u4e07\u5bb6\u6c7d\u8f66\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAQICHEXINQUSHIHUNHEC"}, "006235": {"suoxie": "DFCZXFZTHH", "name": "\u4e1c\u65b9\u57ce\u9547\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGCHENGZHENXIAOFEIZHUTIHUNHE"}, "006237": {"suoxie": "YYJYZQ", "name": "\u6c38\u8d62\u5609\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGJIAYIZHAIQUAN"}, "006240": {"suoxie": "ZRYLJKHHA", "name": "\u4e2d\u878d\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYILIAOJIANKANGHUNHEA"}, "006241": {"suoxie": "ZRYLJKHHC", "name": "\u4e2d\u878d\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYILIAOJIANKANGHUNHEC"}, "006242": {"suoxie": "BYYRCZZQ", "name": "\u5b9d\u76c8\u76c8\u6da6\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGRUNCHUNZHAIZHAIQUAN"}, "006243": {"suoxie": "ZYSXHBHHA", "name": "\u4e2d\u94f6\u53cc\u606f\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINSHUANGXIHUIBAOHUNHEA"}, "006245": {"suoxie": "JSYL2030HHFOF", "name": "\u5609\u5b9e\u517b\u80012030\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIYANGLAO2030HUNHEFOF"}, "006248": {"suoxie": "HXCYBETFLJA", "name": "\u534e\u590f\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGYEBANETFLIANJIEA"}, "006249": {"suoxie": "HXCYBETFLJC", "name": "\u534e\u590f\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGYEBANETFLIANJIEC"}, "006250": {"suoxie": "STMGDLJXHHA", "name": "\u4e0a\u6295\u6469\u6839\u52a8\u529b\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENDONGLIJINGXUANHUNHEA"}, "006251": {"suoxie": "YHXSGP", "name": "\u94f6\u534e\u5174\u76db\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAXINGSHENGGUPIAO"}, "006252": {"suoxie": "YYXFZTA", "name": "\u6c38\u8d62\u6d88\u8d39\u4e3b\u9898A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGXIAOFEIZHUTIA"}, "006253": {"suoxie": "YYXFZTC", "name": "\u6c38\u8d62\u6d88\u8d39\u4e3b\u9898C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGXIAOFEIZHUTIC"}, "006254": {"suoxie": "CCJYZQ", "name": "\u957f\u57ce\u4e45\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGJIUYUEZHAIQUAN"}, "006257": {"suoxie": "XDAYXJZZGPX", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5148\u8fdb\u667a\u9020\u80a1\u7968\u578b", "style": "\u80a1\u7968\u578b", "pinyin": "XINDAAOYINXIANJINZHIZAOGUPIAOXING"}, "006258": {"suoxie": "HCWLWJTLZQC", "name": "\u534e\u5bb8\u672a\u6765\u7a33\u5065\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUACHENWEILAIWENJIANTIANLIZHAIQUANC"}, "006259": {"suoxie": "HTFHLZZHHA", "name": "\u6c47\u6dfb\u5bcc\u7ea2\u5229\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHONGLIZENGZHANGHUNHEA"}, "006260": {"suoxie": "HTFHLZZHHC", "name": "\u6c47\u6dfb\u5bcc\u7ea2\u5229\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHONGLIZENGZHANGHUNHEC"}, "006263": {"suoxie": "YFDXGXXGZSC", "name": "\u6613\u65b9\u8fbe\u9999\u6e2f\u5c0f\u578b\u80a1\u6307\u6570C", "style": "QDII", "pinyin": "YIFANGDAXIANGGANGXIAOXINGGUZHISHUC"}, "006264": {"suoxie": "PAHXZQ", "name": "\u5e73\u5b89\u60e0\u8f69\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXUANZHAIQUAN"}, "006265": {"suoxie": "HTCXXKJGP", "name": "\u7ea2\u571f\u521b\u65b0\u65b0\u79d1\u6280\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HONGTUCHUANGXINXINKEJIGUPIAO"}, "006266": {"suoxie": "YYZNLXHHA", "name": "\u6c38\u8d62\u667a\u80fd\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGZHINENGLINGXIANHUNHEA"}, "006267": {"suoxie": "NDLHHXA", "name": "\u8bfa\u5fb7\u91cf\u5316\u6838\u5fc3A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODELIANGHUAHEXINA"}, "006268": {"suoxie": "NDLHHXC", "name": "\u8bfa\u5fb7\u91cf\u5316\u6838\u5fc3C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODELIANGHUAHEXINC"}, "006269": {"suoxie": "YYZNLXHHC", "name": "\u6c38\u8d62\u667a\u80fd\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGZHINENGLINGXIANHUNHEC"}, "006270": {"suoxie": "HAHXCZHHA", "name": "\u6c47\u5b89\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINCHENGZHANGHUNHEA"}, "006271": {"suoxie": "HAHXCZHHC", "name": "\u6c47\u5b89\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINCHENGZHANGHUNHEC"}, "006274": {"suoxie": "YXYFYYJK", "name": "\u5706\u4fe1\u6c38\u4e30\u533b\u836f\u5065\u5eb7", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYIYAOJIANKANG"}, "006275": {"suoxie": "YYJYZQA", "name": "\u6c38\u8d62\u805a\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGJUYIZHAIQUANA"}, "006276": {"suoxie": "YYJYZQC", "name": "\u6c38\u8d62\u805a\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGJUYIZHAIQUANC"}, "006277": {"suoxie": "ZJRHA", "name": "\u4e2d\u91d1\u745e\u548cA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINRUIHEA"}, "006278": {"suoxie": "ZJRHC", "name": "\u4e2d\u91d1\u745e\u548cC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINRUIHEC"}, "006279": {"suoxie": "ZJRXA", "name": "\u4e2d\u91d1\u745e\u7965A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINRUIXIANGA"}, "006280": {"suoxie": "ZJRXC", "name": "\u4e2d\u91d1\u745e\u7965C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINRUIXIANGC"}, "006281": {"suoxie": "WJRGZNHHA", "name": "\u4e07\u5bb6\u4eba\u5de5\u667a\u80fd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIARENGONGZHINENGHUNHEA"}, "006282": {"suoxie": "STOZDLQDII", "name": "\u4e0a\u6295\u6b27\u6d32\u52a8\u529b(QDII)", "style": "QDII", "pinyin": "SHANGTOUOUZHOUDONGLIQDII"}, "006283": {"suoxie": "PHMGFDCMYXH", "name": "\u9e4f\u534e\u7f8e\u56fd\u623f\u5730\u4ea7\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "PENGHUAMEIGUOFANGDICHANMEIYUANXIANHUI"}, "006284": {"suoxie": "ZSXYSGYDKZFQS", "name": "\u6d59\u5546\u5174\u6c38\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGXINGYONGSANGEYUEDINGKAIZHAIFAQISHI"}, "006285": {"suoxie": "PHQQZDZQDIIMYXHA", "name": "\u9e4f\u534e\u5168\u7403\u4e2d\u77ed\u503a(QDII)\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "PENGHUAQUANQIUZHONGDUANZHAIQDIIMEIYUANXIANHUIA"}, "006286": {"suoxie": "HTMSCIZGAGLJA", "name": "\u534e\u6cf0MSCI\u4e2d\u56fdA\u80a1\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIMSCIZHONGGUOAGULIANJIEA"}, "006287": {"suoxie": "YYSYZQA", "name": "\u6c38\u8d62\u76db\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGSHENGYIZHAIQUANA"}, "006288": {"suoxie": "YYSYZQC", "name": "\u6c38\u8d62\u76db\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGSHENGYIZHAIQUANC"}, "006289": {"suoxie": "HXYL2040FOF", "name": "\u534e\u590f\u517b\u80012040(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAYANGLAO2040FOF"}, "006290": {"suoxie": "NFYL2035FOFA", "name": "\u5357\u65b9\u517b\u80012035(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYANGLAO2035FOFA"}, "006291": {"suoxie": "NFYL2035FOFC", "name": "\u5357\u65b9\u517b\u80012035(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYANGLAO2035FOFC"}, "006292": {"suoxie": "YFDHCYL2043HHFOF", "name": "\u6613\u65b9\u8fbe\u6c47\u8bda\u517b\u80012043\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAHUICHENGYANGLAO2043HUNHEFOF"}, "006293": {"suoxie": "HTMSCIZGAGLJC", "name": "\u534e\u6cf0MSCI\u4e2d\u56fdA\u80a1\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIMSCIZHONGGUOAGULIANJIEC"}, "006294": {"suoxie": "WJWJYLFOF", "name": "\u4e07\u5bb6\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAWENJIANYANGLAOFOF"}, "006295": {"suoxie": "GYYL2035FOF", "name": "\u5de5\u94f6\u517b\u80012035(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINYANGLAO2035FOF"}, "006296": {"suoxie": "PHYL2035HHFOF", "name": "\u9e4f\u534e\u517b\u80012035\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGHUAYANGLAO2035HUNHEFOF"}, "006297": {"suoxie": "FGXWWJYLFOF", "name": "\u5bcc\u56fd\u946b\u65fa\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINWANGWENJIANYANGLAOFOF"}, "006298": {"suoxie": "GFWJYLFOF", "name": "\u5e7f\u53d1\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAWENJIANYANGLAOFOF"}, "006299": {"suoxie": "HYHXJXHHA", "name": "\u6052\u8d8a\u6838\u5fc3\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEHEXINJINGXUANHUNHEA"}, "006300": {"suoxie": "HBBFGDJZQA", "name": "\u534e\u5b9d\u5b9d\u4e30\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOFENGGAODENGJIZHAIQUANA"}, "006301": {"suoxie": "HBBFGDJZQC", "name": "\u534e\u5b9d\u5b9d\u4e30\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOFENGGAODENGJIZHAIQUANC"}, "006302": {"suoxie": "YHHYLDHH", "name": "\u94f6\u534e\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAHANGYELUNDONGHUNHE"}, "006303": {"suoxie": "ZYYLYNDKFOF", "name": "\u4e2d\u94f6\u517b\u8001\u4e00\u5e74\u5b9a\u5f00(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINYANGLAOYINIANDINGKAIFOF"}, "006304": {"suoxie": "ZJYXCZZQ", "name": "\u4e2d\u52a0\u9890\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIXINCHUNZHAIZHAIQUAN"}, "006305": {"suoxie": "YHZHYL2035HHFOF", "name": "\u94f6\u534e\u5c0a\u548c\u517b\u80012035\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAZUNHEYANGLAO2035HUNHEFOF"}, "006306": {"suoxie": "TDTHPHYLMBSNCYHHFOF", "name": "\u6cf0\u8fbe\u6cf0\u548c\u5e73\u8861\u517b\u8001\u76ee\u6807\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIDATAIHEPINGHENGYANGLAOMUBIAOSANNIANCHIYOUHUNHEFOF"}, "006307": {"suoxie": "JSYL2040HHFOF", "name": "\u5609\u5b9e\u517b\u80012040\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIYANGLAO2040HUNHEFOF"}, "006308": {"suoxie": "HTFQQXFHHRMBA", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u6d88\u8d39\u6df7\u5408\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUITIANFUQUANQIUXIAOFEIHUNHERENMINBIA"}, "006309": {"suoxie": "HTFQQXFHHRMBC", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u6d88\u8d39\u6df7\u5408\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "HUITIANFUQUANQIUXIAOFEIHUNHERENMINBIC"}, "006310": {"suoxie": "HTFQQXFHHMYXH", "name": "\u6c47\u6dfb\u5bcc\u5168\u7403\u6d88\u8d39\u6df7\u5408\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "HUITIANFUQUANQIUXIAOFEIHUNHEMEIYUANXIANHUI"}, "006313": {"suoxie": "HYHXJXHHA", "name": "\u6052\u8d8a\u6838\u5fc3\u7cbe\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEHEXINJINGXUANHUNHEA"}, "006314": {"suoxie": "ZRCLYXHHA", "name": "\u4e2d\u878d\u7b56\u7565\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCELUEYOUXUANHUNHEA"}, "006315": {"suoxie": "ZRCLYXHHC", "name": "\u4e2d\u878d\u7b56\u7565\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCELUEYOUXUANHUNHEC"}, "006316": {"suoxie": "PAHCCZZQ", "name": "\u5e73\u5b89\u60e0\u8bda\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUICHENGCHUNZHAIZHAIQUAN"}, "006319": {"suoxie": "YFDARDZA", "name": "\u6613\u65b9\u8fbe\u5b89\u745e\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANRUIDUANZHAIA"}, "006320": {"suoxie": "YFDARDZC", "name": "\u6613\u65b9\u8fbe\u5b89\u745e\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANRUIDUANZHAIC"}, "006321": {"suoxie": "ZOYJYL2035FOFA", "name": "\u4e2d\u6b27\u9884\u89c1\u517b\u80012035(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUJIANYANGLAO2035FOFA"}, "006322": {"suoxie": "ZOYJYL2035FOFC", "name": "\u4e2d\u6b27\u9884\u89c1\u517b\u80012035(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUJIANYANGLAO2035FOFC"}, "006323": {"suoxie": "HXZYJXHHA", "name": "\u5408\u7166\u667a\u8fdc\u5609\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HEXUZHIYUANJIAXUANHUNHEA"}, "006324": {"suoxie": "HXZYJXHHC", "name": "\u5408\u7166\u667a\u8fdc\u5609\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HEXUZHIYUANJIAXUANHUNHEC"}, "006325": {"suoxie": "ZSTR3GYDKZA", "name": "\u62db\u5546\u6dfb\u83633\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRONG3GEYUEDINGKAIZHAIA"}, "006326": {"suoxie": "ZSTR3GYDKZC", "name": "\u62db\u5546\u6dfb\u83633\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRONG3GEYUEDINGKAIZHAIC"}, "006327": {"suoxie": "YFDZZHW50ETFLJRMBA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6d77\u591650ETF\u8054\u63a5\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDAZHONGZHENGHAIWAI50ETFLIANJIERENMINBIA"}, "006328": {"suoxie": "YFDZZHW50ETFLJRMBC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6d77\u591650ETF\u8054\u63a5\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDAZHONGZHENGHAIWAI50ETFLIANJIERENMINBIC"}, "006329": {"suoxie": "YFDZZHW50ETFLJMYA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6d77\u591650ETF\u8054\u63a5\u7f8e\u5143A", "style": "QDII", "pinyin": "YIFANGDAZHONGZHENGHAIWAI50ETFLIANJIEMEIYUANA"}, "006330": {"suoxie": "YFDZZHW50ETFLJMYC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6d77\u591650ETF\u8054\u63a5\u7f8e\u5143C", "style": "QDII", "pinyin": "YIFANGDAZHONGZHENGHAIWAI50ETFLIANJIEMEIYUANC"}, "006331": {"suoxie": "ZYGYQYZC", "name": "\u4e2d\u94f6\u56fd\u6709\u4f01\u4e1a\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINGUOYOUQIYEZHAIC"}, "006332": {"suoxie": "ZSJHZQA", "name": "\u62db\u5546\u91d1\u9e3f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGJINHONGZHAIQUANA"}, "006333": {"suoxie": "ZSJHZQC", "name": "\u62db\u5546\u91d1\u9e3f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGJINHONGZHAIQUANC"}, "006336": {"suoxie": "HDLHJXHH", "name": "\u6cd3\u5fb7\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDELIANGHUAJINGXUANHUNHE"}, "006337": {"suoxie": "HAAPZQA", "name": "\u534e\u5b89\u5b89\u6d66\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANPUZHAIQUANA"}, "006338": {"suoxie": "HAAPZQC", "name": "\u534e\u5b89\u5b89\u6d66\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANPUZHAIQUANC"}, "006339": {"suoxie": "YHMSCIZGAGLJA", "name": "\u94f6\u534eMSCI\u4e2d\u56fdA\u80a1\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAMSCIZHONGGUOAGULIANJIEA"}, "006340": {"suoxie": "GTMAZYCZC", "name": "\u56fd\u6cf0\u6c11\u5b89\u589e\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIMINANZENGYICHUNZHAIC"}, "006341": {"suoxie": "ZJMSCIZLA", "name": "\u4e2d\u91d1MSCI\u8d28\u91cfA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHILIANGA"}, "006342": {"suoxie": "ZJMSCIZLC", "name": "\u4e2d\u91d1MSCI\u8d28\u91cfC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHILIANGC"}, "006345": {"suoxie": "JSCCJYLNDKHH", "name": "\u666f\u987a\u957f\u57ce\u96c6\u82f1\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJIYINGLIANGNIANDINGKAIHUNHE"}, "006346": {"suoxie": "AXLHYXGPA", "name": "\u5b89\u4fe1\u91cf\u5316\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINLIANGHUAYOUXUANGUPIAOA"}, "006347": {"suoxie": "AXLHYXGPC", "name": "\u5b89\u4fe1\u91cf\u5316\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINLIANGHUAYOUXUANGUPIAOC"}, "006348": {"suoxie": "YHSLHHFQS", "name": "\u94f6\u534e\u76db\u5229\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUASHENGLIHUNHEFAQISHI"}, "006349": {"suoxie": "ZJMSCIZGAGJZZSA", "name": "\u4e2d\u91d1MSCI\u4e2d\u56fdA\u80a1\u4ef7\u503c\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHONGGUOAGUJIAZHIZHISHUA"}, "006350": {"suoxie": "ZJMSCIZGAGJZZSC", "name": "\u4e2d\u91d1MSCI\u4e2d\u56fdA\u80a1\u4ef7\u503c\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHONGGUOAGUJIAZHIZHISHUC"}, "006351": {"suoxie": "ZJMSCIZGAGHLZSA", "name": "\u4e2d\u91d1MSCI\u4e2d\u56fdA\u80a1\u7ea2\u5229\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHONGGUOAGUHONGLIZHISHUA"}, "006352": {"suoxie": "ZJMSCIZGAGHLZSC", "name": "\u4e2d\u91d1MSCI\u4e2d\u56fdA\u80a1\u7ea2\u5229\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHONGGUOAGUHONGLIZHISHUC"}, "006353": {"suoxie": "DFHHXYXDKHHA", "name": "\u4e1c\u65b9\u7ea2\u6838\u5fc3\u4f18\u9009\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGHEXINYOUXUANDINGKAIHUNHEA"}, "006354": {"suoxie": "GTMYJQLHPZHH", "name": "\u56fd\u6cf0\u6c11\u88d5\u8fdb\u53d6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINYUJINQULINGHUOPEIZHIHUNHE"}, "006355": {"suoxie": "HBXGDPC", "name": "\u534e\u5b9d\u9999\u6e2f\u5927\u76d8C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOXIANGGANGDAPANC"}, "006360": {"suoxie": "CTHYZDZZQA", "name": "\u8d22\u901a\u9e3f\u76ca\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGYIZHONGDUANZHAIZHAIQUANA"}, "006361": {"suoxie": "CTHYZDZZQC", "name": "\u8d22\u901a\u9e3f\u76ca\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGYIZHONGDUANZHAIZHAIQUANC"}, "006363": {"suoxie": "JXSZJBM60ETFLJC", "name": "\u5efa\u4fe1\u6df1\u8bc1\u57fa\u672c\u976260ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHENZHENGJIBENMIAN60ETFLIANJIEC"}, "006364": {"suoxie": "ZSFYHHA", "name": "\u62db\u5546\u4e30\u97f5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGFENGYUNHUNHEA"}, "006365": {"suoxie": "ZSFYHHC", "name": "\u62db\u5546\u4e30\u97f5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGFENGYUNHUNHEC"}, "006366": {"suoxie": "XYABYXHH", "name": "\u5174\u4e1a\u5b89\u4fdd\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEANBAOYOUXUANHUNHE"}, "006367": {"suoxie": "JYYXCZZQA", "name": "\u4ea4\u94f6\u88d5\u7965\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUXIANGCHUNZHAIZHAIQUANA"}, "006368": {"suoxie": "JYYXCZZQC", "name": "\u4ea4\u94f6\u88d5\u7965\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUXIANGCHUNZHAIZHAIQUANC"}, "006369": {"suoxie": "HYYFGQZXSJHHA", "name": "\u5f18\u6bc5\u8fdc\u65b9\u56fd\u4f01\u8f6c\u578b\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGUOQIZHUANXINGSHENGJIHUNHEA"}, "006370": {"suoxie": "GFDZHJXHHMY", "name": "\u56fd\u5bcc\u5927\u4e2d\u534e\u7cbe\u9009\u6df7\u5408\u7f8e\u5143", "style": "QDII", "pinyin": "GUOFUDAZHONGHUAJINGXUANHUNHEMEIYUAN"}, "006371": {"suoxie": "CAXYHHA", "name": "\u957f\u5b89\u946b\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINYINGHUNHEA"}, "006372": {"suoxie": "CAXYHHC", "name": "\u957f\u5b89\u946b\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGANXINYINGHUNHEC"}, "006373": {"suoxie": "GFQQKJHLHHRMB", "name": "\u56fd\u5bcc\u5168\u7403\u79d1\u6280\u4e92\u8054\u6df7\u5408\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GUOFUQUANQIUKEJIHULIANHUNHERENMINBI"}, "006374": {"suoxie": "GFQQKJHLHHMYXH", "name": "\u56fd\u5bcc\u5168\u7403\u79d1\u6280\u4e92\u8054\u6df7\u5408\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "GUOFUQUANQIUKEJIHULIANHUNHEMEIYUANXIANHUI"}, "006377": {"suoxie": "GFQSDLHH", "name": "\u5e7f\u53d1\u8d8b\u52bf\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAQUSHIDONGLIHUNHE"}, "006378": {"suoxie": "GFHH6GYDKZ", "name": "\u5e7f\u53d1\u6c47\u5b8f6\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIHONG6GEYUEDINGKAIZHAI"}, "006381": {"suoxie": "HXHSETFLJC", "name": "\u534e\u590f\u6052\u751fETF\u8054\u63a5C", "style": "QDII", "pinyin": "HUAXIAHENGSHENGETFLIANJIEC"}, "006382": {"suoxie": "HXZZ500ETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ETFLIANJIEC"}, "006383": {"suoxie": "ZSTYCZA", "name": "\u62db\u5546\u6dfb\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYINGCHUNZHAIA"}, "006384": {"suoxie": "ZSTYCZC", "name": "\u62db\u5546\u6dfb\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYINGCHUNZHAIC"}, "006385": {"suoxie": "HTBXYJZXA", "name": "\u534e\u6cf0\u4fdd\u5174\u7814\u7a76\u667a\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAOXINGYANJIUZHIXUANA"}, "006386": {"suoxie": "HTBXYJZXC", "name": "\u534e\u6cf0\u4fdd\u5174\u7814\u7a76\u667a\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAOXINGYANJIUZHIXUANC"}, "006387": {"suoxie": "BYATDZZQA", "name": "\u5b9d\u76c8\u5b89\u6cf0\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BAOYINGANTAIDUANZHAIZHAIQUANA"}, "006388": {"suoxie": "BYATDZZQC", "name": "\u5b9d\u76c8\u5b89\u6cf0\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BAOYINGANTAIDUANZHAIZHAIQUANC"}, "006389": {"suoxie": "JYTXZDZA", "name": "\u91d1\u9e70\u6dfb\u7965\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINYINGTIANXIANGZHONGDUANZHAIA"}, "006390": {"suoxie": "JYTXZDZC", "name": "\u91d1\u9e70\u6dfb\u7965\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINYINGTIANXIANGZHONGDUANZHAIC"}, "006392": {"suoxie": "ZXBCCXCZHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINBAOCHENGCHUANGXINCHENGZHANGHUNHE"}, "006393": {"suoxie": "ZSTD3GYDKZA", "name": "\u62db\u5546\u6dfb\u5fb73\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANDE3GEYUEDINGKAIZHAIA"}, "006394": {"suoxie": "ZSTD3GYDKZC", "name": "\u62db\u5546\u6dfb\u5fb73\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANDE3GEYUEDINGKAIZHAIC"}, "006395": {"suoxie": "HXSZ50AHYXZSC", "name": "\u534e\u590f\u4e0a\u8bc150AH\u4f18\u9009\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENG50AHYOUXUANZHISHUC"}, "006396": {"suoxie": "CXSLYXHHE", "name": "\u957f\u4fe1\u53cc\u5229\u4f18\u9009\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINSHUANGLIYOUXUANHUNHEE"}, "006397": {"suoxie": "CXNXCZHHE", "name": "\u957f\u4fe1\u5185\u9700\u6210\u957f\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINNEIXUCHENGZHANGHUNHEE"}, "006398": {"suoxie": "BYXYDQKFHHA", "name": "\u5b9d\u76c8\u7965\u9890\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGYIDINGQIKAIFANGHUNHEA"}, "006399": {"suoxie": "BYXYDQKFHHC", "name": "\u5b9d\u76c8\u7965\u9890\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGYIDINGQIKAIFANGHUNHEC"}, "006400": {"suoxie": "KSHHYLDHH", "name": "\u51ef\u77f3\u6cd3\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHIHONGHANGYELUNDONGHUNHE"}, "006401": {"suoxie": "XFLHYXHHA", "name": "\u5148\u950b\u91cf\u5316\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGLIANGHUAYOUXUANHUNHEA"}, "006402": {"suoxie": "XFLHYXHHC", "name": "\u5148\u950b\u91cf\u5316\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANFENGLIANGHUAYOUXUANHUNHEC"}, "006403": {"suoxie": "YHRJZQC", "name": "\u94f6\u6cb3\u777f\u5609\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHERUIJIAZHAIQUANC"}, "006404": {"suoxie": "PYASSRDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u878d\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGRONGDINGKAIZHAIQUAN"}, "006405": {"suoxie": "HFHSCZZQA", "name": "\u534e\u5bcc\u6052\u76db\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGSHENGCHUNZHAIZHAIQUANA"}, "006406": {"suoxie": "HFHSCZZQC", "name": "\u534e\u5bcc\u6052\u76db\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGSHENGCHUNZHAIZHAIQUANC"}, "006408": {"suoxie": "HTFXFSJHH", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUXIAOFEISHENGJIHUNHE"}, "006409": {"suoxie": "FGZZGKHZA", "name": "\u5bcc\u56fd\u4e2d\u503a\u56fd\u5f00\u884c\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAIGUOKAIHANGZHAIA"}, "006410": {"suoxie": "FGZZGKHZC", "name": "\u5bcc\u56fd\u4e2d\u503a\u56fd\u5f00\u884c\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAIGUOKAIHANGZHAIC"}, "006411": {"suoxie": "ZJYZCZZQ", "name": "\u4e2d\u52a0\u9890\u667a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIZHICHUNZHAIZHAIQUAN"}, "006412": {"suoxie": "PAHJDKZ", "name": "\u5e73\u5b89\u5408\u9526\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEJINDINGKAIZHAI"}, "006415": {"suoxie": "YHZDZCJRZDKZ", "name": "\u94f6\u534e\u4e2d\u77ed\u653f\u7b56\u91d1\u878d\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAZHONGDUANZHENGCEJINRONGZHAIDINGKAIZHAI"}, "006416": {"suoxie": "FZFBFLZQA", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGFUBANGFENGLIZHAIQUANA"}, "006417": {"suoxie": "FZFBFLZQC", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGFUBANGFENGLIZHAIQUANC"}, "006419": {"suoxie": "RBYSCYHHA", "name": "\u4eba\u4fdd\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOYOUSHICHANYEHUNHEA"}, "006420": {"suoxie": "RBYSCYHHC", "name": "\u4eba\u4fdd\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOYOUSHICHANYEHUNHEC"}, "006421": {"suoxie": "ZYHXZQ", "name": "\u4e2d\u94f6\u5f18\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHONGXIANGZHAIQUAN"}, "006422": {"suoxie": "JHPWCZA", "name": "\u5609\u5408\u78d0\u7a33\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANWENCHUNZHAIA"}, "006423": {"suoxie": "JHPWCZC", "name": "\u5609\u5408\u78d0\u7a33\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANWENCHUNZHAIC"}, "006424": {"suoxie": "JHJCHHA", "name": "\u5609\u5408\u9526\u7a0b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINCHENGHUNHEA"}, "006425": {"suoxie": "JHJCHHC", "name": "\u5609\u5408\u9526\u7a0b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINCHENGHUNHEC"}, "006427": {"suoxie": "ZSTYCZA", "name": "\u62db\u5546\u6dfb\u60a6\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUECHUNZHAIA"}, "006428": {"suoxie": "ZSTYCZC", "name": "\u62db\u5546\u6dfb\u60a6\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUECHUNZHAIC"}, "006429": {"suoxie": "NAHXHH", "name": "\u8bfa\u5b89\u6052\u946b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANHENGXINHUNHE"}, "006430": {"suoxie": "KSLLTJJYNCYHH", "name": "\u51ef\u77f3\u6f9c\u9f99\u5934\u7ecf\u6d4e\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHILANLONGTOUJINGJIYINIANCHIYOUHUNHE"}, "006431": {"suoxie": "HADLCZA", "name": "\u6c47\u5b89\u9f0e\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANDINGLICHUNZHAIA"}, "006432": {"suoxie": "HADLCZC", "name": "\u6c47\u5b89\u9f0e\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANDINGLICHUNZHAIC"}, "006433": {"suoxie": "PAXLHHC", "name": "\u5e73\u5b89\u946b\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINLIHUNHEC"}, "006434": {"suoxie": "PH3GYZDZA", "name": "\u9e4f\u534e3\u4e2a\u6708\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUA3GEYUEZHONGDUANZHAIA"}, "006435": {"suoxie": "JSCCCXCZHH", "name": "\u666f\u987a\u957f\u57ce\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGCHUANGXINCHENGZHANGHUNHE"}, "006436": {"suoxie": "PYASZDZA", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGZHONGDUANZHAIA"}, "006437": {"suoxie": "PYASZDZC", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGZHONGDUANZHAIC"}, "006438": {"suoxie": "BSYTETFLJA", "name": "\u535a\u65f6\u592e\u8c03ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGDIAOETFLIANJIEA"}, "006439": {"suoxie": "BSYTETFLJC", "name": "\u535a\u65f6\u592e\u8c03ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGDIAOETFLIANJIEC"}, "006440": {"suoxie": "ZXJTZZ500ZQA", "name": "\u4e2d\u4fe1\u5efa\u6295\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGXINJIANTOUZHONGZHENG500ZENGQIANGA"}, "006441": {"suoxie": "ZXJTZZ500ZQC", "name": "\u4e2d\u4fe1\u5efa\u6295\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGXINJIANTOUZHONGZHENG500ZENGQIANGC"}, "006442": {"suoxie": "DXPPJXHHC", "name": "\u4e1c\u5174\u54c1\u724c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGPINPAIJINGXUANHUNHEC"}, "006443": {"suoxie": "YYYYZQA", "name": "\u6c38\u8d62\u88d5\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYUYIZHAIQUANA"}, "006444": {"suoxie": "YYYYZQC", "name": "\u6c38\u8d62\u88d5\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYUYIZHAIQUANC"}, "006445": {"suoxie": "HXQQJXARMB", "name": "\u534e\u590f\u5168\u7403\u805a\u4eabA\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "HUAXIAQUANQIUJUXIANGARENMINBI"}, "006446": {"suoxie": "HXQQJXAMYXH", "name": "\u534e\u590f\u5168\u7403\u805a\u4eabA\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "HUAXIAQUANQIUJUXIANGAMEIYUANXIANHUI"}, "006447": {"suoxie": "HXQQJXAMYXC", "name": "\u534e\u590f\u5168\u7403\u805a\u4eabA\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "HUAXIAQUANQIUJUXIANGAMEIYUANXIANCHAO"}, "006448": {"suoxie": "HXQQJXCRMB", "name": "\u534e\u590f\u5168\u7403\u805a\u4eabC\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "HUAXIAQUANQIUJUXIANGCRENMINBI"}, "006449": {"suoxie": "ZSHJLHJXHH", "name": "\u6d59\u5546\u6c47\u91d1\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGHUIJINLIANGHUAJINGXUANHUNHE"}, "006450": {"suoxie": "JSZYZQ", "name": "\u5609\u5b9e\u81f4\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIYINGZHAIQUAN"}, "006451": {"suoxie": "HFHDJQGKZA", "name": "\u534e\u5bcc\u6052\u5b9a\u4e45\u671f\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGDINGJIUQIGUOKAIZHAIA"}, "006452": {"suoxie": "HFHDJQGKZC", "name": "\u534e\u5bcc\u6052\u5b9a\u4e45\u671f\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGDINGJIUQIGUOKAIZHAIC"}, "006453": {"suoxie": "ZJRLCZZQA", "name": "\u4e2d\u52a0\u745e\u5229\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUILICHUNZHAIZHAIQUANA"}, "006454": {"suoxie": "ZJRLCZZQC", "name": "\u4e2d\u52a0\u745e\u5229\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUILICHUNZHAIZHAIQUANC"}, "006456": {"suoxie": "PH3GYZDZC", "name": "\u9e4f\u534e3\u4e2a\u6708\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUA3GEYUEZHONGDUANZHAIC"}, "006457": {"suoxie": "PAGZYSHHA", "name": "\u5e73\u5b89\u4f30\u503c\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANGUZHIYOUSHIHUNHEA"}, "006458": {"suoxie": "PAGZYSHHC", "name": "\u5e73\u5b89\u4f30\u503c\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANGUZHIYOUSHIHUNHEC"}, "006459": {"suoxie": "RBXYZQA", "name": "\u4eba\u4fdd\u946b\u88d5\u589e\u5f3aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RENBAOXINYUZENGQIANGA"}, "006460": {"suoxie": "RBXYZQC", "name": "\u4eba\u4fdd\u946b\u88d5\u589e\u5f3aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RENBAOXINYUZENGQIANGC"}, "006461": {"suoxie": "RBFZYNDQKFZQ", "name": "\u4eba\u4fdd\u798f\u6cfd\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOFUZEYINIANDINGQIKAIFANGZHAIQUAN"}, "006462": {"suoxie": "XDAYXQDDQKFHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u8d77\u70b9\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINXINQIDIANDINGQIKAIFANGHUNHEC"}, "006464": {"suoxie": "PYASPYCZA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUYICHUNZHAIA"}, "006465": {"suoxie": "PYASPYCZC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUYICHUNZHAIC"}, "006466": {"suoxie": "PYASSZZQZQA", "name": "\u6d66\u94f6\u5b89\u76db\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PUYINANSHENGSHUANGZHAIZENGQIANGZHAIQUANA"}, "006467": {"suoxie": "PYASSZZQZQC", "name": "\u6d66\u94f6\u5b89\u76db\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PUYINANSHENGSHUANGZHAIZENGQIANGZHAIQUANC"}, "006468": {"suoxie": "JSWLCZZQ", "name": "\u5609\u5b9e\u7a33\u8054\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENLIANCHUNZHAIZHAIQUAN"}, "006470": {"suoxie": "GYMBSYYNDKA", "name": "\u5de5\u94f6\u76ee\u6807\u6536\u76ca\u4e00\u5e74\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINMUBIAOSHOUYIYINIANDINGKAIA"}, "006471": {"suoxie": "HRYDRXZQC", "name": "\u534e\u6da6\u5143\u5927\u6da6\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNXINZHAIQUANC"}, "006473": {"suoxie": "ZSZZ15NJCKHA", "name": "\u62db\u5546\u4e2d\u503a1-5\u5e74\u8fdb\u51fa\u53e3\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHONGZHAI15NIANJINCHUKOUHANGA"}, "006474": {"suoxie": "ZSZZ15NJCKHC", "name": "\u62db\u5546\u4e2d\u503a1-5\u5e74\u8fdb\u51fa\u53e3\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHONGZHAI15NIANJINCHUKOUHANGC"}, "006475": {"suoxie": "GTJRCZZQ", "name": "\u56fd\u6cf0\u5609\u777f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJIARUICHUNZHAIZHAIQUAN"}, "006476": {"suoxie": "NFYYC", "name": "\u5357\u65b9\u539f\u6cb9C", "style": "QDII", "pinyin": "NANFANGYUANYOUC"}, "006477": {"suoxie": "ZYHGSJXHH", "name": "\u4e2d\u90ae\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUHUGANGSHENJINGXUANHUNHE"}, "006478": {"suoxie": "CSDYZCLYX", "name": "\u957f\u76db\u591a\u56e0\u5b50\u7b56\u7565\u4f18\u9009", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGSHENGDUOYINZICELUEYOUXUAN"}, "006479": {"suoxie": "GFNSDK100ZSC", "name": "\u5e7f\u53d1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570C", "style": "QDII", "pinyin": "GUANGFANASIDAKE100ZHISHUC"}, "006480": {"suoxie": "GFNSDK100MYXHC", "name": "\u5e7f\u53d1\u7eb3\u65af\u8fbe\u514b100\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "GUANGFANASIDAKE100MEIYUANXIANHUIC"}, "006481": {"suoxie": "HFTSQSDRZQA", "name": "\u6d77\u5bcc\u901a\u4e0a\u6e05\u6240\u77ed\u878d\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAIFUTONGSHANGQINGSUODUANRONGZHAIQUANA"}, "006482": {"suoxie": "GFKZZZQA", "name": "\u5e7f\u53d1\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUANGFAKEZHUANZHAIZHAIQUANA"}, "006483": {"suoxie": "GFKZZZQC", "name": "\u5e7f\u53d1\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUANGFAKEZHUANZHAIZHAIQUANC"}, "006484": {"suoxie": "GFZZ13NGKZA", "name": "\u5e7f\u53d1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI13NIANGUOKAIZHAIA"}, "006485": {"suoxie": "GFZZ13NGKZC", "name": "\u5e7f\u53d1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI13NIANGUOKAIZHAIC"}, "006486": {"suoxie": "GFZZ1000ZSA", "name": "\u5e7f\u53d1\u4e2d\u8bc11000\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG1000ZHISHUA"}, "006487": {"suoxie": "GFZZ1000ZSC", "name": "\u5e7f\u53d1\u4e2d\u8bc11000\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG1000ZHISHUC"}, "006488": {"suoxie": "FRFK13NGKZCZA", "name": "\u5bcc\u8363\u5bcc\u5f001-3\u5e74\u56fd\u5f00\u503a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUKAI13NIANGUOKAIZHAICHUNZHAIA"}, "006489": {"suoxie": "ZSTYCZA", "name": "\u62db\u5546\u6dfb\u88d5\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUCHUNZHAIA"}, "006490": {"suoxie": "ZSTYCZC", "name": "\u62db\u5546\u6dfb\u88d5\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUCHUNZHAIC"}, "006491": {"suoxie": "NF13NGKZA", "name": "\u5357\u65b91-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG13NIANGUOKAIZHAIA"}, "006492": {"suoxie": "NF13NGKZC", "name": "\u5357\u65b91-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG13NIANGUOKAIZHAIC"}, "006493": {"suoxie": "NF35NNFZA", "name": "\u5357\u65b93-5\u5e74\u519c\u53d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG35NIANNONGFAZHAIA"}, "006494": {"suoxie": "NF35NNFZC", "name": "\u5357\u65b93-5\u5e74\u519c\u53d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG35NIANNONGFAZHAIC"}, "006495": {"suoxie": "GLAZFYNDKZ", "name": "\u56fd\u8054\u5b89\u589e\u5bcc\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGFUYINIANDINGKAIZHAI"}, "006496": {"suoxie": "YHAYDZZQA", "name": "\u94f6\u534e\u5b89\u76c8\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANYINGDUANZHAIZHAIQUANA"}, "006497": {"suoxie": "YHAYDZZQC", "name": "\u94f6\u534e\u5b89\u76c8\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANYINGDUANZHAIZHAIQUANC"}, "006498": {"suoxie": "ZKWTWSCZA", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u76db\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGKEWOTUWOSHENGCHUNZHAIA"}, "006499": {"suoxie": "ZKWTWSCZC", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u76db\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGKEWOTUWOSHENGCHUNZHAIC"}, "006500": {"suoxie": "JXRLZQZQA", "name": "\u5efa\u4fe1\u6da6\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINRUNLIZENGQIANGZHAIQUANA"}, "006501": {"suoxie": "JXRLZQZQC", "name": "\u5efa\u4fe1\u6da6\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINRUNLIZENGQIANGZHAIQUANC"}, "006502": {"suoxie": "CTJCDLCYGPA", "name": "\u8d22\u901a\u96c6\u6210\u7535\u8def\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CAITONGJICHENGDIANLUCHANYEGUPIAOA"}, "006503": {"suoxie": "CTJCDLCYGPC", "name": "\u8d22\u901a\u96c6\u6210\u7535\u8def\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CAITONGJICHENGDIANLUCHANYEGUPIAOC"}, "006504": {"suoxie": "GFHCDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u627f\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUICHENGDINGQIKAIFANGZHAIQUAN"}, "006505": {"suoxie": "YYXYZQA", "name": "\u6c38\u8d62\u7965\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGXIANGYIZHAIQUANA"}, "006506": {"suoxie": "YYXYZQC", "name": "\u6c38\u8d62\u7965\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGXIANGYIZHAIQUANC"}, "006507": {"suoxie": "QHKYYZFOF", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u6cfd(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYUZEFOF"}, "006508": {"suoxie": "GLAZYYNDKZ", "name": "\u56fd\u8054\u5b89\u589e\u88d5\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGYUYINIANDINGKAIZHAI"}, "006509": {"suoxie": "GLAZYCZA", "name": "\u56fd\u8054\u5b89\u589e\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGYINGCHUNZHAIA"}, "006510": {"suoxie": "GLAZYCZC", "name": "\u56fd\u8054\u5b89\u589e\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGYINGCHUNZHAIC"}, "006511": {"suoxie": "BDZYHHA", "name": "\u535a\u9053\u5353\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOZHUOYUANHUNHEA"}, "006512": {"suoxie": "BDZYHHC", "name": "\u535a\u9053\u5353\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOZHUOYUANHUNHEC"}, "006513": {"suoxie": "PYCXZQA", "name": "\u9e4f\u626c\u6df3\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNXIANGZHAIQUANA"}, "006514": {"suoxie": "PYCXZQC", "name": "\u9e4f\u626c\u6df3\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNXIANGZHAIQUANC"}, "006515": {"suoxie": "ZSHJDZE", "name": "\u6d59\u5546\u6c47\u91d1\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINDUANZHAIE"}, "006516": {"suoxie": "ZSHJDZA", "name": "\u6d59\u5546\u6c47\u91d1\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINDUANZHAIA"}, "006517": {"suoxie": "NFJYDZA", "name": "\u5357\u65b9\u5409\u5143\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGJIYUANDUANZHAIA"}, "006518": {"suoxie": "NFJYDZC", "name": "\u5357\u65b9\u5409\u5143\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGJIYUANDUANZHAIC"}, "006519": {"suoxie": "HADZZQA", "name": "\u6c47\u5b89\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANDUANZHAIZHAIQUANA"}, "006520": {"suoxie": "HADZZQC", "name": "\u6c47\u5b89\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANDUANZHAIZHAIQUANC"}, "006521": {"suoxie": "HADZZQE", "name": "\u6c47\u5b89\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANDUANZHAIZHAIQUANE"}, "006522": {"suoxie": "CTXXLCHHA", "name": "\u8d22\u901a\u65b0\u5174\u84dd\u7b79\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGXINXINGLANCHOUHUNHEA"}, "006523": {"suoxie": "CTXXLCHHC", "name": "\u8d22\u901a\u65b0\u5174\u84dd\u7b79\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGXINXINGLANCHOUHUNHEC"}, "006524": {"suoxie": "QHKYMSCIZGAGZSA", "name": "\u524d\u6d77\u5f00\u6e90MSCI\u4e2d\u56fdA\u80a1\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANMSCIZHONGGUOAGUZHISHUA"}, "006525": {"suoxie": "QHKYMSCIZGAGZSC", "name": "\u524d\u6d77\u5f00\u6e90MSCI\u4e2d\u56fdA\u80a1\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANMSCIZHONGGUOAGUZHISHUC"}, "006526": {"suoxie": "PHYXHBHHA", "name": "\u9e4f\u534e\u4f18\u9009\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAYOUXUANHUIBAOHUNHEA"}, "006527": {"suoxie": "FGYZFZHHA", "name": "\u5bcc\u56fd\u4f18\u8d28\u53d1\u5c55\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYOUZHIFAZHANHUNHEA"}, "006528": {"suoxie": "FGYZFZHHC", "name": "\u5bcc\u56fd\u4f18\u8d28\u53d1\u5c55\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYOUZHIFAZHANHUNHEC"}, "006529": {"suoxie": "ZOJXLNCYQHHA", "name": "\u4e2d\u6b27\u5320\u5fc3\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIANGXINLIANGNIANCHIYOUQIHUNHEA"}, "006530": {"suoxie": "ZOJXLNCYQHHC", "name": "\u4e2d\u6b27\u5320\u5fc3\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIANGXINLIANGNIANCHIYOUQIHUNHEC"}, "006531": {"suoxie": "HTBRLHQDHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAQUDONGHUNHEC"}, "006532": {"suoxie": "HTBRLHAEFC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u963f\u5c14\u6cd5C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUILIANGHUAAERFAC"}, "006533": {"suoxie": "YFDKRHH", "name": "\u6613\u65b9\u8fbe\u79d1\u878d\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKERONGHUNHE"}, "006534": {"suoxie": "NYHLYSDQKFHH", "name": "\u519c\u94f6\u6c47\u7406\u6c38\u76db\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NONGYINHUILIYONGSHENGDINGQIKAIFANGHUNHE"}, "006535": {"suoxie": "HSQHHJYLA", "name": "\u6052\u751f\u524d\u6d77\u6052\u9526\u88d5\u5229A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HENGSHENGQIANHAIHENGJINYULIA"}, "006536": {"suoxie": "HSQHHJYLC", "name": "\u6052\u751f\u524d\u6d77\u6052\u9526\u88d5\u5229C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HENGSHENGQIANHAIHENGJINYULIC"}, "006537": {"suoxie": "HSQHGGTJXHH", "name": "\u6052\u751f\u524d\u6d77\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGSHENGQIANHAIGANGGUTONGJINGXUANHUNHE"}, "006538": {"suoxie": "DHHXJZ", "name": "\u4e1c\u6d77\u6838\u5fc3\u4ef7\u503c", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGHAIHEXINJIAZHI"}, "006539": {"suoxie": "NFYXJZHHC", "name": "\u5357\u65b9\u4f18\u9009\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYOUXUANJIAZHIHUNHEC"}, "006540": {"suoxie": "NFJYCZHHC", "name": "\u5357\u65b9\u7ee9\u4f18\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIYOUCHENGZHANGHUNHEC"}, "006541": {"suoxie": "NFCFJXHHC", "name": "\u5357\u65b9\u6210\u4efd\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHENGFENJINGXUANHUNHEC"}, "006542": {"suoxie": "CTHLZDZZQA", "name": "\u8d22\u901a\u9e3f\u5229\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGLIZHONGDUANZHAIZHAIQUANA"}, "006543": {"suoxie": "CTHLZDZZQC", "name": "\u8d22\u901a\u9e3f\u5229\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGLIZHONGDUANZHAIZHAIQUANC"}, "006544": {"suoxie": "PAHJCZZQ", "name": "\u5e73\u5b89\u60e0\u805a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIJUCHUNZHAIZHAIQUAN"}, "006545": {"suoxie": "XYZDZA", "name": "\u5174\u94f6\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINZHONGDUANZHAIA"}, "006546": {"suoxie": "XYZDZC", "name": "\u5174\u94f6\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINZHONGDUANZHAIC"}, "006547": {"suoxie": "HTHTSHHHA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5f18\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGHONGHUNHEA"}, "006548": {"suoxie": "HTHTSHHHC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5f18\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTAHONGTUSHENGHONGHUNHEC"}, "006549": {"suoxie": "GJHYCZA", "name": "\u56fd\u91d1\u60e0\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIYINGCHUNZHAIA"}, "006551": {"suoxie": "ZGJZLHHH", "name": "\u4e2d\u5e9a\u4ef7\u503c\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGGENGJIAZHILINGHANGHUNHE"}, "006552": {"suoxie": "GFHX3GYDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u51743\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIXING3GEYUEDINGQIKAIFANGZHAIQUAN"}, "006553": {"suoxie": "GTRYHZZDZC", "name": "\u56fd\u6295\u745e\u94f6\u6052\u6cfd\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTOURUIYINHENGZEZHONGDUANZHAIC"}, "006554": {"suoxie": "HADYZQC", "name": "\u534e\u5b89\u9f0e\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANDINGYIZHAIQUANC"}, "006555": {"suoxie": "PYASQQZNKJQDIIA", "name": "\u6d66\u94f6\u5b89\u76db\u5168\u7403\u667a\u80fd\u79d1\u6280(QDII)A", "style": "QDII", "pinyin": "PUYINANSHENGQUANQIUZHINENGKEJIQDIIA"}, "006556": {"suoxie": "HFTYJJXHHC", "name": "\u6d77\u5bcc\u901a\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGYANJIUJINGXUANHUNHEC"}, "006557": {"suoxie": "HFTYJJXHHA", "name": "\u6d77\u5bcc\u901a\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGYANJIUJINGXUANHUNHEA"}, "006558": {"suoxie": "YYTYZQA", "name": "\u6c38\u8d62\u901a\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTONGYIZHAIQUANA"}, "006559": {"suoxie": "YYTYZQC", "name": "\u6c38\u8d62\u901a\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTONGYIZHAIQUANC"}, "006560": {"suoxie": "HXSCGGETFLJA", "name": "\u534e\u590f\u56db\u5ddd\u56fd\u6539ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASICHUANGUOGAIETFLIANJIEA"}, "006561": {"suoxie": "HXSCGGETFLJC", "name": "\u534e\u590f\u56db\u5ddd\u56fd\u6539ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASICHUANGUOGAIETFLIANJIEC"}, "006562": {"suoxie": "ZODZZQC", "name": "\u4e2d\u6b27\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGOUDUANZHAIZHAIQUANC"}, "006564": {"suoxie": "YXYFJXHBHH", "name": "\u5706\u4fe1\u6c38\u4e30\u7cbe\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGJINGXUANHUIBAOHUNHE"}, "006565": {"suoxie": "GDZTDKZ", "name": "\u5149\u5927\u5c0a\u6cf0\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNTAIDINGKAIZHAI"}, "006567": {"suoxie": "ZTXYLHPZHHA", "name": "\u4e2d\u6cf0\u661f\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGTAIXINGYUANLINGHUOPEIZHIHUNHEA"}, "006568": {"suoxie": "GLAHYLXHH", "name": "\u56fd\u8054\u5b89\u884c\u4e1a\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHANGYELINGXIANHUNHE"}, "006569": {"suoxie": "GLAZZYY100C", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u533b\u836f100C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGYIYAO100C"}, "006570": {"suoxie": "ZJJYA", "name": "\u4e2d\u91d1\u91d1\u5143A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINYUANA"}, "006571": {"suoxie": "ZJJYC", "name": "\u4e2d\u91d1\u91d1\u5143C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINYUANC"}, "006572": {"suoxie": "BYYTCZZQC", "name": "\u5b9d\u76c8\u76c8\u6cf0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGTAICHUNZHAIZHAIQUANC"}, "006573": {"suoxie": "RBHYLDHHA", "name": "\u4eba\u4fdd\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOHANGYELUNDONGHUNHEA"}, "006574": {"suoxie": "RBHYLDHHC", "name": "\u4eba\u4fdd\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOHANGYELUNDONGHUNHEC"}, "006575": {"suoxie": "HAYL2030SNFOF", "name": "\u534e\u5b89\u517b\u80012030\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANYANGLAO2030SANNIANFOF"}, "006576": {"suoxie": "YYCYZQA", "name": "\u6c38\u8d62\u8bda\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHENGYIZHAIQUANA"}, "006577": {"suoxie": "YYCYZQC", "name": "\u6c38\u8d62\u8bda\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHENGYIZHAIQUANC"}, "006578": {"suoxie": "TKZZGGTFYZSA", "name": "\u6cf0\u5eb7\u4e2d\u8bc1\u6e2f\u80a1\u901a\u975e\u94f6\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGZHONGZHENGGANGGUTONGFEIYINZHISHUA"}, "006579": {"suoxie": "TKZZGGTFYZSC", "name": "\u6cf0\u5eb7\u4e2d\u8bc1\u6e2f\u80a1\u901a\u975e\u94f6\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGZHONGZHENGGANGGUTONGFEIYINZHISHUC"}, "006580": {"suoxie": "XQATYLSNFOF", "name": "\u5174\u5168\u5b89\u6cf0\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGQUANANTAIYANGLAOSANNIANFOF"}, "006581": {"suoxie": "JXYXWJYLFOF", "name": "\u5efa\u4fe1\u4f18\u4eab\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINYOUXIANGWENJIANYANGLAOFOF"}, "006582": {"suoxie": "BSFY3GYDKZ", "name": "\u535a\u65f6\u5bcc\u6c383\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYONG3GEYUEDINGKAIZHAI"}, "006583": {"suoxie": "ZXBCJTZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGTAIZHAIQUANA"}, "006584": {"suoxie": "ZXBCJTZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGTAIZHAIQUANC"}, "006585": {"suoxie": "NFBYZQC", "name": "\u5357\u65b9\u5b9d\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGBAOYUANZHAIQUANC"}, "006586": {"suoxie": "NFAYHHC", "name": "\u5357\u65b9\u5b89\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANYUHUNHEC"}, "006587": {"suoxie": "NFYXFHHHC", "name": "\u5357\u65b9\u4f18\u4eab\u5206\u7ea2\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYOUXIANGFENHONGHUNHEC"}, "006588": {"suoxie": "ZJJLCZDKA", "name": "\u4e2d\u52a0\u805a\u5229\u7eaf\u503a\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJULICHUNZHAIDINGKAIA"}, "006589": {"suoxie": "ZJJLCZDKC", "name": "\u4e2d\u52a0\u805a\u5229\u7eaf\u503a\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJULICHUNZHAIDINGKAIC"}, "006590": {"suoxie": "NFXYXLHPZHHC", "name": "\u5357\u65b9\u65b0\u4f18\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGXINYOUXIANGLINGHUOPEIZHIHUNHEC"}, "006591": {"suoxie": "GFJMZDZA", "name": "\u5e7f\u53d1\u666f\u660e\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGMINGZHONGDUANZHAIA"}, "006592": {"suoxie": "GFJMZDZC", "name": "\u5e7f\u53d1\u666f\u660e\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGMINGZHONGDUANZHAIC"}, "006593": {"suoxie": "BDZZ500ZQA", "name": "\u535a\u9053\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BODAOZHONGZHENG500ZENGQIANGA"}, "006594": {"suoxie": "BDZZ500ZQC", "name": "\u535a\u9053\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BODAOZHONGZHENG500ZENGQIANGC"}, "006595": {"suoxie": "GFGGTYZZZHHA", "name": "\u5e7f\u53d1\u6e2f\u80a1\u901a\u4f18\u8d28\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGANGGUTONGYOUZHIZENGZHANGHUNHEA"}, "006596": {"suoxie": "GTJHCZZQ", "name": "\u56fd\u6cf0\u805a\u79be\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUHECHUNZHAIZHAIQUAN"}, "006597": {"suoxie": "GTLXZDZZQA", "name": "\u56fd\u6cf0\u5229\u4eab\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIXIANGZHONGDUANZHAIZHAIQUANA"}, "006598": {"suoxie": "GTLXZDZZQC", "name": "\u56fd\u6cf0\u5229\u4eab\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIXIANGZHONGDUANZHAIZHAIQUANC"}, "006599": {"suoxie": "GSABAFCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u4e30\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANFENGCHUNZHAIZHAIQUAN"}, "006600": {"suoxie": "RBHS300ZS", "name": "\u4eba\u4fdd\u6caa\u6df1300\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RENBAOHUSHEN300ZHISHU"}, "006601": {"suoxie": "GRRTLHPZHHA", "name": "\u56fd\u878d\u878d\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGTAILINGHUOPEIZHIHUNHEA"}, "006602": {"suoxie": "GRRTLHPZHHC", "name": "\u56fd\u878d\u878d\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGTAILINGHUOPEIZHIHUNHEC"}, "006603": {"suoxie": "JSHRJXGP", "name": "\u5609\u5b9e\u4e92\u878d\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHURONGJINGXUANGUPIAO"}, "006604": {"suoxie": "JSXFJXGPA", "name": "\u5609\u5b9e\u6d88\u8d39\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXIAOFEIJINGXUANGUPIAOA"}, "006605": {"suoxie": "JSXFJXGPC", "name": "\u5609\u5b9e\u6d88\u8d39\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIXIAOFEIJINGXUANGUPIAOC"}, "006606": {"suoxie": "HDYFZDZZQA", "name": "\u6cd3\u5fb7\u88d5\u4e30\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HONGDEYUFENGZHONGDUANZHAIZHAIQUANA"}, "006607": {"suoxie": "HDYFZDZZQC", "name": "\u6cd3\u5fb7\u88d5\u4e30\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HONGDEYUFENGZHONGDUANZHAIZHAIQUANC"}, "006608": {"suoxie": "HDYJYXHH", "name": "\u6cd3\u5fb7\u7814\u7a76\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEYANJIUYOUXUANHUNHE"}, "006609": {"suoxie": "SWLXATRLZDZA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u745e\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHENWANLINGXINANTAIRUILIZHONGDUANZHAIA"}, "006610": {"suoxie": "YHYJHHFQS", "name": "\u94f6\u534e\u8fdc\u89c1\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAYUANJIANHUNHEFAQISHI"}, "006611": {"suoxie": "RBZZ500ZS", "name": "\u4eba\u4fdd\u4e2d\u8bc1500\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RENBAOZHONGZHENG500ZHISHU"}, "006612": {"suoxie": "YHXYJXYNDKZ", "name": "\u94f6\u534e\u4fe1\u7528\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJINGXUANYINIANDINGKAIZHAI"}, "006614": {"suoxie": "JSGGTXJJZSC", "name": "\u5609\u5b9e\u6e2f\u80a1\u901a\u65b0\u7ecf\u6d4e\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIGANGGUTONGXINJINGJIZHISHUC"}, "006615": {"suoxie": "GYZLXXCYHHA", "name": "\u5de5\u94f6\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHANLUEXINXINGCHANYEHUNHEA"}, "006616": {"suoxie": "GYZLXXCYHHC", "name": "\u5de5\u94f6\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHANLUEXINXINGCHANYEHUNHEC"}, "006617": {"suoxie": "GYRZDKZQ", "name": "\u5de5\u94f6\u745e\u6cfd\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIZEDINGKAIZHAIQUAN"}, "006618": {"suoxie": "CJKZZZQA", "name": "\u957f\u6c5f\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGJIANGKEZHUANZHAIZHAIQUANA"}, "006619": {"suoxie": "CJKZZZQC", "name": "\u957f\u6c5f\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGJIANGKEZHUANZHAIZHAIQUANC"}, "006620": {"suoxie": "HXYL2045FOFA", "name": "\u534e\u590f\u517b\u80012045(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAYANGLAO2045FOFA"}, "006621": {"suoxie": "HXYL2045FOFC", "name": "\u534e\u590f\u517b\u80012045(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAYANGLAO2045FOFC"}, "006622": {"suoxie": "HXYL2035FOFA", "name": "\u534e\u590f\u517b\u80012035(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYANGLAO2035FOFA"}, "006623": {"suoxie": "HXYL2035FOFC", "name": "\u534e\u590f\u517b\u80012035(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYANGLAO2035FOFC"}, "006624": {"suoxie": "ZTYHJZYXHH", "name": "\u4e2d\u6cf0\u7389\u8861\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGTAIYUHENGJIAZHIYOUXUANHUNHE"}, "006625": {"suoxie": "HAJXCZZQ", "name": "\u6c47\u5b89\u5609\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIAXINCHUNZHAIZHAIQUAN"}, "006626": {"suoxie": "SXZQCDZA", "name": "\u5c71\u897f\u8bc1\u5238\u8d85\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANXIZHENGQUANCHAODUANZHAIA"}, "006627": {"suoxie": "SXZQCDZC", "name": "\u5c71\u897f\u8bc1\u5238\u8d85\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANXIZHENGQUANCHAODUANZHAIC"}, "006629": {"suoxie": "ZSXYZDZA", "name": "\u62db\u5546\u946b\u60a6\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGXINYUEZHONGDUANZHAIA"}, "006630": {"suoxie": "ZSXYZDZC", "name": "\u62db\u5546\u946b\u60a6\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGXINYUEZHONGDUANZHAIC"}, "006631": {"suoxie": "XYZLA", "name": "\u946b\u5143\u81fb\u5229A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHENLIA"}, "006632": {"suoxie": "XYZLC", "name": "\u946b\u5143\u81fb\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHENLIC"}, "006633": {"suoxie": "BSZZ13ZJZZSA", "name": "\u535a\u65f6\u4e2d\u503a1-3\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI13ZHENGJINZHAIZHISHUA"}, "006634": {"suoxie": "BSZZ13ZJZZSC", "name": "\u535a\u65f6\u4e2d\u503a1-3\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI13ZHENGJINZHAIZHISHUC"}, "006635": {"suoxie": "YYWYZQ", "name": "\u6c38\u8d62\u4f1f\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGWEIYIZHAIQUAN"}, "006636": {"suoxie": "HFHXCZZQA", "name": "\u534e\u5bcc\u6052\u6b23\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGXINCHUNZHAIZHAIQUANA"}, "006637": {"suoxie": "HFHXCZZQC", "name": "\u534e\u5bcc\u6052\u6b23\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUHENGXINCHUNZHAIZHAIQUANC"}, "006638": {"suoxie": "RBXSCZA", "name": "\u4eba\u4fdd\u946b\u76db\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOXINSHENGCHUNZHAIA"}, "006639": {"suoxie": "RBXSCZC", "name": "\u4eba\u4fdd\u946b\u76db\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOXINSHENGCHUNZHAIC"}, "006640": {"suoxie": "ZJXY6GYDKZA", "name": "\u4e2d\u91d1\u65b0\u51436\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINYUAN6GEYUEDINGKAIZHAIA"}, "006641": {"suoxie": "ZJXY6GYDKZC", "name": "\u4e2d\u91d1\u65b0\u51436\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINYUAN6GEYUEDINGKAIZHAIC"}, "006642": {"suoxie": "HTBXJNLDK", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u5229\u5b9a\u5f00", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJINIANLIDINGKAI"}, "006644": {"suoxie": "HYYFXFSJHHA", "name": "\u5f18\u6bc5\u8fdc\u65b9\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGXIAOFEISHENGJIHUNHEA"}, "006645": {"suoxie": "YHAFZDQZQ", "name": "\u94f6\u534e\u5b89\u4e30\u4e2d\u77ed\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANFENGZHONGDUANQIZHAIQUAN"}, "006646": {"suoxie": "HTFDZZQA", "name": "\u6c47\u6dfb\u5bcc\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUDUANZHAIZHAIQUANA"}, "006647": {"suoxie": "HTFDZZQC", "name": "\u6c47\u6dfb\u5bcc\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUDUANZHAIZHAIQUANC"}, "006648": {"suoxie": "HADYZHHA", "name": "\u6c47\u5b89\u591a\u56e0\u5b50\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANDUOYINZIHUNHEA"}, "006649": {"suoxie": "HADYZHHC", "name": "\u6c47\u5b89\u591a\u56e0\u5b50\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANDUOYINZIHUNHEC"}, "006650": {"suoxie": "ZSAQZQ", "name": "\u62db\u5546\u5b89\u5e86\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANQINGZHAIQUAN"}, "006652": {"suoxie": "FGJRDCHYHHA", "name": "\u5bcc\u56fd\u91d1\u878d\u5730\u4ea7\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJINRONGDICHANHANGYEHUNHEA"}, "006653": {"suoxie": "NFCLDKZFQS", "name": "\u5357\u65b9\u7545\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHANGLIDINGKAIZHAIFAQISHI"}, "006654": {"suoxie": "HTZJJJXDKZA", "name": "\u534e\u6cf0\u7d2b\u91d1\u5b63\u5b63\u4eab\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINJIJIXIANGDINGKAIZHAIA"}, "006655": {"suoxie": "HTZJJJXDKZC", "name": "\u534e\u6cf0\u7d2b\u91d1\u5b63\u5b63\u4eab\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINJIJIXIANGDINGKAIZHAIC"}, "006656": {"suoxie": "FZFBZZ500ETFLJA", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENG500ETFLIANJIEA"}, "006657": {"suoxie": "FZFBZZ500ETFLJC", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENG500ETFLIANJIEC"}, "006658": {"suoxie": "CTZZXGHLDQZSA", "name": "\u8d22\u901a\u4e2d\u8bc1\u9999\u6e2f\u7ea2\u5229\u7b49\u6743\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZHONGZHENGXIANGGANGHONGLIDENGQUANZHISHUA"}, "006659": {"suoxie": "CTZZXGHLDQZSC", "name": "\u8d22\u901a\u4e2d\u8bc1\u9999\u6e2f\u7ea2\u5229\u7b49\u6743\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZHONGZHENGXIANGGANGHONGLIDENGQUANZHISHUC"}, "006660": {"suoxie": "YYCYZQA", "name": "\u6c38\u8d62\u660c\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHANGYIZHAIQUANA"}, "006661": {"suoxie": "YYCYZQC", "name": "\u6c38\u8d62\u660c\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHANGYIZHAIQUANC"}, "006662": {"suoxie": "YFDAYCDZA", "name": "\u6613\u65b9\u8fbe\u5b89\u60a6\u8d85\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANYUECHAODUANZHAIA"}, "006663": {"suoxie": "YFDAYCDZC", "name": "\u6613\u65b9\u8fbe\u5b89\u60a6\u8d85\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANYUECHAODUANZHAIC"}, "006664": {"suoxie": "YFDAYCDZF", "name": "\u6613\u65b9\u8fbe\u5b89\u60a6\u8d85\u77ed\u503aF", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANYUECHAODUANZHAIF"}, "006665": {"suoxie": "HXDKZQA", "name": "\u534e\u590f\u9f0e\u5eb7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGKANGZHAIQUANA"}, "006666": {"suoxie": "HXDKZQC", "name": "\u534e\u590f\u9f0e\u5eb7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGKANGZHAIQUANC"}, "006667": {"suoxie": "NHRYDQKFZQ", "name": "\u5357\u534e\u745e\u5143\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUIYUANDINGQIKAIFANGZHAIQUAN"}, "006668": {"suoxie": "HXZDZZQA", "name": "\u534e\u590f\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIAZHONGDUANZHAIZHAIQUANA"}, "006669": {"suoxie": "HXZDZZQC", "name": "\u534e\u590f\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIAZHONGDUANZHAIZHAIQUANC"}, "006670": {"suoxie": "GFJXCZZQ", "name": "\u5e7f\u53d1\u666f\u79c0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGXIUCHUNZHAIZHAIQUAN"}, "006671": {"suoxie": "GFXFSJGP", "name": "\u5e7f\u53d1\u6d88\u8d39\u5347\u7ea7\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAXIAOFEISHENGJIGUPIAO"}, "006672": {"suoxie": "GFZCDZZQA", "name": "\u5e7f\u53d1\u62db\u8d22\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZHAOCAIDUANZHAIZHAIQUANA"}, "006673": {"suoxie": "GFZCDZZQC", "name": "\u5e7f\u53d1\u62db\u8d22\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZHAOCAIDUANZHAIZHAIQUANC"}, "006674": {"suoxie": "DCJXCZZQB", "name": "\u5927\u6210\u666f\u65ed\u7eaf\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGXUCHUNZHAIZHAIQUANB"}, "006675": {"suoxie": "BYPPXFGPA", "name": "\u5b9d\u76c8\u54c1\u724c\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGPINPAIXIAOFEIGUPIAOA"}, "006676": {"suoxie": "BYPPXFGPC", "name": "\u5b9d\u76c8\u54c1\u724c\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGPINPAIXIAOFEIGUPIAOC"}, "006677": {"suoxie": "ZYWHDZZQA", "name": "\u4e2d\u94f6\u7a33\u6c47\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYINWENHUIDUANZHAIZHAIQUANA"}, "006678": {"suoxie": "ZYWHDZZQC", "name": "\u4e2d\u94f6\u7a33\u6c47\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYINWENHUIDUANZHAIZHAIQUANC"}, "006679": {"suoxie": "GFDQSSYZSMYXHA", "name": "\u5e7f\u53d1\u9053\u743c\u65af\u77f3\u6cb9\u6307\u6570\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "GUANGFADAOQIONGSISHIYOUZHISHUMEIYUANXIANHUIA"}, "006680": {"suoxie": "GFDQSSYZSMYXHC", "name": "\u5e7f\u53d1\u9053\u743c\u65af\u77f3\u6cb9\u6307\u6570\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "GUANGFADAOQIONGSISHIYOUZHISHUMEIYUANXIANHUIC"}, "006681": {"suoxie": "JSCCJTJLCZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u805a\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIJULICHUNZHAI"}, "006682": {"suoxie": "JSCCZZ500ZQ", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u8bc1500\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGZHONGZHENG500ZENGQIANG"}, "006683": {"suoxie": "FGGYQYZZQD", "name": "\u5bcc\u56fd\u56fd\u6709\u4f01\u4e1a\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOGUOYOUQIYEZHAIZHAIQUAND"}, "006684": {"suoxie": "FGXYZZQD", "name": "\u5bcc\u56fd\u4fe1\u7528\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXINYONGZHAIZHAIQUAND"}, "006687": {"suoxie": "FZFBSZ100ETFLJA", "name": "\u65b9\u6b63\u5bcc\u90a6\u6df1\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGSHENZHENG100ETFLIANJIEA"}, "006688": {"suoxie": "FZFBSZ100ETFLJC", "name": "\u65b9\u6b63\u5bcc\u90a6\u6df1\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGSHENZHENG100ETFLIANJIEC"}, "006689": {"suoxie": "FZFBXHHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u4fe1\u6cd3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGXINHONGHUNHEA"}, "006692": {"suoxie": "JXXFSJGPA", "name": "\u91d1\u4fe1\u6d88\u8d39\u5347\u7ea7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JINXINXIAOFEISHENGJIGUPIAOA"}, "006693": {"suoxie": "JXXFSJGPC", "name": "\u91d1\u4fe1\u6d88\u8d39\u5347\u7ea7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JINXINXIAOFEISHENGJIGUPIAOC"}, "006695": {"suoxie": "XHXRXZDZC", "name": "\u65b0\u534e\u946b\u65e5\u4eab\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINHUAXINRIXIANGZHONGDUANZHAIC"}, "006696": {"suoxie": "HTFYJYXLHPZHH", "name": "\u6c47\u6dfb\u5bcc\u7814\u7a76\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYANJIUYOUXUANLINGHUOPEIZHIHUNHE"}, "006697": {"suoxie": "HBZZYHETFLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGYINHANGETFLIANJIEC"}, "006698": {"suoxie": "HTCXHS300ZQA", "name": "\u7ea2\u571f\u521b\u65b0\u6caa\u6df1300\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGTUCHUANGXINHUSHEN300ZENGQIANGA"}, "006699": {"suoxie": "HTCXHS300ZQC", "name": "\u7ea2\u571f\u521b\u65b0\u6caa\u6df1300\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGTUCHUANGXINHUSHEN300ZENGQIANGC"}, "006700": {"suoxie": "HTCXWJHHA", "name": "\u7ea2\u571f\u521b\u65b0\u7a33\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTUCHUANGXINWENJIANHUNHEA"}, "006701": {"suoxie": "HTCXWJHHC", "name": "\u7ea2\u571f\u521b\u65b0\u7a33\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTUCHUANGXINWENJIANHUNHEC"}, "006704": {"suoxie": "YFDMSCIZGAGLJA", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA\u80a1\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOAGULIANJIEA"}, "006705": {"suoxie": "YFDMSCIZGAGLJC", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA\u80a1\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOAGULIANJIEC"}, "006706": {"suoxie": "ZRJHDQKFZQ", "name": "\u4e2d\u878d\u805a\u6c47\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUHUIDINGQIKAIFANGZHAIQUAN"}, "006707": {"suoxie": "YYHYZQA", "name": "\u6c38\u8d62\u5b8f\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHONGYIZHAIQUANA"}, "006708": {"suoxie": "YYHYZQC", "name": "\u6c38\u8d62\u5b8f\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHONGYIZHAIQUANC"}, "006712": {"suoxie": "QHKYMSCIZGAGXFA", "name": "\u524d\u6d77\u5f00\u6e90MSCI\u4e2d\u56fdA\u80a1\u6d88\u8d39A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANMSCIZHONGGUOAGUXIAOFEIA"}, "006713": {"suoxie": "QHKYMSCIZGAGXFC", "name": "\u524d\u6d77\u5f00\u6e90MSCI\u4e2d\u56fdA\u80a1\u6d88\u8d39C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANMSCIZHONGGUOAGUXIAOFEIC"}, "006714": {"suoxie": "BSFYCZZQ", "name": "\u535a\u65f6\u5bcc\u6e90\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYUANCHUNZHAIZHAIQUAN"}, "006715": {"suoxie": "DFYTCZ1NA", "name": "\u4e1c\u65b9\u6c38\u6cf0\u7eaf\u503a1\u5e74A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGTAICHUNZHAI1NIANA"}, "006716": {"suoxie": "DFYTCZ1NC", "name": "\u4e1c\u65b9\u6c38\u6cf0\u7eaf\u503a1\u5e74C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGTAICHUNZHAI1NIANC"}, "006717": {"suoxie": "PAHJDKZC", "name": "\u5e73\u5b89\u60e0\u91d1\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIJINDINGKAIZHAIC"}, "006718": {"suoxie": "GRRSLTYXHHA", "name": "\u56fd\u878d\u878d\u76db\u9f99\u5934\u4e25\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUORONGRONGSHENGLONGTOUYANXUANHUNHEA"}, "006719": {"suoxie": "GRRSLTYXHHC", "name": "\u56fd\u878d\u878d\u76db\u9f99\u5934\u4e25\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUORONGRONGSHENGLONGTOUYANXUANHUNHEC"}, "006720": {"suoxie": "PAHXYSHHA", "name": "\u5e73\u5b89\u6838\u5fc3\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANHEXINYOUSHIHUNHEA"}, "006721": {"suoxie": "PAHXYSHHC", "name": "\u5e73\u5b89\u6838\u5fc3\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANHEXINYOUSHIHUNHEC"}, "006724": {"suoxie": "GYSZHLETFLJC", "name": "\u5de5\u94f6\u6df1\u8bc1\u7ea2\u5229ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHENZHENGHONGLIETFLIANJIEC"}, "006725": {"suoxie": "GTFYCZZQ", "name": "\u56fd\u6cf0\u4e30\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIFENGYINGCHUNZHAIZHAIQUAN"}, "006727": {"suoxie": "BSZZ35JCKHA", "name": "\u535a\u65f6\u4e2d\u503a3-5\u8fdb\u51fa\u53e3\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35JINCHUKOUHANGA"}, "006728": {"suoxie": "BSZZ35JCKHC", "name": "\u535a\u65f6\u4e2d\u503a3-5\u8fdb\u51fa\u53e3\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35JINCHUKOUHANGC"}, "006729": {"suoxie": "WJZZ500ZSZQA", "name": "\u4e07\u5bb6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAZHONGZHENG500ZHISHUZENGQIANGA"}, "006730": {"suoxie": "WJZZ500ZSZQC", "name": "\u4e07\u5bb6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAZHONGZHENG500ZHISHUZENGQIANGC"}, "006731": {"suoxie": "FZFBFLCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u5bcc\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGFULICHUNZHAIA"}, "006732": {"suoxie": "FZFBFLCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u5bcc\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGFULICHUNZHAIC"}, "006733": {"suoxie": "BSCYBETFLJC", "name": "\u535a\u65f6\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHICHUANGYEBANETFLIANJIEC"}, "006734": {"suoxie": "GJHXDZZQA", "name": "\u56fd\u91d1\u60e0\u946b\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINHUIXINDUANZHAIZHAIQUANA"}, "006735": {"suoxie": "GJHXDZZQC", "name": "\u56fd\u91d1\u60e0\u946b\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINHUIXINDUANZHAIZHAIQUANC"}, "006736": {"suoxie": "GTRYXJZZHH", "name": "\u56fd\u6295\u745e\u94f6\u5148\u8fdb\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINXIANJINZHIZAOHUNHE"}, "006737": {"suoxie": "NAHHZQ", "name": "\u8bfa\u5b89\u6052\u60e0\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANHENGHUIZHAIQUAN"}, "006738": {"suoxie": "GYRXTHZQA", "name": "\u5de5\u94f6\u745e\u4fe1\u6dfb\u6167\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINRUIXINTIANHUIZHAIQUANA"}, "006739": {"suoxie": "GYRXTHZQC", "name": "\u5de5\u94f6\u745e\u4fe1\u6dfb\u6167\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINRUIXINTIANHUIZHAIQUANC"}, "006740": {"suoxie": "GYZLZDZZQA", "name": "\u5de5\u94f6\u5c0a\u5229\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNLIZHONGDUANZHAIZHAIQUANA"}, "006741": {"suoxie": "GYZLZDZZQC", "name": "\u5de5\u94f6\u5c0a\u5229\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNLIZHONGDUANZHAIZHAIQUANC"}, "006742": {"suoxie": "NFZYZQ", "name": "\u5357\u65b9\u81fb\u5143\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHENYUANZHAIQUAN"}, "006743": {"suoxie": "ZRYSCJ50ETFLJA", "name": "\u4e2d\u878d\u592e\u89c6\u8d22\u7ecf50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGYANGSHICAIJING50ETFLIANJIEA"}, "006744": {"suoxie": "ZRYSCJ50ETFLJC", "name": "\u4e2d\u878d\u592e\u89c6\u8d22\u7ecf50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGYANGSHICAIJING50ETFLIANJIEC"}, "006745": {"suoxie": "JYZZ13NNFZZSA", "name": "\u4ea4\u94f6\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINZHONGZHAI13NIANNONGFAZHAIZHISHUA"}, "006746": {"suoxie": "JYZZ13NNFZZSC", "name": "\u4ea4\u94f6\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINZHONGZHAI13NIANNONGFAZHAIZHISHUC"}, "006747": {"suoxie": "DHXLCZ", "name": "\u4e1c\u6d77\u7965\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGHAIXIANGLICHUNZHAI"}, "006748": {"suoxie": "FGZZJZETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u4ef7\u503cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJIAZHIETFLIANJIEA"}, "006750": {"suoxie": "FGDLCZDKZ", "name": "\u5bcc\u56fd\u5fb7\u5229\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUODELICHUNZHAIDINGKAIZHAI"}, "006751": {"suoxie": "FGHLKJGPA", "name": "\u5bcc\u56fd\u4e92\u8054\u79d1\u6280\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOHULIANKEJIGUPIAOA"}, "006752": {"suoxie": "THGGTJXA", "name": "\u5929\u5f18\u6e2f\u80a1\u901a\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGGANGGUTONGJINGXUANA"}, "006753": {"suoxie": "THGGTJXC", "name": "\u5929\u5f18\u6e2f\u80a1\u901a\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGGANGGUTONGJINGXUANC"}, "006754": {"suoxie": "XYYLDKZFQS", "name": "\u946b\u5143\u60a6\u5229\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANYUELIDINGKAIZHAIFAQISHI"}, "006756": {"suoxie": "GTZZSWYYETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u751f\u7269\u533b\u836fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGSHENGWUYIYAOETFLIANJIEA"}, "006757": {"suoxie": "GTZZSWYYETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u751f\u7269\u533b\u836fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGSHENGWUYIYAOETFLIANJIEC"}, "006758": {"suoxie": "NYHLJLZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u7984\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINLUZHAIQUAN"}, "006759": {"suoxie": "YHLHYCHH", "name": "\u94f6\u6cb3\u4e50\u6d3b\u4f18\u8403\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELEHUOYOUCUIHUNHE"}, "006760": {"suoxie": "GJHYCZC", "name": "\u56fd\u91d1\u60e0\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIYINGCHUNZHAIC"}, "006761": {"suoxie": "YHJYZQ", "name": "\u94f6\u6cb3\u5bb6\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJIAYINGZHAIQUAN"}, "006762": {"suoxie": "GTJXCZZQ", "name": "\u56fd\u6cf0\u805a\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUXIANGCHUNZHAIZHAIQUAN"}, "006763": {"suoxie": "HTFYL2030HHFOF", "name": "\u6c47\u6dfb\u5bcc\u517b\u80012030\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYANGLAO2030HUNHEFOF"}, "006764": {"suoxie": "JSCCJTXLCZA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u946b\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIXINLICHUNZHAIA"}, "006767": {"suoxie": "YHJYZQ", "name": "\u94f6\u6cb3\u5609\u88d5\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJIAYUZHAIQUAN"}, "006768": {"suoxie": "HAHGSYXHH", "name": "\u534e\u5b89\u6caa\u6e2f\u6df1\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUGANGSHENYOUXUANHUNHE"}, "006769": {"suoxie": "CCYJJXHHA", "name": "\u957f\u57ce\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYANJIUJINGXUANHUNHEA"}, "006771": {"suoxie": "YYHYZQ", "name": "\u6c38\u8d62\u5408\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHEYIZHAIQUAN"}, "006772": {"suoxie": "HTFFRZDZ", "name": "\u6c47\u6dfb\u5bcc\u4e30\u6da6\u4e2d\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUFENGRUNZHONGDUANZHAI"}, "006773": {"suoxie": "GSABZRZDZZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8363\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOSHOUANBAOZUNRONGZHONGDUANZHAIZHAIQUANA"}, "006774": {"suoxie": "GSABZRZDZZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8363\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOSHOUANBAOZUNRONGZHONGDUANZHAIZHAIQUANC"}, "006775": {"suoxie": "QHKYYZCZHH", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u8d28\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANYOUZHICHENGZHANGHUNHE"}, "006776": {"suoxie": "HXDLZQA", "name": "\u534e\u590f\u9f0e\u7565\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLUEZHAIQUANA"}, "006777": {"suoxie": "HXDLZQC", "name": "\u534e\u590f\u9f0e\u7565\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGLUEZHAIQUANC"}, "006778": {"suoxie": "GFHSZGQYQDIIA", "name": "\u5e7f\u53d1\u6052\u751f\u4e2d\u56fd\u4f01\u4e1a(QDII)A", "style": "QDII", "pinyin": "GUANGFAHENGSHENGZHONGGUOQIYEQDIIA"}, "006779": {"suoxie": "GFHSZGQYQDIIC", "name": "\u5e7f\u53d1\u6052\u751f\u4e2d\u56fd\u4f01\u4e1a(QDII)C", "style": "QDII", "pinyin": "GUANGFAHENGSHENGZHONGGUOQIYEQDIIC"}, "006780": {"suoxie": "GFWJCLHH", "name": "\u5e7f\u53d1\u7a33\u5065\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAWENJIANCELUEHUNHE"}, "006781": {"suoxie": "HFJXGGTJXGP", "name": "\u6c47\u4e30\u664b\u4fe1\u6e2f\u80a1\u901a\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINGANGGUTONGJINGXUANGUPIAO"}, "006782": {"suoxie": "GTXLSGYDKZ", "name": "\u56fd\u6cf0\u4fe1\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIXINLISANGEYUEDINGKAIZHAI"}, "006783": {"suoxie": "HTCXZZ500ZQA", "name": "\u7ea2\u571f\u521b\u65b0\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGTUCHUANGXINZHONGZHENG500ZENGQIANGA"}, "006784": {"suoxie": "HTCXZZ500ZQC", "name": "\u7ea2\u571f\u521b\u65b0\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGTUCHUANGXINZHONGZHENG500ZENGQIANGC"}, "006785": {"suoxie": "DFLHDCLHHA", "name": "\u4e1c\u65b9\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGLIANGHUADUOCELUEHUNHEA"}, "006786": {"suoxie": "TKGGTDXFZSA", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901a\u5927\u6d88\u8d39\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGDAXIAOFEIZHISHUA"}, "006787": {"suoxie": "TKGGTDXFZSC", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901a\u5927\u6d88\u8d39\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGDAXIAOFEIZHISHUC"}, "006788": {"suoxie": "JYTXDKZFQS", "name": "\u91d1\u9e70\u6dfb\u946b\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANXINDINGKAIZHAIFAQISHI"}, "006789": {"suoxie": "ZXBCJFZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGFENGZHAIQUANA"}, "006790": {"suoxie": "ZXBCJFZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGFENGZHAIQUANC"}, "006791": {"suoxie": "JXRXCZZQ", "name": "\u5efa\u4fe1\u777f\u5174\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIXINGCHUNZHAIZHAIQUAN"}, "006792": {"suoxie": "PHGMHLGPMYXH", "name": "\u9e4f\u534e\u6e2f\u7f8e\u4e92\u8054\u80a1\u7968\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "PENGHUAGANGMEIHULIANGUPIAOMEIYUANXIANHUI"}, "006793": {"suoxie": "JYWXDZZQA", "name": "\u4ea4\u94f6\u7a33\u946b\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAOYINWENXINDUANZHAIZHAIQUANA"}, "006794": {"suoxie": "JYWXDZZQC", "name": "\u4ea4\u94f6\u7a33\u946b\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAOYINWENXINDUANZHAIZHAIQUANC"}, "006795": {"suoxie": "GTYXSGYDKZ", "name": "\u56fd\u6cf0\u88d5\u7965\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIYUXIANGSANGEYUEDINGKAIZHAI"}, "006796": {"suoxie": "FGXFSJHHA", "name": "\u5bcc\u56fd\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXIAOFEISHENGJIHUNHEA"}, "006797": {"suoxie": "JSZDZZQA", "name": "\u5609\u5b9e\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIZHONGDUANZHAIZHAIQUANA"}, "006798": {"suoxie": "JSZDZZQC", "name": "\u5609\u5b9e\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIZHONGDUANZHAIZHAIQUANC"}, "006799": {"suoxie": "CTHYZDZZQA", "name": "\u8d22\u901a\u9e3f\u8fd0\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGYUNZHONGDUANZHAIZHAIQUANA"}, "006800": {"suoxie": "CTHYZDZZQC", "name": "\u8d22\u901a\u9e3f\u8fd0\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGYUNZHONGDUANZHAIZHAIQUANC"}, "006801": {"suoxie": "QHLHKJXFHHA", "name": "\u524d\u6d77\u8054\u5408\u79d1\u6280\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHEKEJIXIANFENGHUNHEA"}, "006802": {"suoxie": "QHLHKJXFHHC", "name": "\u524d\u6d77\u8054\u5408\u79d1\u6280\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHEKEJIXIANFENGHUNHEC"}, "006803": {"suoxie": "JSHTJXGP", "name": "\u5609\u5b9e\u4e92\u901a\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHUTONGJINGXUANGUPIAO"}, "006804": {"suoxie": "FGDZZQXA", "name": "\u5bcc\u56fd\u77ed\u503a\u503a\u5238\u578bA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUODUANZHAIZHAIQUANXINGA"}, "006805": {"suoxie": "FGDZZQXC", "name": "\u5bcc\u56fd\u77ed\u503a\u503a\u5238\u578bC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUODUANZHAIZHAIQUANXINGC"}, "006806": {"suoxie": "XBLDTYDZZQA", "name": "\u897f\u90e8\u5229\u5f97\u6dfb\u76c8\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XIBULIDETIANYINGDUANZHAIZHAIQUANA"}, "006807": {"suoxie": "XBLDTYDZZQC", "name": "\u897f\u90e8\u5229\u5f97\u6dfb\u76c8\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XIBULIDETIANYINGDUANZHAIZHAIQUANC"}, "006808": {"suoxie": "XBLDTYDZZQE", "name": "\u897f\u90e8\u5229\u5f97\u6dfb\u76c8\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XIBULIDETIANYINGDUANZHAIZHAIQUANE"}, "006809": {"suoxie": "TKXGYHZSA", "name": "\u6cf0\u5eb7\u9999\u6e2f\u94f6\u884c\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGXIANGGANGYINHANGZHISHUA"}, "006810": {"suoxie": "TKXGYHZSC", "name": "\u6cf0\u5eb7\u9999\u6e2f\u94f6\u884c\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGXIANGGANGYINHANGZHISHUC"}, "006811": {"suoxie": "DCJYZQA", "name": "\u5927\u6210\u666f\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGYINGZHAIQUANA"}, "006812": {"suoxie": "DCHFZQ", "name": "\u5927\u6210\u60e0\u798f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIFUZHAIQUAN"}, "006813": {"suoxie": "BSHYHBHH", "name": "\u535a\u65f6\u6c47\u60a6\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUIYUEHUIBAOHUNHE"}, "006816": {"suoxie": "TKGGTDCZSA", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901a\u5730\u4ea7\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGDICHANZHISHUA"}, "006817": {"suoxie": "TKGGTDCZSC", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901a\u5730\u4ea7\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGDICHANZHISHUC"}, "006818": {"suoxie": "AXYLQDGPA", "name": "\u5b89\u4fe1\u76c8\u5229\u9a71\u52a8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINYINGLIQUDONGGUPIAOA"}, "006819": {"suoxie": "AXYLQDGPC", "name": "\u5b89\u4fe1\u76c8\u5229\u9a71\u52a8\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINYINGLIQUDONGGUPIAOC"}, "006822": {"suoxie": "KSZHHA", "name": "\u51ef\u77f3\u6e5b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHIZHANHUNHEA"}, "006823": {"suoxie": "KSZHHC", "name": "\u51ef\u77f3\u6e5b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHIZHANHUNHEC"}, "006824": {"suoxie": "CJHXXRXDZZQA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u65e5\u4eab\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINXINRIXIANGDUANZHAIZHAIQUANA"}, "006825": {"suoxie": "CJHXXRXDZZQC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u65e5\u4eab\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINXINRIXIANGDUANZHAIZHAIQUANC"}, "006826": {"suoxie": "HBBYZQA", "name": "\u534e\u5b9d\u5b9d\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOYUZHAIQUANA"}, "006827": {"suoxie": "ZJRXCZZQ", "name": "\u4e2d\u52a0\u745e\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUIXINCHUNZHAIZHAIQUAN"}, "006828": {"suoxie": "YHJTZQ", "name": "\u94f6\u6cb3\u4e45\u6cf0\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJIUTAIZHAIQUAN"}, "006829": {"suoxie": "PYLFDZA", "name": "\u9e4f\u626c\u5229\u6ca3\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGLIFENGDUANZHAIA"}, "006830": {"suoxie": "PYLFDZC", "name": "\u9e4f\u626c\u5229\u6ca3\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGLIFENGDUANZHAIC"}, "006831": {"suoxie": "PYLFDZE", "name": "\u9e4f\u626c\u5229\u6ca3\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGLIFENGDUANZHAIE"}, "006832": {"suoxie": "PYTLZQZQA", "name": "\u9e4f\u626c\u6dfb\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGTIANLIZENGQIANGZHAIQUANA"}, "006833": {"suoxie": "PYTLZQZQC", "name": "\u9e4f\u626c\u6dfb\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGTIANLIZENGQIANGZHAIQUANC"}, "006834": {"suoxie": "GYZXDZZQA", "name": "\u5de5\u94f6\u5c0a\u4eab\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNXIANGDUANZHAIZHAIQUANA"}, "006835": {"suoxie": "GYZXDZZQC", "name": "\u5de5\u94f6\u5c0a\u4eab\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNXIANGDUANZHAIZHAIQUANC"}, "006836": {"suoxie": "YYHZYN", "name": "\u6c38\u8d62\u60e0\u6cfd\u4e00\u5e74", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGHUIZEYINIAN"}, "006837": {"suoxie": "YHXYSJHZQC", "name": "\u94f6\u534e\u4fe1\u7528\u56db\u5b63\u7ea2\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGSIJIHONGZHAIQUANC"}, "006838": {"suoxie": "XYRLSGYDKZ", "name": "\u946b\u5143\u8363\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANRONGLISANGEYUEDINGKAIZHAI"}, "006839": {"suoxie": "AXJLZQZQA", "name": "\u5b89\u4fe1\u805a\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINJULIZENGQIANGZHAIQUANA"}, "006840": {"suoxie": "AXJLZQZQC", "name": "\u5b89\u4fe1\u805a\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINJULIZENGQIANGZHAIQUANC"}, "006841": {"suoxie": "JSZXCZZQ", "name": "\u5609\u5b9e\u81f4\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIXIANGCHUNZHAIZHAIQUAN"}, "006842": {"suoxie": "NFGL6GYDKZ", "name": "\u5357\u65b9\u56fd\u52296\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGGUOLI6GEYUEDINGKAIZHAI"}, "006843": {"suoxie": "ZXJTRYHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u6ea2\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOURUIYIHUNHEC"}, "006844": {"suoxie": "ZXJTWLHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOUWENLIHUNHEC"}, "006845": {"suoxie": "ZXJTJLHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u805a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINJIANTOUJULIHUNHEC"}, "006846": {"suoxie": "ZYFJGQZDKZA", "name": "\u4e2d\u94f6\u798f\u5efa\u56fd\u4f01\u503a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFUJIANGUOQIZHAIDINGKAIZHAIA"}, "006847": {"suoxie": "ZYFJGQZDKZC", "name": "\u4e2d\u94f6\u798f\u5efa\u56fd\u4f01\u503a\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINFUJIANGUOQIZHAIDINGKAIZHAIC"}, "006848": {"suoxie": "BSZZ510NFHA", "name": "\u535a\u65f6\u4e2d\u503a5-10\u519c\u53d1\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI510NONGFAHANGA"}, "006849": {"suoxie": "BSZZ510NFHC", "name": "\u535a\u65f6\u4e2d\u503a5-10\u519c\u53d1\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI510NONGFAHANGC"}, "006850": {"suoxie": "YYYLZQ", "name": "\u6c38\u8d62\u9890\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYILIZHAIQUAN"}, "006851": {"suoxie": "PAZDZZQE", "name": "\u5e73\u5b89\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANZHONGDUANZHAIZHAIQUANE"}, "006852": {"suoxie": "YYXLZGDJDZA", "name": "\u6c38\u8d62\u8fc5\u5229\u4e2d\u9ad8\u7b49\u7ea7\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGXUNLIZHONGGAODENGJIDUANZHAIA"}, "006853": {"suoxie": "ZYHXZQ", "name": "\u4e2d\u94f6\u6c47\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHUIXIANGZHAIQUAN"}, "006854": {"suoxie": "RBXZCZA", "name": "\u4eba\u4fdd\u946b\u6cfd\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOXINZECHUNZHAIA"}, "006855": {"suoxie": "RBXZCZC", "name": "\u4eba\u4fdd\u946b\u6cfd\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOXINZECHUNZHAIC"}, "006856": {"suoxie": "YHFT3GYDKZQ", "name": "\u94f6\u6cb3\u4e30\u6cf03\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEFENGTAI3GEYUEDINGKAIZHAIQUAN"}, "006857": {"suoxie": "FCZRLHPZHHA", "name": "\u8702\u5de2\u5353\u777f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FENGCHAOZHUORUILINGHUOPEIZHIHUNHEA"}, "006858": {"suoxie": "FCZRLHPZHHC", "name": "\u8702\u5de2\u5353\u777f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FENGCHAOZHUORUILINGHUOPEIZHIHUNHEC"}, "006859": {"suoxie": "YFDHCYL2033FOF", "name": "\u6613\u65b9\u8fbe\u6c47\u8bda\u517b\u80012033(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAHUICHENGYANGLAO2033FOF"}, "006860": {"suoxie": "YFDHCYL2038FOF", "name": "\u6613\u65b9\u8fbe\u6c47\u8bda\u517b\u80012038(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAHUICHENGYANGLAO2038FOF"}, "006861": {"suoxie": "ZSHYWJYLFOFA", "name": "\u62db\u5546\u548c\u60a6\u7a33\u5065\u517b\u8001(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGHEYUEWENJIANYANGLAOFOFA"}, "006862": {"suoxie": "ZSHYWJYLFOFC", "name": "\u62db\u5546\u548c\u60a6\u7a33\u5065\u517b\u8001(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGHEYUEWENJIANYANGLAOFOFC"}, "006863": {"suoxie": "GLAZNZZHH", "name": "\u56fd\u8054\u5b89\u667a\u80fd\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANZHINENGZHIZAOHUNHE"}, "006864": {"suoxie": "GLAHXZCCLHH", "name": "\u56fd\u8054\u5b89\u6838\u5fc3\u8d44\u4ea7\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHEXINZICHANCELUEHUNHE"}, "006865": {"suoxie": "TKAHCZZQC", "name": "\u6cf0\u5eb7\u5b89\u60e0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANHUICHUNZHAIZHAIQUANC"}, "006867": {"suoxie": "YFDFHZQC", "name": "\u6613\u65b9\u8fbe\u4e30\u534e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAFENGHUAZHAIQUANC"}, "006868": {"suoxie": "HXKJCZGP", "name": "\u534e\u590f\u79d1\u6280\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAKEJICHENGZHANGGUPIAO"}, "006869": {"suoxie": "GFZCXJRZ", "name": "\u5e7f\u53d1\u653f\u7b56\u6027\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHENGCEXINGJINRONGZHAI"}, "006870": {"suoxie": "GFJHZDZA", "name": "\u5e7f\u53d1\u666f\u548c\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGHEZHONGDUANZHAIA"}, "006871": {"suoxie": "GFJHZDZC", "name": "\u5e7f\u53d1\u666f\u548c\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGHEZHONGDUANZHAIC"}, "006872": {"suoxie": "CXYTPHYLFOFA", "name": "\u957f\u4fe1\u9890\u5929\u5e73\u8861\u517b\u8001(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGXINYITIANPINGHENGYANGLAOFOFA"}, "006873": {"suoxie": "CXYTPHYLFOFC", "name": "\u957f\u4fe1\u9890\u5929\u5e73\u8861\u517b\u8001(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGXINYITIANPINGHENGYANGLAOFOFC"}, "006874": {"suoxie": "CJHXHXZDZA", "name": "\u521b\u91d1\u5408\u4fe1\u6052\u5174\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINHENGXINGZHONGDUANZHAIA"}, "006875": {"suoxie": "CJHXHXZDZC", "name": "\u521b\u91d1\u5408\u4fe1\u6052\u5174\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINHENGXINGZHONGDUANZHAIC"}, "006876": {"suoxie": "GTRYWJYLFOFA", "name": "\u56fd\u6295\u745e\u94f6\u7a33\u5065\u517b\u8001(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINWENJIANYANGLAOFOFA"}, "006877": {"suoxie": "TZLHHXJXHHA", "name": "\u5929\u6cbb\u91cf\u5316\u6838\u5fc3\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANZHILIANGHUAHEXINJINGXUANHUNHEA"}, "006878": {"suoxie": "TZLHHXJXHHC", "name": "\u5929\u6cbb\u91cf\u5316\u6838\u5fc3\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANZHILIANGHUAHEXINJINGXUANHUNHEC"}, "006879": {"suoxie": "HAZNSHHHA", "name": "\u534e\u5b89\u667a\u80fd\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHINENGSHENGHUOHUNHEA"}, "006880": {"suoxie": "JYAXWJYLYNFOF", "name": "\u4ea4\u94f6\u5b89\u4eab\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINANXIANGWENJIANYANGLAOYINIANFOF"}, "006881": {"suoxie": "HBDJKHH", "name": "\u534e\u5b9d\u5927\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAODAJIANKANGHUNHE"}, "006882": {"suoxie": "HTBXJKXFA", "name": "\u534e\u6cf0\u4fdd\u5174\u5065\u5eb7\u6d88\u8d39A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJIANKANGXIAOFEIA"}, "006883": {"suoxie": "HTBXJKXFC", "name": "\u534e\u6cf0\u4fdd\u5174\u5065\u5eb7\u6d88\u8d39C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJIANKANGXIAOFEIC"}, "006884": {"suoxie": "HTFAAAJXYCZA", "name": "\u6c47\u6dfb\u5bccAAA\u7ea7\u4fe1\u7528\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUAAAJIXINYONGCHUNZHAIA"}, "006885": {"suoxie": "HTFAAAJXYCZC", "name": "\u6c47\u6dfb\u5bccAAA\u7ea7\u4fe1\u7528\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUAAAJIXINYONGCHUNZHAIC"}, "006886": {"suoxie": "GYYL2050HHFOF", "name": "\u5de5\u94f6\u517b\u80012050\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINYANGLAO2050HUNHEFOF"}, "006887": {"suoxie": "NDXSHHHA", "name": "\u8bfa\u5fb7\u65b0\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEXINSHENGHUOHUNHEA"}, "006888": {"suoxie": "NDXSHHHC", "name": "\u8bfa\u5fb7\u65b0\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEXINSHENGHUOHUNHEC"}, "006889": {"suoxie": "PAHHCZZQ", "name": "\u5e73\u5b89\u60e0\u9e3f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIHONGCHUNZHAIZHAIQUAN"}, "006890": {"suoxie": "STMGLXYXHH", "name": "\u4e0a\u6295\u6469\u6839\u9886\u5148\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENLINGXIANYOUXUANHUNHE"}, "006891": {"suoxie": "HXYL2050WNFOF", "name": "\u534e\u590f\u517b\u80012050\u4e94\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAYANGLAO2050WUNIANFOF"}, "006892": {"suoxie": "XHDLZQC", "name": "\u65b0\u534e\u9f0e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUADINGLIZHAIQUANC"}, "006893": {"suoxie": "HTFFLDZZQA", "name": "\u6c47\u6dfb\u5bcc\u4e30\u5229\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUFENGLIDUANZHAIZHAIQUANA"}, "006894": {"suoxie": "XYYL2035FOFA", "name": "\u5174\u4e1a\u517b\u80012035(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGYEYANGLAO2035FOFA"}, "006895": {"suoxie": "XYYL2035FOFC", "name": "\u5174\u4e1a\u517b\u80012035(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGYEYANGLAO2035FOFC"}, "006896": {"suoxie": "XHJLZQA", "name": "\u65b0\u534e\u805a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAJULIZHAIQUANA"}, "006897": {"suoxie": "XHJLZQC", "name": "\u65b0\u534e\u805a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAJULIZHAIQUANC"}, "006898": {"suoxie": "THHFZQHBZQA", "name": "\u5929\u5f18\u5f18\u4e30\u589e\u5f3a\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGHONGFENGZENGQIANGHUIBAOZHAIQUANA"}, "006899": {"suoxie": "THHFZQHBZQC", "name": "\u5929\u5f18\u5f18\u4e30\u589e\u5f3a\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGHONGFENGZENGQIANGHUIBAOZHAIQUANC"}, "006901": {"suoxie": "SYHXLZQA", "name": "\u4e0a\u94f6\u6167\u7965\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIXIANGLIZHAIQUANA"}, "006902": {"suoxie": "CSAXZDZA", "name": "\u957f\u76db\u5b89\u946b\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGSHENGANXINZHONGDUANZHAIA"}, "006903": {"suoxie": "CSAXZDZC", "name": "\u957f\u76db\u5b89\u946b\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGSHENGANXINZHONGDUANZHAIC"}, "006904": {"suoxie": "TKCYSJHHA", "name": "\u6cf0\u5eb7\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGCHANYESHENGJIHUNHEA"}, "006905": {"suoxie": "TKCYSJHHC", "name": "\u6cf0\u5eb7\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGCHANYESHENGJIHUNHEC"}, "006906": {"suoxie": "CJHXXSYHHE", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u6536\u76ca\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINXINSHOUYIHUNHEE"}, "006907": {"suoxie": "YHAXDZZQA", "name": "\u94f6\u534e\u5b89\u946b\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANXINDUANZHAIZHAIQUANA"}, "006908": {"suoxie": "YHAXDZZQC", "name": "\u94f6\u534e\u5b89\u946b\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANXINDUANZHAIZHAIQUANC"}, "006909": {"suoxie": "HXZLXXCZETFLJA", "name": "\u534e\u590f\u6218\u7565\u65b0\u5174\u6210\u6307ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHANLUEXINXINGCHENGZHIETFLIANJIEA"}, "006910": {"suoxie": "HXZLXXCZETFLJC", "name": "\u534e\u590f\u6218\u7565\u65b0\u5174\u6210\u6307ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHANLUEXINXINGCHENGZHIETFLIANJIEC"}, "006911": {"suoxie": "CJLHJXZXA", "name": "\u957f\u6c5f\u91cf\u5316\u5320\u5fc3\u7504\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAJIANGXINZHENXUANA"}, "006912": {"suoxie": "CCJTHS300ZSC", "name": "\u957f\u57ce\u4e45\u6cf0\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGJIUTAIHUSHEN300ZHISHUC"}, "006913": {"suoxie": "NFHYA", "name": "\u5357\u65b9\u534e\u5143A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHUAYUANA"}, "006914": {"suoxie": "NFHYC", "name": "\u5357\u65b9\u534e\u5143C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHUAYUANC"}, "006915": {"suoxie": "NFHYZQA", "name": "\u5357\u65b9\u4ea8\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGYUANZHAIQUANA"}, "006916": {"suoxie": "NFHYZQC", "name": "\u5357\u65b9\u4ea8\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGYUANZHAIQUANC"}, "006917": {"suoxie": "SYHXLZQC", "name": "\u4e0a\u94f6\u6167\u7965\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIXIANGLIZHAIQUANC"}, "006918": {"suoxie": "GLAAXWJYLFOF", "name": "\u56fd\u8054\u5b89\u5b89\u4eab\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANANXIANGWENJIANYANGLAOFOF"}, "006919": {"suoxie": "GSABTHCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u548c\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIHECHUNZHAIZHAIQUAN"}, "006920": {"suoxie": "JSWHCZZQC", "name": "\u5609\u5b9e\u7a33\u534e\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENHUACHUNZHAIZHAIQUANC"}, "006921": {"suoxie": "NFZCHH", "name": "\u5357\u65b9\u667a\u8bda\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHICHENGHUNHE"}, "006922": {"suoxie": "HBBYZQC", "name": "\u534e\u5b9d\u5b9d\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOYUZHAIQUANC"}, "006923": {"suoxie": "QHKYHGSFZQGPA", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u975e\u5468\u671f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANHUGANGSHENFEIZHOUQIGUPIAOA"}, "006924": {"suoxie": "QHKYHGSFZQGPC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u975e\u5468\u671f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANHUGANGSHENFEIZHOUQIGUPIAOC"}, "006925": {"suoxie": "YYZZ13NZJZZS", "name": "\u6c38\u8d62\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGZHAI13NIANZHENGJINZHAIZHISHU"}, "006926": {"suoxie": "CCLHJXGPA", "name": "\u957f\u57ce\u91cf\u5316\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGLIANGHUAJINGXUANGUPIAOA"}, "006927": {"suoxie": "ZSHJJXDKZFQS", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u946b\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINJUXINDINGKAIZHAIFAQISHI"}, "006928": {"suoxie": "CCCYBZSZQC", "name": "\u957f\u57ce\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGCHUANGYEBANZHISHUZENGQIANGC"}, "006929": {"suoxie": "BSFRCZZQ", "name": "\u535a\u65f6\u5bcc\u878d\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFURONGCHUNZHAIZHAIQUAN"}, "006930": {"suoxie": "TKGGTTMTZSA", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901aTMT\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGTMTZHISHUA"}, "006931": {"suoxie": "TKGGTTMTZSC", "name": "\u6cf0\u5eb7\u6e2f\u80a1\u901aTMT\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGANGGUTONGTMTZHISHUC"}, "006932": {"suoxie": "PA03NZCXJRZA", "name": "\u5e73\u5b890-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN03NIANZHENGCEXINGJINRONGZHAIA"}, "006933": {"suoxie": "PA03NZCXJRZC", "name": "\u5e73\u5b890-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN03NIANZHENGCEXINGJINRONGZHAIC"}, "006934": {"suoxie": "PA35NZCXJRZA", "name": "\u5e73\u5b893-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN35NIANZHENGCEXINGJINRONGZHAIA"}, "006935": {"suoxie": "PA35NZCXJRZC", "name": "\u5e73\u5b893-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN35NIANZHENGCEXINGJINRONGZHAIC"}, "006936": {"suoxie": "HAAS3GYDKZ", "name": "\u534e\u5b89\u5b89\u76db3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANSHENG3GEYUEDINGKAIZHAI"}, "006937": {"suoxie": "GYHS300ZSC", "name": "\u5de5\u94f6\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUSHEN300ZHISHUC"}, "006938": {"suoxie": "PHZZ500ZSLOFC", "name": "\u9e4f\u534e\u4e2d\u8bc1500\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ZHISHULOFC"}, "006939": {"suoxie": "PHHS300ZSLOFC", "name": "\u9e4f\u534e\u6caa\u6df1300\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAHUSHEN300ZHISHULOFC"}, "006941": {"suoxie": "GTHYCZZQ", "name": "\u56fd\u6cf0\u60e0\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIYINGCHUNZHAIZHAIQUAN"}, "006942": {"suoxie": "HTBRLHMXA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u660e\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAMINGXUANA"}, "006943": {"suoxie": "HTBRLHMXC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u660e\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAMINGXUANC"}, "006944": {"suoxie": "YYYLZQ", "name": "\u6c38\u8d62\u60a6\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYUELIZHAIQUAN"}, "006946": {"suoxie": "BYJXDQKFZQ", "name": "\u5b9d\u76c8\u805a\u4eab\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGJUXIANGDINGQIKAIFANGZHAIQUAN"}, "006947": {"suoxie": "HBZDZZQA", "name": "\u534e\u5b9d\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUABAOZHONGDUANZHAIZHAIQUANA"}, "006948": {"suoxie": "HBZDZZQC", "name": "\u534e\u5b9d\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUABAOZHONGDUANZHAIZHAIQUANC"}, "006949": {"suoxie": "QHKYQLDQKFZQ", "name": "\u524d\u6d77\u5f00\u6e90\u4e7e\u5229\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANQIANLIDINGQIKAIFANGZHAIQUAN"}, "006952": {"suoxie": "ZYJYHBHH", "name": "\u4e2d\u94f6\u666f\u5143\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGYINJINGYUANHUIBAOHUNHE"}, "006953": {"suoxie": "HAAYZQA", "name": "\u534e\u5b89\u5b89\u4e1a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANYEZHAIQUANA"}, "006954": {"suoxie": "HAAYZQC", "name": "\u534e\u5b89\u5b89\u4e1a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANYEZHAIQUANC"}, "006955": {"suoxie": "GTHFCZZQ", "name": "\u56fd\u6cf0\u60e0\u5bcc\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIFUCHUNZHAIZHAIQUAN"}, "006956": {"suoxie": "PHYRYNDQKFZQ", "name": "\u9e4f\u534e\u6c38\u6da6\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGRUNYINIANDINGQIKAIFANGZHAIQUAN"}, "006957": {"suoxie": "CJLHJXZXC", "name": "\u957f\u6c5f\u91cf\u5316\u5320\u5fc3\u7504\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAJIANGXINZHENXUANC"}, "006958": {"suoxie": "PHYRYNDQKFZQ", "name": "\u9e4f\u534e\u6c38\u878d\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGRONGYINIANDINGQIKAIFANGZHAIQUAN"}, "006959": {"suoxie": "PYZZ35NNFZZSA", "name": "\u6d66\u94f6\u4e2d\u503a3-5\u5e74\u519c\u53d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINZHONGZHAI35NIANNONGFAZHAIZHISHUA"}, "006960": {"suoxie": "PYZZ35NNFZZSC", "name": "\u6d66\u94f6\u4e2d\u503a3-5\u5e74\u519c\u53d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINZHONGZHAI35NIANNONGFAZHAIZHISHUC"}, "006961": {"suoxie": "NF710NGKZA", "name": "\u5357\u65b97-10\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG710NIANGUOKAIZHAIA"}, "006962": {"suoxie": "NF710NGKZC", "name": "\u5357\u65b97-10\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG710NIANGUOKAIZHAIC"}, "006963": {"suoxie": "ZJYJDKZQA", "name": "\u4e2d\u52a0\u9890\u747e\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIJINDINGKAIZHAIQUANA"}, "006964": {"suoxie": "ZJYJDKZQC", "name": "\u4e2d\u52a0\u9890\u747e\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYIJINDINGKAIZHAIQUANC"}, "006965": {"suoxie": "CTARDZZQA", "name": "\u8d22\u901a\u5b89\u745e\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGANRUIDUANZHAIZHAIQUANA"}, "006966": {"suoxie": "CTARDZZQC", "name": "\u8d22\u901a\u5b89\u745e\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGANRUIDUANZHAIZHAIQUANC"}, "006967": {"suoxie": "CTHYLTJXHHA", "name": "\u8d22\u901a\u884c\u4e1a\u9f99\u5934\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGHANGYELONGTOUJINGXUANHUNHEA"}, "006968": {"suoxie": "CTHYLTJXHHC", "name": "\u8d22\u901a\u884c\u4e1a\u9f99\u5934\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGHANGYELONGTOUJINGXUANHUNHEC"}, "006969": {"suoxie": "YXYFGDZZHH", "name": "\u5706\u4fe1\u6c38\u4e30\u9ad8\u7aef\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGGAODUANZHIZAOHUNHE"}, "006970": {"suoxie": "GFJLCZ", "name": "\u5e7f\u53d1\u666f\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGLICHUNZHAI"}, "006972": {"suoxie": "JYMAHBDKA", "name": "\u91d1\u9e70\u6c11\u5b89\u56de\u62a5\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGMINANHUIBAODINGKAIA"}, "006973": {"suoxie": "TPRYHHA", "name": "\u592a\u5e73\u777f\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIYINGHUNHEA"}, "006974": {"suoxie": "JYXRXZQA", "name": "\u91d1\u9e70\u946b\u65e5\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGXINRIXIANGZHAIQUANA"}, "006975": {"suoxie": "JYXRXZQC", "name": "\u91d1\u9e70\u946b\u65e5\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGXINRIXIANGZHAIQUANC"}, "006976": {"suoxie": "PHHXYSHH", "name": "\u9e4f\u534e\u6838\u5fc3\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAHEXINYOUSHIHUNHE"}, "006977": {"suoxie": "NYHLHTSNDKHH", "name": "\u519c\u94f6\u6c47\u7406\u6d77\u68e0\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILIHAITANGSANNIANDINGKAIHUNHE"}, "006978": {"suoxie": "TKAXCZZQA", "name": "\u6cf0\u5eb7\u5b89\u6b23\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANXINCHUNZHAIZHAIQUANA"}, "006979": {"suoxie": "TKAXCZZQC", "name": "\u6cf0\u5eb7\u5b89\u6b23\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANXINCHUNZHAIZHAIQUANC"}, "006980": {"suoxie": "GSABTHCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u6052\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIHENGCHUNZHAIZHAIQUAN"}, "006981": {"suoxie": "ZJXYYGPA", "name": "\u4e2d\u91d1\u65b0\u533b\u836f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINXINYIYAOGUPIAOA"}, "006984": {"suoxie": "XQHRDKZQFQS", "name": "\u5174\u5168\u6052\u745e\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGRUIDINGKAIZHAIQUANFAQISHI"}, "006985": {"suoxie": "XQHYZQA", "name": "\u5174\u5168\u6052\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGYUZHAIQUANA"}, "006986": {"suoxie": "PAJTYDKZA", "name": "\u5e73\u5b89\u5b63\u6dfb\u76c8\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJITIANYINGDINGKAIZHAIA"}, "006987": {"suoxie": "PAJTYDKZC", "name": "\u5e73\u5b89\u5b63\u6dfb\u76c8\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJITIANYINGDINGKAIZHAIC"}, "006988": {"suoxie": "PAJTYDKZE", "name": "\u5e73\u5b89\u5b63\u6dfb\u76c8\u5b9a\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJITIANYINGDINGKAIZHAIE"}, "006989": {"suoxie": "JXZDZCZZQA", "name": "\u5efa\u4fe1\u4e2d\u77ed\u503a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINZHONGDUANZHAICHUNZHAIZHAIQUANA"}, "006990": {"suoxie": "JXZDZCZZQC", "name": "\u5efa\u4fe1\u4e2d\u77ed\u503a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINZHONGDUANZHAICHUNZHAIZHAIQUANC"}, "006991": {"suoxie": "MSJYKNYLFOF", "name": "\u6c11\u751f\u52a0\u94f6\u5eb7\u5b81\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINKANGNINGYANGLAOFOF"}, "006992": {"suoxie": "JHJCYSJXHH", "name": "\u5609\u5408\u9526\u521b\u4f18\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINCHUANGYOUSHIJINGXUANHUNHE"}, "006993": {"suoxie": "XYCLSGYDKZ", "name": "\u946b\u5143\u627f\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANCHENGLISANGEYUEDINGKAIZHAI"}, "006994": {"suoxie": "GTRASGYDQKFZQ", "name": "\u56fd\u6cf0\u745e\u5b89\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUIANSANGEYUEDINGQIKAIFANGZHAIQUAN"}, "006995": {"suoxie": "NFHL6GYDKZA", "name": "\u5357\u65b9\u60e0\u52296\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHUILI6GEYUEDINGKAIZHAIA"}, "006996": {"suoxie": "NFHL6GYDKZC", "name": "\u5357\u65b9\u60e0\u52296\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHUILI6GEYUEDINGKAIZHAIC"}, "006997": {"suoxie": "PAHTCZZQ", "name": "\u5e73\u5b89\u60e0\u6dfb\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUITIANCHUNZHAIZHAIQUAN"}, "006998": {"suoxie": "GFJXZDZA", "name": "\u5e7f\u53d1\u666f\u5174\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGXINGZHONGDUANZHAIA"}, "006999": {"suoxie": "GFJXZDZC", "name": "\u5e7f\u53d1\u666f\u5174\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGXINGZHONGDUANZHAIC"}, "007000": {"suoxie": "PHZZ13NGKHZA", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13NIANGUOKAIHANGZHAIA"}, "007001": {"suoxie": "PHZZ13NGKHZC", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13NIANGUOKAIHANGZHAIC"}, "007005": {"suoxie": "ZJXYYGPC", "name": "\u4e2d\u91d1\u65b0\u533b\u836f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINXINYIYAOGUPIAOC"}, "007008": {"suoxie": "ZYCZYXYNDKZA", "name": "\u4e2d\u90ae\u7eaf\u503a\u4f18\u9009\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIYOUXUANYINIANDINGKAIZHAIA"}, "007009": {"suoxie": "ZYCZYXYNDKZC", "name": "\u4e2d\u90ae\u7eaf\u503a\u4f18\u9009\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIYOUXUANYINIANDINGKAIZHAIC"}, "007010": {"suoxie": "GSABZZ13NZSA", "name": "\u56fd\u5bff\u5b89\u4fdd\u4e2d\u503a1-3\u5e74\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZHONGZHAI13NIANZHISHUA"}, "007011": {"suoxie": "GSABZZ13NZSC", "name": "\u56fd\u5bff\u5b89\u4fdd\u4e2d\u503a1-3\u5e74\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZHONGZHAI13NIANZHISHUC"}, "007012": {"suoxie": "XCCSHHFQSA", "name": "\u6e58\u8d22\u957f\u987a\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIANGCAICHANGSHUNHUNHEFAQISHIA"}, "007013": {"suoxie": "XCCSHHFQSC", "name": "\u6e58\u8d22\u957f\u987a\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIANGCAICHANGSHUNHUNHEFAQISHIC"}, "007014": {"suoxie": "JHPTDZZQA", "name": "\u5609\u5408\u78d0\u6cf0\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAHEPANTAIDUANZHAIZHAIQUANA"}, "007015": {"suoxie": "JHPTDZZQC", "name": "\u5609\u5408\u78d0\u6cf0\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAHEPANTAIDUANZHAIZHAIQUANC"}, "007016": {"suoxie": "FGRZHBHH", "name": "\u5bcc\u56fd\u777f\u6cfd\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUORUIZEHUIBAOHUNHE"}, "007017": {"suoxie": "PARYZDZA", "name": "\u5e73\u5b89\u5982\u610f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANRUYIZHONGDUANZHAIA"}, "007018": {"suoxie": "PARYZDZC", "name": "\u5e73\u5b89\u5982\u610f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANRUYIZHONGDUANZHAIC"}, "007019": {"suoxie": "PARYZDZE", "name": "\u5e73\u5b89\u5982\u610f\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANRUYIZHONGDUANZHAIE"}, "007020": {"suoxie": "HATXZDZC", "name": "\u534e\u5b89\u6dfb\u946b\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANTIANXINZHONGDUANZHAIC"}, "007021": {"suoxie": "JSZZ13ZJZZSA", "name": "\u5609\u5b9e\u4e2d\u503a1-3\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHAI13ZHENGJINZHAIZHISHUA"}, "007022": {"suoxie": "JSZZ13ZJZZSC", "name": "\u5609\u5b9e\u4e2d\u503a1-3\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHAI13ZHENGJINZHAIZHISHUC"}, "007023": {"suoxie": "ZYZQAZZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANZEZHAIQUANA"}, "007024": {"suoxie": "ZYZQAZZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANZEZHAIQUANC"}, "007025": {"suoxie": "NFXL3GYDKZ", "name": "\u5357\u65b9\u946b\u52293\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXINLI3GEYUEDINGKAIZHAI"}, "007026": {"suoxie": "JXZZ13NGKZA", "name": "\u5efa\u4fe1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI13NIANGUOKAIZHAIA"}, "007027": {"suoxie": "JXZZ13NGKZC", "name": "\u5efa\u4fe1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI13NIANGUOKAIZHAIC"}, "007028": {"suoxie": "YFDZZ500ETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ETFLIANJIEA"}, "007029": {"suoxie": "YFDZZ500ETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ETFLIANJIEC"}, "007032": {"suoxie": "PAKZZZQA", "name": "\u5e73\u5b89\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PINGANKEZHUANZHAIZHAIQUANA"}, "007033": {"suoxie": "PAKZZZQC", "name": "\u5e73\u5b89\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PINGANKEZHUANZHAIZHAIQUANC"}, "007034": {"suoxie": "ZKWTWAZDLLC", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u5b89\u4e2d\u77ed\u5229\u7387C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGKEWOTUWOANZHONGDUANLILVC"}, "007035": {"suoxie": "ZYZZ13NQGKZZS", "name": "\u4e2d\u94f6\u4e2d\u503a1-3\u5e74\u671f\u56fd\u5f00\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGZHAI13NIANQIGUOKAIZHAIZHISHU"}, "007036": {"suoxie": "QHLHYXCZC", "name": "\u524d\u6d77\u8054\u5408\u6c38\u5174\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGXINGCHUNZHAIC"}, "007037": {"suoxie": "HFTJHCZ", "name": "\u6d77\u5bcc\u901a\u805a\u5408\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGJUHECHUNZHAI"}, "007038": {"suoxie": "QHLHTHCZC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u60e0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANHUICHUNZHAIC"}, "007039": {"suoxie": "QHLHHS300ZSC", "name": "\u524d\u6d77\u8054\u5408\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAILIANHEHUSHEN300ZHISHUC"}, "007040": {"suoxie": "QHLHYLHHC", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u9686\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGLONGHUNHEC"}, "007041": {"suoxie": "QHLHYTHHC", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u6d9b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGTAOHUNHEC"}, "007042": {"suoxie": "QHLHYJHHC", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u96bd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEYONGJUANHUNHEC"}, "007043": {"suoxie": "QHLHHXHHC", "name": "\u524d\u6d77\u8054\u5408\u6cd3\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEHONGXINHUNHEC"}, "007044": {"suoxie": "BDHS300ZSZQA", "name": "\u535a\u9053\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BODAOHUSHEN300ZHISHUZENGQIANGA"}, "007045": {"suoxie": "BDHS300ZSZQC", "name": "\u535a\u9053\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BODAOHUSHEN300ZHISHUZENGQIANGC"}, "007046": {"suoxie": "FZFBCXDLHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u521b\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCHUANGXINDONGLIHUNHEC"}, "007047": {"suoxie": "CCHXYSHH", "name": "\u957f\u57ce\u6838\u5fc3\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGHEXINYOUSHIHUNHE"}, "007048": {"suoxie": "PAAXLHPZHHC", "name": "\u5e73\u5b89\u5b89\u5fc3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANXINLINGHUOPEIZHIHUNHEC"}, "007049": {"suoxie": "PAXAHHE", "name": "\u5e73\u5b89\u946b\u5b89\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINANHUNHEE"}, "007050": {"suoxie": "XYHLSGYDKZ", "name": "\u946b\u5143\u6052\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHENGLISANGEYUEDINGKAIZHAI"}, "007053": {"suoxie": "PAJKXDKZA", "name": "\u5e73\u5b89\u5b63\u5f00\u946b\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIKAIXINDINGKAIZHAIA"}, "007054": {"suoxie": "PAJKXDKZC", "name": "\u5e73\u5b89\u5b63\u5f00\u946b\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIKAIXINDINGKAIZHAIC"}, "007055": {"suoxie": "PAJKXDKZE", "name": "\u5e73\u5b89\u5b63\u5f00\u946b\u5b9a\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIKAIXINDINGKAIZHAIE"}, "007056": {"suoxie": "YHJJJXHH", "name": "\u94f6\u534e\u79ef\u6781\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJIJIJINGXUANHUNHE"}, "007057": {"suoxie": "ZTLYDZA", "name": "\u4e2d\u6cf0\u84dd\u6708\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGTAILANYUEDUANZHAIA"}, "007058": {"suoxie": "ZTLYDZC", "name": "\u4e2d\u6cf0\u84dd\u6708\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGTAILANYUEDUANZHAIC"}, "007059": {"suoxie": "HTFYL2040WNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u517b\u80012040\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUITIANFUYANGLAO2040WUNIANCHIYOUHUNHEFOF"}, "007060": {"suoxie": "HTFYL2050HHFOF", "name": "\u6c47\u6dfb\u5bcc\u517b\u80012050\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYANGLAO2050HUNHEFOF"}, "007061": {"suoxie": "ZJJYSGYDKZA", "name": "\u4e2d\u52a0\u805a\u76c8\u56db\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJUYINGSIGEYUEDINGKAIZHAIA"}, "007062": {"suoxie": "ZJJYSGYDKZC", "name": "\u4e2d\u52a0\u805a\u76c8\u56db\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAJUYINGSIGEYUEDINGKAIZHAIC"}, "007063": {"suoxie": "CSYFHBHH", "name": "\u957f\u76db\u7814\u53d1\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGYANFAHUIBAOHUNHE"}, "007064": {"suoxie": "PYASSQSYXDRA", "name": "\u6d66\u94f6\u5b89\u76db\u4e0a\u6e05\u6240\u4f18\u9009\u77ed\u878dA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGSHANGQINGSUOYOUXUANDUANRONGA"}, "007065": {"suoxie": "PYASSQSYXDRC", "name": "\u6d66\u94f6\u5b89\u76db\u4e0a\u6e05\u6240\u4f18\u9009\u77ed\u878dC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGSHANGQINGSUOYOUXUANDUANRONGC"}, "007066": {"suoxie": "PYASXJZZHHA", "name": "\u6d66\u94f6\u5b89\u76db\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGXIANJINZHIZAOHUNHEA"}, "007067": {"suoxie": "PYASXJZZHHC", "name": "\u6d66\u94f6\u5b89\u76db\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGXIANJINZHIZAOHUNHEC"}, "007068": {"suoxie": "PYASPFCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u4e30\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUFENGCHUNZHAIZHAIQUANA"}, "007069": {"suoxie": "PYASPFCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u4e30\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUFENGCHUNZHAIZHAIQUANC"}, "007070": {"suoxie": "BSYZWJYLFOFA", "name": "\u535a\u65f6\u9890\u6cfd\u7a33\u5065\u517b\u8001(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIYIZEWENJIANYANGLAOFOFA"}, "007071": {"suoxie": "BSYZWJYLFOFC", "name": "\u535a\u65f6\u9890\u6cfd\u7a33\u5065\u517b\u8001(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIYIZEWENJIANYANGLAOFOFC"}, "007072": {"suoxie": "MSJYXFHHC", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINFUHUNHEC"}, "007073": {"suoxie": "HFTSQSDRZQC", "name": "\u6d77\u5bcc\u901a\u4e0a\u6e05\u6240\u77ed\u878d\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAIFUTONGSHANGQINGSUODUANRONGZHAIQUANC"}, "007074": {"suoxie": "GSABXLCHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u65b0\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOXINLANCHOUHUNHE"}, "007075": {"suoxie": "FGCYZQC", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHANYEZHAIQUANC"}, "007076": {"suoxie": "HTFZZYYETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u533b\u836fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYIYAOETFLIANJIEA"}, "007077": {"suoxie": "HTFZZYYETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u533b\u836fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYIYAOETFLIANJIEC"}, "007078": {"suoxie": "GY35NGKZZSA", "name": "\u5de5\u94f63-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN35NIANGUOKAIZHAIZHISHUA"}, "007079": {"suoxie": "GY35NGKZZSC", "name": "\u5de5\u94f63-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN35NIANGUOKAIZHAIZHISHUC"}, "007080": {"suoxie": "JXZZ510GKZSA", "name": "\u5efa\u4fe1\u4e2d\u503a5-10\u56fd\u5f00\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI510GUOKAIZHISHUA"}, "007081": {"suoxie": "JXZZ510GKZSC", "name": "\u5efa\u4fe1\u4e2d\u503a5-10\u56fd\u5f00\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI510GUOKAIZHISHUC"}, "007082": {"suoxie": "PAGDZZHHA", "name": "\u5e73\u5b89\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANGAODUANZHIZAOHUNHEA"}, "007083": {"suoxie": "PAGDZZHHC", "name": "\u5e73\u5b89\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANGAODUANZHIZAOHUNHEC"}, "007084": {"suoxie": "TZZXSJHH", "name": "\u5929\u6cbb\u8f6c\u578b\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANZHIZHUANXINGSHENGJIHUNHE"}, "007085": {"suoxie": "ZSRQHHC", "name": "\u62db\u5546\u745e\u5e86\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGRUIQINGHUNHEC"}, "007086": {"suoxie": "YYHLLGYDKZ", "name": "\u6c38\u8d62\u6c47\u5229\u516d\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHUILILIUGEYUEDINGKAIZHAI"}, "007088": {"suoxie": "MSJYHYZQ", "name": "\u6c11\u751f\u52a0\u94f6\u6052\u88d5\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHENGYUZHAIQUAN"}, "007089": {"suoxie": "GTZZ500ZSLHZQC", "name": "\u56fd\u6295\u4e2d\u8bc1500\u6307\u6570\u91cf\u5316\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOUZHONGZHENG500ZHISHULIANGHUAZENGQIANGC"}, "007090": {"suoxie": "HFTYLHHFOF", "name": "\u6d77\u5bcc\u901a\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGYANGLAOHUNHEFOF"}, "007091": {"suoxie": "DXXFYNDKZQ", "name": "\u4e1c\u5174\u5174\u798f\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINGFUYINIANDINGKAIZHAIQUAN"}, "007092": {"suoxie": "XYZZ35NGKZZSA", "name": "\u946b\u5143\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHONGZHAI35NIANGUOKAIZHAIZHISHUA"}, "007093": {"suoxie": "XYZZ35NGKZZSC", "name": "\u946b\u5143\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHONGZHAI35NIANGUOKAIZHAIZHISHUC"}, "007094": {"suoxie": "JXZZGKHZA", "name": "\u5efa\u4fe1\u4e2d\u503a\u56fd\u5f00\u884c\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAIGUOKAIHANGZHAIA"}, "007095": {"suoxie": "JXZZGKHZC", "name": "\u5efa\u4fe1\u4e2d\u503a\u56fd\u5f00\u884c\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAIGUOKAIHANGZHAIC"}, "007096": {"suoxie": "DCHS300ZSC", "name": "\u5927\u6210\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGHUSHEN300ZHISHUC"}, "007097": {"suoxie": "HTFZZ13NGKZA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANGUOKAIZHAIA"}, "007098": {"suoxie": "HTFZZ13NGKZC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANGUOKAIZHAIC"}, "007099": {"suoxie": "AXZXTYZQC", "name": "\u5b89\u4fe1\u5c0a\u4eab\u6dfb\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZUNXIANGTIANYIZHAIQUANC"}, "007100": {"suoxie": "ZYTLZQFQE", "name": "\u4e2d\u94f6\u6dfb\u5229\u503a\u5238\u53d1\u8d77E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINTIANLIZHAIQUANFAQIE"}, "007101": {"suoxie": "ZOYJLNDKHHC", "name": "\u4e2d\u6b27\u8fdc\u89c1\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUANJIANLIANGNIANDINGKAIHUNHEC"}, "007102": {"suoxie": "MSJYTXCZA", "name": "\u6c11\u751f\u52a0\u94f6\u6dfb\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINTIANXINCHUNZHAIA"}, "007103": {"suoxie": "MSJYTXCZC", "name": "\u6c11\u751f\u52a0\u94f6\u6dfb\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINTIANXINCHUNZHAIC"}, "007104": {"suoxie": "YFDHLDQKFZQ", "name": "\u6613\u65b9\u8fbe\u6052\u5229\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGLIDINGQIKAIFANGZHAIQUAN"}, "007105": {"suoxie": "GTFXCZZQ", "name": "\u56fd\u6cf0\u4e30\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIFENGXINCHUNZHAIZHAIQUAN"}, "007107": {"suoxie": "TPMSCIXGJZZQA", "name": "\u592a\u5e73MSCI\u9999\u6e2f\u4ef7\u503c\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIPINGMSCIXIANGGANGJIAZHIZENGQIANGA"}, "007108": {"suoxie": "TPMSCIXGJZZQC", "name": "\u592a\u5e73MSCI\u9999\u6e2f\u4ef7\u503c\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIPINGMSCIXIANGGANGJIAZHIZENGQIANGC"}, "007109": {"suoxie": "NFHGSHXYSHH", "name": "\u5357\u65b9\u6caa\u6e2f\u6df1\u6838\u5fc3\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHUGANGSHENHEXINYOUSHIHUNHE"}, "007110": {"suoxie": "GTRYGGTHHA", "name": "\u56fd\u6295\u745e\u94f6\u6e2f\u80a1\u901a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINGANGGUTONGHUNHEA"}, "007111": {"suoxie": "QHLHGMJKHHC", "name": "\u524d\u6d77\u8054\u5408\u56fd\u6c11\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAILIANHEGUOMINJIANKANGHUNHEC"}, "007113": {"suoxie": "YYGDZZHHA", "name": "\u6c38\u8d62\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGAODUANZHIZAOHUNHEA"}, "007114": {"suoxie": "YYGDZZHHC", "name": "\u6c38\u8d62\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGAODUANZHIZAOHUNHEC"}, "007115": {"suoxie": "JYSAASZQC", "name": "\u91d1\u5143\u987a\u5b89\u6849\u76db\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYUANSHUNANANSHENGZHAIQUANC"}, "007116": {"suoxie": "HBZJZZQ", "name": "\u534e\u5b9d\u653f\u91d1\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOZHENGJINZHAIZHAIQUAN"}, "007117": {"suoxie": "HTZJFTCZFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u6cf0\u7eaf\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINFENGTAICHUNZHAIFAQIA"}, "007118": {"suoxie": "HTZJFTCZFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u6cf0\u7eaf\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINFENGTAICHUNZHAIFAQIC"}, "007119": {"suoxie": "RYCZJZHHA", "name": "\u777f\u8fdc\u6210\u957f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIYUANCHENGZHANGJIAZHIHUNHEA"}, "007120": {"suoxie": "RYCZJZHHC", "name": "\u777f\u8fdc\u6210\u957f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIYUANCHENGZHANGJIAZHIHUNHEC"}, "007121": {"suoxie": "ZJYYCZZQ", "name": "\u4e2d\u52a0\u88d5\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYUYINGCHUNZHAIZHAIQUAN"}, "007122": {"suoxie": "GY13NGKZZSA", "name": "\u5de5\u94f61-3\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANGUOKAIZHAIZHISHUA"}, "007123": {"suoxie": "GY13NGKZZSC", "name": "\u5de5\u94f61-3\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANGUOKAIZHAIZHISHUC"}, "007124": {"suoxie": "GY13NNFZZSA", "name": "\u5de5\u94f61-3\u5e74\u519c\u53d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANNONGFAZHAIZHISHUA"}, "007125": {"suoxie": "GY13NNFZZSC", "name": "\u5de5\u94f61-3\u5e74\u519c\u53d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANNONGFAZHAIZHISHUC"}, "007126": {"suoxie": "BDYHHHA", "name": "\u535a\u9053\u8fdc\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOYUANHANGHUNHEA"}, "007127": {"suoxie": "BDYHHHC", "name": "\u535a\u9053\u8fdc\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOYUANHANGHUNHEC"}, "007128": {"suoxie": "THZQHBZQA", "name": "\u5929\u5f18\u589e\u5f3a\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGZENGQIANGHUIBAOZHAIQUANA"}, "007129": {"suoxie": "THZQHBZQC", "name": "\u5929\u5f18\u589e\u5f3a\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGZENGQIANGHUIBAOZHAIQUANC"}, "007130": {"suoxie": "ZGXPJZGP", "name": "\u4e2d\u5e9a\u5c0f\u76d8\u4ef7\u503c\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGGENGXIAOPANJIAZHIGUPIAO"}, "007132": {"suoxie": "CCGGTHH", "name": "\u957f\u57ce\u6e2f\u80a1\u901a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGGANGGUTONGHUNHE"}, "007133": {"suoxie": "JSCQJZYSGPA", "name": "\u5609\u5b9e\u957f\u9752\u7ade\u4e89\u4f18\u52bf\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHICHANGQINGJINGZHENGYOUSHIGUPIAOA"}, "007134": {"suoxie": "JSCQJZYSGPC", "name": "\u5609\u5b9e\u957f\u9752\u7ade\u4e89\u4f18\u52bf\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHICHANGQINGJINGZHENGYOUSHIGUPIAOC"}, "007135": {"suoxie": "GFZZ100ETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG100ETFLIANJIEA"}, "007136": {"suoxie": "GFZZ100ETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG100ETFLIANJIEC"}, "007137": {"suoxie": "PYYHLHDPA", "name": "\u9e4f\u626c\u5143\u5408\u91cf\u5316\u5927\u76d8A", "style": "\u80a1\u7968\u578b", "pinyin": "PENGYANGYUANHELIANGHUADAPANA"}, "007138": {"suoxie": "PYYHLHDPC", "name": "\u9e4f\u626c\u5143\u5408\u91cf\u5316\u5927\u76d8C", "style": "\u80a1\u7968\u578b", "pinyin": "PENGYANGYUANHELIANGHUADAPANC"}, "007139": {"suoxie": "FGMYJQHGSCZA", "name": "\u5bcc\u56fd\u6c11\u88d5\u8fdb\u53d6\u6caa\u6e2f\u6df1\u6210\u957fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOMINYUJINQUHUGANGSHENCHENGZHANGA"}, "007140": {"suoxie": "FGQQZQQDIIMYXH", "name": "\u5bcc\u56fd\u5168\u7403\u503a\u5238(QDII)\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "FUGUOQUANQIUZHAIQUANQDIIMEIYUANXIANHUI"}, "007141": {"suoxie": "JHWJZZHHA", "name": "\u5609\u5408\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAHEWENJIANZENGZHANGHUNHEA"}, "007142": {"suoxie": "JHWJZZHHC", "name": "\u5609\u5408\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAHEWENJIANZENGZHANGHUNHEC"}, "007143": {"suoxie": "GTRYHS300LHA", "name": "\u56fd\u6295\u745e\u94f6\u6caa\u6df1300\u91cf\u5316A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINHUSHEN300LIANGHUAA"}, "007144": {"suoxie": "GTRYHS300LHC", "name": "\u56fd\u6295\u745e\u94f6\u6caa\u6df1300\u91cf\u5316C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINHUSHEN300LIANGHUAC"}, "007145": {"suoxie": "TKAHCZ6GYDKZQ", "name": "\u6cf0\u5eb7\u5b89\u548c\u7eaf\u503a6\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGANHECHUNZHAI6GEYUEDINGKAIZHAIQUAN"}, "007146": {"suoxie": "PHYJZXHH", "name": "\u9e4f\u534e\u7814\u7a76\u667a\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYANJIUZHIXUANHUNHE"}, "007147": {"suoxie": "BSZZ13NGKHA", "name": "\u535a\u65f6\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI13NIANGUOKAIHANGA"}, "007148": {"suoxie": "BSZZ13NGKHC", "name": "\u535a\u65f6\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI13NIANGUOKAIHANGC"}, "007149": {"suoxie": "NFCYZDZA", "name": "\u5357\u65b9\u521d\u5143\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGCHUYUANZHONGDUANZHAIA"}, "007150": {"suoxie": "NFCYZDZC", "name": "\u5357\u65b9\u521d\u5143\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGCHUYUANZHONGDUANZHAIC"}, "007151": {"suoxie": "QHKYHGSJRHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u805a\u745e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANHUGANGSHENJURUIHUNHE"}, "007152": {"suoxie": "NDCLJX", "name": "\u8bfa\u5fb7\u7b56\u7565\u7cbe\u9009", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODECELUEJINGXUAN"}, "007153": {"suoxie": "HTFZZYHETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYINHANGETFLIANJIEA"}, "007154": {"suoxie": "HTFZZYHETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYINHANGETFLIANJIEC"}, "007155": {"suoxie": "YHZZYQ20ZQZS", "name": "\u94f6\u6cb3\u4e2d\u503a\u592e\u4f0120\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEZHONGZHAIYANGQI20ZHAIQUANZHISHU"}, "007158": {"suoxie": "PAHSDKZ", "name": "\u5e73\u5b89\u5408\u76db\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHESHENGDINGKAIZHAI"}, "007159": {"suoxie": "NFWJYLFOFA", "name": "\u5357\u65b9\u7a33\u5065\u517b\u8001(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGWENJIANYANGLAOFOFA"}, "007160": {"suoxie": "NFWJYLFOFC", "name": "\u5357\u65b9\u7a33\u5065\u517b\u8001(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGWENJIANYANGLAOFOFC"}, "007161": {"suoxie": "NFHQYNDKZ", "name": "\u5357\u65b9\u6052\u5e86\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGQINGYINIANDINGKAIZHAI"}, "007162": {"suoxie": "NFHQYNDKZC", "name": "\u5357\u65b9\u6052\u5e86\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGQINGYINIANDINGKAIZHAIC"}, "007163": {"suoxie": "PYASHBXNYA", "name": "\u6d66\u94f6\u5b89\u76db\u73af\u4fdd\u65b0\u80fd\u6e90A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGHUANBAOXINNENGYUANA"}, "007164": {"suoxie": "PYASHBXNYC", "name": "\u6d66\u94f6\u5b89\u76db\u73af\u4fdd\u65b0\u80fd\u6e90C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGHUANBAOXINNENGYUANC"}, "007165": {"suoxie": "HXZZ13NZJZZSA", "name": "\u534e\u590f\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAZHONGZHAI13NIANZHENGJINZHAIZHISHUA"}, "007166": {"suoxie": "HXZZ13NZJZZSC", "name": "\u534e\u590f\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAZHONGZHAI13NIANZHENGJINZHAIZHISHUC"}, "007167": {"suoxie": "HAAHZQA", "name": "\u534e\u5b89\u5b89\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANHEZHAIQUANA"}, "007168": {"suoxie": "HAAHZQC", "name": "\u534e\u5b89\u5b89\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANHEZHAIQUANC"}, "007169": {"suoxie": "YFDZZ13NGKZA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI13NIANGUOKAIZHAIA"}, "007170": {"suoxie": "YFDZZ13NGKZC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI13NIANGUOKAIZHAIC"}, "007171": {"suoxie": "YFDZZ35NGKHZA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u884c\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI35NIANGUOKAIHANGZHAIA"}, "007172": {"suoxie": "YFDZZ35NGKHZC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u884c\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI35NIANGUOKAIHANGZHAIC"}, "007173": {"suoxie": "ZSTXDKZFQSA", "name": "\u62db\u5546\u6dfb\u65ed\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANXUDINGKAIZHAIFAQISHIA"}, "007174": {"suoxie": "ZSTXDKZFQSC", "name": "\u62db\u5546\u6dfb\u65ed\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANXUDINGKAIZHAIFAQISHIC"}, "007175": {"suoxie": "ZRJTDQKFZQ", "name": "\u4e2d\u878d\u805a\u901a\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUTONGDINGQIKAIFANGZHAIQUAN"}, "007176": {"suoxie": "FGHLCZFQSC", "name": "\u5bcc\u56fd\u6cd3\u5229\u7eaf\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHONGLICHUNZHAIFAQISHIC"}, "007177": {"suoxie": "ZSZNHYYXHHA", "name": "\u6d59\u5546\u667a\u80fd\u884c\u4e1a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHINENGHANGYEYOUXUANHUNHEA"}, "007178": {"suoxie": "ZSZHYQGGXA", "name": "\u6d59\u5546\u4e2d\u534e\u9884\u671f\u9ad8\u80a1\u606fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHONGHUAYUQIGAOGUXIA"}, "007179": {"suoxie": "ZSFSCZZQ", "name": "\u6d59\u5546\u4e30\u987a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGFENGSHUNCHUNZHAIZHAIQUAN"}, "007180": {"suoxie": "HAZZ13NZCJRZA", "name": "\u534e\u5b89\u4e2d\u503a1-3\u5e74\u653f\u7b56\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI13NIANZHENGCEJINRONGZHAIA"}, "007181": {"suoxie": "HAZZ13NZCJRZC", "name": "\u534e\u5b89\u4e2d\u503a1-3\u5e74\u653f\u7b56\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI13NIANZHENGCEJINRONGZHAIC"}, "007182": {"suoxie": "WJHGSLCHHA", "name": "\u4e07\u5bb6\u6caa\u6e2f\u6df1\u84dd\u7b79\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHUGANGSHENLANCHOUHUNHEA"}, "007183": {"suoxie": "WJHGSLCHHC", "name": "\u4e07\u5bb6\u6caa\u6e2f\u6df1\u84dd\u7b79\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHUGANGSHENLANCHOUHUNHEC"}, "007184": {"suoxie": "FCTXCZA", "name": "\u8702\u5de2\u6dfb\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANXINCHUNZHAIA"}, "007185": {"suoxie": "FCTXCZC", "name": "\u8702\u5de2\u6dfb\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANXINCHUNZHAIC"}, "007186": {"suoxie": "HXZZ35NZJZZSA", "name": "\u534e\u590f\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAZHONGZHAI35NIANZHENGJINZHAIZHISHUA"}, "007187": {"suoxie": "HXZZ35NZJZZSC", "name": "\u534e\u590f\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAZHONGZHAI35NIANZHENGJINZHAIZHISHUC"}, "007188": {"suoxie": "JSYL2050HHFOF", "name": "\u5609\u5b9e\u517b\u80012050\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYANGLAO2050HUNHEFOF"}, "007189": {"suoxie": "NHJZQHCZZQA", "name": "\u5357\u534e\u4ef7\u503c\u542f\u822a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUAJIAZHIQIHANGCHUNZHAIZHAIQUANA"}, "007190": {"suoxie": "NHJZQHCZZQC", "name": "\u5357\u534e\u4ef7\u503c\u542f\u822a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUAJIAZHIQIHANGCHUNZHAIZHAIQUANC"}, "007191": {"suoxie": "FGZZJZETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u4ef7\u503cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJIAZHIETFLIANJIEC"}, "007192": {"suoxie": "HYYJJXHHC", "name": "\u6052\u8d8a\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYANJIUJINGXUANHUNHEC"}, "007193": {"suoxie": "HYHXJXHHC", "name": "\u6052\u8d8a\u6838\u5fc3\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEHEXINJINGXUANHUNHEC"}, "007194": {"suoxie": "CCDZZQA", "name": "\u957f\u57ce\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGCHENGDUANZHAIZHAIQUANA"}, "007195": {"suoxie": "CCDZZQC", "name": "\u957f\u57ce\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGCHENGDUANZHAIZHAIQUANC"}, "007196": {"suoxie": "PAHHCZ", "name": "\u5e73\u5b89\u60e0\u5408\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIHECHUNZHAI"}, "007197": {"suoxie": "FGZZ15NNFHA", "name": "\u5bcc\u56fd\u4e2d\u503a1-5\u5e74\u519c\u53d1\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAI15NIANNONGFAHANGA"}, "007198": {"suoxie": "FGZZ15NNFHC", "name": "\u5bcc\u56fd\u4e2d\u503a1-5\u5e74\u519c\u53d1\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAI15NIANNONGFAHANGC"}, "007199": {"suoxie": "YYTLZQA", "name": "\u6c38\u8d62\u6cf0\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTAILIZHAIQUANA"}, "007200": {"suoxie": "YYTLZQC", "name": "\u6c38\u8d62\u6cf0\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTAILIZHAIQUANC"}, "007201": {"suoxie": "MSJYJYCZ", "name": "\u6c11\u751f\u52a0\u94f6\u805a\u76ca\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINJUYICHUNZHAI"}, "007202": {"suoxie": "THYZCZQY", "name": "\u5929\u5f18\u4f18\u8d28\u6210\u957f\u4f01\u4e1a", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGYOUZHICHENGZHANGQIYE"}, "007203": {"suoxie": "YHXDNHH", "name": "\u94f6\u6cb3\u65b0\u52a8\u80fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEXINDONGNENGHUNHE"}, "007204": {"suoxie": "YHMYZJXQDIIA", "name": "\u94f6\u534e\u7f8e\u5143\u503a\u7cbe\u9009(QDII)A", "style": "QDII", "pinyin": "YINHUAMEIYUANZHAIJINGXUANQDIIA"}, "007205": {"suoxie": "YHMYZJXQDIIC", "name": "\u94f6\u534e\u7f8e\u5143\u503a\u7cbe\u9009(QDII)C", "style": "QDII", "pinyin": "YINHUAMEIYUANZHAIJINGXUANQDIIC"}, "007206": {"suoxie": "YHFHSGYDKZ", "name": "\u94f6\u534e\u4e30\u534e\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAFENGHUASANGEYUEDINGKAIZHAI"}, "007207": {"suoxie": "HXCYSNDKHH", "name": "\u534e\u590f\u5e38\u9633\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHANGYANGSANNIANDINGKAIHUNHE"}, "007208": {"suoxie": "ZYZZ13NJQYQ20A", "name": "\u4e2d\u90ae\u4e2d\u503a1-3\u5e74\u4e45\u671f\u592e\u4f0120A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUZHONGZHAI13NIANJIUQIYANGQI20A"}, "007209": {"suoxie": "ZYZZ13NJQYQ20C", "name": "\u4e2d\u90ae\u4e2d\u503a1-3\u5e74\u4e45\u671f\u592e\u4f0120C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUZHONGZHAI13NIANJIUQIYANGQI20C"}, "007210": {"suoxie": "HSRFDZC", "name": "\u534e\u5546\u745e\u4e30\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUASHANGRUIFENGDUANZHAIC"}, "007211": {"suoxie": "HAZDZZQE", "name": "\u6c47\u5b89\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANZHONGDUANZHAIZHAIQUANE"}, "007212": {"suoxie": "SXZQYT3GYDK", "name": "\u5c71\u897f\u8bc1\u5238\u88d5\u6cf03\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANXIZHENGQUANYUTAI3GEYUEDINGKAI"}, "007213": {"suoxie": "HAAP6GYDKZ", "name": "\u534e\u5b89\u5b89\u5e736\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANPING6GEYUEDINGKAIZHAI"}, "007214": {"suoxie": "GTHFCZZQ", "name": "\u56fd\u6cf0\u60e0\u4e30\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIFENGCHUNZHAIZHAIQUAN"}, "007215": {"suoxie": "GSABTRCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u8363\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIRONGCHUNZHAIZHAIQUAN"}, "007216": {"suoxie": "ZSZHYQGGXC", "name": "\u6d59\u5546\u4e2d\u534e\u9884\u671f\u9ad8\u80a1\u606fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHONGHUAYUQIGAOGUXIC"}, "007217": {"suoxie": "ZSZNHYYXHHC", "name": "\u6d59\u5546\u667a\u80fd\u884c\u4e1a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHINENGHANGYEYOUXUANHUNHEC"}, "007218": {"suoxie": "FCTMZDZA", "name": "\u8702\u5de2\u6dfb\u5e42\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FENGCHAOTIANMIZHONGDUANZHAIA"}, "007219": {"suoxie": "FCTMZDZC", "name": "\u8702\u5de2\u6dfb\u5e42\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FENGCHAOTIANMIZHONGDUANZHAIC"}, "007220": {"suoxie": "THHXSGYDKZ", "name": "\u5929\u5f18\u534e\u4eab\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGHUAXIANGSANGEYUEDINGKAIZHAI"}, "007221": {"suoxie": "STMGJCYLSNFOF", "name": "\u4e0a\u6295\u6469\u6839\u9526\u7a0b\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGTOUMOGENJINCHENGYANGLAOSANNIANFOF"}, "007223": {"suoxie": "GYZZ500ETFLJC", "name": "\u5de5\u94f6\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENG500ETFLIANJIEC"}, "007224": {"suoxie": "ZSHQ3GYDKZA", "name": "\u6d59\u5546\u60e0\u6cc93\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIQUAN3GEYUEDINGKAIZHAIA"}, "007225": {"suoxie": "ZSHQ3GYDKZC", "name": "\u6d59\u5546\u60e0\u6cc93\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIQUAN3GEYUEDINGKAIZHAIC"}, "007226": {"suoxie": "HFTZDZZQC", "name": "\u6d77\u5bcc\u901a\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAIFUTONGZHONGDUANZHAIZHAIQUANC"}, "007227": {"suoxie": "HFTZDZZQA", "name": "\u6d77\u5bcc\u901a\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAIFUTONGZHONGDUANZHAIZHAIQUANA"}, "007228": {"suoxie": "HAZZ710NGKZA", "name": "\u534e\u5b89\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI710NIANGUOKAIZHAIA"}, "007229": {"suoxie": "HAZZ710NGKZC", "name": "\u534e\u5b89\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI710NIANGUOKAIZHAIC"}, "007230": {"suoxie": "XQHS300ZSLOFC", "name": "\u5174\u5168\u6caa\u6df1300\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGQUANHUSHEN300ZHISHULOFC"}, "007231": {"suoxie": "GTMAYL2040FOF", "name": "\u56fd\u6cf0\u6c11\u5b89\u517b\u80012040(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIMINANYANGLAO2040FOF"}, "007232": {"suoxie": "WJPHYLMBSNFOF", "name": "\u4e07\u5bb6\u5e73\u8861\u517b\u8001\u76ee\u6807\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "WANJIAPINGHENGYANGLAOMUBIAOSANNIANFOF"}, "007233": {"suoxie": "JYXYHHE", "name": "\u91d1\u9e70\u946b\u76ca\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGXINYIHUNHEE"}, "007234": {"suoxie": "BSYSQYLOFC", "name": "\u535a\u65f6\u4f18\u52bf\u4f01\u4e1a(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYOUSHIQIYELOFC"}, "007235": {"suoxie": "GFJLZQLOFC", "name": "\u5e7f\u53d1\u805a\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJULIZHAIQUANLOFC"}, "007238": {"suoxie": "PAYL2035FOFA", "name": "\u5e73\u5b89\u517b\u80012035(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANGLAO2035FOFA"}, "007239": {"suoxie": "PAYL2035FOFC", "name": "\u5e73\u5b89\u517b\u80012035(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANGLAO2035FOFC"}, "007240": {"suoxie": "SWLXATRLZDZC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u745e\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHENWANLINGXINANTAIRUILIZHONGDUANZHAIC"}, "007241": {"suoxie": "ZOYJYL2050WNCYFOFA", "name": "\u4e2d\u6b27\u9884\u89c1\u517b\u80012050\u4e94\u5e74\u6301\u6709(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUJIANYANGLAO2050WUNIANCHIYOUFOFA"}, "007242": {"suoxie": "ZOYJYL2050WNCYFOFC", "name": "\u4e2d\u6b27\u9884\u89c1\u517b\u80012050\u4e94\u5e74\u6301\u6709(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUJIANYANGLAO2050WUNIANCHIYOUFOFC"}, "007243": {"suoxie": "AXHXJZLHHA", "name": "\u5b89\u4fe1\u6838\u5fc3\u7ade\u4e89\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINHEXINJINGZHENGLIHUNHEA"}, "007244": {"suoxie": "AXHXJZLHHC", "name": "\u5b89\u4fe1\u6838\u5fc3\u7ade\u4e89\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINHEXINJINGZHENGLIHUNHEC"}, "007245": {"suoxie": "AXXRXZDZA", "name": "\u5b89\u4fe1\u946b\u65e5\u4eab\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINXINRIXIANGZHONGDUANZHAIA"}, "007246": {"suoxie": "AXXRXZDZC", "name": "\u5b89\u4fe1\u946b\u65e5\u4eab\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINXINRIXIANGZHONGDUANZHAIC"}, "007247": {"suoxie": "YFDHZWJYLHHFOF", "name": "\u6613\u65b9\u8fbe\u6c47\u667a\u7a33\u5065\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAHUIZHIWENJIANYANGLAOHUNHEFOF"}, "007249": {"suoxie": "GFJHYLSNCYHHFOF", "name": "\u5e7f\u53d1\u5747\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAJUNHENGYANGLAOSANNIANCHIYOUHUNHEFOF"}, "007250": {"suoxie": "GFYL2050HHFOF", "name": "\u5e7f\u53d1\u517b\u80012050\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAYANGLAO2050HUNHEFOF"}, "007251": {"suoxie": "GFRXWJZLHHA", "name": "\u5e7f\u53d1\u777f\u4eab\u7a33\u5065\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFARUIXIANGWENJIANZENGLIHUNHEA"}, "007252": {"suoxie": "GFZZNFZZZSA", "name": "\u5e7f\u53d1\u4e2d\u503a\u519c\u53d1\u503a\u603b\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAINONGFAZHAIZONGZHISHUA"}, "007253": {"suoxie": "GFZZNFZZZSC", "name": "\u5e7f\u53d1\u4e2d\u503a\u519c\u53d1\u503a\u603b\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAINONGFAZHAIZONGZHISHUC"}, "007254": {"suoxie": "GFJHJZHH", "name": "\u5e7f\u53d1\u5747\u8861\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUNHENGJIAZHIHUNHE"}, "007255": {"suoxie": "HBWJYLFOF", "name": "\u534e\u5b9d\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUABAOWENJIANYANGLAOFOF"}, "007256": {"suoxie": "GFHYSGYDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u9633\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIYANGSANGEYUEDINGQIKAIFANGZHAIQUAN"}, "007257": {"suoxie": "KSFHHA", "name": "\u51ef\u77f3\u6ca3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHIFENGHUNHEA"}, "007258": {"suoxie": "KSFHHC", "name": "\u51ef\u77f3\u6ca3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "KAISHIFENGHUNHEC"}, "007259": {"suoxie": "MSJY13NNFHZ", "name": "\u6c11\u751f\u52a0\u94f61-3\u5e74\u519c\u53d1\u884c\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYIN13NIANNONGFAHANGZHAI"}, "007260": {"suoxie": "GTRYSQCZ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u797a\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNQICHUNZHAI"}, "007261": {"suoxie": "RTXFSJHH", "name": "\u878d\u901a\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGXIAOFEISHENGJIHUNHE"}, "007262": {"suoxie": "DFHJLZQA", "name": "\u4e1c\u65b9\u7ea2\u805a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGJULIZHAIQUANA"}, "007263": {"suoxie": "DFHJLZQC", "name": "\u4e1c\u65b9\u7ea2\u805a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGHONGJULIZHAIQUANC"}, "007264": {"suoxie": "RBZGDJXYZA", "name": "\u4eba\u4fdd\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOZHONGGAODENGJIXINYONGZHAIA"}, "007265": {"suoxie": "RBZGDJXYZC", "name": "\u4eba\u4fdd\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOZHONGGAODENGJIXINYONGZHAIC"}, "007266": {"suoxie": "JSXTYDQHHA", "name": "\u5609\u5b9e\u65b0\u6dfb\u76ca\u5b9a\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTIANYIDINGQIHUNHEA"}, "007267": {"suoxie": "JSXTYDQHHC", "name": "\u5609\u5b9e\u65b0\u6dfb\u76ca\u5b9a\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTIANYIDINGQIHUNHEC"}, "007268": {"suoxie": "SXZQYR6GYDKZA", "name": "\u5c71\u897f\u8bc1\u5238\u88d5\u777f6\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANXIZHENGQUANYURUI6GEYUEDINGKAIZHAIA"}, "007269": {"suoxie": "SXZQYR6GYDKZC", "name": "\u5c71\u897f\u8bc1\u5238\u88d5\u777f6\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANXIZHENGQUANYURUI6GEYUEDINGKAIZHAIC"}, "007271": {"suoxie": "PHYL2045HHFOF", "name": "\u9e4f\u534e\u517b\u80012045\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGHUAYANGLAO2045HUNHEFOF"}, "007272": {"suoxie": "JSWJYLMBSNHHFOF", "name": "\u666f\u987a\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e09\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNWENJIANYANGLAOMUBIAOSANNIANHUNHEFOF"}, "007273": {"suoxie": "PHCLWJYLHHFOF", "name": "\u9e4f\u534e\u957f\u4e50\u7a33\u5065\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUACHANGLEWENJIANYANGLAOHUNHEFOF"}, "007274": {"suoxie": "JSCCYLHHFOF", "name": "\u666f\u987a\u957f\u57ce\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGYANGLAOHUNHEFOF"}, "007275": {"suoxie": "YHHS300ZSZQA", "name": "\u94f6\u6cb3\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEHUSHEN300ZHISHUZENGQIANGA"}, "007276": {"suoxie": "YHHS300ZSZQC", "name": "\u94f6\u6cb3\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEHUSHEN300ZHISHUZENGQIANGC"}, "007277": {"suoxie": "HSQHXFSJHH", "name": "\u6052\u751f\u524d\u6d77\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGSHENGQIANHAIXIAOFEISHENGJIHUNHE"}, "007278": {"suoxie": "GTXFSGYDKZ", "name": "\u56fd\u6cf0\u5174\u5bcc\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIXINGFUSANGEYUEDINGKAIZHAI"}, "007279": {"suoxie": "YYZLZQ", "name": "\u6c38\u8d62\u4f17\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGLIZHAIQUAN"}, "007280": {"suoxie": "STMGRBJXGPQDII", "name": "\u4e0a\u6295\u6469\u6839\u65e5\u672c\u7cbe\u9009\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "SHANGTOUMOGENRIBENJINGXUANGUPIAOQDII"}, "007281": {"suoxie": "JHXFSJHH", "name": "\u5609\u5408\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEXIAOFEISHENGJIHUNHE"}, "007282": {"suoxie": "HXDCZQA", "name": "\u534e\u590f\u9f0e\u6df3\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGCHUNZHAIQUANA"}, "007283": {"suoxie": "HXDCZQC", "name": "\u534e\u590f\u9f0e\u6df3\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGCHUNZHAIQUANC"}, "007284": {"suoxie": "GYZZ15NJCKHA", "name": "\u5de5\u94f6\u4e2d\u503a1-5\u5e74\u8fdb\u51fa\u53e3\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINZHONGZHAI15NIANJINCHUKOUHANGA"}, "007285": {"suoxie": "GYZZ15NJCKHC", "name": "\u5de5\u94f6\u4e2d\u503a1-5\u5e74\u8fdb\u51fa\u53e3\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINZHONGZHAI15NIANJINCHUKOUHANGC"}, "007286": {"suoxie": "ZYCZYLSGYDKZ", "name": "\u4e2d\u90ae\u7eaf\u503a\u88d5\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIYULISANGEYUEDINGKAIZHAI"}, "007287": {"suoxie": "HXZYXFZTGPFQSA", "name": "\u5408\u7166\u667a\u8fdc\u6d88\u8d39\u4e3b\u9898\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "HEXUZHIYUANXIAOFEIZHUTIGUPIAOFAQISHIA"}, "007288": {"suoxie": "HXZYXFZTGPFQSC", "name": "\u5408\u7166\u667a\u8fdc\u6d88\u8d39\u4e3b\u9898\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "HEXUZHIYUANXIAOFEIZHUTIGUPIAOFAQISHIC"}, "007289": {"suoxie": "HTFZZ13NNFZA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANNONGFAZHAIA"}, "007290": {"suoxie": "HTFZZ13NNFZC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANNONGFAZHAIC"}, "007291": {"suoxie": "HFJXGGTSHCLHH", "name": "\u6c47\u4e30\u664b\u4fe1\u6e2f\u80a1\u901a\u53cc\u6838\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINGANGGUTONGSHUANGHECELUEHUNHE"}, "007292": {"suoxie": "MSJYXYZQ", "name": "\u6c11\u751f\u52a0\u94f6\u5174\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINGYINGZHAIQUAN"}, "007293": {"suoxie": "CXLXHHC", "name": "\u957f\u4fe1\u5229\u4fe1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIXINHUNHEC"}, "007294": {"suoxie": "CXLXHHE", "name": "\u957f\u4fe1\u5229\u4fe1\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIXINHUNHEE"}, "007295": {"suoxie": "THAYZQA", "name": "\u5929\u5f18\u5b89\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGANYIZHAIQUANA"}, "007296": {"suoxie": "THAYZQC", "name": "\u5929\u5f18\u5b89\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGANYIZHAIQUANC"}, "007297": {"suoxie": "DCYL2040FOFA", "name": "\u5927\u6210\u517b\u80012040(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGYANGLAO2040FOFA"}, "007298": {"suoxie": "DCYL2040FOFC", "name": "\u5927\u6210\u517b\u80012040(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGYANGLAO2040FOFC"}, "007300": {"suoxie": "GLAZZBDTETFLJA", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u534a\u5bfc\u4f53ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGBANDAOTIETFLIANJIEA"}, "007301": {"suoxie": "GLAZZBDTETFLJC", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u534a\u5bfc\u4f53ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGBANDAOTIETFLIANJIEC"}, "007302": {"suoxie": "HBBSCZZQ", "name": "\u534e\u5b9d\u5b9d\u76db\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOSHENGCHUNZHAIZHAIQUAN"}, "007305": {"suoxie": "GLAXKJHH", "name": "\u56fd\u8054\u5b89\u65b0\u79d1\u6280\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANXINKEJIHUNHE"}, "007306": {"suoxie": "HTBRJBMZXA", "name": "\u534e\u6cf0\u67cf\u745e\u57fa\u672c\u9762\u667a\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIBENMIANZHIXUANA"}, "007307": {"suoxie": "HTBRJBMZXC", "name": "\u534e\u6cf0\u67cf\u745e\u57fa\u672c\u9762\u667a\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIBENMIANZHIXUANC"}, "007308": {"suoxie": "HBXFSJHH", "name": "\u534e\u5b9d\u6d88\u8d39\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXIAOFEISHENGJIHUNHE"}, "007309": {"suoxie": "PH910NLLFQSZQA", "name": "\u9e4f\u534e9-10\u5e74\u5229\u7387\u53d1\u8d77\u5f0f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUA910NIANLILVFAQISHIZHAIQUANA"}, "007310": {"suoxie": "YHZSWJYLHHFOF", "name": "\u94f6\u534e\u5c0a\u5c1a\u7a33\u5065\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAZUNSHANGWENJIANYANGLAOHUNHEFOF"}, "007311": {"suoxie": "FZFBTLCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u6dfb\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGTIANLICHUNZHAIA"}, "007312": {"suoxie": "FZFBTLCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u6dfb\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGTIANLICHUNZHAIC"}, "007315": {"suoxie": "HAJYYNCYQZQA", "name": "\u6c47\u5b89\u5609\u76c8\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANJIAYINGYINIANCHIYOUQIZHAIQUANA"}, "007316": {"suoxie": "JYKZZZQA", "name": "\u4ea4\u94f6\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "JIAOYINKEZHUANZHAIZHAIQUANA"}, "007317": {"suoxie": "JYKZZZQC", "name": "\u4ea4\u94f6\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "JIAOYINKEZHUANZHAIZHAIQUANC"}, "007318": {"suoxie": "ZYMFHBHH", "name": "\u4e2d\u94f6\u6c11\u4e30\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINMINFENGHUIBAOHUNHE"}, "007319": {"suoxie": "JSHDZDZZQA", "name": "\u5609\u5b9e\u6c47\u8fbe\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIHUIDAZHONGDUANZHAIZHAIQUANA"}, "007320": {"suoxie": "JSHDZDZZQC", "name": "\u5609\u5b9e\u6c47\u8fbe\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIHUIDAZHONGDUANZHAIZHAIQUANC"}, "007321": {"suoxie": "PHJLZQ", "name": "\u9e4f\u534e\u91d1\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAJINLIZHAIQUAN"}, "007323": {"suoxie": "YYJLZQ", "name": "\u6c38\u8d62\u4e45\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGJIULIZHAIQUAN"}, "007324": {"suoxie": "XYZZ13NGKZZSA", "name": "\u946b\u5143\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHONGZHAI13NIANGUOKAIZHAIZHISHUA"}, "007325": {"suoxie": "XYZZ13NGKZZSC", "name": "\u946b\u5143\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZHONGZHAI13NIANGUOKAIZHAIZHISHUC"}, "007326": {"suoxie": "GTRYXZZHHC", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINZENGZHANGHUNHEC"}, "007327": {"suoxie": "QHLHYHCZA", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u8f89\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGHUICHUNZHAIA"}, "007328": {"suoxie": "ZSTYCZE", "name": "\u62db\u5546\u6dfb\u76c8\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYINGCHUNZHAIE"}, "007329": {"suoxie": "STMGRYCZA", "name": "\u4e0a\u6295\u6469\u6839\u745e\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENRUIYICHUNZHAIA"}, "007330": {"suoxie": "STMGRYCZC", "name": "\u4e0a\u6295\u6469\u6839\u745e\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENRUIYICHUNZHAIC"}, "007331": {"suoxie": "GTHRCZZQ", "name": "\u56fd\u6cf0\u60e0\u878d\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIRONGCHUNZHAIZHAIQUAN"}, "007332": {"suoxie": "JHPSCZA", "name": "\u5609\u5408\u78d0\u6607\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANSHENGCHUNZHAIA"}, "007333": {"suoxie": "JHPSCZC", "name": "\u5609\u5408\u78d0\u6607\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANSHENGCHUNZHAIC"}, "007335": {"suoxie": "ZYZZ13NQNFHZ", "name": "\u4e2d\u94f6\u4e2d\u503a1-3\u5e74\u671f\u519c\u53d1\u884c\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGZHAI13NIANQINONGFAHANGZHAI"}, "007336": {"suoxie": "HAJSCZZQA", "name": "\u6c47\u5b89\u5609\u76db\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIASHENGCHUNZHAIZHAIQUANA"}, "007337": {"suoxie": "HAJSCZZQC", "name": "\u6c47\u5b89\u5609\u76db\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIASHENGCHUNZHAIZHAIQUANC"}, "007338": {"suoxie": "QHLHYHCZC", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u8f89\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGHUICHUNZHAIC"}, "007339": {"suoxie": "YFDHS300ETFLJC", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300ETFLIANJIEC"}, "007340": {"suoxie": "NFKJCXHHA", "name": "\u5357\u65b9\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGKEJICHUANGXINHUNHEA"}, "007341": {"suoxie": "NFKJCXHHC", "name": "\u5357\u65b9\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGKEJICHUANGXINHUNHEC"}, "007342": {"suoxie": "GTRYSZCZZQA", "name": "\u56fd\u6295\u745e\u94f6\u987a\u81fb\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNZHENCHUNZHAIZHAIQUANA"}, "007343": {"suoxie": "JSKJCXHH", "name": "\u5609\u5b9e\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIKEJICHUANGXINHUNHE"}, "007345": {"suoxie": "FGKJCXLHPZHH", "name": "\u5bcc\u56fd\u79d1\u6280\u521b\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOKEJICHUANGXINLINGHUOPEIZHIHUNHE"}, "007346": {"suoxie": "YFDKJCXHH", "name": "\u6613\u65b9\u8fbe\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKEJICHUANGXINHUNHE"}, "007347": {"suoxie": "YYCLZQA", "name": "\u6c38\u8d62\u660c\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHANGLIZHAIQUANA"}, "007348": {"suoxie": "YYCLZQC", "name": "\u6c38\u8d62\u660c\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHANGLIZHAIQUANC"}, "007349": {"suoxie": "HXKJCXHHA", "name": "\u534e\u590f\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAKEJICHUANGXINHUNHEA"}, "007350": {"suoxie": "HXKJCXHHC", "name": "\u534e\u590f\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAKEJICHUANGXINHUNHEC"}, "007351": {"suoxie": "YYTLZQA", "name": "\u6c38\u8d62\u540c\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTONGLIZHAIQUANA"}, "007352": {"suoxie": "YYTLZQC", "name": "\u6c38\u8d62\u540c\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTONGLIZHAIQUANC"}, "007353": {"suoxie": "GYKJCX3NFBHH", "name": "\u5de5\u94f6\u79d1\u6280\u521b\u65b03\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINKEJICHUANGXIN3NIANFENGBIHUNHE"}, "007354": {"suoxie": "CJHXGGTLHGPA", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u91cf\u5316\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGLIANGHUAGUPIAOA"}, "007355": {"suoxie": "HTFKJCXHHA", "name": "\u6c47\u6dfb\u5bcc\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUKEJICHUANGXINHUNHEA"}, "007356": {"suoxie": "HTFKJCXHHC", "name": "\u6c47\u6dfb\u5bcc\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUKEJICHUANGXINHUNHEC"}, "007357": {"suoxie": "CJHXGGTLHGPC", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u91cf\u5316\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGLIANGHUAGUPIAOC"}, "007360": {"suoxie": "YFDZDQMYZQDIIARMB", "name": "\u6613\u65b9\u8fbe\u4e2d\u77ed\u671f\u7f8e\u5143\u503a(QDII)A\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "YIFANGDAZHONGDUANQIMEIYUANZHAIQDIIARENMINBI"}, "007361": {"suoxie": "YFDZDQMYZQDIICRMB", "name": "\u6613\u65b9\u8fbe\u4e2d\u77ed\u671f\u7f8e\u5143\u503a(QDII)C\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "YIFANGDAZHONGDUANQIMEIYUANZHAIQDIICRENMINBI"}, "007362": {"suoxie": "YFDZDQMYZQDIIAMYXH", "name": "\u6613\u65b9\u8fbe\u4e2d\u77ed\u671f\u7f8e\u5143\u503a(QDII)A\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "YIFANGDAZHONGDUANQIMEIYUANZHAIQDIIAMEIYUANXIANHUI"}, "007363": {"suoxie": "YFDZDQMYZQDIICMYXH", "name": "\u6613\u65b9\u8fbe\u4e2d\u77ed\u671f\u7f8e\u5143\u503a(QDII)C\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "YIFANGDAZHONGDUANQIMEIYUANZHAIQDIICMEIYUANXIANHUI"}, "007364": {"suoxie": "YFDZZ13NZJZA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI13NIANZHENGJINZHAIA"}, "007365": {"suoxie": "YFDZZ13NZJZC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI13NIANZHENGJINZHAIC"}, "007366": {"suoxie": "YFDZZ35NZJRZZSA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a3-5\u5e74\u653f\u91d1\u878d\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI35NIANZHENGJINRONGZHAIZHISHUA"}, "007367": {"suoxie": "YFDZZ35NZJRZZSC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a3-5\u5e74\u653f\u91d1\u878d\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI35NIANZHENGJINRONGZHAIZHISHUC"}, "007368": {"suoxie": "ZSHGSJXHHA", "name": "\u6d59\u5546\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUGANGSHENJINGXUANHUNHEA"}, "007369": {"suoxie": "ZSHGSJXHHC", "name": "\u6d59\u5546\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUGANGSHENJINGXUANHUNHEC"}, "007370": {"suoxie": "HAAJDK", "name": "\u534e\u5b89\u5b89\u5609\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANJIADINGKAI"}, "007371": {"suoxie": "GLAZRZCXJRZCZA", "name": "\u56fd\u8054\u5b89\u589e\u745e\u653f\u7b56\u6027\u91d1\u878d\u503a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGRUIZHENGCEXINGJINRONGZHAICHUNZHAIA"}, "007372": {"suoxie": "GLAZRZCXJRZCZC", "name": "\u56fd\u8054\u5b89\u589e\u745e\u653f\u7b56\u6027\u91d1\u878d\u503a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGRUIZHENGCEXINGJINRONGZHAICHUNZHAIC"}, "007373": {"suoxie": "YYZLZQ", "name": "\u6c38\u8d62\u5353\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHUOLIZHAIQUAN"}, "007374": {"suoxie": "YYCLZQ", "name": "\u6c38\u8d62\u6df3\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGCHUNLIZHAIQUAN"}, "007375": {"suoxie": "XBLDJL6GYDKZA", "name": "\u897f\u90e8\u5229\u5f97\u805a\u52296\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJULI6GEYUEDINGKAIZHAIA"}, "007376": {"suoxie": "XBLDJL6GYDKZC", "name": "\u897f\u90e8\u5229\u5f97\u805a\u52296\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJULI6GEYUEDINGKAIZHAIC"}, "007377": {"suoxie": "XBLDJXYNDKZQA", "name": "\u897f\u90e8\u5229\u5f97\u805a\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJUXIANGYINIANDINGKAIZHAIQUANA"}, "007378": {"suoxie": "XBLDJXYNDKZQC", "name": "\u897f\u90e8\u5229\u5f97\u805a\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJUXIANGYINIANDINGKAIZHAIQUANC"}, "007379": {"suoxie": "YFDSZ50ETFLJJJA", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150ETF\u8054\u63a5\u57fa\u91d1A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ETFLIANJIEJIJINA"}, "007380": {"suoxie": "YFDSZ50ETFLJJJC", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150ETF\u8054\u63a5\u57fa\u91d1C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ETFLIANJIEJIJINC"}, "007381": {"suoxie": "GRRXXFYXHHA", "name": "\u56fd\u878d\u878d\u4fe1\u6d88\u8d39\u4e25\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUORONGRONGXINXIAOFEIYANXUANHUNHEA"}, "007382": {"suoxie": "GRRXXFYXHHC", "name": "\u56fd\u878d\u878d\u4fe1\u6d88\u8d39\u4e25\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUORONGRONGXINXIAOFEIYANXUANHUNHEC"}, "007383": {"suoxie": "GRWYZQA", "name": "\u56fd\u878d\u7a33\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUORONGWENYIZHAIQUANA"}, "007384": {"suoxie": "GRWYZQC", "name": "\u56fd\u878d\u7a33\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUORONGWENYIZHAIQUANC"}, "007385": {"suoxie": "HTBXAYDKHH", "name": "\u534e\u6cf0\u4fdd\u5174\u5b89\u76c8\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAOXINGANYINGDINGKAIHUNHE"}, "007386": {"suoxie": "ZSZZ500ZSZQC", "name": "\u6d59\u5546\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHONGZHENG500ZHISHUZENGQIANGC"}, "007387": {"suoxie": "RTTHHHC", "name": "\u878d\u901a\u901a\u6167\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGTONGHUIHUNHEC"}, "007388": {"suoxie": "STMGYJQDA", "name": "\u4e0a\u6295\u6469\u6839\u7814\u7a76\u9a71\u52a8A", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENYANJIUQUDONGA"}, "007389": {"suoxie": "STMGYJQDC", "name": "\u4e0a\u6295\u6469\u6839\u7814\u7a76\u9a71\u52a8C", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENYANJIUQUDONGC"}, "007390": {"suoxie": "SYZZ13NNFHZQZS", "name": "\u4e0a\u94f6\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHU"}, "007391": {"suoxie": "SWLXATFLZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIFENGLIZHAIQUANA"}, "007392": {"suoxie": "SWLXATFLZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIFENGLIZHAIQUANC"}, "007393": {"suoxie": "SYWLSHLHPZHHA", "name": "\u4e0a\u94f6\u672a\u6765\u751f\u6d3b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGYINWEILAISHENGHUOLINGHUOPEIZHIHUNHEA"}, "007394": {"suoxie": "DXXCDZZQA", "name": "\u4e1c\u5174\u5174\u8d22\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGXINGXINGCAIDUANZHAIZHAIQUANA"}, "007395": {"suoxie": "DXXCDZZQC", "name": "\u4e1c\u5174\u5174\u8d22\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGXINGXINGCAIDUANZHAIZHAIQUANC"}, "007396": {"suoxie": "GFJHCZ", "name": "\u5e7f\u53d1\u666f\u8f89\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGHUICHUNZHAI"}, "007397": {"suoxie": "HBHGSZGZQLOFC", "name": "\u534e\u5b9d\u6caa\u6e2f\u6df1\u4e2d\u56fd\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOHUGANGSHENZHONGGUOZENGQIANGLOFC"}, "007398": {"suoxie": "XQPWZLZQC", "name": "\u5174\u5168\u78d0\u7a33\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGQUANPANWENZENGLIZHAIQUANC"}, "007401": {"suoxie": "PYYHWJYLYNFOFA", "name": "\u6d66\u94f6\u9890\u548c\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINYIHEWENJIANYANGLAOYINIANFOFA"}, "007402": {"suoxie": "PYYHWJYLYNFOFC", "name": "\u6d66\u94f6\u9890\u548c\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINYIHEWENJIANYANGLAOYINIANFOFC"}, "007404": {"suoxie": "HBHS300ZSZQC", "name": "\u534e\u5b9d\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOHUSHEN300ZHISHUZENGQIANGC"}, "007405": {"suoxie": "HBZZ100ZSC", "name": "\u534e\u5b9d\u4e2d\u8bc1100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG100ZHISHUC"}, "007406": {"suoxie": "YHRXZQ", "name": "\u94f6\u6cb3\u777f\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHERUIXINZHAIQUAN"}, "007407": {"suoxie": "NYYL2035HHFOF", "name": "\u519c\u94f6\u517b\u80012035\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NONGYINYANGLAO2035HUNHEFOF"}, "007408": {"suoxie": "PYCKZQA", "name": "\u9e4f\u626c\u6df3\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNKAIZHAIQUANA"}, "007409": {"suoxie": "PYCKZQC", "name": "\u9e4f\u626c\u6df3\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNKAIZHAIQUANC"}, "007410": {"suoxie": "PYASZZGGXETFLJA", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u9ad8\u80a1\u606fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGGAOGUXIETFLIANJIEA"}, "007411": {"suoxie": "PYASZZGGXETFLJC", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u9ad8\u80a1\u606fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGGAOGUXIETFLIANJIEC"}, "007412": {"suoxie": "JSCCJYCZHH", "name": "\u666f\u987a\u957f\u57ce\u7ee9\u4f18\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJIYOUCHENGZHANGHUNHE"}, "007413": {"suoxie": "CCZZ500ZSZQC", "name": "\u957f\u57ce\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGZHONGZHENG500ZHISHUZENGQIANGC"}, "007414": {"suoxie": "CJAYZDZLGYDK", "name": "\u957f\u6c5f\u5b89\u76c8\u4e2d\u77ed\u503a\u516d\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGJIANGANYINGZHONGDUANZHAILIUGEYUEDINGKAI"}, "007415": {"suoxie": "NFZYHHA", "name": "\u5357\u65b9\u81f4\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGZHIYUANHUNHEA"}, "007416": {"suoxie": "NFZYHHC", "name": "\u5357\u65b9\u81f4\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGZHIYUANHUNHEC"}, "007417": {"suoxie": "TKXYJXZQA", "name": "\u6cf0\u5eb7\u4fe1\u7528\u7cbe\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGXINYONGJINGXUANZHAIQUANA"}, "007418": {"suoxie": "TKXYJXZQC", "name": "\u6cf0\u5eb7\u4fe1\u7528\u7cbe\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGXINYONGJINGXUANZHAIQUANC"}, "007419": {"suoxie": "GSABTHCZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u5f18\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIHONGCHUNZHAIZHAIQUAN"}, "007420": {"suoxie": "HADX3GYDKZ", "name": "\u534e\u5b89\u9f0e\u4fe13\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANDINGXIN3GEYUEDINGKAIZHAI"}, "007423": {"suoxie": "XBLDJHHHA", "name": "\u897f\u90e8\u5229\u5f97\u805a\u79be\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJUHEHUNHEA"}, "007424": {"suoxie": "XBLDJHHHC", "name": "\u897f\u90e8\u5229\u5f97\u805a\u79be\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJUHEHUNHEC"}, "007425": {"suoxie": "ZSHJZGDJSGYA", "name": "\u6d59\u5546\u6c47\u91d1\u4e2d\u9ad8\u7b49\u7ea7\u4e09\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINZHONGGAODENGJISANGEYUEA"}, "007426": {"suoxie": "ZSHJJYZDZA", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u76c8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINJUYINGZHONGDUANZHAIA"}, "007427": {"suoxie": "YYKLZQ", "name": "\u6c38\u8d62\u51ef\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGKAILIZHAIQUAN"}, "007428": {"suoxie": "CXFRLNDKZQC", "name": "\u957f\u4fe1\u5bcc\u745e\u4e24\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFURUILIANGNIANDINGKAIZHAIQUANC"}, "007429": {"suoxie": "PYCY6GYDKZA", "name": "\u9e4f\u626c\u6df3\u76c86\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNYING6GEYUEDINGKAIZHAIA"}, "007430": {"suoxie": "PYCY6GYDKZC", "name": "\u9e4f\u626c\u6df3\u76c86\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNYING6GEYUEDINGKAIZHAIC"}, "007431": {"suoxie": "ZSZJFHLJ", "name": "\u6d59\u5546\u4e4b\u6c5f\u51e4\u51f0\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHIJIANGFENGHUANGLIANJIE"}, "007432": {"suoxie": "HTBXJY63GYDKZ", "name": "\u534e\u6cf0\u4fdd\u5174\u4e45\u76c863\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGJIUYING63GEYUEDINGKAIZHAI"}, "007433": {"suoxie": "XYHFZCXJRZ", "name": "\u5174\u94f6\u5408\u4e30\u653f\u7b56\u6027\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHEFENGZHENGCEXINGJINRONGZHAI"}, "007435": {"suoxie": "HBBYZQ", "name": "\u534e\u5b9d\u5b9d\u6021\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOYIZHAIQUAN"}, "007439": {"suoxie": "DHKJDLA", "name": "\u4e1c\u6d77\u79d1\u6280\u52a8\u529bA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGHAIKEJIDONGLIA"}, "007440": {"suoxie": "NFXYZQA", "name": "\u5357\u65b9\u65ed\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXUYUANZHAIQUANA"}, "007441": {"suoxie": "NFXYZQC", "name": "\u5357\u65b9\u65ed\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXUYUANZHAIQUANC"}, "007442": {"suoxie": "ZSHJZGDJSGYC", "name": "\u6d59\u5546\u6c47\u91d1\u4e2d\u9ad8\u7b49\u7ea7\u4e09\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINZHONGGAODENGJISANGEYUEC"}, "007443": {"suoxie": "ZSHJJYZDZC", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u76c8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINJUYINGZHONGDUANZHAIC"}, "007445": {"suoxie": "GTRYSYZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNYUEZHAIQUAN"}, "007446": {"suoxie": "ZOZQHBZQLOFC", "name": "\u4e2d\u6b27\u589e\u5f3a\u56de\u62a5\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUZENGQIANGHUIBAOZHAIQUANLOFC"}, "007447": {"suoxie": "PAHTCZ", "name": "\u5e73\u5b89\u60e0\u6cf0\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUITAICHUNZHAI"}, "007448": {"suoxie": "CXHS300ZSZQC", "name": "\u957f\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGXINHUSHEN300ZHISHUZENGQIANGC"}, "007449": {"suoxie": "XQDWJZHHA", "name": "\u5174\u5168\u591a\u7ef4\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANDUOWEIJIAZHIHUNHEA"}, "007450": {"suoxie": "XQDWJZHHC", "name": "\u5174\u5168\u591a\u7ef4\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANDUOWEIJIAZHIHUNHEC"}, "007451": {"suoxie": "YFDHX3GYDKZ", "name": "\u6613\u65b9\u8fbe\u6052\u51743\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGXING3GEYUEDINGKAIZHAI"}, "007454": {"suoxie": "MSJYJYZQ", "name": "\u6c11\u751f\u52a0\u94f6\u5609\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINJIAYINGZHAIQUAN"}, "007455": {"suoxie": "FGLCJXGPQDIIRMB", "name": "\u5bcc\u56fd\u84dd\u7b79\u7cbe\u9009\u80a1\u7968(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "FUGUOLANCHOUJINGXUANGUPIAOQDIIRENMINBI"}, "007456": {"suoxie": "HTF90TDZA", "name": "\u6c47\u6dfb\u5bcc90\u5929\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFU90TIANDUANZHAIA"}, "007457": {"suoxie": "HTF90TDZB", "name": "\u6c47\u6dfb\u5bcc90\u5929\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFU90TIANDUANZHAIB"}, "007458": {"suoxie": "HTF90TDZC", "name": "\u6c47\u6dfb\u5bcc90\u5929\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFU90TIANDUANZHAIC"}, "007459": {"suoxie": "ZSHRCZZQ", "name": "\u6d59\u5546\u60e0\u777f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIRUICHUNZHAIZHAIQUAN"}, "007460": {"suoxie": "HACZCXHH", "name": "\u534e\u5b89\u6210\u957f\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHENGZHANGCHUANGXINHUNHE"}, "007461": {"suoxie": "DBRHZQA", "name": "\u5fb7\u90a6\u9510\u6cd3\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIHONGZHAIQUANA"}, "007462": {"suoxie": "DBRHZQC", "name": "\u5fb7\u90a6\u9510\u6cd3\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIHONGZHAIQUANC"}, "007463": {"suoxie": "DHKJDLC", "name": "\u4e1c\u6d77\u79d1\u6280\u52a8\u529bC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGHAIKEJIDONGLIC"}, "007464": {"suoxie": "JYCYB50ZSA", "name": "\u4ea4\u94f6\u521b\u4e1a\u677f50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINCHUANGYEBAN50ZHISHUA"}, "007465": {"suoxie": "JYCYB50ZSC", "name": "\u4ea4\u94f6\u521b\u4e1a\u677f50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINCHUANGYEBAN50ZHISHUC"}, "007466": {"suoxie": "HTBRZZDBDETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u4f4e\u6ce2\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGDIBODONGETFLIANJIEA"}, "007467": {"suoxie": "HTBRZZDBDETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u4f4e\u6ce2\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGDIBODONGETFLIANJIEC"}, "007468": {"suoxie": "ZXJTJXHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUJINGXUANHUNHEA"}, "007469": {"suoxie": "ZXJTJXHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUJINGXUANHUNHEC"}, "007470": {"suoxie": "BDSBZHGPA", "name": "\u535a\u9053\u53c1\u4f70\u667a\u822a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOSANBAIZHIHANGGUPIAOA"}, "007471": {"suoxie": "BDSBZHGPC", "name": "\u535a\u9053\u53c1\u4f70\u667a\u822a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOSANBAIZHIHANGGUPIAOC"}, "007472": {"suoxie": "HXCLCETFLJA", "name": "\u534e\u590f\u521b\u84dd\u7b79ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGLANCHOUETFLIANJIEA"}, "007473": {"suoxie": "HXCLCETFLJC", "name": "\u534e\u590f\u521b\u84dd\u7b79ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGLANCHOUETFLIANJIEC"}, "007474": {"suoxie": "HXCCZETFLJA", "name": "\u534e\u590f\u521b\u6210\u957fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGCHENGZHANGETFLIANJIEA"}, "007475": {"suoxie": "HXCCZETFLJC", "name": "\u534e\u590f\u521b\u6210\u957fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGCHENGZHANGETFLIANJIEC"}, "007478": {"suoxie": "ZJHTDKZQ", "name": "\u4e2d\u52a0\u6052\u6cf0\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAHENGTAIDINGKAIZHAIQUAN"}, "007480": {"suoxie": "ZJYXCZZQA", "name": "\u4e2d\u52a0\u4f18\u4eab\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYOUXIANGCHUNZHAIZHAIQUANA"}, "007481": {"suoxie": "HXYXJKHH", "name": "\u534e\u590f\u9038\u4eab\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAYIXIANGJIANKANGHUNHE"}, "007482": {"suoxie": "YYZYCZSGY", "name": "\u6c38\u8d62\u667a\u76ca\u7eaf\u503a\u4e09\u4e2a\u6708", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHIYICHUNZHAISANGEYUE"}, "007484": {"suoxie": "XDAYHXKJHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6838\u5fc3\u79d1\u6280\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINHEXINKEJIHUNHE"}, "007485": {"suoxie": "BSZZ35NGKHA", "name": "\u535a\u65f6\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35NIANGUOKAIHANGA"}, "007486": {"suoxie": "BSZZ35NGKHC", "name": "\u535a\u65f6\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35NIANGUOKAIHANGC"}, "007488": {"suoxie": "WJMAZL12GYDKZA", "name": "\u4e07\u5bb6\u6c11\u5b89\u589e\u522912\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAMINANZENGLI12GEYUEDINGKAIZHAIA"}, "007489": {"suoxie": "WJMAZL12GYDKZC", "name": "\u4e07\u5bb6\u6c11\u5b89\u589e\u522912\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAMINANZENGLI12GEYUEDINGKAIZHAIC"}, "007490": {"suoxie": "NFXXCXHHA", "name": "\u5357\u65b9\u4fe1\u606f\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINXICHUANGXINHUNHEA"}, "007491": {"suoxie": "NFXXCXHHC", "name": "\u5357\u65b9\u4fe1\u606f\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINXICHUANGXINHUNHEC"}, "007492": {"suoxie": "SYZCXJRZ", "name": "\u4e0a\u94f6\u653f\u7b56\u6027\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINZHENGCEXINGJINRONGZHAI"}, "007493": {"suoxie": "ZQCYZXHHA", "name": "\u6731\u96c0\u4ea7\u4e1a\u81fb\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUECHANYEZHENXUANHUNHEA"}, "007494": {"suoxie": "ZQCYZXHHC", "name": "\u6731\u96c0\u4ea7\u4e1a\u81fb\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUECHANYEZHENXUANHUNHEC"}, "007495": {"suoxie": "XYZZ13NZJZC", "name": "\u5174\u4e1a\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHAI13NIANZHENGJINZHAIC"}, "007496": {"suoxie": "NYHLFZSNDKZ", "name": "\u519c\u94f6\u6c47\u7406\u4e30\u6cfd\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIFENGZESANNIANDINGKAIZHAI"}, "007497": {"suoxie": "ZGJZLDLHPZHH", "name": "\u4e2d\u5e9a\u4ef7\u503c\u7075\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGGENGJIAZHILINGDONGLINGHUOPEIZHIHUNHE"}, "007498": {"suoxie": "ZGJZLDLHPZHH", "name": "\u4e2d\u5e9a\u4ef7\u503c\u7075\u52a8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGGENGJIAZHILINGDONGLINGHUOPEIZHIHUNHE"}, "007499": {"suoxie": "GDFGLDHHC", "name": "\u5149\u5927\u98ce\u683c\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAFENGGELUNDONGHUNHEC"}, "007500": {"suoxie": "PHZCDKZFQS", "name": "\u9e4f\u534e\u5c0a\u8bda\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNCHENGDINGKAIZHAIFAQISHI"}, "007501": {"suoxie": "WJKC3NFBHHC", "name": "\u4e07\u5bb6\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAKECHUANG3NIANFENGBIHUNHEC"}, "007502": {"suoxie": "QHKYYHHHC", "name": "\u524d\u6d77\u5f00\u6e90\u88d5\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAIKAIYUANYUHEHUNHEC"}, "007505": {"suoxie": "HXAHJJLCGPA", "name": "\u534e\u590fAH\u7ecf\u6d4e\u84dd\u7b79\u80a1\u7968A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAAHJINGJILANCHOUGUPIAOA"}, "007506": {"suoxie": "HXAHJJLCGPC", "name": "\u534e\u590fAH\u7ecf\u6d4e\u84dd\u7b79\u80a1\u7968C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAAHJINGJILANCHOUGUPIAOC"}, "007507": {"suoxie": "DCZZ35NGKZA", "name": "\u5927\u6210\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGZHONGZHAI35NIANGUOKAIZHAIA"}, "007508": {"suoxie": "DCZZ35NGKZC", "name": "\u5927\u6210\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGZHONGZHAI35NIANGUOKAIZHAIC"}, "007509": {"suoxie": "HSRFLHPZHHC", "name": "\u534e\u5546\u6da6\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGRUNFENGLINGHUOPEIZHIHUNHEC"}, "007510": {"suoxie": "NFTYZQA", "name": "\u5357\u65b9\u6cf0\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGTAIYUANZHAIQUANA"}, "007511": {"suoxie": "NFTYZQC", "name": "\u5357\u65b9\u6cf0\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGTAIYUANZHAIQUANC"}, "007512": {"suoxie": "GYHGSGPC", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINHUGANGSHENGUPIAOC"}, "007513": {"suoxie": "BSFF3GYDKZ", "name": "\u535a\u65f6\u5bcc\u4e303\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUFENG3GEYUEDINGKAIZHAI"}, "007515": {"suoxie": "PHWLDZA", "name": "\u9e4f\u534e\u7a33\u5229\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUAWENLIDUANZHAIA"}, "007516": {"suoxie": "RTZRSGYDKZ", "name": "\u878d\u901a\u589e\u6da6\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGRUNSANGEYUEDINGKAIZHAI"}, "007517": {"suoxie": "BSFC3GYDKZ", "name": "\u535a\u65f6\u5bcc\u6df33\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUCHUN3GEYUEDINGKAIZHAI"}, "007518": {"suoxie": "DFAEFYXHHA", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAYOUXUANHUNHEA"}, "007519": {"suoxie": "DFAEFYXHHC", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAYOUXUANHUNHEC"}, "007520": {"suoxie": "FADFLCZA", "name": "\u5bcc\u5b89\u8fbe\u5bcc\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUANDAFULICHUNZHAIA"}, "007521": {"suoxie": "NFRTYF", "name": "\u5357\u65b9\u65e5\u6dfb\u76caF", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGRITIANYIF"}, "007522": {"suoxie": "NFLCJJYXHBE", "name": "\u5357\u65b9\u7406\u8d22\u91d1\u4ea4\u6613\u578b\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGLICAIJINJIAOYIXINGHUOBIE"}, "007523": {"suoxie": "HTFNXZZGPA", "name": "\u6c47\u6dfb\u5bcc\u5185\u9700\u589e\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUNEIXUZENGZHANGGUPIAOA"}, "007524": {"suoxie": "HTFNXZZGPC", "name": "\u6c47\u6dfb\u5bcc\u5185\u9700\u589e\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUNEIXUZENGZHANGGUPIAOC"}, "007525": {"suoxie": "YFDNNHXYNDKZA", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u590f\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGXIAYINIANDINGKAIZHAIA"}, "007526": {"suoxie": "YFDNNHXYNDKZC", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u590f\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGXIAYINIANDINGKAIZHAIC"}, "007527": {"suoxie": "RTLHDCLHHA", "name": "\u878d\u901a\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGLIANGHUADUOCELUEHUNHEA"}, "007528": {"suoxie": "RTLHDCLHHC", "name": "\u878d\u901a\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGLIANGHUADUOCELUEHUNHEC"}, "007529": {"suoxie": "JSHXZDZA", "name": "\u5609\u5b9e\u6c47\u946b\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIHUIXINZHONGDUANZHAIA"}, "007530": {"suoxie": "JSHXZDZC", "name": "\u5609\u5b9e\u6c47\u946b\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHIHUIXINZHONGDUANZHAIC"}, "007531": {"suoxie": "HBQSETFLJC", "name": "\u534e\u5b9d\u5238\u5546ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOQUANSHANGETFLIANJIEC"}, "007532": {"suoxie": "GTSHSGYDKZ", "name": "\u56fd\u6cf0\u76db\u5408\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAISHENGHESANGEYUEDINGKAIZHAI"}, "007533": {"suoxie": "GLCXCZHHA", "name": "\u683c\u6797\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINCHUANGXINCHENGZHANGHUNHEA"}, "007534": {"suoxie": "GLCXCZHHC", "name": "\u683c\u6797\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINCHUANGXINCHENGZHANGHUNHEC"}, "007535": {"suoxie": "ZOYHZQ", "name": "\u4e2d\u6b27\u76c8\u548c\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUYINGHEZHAIQUAN"}, "007536": {"suoxie": "BSFLCZZQ", "name": "\u535a\u65f6\u5bcc\u4e50\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFULECHUNZHAIZHAIQUAN"}, "007537": {"suoxie": "JSCCJTYLCZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u76c8\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIYINGLICHUNZHAI"}, "007538": {"suoxie": "YYHS300A", "name": "\u6c38\u8d62\u6caa\u6df1300A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGHUSHEN300A"}, "007539": {"suoxie": "YYHS300C", "name": "\u6c38\u8d62\u6caa\u6df1300C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGHUSHEN300C"}, "007540": {"suoxie": "HTBXAYZQ", "name": "\u534e\u6cf0\u4fdd\u5174\u5b89\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGANYUEZHAIQUAN"}, "007541": {"suoxie": "XHMSCIZGAGGJETFLJ", "name": "\u65b0\u534eMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAMSCIZHONGGUOAGUGUOJIETFLIANJIE"}, "007542": {"suoxie": "YYKTZGDJZDZA", "name": "\u6c38\u8d62\u5f00\u6cf0\u4e2d\u9ad8\u7b49\u7ea7\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGKAITAIZHONGGAODENGJIZHONGDUANZHAIA"}, "007543": {"suoxie": "YYKTZGDJZDZC", "name": "\u6c38\u8d62\u5f00\u6cf0\u4e2d\u9ad8\u7b49\u7ea7\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGKAITAIZHONGGAODENGJIZHONGDUANZHAIC"}, "007544": {"suoxie": "PHZSDQKFFQSZQ", "name": "\u9e4f\u534e\u5c0a\u665f\u5b9a\u671f\u5f00\u653e\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNSHENGDINGQIKAIFANGFAQISHIZHAIQUAN"}, "007545": {"suoxie": "TPHASGYDKZ", "name": "\u592a\u5e73\u6052\u5b89\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGANSANGEYUEDINGKAIZHAI"}, "007546": {"suoxie": "RTZXCZZQ", "name": "\u878d\u901a\u589e\u4eab\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZENGXIANGCHUNZHAIZHAIQUAN"}, "007548": {"suoxie": "YFDESGZRTZGP", "name": "\u6613\u65b9\u8fbeESG\u8d23\u4efb\u6295\u8d44\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAESGZERENTOUZIGUPIAO"}, "007549": {"suoxie": "ZTKYJZYXHHA", "name": "\u4e2d\u6cf0\u5f00\u9633\u4ef7\u503c\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGTAIKAIYANGJIAZHIYOUXUANHUNHEA"}, "007550": {"suoxie": "DXWLJZHHC", "name": "\u4e1c\u5174\u672a\u6765\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGXINGWEILAIJIAZHIHUNHEC"}, "007551": {"suoxie": "XYZLZQ", "name": "\u946b\u5143\u6cfd\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANZELIZHAIQUAN"}, "007552": {"suoxie": "ZXJTWYDKZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u88d5\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENYUDINGKAIZHAIC"}, "007553": {"suoxie": "ZXJTYGHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u533b\u6539\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGXINJIANTOUYIGAIHUNHEC"}, "007554": {"suoxie": "CTHLCZ", "name": "\u8d22\u901a\u6052\u5229\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGHENGLICHUNZHAI"}, "007555": {"suoxie": "ZHRMCZA", "name": "\u4e2d\u822a\u745e\u660e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIMINGCHUNZHAIA"}, "007556": {"suoxie": "ZHRMCZC", "name": "\u4e2d\u822a\u745e\u660e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIMINGCHUNZHAIC"}, "007557": {"suoxie": "ZJYXZGDJZQA", "name": "\u4e2d\u52a0\u4f18\u9009\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYOUXUANZHONGGAODENGJIZHAIQUANA"}, "007558": {"suoxie": "ZJYXZGDJZQC", "name": "\u4e2d\u52a0\u4f18\u9009\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYOUXUANZHONGGAODENGJIZHAIQUANC"}, "007559": {"suoxie": "XYFLSGYDQKFZ", "name": "\u946b\u5143\u5bcc\u5229\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANFULISANGEYUEDINGQIKAIFANGZHAI"}, "007560": {"suoxie": "ZRHXCZA", "name": "\u4e2d\u878d\u6052\u946b\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGXINCHUNZHAIA"}, "007561": {"suoxie": "ZRHXCZC", "name": "\u4e2d\u878d\u6052\u946b\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGXINCHUNZHAIC"}, "007562": {"suoxie": "JSCCJTCLZQ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u7eaf\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAICHUNLIZHAIQUAN"}, "007563": {"suoxie": "XYHYDKZ", "name": "\u5174\u94f6\u6c47\u9038\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIYIDINGKAIZHAI"}, "007564": {"suoxie": "PYCMZQA", "name": "\u9e4f\u626c\u6df3\u660e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNMINGZHAIQUANA"}, "007565": {"suoxie": "PYCMZQC", "name": "\u9e4f\u626c\u6df3\u660e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNMINGZHAIQUANC"}, "007566": {"suoxie": "ZYNXZQ", "name": "\u4e2d\u94f6\u5b81\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINNINGXIANGZHAIQUAN"}, "007567": {"suoxie": "NFHX39GYA", "name": "\u5357\u65b9\u6052\u65b039\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGXIN39GEYUEA"}, "007568": {"suoxie": "NFHX39GYC", "name": "\u5357\u65b9\u6052\u65b039\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHENGXIN39GEYUEC"}, "007569": {"suoxie": "NFAFHHC", "name": "\u5357\u65b9\u5b89\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANFUHUNHEC"}, "007570": {"suoxie": "FZFBHLJXHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u7ea2\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGHONGLIJINGXUANHUNHEC"}, "007571": {"suoxie": "NFSZ380ETFLJC", "name": "\u5357\u65b9\u4e0a\u8bc1380ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG380ETFLIANJIEC"}, "007572": {"suoxie": "ZJMFCZ", "name": "\u4e2d\u52a0\u6c11\u4e30\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAMINFENGCHUNZHAI"}, "007573": {"suoxie": "NYHLFYSNDKZ", "name": "\u519c\u94f6\u6c47\u7406\u4e30\u76c8\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIFENGYINGSANNIANDINGKAIZHAI"}, "007574": {"suoxie": "BYXJZHHC", "name": "\u5b9d\u76c8\u65b0\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINJIAZHIHUNHEC"}, "007575": {"suoxie": "BYXTHHC", "name": "\u5b9d\u76c8\u7965\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGTAIHUNHEC"}, "007576": {"suoxie": "HXDQSGYDKZQ", "name": "\u534e\u590f\u9f0e\u742a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGQISANGEYUEDINGKAIZHAIQUAN"}, "007577": {"suoxie": "BYXRHHC", "name": "\u5b9d\u76c8\u7965\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGRUIHUNHEC"}, "007578": {"suoxie": "BYXRHHC", "name": "\u5b9d\u76c8\u65b0\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINRUIHUNHEC"}, "007579": {"suoxie": "BYXJZZHHC", "name": "\u5b9d\u76c8\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXIANJINZHIZAOHUNHEC"}, "007580": {"suoxie": "BYZZ100ZSZQC", "name": "\u5b9d\u76c8\u4e2d\u8bc1100\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BAOYINGZHONGZHENG100ZHISHUZENGQIANGC"}, "007581": {"suoxie": "BYHLSYLHPZHHC", "name": "\u5b9d\u76c8\u9e3f\u5229\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGHONGLISHOUYILINGHUOPEIZHIHUNHEC"}, "007582": {"suoxie": "ZTQYZDZA", "name": "\u4e2d\u6cf0\u9752\u6708\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGTAIQINGYUEZHONGDUANZHAIA"}, "007583": {"suoxie": "ZTQYZDZC", "name": "\u4e2d\u6cf0\u9752\u6708\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGTAIQINGYUEZHONGDUANZHAIC"}, "007584": {"suoxie": "PHFXZQ", "name": "\u9e4f\u534e\u4e30\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGXINZHAIQUAN"}, "007585": {"suoxie": "GYRH3GYDKZ", "name": "\u5de5\u94f6\u745e\u5f183\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIHONG3GEYUEDINGKAIZHAI"}, "007586": {"suoxie": "HTBXDCL", "name": "\u534e\u6cf0\u4fdd\u5174\u591a\u7b56\u7565", "style": "\u80a1\u7968\u578b", "pinyin": "HUATAIBAOXINGDUOCELUE"}, "007587": {"suoxie": "ZSFYCZZQA", "name": "\u6d59\u5546\u4e30\u88d5\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGFENGYUCHUNZHAIZHAIQUANA"}, "007588": {"suoxie": "ZSFYCZZQC", "name": "\u6d59\u5546\u4e30\u88d5\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGFENGYUCHUNZHAIZHAIQUANC"}, "007589": {"suoxie": "JSZY42GYDQZQ", "name": "\u5609\u5b9e\u81f4\u514342\u4e2a\u6708\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIYUAN42GEYUEDINGQIZHAIQUAN"}, "007590": {"suoxie": "HBLSLXGP", "name": "\u534e\u5b9d\u7eff\u8272\u9886\u5148\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUABAOLVSELINGXIANGUPIAO"}, "007591": {"suoxie": "HXHY18GYDKZQ", "name": "\u534e\u590f\u6052\u76ca18\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAHENGYI18GEYUEDINGKAIZHAIQUAN"}, "007592": {"suoxie": "HXJZJXHH", "name": "\u534e\u590f\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAJIAZHIJINGXUANHUNHE"}, "007593": {"suoxie": "PYZZ500ZLCZZSA", "name": "\u9e4f\u626c\u4e2d\u8bc1500\u8d28\u91cf\u6210\u957f\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENG500ZHILIANGCHENGZHANGZHISHUA"}, "007594": {"suoxie": "PYZZ500ZLCZZSC", "name": "\u9e4f\u626c\u4e2d\u8bc1500\u8d28\u91cf\u6210\u957f\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENG500ZHILIANGCHENGZHANGZHISHUC"}, "007595": {"suoxie": "ZSTZCZA", "name": "\u62db\u5546\u6dfb\u6cfd\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANZECHUNZHAIA"}, "007596": {"suoxie": "ZSTZCZC", "name": "\u62db\u5546\u6dfb\u6cfd\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANZECHUNZHAIC"}, "007598": {"suoxie": "GFMYCZ", "name": "\u5e7f\u53d1\u6c11\u7389\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAMINYUCHUNZHAI"}, "007600": {"suoxie": "TKRY63GYDKZQ", "name": "\u6cf0\u5eb7\u6da6\u989063\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGRUNYI63GEYUEDINGKAIZHAIQUAN"}, "007603": {"suoxie": "JSCCZDZA", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINGSHUNCHANGCHENGZHONGDUANZHAIA"}, "007604": {"suoxie": "JSCCZDZC", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINGSHUNCHANGCHENGZHONGDUANZHAIC"}, "007605": {"suoxie": "JSHS300HLDBDETFLJA", "name": "\u5609\u5b9e\u6caa\u6df1300\u7ea2\u5229\u4f4e\u6ce2\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300HONGLIDIBODONGETFLIANJIEA"}, "007606": {"suoxie": "JSHS300HLDBDETFLJC", "name": "\u5609\u5b9e\u6caa\u6df1300\u7ea2\u5229\u4f4e\u6ce2\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300HONGLIDIBODONGETFLIANJIEC"}, "007609": {"suoxie": "HAJCZQA", "name": "\u6c47\u5b89\u5609\u8bda\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANJIACHENGZHAIQUANA"}, "007610": {"suoxie": "HAJCZQC", "name": "\u6c47\u5b89\u5609\u8bda\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANJIACHENGZHAIQUANC"}, "007611": {"suoxie": "HAYHCZZQA", "name": "\u6c47\u5b89\u88d5\u548c\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANYUHECHUNZHAIZHAIQUANA"}, "007612": {"suoxie": "HAYHCZZQC", "name": "\u6c47\u5b89\u88d5\u548c\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANYUHECHUNZHAIZHAIQUANC"}, "007613": {"suoxie": "JHYLJKHH", "name": "\u5609\u5408\u533b\u7597\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEYILIAOJIANKANGHUNHE"}, "007616": {"suoxie": "FGTZJXYZZQXA", "name": "\u5bcc\u56fd\u6295\u8d44\u7ea7\u4fe1\u7528\u503a\u503a\u5238\u578bA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTOUZIJIXINYONGZHAIZHAIQUANXINGA"}, "007617": {"suoxie": "FGTZJXYZZQXC", "name": "\u5bcc\u56fd\u6295\u8d44\u7ea7\u4fe1\u7528\u503a\u503a\u5238\u578bC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTOUZIJIXINYONGZHAIZHAIQUANXINGC"}, "007618": {"suoxie": "FGTZJXYZZQXD", "name": "\u5bcc\u56fd\u6295\u8d44\u7ea7\u4fe1\u7528\u503a\u503a\u5238\u578bD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTOUZIJIXINYONGZHAIZHAIQUANXINGD"}, "007619": {"suoxie": "ZORY63GYDKZ", "name": "\u4e2d\u6b27\u6da6\u903863\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOURUNYI63GEYUEDINGKAIZHAI"}, "007622": {"suoxie": "ZOGLYNGDCYZQA", "name": "\u4e2d\u6b27\u6eda\u5229\u4e00\u5e74\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUGUNLIYINIANGUNDONGCHIYOUZHAIQUANA"}, "007623": {"suoxie": "ZOGLYNGDCYZQC", "name": "\u4e2d\u6b27\u6eda\u5229\u4e00\u5e74\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUGUNLIYINIANGUNDONGCHIYOUZHAIQUANC"}, "007628": {"suoxie": "NFQQZQQDIIRMBA", "name": "\u5357\u65b9\u5168\u7403\u503a\u5238(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "NANFANGQUANQIUZHAIQUANQDIIRENMINBIA"}, "007629": {"suoxie": "NFQQZQQDIIRMBC", "name": "\u5357\u65b9\u5168\u7403\u503a\u5238(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "NANFANGQUANQIUZHAIQUANQDIIRENMINBIC"}, "007630": {"suoxie": "NFQQZQQDIIMYXHA", "name": "\u5357\u65b9\u5168\u7403\u503a\u5238(QDII)\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "NANFANGQUANQIUZHAIQUANQDIIMEIYUANXIANHUIA"}, "007631": {"suoxie": "NFQQZQQDIIMYXHC", "name": "\u5357\u65b9\u5168\u7403\u503a\u5238(QDII)\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "NANFANGQUANQIUZHAIQUANQDIIMEIYUANXIANHUIC"}, "007632": {"suoxie": "HRYDAXLHPZHHC", "name": "\u534e\u6da6\u5143\u5927\u5b89\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARUNYUANDAANXINLINGHUOPEIZHIHUNHEC"}, "007635": {"suoxie": "YHTYZDZA", "name": "\u94f6\u6cb3\u5929\u76c8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHETIANYINGZHONGDUANZHAIA"}, "007636": {"suoxie": "YHTYZDZC", "name": "\u94f6\u6cb3\u5929\u76c8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHETIANYINGZHONGDUANZHAIC"}, "007637": {"suoxie": "XYXRXDZC", "name": "\u5174\u94f6\u946b\u65e5\u4eab\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINXINRIXIANGDUANZHAIC"}, "007638": {"suoxie": "QHKYKYPHYLSNFOF", "name": "\u524d\u6d77\u5f00\u6e90\u5eb7\u9890\u5e73\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "QIANHAIKAIYUANKANGYIPINGHENGYANGLAOSANNIANFOF"}, "007639": {"suoxie": "HTF3NFBJZYSHH", "name": "\u6c47\u6dfb\u5bcc3\u5e74\u5c01\u95ed\u7ade\u4e89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFU3NIANFENGBIJINGZHENGYOUSHIHUNHE"}, "007640": {"suoxie": "TDHLYLZQ", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6c38\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIYONGLIZHAIQUAN"}, "007641": {"suoxie": "TDHLXLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u946b\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIXINLIZHAIQUANA"}, "007642": {"suoxie": "TDHLXLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u946b\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIXINLIZHAIQUANC"}, "007643": {"suoxie": "HAWJYLYNFOF", "name": "\u534e\u5b89\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANWENJIANYANGLAOYINIANFOF"}, "007644": {"suoxie": "HBBRZQ", "name": "\u534e\u5b9d\u5b9d\u6da6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAORUNZHAIQUAN"}, "007645": {"suoxie": "PAJXYDKZA", "name": "\u5e73\u5b89\u5b63\u4eab\u88d5\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIXIANGYUDINGKAIZHAIA"}, "007646": {"suoxie": "PAJXYDKZC", "name": "\u5e73\u5b89\u5b63\u4eab\u88d5\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIXIANGYUDINGKAIZHAIC"}, "007647": {"suoxie": "PAJXYDKZE", "name": "\u5e73\u5b89\u5b63\u4eab\u88d5\u5b9a\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIXIANGYUDINGKAIZHAIE"}, "007649": {"suoxie": "BSYZPHYLFOF", "name": "\u535a\u65f6\u9890\u6cfd\u5e73\u8861\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIYIZEPINGHENGYANGLAOFOF"}, "007650": {"suoxie": "GYYL2040FOF", "name": "\u5de5\u94f6\u517b\u80012040(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINYANGLAO2040FOF"}, "007651": {"suoxie": "GYYL2045SNCYHHFOF", "name": "\u5de5\u94f6\u517b\u80012045\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINYANGLAO2045SANNIANCHIYOUHUNHEFOF"}, "007652": {"suoxie": "HXWJYLYNFOF", "name": "\u534e\u590f\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAWENJIANYANGLAOYINIANFOF"}, "007653": {"suoxie": "CSWY6GYA", "name": "\u957f\u76db\u7a33\u76ca6\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGWENYI6GEYUEA"}, "007654": {"suoxie": "CSWY6GYC", "name": "\u957f\u76db\u7a33\u76ca6\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGWENYI6GEYUEC"}, "007655": {"suoxie": "NFDYZDZA", "name": "\u5357\u65b9\u5b9a\u5143\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGDINGYUANZHONGDUANZHAIA"}, "007656": {"suoxie": "NFDYZDZC", "name": "\u5357\u65b9\u5b9a\u5143\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGDINGYUANZHONGDUANZHAIC"}, "007657": {"suoxie": "DFHZZJZLZSA", "name": "\u4e1c\u65b9\u7ea2\u4e2d\u8bc1\u7ade\u4e89\u529b\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGFANGHONGZHONGZHENGJINGZHENGLIZHISHUA"}, "007658": {"suoxie": "DFHZZJZLZSC", "name": "\u4e1c\u65b9\u7ea2\u4e2d\u8bc1\u7ade\u4e89\u529b\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGFANGHONGZHONGZHENGJINGZHENGLIZHISHUC"}, "007659": {"suoxie": "BSFH3GYDKZFQS", "name": "\u535a\u65f6\u5bcc\u6c473\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUHUI3GEYUEDINGKAIZHAIFAQISHI"}, "007660": {"suoxie": "ZSHYJHYLSNFOF", "name": "\u62db\u5546\u548c\u60a6\u5747\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGHEYUEJUNHENGYANGLAOSANNIANFOF"}, "007661": {"suoxie": "NFYL2030SNFOF", "name": "\u5357\u65b9\u517b\u80012030\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYANGLAO2030SANNIANFOF"}, "007662": {"suoxie": "FGXWJHYLSNFOF", "name": "\u5bcc\u56fd\u946b\u65fa\u5747\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOXINWANGJUNHENGYANGLAOSANNIANFOF"}, "007663": {"suoxie": "PAAXLHPZHHC", "name": "\u5e73\u5b89\u5b89\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANXIANGLINGHUOPEIZHIHUNHEC"}, "007664": {"suoxie": "YYCYBZSFQSA", "name": "\u6c38\u8d62\u521b\u4e1a\u677f\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGCHUANGYEBANZHISHUFAQISHIA"}, "007665": {"suoxie": "YYCYBZSFQSC", "name": "\u6c38\u8d62\u521b\u4e1a\u677f\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGCHUANGYEBANZHISHUFAQISHIC"}, "007666": {"suoxie": "HXDHZQA", "name": "\u534e\u590f\u9f0e\u6cd3\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGHONGZHAIQUANA"}, "007667": {"suoxie": "HXDHZQC", "name": "\u534e\u590f\u9f0e\u6cd3\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGHONGZHAIQUANC"}, "007668": {"suoxie": "GFYL2035SNCYQHHFOF", "name": "\u5e7f\u53d1\u517b\u80012035\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAYANGLAO2035SANNIANCHIYOUQIHUNHEFOF"}, "007669": {"suoxie": "TPRYHHC", "name": "\u592a\u5e73\u777f\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIYINGHUNHEC"}, "007670": {"suoxie": "JSSYYHJXZQ", "name": "\u5609\u5b9e\u5546\u4e1a\u94f6\u884c\u7cbe\u9009\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHISHANGYEYINHANGJINGXUANZHAIQUAN"}, "007671": {"suoxie": "JXZZHLQLZSA", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u7ea2\u5229\u6f5c\u529b\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGHONGLIQIANLIZHISHUA"}, "007672": {"suoxie": "JXZZHLQLZSC", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u7ea2\u5229\u6f5c\u529b\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGHONGLIQIANLIZHISHUC"}, "007673": {"suoxie": "ZJARWJYLYNFOF", "name": "\u4e2d\u52a0\u5b89\u745e\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAANRUIWENJIANYANGLAOYINIANFOF"}, "007674": {"suoxie": "GYCYSJGPA", "name": "\u5de5\u94f6\u4ea7\u4e1a\u5347\u7ea7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINCHANYESHENGJIGUPIAOA"}, "007675": {"suoxie": "GYCYSJGPC", "name": "\u5de5\u94f6\u4ea7\u4e1a\u5347\u7ea7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINCHANYESHENGJIGUPIAOC"}, "007676": {"suoxie": "FCTHCZA", "name": "\u8702\u5de2\u6dfb\u6c47\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANHUICHUNZHAIA"}, "007677": {"suoxie": "FCTHCZC", "name": "\u8702\u5de2\u6dfb\u6c47\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANHUICHUNZHAIC"}, "007678": {"suoxie": "TDHLPPSJHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u54c1\u724c\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIPINPAISHENGJIHUNHEA"}, "007679": {"suoxie": "TDHLPPSJHHC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u54c1\u724c\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIPINPAISHENGJIHUNHEC"}, "007680": {"suoxie": "ZJXLSNZQ", "name": "\u4e2d\u52a0\u4eab\u5229\u4e09\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAXIANGLISANNIANZHAIQUAN"}, "007681": {"suoxie": "PHFDZQ", "name": "\u9e4f\u534e\u4e30\u767b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGDENGZHAIQUAN"}, "007682": {"suoxie": "PHJLLNDKZ", "name": "\u9e4f\u534e\u9526\u5229\u4e24\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAJINLILIANGNIANDINGKAIZHAI"}, "007683": {"suoxie": "HSZZJXZQA", "name": "\u534e\u5546\u8f6c\u503a\u7cbe\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUASHANGZHUANZHAIJINGXUANZHAIQUANA"}, "007684": {"suoxie": "HSZZJXZQC", "name": "\u534e\u5546\u8f6c\u503a\u7cbe\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUASHANGZHUANZHAIJINGXUANZHAIQUANC"}, "007685": {"suoxie": "HSDZHYLHGP", "name": "\u534e\u5546\u7535\u5b50\u884c\u4e1a\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGDIANZIHANGYELIANGHUAGUPIAO"}, "007686": {"suoxie": "DFJZWJLHPZHHC", "name": "\u4e1c\u65b9\u4ef7\u503c\u6316\u6398\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGJIAZHIWAJUELINGHUOPEIZHIHUNHEC"}, "007687": {"suoxie": "DFCZSYLHPZHHC", "name": "\u4e1c\u65b9\u6210\u957f\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGCHENGZHANGSHOUYILINGHUOPEIZHIHUNHEC"}, "007688": {"suoxie": "TDYL2040SNCYHHFOFA", "name": "\u6cf0\u8fbe\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIDAYANGLAO2040SANNIANCHIYOUHUNHEFOFA"}, "007689": {"suoxie": "GTRYXNYHHA", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u80fd\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINXINNENGYUANHUNHEA"}, "007690": {"suoxie": "GTRYXNYHHC", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u80fd\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINXINNENGYUANHUNHEC"}, "007691": {"suoxie": "YYZLYNDK", "name": "\u6c38\u8d62\u6cfd\u5229\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZELIYINIANDINGKAI"}, "007692": {"suoxie": "YYDLZQA", "name": "\u6c38\u8d62\u9f0e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGDINGLIZHAIQUANA"}, "007693": {"suoxie": "YYDLZQC", "name": "\u6c38\u8d62\u9f0e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGDINGLIZHAIQUANC"}, "007694": {"suoxie": "TDYL2040SNCYHHFOFC", "name": "\u6cf0\u8fbe\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIDAYANGLAO2040SANNIANCHIYOUHUNHEFOFC"}, "007695": {"suoxie": "TDYL2040SNCYHHFOFE", "name": "\u6cf0\u8fbe\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)E", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIDAYANGLAO2040SANNIANCHIYOUHUNHEFOFE"}, "007696": {"suoxie": "JSRXHB", "name": "\u5609\u5b9e\u878d\u4eab\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIRONGXIANGHUOBI"}, "007699": {"suoxie": "JXRXYNDQKFZQ", "name": "\u5efa\u4fe1\u8363\u79a7\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRONGXIYINIANDINGQIKAIFANGZHAIQUAN"}, "007701": {"suoxie": "GLA6GYDKZA", "name": "\u56fd\u8054\u5b896\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANAN6GEYUEDINGKAIZHAIA"}, "007702": {"suoxie": "GLA6GYDKZC", "name": "\u56fd\u8054\u5b896\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANAN6GEYUEDINGKAIZHAIC"}, "007703": {"suoxie": "WJXSCZA", "name": "\u4e07\u5bb6\u946b\u76db\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINSHENGCHUNZHAIA"}, "007704": {"suoxie": "WJXSCZC", "name": "\u4e07\u5bb6\u946b\u76db\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINSHENGCHUNZHAIC"}, "007705": {"suoxie": "CCHKWJYLYNFOF", "name": "\u957f\u57ce\u6052\u5eb7\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGHENGKANGWENJIANYANGLAOYINIANFOF"}, "007706": {"suoxie": "NFCYZQA", "name": "\u5357\u65b9\u806a\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCONGYUANZHAIQUANA"}, "007707": {"suoxie": "NFCYZQC", "name": "\u5357\u65b9\u806a\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCONGYUANZHAIQUANC"}, "007708": {"suoxie": "ZYRFFDJZXHBA", "name": "\u4e2d\u94f6\u745e\u798f\u6d6e\u52a8\u51c0\u503c\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINRUIFUFUDONGJINGZHIXINGHUOBIA"}, "007709": {"suoxie": "ZYRFFDJZXHBC", "name": "\u4e2d\u94f6\u745e\u798f\u6d6e\u52a8\u51c0\u503c\u578b\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINRUIFUFUDONGJINGZHIXINGHUOBIC"}, "007710": {"suoxie": "GLHTSGYDKZA", "name": "\u683c\u6797\u6cd3\u6cf0\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGTAISANGEYUEDINGKAIZHAIA"}, "007711": {"suoxie": "GLHTSGYDKZC", "name": "\u683c\u6797\u6cd3\u6cf0\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGTAISANGEYUEDINGKAIZHAIC"}, "007712": {"suoxie": "ZYKX3GYDQKFZQ", "name": "\u4e2d\u94f6\u5eb7\u4eab3\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINKANGXIANG3GEYUEDINGQIKAIFANGZHAIQUAN"}, "007713": {"suoxie": "HFKJDNHH", "name": "\u534e\u5bcc\u79d1\u6280\u52a8\u80fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUKEJIDONGNENGHUNHE"}, "007714": {"suoxie": "NFHYLLZZQA", "name": "\u5357\u65b9\u8d3a\u5143\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHEYUANLILVZHAIZHAIQUANA"}, "007715": {"suoxie": "NFHYLLZZQC", "name": "\u5357\u65b9\u8d3a\u5143\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGHEYUANLILVZHAIZHAIQUANC"}, "007716": {"suoxie": "JSZHCZZQ", "name": "\u5609\u5b9e\u81f4\u534e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIHUACHUNZHAIZHAIQUAN"}, "007717": {"suoxie": "GYZXDZZQF", "name": "\u5de5\u94f6\u5c0a\u4eab\u77ed\u503a\u503a\u5238F", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNXIANGDUANZHAIZHAIQUANF"}, "007718": {"suoxie": "ZYCXYLHHA", "name": "\u4e2d\u94f6\u521b\u65b0\u533b\u7597\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCHUANGXINYILIAOHUNHEA"}, "007719": {"suoxie": "YYYLZQA", "name": "\u6c38\u8d62\u5143\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYUANLIZHAIQUANA"}, "007720": {"suoxie": "YYYLZQC", "name": "\u6c38\u8d62\u5143\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYUANLIZHAIQUANC"}, "007721": {"suoxie": "THBP500QDIIFOFA", "name": "\u5929\u5f18\u6807\u666e500(QDII-FOF)A", "style": "QDII", "pinyin": "TIANHONGBIAOPU500QDIIFOFA"}, "007722": {"suoxie": "THBP500QDIIFOFC", "name": "\u5929\u5f18\u6807\u666e500(QDII-FOF)C", "style": "QDII", "pinyin": "TIANHONGBIAOPU500QDIIFOFC"}, "007723": {"suoxie": "PHJR86GYDKZ", "name": "\u9e4f\u534e\u9526\u6da686\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAJINRUN86GEYUEDINGKAIZHAI"}, "007725": {"suoxie": "ZSRWHHA", "name": "\u62db\u5546\u745e\u6587\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGRUIWENHUNHEA"}, "007726": {"suoxie": "ZSRWHHC", "name": "\u62db\u5546\u745e\u6587\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGRUIWENHUNHEC"}, "007729": {"suoxie": "ZSPSQQPZQDIIFOFRMB", "name": "\u62db\u5546\u666e\u76db\u5168\u7403\u914d\u7f6e(QDII-FOF)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "ZHAOSHANGPUSHENGQUANQIUPEIZHIQDIIFOFRENMINBI"}, "007730": {"suoxie": "PAJGJHBC", "name": "\u5e73\u5b89\u91d1\u7ba1\u5bb6\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANJINGUANJIAHUOBIC"}, "007731": {"suoxie": "MSJYCXCZHHA", "name": "\u6c11\u751f\u52a0\u94f6\u6301\u7eed\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINCHIXUCHENGZHANGHUNHEA"}, "007732": {"suoxie": "MSJYCXCZHHC", "name": "\u6c11\u751f\u52a0\u94f6\u6301\u7eed\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINCHIXUCHENGZHANGHUNHEC"}, "007733": {"suoxie": "NFZRHHA", "name": "\u5357\u65b9\u667a\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHIRUIHUNHEA"}, "007734": {"suoxie": "NFZRHHC", "name": "\u5357\u65b9\u667a\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHIRUIHUNHEC"}, "007735": {"suoxie": "JYMAHBDKC", "name": "\u91d1\u9e70\u6c11\u5b89\u56de\u62a5\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGMINANHUIBAODINGKAIC"}, "007736": {"suoxie": "MSJYJXSNDKZ", "name": "\u6c11\u751f\u52a0\u94f6\u805a\u946b\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINJUXINSANNIANDINGKAIZHAI"}, "007737": {"suoxie": "NDYFCX100", "name": "\u8bfa\u5fb7\u7814\u53d1\u521b\u65b0100", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUODEYANFACHUANGXIN100"}, "007738": {"suoxie": "CHWHZQA", "name": "\u6df3\u539a\u7a33\u60e0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENHUIZHAIQUANA"}, "007739": {"suoxie": "CHWHZQC", "name": "\u6df3\u539a\u7a33\u60e0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENHUIZHAIQUANC"}, "007740": {"suoxie": "THXYZQA", "name": "\u5929\u5f18\u4fe1\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINYIZHAIQUANA"}, "007741": {"suoxie": "THXYZQC", "name": "\u5929\u5f18\u4fe1\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINYIZHAIQUANC"}, "007744": {"suoxie": "CSAYCZZQA", "name": "\u957f\u76db\u5b89\u9038\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGANYICHUNZHAIZHAIQUANA"}, "007745": {"suoxie": "CSAYCZZQC", "name": "\u957f\u76db\u5b89\u9038\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGANYICHUNZHAIZHAIQUANC"}, "007746": {"suoxie": "HAXJRL", "name": "\u534e\u5b89\u73b0\u91d1\u6da6\u5229", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANXIANJINRUNLI"}, "007747": {"suoxie": "HFTPHYLFOF", "name": "\u6d77\u5bcc\u901a\u5e73\u8861\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGPINGHENGYANGLAOFOF"}, "007748": {"suoxie": "THYL2035SNFOF", "name": "\u5929\u5f18\u517b\u80012035\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TIANHONGYANGLAO2035SANNIANFOF"}, "007749": {"suoxie": "MSJYPCHHC", "name": "\u6c11\u751f\u52a0\u94f6\u9e4f\u7a0b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINPENGCHENGHUNHEC"}, "007750": {"suoxie": "GFYSZZGP", "name": "\u5e7f\u53d1\u4f18\u52bf\u589e\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYOUSHIZENGZHANGGUPIAO"}, "007751": {"suoxie": "JSZZHLCZDBDA", "name": "\u666f\u987a\u4e2d\u8bc1\u7ea2\u5229\u6210\u957f\u4f4e\u6ce2\u52a8A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENGHONGLICHENGZHANGDIBODONGA"}, "007752": {"suoxie": "ZYZLZQA", "name": "\u4e2d\u94f6\u62db\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHAOLIZHAIQUANA"}, "007753": {"suoxie": "ZYZLZQC", "name": "\u4e2d\u94f6\u62db\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHAOLIZHAIQUANC"}, "007754": {"suoxie": "SYHYLZDQZQA", "name": "\u4e0a\u94f6\u6167\u6c38\u5229\u4e2d\u77ed\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANGYINHUIYONGLIZHONGDUANQIZHAIQUANA"}, "007755": {"suoxie": "SYHYLZDQZQC", "name": "\u4e0a\u94f6\u6167\u6c38\u5229\u4e2d\u77ed\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANGYINHUIYONGLIZHONGDUANQIZHAIQUANC"}, "007756": {"suoxie": "CTJLSGYDKZFQS", "name": "\u8d22\u901a\u4e45\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGJIULISANGEYUEDINGKAIZHAIFAQISHI"}, "007758": {"suoxie": "PALXYNDKZA", "name": "\u5e73\u5b89\u4e50\u4eab\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANLEXIANGYINIANDINGKAIZHAIA"}, "007759": {"suoxie": "PALXYNDKZC", "name": "\u5e73\u5b89\u4e50\u4eab\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANLEXIANGYINIANDINGKAIZHAIC"}, "007760": {"suoxie": "JSZZHLCZDBDC", "name": "\u666f\u987a\u4e2d\u8bc1\u7ea2\u5229\u6210\u957f\u4f4e\u6ce2\u52a8C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENGHONGLICHENGZHANGDIBODONGC"}, "007761": {"suoxie": "XYARSNDKZ", "name": "\u946b\u5143\u5b89\u777f\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANANRUISANNIANDINGKAIZHAI"}, "007762": {"suoxie": "FGTYZQLOFA", "name": "\u5bcc\u56fd\u5929\u76c8\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANYINGZHAIQUANLOFA"}, "007765": {"suoxie": "QHKY13NGKZA", "name": "\u524d\u6d77\u5f00\u6e901-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUAN13NIANGUOKAIZHAIA"}, "007766": {"suoxie": "QHKY13NGKZC", "name": "\u524d\u6d77\u5f00\u6e901-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUAN13NIANGUOKAIZHAIC"}, "007767": {"suoxie": "HTBXZXDK", "name": "\u534e\u6cf0\u4fdd\u5174\u5c0a\u4eab\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAOXINGZUNXIANGDINGKAI"}, "007768": {"suoxie": "XDAYASCZ", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5b89\u76db\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINDAAOYINANSHENGCHUNZHAI"}, "007769": {"suoxie": "DXXRYNDKA", "name": "\u4e1c\u5174\u5174\u745e\u4e00\u5e74\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINGRUIYINIANDINGKAIA"}, "007770": {"suoxie": "TTKTHHA", "name": "\u540c\u6cf0\u5f00\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIKAITAIHUNHEA"}, "007771": {"suoxie": "TTKTHHC", "name": "\u540c\u6cf0\u5f00\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIKAITAIHUNHEC"}, "007772": {"suoxie": "PYASSXDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u714a\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGXUANDINGKAIZHAIQUAN"}, "007773": {"suoxie": "JYSAHQCZ3GYDKZA", "name": "\u91d1\u5143\u987a\u5b89\u6cd3\u6cc9\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANHONGQUANCHUNZHAI3GEYUEDINGKAIZHAIA"}, "007774": {"suoxie": "JYSAHQCZ3GYDKZC", "name": "\u91d1\u5143\u987a\u5b89\u6cd3\u6cc9\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANHONGQUANCHUNZHAI3GEYUEDINGKAIZHAIC"}, "007775": {"suoxie": "HALHXFHHA", "name": "\u6c47\u5b89\u91cf\u5316\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANLIANGHUAXIANFENGHUNHEA"}, "007776": {"suoxie": "HALHXFHHC", "name": "\u6c47\u5b89\u91cf\u5316\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANLIANGHUAXIANFENGHUNHEC"}, "007777": {"suoxie": "ZYYJJXHH", "name": "\u4e2d\u90ae\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUYANJIUJINGXUANHUNHE"}, "007778": {"suoxie": "GFJFCZ", "name": "\u5e7f\u53d1\u666f\u5bcc\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGFUCHUNZHAI"}, "007779": {"suoxie": "YHZHYL2030HHFOF", "name": "\u94f6\u534e\u5c0a\u548c\u517b\u80012030\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YINHUAZUNHEYANGLAO2030HUNHEFOF"}, "007780": {"suoxie": "YHZHYL2040HHFOF", "name": "\u94f6\u534e\u5c0a\u548c\u517b\u80012040\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YINHUAZUNHEYANGLAO2040HUNHEFOF"}, "007781": {"suoxie": "THHXHHFQS", "name": "\u5929\u5f18\u5f18\u65b0\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGHONGXINHUNHEFAQISHI"}, "007782": {"suoxie": "DCMSCIJZ100ETFLJA", "name": "\u5927\u6210MSCI\u4ef7\u503c100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGMSCIJIAZHI100ETFLIANJIEA"}, "007783": {"suoxie": "DCMSCIJZ100ETFLJC", "name": "\u5927\u6210MSCI\u4ef7\u503c100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGMSCIJIAZHI100ETFLIANJIEC"}, "007784": {"suoxie": "GFYQCXQDETFLJA", "name": "\u5e7f\u53d1\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYANGQICHUANGXINQUDONGETFLIANJIEA"}, "007785": {"suoxie": "GFYQCXQDETFLJC", "name": "\u5e7f\u53d1\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYANGQICHUANGXINQUDONGETFLIANJIEC"}, "007786": {"suoxie": "FGZZGQYDYLETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGUOQIYIDAIYILUETFLIANJIEA"}, "007787": {"suoxie": "FGZZGQYDYLETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGUOQIYIDAIYILUETFLIANJIEC"}, "007788": {"suoxie": "YFDZZGQDLFQSLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u56fd\u4f01\u5e26\u8def\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGUOQIDAILUFAQISHILIANJIEA"}, "007789": {"suoxie": "YFDZZGQDLFQSLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u56fd\u4f01\u5e26\u8def\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGUOQIDAILUFAQISHILIANJIEC"}, "007790": {"suoxie": "NFMYDZA", "name": "\u5357\u65b9\u68a6\u5143\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGMENGYUANDUANZHAIA"}, "007791": {"suoxie": "NFMYDZC", "name": "\u5357\u65b9\u68a6\u5143\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGMENGYUANDUANZHAIC"}, "007792": {"suoxie": "JSYQCXQDETFLJA", "name": "\u5609\u5b9e\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIYANGQICHUANGXINQUDONGETFLIANJIEA"}, "007793": {"suoxie": "JSYQCXQDETFLJC", "name": "\u5609\u5b9e\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIYANGQICHUANGXINQUDONGETFLIANJIEC"}, "007794": {"suoxie": "SWLXZZ500ZSYXZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1500\u6307\u6570\u4f18\u9009\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENG500ZHISHUYOUXUANZENGQIANGC"}, "007795": {"suoxie": "SWLXZZ500ZSZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENG500ZHISHUZENGQIANGC"}, "007796": {"suoxie": "BSYQCXQDETFLJA", "name": "\u535a\u65f6\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGQICHUANGXINQUDONGETFLIANJIEA"}, "007797": {"suoxie": "BSYQCXQDETFLJC", "name": "\u535a\u65f6\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGQICHUANGXINQUDONGETFLIANJIEC"}, "007799": {"suoxie": "SWLXZXQY100ZSLOFC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u5c0f\u4f01\u4e1a100\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGXIAOQIYE100ZHISHULOFC"}, "007800": {"suoxie": "SWLXHS300JZZSC", "name": "\u7533\u4e07\u83f1\u4fe1\u6caa\u6df1300\u4ef7\u503c\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINHUSHEN300JIAZHIZHISHUC"}, "007801": {"suoxie": "DCZZHLZSC", "name": "\u5927\u6210\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENGHONGLIZHISHUC"}, "007802": {"suoxie": "XQHTHHA", "name": "\u5174\u5168\u5408\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHETAIHUNHEA"}, "007803": {"suoxie": "XQHTHHC", "name": "\u5174\u5168\u5408\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHETAIHUNHEC"}, "007804": {"suoxie": "SWLXHS300ZSZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINHUSHEN300ZHISHUZENGQIANGC"}, "007805": {"suoxie": "HBFDJZHB", "name": "\u534e\u5b9d\u6d6e\u52a8\u51c0\u503c\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOFUDONGJINGZHIHUOBI"}, "007806": {"suoxie": "JXMSCIZGAGZSZQA", "name": "\u5efa\u4fe1MSCI\u4e2d\u56fdA\u80a1\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINMSCIZHONGGUOAGUZHISHUZENGQIANGA"}, "007807": {"suoxie": "JXMSCIZGAGZSZQC", "name": "\u5efa\u4fe1MSCI\u4e2d\u56fdA\u80a1\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINMSCIZHONGGUOAGUZHISHUZENGQIANGC"}, "007808": {"suoxie": "BXRFLHYXLHPZ", "name": "\u5317\u4fe1\u745e\u4e30\u91cf\u5316\u4f18\u9009\u7075\u6d3b\u914d\u7f6e", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BEIXINRUIFENGLIANGHUAYOUXUANLINGHUOPEIZHI"}, "007809": {"suoxie": "FGYQCXQDETFLJA", "name": "\u5bcc\u56fd\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOYANGQICHUANGXINQUDONGETFLIANJIEA"}, "007810": {"suoxie": "FGYQCXQDETFLJC", "name": "\u5bcc\u56fd\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOYANGQICHUANGXINQUDONGETFLIANJIEC"}, "007811": {"suoxie": "CHXZHHA", "name": "\u6df3\u539a\u4fe1\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUNHOUXINZEHUNHEA"}, "007812": {"suoxie": "CHXZHHC", "name": "\u6df3\u539a\u4fe1\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUNHOUXINZEHUNHEC"}, "007815": {"suoxie": "JSXXKJ100ETFLJA", "name": "\u5609\u5b9e\u65b0\u5174\u79d1\u6280100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIXINXINGKEJI100ETFLIANJIEA"}, "007816": {"suoxie": "JSXXKJ100ETFLJC", "name": "\u5609\u5b9e\u65b0\u5174\u79d1\u6280100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIXINXINGKEJI100ETFLIANJIEC"}, "007817": {"suoxie": "GTZZQZTXSBETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u901a\u4fe1\u8bbe\u5907ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHITONGXINSHEBEIETFLIANJIEA"}, "007818": {"suoxie": "GTZZQZTXSBETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u901a\u4fe1\u8bbe\u5907ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHITONGXINSHEBEIETFLIANJIEC"}, "007819": {"suoxie": "HTZJFYZDZA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u76ca\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIZIJINFENGYIZHONGDUANZHAIA"}, "007820": {"suoxie": "HTZJFYZDZC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u76ca\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIZIJINFENGYIZHONGDUANZHAIC"}, "007821": {"suoxie": "HTZJFLZDZFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u5229\u4e2d\u77ed\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIZIJINFENGLIZHONGDUANZHAIFAQIA"}, "007822": {"suoxie": "HTZJFLZDZFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u5229\u4e2d\u77ed\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIZIJINFENGLIZHONGDUANZHAIFAQIC"}, "007823": {"suoxie": "THHZDZA", "name": "\u5929\u5f18\u5f18\u62e9\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGHONGZEDUANZHAIA"}, "007824": {"suoxie": "THHZDZC", "name": "\u5929\u5f18\u5f18\u62e9\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGHONGZEDUANZHAIC"}, "007825": {"suoxie": "BDZYHHA", "name": "\u535a\u9053\u5fd7\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOZHIYUANHUNHEA"}, "007826": {"suoxie": "BDZYHHC", "name": "\u535a\u9053\u5fd7\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOZHIYUANHUNHEC"}, "007827": {"suoxie": "HRYDLHYXHHC", "name": "\u534e\u6da6\u5143\u5927\u91cf\u5316\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDALIANGHUAYOUXUANHUNHEC"}, "007828": {"suoxie": "CJHXXYHLZQA", "name": "\u521b\u91d1\u5408\u4fe1\u4fe1\u7528\u7ea2\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINXINYONGHONGLIZHAIQUANA"}, "007829": {"suoxie": "CJHXXYHLZQC", "name": "\u521b\u91d1\u5408\u4fe1\u4fe1\u7528\u7ea2\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINXINYONGHONGLIZHAIQUANC"}, "007830": {"suoxie": "JXRRYNDQKFZQ", "name": "\u5efa\u4fe1\u8363\u745e\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRONGRUIYINIANDINGQIKAIFANGZHAIQUAN"}, "007831": {"suoxie": "BDWBZHGPA", "name": "\u535a\u9053\u4f0d\u4f70\u667a\u822a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOWUBAIZHIHANGGUPIAOA"}, "007832": {"suoxie": "BDWBZHGPC", "name": "\u535a\u9053\u4f0d\u4f70\u667a\u822a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOWUBAIZHIHANGGUPIAOC"}, "007833": {"suoxie": "CSWYTLZQA", "name": "\u957f\u76db\u7a33\u6021\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGSHENGWENYITIANLIZHAIQUANA"}, "007834": {"suoxie": "CSWYTLZQC", "name": "\u957f\u76db\u7a33\u6021\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGSHENGWENYITIANLIZHAIQUANC"}, "007835": {"suoxie": "GTXRHH", "name": "\u56fd\u6cf0\u946b\u777f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIXINRUIHUNHE"}, "007836": {"suoxie": "TKRHLNDKZQ", "name": "\u6cf0\u5eb7\u6da6\u548c\u4e24\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGRUNHELIANGNIANDINGKAIZHAIQUAN"}, "007837": {"suoxie": "GSABZYCZA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8000\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNYAOCHUNZHAIA"}, "007838": {"suoxie": "GSABZYCZC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8000\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNYAOCHUNZHAIC"}, "007839": {"suoxie": "HTFZZCSJETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u957f\u4e09\u89d2ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGCHANGSANJIAOETFLIANJIEA"}, "007840": {"suoxie": "HTFZZCSJETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u957f\u4e09\u89d2ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGCHANGSANJIAOETFLIANJIEC"}, "007842": {"suoxie": "NHZZHZWQETFLJA", "name": "\u5357\u534e\u4e2d\u8bc1\u676d\u5dde\u6e7e\u533aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANHUAZHONGZHENGHANGZHOUWANQUETFLIANJIEA"}, "007843": {"suoxie": "NHZZHZWQETFLJC", "name": "\u5357\u534e\u4e2d\u8bc1\u676d\u5dde\u6e7e\u533aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANHUAZHONGZHENGHANGZHOUWANQUETFLIANJIEC"}, "007844": {"suoxie": "HBBPYQSYGPRMBC", "name": "\u534e\u5b9d\u6807\u666e\u6cb9\u6c14\u4e0a\u6e38\u80a1\u7968\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "HUABAOBIAOPUYOUQISHANGYOUGUPIAORENMINBIC"}, "007845": {"suoxie": "BSJJXCYQC", "name": "\u535a\u65f6\u5b63\u5b63\u4eab\u6301\u6709\u671fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJIJIXIANGCHIYOUQIC"}, "007846": {"suoxie": "HFZZ05NZGDJXYZZSA", "name": "\u534e\u5bcc\u4e2d\u503a-0-5\u5e74\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAI05NIANZHONGGAODENGJIXINYONGZHAIZHISHUA"}, "007847": {"suoxie": "HFZZ05NZGDJXYZZSC", "name": "\u534e\u5bcc\u4e2d\u503a-0-5\u5e74\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAI05NIANZHONGGAODENGJIXINYONGZHAIZHISHUC"}, "007848": {"suoxie": "GFJBHHC", "name": "\u5e7f\u53d1\u805a\u5b9d\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUBAOHUNHEC"}, "007850": {"suoxie": "FZFBTRHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u777f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANRUIHUNHEA"}, "007851": {"suoxie": "FZFBTRHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u777f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANRUIHUNHEC"}, "007852": {"suoxie": "GYRA3GYDKCZZQ", "name": "\u5de5\u94f6\u745e\u5b893\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIAN3GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "007853": {"suoxie": "HSJSJHYLHGP", "name": "\u534e\u5546\u8ba1\u7b97\u673a\u884c\u4e1a\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGJISUANJIHANGYELIANGHUAGUPIAO"}, "007854": {"suoxie": "GDJQXFHH", "name": "\u5149\u5927\u666f\u6c14\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAJINGQIXIANFENGHUNHE"}, "007856": {"suoxie": "YFDZZ800ETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1800ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG800ETFLIANJIEA"}, "007857": {"suoxie": "YFDZZ800ETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1800ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG800ETFLIANJIEC"}, "007858": {"suoxie": "PHFDJZXFQSHB", "name": "\u9e4f\u534e\u6d6e\u52a8\u51c0\u503c\u578b\u53d1\u8d77\u5f0f\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAFUDONGJINGZHIXINGFAQISHIHUOBI"}, "007859": {"suoxie": "PA510NQZCXJRZA", "name": "\u5e73\u5b895-10\u5e74\u671f\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN510NIANQIZHENGCEXINGJINRONGZHAIA"}, "007860": {"suoxie": "PA510NQZCXJRZC", "name": "\u5e73\u5b895-10\u5e74\u671f\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGAN510NIANQIZHENGCEXINGJINRONGZHAIC"}, "007861": {"suoxie": "JYSAYLJKHHA", "name": "\u91d1\u5143\u987a\u5b89\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYUANSHUNANYILIAOJIANKANGHUNHEA"}, "007862": {"suoxie": "JYSAYLJKHHC", "name": "\u91d1\u5143\u987a\u5b89\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYUANSHUNANYILIAOJIANKANGHUNHEC"}, "007863": {"suoxie": "CXLTLHPZHHC", "name": "\u957f\u4fe1\u5229\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLITAILINGHUOPEIZHIHUNHEC"}, "007864": {"suoxie": "DFHHBC", "name": "\u4e1c\u65b9\u7ea2\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGHONGHUOBIC"}, "007865": {"suoxie": "DFHHBD", "name": "\u4e1c\u65b9\u7ea2\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGHONGHUOBID"}, "007866": {"suoxie": "CJHXHBC", "name": "\u521b\u91d1\u5408\u4fe1\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "CHUANGJINHEXINHUOBIC"}, "007867": {"suoxie": "HTBRJTYNDK", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u6cf0\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIJINTAIYINIANDINGKAI"}, "007868": {"suoxie": "HTFHXHBA", "name": "\u6c47\u6dfb\u5bcc\u6c47\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUIXINHUOBIA"}, "007869": {"suoxie": "HTFHXHBB", "name": "\u6c47\u6dfb\u5bcc\u6c47\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUIXINHUOBIB"}, "007870": {"suoxie": "PHZX3GYDKFQSZQ", "name": "\u9e4f\u534e\u5c0a\u4fe13\u4e2a\u6708\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNXIN3GEYUEDINGKAIFAQISHIZHAIQUAN"}, "007871": {"suoxie": "GTHXSGYDKZ", "name": "\u56fd\u6cf0\u60e0\u4eab\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIXIANGSANGEYUEDINGKAIZHAI"}, "007872": {"suoxie": "JXWJCLLHPZHH", "name": "\u91d1\u4fe1\u7a33\u5065\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINWENJIANCELUELINGHUOPEIZHIHUNHE"}, "007873": {"suoxie": "HBKJETFLJA", "name": "\u534e\u5b9d\u79d1\u6280ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOKEJIETFLIANJIEA"}, "007874": {"suoxie": "HBKJETFLJC", "name": "\u534e\u5b9d\u79d1\u6280ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOKEJIETFLIANJIEC"}, "007875": {"suoxie": "GRRXHHA", "name": "\u56fd\u878d\u878d\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGXINGHUNHEA"}, "007876": {"suoxie": "GRRXHHC", "name": "\u56fd\u878d\u878d\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUORONGRONGXINGHUNHEC"}, "007877": {"suoxie": "HSHFCZA", "name": "\u60e0\u5347\u548c\u98ce\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEFENGCHUNZHAIA"}, "007878": {"suoxie": "HSHFCZC", "name": "\u60e0\u5347\u548c\u98ce\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEFENGCHUNZHAIC"}, "007879": {"suoxie": "JSZA3GYDQZQ", "name": "\u5609\u5b9e\u81f4\u5b893\u4e2a\u6708\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIZHIAN3GEYUEDINGQIZHAIQUAN"}, "007880": {"suoxie": "ZQCYZXHHA", "name": "\u6731\u96c0\u4ea7\u4e1a\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUECHANYEZHIXUANHUNHEA"}, "007881": {"suoxie": "ZQCYZXHHC", "name": "\u6731\u96c0\u4ea7\u4e1a\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUECHANYEZHIXUANHUNHEC"}, "007882": {"suoxie": "YFDHS300FYLJC", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u975e\u94f6\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300FEIYINLIANJIEC"}, "007883": {"suoxie": "YFDHS300YYLJC", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u533b\u836f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300YIYAOLIANJIEC"}, "007884": {"suoxie": "YFDHS3GYDKHH", "name": "\u6613\u65b9\u8fbe\u6052\u76db3\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAHENGSHENG3GEYUEDINGKAIHUNHE"}, "007885": {"suoxie": "ZRZZ500ETFLJA", "name": "\u4e2d\u878d\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGZHONGZHENG500ETFLIANJIEA"}, "007886": {"suoxie": "ZRZZ500ETFLJC", "name": "\u4e2d\u878d\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGZHONGZHENG500ETFLIANJIEC"}, "007887": {"suoxie": "DFHQYSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u5143\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIYUANSANNIANCHIYOUHUNHEB"}, "007888": {"suoxie": "NYHLJYZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINYINGZHAIQUAN"}, "007889": {"suoxie": "PYASSNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u8bfa\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGNUODINGKAIZHAIQUAN"}, "007890": {"suoxie": "YHJXLNDKZQ", "name": "\u94f6\u6cb3\u805a\u661f\u4e24\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJUXINGLIANGNIANDINGKAIZHAIQUAN"}, "007893": {"suoxie": "PAGZJXHHA", "name": "\u5e73\u5b89\u4f30\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANGUZHIJINGXUANHUNHEA"}, "007894": {"suoxie": "PAGZJXHHC", "name": "\u5e73\u5b89\u4f30\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANGUZHIJINGXUANHUNHEC"}, "007895": {"suoxie": "JSJZCZHH", "name": "\u5609\u5b9e\u4ef7\u503c\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHICHENGZHANGHUNHE"}, "007896": {"suoxie": "YFDYXDZCSGYCYHHFOFA", "name": "\u6613\u65b9\u8fbe\u4f18\u9009\u591a\u8d44\u4ea7\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUXUANDUOZICHANSANGEYUECHIYOUHUNHEFOFA"}, "007897": {"suoxie": "YFDYXDZCSGYCYHHFOFC", "name": "\u6613\u65b9\u8fbe\u4f18\u9009\u591a\u8d44\u4ea7\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUXUANDUOZICHANSANGEYUECHIYOUHUNHEFOFC"}, "007898": {"suoxie": "FGZCJXHHFOF", "name": "\u5bcc\u56fd\u667a\u8bda\u7cbe\u9009\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHICHENGJINGXUANHUNHEFOF"}, "007901": {"suoxie": "HTFZDZA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUZHONGDUANZHAIA"}, "007902": {"suoxie": "HTFZDZC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUZHONGDUANZHAIC"}, "007903": {"suoxie": "CCLHXPGP", "name": "\u957f\u57ce\u91cf\u5316\u5c0f\u76d8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGLIANGHUAXIAOPANGUPIAO"}, "007904": {"suoxie": "GFRYJQ3GYCYHHFOFA", "name": "\u5e7f\u53d1\u9510\u610f\u8fdb\u53d63\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYIJINQU3GEYUECHIYOUHUNHEFOFA"}, "007907": {"suoxie": "FRFK13NGKZCZC", "name": "\u5bcc\u8363\u5bcc\u5f001-3\u5e74\u56fd\u5f00\u503a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUKAI13NIANGUOKAIZHAICHUNZHAIC"}, "007908": {"suoxie": "ZSTY3GYDKZA", "name": "\u62db\u5546\u6dfb\u97f53\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUN3GEYUEDINGKAIZHAIA"}, "007909": {"suoxie": "ZSTY3GYDKZC", "name": "\u62db\u5546\u6dfb\u97f53\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUN3GEYUEDINGKAIZHAIC"}, "007910": {"suoxie": "DCYSJSQHETFLJA", "name": "\u5927\u6210\u6709\u8272\u91d1\u5c5e\u671f\u8d27ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "DACHENGYOUSEJINSHUQIHUOETFLIANJIEA"}, "007911": {"suoxie": "DCYSJSQHETFLJC", "name": "\u5927\u6210\u6709\u8272\u91d1\u5c5e\u671f\u8d27ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "DACHENGYOUSEJINSHUQIHUOETFLIANJIEC"}, "007912": {"suoxie": "XHXRXZDZB", "name": "\u65b0\u534e\u946b\u65e5\u4eab\u4e2d\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINHUAXINRIXIANGZHONGDUANZHAIB"}, "007913": {"suoxie": "CTZGFHLNDKZA", "name": "\u8d22\u901a\u8d44\u7ba1\u4e30\u548c\u4e24\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANFENGHELIANGNIANDINGKAIZHAIA"}, "007914": {"suoxie": "CTZGFHLNDKZC", "name": "\u8d22\u901a\u8d44\u7ba1\u4e30\u548c\u4e24\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANFENGHELIANGNIANDINGKAIZHAIC"}, "007915": {"suoxie": "CTZGHFDZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u798f\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGFUDUANZHAIA"}, "007916": {"suoxie": "CTZGHFDZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u798f\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGFUDUANZHAIC"}, "007920": {"suoxie": "NDDZZQC", "name": "\u8bfa\u5fb7\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NUODEDUANZHAIZHAIQUANC"}, "007923": {"suoxie": "FZFBTXHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANXINHUNHEA"}, "007924": {"suoxie": "FZFBTXHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANXINHUNHEC"}, "007925": {"suoxie": "PAXXHHE", "name": "\u5e73\u5b89\u946b\u4eab\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANXINXIANGHUNHEE"}, "007926": {"suoxie": "WJJXZDZC", "name": "\u4e07\u5bb6\u5bb6\u4eab\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "WANJIAJIAXIANGZHONGDUANZHAIC"}, "007927": {"suoxie": "WJJXZDZE", "name": "\u4e07\u5bb6\u5bb6\u4eab\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "WANJIAJIAXIANGZHONGDUANZHAIE"}, "007928": {"suoxie": "ZJXRLNDKZ", "name": "\u4e2d\u52a0\u4eab\u6da6\u4e24\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAXIANGRUNLIANGNIANDINGKAIZHAI"}, "007930": {"suoxie": "CHWXZQA", "name": "\u6df3\u539a\u7a33\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENXINZHAIQUANA"}, "007931": {"suoxie": "CHWXZQC", "name": "\u6df3\u539a\u7a33\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENXINZHAIQUANC"}, "007932": {"suoxie": "PHZZ500ETFLJA", "name": "\u9e4f\u534e\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ETFLIANJIEA"}, "007933": {"suoxie": "JHYTYXSGYCYHHFOFA", "name": "\u5609\u5408\u6c38\u6cf0\u4f18\u9009\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEYONGTAIYOUXUANSANGEYUECHIYOUHUNHEFOFA"}, "007934": {"suoxie": "JHYTYXSGYCYHHFOFC", "name": "\u5609\u5408\u6c38\u6cf0\u4f18\u9009\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEYONGTAIYOUXUANSANGEYUECHIYOUHUNHEFOFC"}, "007935": {"suoxie": "PAHLCZA", "name": "\u5e73\u5b89\u60e0\u6f9c\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUILANCHUNZHAIA"}, "007936": {"suoxie": "PAHLCZC", "name": "\u5e73\u5b89\u60e0\u6f9c\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUILANCHUNZHAIC"}, "007937": {"suoxie": "HXSLDPQHETFLJA", "name": "\u534e\u590f\u9972\u6599\u8c46\u7c95\u671f\u8d27ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIASILIAODOUPOQIHUOETFLIANJIEA"}, "007938": {"suoxie": "HXSLDPQHETFLJC", "name": "\u534e\u590f\u9972\u6599\u8c46\u7c95\u671f\u8d27ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIASILIAODOUPOQIHUOETFLIANJIEC"}, "007939": {"suoxie": "HXWGJXHHC", "name": "\u534e\u590f\u7f51\u8d2d\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAWANGGOUJINGXUANHUNHEC"}, "007941": {"suoxie": "HSQHHYCZZQA", "name": "\u6052\u751f\u524d\u6d77\u6052\u626c\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYANGCHUNZHAIZHAIQUANA"}, "007942": {"suoxie": "HSQHHYCZZQC", "name": "\u6052\u751f\u524d\u6d77\u6052\u626c\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYANGCHUNZHAIZHAIQUANC"}, "007943": {"suoxie": "FADZZ500ZSZQ", "name": "\u5bcc\u5b89\u8fbe\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUANDAZHONGZHENG500ZHISHUZENGQIANG"}, "007944": {"suoxie": "YYQYSNDK", "name": "\u6c38\u8d62\u4e7e\u5143\u4e09\u5e74\u5b9a\u5f00", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGQIANYUANSANNIANDINGKAI"}, "007945": {"suoxie": "JSCCGGJYLHPZC", "name": "\u666f\u987a\u957f\u57ce\u6539\u9769\u673a\u9047\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGGAIGEJIYULINGHUOPEIZHIC"}, "007946": {"suoxie": "DCZZ13NGKZZSA", "name": "\u5927\u6210\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGZHONGZHAI13NIANGUOKAIZHAIZHISHUA"}, "007947": {"suoxie": "DCZZ13NGKZZSC", "name": "\u5927\u6210\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGZHONGZHAI13NIANGUOKAIZHAIZHISHUC"}, "007948": {"suoxie": "HTFSA39GYDKZ", "name": "\u6c47\u6dfb\u5bcc\u76db\u5b8939\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUSHENGAN39GEYUEDINGKAIZHAI"}, "007949": {"suoxie": "FGZLCZZQ", "name": "\u5bcc\u56fd\u6cfd\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZELICHUNZHAIZHAIQUAN"}, "007950": {"suoxie": "ZSLHJXGPC", "name": "\u62db\u5546\u91cf\u5316\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGLIANGHUAJINGXUANGUPIAOC"}, "007951": {"suoxie": "ZSXYZQZQC", "name": "\u62db\u5546\u4fe1\u7528\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXINYONGZENGQIANGZHAIQUANC"}, "007952": {"suoxie": "ZSCJDSJGPC", "name": "\u62db\u5546\u8d22\u7ecf\u5927\u6570\u636e\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGCAIJINGDASHUJUGUPIAOC"}, "007953": {"suoxie": "PAHWCZ", "name": "\u5e73\u5b89\u60e0\u6587\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIWENCHUNZHAI"}, "007954": {"suoxie": "PAHYCZ", "name": "\u5e73\u5b89\u60e0\u6d8c\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIYONGCHUNZHAI"}, "007955": {"suoxie": "MSJYXXZQD", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u4eab\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINXIANGZHAIQUAND"}, "007956": {"suoxie": "PHWLDZC", "name": "\u9e4f\u534e\u7a33\u5229\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUAWENLIDUANZHAIC"}, "007957": {"suoxie": "HBBHZQ", "name": "\u534e\u5b9d\u5b9d\u60e0\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOHUIZHAIQUAN"}, "007958": {"suoxie": "HTBRYTSGYDKZ", "name": "\u534e\u6cf0\u67cf\u745e\u76ca\u901a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIYITONGSANGEYUEDINGKAIZHAI"}, "007959": {"suoxie": "FZFBTHHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u6052\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANHENGHUNHEA"}, "007960": {"suoxie": "FZFBTHHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u6052\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANHENGHUNHEC"}, "007961": {"suoxie": "PYASRRYHBE", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u76c8\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIYINGHUOBIE"}, "007962": {"suoxie": "BSZZ35ZJRZZSA", "name": "\u535a\u65f6\u4e2d\u503a3-5\u653f\u91d1\u878d\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35ZHENGJINRONGZHAIZHISHUA"}, "007963": {"suoxie": "BSZZ35ZJRZZSC", "name": "\u535a\u65f6\u4e2d\u503a3-5\u653f\u91d1\u878d\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI35ZHENGJINRONGZHAIZHISHUC"}, "007964": {"suoxie": "HBBKZQC", "name": "\u534e\u5b9d\u5b9d\u5eb7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOBAOKANGZHAIQUANC"}, "007965": {"suoxie": "MSJYPZXFGPA", "name": "\u6c11\u751f\u52a0\u94f6\u54c1\u8d28\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINPINZHIXIAOFEIGUPIAOA"}, "007966": {"suoxie": "MSJYPZXFGPC", "name": "\u6c11\u751f\u52a0\u94f6\u54c1\u8d28\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINPINZHIXIAOFEIGUPIAOC"}, "007967": {"suoxie": "DCHJYNDKZQ", "name": "\u5927\u6210\u60e0\u5609\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIJIAYINIANDINGKAIZHAIQUAN"}, "007968": {"suoxie": "HTBRYJJXA", "name": "\u534e\u6cf0\u67cf\u745e\u7814\u7a76\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYANJIUJINGXUANA"}, "007969": {"suoxie": "XBLDDZCZC", "name": "\u897f\u90e8\u5229\u5f97\u5f97\u5c0a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEDEZUNCHUNZHAIC"}, "007970": {"suoxie": "GSABAZ39GYDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u6cfd39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANZE39GEYUEDINGKAIZHAI"}, "007971": {"suoxie": "HTBXHLZDZA", "name": "\u534e\u6cf0\u4fdd\u5174\u6052\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAOXINGHENGLIZHONGDUANZHAIA"}, "007972": {"suoxie": "HTBXHLZDZC", "name": "\u534e\u6cf0\u4fdd\u5174\u6052\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAOXINGHENGLIZHONGDUANZHAIC"}, "007975": {"suoxie": "ZSPSQQPZQDIIFOFMYXH", "name": "\u62db\u5546\u666e\u76db\u5168\u7403\u914d\u7f6e(QDII-FOF)\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "ZHAOSHANGPUSHENGQUANQIUPEIZHIQDIIFOFMEIYUANXIANHUI"}, "007976": {"suoxie": "YFDHJZTRMBC", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1\u4e3b\u9898\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDAHUANGJINZHUTIRENMINBIC"}, "007977": {"suoxie": "YFDHJZTMYXHA", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1\u4e3b\u9898\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "YIFANGDAHUANGJINZHUTIMEIYUANXIANHUIA"}, "007978": {"suoxie": "YFDHJZTMYXHC", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1\u4e3b\u9898\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "YIFANGDAHUANGJINZHUTIMEIYUANXIANHUIC"}, "007979": {"suoxie": "WJHX39GYDKZ", "name": "\u4e07\u5bb6\u60e0\u4eab39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAHUIXIANG39GEYUEDINGKAIZHAI"}, "007981": {"suoxie": "HTHTRXCZA", "name": "\u7ea2\u5854\u7ea2\u571f\u745e\u7965\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTURUIXIANGCHUNZHAIA"}, "007982": {"suoxie": "HTHTRXCZC", "name": "\u7ea2\u5854\u7ea2\u571f\u745e\u7965\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTURUIXIANGCHUNZHAIC"}, "007983": {"suoxie": "SWLXZZYFCX100ETFLJA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u7814\u53d1\u521b\u65b0100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGYANFACHUANGXIN100ETFLIANJIEA"}, "007984": {"suoxie": "SWLXZZYFCX100ETFLJC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u7814\u53d1\u521b\u65b0100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGYANFACHUANGXIN100ETFLIANJIEC"}, "007985": {"suoxie": "BSFYCZ", "name": "\u535a\u65f6\u5bcc\u60a6\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYUECHUNZHAI"}, "007986": {"suoxie": "JSZL3GYDQCZZQ", "name": "\u5609\u5b9e\u81f4\u79843\u4e2a\u6708\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHILU3GEYUEDINGQICHUNZHAIZHAIQUAN"}, "007987": {"suoxie": "PHFQZQ", "name": "\u9e4f\u534e\u4e30\u5e86\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGQINGZHAIQUAN"}, "007988": {"suoxie": "RTTH63GYDKZQA", "name": "\u878d\u901a\u901a\u605263\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGHENG63GEYUEDINGKAIZHAIQUANA"}, "007989": {"suoxie": "RTTH63GYDKZQC", "name": "\u878d\u901a\u901a\u605263\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGHENG63GEYUEDINGKAIZHAIQUANC"}, "007990": {"suoxie": "FGHYSNDKZA", "name": "\u5bcc\u56fd\u6c47\u8fdc\u4e09\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHUIYUANSANNIANDINGKAIZHAIA"}, "007991": {"suoxie": "FGHYSNDKZC", "name": "\u5bcc\u56fd\u6c47\u8fdc\u4e09\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHUIYUANSANNIANDINGKAIZHAIC"}, "007992": {"suoxie": "HXZZQZZQGSETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "007993": {"suoxie": "HXZZQZZQGSETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "007994": {"suoxie": "HXZZ500ZSZQA", "name": "\u534e\u590f\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ZHISHUZENGQIANGA"}, "007995": {"suoxie": "HXZZ500ZSZQC", "name": "\u534e\u590f\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ZHISHUZENGQIANGC"}, "007996": {"suoxie": "BSFSCZZQ", "name": "\u535a\u65f6\u5bcc\u987a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUSHUNCHUNZHAIZHAIQUAN"}, "007997": {"suoxie": "YFDNNHQYNDKZA", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u79cb\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGQIUYINIANDINGKAIZHAIA"}, "007998": {"suoxie": "YFDNNHQYNDKZC", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u79cb\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGQIUYINIANDINGKAIZHAIC"}, "007999": {"suoxie": "GLAHL63GYDKZA", "name": "\u56fd\u8054\u5b89\u6052\u522963\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANHENGLI63GEYUEDINGKAIZHAIA"}, "008000": {"suoxie": "GLAHL63GYDKZC", "name": "\u56fd\u8054\u5b89\u6052\u522963\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANHENGLI63GEYUEDINGKAIZHAIC"}, "008001": {"suoxie": "PHZZ500ETFLJC", "name": "\u9e4f\u534e\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ETFLIANJIEC"}, "008002": {"suoxie": "YHWS39GYDKZ", "name": "\u94f6\u534e\u7a33\u665f39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAWENSHENG39GEYUEDINGKAIZHAI"}, "008003": {"suoxie": "DCTJSNDKZQA", "name": "\u5927\u6210\u901a\u5609\u4e09\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGTONGJIASANNIANDINGKAIZHAIQUANA"}, "008004": {"suoxie": "DCTJSNDKZQC", "name": "\u5927\u6210\u901a\u5609\u4e09\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGTONGJIASANNIANDINGKAIZHAIQUANC"}, "008008": {"suoxie": "YFDWJSYZQC", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAWENJIANSHOUYIZHAIQUANC"}, "008009": {"suoxie": "HSGDZBZZGP", "name": "\u534e\u5546\u9ad8\u7aef\u88c5\u5907\u5236\u9020\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGGAODUANZHUANGBEIZHIZAOGUPIAO"}, "008010": {"suoxie": "QHLHRYDZA", "name": "\u524d\u6d77\u8054\u5408\u6da6\u76c8\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "QIANHAILIANHERUNYINGDUANZHAIA"}, "008011": {"suoxie": "QHLHRYDZC", "name": "\u524d\u6d77\u8054\u5408\u6da6\u76c8\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "QIANHAILIANHERUNYINGDUANZHAIC"}, "008012": {"suoxie": "QHLHCF87GYDKZA", "name": "\u524d\u6d77\u8054\u5408\u6df3\u4e3087\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHECHUNFENG87GEYUEDINGKAIZHAIA"}, "008013": {"suoxie": "QHLHCF87GYDKZC", "name": "\u524d\u6d77\u8054\u5408\u6df3\u4e3087\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHECHUNFENG87GEYUEDINGKAIZHAIC"}, "008014": {"suoxie": "THXLSNDK", "name": "\u5929\u5f18\u946b\u5229\u4e09\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINLISANNIANDINGKAI"}, "008015": {"suoxie": "JSZZ35NGKZZSA", "name": "\u5609\u5b9e\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHAI35NIANGUOKAIZHAIZHISHUA"}, "008016": {"suoxie": "JSZZ35NGKZZSC", "name": "\u5609\u5b9e\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHAI35NIANGUOKAIZHAIZHISHUC"}, "008017": {"suoxie": "GTHXSNDKZ", "name": "\u56fd\u6cf0\u60e0\u4fe1\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIXINSANNIANDINGKAIZHAI"}, "008018": {"suoxie": "HFAX39GYDKZA", "name": "\u534e\u5bcc\u5b89\u517439\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUANXING39GEYUEDINGKAIZHAIA"}, "008019": {"suoxie": "HFAX39GYDKZC", "name": "\u534e\u5bcc\u5b89\u517439\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUANXING39GEYUEDINGKAIZHAIC"}, "008020": {"suoxie": "HFZZRGZNCYETFLJA", "name": "\u534e\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4ea7\u4e1aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGRENGONGZHINENGCHANYEETFLIANJIEA"}, "008021": {"suoxie": "HFZZRGZNCYETFLJC", "name": "\u534e\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4ea7\u4e1aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGRENGONGZHINENGCHANYEETFLIANJIEC"}, "008022": {"suoxie": "JXDZZQF", "name": "\u5efa\u4fe1\u77ed\u503a\u503a\u5238F", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINDUANZHAIZHAIQUANF"}, "008025": {"suoxie": "HTFWJZZHHA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANZENGZHANGHUNHEA"}, "008026": {"suoxie": "HTFWJZZHHC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANZENGZHANGHUNHEC"}, "008027": {"suoxie": "GYTH39GYDKZQA", "name": "\u5de5\u94f6\u6cf0\u548c39\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINTAIHE39GEYUEDINGKAIZHAIQUANA"}, "008028": {"suoxie": "SWLXATGL63GYDKZ", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u5e7f\u522963\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIGUANGLI63GEYUEDINGKAIZHAI"}, "008030": {"suoxie": "NYHLJYZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINYIZHAIQUAN"}, "008031": {"suoxie": "CJHXHJSGYDK", "name": "\u521b\u91d1\u5408\u4fe1\u6c47\u5609\u4e09\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINHUIJIASANGEYUEDINGKAI"}, "008032": {"suoxie": "HFTYSSNDKZQ", "name": "\u6d77\u5bcc\u901a\u88d5\u6607\u4e09\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGYUSHENGSANNIANDINGKAIZHAIQUAN"}, "008033": {"suoxie": "ZJKYHHA", "name": "\u4e2d\u52a0\u79d1\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEYINGHUNHEA"}, "008034": {"suoxie": "ZJKYHHC", "name": "\u4e2d\u52a0\u79d1\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEYINGHUNHEC"}, "008035": {"suoxie": "FCHLZQA", "name": "\u8702\u5de2\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOHENGLIZHAIQUANA"}, "008036": {"suoxie": "FCHLZQC", "name": "\u8702\u5de2\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOHENGLIZHAIQUANC"}, "008037": {"suoxie": "XYXFCZHHA", "name": "\u5174\u94f6\u5148\u950b\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINXIANFENGCHENGZHANGHUNHEA"}, "008038": {"suoxie": "XYXFCZHHC", "name": "\u5174\u94f6\u5148\u950b\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINXIANFENGCHENGZHANGHUNHEC"}, "008039": {"suoxie": "NFCL3GYDKZ", "name": "\u5357\u65b9\u521b\u52293\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHUANGLI3GEYUEDINGKAIZHAI"}, "008040": {"suoxie": "PH05NLLFQSZQ", "name": "\u9e4f\u534e0-5\u5e74\u5229\u7387\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUA05NIANLILVFAQISHIZHAIQUAN"}, "008041": {"suoxie": "CXXLBNDKHHC", "name": "\u957f\u4fe1\u5148\u5229\u534a\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINXIANLIBANNIANDINGKAIHUNHEC"}, "008042": {"suoxie": "XYZZYH50JRZZSA", "name": "\u5174\u4e1a\u4e2d\u8bc1\u94f6\u884c50\u91d1\u878d\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHENGYINHANG50JINRONGZHAIZHISHUA"}, "008043": {"suoxie": "XYZZYH50JRZZSC", "name": "\u5174\u4e1a\u4e2d\u8bc1\u94f6\u884c50\u91d1\u878d\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHENGYINHANG50JINRONGZHAIZHISHUC"}, "008044": {"suoxie": "BYZQHBZQA", "name": "\u535a\u8fdc\u589e\u5f3a\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOYUANZENGQIANGHUIBAOZHAIQUANA"}, "008045": {"suoxie": "BYZQHBZQC", "name": "\u535a\u8fdc\u589e\u5f3a\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOYUANZENGQIANGHUIBAOZHAIQUANC"}, "008046": {"suoxie": "ZRRJ39GYDKZA", "name": "\u4e2d\u878d\u777f\u560939\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIJIA39GEYUEDINGKAIZHAIA"}, "008047": {"suoxie": "ZRRJ39GYDKZC", "name": "\u4e2d\u878d\u777f\u560939\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIJIA39GEYUEDINGKAIZHAIC"}, "008048": {"suoxie": "ZRRX86GYDKZA", "name": "\u4e2d\u878d\u777f\u4eab86\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIXIANG86GEYUEDINGKAIZHAIA"}, "008049": {"suoxie": "ZRRX86GYDKZC", "name": "\u4e2d\u878d\u777f\u4eab86\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGRUIXIANG86GEYUEDINGKAIZHAIC"}, "008050": {"suoxie": "TTHZHHA", "name": "\u540c\u6cf0\u6167\u62e9\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIZEHUNHEA"}, "008051": {"suoxie": "TTHZHHC", "name": "\u540c\u6cf0\u6167\u62e9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIZEHUNHEC"}, "008052": {"suoxie": "GYWC100ETFLJA", "name": "\u5de5\u94f6\u6e7e\u521b100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINWANCHUANG100ETFLIANJIEA"}, "008053": {"suoxie": "GYWC100ETFLJC", "name": "\u5de5\u94f6\u6e7e\u521b100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINWANCHUANG100ETFLIANJIEC"}, "008054": {"suoxie": "HTFZZ710NGKZA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI710NIANGUOKAIZHAIA"}, "008055": {"suoxie": "HTFZZ710NGKZC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGZHAI710NIANGUOKAIZHAIC"}, "008056": {"suoxie": "NFSZ50ZQA", "name": "\u5357\u65b9\u4e0a\u8bc150\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG50ZENGQIANGA"}, "008057": {"suoxie": "NFSZ50ZQC", "name": "\u5357\u65b9\u4e0a\u8bc150\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG50ZENGQIANGC"}, "008058": {"suoxie": "PHXXWJHHA", "name": "\u9e4f\u534e\u946b\u4eab\u7a33\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAXINXIANGWENJIANHUNHEA"}, "008059": {"suoxie": "PHXXWJHHC", "name": "\u9e4f\u534e\u946b\u4eab\u7a33\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAXINXIANGWENJIANHUNHEC"}, "008060": {"suoxie": "JSCCJZBJLHPZHH", "name": "\u666f\u987a\u957f\u57ce\u4ef7\u503c\u8fb9\u9645\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGJIAZHIBIANJILINGHUOPEIZHIHUNHE"}, "008061": {"suoxie": "HSHXHHA", "name": "\u60e0\u5347\u60e0\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIXINHUNHEA"}, "008062": {"suoxie": "HSHXHHC", "name": "\u60e0\u5347\u60e0\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIXINHUNHEC"}, "008063": {"suoxie": "HTFDPHXZCHHA", "name": "\u6c47\u6dfb\u5bcc\u5927\u76d8\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUDAPANHEXINZICHANHUNHEA"}, "008064": {"suoxie": "JXRXSGYDKZ", "name": "\u5efa\u4fe1\u777f\u4fe1\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIXINSANGEYUEDINGKAIZHAI"}, "008065": {"suoxie": "HTFZPJJCZHHA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u79ef\u6781\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANJIJICHENGZHANGHUNHEA"}, "008066": {"suoxie": "HTFZPJJCZHHC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u79ef\u6781\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANJIJICHENGZHANGHUNHEC"}, "008067": {"suoxie": "YXYFFHA", "name": "\u5706\u4fe1\u6c38\u4e30\u4e30\u548cA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YUANXINYONGFENGFENGHEA"}, "008068": {"suoxie": "YXYFFHC", "name": "\u5706\u4fe1\u6c38\u4e30\u4e30\u548cC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YUANXINYONGFENGFENGHEC"}, "008069": {"suoxie": "PYFLZQZA", "name": "\u9e4f\u626c\u5bcc\u5229\u589e\u5f3a\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGFULIZENGQIANGZHAIA"}, "008070": {"suoxie": "PYFLZQZC", "name": "\u9e4f\u626c\u5bcc\u5229\u589e\u5f3a\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGFULIZENGQIANGZHAIC"}, "008071": {"suoxie": "CXLTLHPZHHE", "name": "\u957f\u4fe1\u5229\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLITAILINGHUOPEIZHIHUNHEE"}, "008072": {"suoxie": "JSCCCYBZZZQ", "name": "\u666f\u987a\u957f\u57ce\u521b\u4e1a\u677f\u7efc\u6307\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGCHUANGYEBANZONGZHIZENGQIANG"}, "008075": {"suoxie": "ZSHXYXGPA", "name": "\u62db\u5546\u6838\u5fc3\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGHEXINYOUXUANGUPIAOA"}, "008076": {"suoxie": "ZSHXYXGPC", "name": "\u62db\u5546\u6838\u5fc3\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGHEXINYOUXUANGUPIAOC"}, "008077": {"suoxie": "JTTYLHJZHHA", "name": "\u4e5d\u6cf0\u5929\u5955\u91cf\u5316\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAITIANYILIANGHUAJIAZHIHUNHEA"}, "008079": {"suoxie": "NDDLJXFOF", "name": "\u8bfa\u5fb7\u5927\u7c7b\u7cbe\u9009(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEDALEIJINGXUANFOF"}, "008080": {"suoxie": "NFCYZDZE", "name": "\u5357\u65b9\u521d\u5143\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGCHUYUANZHONGDUANZHAIE"}, "008081": {"suoxie": "HTFXYZQ", "name": "\u6c47\u6dfb\u5bcc\u946b\u8fdc\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINYUANZHAIQUAN"}, "008082": {"suoxie": "GSABYJJXHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOYANJIUJINGXUANHUNHEA"}, "008083": {"suoxie": "GSABYJJXHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOYANJIUJINGXUANHUNHEC"}, "008084": {"suoxie": "HFTXJZZGPC", "name": "\u6d77\u5bcc\u901a\u5148\u8fdb\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGXIANJINZHIZAOGUPIAOC"}, "008085": {"suoxie": "HFTXJZZGPA", "name": "\u6d77\u5bcc\u901a\u5148\u8fdb\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HAIFUTONGXIANJINZHIZAOGUPIAOA"}, "008086": {"suoxie": "HXZZ5GTXZTETFLJA", "name": "\u534e\u590f\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG5GTONGXINZHUTIETFLIANJIEA"}, "008087": {"suoxie": "HXZZ5GTXZTETFLJC", "name": "\u534e\u590f\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG5GTONGXINZHUTIETFLIANJIEC"}, "008088": {"suoxie": "HXZZQZFDCETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u623f\u5730\u4ea7ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIFANGDICHANETFLIANJIEA"}, "008089": {"suoxie": "HXZZQZFDCETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u623f\u5730\u4ea7ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIFANGDICHANETFLIANJIEC"}, "008091": {"suoxie": "ZXBCHLJXHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7ea2\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGHONGLIJINGXUANHUNHEA"}, "008092": {"suoxie": "ZXBCHLJXHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7ea2\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGHONGLIJINGXUANHUNHEC"}, "008093": {"suoxie": "TTHXHHFQSA", "name": "\u540c\u6cf0\u6167\u9009\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIXUANHUNHEFAQISHIA"}, "008094": {"suoxie": "TTHXHHFQSC", "name": "\u540c\u6cf0\u6167\u9009\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIXUANHUNHEFAQISHIC"}, "008095": {"suoxie": "ZYYTJXZQQDIIRMBA", "name": "\u4e2d\u94f6\u4e9a\u592a\u7cbe\u9009\u503a\u5238(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "ZHONGYINYATAIJINGXUANZHAIQUANQDIIRENMINBIA"}, "008096": {"suoxie": "ZYYTJXZQQDIIRMBC", "name": "\u4e2d\u94f6\u4e9a\u592a\u7cbe\u9009\u503a\u5238(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "ZHONGYINYATAIJINGXUANZHAIQUANQDIIRENMINBIC"}, "008097": {"suoxie": "ZYYTJXZQQDIIMYA", "name": "\u4e2d\u94f6\u4e9a\u592a\u7cbe\u9009\u503a\u5238(QDII)\u7f8e\u5143A", "style": "QDII", "pinyin": "ZHONGYINYATAIJINGXUANZHAIQUANQDIIMEIYUANA"}, "008098": {"suoxie": "ZYYTJXZQQDIIMYC", "name": "\u4e2d\u94f6\u4e9a\u592a\u7cbe\u9009\u503a\u5238(QDII)\u7f8e\u5143C", "style": "QDII", "pinyin": "ZHONGYINYATAIJINGXUANZHAIQUANQDIIMEIYUANC"}, "008099": {"suoxie": "GFJZLXHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHILINGXIANHUNHEA"}, "008102": {"suoxie": "ZJXF87GYDKZ", "name": "\u4e2d\u91d1\u946b\u798f87\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINFU87GEYUEDINGKAIZHAI"}, "008104": {"suoxie": "ZJXY1NDKZA", "name": "\u4e2d\u91d1\u946b\u88d51\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINYU1NIANDINGKAIZHAIA"}, "008105": {"suoxie": "ZJXY1NDKZC", "name": "\u4e2d\u91d1\u946b\u88d51\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINYU1NIANDINGKAIZHAIC"}, "008106": {"suoxie": "BSFRCZZQC", "name": "\u535a\u65f6\u5bcc\u745e\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFURUICHUNZHAIZHAIQUANC"}, "008107": {"suoxie": "HSYYYLHYGP", "name": "\u534e\u5546\u533b\u836f\u533b\u7597\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUASHANGYIYAOYILIAOHANGYEGUPIAO"}, "008108": {"suoxie": "GLADZZQA", "name": "\u56fd\u8054\u5b89\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOLIANANDUANZHAIZHAIQUANA"}, "008109": {"suoxie": "GLADZZQC", "name": "\u56fd\u8054\u5b89\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOLIANANDUANZHAIZHAIQUANC"}, "008110": {"suoxie": "JTKYJZHHA", "name": "\u4e5d\u6cf0\u79d1\u76c8\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIKEYINGJIAZHIHUNHEA"}, "008111": {"suoxie": "YHXYJX18GYDKZ", "name": "\u94f6\u534e\u4fe1\u7528\u7cbe\u900918\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJINGXUAN18GEYUEDINGKAIZHAI"}, "008112": {"suoxie": "ZTZZ500ZSZQA", "name": "\u4e2d\u6cf0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIZHONGZHENG500ZHISHUZENGQIANGA"}, "008113": {"suoxie": "ZTZZ500ZSZQC", "name": "\u4e2d\u6cf0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIZHONGZHENG500ZHISHUZENGQIANGC"}, "008114": {"suoxie": "THZZHLDBD100A", "name": "\u5929\u5f18\u4e2d\u8bc1\u7ea2\u5229\u4f4e\u6ce2\u52a8100A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHONGLIDIBODONG100A"}, "008115": {"suoxie": "THZZHLDBD100C", "name": "\u5929\u5f18\u4e2d\u8bc1\u7ea2\u5229\u4f4e\u6ce2\u52a8100C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHONGLIDIBODONG100C"}, "008116": {"suoxie": "YHHSGTJXHH", "name": "\u94f6\u534e\u6caa\u6df1\u80a1\u901a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAHUSHENGUTONGJINGXUANHUNHE"}, "008117": {"suoxie": "BSWX39GYDKZ", "name": "\u535a\u65f6\u7a33\u6b2339\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIWENXIN39GEYUEDINGKAIZHAI"}, "008118": {"suoxie": "JSMQJXYNDKZ", "name": "\u5609\u5b9e\u6c11\u4f01\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIMINQIJINGXUANYINIANDINGKAIZHAI"}, "008119": {"suoxie": "PHJXHH", "name": "\u9e4f\u534e\u91d1\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGHUAJINXIANGHUNHE"}, "008120": {"suoxie": "WJZZCXHHA", "name": "\u4e07\u5bb6\u81ea\u4e3b\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZIZHUCHUANGXINHUNHEA"}, "008121": {"suoxie": "WJZZCXHHC", "name": "\u4e07\u5bb6\u81ea\u4e3b\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZIZHUCHUANGXINHUNHEC"}, "008122": {"suoxie": "NFHYDZA", "name": "\u5357\u65b9\u7693\u5143\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGHAOYUANDUANZHAIA"}, "008123": {"suoxie": "NFHYDZC", "name": "\u5357\u65b9\u7693\u5143\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGHAOYUANDUANZHAIC"}, "008124": {"suoxie": "ZYZZ500ZSZQC", "name": "\u4e2d\u90ae\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYOUZHONGZHENG500ZHISHUZENGQIANGC"}, "008125": {"suoxie": "CJHXZZ13NGKZA", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZHONGZHAI13NIANGUOKAIZHAIA"}, "008126": {"suoxie": "CJHXZZ13NGKZC", "name": "\u521b\u91d1\u5408\u4fe1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZHONGZHAI13NIANGUOKAIZHAIC"}, "008127": {"suoxie": "GFQSYXLHPZHHC", "name": "\u5e7f\u53d1\u8d8b\u52bf\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAQUSHIYOUXUANLINGHUOPEIZHIHUNHEC"}, "008128": {"suoxie": "XCCYGPXA", "name": "\u6e58\u8d22\u957f\u6e90\u80a1\u7968\u578bA", "style": "\u80a1\u7968\u578b", "pinyin": "XIANGCAICHANGYUANGUPIAOXINGA"}, "008129": {"suoxie": "XCCYGPXC", "name": "\u6e58\u8d22\u957f\u6e90\u80a1\u7968\u578bC", "style": "\u80a1\u7968\u578b", "pinyin": "XIANGCAICHANGYUANGUPIAOXINGC"}, "008130": {"suoxie": "GFHY66GYDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u4f1866\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIYOU66GEYUEDINGQIKAIFANGZHAIQUAN"}, "008131": {"suoxie": "JSCCJZYSHH", "name": "\u666f\u987a\u957f\u57ce\u7ade\u4e89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGZHENGYOUSHIHUNHE"}, "008132": {"suoxie": "PHJZQDHH", "name": "\u9e4f\u534e\u4ef7\u503c\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHIQUDONGHUNHE"}, "008133": {"suoxie": "HAYZSHHH", "name": "\u534e\u5b89\u4f18\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUZHISHENGHUOHUNHE"}, "008134": {"suoxie": "PHYXJZGP", "name": "\u9e4f\u534e\u4f18\u9009\u4ef7\u503c\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAYOUXUANJIAZHIGUPIAO"}, "008135": {"suoxie": "HCWLJZXF", "name": "\u534e\u5bb8\u672a\u6765\u4ef7\u503c\u5148\u950b", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUACHENWEILAIJIAZHIXIANFENG"}, "008136": {"suoxie": "JTKYJZHHC", "name": "\u4e5d\u6cf0\u79d1\u76c8\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIKEYINGJIAZHIHUNHEC"}, "008137": {"suoxie": "JTTYLHJZHHC", "name": "\u4e5d\u6cf0\u5929\u5955\u91cf\u5316\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAITIANYILIANGHUAJIAZHIHUNHEC"}, "008138": {"suoxie": "FGLTYSHH", "name": "\u5bcc\u56fd\u9f99\u5934\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOLONGTOUYOUSHIHUNHE"}, "008139": {"suoxie": "XYYNZGDJZ", "name": "\u946b\u5143\u4e00\u5e74\u4e2d\u9ad8\u7b49\u7ea7\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANYINIANZHONGGAODENGJIZHAI"}, "008140": {"suoxie": "HTFJDSYDKHHC", "name": "\u6c47\u6dfb\u5bcc\u7edd\u5bf9\u6536\u76ca\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUITIANFUJUEDUISHOUYIDINGKAIHUNHEC"}, "008142": {"suoxie": "GYRXHJETFLJA", "name": "\u5de5\u94f6\u745e\u4fe1\u9ec4\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GONGYINRUIXINHUANGJINETFLIANJIEA"}, "008143": {"suoxie": "GYRXHJETFLJC", "name": "\u5de5\u94f6\u745e\u4fe1\u9ec4\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GONGYINRUIXINHUANGJINETFLIANJIEC"}, "008144": {"suoxie": "GYZYPZSGYHHFOF", "name": "\u5de5\u94f6\u667a\u8fdc\u914d\u7f6e\u4e09\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINZHIYUANPEIZHISANGEYUEHUNHEFOF"}, "008145": {"suoxie": "XQYXJQSGYCYFOFA", "name": "\u5174\u5168\u4f18\u9009\u8fdb\u53d6\u4e09\u4e2a\u6708\u6301\u6709(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANYOUXUANJINQUSANGEYUECHIYOUFOFA"}, "008146": {"suoxie": "ZYTR6GYA", "name": "\u4e2d\u94f6\u6dfb\u745e6\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINTIANRUI6GEYUEA"}, "008147": {"suoxie": "ZYTR6GYC", "name": "\u4e2d\u94f6\u6dfb\u745e6\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINTIANRUI6GEYUEC"}, "008150": {"suoxie": "JSYJQYJXLNCYQHH", "name": "\u5609\u5b9e\u8fdc\u89c1\u4f01\u4e1a\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYUANJIANQIYEJINGXUANLIANGNIANCHIYOUQIHUNHE"}, "008154": {"suoxie": "JSYYJK100ETFLJA", "name": "\u5609\u5b9e\u533b\u836f\u5065\u5eb7100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIYIYAOJIANKANG100ETFLIANJIEA"}, "008155": {"suoxie": "JSYYJK100ETFLJC", "name": "\u5609\u5b9e\u533b\u836f\u5065\u5eb7100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIYIYAOJIANKANG100ETFLIANJIEC"}, "008156": {"suoxie": "ZJZZ13NZJZA", "name": "\u4e2d\u91d1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINZHONGZHAI13NIANZHENGJINZHAIA"}, "008157": {"suoxie": "ZJZZ13NZJZC", "name": "\u4e2d\u91d1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINZHONGZHAI13NIANZHENGJINZHAIC"}, "008158": {"suoxie": "ZSSXYX3GYCYFOFA", "name": "\u62db\u5546\u76db\u946b\u4f18\u90093\u4e2a\u6708\u6301\u6709(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGSHENGXINYOUXUAN3GEYUECHIYOUFOFA"}, "008159": {"suoxie": "ZSSXYX3GYCYFOFC", "name": "\u62db\u5546\u76db\u946b\u4f18\u90093\u4e2a\u6708\u6301\u6709(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGSHENGXINYOUXUAN3GEYUECHIYOUFOFC"}, "008160": {"suoxie": "QHLHCA3NDKZQ", "name": "\u524d\u6d77\u8054\u5408\u6df3\u5b893\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHECHUNAN3NIANDINGKAIZHAIQUAN"}, "008161": {"suoxie": "GFHD3GYDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u8fbe3\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIDA3GEYUEDINGQIKAIFANGZHAIQUAN"}, "008162": {"suoxie": "PYASJJDJQHHC", "name": "\u6d66\u94f6\u5b89\u76db\u7ecf\u6d4e\u5e26\u5d1b\u8d77\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGJINGJIDAIJUEQIHUNHEC"}, "008163": {"suoxie": "NFDPHL50ETFLJA", "name": "\u5357\u65b9\u5927\u76d8\u7ea2\u522950ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDAPANHONGLI50ETFLIANJIEA"}, "008164": {"suoxie": "NFDPHL50ETFLJC", "name": "\u5357\u65b9\u5927\u76d8\u7ea2\u522950ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGDAPANHONGLI50ETFLIANJIEC"}, "008165": {"suoxie": "DXXYSNDK", "name": "\u4e1c\u5174\u946b\u8fdc\u4e09\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINYUANSANNIANDINGKAI"}, "008166": {"suoxie": "GYXFGPA", "name": "\u5de5\u94f6\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXIAOFEIGUPIAOA"}, "008167": {"suoxie": "GYXFGPC", "name": "\u5de5\u94f6\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXIAOFEIGUPIAOC"}, "008168": {"suoxie": "HTFJJCZSGYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u805a\u7126\u6210\u957f\u4e09\u4e2a\u6708\u6df7\u5408FOF", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUJIAOCHENGZHANGSANGEYUEHUNHEFOF"}, "008169": {"suoxie": "HTFHXYSSGYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u6838\u5fc3\u4f18\u52bf\u4e09\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHEXINYOUSHISANGEYUEHUNHEFOF"}, "008170": {"suoxie": "BSFTCZZQ", "name": "\u535a\u65f6\u5bcc\u6dfb\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUTIANCHUNZHAIZHAIQUAN"}, "008171": {"suoxie": "CCJYLGYDKZA", "name": "\u957f\u57ce\u5609\u88d5\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIAYULIUGEYUEDINGKAIZHAIA"}, "008172": {"suoxie": "CCJYLGYDKZC", "name": "\u957f\u57ce\u5609\u88d5\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIAYULIUGEYUEDINGKAIZHAIC"}, "008173": {"suoxie": "XQWTZQC", "name": "\u5174\u5168\u7a33\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANWENTAIZHAIQUANC"}, "008174": {"suoxie": "GTLCJXHHA", "name": "\u56fd\u6cf0\u84dd\u7b79\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAILANCHOUJINGXUANHUNHEA"}, "008175": {"suoxie": "GTLCJXHHC", "name": "\u56fd\u6cf0\u84dd\u7b79\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAILANCHOUJINGXUANHUNHEC"}, "008176": {"suoxie": "CXLBZQC", "name": "\u957f\u4fe1\u5229\u4fdd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIBAOZHAIQUANC"}, "008177": {"suoxie": "JXGGXZTGP", "name": "\u5efa\u4fe1\u9ad8\u80a1\u606f\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINGAOGUXIZHUTIGUPIAO"}, "008178": {"suoxie": "TTHYHHA", "name": "\u540c\u6cf0\u6167\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIYINGHUNHEA"}, "008179": {"suoxie": "TTHYHHC", "name": "\u540c\u6cf0\u6167\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUIYINGHUNHEC"}, "008180": {"suoxie": "TTHLHHA", "name": "\u540c\u6cf0\u6167\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUILIHUNHEA"}, "008181": {"suoxie": "TTHLHHC", "name": "\u540c\u6cf0\u6167\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIHUILIHUNHEC"}, "008182": {"suoxie": "FZFBXHHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u4fe1\u6cd3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGXINHONGHUNHEC"}, "008183": {"suoxie": "ZSPSQQPZQDIIFOFMYXC", "name": "\u62db\u5546\u666e\u76db\u5168\u7403\u914d\u7f6e(QDII-FOF)\u7f8e\u5143\u73b0\u949e", "style": "QDII", "pinyin": "ZHAOSHANGPUSHENGQUANQIUPEIZHIQDIIFOFMEIYUANXIANCHAO"}, "008184": {"suoxie": "XHHS300ZSZQC", "name": "\u65b0\u534e\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAHUSHEN300ZHISHUZENGQIANGC"}, "008185": {"suoxie": "NAYJYXHHA", "name": "\u8bfa\u5b89\u7814\u7a76\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANYANJIUYOUXUANHUNHEA"}, "008186": {"suoxie": "CHXRHHA", "name": "\u6df3\u539a\u4fe1\u777f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINRUIHUNHEA"}, "008187": {"suoxie": "CHXRHHC", "name": "\u6df3\u539a\u4fe1\u777f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINRUIHUNHEC"}, "008188": {"suoxie": "QHKYWJZZSNHH", "name": "\u524d\u6d77\u5f00\u6e90\u7a33\u5065\u589e\u957f\u4e09\u5e74\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANWENJIANZENGZHANGSANNIANHUNHE"}, "008189": {"suoxie": "GTZZGTETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u94a2\u94c1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGTIEETFLIANJIEA"}, "008190": {"suoxie": "GTZZGTETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u94a2\u94c1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGTIEETFLIANJIEC"}, "008191": {"suoxie": "BSHLHBA", "name": "\u535a\u65f6\u5408\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIHELIHUOBIA"}, "008192": {"suoxie": "BSXRHBA", "name": "\u535a\u65f6\u5174\u8363\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXINGRONGHUOBIA"}, "008193": {"suoxie": "BSXSHBA", "name": "\u535a\u65f6\u5174\u76db\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXINGSHENGHUOBIA"}, "008199": {"suoxie": "HXCX100ETFLJA", "name": "\u534e\u590f\u521b\u65b0100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGXIN100ETFLIANJIEA"}, "008200": {"suoxie": "HXCX100ETFLJC", "name": "\u534e\u590f\u521b\u65b0100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGXIN100ETFLIANJIEC"}, "008201": {"suoxie": "YHMSCIZGAGLJC", "name": "\u94f6\u534eMSCI\u4e2d\u56fdA\u80a1\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAMSCIZHONGGUOAGULIANJIEC"}, "008202": {"suoxie": "ZYHY9GYCYQZQA", "name": "\u4e2d\u94f6\u6052\u88d59\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHENGYU9GEYUECHIYOUQIZHAIQUANA"}, "008203": {"suoxie": "ZYHY9GYCYQZQC", "name": "\u4e2d\u94f6\u6052\u88d59\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHENGYU9GEYUECHIYOUQIZHAIQUANC"}, "008204": {"suoxie": "JYWLZDZZQA", "name": "\u4ea4\u94f6\u7a33\u5229\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAOYINWENLIZHONGDUANZHAIZHAIQUANA"}, "008205": {"suoxie": "JYWLZDZZQC", "name": "\u4ea4\u94f6\u7a33\u5229\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIAOYINWENLIZHONGDUANZHAIZHAIQUANC"}, "008206": {"suoxie": "GTJRCZZQ", "name": "\u56fd\u6cf0\u805a\u745e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJURUICHUNZHAIZHAIQUAN"}, "008207": {"suoxie": "GTHRCZZQ", "name": "\u56fd\u6cf0\u5408\u878d\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHERONGCHUNZHAIZHAIQUAN"}, "008208": {"suoxie": "BDJTHBHH", "name": "\u535a\u9053\u5609\u6cf0\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIATAIHUIBAOHUNHE"}, "008209": {"suoxie": "NFBTYNHHA", "name": "\u5357\u65b9\u5b9d\u6cf0\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOTAIYINIANHUNHEA"}, "008210": {"suoxie": "NFBTYNHHC", "name": "\u5357\u65b9\u5b9d\u6cf0\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOTAIYINIANHUNHEC"}, "008211": {"suoxie": "YHYSZQ", "name": "\u94f6\u534e\u6c38\u76db\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAYONGSHENGZHAIQUAN"}, "008212": {"suoxie": "HXXJYHHC", "name": "\u534e\u590f\u65b0\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINJIYUHUNHEC"}, "008213": {"suoxie": "HXXQDHHC", "name": "\u534e\u590f\u65b0\u8d77\u70b9\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINQIDIANHUNHEC"}, "008214": {"suoxie": "HAXFDKZA", "name": "\u534e\u5b89\u946b\u798f\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINFUDINGKAIZHAIA"}, "008215": {"suoxie": "HAXFDKZC", "name": "\u534e\u5b89\u946b\u798f\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINFUDINGKAIZHAIC"}, "008216": {"suoxie": "NYHLPB13NLLZZS", "name": "\u519c\u94f6\u6c47\u7406\u5f6d\u535a1-3\u5e74\u5229\u7387\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIPENGBO13NIANLILVZHAIZHISHU"}, "008217": {"suoxie": "GTJYSNDQKFZQ", "name": "\u56fd\u6cf0\u805a\u76c8\u4e09\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUYINGSANNIANDINGQIKAIFANGZHAIQUAN"}, "008219": {"suoxie": "XBLDZYSNDKZQ", "name": "\u897f\u90e8\u5229\u5f97\u5c0a\u9038\u4e09\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEZUNYISANNIANDINGKAIZHAIQUAN"}, "008221": {"suoxie": "XYJXLHPZHHC", "name": "\u5174\u4e1a\u805a\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGYEJUXINLINGHUOPEIZHIHUNHEC"}, "008222": {"suoxie": "XYJYZQC", "name": "\u5174\u4e1a\u673a\u9047\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYEJIYUZHAIQUANC"}, "008223": {"suoxie": "JYYTLNDQKFZQ", "name": "\u4ea4\u94f6\u88d5\u6cf0\u4e24\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUTAILIANGNIANDINGQIKAIFANGZHAIQUAN"}, "008224": {"suoxie": "JYSAHF87GYDKZA", "name": "\u91d1\u5143\u987a\u5b89\u6cd3\u4e3087\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANHONGFENG87GEYUEDINGKAIZHAIA"}, "008225": {"suoxie": "JYSAHF87GYDKZC", "name": "\u91d1\u5143\u987a\u5b89\u6cd3\u4e3087\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANHONGFENG87GEYUEDINGKAIZHAIC"}, "008226": {"suoxie": "NFYL3GYDKZ", "name": "\u5357\u65b9\u8fdc\u52293\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGYUANLI3GEYUEDINGKAIZHAI"}, "008227": {"suoxie": "BYYJJXHHA", "name": "\u5b9d\u76c8\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGYANJIUJINGXUANHUNHEA"}, "008228": {"suoxie": "BYYJJXHHC", "name": "\u5b9d\u76c8\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGYANJIUJINGXUANHUNHEC"}, "008229": {"suoxie": "XYASLNDKZ", "name": "\u946b\u5143\u5b89\u7855\u4e24\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANANSHUOLIANGNIANDINGKAIZHAI"}, "008231": {"suoxie": "HFTYT30GYDKZ", "name": "\u6d77\u5bcc\u901a\u88d5\u901a30\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGYUTONG30GEYUEDINGKAIZHAI"}, "008232": {"suoxie": "ZYHY12GYCYQZQA", "name": "\u4e2d\u94f6\u6052\u4f1812\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHENGYOU12GEYUECHIYOUQIZHAIQUANA"}, "008233": {"suoxie": "ZYHY12GYCYQZQC", "name": "\u4e2d\u94f6\u6052\u4f1812\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHENGYOU12GEYUECHIYOUQIZHAIQUANC"}, "008234": {"suoxie": "GDXFZTGP", "name": "\u5149\u5927\u6d88\u8d39\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDAXIAOFEIZHUTIGUPIAO"}, "008236": {"suoxie": "ZSSZ100ETFLJA", "name": "\u62db\u5546\u6df1\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENG100ETFLIANJIEA"}, "008237": {"suoxie": "ZSSZ100ETFLJC", "name": "\u62db\u5546\u6df1\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENG100ETFLIANJIEC"}, "008238": {"suoxie": "ZTHS300ZQA", "name": "\u4e2d\u6cf0\u6caa\u6df1300\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIHUSHEN300ZENGQIANGA"}, "008239": {"suoxie": "ZTHS300ZQC", "name": "\u4e2d\u6cf0\u6caa\u6df1300\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIHUSHEN300ZENGQIANGC"}, "008240": {"suoxie": "DCSZ50A", "name": "\u4e1c\u8d22\u4e0a\u8bc150A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAISHANGZHENG50A"}, "008241": {"suoxie": "DCSZ50C", "name": "\u4e1c\u8d22\u4e0a\u8bc150C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAISHANGZHENG50C"}, "008242": {"suoxie": "YDTYCZZQA", "name": "\u82f1\u5927\u901a\u76c8\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDATONGYINGCHUNZHAIZHAIQUANA"}, "008243": {"suoxie": "YDTYCZZQC", "name": "\u82f1\u5927\u901a\u76c8\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDATONGYINGCHUNZHAIZHAIQUANC"}, "008244": {"suoxie": "SYXZHH", "name": "\u4e0a\u94f6\u946b\u5353\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINXINZHUOHUNHE"}, "008245": {"suoxie": "YXYFZYHHA", "name": "\u5706\u4fe1\u6c38\u4e30\u81f4\u4f18\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGZHIYOUHUNHEA"}, "008246": {"suoxie": "YXYFZYHHC", "name": "\u5706\u4fe1\u6c38\u4e30\u81f4\u4f18\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGZHIYOUHUNHEC"}, "008249": {"suoxie": "GYSZ100ETFLJA", "name": "\u5de5\u94f6\u6df1\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHENZHENG100ETFLIANJIEA"}, "008250": {"suoxie": "GYSZ100ETFLJC", "name": "\u5de5\u94f6\u6df1\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHENZHENG100ETFLIANJIEC"}, "008251": {"suoxie": "HAYCLHJXHHA", "name": "\u6c47\u5b89\u5b9c\u521b\u91cf\u5316\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANYICHUANGLIANGHUAJINGXUANHUNHEA"}, "008252": {"suoxie": "HAYCLHJXHHC", "name": "\u6c47\u5b89\u5b9c\u521b\u91cf\u5316\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANYICHUANGLIANGHUAJINGXUANHUNHEC"}, "008253": {"suoxie": "HBZYHHQDIIA", "name": "\u534e\u5b9d\u81f4\u8fdc\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "HUABAOZHIYUANHUNHEQDIIA"}, "008254": {"suoxie": "HBZYHHQDIIC", "name": "\u534e\u5b9d\u81f4\u8fdc\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "HUABAOZHIYUANHUNHEQDIIC"}, "008255": {"suoxie": "XBLDFTZQA", "name": "\u897f\u90e8\u5229\u5f97\u6ca3\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEFENGTAIZHAIQUANA"}, "008256": {"suoxie": "NF15NGKZA", "name": "\u5357\u65b91-5\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG15NIANGUOKAIZHAIA"}, "008257": {"suoxie": "NF15NGKZC", "name": "\u5357\u65b91-5\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG15NIANGUOKAIZHAIC"}, "008258": {"suoxie": "ZYZQZZ500ETFLJA", "name": "\u4e2d\u94f6\u8bc1\u5238\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHENGQUANZHONGZHENG500ETFLIANJIEA"}, "008259": {"suoxie": "ZYZQZZ500ETFLJC", "name": "\u4e2d\u94f6\u8bc1\u5238\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHENGQUANZHONGZHENG500ETFLIANJIEC"}, "008260": {"suoxie": "CCJZYXHH", "name": "\u957f\u57ce\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHIYOUXUANHUNHE"}, "008261": {"suoxie": "ZSYJYXGPA", "name": "\u62db\u5546\u7814\u7a76\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGYANJIUYOUXUANGUPIAOA"}, "008262": {"suoxie": "ZSYJYXGPC", "name": "\u62db\u5546\u7814\u7a76\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGYANJIUYOUXUANGUPIAOC"}, "008263": {"suoxie": "DFHPZYXDKHH", "name": "\u4e1c\u65b9\u7ea2\u54c1\u8d28\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGPINZHIYOUXUANDINGKAIHUNHE"}, "008264": {"suoxie": "NFESGGPA", "name": "\u5357\u65b9ESG\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGESGGUPIAOA"}, "008265": {"suoxie": "NFESGGPC", "name": "\u5357\u65b9ESG\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGESGGUPIAOC"}, "008266": {"suoxie": "HXDMZQA", "name": "\u534e\u590f\u9f0e\u660e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGMINGZHAIQUANA"}, "008267": {"suoxie": "HXDMZQC", "name": "\u534e\u590f\u9f0e\u660e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGMINGZHAIQUANC"}, "008268": {"suoxie": "GTTRYNDKZ", "name": "\u56fd\u6cf0\u6dfb\u745e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAITIANRUIYINIANDINGKAIZHAI"}, "008269": {"suoxie": "DCRXHHA", "name": "\u5927\u6210\u777f\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGRUIXIANGHUNHEA"}, "008270": {"suoxie": "DCRXHHC", "name": "\u5927\u6210\u777f\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGRUIXIANGHUNHEC"}, "008271": {"suoxie": "DCYSQYHHA", "name": "\u5927\u6210\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYOUSHIQIYEHUNHEA"}, "008272": {"suoxie": "DCYSQYHHC", "name": "\u5927\u6210\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYOUSHIQIYEHUNHEC"}, "008273": {"suoxie": "GFYZSHHH", "name": "\u5e7f\u53d1\u4f18\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAYOUZHISHENGHUOHUNHE"}, "008274": {"suoxie": "DCHYXFHHA", "name": "\u5927\u6210\u884c\u4e1a\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHANGYEXIANFENGHUNHEA"}, "008275": {"suoxie": "DCHYXFHHC", "name": "\u5927\u6210\u884c\u4e1a\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHANGYEXIANFENGHUNHEC"}, "008276": {"suoxie": "CTZGJZFXHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u53d1\u73b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIAZHIFAXIANHUNHEA"}, "008277": {"suoxie": "CTZGHYJXHH", "name": "\u8d22\u901a\u8d44\u7ba1\u884c\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANHANGYEJINGXUANHUNHE"}, "008278": {"suoxie": "GTHXYNDQKFZQ", "name": "\u56fd\u6cf0\u60e0\u946b\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIXINYINIANDINGQIKAIFANGZHAIQUAN"}, "008279": {"suoxie": "GTZZMTETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7164\u70adETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGMEITANETFLIANJIEA"}, "008280": {"suoxie": "GTZZMTETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7164\u70adETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGMEITANETFLIANJIEC"}, "008281": {"suoxie": "GTCESBDTXPHYETFLJA", "name": "\u56fd\u6cf0CES\u534a\u5bfc\u4f53\u82af\u7247\u884c\u4e1aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAICESBANDAOTIXINPIANHANGYEETFLIANJIEA"}, "008282": {"suoxie": "GTCESBDTXPHYETFLJC", "name": "\u56fd\u6cf0CES\u534a\u5bfc\u4f53\u82af\u7247\u884c\u4e1aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAICESBANDAOTIXINPIANHANGYEETFLIANJIEC"}, "008283": {"suoxie": "YFDJRHYGP", "name": "\u6613\u65b9\u8fbe\u91d1\u878d\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAJINRONGHANGYEGUPIAO"}, "008284": {"suoxie": "YFDQQYYHYQDIIRMB", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u533b\u836f\u884c\u4e1a(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "YIFANGDAQUANQIUYIYAOHANGYEQDIIRENMINBI"}, "008285": {"suoxie": "YFDQQYYHYQDIIMY", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u533b\u836f\u884c\u4e1a(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "YIFANGDAQUANQIUYIYAOHANGYEQDIIMEIYUAN"}, "008286": {"suoxie": "YFDYJJXGP", "name": "\u6613\u65b9\u8fbe\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAYANJIUJINGXUANGUPIAO"}, "008287": {"suoxie": "CCJXLNDKZA", "name": "\u957f\u57ce\u5609\u946b\u4e24\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIAXINLIANGNIANDINGKAIZHAIA"}, "008288": {"suoxie": "CCJXLNDKZC", "name": "\u957f\u57ce\u5609\u946b\u4e24\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIAXINLIANGNIANDINGKAIZHAIC"}, "008289": {"suoxie": "GSABTXCZYNDKZQFQS", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u7965\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIXIANGCHUNZHAIYINIANDINGKAIZHAIQUANFAQISHI"}, "008290": {"suoxie": "HAXDSHHH", "name": "\u534e\u5b89\u73b0\u4ee3\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANXIANDAISHENGHUOHUNHE"}, "008291": {"suoxie": "MSJYHS300ETFLJA", "name": "\u6c11\u751f\u52a0\u94f6\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINHUSHEN300ETFLIANJIEA"}, "008292": {"suoxie": "MSJYHS300ETFLJC", "name": "\u6c11\u751f\u52a0\u94f6\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINHUSHEN300ETFLIANJIEC"}, "008293": {"suoxie": "NYHLCXYLHH", "name": "\u519c\u94f6\u6c47\u7406\u521b\u65b0\u533b\u7597\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILICHUANGXINYILIAOHUNHE"}, "008294": {"suoxie": "ZQQYYSA", "name": "\u6731\u96c0\u4f01\u4e1a\u4f18\u80dcA", "style": "\u80a1\u7968\u578b", "pinyin": "ZHUQUEQIYEYOUSHENGA"}, "008295": {"suoxie": "ZQQYYSC", "name": "\u6731\u96c0\u4f01\u4e1a\u4f18\u80dcC", "style": "\u80a1\u7968\u578b", "pinyin": "ZHUQUEQIYEYOUSHENGC"}, "008296": {"suoxie": "GFHLYNDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u5229\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUILIYINIANDINGQIKAIFANGZHAIQUAN"}, "008297": {"suoxie": "GFJZYSHH", "name": "\u5e7f\u53d1\u4ef7\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIYOUSHIHUNHE"}, "008298": {"suoxie": "HXZZYHETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYINHANGETFLIANJIEA"}, "008299": {"suoxie": "HXZZYHETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYINHANGETFLIANJIEC"}, "008300": {"suoxie": "RBLHRJHHA", "name": "\u4eba\u4fdd\u91cf\u5316\u9510\u8fdb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOLIANGHUARUIJINHUNHEA"}, "008301": {"suoxie": "RBLHRJHHC", "name": "\u4eba\u4fdd\u91cf\u5316\u9510\u8fdb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RENBAOLIANGHUARUIJINHUNHEC"}, "008302": {"suoxie": "YYYHZQ", "name": "\u6c38\u8d62\u6613\u5f18\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGYIHONGZHAIQUAN"}, "008303": {"suoxie": "BYLTYXGPA", "name": "\u5b9d\u76c8\u9f99\u5934\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGLONGTOUYOUXUANGUPIAOA"}, "008304": {"suoxie": "BYLTYXGPC", "name": "\u5b9d\u76c8\u9f99\u5934\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGLONGTOUYOUXUANGUPIAOC"}, "008305": {"suoxie": "DMLHPZHHC", "name": "\u5927\u6469\u91cf\u5316\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOLIANGHUAPEIZHIHUNHEC"}, "008306": {"suoxie": "FZFBTXHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u7487\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANXUANHUNHEA"}, "008307": {"suoxie": "FZFBTXHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u5929\u7487\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGTIANXUANHUNHEC"}, "008308": {"suoxie": "HXJLJXHH", "name": "\u534e\u590f\u89c1\u9f99\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAJIANLONGJINGXUANHUNHE"}, "008311": {"suoxie": "YXYFYXJZA", "name": "\u5706\u4fe1\u6c38\u4e30\u4f18\u9009\u4ef7\u503cA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYOUXUANJIAZHIA"}, "008312": {"suoxie": "YXYFYXJZC", "name": "\u5706\u4fe1\u6c38\u4e30\u4f18\u9009\u4ef7\u503cC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYOUXUANJIAZHIC"}, "008313": {"suoxie": "GDYJJXHH", "name": "\u5149\u5927\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANJIUJINGXUANHUNHE"}, "008314": {"suoxie": "STMGHXCZA", "name": "\u4e0a\u6295\u6469\u6839\u6167\u9009\u6210\u957fA", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHUIXUANCHENGZHANGA"}, "008315": {"suoxie": "STMGHXCZC", "name": "\u4e0a\u6295\u6469\u6839\u6167\u9009\u6210\u957fC", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHUIXUANCHENGZHANGC"}, "008316": {"suoxie": "FCTY66GYDKZ", "name": "\u8702\u5de2\u6dfb\u8dc366\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYUE66GEYUEDINGKAIZHAI"}, "008317": {"suoxie": "GDRYHH", "name": "\u5149\u5927\u777f\u76c8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDARUIYINGHUNHE"}, "008318": {"suoxie": "BDJHHHA", "name": "\u535a\u9053\u4e45\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIUHANGHUNHEA"}, "008319": {"suoxie": "BDJHHHC", "name": "\u535a\u9053\u4e45\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIUHANGHUNHEC"}, "008320": {"suoxie": "PHQQZDZQDIIRMBC", "name": "\u9e4f\u534e\u5168\u7403\u4e2d\u77ed\u503a(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "PENGHUAQUANQIUZHONGDUANZHAIQDIIRENMINBIC"}, "008321": {"suoxie": "PHQQZDZQDIIMYXHC", "name": "\u9e4f\u534e\u5168\u7403\u4e2d\u77ed\u503a(QDII)\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "PENGHUAQUANQIUZHONGDUANZHAIQDIIMEIYUANXIANHUIC"}, "008322": {"suoxie": "DFZX18GYDKZQA", "name": "\u4e1c\u65b9\u5353\u884c18\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHUOXING18GEYUEDINGKAIZHAIQUANA"}, "008323": {"suoxie": "DFZX18GYDKZQC", "name": "\u4e1c\u65b9\u5353\u884c18\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHUOXING18GEYUEDINGKAIZHAIQUANC"}, "008324": {"suoxie": "BYXLWJPZHHA", "name": "\u5b9d\u76c8\u7965\u5229\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGLIWENJIANPEIZHIHUNHEA"}, "008325": {"suoxie": "BYXLWJPZHHC", "name": "\u5b9d\u76c8\u7965\u5229\u7a33\u5065\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGLIWENJIANPEIZHIHUNHEC"}, "008326": {"suoxie": "DCTXA", "name": "\u4e1c\u8d22\u901a\u4fe1A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAITONGXINA"}, "008327": {"suoxie": "DCTXC", "name": "\u4e1c\u8d22\u901a\u4fe1C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAITONGXINC"}, "008328": {"suoxie": "NAXXCYHH", "name": "\u8bfa\u5b89\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANXINXINGCHANYEHUNHE"}, "008331": {"suoxie": "WJKZZZQA", "name": "\u4e07\u5bb6\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "WANJIAKEZHUANZHAIZHAIQUANA"}, "008332": {"suoxie": "WJKZZZQC", "name": "\u4e07\u5bb6\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "WANJIAKEZHUANZHAIZHAIQUANC"}, "008333": {"suoxie": "JSCCHL39GYDKZ", "name": "\u666f\u987a\u957f\u57ce\u5f18\u522939\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGHONGLI39GEYUEDINGKAIZHAI"}, "008336": {"suoxie": "BYXYZQHBHHA", "name": "\u5b9d\u76c8\u7965\u88d5\u589e\u5f3a\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGYUZENGQIANGHUIBAOHUNHEA"}, "008337": {"suoxie": "BYXYZQHBHHC", "name": "\u5b9d\u76c8\u7965\u88d5\u589e\u5f3a\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGYUZENGQIANGHUIBAOHUNHEC"}, "008338": {"suoxie": "JSAY39GYDQCZA", "name": "\u5609\u5b9e\u5b89\u514339\u4e2a\u6708\u5b9a\u671f\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIANYUAN39GEYUEDINGQICHUNZHAIA"}, "008339": {"suoxie": "JSAY39GYDQCZC", "name": "\u5609\u5b9e\u5b89\u514339\u4e2a\u6708\u5b9a\u671f\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIANYUAN39GEYUEDINGQICHUNZHAIC"}, "008340": {"suoxie": "HFZZAHXYZA", "name": "\u534e\u5bcc\u4e2d\u503a-\u5b89\u5fbd\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAIANHUIXINYONGZHAIA"}, "008341": {"suoxie": "HFZZAHXYZC", "name": "\u534e\u5bcc\u4e2d\u503a-\u5b89\u5fbd\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAIANHUIXINYONGZHAIC"}, "008342": {"suoxie": "JTKXCLJXHHA", "name": "\u4e5d\u6cf0\u79d1\u946b\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIKEXINCELUEJINGXUANHUNHEA"}, "008343": {"suoxie": "JTKXCLJXHHC", "name": "\u4e5d\u6cf0\u79d1\u946b\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIKEXINCELUEJINGXUANHUNHEC"}, "008344": {"suoxie": "JXRYYNDQKFZQ", "name": "\u5efa\u4fe1\u777f\u9633\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIYANGYINIANDINGQIKAIFANGZHAIQUAN"}, "008345": {"suoxie": "NHRZZQA", "name": "\u5357\u534e\u745e\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANHUARUIZEZHAIQUANA"}, "008346": {"suoxie": "NHRZZQC", "name": "\u5357\u534e\u745e\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANHUARUIZEZHAIQUANC"}, "008347": {"suoxie": "ZXJTZXHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUZHENXUANHUNHEA"}, "008348": {"suoxie": "ZXJTZXHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUZHENXUANHUNHEC"}, "008349": {"suoxie": "HXHT64GYDKZQ", "name": "\u534e\u590f\u6052\u6cf064\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAHENGTAI64GEYUEDINGKAIZHAIQUAN"}, "008352": {"suoxie": "JYYKCZYNDK", "name": "\u4ea4\u94f6\u88d5\u5764\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUKUNCHUNZHAIYINIANDINGKAI"}, "008353": {"suoxie": "TDHLXFHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXIAOFEIHUNHEA"}, "008354": {"suoxie": "TDHLXFHHC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXIAOFEIHUNHEC"}, "008355": {"suoxie": "NYHLJQYNDKZ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u797a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINQIYINIANDINGKAIZHAI"}, "008356": {"suoxie": "ZJKFJZJXHH", "name": "\u4e2d\u52a0\u79d1\u4e30\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEFENGJIAZHIJINGXUANHUNHE"}, "008359": {"suoxie": "HAYLCXHHA", "name": "\u534e\u5b89\u533b\u7597\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYILIAOCHUANGXINHUNHEA"}, "008361": {"suoxie": "NFZLYNZQ", "name": "\u5357\u65b9\u62db\u5229\u4e00\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHAOLIYINIANZHAIQUAN"}, "008362": {"suoxie": "GFHCYNDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u6210\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUICHENGYINIANDINGQIKAIFANGZHAIQUAN"}, "008363": {"suoxie": "GFMFYNDQKFZQ", "name": "\u5e7f\u53d1\u6c11\u4e30\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAMINFENGYINIANDINGQIKAIFANGZHAIQUAN"}, "008366": {"suoxie": "GFHMYNDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u660e\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIMINGYINIANDINGQIKAIFANGZHAIQUAN"}, "008367": {"suoxie": "FGYZSYZQQDIIRMB", "name": "\u5bcc\u56fd\u4e9a\u6d32\u6536\u76ca\u503a\u5238(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "FUGUOYAZHOUSHOUYIZHAIQUANQDIIRENMINBI"}, "008368": {"suoxie": "FGYZSYZQQDIIMY", "name": "\u5bcc\u56fd\u4e9a\u6d32\u6536\u76ca\u503a\u5238(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOYAZHOUSHOUYIZHAIQUANQDIIMEIYUAN"}, "008369": {"suoxie": "FCFXYNDK", "name": "\u8702\u5de2\u4e30\u946b\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGXINYINIANDINGKAI"}, "008370": {"suoxie": "GTYJJXLNCYHH", "name": "\u56fd\u6cf0\u7814\u7a76\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIYANJIUJINGXUANLIANGNIANCHIYOUHUNHE"}, "008371": {"suoxie": "HAHZJXLNCYQHH", "name": "\u534e\u5b89\u6c47\u667a\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUIZHIJINGXUANLIANGNIANCHIYOUQIHUNHE"}, "008372": {"suoxie": "FGAEFLNCYQHH", "name": "\u5bcc\u56fd\u963f\u5c14\u6cd5\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOAERFALIANGNIANCHIYOUQIHUNHE"}, "008373": {"suoxie": "HTBRJQHBA", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u56de\u62a5A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIHUIBAOA"}, "008374": {"suoxie": "HTBRJQHBC", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u56de\u62a5C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIHUIBAOC"}, "008375": {"suoxie": "ZOQHSNHHA", "name": "\u4e2d\u6b27\u542f\u822a\u4e09\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUQIHANGSANNIANHUNHEA"}, "008376": {"suoxie": "ZOQHSNHHC", "name": "\u4e2d\u6b27\u542f\u822a\u4e09\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUQIHANGSANNIANHUNHEC"}, "008378": {"suoxie": "XQSHJZSNCYHH", "name": "\u5174\u5168\u793e\u4f1a\u4ef7\u503c\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANSHEHUIJIAZHISANNIANCHIYOUHUNHE"}, "008381": {"suoxie": "QHKYXXCYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANXINXINGCHANYEHUNHEA"}, "008382": {"suoxie": "RTCYQSGP", "name": "\u878d\u901a\u4ea7\u4e1a\u8d8b\u52bf\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "RONGTONGCHANYEQUSHIGUPIAO"}, "008383": {"suoxie": "ZSAXSYZQA", "name": "\u62db\u5546\u5b89\u5fc3\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANXINSHOUYIZHAIQUANA"}, "008384": {"suoxie": "YHHYYNCYQHHA", "name": "\u94f6\u534e\u6c47\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAHUIYIYINIANCHIYOUQIHUNHEA"}, "008385": {"suoxie": "YHHYYNCYQHHC", "name": "\u94f6\u534e\u6c47\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAHUIYIYINIANCHIYOUQIHUNHEC"}, "008386": {"suoxie": "QHLHHXDKZQ", "name": "\u524d\u6d77\u8054\u5408\u6cd3\u65ed\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEHONGXUDINGKAIZHAIQUAN"}, "008390": {"suoxie": "GLAHS300ETFLJA", "name": "\u56fd\u8054\u5b89\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANHUSHEN300ETFLIANJIEA"}, "008391": {"suoxie": "GLAHS300ETFLJC", "name": "\u56fd\u8054\u5b89\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANHUSHEN300ETFLIANJIEC"}, "008392": {"suoxie": "XYYZZLZQC", "name": "\u5174\u4e1a\u4f18\u503a\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYOUZHAIZENGLIZHAIQUANC"}, "008393": {"suoxie": "BSXJSYHBC", "name": "\u535a\u65f6\u73b0\u91d1\u6536\u76ca\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINSHOUYIHUOBIC"}, "008394": {"suoxie": "FZFBHLCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u6052\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHENGLICHUNZHAIA"}, "008395": {"suoxie": "FZFBHLCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u6052\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHENGLICHUNZHAIC"}, "008396": {"suoxie": "BSZZ500ETFLJA", "name": "\u535a\u65f6\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG500ETFLIANJIEA"}, "008397": {"suoxie": "BSZZ500ETFLJC", "name": "\u535a\u65f6\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG500ETFLIANJIEC"}, "008398": {"suoxie": "HTFXFZ", "name": "\u6c47\u6dfb\u5bcc\u946b\u798f\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINFUZHAI"}, "008399": {"suoxie": "HTBRZZKJETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u79d1\u6280ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGKEJIETFLIANJIEA"}, "008400": {"suoxie": "HTBRZZKJETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u79d1\u6280ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGKEJIETFLIANJIEC"}, "008404": {"suoxie": "HTZJTYHHA", "name": "\u534e\u6cf0\u7d2b\u91d1\u6cf0\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIZIJINTAIYINGHUNHEA"}, "008405": {"suoxie": "HTZJTYHHC", "name": "\u534e\u6cf0\u7d2b\u91d1\u6cf0\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIZIJINTAIYINGHUNHEC"}, "008406": {"suoxie": "XYHYDKZ", "name": "\u5174\u94f6\u6c47\u88d5\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIYUDINGKAIZHAI"}, "008407": {"suoxie": "HSHSGTXFHYLTA", "name": "\u6052\u751f\u6caa\u6df1\u6e2f\u901a\u7ec6\u5206\u884c\u4e1a\u9f99\u5934A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HENGSHENGHUSHENGANGTONGXIFENHANGYELONGTOUA"}, "008408": {"suoxie": "HSHSGTXFHYLTC", "name": "\u6052\u751f\u6caa\u6df1\u6e2f\u901a\u7ec6\u5206\u884c\u4e1a\u9f99\u5934C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HENGSHENGHUSHENGANGTONGXIFENHANGYELONGTOUC"}, "008409": {"suoxie": "JSCCJTYLCZZQ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u88d5\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIYULICHUNZHAIZHAIQUAN"}, "008411": {"suoxie": "BSFXCZZQ", "name": "\u535a\u65f6\u5bcc\u4fe1\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUXINCHUNZHAIZHAIQUAN"}, "008412": {"suoxie": "CSJZYSGPA", "name": "\u957f\u76db\u7ade\u4e89\u4f18\u52bf\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGSHENGJINGZHENGYOUSHIGUPIAOA"}, "008413": {"suoxie": "CSJZYSGPC", "name": "\u957f\u76db\u7ade\u4e89\u4f18\u52bf\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGSHENGJINGZHENGYOUSHIGUPIAOC"}, "008414": {"suoxie": "GTHTYNDQKFZQ", "name": "\u56fd\u6cf0\u60e0\u6cf0\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUITAIYINIANDINGQIKAIFANGZHAIQUAN"}, "008415": {"suoxie": "GTDZZLNCYQHH", "name": "\u56fd\u6cf0\u5927\u5236\u9020\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIDAZHIZAOLIANGNIANCHIYOUQIHUNHE"}, "008416": {"suoxie": "PYJRSNDKHHA", "name": "\u9e4f\u626c\u666f\u745e\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGRUISANNIANDINGKAIHUNHEA"}, "008417": {"suoxie": "PYJRSNDKHHC", "name": "\u9e4f\u626c\u666f\u745e\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGRUISANNIANDINGKAIHUNHEC"}, "008418": {"suoxie": "HSHZHHA", "name": "\u60e0\u5347\u60e0\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIZEHUNHEA"}, "008419": {"suoxie": "HSHZHHC", "name": "\u60e0\u5347\u60e0\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIZEHUNHEC"}, "008420": {"suoxie": "GFZTHHA", "name": "\u5e7f\u53d1\u62db\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAZHAOTAIHUNHEA"}, "008421": {"suoxie": "GFZTHHC", "name": "\u5e7f\u53d1\u62db\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAZHAOTAIHUNHEC"}, "008422": {"suoxie": "ZRYFCXHHA", "name": "\u4e2d\u878d\u7814\u53d1\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYANFACHUANGXINHUNHEA"}, "008423": {"suoxie": "ZRYFCXHHC", "name": "\u4e2d\u878d\u7814\u53d1\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYANFACHUANGXINHUNHEC"}, "008424": {"suoxie": "ZRPPYXHHA", "name": "\u4e2d\u878d\u54c1\u724c\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGPINPAIYOUXUANHUNHEA"}, "008425": {"suoxie": "ZRPPYXHHC", "name": "\u4e2d\u878d\u54c1\u724c\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGPINPAIYOUXUANHUNHEC"}, "008426": {"suoxie": "HAADZQA", "name": "\u534e\u5b89\u5b89\u6566\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANDUNZHAIQUANA"}, "008427": {"suoxie": "HAADZQC", "name": "\u534e\u5b89\u5b89\u6566\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANDUNZHAIQUANC"}, "008428": {"suoxie": "DFHXYLNDKXYZ", "name": "\u4e1c\u65b9\u7ea2\u946b\u88d5\u4e24\u5e74\u5b9a\u5f00\u4fe1\u7528\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGXINYULIANGNIANDINGKAIXINYONGZHAI"}, "008429": {"suoxie": "ZXBCJYWNDKZ", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5609\u88d5\u4e94\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJIAYUWUNIANDINGKAIZHAI"}, "008433": {"suoxie": "KSQDZA", "name": "\u51ef\u77f3\u5c90\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "KAISHIQIDUANZHAIA"}, "008434": {"suoxie": "KSQDZC", "name": "\u51ef\u77f3\u5c90\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "KAISHIQIDUANZHAIC"}, "008435": {"suoxie": "CXZZZZJKJHZ50ZSA", "name": "\u957f\u4fe1\u4e2d\u8bc1\u8f6c\u503a\u53ca\u53ef\u4ea4\u6362\u503a50\u6307\u6570A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGXINZHONGZHENGZHUANZHAIJIKEJIAOHUANZHAI50ZHISHUA"}, "008436": {"suoxie": "CXZZZZJKJHZ50ZSC", "name": "\u957f\u4fe1\u4e2d\u8bc1\u8f6c\u503a\u53ca\u53ef\u4ea4\u6362\u503a50\u6307\u6570C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGXINZHONGZHENGZHUANZHAIJIKEJIAOHUANZHAI50ZHISHUC"}, "008437": {"suoxie": "JTHYYXHHA", "name": "\u4e5d\u6cf0\u884c\u4e1a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIHANGYEYOUXUANHUNHEA"}, "008438": {"suoxie": "JTHYYXHHC", "name": "\u4e5d\u6cf0\u884c\u4e1a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIHANGYEYOUXUANHUNHEC"}, "008443": {"suoxie": "JTDTCLHHA", "name": "\u4e5d\u6cf0\u52a8\u6001\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIDONGTAICELUEHUNHEA"}, "008444": {"suoxie": "JTDTCLHHC", "name": "\u4e5d\u6cf0\u52a8\u6001\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIDONGTAICELUEHUNHEC"}, "008445": {"suoxie": "RTCYQSXFGP", "name": "\u878d\u901a\u4ea7\u4e1a\u8d8b\u52bf\u5148\u950b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "RONGTONGCHANYEQUSHIXIANFENGGUPIAO"}, "008448": {"suoxie": "DBDZA", "name": "\u5fb7\u90a6\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DEBANGDUANZHAIA"}, "008449": {"suoxie": "DBDZC", "name": "\u5fb7\u90a6\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DEBANGDUANZHAIC"}, "008452": {"suoxie": "XQHXZQA", "name": "\u5174\u5168\u6052\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGXINZHAIQUANA"}, "008453": {"suoxie": "XQHXZQC", "name": "\u5174\u5168\u6052\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGXINZHAIQUANC"}, "008454": {"suoxie": "ZXBCZZ13NGKHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGZHONGZHAI13NIANGUOKAIHANGA"}, "008455": {"suoxie": "ZXBCZZ13NGKHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGZHONGZHAI13NIANGUOKAIHANGC"}, "008456": {"suoxie": "ZSRYHHA", "name": "\u62db\u5546\u745e\u9633\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIYANGHUNHEA"}, "008457": {"suoxie": "ZSRYHHC", "name": "\u62db\u5546\u745e\u9633\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIYANGHUNHEC"}, "008460": {"suoxie": "ZSZH39GYDKZ", "name": "\u62db\u5546\u62db\u548c39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOHE39GEYUEDINGKAIZHAI"}, "008461": {"suoxie": "PAYFSGYCYHHFOFA", "name": "\u5e73\u5b89\u76c8\u4e30\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYINGFENGSANGEYUECHIYOUHUNHEFOFA"}, "008462": {"suoxie": "PAYFSGYCYHHFOFC", "name": "\u5e73\u5b89\u76c8\u4e30\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYINGFENGSANGEYUECHIYOUHUNHEFOFC"}, "008463": {"suoxie": "ZSTR1NDKZA", "name": "\u62db\u5546\u6dfb\u745e1\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRUI1NIANDINGKAIZHAIA"}, "008464": {"suoxie": "ZSTR1NDKZC", "name": "\u62db\u5546\u6dfb\u745e1\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANRUI1NIANDINGKAIZHAIC"}, "008465": {"suoxie": "FCTYCZA", "name": "\u8702\u5de2\u6dfb\u76ca\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYICHUNZHAIA"}, "008466": {"suoxie": "FCTYCZC", "name": "\u8702\u5de2\u6dfb\u76ca\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYICHUNZHAIC"}, "008467": {"suoxie": "BDJRHHA", "name": "\u535a\u9053\u5609\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BODAOJIARUIHUNHEA"}, "008468": {"suoxie": "BDJRHHC", "name": "\u535a\u9053\u5609\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BODAOJIARUIHUNHEC"}, "008469": {"suoxie": "ZQAXHBZQA", "name": "\u6731\u96c0\u5b89\u946b\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHUQUEANXINHUIBAOZHAIQUANA"}, "008470": {"suoxie": "ZQAXHBZQC", "name": "\u6731\u96c0\u5b89\u946b\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHUQUEANXINHUIBAOZHAIQUANC"}, "008471": {"suoxie": "GYTYSNDKZQA", "name": "\u5de5\u94f6\u6cf0\u9890\u4e09\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINTAIYISANNIANDINGKAIZHAIQUANA"}, "008472": {"suoxie": "GYTYSNDKZQC", "name": "\u5de5\u94f6\u6cf0\u9890\u4e09\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINTAIYISANNIANDINGKAIZHAIQUANC"}, "008475": {"suoxie": "ZSMAZYZQA", "name": "\u62db\u5546\u6c11\u5b89\u589e\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGMINANZENGYIZHAIQUANA"}, "008476": {"suoxie": "ZSMAZYZQC", "name": "\u62db\u5546\u6c11\u5b89\u589e\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGMINANZENGYIZHAIQUANC"}, "008477": {"suoxie": "AXJZQDSNCYHH", "name": "\u5b89\u4fe1\u4ef7\u503c\u9a71\u52a8\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIQUDONGSANNIANCHIYOUHUNHE"}, "008478": {"suoxie": "THXY39GYDKZ", "name": "\u5929\u5f18\u946b\u610f39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINYI39GEYUEDINGKAIZHAI"}, "008479": {"suoxie": "JSCCTSHBHH", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u7533\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGTAISHENHUIBAOHUNHE"}, "008480": {"suoxie": "YYGXYXA", "name": "\u6c38\u8d62\u80a1\u606f\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGUXIYOUXUANA"}, "008481": {"suoxie": "YYGXYXC", "name": "\u6c38\u8d62\u80a1\u606f\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGUXIYOUXUANC"}, "008482": {"suoxie": "GFYQ80ZQZSA", "name": "\u5e7f\u53d1\u592e\u4f0180\u503a\u5238\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAYANGQI80ZHAIQUANZHISHUA"}, "008483": {"suoxie": "GFYQ80ZQZSC", "name": "\u5e7f\u53d1\u592e\u4f0180\u503a\u5238\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAYANGQI80ZHAIQUANZHISHUC"}, "008484": {"suoxie": "GLHYYNDKZA", "name": "\u683c\u6797\u6cd3\u88d5\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGYUYINIANDINGKAIZHAIA"}, "008485": {"suoxie": "GLHYYNDKZC", "name": "\u683c\u6797\u6cd3\u88d5\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGYUYINIANDINGKAIZHAIC"}, "008486": {"suoxie": "DBDRYNDKZ", "name": "\u5fb7\u90a6\u5fb7\u745e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGDERUIYINIANDINGKAIZHAI"}, "008487": {"suoxie": "ZXJTWYZQ", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENYUEZHAIQUAN"}, "008488": {"suoxie": "HSHYWJHH", "name": "\u534e\u5546\u6052\u76ca\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGHENGYIWENJIANHUNHE"}, "008489": {"suoxie": "HSHC39GYDKLLZA", "name": "\u534e\u5546\u9e3f\u754539\u4e2a\u6708\u5b9a\u5f00\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUASHANGHONGCHANG39GEYUEDINGKAILILVZHAIA"}, "008490": {"suoxie": "HSHC39GYDKLLZC", "name": "\u534e\u5546\u9e3f\u754539\u4e2a\u6708\u5b9a\u5f00\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUASHANGHONGCHANG39GEYUEDINGKAILILVZHAIC"}, "008491": {"suoxie": "WJZQYSQYHHA", "name": "\u4e07\u5bb6\u5468\u671f\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHOUQIYOUSHIQIYEHUNHEA"}, "008492": {"suoxie": "WJZQYSQYHHC", "name": "\u4e07\u5bb6\u5468\u671f\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHOUQIYOUSHIQIYEHUNHEC"}, "008493": {"suoxie": "PHZTYNDKFQSZQ", "name": "\u9e4f\u534e\u5c0a\u6cf0\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNTAIYINIANDINGKAIFAQISHIZHAIQUAN"}, "008495": {"suoxie": "JSCCJTTLYNDKZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u6dfb\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAITIANLIYINIANDINGKAIZHAI"}, "008496": {"suoxie": "GTHRYNDKZ", "name": "\u56fd\u6cf0\u60e0\u745e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIHUIRUIYINIANDINGKAIZHAI"}, "008497": {"suoxie": "PYPLZDZA", "name": "\u9e4f\u626c\u6d66\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGPULIZHONGDUANZHAIA"}, "008498": {"suoxie": "PYPLZDZC", "name": "\u9e4f\u626c\u6d66\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGPULIZHONGDUANZHAIC"}, "008499": {"suoxie": "PYJKHHA", "name": "\u9e4f\u626c\u666f\u79d1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGKEHUNHEA"}, "008500": {"suoxie": "PYJKHHC", "name": "\u9e4f\u626c\u666f\u79d1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGKEHUNHEC"}, "008501": {"suoxie": "PYJLLGYCYQZQA", "name": "\u9e4f\u626c\u805a\u5229\u516d\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGJULILIUGEYUECHIYOUQIZHAIQUANA"}, "008502": {"suoxie": "PYJLLGYCYQZQC", "name": "\u9e4f\u626c\u805a\u5229\u516d\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGYANGJULILIUGEYUECHIYOUQIZHAIQUANC"}, "008503": {"suoxie": "GSABTRCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u745e\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIRUICHUNZHAIYINIANDINGKAIZHAI"}, "008504": {"suoxie": "GTXYHLZQC", "name": "\u56fd\u6cf0\u4fe1\u7528\u4e92\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIXINYONGHULIZHAIQUANC"}, "008505": {"suoxie": "ZSZDZZQA", "name": "\u6d59\u5546\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGZHONGDUANZHAIZHAIQUANA"}, "008506": {"suoxie": "ZSZDZZQC", "name": "\u6d59\u5546\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGZHONGDUANZHAIZHAIQUANC"}, "008507": {"suoxie": "JYNHQDHH", "name": "\u4ea4\u94f6\u5185\u6838\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINNEIHEQUDONGHUNHE"}, "008508": {"suoxie": "ZRJJYNDKZQ", "name": "\u4e2d\u878d\u805a\u9526\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUJINYINIANDINGKAIZHAIQUAN"}, "008509": {"suoxie": "NFDLYNDKZ", "name": "\u5357\u65b9\u5f97\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGDELIYINIANDINGKAIZHAI"}, "008510": {"suoxie": "NFDLYNZQ", "name": "\u5357\u65b9\u9f0e\u5229\u4e00\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGDINGLIYINIANZHAIQUAN"}, "008511": {"suoxie": "BYHSZQA", "name": "\u5b9d\u76c8\u9e3f\u76db\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BAOYINGHONGSHENGZHAIQUANA"}, "008512": {"suoxie": "BYHSZQC", "name": "\u5b9d\u76c8\u9e3f\u76db\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BAOYINGHONGSHENGZHAIQUANC"}, "008513": {"suoxie": "NFBFHHA", "name": "\u5357\u65b9\u5b9d\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOFENGHUNHEA"}, "008514": {"suoxie": "NFBFHHC", "name": "\u5357\u65b9\u5b9d\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOFENGHUNHEC"}, "008515": {"suoxie": "GFJBMYXHH", "name": "\u56fd\u5bcc\u57fa\u672c\u9762\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJIBENMIANYOUXUANHUNHE"}, "008516": {"suoxie": "PYASSXYNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u7199\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGXIYINIANDINGKAIZHAIQUAN"}, "008517": {"suoxie": "XYJHYNDKZQFQS", "name": "\u5174\u4e1a\u5609\u534e\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIAHUAYINIANDINGKAIZHAIQUANFAQISHI"}, "008518": {"suoxie": "HYYFJJXDLHH", "name": "\u5f18\u6bc5\u8fdc\u65b9\u7ecf\u6d4e\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGJINGJIXINDONGLIHUNHE"}, "008519": {"suoxie": "ZJZZHGSYXXF50ZSA", "name": "\u4e2d\u91d1\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4f18\u9009\u6d88\u8d3950\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINZHONGZHENGHUGANGSHENYOUXUANXIAOFEI50ZHISHUA"}, "008520": {"suoxie": "ZJZZHGSYXXF50ZSC", "name": "\u4e2d\u91d1\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4f18\u9009\u6d88\u8d3950\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINZHONGZHENGHUGANGSHENYOUXUANXIAOFEI50ZHISHUC"}, "008521": {"suoxie": "FGHYCZ63GYDKZ", "name": "\u5bcc\u56fd\u6c47\u4f18\u7eaf\u503a63\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHUIYOUCHUNZHAI63GEYUEDINGKAIZHAI"}, "008522": {"suoxie": "FRFHYNDKZ", "name": "\u5bcc\u8363\u5bcc\u5408\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUHEYINIANDINGKAIZHAI"}, "008523": {"suoxie": "AXFZ39GYDKZ", "name": "\u5b89\u4fe1\u4e30\u6cfd39\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINFENGZE39GEYUEDINGKAIZHAI"}, "008524": {"suoxie": "HTBRJRZQA", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIJINRUIZHAIQUANA"}, "008525": {"suoxie": "HTBRJRZQC", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIJINRUIZHAIQUANC"}, "008526": {"suoxie": "HTBRHYJXA", "name": "\u534e\u6cf0\u67cf\u745e\u884c\u4e1a\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIHANGYEJINGXUANA"}, "008527": {"suoxie": "HTBRHYJXC", "name": "\u534e\u6cf0\u67cf\u745e\u884c\u4e1a\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIHANGYEJINGXUANC"}, "008528": {"suoxie": "HTBRZLCZA", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u6210\u957fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGCHENGZHANGA"}, "008529": {"suoxie": "HAXLZQA", "name": "\u6c47\u5b89\u4fe1\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANXINLIZHAIQUANA"}, "008530": {"suoxie": "HAXLZQC", "name": "\u6c47\u5b89\u4fe1\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANXINLIZHAIQUANC"}, "008531": {"suoxie": "HSHMHHA", "name": "\u60e0\u5347\u60e0\u6c11\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUISHENGHUIMINHUNHEA"}, "008532": {"suoxie": "HSHMHHC", "name": "\u60e0\u5347\u60e0\u6c11\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUISHENGHUIMINHUNHEC"}, "008533": {"suoxie": "HSHXHHA", "name": "\u60e0\u5347\u60e0\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUIXINGHUNHEA"}, "008534": {"suoxie": "HSHXHHC", "name": "\u60e0\u5347\u60e0\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUIXINGHUNHEC"}, "008535": {"suoxie": "XYHSDKZA", "name": "\u5174\u94f6\u5408\u76db\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHESHENGDINGKAIZHAIA"}, "008536": {"suoxie": "XYHSDKZC", "name": "\u5174\u94f6\u5408\u76db\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHESHENGDINGKAIZHAIC"}, "008537": {"suoxie": "XYYJJXGPA", "name": "\u5174\u94f6\u7814\u7a76\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "XINGYINYANJIUJINGXUANGUPIAOA"}, "008538": {"suoxie": "XYYJJXGPC", "name": "\u5174\u94f6\u7814\u7a76\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "XINGYINYANJIUJINGXUANGUPIAOC"}, "008539": {"suoxie": "GYKYLLZZQA", "name": "\u5de5\u94f6\u5f00\u5143\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINKAIYUANLILVZHAIZHAIQUANA"}, "008540": {"suoxie": "GYKYLLZZQC", "name": "\u5de5\u94f6\u5f00\u5143\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINKAIYUANLILVZHAIZHAIQUANC"}, "008541": {"suoxie": "XBLDXXHHA", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XIBULIDEXINXIANGHUNHEA"}, "008542": {"suoxie": "XBLDXXHHC", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XIBULIDEXINXIANGHUNHEC"}, "008543": {"suoxie": "XBLDXRHHA", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XIBULIDEXINRUIHUNHEA"}, "008544": {"suoxie": "XBLDXRHHC", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XIBULIDEXINRUIHUNHEC"}, "008545": {"suoxie": "HDFRSNCYQHH", "name": "\u6cd3\u5fb7\u4e30\u6da6\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEFENGRUNSANNIANCHIYOUQIHUNHE"}, "008546": {"suoxie": "NFCYYSLNHHA", "name": "\u5357\u65b9\u4ea7\u4e1a\u4f18\u52bf\u4e24\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHANYEYOUSHILIANGNIANHUNHEA"}, "008547": {"suoxie": "BDAY6GYDKHH", "name": "\u535a\u9053\u5b89\u8fdc6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BODAOANYUAN6GEYUEDINGKAIHUNHE"}, "008548": {"suoxie": "ZSHYCZC", "name": "\u6d59\u5546\u60e0\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIYINGCHUNZHAIC"}, "008549": {"suoxie": "HAZZGXXYZA", "name": "\u6c47\u5b89\u4e2d\u503a-\u5e7f\u897f\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANZHONGZHAIGUANGXIXINYONGZHAIA"}, "008550": {"suoxie": "HAZZGXXYZC", "name": "\u6c47\u5b89\u4e2d\u503a-\u5e7f\u897f\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANZHONGZHAIGUANGXIXINYONGZHAIC"}, "008551": {"suoxie": "DCYYA", "name": "\u4e1c\u8d22\u533b\u836fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYIYAOA"}, "008552": {"suoxie": "DCYYC", "name": "\u4e1c\u8d22\u533b\u836fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYIYAOC"}, "008553": {"suoxie": "WJYL2035SNCYQHHFOF", "name": "\u4e07\u5bb6\u517b\u80012035\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "WANJIAYANGLAO2035SANNIANCHIYOUQIHUNHEFOF"}, "008554": {"suoxie": "JSCCJTHLDKZC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u6c47\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIHUILIDINGKAIZHAIC"}, "008555": {"suoxie": "HSLTYSHH", "name": "\u534e\u5546\u9f99\u5934\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGLONGTOUYOUSHIHUNHE"}, "008556": {"suoxie": "YFDYFZQA", "name": "\u6613\u65b9\u8fbe\u88d5\u5bcc\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUFUZHAIQUANA"}, "008557": {"suoxie": "YFDYFZQC", "name": "\u6613\u65b9\u8fbe\u88d5\u5bcc\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUFUZHAIQUANC"}, "008558": {"suoxie": "YYBLZQA", "name": "\u6c38\u8d62\u90a6\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGBANGLIZHAIQUANA"}, "008559": {"suoxie": "YYBLZQC", "name": "\u6c38\u8d62\u90a6\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGBANGLIZHAIQUANC"}, "008560": {"suoxie": "ZYCY39GYDKZA", "name": "\u4e2d\u90ae\u6df3\u60a639\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNYUE39GEYUEDINGKAIZHAIA"}, "008561": {"suoxie": "ZYCY39GYDKZC", "name": "\u4e2d\u90ae\u6df3\u60a639\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNYUE39GEYUEDINGKAIZHAIC"}, "008563": {"suoxie": "YHZYWJPZHHA", "name": "\u94f6\u6cb3\u81fb\u4f18\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHEZHENYOUWENJIANPEIZHIHUNHEA"}, "008564": {"suoxie": "YHZYWJPZHHC", "name": "\u94f6\u6cb3\u81fb\u4f18\u7a33\u5065\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHEZHENYOUWENJIANPEIZHIHUNHEC"}, "008565": {"suoxie": "TKAZZDZA", "name": "\u6cf0\u5eb7\u5b89\u6cfd\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIKANGANZEZHONGDUANZHAIA"}, "008566": {"suoxie": "FCTYCZA", "name": "\u8702\u5de2\u6dfb\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYINGCHUNZHAIA"}, "008567": {"suoxie": "FCTYCZC", "name": "\u8702\u5de2\u6dfb\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYINGCHUNZHAIC"}, "008568": {"suoxie": "FCFYYNDKZFQS", "name": "\u8702\u5de2\u4e30\u4e1a\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGYEYINIANDINGKAIZHAIFAQISHI"}, "008571": {"suoxie": "JXMDCZA", "name": "\u91d1\u4fe1\u6c11\u8fbe\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINXINMINDACHUNZHAIA"}, "008572": {"suoxie": "JXMDCZC", "name": "\u91d1\u4fe1\u6c11\u8fbe\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINXINMINDACHUNZHAIC"}, "008574": {"suoxie": "ZJ13NZJZZS", "name": "\u4e2d\u52a01-3\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIA13NIANZHENGJINZHAIZHISHU"}, "008575": {"suoxie": "CTYH63GYDKZ", "name": "\u8d22\u901a\u88d5\u60e063\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGYUHUI63GEYUEDINGKAIZHAI"}, "008578": {"suoxie": "DHXSDZA", "name": "\u4e1c\u6d77\u7965\u82cf\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGHAIXIANGSUDUANZHAIA"}, "008579": {"suoxie": "DHXSDZC", "name": "\u4e1c\u6d77\u7965\u82cf\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGHAIXIANGSUDUANZHAIC"}, "008580": {"suoxie": "JTJJCZ3GYA", "name": "\u4e5d\u6cf0\u4e45\u5609\u7eaf\u503a3\u4e2a\u6708A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIUTAIJIUJIACHUNZHAI3GEYUEA"}, "008581": {"suoxie": "JTJJCZ3GYC", "name": "\u4e5d\u6cf0\u4e45\u5609\u7eaf\u503a3\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIUTAIJIUJIACHUNZHAI3GEYUEC"}, "008582": {"suoxie": "XYJFZQ", "name": "\u5174\u94f6\u805a\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINJUFENGZHAIQUAN"}, "008583": {"suoxie": "XBLDZZ13NZJZZSA", "name": "\u897f\u90e8\u5229\u5f97\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEZHONGZHAI13NIANZHENGJINZHAIZHISHUA"}, "008584": {"suoxie": "XBLDZZ13NZJZZSC", "name": "\u897f\u90e8\u5229\u5f97\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEZHONGZHAI13NIANZHENGJINZHAIZHISHUC"}, "008585": {"suoxie": "HXZZRGZNZTETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEA"}, "008586": {"suoxie": "HXZZRGZNZTETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEC"}, "008587": {"suoxie": "CHZDZZQA", "name": "\u6df3\u539a\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUNHOUZHONGDUANZHAIZHAIQUANA"}, "008588": {"suoxie": "CHZDZZQC", "name": "\u6df3\u539a\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUNHOUZHONGDUANZHAIZHAIQUANC"}, "008590": {"suoxie": "THZZQZZQGSETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "008591": {"suoxie": "THZZQZZQGSETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "008592": {"suoxie": "THHS300ZSZQA", "name": "\u5929\u5f18\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUSHEN300ZHISHUZENGQIANGA"}, "008593": {"suoxie": "THHS300ZSZQC", "name": "\u5929\u5f18\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUSHEN300ZHISHUZENGQIANGC"}, "008594": {"suoxie": "PAHRDKZ", "name": "\u5e73\u5b89\u5408\u6da6\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHERUNDINGKAIZHAI"}, "008595": {"suoxie": "PAHZCZZQ", "name": "\u5e73\u5b89\u60e0\u667a\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIZHICHUNZHAIZHAIQUAN"}, "008596": {"suoxie": "PALS39GYDKZA", "name": "\u5e73\u5b89\u4e50\u987a39\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANLESHUN39GEYUEDINGKAIZHAIA"}, "008597": {"suoxie": "PALS39GYDKZC", "name": "\u5e73\u5b89\u4e50\u987a39\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANLESHUN39GEYUEDINGKAIZHAIC"}, "008598": {"suoxie": "BYZZ13NGKZA", "name": "\u5b9d\u76c8\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGZHONGZHAI13NIANGUOKAIZHAIA"}, "008599": {"suoxie": "BYZZ13NGKZC", "name": "\u5b9d\u76c8\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGZHONGZHAI13NIANGUOKAIZHAIC"}, "008602": {"suoxie": "FZFBXXCZHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u65b0\u5174\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGXINXINGCHENGZHANGHUNHEA"}, "008603": {"suoxie": "FZFBXXCZHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u65b0\u5174\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGXINXINGCHENGZHANGHUNHEC"}, "008604": {"suoxie": "GFWALHPZC", "name": "\u5e7f\u53d1\u7a33\u5b89\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAWENANLINGHUOPEIZHIC"}, "008605": {"suoxie": "ZSHMCZC", "name": "\u6d59\u5546\u60e0\u6c11\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIMINCHUNZHAIC"}, "008606": {"suoxie": "GFHZYNDKZA", "name": "\u5e7f\u53d1\u6c47\u62e9\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIZEYINIANDINGKAIZHAIA"}, "008607": {"suoxie": "GFHZYNDKZC", "name": "\u5e7f\u53d1\u6c47\u62e9\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIZEYINIANDINGKAIZHAIC"}, "008608": {"suoxie": "GFHPSNDQKFZQ", "name": "\u5e7f\u53d1\u6c47\u6d66\u4e09\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIPUSANNIANDINGQIKAIFANGZHAIQUAN"}, "008609": {"suoxie": "GFYL2040FOF", "name": "\u5e7f\u53d1\u517b\u80012040(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAYANGLAO2040FOF"}, "008610": {"suoxie": "HFTTXSYZQC", "name": "\u6d77\u5bcc\u901a\u6dfb\u946b\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGTIANXINSHOUYIZHAIQUANC"}, "008611": {"suoxie": "HFTTXSYZQA", "name": "\u6d77\u5bcc\u901a\u6dfb\u946b\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGTIANXINSHOUYIZHAIQUANA"}, "008612": {"suoxie": "GTRYSHCZZQ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u6052\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNHENGCHUNZHAIZHAIQUAN"}, "008613": {"suoxie": "ZSHJAX66GYDQA", "name": "\u6d59\u5546\u6c47\u91d1\u5b89\u4eab66\u4e2a\u6708\u5b9a\u671fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINANXIANG66GEYUEDINGQIA"}, "008614": {"suoxie": "ZSHJAX66GYDQC", "name": "\u6d59\u5546\u6c47\u91d1\u5b89\u4eab66\u4e2a\u6708\u5b9a\u671fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINANXIANG66GEYUEDINGQIC"}, "008615": {"suoxie": "ZSHJJHLNDKZA", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u6cd3\u4e24\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINJUHONGLIANGNIANDINGKAIZHAIA"}, "008616": {"suoxie": "ZSHJJHLNDKZC", "name": "\u6d59\u5546\u6c47\u91d1\u805a\u6cd3\u4e24\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIJINJUHONGLIANGNIANDINGKAIZHAIC"}, "008617": {"suoxie": "GSWJYLYNHHFOF", "name": "\u56fd\u5bff\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUWENJIANYANGLAOYINIANHUNHEFOF"}, "008618": {"suoxie": "YYYYJKA", "name": "\u6c38\u8d62\u533b\u836f\u5065\u5eb7A", "style": "\u80a1\u7968\u578b", "pinyin": "YONGYINGYIYAOJIANKANGA"}, "008619": {"suoxie": "YYYYJKC", "name": "\u6c38\u8d62\u533b\u836f\u5065\u5eb7C", "style": "\u80a1\u7968\u578b", "pinyin": "YONGYINGYIYAOJIANKANGC"}, "008620": {"suoxie": "JSZN3GYDKCZZQ", "name": "\u5609\u5b9e\u81f4\u5b813\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHINING3GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "008621": {"suoxie": "THYYWJYLYNFOF", "name": "\u5929\u5f18\u6c38\u88d5\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYONGYUWENJIANYANGLAOYINIANFOF"}, "008622": {"suoxie": "PHGXLTETFLJA", "name": "\u9e4f\u534e\u80a1\u606f\u9f99\u5934ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUXILONGTOUETFLIANJIEA"}, "008623": {"suoxie": "PHGXLTETFLJC", "name": "\u9e4f\u534e\u80a1\u606f\u9f99\u5934ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUXILONGTOUETFLIANJIEC"}, "008624": {"suoxie": "HAYX12GYDKCZZQ", "name": "\u6c47\u5b89\u88d5\u946b12\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANYUXIN12GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "008625": {"suoxie": "GFPHYLSNHHFOF", "name": "\u56fd\u5bcc\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOFUPINGHENGYANGLAOSANNIANHUNHEFOF"}, "008626": {"suoxie": "NF05NJSCTZA", "name": "\u5357\u65b90-5\u5e74\u6c5f\u82cf\u57ce\u6295\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG05NIANJIANGSUCHENGTOUZHAIA"}, "008627": {"suoxie": "NF05NJSCTZC", "name": "\u5357\u65b90-5\u5e74\u6c5f\u82cf\u57ce\u6295\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG05NIANJIANGSUCHENGTOUZHAIC"}, "008628": {"suoxie": "DCHXYNDKZQ", "name": "\u5927\u6210\u60e0\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIXIANGYINIANDINGKAIZHAIQUAN"}, "008629": {"suoxie": "DCJRWJPZHHA", "name": "\u5927\u6210\u666f\u745e\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGJINGRUIWENJIANPEIZHIHUNHEA"}, "008630": {"suoxie": "DCJRWJPZHHC", "name": "\u5927\u6210\u666f\u745e\u7a33\u5065\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGJINGRUIWENJIANPEIZHIHUNHEC"}, "008631": {"suoxie": "GTMZPHYLFOF", "name": "\u56fd\u6cf0\u6c11\u6cfd\u5e73\u8861\u517b\u8001FOF", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOTAIMINZEPINGHENGYANGLAOFOF"}, "008632": {"suoxie": "NFJYDZE", "name": "\u5357\u65b9\u5409\u5143\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGJIYUANDUANZHAIE"}, "008633": {"suoxie": "WJKJCXHHA", "name": "\u4e07\u5bb6\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAKEJICHUANGXINHUNHEA"}, "008634": {"suoxie": "WJKJCXHHC", "name": "\u4e07\u5bb6\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAKEJICHUANGXINHUNHEC"}, "008635": {"suoxie": "HAKJCXHH", "name": "\u534e\u5b89\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANKEJICHUANGXINHUNHE"}, "008636": {"suoxie": "QHLHTRCZA", "name": "\u524d\u6d77\u8054\u5408\u6cf0\u745e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETAIRUICHUNZHAIA"}, "008637": {"suoxie": "GJHXYNDK", "name": "\u56fd\u91d1\u60e0\u4eab\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIXIANGYINIANDINGKAI"}, "008638": {"suoxie": "GFKJCXHHA", "name": "\u5e7f\u53d1\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAKEJICHUANGXINHUNHEA"}, "008639": {"suoxie": "ZOYJYL2025YNCYFOF", "name": "\u4e2d\u6b27\u9884\u89c1\u517b\u80012025\u4e00\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUYUJIANYANGLAO2025YINIANCHIYOUFOF"}, "008640": {"suoxie": "FZFBKJCXA", "name": "\u65b9\u6b63\u5bcc\u90a6\u79d1\u6280\u521b\u65b0A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGKEJICHUANGXINA"}, "008641": {"suoxie": "FZFBKJCXC", "name": "\u65b9\u6b63\u5bcc\u90a6\u79d1\u6280\u521b\u65b0C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGKEJICHUANGXINC"}, "008642": {"suoxie": "GJHYCZA", "name": "\u56fd\u91d1\u60e0\u8fdc\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIYUANCHUNZHAIA"}, "008643": {"suoxie": "GJHYCZC", "name": "\u56fd\u91d1\u60e0\u8fdc\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIYUANCHUNZHAIC"}, "008644": {"suoxie": "THJJXSGYDKA", "name": "\u5929\u5f18\u5b63\u5b63\u5174\u4e09\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGJIJIXINGSANGEYUEDINGKAIA"}, "008645": {"suoxie": "THJJXSGYDKC", "name": "\u5929\u5f18\u5b63\u5b63\u5174\u4e09\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGJIJIXINGSANGEYUEDINGKAIC"}, "008646": {"suoxie": "THZLDZA", "name": "\u5929\u5f18\u589e\u5229\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGZENGLIDUANZHAIA"}, "008647": {"suoxie": "THZLDZC", "name": "\u5929\u5f18\u589e\u5229\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGZENGLIDUANZHAIC"}, "008648": {"suoxie": "JSZYYNDQCZZQ", "name": "\u5609\u5b9e\u81f4\u4e1a\u4e00\u5e74\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIYEYINIANDINGQICHUNZHAIZHAIQUAN"}, "008649": {"suoxie": "HTBRJX39GYDKZ", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u517439\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIJINXING39GEYUEDINGKAIZHAI"}, "008650": {"suoxie": "HTBRYSYNDKZQ", "name": "\u534e\u6cf0\u67cf\u745e\u76ca\u5546\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIYISHANGYINIANDINGKAIZHAIQUAN"}, "008651": {"suoxie": "BSFJYNQDKZFQS", "name": "\u535a\u65f6\u5bcc\u8fdb\u4e00\u5e74\u671f\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUJINYINIANQIDINGKAIZHAIFAQISHI"}, "008652": {"suoxie": "CCZZ13NZJZA", "name": "\u957f\u57ce\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI13NIANZHENGJINZHAIA"}, "008653": {"suoxie": "CCZZ13NZJZC", "name": "\u957f\u57ce\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI13NIANZHENGJINZHAIC"}, "008654": {"suoxie": "NDHYYNDK", "name": "\u8bfa\u5fb7\u6c47\u76c8\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUODEHUIYINGYINIANDINGKAI"}, "008655": {"suoxie": "ZSKJCXHHA", "name": "\u62db\u5546\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGKEJICHUANGXINHUNHEA"}, "008656": {"suoxie": "ZSKJCXHHC", "name": "\u62db\u5546\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGKEJICHUANGXINHUNHEC"}, "008657": {"suoxie": "JSCCKJCXHH", "name": "\u666f\u987a\u957f\u57ce\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGKEJICHUANGXINHUNHE"}, "008658": {"suoxie": "NYGZJZJZ15NZS", "name": "\u519c\u94f6\u56fd\u503a\u53ca\u653f\u91d1\u503a1-5\u5e74\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINGUOZHAIJIZHENGJINZHAI15NIANZHISHU"}, "008659": {"suoxie": "ZYCX66GYDKZ", "name": "\u4e2d\u90ae\u6df3\u4eab66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNXIANG66GEYUEDINGKAIZHAI"}, "008661": {"suoxie": "JSZRYNDQZQ", "name": "\u5609\u5b9e\u81f4\u878d\u4e00\u5e74\u5b9a\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIRONGYINIANDINGQIZHAIQUAN"}, "008662": {"suoxie": "ZYAXYNDKZFQS", "name": "\u4e2d\u94f6\u6fb3\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINAOXIANGYINIANDINGKAIZHAIFAQISHI"}, "008663": {"suoxie": "ZYTXYNDKZ", "name": "\u4e2d\u94f6\u540c\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINTONGXIANGYINIANDINGKAIZHAI"}, "008664": {"suoxie": "JSXHYNCYQHHA", "name": "\u5609\u5b9e\u946b\u548c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINHEYINIANCHIYOUQIHUNHEA"}, "008665": {"suoxie": "JSXHYNCYQHHC", "name": "\u5609\u5b9e\u946b\u548c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINHEYINIANCHIYOUQIHUNHEC"}, "008666": {"suoxie": "GTXLYNCYQHHA", "name": "\u56fd\u6cf0\u946b\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIXINLIYINIANCHIYOUQIHUNHEA"}, "008667": {"suoxie": "GTXLYNCYQHHC", "name": "\u56fd\u6cf0\u946b\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIXINLIYINIANCHIYOUQIHUNHEC"}, "008668": {"suoxie": "XBLDSYYNDKZQ", "name": "\u897f\u90e8\u5229\u5f97\u53cc\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDESHUANGYINGYINIANDINGKAIZHAIQUAN"}, "008669": {"suoxie": "FZFBHL39GYDKZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u79be\u522939\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHELI39GEYUEDINGKAIZHAIA"}, "008670": {"suoxie": "FZFBHL39GYDKZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u79be\u522939\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGHELI39GEYUEDINGKAIZHAIC"}, "008671": {"suoxie": "YHKJCXHH", "name": "\u94f6\u534e\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAKEJICHUANGXINHUNHE"}, "008672": {"suoxie": "BYXZHHA", "name": "\u5b9d\u76c8\u7965\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGZEHUNHEA"}, "008673": {"suoxie": "BYXZHHC", "name": "\u5b9d\u76c8\u7965\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGZEHUNHEC"}, "008674": {"suoxie": "BSWY63GYDKZ", "name": "\u535a\u65f6\u7a33\u60a663\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIWENYUE63GEYUEDINGKAIZHAI"}, "008675": {"suoxie": "HAXPDKZA", "name": "\u534e\u5b89\u946b\u6d66\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINPUDINGKAIZHAIA"}, "008676": {"suoxie": "HAXPDKZC", "name": "\u534e\u5b89\u946b\u6d66\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINPUDINGKAIZHAIC"}, "008677": {"suoxie": "YHZZ13NGKHZQ", "name": "\u94f6\u534e\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAZHONGZHAI13NIANGUOKAIHANGZHAIQUAN"}, "008678": {"suoxie": "CTXLCZ12GYDKZ", "name": "\u8d22\u901a\u5174\u5229\u7eaf\u503a12\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGXINGLICHUNZHAI12GEYUEDINGKAIZHAI"}, "008681": {"suoxie": "PHJZCZHH", "name": "\u9e4f\u534e\u4ef7\u503c\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHICHENGZHANGHUNHE"}, "008682": {"suoxie": "FGZZHLZSZQC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHONGLIZHISHUZENGQIANGC"}, "008684": {"suoxie": "BYYXCZZQA", "name": "\u5b9d\u76c8\u76c8\u65ed\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGXUCHUNZHAIZHAIQUANA"}, "008685": {"suoxie": "BYYXCZZQC", "name": "\u5b9d\u76c8\u76c8\u65ed\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGXUCHUNZHAIZHAIQUANC"}, "008686": {"suoxie": "DCJYZDZA", "name": "\u5927\u6210\u666f\u4f18\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGYOUZHONGDUANZHAIA"}, "008687": {"suoxie": "DCJYZDZC", "name": "\u5927\u6210\u666f\u4f18\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGYOUZHONGDUANZHAIC"}, "008688": {"suoxie": "DCJLCZZQA", "name": "\u5927\u6210\u666f\u4e50\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGLECHUNZHAIZHAIQUANA"}, "008689": {"suoxie": "DCJLCZZQC", "name": "\u5927\u6210\u666f\u4e50\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGLECHUNZHAIZHAIQUANC"}, "008690": {"suoxie": "PAZLLGYDKZA", "name": "\u5e73\u5b89\u589e\u5229\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGLILIUGEYUEDINGKAIZHAIA"}, "008691": {"suoxie": "PAZLLGYDKZC", "name": "\u5e73\u5b89\u589e\u5229\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGLILIUGEYUEDINGKAIZHAIC"}, "008692": {"suoxie": "PAZLLGYDKZE", "name": "\u5e73\u5b89\u589e\u5229\u516d\u4e2a\u6708\u5b9a\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGLILIUGEYUEDINGKAIZHAIE"}, "008693": {"suoxie": "MSJY39GYDQCZ", "name": "\u6c11\u751f\u52a0\u94f639\u4e2a\u6708\u5b9a\u671f\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYIN39GEYUEDINGQICHUNZHAI"}, "008694": {"suoxie": "PAYSCDZA", "name": "\u5e73\u5b89\u5143\u76db\u8d85\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANSHENGCHAODUANZHAIA"}, "008695": {"suoxie": "PAYSCDZC", "name": "\u5e73\u5b89\u5143\u76db\u8d85\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANSHENGCHAODUANZHAIC"}, "008696": {"suoxie": "PAYSCDZE", "name": "\u5e73\u5b89\u5143\u76db\u8d85\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANSHENGCHAODUANZHAIE"}, "008697": {"suoxie": "JYYL2035SNFOF", "name": "\u4ea4\u94f6\u517b\u80012035\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIAOYINYANGLAO2035SANNIANFOF"}, "008698": {"suoxie": "QHLHYJCZA", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u5609\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGJIACHUNZHAIA"}, "008699": {"suoxie": "QHLHYJCZC", "name": "\u524d\u6d77\u8054\u5408\u6cf3\u5609\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHEYONGJIACHUNZHAIC"}, "008700": {"suoxie": "TKRF3YDKZQ", "name": "\u6cf0\u5eb7\u745e\u4e303\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGRUIFENG3YUEDINGKAIZHAIQUAN"}, "008701": {"suoxie": "HXHJETFLJA", "name": "\u534e\u590f\u9ec4\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIAHUANGJINETFLIANJIEA"}, "008702": {"suoxie": "HXHJETFLJC", "name": "\u534e\u590f\u9ec4\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIAHUANGJINETFLIANJIEC"}, "008703": {"suoxie": "QHLHTRCZC", "name": "\u524d\u6d77\u8054\u5408\u6cf0\u745e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETAIRUICHUNZHAIC"}, "008704": {"suoxie": "GFGGXYXHHA", "name": "\u5e7f\u53d1\u9ad8\u80a1\u606f\u4f18\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGAOGUXIYOUXIANGHUNHEA"}, "008705": {"suoxie": "GFGGXYXHHC", "name": "\u5e7f\u53d1\u9ad8\u80a1\u606f\u4f18\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGAOGUXIYOUXIANGHUNHEC"}, "008706": {"suoxie": "JXFS100ZSQDIIRMBC", "name": "\u5efa\u4fe1\u5bcc\u65f6100\u6307\u6570(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "JIANXINFUSHI100ZHISHUQDIIRENMINBIC"}, "008707": {"suoxie": "JXFS100ZSQDIIMYXHA", "name": "\u5efa\u4fe1\u5bcc\u65f6100\u6307\u6570(QDII)\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "JIANXINFUSHI100ZHISHUQDIIMEIYUANXIANHUIA"}, "008708": {"suoxie": "JXFS100ZSQDIIMYXHC", "name": "\u5efa\u4fe1\u5bcc\u65f6100\u6307\u6570(QDII)\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "JIANXINFUSHI100ZHISHUQDIIMEIYUANXIANHUIC"}, "008709": {"suoxie": "YHLTGP", "name": "\u94f6\u6cb3\u9f99\u5934\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINHELONGTOUGUPIAO"}, "008710": {"suoxie": "BYYSCZZQA", "name": "\u5b9d\u76c8\u76c8\u987a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGSHUNCHUNZHAIZHAIQUANA"}, "008711": {"suoxie": "BYYSCZZQC", "name": "\u5b9d\u76c8\u76c8\u987a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGSHUNCHUNZHAIZHAIQUANC"}, "008712": {"suoxie": "JSCCPZCZHH", "name": "\u666f\u987a\u957f\u57ce\u54c1\u8d28\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGPINZHICHENGZHANGHUNHE"}, "008713": {"suoxie": "GTZZQZJYDQETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5bb6\u7528\u7535\u5668ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIAYONGDIANQIETFLIANJIEA"}, "008714": {"suoxie": "GTZZQZJYDQETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5bb6\u7528\u7535\u5668ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIAYONGDIANQIETFLIANJIEC"}, "008715": {"suoxie": "JSCCJZQDYNCYHH", "name": "\u666f\u987a\u957f\u57ce\u4ef7\u503c\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGJIAZHIQUDONGYINIANCHIYOUHUNHE"}, "008716": {"suoxie": "PHYZHBLNDKHH", "name": "\u9e4f\u534e\u4f18\u8d28\u56de\u62a5\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYOUZHIHUIBAOLIANGNIANDINGKAIHUNHE"}, "008717": {"suoxie": "DBRH39GYDKZA", "name": "\u5fb7\u90a6\u9510\u605239\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIHENG39GEYUEDINGKAIZHAIA"}, "008718": {"suoxie": "DBRH39GYDKZC", "name": "\u5fb7\u90a6\u9510\u605239\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIHENG39GEYUEDINGKAIZHAIC"}, "008719": {"suoxie": "DBASHHA", "name": "\u5fb7\u90a6\u5b89\u987a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANSHUNHUNHEA"}, "008720": {"suoxie": "DBASHHC", "name": "\u5fb7\u90a6\u5b89\u987a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANSHUNHUNHEC"}, "008721": {"suoxie": "HSHYYNDKZ", "name": "\u534e\u5546\u9e3f\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUASHANGHONGYIYINIANDINGKAIZHAI"}, "008722": {"suoxie": "YYXYCZYNDKFQS", "name": "\u6c38\u8d62\u6b23\u76ca\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGXINYICHUNZHAIYINIANDINGKAIFAQISHI"}, "008723": {"suoxie": "YYXXHH", "name": "\u6c38\u8d62\u946b\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGXINXIANGHUNHE"}, "008724": {"suoxie": "HDYRSNDKZQ", "name": "\u6cd3\u5fb7\u88d5\u745e\u4e09\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGDEYURUISANNIANDINGKAIZHAIQUAN"}, "008726": {"suoxie": "PATYZQA", "name": "\u5e73\u5b89\u6dfb\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANTIANYUZHAIQUANA"}, "008727": {"suoxie": "PATYZQC", "name": "\u5e73\u5b89\u6dfb\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANTIANYUZHAIQUANC"}, "008728": {"suoxie": "TTHLCZA", "name": "\u540c\u6cf0\u6052\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAIHENGLICHUNZHAIA"}, "008729": {"suoxie": "TTHLCZC", "name": "\u540c\u6cf0\u6052\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAIHENGLICHUNZHAIC"}, "008730": {"suoxie": "THCXYNDK", "name": "\u5929\u5f18\u7eaf\u4eab\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGCHUNXIANGYINIANDINGKAI"}, "008731": {"suoxie": "ZSTHCZA", "name": "\u62db\u5546\u6dfb\u6d69\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANHAOCHUNZHAIA"}, "008732": {"suoxie": "ZSTHCZC", "name": "\u62db\u5546\u6dfb\u6d69\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANHAOCHUNZHAIC"}, "008733": {"suoxie": "YFDYLCHBB", "name": "\u6613\u65b9\u8fbe\u6613\u7406\u8d22\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAYILICAIHUOBIB"}, "008734": {"suoxie": "JYKRKJCXHHA", "name": "\u4ea4\u94f6\u79d1\u9510\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINKERUIKEJICHUANGXINHUNHEA"}, "008735": {"suoxie": "HASXSNDKCZZQ", "name": "\u6c47\u5b89\u76db\u946b\u4e09\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANSHENGXINSANNIANDINGKAICHUNZHAIZHAIQUAN"}, "008736": {"suoxie": "NFGGXGPA", "name": "\u5357\u65b9\u9ad8\u80a1\u606f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGGAOGUXIGUPIAOA"}, "008737": {"suoxie": "NFGGXGPC", "name": "\u5357\u65b9\u9ad8\u80a1\u606f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGGAOGUXIGUPIAOC"}, "008738": {"suoxie": "THXXYNDK", "name": "\u5929\u5f18\u5174\u4eab\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINGXIANGYINIANDINGKAI"}, "008739": {"suoxie": "ZOTYYNDQKFZQ", "name": "\u4e2d\u6b27\u540c\u76ca\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUTONGYIYINIANDINGQIKAIFANGZHAIQUAN"}, "008740": {"suoxie": "GSABZSSZZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u76db\u53cc\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUOSHOUANBAOZUNSHENGSHUANGZHAIZHAIQUANA"}, "008741": {"suoxie": "GSABZSSZZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u76db\u53cc\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUOSHOUANBAOZUNSHENGSHUANGZHAIZHAIQUANC"}, "008742": {"suoxie": "TZTDLHBB", "name": "\u5929\u6cbb\u5929\u5f97\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANZHITIANDELIHUOBIB"}, "008743": {"suoxie": "NFJL18GYDKZQA", "name": "\u5357\u65b9\u96c6\u522918\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGJILI18GEYUEDINGKAIZHAIQUANA"}, "008744": {"suoxie": "NFJL18GYDKZQC", "name": "\u5357\u65b9\u96c6\u522918\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGJILI18GEYUEDINGKAIZHAIQUANC"}, "008745": {"suoxie": "NFZLYNZQ", "name": "\u5357\u65b9\u5c0a\u5229\u4e00\u5e74\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZUNLIYINIANZHAIQUAN"}, "008746": {"suoxie": "CTDLCZA", "name": "\u8d22\u901a\u591a\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGDUOLICHUNZHAIA"}, "008747": {"suoxie": "DCJTCZZQA", "name": "\u5927\u6210\u666f\u6cf0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGTAICHUNZHAIZHAIQUANA"}, "008748": {"suoxie": "DCJTCZZQC", "name": "\u5927\u6210\u666f\u6cf0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGTAICHUNZHAIZHAIQUANC"}, "008749": {"suoxie": "FGZZKJ50CLETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u628050\u7b56\u7565ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKEJI50CELUEETFLIANJIEA"}, "008750": {"suoxie": "FGZZKJ50CLETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u628050\u7b56\u7565ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKEJI50CELUEETFLIANJIEC"}, "008751": {"suoxie": "DCQQMYZQDIIARMB", "name": "\u5927\u6210\u5168\u7403\u7f8e\u5143\u503a(QDII)A\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "DACHENGQUANQIUMEIYUANZHAIQDIIARENMINBI"}, "008752": {"suoxie": "DCQQMYZQDIICRMB", "name": "\u5927\u6210\u5168\u7403\u7f8e\u5143\u503a(QDII)C\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "DACHENGQUANQIUMEIYUANZHAIQDIICRENMINBI"}, "008753": {"suoxie": "DCXXPHYLSNFOF", "name": "\u5927\u6210\u5174\u4eab\u5e73\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGXINGXIANGPINGHENGYANGLAOSANNIANFOF"}, "008754": {"suoxie": "TKRF3YCYHHFOFA", "name": "\u6cf0\u5eb7\u777f\u798f3\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGRUIFU3YUECHIYOUHUNHEFOFA"}, "008755": {"suoxie": "TKRF3YCYHHFOFC", "name": "\u6cf0\u5eb7\u777f\u798f3\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGRUIFU3YUECHIYOUHUNHEFOFC"}, "008756": {"suoxie": "MSJYRXYNDKZQFQS", "name": "\u6c11\u751f\u52a0\u94f6\u745e\u590f\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINRUIXIAYINIANDINGKAIZHAIQUANFAQISHI"}, "008757": {"suoxie": "JTJXHHA", "name": "\u4e5d\u6cf0\u805a\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIUTAIJUXINHUNHEA"}, "008758": {"suoxie": "JTJXHHC", "name": "\u4e5d\u6cf0\u805a\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIUTAIJUXINHUNHEC"}, "008759": {"suoxie": "STMGRT38GYDKA", "name": "\u4e0a\u6295\u6469\u6839\u745e\u6cf038\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENRUITAI38GEYUEDINGKAIA"}, "008760": {"suoxie": "STMGRT38GYDKC", "name": "\u4e0a\u6295\u6469\u6839\u745e\u6cf038\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENRUITAI38GEYUEDINGKAIC"}, "008761": {"suoxie": "NFJYZDLLZ", "name": "\u5357\u65b9\u9a8f\u5143\u4e2d\u77ed\u5229\u7387\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGJUNYUANZHONGDUANLILVZHAI"}, "008762": {"suoxie": "THHXYNDK", "name": "\u5929\u5f18\u6052\u4eab\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGHENGXIANGYINIANDINGKAI"}, "008763": {"suoxie": "THYNSCGPQDIIA", "name": "\u5929\u5f18\u8d8a\u5357\u5e02\u573a\u80a1\u7968(QDII)A", "style": "QDII", "pinyin": "TIANHONGYUENANSHICHANGGUPIAOQDIIA"}, "008764": {"suoxie": "THYNSCGPQDIIC", "name": "\u5929\u5f18\u8d8a\u5357\u5e02\u573a\u80a1\u7968(QDII)C", "style": "QDII", "pinyin": "TIANHONGYUENANSHICHANGGUPIAOQDIIC"}, "008765": {"suoxie": "ZJRXCZZQA", "name": "\u4e2d\u52a0\u745e\u4eab\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUIXIANGCHUNZHAIZHAIQUANA"}, "008766": {"suoxie": "CTZGHS12GYDKZQA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u76db12\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGSHENG12GEYUEDINGKAIZHAIQUANA"}, "008767": {"suoxie": "CTZGHS12GYDKZQC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u76db12\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGSHENG12GEYUEDINGKAIZHAIQUANC"}, "008768": {"suoxie": "CJSZCDPLHA", "name": "\u521b\u91d1\u4e0a\u8bc1\u8d85\u5927\u76d8\u91cf\u5316A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINSHANGZHENGCHAODAPANLIANGHUAA"}, "008769": {"suoxie": "CJSZCDPLHC", "name": "\u521b\u91d1\u4e0a\u8bc1\u8d85\u5927\u76d8\u91cf\u5316C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINSHANGZHENGCHAODAPANLIANGHUAC"}, "008770": {"suoxie": "DFHAXZXYNCYHH", "name": "\u4e1c\u65b9\u7ea2\u5b89\u946b\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGANXINZHENXUANYINIANCHIYOUHUNHE"}, "008771": {"suoxie": "NFZYZQA", "name": "\u5357\u65b9\u662d\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHAOYUANZHAIQUANA"}, "008772": {"suoxie": "NFZYZQC", "name": "\u5357\u65b9\u662d\u5143\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHAOYUANZHAIQUANC"}, "008773": {"suoxie": "ZYJTHBHH", "name": "\u4e2d\u94f6\u666f\u6cf0\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINJINGTAIHUIBAOHUNHE"}, "008774": {"suoxie": "ZSXFZDZA", "name": "\u62db\u5546\u946b\u798f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGXINFUZHONGDUANZHAIA"}, "008775": {"suoxie": "ZSXFZDZC", "name": "\u62db\u5546\u946b\u798f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGXINFUZHONGDUANZHAIC"}, "008776": {"suoxie": "HAHS300ETFLJA", "name": "\u534e\u5b89\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANHUSHEN300ETFLIANJIEA"}, "008777": {"suoxie": "HAHS300ETFLJC", "name": "\u534e\u5b89\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANHUSHEN300ETFLIANJIEC"}, "008778": {"suoxie": "JSZZ500ZSZQA", "name": "\u5609\u5b9e\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500ZHISHUZENGQIANGA"}, "008779": {"suoxie": "JSZZ500ZSZQC", "name": "\u5609\u5b9e\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500ZHISHUZENGQIANGC"}, "008780": {"suoxie": "NFNLYNDKZFQS", "name": "\u5357\u65b9\u5b81\u5229\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGNINGLIYINIANDINGKAIZHAIFAQISHI"}, "008783": {"suoxie": "NFLYZDLLZA", "name": "\u5357\u65b9\u4e50\u5143\u4e2d\u77ed\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGLEYUANZHONGDUANLILVZHAIA"}, "008784": {"suoxie": "NFLYZDLLZC", "name": "\u5357\u65b9\u4e50\u5143\u4e2d\u77ed\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGLEYUANZHONGDUANLILVZHAIC"}, "008785": {"suoxie": "ZJBYCZZQ", "name": "\u4e2d\u52a0\u535a\u88d5\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIABOYUCHUNZHAIZHAIQUAN"}, "008786": {"suoxie": "CCJKSHLHPZHH", "name": "\u957f\u57ce\u5065\u5eb7\u751f\u6d3b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIANKANGSHENGHUOLINGHUOPEIZHIHUNHE"}, "008787": {"suoxie": "RTZZ13NGKHZQZSA", "name": "\u878d\u901a\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHUA"}, "008788": {"suoxie": "RTZZ13NGKHZQZSC", "name": "\u878d\u901a\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHUC"}, "008791": {"suoxie": "ZSAHZQA", "name": "\u62db\u5546\u5b89\u534e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANHUAZHAIQUANA"}, "008792": {"suoxie": "ZSAHZQC", "name": "\u62db\u5546\u5b89\u534e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANHUAZHAIQUANC"}, "008793": {"suoxie": "BDJYHHA", "name": "\u535a\u9053\u5609\u5143\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIAYUANHUNHEA"}, "008794": {"suoxie": "BDJYHHC", "name": "\u535a\u9053\u5609\u5143\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIAYUANHUNHEC"}, "008795": {"suoxie": "HFTAEFDCHHC", "name": "\u6d77\u5bcc\u901a\u963f\u5c14\u6cd5\u5bf9\u51b2\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HAIFUTONGAERFADUICHONGHUNHEC"}, "008796": {"suoxie": "ZRHACZA", "name": "\u4e2d\u878d\u6052\u5b89\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGANCHUNZHAIA"}, "008797": {"suoxie": "ZRHACZC", "name": "\u4e2d\u878d\u6052\u5b89\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGANCHUNZHAIC"}, "008798": {"suoxie": "GJHALLZA", "name": "\u56fd\u91d1\u60e0\u5b89\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIANLILVZHAIA"}, "008799": {"suoxie": "GJHALLZC", "name": "\u56fd\u91d1\u60e0\u5b89\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIANLILVZHAIC"}, "008802": {"suoxie": "PYASSHYNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u6656\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGHUIYINIANDINGKAIZHAIQUAN"}, "008803": {"suoxie": "HFTRH6GYZQ", "name": "\u6d77\u5bcc\u901a\u745e\u5f186\u4e2a\u6708\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIHONG6GEYUEZHAIQUAN"}, "008804": {"suoxie": "ZSTHCZA", "name": "\u62db\u5546\u6dfb\u534e\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANHUACHUNZHAIA"}, "008805": {"suoxie": "ZSTHCZC", "name": "\u62db\u5546\u6dfb\u534e\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANHUACHUNZHAIC"}, "008806": {"suoxie": "XYJLYNDKZ", "name": "\u946b\u5143\u9526\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANJINLIYINIANDINGKAIZHAI"}, "008807": {"suoxie": "PYCYYNDKZFQS", "name": "\u9e4f\u626c\u6df3\u60a6\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNYUEYINIANDINGKAIZHAIFAQISHI"}, "008808": {"suoxie": "XHAXHZ39GYDKZC", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u6cfd39\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIZE39GEYUEDINGKAIZHAIC"}, "008809": {"suoxie": "AXMWZZHHA", "name": "\u5b89\u4fe1\u6c11\u7a33\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINMINWENZENGZHANGHUNHEA"}, "008810": {"suoxie": "AXMWZZHHC", "name": "\u5b89\u4fe1\u6c11\u7a33\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINMINWENZENGZHANGHUNHEC"}, "008811": {"suoxie": "PHKJCXHH", "name": "\u9e4f\u534e\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAKEJICHUANGXINHUNHE"}, "008813": {"suoxie": "ZSZZ13NGKZA", "name": "\u62db\u5546\u4e2d\u503a-1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHONGZHAI13NIANGUOKAIZHAIA"}, "008814": {"suoxie": "ZSZZ13NGKZC", "name": "\u62db\u5546\u4e2d\u503a-1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHONGZHAI13NIANGUOKAIZHAIC"}, "008817": {"suoxie": "HBKZZZQC", "name": "\u534e\u5b9d\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUABAOKEZHUANZHAIZHAIQUANC"}, "008818": {"suoxie": "HAHX12GYDKCZZQ", "name": "\u6c47\u5b89\u6052\u946b12\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANHENGXIN12GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "008819": {"suoxie": "NYHLCLQSHH", "name": "\u519c\u94f6\u6c47\u7406\u7b56\u7565\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILICELUEQUSHIHUNHE"}, "008820": {"suoxie": "DCJYZDZA", "name": "\u5927\u6210\u666f\u60a6\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGYUEZHONGDUANZHAIA"}, "008821": {"suoxie": "DCJYZDZC", "name": "\u5927\u6210\u666f\u60a6\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGJINGYUEZHONGDUANZHAIC"}, "008822": {"suoxie": "JSCCZZ13NGKHZQZSA", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHUA"}, "008823": {"suoxie": "JSCCZZ13NGKHZQZSC", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHUC"}, "008825": {"suoxie": "MSRYYNDKZFQS", "name": "\u6c11\u751f\u745e\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGRUIYINGYINIANDINGKAIZHAIFAQISHI"}, "008826": {"suoxie": "THCXYNDK", "name": "\u5929\u5f18\u6210\u4eab\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGCHENGXIANGYINIANDINGKAI"}, "008827": {"suoxie": "JXYSZSSNYHGQHETFLJA", "name": "\u5efa\u4fe1\u6613\u76db\u90d1\u5546\u6240\u80fd\u6e90\u5316\u5de5\u671f\u8d27ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINYISHENGZHENGSHANGSUONENGYUANHUAGONGQIHUOETFLIANJIEA"}, "008828": {"suoxie": "JXYSZSSNYHGQHETFLJC", "name": "\u5efa\u4fe1\u6613\u76db\u90d1\u5546\u6240\u80fd\u6e90\u5316\u5de5\u671f\u8d27ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINYISHENGZHENGSHANGSUONENGYUANHUAGONGQIHUOETFLIANJIEC"}, "008829": {"suoxie": "BSFYYNDKZFQS", "name": "\u535a\u65f6\u5bcc\u6d0b\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUYANGYINIANDINGKAIZHAIFAQISHI"}, "008830": {"suoxie": "HFTAYDCHHC", "name": "\u6d77\u5bcc\u901a\u5b89\u76ca\u5bf9\u51b2\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HAIFUTONGANYIDUICHONGHUNHEC"}, "008831": {"suoxie": "HFTAYDCHHA", "name": "\u6d77\u5bcc\u901a\u5b89\u76ca\u5bf9\u51b2\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HAIFUTONGANYIDUICHONGHUNHEA"}, "008832": {"suoxie": "HFTZZCSJLXETFLJ", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1\u957f\u4e09\u89d2\u9886\u5148ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENGCHANGSANJIAOLINGXIANETFLIANJIE"}, "008833": {"suoxie": "YHHYYNCYQHHA", "name": "\u94f6\u534e\u6c47\u76c8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAHUIYINGYINIANCHIYOUQIHUNHEA"}, "008834": {"suoxie": "YHHYYNCYQHHC", "name": "\u94f6\u534e\u6c47\u76c8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAHUIYINGYINIANCHIYOUQIHUNHEC"}, "008835": {"suoxie": "FGLHSGYCYQHHA", "name": "\u5bcc\u56fd\u91cf\u5316\u4e09\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "FUGUOLIANGHUASANGEYUECHIYOUQIHUNHEA"}, "008836": {"suoxie": "FGLHSGYCYQHHC", "name": "\u5bcc\u56fd\u91cf\u5316\u4e09\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "FUGUOLIANGHUASANGEYUECHIYOUQIHUNHEC"}, "008837": {"suoxie": "RTTYHH", "name": "\u878d\u901a\u901a\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGTONGYIHUNHE"}, "008838": {"suoxie": "DBLHDCHHA", "name": "\u5fb7\u90a6\u91cf\u5316\u5bf9\u51b2\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "DEBANGLIANGHUADUICHONGHUNHEA"}, "008839": {"suoxie": "DBLHDCHHC", "name": "\u5fb7\u90a6\u91cf\u5316\u5bf9\u51b2\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "DEBANGLIANGHUADUICHONGHUNHEC"}, "008840": {"suoxie": "DBDXFHHA", "name": "\u5fb7\u90a6\u5927\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGDAXIAOFEIHUNHEA"}, "008841": {"suoxie": "DBDXFHHC", "name": "\u5fb7\u90a6\u5927\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGDAXIAOFEIHUNHEC"}, "008842": {"suoxie": "TTYJHHA", "name": "\u540c\u6cf0\u8fdc\u89c1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TONGTAIYUANJIANHUNHEA"}, "008843": {"suoxie": "TTYJHHC", "name": "\u540c\u6cf0\u8fdc\u89c1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TONGTAIYUANJIANHUNHEC"}, "008844": {"suoxie": "STMGZZ13NGKZZSA", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENZHONGZHAI13NIANGUOKAIZHAIZHISHUA"}, "008845": {"suoxie": "STMGZZ13NGKZZSC", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENZHONGZHAI13NIANGUOKAIZHAIZHISHUC"}, "008846": {"suoxie": "DCMWZZHHA", "name": "\u5927\u6210\u6c11\u7a33\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGMINWENZENGZHANGHUNHEA"}, "008847": {"suoxie": "DCMWZZHHC", "name": "\u5927\u6210\u6c11\u7a33\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGMINWENZENGZHANGHUNHEC"}, "008848": {"suoxie": "ZRZXDC3GYDKHH", "name": "\u4e2d\u878d\u667a\u9009\u5bf9\u51b23\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ZHONGRONGZHIXUANDUICHONG3GEYUEDINGKAIHUNHE"}, "008850": {"suoxie": "JSCCJZWJDKHH", "name": "\u666f\u987a\u957f\u57ce\u4ef7\u503c\u7a33\u8fdb\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGJIAZHIWENJINDINGKAIHUNHE"}, "008851": {"suoxie": "JSCCLHDCCLSGYDK", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u5bf9\u51b2\u7b56\u7565\u4e09\u4e2a\u6708\u5b9a\u5f00", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "JINGSHUNCHANGCHENGLIANGHUADUICHONGCELUESANGEYUEDINGKAI"}, "008854": {"suoxie": "NFNXZZLNGPA", "name": "\u5357\u65b9\u5185\u9700\u589e\u957f\u4e24\u5e74\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGNEIXUZENGZHANGLIANGNIANGUPIAOA"}, "008855": {"suoxie": "NFNXZZLNGPC", "name": "\u5357\u65b9\u5185\u9700\u589e\u957f\u4e24\u5e74\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGNEIXUZENGZHANGLIANGNIANGUPIAOC"}, "008856": {"suoxie": "HXATDCCL3GYDKHH", "name": "\u534e\u590f\u5b89\u6cf0\u5bf9\u51b2\u7b56\u75653\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HUAXIAANTAIDUICHONGCELUE3GEYUEDINGKAIHUNHE"}, "008857": {"suoxie": "HXDHZQA", "name": "\u534e\u590f\u9f0e\u822a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGHANGZHAIQUANA"}, "008858": {"suoxie": "HXDHZQC", "name": "\u534e\u590f\u9f0e\u822a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGHANGZHAIQUANC"}, "008860": {"suoxie": "MSJYLTYXGP", "name": "\u6c11\u751f\u52a0\u94f6\u9f99\u5934\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINLONGTOUYOUXUANGUPIAO"}, "008861": {"suoxie": "XBLDGGTXJYHHA", "name": "\u897f\u90e8\u5229\u5f97\u6e2f\u80a1\u901a\u65b0\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEGANGGUTONGXINJIYUHUNHEA"}, "008862": {"suoxie": "ZYZQHYDKZ", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u8fdc\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIYUANDINGKAIZHAI"}, "008863": {"suoxie": "ZYZQHXDQKFZQ", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u5174\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIXINGDINGQIKAIFANGZHAIQUAN"}, "008864": {"suoxie": "XYZDZA", "name": "\u946b\u5143\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINYUANZHONGDUANZHAIA"}, "008865": {"suoxie": "XYZDZC", "name": "\u946b\u5143\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINYUANZHONGDUANZHAIC"}, "008866": {"suoxie": "BSCYXQSHHA", "name": "\u535a\u65f6\u4ea7\u4e1a\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEXINQUSHIHUNHEA"}, "008867": {"suoxie": "BSCYXQSHHC", "name": "\u535a\u65f6\u4ea7\u4e1a\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEXINQUSHIHUNHEC"}, "008868": {"suoxie": "MSJYJYZQ", "name": "\u6c11\u751f\u52a0\u94f6\u5609\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINJIAYIZHAIQUAN"}, "008869": {"suoxie": "DCHXHHA", "name": "\u5927\u6210\u6052\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGHENGXIANGHUNHEA"}, "008870": {"suoxie": "DCHXHHC", "name": "\u5927\u6210\u6052\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGHENGXIANGHUNHEC"}, "008871": {"suoxie": "DCRYLYCYGPA", "name": "\u5927\u6210\u777f\u88d5\u516d\u6708\u6301\u6709\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGRUIYULIUYUECHIYOUGUPIAOA"}, "008872": {"suoxie": "DCRYLYCYGPC", "name": "\u5927\u6210\u777f\u88d5\u516d\u6708\u6301\u6709\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGRUIYULIUYUECHIYOUGUPIAOC"}, "008873": {"suoxie": "GSABZCCZA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8bda\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNCHENGCHUNZHAIA"}, "008874": {"suoxie": "GSABZCCZC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u8bda\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNCHENGCHUNZHAIC"}, "008875": {"suoxie": "GSABZHLLZZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u6052\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNHENGLILVZHAIZHAIQUANA"}, "008876": {"suoxie": "GSABZHLLZZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u6052\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNHENGLILVZHAIZHAIQUANC"}, "008877": {"suoxie": "GLAZSYNDKZ", "name": "\u56fd\u8054\u5b89\u589e\u76db\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGSHENGYINIANDINGKAIZHAI"}, "008878": {"suoxie": "GLAXLCHLYNDKHH", "name": "\u56fd\u8054\u5b89\u65b0\u84dd\u7b79\u7ea2\u5229\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANXINLANCHOUHONGLIYINIANDINGKAIHUNHE"}, "008879": {"suoxie": "GLAZZ13NZJRZ", "name": "\u56fd\u8054\u5b89\u4e2d\u503a1-3\u5e74\u653f\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZHONGZHAI13NIANZHENGJINRONGZHAI"}, "008880": {"suoxie": "GLAZSCZA", "name": "\u56fd\u8054\u5b89\u589e\u987a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGSHUNCHUNZHAIA"}, "008881": {"suoxie": "GLAZSCZC", "name": "\u56fd\u8054\u5b89\u589e\u987a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGSHUNCHUNZHAIC"}, "008882": {"suoxie": "GLAZQCZA", "name": "\u56fd\u8054\u5b89\u589e\u797a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGQICHUNZHAIA"}, "008883": {"suoxie": "GLAZQCZC", "name": "\u56fd\u8054\u5b89\u589e\u797a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGQICHUNZHAIC"}, "008884": {"suoxie": "BYBRHHA", "name": "\u535a\u8fdc\u535a\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOYUANBORUIHUNHEA"}, "008885": {"suoxie": "BYBRHHC", "name": "\u535a\u8fdc\u535a\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOYUANBORUIHUNHEC"}, "008886": {"suoxie": "MSZYPZ6GYHHFOF", "name": "\u6c11\u751f\u5353\u8d8a\u914d\u7f6e6\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "MINSHENGZHUOYUEPEIZHI6GEYUEHUNHEFOF"}, "008887": {"suoxie": "HXGZBDTXPETFLJA", "name": "\u534e\u590f\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAGUOZHENGBANDAOTIXINPIANETFLIANJIEA"}, "008888": {"suoxie": "HXGZBDTXPETFLJC", "name": "\u534e\u590f\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAGUOZHENGBANDAOTIXINPIANETFLIANJIEC"}, "008889": {"suoxie": "YHZZ5GTXZTETFLJA", "name": "\u94f6\u534e\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENG5GTONGXINZHUTIETFLIANJIEA"}, "008890": {"suoxie": "ZYJZYXDKHH", "name": "\u4e2d\u90ae\u4ef7\u503c\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUJIAZHIYOUXUANDINGKAIHUNHE"}, "008891": {"suoxie": "AXJZCZHHA", "name": "\u5b89\u4fe1\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHICHENGZHANGHUNHEA"}, "008892": {"suoxie": "AXJZCZHHC", "name": "\u5b89\u4fe1\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHICHENGZHANGHUNHEC"}, "008893": {"suoxie": "CJHXXLHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINLIHUNHEA"}, "008894": {"suoxie": "CJHXXLHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINLIHUNHEC"}, "008895": {"suoxie": "SWLXLHDCCLHH", "name": "\u7533\u4e07\u83f1\u4fe1\u91cf\u5316\u5bf9\u51b2\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "SHENWANLINGXINLIANGHUADUICHONGCELUEHUNHE"}, "008896": {"suoxie": "XYDTYNDKZQFQS", "name": "\u5174\u4e1a\u9f0e\u6cf0\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEDINGTAIYINIANDINGKAIZHAIQUANFAQISHI"}, "008897": {"suoxie": "SYKZZJXZQ", "name": "\u4e0a\u94f6\u53ef\u8f6c\u503a\u7cbe\u9009\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "SHANGYINKEZHUANZHAIJINGXUANZHAIQUAN"}, "008898": {"suoxie": "GSCJX88ETFLJA", "name": "\u56fd\u5bff\u521b\u7cbe\u900988ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUCHUANGJINGXUAN88ETFLIANJIEA"}, "008899": {"suoxie": "GSCJX88ETFLJC", "name": "\u56fd\u5bff\u521b\u7cbe\u900988ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUCHUANGJINGXUAN88ETFLIANJIEC"}, "008900": {"suoxie": "GLAZTYNDKZFQS", "name": "\u56fd\u8054\u5b89\u589e\u6cf0\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANZENGTAIYINIANDINGKAIZHAIFAQISHI"}, "008901": {"suoxie": "FGNXZZHHA", "name": "\u5bcc\u56fd\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUONEIXUZENGZHANGHUNHEA"}, "008902": {"suoxie": "GSABTJCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u5409\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIJICHUNZHAIYINIANDINGKAIZHAI"}, "008903": {"suoxie": "GFKJXFHH", "name": "\u5e7f\u53d1\u79d1\u6280\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAKEJIXIANFENGHUNHE"}, "008904": {"suoxie": "HAATYNDKZ", "name": "\u534e\u5b89\u5b89\u817e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANANTENGYINIANDINGKAIZHAI"}, "008905": {"suoxie": "JHJPTLHHA", "name": "\u5609\u5408\u9526\u9e4f\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEJINPENGTIANLIHUNHEA"}, "008906": {"suoxie": "JHJPTLHHC", "name": "\u5609\u5408\u9526\u9e4f\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEJINPENGTIANLIHUNHEC"}, "008907": {"suoxie": "HTFZZGQYDYLETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGUOQIYIDAIYILUETFLIANJIEA"}, "008908": {"suoxie": "HTFZZGQYDYLETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGUOQIYIDAIYILUETFLIANJIEC"}, "008909": {"suoxie": "CJHXXYHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINXINYIHUNHEA"}, "008910": {"suoxie": "CJHXXYHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINXINYIHUNHEC"}, "008911": {"suoxie": "PAYFZDZZQA", "name": "\u5e73\u5b89\u5143\u4e30\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANFENGZHONGDUANZHAIZHAIQUANA"}, "008912": {"suoxie": "PAYFZDZZQC", "name": "\u5e73\u5b89\u5143\u4e30\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANFENGZHONGDUANZHAIZHAIQUANC"}, "008913": {"suoxie": "PAYFZDZZQE", "name": "\u5e73\u5b89\u5143\u4e30\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANFENGZHONGDUANZHAIZHAIQUANE"}, "008916": {"suoxie": "HXZZZJGZCXFZETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u6d59\u6c5f\u56fd\u8d44\u521b\u65b0\u53d1\u5c55ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHEJIANGGUOZICHUANGXINFAZHANETFLIANJIEA"}, "008917": {"suoxie": "HXZZZJGZCXFZETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u6d59\u6c5f\u56fd\u8d44\u521b\u65b0\u53d1\u5c55ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHEJIANGGUOZICHUANGXINFAZHANETFLIANJIEC"}, "008918": {"suoxie": "CXXRHHC", "name": "\u957f\u4fe1\u5148\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINXIANRUIHUNHEC"}, "008919": {"suoxie": "YYKJQDA", "name": "\u6c38\u8d62\u79d1\u6280\u9a71\u52a8A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGKEJIQUDONGA"}, "008920": {"suoxie": "YYKJQDC", "name": "\u6c38\u8d62\u79d1\u6280\u9a71\u52a8C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGKEJIQUDONGC"}, "008921": {"suoxie": "GTJXCZZQ", "name": "\u56fd\u6cf0\u805a\u946b\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUXINCHUNZHAIZHAIQUAN"}, "008922": {"suoxie": "CTHYZDZZQE", "name": "\u8d22\u901a\u9e3f\u8fd0\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGHONGYUNZHONGDUANZHAIZHAIQUANE"}, "008923": {"suoxie": "JXYLJKHYGPA", "name": "\u5efa\u4fe1\u533b\u7597\u5065\u5eb7\u884c\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINYILIAOJIANKANGHANGYEGUPIAOA"}, "008924": {"suoxie": "JXYLJKHYGPC", "name": "\u5efa\u4fe1\u533b\u7597\u5065\u5eb7\u884c\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINYILIAOJIANKANGHANGYEGUPIAOC"}, "008925": {"suoxie": "PHZDYNDKFQSZQ", "name": "\u9e4f\u534e\u5c0a\u8fbe\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNDAYINIANDINGKAIFAQISHIZHAIQUAN"}, "008926": {"suoxie": "TKHS300ETFLJA", "name": "\u6cf0\u5eb7\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGHUSHEN300ETFLIANJIEA"}, "008927": {"suoxie": "TKHS300ETFLJC", "name": "\u6cf0\u5eb7\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGHUSHEN300ETFLIANJIEC"}, "008928": {"suoxie": "TDXFHLZSA", "name": "\u6cf0\u8fbe\u6d88\u8d39\u7ea2\u5229\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAXIAOFEIHONGLIZHISHUA"}, "008929": {"suoxie": "TDXFHLZSC", "name": "\u6cf0\u8fbe\u6d88\u8d39\u7ea2\u5229\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAXIAOFEIHONGLIZHISHUC"}, "008930": {"suoxie": "ZJARPHYLSNFOF", "name": "\u4e2d\u52a0\u5b89\u745e\u5e73\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGJIAANRUIPINGHENGYANGLAOSANNIANFOF"}, "008933": {"suoxie": "THZZ13NGKZ", "name": "\u5929\u5f18\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGZHONGZHAI13NIANGUOKAIZHAI"}, "008934": {"suoxie": "DCKJXFGPA", "name": "\u5927\u6210\u79d1\u6280\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGKEJIXIAOFEIGUPIAOA"}, "008935": {"suoxie": "DCKJXFGPC", "name": "\u5927\u6210\u79d1\u6280\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGKEJIXIAOFEIGUPIAOC"}, "008936": {"suoxie": "ZYCYZZQC", "name": "\u4e2d\u94f6\u4ea7\u4e1a\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINCHANYEZHAIZHAIQUANC"}, "008937": {"suoxie": "NDAYCZ", "name": "\u8bfa\u5fb7\u5b89\u76c8\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUODEANYINGCHUNZHAI"}, "008938": {"suoxie": "DCHXYNDKZQ", "name": "\u5927\u6210\u60e0\u5174\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIXINGYINIANDINGKAIZHAIQUAN"}, "008939": {"suoxie": "HTZJYYG3YGDZA", "name": "\u534e\u6cf0\u7d2b\u91d1\u6708\u6708\u8d2d3\u6708\u6eda\u52a8\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINYUEYUEGOU3YUEGUNDONGZHAIA"}, "008940": {"suoxie": "HTZJYYG3YGDZC", "name": "\u534e\u6cf0\u7d2b\u91d1\u6708\u6708\u8d2d3\u6708\u6eda\u52a8\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINYUEYUEGOU3YUEGUNDONGZHAIC"}, "008941": {"suoxie": "HTZJZZG3YGDZA", "name": "\u534e\u6cf0\u7d2b\u91d1\u5468\u5468\u8d2d3\u6708\u6eda\u52a8\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINZHOUZHOUGOU3YUEGUNDONGZHAIA"}, "008942": {"suoxie": "HTZJZZG3YGDZC", "name": "\u534e\u6cf0\u7d2b\u91d1\u5468\u5468\u8d2d3\u6708\u6eda\u52a8\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINZHOUZHOUGOU3YUEGUNDONGZHAIC"}, "008944": {"suoxie": "STMGMSCIZGAGETFLJA", "name": "\u4e0a\u6295\u6469\u6839MSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUMOGENMSCIZHONGGUOAGUETFLIANJIEA"}, "008945": {"suoxie": "STMGMSCIZGAGETFLJC", "name": "\u4e0a\u6295\u6469\u6839MSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUMOGENMSCIZHONGGUOAGUETFLIANJIEC"}, "008947": {"suoxie": "HXDYZQA", "name": "\u534e\u590f\u9f0e\u6e90\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGYUANZHAIQUANA"}, "008948": {"suoxie": "HXDYZQC", "name": "\u534e\u590f\u9f0e\u6e90\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGYUANZHAIQUANC"}, "008949": {"suoxie": "PAJXYXHHA", "name": "\u5e73\u5b89\u5320\u5fc3\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIANGXINYOUXUANHUNHEA"}, "008950": {"suoxie": "PAJXYXHHC", "name": "\u5e73\u5b89\u5320\u5fc3\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIANGXINYOUXUANHUNHEC"}, "008951": {"suoxie": "PHZYYNDKZ", "name": "\u9e4f\u534e\u5c0a\u88d5\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNYUYINIANDINGKAIZHAI"}, "008952": {"suoxie": "ZXJTGQZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u6842\u4f01\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUGUIQIZHAIA"}, "008953": {"suoxie": "ZXJTGQZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u6842\u4f01\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUGUIQIZHAIC"}, "008954": {"suoxie": "AXJZHBSNCYHHA", "name": "\u5b89\u4fe1\u4ef7\u503c\u56de\u62a5\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIHUIBAOSANNIANCHIYOUHUNHEA"}, "008955": {"suoxie": "JYCXLHHH", "name": "\u4ea4\u94f6\u521b\u65b0\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHUANGXINLINGHANGHUNHE"}, "008956": {"suoxie": "PHZZ35NGKZZSA", "name": "\u9e4f\u534e\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI35NIANGUOKAIZHAIZHISHUA"}, "008957": {"suoxie": "PHZZ35NGKZZSC", "name": "\u9e4f\u534e\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI35NIANGUOKAIZHAIZHISHUC"}, "008958": {"suoxie": "JSHBJXGP", "name": "\u5609\u5b9e\u56de\u62a5\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIHUIBAOJINGXUANGUPIAO"}, "008959": {"suoxie": "CJHXHLCDZZQE", "name": "\u521b\u91d1\u5408\u4fe1\u6052\u5229\u8d85\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINHENGLICHAODUANZHAIZHAIQUANE"}, "008960": {"suoxie": "CXGFJGLHHHC", "name": "\u957f\u4fe1\u56fd\u9632\u519b\u5de5\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINGUOFANGJUNGONGLIANGHUAHUNHEC"}, "008961": {"suoxie": "HSKJCXHH", "name": "\u534e\u5546\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGKEJICHUANGXINHUNHE"}, "008962": {"suoxie": "JXKJCXHHA", "name": "\u5efa\u4fe1\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINKEJICHUANGXINHUNHEA"}, "008963": {"suoxie": "JXKJCXHHC", "name": "\u5efa\u4fe1\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINKEJICHUANGXINHUNHEC"}, "008964": {"suoxie": "HTZJZZ15NGKZZSA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHONGZHAI15NIANGUOKAIZHAIZHISHUA"}, "008965": {"suoxie": "HTZJZZ15NGKZZSC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHONGZHAI15NIANGUOKAIZHAIZHISHUC"}, "008966": {"suoxie": "BSCZYXLNFBHHA", "name": "\u535a\u65f6\u6210\u957f\u4f18\u9009\u4e24\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHENGZHANGYOUXUANLIANGNIANFENGBIHUNHEA"}, "008967": {"suoxie": "BSCZYXLNFBHHC", "name": "\u535a\u65f6\u6210\u957f\u4f18\u9009\u4e24\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHENGZHANGYOUXUANLIANGNIANFENGBIHUNHEC"}, "008968": {"suoxie": "SWLXATDLYNDKZ", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u9f0e\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIDINGLIYINIANDINGKAIZHAI"}, "008969": {"suoxie": "RYJHJZSNCYHHA", "name": "\u777f\u8fdc\u5747\u8861\u4ef7\u503c\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIYUANJUNHENGJIAZHISANNIANCHIYOUHUNHEA"}, "008970": {"suoxie": "RYJHJZSNCYHHC", "name": "\u777f\u8fdc\u5747\u8861\u4ef7\u503c\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIYUANJUNHENGJIAZHISANNIANCHIYOUHUNHEC"}, "008972": {"suoxie": "DCHSZHZXXGZSC", "name": "\u5927\u6210\u6052\u751f\u7efc\u5408\u4e2d\u5c0f\u578b\u80a1\u6307\u6570C", "style": "QDII", "pinyin": "DACHENGHENGSHENGZONGHEZHONGXIAOXINGGUZHISHUC"}, "008973": {"suoxie": "DCZHHSG300ZSLOFC", "name": "\u5927\u6210\u4e2d\u534e\u6caa\u6df1\u6e2f300\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGHUAHUSHENGANG300ZHISHULOFC"}, "008974": {"suoxie": "CCWJZLZQC", "name": "\u957f\u57ce\u7a33\u5065\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGWENJIANZENGLIZHAIQUANC"}, "008975": {"suoxie": "FGZZXF50ETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6d88\u8d3950ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIAOFEI50ETFLIANJIEA"}, "008976": {"suoxie": "FGZZXF50ETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6d88\u8d3950ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIAOFEI50ETFLIANJIEC"}, "008978": {"suoxie": "YHCFHHFQS", "name": "\u94f6\u534e\u957f\u4e30\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUACHANGFENGHUNHEFAQISHI"}, "008979": {"suoxie": "WJMFHBYNCYHH", "name": "\u4e07\u5bb6\u6c11\u4e30\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAMINFENGHUIBAOYINIANCHIYOUHUNHE"}, "008980": {"suoxie": "ZYKJCXJXHHA", "name": "\u4e2d\u90ae\u79d1\u6280\u521b\u65b0\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUKEJICHUANGXINJINGXUANHUNHEA"}, "008981": {"suoxie": "ZYKJCXJXHHC", "name": "\u4e2d\u90ae\u79d1\u6280\u521b\u65b0\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUKEJICHUANGXINJINGXUANHUNHEC"}, "008982": {"suoxie": "HTZJZX3YDKZ", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u946b3\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINZHIXIN3YUEDINGKAIZHAI"}, "008983": {"suoxie": "CTKJCXHHA", "name": "\u8d22\u901a\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGKEJICHUANGXINHUNHEA"}, "008984": {"suoxie": "CTKJCXHHC", "name": "\u8d22\u901a\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGKEJICHUANGXINHUNHEC"}, "008985": {"suoxie": "DFHQDSNCYHH", "name": "\u4e1c\u65b9\u7ea2\u542f\u4e1c\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIDONGSANNIANCHIYOUHUNHE"}, "008986": {"suoxie": "GFSHJETFLJA", "name": "\u5e7f\u53d1\u4e0a\u6d77\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUANGFASHANGHAIJINETFLIANJIEA"}, "008987": {"suoxie": "GFSHJETFLJC", "name": "\u5e7f\u53d1\u4e0a\u6d77\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUANGFASHANGHAIJINETFLIANJIEC"}, "008988": {"suoxie": "DCKJCXHHA", "name": "\u5927\u6210\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGKEJICHUANGXINHUNHEA"}, "008989": {"suoxie": "DCKJCXHHC", "name": "\u5927\u6210\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGKEJICHUANGXINHUNHEC"}, "008990": {"suoxie": "DFHJXZXYNCYHH", "name": "\u4e1c\u65b9\u7ea2\u5320\u5fc3\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJIANGXINZHENXUANYINIANCHIYOUHUNHE"}, "008991": {"suoxie": "SWLXAXHXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u6167\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINANXINHUIXUANHUNHEA"}, "008992": {"suoxie": "SWLXAXHXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u6167\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINANXINHUIXUANHUNHEC"}, "008993": {"suoxie": "HTFDCLCZA", "name": "\u6c47\u6dfb\u5bcc\u591a\u7b56\u7565\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUDUOCELUECHUNZHAIA"}, "008994": {"suoxie": "HTFDCLCZC", "name": "\u6c47\u6dfb\u5bcc\u591a\u7b56\u7565\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUDUOCELUECHUNZHAIC"}, "008995": {"suoxie": "ZYZQAPZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6c9b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANPEIZHAIQUANA"}, "008996": {"suoxie": "ZYZQAPZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6c9b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANPEIZHAIQUANC"}, "008997": {"suoxie": "TTJZYSHHA", "name": "\u540c\u6cf0\u7ade\u4e89\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIJINGZHENGYOUSHIHUNHEA"}, "008998": {"suoxie": "TTJZYSHHC", "name": "\u540c\u6cf0\u7ade\u4e89\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIJINGZHENGYOUSHIHUNHEC"}, "008999": {"suoxie": "JSJYJL6GYCYQZQA", "name": "\u666f\u987a\u666f\u9890\u5609\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNJINGYIJIALI6GEYUECHIYOUQIZHAIQUANA"}, "009000": {"suoxie": "JSJYJL6GYCYQZQC", "name": "\u666f\u987a\u666f\u9890\u5609\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNJINGYIJIALI6GEYUECHIYOUQIZHAIQUANC"}, "009001": {"suoxie": "CCTLCZA", "name": "\u957f\u57ce\u6cf0\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGTAILICHUNZHAIA"}, "009002": {"suoxie": "CCTLCZC", "name": "\u957f\u57ce\u6cf0\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGTAILICHUNZHAIC"}, "009003": {"suoxie": "ZYAKPHYLSNFOF", "name": "\u4e2d\u94f6\u5b89\u5eb7\u5e73\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGYINANKANGPINGHENGYANGLAOSANNIANFOF"}, "009004": {"suoxie": "HFTZZ500ZQC", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENG500ZENGQIANGC"}, "009005": {"suoxie": "CJHXXQHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u797a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINQIHUNHEA"}, "009006": {"suoxie": "CJHXXQHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u797a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINQIHUNHEC"}, "009007": {"suoxie": "XQHGSLNCYHH", "name": "\u5174\u5168\u6caa\u6e2f\u6df1\u4e24\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHUGANGSHENLIANGNIANCHIYOUHUNHE"}, "009008": {"suoxie": "PAKJCXHHA", "name": "\u5e73\u5b89\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANKEJICHUANGXINHUNHEA"}, "009009": {"suoxie": "PAKJCXHHC", "name": "\u5e73\u5b89\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANKEJICHUANGXINHUNHEC"}, "009010": {"suoxie": "HXXYYNCYHH", "name": "\u534e\u590f\u5174\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINGYANGYINIANCHIYOUHUNHE"}, "009011": {"suoxie": "HXRYYNCYHH", "name": "\u534e\u590f\u777f\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIARUIYANGYINIANCHIYOUHUNHE"}, "009012": {"suoxie": "PACYBETFLJA", "name": "\u5e73\u5b89\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANCHUANGYEBANETFLIANJIEA"}, "009013": {"suoxie": "PACYBETFLJC", "name": "\u5e73\u5b89\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANCHUANGYEBANETFLIANJIEC"}, "009014": {"suoxie": "HDRZHH", "name": "\u6cd3\u5fb7\u777f\u6cfd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDERUIZEHUNHE"}, "009015": {"suoxie": "HDRXYNCYQHHA", "name": "\u6cd3\u5fb7\u777f\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDERUIXIANGYINIANCHIYOUQIHUNHEA"}, "009016": {"suoxie": "HDRXYNCYQHHC", "name": "\u6cd3\u5fb7\u777f\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDERUIXIANGYINIANCHIYOUQIHUNHEC"}, "009017": {"suoxie": "YHGGTJXGPFQSA", "name": "\u94f6\u534e\u6e2f\u80a1\u901a\u7cbe\u9009\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAGANGGUTONGJINGXUANGUPIAOFAQISHIA"}, "009018": {"suoxie": "XBLDJT18GYDKZA", "name": "\u897f\u90e8\u5229\u5f97\u805a\u6cf018\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJUTAI18GEYUEDINGKAIZHAIA"}, "009019": {"suoxie": "XBLDJT18GYDKZC", "name": "\u897f\u90e8\u5229\u5f97\u805a\u6cf018\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEJUTAI18GEYUEDINGKAIZHAIC"}, "009021": {"suoxie": "PHFCZQA", "name": "\u9e4f\u534e\u4e30\u8bda\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGCHENGZHAIQUANA"}, "009022": {"suoxie": "PHFCZQC", "name": "\u9e4f\u534e\u4e30\u8bda\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGCHENGZHAIQUANC"}, "009023": {"suoxie": "PHWJHBHH", "name": "\u9e4f\u534e\u7a33\u5065\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAWENJIANHUIBAOHUNHE"}, "009024": {"suoxie": "HFTKJCXHHC", "name": "\u6d77\u5bcc\u901a\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGKEJICHUANGXINHUNHEC"}, "009025": {"suoxie": "HFTKJCXHHA", "name": "\u6d77\u5bcc\u901a\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGKEJICHUANGXINHUNHEA"}, "009026": {"suoxie": "ZYGZLFZJYHH", "name": "\u4e2d\u94f6\u9ad8\u8d28\u91cf\u53d1\u5c55\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINGAOZHILIANGFAZHANJIYUHUNHE"}, "009027": {"suoxie": "PYASAYHBYNHHA", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u8fdc\u56de\u62a5\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANYUANHUIBAOYINIANHUNHEA"}, "009028": {"suoxie": "PYASAYHBYNHHC", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u8fdc\u56de\u62a5\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANYUANHUIBAOYINIANHUNHEC"}, "009029": {"suoxie": "GYGZLCZHHA", "name": "\u5de5\u94f6\u9ad8\u8d28\u91cf\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINGAOZHILIANGCHENGZHANGHUNHEA"}, "009030": {"suoxie": "GYGZLCZHHC", "name": "\u5de5\u94f6\u9ad8\u8d28\u91cf\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINGAOZHILIANGCHENGZHANGHUNHEC"}, "009031": {"suoxie": "GYJHYNDKHHA", "name": "\u5de5\u94f6\u805a\u548c\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUHEYINIANDINGKAIHUNHEA"}, "009032": {"suoxie": "GYJHYNDKHHC", "name": "\u5de5\u94f6\u805a\u548c\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUHEYINIANDINGKAIHUNHEC"}, "009033": {"suoxie": "JXSHJETFLJA", "name": "\u5efa\u4fe1\u4e0a\u6d77\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINSHANGHAIJINETFLIANJIEA"}, "009034": {"suoxie": "JXSHJETFLJC", "name": "\u5efa\u4fe1\u4e0a\u6d77\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINSHANGHAIJINETFLIANJIEC"}, "009035": {"suoxie": "PYASZZ13NGKZA", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGZHONGZHAI13NIANGUOKAIZHAIA"}, "009036": {"suoxie": "PYASZZ13NGKZC", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGZHONGZHAI13NIANGUOKAIZHAIC"}, "009037": {"suoxie": "PYASPQCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u5e86\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUQINGCHUNZHAIZHAIQUANA"}, "009038": {"suoxie": "PYASPQCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u5e86\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUQINGCHUNZHAIZHAIQUANC"}, "009041": {"suoxie": "PYASPTCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u5929\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUTIANCHUNZHAIZHAIQUANA"}, "009042": {"suoxie": "PYASPTCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u5929\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUTIANCHUNZHAIZHAIQUANC"}, "009043": {"suoxie": "JTJXLHGP", "name": "\u4e5d\u6cf0\u4e45\u4fe1\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUXINLIANGHUAGUPIAO"}, "009044": {"suoxie": "PYASSYYNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u6bc5\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGYIYINIANDINGKAIZHAIQUAN"}, "009045": {"suoxie": "PYASSZYNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u667a\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGZHIYINIANDINGKAIZHAIQUAN"}, "009046": {"suoxie": "DCCYBA", "name": "\u4e1c\u8d22\u521b\u4e1a\u677fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAICHUANGYEBANA"}, "009047": {"suoxie": "DCCYBC", "name": "\u4e1c\u8d22\u521b\u4e1a\u677fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAICHUANGYEBANC"}, "009048": {"suoxie": "PYASKCSNFBHH", "name": "\u6d66\u94f6\u5b89\u76db\u79d1\u521b\u4e09\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGKECHUANGSANNIANFENGBIHUNHE"}, "009049": {"suoxie": "YFDGDZZHH", "name": "\u6613\u65b9\u8fbe\u9ad8\u7aef\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGAODUANZHIZAOHUNHE"}, "009050": {"suoxie": "YFDHYYNDKZ", "name": "\u6613\u65b9\u8fbe\u6052\u88d5\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGYUYINIANDINGKAIZHAI"}, "009051": {"suoxie": "YFDZZHLETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u7ea2\u5229ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGHONGLIETFLIANJIEA"}, "009052": {"suoxie": "YFDZZHLETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u7ea2\u5229ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGHONGLIETFLIANJIEC"}, "009053": {"suoxie": "PAHQDKZ", "name": "\u5e73\u5b89\u5408\u5e86\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEQINGDINGKAIZHAI"}, "009054": {"suoxie": "YXYFFTHH", "name": "\u5706\u4fe1\u6c38\u4e30\u6ca3\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YUANXINYONGFENGFENGTAIHUNHE"}, "009055": {"suoxie": "YXYFDWQA", "name": "\u5706\u4fe1\u6c38\u4e30\u5927\u6e7e\u533aA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGDAWANQUA"}, "009056": {"suoxie": "YXYFDWQC", "name": "\u5706\u4fe1\u6c38\u4e30\u5927\u6e7e\u533aC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGDAWANQUC"}, "009057": {"suoxie": "BSKJCXHHA", "name": "\u535a\u65f6\u79d1\u6280\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIKEJICHUANGXINHUNHEA"}, "009058": {"suoxie": "BSKJCXHHC", "name": "\u535a\u65f6\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIKEJICHUANGXINHUNHEC"}, "009059": {"suoxie": "NFHS300ZQA", "name": "\u5357\u65b9\u6caa\u6df1300\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300ZENGQIANGA"}, "009060": {"suoxie": "NFHS300ZQC", "name": "\u5357\u65b9\u6caa\u6df1300\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300ZENGQIANGC"}, "009062": {"suoxie": "CTZHCZHHA", "name": "\u8d22\u901a\u667a\u6167\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZHIHUICHENGZHANGHUNHEA"}, "009063": {"suoxie": "CTZHCZHHC", "name": "\u8d22\u901a\u667a\u6167\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZHIHUICHENGZHANGHUNHEC"}, "009064": {"suoxie": "PYJWLGYCYQHHA", "name": "\u9e4f\u626c\u666f\u6c83\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGWOLIUGEYUECHIYOUQIHUNHEA"}, "009065": {"suoxie": "PYJWLGYCYQHHC", "name": "\u9e4f\u626c\u666f\u6c83\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGWOLIUGEYUECHIYOUQIHUNHEC"}, "009067": {"suoxie": "GTZZXNYQCETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINNENGYUANQICHEETFLIANJIEA"}, "009068": {"suoxie": "GTZZXNYQCETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINNENGYUANQICHEETFLIANJIEC"}, "009069": {"suoxie": "DCRXGPA", "name": "\u5927\u6210\u777f\u946b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGRUIXINGUPIAOA"}, "009070": {"suoxie": "DCRXGPC", "name": "\u5927\u6210\u777f\u946b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGRUIXINGUPIAOC"}, "009071": {"suoxie": "DBAXHHA", "name": "\u5fb7\u90a6\u5b89\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANXINHUNHEA"}, "009072": {"suoxie": "DBAXHHC", "name": "\u5fb7\u90a6\u5b89\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANXINHUNHEC"}, "009073": {"suoxie": "DBHLHHA", "name": "\u5fb7\u90a6\u60e0\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGHUILIHUNHEA"}, "009074": {"suoxie": "DBHLHHC", "name": "\u5fb7\u90a6\u60e0\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGHUILIHUNHEC"}, "009076": {"suoxie": "GYYXHH", "name": "\u5de5\u94f6\u5706\u5174\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYUANXINGHUNHE"}, "009077": {"suoxie": "HTCXWJHHA", "name": "\u7ea2\u571f\u521b\u65b0\u7a33\u8fdb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTUCHUANGXINWENJINHUNHEA"}, "009078": {"suoxie": "HTCXWJHHC", "name": "\u7ea2\u571f\u521b\u65b0\u7a33\u8fdb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTUCHUANGXINWENJINHUNHEC"}, "009079": {"suoxie": "NFYGADWQETFLJA", "name": "\u5357\u65b9\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYUEGANGAODAWANQUETFLIANJIEA"}, "009080": {"suoxie": "NFYGADWQETFLJC", "name": "\u5357\u65b9\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYUEGANGAODAWANQUETFLIANJIEC"}, "009081": {"suoxie": "ZXBCJFYNDKZFQS", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5609\u4e30\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJIAFENGYINIANDINGKAIZHAIFAQISHI"}, "009082": {"suoxie": "HXDJZQA", "name": "\u534e\u590f\u9f0e\u4f73\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGJIAZHAIQUANA"}, "009083": {"suoxie": "HXDJZQC", "name": "\u534e\u590f\u9f0e\u4f73\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGJIAZHAIQUANC"}, "009084": {"suoxie": "SWLXATXLCZYNDKZ", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u946b\u5229\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIXINLICHUNZHAIYINIANDINGKAIZHAI"}, "009085": {"suoxie": "YHFXYNCYQHH", "name": "\u94f6\u534e\u4e30\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFENGXIANGYINIANCHIYOUQIHUNHE"}, "009086": {"suoxie": "PHJZGYLNCYQHH", "name": "\u9e4f\u534e\u4ef7\u503c\u5171\u8d62\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHIGONGYINGLIANGNIANCHIYOUQIHUNHE"}, "009087": {"suoxie": "TPZZ13NZCXJRZA", "name": "\u592a\u5e73\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIA"}, "009088": {"suoxie": "TPZZ13NZCXJRZC", "name": "\u592a\u5e73\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIC"}, "009089": {"suoxie": "JSWGSYZQA", "name": "\u5609\u5b9e\u7a33\u56fa\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENGUSHOUYIZHAIQUANA"}, "009091": {"suoxie": "XYHYYNDKZFQS", "name": "\u5174\u94f6\u6c47\u60a6\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIYUEYINIANDINGKAIZHAIFAQISHI"}, "009092": {"suoxie": "FGXCLXNYHHA", "name": "\u5bcc\u56fd\u65b0\u6750\u6599\u65b0\u80fd\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINCAILIAOXINNENGYUANHUNHEA"}, "009093": {"suoxie": "HTBRHLZDZA", "name": "\u534e\u6cf0\u67cf\u745e\u9e3f\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAIRUIHONGLIZHONGDUANZHAIA"}, "009094": {"suoxie": "HTBRHLZDZC", "name": "\u534e\u6cf0\u67cf\u745e\u9e3f\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAIRUIHONGLIZHONGDUANZHAIC"}, "009095": {"suoxie": "HTBRHLZDZE", "name": "\u534e\u6cf0\u67cf\u745e\u9e3f\u5229\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAIRUIHONGLIZHONGDUANZHAIE"}, "009096": {"suoxie": "PHAZHHA", "name": "\u9e4f\u534e\u5b89\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANZEHUNHEA"}, "009097": {"suoxie": "PHAZHHC", "name": "\u9e4f\u534e\u5b89\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANZEHUNHEC"}, "009098": {"suoxie": "JSCCJZLHHH", "name": "\u666f\u987a\u957f\u57ce\u4ef7\u503c\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJIAZHILINGHANGHUNHE"}, "009099": {"suoxie": "XHYNBHBE", "name": "\u65b0\u534e\u58f9\u8bfa\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "XINHUAYINUOBAOHUOBIE"}, "009100": {"suoxie": "AXWJZLHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANZENGLIHUNHEA"}, "009101": {"suoxie": "AXWJZLHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANZENGLIHUNHEC"}, "009102": {"suoxie": "PYHLYXHHA", "name": "\u9e4f\u626c\u7ea2\u5229\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGHONGLIYOUXUANHUNHEA"}, "009103": {"suoxie": "PYHLYXHHC", "name": "\u9e4f\u626c\u7ea2\u5229\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGHONGLIYOUXUANHUNHEC"}, "009104": {"suoxie": "XHCZTLZQFQB", "name": "\u65b0\u534e\u7eaf\u503a\u6dfb\u5229\u503a\u5238\u53d1\u8d77B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUACHUNZHAITIANLIZHAIQUANFAQIB"}, "009105": {"suoxie": "XYJRYNDKZQ", "name": "\u5174\u4e1a\u5609\u8363\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIARONGYINIANDINGKAIZHAIQUAN"}, "009106": {"suoxie": "JHTSZXGPA", "name": "\u5609\u5408\u540c\u987a\u667a\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIAHETONGSHUNZHIXUANGUPIAOA"}, "009107": {"suoxie": "JHTSZXGPC", "name": "\u5609\u5408\u540c\u987a\u667a\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIAHETONGSHUNZHIXUANGUPIAOC"}, "009108": {"suoxie": "FGHLJXHHQDIIRMB", "name": "\u5bcc\u56fd\u7ea2\u5229\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "FUGUOHONGLIJINGXUANHUNHEQDIIRENMINBI"}, "009109": {"suoxie": "BYZYCZZQA", "name": "\u535a\u8fdc\u589e\u76ca\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOYUANZENGYICHUNZHAIZHAIQUANA"}, "009110": {"suoxie": "BYZYCZZQC", "name": "\u535a\u8fdc\u589e\u76ca\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOYUANZENGYICHUNZHAIZHAIQUANC"}, "009111": {"suoxie": "BYSZZLHHA", "name": "\u535a\u8fdc\u53cc\u503a\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOYUANSHUANGZHAIZENGLIHUNHEA"}, "009112": {"suoxie": "BYSZZLHHC", "name": "\u535a\u8fdc\u53cc\u503a\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOYUANSHUANGZHAIZENGLIHUNHEC"}, "009113": {"suoxie": "ZSHJZYYX3GYFOF", "name": "\u6d59\u5546\u6c47\u91d1\u5353\u8d8a\u4f18\u90093\u4e2a\u6708(FOF)", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGHUIJINZHUOYUEYOUXUAN3GEYUEFOF"}, "009114": {"suoxie": "PYJHHBLHPZHHA", "name": "\u9e4f\u626c\u666f\u6cd3\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGJINGHONGHUIBAOLINGHUOPEIZHIHUNHEA"}, "009115": {"suoxie": "PYJHHBLHPZHHC", "name": "\u9e4f\u626c\u666f\u6cd3\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGYANGJINGHONGHUIBAOLINGHUOPEIZHIHUNHEC"}, "009116": {"suoxie": "DXZZXF50A", "name": "\u4e1c\u5174\u4e2d\u8bc1\u6d88\u8d3950A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGXINGZHONGZHENGXIAOFEI50A"}, "009117": {"suoxie": "DXZZXF50C", "name": "\u4e1c\u5174\u4e2d\u8bc1\u6d88\u8d3950C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGXINGZHONGZHENGXIAOFEI50C"}, "009118": {"suoxie": "TPHRCZZQ", "name": "\u592a\u5e73\u6052\u777f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGRUICHUNZHAIZHAIQUAN"}, "009119": {"suoxie": "GFPZHBHHA", "name": "\u5e7f\u53d1\u54c1\u8d28\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAPINZHIHUIBAOHUNHEA"}, "009120": {"suoxie": "GFPZHBHHC", "name": "\u5e7f\u53d1\u54c1\u8d28\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAPINZHIHUIBAOHUNHEC"}, "009121": {"suoxie": "GFZXHHA", "name": "\u5e7f\u53d1\u62db\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAZHAOXIANGHUNHEA"}, "009124": {"suoxie": "HTBXKRHHA", "name": "\u534e\u6cf0\u4fdd\u5174\u79d1\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAOXINGKERONGHUNHEA"}, "009125": {"suoxie": "HTBXKRHHC", "name": "\u534e\u6cf0\u4fdd\u5174\u79d1\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAOXINGKERONGHUNHEC"}, "009126": {"suoxie": "JSJCCYYXGPA", "name": "\u5609\u5b9e\u57fa\u7840\u4ea7\u4e1a\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIJICHUCHANYEYOUXUANGUPIAOA"}, "009127": {"suoxie": "JSJCCYYXGPC", "name": "\u5609\u5b9e\u57fa\u7840\u4ea7\u4e1a\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIJICHUCHANYEYOUXUANGUPIAOC"}, "009128": {"suoxie": "MYJZCQHHA", "name": "\u660e\u4e9a\u4ef7\u503c\u957f\u9752\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINGYAJIAZHICHANGQINGHUNHEA"}, "009129": {"suoxie": "MYJZCQHHC", "name": "\u660e\u4e9a\u4ef7\u503c\u957f\u9752\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINGYAJIAZHICHANGQINGHUNHEC"}, "009130": {"suoxie": "PYJHLGYCYHHA", "name": "\u9e4f\u626c\u666f\u6052\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGHENGLIUGEYUECHIYOUHUNHEA"}, "009131": {"suoxie": "PYJHLGYCYHHC", "name": "\u9e4f\u626c\u666f\u6052\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGHENGLIUGEYUECHIYOUHUNHEC"}, "009132": {"suoxie": "GFXPCZHHLOFC", "name": "\u5e7f\u53d1\u5c0f\u76d8\u6210\u957f\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOPANCHENGZHANGHUNHELOFC"}, "009133": {"suoxie": "HAJLHHA", "name": "\u6c47\u5b89\u5609\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANJIALIHUNHEA"}, "009134": {"suoxie": "HAJLHHC", "name": "\u6c47\u5b89\u5609\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANJIALIHUNHEC"}, "009135": {"suoxie": "GFHLYNCYHHA", "name": "\u5e7f\u53d1\u6052\u9686\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGLONGYINIANCHIYOUHUNHEA"}, "009136": {"suoxie": "GFHLYNCYHHC", "name": "\u5e7f\u53d1\u6052\u9686\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGLONGYINIANCHIYOUHUNHEC"}, "009137": {"suoxie": "JSRHLNCYQHH", "name": "\u5609\u5b9e\u745e\u548c\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIRUIHELIANGNIANCHIYOUQIHUNHE"}, "009138": {"suoxie": "JSRCLNCYQHHA", "name": "\u5609\u5b9e\u745e\u6210\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIRUICHENGLIANGNIANCHIYOUQIHUNHEA"}, "009139": {"suoxie": "JSRCLNCYQHHC", "name": "\u5609\u5b9e\u745e\u6210\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIRUICHENGLIANGNIANCHIYOUQIHUNHEC"}, "009140": {"suoxie": "YYJZLJXHH", "name": "\u6c38\u8d62\u7ade\u4e89\u529b\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGJINGZHENGLIJINGXUANHUNHE"}, "009141": {"suoxie": "TDJZCQHHA", "name": "\u6cf0\u8fbe\u4ef7\u503c\u957f\u9752\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAJIAZHICHANGQINGHUNHEA"}, "009142": {"suoxie": "TDJZCQHHC", "name": "\u6cf0\u8fbe\u4ef7\u503c\u957f\u9752\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAJIAZHICHANGQINGHUNHEC"}, "009143": {"suoxie": "STMGYLYNHHFOF", "name": "\u4e0a\u6295\u6469\u6839\u517b\u8001\u4e00\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGTOUMOGENYANGLAOYINIANHUNHEFOF"}, "009144": {"suoxie": "BSRSWJTLHHA", "name": "\u535a\u65f6\u8363\u5347\u7a33\u5065\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIRONGSHENGWENJIANTIANLIHUNHEA"}, "009145": {"suoxie": "BSRSWJTLHHC", "name": "\u535a\u65f6\u8363\u5347\u7a33\u5065\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIRONGSHENGWENJIANTIANLIHUNHEC"}, "009146": {"suoxie": "XHJXCZ3GYHHFOF", "name": "\u65b0\u534e\u7cbe\u9009\u6210\u957f3\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINHUAJINGXUANCHENGZHANG3GEYUEHUNHEFOF"}, "009147": {"suoxie": "JXXNYHYGPA", "name": "\u5efa\u4fe1\u65b0\u80fd\u6e90\u884c\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINXINNENGYUANHANGYEGUPIAOA"}, "009148": {"suoxie": "PAHJDKZ", "name": "\u5e73\u5b89\u5408\u805a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEJUDINGKAIZHAI"}, "009149": {"suoxie": "FGJDSYDCLHHC", "name": "\u5bcc\u56fd\u7edd\u5bf9\u6536\u76ca\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "FUGUOJUEDUISHOUYIDUOCELUEHUNHEC"}, "009151": {"suoxie": "GSCLYX3GYCYHHFOF", "name": "\u56fd\u5bff\u7b56\u7565\u4f18\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUCELUEYOUXUAN3GEYUECHIYOUHUNHEFOF"}, "009152": {"suoxie": "NFRSSNHHA", "name": "\u5357\u65b9\u745e\u76db\u4e09\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGRUISHENGSANNIANHUNHEA"}, "009153": {"suoxie": "NFRSSNHHC", "name": "\u5357\u65b9\u745e\u76db\u4e09\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGRUISHENGSANNIANHUNHEC"}, "009154": {"suoxie": "HFTFYHHA", "name": "\u6d77\u5bcc\u901a\u5bcc\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFUYINGHUNHEA"}, "009155": {"suoxie": "HFTFYHHC", "name": "\u6d77\u5bcc\u901a\u5bcc\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFUYINGHUNHEC"}, "009156": {"suoxie": "HFTFZHHA", "name": "\u6d77\u5bcc\u901a\u5bcc\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFUZEHUNHEA"}, "009157": {"suoxie": "HFTFZHHC", "name": "\u6d77\u5bcc\u901a\u5bcc\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFUZEHUNHEC"}, "009159": {"suoxie": "QHLHZX3GYCYHHFOFA", "name": "\u524d\u6d77\u8054\u5408\u667a\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAILIANHEZHIXUAN3GEYUECHIYOUHUNHEFOFA"}, "009160": {"suoxie": "QHLHZX3GYCYHHFOFC", "name": "\u524d\u6d77\u8054\u5408\u667a\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAILIANHEZHIXUAN3GEYUECHIYOUHUNHEFOFC"}, "009161": {"suoxie": "STJCYLWNHHFOF", "name": "\u4e0a\u6295\u9526\u7a0b\u517b\u8001\u4e94\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUJINCHENGYANGLAOWUNIANHUNHEFOF"}, "009162": {"suoxie": "FGYYCZ30GP", "name": "\u5bcc\u56fd\u533b\u836f\u6210\u957f30\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOYIYAOCHENGZHANG30GUPIAO"}, "009163": {"suoxie": "GFYLBJGPC", "name": "\u5e7f\u53d1\u533b\u7597\u4fdd\u5065\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYILIAOBAOJIANGUPIAOC"}, "009164": {"suoxie": "ZJJQLGYDKHHA", "name": "\u4e2d\u52a0\u805a\u5e86\u516d\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJUQINGLIUGEYUEDINGKAIHUNHEA"}, "009165": {"suoxie": "ZJJQLGYDKHHC", "name": "\u4e2d\u52a0\u805a\u5e86\u516d\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJUQINGLIUGEYUEDINGKAIHUNHEC"}, "009166": {"suoxie": "PAHX1NDKZ", "name": "\u5e73\u5b89\u5408\u4eab1\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEXIANG1NIANDINGKAIZHAI"}, "009167": {"suoxie": "BSFCYNDKZFQS", "name": "\u535a\u65f6\u5bcc\u707f\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUCANYINIANDINGKAIZHAIFAQISHI"}, "009168": {"suoxie": "BSFXCZZQC", "name": "\u535a\u65f6\u5bcc\u7965\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUXIANGCHUNZHAIZHAIQUANC"}, "009169": {"suoxie": "XCCXLHPZHHA", "name": "\u6e58\u8d22\u957f\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGXINGLINGHUOPEIZHIHUNHEA"}, "009170": {"suoxie": "XCCXLHPZHHC", "name": "\u6e58\u8d22\u957f\u5174\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGXINGLINGHUOPEIZHIHUNHEC"}, "009171": {"suoxie": "YYZZ15NGKZZSA", "name": "\u6c38\u8d62\u4e2d\u503a-1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGZHAI15NIANGUOKAIZHAIZHISHUA"}, "009172": {"suoxie": "YYZZ15NGKZZSC", "name": "\u6c38\u8d62\u4e2d\u503a-1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGZHAI15NIANGUOKAIZHAIZHISHUC"}, "009174": {"suoxie": "DFHYHWJYLLNFOF", "name": "\u4e1c\u65b9\u7ea2\u9890\u548c\u7a33\u5065\u517b\u8001\u4e24\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGYIHEWENJIANYANGLAOLIANGNIANFOF"}, "009175": {"suoxie": "ZSZZ15NZCXJRZA", "name": "\u6d59\u5546\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIA"}, "009176": {"suoxie": "ZSZZ15NZCXJRZC", "name": "\u6d59\u5546\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIC"}, "009177": {"suoxie": "DFYY18GYDKZQA", "name": "\u4e1c\u65b9\u6c38\u60a618\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGYUE18GEYUEDINGKAIZHAIQUANA"}, "009178": {"suoxie": "DFYY18GYDKZQC", "name": "\u4e1c\u65b9\u6c38\u60a618\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGYONGYUE18GEYUEDINGKAIZHAIQUANC"}, "009179": {"suoxie": "JSZZZYXFETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGZHUYAOXIAOFEIETFLIANJIEA"}, "009180": {"suoxie": "JSZZZYXFETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGZHUYAOXIAOFEIETFLIANJIEC"}, "009181": {"suoxie": "ZSZDXWJHBYNCYHHA", "name": "\u6d59\u5546\u667a\u591a\u5174\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOXINGWENJIANHUIBAOYINIANCHIYOUHUNHEA"}, "009182": {"suoxie": "ZSZDXWJHBYNCYHHC", "name": "\u6d59\u5546\u667a\u591a\u5174\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOXINGWENJIANHUIBAOYINIANCHIYOUHUNHEC"}, "009183": {"suoxie": "DFHYHPHYLSNFOF", "name": "\u4e1c\u65b9\u7ea2\u9890\u548c\u5e73\u8861\u517b\u8001\u4e09\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DONGFANGHONGYIHEPINGHENGYANGLAOSANNIANFOF"}, "009184": {"suoxie": "DFHYHJJYLWNFOF", "name": "\u4e1c\u65b9\u7ea2\u9890\u548c\u79ef\u6781\u517b\u8001\u4e94\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGYIHEJIJIYANGLAOWUNIANFOF"}, "009185": {"suoxie": "NYHLYL3YCYFOF", "name": "\u519c\u94f6\u6c47\u7406\u6c38\u4e503\u6708\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NONGYINHUILIYONGLE3YUECHIYOUFOF"}, "009186": {"suoxie": "THJXSGYDKA", "name": "\u5929\u5f18\u805a\u65b0\u4e09\u4e2a\u6708\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGJUXINSANGEYUEDINGKAIA"}, "009187": {"suoxie": "THJXSGYDKC", "name": "\u5929\u5f18\u805a\u65b0\u4e09\u4e2a\u6708\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGJUXINSANGEYUEDINGKAIC"}, "009188": {"suoxie": "PHGXJXHH", "name": "\u9e4f\u534e\u80a1\u606f\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAGUXIJINGXUANHUNHE"}, "009189": {"suoxie": "HBCZCLHH", "name": "\u534e\u5b9d\u6210\u957f\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOCHENGZHANGCELUEHUNHE"}, "009190": {"suoxie": "JSCCHXYXHH", "name": "\u666f\u987a\u957f\u57ce\u6838\u5fc3\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGHEXINYOUXUANHUNHE"}, "009191": {"suoxie": "ZXBCJYZDZA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u88d5\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINBAOCHENGJINGYUZHONGDUANZHAIA"}, "009192": {"suoxie": "ZXBCJYZDZC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u88d5\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGXINBAOCHENGJINGYUZHONGDUANZHAIC"}, "009193": {"suoxie": "FGHLJXHHQDIIMY", "name": "\u5bcc\u56fd\u7ea2\u5229\u7cbe\u9009\u6df7\u5408(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOHONGLIJINGXUANHUNHEQDIIMEIYUAN"}, "009194": {"suoxie": "TDHLZZJYZSJJA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u8bc1\u7ee9\u4f18\u6307\u6570\u57fa\u91d1A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAHONGLIZHONGZHENGJIYOUZHISHUJIJINA"}, "009195": {"suoxie": "TDHLZZJYZSJJC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u8bc1\u7ee9\u4f18\u6307\u6570\u57fa\u91d1C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAHONGLIZHONGZHENGJIYOUZHISHUJIJINC"}, "009196": {"suoxie": "BXRFDSZDZA", "name": "\u5317\u4fe1\u745e\u4e30\u9f0e\u76db\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BEIXINRUIFENGDINGSHENGZHONGDUANZHAIA"}, "009197": {"suoxie": "BXRFDSZDZC", "name": "\u5317\u4fe1\u745e\u4e30\u9f0e\u76db\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BEIXINRUIFENGDINGSHENGZHONGDUANZHAIC"}, "009198": {"suoxie": "QHKYHJETFLJ", "name": "\u524d\u6d77\u5f00\u6e90\u9ec4\u91d1ETF\u8054\u63a5", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "QIANHAIKAIYUANHUANGJINETFLIANJIE"}, "009199": {"suoxie": "WJJZYSYNCYQHH", "name": "\u4e07\u5bb6\u4ef7\u503c\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJIAZHIYOUSHIYINIANCHIYOUQIHUNHE"}, "009200": {"suoxie": "HAJXHH", "name": "\u534e\u5b89\u91d1\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJINXIANGHUNHE"}, "009201": {"suoxie": "ZYYXYNDKHHA", "name": "\u4e2d\u90ae\u4f18\u4eab\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOUYOUXIANGYINIANDINGKAIHUNHEA"}, "009202": {"suoxie": "ZYYXYNDKHHC", "name": "\u4e2d\u90ae\u4f18\u4eab\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOUYOUXIANGYINIANDINGKAIHUNHEC"}, "009203": {"suoxie": "PYWLZQA", "name": "\u9e4f\u626c\u7a33\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGWENLIZHAIQUANA"}, "009204": {"suoxie": "PYWLZQC", "name": "\u9e4f\u626c\u7a33\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGWENLIZHAIQUANC"}, "009205": {"suoxie": "XYFYWYHBHHA", "name": "\u5174\u94f6\u4e30\u8fd0\u7a33\u76ca\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYINFENGYUNWENYIHUIBAOHUNHEA"}, "009206": {"suoxie": "XYFYWYHBHHC", "name": "\u5174\u94f6\u4e30\u8fd0\u7a33\u76ca\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYINFENGYUNWENYIHUIBAOHUNHEC"}, "009207": {"suoxie": "XYHZDKZ", "name": "\u5174\u94f6\u6c47\u667a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIZHIDINGKAIZHAI"}, "009208": {"suoxie": "JXHS300ZSZQLOFC", "name": "\u5efa\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300ZHISHUZENGQIANGLOFC"}, "009209": {"suoxie": "CXWLYNCYQHHFOF", "name": "\u957f\u4fe1\u7a33\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENLIYINIANCHIYOUQIHUNHEFOF"}, "009210": {"suoxie": "ZOJHSNHHA", "name": "\u4e2d\u6b27\u5609\u548c\u4e09\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAHESANNIANHUNHEA"}, "009211": {"suoxie": "ZOJHSNHHC", "name": "\u4e2d\u6b27\u5609\u548c\u4e09\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAHESANNIANHUNHEC"}, "009212": {"suoxie": "YFDHM39GYDKZQ", "name": "\u6613\u65b9\u8fbe\u6052\u830239\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGMAO39GEYUEDINGKAIZHAIQUAN"}, "009213": {"suoxie": "YFDRYATFOFA", "name": "\u6613\u65b9\u8fbe\u5982\u610f\u5b89\u6cf0(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDARUYIANTAIFOFA"}, "009214": {"suoxie": "YFDRYATFOFC", "name": "\u6613\u65b9\u8fbe\u5982\u610f\u5b89\u6cf0(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDARUYIANTAIFOFC"}, "009215": {"suoxie": "YFDRCHHA", "name": "\u6613\u65b9\u8fbe\u745e\u5ddd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICHUANHUNHEA"}, "009216": {"suoxie": "YFDRCHHC", "name": "\u6613\u65b9\u8fbe\u745e\u5ddd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUICHUANHUNHEC"}, "009217": {"suoxie": "BSRFHBSNFBHHA", "name": "\u535a\u65f6\u8363\u4e30\u56de\u62a5\u4e09\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGFENGHUIBAOSANNIANFENGBIHUNHEA"}, "009218": {"suoxie": "BSRFHBSNFBHHC", "name": "\u535a\u65f6\u8363\u4e30\u56de\u62a5\u4e09\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGFENGHUIBAOSANNIANFENGBIHUNHEC"}, "009219": {"suoxie": "DCPBNFHZ13NZSA", "name": "\u5927\u6210\u5f6d\u535a\u519c\u53d1\u884c\u503a1-3\u5e74\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGPENGBONONGFAHANGZHAI13NIANZHISHUA"}, "009220": {"suoxie": "DCPBNFHZ13NZSC", "name": "\u5927\u6210\u5f6d\u535a\u519c\u53d1\u884c\u503a1-3\u5e74\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGPENGBONONGFAHANGZHAI13NIANZHISHUC"}, "009223": {"suoxie": "BYXDFWYHHA", "name": "\u5b9d\u76c8\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGXIANDAIFUWUYEHUNHEA"}, "009224": {"suoxie": "BYXDFWYHHC", "name": "\u5b9d\u76c8\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGXIANDAIFUWUYEHUNHEC"}, "009225": {"suoxie": "THZZZMHLWQDIIA", "name": "\u5929\u5f18\u4e2d\u8bc1\u4e2d\u7f8e\u4e92\u8054\u7f51(QDII)A", "style": "QDII", "pinyin": "TIANHONGZHONGZHENGZHONGMEIHULIANWANGQDIIA"}, "009226": {"suoxie": "THZZZMHLWQDIIC", "name": "\u5929\u5f18\u4e2d\u8bc1\u4e2d\u7f8e\u4e92\u8054\u7f51(QDII)C", "style": "QDII", "pinyin": "TIANHONGZHONGZHENGZHONGMEIHULIANWANGQDIIC"}, "009227": {"suoxie": "PAZXLGYDKZA", "name": "\u5e73\u5b89\u589e\u946b\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGXINLIUGEYUEDINGKAIZHAIA"}, "009228": {"suoxie": "PAZXLGYDKZC", "name": "\u5e73\u5b89\u589e\u946b\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGXINLIUGEYUEDINGKAIZHAIC"}, "009229": {"suoxie": "PAZXLGYDKZE", "name": "\u5e73\u5b89\u589e\u946b\u516d\u4e2a\u6708\u5b9a\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZENGXINLIUGEYUEDINGKAIZHAIE"}, "009230": {"suoxie": "PHAHHHA", "name": "\u9e4f\u534e\u5b89\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANHEHUNHEA"}, "009231": {"suoxie": "PHAHHHC", "name": "\u9e4f\u534e\u5b89\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANHEHUNHEC"}, "009232": {"suoxie": "PHAHHHA", "name": "\u9e4f\u534e\u5b89\u60e0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANHUIHUNHEA"}, "009233": {"suoxie": "PHAHHHC", "name": "\u9e4f\u534e\u5b89\u60e0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANHUIHUNHEC"}, "009234": {"suoxie": "PHYZQYHH", "name": "\u9e4f\u534e\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYOUZHIQIYEHUNHE"}, "009235": {"suoxie": "JSCCHY66GYDKZ", "name": "\u666f\u987a\u957f\u57ce\u5f18\u8fdc66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGHONGYUAN66GEYUEDINGKAIZHAI"}, "009236": {"suoxie": "ZXJTWTYNDKZQ", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u6cf0\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENTAIYINIANDINGKAIZHAIQUAN"}, "009237": {"suoxie": "XYLSCZYNDKZQA", "name": "\u5174\u4e1a\u7eff\u8272\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYELVSECHUNZHAIYINIANDINGKAIZHAIQUANA"}, "009238": {"suoxie": "XYLSCZYNDKZQC", "name": "\u5174\u4e1a\u7eff\u8272\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYELVSECHUNZHAIYINIANDINGKAIZHAIQUANC"}, "009239": {"suoxie": "RTRGZNZSLOFC", "name": "\u878d\u901a\u4eba\u5de5\u667a\u80fd\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGRENGONGZHINENGZHISHULOFC"}, "009240": {"suoxie": "TKLCYSYNCYGP", "name": "\u6cf0\u5eb7\u84dd\u7b79\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "TAIKANGLANCHOUYOUSHIYINIANCHIYOUGUPIAO"}, "009241": {"suoxie": "RTLXCZHHLOFC", "name": "\u878d\u901a\u9886\u5148\u6210\u957f\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGLINGXIANCHENGZHANGHUNHELOFC"}, "009242": {"suoxie": "ZJHXZZHHA", "name": "\u4e2d\u52a0\u6838\u5fc3\u667a\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAHEXINZHIZAOHUNHEA"}, "009243": {"suoxie": "ZJHXZZHHC", "name": "\u4e2d\u52a0\u6838\u5fc3\u667a\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAHEXINZHIZAOHUNHEC"}, "009244": {"suoxie": "GSABWF6GYCYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u4e306\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENFENG6GEYUECHIYOUHUNHEA"}, "009245": {"suoxie": "GSABWF6GYCYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u4e306\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENFENG6GEYUECHIYOUHUNHEC"}, "009246": {"suoxie": "DMESGLHHH", "name": "\u5927\u6469ESG\u91cf\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOESGLIANGHUAHUNHE"}, "009247": {"suoxie": "YFDPHJGYCYHHA", "name": "\u6613\u65b9\u8fbe\u78d0\u6052\u4e5d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANHENGJIUGEYUECHIYOUHUNHEA"}, "009248": {"suoxie": "YFDPHJGYCYHHC", "name": "\u6613\u65b9\u8fbe\u78d0\u6052\u4e5d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANHENGJIUGEYUECHIYOUHUNHEC"}, "009249": {"suoxie": "YFDPTYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u78d0\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANTAIYINIANCHIYOUQIHUNHEA"}, "009250": {"suoxie": "YFDPTYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u78d0\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANTAIYINIANCHIYOUQIHUNHEC"}, "009251": {"suoxie": "GDHBB", "name": "\u5149\u5927\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAHUOBIB"}, "009252": {"suoxie": "FCTYCZA", "name": "\u8702\u5de2\u6dfb\u5143\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYUANCHUNZHAIA"}, "009253": {"suoxie": "FCTYCZC", "name": "\u8702\u5de2\u6dfb\u5143\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANYUANCHUNZHAIC"}, "009254": {"suoxie": "FCTX87GYDK", "name": "\u8702\u5de2\u6dfb\u79a787\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOTIANXI87GEYUEDINGKAI"}, "009255": {"suoxie": "ZYTS39GYDQKFZQ", "name": "\u4e2d\u94f6\u6dfb\u76db39\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINTIANSHENG39GEYUEDINGQIKAIFANGZHAIQUAN"}, "009256": {"suoxie": "MSJYXTZQ", "name": "\u6c11\u751f\u52a0\u94f6\u946b\u901a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINXINTONGZHAIQUAN"}, "009257": {"suoxie": "GYZLZDZZQF", "name": "\u5de5\u94f6\u5c0a\u5229\u4e2d\u77ed\u503a\u503a\u5238F", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNLIZHONGDUANZHAIZHAIQUANF"}, "009258": {"suoxie": "XBLDJRLHPZHHC", "name": "\u897f\u90e8\u5229\u5f97\u666f\u745e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJINGRUILINGHUOPEIZHIHUNHEC"}, "009260": {"suoxie": "MSJYJL6GYHHA", "name": "\u6c11\u751f\u52a0\u94f6\u805a\u52296\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINJULI6GEYUEHUNHEA"}, "009261": {"suoxie": "MSJYJL6GYHHC", "name": "\u6c11\u751f\u52a0\u94f6\u805a\u52296\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINJULI6GEYUEHUNHEC"}, "009262": {"suoxie": "MSJYZZ15NZJZZS", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINZHONGZHAI15NIANZHENGJINZHAIZHISHU"}, "009263": {"suoxie": "HBHLJXHHA", "name": "\u534e\u5b9d\u7ea2\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOHONGLIJINGXUANHUNHEA"}, "009264": {"suoxie": "HDRXSNCYQHH", "name": "\u6cd3\u5fb7\u745e\u5174\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDERUIXINGSANNIANCHIYOUQIHUNHE"}, "009265": {"suoxie": "YFDXFJXGP", "name": "\u6613\u65b9\u8fbe\u6d88\u8d39\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAXIAOFEIJINGXUANGUPIAO"}, "009266": {"suoxie": "PYJHLGYCYHH", "name": "\u9e4f\u626c\u666f\u5408\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGHELIUGEYUECHIYOUHUNHE"}, "009267": {"suoxie": "GFSZTLZQE", "name": "\u5e7f\u53d1\u53cc\u503a\u6dfb\u5229\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFASHUANGZHAITIANLIZHAIQUANE"}, "009268": {"suoxie": "CJHXWJZL6GYCYQA", "name": "\u521b\u91d1\u5408\u4fe1\u7a33\u5065\u589e\u52296\u4e2a\u6708\u6301\u6709\u671fA", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINWENJIANZENGLI6GEYUECHIYOUQIA"}, "009269": {"suoxie": "CJHXWJZL6GYCYQC", "name": "\u521b\u91d1\u5408\u4fe1\u7a33\u5065\u589e\u52296\u4e2a\u6708\u6301\u6709\u671fC", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINWENJIANZENGLI6GEYUECHIYOUQIC"}, "009270": {"suoxie": "RTNXCLLHPZHHC", "name": "\u878d\u901a\u9006\u5411\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGNIXIANGCELUELINGHUOPEIZHIHUNHEC"}, "009271": {"suoxie": "BSXYYXZQA", "name": "\u535a\u65f6\u4fe1\u7528\u4f18\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIXINYONGYOUXUANZHAIQUANA"}, "009272": {"suoxie": "BSXYYXZQC", "name": "\u535a\u65f6\u4fe1\u7528\u4f18\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIXINYONGYOUXUANZHAIQUANC"}, "009273": {"suoxie": "RTZGF1HLHPZHHC", "name": "\u878d\u901a\u4e2d\u56fd\u98ce1\u53f7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHONGGUOFENG1HAOLINGHUOPEIZHIHUNHEC"}, "009274": {"suoxie": "RTJKCYLHPZHHC", "name": "\u878d\u901a\u5065\u5eb7\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGJIANKANGCHANYELINGHUOPEIZHIHUNHEC"}, "009275": {"suoxie": "RTYLBJHYHHC", "name": "\u878d\u901a\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGYILIAOBAOJIANHANGYEHUNHEC"}, "009276": {"suoxie": "PH910NLLFQSZQC", "name": "\u9e4f\u534e9-10\u5e74\u5229\u7387\u53d1\u8d77\u5f0f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUA910NIANLILVFAQISHIZHAIQUANC"}, "009277": {"suoxie": "RTHYJQHHC", "name": "\u878d\u901a\u884c\u4e1a\u666f\u6c14\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGHANGYEJINGQIHUNHEC"}, "009278": {"suoxie": "TTHXCZA", "name": "\u540c\u6cf0\u6052\u5174\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAIHENGXINGCHUNZHAIA"}, "009279": {"suoxie": "TTHXCZC", "name": "\u540c\u6cf0\u6052\u5174\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAIHENGXINGCHUNZHAIC"}, "009280": {"suoxie": "CXZZ13NZJZZSA", "name": "\u957f\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINZHONGZHAI13NIANZHENGJINZHAIZHISHUA"}, "009281": {"suoxie": "CXZZ13NZJZZSC", "name": "\u957f\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINZHONGZHAI13NIANZHENGJINZHAIZHISHUC"}, "009282": {"suoxie": "BYYHCZA", "name": "\u5b9d\u76c8\u76c8\u8f89\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGHUICHUNZHAIA"}, "009283": {"suoxie": "BYYHCZC", "name": "\u5b9d\u76c8\u76c8\u8f89\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGHUICHUNZHAIC"}, "009284": {"suoxie": "SYHFLZQ", "name": "\u4e0a\u94f6\u6167\u4e30\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIFENGLIZHAIQUAN"}, "009285": {"suoxie": "TKZTZXYNCYQHHA", "name": "\u6cf0\u5eb7\u62db\u6cf0\u5c0a\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGZHAOTAIZUNXIANGYINIANCHIYOUQIHUNHEA"}, "009286": {"suoxie": "TKZTZXYNCYQHHC", "name": "\u6cf0\u5eb7\u62db\u6cf0\u5c0a\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGZHAOTAIZUNXIANGYINIANCHIYOUQIHUNHEC"}, "009287": {"suoxie": "HSHYCZZQA", "name": "\u60e0\u5347\u548c\u88d5\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYUCHUNZHAIZHAIQUANA"}, "009288": {"suoxie": "HSHYCZZQC", "name": "\u60e0\u5347\u548c\u88d5\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYUCHUNZHAIZHAIQUANC"}, "009289": {"suoxie": "FGCJJJDCZZQ", "name": "\u5bcc\u56fd\u957f\u6c5f\u7ecf\u6d4e\u5e26\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHANGJIANGJINGJIDAICHUNZHAIZHAIQUAN"}, "009290": {"suoxie": "FGTXYNCYQZQA", "name": "\u5bcc\u56fd\u6dfb\u4eab\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTIANXIANGYINIANCHIYOUQIZHAIQUANA"}, "009291": {"suoxie": "FGTXYNCYQZQC", "name": "\u5bcc\u56fd\u6dfb\u4eab\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOTIANXIANGYINIANCHIYOUQIZHAIQUANC"}, "009292": {"suoxie": "YFDNNHCDKZA", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u6625\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGCHUNDINGKAIZHAIA"}, "009293": {"suoxie": "YFDNNHCDKZC", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u6625\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGCHUNDINGKAIZHAIC"}, "009294": {"suoxie": "JSZYCZZQ", "name": "\u5609\u5b9e\u81f4\u76ca\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIYICHUNZHAIZHAIQUAN"}, "009295": {"suoxie": "MSRZYNDKZ", "name": "\u6c11\u751f\u777f\u667a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGRUIZHIYINIANDINGKAIZHAI"}, "009296": {"suoxie": "NFYHYNHHA", "name": "\u5357\u65b9\u8a89\u6167\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUHUIYINIANHUNHEA"}, "009297": {"suoxie": "NFYHYNHHC", "name": "\u5357\u65b9\u8a89\u6167\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUHUIYINIANHUNHEC"}, "009298": {"suoxie": "YDAHCZA", "name": "\u82f1\u5927\u5b89\u60e0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDAANHUICHUNZHAIA"}, "009299": {"suoxie": "YDAHCZC", "name": "\u82f1\u5927\u5b89\u60e0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDAANHUICHUNZHAIC"}, "009300": {"suoxie": "XBLDZZ500ZSZQC", "name": "\u897f\u90e8\u5229\u5f97\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEZHONGZHENG500ZHISHUZENGQIANGC"}, "009301": {"suoxie": "HSQHDZZQA", "name": "\u6052\u751f\u524d\u6d77\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HENGSHENGQIANHAIDUANZHAIZHAIQUANA"}, "009302": {"suoxie": "HSQHDZZQC", "name": "\u6052\u751f\u524d\u6d77\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HENGSHENGQIANHAIDUANZHAIZHAIQUANC"}, "009303": {"suoxie": "HSQHHYWNDKZA", "name": "\u6052\u751f\u524d\u6d77\u6052\u9890\u4e94\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYIWUNIANDINGKAIZHAIA"}, "009304": {"suoxie": "HSQHHYWNDKZC", "name": "\u6052\u751f\u524d\u6d77\u6052\u9890\u4e94\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYIWUNIANDINGKAIZHAIC"}, "009305": {"suoxie": "HAHL39GYDKCZZQ", "name": "\u6c47\u5b89\u6052\u522939\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANHENGLI39GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "009306": {"suoxie": "PAHMCZ", "name": "\u5e73\u5b89\u60e0\u94ed\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIMINGCHUNZHAI"}, "009308": {"suoxie": "THAKYYHHC", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u517b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIYANGHUNHEC"}, "009309": {"suoxie": "GSABZQ6GYCYQZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5e866\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNQING6GEYUECHIYOUQIZHAIQUANA"}, "009310": {"suoxie": "GSABZQ6GYCYQZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5e866\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOZUNQING6GEYUECHIYOUQIZHAIQUANC"}, "009311": {"suoxie": "CJHXXRXDZZQE", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u65e5\u4eab\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHEXINXINRIXIANGDUANZHAIZHAIQUANE"}, "009312": {"suoxie": "QHLHJZYXHHA", "name": "\u524d\u6d77\u8054\u5408\u4ef7\u503c\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHEJIAZHIYOUXUANHUNHEA"}, "009313": {"suoxie": "QHLHJZYXHHC", "name": "\u524d\u6d77\u8054\u5408\u4ef7\u503c\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHEJIAZHIYOUXUANHUNHEC"}, "009314": {"suoxie": "GFSQSJHHC", "name": "\u5e7f\u53d1\u53cc\u64ce\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHUANGQINGSHENGJIHUNHEC"}, "009315": {"suoxie": "JYZZ13NZJZZSA", "name": "\u4ea4\u94f6\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINZHONGZHAI13NIANZHENGJINZHAIZHISHUA"}, "009316": {"suoxie": "JYZZ13NZJZZSC", "name": "\u4ea4\u94f6\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINZHONGZHAI13NIANZHENGJINZHAIZHISHUC"}, "009317": {"suoxie": "JXHXJZLHH", "name": "\u91d1\u4fe1\u6838\u5fc3\u7ade\u4e89\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINXINHEXINJINGZHENGLIHUNHE"}, "009318": {"suoxie": "NFCZXFHHA", "name": "\u5357\u65b9\u6210\u957f\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHENGZHANGXIANFENGHUNHEA"}, "009319": {"suoxie": "NFCZXFHHC", "name": "\u5357\u65b9\u6210\u957f\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHENGZHANGXIANFENGHUNHEC"}, "009320": {"suoxie": "ZXBCZZ13NNFHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGZHONGZHAI13NIANNONGFAHANGA"}, "009321": {"suoxie": "ZXBCZZ13NNFHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGZHONGZHAI13NIANNONGFAHANGC"}, "009322": {"suoxie": "GFRYJQ3GYCYHHFOFC", "name": "\u5e7f\u53d1\u9510\u610f\u8fdb\u53d63\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYIJINQU3GEYUECHIYOUHUNHEFOFC"}, "009323": {"suoxie": "BSFTYNDKZFQS", "name": "\u535a\u65f6\u5bcc\u901a\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUTONGYINIANDINGKAIZHAIFAQISHI"}, "009324": {"suoxie": "CCZZ35NGKZZSA", "name": "\u957f\u57ce\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI35NIANGUOKAIZHAIZHISHUA"}, "009325": {"suoxie": "CCZZ35NGKZZSC", "name": "\u957f\u57ce\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI35NIANGUOKAIZHAIZHISHUC"}, "009326": {"suoxie": "GFWJZZHHC", "name": "\u5e7f\u53d1\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAWENJIANZENGZHANGHUNHEC"}, "009327": {"suoxie": "DXXSHHA", "name": "\u4e1c\u5174\u5174\u665f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGXINGSHENGHUNHEA"}, "009328": {"suoxie": "DXXSHHC", "name": "\u4e1c\u5174\u5174\u665f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGXINGSHENGHUNHEC"}, "009329": {"suoxie": "HBZZXFLTZSLOFC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u6d88\u8d39\u9f99\u5934\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIAOFEILONGTOUZHISHULOFC"}, "009330": {"suoxie": "PHCZJZHHA", "name": "\u9e4f\u534e\u6210\u957f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGJIAZHIHUNHEA"}, "009331": {"suoxie": "PHCZJZHHC", "name": "\u9e4f\u534e\u6210\u957f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGJIAZHIHUNHEC"}, "009332": {"suoxie": "BSHYCYQHHA", "name": "\u535a\u65f6\u6052\u88d5\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUCHIYOUQIHUNHEA"}, "009333": {"suoxie": "BSHYCYQHHC", "name": "\u535a\u65f6\u6052\u88d5\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUCHIYOUQIHUNHEC"}, "009334": {"suoxie": "FGRX18GYDKHHA", "name": "\u5bcc\u56fd\u878d\u4eab18\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUORONGXIANG18GEYUEDINGKAIHUNHEA"}, "009335": {"suoxie": "GYWJYLMBYNCYHHFQFOF", "name": "\u5de5\u94f6\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFAQIFOF"}, "009336": {"suoxie": "PAZZ500ZSZQA", "name": "\u5e73\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENG500ZHISHUZENGQIANGA"}, "009337": {"suoxie": "PAZZ500ZSZQC", "name": "\u5e73\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENG500ZHISHUZENGQIANGC"}, "009338": {"suoxie": "WJMRXH6GYCYZA", "name": "\u4e07\u5bb6\u6c11\u745e\u7965\u548c6\u4e2a\u6708\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAMINRUIXIANGHE6GEYUECHIYOUZHAIA"}, "009339": {"suoxie": "WJMRXH6GYCYZC", "name": "\u4e07\u5bb6\u6c11\u745e\u7965\u548c6\u4e2a\u6708\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAMINRUIXIANGHE6GEYUECHIYOUZHAIC"}, "009340": {"suoxie": "GYYL2055HHFOF", "name": "\u5de5\u94f6\u517b\u80012055\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINYANGLAO2055HUNHEFOF"}, "009341": {"suoxie": "YFDJHCZGP", "name": "\u6613\u65b9\u8fbe\u5747\u8861\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAJUNHENGCHENGZHANGGUPIAO"}, "009342": {"suoxie": "YFDYZQYSNCYQHH", "name": "\u6613\u65b9\u8fbe\u4f18\u8d28\u4f01\u4e1a\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUZHIQIYESANNIANCHIYOUQIHUNHE"}, "009343": {"suoxie": "TKCJJJDZQA", "name": "\u6cf0\u5eb7\u957f\u6c5f\u7ecf\u6d4e\u5e26\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGCHANGJIANGJINGJIDAIZHAIQUANA"}, "009344": {"suoxie": "TKCJJJDZQC", "name": "\u6cf0\u5eb7\u957f\u6c5f\u7ecf\u6d4e\u5e26\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIKANGCHANGJIANGJINGJIDAIZHAIQUANC"}, "009345": {"suoxie": "ZYSXHBYNCYHHA", "name": "\u4e2d\u94f6\u987a\u5174\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNXINGHUIBAOYINIANCHIYOUHUNHEA"}, "009346": {"suoxie": "ZYSXHBYNCYHHC", "name": "\u4e2d\u94f6\u987a\u5174\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNXINGHUIBAOYINIANCHIYOUHUNHEC"}, "009347": {"suoxie": "ZRJZCZ6GYCYHHA", "name": "\u4e2d\u878d\u4ef7\u503c\u6210\u957f6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJIAZHICHENGZHANG6GEYUECHIYOUHUNHEA"}, "009348": {"suoxie": "ZRJZCZ6GYCYHHC", "name": "\u4e2d\u878d\u4ef7\u503c\u6210\u957f6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJIAZHICHENGZHANG6GEYUECHIYOUHUNHEC"}, "009349": {"suoxie": "QHLHTZZQA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u6cfd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANZEZHAIQUANA"}, "009350": {"suoxie": "QHLHTZZQC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u6cfd\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAILIANHETIANZEZHAIQUANC"}, "009351": {"suoxie": "NFYF18GYHHA", "name": "\u5357\u65b9\u8a89\u4e3018\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUFENG18GEYUEHUNHEA"}, "009352": {"suoxie": "NFYF18GYHHC", "name": "\u5357\u65b9\u8a89\u4e3018\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUFENG18GEYUEHUNHEC"}, "009353": {"suoxie": "ZSKCYGYGDCYHHA", "name": "\u6d59\u5546\u79d1\u521b\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGKECHUANGYIGEYUEGUNDONGCHIYOUHUNHEA"}, "009354": {"suoxie": "ZSKCYGYGDCYHHC", "name": "\u6d59\u5546\u79d1\u521b\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGKECHUANGYIGEYUEGUNDONGCHIYOUHUNHEC"}, "009355": {"suoxie": "TDTHWJYLYNFOF", "name": "\u6cf0\u8fbe\u6cf0\u548c\u7a33\u5065\u517b\u8001\u4e00\u5e74(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDATAIHEWENJIANYANGLAOYINIANFOF"}, "009356": {"suoxie": "BSJJLCYQZQA", "name": "\u535a\u65f6\u5b63\u5b63\u4e50\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJIJILECHIYOUQIZHAIQUANA"}, "009357": {"suoxie": "BSJJLCYQZQC", "name": "\u535a\u65f6\u5b63\u5b63\u4e50\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJIJILECHIYOUQIZHAIQUANC"}, "009358": {"suoxie": "XYWJSLYNCYQZQA", "name": "\u5174\u4e1a\u7a33\u5065\u53cc\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYEWENJIANSHUANGLIYINIANCHIYOUQIZHAIQUANA"}, "009359": {"suoxie": "XYWJSLYNCYQZQC", "name": "\u5174\u4e1a\u7a33\u5065\u53cc\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGYEWENJIANSHUANGLIYINIANCHIYOUQIZHAIQUANC"}, "009360": {"suoxie": "ZSCXZZHHA", "name": "\u62db\u5546\u521b\u65b0\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGCHUANGXINZENGZHANGHUNHEA"}, "009361": {"suoxie": "ZSCXZZHHC", "name": "\u62db\u5546\u521b\u65b0\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGCHUANGXINZENGZHANGHUNHEC"}, "009362": {"suoxie": "ZSFYJJPZHHA", "name": "\u62db\u5546\u4e30\u76c8\u79ef\u6781\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGFENGYINGJIJIPEIZHIHUNHEA"}, "009363": {"suoxie": "ZSFYJJPZHHC", "name": "\u62db\u5546\u4e30\u76c8\u79ef\u6781\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGFENGYINGJIJIPEIZHIHUNHEC"}, "009364": {"suoxie": "GYKJCX6GYDKHHA", "name": "\u5de5\u94f6\u79d1\u6280\u521b\u65b06\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINKEJICHUANGXIN6GEYUEDINGKAIHUNHEA"}, "009365": {"suoxie": "GYKJCX6GYDKHHC", "name": "\u5de5\u94f6\u79d1\u6280\u521b\u65b06\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINKEJICHUANGXIN6GEYUEDINGKAIHUNHEC"}, "009366": {"suoxie": "PYASKJCXYNDKHHA", "name": "\u6d66\u94f6\u5b89\u76db\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGKEJICHUANGXINYINIANDINGKAIHUNHEA"}, "009367": {"suoxie": "PYASKJCXYNDKHHC", "name": "\u6d66\u94f6\u5b89\u76db\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGKEJICHUANGXINYINIANDINGKAIHUNHEC"}, "009368": {"suoxie": "PYASJZJXHHA", "name": "\u6d66\u94f6\u5b89\u76db\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGJIAZHIJINGXUANHUNHEA"}, "009369": {"suoxie": "PYASJZJXHHC", "name": "\u6d66\u94f6\u5b89\u76db\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGJIAZHIJINGXUANHUNHEC"}, "009370": {"suoxie": "PYASRHYX3GYCYHHFOFA", "name": "\u6d66\u94f6\u5b89\u76db\u777f\u548c\u4f18\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGRUIHEYOUXUAN3GEYUECHIYOUHUNHEFOFA"}, "009371": {"suoxie": "PYASRHYX3GYCYHHFOFC", "name": "\u6d66\u94f6\u5b89\u76db\u777f\u548c\u4f18\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGRUIHEYOUXUAN3GEYUECHIYOUHUNHEFOFC"}, "009372": {"suoxie": "PYASJHWJYNCYHHFOFA", "name": "\u6d66\u94f6\u5b89\u76db\u5609\u548c\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGJIAHEWENJIANYINIANCHIYOUHUNHEFOFA"}, "009373": {"suoxie": "PYASJHWJYNCYHHFOFC", "name": "\u6d66\u94f6\u5b89\u76db\u5609\u548c\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGJIAHEWENJIANYINIANCHIYOUHUNHEFOFC"}, "009374": {"suoxie": "PYASMSCIZGAGETFLJA", "name": "\u6d66\u94f6\u5b89\u76dbMSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGMSCIZHONGGUOAGUETFLIANJIEA"}, "009375": {"suoxie": "PYASMSCIZGAGETFLJC", "name": "\u6d66\u94f6\u5b89\u76dbMSCI\u4e2d\u56fdA\u80a1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGMSCIZHONGGUOAGUETFLIANJIEC"}, "009376": {"suoxie": "JSCCCZLHHH", "name": "\u666f\u987a\u957f\u57ce\u6210\u957f\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGCHENGZHANGLINGHANGHUNHE"}, "009377": {"suoxie": "ZSRHYNCYQHHA", "name": "\u62db\u5546\u745e\u6052\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHENGYINIANCHIYOUQIHUNHEA"}, "009378": {"suoxie": "ZSRHYNCYQHHC", "name": "\u62db\u5546\u745e\u6052\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHENGYINIANCHIYOUQIHUNHEC"}, "009379": {"suoxie": "ZYCZYXGPA", "name": "\u4e2d\u94f6\u6210\u957f\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINCHENGZHANGYOUXUANGUPIAOA"}, "009380": {"suoxie": "FADKJLHHH", "name": "\u5bcc\u5b89\u8fbe\u79d1\u6280\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDAKEJILINGHANGHUNHE"}, "009381": {"suoxie": "HAHXZCHHA", "name": "\u6c47\u5b89\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINZICHANHUNHEA"}, "009382": {"suoxie": "HAHXZCHHC", "name": "\u6c47\u5b89\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINZICHANHUNHEC"}, "009383": {"suoxie": "RBWJPZSGYCYFOF", "name": "\u4eba\u4fdd\u7a33\u8fdb\u914d\u7f6e\u4e09\u4e2a\u6708\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RENBAOWENJINPEIZHISANGEYUECHIYOUFOF"}, "009384": {"suoxie": "DMMSCIZGAGZQA", "name": "\u5927\u6469MSCI\u4e2d\u56fdA\u80a1\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DAMOMSCIZHONGGUOAGUZENGQIANGA"}, "009385": {"suoxie": "THYYPHYLSNCYFOF", "name": "\u5929\u5f18\u6c38\u88d5\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TIANHONGYONGYUPINGHENGYANGLAOSANNIANCHIYOUFOF"}, "009386": {"suoxie": "CJHXTX39GY", "name": "\u521b\u91d1\u5408\u4fe1\u6cf0\u4eab39\u4e2a\u6708", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINTAIXIANG39GEYUE"}, "009387": {"suoxie": "JSWFHHA", "name": "\u5609\u5b9e\u7a33\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENFUHUNHEA"}, "009388": {"suoxie": "JSWFHHC", "name": "\u5609\u5b9e\u7a33\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENFUHUNHEC"}, "009391": {"suoxie": "HTFYZCZHHA", "name": "\u6c47\u6dfb\u5bcc\u4f18\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUZHICHENGZHANGHUNHEA"}, "009392": {"suoxie": "HTFYZCZHHC", "name": "\u6c47\u6dfb\u5bcc\u4f18\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUZHICHENGZHANGHUNHEC"}, "009394": {"suoxie": "YHTLJXHH", "name": "\u94f6\u534e\u540c\u529b\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUATONGLIJINGXUANHUNHE"}, "009395": {"suoxie": "XYAXHBHH", "name": "\u946b\u5143\u5b89\u946b\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINYUANANXINHUIBAOHUNHE"}, "009396": {"suoxie": "DCACZQA", "name": "\u5927\u6210\u5b89\u8bda\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGANCHENGZHAIQUANA"}, "009397": {"suoxie": "DCACZQC", "name": "\u5927\u6210\u5b89\u8bda\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGANCHENGZHAIQUANC"}, "009398": {"suoxie": "HFCZQYJXGP", "name": "\u534e\u5bcc\u6210\u957f\u4f01\u4e1a\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUAFUCHENGZHANGQIYEJINGXUANGUPIAO"}, "009399": {"suoxie": "HRXAJSFZSNDK", "name": "\u534e\u878d\u96c4\u5b89\u5efa\u8bbe\u53d1\u5c55\u4e09\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARONGXIONGANJIANSHEFAZHANSANNIANDINGKAI"}, "009400": {"suoxie": "HATR6GYHHA", "name": "\u534e\u5b89\u6dfb\u745e6\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANRUI6GEYUEHUNHEA"}, "009401": {"suoxie": "HATR6GYHHC", "name": "\u534e\u5b89\u6dfb\u745e6\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANRUI6GEYUEHUNHEC"}, "009402": {"suoxie": "JYQMHHA", "name": "\u4ea4\u94f6\u542f\u660e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQIMINGHUNHEA"}, "009403": {"suoxie": "PAHYCZC", "name": "\u5e73\u5b89\u60e0\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIYINGCHUNZHAIC"}, "009404": {"suoxie": "PAHXCZC", "name": "\u5e73\u5b89\u60e0\u4eab\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXIANGCHUNZHAIC"}, "009405": {"suoxie": "PAHLCZC", "name": "\u5e73\u5b89\u60e0\u9686\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUILONGCHUNZHAIC"}, "009406": {"suoxie": "PAGDJZC", "name": "\u5e73\u5b89\u9ad8\u7b49\u7ea7\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANGAODENGJIZHAIC"}, "009407": {"suoxie": "GLHYCZA", "name": "\u683c\u6797\u6cd3\u8fdc\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGYUANCHUNZHAIA"}, "009408": {"suoxie": "GLHYCZC", "name": "\u683c\u6797\u6cd3\u8fdc\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGYUANCHUNZHAIC"}, "009409": {"suoxie": "HATF18GYHHA", "name": "\u534e\u5b89\u6dfb\u798f18\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANFU18GEYUEHUNHEA"}, "009410": {"suoxie": "HATF18GYHHC", "name": "\u534e\u5b89\u6dfb\u798f18\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANFU18GEYUEHUNHEC"}, "009411": {"suoxie": "ZYKJCXYNDKHH", "name": "\u4e2d\u94f6\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINKEJICHUANGXINYINIANDINGKAIHUNHE"}, "009412": {"suoxie": "YFDZYYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u62db\u6613\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAZHAOYIYINIANCHIYOUQIHUNHEA"}, "009413": {"suoxie": "YFDZYYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u62db\u6613\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAZHAOYIYINIANCHIYOUQIHUNHEC"}, "009414": {"suoxie": "ZYDJKGPA", "name": "\u4e2d\u94f6\u5927\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINDAJIANKANGGUPIAOA"}, "009415": {"suoxie": "ZYRXLNDKHHA", "name": "\u4e2d\u90ae\u745e\u4eab\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOURUIXIANGLIANGNIANDINGKAIHUNHEA"}, "009416": {"suoxie": "ZYRXLNDKHHC", "name": "\u4e2d\u90ae\u745e\u4eab\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOURUIXIANGLIANGNIANDINGKAIHUNHEC"}, "009417": {"suoxie": "GTRYSRDKZQA", "name": "\u56fd\u6295\u745e\u94f6\u987a\u8363\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNRONGDINGKAIZHAIQUANA"}, "009418": {"suoxie": "GTRYSRDKZQC", "name": "\u56fd\u6295\u745e\u94f6\u987a\u8363\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNRONGDINGKAIZHAIQUANC"}, "009419": {"suoxie": "BYXMYNDKHHA", "name": "\u5b9d\u76c8\u7965\u660e\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGMINGYINIANDINGKAIHUNHEA"}, "009420": {"suoxie": "BYXMYNDKHHC", "name": "\u5b9d\u76c8\u7965\u660e\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGMINGYINIANDINGKAIHUNHEC"}, "009421": {"suoxie": "GYPBGKZ13NZSA", "name": "\u5de5\u94f6\u5f6d\u535a\u56fd\u5f00\u503a1-3\u5e74\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINPENGBOGUOKAIZHAI13NIANZHISHUA"}, "009422": {"suoxie": "GYPBGKZ13NZSC", "name": "\u5de5\u94f6\u5f6d\u535a\u56fd\u5f00\u503a1-3\u5e74\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINPENGBOGUOKAIZHAI13NIANZHISHUC"}, "009423": {"suoxie": "ZSRXWJPZHHA", "name": "\u62db\u5546\u745e\u4fe1\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIXINWENJIANPEIZHIHUNHEA"}, "009424": {"suoxie": "ZSRXWJPZHHC", "name": "\u62db\u5546\u745e\u4fe1\u7a33\u5065\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIXINWENJIANPEIZHIHUNHEC"}, "009426": {"suoxie": "PYJHLGYCYQHHA", "name": "\u9e4f\u626c\u666f\u60e0\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGHUILIUGEYUECHIYOUQIHUNHEA"}, "009427": {"suoxie": "PYJHLGYCYQHHC", "name": "\u9e4f\u626c\u666f\u60e0\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGHUILIUGEYUECHIYOUQIHUNHEC"}, "009428": {"suoxie": "PYJFLGYCYQHHA", "name": "\u9e4f\u626c\u666f\u6ca3\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGFENGLIUGEYUECHIYOUQIHUNHEA"}, "009429": {"suoxie": "PYJFLGYCYQHHC", "name": "\u9e4f\u626c\u666f\u6ca3\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGFENGLIUGEYUECHIYOUQIHUNHEC"}, "009432": {"suoxie": "DBKJCXYNDKHHA", "name": "\u5fb7\u90a6\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGKEJICHUANGXINYINIANDINGKAIHUNHEA"}, "009433": {"suoxie": "DBKJCXYNDKHHC", "name": "\u5fb7\u90a6\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGKEJICHUANGXINYINIANDINGKAIHUNHEC"}, "009434": {"suoxie": "CHWJZQA", "name": "\u6df3\u539a\u7a33\u5609\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENJIAZHAIQUANA"}, "009435": {"suoxie": "CHWJZQC", "name": "\u6df3\u539a\u7a33\u5609\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENJIAZHAIQUANC"}, "009436": {"suoxie": "CCHTYL2040SNCYHHFOF", "name": "\u957f\u57ce\u6052\u6cf0\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGHENGTAIYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "009437": {"suoxie": "XDAYKCYNDKHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u79d1\u521b\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINKECHUANGYINIANDINGKAIHUNHEA"}, "009438": {"suoxie": "XDAYKCYNDKHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u79d1\u521b\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINKECHUANGYINIANDINGKAIHUNHEC"}, "009439": {"suoxie": "XBLDGQHLLOFC", "name": "\u897f\u90e8\u5229\u5f97\u56fd\u4f01\u7ea2\u5229(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEGUOQIHONGLILOFC"}, "009440": {"suoxie": "GDYXHHA", "name": "\u5149\u5927\u88d5\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAYUXINHUNHEA"}, "009441": {"suoxie": "GDYXHHC", "name": "\u5149\u5927\u88d5\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAYUXINHUNHEC"}, "009442": {"suoxie": "ZYYL2040SNCYQHHFOF", "name": "\u4e2d\u94f6\u517b\u80012040\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINYANGLAO2040SANNIANCHIYOUQIHUNHEFOF"}, "009443": {"suoxie": "GYTH39GYDKZQC", "name": "\u5de5\u94f6\u6cf0\u548c39\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINTAIHE39GEYUEDINGKAIZHAIQUANC"}, "009444": {"suoxie": "GTTFYNDQKFZQ", "name": "\u56fd\u6cf0\u6dfb\u798f\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAITIANFUYINIANDINGQIKAIFANGZHAIQUAN"}, "009445": {"suoxie": "HXSQS13NGDJGQZPA", "name": "\u534e\u590f\u4e0a\u6e05\u62401-3\u5e74\u9ad8\u7b49\u7ea7\u56fd\u4f01\u4e2d\u7968A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIASHANGQINGSUO13NIANGAODENGJIGUOQIZHONGPIAOA"}, "009446": {"suoxie": "HXSQS13NGDJGQZPC", "name": "\u534e\u590f\u4e0a\u6e05\u62401-3\u5e74\u9ad8\u7b49\u7ea7\u56fd\u4f01\u4e2d\u7968C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIASHANGQINGSUO13NIANGAODENGJIGUOQIZHONGPIAOC"}, "009447": {"suoxie": "CTKJCXYNDKHH", "name": "\u8d22\u901a\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGKEJICHUANGXINYINIANDINGKAIHUNHE"}, "009448": {"suoxie": "TKSRYNCYQHHA", "name": "\u6cf0\u5eb7\u7533\u6da6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGSHENRUNYINIANCHIYOUQIHUNHEA"}, "009449": {"suoxie": "TKSRYNCYQHHC", "name": "\u6cf0\u5eb7\u7533\u6da6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGSHENRUNYINIANCHIYOUQIHUNHEC"}, "009450": {"suoxie": "ZJXH1N", "name": "\u4e2d\u91d1\u65b0\u8f891\u5e74", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINHUI1NIAN"}, "009451": {"suoxie": "ZJXS1NDKZ", "name": "\u4e2d\u91d1\u65b0\u76db1\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINSHENG1NIANDINGKAIZHAI"}, "009452": {"suoxie": "GDZYCZYNDKZ", "name": "\u5149\u5927\u5c0a\u88d5\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNYUCHUNZHAIYINIANDINGKAIZHAI"}, "009453": {"suoxie": "PAHX1NDKZ", "name": "\u5e73\u5b89\u5408\u51741\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEXING1NIANDINGKAIZHAI"}, "009456": {"suoxie": "DFWJHBZQC", "name": "\u4e1c\u65b9\u7a33\u5065\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGWENJIANHUIBAOZHAIQUANC"}, "009457": {"suoxie": "HTCXCZA", "name": "\u7ea2\u571f\u521b\u65b0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTUCHUANGXINCHUNZHAIA"}, "009458": {"suoxie": "HTCXCZC", "name": "\u7ea2\u571f\u521b\u65b0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTUCHUANGXINCHUNZHAIC"}, "009459": {"suoxie": "CJHXJAX3GYC", "name": "\u521b\u91d1\u5408\u4fe1\u5b63\u5b89\u946b3\u4e2a\u6708C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINJIANXIN3GEYUEC"}, "009460": {"suoxie": "AXXYWJYLYNCYHHFOF", "name": "\u5b89\u4fe1\u79a7\u60a6\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINXIYUEWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "009461": {"suoxie": "DFZC3GYDKZQA", "name": "\u4e1c\u65b9\u81fb\u84033\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENCUI3GEYUEDINGKAIZHAIQUANA"}, "009462": {"suoxie": "DFZC3GYDKZQC", "name": "\u4e1c\u65b9\u81fb\u84033\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENCUI3GEYUEDINGKAIZHAIQUANC"}, "009463": {"suoxie": "DFZHCZZQA", "name": "\u4e1c\u65b9\u81fb\u6167\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENHUICHUNZHAIZHAIQUANA"}, "009464": {"suoxie": "DFZHCZZQC", "name": "\u4e1c\u65b9\u81fb\u6167\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENHUICHUNZHAIZHAIQUANC"}, "009465": {"suoxie": "DFKZZZQA", "name": "\u4e1c\u65b9\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "DONGFANGKEZHUANZHAIZHAIQUANA"}, "009466": {"suoxie": "DFKZZZQC", "name": "\u4e1c\u65b9\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "DONGFANGKEZHUANZHAIZHAIQUANC"}, "009467": {"suoxie": "HTCXKJCX3GYDKHHA", "name": "\u7ea2\u571f\u521b\u65b0\u79d1\u6280\u521b\u65b03\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTUCHUANGXINKEJICHUANGXIN3GEYUEDINGKAIHUNHEA"}, "009468": {"suoxie": "BSJKCZSZDQKSHHHA", "name": "\u535a\u65f6\u5065\u5eb7\u6210\u957f\u53cc\u5468\u5b9a\u671f\u53ef\u8d4e\u56de\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIANKANGCHENGZHANGSHUANGZHOUDINGQIKESHUHUIHUNHEA"}, "009469": {"suoxie": "BSJKCZSZDQKSHHHC", "name": "\u535a\u65f6\u5065\u5eb7\u6210\u957f\u53cc\u5468\u5b9a\u671f\u53ef\u8d4e\u56de\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIANKANGCHENGZHANGSHUANGZHOUDINGQIKESHUHUIHUNHEC"}, "009470": {"suoxie": "DFXLHHA", "name": "\u4e1c\u65b9\u6b23\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINLIHUNHEA"}, "009471": {"suoxie": "DFXLHHC", "name": "\u4e1c\u65b9\u6b23\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINLIHUNHEC"}, "009472": {"suoxie": "GFSZ100ZSLOFC", "name": "\u5e7f\u53d1\u6df1\u8bc1100\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFASHENZHENG100ZHISHULOFC"}, "009474": {"suoxie": "GTZYYSHH", "name": "\u56fd\u6cf0\u81f4\u8fdc\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIZHIYUANYOUSHIHUNHE"}, "009475": {"suoxie": "HFJXHYHH", "name": "\u6c47\u4e30\u664b\u4fe1\u6167\u76c8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUIFENGJINXINHUIYINGHUNHE"}, "009476": {"suoxie": "JXSPYLHYGPA", "name": "\u5efa\u4fe1\u98df\u54c1\u996e\u6599\u884c\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINSHIPINYINLIAOHANGYEGUPIAOA"}, "009477": {"suoxie": "ZYSHJETFLJA", "name": "\u4e2d\u94f6\u4e0a\u6d77\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "ZHONGYINSHANGHAIJINETFLIANJIEA"}, "009478": {"suoxie": "ZYSHJETFLJC", "name": "\u4e2d\u94f6\u4e0a\u6d77\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "ZHONGYINSHANGHAIJINETFLIANJIEC"}, "009479": {"suoxie": "ZYZZ100ETFLJA", "name": "\u4e2d\u94f6\u4e2d\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG100ETFLIANJIEA"}, "009480": {"suoxie": "ZYZZ100ETFLJC", "name": "\u4e2d\u94f6\u4e2d\u8bc1100ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG100ETFLIANJIEC"}, "009481": {"suoxie": "GTHYYNCYQHHA", "name": "\u56fd\u6cf0\u5b8f\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHONGYIYINIANCHIYOUQIHUNHEA"}, "009482": {"suoxie": "GTHYYNCYQHHC", "name": "\u56fd\u6cf0\u5b8f\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHONGYIYINIANCHIYOUQIHUNHEC"}, "009483": {"suoxie": "PHPLZQA", "name": "\u9e4f\u534e\u666e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAPULIZHAIQUANA"}, "009484": {"suoxie": "PHPLZQC", "name": "\u9e4f\u534e\u666e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAPULIZHAIQUANC"}, "009485": {"suoxie": "GSABJBPHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u805a\u5b9d\u76c6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOJUBAOPENHUOBIB"}, "009486": {"suoxie": "GDRHHHA", "name": "\u5149\u5927\u745e\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDARUIHEHUNHEA"}, "009487": {"suoxie": "GDRHHHC", "name": "\u5149\u5927\u745e\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDARUIHEHUNHEC"}, "009488": {"suoxie": "ZYJZJXHHA", "name": "\u4e2d\u90ae\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUJIAZHIJINGXUANHUNHEA"}, "009489": {"suoxie": "ZYJZJXHHC", "name": "\u4e2d\u90ae\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUJIAZHIJINGXUANHUNHEC"}, "009490": {"suoxie": "TKKJCXYNDKHH", "name": "\u6cf0\u5eb7\u79d1\u6280\u521b\u65b0\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGKEJICHUANGXINYINIANDINGKAIHUNHE"}, "009491": {"suoxie": "BYCXQDGPA", "name": "\u5b9d\u76c8\u521b\u65b0\u9a71\u52a8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGCHUANGXINQUDONGGUPIAOA"}, "009492": {"suoxie": "BYCXQDGPC", "name": "\u5b9d\u76c8\u521b\u65b0\u9a71\u52a8\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGCHUANGXINQUDONGGUPIAOC"}, "009493": {"suoxie": "DCZX18YDKHHA", "name": "\u5927\u6210\u5c0a\u4eab18\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGZUNXIANG18YUEDINGKAIHUNHEA"}, "009494": {"suoxie": "DCZX18YDKHHC", "name": "\u5927\u6210\u5c0a\u4eab18\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGZUNXIANG18YUEDINGKAIHUNHEC"}, "009495": {"suoxie": "DCJXZGDJZQA", "name": "\u5927\u6210\u666f\u8f69\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGXUANZHONGGAODENGJIZHAIQUANA"}, "009496": {"suoxie": "DCJXZGDJZQC", "name": "\u5927\u6210\u666f\u8f69\u4e2d\u9ad8\u7b49\u7ea7\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGXUANZHONGGAODENGJIZHAIQUANC"}, "009499": {"suoxie": "JSCCAXHBHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u946b\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANXINHUIBAOHUNHEA"}, "009500": {"suoxie": "GSABGGXHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u9ad8\u80a1\u606f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOGAOGUXIHUNHEA"}, "009501": {"suoxie": "GSABGGXHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u9ad8\u80a1\u606f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOGAOGUXIHUNHEC"}, "009502": {"suoxie": "GSABCXYYGPA", "name": "\u56fd\u5bff\u5b89\u4fdd\u521b\u65b0\u533b\u836f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOSHOUANBAOCHUANGXINYIYAOGUPIAOA"}, "009503": {"suoxie": "GSABCXYYGPC", "name": "\u56fd\u5bff\u5b89\u4fdd\u521b\u65b0\u533b\u836f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOSHOUANBAOCHUANGXINYIYAOGUPIAOC"}, "009504": {"suoxie": "FGSHJETFLJA", "name": "\u5bcc\u56fd\u4e0a\u6d77\u91d1ETF\u8054\u63a5A", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "FUGUOSHANGHAIJINETFLIANJIEA"}, "009505": {"suoxie": "FGSHJETFLJC", "name": "\u5bcc\u56fd\u4e0a\u6d77\u91d1ETF\u8054\u63a5C", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "FUGUOSHANGHAIJINETFLIANJIEC"}, "009506": {"suoxie": "FRFHLNDKZ", "name": "\u5bcc\u8363\u5bcc\u6052\u4e24\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGFUHENGLIANGNIANDINGKAIZHAI"}, "009507": {"suoxie": "GJXYYYXFA", "name": "\u56fd\u91d1\u946b\u610f\u533b\u836f\u6d88\u8d39A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINYIYIYAOXIAOFEIA"}, "009508": {"suoxie": "GJXYYYXFC", "name": "\u56fd\u91d1\u946b\u610f\u533b\u836f\u6d88\u8d39C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINYIYIYAOXIAOFEIC"}, "009509": {"suoxie": "PAHRCZ", "name": "\u5e73\u5b89\u60e0\u6da6\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIRUNCHUNZHAI"}, "009510": {"suoxie": "THTLZQLOFE", "name": "\u5929\u5f18\u540c\u5229\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGTONGLIZHAIQUANLOFE"}, "009511": {"suoxie": "XDAYYJYXHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u7814\u7a76\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYANJIUYOUXUANHUNHEA"}, "009512": {"suoxie": "THTLZQLOFE", "name": "\u5929\u5f18\u6dfb\u5229\u503a\u5238(LOF)E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGTIANLIZHAIQUANLOFE"}, "009513": {"suoxie": "CJTSCYBJXGPA", "name": "\u521b\u91d1\u540c\u987a\u521b\u4e1a\u677f\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINTONGSHUNCHUANGYEBANJINGXUANGUPIAOA"}, "009514": {"suoxie": "CJTSCYBJXGPC", "name": "\u521b\u91d1\u540c\u987a\u521b\u4e1a\u677f\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINTONGSHUNCHUANGYEBANJINGXUANGUPIAOC"}, "009515": {"suoxie": "ZOZYWJYNHHA", "name": "\u4e2d\u6b27\u771f\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUZHENYIWENJIANYINIANHUNHEA"}, "009516": {"suoxie": "ZOZYWJYNHHC", "name": "\u4e2d\u6b27\u771f\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUZHENYIWENJIANYINIANHUNHEC"}, "009517": {"suoxie": "RBFX3GYDKZA", "name": "\u4eba\u4fdd\u798f\u6b233\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOFUXIN3GEYUEDINGKAIZHAIA"}, "009518": {"suoxie": "RBFX3GYDKZC", "name": "\u4eba\u4fdd\u798f\u6b233\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RENBAOFUXIN3GEYUEDINGKAIZHAIC"}, "009519": {"suoxie": "ZODLZQE", "name": "\u4e2d\u6b27\u9f0e\u5229\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUDINGLIZHAIQUANE"}, "009520": {"suoxie": "ZODLZQC", "name": "\u4e2d\u6b27\u9f0e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUDINGLIZHAIQUANC"}, "009523": {"suoxie": "BYJF39GYDKZA", "name": "\u5b9d\u76c8\u805a\u798f39\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGJUFU39GEYUEDINGKAIZHAIA"}, "009524": {"suoxie": "BYJF39GYDKZC", "name": "\u5b9d\u76c8\u805a\u798f39\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGJUFU39GEYUEDINGKAIZHAIC"}, "009525": {"suoxie": "GFJRYNCYHHA", "name": "\u5e7f\u53d1\u805a\u8363\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJURONGYINIANCHIYOUHUNHEA"}, "009526": {"suoxie": "GFJRYNCYHHC", "name": "\u5e7f\u53d1\u805a\u8363\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJURONGYINIANCHIYOUHUNHEC"}, "009527": {"suoxie": "ZSHJXXXFHH", "name": "\u6d59\u5546\u6c47\u91d1\u65b0\u5174\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGHUIJINXINXINGXIAOFEIHUNHE"}, "009528": {"suoxie": "JXHBSDFZFZZS", "name": "\u5efa\u4fe1\u6e56\u5317\u7701\u5730\u65b9\u653f\u5e9c\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINHUBEISHENGDIFANGZHENGFUZHAIZHISHU"}, "009529": {"suoxie": "ZRZZ15NGKHA", "name": "\u4e2d\u878d\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGZHONGZHAI15NIANGUOKAIHANGA"}, "009530": {"suoxie": "ZRZZ15NGKHC", "name": "\u4e2d\u878d\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGZHONGZHAI15NIANGUOKAIHANGC"}, "009531": {"suoxie": "JTRH18GYDKHH", "name": "\u4e5d\u6cf0\u9510\u548c18\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAIRUIHE18GEYUEDINGKAIHUNHE"}, "009532": {"suoxie": "GFJMZDZE", "name": "\u5e7f\u53d1\u666f\u660e\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAJINGMINGZHONGDUANZHAIE"}, "009533": {"suoxie": "TPHZ63GYDK", "name": "\u592a\u5e73\u6052\u6cfd63\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGZE63GEYUEDINGKAI"}, "009534": {"suoxie": "NFSYZDQLLZA", "name": "\u5357\u65b9\u5347\u5143\u4e2d\u77ed\u671f\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGSHENGYUANZHONGDUANQILILVZHAIA"}, "009535": {"suoxie": "NFSYZDQLLZC", "name": "\u5357\u65b9\u5347\u5143\u4e2d\u77ed\u671f\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGSHENGYUANZHONGDUANQILILVZHAIC"}, "009536": {"suoxie": "HTFWJZYYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u589e\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANZENGYIYINIANCHIYOUHUNHEA"}, "009537": {"suoxie": "TPHYYXGPA", "name": "\u592a\u5e73\u884c\u4e1a\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGHANGYEYOUXUANGUPIAOA"}, "009538": {"suoxie": "TPHYYXGPC", "name": "\u592a\u5e73\u884c\u4e1a\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGHANGYEYOUXUANGUPIAOC"}, "009539": {"suoxie": "XYRJHHA", "name": "\u5174\u4e1a\u777f\u8fdb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYERUIJINHUNHEA"}, "009540": {"suoxie": "XYRJHHC", "name": "\u5174\u4e1a\u777f\u8fdb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYERUIJINHUNHEC"}, "009541": {"suoxie": "YHZZ13NNFHZQZS", "name": "\u94f6\u534e\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHU"}, "009542": {"suoxie": "YHFLJXHHA", "name": "\u94f6\u534e\u5bcc\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFULIJINGXUANHUNHEA"}, "009543": {"suoxie": "SWLXATFLSNDKA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u5bcc\u5229\u4e09\u5e74\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIFULISANNIANDINGKAIA"}, "009544": {"suoxie": "SWLXATFLSNDKC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u6cf0\u5bcc\u5229\u4e09\u5e74\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINANTAIFULISANNIANDINGKAIC"}, "009545": {"suoxie": "BSXRWJHHA", "name": "\u535a\u65f6\u946b\u8363\u7a33\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIXINRONGWENJIANHUNHEA"}, "009546": {"suoxie": "BSXRWJHHC", "name": "\u535a\u65f6\u946b\u8363\u7a33\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIXINRONGWENJIANHUNHEC"}, "009548": {"suoxie": "HTFZPJZJXHHA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANJIAZHIJINGXUANHUNHEA"}, "009549": {"suoxie": "HTFZPJZJXHHC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANJIAZHIJINGXUANHUNHEC"}, "009550": {"suoxie": "HTFKFYS6GYCYQGPA", "name": "\u6c47\u6dfb\u5bcc\u5f00\u653e\u4f18\u52bf6\u4e2a\u6708\u6301\u6709\u671f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUKAIFANGYOUSHI6GEYUECHIYOUQIGUPIAOA"}, "009551": {"suoxie": "HTFKFYS6GYCYQGPC", "name": "\u6c47\u6dfb\u5bcc\u5f00\u653e\u4f18\u52bf6\u4e2a\u6708\u6301\u6709\u671f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUKAIFANGYOUSHI6GEYUECHIYOUQIGUPIAOC"}, "009552": {"suoxie": "CTZGFQ39GYDKZA", "name": "\u8d22\u901a\u8d44\u7ba1\u4e30\u4e7e39\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANFENGQIAN39GEYUEDINGKAIZHAIA"}, "009553": {"suoxie": "CTZGFQ39GYDKZC", "name": "\u8d22\u901a\u8d44\u7ba1\u4e30\u4e7e39\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANFENGQIAN39GEYUEDINGKAIZHAIC"}, "009554": {"suoxie": "JXZZ13NNFHZQZSA", "name": "\u5efa\u4fe1\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHUA"}, "009555": {"suoxie": "JXZZ13NNFHZQZSC", "name": "\u5efa\u4fe1\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHUC"}, "009556": {"suoxie": "XQHFSNCYHH", "name": "\u5174\u5168\u5408\u4e30\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHEFENGSANNIANCHIYOUHUNHE"}, "009557": {"suoxie": "SWLXCYBLHJXGPA", "name": "\u7533\u4e07\u83f1\u4fe1\u521b\u4e1a\u677f\u91cf\u5316\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINCHUANGYEBANLIANGHUAJINGXUANGUPIAOA"}, "009558": {"suoxie": "JSWH6GYCYQHHA", "name": "\u5609\u5b9e\u7a33\u60e06\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENHUI6GEYUECHIYOUQIHUNHEA"}, "009559": {"suoxie": "JSWH6GYCYQHHC", "name": "\u5609\u5b9e\u7a33\u60e06\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENHUI6GEYUECHIYOUQIHUNHEC"}, "009560": {"suoxie": "SYZZ13NGKHZQZS", "name": "\u4e0a\u94f6\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHU"}, "009561": {"suoxie": "BSFSYNDKZFQS", "name": "\u535a\u65f6\u5bcc\u76db\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUSHENGYINIANDINGKAIZHAIFAQISHI"}, "009562": {"suoxie": "GYQQGPQDIIMY", "name": "\u5de5\u94f6\u5168\u7403\u80a1\u7968(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "GONGYINQUANQIUGUPIAOQDIIMEIYUAN"}, "009563": {"suoxie": "GYQQGPQDIIGB", "name": "\u5de5\u94f6\u5168\u7403\u80a1\u7968(QDII)\u6e2f\u5e01", "style": "QDII", "pinyin": "GONGYINQUANQIUGUPIAOQDIIGANGBI"}, "009564": {"suoxie": "HAXFLTHHA", "name": "\u6c47\u5b89\u6d88\u8d39\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANXIAOFEILONGTOUHUNHEA"}, "009565": {"suoxie": "HAXFLTHHC", "name": "\u6c47\u5b89\u6d88\u8d39\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANXIAOFEILONGTOUHUNHEC"}, "009566": {"suoxie": "HAHYSNCYQHH", "name": "\u6c47\u5b89\u6cd3\u9633\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHONGYANGSANNIANCHIYOUQIHUNHE"}, "009567": {"suoxie": "SXZQYFYNDKZ", "name": "\u5c71\u897f\u8bc1\u5238\u88d5\u4e30\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANXIZHENGQUANYUFENGYINIANDINGKAIZHAI"}, "009568": {"suoxie": "ZSZDBWJYNCYQA", "name": "\u6d59\u5546\u667a\u591a\u5b9d\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671fA", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOBAOWENJIANYINIANCHIYOUQIA"}, "009569": {"suoxie": "ZSZDBWJYNCYQC", "name": "\u6d59\u5546\u667a\u591a\u5b9d\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671fC", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOBAOWENJIANYINIANCHIYOUQIC"}, "009570": {"suoxie": "PHJXJXHHA", "name": "\u9e4f\u534e\u5320\u5fc3\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIANGXINJINGXUANHUNHEA"}, "009571": {"suoxie": "PHJXJXHHC", "name": "\u9e4f\u534e\u5320\u5fc3\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIANGXINJINGXUANHUNHEC"}, "009572": {"suoxie": "NFYL2040SNCYHHFOF", "name": "\u5357\u65b9\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "009573": {"suoxie": "NFYL2045SNCYHHFOF", "name": "\u5357\u65b9\u517b\u80012045\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYANGLAO2045SANNIANCHIYOUHUNHEFOF"}, "009574": {"suoxie": "DBAY6GYCYQHHA", "name": "\u5fb7\u90a6\u5b89\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANYI6GEYUECHIYOUQIHUNHEA"}, "009575": {"suoxie": "DBAY6GYCYQHHC", "name": "\u5fb7\u90a6\u5b89\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DEBANGANYI6GEYUECHIYOUQIHUNHEC"}, "009576": {"suoxie": "DFHZYSNCYHH", "name": "\u4e1c\u65b9\u7ea2\u667a\u8fdc\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGZHIYUANSANNIANCHIYOUHUNHE"}, "009577": {"suoxie": "SYJYYYNDKZQ", "name": "\u4e0a\u94f6\u805a\u6c38\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUYONGYIYINIANDINGKAIZHAIQUAN"}, "009578": {"suoxie": "SYJDYYNDKZQ", "name": "\u4e0a\u94f6\u805a\u5fb7\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUDEYIYINIANDINGKAIZHAIQUAN"}, "009579": {"suoxie": "DFHXA39GYDKZQ", "name": "\u4e1c\u65b9\u7ea2\u946b\u5b8939\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGXINAN39GEYUEDINGKAIZHAIQUAN"}, "009580": {"suoxie": "ZSSZZQZQLOFD", "name": "\u62db\u5546\u53cc\u503a\u589e\u5f3a\u503a\u5238(LOF)D", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGSHUANGZHAIZENGQIANGZHAIQUANLOFD"}, "009581": {"suoxie": "GSZZ35NZJZZSA", "name": "\u56fd\u5bff\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUZHONGZHAI35NIANZHENGJINZHAIZHISHUA"}, "009582": {"suoxie": "GSZZ35NZJZZSC", "name": "\u56fd\u5bff\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUZHONGZHAI35NIANZHENGJINZHAIZHISHUC"}, "009583": {"suoxie": "CHAY87GYDKZ", "name": "\u6df3\u539a\u5b89\u88d587\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUANYU87GEYUEDINGKAIZHAI"}, "009584": {"suoxie": "HF63GYDQKFZQ", "name": "\u534e\u5bcc63\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFU63GEYUEDINGQIKAIFANGZHAIQUAN"}, "009585": {"suoxie": "ZXJTWF63GYDKZ", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u4e3063\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENFENG63GEYUEDINGKAIZHAI"}, "009586": {"suoxie": "FGFQBHBB", "name": "\u5bcc\u56fd\u5bcc\u94b1\u5305\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOFUQIANBAOHUOBIB"}, "009587": {"suoxie": "GSABRH66GYDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u745e\u548c66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAORUIHE66GEYUEDINGKAIZHAI"}, "009588": {"suoxie": "HTFXJBHBB", "name": "\u6c47\u6dfb\u5bcc\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUXIANJINBAOHUOBIB"}, "009589": {"suoxie": "HTFXJBHBC", "name": "\u6c47\u6dfb\u5bcc\u73b0\u91d1\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUXIANJINBAOHUOBIC"}, "009590": {"suoxie": "DFSSLHPZHHC", "name": "\u4e1c\u65b9\u76db\u4e16\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGSHENGSHILINGHUOPEIZHIHUNHEC"}, "009591": {"suoxie": "BSYJJXCYQHHA", "name": "\u535a\u65f6\u7814\u7a76\u7cbe\u9009\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUJINGXUANCHIYOUQIHUNHEA"}, "009592": {"suoxie": "BSYJJXCYQHHC", "name": "\u535a\u65f6\u7814\u7a76\u7cbe\u9009\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUJINGXUANCHIYOUQIHUNHEC"}, "009593": {"suoxie": "GTZZ13NGKZA", "name": "\u56fd\u6cf0\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIZHONGZHAI13NIANGUOKAIZHAIA"}, "009594": {"suoxie": "GTZZ13NGKZC", "name": "\u56fd\u6cf0\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIZHONGZHAI13NIANGUOKAIZHAIC"}, "009595": {"suoxie": "SXZQYSYNDKHH", "name": "\u5c71\u897f\u8bc1\u5238\u88d5\u76db\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANXIZHENGQUANYUSHENGYINIANDINGKAIHUNHE"}, "009596": {"suoxie": "TKCXCZHHA", "name": "\u6cf0\u5eb7\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGCHUANGXINCHENGCHANGHUNHEA"}, "009597": {"suoxie": "TKCXCZHHC", "name": "\u6cf0\u5eb7\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGCHUANGXINCHENGCHANGHUNHEC"}, "009598": {"suoxie": "JSCCKCSNDKHH", "name": "\u666f\u987a\u957f\u57ce\u79d1\u521b\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINGSHUNCHANGCHENGKECHUANGSANNIANDINGKAIHUNHE"}, "009599": {"suoxie": "JSZJCZZQ", "name": "\u5609\u5b9e\u81f4\u5609\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIJIACHUNZHAIZHAIQUAN"}, "009600": {"suoxie": "JSAZYNDKZCZ", "name": "\u5609\u5b9e\u5b89\u6cfd\u4e00\u5e74\u5b9a\u5f00\u503a\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIANZEYINIANDINGKAIZHAICHUNZHAI"}, "009601": {"suoxie": "ZSKJDL3GYGPA", "name": "\u62db\u5546\u79d1\u6280\u52a8\u529b3\u4e2a\u6708\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGKEJIDONGLI3GEYUEGUPIAOA"}, "009602": {"suoxie": "ZSKJDL3GYGPC", "name": "\u62db\u5546\u79d1\u6280\u52a8\u529b3\u4e2a\u6708\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGKEJIDONGLI3GEYUEGUPIAOC"}, "009603": {"suoxie": "GJHXDZZQE", "name": "\u56fd\u91d1\u60e0\u946b\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINHUIXINDUANZHAIZHAIQUANE"}, "009604": {"suoxie": "GJHYCZE", "name": "\u56fd\u91d1\u60e0\u76c8\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIYINGCHUNZHAIE"}, "009605": {"suoxie": "AXYSYNDKZQ", "name": "\u5b89\u4fe1\u6c38\u987a\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGSHUNYINIANDINGKAIZHAIQUAN"}, "009606": {"suoxie": "CXWJJXHHA", "name": "\u957f\u4fe1\u7a33\u5065\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANJINGXUANHUNHEA"}, "009607": {"suoxie": "CXWJJXHHC", "name": "\u957f\u4fe1\u7a33\u5065\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANJINGXUANHUNHEC"}, "009608": {"suoxie": "GFZZ500ZSZQA", "name": "\u5e7f\u53d1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ZHISHUZENGQIANGA"}, "009609": {"suoxie": "GFZZ500ZSZQC", "name": "\u5e7f\u53d1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ZHISHUZENGQIANGC"}, "009610": {"suoxie": "THYLZQC", "name": "\u5929\u5f18\u6c38\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGYONGLIZHAIQUANC"}, "009611": {"suoxie": "XQHXYNCYHHA", "name": "\u5174\u5168\u6c47\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANHUIXIANGYINIANCHIYOUHUNHEA"}, "009612": {"suoxie": "XQHXYNCYHHC", "name": "\u5174\u5168\u6c47\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANHUIXIANGYINIANCHIYOUHUNHEC"}, "009613": {"suoxie": "SYZZ500ZSZQXA", "name": "\u4e0a\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a\u578bA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGYINZHONGZHENG500ZHISHUZENGQIANGXINGA"}, "009614": {"suoxie": "SYZZ500ZSZQXC", "name": "\u4e0a\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a\u578bC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGYINZHONGZHENG500ZHISHUZENGQIANGXINGC"}, "009615": {"suoxie": "NF02NGKZA", "name": "\u5357\u65b90-2\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG02NIANGUOKAIZHAIA"}, "009616": {"suoxie": "NF02NGKZC", "name": "\u5357\u65b90-2\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG02NIANGUOKAIZHAIC"}, "009617": {"suoxie": "DXXLZQC", "name": "\u4e1c\u5174\u5174\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGXINGXINGLIZHAIQUANC"}, "009618": {"suoxie": "JYQHHHA", "name": "\u4ea4\u94f6\u542f\u6c47\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQIHUIHUNHEA"}, "009619": {"suoxie": "BSNXXFZTHHA", "name": "\u535a\u65f6\u5973\u6027\u6d88\u8d39\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHINVXINGXIAOFEIZHUTIHUNHEA"}, "009620": {"suoxie": "BSNXXFZTHHC", "name": "\u535a\u65f6\u5973\u6027\u6d88\u8d39\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHINVXINGXIAOFEIZHUTIHUNHEC"}, "009621": {"suoxie": "ZOXYWJ6GYHHA", "name": "\u4e2d\u6b27\u5fc3\u76ca\u7a33\u50656\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUXINYIWENJIAN6GEYUEHUNHEA"}, "009622": {"suoxie": "ZOXYWJ6GYHHC", "name": "\u4e2d\u6b27\u5fc3\u76ca\u7a33\u50656\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUXINYIWENJIAN6GEYUEHUNHEC"}, "009623": {"suoxie": "CCCXQDHH", "name": "\u957f\u57ce\u521b\u65b0\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGCHUANGXINQUDONGHUNHE"}, "009624": {"suoxie": "AXWJAEFDKHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u963f\u5c14\u6cd5\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ANXINWENJIANAERFADINGKAIHUNHEC"}, "009625": {"suoxie": "THZZ35NZJZ", "name": "\u5929\u5f18\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGZHONGZHAI35NIANZHENGJINZHAI"}, "009626": {"suoxie": "PYASYL2040SNCYHHFOF", "name": "\u6d66\u94f6\u5b89\u76db\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PUYINANSHENGYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "009627": {"suoxie": "THRXSGYDKA", "name": "\u5929\u5f18\u777f\u65b0\u4e09\u4e2a\u6708\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGRUIXINSANGEYUEDINGKAIA"}, "009628": {"suoxie": "THRXSGYDKC", "name": "\u5929\u5f18\u777f\u65b0\u4e09\u4e2a\u6708\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGRUIXINSANGEYUEDINGKAIC"}, "009630": {"suoxie": "PYASESGZRTZHHA", "name": "\u6d66\u94f6\u5b89\u76dbESG\u8d23\u4efb\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGESGZERENTOUZIHUNHEA"}, "009631": {"suoxie": "PYASESGZRTZHHC", "name": "\u6d66\u94f6\u5b89\u76dbESG\u8d23\u4efb\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGESGZERENTOUZIHUNHEC"}, "009632": {"suoxie": "PYASPJ87GYDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u560987\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUJIA87GEYUEDINGKAIZHAIA"}, "009633": {"suoxie": "PYASPJ87GYDKZC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u560987\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUJIA87GEYUEDINGKAIZHAIC"}, "009634": {"suoxie": "PHARLNCYQHHA", "name": "\u9e4f\u534e\u5b89\u777f\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRUILIANGNIANCHIYOUQIHUNHEA"}, "009635": {"suoxie": "PHARLNCYQHHC", "name": "\u9e4f\u534e\u5b89\u777f\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRUILIANGNIANCHIYOUQIHUNHEC"}, "009636": {"suoxie": "HTBRJQYXA", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIYOUXUANA"}, "009637": {"suoxie": "ZSXYTLZQLOFC", "name": "\u62db\u5546\u4fe1\u7528\u6dfb\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXINYONGTIANLIZHAIQUANLOFC"}, "009638": {"suoxie": "HTZJZZG12GYGDZFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u5468\u5468\u8d2d12\u4e2a\u6708\u6eda\u52a8\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINZHOUZHOUGOU12GEYUEGUNDONGZHAIFAQIA"}, "009639": {"suoxie": "HTZJZZG12GYGDZFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u5468\u5468\u8d2d12\u4e2a\u6708\u6eda\u52a8\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINZHOUZHOUGOU12GEYUEGUNDONGZHAIFAQIC"}, "009640": {"suoxie": "ZYZQYXHYLTHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u4f18\u9009\u884c\u4e1a\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANYOUXUANHANGYELONGTOUHUNHEA"}, "009641": {"suoxie": "ZYZQYXHYLTHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u4f18\u9009\u884c\u4e1a\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANYOUXUANHANGYELONGTOUHUNHEC"}, "009642": {"suoxie": "FGRLCZYNDKZFQS", "name": "\u5bcc\u56fd\u8363\u5229\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUORONGLICHUNZHAIYINIANDINGKAIZHAIFAQISHI"}, "009643": {"suoxie": "JSZXYNDQCZZQ", "name": "\u5609\u5b9e\u81f4\u4fe1\u4e00\u5e74\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIXINYINIANDINGQICHUNZHAIZHAIQUAN"}, "009644": {"suoxie": "DFAEFYSCYHHA", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAYOUSHICHANYEHUNHEA"}, "009645": {"suoxie": "DFAEFYSCYHHC", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAYOUSHICHANYEHUNHEC"}, "009646": {"suoxie": "NFHXCZHHA", "name": "\u5357\u65b9\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHEXINCHENGZHANGHUNHEA"}, "009647": {"suoxie": "NFHXCZHHC", "name": "\u5357\u65b9\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHEXINCHENGZHANGHUNHEC"}, "009648": {"suoxie": "ZORDDQKFHHC", "name": "\u4e2d\u6b27\u777f\u8fbe\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURUIDADINGQIKAIFANGHUNHEC"}, "009649": {"suoxie": "JSJXPHHHA", "name": "\u5609\u5b9e\u7cbe\u9009\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIJINGXUANPINGHENGHUNHEA"}, "009650": {"suoxie": "JSJXPHHHC", "name": "\u5609\u5b9e\u7cbe\u9009\u5e73\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIJINGXUANPINGHENGHUNHEC"}, "009651": {"suoxie": "HFTCZZXHHA", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGZHENXUANHUNHEA"}, "009652": {"suoxie": "HFTCZZXHHC", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGZHENXUANHUNHEC"}, "009653": {"suoxie": "DCFXHBHHA", "name": "\u5927\u6210\u4e30\u4eab\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGFENGXIANGHUIBAOHUNHEA"}, "009654": {"suoxie": "DCFXHBHHC", "name": "\u5927\u6210\u4e30\u4eab\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGFENGXIANGHUIBAOHUNHEC"}, "009655": {"suoxie": "GYZYZDZA", "name": "\u5de5\u94f6\u5c0a\u76ca\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNYIZHONGDUANZHAIA"}, "009656": {"suoxie": "HAZZ15NGKHZA", "name": "\u534e\u5b89\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u884c\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI15NIANGUOKAIHANGZHAIA"}, "009657": {"suoxie": "HAZZ15NGKHZC", "name": "\u534e\u5b89\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u884c\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGZHAI15NIANGUOKAIHANGZHAIC"}, "009658": {"suoxie": "HFJXZXPDBDCLGPA", "name": "\u6c47\u4e30\u664b\u4fe1\u4e2d\u5c0f\u76d8\u4f4e\u6ce2\u52a8\u7b56\u7565\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINZHONGXIAOPANDIBODONGCELUEGUPIAOA"}, "009659": {"suoxie": "MSJYXDNYNDKHHA", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u52a8\u80fd\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINDONGNENGYINIANDINGKAIHUNHEA"}, "009660": {"suoxie": "MSJYXDNYNDKHHC", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u52a8\u80fd\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINDONGNENGYINIANDINGKAIHUNHEC"}, "009661": {"suoxie": "PAYJRXHHA", "name": "\u5e73\u5b89\u7814\u7a76\u777f\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANJIURUIXUANHUNHEA"}, "009662": {"suoxie": "PAYJRXHHC", "name": "\u5e73\u5b89\u7814\u7a76\u777f\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANJIURUIXUANHUNHEC"}, "009663": {"suoxie": "HTZJKC3NFBHHA", "name": "\u534e\u6cf0\u7d2b\u91d1\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIZIJINKECHUANG3NIANFENGBIHUNHEA"}, "009664": {"suoxie": "HTFYLJJCZYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u533b\u7597\u79ef\u6781\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYILIAOJIJICHENGZZHANGYINIANCHIYOUHUNHEA"}, "009665": {"suoxie": "HTFYLJJCZYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u533b\u7597\u79ef\u6781\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYILIAOJIJICHENGZHANGYINIANCHIYOUHUNHEC"}, "009666": {"suoxie": "XQHX88GYDKZQ", "name": "\u5174\u5168\u6052\u796588\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGXIANG88GEYUEDINGKAIZHAIQUAN"}, "009667": {"suoxie": "PHAQHHA", "name": "\u9e4f\u534e\u5b89\u5e86\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANQINGHUNHEA"}, "009668": {"suoxie": "PHAQHHC", "name": "\u9e4f\u534e\u5b89\u5e86\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANQINGHUNHEC"}, "009669": {"suoxie": "CXLHJZQDHHC", "name": "\u957f\u4fe1\u91cf\u5316\u4ef7\u503c\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINLIANGHUAJIAZHIQUDONGHUNHEC"}, "009670": {"suoxie": "DFHYFCZZQ", "name": "\u4e1c\u65b9\u7ea2\u76ca\u4e30\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGYIFENGCHUNZHAIZHAIQUAN"}, "009671": {"suoxie": "PAHZHHA", "name": "\u5e73\u5b89\u6052\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGZEHUNHEA"}, "009672": {"suoxie": "PAHZHHC", "name": "\u5e73\u5b89\u6052\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGZEHUNHEC"}, "009673": {"suoxie": "JHHK63GYDKZQA", "name": "\u5609\u5408\u6167\u5eb763\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEHUIKANG63GEYUEDINGKAIZHAIQUANA"}, "009674": {"suoxie": "JHHK63GYDKZQC", "name": "\u5609\u5408\u6167\u5eb763\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEHUIKANG63GEYUEDINGKAIZHAIQUANC"}, "009675": {"suoxie": "ZRRHSXYNDKZQA", "name": "\u4e2d\u878d\u878d\u6167\u53cc\u6b23\u4e00\u5e74\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGRONGRONGHUISHUANGXINYINIANDINGKAIZHAIQUANA"}, "009676": {"suoxie": "ZRRHSXYNDKZQC", "name": "\u4e2d\u878d\u878d\u6167\u53cc\u6b23\u4e00\u5e74\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGRONGRONGHUISHUANGXINYINIANDINGKAIZHAIQUANC"}, "009677": {"suoxie": "ZSZDYWJYNCYQHHA", "name": "\u6d59\u5546\u667a\u591a\u76ca\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOYIWENJIANYINIANCHIYOUQIHUNHEA"}, "009678": {"suoxie": "ZSZDYWJYNCYQHHC", "name": "\u6d59\u5546\u667a\u591a\u76ca\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOYIWENJIANYINIANCHIYOUQIHUNHEC"}, "009679": {"suoxie": "ZSHL39GYDKZQ", "name": "\u6d59\u5546\u60e0\u968639\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUILONG39GEYUEDINGKAIZHAIQUAN"}, "009681": {"suoxie": "NFCXJXYNDKHHA", "name": "\u5357\u65b9\u521b\u65b0\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINJINGXUANYINIANDINGKAIHUNHEA"}, "009682": {"suoxie": "NFCXJXYNDKHHC", "name": "\u5357\u65b9\u521b\u65b0\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINJINGXUANYINIANDINGKAIHUNHEC"}, "009683": {"suoxie": "HTFCXZZYNDKHHA", "name": "\u6c47\u6dfb\u5bcc\u521b\u65b0\u589e\u957f\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHUANGXINZENGZHANGYINIANDINGKAIHUNHEA"}, "009684": {"suoxie": "HTFCXZZYNDKHHC", "name": "\u6c47\u6dfb\u5bcc\u521b\u65b0\u589e\u957f\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHUANGXINZENGZHANGYINIANDINGKAIHUNHEC"}, "009685": {"suoxie": "JSCCJTBLYNDKZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u5b9d\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIBAOLIYINIANDINGKAIZHAI"}, "009686": {"suoxie": "HXPLYNDKHHA", "name": "\u534e\u590f\u78d0\u5229\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAPANLIYINIANDINGKAIHUNHEA"}, "009687": {"suoxie": "HXPLYNDKHHC", "name": "\u534e\u590f\u78d0\u5229\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAPANLIYINIANDINGKAIHUNHEC"}, "009688": {"suoxie": "WJXDLYYGYNGDHH", "name": "\u4e07\u5bb6\u946b\u52a8\u529b\u6708\u6708\u8d2d\u4e00\u5e74\u6eda\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAXINDONGLIYUEYUEGOUYINIANGUNDONGHUNHE"}, "009689": {"suoxie": "YFDRJHHA", "name": "\u6613\u65b9\u8fbe\u745e\u9526\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIJINHUNHEA"}, "009690": {"suoxie": "YFDRJHHC", "name": "\u6613\u65b9\u8fbe\u745e\u9526\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIJINHUNHEC"}, "009691": {"suoxie": "GTHYHHA", "name": "\u56fd\u6cf0\u6d69\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHAOYIHUNHEA"}, "009692": {"suoxie": "GTHYHHC", "name": "\u56fd\u6cf0\u6d69\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHAOYIHUNHEC"}, "009693": {"suoxie": "FGJJCZYNDKHH", "name": "\u5bcc\u56fd\u79ef\u6781\u6210\u957f\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIJICHENGZHANGYINIANDINGKAIHUNHE"}, "009695": {"suoxie": "ZSCZJXYNDKHHA", "name": "\u62db\u5546\u6210\u957f\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGCHENGZHANGJINGXUANYINIANDINGKAIHUNHEA"}, "009696": {"suoxie": "ZSCZJXYNDKHHC", "name": "\u62db\u5546\u6210\u957f\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGCHENGZHANGJINGXUANYINIANDINGKAIHUNHEC"}, "009697": {"suoxie": "HXCZJX6GYDKHHA", "name": "\u534e\u590f\u6210\u957f\u7cbe\u90096\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGJINGXUAN6GEYUEDINGKAIHUNHEA"}, "009698": {"suoxie": "HXCZJX6GYDKHHC", "name": "\u534e\u590f\u6210\u957f\u7cbe\u90096\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGJINGXUAN6GEYUEDINGKAIHUNHEC"}, "009699": {"suoxie": "CXPR87GYDKZQ", "name": "\u957f\u4fe1\u6d66\u745e87\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINPURUI87GEYUEDINGKAIZHAIQUAN"}, "009700": {"suoxie": "CJTLHHA", "name": "\u957f\u6c5f\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGJIANGTIANLIHUNHEA"}, "009701": {"suoxie": "CJTLHHC", "name": "\u957f\u6c5f\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGJIANGTIANLIHUNHEC"}, "009702": {"suoxie": "PHZZ13NNFHZQZSA", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHUA"}, "009703": {"suoxie": "PHZZ13NNFHZQZSC", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13NIANNONGFAHANGZHAIQUANZHISHUC"}, "009704": {"suoxie": "NFJQQDHHA", "name": "\u5357\u65b9\u666f\u6c14\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJINGQIQUDONGHUNHEA"}, "009705": {"suoxie": "NFJQQDHHC", "name": "\u5357\u65b9\u666f\u6c14\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJINGQIQUDONGHUNHEC"}, "009706": {"suoxie": "MSJYCZHHHC", "name": "\u6c11\u751f\u52a0\u94f6\u57ce\u9547\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINCHENGZHENHUAHUNHEC"}, "009707": {"suoxie": "GYXXZZHHA", "name": "\u5de5\u94f6\u65b0\u5174\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINXINGZHIZAOHUNHEA"}, "009708": {"suoxie": "GYXXZZHHC", "name": "\u5de5\u94f6\u65b0\u5174\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINXINGZHIZAOHUNHEC"}, "009709": {"suoxie": "MSJYCLJXHHC", "name": "\u6c11\u751f\u52a0\u94f6\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINCELUEJINGXUANHUNHEC"}, "009710": {"suoxie": "NDXSLHPZHHC", "name": "\u8bfa\u5fb7\u65b0\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODEXINSHENGLINGHUOPEIZHIHUNHEC"}, "009711": {"suoxie": "ZSTS78GYDKZ", "name": "\u62db\u5546\u6dfb\u76db78\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANSHENG78GEYUEDINGKAIZHAI"}, "009712": {"suoxie": "XDAYHGJHBB", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7ba1\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUIGUANJIAHUOBIB"}, "009713": {"suoxie": "XDAYHGJHBD", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6167\u7ba1\u5bb6\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "XINDAAOYINHUIGUANJIAHUOBID"}, "009714": {"suoxie": "HAJYJXHH", "name": "\u534e\u5b89\u805a\u4f18\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUYOUJINGXUANHUNHE"}, "009715": {"suoxie": "HTFCLZZLNFBHH", "name": "\u6c47\u6dfb\u5bcc\u7b56\u7565\u589e\u957f\u4e24\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUCELUEZENGZHANGLIANGNIANFENGBIHUNHE"}, "009716": {"suoxie": "BSHSCYQHHA", "name": "\u535a\u65f6\u6052\u76db\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGSHENGCHIYOUQIHUNHEA"}, "009717": {"suoxie": "BSHSCYQHHC", "name": "\u535a\u65f6\u6052\u76db\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGSHENGCHIYOUQIHUNHEC"}, "009718": {"suoxie": "ZSZHYNDQKFHHA", "name": "\u62db\u5546\u589e\u6d69\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGZENGHAOYINIANDINGQIKAIFANGHUNHEA"}, "009719": {"suoxie": "ZSZHYNDQKFHHC", "name": "\u62db\u5546\u589e\u6d69\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGZENGHAOYINIANDINGQIKAIFANGHUNHEC"}, "009720": {"suoxie": "MSJYJQHYHHC", "name": "\u6c11\u751f\u52a0\u94f6\u666f\u6c14\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINJINGQIHANGYEHUNHEC"}, "009721": {"suoxie": "PAZZ15NZCXJRZA", "name": "\u5e73\u5b89\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIA"}, "009722": {"suoxie": "PAZZ15NZCXJRZC", "name": "\u5e73\u5b89\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIC"}, "009725": {"suoxie": "DFHYZZXYNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u4f18\u8d28\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGYOUZHIZHENXUANYINIANCHIYOUHUNHEA"}, "009726": {"suoxie": "ZSZZ500DQZZSZQA", "name": "\u62db\u5546\u4e2d\u8bc1500\u7b49\u6743\u91cd\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG500DENGQUANZHONGZHISHUZENGQIANGA"}, "009727": {"suoxie": "ZSZZ500DQZZSZQC", "name": "\u62db\u5546\u4e2d\u8bc1500\u7b49\u6743\u91cd\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENG500DENGQUANZHONGZHISHUZENGQIANGC"}, "009728": {"suoxie": "ZYZQATZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHENGQUANANTAIZHAIQUANA"}, "009729": {"suoxie": "ZYZQATZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZHENGQUANANTAIZHAIQUANC"}, "009730": {"suoxie": "ZXBCAXHBZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5b89\u946b\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINBAOCHENGANXINHUIBAOZHAIQUANA"}, "009731": {"suoxie": "ZXBCAXHBZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5b89\u946b\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINBAOCHENGANXINHUIBAOZHAIQUANC"}, "009732": {"suoxie": "XYWT66GYDKZQ", "name": "\u5174\u4e1a\u7a33\u6cf066\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEWENTAI66GEYUEDINGKAIZHAIQUAN"}, "009733": {"suoxie": "CJHXGGTDXFJXGPA", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u5927\u6d88\u8d39\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGDAXIAOFEIJINGXUANGUPIAOA"}, "009734": {"suoxie": "CJHXGGTDXFJXGPC", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u5927\u6d88\u8d39\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGDAXIAOFEIJINGXUANGUPIAOC"}, "009735": {"suoxie": "THZQHBZQE", "name": "\u5929\u5f18\u589e\u5f3a\u56de\u62a5\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGZENGQIANGHUIBAOZHAIQUANE"}, "009736": {"suoxie": "HTFWJSYHHA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANSHOUYIHUNHEA"}, "009737": {"suoxie": "HTFWJSYHHC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANSHOUYIHUNHEC"}, "009738": {"suoxie": "GLHA63GYDKZ", "name": "\u683c\u6797\u6cd3\u5b8963\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGAN63GEYUEDINGKAIZHAI"}, "009740": {"suoxie": "BSYJZXCYQHHA", "name": "\u535a\u65f6\u7814\u7a76\u81fb\u9009\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUZHENXUANCHIYOUQIHUNHEA"}, "009741": {"suoxie": "BSYJZXCYQHHC", "name": "\u535a\u65f6\u7814\u7a76\u81fb\u9009\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUZHENXUANCHIYOUQIHUNHEC"}, "009742": {"suoxie": "PHZZ03NAAYXXYZA", "name": "\u9e4f\u534e\u4e2d\u503a-0-3\u5e74AA+\u4f18\u9009\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI03NIANAAYOUXUANXINYONGZHAIA"}, "009743": {"suoxie": "PHZZ03NAAYXXYZC", "name": "\u9e4f\u534e\u4e2d\u503a-0-3\u5e74AA+\u4f18\u9009\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI03NIANAAYOUXUANXINYONGZHAIC"}, "009744": {"suoxie": "YDXJBHBB", "name": "\u82f1\u5927\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINGDAXIANJINBAOHUOBIB"}, "009745": {"suoxie": "PHZZ13YHPJAAAZSA", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u9690\u542b\u8bc4\u7ea7AAA\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13YINHANPINGJIAAAZHISHUA"}, "009746": {"suoxie": "PHZZ13YHPJAAAZSC", "name": "\u9e4f\u534e\u4e2d\u503a1-3\u9690\u542b\u8bc4\u7ea7AAA\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHAI13YINHANPINGJIAAAZHISHUC"}, "009747": {"suoxie": "ZKWTWXCZJXHHC", "name": "\u4e2d\u79d1\u6c83\u571f\u6c83\u946b\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGKEWOTUWOXINCHENGZHANGJINGXUANHUNHEC"}, "009748": {"suoxie": "HFJXHACZ63GYDKZ", "name": "\u6c47\u4e30\u664b\u4fe1\u60e0\u5b89\u7eaf\u503a63\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIFENGJINXINHUIANCHUNZHAI63GEYUEDINGKAIZHAI"}, "009749": {"suoxie": "XBLDZT86GYDKZ", "name": "\u897f\u90e8\u5229\u5f97\u5c0a\u6cf086\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEZUNTAI86GEYUEDINGKAIZHAI"}, "009750": {"suoxie": "HAJZLCHHA", "name": "\u6c47\u5b89\u4ef7\u503c\u84dd\u7b79\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANJIAZHILANCHOUHUNHEA"}, "009751": {"suoxie": "HAJZLCHHC", "name": "\u6c47\u5b89\u4ef7\u503c\u84dd\u7b79\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANJIAZHILANCHOUHUNHEC"}, "009752": {"suoxie": "DMLDYXZQA", "name": "\u5927\u6469\u7075\u52a8\u4f18\u9009\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DAMOLINGDONGYOUXUANZHAIQUANA"}, "009753": {"suoxie": "ZOMYWJLNHHA", "name": "\u4e2d\u6b27\u7f8e\u76ca\u7a33\u5065\u4e24\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUMEIYIWENJIANLIANGNIANHUNHEA"}, "009754": {"suoxie": "ZOMYWJLNHHC", "name": "\u4e2d\u6b27\u7f8e\u76ca\u7a33\u5065\u4e24\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUMEIYIWENJIANLIANGNIANHUNHEC"}, "009755": {"suoxie": "JSCCAXHBHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u946b\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANXINHUIBAOHUNHEC"}, "009756": {"suoxie": "HBBLDKZQ", "name": "\u534e\u5b9d\u5b9d\u5229\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOLIDINGKAIZHAIQUAN"}, "009757": {"suoxie": "HBZZ13NGKHZQZS", "name": "\u534e\u5b9d\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHU"}, "009758": {"suoxie": "FGKZZC", "name": "\u5bcc\u56fd\u53ef\u8f6c\u503aC", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "FUGUOKEZHUANZHAIC"}, "009759": {"suoxie": "PYCA66GYDKZA", "name": "\u9e4f\u626c\u6df3\u5b8966\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNAN66GEYUEDINGKAIZHAIA"}, "009760": {"suoxie": "PYCA66GYDKZC", "name": "\u9e4f\u626c\u6df3\u5b8966\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNAN66GEYUEDINGKAIZHAIC"}, "009761": {"suoxie": "GDZH87GYDKZ", "name": "\u5149\u5927\u5c0a\u540887\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAZUNHE87GEYUEDINGKAIZHAI"}, "009762": {"suoxie": "GJGXFQC", "name": "\u56fd\u91d1\u56fd\u946b\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOJINGUOXINFAQIC"}, "009763": {"suoxie": "HSHYZQA", "name": "\u60e0\u5347\u548c\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUISHENGHEYUEZHAIQUANA"}, "009764": {"suoxie": "HSHYZQC", "name": "\u60e0\u5347\u548c\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUISHENGHEYUEZHAIQUANC"}, "009765": {"suoxie": "HSHX88GYDKZ", "name": "\u60e0\u5347\u548c\u716688\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEXU88GEYUEDINGKAIZHAI"}, "009766": {"suoxie": "AXPWSL3GYCYHHA", "name": "\u5b89\u4fe1\u5e73\u7a33\u53cc\u52293\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINPINGWENSHUANGLI3GEYUECHIYOUHUNHEA"}, "009767": {"suoxie": "AXPWSL3GYCYHHC", "name": "\u5b89\u4fe1\u5e73\u7a33\u53cc\u52293\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINPINGWENSHUANGLI3GEYUECHIYOUHUNHEC"}, "009770": {"suoxie": "YDAX66GYDQKFZQ", "name": "\u82f1\u5927\u5b89\u946b66\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDAANXIN66GEYUEDINGQIKAIFANGZHAIQUAN"}, "009771": {"suoxie": "HAJHCZZQC", "name": "\u6c47\u5b89\u5609\u6c47\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANJIAHUICHUNZHAIZHAIQUANC"}, "009772": {"suoxie": "JSPBGKZ15NZSA", "name": "\u5609\u5b9e\u5f6d\u535a\u56fd\u5f00\u503a1-5\u5e74\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIPENGBOGUOKAIZHAI15NIANZHISHUA"}, "009773": {"suoxie": "JSPBGKZ15NZSC", "name": "\u5609\u5b9e\u5f6d\u535a\u56fd\u5f00\u503a1-5\u5e74\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIPENGBOGUOKAIZHAI15NIANZHISHUC"}, "009774": {"suoxie": "CTZGYXHBYNCYQHH", "name": "\u8d22\u901a\u8d44\u7ba1\u4f18\u9009\u56de\u62a5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANYOUXUANHUIBAOYINIANCHIYOUQIHUNHE"}, "009775": {"suoxie": "HFJXZXPDBDCLGPC", "name": "\u6c47\u4e30\u664b\u4fe1\u4e2d\u5c0f\u76d8\u4f4e\u6ce2\u52a8\u7b56\u7565\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINZHONGXIAOPANDIBODONGCELUEGUPIAOC"}, "009776": {"suoxie": "ZOAEFHHA", "name": "\u4e2d\u6b27\u963f\u5c14\u6cd5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUAERFAHUNHEA"}, "009777": {"suoxie": "ZOAEFHHC", "name": "\u4e2d\u6b27\u963f\u5c14\u6cd5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUAERFAHUNHEC"}, "009778": {"suoxie": "CXXFSJHHA", "name": "\u957f\u4fe1\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINXIAOFEISHENGJIHUNHEA"}, "009779": {"suoxie": "CXXFSJHHC", "name": "\u957f\u4fe1\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINXIAOFEISHENGJIHUNHEC"}, "009780": {"suoxie": "DBRZ86GYDKZ", "name": "\u5fb7\u90a6\u9510\u6cfd86\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIZE86GEYUEDINGKAIZHAI"}, "009781": {"suoxie": "NFCYYSLNHHC", "name": "\u5357\u65b9\u4ea7\u4e1a\u4f18\u52bf\u4e24\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHANYEYOUSHILIANGNIANHUNHEC"}, "009782": {"suoxie": "FGXQHB12GYCYQHHA", "name": "\u5bcc\u56fd\u5174\u6cc9\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINGQUANHUIBAO12GEYUECHIYOUQIHUNHEA"}, "009783": {"suoxie": "FGXQHB12GYCYQHHC", "name": "\u5bcc\u56fd\u5174\u6cc9\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINGQUANHUIBAO12GEYUECHIYOUQIHUNHEC"}, "009784": {"suoxie": "AXZXTLLLZA", "name": "\u5b89\u4fe1\u5c0a\u4eab\u6dfb\u5229\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZUNXIANGTIANLILILVZHAIA"}, "009785": {"suoxie": "AXZXTLLLZC", "name": "\u5b89\u4fe1\u5c0a\u4eab\u6dfb\u5229\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZUNXIANGTIANLILILVZHAIC"}, "009786": {"suoxie": "HAJY07NJRZDKZ", "name": "\u534e\u5b89\u9526\u6e900-7\u5e74\u91d1\u878d\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANJINYUAN07NIANJINRONGZHAIDINGKAIZHAI"}, "009787": {"suoxie": "PHJHDZC3GYCYQHHFOF", "name": "\u9e4f\u534e\u805a\u5408\u591a\u8d44\u4ea73\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAJUHEDUOZICHAN3GEYUECHIYOUQIHUNHEFOF"}, "009788": {"suoxie": "QHKYHZLNDKZQ", "name": "\u524d\u6d77\u5f00\u6e90\u60e0\u6cfd\u4e24\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANHUIZELIANGNIANDINGKAIZHAIQUAN"}, "009789": {"suoxie": "FADKJCXHH", "name": "\u5bcc\u5b89\u8fbe\u79d1\u6280\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDAKEJICHUANGXINHUNHE"}, "009790": {"suoxie": "GSABZJBHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u589e\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOZENGJINBAOHUOBIB"}, "009791": {"suoxie": "ZOCYBLNDKHHC", "name": "\u4e2d\u6b27\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHUANGYEBANLIANGNIANDINGKAIHUNHEC"}, "009792": {"suoxie": "GYRYZQA", "name": "\u5de5\u94f6\u745e\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIYIZHAIQUANA"}, "009793": {"suoxie": "GYRYZQC", "name": "\u5de5\u94f6\u745e\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIYIZHAIQUANC"}, "009794": {"suoxie": "TPZXYNDKGP", "name": "\u592a\u5e73\u667a\u9009\u4e00\u5e74\u5b9a\u5f00\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGZHIXUANYINIANDINGKAIGUPIAO"}, "009795": {"suoxie": "JSYJJXLNCYQHH", "name": "\u5609\u5b9e\u8fdc\u89c1\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYUANJIANJINGXUANLIANGNIANCHIYOUQIHUNHE"}, "009796": {"suoxie": "DCHXYNCYHHA", "name": "\u5927\u6210\u6c47\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHUIXIANGYINIANCHIYOUHUNHEA"}, "009797": {"suoxie": "DCHXYNCYHHC", "name": "\u5927\u6210\u6c47\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHUIXIANGYINIANCHIYOUHUNHEC"}, "009798": {"suoxie": "DCCYBLNDKHHC", "name": "\u5927\u6210\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHUANGYEBANLIANGNIANDINGKAIHUNHEC"}, "009799": {"suoxie": "ZYZQAHSNDKZ", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u6c47\u4e09\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANHUISANNIANDINGKAIZHAI"}, "009800": {"suoxie": "CSZZJXHHA", "name": "\u957f\u76db\u5236\u9020\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGZHIZAOJINGXUANHUNHEA"}, "009801": {"suoxie": "CSZZJXHHC", "name": "\u957f\u76db\u5236\u9020\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGZHIZAOJINGXUANHUNHEC"}, "009802": {"suoxie": "DHXTSNDKZFQS", "name": "\u4e1c\u6d77\u7965\u6cf0\u4e09\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGHAIXIANGTAISANNIANDINGKAIZHAIFAQISHI"}, "009803": {"suoxie": "YFDZZ710NGKZC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAI710NIANGUOKAIZHAIC"}, "009804": {"suoxie": "GTYJYSHH", "name": "\u56fd\u6cf0\u7814\u7a76\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIYANJIUYOUSHIHUNHE"}, "009805": {"suoxie": "GTYYJKGPA", "name": "\u56fd\u6cf0\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIYIYAOJIANKANGGUPIAOA"}, "009806": {"suoxie": "DFHZYZXYNHHA", "name": "\u4e1c\u65b9\u7ea2\u62db\u76c8\u7504\u9009\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHAOYINGZHENXUANYINIANHUNHEA"}, "009807": {"suoxie": "DFHZYZXYNHHC", "name": "\u4e1c\u65b9\u7ea2\u62db\u76c8\u7504\u9009\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHAOYINGZHENXUANYINIANHUNHEC"}, "009808": {"suoxie": "YFDCXCZHH", "name": "\u6613\u65b9\u8fbe\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHUANGXINCHENGZHANGHUNHE"}, "009809": {"suoxie": "YFDHZ63GYDKZFQS", "name": "\u6613\u65b9\u8fbe\u6052\u667a63\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAHENGZHI63GEYUEDINGKAIZHAIFAQISHI"}, "009810": {"suoxie": "YFDYTYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u901a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUETONGYINIANCHIYOUQIHUNHEA"}, "009811": {"suoxie": "YFDYTYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u901a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUETONGYINIANCHIYOUQIHUNHEC"}, "009812": {"suoxie": "YFDYXYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u5174\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXINGYINIANCHIYOUQIHUNHEA"}, "009813": {"suoxie": "YFDYXYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u5174\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXINGYINIANCHIYOUQIHUNHEC"}, "009814": {"suoxie": "TDHLLY66GYDKZA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e50\u76c866\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLILEYING66GEYUEDINGKAIZHAIA"}, "009815": {"suoxie": "TDHLLY66GYDKZC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e50\u76c866\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLILEYING66GEYUEDINGKAIZHAIC"}, "009816": {"suoxie": "DMFY63GYKFZQ", "name": "\u5927\u6469\u4e30\u88d563\u4e2a\u6708\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DAMOFENGYU63GEYUEKAIFANGZHAIQUAN"}, "009817": {"suoxie": "HTHTWJJXHHA", "name": "\u7ea2\u5854\u7ea2\u571f\u7a33\u5065\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTAHONGTUWENJIANJINGXUANHUNHEA"}, "009818": {"suoxie": "HTHTWJJXHHC", "name": "\u7ea2\u5854\u7ea2\u571f\u7a33\u5065\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGTAHONGTUWENJIANJINGXUANHUNHEC"}, "009820": {"suoxie": "JSPH6GYCYQHHA", "name": "\u5609\u5b9e\u6d66\u60e06\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIPUHUI6GEYUECHIYOUQIHUNHEA"}, "009821": {"suoxie": "JSPH6GYCYQHHC", "name": "\u5609\u5b9e\u6d66\u60e06\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIPUHUI6GEYUECHIYOUQIHUNHEC"}, "009822": {"suoxie": "PHZHYNCYQHHA", "name": "\u9e4f\u534e\u62db\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZHAOHUAYINIANCHIYOUQIHUNHEA"}, "009823": {"suoxie": "PHZHYNCYQHHC", "name": "\u9e4f\u534e\u62db\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZHAOHUAYINIANCHIYOUQIHUNHEC"}, "009824": {"suoxie": "PHTLBHBB", "name": "\u9e4f\u534e\u6dfb\u5229\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUATIANLIBAOHUOBIB"}, "009826": {"suoxie": "MSJYJY6GYCYQZQA", "name": "\u6c11\u751f\u52a0\u94f6\u5bb6\u76c86\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGJIAYINJIAYING6GEYUECHIYOUQIZHAIQUANA"}, "009827": {"suoxie": "MSJYJY6GYCYQZQC", "name": "\u6c11\u751f\u52a0\u94f6\u5bb6\u76c86\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGJIAYINJIAYING6GEYUECHIYOUQIZHAIQUANC"}, "009828": {"suoxie": "RTZXSDLLHPZHHC", "name": "\u878d\u901a\u8f6c\u578b\u4e09\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGZHUANXINGSANDONGLILINGHUOPEIZHIHUNHEC"}, "009829": {"suoxie": "CCYXZQLGYCYHHA", "name": "\u957f\u57ce\u4f18\u9009\u589e\u5f3a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANZENGQIANGLIUGEYUECHIYOUHUNHEA"}, "009830": {"suoxie": "CCYXZQLGYCYHHC", "name": "\u957f\u57ce\u4f18\u9009\u589e\u5f3a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANZENGQIANGLIUGEYUECHIYOUHUNHEC"}, "009831": {"suoxie": "CCWLCZA", "name": "\u957f\u57ce\u7a33\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGWENLICHUNZHAIA"}, "009832": {"suoxie": "CCWLCZC", "name": "\u957f\u57ce\u7a33\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGWENLICHUNZHAIC"}, "009833": {"suoxie": "CJHXTB66GYDKZ", "name": "\u521b\u91d1\u5408\u4fe1\u6cf0\u535a66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINTAIBO66GEYUEDINGKAIZHAI"}, "009834": {"suoxie": "DFHXT66GYDKZ", "name": "\u4e1c\u65b9\u7ea2\u946b\u6cf066\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGXINTAI66GEYUEDINGKAIZHAI"}, "009835": {"suoxie": "RTXNYQCZTJXHHC", "name": "\u878d\u901a\u65b0\u80fd\u6e90\u6c7d\u8f66\u4e3b\u9898\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGXINNENGYUANQICHEZHUTIJINGXUANHUNHEC"}, "009836": {"suoxie": "BHHJHY87GYDKZ", "name": "\u6e24\u6d77\u6c47\u91d1\u6c47\u88d587\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOHAIHUIJINHUIYU87GEYUEDINGKAIZHAI"}, "009837": {"suoxie": "HXPRYNDKHHA", "name": "\u534e\u590f\u78d0\u9510\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAPANRUIYINIANDINGKAIHUNHEA"}, "009838": {"suoxie": "HXPRYNDKHHC", "name": "\u534e\u590f\u78d0\u9510\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAPANRUIYINIANDINGKAIHUNHEC"}, "009839": {"suoxie": "GJHF39GYDKZ", "name": "\u56fd\u91d1\u60e0\u4e3039\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOJINHUIFENG39GEYUEDINGKAIZHAI"}, "009840": {"suoxie": "DCLHJXA", "name": "\u4e1c\u8d22\u91cf\u5316\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAILIANGHUAJINGXUANA"}, "009841": {"suoxie": "DCLHJXC", "name": "\u4e1c\u8d22\u91cf\u5316\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAILIANGHUAJINGXUANC"}, "009842": {"suoxie": "DFHMJYXLNDKHH", "name": "\u4e1c\u65b9\u7ea2\u660e\u9274\u4f18\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGMINGJIANYOUXUANLIANGNIANDINGKAIHUNHE"}, "009844": {"suoxie": "HTZJFA27GYDKZQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u5b8927\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINFENGAN27GEYUEDINGKAIZHAIQUANA"}, "009845": {"suoxie": "HTZJFA27GYDKZQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u5b8927\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIZIJINFENGAN27GEYUEDINGKAIZHAIQUANC"}, "009846": {"suoxie": "GFGGTYJJZHH", "name": "\u56fd\u5bcc\u6e2f\u80a1\u901a\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUGANGGUTONGYUANJIANJIAZHIHUNHE"}, "009847": {"suoxie": "YXYFYJJXHHA", "name": "\u5706\u4fe1\u6c38\u4e30\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYANJIUJINGXUANHUNHEA"}, "009848": {"suoxie": "YXYFYJJXHHC", "name": "\u5706\u4fe1\u6c38\u4e30\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGYANJIUJINGXUANHUNHEC"}, "009849": {"suoxie": "AXWJJSYNCYQHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u805a\u7533\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINWENJIANJUSHENYINIANCHIYOUQIHUNHEA"}, "009851": {"suoxie": "SYJYY42GYDKZQ", "name": "\u4e0a\u94f6\u805a\u8fdc\u76c842\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUYUANYING42GEYUEDINGKAIZHAIQUAN"}, "009852": {"suoxie": "YHPZXFGP", "name": "\u94f6\u534e\u54c1\u8d28\u6d88\u8d39\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAPINZHIXIAOFEIGUPIAO"}, "009853": {"suoxie": "ZJYSQYHHA", "name": "\u4e2d\u52a0\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAYOUSHIQIYEHUNHEA"}, "009854": {"suoxie": "ZJYSQYHHC", "name": "\u4e2d\u52a0\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAYOUSHIQIYEHUNHEC"}, "009855": {"suoxie": "ZJXXCZHHA", "name": "\u4e2d\u52a0\u65b0\u5174\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXINXINGCHENGZHANGHUNHEA"}, "009856": {"suoxie": "ZJXXCZHHC", "name": "\u4e2d\u52a0\u65b0\u5174\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXINXINGCHENGZHANGHUNHEC"}, "009857": {"suoxie": "BSJZZXCYQHHA", "name": "\u535a\u65f6\u4ef7\u503c\u81fb\u9009\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIJIAZHIZHENXUANCHIYOUQIHUNHEA"}, "009858": {"suoxie": "BSJZZXCYQHHC", "name": "\u535a\u65f6\u4ef7\u503c\u81fb\u9009\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIJIAZHIZHENXUANCHIYOUQIHUNHEC"}, "009859": {"suoxie": "YHLXHH", "name": "\u94f6\u534e\u4e50\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUALEXIANGHUNHE"}, "009860": {"suoxie": "YFDZZYHZSLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGYINHANGZHISHULOFC"}, "009861": {"suoxie": "PHXXCZHHA", "name": "\u9e4f\u534e\u65b0\u5174\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINXINGCHENGCHANGHUNHEA"}, "009862": {"suoxie": "PHXXCZHHC", "name": "\u9e4f\u534e\u65b0\u5174\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINXINGCHENGCHANGHUNHEC"}, "009863": {"suoxie": "FGCXQSGP", "name": "\u5bcc\u56fd\u521b\u65b0\u8d8b\u52bf\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOCHUANGXINQUSHIGUPIAO"}, "009864": {"suoxie": "ZSJQYXGPA", "name": "\u62db\u5546\u666f\u6c14\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGJINGQIYOUXUANGUPIAOA"}, "009865": {"suoxie": "ZSJQYXGPC", "name": "\u62db\u5546\u666f\u6c14\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGJINGQIYOUXUANGUPIAOC"}, "009866": {"suoxie": "YYRN87GYDKZ", "name": "\u6c38\u8d62\u745e\u5b8187\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGRUINING87GEYUEDINGKAIZHAI"}, "009867": {"suoxie": "GYCXJXYNDKHHA", "name": "\u5de5\u94f6\u521b\u65b0\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGXINJINGXUANYINIANDINGKAIHUNHEA"}, "009868": {"suoxie": "GYCXJXYNDKHHC", "name": "\u5de5\u94f6\u521b\u65b0\u7cbe\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGXINJINGXUANYINIANDINGKAIHUNHEC"}, "009869": {"suoxie": "JSCYXFHHA", "name": "\u5609\u5b9e\u4ea7\u4e1a\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYEXIANFENGHUNHEA"}, "009870": {"suoxie": "JSCYXFHHC", "name": "\u5609\u5b9e\u4ea7\u4e1a\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYEXIANFENGHUNHEC"}, "009871": {"suoxie": "JSCCJRSYZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u745e\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGRUISHOUYIZHAIQUANC"}, "009872": {"suoxie": "ZOZRTZHHA", "name": "\u4e2d\u6b27\u8d23\u4efb\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZERENTOUZIHUNHEA"}, "009873": {"suoxie": "ZOZRTZHHC", "name": "\u4e2d\u6b27\u8d23\u4efb\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZERENTOUZIHUNHEC"}, "009874": {"suoxie": "JTJRLHGP", "name": "\u4e5d\u6cf0\u4e45\u777f\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIURUILIANGHUAGUPIAO"}, "009875": {"suoxie": "THZXSPYLGPA", "name": "\u5929\u5f18\u7504\u9009\u98df\u54c1\u996e\u6599\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TIANHONGZHENXUANSHIPINYINLIAOGUPIAOA"}, "009876": {"suoxie": "THZXSPYLGPC", "name": "\u5929\u5f18\u7504\u9009\u98df\u54c1\u996e\u6599\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TIANHONGZHENXUANSHIPINYINLIAOGUPIAOC"}, "009877": {"suoxie": "ZYNHQDGPA", "name": "\u4e2d\u94f6\u5185\u6838\u9a71\u52a8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINNEIHEQUDONGGUPIAOA"}, "009878": {"suoxie": "PADTJJHHA", "name": "\u5e73\u5b89\u4f4e\u78b3\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANDITANJINGJIHUNHEA"}, "009879": {"suoxie": "PADTJJHHC", "name": "\u5e73\u5b89\u4f4e\u78b3\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANDITANJINGJIHUNHEC"}, "009880": {"suoxie": "AXCZDLYNCYHH", "name": "\u5b89\u4fe1\u6210\u957f\u52a8\u529b\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINCHENGZHANGDONGLIYINIANCHIYOUHUNHE"}, "009881": {"suoxie": "GFZZYLZSLOFC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u533b\u7597\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGYILIAOZHISHULOFC"}, "009882": {"suoxie": "HRYDHXDLHHA", "name": "\u534e\u6da6\u5143\u5927\u6838\u5fc3\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAHEXINDONGLIHUNHEA"}, "009883": {"suoxie": "HRYDHXDLHHC", "name": "\u534e\u6da6\u5143\u5927\u6838\u5fc3\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAHEXINDONGLIHUNHEC"}, "009884": {"suoxie": "MSJYKNPHYLMBSNCYHHFOF", "name": "\u6c11\u751f\u52a0\u94f6\u5eb7\u5b81\u5e73\u8861\u517b\u8001\u76ee\u6807\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "MINSHENGJIAYINKANGNINGPINGHENGYANGLAOMUBIAOSANNIANCHIYOUHUNHEFOF"}, "009885": {"suoxie": "XHJQHYHHA", "name": "\u65b0\u534e\u666f\u6c14\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAJINGQIHANGYEHUNHEA"}, "009886": {"suoxie": "XHJQHYHHC", "name": "\u65b0\u534e\u666f\u6c14\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAJINGQIHANGYEHUNHEC"}, "009887": {"suoxie": "GFWJYXLGYCYQHHA", "name": "\u5e7f\u53d1\u7a33\u5065\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAWENJIANYOUXUANLIUGEYUECHIYOUQIHUNHEA"}, "009888": {"suoxie": "GFWJYXLGYCYQHHC", "name": "\u5e7f\u53d1\u7a33\u5065\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAWENJIANYOUXUANLIUGEYUECHIYOUQIHUNHEC"}, "009889": {"suoxie": "HRYDRX39GYDKZA", "name": "\u534e\u6da6\u5143\u5927\u6da6\u79a739\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNXI39GEYUEDINGKAIZHAIA"}, "009890": {"suoxie": "HRYDRX39GYDKZC", "name": "\u534e\u6da6\u5143\u5927\u6da6\u79a739\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARUNYUANDARUNXI39GEYUEDINGKAIZHAIC"}, "009891": {"suoxie": "RTCYQSZXGP", "name": "\u878d\u901a\u4ea7\u4e1a\u8d8b\u52bf\u81fb\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "RONGTONGCHANYEQUSHIZHENXUANGUPIAO"}, "009892": {"suoxie": "FGCZCLHH", "name": "\u5bcc\u56fd\u6210\u957f\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHENGZHANGCELUEHUNHE"}, "009893": {"suoxie": "DMYYAHHHA", "name": "\u5927\u6469\u4f18\u60a6\u5b89\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOYOUYUEANHEHUNHEA"}, "009894": {"suoxie": "QHKYHY39GYDKZQ", "name": "\u524d\u6d77\u5f00\u6e90\u60e0\u76c839\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANHUIYING39GEYUEDINGKAIZHAIQUAN"}, "009895": {"suoxie": "STMGRS87GYDKZ", "name": "\u4e0a\u6295\u6469\u6839\u745e\u76db87\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENRUISHENG87GEYUEDINGKAIZHAI"}, "009896": {"suoxie": "GFGGTCZJXGPA", "name": "\u5e7f\u53d1\u6e2f\u80a1\u901a\u6210\u957f\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAGANGGUTONGCHENGZHANGJINGXUANGUPIAOA"}, "009897": {"suoxie": "GFGGTCZJXGPC", "name": "\u5e7f\u53d1\u6e2f\u80a1\u901a\u6210\u957f\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAGANGGUTONGCHENGZHANGJINGXUANGUPIAOC"}, "009898": {"suoxie": "MSJYYYJKGPA", "name": "\u6c11\u751f\u52a0\u94f6\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINYIYAOJIANKANGGUPIAOA"}, "009899": {"suoxie": "SYNXZZGP", "name": "\u4e0a\u94f6\u5185\u9700\u589e\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGYINNEIXUZENGZHANGGUPIAO"}, "009900": {"suoxie": "YFDPGLGYCYQHHA", "name": "\u6613\u65b9\u8fbe\u78d0\u56fa\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANGULIUGEYUECHIYOUQIHUNHEA"}, "009901": {"suoxie": "YFDPGLGYCYQHHC", "name": "\u6613\u65b9\u8fbe\u78d0\u56fa\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAPANGULIUGEYUECHIYOUQIHUNHEC"}, "009902": {"suoxie": "YFDYXYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXIANGYINIANCHIYOUHUNHEA"}, "009903": {"suoxie": "YFDYXYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXIANGYINIANCHIYOUHUNHEC"}, "009904": {"suoxie": "MSJYZZ200ZSZQA", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1200\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG200ZHISHUZENGQIANGA"}, "009905": {"suoxie": "MSJYZZ200ZSZQC", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1200\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG200ZHISHUZENGQIANGC"}, "009906": {"suoxie": "NDAR39GYDK", "name": "\u8bfa\u5fb7\u5b89\u745e39\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUODEANRUI39GEYUEDINGKAI"}, "009907": {"suoxie": "XCCZLHPZHHA", "name": "\u6e58\u8d22\u957f\u6cfd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGZELINGHUOPEIZHIHUNHEA"}, "009908": {"suoxie": "XCCZLHPZHHC", "name": "\u6e58\u8d22\u957f\u6cfd\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGZELINGHUOPEIZHIHUNHEC"}, "009909": {"suoxie": "JSDLXFHHA", "name": "\u5609\u5b9e\u52a8\u529b\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIDONGLIXIANFENGHUNHEA"}, "009910": {"suoxie": "JSDLXFHHC", "name": "\u5609\u5b9e\u52a8\u529b\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIDONGLIXIANFENGHUNHEC"}, "009911": {"suoxie": "CXJZLCLNDKC", "name": "\u957f\u4fe1\u4ef7\u503c\u84dd\u7b79\u4e24\u5e74\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINJIAZHILANCHOULIANGNIANDINGKAIC"}, "009912": {"suoxie": "JTTFGGHHC", "name": "\u4e5d\u6cf0\u5929\u5bcc\u6539\u9769\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAITIANFUGAIGEHUNHEC"}, "009913": {"suoxie": "ZXBCCZDLHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u6210\u957f\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGCHENGZHANGDONGLIHUNHEA"}, "009914": {"suoxie": "FGCZDLHH", "name": "\u5bcc\u56fd\u6210\u957f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHENGZHANGDONGLIHUNHE"}, "009916": {"suoxie": "GLHLZQZQA", "name": "\u683c\u6797\u6cd3\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GELINHONGLIZENGQIANGZHAIQUANA"}, "009917": {"suoxie": "GLHLZQZQC", "name": "\u683c\u6797\u6cd3\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GELINHONGLIZENGQIANGZHAIQUANC"}, "009918": {"suoxie": "SYHXCZHHA", "name": "\u4e0a\u94f6\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINHEXINCHENGZHANGHUNHEA"}, "009919": {"suoxie": "SYHXCZHHC", "name": "\u4e0a\u94f6\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINHEXINCHENGZHANGHUNHEC"}, "009920": {"suoxie": "PHNNHYNCYQZQA", "name": "\u9e4f\u534e\u5e74\u5e74\u7ea2\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUANIANNIANHONGYINIANCHIYOUQIZHAIQUANA"}, "009921": {"suoxie": "PHNNHYNCYQZQC", "name": "\u9e4f\u534e\u5e74\u5e74\u7ea2\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUANIANNIANHONGYINIANCHIYOUQIZHAIQUANC"}, "009922": {"suoxie": "HXDFZQA", "name": "\u534e\u590f\u9f0e\u5bcc\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGFUZHAIQUANA"}, "009923": {"suoxie": "HXDFZQC", "name": "\u534e\u590f\u9f0e\u5bcc\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGFUZHAIQUANC"}, "009924": {"suoxie": "ZYZZ15NGKZZS", "name": "\u4e2d\u94f6\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHONGZHAI15NIANGUOKAIZHAIZHISHU"}, "009925": {"suoxie": "BSHL6GYCYQZQA", "name": "\u535a\u65f6\u6052\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHENGLI6GEYUECHIYOUQIZHAIQUANA"}, "009926": {"suoxie": "BSHL6GYCYQZQC", "name": "\u535a\u65f6\u6052\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHENGLI6GEYUECHIYOUQIZHAIQUANC"}, "009927": {"suoxie": "GYJL18GYDKHHA", "name": "\u5de5\u94f6\u805a\u522918\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJULI18GEYUEDINGKAIHUNHEA"}, "009928": {"suoxie": "GYJL18GYDKHHC", "name": "\u5de5\u94f6\u805a\u522918\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJULI18GEYUEDINGKAIHUNHEC"}, "009929": {"suoxie": "NFCXQDHHA", "name": "\u5357\u65b9\u521b\u65b0\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINQUDONGHUNHEA"}, "009930": {"suoxie": "NFCXQDHHC", "name": "\u5357\u65b9\u521b\u65b0\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINQUDONGHUNHEC"}, "009931": {"suoxie": "CHXXYNCYQHHA", "name": "\u6df3\u539a\u6b23\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINXIANGYINIANCHIYOUQIHUNHEA"}, "009932": {"suoxie": "YYWJZZYNCYHHA", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGWENJIANZENGZHANGYINIANCHIYOUHUNHEA"}, "009933": {"suoxie": "PYASPH66GYDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u534e66\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUHUA66GEYUEDINGKAIZHAIA"}, "009934": {"suoxie": "PYASPH66GYDKZC", "name": "\u6d66\u94f6\u5b89\u76db\u666e\u534e66\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGPUHUA66GEYUEDINGKAIZHAIC"}, "009937": {"suoxie": "DFXYYNCYQHHA", "name": "\u4e1c\u65b9\u6b23\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINYIYINIANCHIYOUQIHUNHEA"}, "009938": {"suoxie": "DFXYYNCYQHHC", "name": "\u4e1c\u65b9\u6b23\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGXINYIYINIANCHIYOUQIHUNHEC"}, "009939": {"suoxie": "CHXXYNCYQHHC", "name": "\u6df3\u539a\u6b23\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINXIANGYINIANCHIYOUQIHUNHEC"}, "009940": {"suoxie": "GLWJJZHHA", "name": "\u683c\u6797\u7a33\u5065\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINWENJIANJIAZHIHUNHEA"}, "009941": {"suoxie": "GLWJJZHHC", "name": "\u683c\u6797\u7a33\u5065\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GELINWENJIANJIAZHIHUNHEC"}, "009942": {"suoxie": "CTZGHYZDZZQE", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u76ca\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGYIZHONGDUANZHAIZHAIQUANE"}, "009943": {"suoxie": "PYASWJFLZQA", "name": "\u6d66\u94f6\u5b89\u76db\u7a33\u5065\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGWENJIANFENGLIZHAIQUANA"}, "009944": {"suoxie": "PYASWJFLZQC", "name": "\u6d66\u94f6\u5b89\u76db\u7a33\u5065\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGWENJIANFENGLIZHAIQUANC"}, "009947": {"suoxie": "HBBHZQ", "name": "\u534e\u5b9d\u5b9d\u6cd3\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAOHONGZHAIQUAN"}, "009950": {"suoxie": "CTZGJHJZYNCYQHH", "name": "\u8d22\u901a\u8d44\u7ba1\u5747\u8861\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJUNHENGJIAZHIYINIANCHIYOUQIHUNHE"}, "009951": {"suoxie": "GFWJHBHHA", "name": "\u5e7f\u53d1\u7a33\u5065\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAWENJIANHUIBAOHUNHEA"}, "009952": {"suoxie": "GFWJHBHHC", "name": "\u5e7f\u53d1\u7a33\u5065\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAWENJIANHUIBAOHUNHEC"}, "009953": {"suoxie": "HTBRJQZQ", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u4e7e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIJINQIANZHAIQUAN"}, "009954": {"suoxie": "BXRFYXCZ", "name": "\u5317\u4fe1\u745e\u4e30\u4f18\u9009\u6210\u957f", "style": "\u80a1\u7968\u578b", "pinyin": "BEIXINRUIFENGYOUXUANCHENGZHANG"}, "009955": {"suoxie": "GFXYHHC", "name": "\u5e7f\u53d1\u946b\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINYUHUNHEC"}, "009956": {"suoxie": "GFHYHHA", "name": "\u5e7f\u53d1\u6052\u8a89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYUHUNHEA"}, "009957": {"suoxie": "GFHYHHC", "name": "\u5e7f\u53d1\u6052\u8a89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYUHUNHEC"}, "009958": {"suoxie": "CAXYXFHHA", "name": "\u957f\u5b89\u946b\u60a6\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXINYUEXIAOFEIHUNHEA"}, "009959": {"suoxie": "CAXYXFHHC", "name": "\u957f\u5b89\u946b\u60a6\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXINYUEXIAOFEIHUNHEC"}, "009960": {"suoxie": "YHDYJYHH", "name": "\u94f6\u534e\u591a\u5143\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUADUOYUANJIYUHUNHE"}, "009967": {"suoxie": "BSRTHH", "name": "\u535a\u65f6\u8363\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGTAIHUNHE"}, "009968": {"suoxie": "JYNXCZHHA", "name": "\u91d1\u9e70\u5185\u9700\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGNEIXUCHENGZHANGHUNHEA"}, "009969": {"suoxie": "JYNXCZHHC", "name": "\u91d1\u9e70\u5185\u9700\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGNEIXUCHENGZHANGHUNHEC"}, "009970": {"suoxie": "CTNXZZ12GYDKHH", "name": "\u8d22\u901a\u5185\u9700\u589e\u957f12\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGNEIXUZENGZHANG12GEYUEDINGKAIHUNHE"}, "009975": {"suoxie": "HBBPMGXFRMBC", "name": "\u534e\u5b9d\u6807\u666e\u7f8e\u56fd\u6d88\u8d39\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "HUABAOBIAOPUMEIGUOXIAOFEIRENMINBIC"}, "009976": {"suoxie": "DFJZTHBB", "name": "\u4e1c\u65b9\u91d1\u8bc1\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGJINZHENGTONGHUOBIB"}, "009977": {"suoxie": "YHZLYNCYQHHA", "name": "\u94f6\u534e\u62db\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAZHAOLIYINIANCHIYOUQIHUNHEA"}, "009978": {"suoxie": "YHZLYNCYQHHC", "name": "\u94f6\u534e\u62db\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAZHAOLIYINIANCHIYOUQIHUNHEC"}, "009979": {"suoxie": "XHAXHR88GYDKZA", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u878d88\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIRONG88GEYUEDINGKAIZHAIA"}, "009980": {"suoxie": "XHAXHR88GYDKZC", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u878d88\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIRONG88GEYUEDINGKAIZHAIC"}, "009981": {"suoxie": "WJCYBZSZQA", "name": "\u4e07\u5bb6\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIACHUANGYEBANZHISHUZENGQIANGA"}, "009982": {"suoxie": "WJCYBZSZQC", "name": "\u4e07\u5bb6\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIACHUANGYEBANZHISHUZENGQIANGC"}, "009983": {"suoxie": "YYGGTPZSHHXHH", "name": "\u6c38\u8d62\u6e2f\u80a1\u901a\u54c1\u8d28\u751f\u6d3b\u6167\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGANGGUTONGPINZHISHENGHUOHUIXUANHUNHE"}, "009984": {"suoxie": "PHQHLNFBYZHH", "name": "\u9e4f\u534e\u542f\u822a\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAQIHANGLIANGNIANFENGBIYUNZUOHUNHE"}, "009985": {"suoxie": "YYXLZGDJDZE", "name": "\u6c38\u8d62\u8fc5\u5229\u4e2d\u9ad8\u7b49\u7ea7\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGXUNLIZHONGGAODENGJIDUANZHAIE"}, "009986": {"suoxie": "THCXLHA", "name": "\u5929\u5f18\u521b\u65b0\u9886\u822aA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGCHUANGXINLINGHANGA"}, "009987": {"suoxie": "THCXLHC", "name": "\u5929\u5f18\u521b\u65b0\u9886\u822aC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGCHUANGXINLINGHANGC"}, "009988": {"suoxie": "XDAYLCJXGP", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u84dd\u7b79\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINDAAOYINLANCHOUJINGXUANGUPIAO"}, "009989": {"suoxie": "HBYJJXHH", "name": "\u534e\u5b9d\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOYANJIUJINGXUANHUNHE"}, "009990": {"suoxie": "HTBRPZYXA", "name": "\u534e\u6cf0\u67cf\u745e\u54c1\u8d28\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIPINZHIYOUXUANA"}, "009991": {"suoxie": "HTBRPZYXC", "name": "\u534e\u6cf0\u67cf\u745e\u54c1\u8d28\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIPINZHIYOUXUANC"}, "009992": {"suoxie": "JSCCLHCZYHHH", "name": "\u666f\u987a\u957f\u57ce\u91cf\u5316\u6210\u957f\u6f14\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGLIANGHUACHENGZHANGYANHUAHUNHE"}, "009993": {"suoxie": "JSQYCXHH", "name": "\u5609\u5b9e\u524d\u6cbf\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIQIANYANCHUANGXINHUNHE"}, "009994": {"suoxie": "JSCXXFHHA", "name": "\u5609\u5b9e\u521b\u65b0\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHUANGXINXIANFENGHUNHEA"}, "009995": {"suoxie": "JSCXXFHHC", "name": "\u5609\u5b9e\u521b\u65b0\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHUANGXINXIANFENGHUNHEC"}, "009998": {"suoxie": "STMGHJLNCYQ", "name": "\u4e0a\u6295\u6469\u6839\u6167\u89c1\u4e24\u5e74\u6301\u6709\u671f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENHUIJIANLIANGNIANCHIYOUQI"}, "009999": {"suoxie": "DFZGHLHH", "name": "\u4e1c\u65b9\u4e2d\u56fd\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGZHONGGUOHONGLIHUNHE"}, "010000": {"suoxie": "CCZGZZLHPZHHC", "name": "\u957f\u57ce\u4e2d\u56fd\u667a\u9020\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGZHONGGUOZHIZAOLINGHUOPEIZHIHUNHEC"}, "010001": {"suoxie": "CJHXYJJXGPA", "name": "\u521b\u91d1\u5408\u4fe1\u7814\u7a76\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYANJIUJINGXUANGUPIAOA"}, "010002": {"suoxie": "CJHXYJJXGPC", "name": "\u521b\u91d1\u5408\u4fe1\u7814\u7a76\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYANJIUJINGXUANGUPIAOC"}, "010003": {"suoxie": "JSCCDZXXCYGPA", "name": "\u666f\u987a\u957f\u57ce\u7535\u5b50\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGDIANZIXINXICHANYEGUPIAOA"}, "010004": {"suoxie": "JSCCDZXXCYGPC", "name": "\u666f\u987a\u957f\u57ce\u7535\u5b50\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGDIANZIXINXICHANYEGUPIAOC"}, "010005": {"suoxie": "PYXJTLHBE", "name": "\u9e4f\u626c\u73b0\u91d1\u901a\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGYANGXIANJINTONGLIHUOBIE"}, "010006": {"suoxie": "NFYDYNCYQHHA", "name": "\u5357\u65b9\u8a89\u9f0e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUDINGYINIANCHIYOUQIHUNHEA"}, "010007": {"suoxie": "NFYDYNCYQHHC", "name": "\u5357\u65b9\u8a89\u9f0e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUDINGYINIANCHIYOUQIHUNHEC"}, "010008": {"suoxie": "ZRCZYXHHA", "name": "\u4e2d\u878d\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCHENGZHANGYOUXUANHUNHEA"}, "010009": {"suoxie": "ZRCZYXHHC", "name": "\u4e2d\u878d\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCHENGZHANGYOUXUANHUNHEC"}, "010010": {"suoxie": "GTRYGGT6GYDKGP", "name": "\u56fd\u6295\u745e\u94f6\u6e2f\u80a1\u901a6\u4e2a\u6708\u5b9a\u5f00\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTOURUIYINGANGGUTONG6GEYUEDINGKAIGUPIAO"}, "010011": {"suoxie": "JSJYZL6GYCYQZQA", "name": "\u666f\u987a\u666f\u9890\u62db\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNJINGYIZHAOLI6GEYUECHIYOUQIZHAIQUANA"}, "010012": {"suoxie": "JSJYZL6GYCYQZQC", "name": "\u666f\u987a\u666f\u9890\u62db\u52296\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNJINGYIZHAOLI6GEYUECHIYOUQIZHAIQUANC"}, "010013": {"suoxie": "YFDXXHYJXGP", "name": "\u6613\u65b9\u8fbe\u4fe1\u606f\u884c\u4e1a\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAXINXIHANGYEJINGXUANGUPIAO"}, "010014": {"suoxie": "HXDQZQA", "name": "\u534e\u590f\u9f0e\u6e05\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGQINGZHAIQUANA"}, "010015": {"suoxie": "HXDQZQC", "name": "\u534e\u590f\u9f0e\u6e05\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGQINGZHAIQUANC"}, "010016": {"suoxie": "HXKJQY6GYDKHHA", "name": "\u534e\u590f\u79d1\u6280\u524d\u6cbf6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAKEJIQIANYAN6GEYUEDINGKAIHUNHEA"}, "010017": {"suoxie": "HXKJQY6GYDKHHC", "name": "\u534e\u590f\u79d1\u6280\u524d\u6cbf6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAKEJIQIANYAN6GEYUEDINGKAIHUNHEC"}, "010018": {"suoxie": "ZSRZYNCYQHHA", "name": "\u62db\u5546\u745e\u6cfd\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIZEYINIANCHIYOUQIHUNHEA"}, "010019": {"suoxie": "ZSRZYNCYQHHC", "name": "\u62db\u5546\u745e\u6cfd\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIZEYINIANCHIYOUQIHUNHEC"}, "010020": {"suoxie": "HXXSJJZTJXHH", "name": "\u534e\u590f\u7ebf\u4e0a\u7ecf\u6d4e\u4e3b\u9898\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANSHANGJINGJIZHUTIJINGXUANHUNHE"}, "010021": {"suoxie": "GFYQJXHHC", "name": "\u5e7f\u53d1\u4f18\u4f01\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAYOUQIJINGXUANHUNHEC"}, "010022": {"suoxie": "GFXFPJXHHC", "name": "\u5e7f\u53d1\u6d88\u8d39\u54c1\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOFEIPINJINGXUANHUNHEC"}, "010023": {"suoxie": "GFZZYJXHHC", "name": "\u5e7f\u53d1\u5236\u9020\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZHIZAOYEJINGXUANHUNHEC"}, "010024": {"suoxie": "GFHGSXQDGPC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u65b0\u8d77\u70b9\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAHUGANGSHENXINQIDIANGUPIAOC"}, "010025": {"suoxie": "GFJFHHC", "name": "\u5e7f\u53d1\u805a\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUFENGHUNHEC"}, "010026": {"suoxie": "GFJRHHC", "name": "\u5e7f\u53d1\u805a\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJURUIHUNHEC"}, "010027": {"suoxie": "JSCCHXZJYNCYHH", "name": "\u666f\u987a\u957f\u57ce\u6838\u5fc3\u4e2d\u666f\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGHEXINZHONGJINGYINIANCHIYOUHUNHE"}, "010028": {"suoxie": "HTBRCXSJHHC", "name": "\u534e\u6cf0\u67cf\u745e\u521b\u65b0\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUICHUANGXINSHENGJIHUNHEC"}, "010029": {"suoxie": "FGWJHB12GYCYQHHA", "name": "\u5bcc\u56fd\u7a33\u8fdb\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOWENJINHUIBAO12GEYUECHIYOUQIHUNHEA"}, "010030": {"suoxie": "FGWJHB12GYCYQHHC", "name": "\u5bcc\u56fd\u7a33\u8fdb\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOWENJINHUIBAO12GEYUECHIYOUQIHUNHEC"}, "010031": {"suoxie": "HTBRSWYYHHC", "name": "\u534e\u6cf0\u67cf\u745e\u751f\u7269\u533b\u836f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUISHENGWUYIYAOHUNHEC"}, "010032": {"suoxie": "HTBRXXCYHHC", "name": "\u534e\u6cf0\u67cf\u745e\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIXINXINGCHANYEHUNHEC"}, "010033": {"suoxie": "AXCZJXHHA", "name": "\u5b89\u4fe1\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINCHENGZHANGJINGXUANHUNHEA"}, "010034": {"suoxie": "AXCZJXHHC", "name": "\u5b89\u4fe1\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINCHENGZHANGJINGXUANHUNHEC"}, "010035": {"suoxie": "PAGDJZE", "name": "\u5e73\u5b89\u9ad8\u7b49\u7ea7\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANGAODENGJIZHAIE"}, "010036": {"suoxie": "GFHTLGYCYQHHA", "name": "\u5e7f\u53d1\u6052\u901a\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGTONGLIUGEYUECHIYOUQIHUNHEA"}, "010037": {"suoxie": "HTBRJZZZHHC", "name": "\u534e\u6cf0\u67cf\u745e\u4ef7\u503c\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIAZHIZENGZHANGHUNHEC"}, "010038": {"suoxie": "GFHTLGYCYQHHC", "name": "\u5e7f\u53d1\u6052\u901a\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGTONGLIUGEYUECHIYOUQIHUNHEC"}, "010041": {"suoxie": "JSGGYSHHA", "name": "\u5609\u5b9e\u6e2f\u80a1\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIGANGGUYOUSHIHUNHEA"}, "010042": {"suoxie": "JSGGYSHHC", "name": "\u5609\u5b9e\u6e2f\u80a1\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIGANGGUYOUSHIHUNHEC"}, "010043": {"suoxie": "THAKYHHHA", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIHEHUNHEA"}, "010044": {"suoxie": "THAKYHHHC", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIHEHUNHEC"}, "010045": {"suoxie": "HTFWJTYYNCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6dfb\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANTIANYINGYINIANCHIYOUHUNHE"}, "010046": {"suoxie": "BSJFAYNCYHHFOFA", "name": "\u535a\u65f6\u91d1\u798f\u5b89\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIJINFUANYINIANCHIYOUHUNHEFOFA"}, "010047": {"suoxie": "BSJFAYNCYHHFOFC", "name": "\u535a\u65f6\u91d1\u798f\u5b89\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIJINFUANYINIANCHIYOUHUNHEFOFC"}, "010048": {"suoxie": "PADZI", "name": "\u5e73\u5b89\u77ed\u503aI", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANDUANZHAII"}, "010049": {"suoxie": "CCCZXFHHA", "name": "\u957f\u57ce\u6210\u957f\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGCHENGZHANGXIANFENGHUNHEA"}, "010050": {"suoxie": "CCCZXFHHC", "name": "\u957f\u57ce\u6210\u957f\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGCHENGZHANGXIANFENGHUNHEC"}, "010051": {"suoxie": "CCGZBHBC", "name": "\u957f\u57ce\u5de5\u8d44\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGGONGZIBAOHUOBIC"}, "010052": {"suoxie": "CCJJCXCZHHC", "name": "\u957f\u57ce\u4e45\u5609\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUJIACHUANGXINCHENGZHANGHUNHEC"}, "010053": {"suoxie": "AXJLZQZQB", "name": "\u5b89\u4fe1\u805a\u5229\u589e\u5f3a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINJULIZENGQIANGZHAIQUANB"}, "010054": {"suoxie": "WJJKCYHHA", "name": "\u4e07\u5bb6\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJIANKANGCHANYEHUNHEA"}, "010055": {"suoxie": "WJJKCYHHC", "name": "\u4e07\u5bb6\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJIANKANGCHANYEHUNHEC"}, "010056": {"suoxie": "PARXYNDKHHA", "name": "\u5e73\u5b89\u745e\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANRUIXINGYINIANDINGKAIHUNHEA"}, "010057": {"suoxie": "PARXYNDKHHC", "name": "\u5e73\u5b89\u745e\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANRUIXINGYINIANDINGKAIHUNHEC"}, "010058": {"suoxie": "THRCYNCYQHH", "name": "\u5929\u5f18\u8363\u521b\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGRONGCHUANGYINIANCHIYOUQIHUNHE"}, "010059": {"suoxie": "DFHDY3GYDKHH", "name": "\u4e1c\u65b9\u7ea2\u9f0e\u51433\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGDINGYUAN3GEYUEDINGKAIHUNHE"}, "010060": {"suoxie": "HTBRJLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAIRUIJINGLIHUNHEA"}, "010061": {"suoxie": "HTBRJLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAIRUIJINGLIHUNHEC"}, "010062": {"suoxie": "NFHYJXYNHHA", "name": "\u5357\u65b9\u884c\u4e1a\u7cbe\u9009\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHANGYEJINGXUANYINIANHUNHEA"}, "010063": {"suoxie": "NFHYJXYNHHC", "name": "\u5357\u65b9\u884c\u4e1a\u7cbe\u9009\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHANGYEJINGXUANYINIANHUNHEC"}, "010064": {"suoxie": "YXYFXYA", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u7814A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXINGYANA"}, "010065": {"suoxie": "YXYFXYC", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u7814C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXINGYANC"}, "010066": {"suoxie": "FZFBZZ500ZSZQA", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENG500ZHISHUZENGQIANGA"}, "010067": {"suoxie": "FZFBZZ500ZSZQC", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENG500ZHISHUZENGQIANGC"}, "010068": {"suoxie": "GYSYZQA", "name": "\u5de5\u94f6\u53cc\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINSHUANGYINGZHAIQUANA"}, "010069": {"suoxie": "GYSYZQC", "name": "\u5de5\u94f6\u53cc\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINSHUANGYINGZHAIQUANC"}, "010070": {"suoxie": "FZFBESGZTTZHHA", "name": "\u65b9\u6b63\u5bcc\u90a6ESG\u4e3b\u9898\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGESGZHUTITOUZIHUNHEA"}, "010071": {"suoxie": "FZFBESGZTTZHHC", "name": "\u65b9\u6b63\u5bcc\u90a6ESG\u4e3b\u9898\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGESGZHUTITOUZIHUNHEC"}, "010072": {"suoxie": "FZFBCLJXA", "name": "\u65b9\u6b63\u5bcc\u90a6\u7b56\u7565\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCELUEJINGXUANA"}, "010073": {"suoxie": "FZFBCLJXC", "name": "\u65b9\u6b63\u5bcc\u90a6\u7b56\u7565\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCELUEJINGXUANC"}, "010074": {"suoxie": "NFYLYNHHA", "name": "\u5357\u65b9\u8a89\u9686\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYULONGYINIANHUNHEA"}, "010075": {"suoxie": "NFYLYNHHC", "name": "\u5357\u65b9\u8a89\u9686\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYULONGYINIANHUNHEC"}, "010076": {"suoxie": "XCCHLHPZHHA", "name": "\u6e58\u8d22\u957f\u5f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGHONGLINGHUOPEIZHIHUNHEA"}, "010077": {"suoxie": "XCCHLHPZHHC", "name": "\u6e58\u8d22\u957f\u5f18\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIANGCAICHANGHONGLINGHUOPEIZHIHUNHEC"}, "010078": {"suoxie": "BSHRYNHHA", "name": "\u535a\u65f6\u6052\u8363\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRONGYINIANHUNHEA"}, "010079": {"suoxie": "BSHRYNHHC", "name": "\u535a\u65f6\u6052\u8363\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRONGYINIANHUNHEC"}, "010080": {"suoxie": "ZOYSCZSGYDKHH", "name": "\u4e2d\u6b27\u4f18\u52bf\u6210\u957f\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYOUSHICHENGZHANGSANGEYUEDINGKAIHUNHE"}, "010081": {"suoxie": "TKHZHHA", "name": "\u6cf0\u5eb7\u6d69\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGHAOZEHUNHEA"}, "010082": {"suoxie": "TKHZHHC", "name": "\u6cf0\u5eb7\u6d69\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGHAOZEHUNHEC"}, "010083": {"suoxie": "ZYXXLLZ", "name": "\u4e2d\u94f6\u6b23\u4eab\u5229\u7387\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINXINXIANGLILVZHAI"}, "010084": {"suoxie": "FCFRZQA", "name": "\u8702\u5de2\u4e30\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOFENGRUIZHAIQUANA"}, "010085": {"suoxie": "FCFRZQC", "name": "\u8702\u5de2\u4e30\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOFENGRUIZHAIQUANC"}, "010086": {"suoxie": "ZYCZFLZQA", "name": "\u4e2d\u90ae\u7eaf\u503a\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIFENGLIZHAIQUANA"}, "010087": {"suoxie": "ZYCZFLZQC", "name": "\u4e2d\u90ae\u7eaf\u503a\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUCHUNZHAIFENGLIZHAIQUANC"}, "010088": {"suoxie": "GYYZCZHHA", "name": "\u5de5\u94f6\u4f18\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYOUZHICHENGZHANGHUNHEA"}, "010089": {"suoxie": "GYYZCZHHC", "name": "\u5de5\u94f6\u4f18\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYOUZHICHENGZHANGHUNHEC"}, "010090": {"suoxie": "ZXJTYYJKA", "name": "\u4e2d\u4fe1\u5efa\u6295\u533b\u836f\u5065\u5eb7A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUYIYAOJIANKANGA"}, "010091": {"suoxie": "ZXJTYYJKC", "name": "\u4e2d\u4fe1\u5efa\u6295\u533b\u836f\u5065\u5eb7C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUYIYAOJIANKANGC"}, "010092": {"suoxie": "YYHJXYZA", "name": "\u6c38\u8d62\u534e\u5609\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHUAJIAXINYONGZHAIA"}, "010093": {"suoxie": "XBLDGGTXJYHHC", "name": "\u897f\u90e8\u5229\u5f97\u6e2f\u80a1\u901a\u65b0\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEGANGGUTONGXINJIYUHUNHEC"}, "010094": {"suoxie": "JYCYJYHH", "name": "\u4ea4\u94f6\u4ea7\u4e1a\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHANYEJIYUHUNHE"}, "010096": {"suoxie": "BYXXSGYZQA", "name": "\u535a\u8fdc\u946b\u4eab\u4e09\u4e2a\u6708\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOYUANXINXIANGSANGEYUEZHAIQUANA"}, "010097": {"suoxie": "BYXXSGYZQC", "name": "\u535a\u8fdc\u946b\u4eab\u4e09\u4e2a\u6708\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOYUANXINXIANGSANGEYUEZHAIQUANC"}, "010098": {"suoxie": "BYXXSGYZQE", "name": "\u535a\u8fdc\u946b\u4eab\u4e09\u4e2a\u6708\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOYUANXINXIANGSANGEYUEZHAIQUANE"}, "010099": {"suoxie": "MSJYHZ3GYDKZ", "name": "\u6c11\u751f\u52a0\u94f6\u6c47\u667a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHUIZHI3GEYUEDINGKAIZHAI"}, "010102": {"suoxie": "XBLDXHZQZQA", "name": "\u897f\u90e8\u5229\u5f97\u946b\u6cd3\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEXINHONGZENGQIANGZHAIQUANA"}, "010103": {"suoxie": "XBLDXHZQZQC", "name": "\u897f\u90e8\u5229\u5f97\u946b\u6cd3\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEXINHONGZENGQIANGZHAIQUANC"}, "010104": {"suoxie": "JSCCXFJXHHA", "name": "\u666f\u987a\u957f\u57ce\u6d88\u8d39\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGXIAOFEIJINGXUANHUNHEA"}, "010105": {"suoxie": "JSCCXFJXHHC", "name": "\u666f\u987a\u957f\u57ce\u6d88\u8d39\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGXIAOFEIJINGXUANHUNHEC"}, "010106": {"suoxie": "HXHXKJ6GYDKHHA", "name": "\u534e\u590f\u6838\u5fc3\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINKEJI6GEYUEDINGKAIHUNHEA"}, "010107": {"suoxie": "HXHXKJ6GYDKHHC", "name": "\u534e\u590f\u6838\u5fc3\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINKEJI6GEYUEDINGKAIHUNHEC"}, "010108": {"suoxie": "JSCCHXZJHH", "name": "\u666f\u987a\u957f\u57ce\u6838\u5fc3\u62db\u666f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGHEXINZHAOJINGHUNHE"}, "010109": {"suoxie": "FGJZZZHH", "name": "\u5bcc\u56fd\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIAZHIZENGZHANGHUNHE"}, "010110": {"suoxie": "GFYYJKHHA", "name": "\u5e7f\u53d1\u533b\u836f\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAYIYAOJIANKANGHUNHEA"}, "010111": {"suoxie": "GFYYJKHHC", "name": "\u5e7f\u53d1\u533b\u836f\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAYIYAOJIANKANGHUNHEC"}, "010112": {"suoxie": "GFYJJXGPA", "name": "\u5e7f\u53d1\u7814\u7a76\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYANJIUJINGXUANGUPIAOA"}, "010113": {"suoxie": "GFYJJXGPC", "name": "\u5e7f\u53d1\u7814\u7a76\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYANJIUJINGXUANGUPIAOC"}, "010114": {"suoxie": "HBXXCZHH", "name": "\u534e\u5b9d\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXINXINGCHENGZHANGHUNHE"}, "010115": {"suoxie": "YFDYJCZHHA", "name": "\u6613\u65b9\u8fbe\u8fdc\u89c1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYUANJIANCHENGZHANGHUNHEA"}, "010116": {"suoxie": "MSJYXXCYHHA", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINXINGCHANYEHUNHEA"}, "010117": {"suoxie": "MSJYXXCYHHC", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINXINGCHANYEHUNHEC"}, "010118": {"suoxie": "THDYSYZQA", "name": "\u5929\u5f18\u591a\u5143\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGDUOYUANSHOUYIZHAIQUANA"}, "010119": {"suoxie": "THDYSYZQC", "name": "\u5929\u5f18\u591a\u5143\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGDUOYUANSHOUYIZHAIQUANC"}, "010120": {"suoxie": "JTJFLHGPA", "name": "\u4e5d\u6cf0\u4e45\u798f\u91cf\u5316\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUFULIANGHUAGUPIAOA"}, "010121": {"suoxie": "JTJFLHGPC", "name": "\u4e5d\u6cf0\u4e45\u798f\u91cf\u5316\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUFULIANGHUAGUPIAOC"}, "010122": {"suoxie": "HTBRYSLHHHA", "name": "\u534e\u6cf0\u67cf\u745e\u4f18\u52bf\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYOUSHILINGHANGHUNHEA"}, "010123": {"suoxie": "HTBRYSLHHHC", "name": "\u534e\u6cf0\u67cf\u745e\u4f18\u52bf\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYOUSHILINGHANGHUNHEC"}, "010124": {"suoxie": "XYJQYXHHA", "name": "\u5174\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINJINGQIYOUXUANHUNHEA"}, "010125": {"suoxie": "XYJQYXHHC", "name": "\u5174\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINJINGQIYOUXUANHUNHEC"}, "010126": {"suoxie": "PAJZCZHHA", "name": "\u5e73\u5b89\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIAZHICHENGZHANGHUNHEA"}, "010127": {"suoxie": "PAJZCZHHC", "name": "\u5e73\u5b89\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIAZHICHENGZHANGHUNHEC"}, "010128": {"suoxie": "BYFZXDNGPA", "name": "\u5b9d\u76c8\u53d1\u5c55\u65b0\u52a8\u80fd\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGFAZHANXINDONGNENGGUPIAOA"}, "010129": {"suoxie": "BYFZXDNGPC", "name": "\u5b9d\u76c8\u53d1\u5c55\u65b0\u52a8\u80fd\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGFAZHANXINDONGNENGGUPIAOC"}, "010130": {"suoxie": "HFTHZYNDKHHA", "name": "\u6d77\u5bcc\u901a\u60e0\u589e\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGHUIZENGYINIANDINGKAIHUNHEA"}, "010131": {"suoxie": "HFTHZYNDKHHC", "name": "\u6d77\u5bcc\u901a\u60e0\u589e\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGHUIZENGYINIANDINGKAIHUNHEC"}, "010132": {"suoxie": "NFCXCZHHA", "name": "\u5357\u65b9\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINCHENGZHANGHUNHEA"}, "010133": {"suoxie": "NFCXCZHHC", "name": "\u5357\u65b9\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGXINCHENGZHANGHUNHEC"}, "010134": {"suoxie": "GFXJJHHC", "name": "\u5e7f\u53d1\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINJINGJIHUNHEC"}, "010135": {"suoxie": "TDGYF6GYCYHHA", "name": "\u6cf0\u8fbe\u9ad8\u7814\u53d16\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAGAOYANFA6GEYUECHIYOUHUNHEA"}, "010136": {"suoxie": "TDGYF6GYCYHHC", "name": "\u6cf0\u8fbe\u9ad8\u7814\u53d16\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAGAOYANFA6GEYUECHIYOUHUNHEC"}, "010137": {"suoxie": "HTBRLHCXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u521b\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUACHUANGXIANGHUNHEA"}, "010138": {"suoxie": "HTBRLHCXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u521b\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUACHUANGXIANGHUNHEC"}, "010139": {"suoxie": "BYYPCZZQA", "name": "\u5b9d\u76c8\u76c8\u6c9b\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGPEICHUNZHAIZHAIQUANA"}, "010140": {"suoxie": "BYYPCZZQC", "name": "\u5b9d\u76c8\u76c8\u6c9b\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGYINGPEICHUNZHAIZHAIQUANC"}, "010141": {"suoxie": "ZQQYYXA", "name": "\u6731\u96c0\u4f01\u4e1a\u4f18\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHUQUEQIYEYOUXUANA"}, "010142": {"suoxie": "ZQQYYXC", "name": "\u6731\u96c0\u4f01\u4e1a\u4f18\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHUQUEQIYEYOUXUANC"}, "010143": {"suoxie": "JYSLDQXHH", "name": "\u4ea4\u94f6\u65bd\u7f57\u5fb7\u542f\u6b23\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINSHILUODEQIXINHUNHE"}, "010144": {"suoxie": "GTGZYYWSHYZSC", "name": "\u56fd\u6cf0\u56fd\u8bc1\u533b\u836f\u536b\u751f\u884c\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGYIYAOWEISHENGHANGYEZHISHUC"}, "010145": {"suoxie": "GLZDZZQA", "name": "\u683c\u6797\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GELINZHONGDUANZHAIZHAIQUANA"}, "010146": {"suoxie": "GLZDZZQC", "name": "\u683c\u6797\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GELINZHONGDUANZHAIZHAIQUANC"}, "010147": {"suoxie": "BDJXYNCYQHH", "name": "\u535a\u9053\u5609\u5174\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIAXINGYINIANCHIYOUQIHUNHE"}, "010148": {"suoxie": "ZSZXJJDNHHA", "name": "\u6d59\u5546\u667a\u9009\u7ecf\u6d4e\u52a8\u80fd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANJINGJIDONGNENGHUNHEA"}, "010149": {"suoxie": "ZSZXJJDNHHC", "name": "\u6d59\u5546\u667a\u9009\u7ecf\u6d4e\u52a8\u80fd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANJINGJIDONGNENGHUNHEC"}, "010150": {"suoxie": "NFJXHHC", "name": "\u5357\u65b9\u541b\u4fe1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJUNXINHUNHEC"}, "010151": {"suoxie": "DCXFJXA", "name": "\u4e1c\u8d22\u6d88\u8d39\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAIXIAOFEIJINGXUANA"}, "010152": {"suoxie": "DCXFJXC", "name": "\u4e1c\u8d22\u6d88\u8d39\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAIXIAOFEIJINGXUANC"}, "010153": {"suoxie": "ZJZZ500ZSZQA", "name": "\u4e2d\u52a0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJIAZHONGZHENG500ZHISHUZENGQIANGA"}, "010154": {"suoxie": "ZJZZ500ZSZQC", "name": "\u4e2d\u52a0\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJIAZHONGZHENG500ZHISHUZENGQIANGC"}, "010155": {"suoxie": "CSHXCZHHA", "name": "\u957f\u76db\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGHEXINCHENGZHANGHUNHEA"}, "010156": {"suoxie": "CSHXCZHHC", "name": "\u957f\u76db\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGHEXINCHENGZHANGHUNHEC"}, "010157": {"suoxie": "HAZZ500ZQA", "name": "\u6c47\u5b89\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANZHONGZHENG500ZENGQIANGA"}, "010158": {"suoxie": "HAZZ500ZQC", "name": "\u6c47\u5b89\u4e2d\u8bc1500\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANZHONGZHENG500ZENGQIANGC"}, "010159": {"suoxie": "ZYYLBJHHC", "name": "\u4e2d\u94f6\u533b\u7597\u4fdd\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYILIAOBAOJIANHUNHEC"}, "010160": {"suoxie": "GFGDZZGPC", "name": "\u5e7f\u53d1\u9ad8\u7aef\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAGAODUANZHIZAOGUPIAOC"}, "010161": {"suoxie": "GFRAJXGPA", "name": "\u5e7f\u53d1\u745e\u5b89\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFARUIANJINGXUANGUPIAOA"}, "010162": {"suoxie": "GFRAJXGPC", "name": "\u5e7f\u53d1\u745e\u5b89\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFARUIANJINGXUANGUPIAOC"}, "010163": {"suoxie": "CTZGJZJXYNCYHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIAZHIJINGXUANYINIANCHIYOUHUNHEA"}, "010164": {"suoxie": "CTZGJZJXYNCYHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIAZHIJINGXUANYINIANCHIYOUHUNHEC"}, "010165": {"suoxie": "TPFHYNDKZQFQS", "name": "\u592a\u5e73\u4e30\u548c\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIPINGFENGHEYINIANDINGKAIZHAIQUANFAQISHI"}, "010166": {"suoxie": "ZSXHYX1NCYQHH", "name": "\u62db\u5546\u5174\u548c\u4f18\u90091\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGXINGHEYOUXUAN1NIANCHIYOUQIHUNHE"}, "010167": {"suoxie": "ZYDCLHHC", "name": "\u4e2d\u94f6\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINDUOCELUEHUNHEC"}, "010168": {"suoxie": "THALDZA", "name": "\u5929\u5f18\u5b89\u5229\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGANLIDUANZHAIA"}, "010169": {"suoxie": "THALDZC", "name": "\u5929\u5f18\u5b89\u5229\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGANLIDUANZHAIC"}, "010170": {"suoxie": "ZYZQXR6GYCYA", "name": "\u4e2d\u94f6\u8bc1\u5238\u946b\u745e6\u4e2a\u6708\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANXINRUI6GEYUECHIYOUA"}, "010171": {"suoxie": "ZYZQXR6GYCYC", "name": "\u4e2d\u94f6\u8bc1\u5238\u946b\u745e6\u4e2a\u6708\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANXINRUI6GEYUECHIYOUC"}, "010172": {"suoxie": "ZYXHBLHPZHHC", "name": "\u4e2d\u94f6\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINHUIBAOLINGHUOPEIZHIHUNHEC"}, "010173": {"suoxie": "YFDZJBHBB", "name": "\u6613\u65b9\u8fbe\u589e\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAZENGJINBAOHUOBIB"}, "010174": {"suoxie": "YDZXZQA", "name": "\u82f1\u5927\u667a\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINGDAZHIXIANGZHAIQUANA"}, "010175": {"suoxie": "YDZXZQC", "name": "\u82f1\u5927\u667a\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINGDAZHIXIANGZHAIQUANC"}, "010176": {"suoxie": "ZJXXXFHHA", "name": "\u4e2d\u52a0\u65b0\u5174\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXINXINGXIAOFEIHUNHEA"}, "010177": {"suoxie": "ZJXXXFHHC", "name": "\u4e2d\u52a0\u65b0\u5174\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXINXINGXIAOFEIHUNHEC"}, "010178": {"suoxie": "DCQYNLQDHHA", "name": "\u5927\u6210\u4f01\u4e1a\u80fd\u529b\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGQIYENENGLIQUDONGHUNHEA"}, "010179": {"suoxie": "DCQYNLQDHHC", "name": "\u5927\u6210\u4f01\u4e1a\u80fd\u529b\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGQIYENENGLIQUDONGHUNHEC"}, "010180": {"suoxie": "HXKJLTLNDKHH", "name": "\u534e\u590f\u79d1\u6280\u9f99\u5934\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAKEJILONGTOULIANGNIANDINGKAIHUNHE"}, "010181": {"suoxie": "XYYSCYHHA", "name": "\u5174\u4e1a\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEYOUSHICHANYEHUNHEA"}, "010182": {"suoxie": "XYYSCYHHC", "name": "\u5174\u4e1a\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEYOUSHICHANYEHUNHEC"}, "010183": {"suoxie": "NFCYBETFLJE", "name": "\u5357\u65b9\u521b\u4e1a\u677fETF\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGCHUANGYEBANETFLIANJIEE"}, "010186": {"suoxie": "JSHXCZHHA", "name": "\u5609\u5b9e\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIHEXINCHENGZHANGHUNHEA"}, "010187": {"suoxie": "JSHXCZHHC", "name": "\u5609\u5b9e\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIHEXINCHENGZHANGHUNHEC"}, "010188": {"suoxie": "ZOTYYNHHA", "name": "\u4e2d\u6b27\u6dfb\u76ca\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUTIANYIYINIANHUNHEA"}, "010189": {"suoxie": "ZOTYYNHHC", "name": "\u4e2d\u6b27\u6dfb\u76ca\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUTIANYIYINIANHUNHEC"}, "010190": {"suoxie": "JSJZFXSGYDKHH", "name": "\u5609\u5b9e\u4ef7\u503c\u53d1\u73b0\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHIFAXIANSANGEYUEDINGKAIHUNHE"}, "010191": {"suoxie": "HXDXZQA", "name": "\u534e\u590f\u9f0e\u4fe1\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXINZHAIQUANA"}, "010192": {"suoxie": "HXDXZQC", "name": "\u534e\u590f\u9f0e\u4fe1\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGXINZHAIQUANC"}, "010193": {"suoxie": "NYYL2045WNCYHHFOF", "name": "\u519c\u94f6\u517b\u80012045\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINYANGLAO2045WUNIANCHIYOUHUNHEFOF"}, "010194": {"suoxie": "BSRX15GYDKHHA", "name": "\u535a\u65f6\u777f\u796515\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIRUIXIANG15GEYUEDINGKAIHUNHEA"}, "010195": {"suoxie": "BSRX15GYDKHHC", "name": "\u535a\u65f6\u777f\u796515\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIRUIXIANG15GEYUEDINGKAIHUNHEC"}, "010196": {"suoxie": "YFDHXYSGPA", "name": "\u6613\u65b9\u8fbe\u6838\u5fc3\u4f18\u52bf\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAHEXINYOUSHIGUPIAOA"}, "010197": {"suoxie": "YFDHXYSGPC", "name": "\u6613\u65b9\u8fbe\u6838\u5fc3\u4f18\u52bf\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAHEXINYOUSHIGUPIAOC"}, "010198": {"suoxie": "YFDJZYSQYHHA", "name": "\u6613\u65b9\u8fbe\u7ade\u4e89\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAJINGZHENGYOUSHIQIYEHUNHEA"}, "010199": {"suoxie": "CXTLAXSYHHA", "name": "\u957f\u4fe1\u6dfb\u5229\u5b89\u5fc3\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINTIANLIANXINSHOUYIHUNHEA"}, "010200": {"suoxie": "CXTLAXSYHHC", "name": "\u957f\u4fe1\u6dfb\u5229\u5b89\u5fc3\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINTIANLIANXINSHOUYIHUNHEC"}, "010201": {"suoxie": "NYHLZZYNDKHH", "name": "\u519c\u94f6\u6c47\u7406\u667a\u589e\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILIZHIZENGYINIANDINGKAIHUNHE"}, "010202": {"suoxie": "THZZKJ100ZSZQA", "name": "\u5929\u5f18\u4e2d\u8bc1\u79d1\u6280100\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGKEJI100ZHISHUZENGQIANGA"}, "010203": {"suoxie": "THZZKJ100ZSZQC", "name": "\u5929\u5f18\u4e2d\u8bc1\u79d1\u6280100\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGKEJI100ZHISHUZENGQIANGC"}, "010204": {"suoxie": "ZYGGTYSCZGP", "name": "\u4e2d\u94f6\u6e2f\u80a1\u901a\u4f18\u52bf\u6210\u957f\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINGANGGUTONGYOUSHICHENGZHANGGUPIAO"}, "010205": {"suoxie": "GSABYAHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOSHOUANBAOYUANHUNHEA"}, "010206": {"suoxie": "GSABYAHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOSHOUANBAOYUANHUNHEC"}, "010208": {"suoxie": "PARZLHBB", "name": "\u5e73\u5b89\u65e5\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANRIZENGLIHUOBIB"}, "010210": {"suoxie": "GTZZJSJZTETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u8ba1\u7b97\u673a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGJISUANJIZHUTIETFLIANJIEC"}, "010211": {"suoxie": "JSCCSXHBHHA", "name": "\u666f\u987a\u957f\u57ce\u987a\u946b\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGSHUNXINHUIBAOHUNHEA"}, "010212": {"suoxie": "JSCCSXHBHHC", "name": "\u666f\u987a\u957f\u57ce\u987a\u946b\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGSHUNXINHUIBAOHUNHEC"}, "010213": {"suoxie": "ZOHLWXFHHA", "name": "\u4e2d\u6b27\u4e92\u8054\u7f51\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHULIANWANGXIANFENGHUNHEA"}, "010214": {"suoxie": "ZOHLWXFHHC", "name": "\u4e2d\u6b27\u4e92\u8054\u7f51\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHULIANWANGXIANFENGHUNHEC"}, "010215": {"suoxie": "ZODYWJYNHHA", "name": "\u4e2d\u6b27\u8fbe\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUDAYIWENJIANYINIANHUNHEA"}, "010216": {"suoxie": "ZODYWJYNHHC", "name": "\u4e2d\u6b27\u8fbe\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUDAYIWENJIANYINIANHUNHEC"}, "010217": {"suoxie": "ZYTXFLWJYLMBYNCYHHFOF", "name": "\u4e2d\u94f6\u6dfb\u79a7\u4e30\u7984\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINTIANXIFENGLUWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "010219": {"suoxie": "HTFWJTYYNCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6dfb\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANTIANYIYINIANCHIYOUHUNHE"}, "010220": {"suoxie": "HFTXFHXHHA", "name": "\u6d77\u5bcc\u901a\u6d88\u8d39\u6838\u5fc3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGXIAOFEIHEXINHUNHEA"}, "010221": {"suoxie": "HFTXFHXHHC", "name": "\u6d77\u5bcc\u901a\u6d88\u8d39\u6838\u5fc3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGXIAOFEIHEXINHUNHEC"}, "010222": {"suoxie": "DMMFYHYNCYHH", "name": "\u5927\u6469\u6c11\u4e30\u76c8\u548c\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DAMOMINFENGYINGHEYINIANCHIYOUHUNHE"}, "010223": {"suoxie": "BSSJXCYQZQA", "name": "\u535a\u65f6\u53cc\u5b63\u4eab\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGJIXIANGCHIYOUQIZHAIQUANA"}, "010224": {"suoxie": "HFTZZ100ZSLOFC", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1100\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENG100ZHISHULOFC"}, "010225": {"suoxie": "DFHQHSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u822a\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHANGSANNIANCHIYOUHUNHEB"}, "010226": {"suoxie": "BSSJXCYQZQB", "name": "\u535a\u65f6\u53cc\u5b63\u4eab\u6301\u6709\u671f\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGJIXIANGCHIYOUQIZHAIQUANB"}, "010227": {"suoxie": "BSSJXCYQZQC", "name": "\u535a\u65f6\u53cc\u5b63\u4eab\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGJIXIANGCHIYOUQIZHAIQUANC"}, "010228": {"suoxie": "PADHHHLOFC", "name": "\u5e73\u5b89\u9f0e\u5f18\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANDINGHONGHUNHELOFC"}, "010229": {"suoxie": "PADHHHLOFD", "name": "\u5e73\u5b89\u9f0e\u5f18\u6df7\u5408(LOF)D", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANDINGHONGHUNHELOFD"}, "010230": {"suoxie": "NFBCHHA", "name": "\u5357\u65b9\u5b9d\u660c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOCHANGHUNHEA"}, "010231": {"suoxie": "NFBCHHC", "name": "\u5357\u65b9\u5b9d\u660c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOCHANGHUNHEC"}, "010232": {"suoxie": "GSABTACZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u6cf0\u5b89\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOTAIANCHUNZHAIZHAIQUAN"}, "010233": {"suoxie": "NYJRYNDKZ", "name": "\u519c\u94f6\u91d1\u6da6\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINJINRUNYINIANDINGKAIZHAI"}, "010234": {"suoxie": "HTBRLHZQHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u589e\u5f3a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAZENGQIANGHUNHEC"}, "010235": {"suoxie": "GFZYYXGPC", "name": "\u5e7f\u53d1\u8d44\u6e90\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAZIYUANYOUXUANGUPIAOC"}, "010236": {"suoxie": "GFDZXXCMGPC", "name": "\u5e7f\u53d1\u7535\u5b50\u4fe1\u606f\u4f20\u5a92\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFADIANZIXINXICHUANMEIGUPIAOC"}, "010237": {"suoxie": "AXCXXFHHFQA", "name": "\u5b89\u4fe1\u521b\u65b0\u5148\u950b\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINCHUANGXINXIANFENGHUNHEFAQIA"}, "010238": {"suoxie": "AXCXXFHHFQC", "name": "\u5b89\u4fe1\u521b\u65b0\u5148\u950b\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINCHUANGXINXIANFENGHUNHEFAQIC"}, "010239": {"suoxie": "PARSLGYCYHHA", "name": "\u5e73\u5b89\u745e\u5c1a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANRUISHANGLIUGEYUECHIYOUHUNHEA"}, "010240": {"suoxie": "PAJJX3GYCYZQA", "name": "\u5e73\u5b89\u5b63\u5b63\u4eab3\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIJIXIANG3GEYUECHIYOUZHAIQUANA"}, "010241": {"suoxie": "PAJJX3GYCYZQC", "name": "\u5e73\u5b89\u5b63\u5b63\u4eab3\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANJIJIXIANG3GEYUECHIYOUZHAIQUANC"}, "010242": {"suoxie": "PAWJZZHHA", "name": "\u5e73\u5b89\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANWENJIANZENGZHANGHUNHEA"}, "010243": {"suoxie": "PAWJZZHHC", "name": "\u5e73\u5b89\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANWENJIANZENGZHANGHUNHEC"}, "010244": {"suoxie": "PARSLGYCYHHC", "name": "\u5e73\u5b89\u745e\u5c1a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANRUISHANGLIUGEYUECHIYOUHUNHEC"}, "010245": {"suoxie": "GFPPXFGPFQSC", "name": "\u5e7f\u53d1\u54c1\u724c\u6d88\u8d39\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAPINPAIXIAOFEIGUPIAOFAQISHIC"}, "010246": {"suoxie": "HTBRLHXXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u5148\u884c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAXIANXINGHUNHEC"}, "010247": {"suoxie": "HSHTCZA", "name": "\u60e0\u5347\u548c\u6cf0\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHETAICHUNZHAIA"}, "010248": {"suoxie": "HSHTCZC", "name": "\u60e0\u5347\u548c\u6cf0\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHETAICHUNZHAIC"}, "010249": {"suoxie": "GJHCZQA", "name": "\u56fd\u91d1\u60e0\u8bda\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOJINHUICHENGZHAIQUANA"}, "010250": {"suoxie": "GJHCZQC", "name": "\u56fd\u91d1\u60e0\u8bda\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOJINHUICHENGZHAIQUANC"}, "010251": {"suoxie": "CJAXCZ18GYDKZA", "name": "\u957f\u6c5f\u5b89\u4eab\u7eaf\u503a18\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGANXIANGCHUNZHAI18GEYUEDINGKAIZHAIA"}, "010252": {"suoxie": "CJAXCZ18GYDKZC", "name": "\u957f\u6c5f\u5b89\u4eab\u7eaf\u503a18\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGJIANGANXIANGCHUNZHAI18GEYUEDINGKAIZHAIC"}, "010253": {"suoxie": "XYZZ500ZSZQA", "name": "\u5174\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENG500ZHISHUZENGQIANGA"}, "010254": {"suoxie": "JSFNYNDQCZZQA", "name": "\u5609\u5b9e\u4e30\u5e74\u4e00\u5e74\u5b9a\u671f\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIFENGNIANYINIANDINGQICHUNZHAIZHAIQUANA"}, "010255": {"suoxie": "JSFNYNDQCZZQC", "name": "\u5609\u5b9e\u4e30\u5e74\u4e00\u5e74\u5b9a\u671f\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIFENGNIANYINIANDINGQICHUNZHAIZHAIQUANC"}, "010256": {"suoxie": "NYHLJHZQC", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u6c47\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINHUIZHAIQUANC"}, "010257": {"suoxie": "THDLYN", "name": "\u5929\u5f18\u591a\u5229\u4e00\u5e74", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGDUOLIYINIAN"}, "010258": {"suoxie": "CHWYZQA", "name": "\u6df3\u539a\u7a33\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENYUEZHAIQUANA"}, "010259": {"suoxie": "CHWYZQC", "name": "\u6df3\u539a\u7a33\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENYUEZHAIQUANC"}, "010260": {"suoxie": "HFTCLSYZQA", "name": "\u6d77\u5bcc\u901a\u7b56\u7565\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGCELUESHOUYIZHAIQUANA"}, "010261": {"suoxie": "HFTCLSYZQC", "name": "\u6d77\u5bcc\u901a\u7b56\u7565\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGCELUESHOUYIZHAIQUANC"}, "010262": {"suoxie": "HFTZZ13NNFZA", "name": "\u6d77\u5bcc\u901a\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGZHONGZHAI13NIANNONGFAZHAIA"}, "010263": {"suoxie": "HFTZZ13NNFZC", "name": "\u6d77\u5bcc\u901a\u4e2d\u503a1-3\u5e74\u519c\u53d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGZHONGZHAI13NIANNONGFAZHAIC"}, "010264": {"suoxie": "PHCZZXHHA", "name": "\u9e4f\u534e\u6210\u957f\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGZHIXUANHUNHEA"}, "010265": {"suoxie": "PHCZZXHHC", "name": "\u9e4f\u534e\u6210\u957f\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGZHIXUANHUNHEC"}, "010266": {"suoxie": "XQATWJYLYNCYHHFOF", "name": "\u5174\u5168\u5b89\u6cf0\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANANTAIWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "010267": {"suoxie": "XQATYLWNCYHHFOF", "name": "\u5174\u5168\u5b89\u6cf0\u517b\u8001\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANANTAIYANGLAOWUNIANCHIYOUHUNHEFOF"}, "010268": {"suoxie": "TPRAHHA", "name": "\u592a\u5e73\u777f\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIANHUNHEA"}, "010269": {"suoxie": "TPRAHHC", "name": "\u592a\u5e73\u777f\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIANHUNHEC"}, "010270": {"suoxie": "HAJYYNCYQZQC", "name": "\u6c47\u5b89\u5609\u76c8\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUIANJIAYINGYINIANCHIYOUQIZHAIQUANC"}, "010271": {"suoxie": "GFJZCZYNCYQHHA", "name": "\u56fd\u5bcc\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJIAZHICHENGZHANGYINIANCHIYOUQIHUNHEA"}, "010272": {"suoxie": "GFJZCZYNCYQHHC", "name": "\u56fd\u5bcc\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJIAZHICHENGZHANGYINIANCHIYOUQIHUNHEC"}, "010273": {"suoxie": "JSJZCQHHA", "name": "\u5609\u5b9e\u4ef7\u503c\u957f\u9752\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHICHANGQINGHUNHEA"}, "010274": {"suoxie": "JSJZCQHHC", "name": "\u5609\u5b9e\u4ef7\u503c\u957f\u9752\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHICHANGQINGHUNHEC"}, "010275": {"suoxie": "JSYZJXHHA", "name": "\u5609\u5b9e\u4f18\u8d28\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUZHIJINGXUANHUNHEA"}, "010276": {"suoxie": "JSYZJXHHC", "name": "\u5609\u5b9e\u4f18\u8d28\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUZHIJINGXUANHUNHEC"}, "010277": {"suoxie": "JSMATSWJYLYNCYQHHFOF", "name": "\u5609\u5b9e\u6c11\u5b89\u6dfb\u5c81\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIMINANTIANSUIWENJIANYANGLAOYINIANCHIYOUQIHUNHEFOF"}, "010278": {"suoxie": "NHRT39GYDKA", "name": "\u5357\u534e\u745e\u6cf039\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUITAI39GEYUEDINGKAIA"}, "010279": {"suoxie": "NHRT39GYDKC", "name": "\u5357\u534e\u745e\u6cf039\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUITAI39GEYUEDINGKAIC"}, "010281": {"suoxie": "HXBSYLYNCYHHFOF", "name": "\u534e\u590f\u4fdd\u5b88\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIABAOSHOUYANGLAOYINIANCHIYOUHUNHEFOF"}, "010282": {"suoxie": "ZXJTZXSHHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u667a\u4eab\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUZHIXIANGSHENGHUOHUNHEA"}, "010283": {"suoxie": "ZXJTZXSHHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u667a\u4eab\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUZHIXIANGSHENGHUOHUNHEC"}, "010284": {"suoxie": "CCJZCZLGYCYQHHA", "name": "\u957f\u57ce\u4ef7\u503c\u6210\u957f\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHICHENGZHANGLIUGEYUECHIYOUQIHUNHEA"}, "010285": {"suoxie": "CCJZCZLGYCYQHHC", "name": "\u957f\u57ce\u4ef7\u503c\u6210\u957f\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHICHENGZHANGLIUGEYUECHIYOUQIHUNHEC"}, "010286": {"suoxie": "HFTCZJZHHA", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGJIAZHIHUNHEA"}, "010287": {"suoxie": "HFTCZJZHHC", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGJIAZHIHUNHEC"}, "010288": {"suoxie": "MSJYXJBHBB", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINBAOHUOBIB"}, "010289": {"suoxie": "JSCCCYQSHH", "name": "\u666f\u987a\u957f\u57ce\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGCHANYEQUSHIHUNHE"}, "010291": {"suoxie": "HTBRYJJXC", "name": "\u534e\u6cf0\u67cf\u745e\u7814\u7a76\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYANJIUJINGXUANC"}, "010292": {"suoxie": "DFHHXYXDKHHC", "name": "\u4e1c\u65b9\u7ea2\u6838\u5fc3\u4f18\u9009\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGHEXINYOUXUANDINGKAIHUNHEC"}, "010293": {"suoxie": "HSLHYZJXHH", "name": "\u534e\u5546\u91cf\u5316\u4f18\u8d28\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGLIANGHUAYOUZHIJINGXUANHUNHE"}, "010294": {"suoxie": "HTHTSX39GYDKZA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u517439\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTUSHENGXING39GEYUEDINGKAIZHAIA"}, "010295": {"suoxie": "HTHTSX39GYDKZC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u517439\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTUSHENGXING39GEYUEDINGKAIZHAIC"}, "010296": {"suoxie": "WJHLHTZGYSA", "name": "\u4e07\u5bb6\u4e92\u8054\u4e92\u901a\u4e2d\u56fd\u4f18\u52bfA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHULIANHUTONGZHONGGUOYOUSHIA"}, "010297": {"suoxie": "WJHLHTZGYSC", "name": "\u4e07\u5bb6\u4e92\u8054\u4e92\u901a\u4e2d\u56fd\u4f18\u52bfC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHULIANHUTONGZHONGGUOYOUSHIC"}, "010298": {"suoxie": "HTFPPQDLGYCYHH", "name": "\u6c47\u6dfb\u5bcc\u54c1\u724c\u9a71\u52a8\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUPINPAIQUDONGLIUGEYUECHIYOUHUNHE"}, "010299": {"suoxie": "NFCYSJHHA", "name": "\u5357\u65b9\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHANYESHENGJIHUNHEA"}, "010300": {"suoxie": "NFCYSJHHC", "name": "\u5357\u65b9\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHANYESHENGJIHUNHEC"}, "010301": {"suoxie": "DCCZXFHHA", "name": "\u8fbe\u8bda\u6210\u957f\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGXIANFENGHUNHEA"}, "010302": {"suoxie": "DCCZXFHHC", "name": "\u8fbe\u8bda\u6210\u957f\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGXIANFENGHUNHEC"}, "010303": {"suoxie": "HTBRLHCYHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u521b\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUACHUANGYINGHUNHEA"}, "010304": {"suoxie": "HTBRLHCYHHC", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u521b\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUACHUANGYINGHUNHEC"}, "010305": {"suoxie": "HXCXQDHHA", "name": "\u534e\u590f\u521b\u65b0\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINQUDONGHUNHEA"}, "010306": {"suoxie": "HXCXQDHHC", "name": "\u534e\u590f\u521b\u65b0\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINQUDONGHUNHEC"}, "010307": {"suoxie": "DCXXCYJXA", "name": "\u4e1c\u8d22\u4fe1\u606f\u4ea7\u4e1a\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAIXINXICHANYEJINGXUANA"}, "010308": {"suoxie": "DCXXCYJXC", "name": "\u4e1c\u8d22\u4fe1\u606f\u4ea7\u4e1a\u7cbe\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAIXINXICHANYEJINGXUANC"}, "010309": {"suoxie": "DBRYLLZZQA", "name": "\u5fb7\u90a6\u9510\u88d5\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIYULILVZHAIZHAIQUANA"}, "010310": {"suoxie": "DBRYLLZZQC", "name": "\u5fb7\u90a6\u9510\u88d5\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIYULILVZHAIZHAIQUANC"}, "010311": {"suoxie": "ZYLHJZHHC", "name": "\u4e2d\u94f6\u91cf\u5316\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINLIANGHUAJIAZHIHUNHEC"}, "010312": {"suoxie": "ZYJRDCHHC", "name": "\u4e2d\u94f6\u91d1\u878d\u5730\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINJINRONGDICHANHUNHEC"}, "010313": {"suoxie": "SYXHHHA", "name": "\u4e0a\u94f6\u946b\u6052\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINXINHENGHUNHEA"}, "010314": {"suoxie": "DMNXZZHHA", "name": "\u5927\u6469\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMONEIXUZENGZHANGHUNHEA"}, "010320": {"suoxie": "HAYL2040SNCYHHFOF", "name": "\u534e\u5b89\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "010321": {"suoxie": "ZYDJKGPC", "name": "\u4e2d\u94f6\u5927\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINDAJIANKANGGUPIAOC"}, "010322": {"suoxie": "DMXXCYGP", "name": "\u5927\u6469\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "DAMOXINXINGCHANYEGUPIAO"}, "010323": {"suoxie": "HAPHYLSNCYFOF", "name": "\u534e\u5b89\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANPINGHENGYANGLAOSANNIANCHIYOUFOF"}, "010324": {"suoxie": "GFZCDZZQE", "name": "\u5e7f\u53d1\u62db\u8d22\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZHAOCAIDUANZHAIZHAIQUANE"}, "010325": {"suoxie": "SWLXSYBHBE", "name": "\u7533\u4e07\u83f1\u4fe1\u6536\u76ca\u5b9d\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "SHENWANLINGXINSHOUYIBAOHUOBIE"}, "010326": {"suoxie": "BSXFCXHHA", "name": "\u535a\u65f6\u6d88\u8d39\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXIAOFEICHUANGXINHUNHEA"}, "010327": {"suoxie": "BSXFCXHHC", "name": "\u535a\u65f6\u6d88\u8d39\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXIAOFEICHUANGXINHUNHEC"}, "010328": {"suoxie": "BSRHLHPZHHA", "name": "\u535a\u65f6\u8363\u534e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGHUALINGHUOPEIZHIHUNHEA"}, "010329": {"suoxie": "BSRHLHPZHHC", "name": "\u535a\u65f6\u8363\u534e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRONGHUALINGHUOPEIZHIHUNHEC"}, "010330": {"suoxie": "DWXXCZHHA", "name": "\u4e1c\u5434\u5174\u4eab\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXINGXIANGCHENGZHANGHUNHEA"}, "010331": {"suoxie": "THXFGPA", "name": "\u5929\u5f18\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TIANHONGXIAOFEIGUPIAOA"}, "010332": {"suoxie": "THXFGPC", "name": "\u5929\u5f18\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TIANHONGXIAOFEIGUPIAOC"}, "010333": {"suoxie": "HXHXZCHHA", "name": "\u534e\u590f\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINZICHANHUNHEA"}, "010334": {"suoxie": "HXHXZCHHC", "name": "\u534e\u590f\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINZICHANHUNHEC"}, "010335": {"suoxie": "HBJZYSHH", "name": "\u534e\u5b9d\u7ade\u4e89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOJINGZHENGYOUSHIHUNHE"}, "010336": {"suoxie": "ZOYXSHHHA", "name": "\u4e2d\u6b27\u60a6\u4eab\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUEXIANGSHENGHUOHUNHEA"}, "010337": {"suoxie": "ZOYXSHHHC", "name": "\u4e2d\u6b27\u60a6\u4eab\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUEXIANGSHENGHUOHUNHEC"}, "010338": {"suoxie": "GTRYYJCZHHA", "name": "\u56fd\u6295\u745e\u94f6\u8fdc\u89c1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINYUANJIANCHENGZHANGHUNHEA"}, "010339": {"suoxie": "GTRYYJCZHHC", "name": "\u56fd\u6295\u745e\u94f6\u8fdc\u89c1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINYUANJIANCHENGZHANGHUNHEC"}, "010340": {"suoxie": "YFDGZLYXSNCY", "name": "\u6613\u65b9\u8fbe\u9ad8\u8d28\u91cf\u4e25\u9009\u4e09\u5e74\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGAOZHILIANGYANXUANSANNIANCHIYOU"}, "010341": {"suoxie": "ZSCYJXGPA", "name": "\u62db\u5546\u4ea7\u4e1a\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGCHANYEJINGXUANGUPIAOA"}, "010342": {"suoxie": "ZSCYJXGPC", "name": "\u62db\u5546\u4ea7\u4e1a\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGCHANYEJINGXUANGUPIAOC"}, "010343": {"suoxie": "HBFS100QDIIA", "name": "\u534e\u5b9d\u5bcc\u65f6100(QDII)A", "style": "QDII", "pinyin": "HUABAOFUSHI100QDIIA"}, "010344": {"suoxie": "HBFS100QDIIC", "name": "\u534e\u5b9d\u5bcc\u65f6100(QDII)C", "style": "QDII", "pinyin": "HUABAOFUSHI100QDIIC"}, "010345": {"suoxie": "HTBRCZZXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u6210\u957f\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUICHENGZHANGZHIXUANHUNHEA"}, "010346": {"suoxie": "HTBRCZZXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u6210\u957f\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUICHENGZHANGZHIXUANHUNHEC"}, "010347": {"suoxie": "NYCLSYYNCYHH", "name": "\u519c\u94f6\u7b56\u7565\u6536\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINCELUESHOUYIYINIANCHIYOUHUNHE"}, "010348": {"suoxie": "JSCCTBSGYDKHH", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u4fdd\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGTAIBAOSANGEYUEDINGKAIHUNHE"}, "010349": {"suoxie": "NADTJJGPC", "name": "\u8bfa\u5b89\u4f4e\u78b3\u7ecf\u6d4e\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANDITANJINGJIGUPIAOC"}, "010350": {"suoxie": "JSCCPZCQHH", "name": "\u666f\u987a\u957f\u57ce\u54c1\u8d28\u957f\u9752\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGPINZHICHANGQINGHUNHE"}, "010351": {"suoxie": "NAZZ100ZSC", "name": "\u8bfa\u5b89\u4e2d\u8bc1100\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANZHONGZHENG100ZHISHUC"}, "010352": {"suoxie": "NAHS300ZSZQC", "name": "\u8bfa\u5b89\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANHUSHEN300ZHISHUZENGQIANGC"}, "010353": {"suoxie": "NFCYCZA", "name": "\u5357\u65b9\u5d07\u5143\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHONGYUANCHUNZHAIA"}, "010354": {"suoxie": "NFCYCZC", "name": "\u5357\u65b9\u5d07\u5143\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGCHONGYUANCHUNZHAIC"}, "010355": {"suoxie": "NAZZ500ZSZQC", "name": "\u8bfa\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANZHONGZHENG500ZHISHUZENGQIANGC"}, "010356": {"suoxie": "NACYBZSZQLOFC", "name": "\u8bfa\u5b89\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANCHUANGYEBANZHISHUZENGQIANGLOFC"}, "010357": {"suoxie": "NFAEFHHA", "name": "\u5357\u65b9\u963f\u5c14\u6cd5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGAERFAHUNHEA"}, "010358": {"suoxie": "NFAEFHHC", "name": "\u5357\u65b9\u963f\u5c14\u6cd5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGAERFAHUNHEC"}, "010361": {"suoxie": "JSPZYXGPA", "name": "\u5609\u5b9e\u54c1\u8d28\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIPINZHIYOUXUANGUPIAOA"}, "010362": {"suoxie": "JSPZYXGPC", "name": "\u5609\u5b9e\u54c1\u8d28\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIASHIPINZHIYOUXUANGUPIAOC"}, "010363": {"suoxie": "XDAYJXZXLNCYQHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5320\u5fc3\u81fb\u9009\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINJIANGXINZHENXUANLIANGNIANCHIYOUQIHUNHE"}, "010364": {"suoxie": "PHKTJGZSLOFC", "name": "\u9e4f\u534e\u7a7a\u5929\u519b\u5de5\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAKONGTIANJUNGONGZHISHULOFC"}, "010365": {"suoxie": "PHXGYHZSLOFC", "name": "\u9e4f\u534e\u9999\u6e2f\u94f6\u884c\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAXIANGGANGYINHANGZHISHULOFC"}, "010366": {"suoxie": "PHZZYYWSLOFC", "name": "\u9e4f\u534e\u4e2d\u8bc1\u533b\u836f\u536b\u751f(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYIYAOWEISHENGLOFC"}, "010367": {"suoxie": "ZRJRYNCYHHA", "name": "\u4e2d\u878d\u666f\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGRUIYINIANCHIYOUHUNHEA"}, "010368": {"suoxie": "ZRJRYNCYHHC", "name": "\u4e2d\u878d\u666f\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGRUIYINIANCHIYOUHUNHEC"}, "010369": {"suoxie": "DCZXYNCYHHA", "name": "\u5927\u6210\u5353\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGZHUOXIANGYINIANCHIYOUHUNHEA"}, "010370": {"suoxie": "DCZXYNCYHHC", "name": "\u5927\u6210\u5353\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGZHUOXIANGYINIANCHIYOUHUNHEC"}, "010371": {"suoxie": "DCCZJQHHA", "name": "\u5927\u6210\u6210\u957f\u8fdb\u53d6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGJINQUHUNHEA"}, "010372": {"suoxie": "DCCZJQHHC", "name": "\u5927\u6210\u6210\u957f\u8fdb\u53d6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGJINQUHUNHEC"}, "010373": {"suoxie": "XBLDJXYNDKHHA", "name": "\u897f\u90e8\u5229\u5f97\u805a\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XIBULIDEJUXINGYINIANDINGKAIHUNHEA"}, "010374": {"suoxie": "XBLDJXYNDKHHC", "name": "\u897f\u90e8\u5229\u5f97\u805a\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XIBULIDEJUXINGYINIANDINGKAIHUNHEC"}, "010375": {"suoxie": "GJXYJJXDNA", "name": "\u56fd\u91d1\u946b\u60a6\u7ecf\u6d4e\u65b0\u52a8\u80fdA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINYUEJINGJIXINDONGNENGA"}, "010376": {"suoxie": "GJXYJJXDNC", "name": "\u56fd\u91d1\u946b\u60a6\u7ecf\u6d4e\u65b0\u52a8\u80fdC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINYUEJINGJIXINDONGNENGC"}, "010377": {"suoxie": "GFJZHXHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u6838\u5fc3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIHEXINHUNHEA"}, "010378": {"suoxie": "GFJZHXHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u6838\u5fc3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIHEXINHUNHEC"}, "010379": {"suoxie": "GFJHYXHHA", "name": "\u5e7f\u53d1\u5747\u8861\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUNHENGYOUXUANHUNHEA"}, "010380": {"suoxie": "GFJHYXHHC", "name": "\u5e7f\u53d1\u5747\u8861\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUNHENGYOUXUANHUNHEC"}, "010381": {"suoxie": "ZSZXJZHHA", "name": "\u6d59\u5546\u667a\u9009\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANJIAZHIHUNHEA"}, "010382": {"suoxie": "ZSZXJZHHC", "name": "\u6d59\u5546\u667a\u9009\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANJIAZHIHUNHEC"}, "010383": {"suoxie": "BYJCCYHHA", "name": "\u5b9d\u76c8\u57fa\u7840\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGJICHUCHANYEHUNHEA"}, "010384": {"suoxie": "BYJCCYHHC", "name": "\u5b9d\u76c8\u57fa\u7840\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGJICHUCHANYEHUNHEC"}, "010385": {"suoxie": "HAHJJXHHA", "name": "\u534e\u5b89\u6c47\u5609\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUIJIAJINGXUANHUNHEA"}, "010386": {"suoxie": "HAHJJXHHC", "name": "\u534e\u5b89\u6c47\u5609\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUIJIAJINGXUANHUNHEC"}, "010387": {"suoxie": "YFDYYSWGPA", "name": "\u6613\u65b9\u8fbe\u533b\u836f\u751f\u7269\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAYIYAOSHENGWUGUPIAOA"}, "010388": {"suoxie": "YFDYYSWGPC", "name": "\u6613\u65b9\u8fbe\u533b\u836f\u751f\u7269\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAYIYAOSHENGWUGUPIAOC"}, "010389": {"suoxie": "YFDKYHHA", "name": "\u6613\u65b9\u8fbe\u79d1\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKEYIHUNHEA"}, "010390": {"suoxie": "YFDKYHHC", "name": "\u6613\u65b9\u8fbe\u79d1\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKEYIHUNHEC"}, "010391": {"suoxie": "YFDZLXXCYGPA", "name": "\u6613\u65b9\u8fbe\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAZHANLUEXINXINGCHANYEGUPIAOA"}, "010392": {"suoxie": "YFDZLXXCYGPC", "name": "\u6613\u65b9\u8fbe\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAZHANLUEXINXINGCHANYEGUPIAOC"}, "010393": {"suoxie": "GYJKSHHHA", "name": "\u5de5\u94f6\u5065\u5eb7\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJIANKANGSHENGHUOHUNHEA"}, "010394": {"suoxie": "GYJKSHHHC", "name": "\u5de5\u94f6\u5065\u5eb7\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJIANKANGSHENGHUOHUNHEC"}, "010395": {"suoxie": "RTWJTYLHPZHHA", "name": "\u878d\u901a\u7a33\u5065\u6dfb\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGWENJIANTIANYINGLINGHUOPEIZHIHUNHEA"}, "010396": {"suoxie": "RTWJTYLHPZHHC", "name": "\u878d\u901a\u7a33\u5065\u6dfb\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGWENJIANTIANYINGLINGHUOPEIZHIHUNHEC"}, "010397": {"suoxie": "ZJRHCZZQ", "name": "\u4e2d\u52a0\u745e\u5408\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUIHECHUNZHAIZHAIQUAN"}, "010398": {"suoxie": "ZJKXHHA", "name": "\u4e2d\u52a0\u79d1\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEXIANGHUNHEA"}, "010399": {"suoxie": "ZJKXHHC", "name": "\u4e2d\u52a0\u79d1\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEXIANGHUNHEC"}, "010400": {"suoxie": "FGRTSGYDKHHFQS", "name": "\u5bcc\u56fd\u878d\u6cf0\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUORONGTAISANGEYUEDINGKAIHUNHEFAQISHI"}, "010401": {"suoxie": "XHAKDYSYYNCYA", "name": "\u65b0\u534e\u5b89\u5eb7\u591a\u5143\u6536\u76ca\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINHUAANKANGDUOYUANSHOUYIYINIANCHIYOUA"}, "010402": {"suoxie": "XHAKDYSYYNCYC", "name": "\u65b0\u534e\u5b89\u5eb7\u591a\u5143\u6536\u76ca\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINHUAANKANGDUOYUANSHOUYIYINIANCHIYOUC"}, "010403": {"suoxie": "HSJQYXHH", "name": "\u534e\u5546\u666f\u6c14\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJINGQIYOUXUANHUNHE"}, "010404": {"suoxie": "BDSL6GYCYQHH", "name": "\u535a\u9053\u76db\u52296\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BODAOSHENGLI6GEYUECHIYOUQIHUNHE"}, "010405": {"suoxie": "HSYYJK6GYCYQHH", "name": "\u60e0\u5347\u533b\u836f\u5065\u5eb76\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUISHENGYIYAOJIANKANG6GEYUECHIYOUQIHUNHE"}, "010406": {"suoxie": "AXZZ13NZCXJRZA", "name": "\u5b89\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIA"}, "010407": {"suoxie": "AXZZ13NZCXJRZC", "name": "\u5b89\u4fe1\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIC"}, "010408": {"suoxie": "AXHY6GYCYHH", "name": "\u5b89\u4fe1\u6d69\u76c86\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINHAOYING6GEYUECHIYOUHUNHE"}, "010409": {"suoxie": "FGXFJX30GP", "name": "\u5bcc\u56fd\u6d88\u8d39\u7cbe\u900930\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOXIAOFEIJINGXUAN30GUPIAO"}, "010410": {"suoxie": "CCPZCZHHA", "name": "\u957f\u57ce\u54c1\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGPINZHICHENGZHANGHUNHEA"}, "010411": {"suoxie": "CCPZCZHHC", "name": "\u957f\u57ce\u54c1\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGPINZHICHENGZHANGHUNHEC"}, "010412": {"suoxie": "HAJHYXHH", "name": "\u6c47\u5b89\u5747\u8861\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANJUNHENGYOUXUANHUNHE"}, "010413": {"suoxie": "CTZGCRYNCYHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u5bb8\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANCHENRUIYINIANCHIYOUHUNHEA"}, "010414": {"suoxie": "CTZGCRYNCYHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u5bb8\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANCHENRUIYINIANCHIYOUHUNHEC"}, "010415": {"suoxie": "HTBRZLJXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGJINGXUANHUNHEA"}, "010416": {"suoxie": "HTBRZLJXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGJINGXUANHUNHEC"}, "010417": {"suoxie": "ZSSY3GYDKHH", "name": "\u62db\u5546\u76db\u6d0b3\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGSHENGYANG3GEYUEDINGKAIHUNHE"}, "010418": {"suoxie": "CTJQHYHH", "name": "\u8d22\u901a\u666f\u6c14\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGJINGQIHANGYEHUNHE"}, "010419": {"suoxie": "SWLXZZHBCYZSLOFC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGHUANBAOCHANYEZHISHULOFC"}, "010420": {"suoxie": "MSJYCZYXGP", "name": "\u6c11\u751f\u52a0\u94f6\u6210\u957f\u4f18\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINCHENGZHANGYOUXUANGUPIAO"}, "010421": {"suoxie": "HFTXFYXHHA", "name": "\u6d77\u5bcc\u901a\u6d88\u8d39\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGXIAOFEIYOUXUANHUNHEA"}, "010422": {"suoxie": "HFTXFYXHHC", "name": "\u6d77\u5bcc\u901a\u6d88\u8d39\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGXIAOFEIYOUXUANHUNHEC"}, "010423": {"suoxie": "GTRYJZCZYNCYHHA", "name": "\u56fd\u6295\u745e\u94f6\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINJIAZHICHENGZHANGYINIANCHIYOUHUNHEA"}, "010424": {"suoxie": "GTRYJZCZYNCYHHC", "name": "\u56fd\u6295\u745e\u94f6\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINJIAZHICHENGZHANGYINIANCHIYOUHUNHEC"}, "010425": {"suoxie": "GTRYKFSJJXHHA", "name": "\u56fd\u6295\u745e\u94f6\u5f00\u653e\u89c6\u89d2\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINKAIFANGSHIJIAOJINGXUANHUNHEA"}, "010426": {"suoxie": "GTRYKFSJJXHHC", "name": "\u56fd\u6295\u745e\u94f6\u5f00\u653e\u89c6\u89d2\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINKAIFANGSHIJIAOJINGXUANHUNHEC"}, "010427": {"suoxie": "XYCLZXHHA", "name": "\u5174\u94f6\u7b56\u7565\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINCELUEZHIXUANHUNHEA"}, "010428": {"suoxie": "XYCLZXHHC", "name": "\u5174\u94f6\u7b56\u7565\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINCELUEZHIXUANHUNHEC"}, "010429": {"suoxie": "ZORJHH", "name": "\u4e2d\u6b27\u777f\u89c1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOURUIJIANHUNHE"}, "010430": {"suoxie": "ZSAYZQA", "name": "\u62db\u5546\u5b89\u9633\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANYANGZHAIQUANA"}, "010431": {"suoxie": "ZSAYZQC", "name": "\u62db\u5546\u5b89\u9633\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANYANGZHAIQUANC"}, "010432": {"suoxie": "GFZXQY300ETFLJC", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u4f01\u4e1a300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGXIAOQIYE300ETFLIANJIEC"}, "010433": {"suoxie": "GFXXCYHHC", "name": "\u5e7f\u53d1\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAXINXINGCHANYEHUNHEC"}, "010434": {"suoxie": "HTCXYLBJGP", "name": "\u7ea2\u571f\u521b\u65b0\u533b\u7597\u4fdd\u5065\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HONGTUCHUANGXINYILIAOBAOJIANGUPIAO"}, "010435": {"suoxie": "FGSZZQZQA", "name": "\u5bcc\u56fd\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHUANGZHAIZENGQIANGZHAIQUANA"}, "010436": {"suoxie": "FGSZZQZQC", "name": "\u5bcc\u56fd\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHUANGZHAIZENGQIANGZHAIQUANC"}, "010437": {"suoxie": "JSJZLYXHHA", "name": "\u5609\u5b9e\u7ade\u4e89\u529b\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJINGZHENGLIYOUXUANHUNHEA"}, "010438": {"suoxie": "JSJZLYXHHC", "name": "\u5609\u5b9e\u7ade\u4e89\u529b\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJINGZHENGLIYOUXUANHUNHEC"}, "010439": {"suoxie": "HTFWJHYYNCYQHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6c47\u76c8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANHUIYINGYINIANCHIYOUQIHUNHE"}, "010440": {"suoxie": "NDAHCZ", "name": "\u8bfa\u5fb7\u5b89\u9e3f\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUODEANHONGCHUNZHAI"}, "010442": {"suoxie": "DFHQSSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u76db\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQISHENGSANNIANCHIYOUHUNHEB"}, "010443": {"suoxie": "DFHQCSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u7a0b\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQICHENGSANNIANCHIYOUHUNHEB"}, "010444": {"suoxie": "NFYSYNCYQHHA", "name": "\u5357\u65b9\u8a89\u5c1a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUSHANGYINIANCHIYOUQIHUNHEA"}, "010445": {"suoxie": "NFYSYNCYQHHC", "name": "\u5357\u65b9\u8a89\u5c1a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUSHANGYINIANCHIYOUQIHUNHEC"}, "010446": {"suoxie": "GTJFSGYDKHH", "name": "\u56fd\u6cf0\u91d1\u798f\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINFUSANGEYUEDINGKAIHUNHE"}, "010447": {"suoxie": "ZYWLCZHHA", "name": "\u4e2d\u90ae\u672a\u6765\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUWEILAICHENGZHANGHUNHEA"}, "010448": {"suoxie": "ZYWLCZHHC", "name": "\u4e2d\u90ae\u672a\u6765\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUWEILAICHENGZHANGHUNHEC"}, "010449": {"suoxie": "GFHYZQA", "name": "\u5e7f\u53d1\u6052\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAHENGYUEZHAIQUANA"}, "010450": {"suoxie": "GFHYZQC", "name": "\u5e7f\u53d1\u6052\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAHENGYUEZHAIQUANC"}, "010451": {"suoxie": "GFHYZQE", "name": "\u5e7f\u53d1\u6052\u60a6\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAHENGYUEZHAIQUANE"}, "010452": {"suoxie": "GFRFJXHHA", "name": "\u5e7f\u53d1\u745e\u798f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIFUJINGXUANHUNHEA"}, "010453": {"suoxie": "GFRFJXHHC", "name": "\u5e7f\u53d1\u745e\u798f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIFUJINGXUANHUNHEC"}, "010454": {"suoxie": "JYNXZZYNCYHH", "name": "\u4ea4\u94f6\u5185\u9700\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINNEIXUZENGZHANGYINIANCHIYOUHUNHE"}, "010455": {"suoxie": "BSCYJXHHA", "name": "\u535a\u65f6\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEJINGXUANHUNHEA"}, "010456": {"suoxie": "BSCYJXHHC", "name": "\u535a\u65f6\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEJINGXUANHUNHEC"}, "010457": {"suoxie": "GFRXHHA", "name": "\u5e7f\u53d1\u777f\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIXINHUNHEA"}, "010458": {"suoxie": "GFRXHHC", "name": "\u5e7f\u53d1\u777f\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIXINHUNHEC"}, "010459": {"suoxie": "XYQLZQ", "name": "\u946b\u5143\u4e7e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANQIANLIZHAIQUAN"}, "010460": {"suoxie": "XYYJJXHH", "name": "\u5174\u4e1a\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEYANJIUJINGXUANHUNHE"}, "010461": {"suoxie": "MSJYKLHH", "name": "\u6c11\u751f\u52a0\u94f6\u5eb7\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINKANGLIHUNHE"}, "010462": {"suoxie": "ZXBCJR66GYDKZ", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5609\u6da666\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJIARUN66GEYUEDINGKAIZHAI"}, "010463": {"suoxie": "PYCW66GYDKZA", "name": "\u9e4f\u626c\u6df3\u7a3366\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNWEN66GEYUEDINGKAIZHAIA"}, "010464": {"suoxie": "PYCW66GYDKZC", "name": "\u9e4f\u626c\u6df3\u7a3366\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNWEN66GEYUEDINGKAIZHAIC"}, "010465": {"suoxie": "PYJCHHA", "name": "\u9e4f\u626c\u666f\u521b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGCHUANGHUNHEA"}, "010466": {"suoxie": "PYJCHHC", "name": "\u9e4f\u626c\u666f\u521b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGCHUANGHUNHEC"}, "010467": {"suoxie": "HAJR05NJRZDKZ", "name": "\u534e\u5b89\u9526\u6eb60-5\u5e74\u91d1\u878d\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANJINRONG05NIANJINRONGZHAIDINGKAIZHAI"}, "010468": {"suoxie": "GFHB63GYDQKFZQ", "name": "\u56fd\u5bcc\u6052\u535a63\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOFUHENGBO63GEYUEDINGQIKAIFANGZHAIQUAN"}, "010469": {"suoxie": "YXYFJYGPA", "name": "\u5706\u4fe1\u6c38\u4e30\u805a\u4f18\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "YUANXINYONGFENGJUYOUGUPIAOA"}, "010470": {"suoxie": "YXYFJYGPC", "name": "\u5706\u4fe1\u6c38\u4e30\u805a\u4f18\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "YUANXINYONGFENGJUYOUGUPIAOC"}, "010471": {"suoxie": "YFDNNHSCZYNDKA", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u5b9e\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGSHICHUNZHAIYINIANDINGKAIA"}, "010472": {"suoxie": "YFDNNHSCZYNDKC", "name": "\u6613\u65b9\u8fbe\u5e74\u5e74\u6052\u5b9e\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDANIANNIANHENGSHICHUNZHAIYINIANDINGKAIC"}, "010473": {"suoxie": "HFAHZQA", "name": "\u534e\u5bcc\u5b89\u534e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANHUAZHAIQUANA"}, "010474": {"suoxie": "HFAHZQC", "name": "\u534e\u5bcc\u5b89\u534e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANHUAZHAIQUANC"}, "010475": {"suoxie": "STAXHBYNCYZA", "name": "\u4e0a\u6295\u5b89\u4eab\u56de\u62a5\u4e00\u5e74\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUANXIANGHUIBAOYINIANCHIYOUZHAIA"}, "010476": {"suoxie": "TPHJCZ", "name": "\u592a\u5e73\u6052\u4e45\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGJIUCHUNZHAI"}, "010477": {"suoxie": "JSCCJTYLCZZQ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u76ca\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIYILICHUNZHAIZHAIQUAN"}, "010478": {"suoxie": "JSCCTXHBHH", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u7965\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGTAIXIANGHUIBAOHUNHE"}, "010479": {"suoxie": "PHFYZQ", "name": "\u9e4f\u534e\u4e30\u9890\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGYIZHAIQUAN"}, "010480": {"suoxie": "HTFWJSYYNCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u8fdb\u53cc\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJINSHUANGYINGYINIANCHIYOUHUNHE"}, "010481": {"suoxie": "HTFGZLCZJX2NHH", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u8d28\u91cf\u6210\u957f\u7cbe\u90092\u5e74\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUGAOZHILIANGCHENGZHANGJINGXUAN2NIANHUNHE"}, "010482": {"suoxie": "HTFSH66GYDKZ", "name": "\u6c47\u6dfb\u5bcc\u76db\u548c66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUSHENGHE66GEYUEDINGKAIZHAI"}, "010483": {"suoxie": "JYQDHH", "name": "\u4ea4\u94f6\u542f\u9053\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQIDAOHUNHE"}, "010484": {"suoxie": "ZYLHJXHHC", "name": "\u4e2d\u94f6\u91cf\u5316\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINLIANGHUAJINGXUANHUNHEC"}, "010485": {"suoxie": "ZHRC87GYDKZA", "name": "\u4e2d\u822a\u745e\u666887\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUICHEN87GEYUEDINGKAIZHAIA"}, "010486": {"suoxie": "ZHRC87GYDKZC", "name": "\u4e2d\u822a\u745e\u666887\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUICHEN87GEYUEDINGKAIZHAIC"}, "010487": {"suoxie": "ZYSYHBYNCYHH", "name": "\u4e2d\u94f6\u987a\u76c8\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNYINGHUIBAOYINIANCHIYOUHUNHE"}, "010488": {"suoxie": "PHYXCZHHA", "name": "\u9e4f\u534e\u4f18\u9009\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYOUXUANCHENGCHANGHUNHEA"}, "010489": {"suoxie": "PHYXCZHHC", "name": "\u9e4f\u534e\u4f18\u9009\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYOUXUANCHENGCHANGHUNHEC"}, "010490": {"suoxie": "PHGZLZZHHA", "name": "\u9e4f\u534e\u9ad8\u8d28\u91cf\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAGAOZHILIANGZENGZHANGHUNHEA"}, "010491": {"suoxie": "PHGZLZZHHC", "name": "\u9e4f\u534e\u9ad8\u8d28\u91cf\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAGAOZHILIANGZENGZHANGHUNHEC"}, "010493": {"suoxie": "ZHRYYNDKZA", "name": "\u4e2d\u822a\u745e\u6631\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHANGRUIYUYINIANDINGKAIZHAIA"}, "010494": {"suoxie": "ZHRYYNDKZC", "name": "\u4e2d\u822a\u745e\u6631\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHANGRUIYUYINIANDINGKAIZHAIC"}, "010495": {"suoxie": "CJHXCXQDGPA", "name": "\u521b\u91d1\u5408\u4fe1\u521b\u65b0\u9a71\u52a8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINCHUANGXINQUDONGGUPIAOA"}, "010496": {"suoxie": "CJHXCXQDGPC", "name": "\u521b\u91d1\u5408\u4fe1\u521b\u65b0\u9a71\u52a8\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINCHUANGXINQUDONGGUPIAOC"}, "010497": {"suoxie": "GDBDXZZ15NZJZA", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDABAODEXINZHONGZHAI15NIANZHENGJINZHAIA"}, "010498": {"suoxie": "GJHNZDQLLZA", "name": "\u56fd\u91d1\u60e0\u5b81\u4e2d\u77ed\u671f\u5229\u7387\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINHUININGZHONGDUANQILILVZHAIA"}, "010499": {"suoxie": "GJHNZDQLLZC", "name": "\u56fd\u91d1\u60e0\u5b81\u4e2d\u77ed\u671f\u5229\u7387\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOJINHUININGZHONGDUANQILILVZHAIC"}, "010500": {"suoxie": "ZYCXYLHHC", "name": "\u4e2d\u94f6\u521b\u65b0\u533b\u7597\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCHUANGXINYILIAOHUNHEC"}, "010501": {"suoxie": "ZTQYAY66GYDKZ", "name": "\u4e2d\u6cf0\u9752\u6708\u5b89\u76c866\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGTAIQINGYUEANYING66GEYUEDINGKAIZHAI"}, "010502": {"suoxie": "CTYT87GYDKZ", "name": "\u8d22\u901a\u88d5\u6cf087\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGYUTAI87GEYUEDINGKAIZHAI"}, "010503": {"suoxie": "ZSWXHHA", "name": "\u62db\u5546\u7a33\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGWENXINGHUNHEA"}, "010504": {"suoxie": "ZSWXHHC", "name": "\u62db\u5546\u7a33\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGWENXINGHUNHEC"}, "010505": {"suoxie": "SWLXCYBLHJXGPC", "name": "\u7533\u4e07\u83f1\u4fe1\u521b\u4e1a\u677f\u91cf\u5316\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINCHUANGYEBANLIANGHUAJINGXUANGUPIAOC"}, "010506": {"suoxie": "DFHRXSNDKHHC", "name": "\u4e1c\u65b9\u7ea2\u777f\u73ba\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIXISANNIANDINGKAIHUNHEC"}, "010507": {"suoxie": "ZSTJ1NDKZFQS", "name": "\u62db\u5546\u6dfb\u95261\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANJIN1NIANDINGKAIZHAIFAQISHI"}, "010508": {"suoxie": "BSXKHHA", "name": "\u535a\u65f6\u946b\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINKANGHUNHEA"}, "010509": {"suoxie": "ZYPBZCXYHZQ15N", "name": "\u4e2d\u94f6\u5f6d\u535a\u653f\u7b56\u6027\u94f6\u884c\u503a\u52381-5\u5e74", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINPENGBOZHENGCEXINGYINHANGZHAIQUAN15NIAN"}, "010510": {"suoxie": "GY14TLCZQFQC", "name": "\u5de5\u94f614\u5929\u7406\u8d22\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN14TIANLICAIZHAIQUANFAQIC"}, "010511": {"suoxie": "BSXKHHC", "name": "\u535a\u65f6\u946b\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINKANGHUNHEC"}, "010512": {"suoxie": "GY7TLCZQC", "name": "\u5de5\u94f67\u5929\u7406\u8d22\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN7TIANLICAIZHAIQUANC"}, "010513": {"suoxie": "CHYJZQA", "name": "\u6df3\u539a\u76ca\u52a0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUNHOUYIJIAZHAIQUANA"}, "010514": {"suoxie": "CHYJZQC", "name": "\u6df3\u539a\u76ca\u52a0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUNHOUYIJIAZHAIQUANC"}, "010515": {"suoxie": "FGTXHBHHA", "name": "\u5bcc\u56fd\u5929\u5174\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTIANXINGHUIBAOHUNHEA"}, "010516": {"suoxie": "JHZZ13NZJZZSA", "name": "\u5609\u5408\u4e2d\u503a-1-3\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEZHONGZHAI13NIANZHENGJINZHAIZHISHUA"}, "010517": {"suoxie": "JHZZ13NZJZZSC", "name": "\u5609\u5408\u4e2d\u503a-1-3\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEZHONGZHAI13NIANZHENGJINZHAIZHISHUC"}, "010518": {"suoxie": "HXXFKJYNDKHHA", "name": "\u534e\u590f\u5148\u950b\u79d1\u6280\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANFENGKEJIYINIANDINGKAIHUNHEA"}, "010519": {"suoxie": "HXXFKJYNDKHHC", "name": "\u534e\u590f\u5148\u950b\u79d1\u6280\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANFENGKEJIYINIANDINGKAIHUNHEC"}, "010520": {"suoxie": "XYZZ35NZCXJRZA", "name": "\u5174\u4e1a\u4e2d\u503a3-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHAI35NIANZHENGCEXINGJINRONGZHAIA"}, "010521": {"suoxie": "XYZZ35NZCXJRZC", "name": "\u5174\u4e1a\u4e2d\u503a3-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEZHONGZHAI35NIANZHENGCEXINGJINRONGZHAIC"}, "010522": {"suoxie": "HATXYNCYQHHA", "name": "\u534e\u5b89\u6dfb\u79a7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANXIYINIANCHIYOUQIHUNHEA"}, "010523": {"suoxie": "HATXYNCYQHHC", "name": "\u534e\u5b89\u6dfb\u79a7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANXIYINIANCHIYOUQIHUNHEC"}, "010524": {"suoxie": "YHZZ5GTXZTETFLJC", "name": "\u94f6\u534e\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENG5GTONGXINZHUTIETFLIANJIEC"}, "010525": {"suoxie": "FGTXHBHHC", "name": "\u5bcc\u56fd\u5929\u5174\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTIANXINGHUIBAOHUNHEC"}, "010527": {"suoxie": "JSCCJTYLYNDKCZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u4f18\u5229\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIYOULIYINIANDINGKAICHUNZHAI"}, "010529": {"suoxie": "GFZZ15NGKZZSA", "name": "\u5e7f\u53d1\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI15NIANGUOKAIZHAIZHISHUA"}, "010530": {"suoxie": "GFZZ15NGKZZSC", "name": "\u5e7f\u53d1\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI15NIANGUOKAIZHAIZHISHUC"}, "010531": {"suoxie": "SWZZSWDZHYTZZSLOFC", "name": "\u7533\u4e07\u4e2d\u8bc1\u7533\u4e07\u7535\u5b50\u884c\u4e1a\u6295\u8d44\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANZHONGZHENGSHENWANDIANZIHANGYETOUZIZHISHULOFC"}, "010532": {"suoxie": "GFHXYNCYQHHA", "name": "\u5e7f\u53d1\u6052\u4fe1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXINYINIANCHIYOUQIHUNHEA"}, "010533": {"suoxie": "GFHXYNCYQHHC", "name": "\u5e7f\u53d1\u6052\u4fe1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXINYINIANCHIYOUQIHUNHEC"}, "010534": {"suoxie": "GFJHZZHHA", "name": "\u5e7f\u53d1\u5747\u8861\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUNHENGZENGZHANGHUNHEA"}, "010535": {"suoxie": "GFJHZZHHC", "name": "\u5e7f\u53d1\u5747\u8861\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAJUNHENGZENGZHANGHUNHEC"}, "010536": {"suoxie": "TKYSQYHHA", "name": "\u6cf0\u5eb7\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGYOUSHIQIYEHUNHEA"}, "010537": {"suoxie": "TKYSQYHHC", "name": "\u6cf0\u5eb7\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGYOUSHIQIYEHUNHEC"}, "010538": {"suoxie": "GTHYHH", "name": "\u56fd\u6cf0\u60e0\u5143\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOTAIHUIYUANHUNHE"}, "010539": {"suoxie": "ZSZDJWJYNCYA", "name": "\u6d59\u5546\u667a\u591a\u91d1\u7a33\u5065\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOJINWENJIANYINIANCHIYOUA"}, "010540": {"suoxie": "ZSZDJWJYNCYC", "name": "\u6d59\u5546\u667a\u591a\u91d1\u7a33\u5065\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOJINWENJIANYINIANCHIYOUC"}, "010541": {"suoxie": "GSABWH6GYCYQHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u548c6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENHE6GEYUECHIYOUQIHUNHEA"}, "010542": {"suoxie": "GSABWH6GYCYQHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u548c6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENHE6GEYUECHIYOUQIHUNHEC"}, "010543": {"suoxie": "ZJKXHHA", "name": "\u4e2d\u52a0\u79d1\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEXINHUNHEA"}, "010544": {"suoxie": "ZJKXHHC", "name": "\u4e2d\u52a0\u79d1\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKEXINHUNHEC"}, "010545": {"suoxie": "ZJJLCYQHHA", "name": "\u4e2d\u52a0\u805a\u9686\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJULONGCHIYOUQIHUNHEA"}, "010546": {"suoxie": "ZJJLCYQHHC", "name": "\u4e2d\u52a0\u805a\u9686\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJULONGCHIYOUQIHUNHEC"}, "010547": {"suoxie": "BSHJCYQHHA", "name": "\u535a\u65f6\u6052\u8fdb\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGJINCHIYOUQIHUNHEA"}, "010548": {"suoxie": "BSHJCYQHHC", "name": "\u535a\u65f6\u6052\u8fdb\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGJINCHIYOUQIHUNHEC"}, "010549": {"suoxie": "FGJHCLHH", "name": "\u5bcc\u56fd\u5747\u8861\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNHENGCELUEHUNHE"}, "010550": {"suoxie": "HSSQLHHH", "name": "\u534e\u5546\u53cc\u64ce\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGSHUANGQINGLINGHANGHUNHE"}, "010551": {"suoxie": "CHXYYNCYQHH", "name": "\u6df3\u539a\u6b23\u9890\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINYIYINIANCHIYOUQIHUNHE"}, "010552": {"suoxie": "ZSZXLHSNCYHHA", "name": "\u6d59\u5546\u667a\u9009\u9886\u822a\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANLINGHANGSANNIANCHIYOUHUNHEA"}, "010553": {"suoxie": "ZSZXLHSNCYHHC", "name": "\u6d59\u5546\u667a\u9009\u9886\u822a\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANLINGHANGSANNIANCHIYOUHUNHEC"}, "010554": {"suoxie": "HAXXXFHHA", "name": "\u534e\u5b89\u65b0\u5174\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANXINXINGXIAOFEIHUNHEA"}, "010555": {"suoxie": "HAXXXFHHC", "name": "\u534e\u5b89\u65b0\u5174\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANXINXINGXIAOFEIHUNHEC"}, "010556": {"suoxie": "HTFHS300ZSZQC", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300ZHISHUZENGQIANGC"}, "010557": {"suoxie": "HTFSZSHLGYCYHH", "name": "\u6c47\u6dfb\u5bcc\u6570\u5b57\u751f\u6d3b\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHUZISHENGHUOLIUGEYUECHIYOUHUNHE"}, "010558": {"suoxie": "HAXLYXHHA", "name": "\u6c47\u5b89\u946b\u5229\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANXINLIYOUXUANHUNHEA"}, "010559": {"suoxie": "HAXLYXHHC", "name": "\u6c47\u5b89\u946b\u5229\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANXINLIYOUXUANHUNHEC"}, "010560": {"suoxie": "YYWJZL18GYCYHHA", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u522918\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGWENJIANZENGLI18GEYUECHIYOUHUNHEA"}, "010561": {"suoxie": "YHJCXPJZETFLJ", "name": "\u94f6\u534e\u5de8\u6f6e\u5c0f\u76d8\u4ef7\u503cETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAJUCHAOXIAOPANJIAZHIETFLIANJIE"}, "010562": {"suoxie": "YYCZLHHHA", "name": "\u6c38\u8d62\u6210\u957f\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGCHENGZHANGLINGHANGHUNHEA"}, "010563": {"suoxie": "YYCZLHHHC", "name": "\u6c38\u8d62\u6210\u957f\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGCHENGZHANGLINGHANGHUNHEC"}, "010564": {"suoxie": "MSJYRLHH", "name": "\u6c11\u751f\u52a0\u94f6\u745e\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINRUILIHUNHE"}, "010565": {"suoxie": "DFHRDZZQA", "name": "\u4e1c\u65b9\u6052\u745e\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGFANGHENGRUIDUANZHAIZHAIQUANA"}, "010566": {"suoxie": "DFHRDZZQB", "name": "\u4e1c\u65b9\u6052\u745e\u77ed\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGFANGHENGRUIDUANZHAIZHAIQUANB"}, "010567": {"suoxie": "DFHRDZZQC", "name": "\u4e1c\u65b9\u6052\u745e\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGFANGHENGRUIDUANZHAIZHAIQUANC"}, "010568": {"suoxie": "HFTHRJXHHA", "name": "\u6d77\u5bcc\u901a\u60e0\u777f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGHUIRUIJINGXUANHUNHEA"}, "010569": {"suoxie": "HFTHRJXHHC", "name": "\u6d77\u5bcc\u901a\u60e0\u777f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGHUIRUIJINGXUANHUNHEC"}, "010570": {"suoxie": "XWCXLHHHA", "name": "\u65b0\u6c83\u521b\u65b0\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINWOCHUANGXINLINGHANGHUNHEA"}, "010571": {"suoxie": "XWCXLHHHC", "name": "\u65b0\u6c83\u521b\u65b0\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINWOCHUANGXINLINGHANGHUNHEC"}, "010572": {"suoxie": "YFDZZWDSWKJZSLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4e07\u5f97\u751f\u7269\u79d1\u6280\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGWANDESHENGWUKEJIZHISHULOFC"}, "010573": {"suoxie": "HRYDFSZGA50ZSC", "name": "\u534e\u6da6\u5143\u5927\u5bcc\u65f6\u4e2d\u56fdA50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUARUNYUANDAFUSHIZHONGGUOA50ZHISHUC"}, "010576": {"suoxie": "MSJYXLHH", "name": "\u6c11\u751f\u52a0\u94f6\u5174\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINXINGLIHUNHE"}, "010580": {"suoxie": "CSWX63GYDKZ", "name": "\u957f\u76db\u7a33\u946b63\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGWENXIN63GEYUEDINGKAIZHAI"}, "010581": {"suoxie": "ZXJTZZ35NZJZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUZHONGZHAI35NIANZHENGJINZHAIA"}, "010582": {"suoxie": "ZXJTZZ35NZJZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUZHONGZHAI35NIANZHENGJINZHAIC"}, "010583": {"suoxie": "FGLCJXGPQDIIMY", "name": "\u5bcc\u56fd\u84dd\u7b79\u7cbe\u9009\u80a1\u7968(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOLANCHOUJINGXUANGUPIAOQDIIMEIYUAN"}, "010584": {"suoxie": "BHHJXDNZTHH", "name": "\u6e24\u6d77\u6c47\u91d1\u65b0\u52a8\u80fd\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOHAIHUIJINXINDONGNENGZHUTIHUNHE"}, "010585": {"suoxie": "CJHXYYXFGPA", "name": "\u521b\u91d1\u5408\u4fe1\u533b\u836f\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYIYAOXIAOFEIGUPIAOA"}, "010586": {"suoxie": "CJHXYYXFGPC", "name": "\u521b\u91d1\u5408\u4fe1\u533b\u836f\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINYIYAOXIAOFEIGUPIAOC"}, "010587": {"suoxie": "PYXJZZHHA", "name": "\u9e4f\u626c\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGXIANJINZHIZAOHUNHEA"}, "010588": {"suoxie": "PYXJZZHHC", "name": "\u9e4f\u626c\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGXIANJINZHIZAOHUNHEC"}, "010589": {"suoxie": "PYJAYNCYQHHA", "name": "\u9e4f\u626c\u666f\u5b89\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGANYINIANCHIYOUQIHUNHEA"}, "010590": {"suoxie": "PYJAYNCYQHHC", "name": "\u9e4f\u626c\u666f\u5b89\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGANYINIANCHIYOUQIHUNHEC"}, "010591": {"suoxie": "FGZGZXPHHQDIIMY", "name": "\u5bcc\u56fd\u4e2d\u56fd\u4e2d\u5c0f\u76d8\u6df7\u5408(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOZHONGGUOZHONGXIAOPANHUNHEQDIIMEIYUAN"}, "010592": {"suoxie": "NFYYCXGPA", "name": "\u5357\u65b9\u533b\u836f\u521b\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGYIYAOCHUANGXINGUPIAOA"}, "010593": {"suoxie": "NFYYCXGPC", "name": "\u5357\u65b9\u533b\u836f\u521b\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGYIYAOCHUANGXINGUPIAOC"}, "010594": {"suoxie": "GFRXSNCYQHH", "name": "\u5e7f\u53d1\u777f\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIXUANSANNIANCHIYOUQIHUNHE"}, "010595": {"suoxie": "GFCZJXHHA", "name": "\u5e7f\u53d1\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHENGZHANGJINGXUANHUNHEA"}, "010596": {"suoxie": "GFCZJXHHC", "name": "\u5e7f\u53d1\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHENGZHANGJINGXUANHUNHEC"}, "010597": {"suoxie": "CJHXJWLHPZHHA", "name": "\u521b\u91d1\u5408\u4fe1\u666f\u96ef\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINJINGWENLINGHUOPEIZHIHUNHEA"}, "010598": {"suoxie": "CJHXJWLHPZHHC", "name": "\u521b\u91d1\u5408\u4fe1\u666f\u96ef\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHUANGJINHEXINJINGWENLINGHUOPEIZHIHUNHEC"}, "010599": {"suoxie": "HTFGZLCZ30YNHHA", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u8d28\u91cf\u6210\u957f30\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUGAOZHILIANGCHENGZHANG30YINIANHUNHEA"}, "010600": {"suoxie": "GDARYNCYA", "name": "\u5149\u5927\u5b89\u745e\u4e00\u5e74\u6301\u6709A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANRUIYINIANCHIYOUA"}, "010601": {"suoxie": "GDARYNCYC", "name": "\u5149\u5927\u5b89\u745e\u4e00\u5e74\u6301\u6709C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAANRUIYINIANCHIYOUC"}, "010602": {"suoxie": "CCJHYXHH", "name": "\u957f\u57ce\u5747\u8861\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJUNHENGYOUXUANHUNHE"}, "010603": {"suoxie": "CCZZ510NGKZZSA", "name": "\u957f\u57ce\u4e2d\u503a5-10\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI510NIANGUOKAIZHAIZHISHUA"}, "010604": {"suoxie": "CCZZ510NGKZZSC", "name": "\u957f\u57ce\u4e2d\u503a5-10\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI510NIANGUOKAIZHAIZHISHUC"}, "010605": {"suoxie": "CJHXXXHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u7965\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINXIANGHUNHEA"}, "010606": {"suoxie": "CJHXXXHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u7965\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINXIANGHUNHEC"}, "010607": {"suoxie": "XWAX87GYDKZ", "name": "\u65b0\u6c83\u5b89\u946b87\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINWOANXIN87GEYUEDINGKAIZHAI"}, "010608": {"suoxie": "HTBRZLLXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGLINGXIANHUNHEA"}, "010609": {"suoxie": "HTBRZLLXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGLINGXIANHUNHEC"}, "010610": {"suoxie": "STMGYJLNCYHH", "name": "\u4e0a\u6295\u6469\u6839\u8fdc\u89c1\u4e24\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENYUANJIANLIANGNIANCHIYOUHUNHE"}, "010611": {"suoxie": "WJZLFZCYHHA", "name": "\u4e07\u5bb6\u6218\u7565\u53d1\u5c55\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHANLUEFAZHANCHANYEHUNHEA"}, "010612": {"suoxie": "WJZLFZCYHHC", "name": "\u4e07\u5bb6\u6218\u7565\u53d1\u5c55\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAZHANLUEFAZHANCHANYEHUNHEC"}, "010613": {"suoxie": "ZRCYQSYNDKHHA", "name": "\u4e2d\u878d\u4ea7\u4e1a\u8d8b\u52bf\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCHANYEQUSHIYINIANDINGKAIHUNHEA"}, "010614": {"suoxie": "ZRCYQSYNDKHHC", "name": "\u4e2d\u878d\u4ea7\u4e1a\u8d8b\u52bf\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCHANYEQUSHIYINIANDINGKAIHUNHEC"}, "010615": {"suoxie": "GJZZCXHHA", "name": "\u56fd\u91d1\u81ea\u4e3b\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINZIZHUCHUANGXINHUNHEA"}, "010616": {"suoxie": "GJZZCXHHC", "name": "\u56fd\u91d1\u81ea\u4e3b\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINZIZHUCHUANGXINHUNHEC"}, "010617": {"suoxie": "XYXFJXHHA", "name": "\u5174\u4e1a\u6d88\u8d39\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXIAOFEIJINGXUANHUNHEA"}, "010618": {"suoxie": "XYXFJXHHC", "name": "\u5174\u4e1a\u6d88\u8d39\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXIAOFEIJINGXUANHUNHEC"}, "010619": {"suoxie": "HATL6GYZQA", "name": "\u534e\u5b89\u6dfb\u52296\u4e2a\u6708\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANTIANLI6GEYUEZHAIQUANA"}, "010620": {"suoxie": "HATL6GYZQC", "name": "\u534e\u5b89\u6dfb\u52296\u4e2a\u6708\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANTIANLI6GEYUEZHAIQUANC"}, "010621": {"suoxie": "YYTN63GYDKZ", "name": "\u6c38\u8d62\u6cf0\u5b8163\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGTAINING63GEYUEDINGKAIZHAI"}, "010622": {"suoxie": "HYCZJXHHA", "name": "\u6052\u8d8a\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUECHENGZHANGJINGXUANHUNHEA"}, "010623": {"suoxie": "HYCZJXHHC", "name": "\u6052\u8d8a\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUECHENGZHANGJINGXUANHUNHEC"}, "010624": {"suoxie": "FGWJZZHHA", "name": "\u5bcc\u56fd\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANZENGZHANGHUNHEA"}, "010625": {"suoxie": "FGWJZZHHC", "name": "\u5bcc\u56fd\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANZENGZHANGHUNHEC"}, "010626": {"suoxie": "HRRY63GYDKZQ", "name": "\u534e\u878d\u8363\u8d6263\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUARONGRONGYING63GEYUEDINGKAIZHAIQUAN"}, "010627": {"suoxie": "CHAX87GYDKZ", "name": "\u6df3\u539a\u5b89\u5fc387\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUANXIN87GEYUEDINGKAIZHAI"}, "010628": {"suoxie": "GFRXSGYDKHH", "name": "\u5e7f\u53d1\u745e\u8f69\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIXUANSANGEYUEDINGKAIHUNHE"}, "010629": {"suoxie": "GFKZZZQE", "name": "\u5e7f\u53d1\u53ef\u8f6c\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "GUANGFAKEZHUANZHAIZHAIQUANE"}, "010630": {"suoxie": "HSHRXLZQA", "name": "\u60e0\u5347\u548c\u777f\u5174\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUISHENGHERUIXINGLIZHAIQUANA"}, "010631": {"suoxie": "HSHY66GYDKZQ", "name": "\u60e0\u5347\u548c\u97f566\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYUN66GEYUEDINGKAIZHAIQUAN"}, "010632": {"suoxie": "GYRDYNDKCZFQS", "name": "\u5de5\u94f6\u745e\u8fbe\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIDAYINIANDINGKAICHUNZHAIFAQISHI"}, "010633": {"suoxie": "HSHRXLZQC", "name": "\u60e0\u5347\u548c\u777f\u5174\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUISHENGHERUIXINGLIZHAIQUANC"}, "010634": {"suoxie": "THHYZQA", "name": "\u5929\u5f18\u5408\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGHEYIZHAIQUANA"}, "010635": {"suoxie": "THHYZQC", "name": "\u5929\u5f18\u5408\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGHEYIZHAIQUANC"}, "010636": {"suoxie": "CTAYHHA", "name": "\u8d22\u901a\u5b89\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGANYINGHUNHEA"}, "010637": {"suoxie": "CTAYHHC", "name": "\u8d22\u901a\u5b89\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGANYINGHUNHEC"}, "010638": {"suoxie": "YHXYJX15GYDKZ", "name": "\u94f6\u534e\u4fe1\u7528\u7cbe\u900915\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJINGXUAN15GEYUEDINGKAIZHAI"}, "010639": {"suoxie": "SYJYX87GYDKZ", "name": "\u4e0a\u94f6\u805a\u8fdc\u946b87\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINJUYUANXIN87GEYUEDINGKAIZHAI"}, "010640": {"suoxie": "CTWJHB6GYCYHHA", "name": "\u8d22\u901a\u7a33\u8fdb\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGWENJINHUIBAO6GEYUECHIYOUHUNHEA"}, "010641": {"suoxie": "CTWJHB6GYCYHHC", "name": "\u8d22\u901a\u7a33\u8fdb\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGWENJINHUIBAO6GEYUECHIYOUHUNHEC"}, "010642": {"suoxie": "NYHLRXYNCYHH", "name": "\u519c\u94f6\u6c47\u7406\u745e\u7965\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NONGYINHUILIRUIXIANGYINIANCHIYOUHUNHE"}, "010643": {"suoxie": "PAYL2025YNCYQHHFOF", "name": "\u5e73\u5b89\u517b\u80012025\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANYANGLAO2025YINIANCHIYOUQIHUNHEFOF"}, "010644": {"suoxie": "FGQQJKSHZTHHQDIIRMB", "name": "\u5bcc\u56fd\u5168\u7403\u5065\u5eb7\u751f\u6d3b\u4e3b\u9898\u6df7\u5408(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "FUGUOQUANQIUJIANKANGSHENGHUOZHUTIHUNHEQDIIRENMINBI"}, "010645": {"suoxie": "FGQQJKSHZTHHQDIIMY", "name": "\u5bcc\u56fd\u5168\u7403\u5065\u5eb7\u751f\u6d3b\u4e3b\u9898\u6df7\u5408(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOQUANQIUJIANKANGSHENGHUOZHUTIHUNHEQDIIMEIYUAN"}, "010646": {"suoxie": "RTJZQSHHA", "name": "\u878d\u901a\u4ef7\u503c\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGJIAZHIQUSHIHUNHEA"}, "010647": {"suoxie": "RTJZQSHHC", "name": "\u878d\u901a\u4ef7\u503c\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGJIAZHIQUSHIHUNHEC"}, "010651": {"suoxie": "PASJZX6GYCYZQA", "name": "\u5e73\u5b89\u53cc\u5b63\u589e\u4eab6\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANSHUANGJIZENGXIANG6GEYUECHIYOUZHAIQUANA"}, "010652": {"suoxie": "PASJZX6GYCYZQC", "name": "\u5e73\u5b89\u53cc\u5b63\u589e\u4eab6\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANSHUANGJIZENGXIANG6GEYUECHIYOUZHAIQUANC"}, "010653": {"suoxie": "NYHLJYZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u7389\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINYUZHAIQUAN"}, "010654": {"suoxie": "THYYCXA", "name": "\u5929\u5f18\u533b\u836f\u521b\u65b0A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGYIYAOCHUANGXINA"}, "010655": {"suoxie": "THYYCXC", "name": "\u5929\u5f18\u533b\u836f\u521b\u65b0C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGYIYAOCHUANGXINC"}, "010657": {"suoxie": "HFTXRHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u777f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGXINRUIHUNHEA"}, "010658": {"suoxie": "HFTXRHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u777f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGXINRUIHUNHEC"}, "010659": {"suoxie": "MSJYZLLXHHA", "name": "\u6c11\u751f\u52a0\u94f6\u8d28\u91cf\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINZHILIANGLINGXIANHUNHEA"}, "010660": {"suoxie": "MSJYZLLXHHC", "name": "\u6c11\u751f\u52a0\u94f6\u8d28\u91cf\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINZHILIANGLINGXIANHUNHEC"}, "010661": {"suoxie": "AXWJJSYNCYQHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u805a\u7533\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINWENJIANJUSHENYINIANCHIYOUQIHUNHEC"}, "010662": {"suoxie": "FGJHYXHH", "name": "\u5bcc\u56fd\u5747\u8861\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNHENGYOUXUANHUNHE"}, "010663": {"suoxie": "CJJHCZHHFQSA", "name": "\u957f\u6c5f\u5747\u8861\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGJUNHENGCHENGZHANGHUNHEFAQISHIA"}, "010664": {"suoxie": "CJJHCZHHFQSC", "name": "\u957f\u6c5f\u5747\u8861\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGJUNHENGCHENGZHANGHUNHEFAQISHIC"}, "010665": {"suoxie": "BSGDZBHHA", "name": "\u535a\u65f6\u9ad8\u7aef\u88c5\u5907\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGAODUANZHUANGBEIHUNHEA"}, "010666": {"suoxie": "BSGDZBHHC", "name": "\u535a\u65f6\u9ad8\u7aef\u88c5\u5907\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGAODUANZHUANGBEIHUNHEC"}, "010667": {"suoxie": "AXJZHBSNCYHHC", "name": "\u5b89\u4fe1\u4ef7\u503c\u56de\u62a5\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIHUIBAOSANNIANCHIYOUHUNHEC"}, "010668": {"suoxie": "GYYXDCLHPZHHFQA", "name": "\u5de5\u94f6\u4f18\u9009\u5bf9\u51b2\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GONGYINYOUXUANDUICHONGLINGHUOPEIZHIHUNHEFAQIA"}, "010669": {"suoxie": "GYYXDCLHPZHHFQC", "name": "\u5de5\u94f6\u4f18\u9009\u5bf9\u51b2\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GONGYINYOUXUANDUICHONGLINGHUOPEIZHIHUNHEFAQIC"}, "010671": {"suoxie": "JSCCDZHHHQDIIMY", "name": "\u666f\u987a\u957f\u57ce\u5927\u4e2d\u534e\u6df7\u5408(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "JINGSHUNCHANGCHENGDAZHONGHUAHUNHEQDIIMEIYUAN"}, "010673": {"suoxie": "XQZZ800LGYCYZSA", "name": "\u5174\u5168\u4e2d\u8bc1800\u516d\u4e2a\u6708\u6301\u6709\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGQUANZHONGZHENG800LIUGEYUECHIYOUZHISHUA"}, "010674": {"suoxie": "XQZZ800LGYCYZSC", "name": "\u5174\u5168\u4e2d\u8bc1800\u516d\u4e2a\u6708\u6301\u6709\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGQUANZHONGZHENG800LIUGEYUECHIYOUZHISHUC"}, "010676": {"suoxie": "GDXJYHH", "name": "\u5149\u5927\u65b0\u673a\u9047\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAXINJIYUHUNHE"}, "010677": {"suoxie": "GYZZCMZSLOFC", "name": "\u5de5\u94f6\u4e2d\u8bc1\u4f20\u5a92\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGCHUANMEIZHISHULOFC"}, "010678": {"suoxie": "ZOJHCZHHA", "name": "\u4e2d\u6b27\u5747\u8861\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJUNHENGCHENGZHANGHUNHEA"}, "010679": {"suoxie": "ZOJHCZHHC", "name": "\u4e2d\u6b27\u5747\u8861\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJUNHENGCHENGZHANGHUNHEC"}, "010680": {"suoxie": "HXXXCZGPA", "name": "\u534e\u590f\u65b0\u5174\u6210\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAXINXINGCHENGZHANGGUPIAOA"}, "010681": {"suoxie": "HXXXCZGPC", "name": "\u534e\u590f\u65b0\u5174\u6210\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAXINXINGCHENGZHANGGUPIAOC"}, "010683": {"suoxie": "ZRJY6GYCYQHHA", "name": "\u4e2d\u878d\u666f\u98906\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGYI6GEYUECHIYOUQIHUNHEA"}, "010684": {"suoxie": "ZRJY6GYCYQHHC", "name": "\u4e2d\u878d\u666f\u98906\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGYI6GEYUECHIYOUQIHUNHEC"}, "010685": {"suoxie": "GYQYYLGPC", "name": "\u5de5\u94f6\u524d\u6cbf\u533b\u7597\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINQIANYANYILIAOGUPIAOC"}, "010687": {"suoxie": "GYWTCYGPC", "name": "\u5de5\u94f6\u6587\u4f53\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINWENTICHANYEGUPIAOC"}, "010688": {"suoxie": "ZSRDYNCYQHHA", "name": "\u62db\u5546\u745e\u5fb7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIDEYINIANCHIYOUQIHUNHEA"}, "010689": {"suoxie": "ZSRDYNCYQHHC", "name": "\u62db\u5546\u745e\u5fb7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIDEYINIANCHIYOUQIHUNHEC"}, "010690": {"suoxie": "WJHLHTHXZCLHA", "name": "\u4e07\u5bb6\u4e92\u8054\u4e92\u901a\u6838\u5fc3\u8d44\u4ea7\u91cf\u5316A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHULIANHUTONGHEXINZICHANLIANGHUAA"}, "010691": {"suoxie": "WJHLHTHXZCLHC", "name": "\u4e07\u5bb6\u4e92\u8054\u4e92\u901a\u6838\u5fc3\u8d44\u4ea7\u91cf\u5316C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHULIANHUTONGHEXINZICHANLIANGHUAC"}, "010692": {"suoxie": "HXHXJZHHA", "name": "\u534e\u590f\u6838\u5fc3\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINJIAZHIHUNHEA"}, "010693": {"suoxie": "HXHXJZHHC", "name": "\u534e\u590f\u6838\u5fc3\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINJIAZHIHUNHEC"}, "010694": {"suoxie": "WJNXZZYNCYHH", "name": "\u4e07\u5bb6\u5185\u9700\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIANEIXUZENGZHANGYINIANCHIYOUHUNHE"}, "010695": {"suoxie": "HXPYYNDKHH", "name": "\u534e\u590f\u78d0\u76ca\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAPANYIYINIANDINGKAIHUNHE"}, "010696": {"suoxie": "GYJRDCHHC", "name": "\u5de5\u94f6\u91d1\u878d\u5730\u4ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINRONGDICHANHUNHEC"}, "010697": {"suoxie": "ZRHYXF6GYCYHHA", "name": "\u4e2d\u878d\u884c\u4e1a\u5148\u950b6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGHANGYEXIANFENG6GEYUECHIYOUHUNHEA"}, "010698": {"suoxie": "ZRHYXF6GYCYHHC", "name": "\u4e2d\u878d\u884c\u4e1a\u5148\u950b6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGHANGYEXIANFENG6GEYUECHIYOUHUNHEC"}, "010699": {"suoxie": "DFHCXQSHH", "name": "\u4e1c\u65b9\u7ea2\u521b\u65b0\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGCHUANGXINQUSHIHUNHE"}, "010700": {"suoxie": "DFHJFYXLNDKHH", "name": "\u4e1c\u65b9\u7ea2\u9526\u4e30\u4f18\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINFENGYOUXUANLIANGNIANDINGKAIHUNHE"}, "010701": {"suoxie": "HYNXQDHHA", "name": "\u6052\u8d8a\u5185\u9700\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUENEIXUQUDONGHUNHEA"}, "010702": {"suoxie": "HYNXQDHHC", "name": "\u6052\u8d8a\u5185\u9700\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUENEIXUQUDONGHUNHEC"}, "010703": {"suoxie": "CTZXXFGPA", "name": "\u8d22\u901a\u667a\u9009\u6d88\u8d39\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CAITONGZHIXUANXIAOFEIGUPIAOA"}, "010704": {"suoxie": "CTZXXFGPC", "name": "\u8d22\u901a\u667a\u9009\u6d88\u8d39\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CAITONGZHIXUANXIAOFEIGUPIAOC"}, "010706": {"suoxie": "JSCCJLCZHH", "name": "\u666f\u987a\u957f\u57ce\u666f\u9a8a\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGLICHENGZHANGHUNHE"}, "010707": {"suoxie": "AXPWHYYNCYHHA", "name": "\u5b89\u4fe1\u5e73\u7a33\u5408\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINPINGWENHEYINGYINIANCHIYOUHUNHEA"}, "010708": {"suoxie": "AXPWHYYNCYHHC", "name": "\u5b89\u4fe1\u5e73\u7a33\u5408\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINPINGWENHEYINGYINIANCHIYOUHUNHEC"}, "010709": {"suoxie": "AXYYJKGPA", "name": "\u5b89\u4fe1\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINYIYAOJIANKANGGUPIAOA"}, "010710": {"suoxie": "AXYYJKGPC", "name": "\u5b89\u4fe1\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINYIYAOJIANKANGGUPIAOC"}, "010711": {"suoxie": "HFGCYXHHFQS", "name": "\u534e\u5bcc\u56fd\u6f6e\u4f18\u9009\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUGUOCHAOYOUXUANHUNHEFAQISHI"}, "010712": {"suoxie": "ZOJLHHA", "name": "\u4e2d\u6b27\u747e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGOUJINLIHUNHEA"}, "010713": {"suoxie": "ZOJLHHC", "name": "\u4e2d\u6b27\u747e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGOUJINLIHUNHEC"}, "010714": {"suoxie": "DFHYJJZHH", "name": "\u4e1c\u65b9\u7ea2\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGYUANJIANJIAZHIHUNHE"}, "010715": {"suoxie": "CTZGXFSJYNCYA", "name": "\u8d22\u901a\u8d44\u7ba1\u6d88\u8d39\u5347\u7ea7\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANXIAOFEISHENGJIYINIANCHIYOUA"}, "010716": {"suoxie": "CTZGXFSJYNCYC", "name": "\u8d22\u901a\u8d44\u7ba1\u6d88\u8d39\u5347\u7ea7\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANXIAOFEISHENGJIYINIANCHIYOUC"}, "010717": {"suoxie": "QHKYYZQY6GYCYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u8d28\u4f01\u4e1a6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANYOUZHIQIYE6GEYUECHIYOUHUNHEA"}, "010718": {"suoxie": "QHKYYZQY6GYCYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u8d28\u4f01\u4e1a6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANYOUZHIQIYE6GEYUECHIYOUHUNHEC"}, "010719": {"suoxie": "DWRY63GYDKZ", "name": "\u4e1c\u5434\u745e\u76c863\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGWURUIYING63GEYUEDINGKAIZHAI"}, "010723": {"suoxie": "ZOJZCZHHA", "name": "\u4e2d\u6b27\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAZHICHENGZHANGHUNHEA"}, "010724": {"suoxie": "ZOJZCZHHC", "name": "\u4e2d\u6b27\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAZHICHENGZHANGHUNHEC"}, "010725": {"suoxie": "PHAXYNCYQHHA", "name": "\u9e4f\u534e\u5b89\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANXIANGYINIANCHIYOUQIHUNHEA"}, "010726": {"suoxie": "PHAXYNCYQHHC", "name": "\u9e4f\u534e\u5b89\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANXIANGYINIANCHIYOUQIHUNHEC"}, "010727": {"suoxie": "JXXJZLHBB", "name": "\u5efa\u4fe1\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINZENGLIHUOBIB"}, "010728": {"suoxie": "ZTXCJZYNCYHHA", "name": "\u4e2d\u6cf0\u5174\u8bda\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGCHENGJIAZHIYINIANCHIYOUHUNHEA"}, "010729": {"suoxie": "ZTXCJZYNCYHHC", "name": "\u4e2d\u6cf0\u5174\u8bda\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGCHENGJIAZHIYINIANCHIYOUHUNHEC"}, "010730": {"suoxie": "YHXJLNCYQHH", "name": "\u94f6\u534e\u5fc3\u4f73\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINJIALIANGNIANCHIYOUQIHUNHE"}, "010731": {"suoxie": "GFCXYLLNCYHHA", "name": "\u5e7f\u53d1\u521b\u65b0\u533b\u7597\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHUANGXINYILIAOLIANGNIANCHIYOUHUNHEA"}, "010732": {"suoxie": "GFCXYLLNCYHHC", "name": "\u5e7f\u53d1\u521b\u65b0\u533b\u7597\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHUANGXINYILIAOLIANGNIANCHIYOUHUNHEC"}, "010733": {"suoxie": "HTHTRJCZA", "name": "\u7ea2\u5854\u7ea2\u571f\u745e\u666f\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTURUIJINGCHUNZHAIA"}, "010734": {"suoxie": "HTHTRJCZC", "name": "\u7ea2\u5854\u7ea2\u571f\u745e\u666f\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HONGTAHONGTURUIJINGCHUNZHAIC"}, "010735": {"suoxie": "SWWJYLYNCYHHFQSFOF", "name": "\u7533\u4e07\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANWENJIANYANGLAOYINIANCHIYOUHUNHEFAQISHIFOF"}, "010736": {"suoxie": "YFDHS300ZSZQA", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300ZHISHUZENGQIANGA"}, "010737": {"suoxie": "YFDHS300ZSZQC", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300ZHISHUZENGQIANGC"}, "010738": {"suoxie": "DCYXSJYNCYHHA", "name": "\u5927\u6210\u4f18\u9009\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYOUXUANSHENGJIYINIANCHIYOUHUNHEA"}, "010739": {"suoxie": "DCYXSJYNCYHHC", "name": "\u5927\u6210\u4f18\u9009\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYOUXUANSHENGJIYINIANCHIYOUHUNHEC"}, "010740": {"suoxie": "HAHXJZHHA", "name": "\u6c47\u5b89\u6838\u5fc3\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINJIAZHIHUNHEA"}, "010741": {"suoxie": "HAHXJZHHC", "name": "\u6c47\u5b89\u6838\u5fc3\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANHEXINJIAZHIHUNHEC"}, "010742": {"suoxie": "NFNYYNCYQHHA", "name": "\u5357\u65b9\u5b81\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGNINGYUEYINIANCHIYOUQIHUNHEA"}, "010743": {"suoxie": "NFNYYNCYQHHC", "name": "\u5357\u65b9\u5b81\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGNINGYUEYINIANCHIYOUQIHUNHEC"}, "010744": {"suoxie": "GYLDJZHHA", "name": "\u5de5\u94f6\u7075\u52a8\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINLINGDONGJIAZHIHUNHEA"}, "010745": {"suoxie": "GYLDJZHHC", "name": "\u5de5\u94f6\u7075\u52a8\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINLINGDONGJIAZHIHUNHEC"}, "010746": {"suoxie": "FADCSJQYZTHH", "name": "\u5bcc\u5b89\u8fbe\u957f\u4e09\u89d2\u533a\u57df\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDACHANGSANJIAOQUYUZHUTIHUNHE"}, "010749": {"suoxie": "ZSCYBZSZQA", "name": "\u6d59\u5546\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGCHUANGYEBANZHISHUZENGQIANGA"}, "010750": {"suoxie": "ZSCYBZSZQC", "name": "\u6d59\u5546\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGCHUANGYEBANZHISHUZENGQIANGC"}, "010751": {"suoxie": "BYYZCZHHA", "name": "\u5b9d\u76c8\u4f18\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGYOUZHICHENGZHANGHUNHEA"}, "010752": {"suoxie": "BYYZCZHHC", "name": "\u5b9d\u76c8\u4f18\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGYOUZHICHENGZHANGHUNHEC"}, "010753": {"suoxie": "ZSZXCZD", "name": "\u62db\u5546\u62db\u65ed\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXUCHUNZHAID"}, "010754": {"suoxie": "ZSHGSKJCXHHC", "name": "\u62db\u5546\u6caa\u6e2f\u6df1\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGHUGANGSHENKEJICHUANGXINHUNHEC"}, "010755": {"suoxie": "BDRJYNCYQHH", "name": "\u535a\u9053\u777f\u89c1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAORUIJIANYINIANCHIYOUQIHUNHE"}, "010756": {"suoxie": "XHYXHHFQSA", "name": "\u5174\u534e\u6c38\u5174\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUAYONGXINGHUNHEFAQISHIA"}, "010757": {"suoxie": "XHYXHHFQSC", "name": "\u5174\u534e\u6c38\u5174\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUAYONGXINGHUNHEFAQISHIC"}, "010758": {"suoxie": "GTRYSJYNDKZ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u666f\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNJINGYINIANDINGKAIZHAI"}, "010761": {"suoxie": "HSZXHBHH", "name": "\u534e\u5546\u7504\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGZHENXUANHUIBAOHUNHE"}, "010762": {"suoxie": "BSHKCYQHHA", "name": "\u535a\u65f6\u6052\u5eb7\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGKANGCHIYOUQIHUNHEA"}, "010763": {"suoxie": "BSHKCYQHHC", "name": "\u535a\u65f6\u6052\u5eb7\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGKANGCHIYOUQIHUNHEC"}, "010764": {"suoxie": "JTRS18GYFBHH", "name": "\u4e5d\u6cf0\u9510\u534718\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAIRUISHENG18GEYUEFENGBIHUNHE"}, "010765": {"suoxie": "GSABHFHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u534e\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOHUAFENGHUNHEA"}, "010766": {"suoxie": "GSABHFHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u534e\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOHUAFENGHUNHEC"}, "010767": {"suoxie": "JXLLZCLCZZQA", "name": "\u5efa\u4fe1\u5229\u7387\u503a\u7b56\u7565\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINLILVZHAICELUECHUNZHAIZHAIQUANA"}, "010768": {"suoxie": "JXLLZCLCZZQC", "name": "\u5efa\u4fe1\u5229\u7387\u503a\u7b56\u7565\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINLILVZHAICELUECHUNZHAIZHAIQUANC"}, "010769": {"suoxie": "THZZNYZTA", "name": "\u5929\u5f18\u4e2d\u8bc1\u519c\u4e1a\u4e3b\u9898A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGNONGYEZHUTIA"}, "010770": {"suoxie": "THZZNYZTC", "name": "\u5929\u5f18\u4e2d\u8bc1\u519c\u4e1a\u4e3b\u9898C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGNONGYEZHUTIC"}, "010771": {"suoxie": "THGZXF100ZSZQA", "name": "\u5929\u5f18\u56fd\u8bc1\u6d88\u8d39100\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGXIAOFEI100ZHISHUZENGQIANGA"}, "010772": {"suoxie": "THGZXF100ZSZQC", "name": "\u5929\u5f18\u56fd\u8bc1\u6d88\u8d39100\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGXIAOFEI100ZHISHUZENGQIANGC"}, "010773": {"suoxie": "JSCCTYHBHHA", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u9633\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGTAIYANGHUIBAOHUNHEA"}, "010774": {"suoxie": "JSCCTYHBHHC", "name": "\u666f\u987a\u957f\u57ce\u6cf0\u9633\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGTAIYANGHUIBAOHUNHEC"}, "010775": {"suoxie": "BSHXCYQHHA", "name": "\u535a\u65f6\u6052\u65ed\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXUCHIYOUQIHUNHEA"}, "010776": {"suoxie": "BSHXCYQHHC", "name": "\u535a\u65f6\u6052\u65ed\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXUCHIYOUQIHUNHEC"}, "010777": {"suoxie": "ZSZXJJGPA", "name": "\u6d59\u5546\u667a\u9009\u5bb6\u5c45\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGZHIXUANJIAJUGUPIAOA"}, "010778": {"suoxie": "ZSZXJJGPC", "name": "\u6d59\u5546\u667a\u9009\u5bb6\u5c45\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGZHIXUANJIAJUGUPIAOC"}, "010779": {"suoxie": "XBLDLHYXYNCYA", "name": "\u897f\u90e8\u5229\u5f97\u91cf\u5316\u4f18\u9009\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDELIANGHUAYOUXUANYINIANCHIYOUA"}, "010780": {"suoxie": "XBLDLHYXYNCYC", "name": "\u897f\u90e8\u5229\u5f97\u91cf\u5316\u4f18\u9009\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDELIANGHUAYOUXUANYINIANCHIYOUC"}, "010781": {"suoxie": "XYJSYNCYQHHA", "name": "\u5174\u4e1a\u805a\u7533\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUSHENYINIANCHIYOUQIHUNHEA"}, "010782": {"suoxie": "XYJSYNCYQHHC", "name": "\u5174\u4e1a\u805a\u7533\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUSHENYINIANCHIYOUQIHUNHEC"}, "010783": {"suoxie": "DBHGSLTHHA", "name": "\u5fb7\u90a6\u6caa\u6e2f\u6df1\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGHUGANGSHENLONGTOUHUNHEA"}, "010784": {"suoxie": "DBHGSLTHHC", "name": "\u5fb7\u90a6\u6caa\u6e2f\u6df1\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGHUGANGSHENLONGTOUHUNHEC"}, "010785": {"suoxie": "BSCYBZSA", "name": "\u535a\u65f6\u521b\u4e1a\u677f\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHICHUANGYEBANZHISHUA"}, "010786": {"suoxie": "BSCYBZSC", "name": "\u535a\u65f6\u521b\u4e1a\u677f\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHICHUANGYEBANZHISHUC"}, "010787": {"suoxie": "HAYSQYHHA", "name": "\u534e\u5b89\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHIQIYEHUNHEA"}, "010788": {"suoxie": "HAYSQYHHC", "name": "\u534e\u5b89\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHIQIYEHUNHEC"}, "010789": {"suoxie": "HTFHSZSC", "name": "\u6c47\u6dfb\u5bcc\u6052\u751f\u6307\u6570C", "style": "QDII", "pinyin": "HUITIANFUHENGSHENGZHISHUC"}, "010790": {"suoxie": "HFTJHZXHHA", "name": "\u6d77\u5bcc\u901a\u5747\u8861\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGJUNHENGZHENXUANHUNHEA"}, "010791": {"suoxie": "HFTJHZXHHC", "name": "\u6d77\u5bcc\u901a\u5747\u8861\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGJUNHENGZHENXUANHUNHEC"}, "010792": {"suoxie": "HACZXFHHA", "name": "\u534e\u5b89\u6210\u957f\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHENGZHANGXIANFENGHUNHEA"}, "010793": {"suoxie": "HACZXFHHC", "name": "\u534e\u5b89\u6210\u957f\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHENGZHANGXIANFENGHUNHEC"}, "010794": {"suoxie": "DHXX66GYDK", "name": "\u4e1c\u6d77\u946b\u4eab66\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGHAIXINXIANG66GEYUEDINGKAI"}, "010795": {"suoxie": "MSJZFXYNCYHHA", "name": "\u6c11\u751f\u4ef7\u503c\u53d1\u73b0\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAZHIFAXIANYINIANCHIYOUHUNHEA"}, "010796": {"suoxie": "MSJZFXYNCYHHC", "name": "\u6c11\u751f\u4ef7\u503c\u53d1\u73b0\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAZHIFAXIANYINIANCHIYOUHUNHEC"}, "010797": {"suoxie": "CCYXHBLGYCYHHA", "name": "\u957f\u57ce\u4f18\u9009\u56de\u62a5\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANHUIBAOLIUGEYUECHIYOUHUNHEA"}, "010798": {"suoxie": "CCYXHBLGYCYHHC", "name": "\u957f\u57ce\u4f18\u9009\u56de\u62a5\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANHUIBAOLIUGEYUECHIYOUHUNHEC"}, "010799": {"suoxie": "CCYXWJLGYCYHHA", "name": "\u957f\u57ce\u4f18\u9009\u7a33\u8fdb\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANWENJINLIUGEYUECHIYOUHUNHEA"}, "010800": {"suoxie": "CCYXWJLGYCYHHC", "name": "\u957f\u57ce\u4f18\u9009\u7a33\u8fdb\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANWENJINLIUGEYUECHIYOUHUNHEC"}, "010801": {"suoxie": "CJLHXFJXGPA", "name": "\u957f\u6c5f\u91cf\u5316\u6d88\u8d39\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAXIAOFEIJINGXUANGUPIAOA"}, "010802": {"suoxie": "CJLHXFJXGPC", "name": "\u957f\u6c5f\u91cf\u5316\u6d88\u8d39\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAXIAOFEIJINGXUANGUPIAOC"}, "010803": {"suoxie": "THQXZQA", "name": "\u5929\u5f18\u5e86\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGQINGXIANGZHAIQUANA"}, "010804": {"suoxie": "THQXZQC", "name": "\u5929\u5f18\u5e86\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGQINGXIANGZHAIQUANC"}, "010805": {"suoxie": "DCXNYCA", "name": "\u4e1c\u8d22\u65b0\u80fd\u6e90\u8f66A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINNENGYUANCHEA"}, "010806": {"suoxie": "DCXNYCC", "name": "\u4e1c\u8d22\u65b0\u80fd\u6e90\u8f66C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINNENGYUANCHEC"}, "010807": {"suoxie": "RTWXZY6GYCYQHHA", "name": "\u878d\u901a\u7a33\u4fe1\u589e\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENXINZENGYI6GEYUECHIYOUQIHUNHEA"}, "010808": {"suoxie": "DCCLXFHHA", "name": "\u8fbe\u8bda\u7b56\u7565\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCELUEXIANFENGHUNHEA"}, "010809": {"suoxie": "DCCLXFHHC", "name": "\u8fbe\u8bda\u7b56\u7565\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCELUEXIANFENGHUNHEC"}, "010810": {"suoxie": "XCJYZDZA", "name": "\u6e58\u8d22\u4e45\u76c8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XIANGCAIJIUYINGZHONGDUANZHAIA"}, "010811": {"suoxie": "XCJYZDZC", "name": "\u6e58\u8d22\u4e45\u76c8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XIANGCAIJIUYINGZHONGDUANZHAIC"}, "010812": {"suoxie": "ZYZLXXCYGPC", "name": "\u4e2d\u94f6\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHANLUEXINXINGCHANYEGUPIAOC"}, "010813": {"suoxie": "HATYYNCYHHA", "name": "\u534e\u5b89\u6dfb\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANYIYINIANCHIYOUHUNHEA"}, "010814": {"suoxie": "HATYYNCYHHC", "name": "\u534e\u5b89\u6dfb\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANYIYINIANCHIYOUHUNHEC"}, "010815": {"suoxie": "NYXXXFGP", "name": "\u519c\u94f6\u65b0\u5174\u6d88\u8d39\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NONGYINXINXINGXIAOFEIGUPIAO"}, "010816": {"suoxie": "YHYXYNCYQZQ", "name": "\u94f6\u534e\u8fdc\u5174\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHUAYUANXINGYINIANCHIYOUQIZHAIQUAN"}, "010817": {"suoxie": "GLAXW3GYCYHHA", "name": "\u56fd\u8054\u5b89\u946b\u7a333\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINWEN3GEYUECHIYOUHUNHEA"}, "010818": {"suoxie": "GLAXW3GYCYHHC", "name": "\u56fd\u8054\u5b89\u946b\u7a333\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINWEN3GEYUECHIYOUHUNHEC"}, "010819": {"suoxie": "AXWJHB6GYHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u56de\u62a56\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANHUIBAO6GEYUEHUNHEA"}, "010820": {"suoxie": "AXWJHB6GYHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u56de\u62a56\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANHUIBAO6GEYUEHUNHEC"}, "010821": {"suoxie": "DFHDYCLHHB", "name": "\u4e1c\u65b9\u7ea2\u591a\u5143\u7b56\u7565\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGDUOYUANCELUEHUNHEB"}, "010822": {"suoxie": "JSCCSAHBHHA", "name": "\u666f\u987a\u957f\u57ce\u987a\u5b89\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGSHUNANHUIBAOHUNHEA"}, "010823": {"suoxie": "JSCCSAHBHHC", "name": "\u666f\u987a\u957f\u57ce\u987a\u5b89\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGSHUNANHUIBAOHUNHEC"}, "010824": {"suoxie": "THCXCZHHFQSA", "name": "\u5929\u5f18\u521b\u65b0\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGCHUANGXINCHENGZHANGHUNHEFAQISHIA"}, "010825": {"suoxie": "THCXCZHHFQSC", "name": "\u5929\u5f18\u521b\u65b0\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGCHUANGXINCHENGZHANGHUNHEFAQISHIC"}, "010826": {"suoxie": "DCCYQSHHA", "name": "\u5927\u6210\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHANYEQUSHIHUNHEA"}, "010827": {"suoxie": "DCCYQSHHC", "name": "\u5927\u6210\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHANYEQUSHIHUNHEC"}, "010828": {"suoxie": "GSABWYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u60a6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENYUEHUNHEA"}, "010829": {"suoxie": "GSABWYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u60a6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENYUEHUNHEC"}, "010830": {"suoxie": "GTTL9GYCYQHHA", "name": "\u56fd\u6cf0\u901a\u52299\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAITONGLI9GEYUECHIYOUQIHUNHEA"}, "010831": {"suoxie": "GTTL9GYCYQHHC", "name": "\u56fd\u6cf0\u901a\u52299\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAITONGLI9GEYUECHIYOUQIHUNHEC"}, "010832": {"suoxie": "GTHYHHA", "name": "\u56fd\u6cf0\u5408\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHEYIHUNHEA"}, "010833": {"suoxie": "GTHYHHC", "name": "\u56fd\u6cf0\u5408\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIHEYIHUNHEC"}, "010834": {"suoxie": "GTTY18GYCYQHHA", "name": "\u56fd\u6cf0\u540c\u76ca18\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAITONGYI18GEYUECHIYOUQIHUNHEA"}, "010835": {"suoxie": "GTTY18GYCYQHHC", "name": "\u56fd\u6cf0\u540c\u76ca18\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAITONGYI18GEYUECHIYOUQIHUNHEC"}, "010836": {"suoxie": "GTRTCZZQ", "name": "\u56fd\u6cf0\u745e\u6cf0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUITAICHUNZHAIZHAIQUAN"}, "010837": {"suoxie": "GLHJZQA", "name": "\u683c\u6797\u6cd3\u666f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GELINHONGJINGZHAIQUANA"}, "010838": {"suoxie": "GLHJZQC", "name": "\u683c\u6797\u6cd3\u666f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GELINHONGJINGZHAIQUANC"}, "010839": {"suoxie": "YFDRALHPZHHA", "name": "\u6613\u65b9\u8fbe\u745e\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIANLINGHUOPEIZHIHUNHEA"}, "010840": {"suoxie": "YFDRALHPZHHC", "name": "\u6613\u65b9\u8fbe\u745e\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIANLINGHUOPEIZHIHUNHEC"}, "010841": {"suoxie": "HBHLJXHHC", "name": "\u534e\u5b9d\u7ea2\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOHONGLIJINGXUANHUNHEC"}, "010842": {"suoxie": "HBKJXFHHC", "name": "\u534e\u5b9d\u79d1\u6280\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOKEJIXIANFENGHUNHEC"}, "010843": {"suoxie": "FGTRHBHHA", "name": "\u5bcc\u56fd\u5929\u6da6\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOTIANRUNHUIBAOHUNHEA"}, "010844": {"suoxie": "FGTRHBHHC", "name": "\u5bcc\u56fd\u5929\u6da6\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOTIANRUNHUIBAOHUNHEC"}, "010845": {"suoxie": "TDHLBKHB12GYHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6ce2\u63a7\u56de\u62a512\u4e2a\u6708\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIBOKONGHUIBAO12GEYUEHUNHE"}, "010846": {"suoxie": "NFZYYX3GYCYQHHA", "name": "\u5357\u65b9\u5353\u8d8a\u4f18\u90093\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHUOYUEYOUXUAN3GEYUECHIYOUQIHUNHEA"}, "010847": {"suoxie": "NFZYYX3GYCYQHHC", "name": "\u5357\u65b9\u5353\u8d8a\u4f18\u90093\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHUOYUEYOUXUAN3GEYUECHIYOUQIHUNHEC"}, "010848": {"suoxie": "BHHJXRYNDQKFZQ", "name": "\u6e24\u6d77\u6c47\u91d1\u5174\u8363\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOHAIHUIJINXINGRONGYINIANDINGQIKAIFANGZHAIQUAN"}, "010849": {"suoxie": "YFDJZYSQYHHC", "name": "\u6613\u65b9\u8fbe\u7ade\u4e89\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAJINGZHENGYOUSHIQIYEHUNHEC"}, "010850": {"suoxie": "HFTFLSGYCYHHA", "name": "\u6d77\u5bcc\u901a\u5bcc\u5229\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFULISANGEYUECHIYOUHUNHEA"}, "010851": {"suoxie": "HFTFLSGYCYHHC", "name": "\u6d77\u5bcc\u901a\u5bcc\u5229\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFULISANGEYUECHIYOUHUNHEC"}, "010852": {"suoxie": "ZONXCZHHA", "name": "\u4e2d\u6b27\u5185\u9700\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUNEIXUCHENGZHANGHUNHEA"}, "010853": {"suoxie": "ZONXCZHHC", "name": "\u4e2d\u6b27\u5185\u9700\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUNEIXUCHENGZHANGHUNHEC"}, "010854": {"suoxie": "HTFHS300JBMZQZSA", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300JIBENMIANZENGQIANGZHISHUA"}, "010855": {"suoxie": "HTFHS300JBMZQZSC", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300JIBENMIANZENGQIANGZHISHUC"}, "010856": {"suoxie": "MSJYHZZQ", "name": "\u6c11\u751f\u52a0\u94f6\u6052\u6cfd\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHENGZEZHAIQUAN"}, "010857": {"suoxie": "BYXLYNCYQHHA", "name": "\u5b9d\u76c8\u7965\u4e50\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGLEYINIANCHIYOUQIHUNHEA"}, "010858": {"suoxie": "BYXLYNCYQHHC", "name": "\u5b9d\u76c8\u7965\u4e50\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGLEYINIANCHIYOUQIHUNHEC"}, "010859": {"suoxie": "FGZZ02NGKZZSA", "name": "\u5bcc\u56fd\u4e2d\u503a0-2\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAI02NIANGUOKAIZHAIZHISHUA"}, "010860": {"suoxie": "FGZZ02NGKZZSC", "name": "\u5bcc\u56fd\u4e2d\u503a0-2\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHAI02NIANGUOKAIZHAIZHISHUC"}, "010861": {"suoxie": "CXQYYXYNCYHH", "name": "\u957f\u4fe1\u4f01\u4e1a\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINQIYEYOUXUANYINIANCHIYOUHUNHE"}, "010862": {"suoxie": "DFHQYSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u9633\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIYANGSANNIANCHIYOUHUNHEB"}, "010863": {"suoxie": "PHAY5GYCYQHHA", "name": "\u9e4f\u534e\u5b89\u88d55\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYU5GEYUECHIYOUQIHUNHEA"}, "010864": {"suoxie": "HDZYHHA", "name": "\u6cd3\u5fb7\u5353\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEZHUOYUANHUNHEA"}, "010865": {"suoxie": "HDZYHHC", "name": "\u6cd3\u5fb7\u5353\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDEZHUOYUANHUNHEC"}, "010866": {"suoxie": "JSFKWJYLYNCYHHFOF", "name": "\u5609\u5b9e\u798f\u5eb7\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIFUKANGWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "010868": {"suoxie": "HBAYHH", "name": "\u534e\u5b9d\u5b89\u76c8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUABAOANYINGHUNHE"}, "010869": {"suoxie": "HTFWJXXYNCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u6b23\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANXINXIANGYINIANCHIYOUHUNHE"}, "010870": {"suoxie": "HTFWJXTYLGYCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u946b\u6dfb\u76ca\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANXINTIANYILIUGEYUECHIYOUHUNHE"}, "010871": {"suoxie": "ZYXYZLZQLOFC", "name": "\u4e2d\u94f6\u4fe1\u7528\u589e\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINXINYONGZENGLIZHAIQUANLOFC"}, "010872": {"suoxie": "BSHS300ZSZQA", "name": "\u535a\u65f6\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUSHEN300ZHISHUZENGQIANGA"}, "010873": {"suoxie": "BSHS300ZSZQC", "name": "\u535a\u65f6\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUSHEN300ZHISHUZENGQIANGC"}, "010874": {"suoxie": "TKPZSHHHA", "name": "\u6cf0\u5eb7\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGPINZHISHENGHUOHUNHEA"}, "010875": {"suoxie": "TKPZSHHHC", "name": "\u6cf0\u5eb7\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGPINZHISHENGHUOHUNHEC"}, "010876": {"suoxie": "ZSZXXFYNCYHHA", "name": "\u6d59\u5546\u667a\u9009\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANXIANFENGYINIANCHIYOUHUNHEA"}, "010877": {"suoxie": "ZSZXXFYNCYHHC", "name": "\u6d59\u5546\u667a\u9009\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGZHIXUANXIANFENGYINIANCHIYOUHUNHEC"}, "010878": {"suoxie": "NDYSCY", "name": "\u8bfa\u5fb7\u4f18\u52bf\u4ea7\u4e1a", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEYOUSHICHANYE"}, "010879": {"suoxie": "NFBSHHA", "name": "\u5357\u65b9\u5b9d\u5347\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOSHENGHUNHEA"}, "010880": {"suoxie": "NFBSHHC", "name": "\u5357\u65b9\u5b9d\u5347\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOSHENGHUNHEC"}, "010881": {"suoxie": "NFBSHHA", "name": "\u5357\u65b9\u5b9d\u987a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOSHUNHUNHEA"}, "010882": {"suoxie": "NFBSHHC", "name": "\u5357\u65b9\u5b9d\u987a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOSHUNHUNHEC"}, "010883": {"suoxie": "XCZHJC", "name": "\u4fe1\u8bda\u667a\u60e0\u91d1C", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGZHIHUIJINC"}, "010884": {"suoxie": "ZYZXZQ", "name": "\u4e2d\u94f6\u81fb\u4eab\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENXIANGZHAIQUAN"}, "010885": {"suoxie": "CSYSQYJXHHA", "name": "\u957f\u76db\u4f18\u52bf\u4f01\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGYOUSHIQIYEJINGXUANHUNHEA"}, "010886": {"suoxie": "CSYSQYJXHHC", "name": "\u957f\u76db\u4f18\u52bf\u4f01\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGYOUSHIQIYEJINGXUANHUNHEC"}, "010887": {"suoxie": "NFXFSJHHA", "name": "\u5357\u65b9\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXIAOFEISHENGJIHUNHEA"}, "010888": {"suoxie": "NFXFSJHHC", "name": "\u5357\u65b9\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXIAOFEISHENGJIHUNHEC"}, "010889": {"suoxie": "GYCYBLNDKHHC", "name": "\u5de5\u94f6\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGYEBANLIANGNIANDINGKAIHUNHEC"}, "010890": {"suoxie": "JYHFLGYCYHHA", "name": "\u4ea4\u94f6\u9e3f\u798f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGFULIUGEYUECHIYOUHUNHEA"}, "010891": {"suoxie": "JYHFLGYCYHHC", "name": "\u4ea4\u94f6\u9e3f\u798f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGFULIUGEYUECHIYOUHUNHEC"}, "010892": {"suoxie": "ZYZQJXHYGPA", "name": "\u4e2d\u94f6\u8bc1\u5238\u7cbe\u9009\u884c\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHENGQUANJINGXUANHANGYEGUPIAOA"}, "010893": {"suoxie": "ZYZQJXHYGPC", "name": "\u4e2d\u94f6\u8bc1\u5238\u7cbe\u9009\u884c\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHENGQUANJINGXUANHANGYEGUPIAOC"}, "010894": {"suoxie": "PHHZYXHHA", "name": "\u9e4f\u534e\u6c47\u667a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAHUIZHIYOUXUANHUNHEA"}, "010895": {"suoxie": "PHHZYXHHC", "name": "\u9e4f\u534e\u6c47\u667a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAHUIZHIYOUXUANHUNHEC"}, "010896": {"suoxie": "TPJZZZGPA", "name": "\u592a\u5e73\u4ef7\u503c\u589e\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGJIAZHIZENGZHANGGUPIAOA"}, "010897": {"suoxie": "TPJZZZGPC", "name": "\u592a\u5e73\u4ef7\u503c\u589e\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGJIAZHIZENGZHANGGUPIAOC"}, "010898": {"suoxie": "YHCYDLHH", "name": "\u94f6\u6cb3\u4ea7\u4e1a\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHANYEDONGLIHUNHE"}, "010899": {"suoxie": "SYHHSYZQZQA", "name": "\u4e0a\u94f6\u6167\u6052\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGYINHUIHENGSHOUYIZENGQIANGZHAIQUANA"}, "010900": {"suoxie": "ZOSYWJYNHHA", "name": "\u4e2d\u6b27\u751f\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUSHENGYIWENJIANYINIANHUNHEA"}, "010901": {"suoxie": "ZOSYWJYNHHC", "name": "\u4e2d\u6b27\u751f\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUSHENGYIWENJIANYINIANHUNHEC"}, "010902": {"suoxie": "BSCZLHHHA", "name": "\u535a\u65f6\u6210\u957f\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHENGZHANGLINGHANGHUNHEA"}, "010903": {"suoxie": "BSCZLHHHC", "name": "\u535a\u65f6\u6210\u957f\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHENGZHANGLINGHANGHUNHEC"}, "010904": {"suoxie": "BSSJX6GYCYHHA", "name": "\u535a\u65f6\u53cc\u5b63\u946b6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHISHUANGJIXIN6GEYUECHIYOUHUNHEA"}, "010905": {"suoxie": "BSSJX6GYCYHHC", "name": "\u535a\u65f6\u53cc\u5b63\u946b6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHISHUANGJIXIN6GEYUECHIYOUHUNHEC"}, "010906": {"suoxie": "BYYXHHA", "name": "\u535a\u8fdc\u4f18\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOYUANYOUXIANGHUNHEA"}, "010907": {"suoxie": "BYYXHHC", "name": "\u535a\u8fdc\u4f18\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOYUANYOUXIANGHUNHEC"}, "010908": {"suoxie": "DCHS300ZQFQSA", "name": "\u5927\u6210\u6caa\u6df1300\u589e\u5f3a\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGHUSHEN300ZENGQIANGFAQISHIA"}, "010909": {"suoxie": "DCHS300ZQFQSC", "name": "\u5927\u6210\u6caa\u6df1300\u589e\u5f3a\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGHUSHEN300ZENGQIANGFAQISHIC"}, "010912": {"suoxie": "GTCZJZHHA", "name": "\u56fd\u6cf0\u6210\u957f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAICHENGZHANGJIAZHIHUNHEA"}, "010913": {"suoxie": "GTCZJZHHC", "name": "\u56fd\u6cf0\u6210\u957f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAICHENGZHANGJIAZHIHUNHEC"}, "010914": {"suoxie": "CSCZJXHHA", "name": "\u957f\u76db\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGCHENGZHANGJINGXUANHUNHEA"}, "010915": {"suoxie": "CSCZJXHHC", "name": "\u957f\u76db\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGCHENGZHANGJINGXUANHUNHEC"}, "010916": {"suoxie": "JYZXHBHH", "name": "\u4ea4\u94f6\u81fb\u9009\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINZHENXUANHUIBAOHUNHE"}, "010917": {"suoxie": "DBRXZQA", "name": "\u5fb7\u90a6\u9510\u7965\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIXIANGZHAIQUANA"}, "010918": {"suoxie": "DBRXZQC", "name": "\u5fb7\u90a6\u9510\u7965\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIXIANGZHAIQUANC"}, "010919": {"suoxie": "PHZRYNCYQHHA", "name": "\u9e4f\u534e\u62db\u6da6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZHAORUNYINIANCHIYOUQIHUNHEA"}, "010920": {"suoxie": "PHZRYNCYQHHC", "name": "\u9e4f\u534e\u62db\u6da6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZHAORUNYINIANCHIYOUQIHUNHEC"}, "010922": {"suoxie": "ZQJXYNCY", "name": "\u6731\u96c0\u5320\u5fc3\u4e00\u5e74\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUEJIANGXINYINIANCHIYOU"}, "010923": {"suoxie": "YYXXHH", "name": "\u6c38\u8d62\u946b\u6b23\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGXINXINHUNHE"}, "010924": {"suoxie": "BSSJX6GYCYHHB", "name": "\u535a\u65f6\u53cc\u5b63\u946b6\u4e2a\u6708\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHISHUANGJIXIN6GEYUECHIYOUHUNHEB"}, "010925": {"suoxie": "XYKJZZ1GYGDHHA", "name": "\u5174\u94f6\u79d1\u6280\u589e\u957f1\u4e2a\u6708\u6eda\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINKEJIZENGZHANG1GEYUEGUNDONGHUNHEA"}, "010926": {"suoxie": "XYKJZZ1GYGDHHC", "name": "\u5174\u94f6\u79d1\u6280\u589e\u957f1\u4e2a\u6708\u6eda\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINKEJIZENGZHANG1GEYUEGUNDONGHUNHEC"}, "010927": {"suoxie": "DCYJZLZQA", "name": "\u5927\u6210\u5143\u5409\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGYUANJIZENGLIZHAIQUANA"}, "010928": {"suoxie": "DCYJZLZQC", "name": "\u5927\u6210\u5143\u5409\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGYUANJIZENGLIZHAIQUANC"}, "010929": {"suoxie": "DCHXJZZXHHA", "name": "\u5927\u6210\u6838\u5fc3\u4ef7\u503c\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHEXINJIAZHIZHENXUANHUNHEA"}, "010930": {"suoxie": "DCHXJZZXHHC", "name": "\u5927\u6210\u6838\u5fc3\u4ef7\u503c\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHEXINJIAZHIZHENXUANHUNHEC"}, "010931": {"suoxie": "GLAXY1GYCYHHA", "name": "\u56fd\u8054\u5b89\u946b\u51431\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINYUAN1GEYUECHIYOUHUNHEA"}, "010932": {"suoxie": "GLAXY1GYCYHHC", "name": "\u56fd\u8054\u5b89\u946b\u51431\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANXINYUAN1GEYUECHIYOUHUNHEC"}, "010933": {"suoxie": "ZYWHDZZQE", "name": "\u4e2d\u94f6\u7a33\u6c47\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYINWENHUIDUANZHAIZHAIQUANE"}, "010934": {"suoxie": "GSABWF6GYCYQHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u798f6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENFU6GEYUECHIYOUQIHUNHEA"}, "010935": {"suoxie": "GSABWF6GYCYQHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u798f6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENFU6GEYUECHIYOUQIHUNHEC"}, "010936": {"suoxie": "JYJHCZYNHHA", "name": "\u4ea4\u94f6\u5747\u8861\u6210\u957f\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJUNHENGCHENGZHANGYINIANHUNHEA"}, "010937": {"suoxie": "JYJHCZYNHHC", "name": "\u4ea4\u94f6\u5747\u8861\u6210\u957f\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJUNHENGCHENGZHANGYINIANHUNHEC"}, "010938": {"suoxie": "DMZHYNCYQHHA", "name": "\u5927\u6469\u62db\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DAMOZHAOHUIYINIANCHIYOUQIHUNHEA"}, "010939": {"suoxie": "DMZHYNCYQHHC", "name": "\u5927\u6469\u62db\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DAMOZHAOHUIYINIANCHIYOUQIHUNHEC"}, "010940": {"suoxie": "DCAXDLLGYCYHHA", "name": "\u5927\u6210\u5b89\u4eab\u5f97\u5229\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGANXIANGDELILIUGEYUECHIYOUHUNHEA"}, "010941": {"suoxie": "DCAXDLLGYCYHHC", "name": "\u5927\u6210\u5b89\u4eab\u5f97\u5229\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGANXIANGDELILIUGEYUECHIYOUHUNHEC"}, "010942": {"suoxie": "ZSRL6GYCYQHHA", "name": "\u62db\u5546\u745e\u4e506\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUILE6GEYUECHIYOUQIHUNHEA"}, "010943": {"suoxie": "ZSRL6GYCYQHHC", "name": "\u62db\u5546\u745e\u4e506\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUILE6GEYUECHIYOUQIHUNHEC"}, "010944": {"suoxie": "ZSSYMSYXA", "name": "\u62db\u5546\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGSHANGYEMOSHIYOUXUANA"}, "010945": {"suoxie": "ZSSYMSYXC", "name": "\u62db\u5546\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGSHANGYEMOSHIYOUXUANC"}, "010946": {"suoxie": "ZYZQHFYNDKZQFQS", "name": "\u4e2d\u94f6\u8bc1\u5238\u6c47\u798f\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANHUIFUYINIANDINGKAIZHAIQUANFAQISHI"}, "010947": {"suoxie": "ZOJXHHA", "name": "\u4e2d\u6b27\u5609\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAXUANHUNHEA"}, "010948": {"suoxie": "ZOJXHHC", "name": "\u4e2d\u6b27\u5609\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAXUANHUNHEC"}, "010951": {"suoxie": "ZJCZJXHHA", "name": "\u4e2d\u91d1\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINCHENGZHANGJINGXUANHUNHEA"}, "010952": {"suoxie": "ZJCZJXHHC", "name": "\u4e2d\u91d1\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJINCHENGZHANGJINGXUANHUNHEC"}, "010953": {"suoxie": "THGZA50ZSA", "name": "\u5929\u5f18\u56fd\u8bc1A50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGA50ZHISHUA"}, "010954": {"suoxie": "THGZA50ZSC", "name": "\u5929\u5f18\u56fd\u8bc1A50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGA50ZHISHUC"}, "010955": {"suoxie": "THZZZNQCZSFQSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGZHINENGQICHEZHISHUFAQISHIA"}, "010956": {"suoxie": "THZZZNQCZSFQSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGZHINENGQICHEZHISHUFAQISHIC"}, "010957": {"suoxie": "JTJALHA", "name": "\u4e5d\u6cf0\u4e45\u5b89\u91cf\u5316A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUANLIANGHUAA"}, "010958": {"suoxie": "XCYL2035SNCYHHFOF", "name": "\u4fe1\u8bda\u517b\u80012035\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINCHENGYANGLAO2035SANNIANCHIYOUHUNHEFOF"}, "010959": {"suoxie": "DCHZYNDKZQFQS", "name": "\u5927\u6210\u60e0\u6cfd\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIZEYINIANDINGKAIZHAIQUANFAQISHI"}, "010960": {"suoxie": "DCHHYNDKZQFQS", "name": "\u5927\u6210\u60e0\u6052\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIHENGYINIANDINGKAIZHAIQUANFAQISHI"}, "010961": {"suoxie": "JTJALHC", "name": "\u4e5d\u6cf0\u4e45\u5b89\u91cf\u5316C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUANLIANGHUAC"}, "010962": {"suoxie": "ZYXXXFCZHHC", "name": "\u4e2d\u94f6\u946b\u65b0\u6d88\u8d39\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINXINXINXIAOFEICHENGZHANGHUNHEC"}, "010963": {"suoxie": "XDAYZQDLHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u5468\u671f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHOUQIDONGLIHUNHE"}, "010964": {"suoxie": "PHKZZZQC", "name": "\u9e4f\u534e\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "PENGHUAKEZHUANZHAIZHAIQUANC"}, "010965": {"suoxie": "ZYXXXFCZHHA", "name": "\u4e2d\u94f6\u946b\u65b0\u6d88\u8d39\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINXINXINXIAOFEICHENGZHANGHUNHEA"}, "010966": {"suoxie": "FGCZLHHH", "name": "\u5bcc\u56fd\u6210\u957f\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHENGZHANGLINGHANGHUNHE"}, "010967": {"suoxie": "BDJFHHA", "name": "\u535a\u9053\u5609\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIAFENGHUNHEA"}, "010968": {"suoxie": "BDJFHHC", "name": "\u535a\u9053\u5609\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOJIAFENGHUNHEC"}, "010969": {"suoxie": "HXAY6GYCYQHHA", "name": "\u534e\u590f\u5b89\u96336\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAANYANG6GEYUECHIYOUQIHUNHEA"}, "010970": {"suoxie": "HXAY6GYCYQHHC", "name": "\u534e\u590f\u5b89\u96336\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAANYANG6GEYUECHIYOUQIHUNHEC"}, "010971": {"suoxie": "HXYXLGYCYQHHA", "name": "\u534e\u590f\u6c38\u946b\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGXINLIUGEYUECHIYOUQIHUNHEA"}, "010972": {"suoxie": "HXYXLGYCYQHHC", "name": "\u534e\u590f\u6c38\u946b\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGXINLIUGEYUECHIYOUQIHUNHEC"}, "010973": {"suoxie": "BSJYCZZQC", "name": "\u535a\u65f6\u805a\u6e90\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIJUYUANCHUNZHAIZHAIQUANC"}, "010976": {"suoxie": "HSHY87GYDKZ", "name": "\u534e\u5546\u9e3f\u76c887\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUASHANGHONGYING87GEYUEDINGKAIZHAI"}, "010977": {"suoxie": "HXHY6GYCYQHHA", "name": "\u534e\u590f\u9e3f\u96336\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHONGYANG6GEYUECHIYOUQIHUNHEA"}, "010978": {"suoxie": "HXHY6GYCYQHHC", "name": "\u534e\u590f\u9e3f\u96336\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHONGYANG6GEYUECHIYOUQIHUNHEC"}, "010979": {"suoxie": "HXDRZQA", "name": "\u534e\u590f\u9f0e\u6da6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGRUNZHAIQUANA"}, "010980": {"suoxie": "HXDRZQC", "name": "\u534e\u590f\u9f0e\u6da6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIADINGRUNZHAIQUANC"}, "010981": {"suoxie": "XQHHYNCYHHA", "name": "\u5174\u5168\u6c47\u8679\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANHUIHONGYINIANCHIYOUHUNHEA"}, "010982": {"suoxie": "XQHHYNCYHHC", "name": "\u5174\u5168\u6c47\u8679\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANHUIHONGYINIANCHIYOUHUNHEC"}, "010983": {"suoxie": "XYHZ87GYDKZ", "name": "\u5174\u94f6\u6c47\u6cfd87\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIZE87GEYUEDINGKAIZHAI"}, "010984": {"suoxie": "GSABWAHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENANHUNHEA"}, "010985": {"suoxie": "GSABWAHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENANHUNHEC"}, "010986": {"suoxie": "YHXYJJHZQC", "name": "\u94f6\u534e\u4fe1\u7528\u5b63\u5b63\u7ea2\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJIJIHONGZHAIQUANC"}, "010987": {"suoxie": "ZRXRJXYNCYHHA", "name": "\u4e2d\u878d\u946b\u9510\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGXINRUIJINGXUANYINIANCHIYOUHUNHEA"}, "010988": {"suoxie": "ZRXRJXYNCYHHC", "name": "\u4e2d\u878d\u946b\u9510\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGXINRUIJINGXUANYINIANCHIYOUHUNHEC"}, "010989": {"suoxie": "NFFDCETFLJE", "name": "\u5357\u65b9\u623f\u5730\u4ea7ETF\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGFANGDICHANETFLIANJIEE"}, "010990": {"suoxie": "NFYSJSETFLJE", "name": "\u5357\u65b9\u6709\u8272\u91d1\u5c5eETF\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYOUSEJINSHUETFLIANJIEE"}, "010991": {"suoxie": "CSTXHYPZHHC", "name": "\u957f\u76db\u540c\u946b\u884c\u4e1a\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGTONGXINHANGYEPEIZHIHUNHEC"}, "010992": {"suoxie": "DCZZ500A", "name": "\u4e1c\u8d22\u4e2d\u8bc1500A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIZHONGZHENG500A"}, "010993": {"suoxie": "DCZZ500C", "name": "\u4e1c\u8d22\u4e2d\u8bc1500C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIZHONGZHENG500C"}, "010994": {"suoxie": "BSCXJJHHA", "name": "\u535a\u65f6\u521b\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGXINJINGJIHUNHEA"}, "010995": {"suoxie": "BSCXJJHHC", "name": "\u535a\u65f6\u521b\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGXINJINGJIHUNHEC"}, "010996": {"suoxie": "ZSPZSJHHA", "name": "\u62db\u5546\u54c1\u8d28\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHISHENGJIHUNHEA"}, "010997": {"suoxie": "ZSPZSJHHC", "name": "\u62db\u5546\u54c1\u8d28\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHISHENGJIHUNHEC"}, "010998": {"suoxie": "BDXFZH", "name": "\u535a\u9053\u6d88\u8d39\u667a\u822a", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOXIAOFEIZHIHANG"}, "010999": {"suoxie": "XHRFHHA", "name": "\u5174\u534e\u745e\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGHUARUIFENGHUNHEA"}, "011000": {"suoxie": "XHRFHHC", "name": "\u5174\u534e\u745e\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGHUARUIFENGHUNHEC"}, "011001": {"suoxie": "ZYXRJZYNCYHH", "name": "\u4e2d\u90ae\u5174\u8363\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUXINGRONGJIAZHIYINIANCHIYOUHUNHE"}, "011002": {"suoxie": "TTDJKZTHHA", "name": "\u540c\u6cf0\u5927\u5065\u5eb7\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIDAJIANKANGZHUTIHUNHEA"}, "011003": {"suoxie": "TTDJKZTHHC", "name": "\u540c\u6cf0\u5927\u5065\u5eb7\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIDAJIANKANGZHUTIHUNHEC"}, "011004": {"suoxie": "YYXSHH", "name": "\u6c38\u8d62\u946b\u76db\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGXINSHENGHUNHE"}, "011006": {"suoxie": "GYYFSNCYQHH", "name": "\u5de5\u94f6\u5706\u4e30\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYUANFENGSANNIANCHIYOUQIHUNHE"}, "011007": {"suoxie": "GTRYSZCZZQC", "name": "\u56fd\u6295\u745e\u94f6\u987a\u81fb\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNZHENCHUNZHAIZHAIQUANC"}, "011008": {"suoxie": "GSABZHDZZQA", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5f18\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOSHOUANBAOZUNHONGDUANZHAIZHAIQUANA"}, "011009": {"suoxie": "GSABZHDZZQC", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5f18\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOSHOUANBAOZUNHONGDUANZHAIZHAIQUANC"}, "011010": {"suoxie": "GSABZHDZZQE", "name": "\u56fd\u5bff\u5b89\u4fdd\u5c0a\u5f18\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOSHOUANBAOZUNHONGDUANZHAIZHAIQUANE"}, "011011": {"suoxie": "RTCYQS2NFBYZHH", "name": "\u878d\u901a\u4ea7\u4e1a\u8d8b\u52bf2\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGCHANYEQUSHI2NIANFENGBIYUNZUOHUNHE"}, "011012": {"suoxie": "TKAZZDZC", "name": "\u6cf0\u5eb7\u5b89\u6cfd\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIKANGANZEZHONGDUANZHAIC"}, "011013": {"suoxie": "CCXF30GPA", "name": "\u957f\u57ce\u6d88\u8d3930\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGXIAOFEI30GUPIAOA"}, "011014": {"suoxie": "CCXF30GPC", "name": "\u957f\u57ce\u6d88\u8d3930\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGXIAOFEI30GUPIAOC"}, "011015": {"suoxie": "JHJYHBHHA", "name": "\u5609\u5408\u9526\u5143\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEJINYUANHUIBAOHUNHEA"}, "011016": {"suoxie": "JHJYHBHHC", "name": "\u5609\u5408\u9526\u5143\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAHEJINYUANHUIBAOHUNHEC"}, "011017": {"suoxie": "PYJMYNHH", "name": "\u9e4f\u626c\u666f\u660e\u4e00\u5e74\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGMINGYINIANHUNHE"}, "011018": {"suoxie": "JSCCAZHBYNCYHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u6cfd\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANZEHUIBAOYINIANCHIYOUHUNHEA"}, "011019": {"suoxie": "JSCCAZHBYNCYHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u6cfd\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANZEHUIBAOYINIANCHIYOUHUNHEC"}, "011020": {"suoxie": "CTZGXFJXHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u6d88\u8d39\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGZIGUANXIAOFEIJINGXUANHUNHEC"}, "011021": {"suoxie": "HTFHLWHXZCLGYCYHHA", "name": "\u6c47\u6dfb\u5bcc\u4e92\u8054\u7f51\u6838\u5fc3\u8d44\u4ea7\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHULIANWANGHEXINZICHANLIUGEYUECHIYOUHUNHEA"}, "011022": {"suoxie": "HTFHLWHXZCLGYCYHHC", "name": "\u6c47\u6dfb\u5bcc\u4e92\u8054\u7f51\u6838\u5fc3\u8d44\u4ea7\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUHULIANWANGHEXINZICHANLIUGEYUECHIYOUHUNHEC"}, "011023": {"suoxie": "MSJYHLHH", "name": "\u6c11\u751f\u52a0\u94f6\u6c47\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINHUILIHUNHE"}, "011024": {"suoxie": "DXXLZQD", "name": "\u4e1c\u5174\u5174\u5229\u503a\u5238D", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGXINGXINGLIZHAIQUAND"}, "011025": {"suoxie": "JTJYLHA", "name": "\u4e5d\u6cf0\u4e45\u5143\u91cf\u5316A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUYUANLIANGHUAA"}, "011026": {"suoxie": "JTJYLHC", "name": "\u4e5d\u6cf0\u4e45\u5143\u91cf\u5316C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIJIUYUANLIANGHUAC"}, "011027": {"suoxie": "GSABWHHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5f18\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENHONGHUNHEA"}, "011028": {"suoxie": "GSABWHHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u5f18\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENHONGHUNHEC"}, "011029": {"suoxie": "AXYYYNDKZQ", "name": "\u5b89\u4fe1\u6c38\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGYINGYINIANDINGKAIZHAIQUAN"}, "011030": {"suoxie": "DCJZXFLHPZA", "name": "\u8fbe\u8bda\u4ef7\u503c\u5148\u950b\u7075\u6d3b\u914d\u7f6eA", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJIAZHIXIANFENGLINGHUOPEIZHIA"}, "011031": {"suoxie": "DCJZXFLHPZC", "name": "\u8fbe\u8bda\u4ef7\u503c\u5148\u950b\u7075\u6d3b\u914d\u7f6eC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGJIAZHIXIANFENGLINGHUOPEIZHIC"}, "011032": {"suoxie": "DFHRZSNDKHHC", "name": "\u4e1c\u65b9\u7ea2\u777f\u6cfd\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIZESANNIANDINGKAIHUNHEC"}, "011033": {"suoxie": "NFBHHHA", "name": "\u5357\u65b9\u5b9d\u6052\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOHENGHUNHEA"}, "011034": {"suoxie": "NFBHHHC", "name": "\u5357\u65b9\u5b9d\u6052\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOHENGHUNHEC"}, "011035": {"suoxie": "JSZZXTCYETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXITUCHANYEETFLIANJIEA"}, "011036": {"suoxie": "JSZZXTCYETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXITUCHANYEETFLIANJIEC"}, "011037": {"suoxie": "FGCQCZHH", "name": "\u5bcc\u56fd\u957f\u671f\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHANGQICHENGZHANGHUNHE"}, "011038": {"suoxie": "XHLLZZQA", "name": "\u65b0\u534e\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUALILVZHAIZHAIQUANA"}, "011039": {"suoxie": "XHLLZZQC", "name": "\u65b0\u534e\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUALILVZHAIZHAIQUANC"}, "011040": {"suoxie": "THGZSWYYETFFQSLJA", "name": "\u5929\u5f18\u56fd\u8bc1\u751f\u7269\u533b\u836fETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGSHENGWUYIYAOETFFAQISHILIANJIEA"}, "011041": {"suoxie": "THGZSWYYETFFQSLJC", "name": "\u5929\u5f18\u56fd\u8bc1\u751f\u7269\u533b\u836fETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGSHENGWUYIYAOETFFAQISHILIANJIEC"}, "011042": {"suoxie": "GTJZXFGPA", "name": "\u56fd\u6cf0\u4ef7\u503c\u5148\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIJIAZHIXIANFENGGUPIAOA"}, "011043": {"suoxie": "GTJZXFGPC", "name": "\u56fd\u6cf0\u4ef7\u503c\u5148\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIJIAZHIXIANFENGGUPIAOC"}, "011044": {"suoxie": "ZYSZHBYNCYQHHA", "name": "\u4e2d\u94f6\u987a\u6cfd\u56de\u62a5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNZEHUIBAOYINIANCHIYOUQIHUNHEA"}, "011045": {"suoxie": "ZYSZHBYNCYQHHC", "name": "\u4e2d\u94f6\u987a\u6cfd\u56de\u62a5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNZEHUIBAOYINIANCHIYOUQIHUNHEC"}, "011046": {"suoxie": "FGYZQYHHA", "name": "\u5bcc\u56fd\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYOUZHIQIYEHUNHEA"}, "011047": {"suoxie": "FGYZQYHHC", "name": "\u5bcc\u56fd\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYOUZHIQIYEHUNHEC"}, "011048": {"suoxie": "THHXHHA", "name": "\u5929\u5f18\u6052\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGHENGXINHUNHEA"}, "011049": {"suoxie": "THHXHHC", "name": "\u5929\u5f18\u6052\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGHENGXINHUNHEC"}, "011050": {"suoxie": "THYXHHA", "name": "\u5929\u5f18\u88d5\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYUXINHUNHEA"}, "011051": {"suoxie": "THYXHHC", "name": "\u5929\u5f18\u88d5\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYUXINHUNHEC"}, "011052": {"suoxie": "PHHYYNCYQHHA", "name": "\u9e4f\u534e\u5f18\u88d5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAHONGYUYINIANCHIYOUQIHUNHEA"}, "011053": {"suoxie": "PHHYYNCYQHHC", "name": "\u9e4f\u534e\u5f18\u88d5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAHONGYUYINIANCHIYOUQIHUNHEC"}, "011054": {"suoxie": "SWLXAXZXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINANXINZHIXUANHUNHEA"}, "011055": {"suoxie": "SWLXAXZXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b89\u946b\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINANXINZHIXUANHUNHEC"}, "011056": {"suoxie": "BSHXHBYNCYQHH", "name": "\u535a\u65f6\u6c47\u5174\u56de\u62a5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIXINGHUIBAOYINIANCHIYOUQIHUNHE"}, "011057": {"suoxie": "HTFFLDZZQC", "name": "\u6c47\u6dfb\u5bcc\u4e30\u5229\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUFENGLIDUANZHAIZHAIQUANC"}, "011058": {"suoxie": "JSCCCZLTYNCYHHA", "name": "\u666f\u987a\u957f\u57ce\u6210\u957f\u9f99\u5934\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGCHENGZHANGLONGTOUYINIANCHIYOUHUNHEA"}, "011059": {"suoxie": "JSCCCZLTYNCYHHC", "name": "\u666f\u987a\u957f\u57ce\u6210\u957f\u9f99\u5934\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGCHENGZHANGLONGTOUYINIANCHIYOUHUNHEC"}, "011060": {"suoxie": "XBLDCLYXHHC", "name": "\u897f\u90e8\u5229\u5f97\u7b56\u7565\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDECELUEYOUXUANHUNHEC"}, "011061": {"suoxie": "GFAYHBHHC", "name": "\u5e7f\u53d1\u5b89\u60a6\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANYUEHUIBAOHUNHEC"}, "011062": {"suoxie": "GFZZ710NGKZZSE", "name": "\u5e7f\u53d1\u4e2d\u503a7-10\u5e74\u56fd\u5f00\u503a\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAZHONGZHAI710NIANGUOKAIZHAIZHISHUE"}, "011063": {"suoxie": "GSABXQBHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u946b\u94b1\u5305\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOXINQIANBAOHUOBIB"}, "011064": {"suoxie": "NFYXYNCYQHHA", "name": "\u5357\u65b9\u8a89\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUXIANGYINIANCHIYOUQIHUNHEA"}, "011065": {"suoxie": "NFYXYNCYQHHC", "name": "\u5357\u65b9\u8a89\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUXIANGYINIANCHIYOUQIHUNHEC"}, "011066": {"suoxie": "DCGXJSCYGPC", "name": "\u5927\u6210\u9ad8\u65b0\u6280\u672f\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGGAOXINJISHUCHANYEGUPIAOC"}, "011067": {"suoxie": "CTZGHDCZI", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8fbe\u7eaf\u503aI", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANHONGDACHUNZHAII"}, "011068": {"suoxie": "HBZYYXHHC", "name": "\u534e\u5b9d\u8d44\u6e90\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOZIYUANYOUXUANHUNHEC"}, "011069": {"suoxie": "GYCZJXHHA", "name": "\u5de5\u94f6\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHENGZHANGJINGXUANHUNHEA"}, "011070": {"suoxie": "GYCZJXHHC", "name": "\u5de5\u94f6\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHENGZHANGJINGXUANHUNHEC"}, "011071": {"suoxie": "PHAYYNCYQHHA", "name": "\u9e4f\u534e\u5b89\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYUEYINIANCHIYOUQIHUNHEA"}, "011072": {"suoxie": "PHAYYNCYQHHC", "name": "\u9e4f\u534e\u5b89\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYUEYINIANCHIYOUQIHUNHEC"}, "011073": {"suoxie": "PHARHHA", "name": "\u9e4f\u534e\u5b89\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRUNHUNHEA"}, "011074": {"suoxie": "PHARHHC", "name": "\u9e4f\u534e\u5b89\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRUNHUNHEC"}, "011075": {"suoxie": "DCHXCXYNDKHHA", "name": "\u5927\u6210\u6052\u4eab\u6625\u6653\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHENGXIANGCHUNXIAOYINIANDINGKAIHUNHEA"}, "011076": {"suoxie": "DCHXCXYNDKHHC", "name": "\u5927\u6210\u6052\u4eab\u6625\u6653\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHENGXIANGCHUNXIAOYINIANDINGKAIHUNHEC"}, "011077": {"suoxie": "HFJXCXXF", "name": "\u6c47\u4e30\u664b\u4fe1\u521b\u65b0\u5148\u950b", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINCHUANGXINXIANFENG"}, "011078": {"suoxie": "NDPZXF6GYCYHH", "name": "\u8bfa\u5fb7\u54c1\u8d28\u6d88\u8d396\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEPINZHIXIAOFEI6GEYUECHIYOUHUNHE"}, "011079": {"suoxie": "JSZHYNDQCZZQ", "name": "\u5609\u5b9e\u81f4\u6cd3\u4e00\u5e74\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIHONGYINIANDINGQICHUNZHAIZHAIQUAN"}, "011080": {"suoxie": "PHZHYNDKFQSZQ", "name": "\u9e4f\u534e\u5c0a\u548c\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZUNHEYINIANDINGKAIFAQISHIZHAIQUAN"}, "011081": {"suoxie": "GTRYGGTHHC", "name": "\u56fd\u6295\u745e\u94f6\u6e2f\u80a1\u901a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINGANGGUTONGHUNHEC"}, "011082": {"suoxie": "GTRYYLBJHHC", "name": "\u56fd\u6295\u745e\u94f6\u533b\u7597\u4fdd\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINYILIAOBAOJIANHUNHEC"}, "011083": {"suoxie": "YHJL87GYDKZQ", "name": "\u94f6\u6cb3\u805a\u522987\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJULI87GEYUEDINGKAIZHAIQUAN"}, "011084": {"suoxie": "CTZGXTY6GYCYQHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u6dfb\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINTIANYI6GEYUECHIYOUQIHUNHEA"}, "011085": {"suoxie": "CTZGXTY6GYCYQHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u6dfb\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINTIANYI6GEYUECHIYOUQIHUNHEC"}, "011086": {"suoxie": "YFDRKHHA", "name": "\u6613\u65b9\u8fbe\u745e\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIKANGHUNHEA"}, "011087": {"suoxie": "YFDRKHHC", "name": "\u6613\u65b9\u8fbe\u745e\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDARUIKANGHUNHEC"}, "011088": {"suoxie": "JSCCJTHLYNDKZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u6052\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIHENGLIYINIANDINGKAIZHAI"}, "011089": {"suoxie": "JSCCJYHLYNCYQZQA", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u60e0\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYIHUILIYINIANCHIYOUQIZHAIQUANA"}, "011090": {"suoxie": "JSCCJYHLYNCYQZQC", "name": "\u666f\u987a\u957f\u57ce\u666f\u9890\u60e0\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGYIHUILIYINIANCHIYOUQIZHAIQUANC"}, "011091": {"suoxie": "GYSX6GYCYQZQA", "name": "\u5de5\u94f6\u53cc\u73ba6\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINSHUANGXI6GEYUECHIYOUQIZHAIQUANA"}, "011092": {"suoxie": "GYSX6GYCYQZQC", "name": "\u5de5\u94f6\u53cc\u73ba6\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINSHUANGXI6GEYUECHIYOUQIZHAIQUANC"}, "011093": {"suoxie": "YYHZYNDKHH", "name": "\u6c38\u8d62\u5b8f\u6cfd\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGHONGZEYINIANDINGKAIHUNHE"}, "011094": {"suoxie": "NDASCZ", "name": "\u8bfa\u5fb7\u5b89\u76db\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUODEANSHENGCHUNZHAI"}, "011095": {"suoxie": "BSHZHHA", "name": "\u535a\u65f6\u6052\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGZEHUNHEA"}, "011096": {"suoxie": "BSHZHHC", "name": "\u535a\u65f6\u6052\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGZEHUNHEC"}, "011097": {"suoxie": "DCYCJXHHA", "name": "\u8fbe\u8bda\u5b9c\u521b\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYICHUANGJINGXUANHUNHEA"}, "011098": {"suoxie": "DCYCJXHHC", "name": "\u8fbe\u8bda\u5b9c\u521b\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYICHUANGJINGXUANHUNHEC"}, "011099": {"suoxie": "FGJZCZHHA", "name": "\u5bcc\u56fd\u4ef7\u503c\u521b\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIAZHICHUANGZAOHUNHEA"}, "011100": {"suoxie": "FGJZCZHHC", "name": "\u5bcc\u56fd\u4ef7\u503c\u521b\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIAZHICHUANGZAOHUNHEC"}, "011101": {"suoxie": "YXYFRF66GYDKZ", "name": "\u5706\u4fe1\u6c38\u4e30\u745e\u4e3066\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YUANXINYONGFENGRUIFENG66GEYUEDINGKAIZHAI"}, "011102": {"suoxie": "THZZGFCYZSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGGUANGFUCHANYEZHISHUA"}, "011103": {"suoxie": "THZZGFCYZSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGGUANGFUCHANYEZHISHUC"}, "011104": {"suoxie": "GDZNQCZTGP", "name": "\u5149\u5927\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDAZHINENGQICHEZHUTIGUPIAO"}, "011105": {"suoxie": "CXWJJH6GYCYQHHA", "name": "\u957f\u4fe1\u7a33\u5065\u5747\u88616\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANJUNHENG6GEYUECHIYOUQIHUNHEA"}, "011106": {"suoxie": "CXWJJH6GYCYQHHC", "name": "\u957f\u4fe1\u7a33\u5065\u5747\u88616\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANJUNHENG6GEYUECHIYOUQIHUNHEC"}, "011107": {"suoxie": "JTTXLHZXA", "name": "\u4e5d\u6cf0\u5929\u5174\u91cf\u5316\u667a\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAITIANXINGLIANGHUAZHIXUANA"}, "011108": {"suoxie": "JTTXLHZXC", "name": "\u4e5d\u6cf0\u5929\u5174\u91cf\u5316\u667a\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAITIANXINGLIANGHUAZHIXUANC"}, "011109": {"suoxie": "NFHY6GYCYQZQA", "name": "\u5357\u65b9\u6656\u51436\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGHUIYUAN6GEYUECHIYOUQIZHAIQUANA"}, "011110": {"suoxie": "NFHY6GYCYQZQC", "name": "\u5357\u65b9\u6656\u51436\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGHUIYUAN6GEYUECHIYOUQIZHAIQUANC"}, "011111": {"suoxie": "HTBRHYYXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u884c\u4e1a\u4e25\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIHANGYEYANXUANHUNHEA"}, "011112": {"suoxie": "HTBRHYYXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u884c\u4e1a\u4e25\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIHANGYEYANXUANHUNHEC"}, "011113": {"suoxie": "FGJGZTHHC", "name": "\u5bcc\u56fd\u519b\u5de5\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNGONGZHUTIHUNHEC"}, "011114": {"suoxie": "FGHGSHYJXHHC", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u884c\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHUGANGSHENHANGYEJINGXUANHUNHEC"}, "011115": {"suoxie": "HFTLLZZQA", "name": "\u6d77\u5bcc\u901a\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGLILVZHAIZHAIQUANA"}, "011116": {"suoxie": "HFTLLZZQC", "name": "\u6d77\u5bcc\u901a\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGLILVZHAIZHAIQUANC"}, "011117": {"suoxie": "FGHGSYJQDHHXC", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u4e1a\u7ee9\u9a71\u52a8\u6df7\u5408\u578bC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHUGANGSHENYEJIQUDONGHUNHEXINGC"}, "011118": {"suoxie": "HTFWJRXYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u777f\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANRUIXUANYINIANCHIYOUHUNHEA"}, "011119": {"suoxie": "HTFWJRXYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u777f\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANRUIXUANYINIANCHIYOUHUNHEC"}, "011120": {"suoxie": "FGCXKJHHC", "name": "\u5bcc\u56fd\u521b\u65b0\u79d1\u6280\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHUANGXINKEJIHUNHEC"}, "011121": {"suoxie": "GFXCHHA", "name": "\u5e7f\u53d1\u5174\u8bda\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINGCHENGHUNHEA"}, "011122": {"suoxie": "HTFESGKCXCZA", "name": "\u6c47\u6dfb\u5bccESG\u53ef\u6301\u7eed\u6210\u957fA", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUESGKECHIXUCHENGZHANGA"}, "011123": {"suoxie": "HTFESGKCXCZC", "name": "\u6c47\u6dfb\u5bccESG\u53ef\u6301\u7eed\u6210\u957fC", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUESGKECHIXUCHENGZHANGC"}, "011124": {"suoxie": "FGJRDCHYHHC", "name": "\u5bcc\u56fd\u91d1\u878d\u5730\u4ea7\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJINRONGDICHANHANGYEHUNHEC"}, "011125": {"suoxie": "FGWTJKGPC", "name": "\u5bcc\u56fd\u6587\u4f53\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOWENTIJIANKANGGUPIAOC"}, "011126": {"suoxie": "FGHLKJGPC", "name": "\u5bcc\u56fd\u4e92\u8054\u79d1\u6280\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOHULIANKEJIGUPIAOC"}, "011127": {"suoxie": "FGQJNYCYHHC", "name": "\u5bcc\u56fd\u6e05\u6d01\u80fd\u6e90\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOQINGJIENENGYUANCHANYEHUNHEC"}, "011128": {"suoxie": "HAJZSHHHA", "name": "\u534e\u5b89\u7cbe\u81f4\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJINGZHISHENGHUOHUNHEA"}, "011129": {"suoxie": "HAJZSHHHC", "name": "\u534e\u5b89\u7cbe\u81f4\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJINGZHISHENGHUOHUNHEC"}, "011130": {"suoxie": "GFXCHHC", "name": "\u5e7f\u53d1\u5174\u8bda\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINGCHENGHUNHEC"}, "011131": {"suoxie": "FGHGSJZHHC", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHUGANGSHENJIAZHIHUNHEC"}, "011132": {"suoxie": "PYHS300ZLCZDBDA", "name": "\u9e4f\u626c\u6caa\u6df1300\u8d28\u91cf\u6210\u957f\u4f4e\u6ce2\u52a8A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGHUSHEN300ZHILIANGCHENGZHANGDIBODONGA"}, "011133": {"suoxie": "PYHS300ZLCZDBDC", "name": "\u9e4f\u626c\u6caa\u6df1300\u8d28\u91cf\u6210\u957f\u4f4e\u6ce2\u52a8C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGHUSHEN300ZHILIANGCHENGZHANGDIBODONGC"}, "011134": {"suoxie": "GFJZYXHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIYOUXUANHUNHEA"}, "011135": {"suoxie": "GFJZYXHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIYOUXUANHUNHEC"}, "011136": {"suoxie": "GFSXHHA", "name": "\u5e7f\u53d1\u76db\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGXINGHUNHEA"}, "011137": {"suoxie": "GFSXHHC", "name": "\u5e7f\u53d1\u76db\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGXINGHUNHEC"}, "011138": {"suoxie": "GFJHLGYCYQHHA", "name": "\u5e7f\u53d1\u805a\u9e3f\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUHONGLIUGEYUECHIYOUQIHUNHEA"}, "011139": {"suoxie": "GFJHLGYCYQHHC", "name": "\u5e7f\u53d1\u805a\u9e3f\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUHONGLIUGEYUECHIYOUQIHUNHEC"}, "011140": {"suoxie": "GFJHLGYCYQHHE", "name": "\u5e7f\u53d1\u805a\u9e3f\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUHONGLIUGEYUECHIYOUQIHUNHEE"}, "011141": {"suoxie": "NFJYZGDJXYZZQ", "name": "\u5357\u65b9\u666f\u5143\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJINGYUANZHONGGAODENGJIXINYONGZHAIZHAIQUAN"}, "011142": {"suoxie": "CJHXXCLXNYGPA", "name": "\u521b\u91d1\u5408\u4fe1\u65b0\u6750\u6599\u65b0\u80fd\u6e90\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINCAILIAOXINNENGYUANGUPIAOA"}, "011143": {"suoxie": "CJHXXCLXNYGPC", "name": "\u521b\u91d1\u5408\u4fe1\u65b0\u6750\u6599\u65b0\u80fd\u6e90\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINCAILIAOXINNENGYUANGUPIAOC"}, "011144": {"suoxie": "HAHHJXHHA", "name": "\u534e\u5b89\u6c47\u5b8f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUIHONGJINGXUANHUNHEA"}, "011145": {"suoxie": "HAHHJXHHC", "name": "\u534e\u5b89\u6c47\u5b8f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUIHONGJINGXUANHUNHEC"}, "011146": {"suoxie": "CJQHBHZRTZGPA", "name": "\u521b\u91d1\u6c14\u5019\u53d8\u5316\u8d23\u4efb\u6295\u8d44\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINQIHOUBIANHUAZERENTOUZIGUPIAOA"}, "011147": {"suoxie": "CJQHBHZRTZGPC", "name": "\u521b\u91d1\u6c14\u5019\u53d8\u5316\u8d23\u4efb\u6295\u8d44\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINQIHOUBIANHUAZERENTOUZIGUPIAOC"}, "011148": {"suoxie": "NFJGGGLHPZHHC", "name": "\u5357\u65b9\u519b\u5de5\u6539\u9769\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJUNGONGGAIGELINGHUOPEIZHIHUNHEC"}, "011149": {"suoxie": "CJHXESGZRTZGPA", "name": "\u521b\u91d1\u5408\u4fe1ESG\u8d23\u4efb\u6295\u8d44\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINESGZERENTOUZIGUPIAOA"}, "011150": {"suoxie": "CJHXESGZRTZGPC", "name": "\u521b\u91d1\u5408\u4fe1ESG\u8d23\u4efb\u6295\u8d44\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINESGZERENTOUZIGUPIAOC"}, "011151": {"suoxie": "FGYLBJHYHHC", "name": "\u5bcc\u56fd\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYILIAOBAOJIANHANGYEHUNHEC"}, "011152": {"suoxie": "GFXHHBHH", "name": "\u56fd\u5bcc\u5174\u6d77\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUXINGHAIHUIBAOHUNHE"}, "011153": {"suoxie": "HBXXXFHHA", "name": "\u534e\u5b9d\u65b0\u5174\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXINXINGXIAOFEIHUNHEA"}, "011154": {"suoxie": "HBXXXFHHC", "name": "\u534e\u5b9d\u65b0\u5174\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXINXINGXIAOFEIHUNHEC"}, "011155": {"suoxie": "JYZRTZHHA", "name": "\u91d1\u9e70\u8d23\u4efb\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGZERENTOUZIHUNHEA"}, "011156": {"suoxie": "JYZRTZHHC", "name": "\u91d1\u9e70\u8d23\u4efb\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGZERENTOUZIHUNHEC"}, "011157": {"suoxie": "HYYFGGTZXLHHHA", "name": "\u5f18\u6bc5\u8fdc\u65b9\u6e2f\u80a1\u901a\u667a\u9009\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGANGGUTONGZHIXUANLINGHANGHUNHEA"}, "011158": {"suoxie": "HYYFGGTZXLHHHC", "name": "\u5f18\u6bc5\u8fdc\u65b9\u6e2f\u80a1\u901a\u667a\u9009\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGANGGUTONGZHIXUANLINGHANGHUNHEC"}, "011159": {"suoxie": "DCZXPHHLOFC", "name": "\u5927\u6210\u4e2d\u5c0f\u76d8\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHONGXIAOPANHUNHELOFC"}, "011160": {"suoxie": "FGZLCZ6GYCYHHA", "name": "\u5bcc\u56fd\u8d28\u91cf\u6210\u957f6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHILIANGCHENGZHANG6GEYUECHIYOUHUNHEA"}, "011161": {"suoxie": "FGZLCZ6GYCYHHC", "name": "\u5bcc\u56fd\u8d28\u91cf\u6210\u957f6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHILIANGCHENGZHANG6GEYUECHIYOUHUNHEC"}, "011162": {"suoxie": "BSGGTLXQSHHA", "name": "\u535a\u65f6\u6e2f\u80a1\u901a\u9886\u5148\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGANGGUTONGLINGXIANQUSHIHUNHEA"}, "011163": {"suoxie": "BSGGTLXQSHHC", "name": "\u535a\u65f6\u6e2f\u80a1\u901a\u9886\u5148\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGANGGUTONGLINGXIANQUSHIHUNHEC"}, "011164": {"suoxie": "FGXYYX12GYCYHHA", "name": "\u5bcc\u56fd\u5174\u8fdc\u4f18\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINGYUANYOUXUAN12GEYUECHIYOUHUNHEA"}, "011165": {"suoxie": "FGXYYX12GYCYHHC", "name": "\u5bcc\u56fd\u5174\u8fdc\u4f18\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINGYUANYOUXUAN12GEYUECHIYOUHUNHEC"}, "011166": {"suoxie": "WJLJZJRCJRZ", "name": "\u4e07\u5bb6\u9646\u5bb6\u5634\u91d1\u878d\u57ce\u91d1\u878d\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIALUJIAZUIJINRONGCHENGJINRONGZHAI"}, "011167": {"suoxie": "JSCCJQCZ", "name": "\u666f\u987a\u957f\u57ce\u666f\u6c14\u6210\u957f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGQICHENGZHANG"}, "011168": {"suoxie": "JSRXAJSL18GYCYQZQ", "name": "\u5609\u5b9e\u777f\u4eab\u5b89\u4e45\u53cc\u522918\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIRUIXIANGANJIUSHUANGLI18GEYUECHIYOUQIZHAIQUAN"}, "011169": {"suoxie": "JXZXHH", "name": "\u5efa\u4fe1\u81fb\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINZHENXUANHUNHE"}, "011170": {"suoxie": "BYZHSHHHA", "name": "\u5b9d\u76c8\u667a\u6167\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGZHIHUISHENGHUOHUNHEA"}, "011171": {"suoxie": "BYZHSHHHC", "name": "\u5b9d\u76c8\u667a\u6167\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGZHIHUISHENGHUOHUNHEC"}, "011172": {"suoxie": "GFLXLHPZHHC", "name": "\u5e7f\u53d1\u5229\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFALIXINLINGHUOPEIZHIHUNHEC"}, "011173": {"suoxie": "YHXXYNCYQHH", "name": "\u94f6\u534e\u5fc3\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINXIANGYINIANCHIYOUQIHUNHE"}, "011174": {"suoxie": "ZGJZPZYNCYQHH", "name": "\u4e2d\u5e9a\u4ef7\u503c\u54c1\u8d28\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGGENGJIAZHIPINZHIYINIANCHIYOUQIHUNHE"}, "011175": {"suoxie": "PAHXHHA", "name": "\u5e73\u5b89\u6052\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGXINHUNHEA"}, "011176": {"suoxie": "PAHXHHC", "name": "\u5e73\u5b89\u6052\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGXINHUNHEC"}, "011177": {"suoxie": "BSHRHBYNCYHHA", "name": "\u535a\u65f6\u6c47\u878d\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUIRONGHUIBAOYINIANCHIYOUHUNHEA"}, "011178": {"suoxie": "BSHRHBYNCYHHC", "name": "\u535a\u65f6\u6c47\u878d\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUIRONGHUIBAOYINIANCHIYOUHUNHEC"}, "011179": {"suoxie": "ZSZXSPYLGPA", "name": "\u6d59\u5546\u667a\u9009\u98df\u54c1\u996e\u6599\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGZHIXUANSHIPINYINLIAOGUPIAOA"}, "011180": {"suoxie": "ZSZXSPYLGPC", "name": "\u6d59\u5546\u667a\u9009\u98df\u54c1\u996e\u6599\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGZHIXUANSHIPINYINLIAOGUPIAOC"}, "011181": {"suoxie": "CSCZLTHHA", "name": "\u957f\u76db\u6210\u957f\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGCHENGZHANGLONGTOUHUNHEA"}, "011182": {"suoxie": "CSCZLTHHC", "name": "\u957f\u76db\u6210\u957f\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGCHENGZHANGLONGTOUHUNHEC"}, "011183": {"suoxie": "GFNXZZHHC", "name": "\u5e7f\u53d1\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFANEIXUZENGZHANGHUNHEC"}, "011184": {"suoxie": "DFAEFZYHHA", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u62db\u9633\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAZHAOYANGHUNHEA"}, "011185": {"suoxie": "DFAEFZYHHC", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u62db\u9633\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFAZHAOYANGHUNHEC"}, "011186": {"suoxie": "XDAYZCJXHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u81f3\u8bda\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHICHENGJINGXUANHUNHEA"}, "011187": {"suoxie": "ZJSYCZZQ", "name": "\u4e2d\u52a0\u7a57\u76c8\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIASUIYINGCHUNZHAIZHAIQUAN"}, "011188": {"suoxie": "XDAYXYHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u661f\u5955\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINXINGYIHUNHEA"}, "011189": {"suoxie": "JXZHYXYNCYQHHMOM", "name": "\u5efa\u4fe1\u667a\u6c47\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(MOM)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINZHIHUIYOUXUANYINIANCHIYOUQIHUNHEMOM"}, "011190": {"suoxie": "ZSRA1NCYQHHA", "name": "\u62db\u5546\u745e\u5b891\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIAN1NIANCHIYOUQIHUNHEA"}, "011191": {"suoxie": "ZSRA1NCYQHHC", "name": "\u62db\u5546\u745e\u5b891\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIAN1NIANCHIYOUQIHUNHEC"}, "011192": {"suoxie": "GFHRSGYCYQHHA", "name": "\u5e7f\u53d1\u6052\u8363\u4e09\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGRONGSANGEYUECHIYOUQIHUNHEA"}, "011193": {"suoxie": "GFHRSGYCYQHHC", "name": "\u5e7f\u53d1\u6052\u8363\u4e09\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGRONGSANGEYUECHIYOUQIHUNHEC"}, "011194": {"suoxie": "GFRMLNCYQHHA", "name": "\u5e7f\u53d1\u777f\u94ed\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIMINGLIANGNIANCHIYOUQIHUNHEA"}, "011195": {"suoxie": "GFRMLNCYQHHC", "name": "\u5e7f\u53d1\u777f\u94ed\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIMINGLIANGNIANCHIYOUQIHUNHEC"}, "011196": {"suoxie": "STMGYSCZHHA", "name": "\u4e0a\u6295\u6469\u6839\u4f18\u52bf\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENYOUSHICHENGZHANGHUNHEA"}, "011197": {"suoxie": "STMGYSCZHHC", "name": "\u4e0a\u6295\u6469\u6839\u4f18\u52bf\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENYOUSHICHENGZHANGHUNHEC"}, "011198": {"suoxie": "JYSLDXXHBHHA", "name": "\u4ea4\u94f6\u65bd\u7f57\u5fb7\u946b\u9009\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINSHILUODEXINXUANHUIBAOHUNHEA"}, "011199": {"suoxie": "JYSLDXXHBHHC", "name": "\u4ea4\u94f6\u65bd\u7f57\u5fb7\u946b\u9009\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINSHILUODEXINXUANHUIBAOHUNHEC"}, "011201": {"suoxie": "CTYSHYLDHHA", "name": "\u8d22\u901a\u4f18\u52bf\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGYOUSHIHANGYELUNDONGHUNHEA"}, "011202": {"suoxie": "CTYSHYLDHHC", "name": "\u8d22\u901a\u4f18\u52bf\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGYOUSHIHANGYELUNDONGHUNHEC"}, "011203": {"suoxie": "YYHTYHHA", "name": "\u6c38\u8d62\u60e0\u6dfb\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGHUITIANYIHUNHEA"}, "011204": {"suoxie": "YYHTYHHC", "name": "\u6c38\u8d62\u60e0\u6dfb\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGHUITIANYIHUNHEC"}, "011205": {"suoxie": "XYZZ500ZSZQC", "name": "\u5174\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENG500ZHISHUZENGQIANGC"}, "011206": {"suoxie": "CJHXJZYSHHA", "name": "\u521b\u91d1\u5408\u4fe1\u7ade\u4e89\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINJINGZHENGYOUSHIHUNHEA"}, "011207": {"suoxie": "CJHXJZYSHHC", "name": "\u521b\u91d1\u5408\u4fe1\u7ade\u4e89\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINJINGZHENGYOUSHIHUNHEC"}, "011212": {"suoxie": "FGWJCL6GYCYHHA", "name": "\u5bcc\u56fd\u7a33\u5065\u7b56\u75656\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANCELUE6GEYUECHIYOUHUNHEA"}, "011213": {"suoxie": "FGWJCL6GYCYHHC", "name": "\u5bcc\u56fd\u7a33\u5065\u7b56\u75656\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANCELUE6GEYUECHIYOUHUNHEC"}, "011214": {"suoxie": "ZSHRYNDKHHFQSMOMA", "name": "\u62db\u5546\u60e0\u6da6\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f(MOM)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGHUIRUNYINIANDINGKAIHUNHEFAQISHIMOMA"}, "011215": {"suoxie": "ZSHRYNDKHHFQSMOMC", "name": "\u62db\u5546\u60e0\u6da6\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f(MOM)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGHUIRUNYINIANDINGKAIHUNHEFAQISHIMOMC"}, "011216": {"suoxie": "NFYZQYHHA", "name": "\u5357\u65b9\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYOUZHIQIYEHUNHEA"}, "011217": {"suoxie": "NFYZQYHHC", "name": "\u5357\u65b9\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYOUZHIQIYEHUNHEC"}, "011220": {"suoxie": "NFJXYXGPA", "name": "\u5357\u65b9\u5320\u5fc3\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGJIANGXINYOUXUANGUPIAOA"}, "011221": {"suoxie": "NFJXYXGPC", "name": "\u5357\u65b9\u5320\u5fc3\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGJIANGXINYOUXUANGUPIAOC"}, "011222": {"suoxie": "JXXJTYHBC", "name": "\u5efa\u4fe1\u73b0\u91d1\u6dfb\u76ca\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINTIANYIHUOBIC"}, "011223": {"suoxie": "XDAYXYHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u661f\u5955\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINXINGYIHUNHEC"}, "011224": {"suoxie": "JTYTLHGPA", "name": "\u4e5d\u6cf0\u76c8\u6cf0\u91cf\u5316\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIYINGTAILIANGHUAGUPIAOA"}, "011225": {"suoxie": "JTYTLHGPC", "name": "\u4e5d\u6cf0\u76c8\u6cf0\u91cf\u5316\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAIYINGTAILIANGHUAGUPIAOC"}, "011228": {"suoxie": "XBLDLHCZHHC", "name": "\u897f\u90e8\u5229\u5f97\u91cf\u5316\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDELIANGHUACHENGZHANGHUNHEC"}, "011229": {"suoxie": "CJHXSZJJZTGPA", "name": "\u521b\u91d1\u5408\u4fe1\u6570\u5b57\u7ecf\u6d4e\u4e3b\u9898\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINSHUZIJINGJIZHUTIGUPIAOA"}, "011230": {"suoxie": "CJHXSZJJZTGPC", "name": "\u521b\u91d1\u5408\u4fe1\u6570\u5b57\u7ecf\u6d4e\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINSHUZIJINGJIZHUTIGUPIAOC"}, "011231": {"suoxie": "GDJHHHA", "name": "\u5149\u5927\u9526\u5f18\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAJINHONGHUNHEA"}, "011232": {"suoxie": "GDJHHHC", "name": "\u5149\u5927\u9526\u5f18\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAJINHONGHUNHEC"}, "011233": {"suoxie": "TKFTPHYLSNCYHHFOF", "name": "\u6cf0\u5eb7\u798f\u6cf0\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIKANGFUTAIPINGHENGYANGLAOSANNIANCHIYOUHUNHEFOF"}, "011234": {"suoxie": "TDHLZZ15NGKZA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIZHONGZHAI15NIANGUOKAIZHAIA"}, "011235": {"suoxie": "TDHLZZ15NGKZC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIZHONGZHAI15NIANGUOKAIZHAIC"}, "011236": {"suoxie": "STMGHYRXGPA", "name": "\u4e0a\u6295\u6469\u6839\u884c\u4e1a\u777f\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHANGYERUIXUANGUPIAOA"}, "011237": {"suoxie": "STMGHYRXGPC", "name": "\u4e0a\u6295\u6469\u6839\u884c\u4e1a\u777f\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHANGYERUIXUANGUPIAOC"}, "011238": {"suoxie": "HAJHJXHHA", "name": "\u534e\u5b89\u805a\u6052\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUHENGJINGXUANHUNHEA"}, "011239": {"suoxie": "HAJHJXHHC", "name": "\u534e\u5b89\u805a\u6052\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUHENGJINGXUANHUNHEC"}, "011240": {"suoxie": "DWHYLDHHC", "name": "\u4e1c\u5434\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUHANGYELUNDONGHUNHEC"}, "011241": {"suoxie": "DWSDLHHC", "name": "\u4e1c\u5434\u53cc\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUSHUANGDONGLIHUNHEC"}, "011242": {"suoxie": "DWJQCLHHC", "name": "\u4e1c\u5434\u8fdb\u53d6\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUJINQUCELUEHUNHEC"}, "011243": {"suoxie": "WJHYHB6GYCYQHHA", "name": "\u4e07\u5bb6\u60e0\u88d5\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAHUIYUHUIBAO6GEYUECHIYOUQIHUNHEA"}, "011244": {"suoxie": "WJHYHB6GYCYQHHC", "name": "\u4e07\u5bb6\u60e0\u88d5\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAHUIYUHUIBAO6GEYUECHIYOUQIHUNHEC"}, "011245": {"suoxie": "ZJRXCZZQC", "name": "\u4e2d\u52a0\u745e\u4eab\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIARUIXIANGCHUNZHAIZHAIQUANC"}, "011246": {"suoxie": "JSAEFYXHHA", "name": "\u5609\u5b9e\u963f\u5c14\u6cd5\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIAERFAYOUXUANHUNHEA"}, "011247": {"suoxie": "JSAEFYXHHC", "name": "\u5609\u5b9e\u963f\u5c14\u6cd5\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIAERFAYOUXUANHUNHEC"}, "011248": {"suoxie": "JSPZHBHH", "name": "\u5609\u5b9e\u54c1\u8d28\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIPINZHIHUIBAOHUNHE"}, "011249": {"suoxie": "JSWYHHA", "name": "\u5609\u5b9e\u7a33\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENYUHUNHEA"}, "011250": {"suoxie": "JSWYHHC", "name": "\u5609\u5b9e\u7a33\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENYUHUNHEC"}, "011251": {"suoxie": "HAJJJXHHA", "name": "\u534e\u5b89\u805a\u5609\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUJIAJINGXUANHUNHEA"}, "011252": {"suoxie": "HAJJJXHHC", "name": "\u534e\u5b89\u805a\u5609\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUJIAJINGXUANHUNHEC"}, "011253": {"suoxie": "HB15NZJZZS", "name": "\u534e\u5b9d1-5\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAO15NIANZHENGJINZHAIZHISHU"}, "011254": {"suoxie": "CJLHKJJXYGYGDCYGPA", "name": "\u957f\u6c5f\u91cf\u5316\u79d1\u6280\u7cbe\u9009\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAKEJIJINGXUANYIGEYUEGUNDONGCHIYOUGUPIAOA"}, "011255": {"suoxie": "CJLHKJJXYGYGDCYGPC", "name": "\u957f\u6c5f\u91cf\u5316\u79d1\u6280\u7cbe\u9009\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGJIANGLIANGHUAKEJIJINGXUANYIGEYUEGUNDONGCHIYOUGUPIAOC"}, "011256": {"suoxie": "JYHGYNHHA", "name": "\u4ea4\u94f6\u9e3f\u5149\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGGUANGYINIANHUNHEA"}, "011257": {"suoxie": "JYHGYNHHC", "name": "\u4ea4\u94f6\u9e3f\u5149\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGGUANGYINIANHUNHEC"}, "011258": {"suoxie": "ZSBZJKXD", "name": "\u62db\u5546\u4fdd\u8bc1\u91d1\u5feb\u7ebfD", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGBAOZHENGJINKUAIXIAND"}, "011259": {"suoxie": "HTFGZLCZ30YNHHC", "name": "\u6c47\u6dfb\u5bcc\u9ad8\u8d28\u91cf\u6210\u957f30\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUGAOZHILIANGCHENGZHANG30YINIANHUNHEC"}, "011260": {"suoxie": "JYXNYHHA", "name": "\u91d1\u9e70\u65b0\u80fd\u6e90\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGXINNENGYUANHUNHEA"}, "011261": {"suoxie": "JYXNYHHC", "name": "\u91d1\u9e70\u65b0\u80fd\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGXINNENGYUANHUNHEC"}, "011262": {"suoxie": "HXDYZQA", "name": "\u534e\u590f\u9f0e\u82f1\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYINGZHAIQUANA"}, "011263": {"suoxie": "HXDYZQC", "name": "\u534e\u590f\u9f0e\u82f1\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYINGZHAIQUANC"}, "011264": {"suoxie": "ZOXQSHHX", "name": "\u4e2d\u6b27\u65b0\u8d8b\u52bf\u6df7\u5408X", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINQUSHIHUNHEX"}, "011265": {"suoxie": "CSATYNCYQHHA", "name": "\u957f\u76db\u5b89\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGANTAIYINIANCHIYOUQIHUNHEA"}, "011266": {"suoxie": "CSATYNCYQHHC", "name": "\u957f\u76db\u5b89\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGANTAIYINIANCHIYOUQIHUNHEC"}, "011267": {"suoxie": "CSXSWJYNCYA", "name": "\u957f\u76db\u946b\u76db\u7a33\u5065\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGXINSHENGWENJIANYINIANCHIYOUA"}, "011268": {"suoxie": "CSXSWJYNCYC", "name": "\u957f\u76db\u946b\u76db\u7a33\u5065\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGXINSHENGWENJIANYINIANCHIYOUC"}, "011269": {"suoxie": "ZYZQYSZZGPA", "name": "\u4e2d\u94f6\u8bc1\u5238\u4f18\u52bf\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHENGQUANYOUSHIZHIZAOGUPIAOA"}, "011270": {"suoxie": "ZYZQYSZZGPC", "name": "\u4e2d\u94f6\u8bc1\u5238\u4f18\u52bf\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHENGQUANYOUSHIZHIZAOGUPIAOC"}, "011271": {"suoxie": "HTFJZCZJHTZHHA", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u6210\u957f\u5747\u8861\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHICHENGZHANGJUNHENGTOUZIHUNHEA"}, "011272": {"suoxie": "HTFJZCZJHTZHHC", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u6210\u957f\u5747\u8861\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHICHENGZHANGJUNHENGTOUZIHUNHEC"}, "011273": {"suoxie": "TXJQQD12GYCYHHA", "name": "\u6cf0\u4fe1\u666f\u6c14\u9a71\u52a812\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINJINGQIQUDONG12GEYUECHIYOUHUNHEA"}, "011274": {"suoxie": "TXJQQD12GYCYHHC", "name": "\u6cf0\u4fe1\u666f\u6c14\u9a71\u52a812\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINJINGQIQUDONG12GEYUECHIYOUHUNHEC"}, "011275": {"suoxie": "JYCZDLYNCYHHA", "name": "\u4ea4\u94f6\u6210\u957f\u52a8\u529b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANGDONGLIYINIANCHIYOUHUNHEA"}, "011276": {"suoxie": "JYCZDLYNCYHHC", "name": "\u4ea4\u94f6\u6210\u957f\u52a8\u529b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANGDONGLIYINIANCHIYOUHUNHEC"}, "011277": {"suoxie": "SYKJQDSZDQKSHHHA", "name": "\u4e0a\u94f6\u79d1\u6280\u9a71\u52a8\u53cc\u5468\u5b9a\u671f\u53ef\u8d4e\u56de\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINKEJIQUDONGSHUANGZHOUDINGQIKESHUHUIHUNHEA"}, "011278": {"suoxie": "HXNXQDHHA", "name": "\u534e\u590f\u5185\u9700\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIANEIXUQUDONGHUNHEA"}, "011279": {"suoxie": "HXNXQDHHC", "name": "\u534e\u590f\u5185\u9700\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIANEIXUQUDONGHUNHEC"}, "011280": {"suoxie": "HBSZZQZQA", "name": "\u534e\u5b9d\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOSHUANGZHAIZENGQIANGZHAIQUANA"}, "011281": {"suoxie": "HBSZZQZQC", "name": "\u534e\u5b9d\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOSHUANGZHAIZENGQIANGZHAIQUANC"}, "011282": {"suoxie": "HXXFLTHHA", "name": "\u534e\u590f\u6d88\u8d39\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIAOFEILONGTOUHUNHEA"}, "011283": {"suoxie": "HXXFLTHHC", "name": "\u534e\u590f\u6d88\u8d39\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIAOFEILONGTOUHUNHEC"}, "011284": {"suoxie": "ZXBCLTJXHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u9f99\u817e\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGLONGTENGJINGXUANHUNHE"}, "011285": {"suoxie": "MSJZYX6GYCYGPA", "name": "\u6c11\u751f\u4ef7\u503c\u4f18\u90096\u4e2a\u6708\u6301\u6709\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAZHIYOUXUAN6GEYUECHIYOUGUPIAOA"}, "011286": {"suoxie": "MSJZYX6GYCYGPC", "name": "\u6c11\u751f\u4ef7\u503c\u4f18\u90096\u4e2a\u6708\u6301\u6709\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAZHIYOUXUAN6GEYUECHIYOUGUPIAOC"}, "011287": {"suoxie": "QHKYJHSNCYHH", "name": "\u524d\u6d77\u5f00\u6e90\u805a\u6167\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANJUHUISANNIANCHIYOUHUNHE"}, "011288": {"suoxie": "SYYLJKHHA", "name": "\u4e0a\u94f6\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINYILIAOJIANKANGHUNHEA"}, "011289": {"suoxie": "SYYLJKHHC", "name": "\u4e0a\u94f6\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINYILIAOJIANKANGHUNHEC"}, "011290": {"suoxie": "QHLHTRYNCYHHA", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAILIANHETIANRUIYINIANCHIYOUHUNHEA"}, "011291": {"suoxie": "QHLHTRYNCYHHC", "name": "\u524d\u6d77\u8054\u5408\u6dfb\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "QIANHAILIANHETIANRUIYINIANCHIYOUHUNHEC"}, "011292": {"suoxie": "ZSTYCZD", "name": "\u62db\u5546\u6dfb\u88d5\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUCHUNZHAID"}, "011293": {"suoxie": "ZJHYYNCYQHH", "name": "\u4e2d\u91d1\u6052\u8fdc\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJINHENGYUANYINIANCHIYOUQIHUNHE"}, "011294": {"suoxie": "ZSTY1NDKZQFQS", "name": "\u62db\u5546\u6dfb\u90381\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYI1NIANDINGKAIZHAIQUANFAQISHI"}, "011295": {"suoxie": "XCLHAEFGPC", "name": "\u4fe1\u8bda\u91cf\u5316\u963f\u5c14\u6cd5\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "XINCHENGLIANGHUAAERFAGUPIAOC"}, "011296": {"suoxie": "HTFYSHYYNDKHHA", "name": "\u6c47\u6dfb\u5bcc\u4f18\u52bf\u884c\u4e1a\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUSHIHANGYEYINIANDINGKAIHUNHEA"}, "011297": {"suoxie": "HTFYSHYYNDKHHC", "name": "\u6c47\u6dfb\u5bcc\u4f18\u52bf\u884c\u4e1a\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUSHIHANGYEYINIANDINGKAIHUNHEC"}, "011298": {"suoxie": "YFDYAYNCYZQA", "name": "\u6613\u65b9\u8fbe\u60a6\u5b89\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUEANYINIANCHIYOUZHAIQUANA"}, "011299": {"suoxie": "YFDYAYNCYZQC", "name": "\u6613\u65b9\u8fbe\u60a6\u5b89\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAYUEANYINIANCHIYOUZHAIQUANC"}, "011300": {"suoxie": "YFDZZYSHHA", "name": "\u6613\u65b9\u8fbe\u667a\u9020\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAZHIZAOYOUSHIHUNHEA"}, "011301": {"suoxie": "YFDZZYSHHC", "name": "\u6613\u65b9\u8fbe\u667a\u9020\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAZHIZAOYOUSHIHUNHEC"}, "011302": {"suoxie": "YFDYYYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEYINGYINIANCHIYOUHUNHEA"}, "011303": {"suoxie": "YFDYYYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEYINGYINIANCHIYOUHUNHEC"}, "011304": {"suoxie": "GYCXCZHHA", "name": "\u5de5\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGXINCHENGZHANGHUNHEA"}, "011305": {"suoxie": "GYCXCZHHC", "name": "\u5de5\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGXINCHENGZHANGHUNHEC"}, "011306": {"suoxie": "FGDTXJJHHC", "name": "\u5bcc\u56fd\u4f4e\u78b3\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODITANXINJINGJIHUNHEC"}, "011307": {"suoxie": "FGTYJZHHC", "name": "\u5bcc\u56fd\u5929\u76ca\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANYIJIAZHIHUNHEC"}, "011308": {"suoxie": "FGSWYYKJHHC", "name": "\u5bcc\u56fd\u751f\u7269\u533b\u836f\u79d1\u6280\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOSHENGWUYIYAOKEJIHUNHEC"}, "011309": {"suoxie": "FGXFZTHHC", "name": "\u5bcc\u56fd\u6d88\u8d39\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXIAOFEIZHUTIHUNHEC"}, "011310": {"suoxie": "ZRHYCZA", "name": "\u4e2d\u878d\u6052\u9633\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYANGCHUNZHAIA"}, "011311": {"suoxie": "ZRHYCZC", "name": "\u4e2d\u878d\u6052\u9633\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYANGCHUNZHAIC"}, "011312": {"suoxie": "DFHQRSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u745e\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIRUISANNIANCHIYOUHUNHEB"}, "011313": {"suoxie": "DFHQHSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u534e\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHUASANNIANCHIYOUHUNHEB"}, "011314": {"suoxie": "NYCXCZHH", "name": "\u519c\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINCHUANGXINCHENGZHANGHUNHE"}, "011315": {"suoxie": "YYGGTYZCZYNHH", "name": "\u6c38\u8d62\u6e2f\u80a1\u901a\u4f18\u8d28\u6210\u957f\u4e00\u5e74\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGGANGGUTONGYOUZHICHENGZHANGYINIANHUNHE"}, "011316": {"suoxie": "THCYB300ETFFQSLJA", "name": "\u5929\u5f18\u521b\u4e1a\u677f300ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBAN300ETFFAQISHILIANJIEA"}, "011317": {"suoxie": "THCYB300ETFFQSLJC", "name": "\u5929\u5f18\u521b\u4e1a\u677f300ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBAN300ETFFAQISHILIANJIEC"}, "011319": {"suoxie": "GTSZZHETFLJA", "name": "\u56fd\u6cf0\u4e0a\u8bc1\u7efc\u5408ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENGZONGHEETFLIANJIEA"}, "011320": {"suoxie": "GTSZZHETFLJC", "name": "\u56fd\u6cf0\u4e0a\u8bc1\u7efc\u5408ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENGZONGHEETFLIANJIEC"}, "011321": {"suoxie": "GTDJKGPC", "name": "\u56fd\u6cf0\u5927\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIDAJIANKANGGUPIAOC"}, "011322": {"suoxie": "GTZNZBGPC", "name": "\u56fd\u6cf0\u667a\u80fd\u88c5\u5907\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIZHINENGZHUANGBEIGUPIAOC"}, "011323": {"suoxie": "GTZNQCGPC", "name": "\u56fd\u6cf0\u667a\u80fd\u6c7d\u8f66\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIZHINENGQICHEGUPIAOC"}, "011324": {"suoxie": "GTJZJXLHPZHHC", "name": "\u56fd\u6cf0\u4ef7\u503c\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIAZHIJINGXUANLINGHUOPEIZHIHUNHEC"}, "011325": {"suoxie": "GTJYYSJXHHC", "name": "\u56fd\u6cf0\u6c5f\u6e90\u4f18\u52bf\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIANGYUANYOUSHIJINGXUANHUNHEC"}, "011326": {"suoxie": "GTYYJKGPC", "name": "\u56fd\u6cf0\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIYIYAOJIANKANGGUPIAOC"}, "011327": {"suoxie": "TPFYYNDKZQFQS", "name": "\u592a\u5e73\u4e30\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIPINGFENGYINGYINIANDINGKAIZHAIQUANFAQISHI"}, "011328": {"suoxie": "JSCCXNYCYGPA", "name": "\u666f\u987a\u957f\u57ce\u65b0\u80fd\u6e90\u4ea7\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGXINNENGYUANCHANYEGUPIAOA"}, "011329": {"suoxie": "JSCCXNYCYGPC", "name": "\u666f\u987a\u957f\u57ce\u65b0\u80fd\u6e90\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGXINNENGYUANCHANYEGUPIAOC"}, "011330": {"suoxie": "PHJXQYYNCYHHMOM", "name": "\u9e4f\u534e\u7cbe\u9009\u7fa4\u82f1\u4e00\u5e74\u6301\u6709\u6df7\u5408MOM", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAJINGXUANQUNYINGYINIANCHIYOUHUNHEMOM"}, "011331": {"suoxie": "PHYJCZHHA", "name": "\u9e4f\u534e\u8fdc\u89c1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYUANJIANCHENGZHANGHUNHEA"}, "011332": {"suoxie": "PHYJCZHHC", "name": "\u9e4f\u534e\u8fdc\u89c1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYUANJIANCHENGZHANGHUNHEC"}, "011333": {"suoxie": "PHPZYXHHA", "name": "\u9e4f\u534e\u54c1\u8d28\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHIYOUXUANHUNHEA"}, "011334": {"suoxie": "PHPZYXHHC", "name": "\u9e4f\u534e\u54c1\u8d28\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHIYOUXUANHUNHEC"}, "011335": {"suoxie": "YHYYHH", "name": "\u94f6\u6cb3\u533b\u836f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEYIYAOHUNHE"}, "011336": {"suoxie": "XQHJYNCYHHA", "name": "\u5174\u5168\u6c47\u5409\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGQUANHUIJIYINIANCHIYOUHUNHEA"}, "011337": {"suoxie": "XQHJYNCYHHC", "name": "\u5174\u5168\u6c47\u5409\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGQUANHUIJIYINIANCHIYOUHUNHEC"}, "011338": {"suoxie": "XQHYLNCYHHA", "name": "\u5174\u5168\u5408\u8fdc\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHEYUANLIANGNIANCHIYOUHUNHEA"}, "011339": {"suoxie": "XQHYLNCYHHC", "name": "\u5174\u5168\u5408\u8fdc\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHEYUANLIANGNIANCHIYOUHUNHEC"}, "011340": {"suoxie": "BSZLXCLZTHHA", "name": "\u535a\u65f6\u6218\u7565\u65b0\u6750\u6599\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHANLUEXINCAILIAOZHUTIHUNHEA"}, "011341": {"suoxie": "BSZLXCLZTHHC", "name": "\u535a\u65f6\u6218\u7565\u65b0\u6750\u6599\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHANLUEXINCAILIAOZHUTIHUNHEC"}, "011344": {"suoxie": "JSCCRJYNCYHHA", "name": "\u666f\u987a\u957f\u57ce\u878d\u666f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGRONGJINGYINIANCHIYOUHUNHEA"}, "011345": {"suoxie": "JSCCRJYNCYHHC", "name": "\u666f\u987a\u957f\u57ce\u878d\u666f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGRONGJINGYINIANCHIYOUHUNHEC"}, "011346": {"suoxie": "CHXCYNCYHH", "name": "\u6df3\u539a\u946b\u6df3\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINCHUNYINIANCHIYOUHUNHE"}, "011347": {"suoxie": "YFDNYYNCYHHA", "name": "\u6613\u65b9\u8fbe\u5b81\u6613\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDANINGYIYINIANCHIYOUHUNHEA"}, "011348": {"suoxie": "YFDNYYNCYHHC", "name": "\u6613\u65b9\u8fbe\u5b81\u6613\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDANINGYIYINIANCHIYOUHUNHEC"}, "011349": {"suoxie": "CHXDFWYGPA", "name": "\u6df3\u539a\u73b0\u4ee3\u670d\u52a1\u4e1a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUNHOUXIANDAIFUWUYEGUPIAOA"}, "011350": {"suoxie": "CHXDFWYGPC", "name": "\u6df3\u539a\u73b0\u4ee3\u670d\u52a1\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUNHOUXIANDAIFUWUYEGUPIAOC"}, "011351": {"suoxie": "JYNNYYYNCYHHA", "name": "\u91d1\u9e70\u5e74\u5e74\u90ae\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGNIANNIANYOUYIYINIANCHIYOUHUNHEA"}, "011352": {"suoxie": "JYNNYYYNCYHHC", "name": "\u91d1\u9e70\u5e74\u5e74\u90ae\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGNIANNIANYOUYIYINIANCHIYOUHUNHEC"}, "011353": {"suoxie": "ZRJSYNCYHHA", "name": "\u4e2d\u878d\u666f\u76db\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGSHENGYINIANCHIYOUHUNHEA"}, "011354": {"suoxie": "ZRJSYNCYHHC", "name": "\u4e2d\u878d\u666f\u76db\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGSHENGYINIANCHIYOUHUNHEC"}, "011355": {"suoxie": "HTBRGGTSDJYHHA", "name": "\u534e\u6cf0\u67cf\u745e\u6e2f\u80a1\u901a\u65f6\u4ee3\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIGANGGUTONGSHIDAIJIYUHUNHEA"}, "011356": {"suoxie": "HTBRGGTSDJYHHC", "name": "\u534e\u6cf0\u67cf\u745e\u6e2f\u80a1\u901a\u65f6\u4ee3\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIGANGGUTONGSHIDAIJIYUHUNHEC"}, "011357": {"suoxie": "HTBRPZCZHHA", "name": "\u534e\u6cf0\u67cf\u745e\u54c1\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIPINZHICHENGZHANGHUNHEA"}, "011358": {"suoxie": "HTBRPZCZHHC", "name": "\u534e\u6cf0\u67cf\u745e\u54c1\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIPINZHICHENGZHANGHUNHEC"}, "011359": {"suoxie": "CCYXTLYNHHA", "name": "\u957f\u57ce\u4f18\u9009\u6dfb\u5229\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANTIANLIYINIANHUNHEA"}, "011360": {"suoxie": "CCYXTLYNHHC", "name": "\u957f\u57ce\u4f18\u9009\u6dfb\u5229\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANTIANLIYINIANHUNHEC"}, "011361": {"suoxie": "HXBRYNCYHHMOMA", "name": "\u534e\u590f\u535a\u9510\u4e00\u5e74\u6301\u6709\u6df7\u5408(MOM)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIABORUIYINIANCHIYOUHUNHEMOMA"}, "011362": {"suoxie": "HXBRYNCYHHMOMC", "name": "\u534e\u590f\u535a\u9510\u4e00\u5e74\u6301\u6709\u6df7\u5408(MOM)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIABORUIYINIANCHIYOUHUNHEMOMC"}, "011363": {"suoxie": "NFXRJZYNCYHHA", "name": "\u5357\u65b9\u5174\u6da6\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINGRUNJIAZHIYINIANCHIYOUHUNHEA"}, "011364": {"suoxie": "NFXRJZYNCYHHC", "name": "\u5357\u65b9\u5174\u6da6\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINGRUNJIAZHIYINIANCHIYOUHUNHEC"}, "011365": {"suoxie": "PYJYHHA", "name": "\u9e4f\u626c\u666f\u9890\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGYIHUNHEA"}, "011366": {"suoxie": "PYJYHHC", "name": "\u9e4f\u626c\u666f\u9890\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PENGYANGJINGYIHUNHEC"}, "011367": {"suoxie": "CJQLYNDKHHMOMA", "name": "\u521b\u91d1\u7fa4\u529b\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408(MOM)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINQUNLIYINIANDINGKAIHUNHEMOMA"}, "011368": {"suoxie": "CJQLYNDKHHMOMC", "name": "\u521b\u91d1\u7fa4\u529b\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408(MOM)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINQUNLIYINIANDINGKAIHUNHEMOMC"}, "011369": {"suoxie": "HSJHCZHHA", "name": "\u534e\u5546\u5747\u8861\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJUNHENGCHENGZHANGHUNHEA"}, "011370": {"suoxie": "HSJHCZHHC", "name": "\u534e\u5546\u5747\u8861\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJUNHENGCHENGZHANGHUNHEC"}, "011371": {"suoxie": "HSYJJZA", "name": "\u534e\u5546\u8fdc\u89c1\u4ef7\u503cA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGYUANJIANJIAZHIA"}, "011372": {"suoxie": "HSYJJZC", "name": "\u534e\u5546\u8fdc\u89c1\u4ef7\u503cC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGYUANJIANJIAZHIC"}, "011373": {"suoxie": "ZSQYYLBJGPA", "name": "\u62db\u5546\u524d\u6cbf\u533b\u7597\u4fdd\u5065\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGQIANYANYILIAOBAOJIANGUPIAOA"}, "011374": {"suoxie": "ZSQYYLBJGPC", "name": "\u62db\u5546\u524d\u6cbf\u533b\u7597\u4fdd\u5065\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGQIANYANYILIAOBAOJIANGUPIAOC"}, "011375": {"suoxie": "HBAYHH", "name": "\u534e\u5b9d\u5b89\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUABAOANYIHUNHE"}, "011376": {"suoxie": "HBAXHH", "name": "\u534e\u5b9d\u5b89\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUABAOANXIANGHUNHE"}, "011377": {"suoxie": "CJHXJJCZGPA", "name": "\u521b\u91d1\u5408\u4fe1\u79ef\u6781\u6210\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINJIJICHENGZHANGGUPIAOA"}, "011378": {"suoxie": "CJHXJJCZGPC", "name": "\u521b\u91d1\u5408\u4fe1\u79ef\u6781\u6210\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINJIJICHENGZHANGGUPIAOC"}, "011383": {"suoxie": "FADYYCXHH", "name": "\u5bcc\u5b89\u8fbe\u533b\u836f\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDAYIYAOCHUANGXINHUNHE"}, "011384": {"suoxie": "NFYJHBGPA", "name": "\u5357\u65b9\u8fdc\u89c1\u56de\u62a5\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGYUANJIANHUIBAOGUPIAOA"}, "011385": {"suoxie": "NFYJHBGPC", "name": "\u5357\u65b9\u8fdc\u89c1\u56de\u62a5\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGYUANJIANHUIBAOGUPIAOC"}, "011387": {"suoxie": "GYNR6GYCYQHHA", "name": "\u5de5\u94f6\u5b81\u745e6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINNINGRUI6GEYUECHIYOUQIHUNHEA"}, "011388": {"suoxie": "GYNR6GYCYQHHC", "name": "\u5de5\u94f6\u5b81\u745e6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINNINGRUI6GEYUECHIYOUQIHUNHEC"}, "011389": {"suoxie": "GDJCHH", "name": "\u56fd\u90fd\u805a\u6210\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUODUJUCHENGHUNHE"}, "011390": {"suoxie": "HATX6GYCYHH", "name": "\u534e\u5b89\u6dfb\u79656\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANTIANXIANG6GEYUECHIYOUHUNHE"}, "011391": {"suoxie": "MSJYXZLHHC", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u6218\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINXINZHANLUEHUNHEC"}, "011392": {"suoxie": "PAXXHBYNDKHH", "name": "\u5e73\u5b89\u5174\u946b\u56de\u62a5\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANXINGXINHUIBAOYINIANDINGKAIHUNHE"}, "011393": {"suoxie": "ZORYWJYNHHA", "name": "\u4e2d\u6b27\u878d\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURONGYIWENJIANYINIANHUNHEA"}, "011394": {"suoxie": "ZORYWJYNHHC", "name": "\u4e2d\u6b27\u878d\u76ca\u7a33\u5065\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURONGYIWENJIANYINIANHUNHEC"}, "011395": {"suoxie": "BSHY6GYCYQHHA", "name": "\u535a\u65f6\u6052\u51436\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUAN6GEYUECHIYOUQIHUNHEA"}, "011396": {"suoxie": "BSHY6GYCYQHHC", "name": "\u535a\u65f6\u6052\u51436\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUAN6GEYUECHIYOUQIHUNHEC"}, "011397": {"suoxie": "ZSRH1NCYQHHA", "name": "\u62db\u5546\u745e\u548c1\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHE1NIANCHIYOUQIHUNHEA"}, "011398": {"suoxie": "ZSRH1NCYQHHC", "name": "\u62db\u5546\u745e\u548c1\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHE1NIANCHIYOUQIHUNHEC"}, "011399": {"suoxie": "HTFSZWLHHA", "name": "\u6c47\u6dfb\u5bcc\u6570\u5b57\u672a\u6765\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHUZIWEILAIHUNHEA"}, "011400": {"suoxie": "HTFSZWLHHC", "name": "\u6c47\u6dfb\u5bcc\u6570\u5b57\u672a\u6765\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHUZIWEILAIHUNHEC"}, "011401": {"suoxie": "HTFCZJXHHA", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGJINGXUANHUNHEA"}, "011402": {"suoxie": "HTFCZJXHHC", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGJINGXUANHUNHEC"}, "011403": {"suoxie": "RTXXCZHHA", "name": "\u878d\u901a\u946b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGXINXINCHENGZHANGHUNHEA"}, "011404": {"suoxie": "RTXXCZHHC", "name": "\u878d\u901a\u946b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGXINXINCHENGZHANGHUNHEC"}, "011405": {"suoxie": "YHWJZZYNCYQHH", "name": "\u94f6\u534e\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAWENJIANZENGZHANGYINIANCHIYOUQIHUNHE"}, "011408": {"suoxie": "THYXHHA", "name": "\u5929\u5f18\u76ca\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYIXINHUNHEA"}, "011409": {"suoxie": "THYXHHC", "name": "\u5929\u5f18\u76ca\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYIXINHUNHEC"}, "011410": {"suoxie": "ZXJTLHJQA", "name": "\u4e2d\u4fe1\u5efa\u6295\u91cf\u5316\u8fdb\u53d6A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULIANGHUAJINQUA"}, "011411": {"suoxie": "ZXJTLHJQC", "name": "\u4e2d\u4fe1\u5efa\u6295\u91cf\u5316\u8fdb\u53d6C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULIANGHUAJINQUC"}, "011412": {"suoxie": "YFDYJCZHHC", "name": "\u6613\u65b9\u8fbe\u8fdc\u89c1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYUANJIANCHENGZHANGHUNHEC"}, "011413": {"suoxie": "FGAYHBB", "name": "\u5bcc\u56fd\u5b89\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOANYIHUOBIB"}, "011414": {"suoxie": "PHNHYNCYQHHA", "name": "\u9e4f\u534e\u5b81\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUANINGHUAYINIANCHIYOUQIHUNHEA"}, "011415": {"suoxie": "PHNHYNCYQHHC", "name": "\u9e4f\u534e\u5b81\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUANINGHUAYINIANCHIYOUQIHUNHEC"}, "011416": {"suoxie": "HYJXZQA", "name": "\u6052\u8d8a\u5609\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HENGYUEJIAXINZHAIQUANA"}, "011417": {"suoxie": "HYJXZQC", "name": "\u6052\u8d8a\u5609\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HENGYUEJIAXINZHAIQUANC"}, "011418": {"suoxie": "HTFXFJXLNCYGPA", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUXIAOFEIJINGXUANLIANGNIANCHIYOUGUPIAOA"}, "011419": {"suoxie": "HTFXFJXLNCYGPC", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u7cbe\u9009\u4e24\u5e74\u6301\u6709\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUXIAOFEIJINGXUANLIANGNIANCHIYOUGUPIAOC"}, "011420": {"suoxie": "GFQQKJSGYDKHHQDIIRMBA", "name": "\u5e7f\u53d1\u5168\u7403\u79d1\u6280\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "GUANGFAQUANQIUKEJISANGEYUEDINGKAIHUNHEQDIIRENMINBIA"}, "011421": {"suoxie": "GFQQKJSGYDKHHQDIIMYA", "name": "\u5e7f\u53d1\u5168\u7403\u79d1\u6280\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408(QDII)\u7f8e\u5143A", "style": "QDII", "pinyin": "GUANGFAQUANQIUKEJISANGEYUEDINGKAIHUNHEQDIIMEIYUANA"}, "011422": {"suoxie": "GFQQKJSGYDKHHQDIIRMBC", "name": "\u5e7f\u53d1\u5168\u7403\u79d1\u6280\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "GUANGFAQUANQIUKEJISANGEYUEDINGKAIHUNHEQDIIRENMINBIC"}, "011423": {"suoxie": "GFQQKJSGYDKHHQDIIMYC", "name": "\u5e7f\u53d1\u5168\u7403\u79d1\u6280\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408(QDII)\u7f8e\u5143C", "style": "QDII", "pinyin": "GUANGFAQUANQIUKEJISANGEYUEDINGKAIHUNHEQDIIMEIYUANC"}, "011424": {"suoxie": "HTFWYZZGPC", "name": "\u6c47\u6dfb\u5bcc\u5916\u5ef6\u589e\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUITIANFUWAIYANZENGZHANGGUPIAOC"}, "011425": {"suoxie": "GFYSCZGPA", "name": "\u5e7f\u53d1\u4f18\u52bf\u6210\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYOUSHICHENGZHANGGUPIAOA"}, "011426": {"suoxie": "GFYSCZGPC", "name": "\u5e7f\u53d1\u4f18\u52bf\u6210\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAYOUSHICHENGZHANGGUPIAOC"}, "011427": {"suoxie": "GFJZQDHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIQUDONGHUNHEA"}, "011428": {"suoxie": "GFJZQDHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIQUDONGHUNHEC"}, "011429": {"suoxie": "QHKYMYJQ", "name": "\u524d\u6d77\u5f00\u6e90\u6c11\u88d5\u8fdb\u53d6", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANMINYUJINQU"}, "011430": {"suoxie": "GFGZYSHHC", "name": "\u5e7f\u53d1\u4f30\u503c\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGUZHIYOUSHIHUNHEC"}, "011431": {"suoxie": "TDHLXFFWHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d88\u8d39\u670d\u52a1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXIAOFEIFUWUHUNHEA"}, "011432": {"suoxie": "TDHLXFFWHHC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6d88\u8d39\u670d\u52a1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXIAOFEIFUWUHUNHEC"}, "011433": {"suoxie": "ZJJYYNDKHHA", "name": "\u4e2d\u52a0\u805a\u4f18\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJUYOUYINIANDINGKAIHUNHEA"}, "011434": {"suoxie": "ZJJYYNDKHHC", "name": "\u4e2d\u52a0\u805a\u4f18\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAJUYOUYINIANDINGKAIHUNHEC"}, "011435": {"suoxie": "ZOYJJXHHA", "name": "\u4e2d\u6b27\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYANJIUJINGXUANHUNHEA"}, "011436": {"suoxie": "ZOYJJXHHC", "name": "\u4e2d\u6b27\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYANJIUJINGXUANHUNHEC"}, "011437": {"suoxie": "ZTKYJZYXHHC", "name": "\u4e2d\u6cf0\u5f00\u9633\u4ef7\u503c\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGTAIKAIYANGJIAZHIYOUXUANHUNHEC"}, "011438": {"suoxie": "HTHTSCYXHHXA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u660c\u4f18\u9009\u6df7\u5408\u578bA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGCHANGYOUXUANHUNHEXINGA"}, "011439": {"suoxie": "HTHTSCYXHHXC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u660c\u4f18\u9009\u6df7\u5408\u578bC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGCHANGYOUXUANHUNHEXINGC"}, "011442": {"suoxie": "CJHXXRHHA", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINRUIHUNHEA"}, "011443": {"suoxie": "CJHXXRHHC", "name": "\u521b\u91d1\u5408\u4fe1\u946b\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINXINRUIHUNHEC"}, "011444": {"suoxie": "CJHXRYHHA", "name": "\u521b\u91d1\u5408\u4fe1\u745e\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINRUIYUHUNHEA"}, "011445": {"suoxie": "CJHXRYHHC", "name": "\u521b\u91d1\u5408\u4fe1\u745e\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINRUIYUHUNHEC"}, "011446": {"suoxie": "CJXNYCYHHFQA", "name": "\u957f\u6c5f\u65b0\u80fd\u6e90\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGXINNENGYUANCHANYEHUNHEFAQIA"}, "011447": {"suoxie": "CJXNYCYHHFQC", "name": "\u957f\u6c5f\u65b0\u80fd\u6e90\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGXINNENGYUANCHANYEHUNHEFAQIC"}, "011448": {"suoxie": "ZYZQJHCZHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5747\u8861\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANJUNHENGCHENGZHANGHUNHEA"}, "011449": {"suoxie": "ZYZQJHCZHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5747\u8861\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANJUNHENGCHENGZHANGHUNHEC"}, "011450": {"suoxie": "ZSQYYXHHA", "name": "\u62db\u5546\u4f01\u4e1a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGQIYEYOUXUANHUNHEA"}, "011451": {"suoxie": "ZSQYYXHHC", "name": "\u62db\u5546\u4f01\u4e1a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGQIYEYOUXUANHUNHEC"}, "011452": {"suoxie": "HTBRZLCZC", "name": "\u534e\u6cf0\u67cf\u745e\u8d28\u91cf\u6210\u957fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIZHILIANGCHENGZHANGC"}, "011453": {"suoxie": "HTBRYLJKC", "name": "\u534e\u6cf0\u67cf\u745e\u533b\u7597\u5065\u5eb7C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYILIAOJIANKANGC"}, "011454": {"suoxie": "HTBRJQYXC", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u4f18\u9009C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIYOUXUANC"}, "011455": {"suoxie": "CCJZYSLGYHHA", "name": "\u957f\u57ce\u7ade\u4e89\u4f18\u52bf\u516d\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJINGZHENGYOUSHILIUGEYUEHUNHEA"}, "011456": {"suoxie": "CCJZYSLGYHHC", "name": "\u957f\u57ce\u7ade\u4e89\u4f18\u52bf\u516d\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJINGZHENGYOUSHILIUGEYUEHUNHEC"}, "011457": {"suoxie": "XHHYLTZTGP", "name": "\u65b0\u534e\u884c\u4e1a\u9f99\u5934\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINHUAHANGYELONGTOUZHUTIGUPIAO"}, "011458": {"suoxie": "DFXXJZCZYNCYHHA", "name": "\u4e1c\u65b9\u946b\u4eab\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGXINXIANGJIAZHICHENGZHANGYINIANCHIYOUHUNHEA"}, "011459": {"suoxie": "DFXXJZCZYNCYHHC", "name": "\u4e1c\u65b9\u946b\u4eab\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGXINXIANGJIAZHICHENGZHANGYINIANCHIYOUHUNHEC"}, "011460": {"suoxie": "PHCXCZHHA", "name": "\u9e4f\u534e\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINCHENGZHANGHUNHEA"}, "011461": {"suoxie": "PHCXCZHHC", "name": "\u9e4f\u534e\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINCHENGZHANGHUNHEC"}, "011462": {"suoxie": "DWXXCZHHC", "name": "\u4e1c\u5434\u5174\u4eab\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXINGXIANGCHENGZHANGHUNHEC"}, "011463": {"suoxie": "CCLHJXGPC", "name": "\u957f\u57ce\u91cf\u5316\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGLIANGHUAJINGXUANGUPIAOC"}, "011464": {"suoxie": "NHRLCZA", "name": "\u5357\u534e\u745e\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUILICHUNZHAIA"}, "011465": {"suoxie": "NHRLCZC", "name": "\u5357\u534e\u745e\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANHUARUILICHUNZHAIC"}, "011466": {"suoxie": "XYYLBJHHA", "name": "\u5174\u4e1a\u533b\u7597\u4fdd\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEYILIAOBAOJIANHUNHEA"}, "011467": {"suoxie": "XYYLBJHHC", "name": "\u5174\u4e1a\u533b\u7597\u4fdd\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEYILIAOBAOJIANHUNHEC"}, "011468": {"suoxie": "GFJZYSSNCYQHHA", "name": "\u56fd\u5bcc\u7ade\u4e89\u4f18\u52bf\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJINGZHENGYOUSHISANNIANCHIYOUQIHUNHEA"}, "011469": {"suoxie": "GFJZYSSNCYQHHC", "name": "\u56fd\u5bcc\u7ade\u4e89\u4f18\u52bf\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJINGZHENGYOUSHISANNIANCHIYOUQIHUNHEC"}, "011470": {"suoxie": "DWXCYJXGPC", "name": "\u4e1c\u5434\u65b0\u4ea7\u4e1a\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUXINCHANYEJINGXUANGUPIAOC"}, "011471": {"suoxie": "PHZYCZHHA", "name": "\u9e4f\u534e\u81f4\u8fdc\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAZHIYUANCHENGZHANGHUNHEA"}, "011472": {"suoxie": "PHZYCZHHC", "name": "\u9e4f\u534e\u81f4\u8fdc\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAZHIYUANCHENGZHANGHUNHEC"}, "011473": {"suoxie": "GYZLZXGPC", "name": "\u5de5\u94f6\u6218\u7565\u8f6c\u578b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINZHANLUEZHUANXINGGUPIAOC"}, "011474": {"suoxie": "GYXXCYHHC", "name": "\u5de5\u94f6\u4fe1\u606f\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINXICHANYEHUNHEC"}, "011475": {"suoxie": "GYXFFWHHC", "name": "\u5de5\u94f6\u6d88\u8d39\u670d\u52a1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXIAOFEIFUWUHUNHEC"}, "011476": {"suoxie": "GYXLCGPC", "name": "\u5de5\u94f6\u65b0\u84dd\u7b79\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXINLANCHOUGUPIAOC"}, "011477": {"suoxie": "GYZHBLHPZHHC", "name": "\u5de5\u94f6\u603b\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINZONGHUIBAOLINGHUOPEIZHIHUNHEC"}, "011478": {"suoxie": "GYMLCZGPC", "name": "\u5de5\u94f6\u7f8e\u4e3d\u57ce\u9547\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINMEILICHENGZHENGUPIAOC"}, "011479": {"suoxie": "GFCXHHA", "name": "\u5e7f\u53d1\u8bda\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHENGXIANGHUNHEA"}, "011480": {"suoxie": "GFCXHHC", "name": "\u5e7f\u53d1\u8bda\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHENGXIANGHUNHEC"}, "011481": {"suoxie": "GFRJYNDQKFHH", "name": "\u5e7f\u53d1\u745e\u9526\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIJINYINIANDINGQIKAIFANGHUNHE"}, "011482": {"suoxie": "ZYSNHB6GYCYHHA", "name": "\u4e2d\u94f6\u987a\u5b81\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNNINGHUIBAO6GEYUECHIYOUHUNHEA"}, "011483": {"suoxie": "ZYSNHB6GYCYHHC", "name": "\u4e2d\u94f6\u987a\u5b81\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINSHUNNINGHUIBAO6GEYUECHIYOUHUNHEC"}, "011484": {"suoxie": "SWLXYXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b9c\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINYIXUANHUNHEA"}, "011485": {"suoxie": "SWLXYXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b9c\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINYIXUANHUNHEC"}, "011486": {"suoxie": "BSCXJXHHA", "name": "\u535a\u65f6\u521b\u65b0\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGXINJINGXUANHUNHEA"}, "011487": {"suoxie": "BSCXJXHHC", "name": "\u535a\u65f6\u521b\u65b0\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGXINJINGXUANHUNHEC"}, "011488": {"suoxie": "SWLXLXHH", "name": "\u7533\u4e07\u83f1\u4fe1\u4e50\u4eab\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINLEXIANGHUNHE"}, "011489": {"suoxie": "CJHXSJX6GYCYA", "name": "\u521b\u91d1\u5408\u4fe1\u53cc\u5b63\u4eab6\u4e2a\u6708\u6301\u6709A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINSHUANGJIXIANG6GEYUECHIYOUA"}, "011490": {"suoxie": "CJHXSJX6GYCYC", "name": "\u521b\u91d1\u5408\u4fe1\u53cc\u5b63\u4eab6\u4e2a\u6708\u6301\u6709C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINSHUANGJIXIANG6GEYUECHIYOUC"}, "011491": {"suoxie": "NFXJBHBB", "name": "\u5357\u65b9\u85aa\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXINJINBAOHUOBIB"}, "011492": {"suoxie": "HTZJFRZQFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u777f\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINFENGRUIZHAIQUANFAQIA"}, "011493": {"suoxie": "HTZJFRZQFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u777f\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINFENGRUIZHAIQUANFAQIC"}, "011494": {"suoxie": "HTZJFHPZHHA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u548c\u504f\u503a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIZIJINFENGHEPIANZHAIHUNHEA"}, "011495": {"suoxie": "HTZJFHPZHHC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e30\u548c\u504f\u503a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIZIJINFENGHEPIANZHAIHUNHEC"}, "011496": {"suoxie": "HTZJYYF1GYGDZQFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u6708\u6708\u53d11\u4e2a\u6708\u6eda\u52a8\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINYUEYUEFA1GEYUEGUNDONGZHAIQUANFAQIA"}, "011497": {"suoxie": "HTZJYYF1GYGDZQFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u6708\u6708\u53d11\u4e2a\u6708\u6eda\u52a8\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIZIJINYUEYUEFA1GEYUEGUNDONGZHAIQUANFAQIC"}, "011498": {"suoxie": "FGHS300JBMJXGPA", "name": "\u5bcc\u56fd\u6caa\u6df1300\u57fa\u672c\u9762\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOHUSHEN300JIBENMIANJINGXUANGUPIAOA"}, "011499": {"suoxie": "FGHS300JBMJXGPC", "name": "\u5bcc\u56fd\u6caa\u6df1300\u57fa\u672c\u9762\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOHUSHEN300JIBENMIANJINGXUANGUPIAOC"}, "011500": {"suoxie": "JTLHXXCY", "name": "\u4e5d\u6cf0\u91cf\u5316\u65b0\u5174\u4ea7\u4e1a", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAILIANGHUAXINXINGCHANYE"}, "011501": {"suoxie": "FZFBHFYNDKHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u6c47\u798f\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGHUIFUYINIANDINGKAIHUNHEA"}, "011502": {"suoxie": "FZFBHFYNDKHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u6c47\u798f\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGHUIFUYINIANDINGKAIHUNHEC"}, "011503": {"suoxie": "JXZNSHHH", "name": "\u5efa\u4fe1\u667a\u80fd\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINZHINENGSHENGHUOHUNHE"}, "011504": {"suoxie": "SYFYHHA", "name": "\u4e0a\u94f6\u4e30\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINFENGYIHUNHEA"}, "011505": {"suoxie": "SYFYHHC", "name": "\u4e0a\u94f6\u4e30\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINFENGYIHUNHEC"}, "011506": {"suoxie": "JXGDZBGPA", "name": "\u5efa\u4fe1\u9ad8\u7aef\u88c5\u5907\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINGAODUANZHUANGBEIGUPIAOA"}, "011507": {"suoxie": "JXGDZBGPC", "name": "\u5efa\u4fe1\u9ad8\u7aef\u88c5\u5907\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINGAODUANZHUANGBEIGUPIAOC"}, "011508": {"suoxie": "YFDYHYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u5f18\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEHONGYINIANCHIYOUQIHUNHEA"}, "011509": {"suoxie": "YFDYHYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u5f18\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEHONGYINIANCHIYOUQIHUNHEC"}, "011510": {"suoxie": "GSABWXYNCYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u946b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENXINYINIANCHIYOUHUNHEA"}, "011511": {"suoxie": "GSABWXYNCYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u946b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENXINYINIANCHIYOUHUNHEC"}, "011512": {"suoxie": "THZZXNYCA", "name": "\u5929\u5f18\u4e2d\u8bc1\u65b0\u80fd\u6e90\u8f66A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINNENGYUANCHEA"}, "011513": {"suoxie": "THZZXNYCC", "name": "\u5929\u5f18\u4e2d\u8bc1\u65b0\u80fd\u6e90\u8f66C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINNENGYUANCHEC"}, "011514": {"suoxie": "ZHHYHHA", "name": "\u4e2d\u6d77\u6d77\u8a89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHAIHAIYUHUNHEA"}, "011515": {"suoxie": "ZHHYHHC", "name": "\u4e2d\u6d77\u6d77\u8a89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHAIHAIYUHUNHEC"}, "011516": {"suoxie": "JSPYYNCYQHHA", "name": "\u5609\u5b9e\u6d66\u76c8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIPUYINGYINIANCHIYOUQIHUNHEA"}, "011517": {"suoxie": "JSPYYNCYQHHC", "name": "\u5609\u5b9e\u6d66\u76c8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIPUYINGYINIANCHIYOUQIHUNHEC"}, "011518": {"suoxie": "JSJZZXHH", "name": "\u5609\u5b9e\u4ef7\u503c\u81fb\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHIZHENXUANHUNHE"}, "011521": {"suoxie": "PYJYYNCYHHA", "name": "\u9e4f\u626c\u666f\u6e90\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGYUANYINIANCHIYOUHUNHEA"}, "011522": {"suoxie": "PYJYYNCYHHC", "name": "\u9e4f\u626c\u666f\u6e90\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGYUANYINIANCHIYOUHUNHEC"}, "011523": {"suoxie": "QHLHCYQSHHA", "name": "\u524d\u6d77\u8054\u5408\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHECHANYEQUSHIHUNHEA"}, "011524": {"suoxie": "QHLHCYQSHHC", "name": "\u524d\u6d77\u8054\u5408\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAILIANHECHANYEQUSHIHUNHEC"}, "011525": {"suoxie": "ZXBCFYYNCYQHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e30\u88d5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINBAOCHENGFENGYUYINIANCHIYOUQIHUNHEA"}, "011526": {"suoxie": "ZXBCFYYNCYQHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e30\u88d5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINBAOCHENGFENGYUYINIANCHIYOUQIHUNHEC"}, "011527": {"suoxie": "BSHY6GYCYHHA", "name": "\u535a\u65f6\u6052\u60a66\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUE6GEYUECHIYOUHUNHEA"}, "011528": {"suoxie": "BSHY6GYCYHHC", "name": "\u535a\u65f6\u6052\u60a66\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYUE6GEYUECHIYOUHUNHEC"}, "011529": {"suoxie": "SYHXYZQ", "name": "\u4e0a\u94f6\u6167\u5174\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIXINGYINGZHAIQUAN"}, "011530": {"suoxie": "HDYZZLLHPZHH", "name": "\u6cd3\u5fb7\u4f18\u8d28\u6cbb\u7406\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDEYOUZHIZHILILINGHUOPEIZHIHUNHE"}, "011531": {"suoxie": "ZQHXYNCYHH", "name": "\u6731\u96c0\u6052\u5fc3\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHUQUEHENGXINYINIANCHIYOUHUNHE"}, "011532": {"suoxie": "GYJFHHA", "name": "\u5de5\u94f6\u805a\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUFENGHUNHEA"}, "011533": {"suoxie": "GYJFHHC", "name": "\u5de5\u94f6\u805a\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUFENGHUNHEC"}, "011534": {"suoxie": "WJMRXM6GYCYHHA", "name": "\u4e07\u5bb6\u6c11\u745e\u7965\u660e6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAMINRUIXIANGMING6GEYUECHIYOUHUNHEA"}, "011535": {"suoxie": "WJMRXM6GYCYHHC", "name": "\u4e07\u5bb6\u6c11\u745e\u7965\u660e6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAMINRUIXIANGMING6GEYUECHIYOUHUNHEC"}, "011536": {"suoxie": "HSHYHHA", "name": "\u60e0\u5347\u60e0\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUIYIHUNHEA"}, "011537": {"suoxie": "HSHYHHC", "name": "\u60e0\u5347\u60e0\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUIYIHUNHEC"}, "011538": {"suoxie": "CCYXTRLGYCYHHA", "name": "\u957f\u57ce\u4f18\u9009\u6dfb\u745e\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANTIANRUILIUGEYUECHIYOUHUNHEA"}, "011539": {"suoxie": "CCYXTRLGYCYHHC", "name": "\u957f\u57ce\u4f18\u9009\u6dfb\u745e\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGCHENGYOUXUANTIANRUILIUGEYUECHIYOUHUNHEC"}, "011542": {"suoxie": "PHYJHBSNCYHH", "name": "\u9e4f\u534e\u8fdc\u89c1\u56de\u62a5\u4e09\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYUANJIANHUIBAOSANNIANCHIYOUHUNHE"}, "011543": {"suoxie": "ZJKRHHA", "name": "\u4e2d\u52a0\u79d1\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKERUIHUNHEA"}, "011544": {"suoxie": "ZJKRHHC", "name": "\u4e2d\u52a0\u79d1\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAKERUIHUNHEC"}, "011545": {"suoxie": "CJHS300ZSZQFQSA", "name": "\u957f\u6c5f\u6caa\u6df1300\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGJIANGHUSHEN300ZHISHUZENGQIANGFAQISHIA"}, "011546": {"suoxie": "CJHS300ZSZQFQSC", "name": "\u957f\u6c5f\u6caa\u6df1300\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGJIANGHUSHEN300ZHISHUZENGQIANGFAQISHIC"}, "011547": {"suoxie": "HXHLHBC", "name": "\u534e\u590f\u60e0\u5229\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAHUILIHUOBIC"}, "011548": {"suoxie": "JTJHHHA", "name": "\u4e5d\u6cf0\u4e45\u6167\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIUTAIJIUHUIHUNHEA"}, "011549": {"suoxie": "JTJHHHC", "name": "\u4e5d\u6cf0\u4e45\u6167\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIUTAIJIUHUIHUNHEC"}, "011550": {"suoxie": "XCCXCZYNCYQHHA", "name": "\u6e58\u8d22\u521b\u65b0\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIANGCAICHUANGXINCHENGZHANGYINIANCHIYOUQIHUNHEA"}, "011551": {"suoxie": "XCCXCZYNCYQHHC", "name": "\u6e58\u8d22\u521b\u65b0\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIANGCAICHUANGXINCHENGZHANGYINIANCHIYOUQIHUNHEC"}, "011552": {"suoxie": "PHMFYH6GYCYHHA", "name": "\u9e4f\u534e\u6c11\u4e30\u76c8\u548c6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAMINFENGYINGHE6GEYUECHIYOUHUNHEA"}, "011553": {"suoxie": "PHMFYH6GYCYHHC", "name": "\u9e4f\u534e\u6c11\u4e30\u76c8\u548c6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAMINFENGYINGHE6GEYUECHIYOUHUNHEC"}, "011554": {"suoxie": "HFTXLHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGXINLIHUNHEA"}, "011555": {"suoxie": "HFTXLHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGXINLIHUNHEC"}, "011556": {"suoxie": "FGMYJQHGSCZC", "name": "\u5bcc\u56fd\u6c11\u88d5\u8fdb\u53d6\u6caa\u6e2f\u6df1\u6210\u957fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOMINYUJINQUHUGANGSHENCHENGZHANGC"}, "011557": {"suoxie": "PAWJYLYNCYQHHFOF", "name": "\u5e73\u5b89\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANWENJIANYANGLAOYINIANCHIYOUQIHUNHEFOF"}, "011558": {"suoxie": "THNHLGYA", "name": "\u5929\u5f18\u5b81\u5f18\u516d\u4e2a\u6708A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGNINGHONGLIUGEYUEA"}, "011559": {"suoxie": "THNHLGYC", "name": "\u5929\u5f18\u5b81\u5f18\u516d\u4e2a\u6708C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGNINGHONGLIUGEYUEC"}, "011562": {"suoxie": "HTFWJYHYNCYHH", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u76c8\u548c\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANYINGHEYINIANCHIYOUHUNHE"}, "011565": {"suoxie": "FGZQYSHHC", "name": "\u5bcc\u56fd\u5468\u671f\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHOUQIYOUSHIHUNHEC"}, "011566": {"suoxie": "FGMLZGHHC", "name": "\u5bcc\u56fd\u7f8e\u4e3d\u4e2d\u56fd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOMEILIZHONGGUOHUNHEC"}, "011567": {"suoxie": "FGXFSJHHC", "name": "\u5bcc\u56fd\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXIAOFEISHENGJIHUNHEC"}, "011568": {"suoxie": "PHCYSJHHA", "name": "\u9e4f\u534e\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHANYESHENGJIHUNHEA"}, "011569": {"suoxie": "PHCYSJHHC", "name": "\u9e4f\u534e\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHANYESHENGJIHUNHEC"}, "011570": {"suoxie": "PHXYJZYNCYQHHA", "name": "\u9e4f\u534e\u946b\u8fdc\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINYUANJIAZHIYINIANCHIYOUQIHUNHEA"}, "011571": {"suoxie": "PHXYJZYNCYQHHC", "name": "\u9e4f\u534e\u946b\u8fdc\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINYUANJIAZHIYINIANCHIYOUQIHUNHEC"}, "011572": {"suoxie": "PHARHHA", "name": "\u9e4f\u534e\u5b89\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRONGHUNHEA"}, "011573": {"suoxie": "PHARHHC", "name": "\u9e4f\u534e\u5b89\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANRONGHUNHEC"}, "011574": {"suoxie": "PHLHYNCYHHA", "name": "\u9e4f\u534e\u9886\u822a\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUALINGHANGYINIANCHIYOUHUNHEA"}, "011575": {"suoxie": "PHLHYNCYHHC", "name": "\u9e4f\u534e\u9886\u822a\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUALINGHANGYINIANCHIYOUHUNHEC"}, "011576": {"suoxie": "PHACHHA", "name": "\u9e4f\u534e\u5b89\u8bda\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANCHENGHUNHEA"}, "011577": {"suoxie": "PHACHHC", "name": "\u9e4f\u534e\u5b89\u8bda\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANCHENGHUNHEC"}, "011578": {"suoxie": "HFJXHXCZA", "name": "\u6c47\u4e30\u664b\u4fe1\u6838\u5fc3\u6210\u957fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINHEXINCHENGZHANGA"}, "011579": {"suoxie": "HFJXHXCZC", "name": "\u6c47\u4e30\u664b\u4fe1\u6838\u5fc3\u6210\u957fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINHEXINCHENGZHANGC"}, "011580": {"suoxie": "MSJYWJPZ6GYHHFOF", "name": "\u6c11\u751f\u52a0\u94f6\u7a33\u5065\u914d\u7f6e6\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINWENJIANPEIZHI6GEYUEHUNHEFOF"}, "011581": {"suoxie": "PHAY5GYCYHHA", "name": "\u9e4f\u534e\u5b89\u6e905\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYUAN5GEYUECHIYOUHUNHEA"}, "011582": {"suoxie": "PHAY5GYCYHHC", "name": "\u9e4f\u534e\u5b89\u6e905\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYUAN5GEYUECHIYOUHUNHEC"}, "011583": {"suoxie": "DCGGJXHHQDIIA", "name": "\u5927\u6210\u6e2f\u80a1\u7cbe\u9009\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "DACHENGGANGGUJINGXUANHUNHEQDIIA"}, "011584": {"suoxie": "DCGGJXHHQDIIC", "name": "\u5927\u6210\u6e2f\u80a1\u7cbe\u9009\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "DACHENGGANGGUJINGXUANHUNHEQDIIC"}, "011585": {"suoxie": "BSCYHXHHA", "name": "\u535a\u65f6\u4ea7\u4e1a\u6167\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHANYEHUIXUANHUNHEA"}, "011586": {"suoxie": "BSCYHXHHC", "name": "\u535a\u65f6\u4ea7\u4e1a\u6167\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHANYEHUIXUANHUNHEC"}, "011587": {"suoxie": "DFHXHPHLNHHFOF", "name": "\u4e1c\u65b9\u7ea2\u6b23\u548c\u5e73\u8861\u4e24\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DONGFANGHONGXINHEPINGHENGLIANGNIANHUNHEFOF"}, "011588": {"suoxie": "QHKYCFJXHH", "name": "\u524d\u6d77\u5f00\u6e90\u6210\u4efd\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANCHENGFENJINGXUANHUNHE"}, "011589": {"suoxie": "JTTLLHA", "name": "\u4e5d\u6cf0\u5929\u5229\u91cf\u5316A", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAITIANLILIANGHUAA"}, "011590": {"suoxie": "JTTLLHC", "name": "\u4e5d\u6cf0\u5929\u5229\u91cf\u5316C", "style": "\u80a1\u7968\u578b", "pinyin": "JIUTAITIANLILIANGHUAC"}, "011591": {"suoxie": "MSJYWJPZ9GYCYHHFOF", "name": "\u6c11\u751f\u52a0\u94f6\u7a33\u5065\u914d\u7f6e9\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINWENJIANPEIZHI9GEYUECHIYOUHUNHEFOF"}, "011592": {"suoxie": "BSJGZTGPC", "name": "\u535a\u65f6\u519b\u5de5\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIJUNGONGZHUTIGUPIAOC"}, "011593": {"suoxie": "NYHLARYNCYHHFOF", "name": "\u519c\u94f6\u6c47\u7406\u5b89\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHUILIANRUIYINIANCHIYOUHUNHEFOF"}, "011594": {"suoxie": "GTRYWJYLFOFC", "name": "\u56fd\u6295\u745e\u94f6\u7a33\u5065\u517b\u8001(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINWENJIANYANGLAOFOFC"}, "011595": {"suoxie": "RTHXQSHHA", "name": "\u878d\u901a\u6838\u5fc3\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGHEXINQUSHIHUNHEA"}, "011596": {"suoxie": "RTHXQSHHC", "name": "\u878d\u901a\u6838\u5fc3\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGHEXINQUSHIHUNHEC"}, "011597": {"suoxie": "HTFDCLCZE", "name": "\u6c47\u6dfb\u5bcc\u591a\u7b56\u7565\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUDUOCELUECHUNZHAIE"}, "011598": {"suoxie": "XDAYYYJKHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u533b\u836f\u5065\u5eb7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYIYAOJIANKANGHUNHE"}, "011599": {"suoxie": "GLAJXKJ1GYCYHH", "name": "\u56fd\u8054\u5b89\u5320\u5fc3\u79d1\u62801\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANJIANGXINKEJI1GEYUECHIYOUHUNHE"}, "011600": {"suoxie": "YHHZSGYCYFOF", "name": "\u94f6\u534e\u534e\u667a\u4e09\u4e2a\u6708\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAHUAZHISANGEYUECHIYOUFOF"}, "011601": {"suoxie": "QHKYGGWSGPA", "name": "\u524d\u6d77\u5f00\u6e90\u516c\u5171\u536b\u751f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANGONGGONGWEISHENGGUPIAOA"}, "011602": {"suoxie": "QHKYGGWSGPC", "name": "\u524d\u6d77\u5f00\u6e90\u516c\u5171\u536b\u751f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANGONGGONGWEISHENGGUPIAOC"}, "011603": {"suoxie": "XYGDZZHHA", "name": "\u5174\u4e1a\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEGAODUANZHIZAOHUNHEA"}, "011604": {"suoxie": "XYGDZZHHC", "name": "\u5174\u4e1a\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEGAODUANZHIZAOHUNHEC"}, "011605": {"suoxie": "JYZXYNCYHHFOFA", "name": "\u4ea4\u94f6\u62db\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINZHAOXIANGYINIANCHIYOUHUNHEFOFA"}, "011606": {"suoxie": "JYZXYNCYHHFOFC", "name": "\u4ea4\u94f6\u62db\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINZHAOXIANGYINIANCHIYOUHUNHEFOFC"}, "011607": {"suoxie": "MSZZNDZYZTZSC", "name": "\u6c11\u751f\u4e2d\u8bc1\u5185\u5730\u8d44\u6e90\u4e3b\u9898\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGZHONGZHENGNEIDIZIYUANZHUTIZHISHUC"}, "011608": {"suoxie": "YFDKCB50ETFLJA", "name": "\u6613\u65b9\u8fbe\u79d1\u521b\u677f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAKECHUANGBAN50ETFLIANJIEA"}, "011609": {"suoxie": "YFDKCB50ETFLJC", "name": "\u6613\u65b9\u8fbe\u79d1\u521b\u677f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAKECHUANGBAN50ETFLIANJIEC"}, "011610": {"suoxie": "HTBRKCB50ETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u79d1\u521b\u677f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIKECHUANGBAN50ETFLIANJIEA"}, "011611": {"suoxie": "HTBRKCB50ETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u79d1\u521b\u677f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIKECHUANGBAN50ETFLIANJIEC"}, "011612": {"suoxie": "HXKCB50ETFLJA", "name": "\u534e\u590f\u79d1\u521b\u677f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAKECHUANGBAN50ETFLIANJIEA"}, "011613": {"suoxie": "HXKCB50ETFLJC", "name": "\u534e\u590f\u79d1\u521b\u677f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAKECHUANGBAN50ETFLIANJIEC"}, "011614": {"suoxie": "GYKCB50ETFLJA", "name": "\u5de5\u94f6\u79d1\u521b\u677f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINKECHUANGBAN50ETFLIANJIEA"}, "011615": {"suoxie": "GYKCB50ETFLJC", "name": "\u5de5\u94f6\u79d1\u521b\u677f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINKECHUANGBAN50ETFLIANJIEC"}, "011616": {"suoxie": "GTRYRXC", "name": "\u56fd\u6295\u745e\u94f6\u745e\u7965C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUIXIANGC"}, "011617": {"suoxie": "HTFAAAJXYCZE", "name": "\u6c47\u6dfb\u5bccAAA\u7ea7\u4fe1\u7528\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUAAAJIXINYONGCHUNZHAIE"}, "011618": {"suoxie": "GTRYRTDCLHHC", "name": "\u56fd\u6295\u745e\u94f6\u745e\u6cf0\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUITAIDUOCELUEHUNHEC"}, "011619": {"suoxie": "PYCXSGYDQKFZQA", "name": "\u9e4f\u626c\u6df3\u5174\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNXINGSANGEYUEDINGQIKAIFANGZHAIQUANA"}, "011620": {"suoxie": "PYCXSGYDQKFZQC", "name": "\u9e4f\u626c\u6df3\u5174\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNXINGSANGEYUEDINGQIKAIFANGZHAIQUANC"}, "011622": {"suoxie": "HTFDZZQE", "name": "\u6c47\u6dfb\u5bcc\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUDUANZHAIZHAIQUANE"}, "011623": {"suoxie": "HTFZDZE", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUZHONGDUANZHAIE"}, "011624": {"suoxie": "HXZXZQA", "name": "\u534e\u590f\u5353\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHUOXIANGZHAIQUANA"}, "011625": {"suoxie": "HXZXZQC", "name": "\u534e\u590f\u5353\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHUOXIANGZHAIQUANC"}, "011626": {"suoxie": "JSJXHBHHA", "name": "\u5609\u5b9e\u5320\u5fc3\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIANGXINHUIBAOHUNHEA"}, "011627": {"suoxie": "JSJXHBHHC", "name": "\u5609\u5b9e\u5320\u5fc3\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIANGXINHUIBAOHUNHEC"}, "011628": {"suoxie": "JSZM3GYDQCZZQ", "name": "\u5609\u5b9e\u81f4\u660e3\u4e2a\u6708\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIMING3GEYUEDINGQICHUNZHAIZHAIQUAN"}, "011630": {"suoxie": "DCYSJSA", "name": "\u4e1c\u8d22\u6709\u8272\u91d1\u5c5eA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYOUSEJINSHUA"}, "011631": {"suoxie": "DCYSJSC", "name": "\u4e1c\u8d22\u6709\u8272\u91d1\u5c5eC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYOUSEJINSHUC"}, "011634": {"suoxie": "GSABAYCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u60a6\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANYUECHUNZHAIYINIANDINGKAIZHAI"}, "011635": {"suoxie": "FGGGTCLJXHHA", "name": "\u5bcc\u56fd\u6e2f\u80a1\u901a\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGANGGUTONGCELUEJINGXUANHUNHEA"}, "011636": {"suoxie": "FGGGTCLJXHHC", "name": "\u5bcc\u56fd\u6e2f\u80a1\u901a\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGANGGUTONGCELUEJINGXUANHUNHEC"}, "011637": {"suoxie": "GFHGSJZCZHHA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJIAZHICHENGZHANGHUNHEA"}, "011638": {"suoxie": "GFHGSJZCZHHC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJIAZHICHENGZHANGHUNHEC"}, "011641": {"suoxie": "FGDLCZYNDKZFQS", "name": "\u5bcc\u56fd\u8fbe\u5229\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUODALICHUNZHAIYINIANDINGKAIZHAIFAQISHI"}, "011642": {"suoxie": "CTZGRH1NDKZ", "name": "\u8d22\u901a\u8d44\u7ba1\u777f\u61671\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANRUIHUI1NIANDINGKAIZHAI"}, "011643": {"suoxie": "JSSDXFSNCYHHA", "name": "\u5609\u5b9e\u65f6\u4ee3\u5148\u950b\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHISHIDAIXIANFENGSANNIANCHIYOUHUNHEA"}, "011644": {"suoxie": "JSSDXFSNCYHHC", "name": "\u5609\u5b9e\u65f6\u4ee3\u5148\u950b\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHISHIDAIXIANFENGSANNIANCHIYOUHUNHEC"}, "011645": {"suoxie": "GTHXJZLNCYQGPA", "name": "\u56fd\u6cf0\u6838\u5fc3\u4ef7\u503c\u4e24\u5e74\u6301\u6709\u671f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIHEXINJIAZHILIANGNIANCHIYOUQIGUPIAOA"}, "011646": {"suoxie": "GTHXJZLNCYQGPC", "name": "\u56fd\u6cf0\u6838\u5fc3\u4ef7\u503c\u4e24\u5e74\u6301\u6709\u671f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIHEXINJIAZHILIANGNIANCHIYOUQIGUPIAOC"}, "011647": {"suoxie": "BSGGTHLJXHHA", "name": "\u535a\u65f6\u6e2f\u80a1\u901a\u7ea2\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGANGGUTONGHONGLIJINGXUANHUNHEA"}, "011648": {"suoxie": "BSGGTHLJXHHC", "name": "\u535a\u65f6\u6e2f\u80a1\u901a\u7ea2\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIGANGGUTONGHONGLIJINGXUANHUNHEC"}, "011649": {"suoxie": "YFDNXTZHHA", "name": "\u6613\u65b9\u8fbe\u9006\u5411\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDANIXIANGTOUZIHUNHEA"}, "011650": {"suoxie": "YFDNXTZHHC", "name": "\u6613\u65b9\u8fbe\u9006\u5411\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDANIXIANGTOUZIHUNHEC"}, "011651": {"suoxie": "ZSGGTHXJXGPA", "name": "\u62db\u5546\u6e2f\u80a1\u901a\u6838\u5fc3\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGGANGGUTONGHEXINJINGXUANGUPIAOA"}, "011652": {"suoxie": "ZSGGTHXJXGPC", "name": "\u62db\u5546\u6e2f\u80a1\u901a\u6838\u5fc3\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGGANGGUTONGHEXINJINGXUANGUPIAOC"}, "011653": {"suoxie": "GTXXWJ6GYZQA", "name": "\u56fd\u6cf0\u946b\u4eab\u7a33\u50656\u4e2a\u6708\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIXINXIANGWENJIAN6GEYUEZHAIQUANA"}, "011654": {"suoxie": "GTXXWJ6GYZQC", "name": "\u56fd\u6cf0\u946b\u4eab\u7a33\u50656\u4e2a\u6708\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIXINXIANGWENJIAN6GEYUEZHAIQUANC"}, "011655": {"suoxie": "THXYYNDK", "name": "\u5929\u5f18\u5174\u76ca\u4e00\u5e74\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGXINGYIYINIANDINGKAI"}, "011656": {"suoxie": "THJJJFQZA", "name": "\u5929\u5f18\u4eac\u6d25\u5180\u53d1\u8d77\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGJINGJINJIFAQIZHAIA"}, "011657": {"suoxie": "THJJJFQZC", "name": "\u5929\u5f18\u4eac\u6d25\u5180\u53d1\u8d77\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGJINGJINJIFAQIZHAIC"}, "011658": {"suoxie": "HTFZGDJXYZA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGGAODENGJIXINYONGZHAIA"}, "011659": {"suoxie": "HTFZGDJXYZC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGGAODENGJIXINYONGZHAIC"}, "011660": {"suoxie": "HTFZGDJXYZE", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUZHONGGAODENGJIXINYONGZHAIE"}, "011661": {"suoxie": "HFZZ13NGKZA", "name": "\u534e\u5bcc\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAI13NIANGUOKAIZHAIA"}, "011662": {"suoxie": "HFZZ13NGKZC", "name": "\u534e\u5bcc\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUZHONGZHAI13NIANGUOKAIZHAIC"}, "011663": {"suoxie": "HAYJQDHHA", "name": "\u534e\u5b89\u7814\u7a76\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUQUDONGHUNHEA"}, "011664": {"suoxie": "HAYJQDHHC", "name": "\u534e\u5b89\u7814\u7a76\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUQUDONGHUNHEC"}, "011665": {"suoxie": "HTFSZJJSNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u6570\u5b57\u7ecf\u6d4e\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHUZIJINGJISANNIANCHIYOUHUNHEA"}, "011666": {"suoxie": "HTFSZJJSNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u6570\u5b57\u7ecf\u6d4e\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHUZIJINGJISANNIANCHIYOUHUNHEC"}, "011667": {"suoxie": "DCGDZZA", "name": "\u4e1c\u8d22\u9ad8\u7aef\u5236\u9020A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIGAODUANZHIZAOA"}, "011668": {"suoxie": "DCGDZZC", "name": "\u4e1c\u8d22\u9ad8\u7aef\u5236\u9020C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIGAODUANZHIZAOC"}, "011669": {"suoxie": "CXYZQYHHA", "name": "\u957f\u4fe1\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINYOUZHIQIYEHUNHEA"}, "011670": {"suoxie": "CXYZQYHHC", "name": "\u957f\u4fe1\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINYOUZHIQIYEHUNHEC"}, "011671": {"suoxie": "ZXJTSL3GYCYZQA", "name": "\u4e2d\u4fe1\u5efa\u6295\u53cc\u52293\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINJIANTOUSHUANGLI3GEYUECHIYOUZHAIQUANA"}, "011672": {"suoxie": "ZXJTSL3GYCYZQC", "name": "\u4e2d\u4fe1\u5efa\u6295\u53cc\u52293\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINJIANTOUSHUANGLI3GEYUECHIYOUZHAIQUANC"}, "011673": {"suoxie": "CCYYKJLGYCYHHA", "name": "\u957f\u57ce\u533b\u836f\u79d1\u6280\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYIYAOKEJILIUGEYUECHIYOUHUNHEA"}, "011674": {"suoxie": "CCYYKJLGYCYHHC", "name": "\u957f\u57ce\u533b\u836f\u79d1\u6280\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYIYAOKEJILIUGEYUECHIYOUHUNHEC"}, "011675": {"suoxie": "CCZZ15NGKZZSA", "name": "\u957f\u57ce\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI15NIANGUOKAIZHAIZHISHUA"}, "011676": {"suoxie": "CCZZ15NGKZZSC", "name": "\u957f\u57ce\u4e2d\u503a1-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHONGZHAI15NIANGUOKAIZHAIZHISHUC"}, "011677": {"suoxie": "ZYRFHBHHA", "name": "\u4e2d\u94f6\u777f\u4e30\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINRUIFENGHUIBAOHUNHEA"}, "011678": {"suoxie": "ZYRFHBHHC", "name": "\u4e2d\u94f6\u777f\u4e30\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINRUIFENGHUIBAOHUNHEC"}, "011681": {"suoxie": "HTFJHJXLGYCYHHA", "name": "\u6c47\u6dfb\u5bcc\u5747\u8861\u7cbe\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUNHENGJINGXUANLIUGEYUECHIYOUHUNHEA"}, "011682": {"suoxie": "HTFJHJXLGYCYHHC", "name": "\u6c47\u6dfb\u5bcc\u5747\u8861\u7cbe\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUNHENGJINGXUANLIUGEYUECHIYOUHUNHEC"}, "011683": {"suoxie": "HXDHYNDKZ", "name": "\u534e\u590f\u9f0e\u534e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGHUAYINIANDINGKAIZHAI"}, "011684": {"suoxie": "PAYL2045WNCYHHFOF", "name": "\u5e73\u5b89\u517b\u80012045\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANGLAO2045WUNIANCHIYOUHUNHEFOF"}, "011685": {"suoxie": "CJHXXJZBGPA", "name": "\u521b\u91d1\u5408\u4fe1\u5148\u8fdb\u88c5\u5907\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXIANJINZHUANGBEIGUPIAOA"}, "011686": {"suoxie": "CJHXXJZBGPC", "name": "\u521b\u91d1\u5408\u4fe1\u5148\u8fdb\u88c5\u5907\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXIANJINZHUANGBEIGUPIAOC"}, "011687": {"suoxie": "YFDLTYXLNCYHHA", "name": "\u6613\u65b9\u8fbe\u9f99\u5934\u4f18\u9009\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDALONGTOUYOUXUANLIANGNIANCHIYOUHUNHEA"}, "011688": {"suoxie": "YFDLTYXLNCYHHC", "name": "\u6613\u65b9\u8fbe\u9f99\u5934\u4f18\u9009\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDALONGTOUYOUXUANLIANGNIANCHIYOUHUNHEC"}, "011690": {"suoxie": "ZSPZFXHHA", "name": "\u62db\u5546\u54c1\u8d28\u53d1\u73b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHIFAXIANHUNHEA"}, "011691": {"suoxie": "ZSPZFXHHC", "name": "\u62db\u5546\u54c1\u8d28\u53d1\u73b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHIFAXIANHUNHEC"}, "011692": {"suoxie": "HAYJZXHHA", "name": "\u534e\u5b89\u7814\u7a76\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUZHIXUANHUNHEA"}, "011693": {"suoxie": "HAYJZXHHC", "name": "\u534e\u5b89\u7814\u7a76\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUZHIXUANHUNHEC"}, "011694": {"suoxie": "HTZJXXKJ6GYDKA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4fe1\u606f\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIZIJINXINXIKEJI6GEYUEDINGKAIA"}, "011695": {"suoxie": "HTZJXXKJ6GYDKC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4fe1\u606f\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIZIJINXINXIKEJI6GEYUEDINGKAIC"}, "011696": {"suoxie": "NFHRJQJX3GYHHFOFA", "name": "\u5357\u65b9\u6d69\u777f\u8fdb\u53d6\u4eac\u90093\u4e2a\u6708\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHAORUIJINQUJINGXUAN3GEYUEHUNHEFOFA"}, "011697": {"suoxie": "NFHRJQJX3GYHHFOFC", "name": "\u5357\u65b9\u6d69\u777f\u8fdb\u53d6\u4eac\u90093\u4e2a\u6708\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHAORUIJINQUJINGXUAN3GEYUEHUNHEFOFC"}, "011698": {"suoxie": "NFJHHBHHA", "name": "\u5357\u65b9\u5747\u8861\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGJUNHENGHUIBAOHUNHEA"}, "011699": {"suoxie": "FCFHZQA", "name": "\u8702\u5de2\u4e30\u534e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGHUAZHAIQUANA"}, "011700": {"suoxie": "FCFHZQC", "name": "\u8702\u5de2\u4e30\u534e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGHUAZHAIQUANC"}, "011701": {"suoxie": "NFJHHBHHC", "name": "\u5357\u65b9\u5747\u8861\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGJUNHENGHUIBAOHUNHEC"}, "011702": {"suoxie": "GFRXWJZLHHC", "name": "\u5e7f\u53d1\u777f\u4eab\u7a33\u5065\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFARUIXIANGWENJIANZENGLIHUNHEC"}, "011703": {"suoxie": "ZJXRYXYNCYHH", "name": "\u4e2d\u91d1\u946b\u745e\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINXINRUIYOUXUANYINIANCHIYOUHUNHE"}, "011704": {"suoxie": "DFAEFCYXFHHA", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4ea7\u4e1a\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFACHANYEXIANFENGHUNHEA"}, "011705": {"suoxie": "DFAEFCYXFHHC", "name": "\u4e1c\u65b9\u963f\u5c14\u6cd5\u4ea7\u4e1a\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGAERFACHANYEXIANFENGHUNHEC"}, "011706": {"suoxie": "CXBP100DQZZSMY", "name": "\u957f\u4fe1\u6807\u666e100\u7b49\u6743\u91cd\u6307\u6570\u7f8e\u5143", "style": "QDII", "pinyin": "CHANGXINBIAOPU100DENGQUANZHONGZHISHUMEIYUAN"}, "011707": {"suoxie": "DWPZYHHHC", "name": "\u4e1c\u5434\u914d\u7f6e\u4f18\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUPEIZHIYOUHUAHUNHEC"}, "011708": {"suoxie": "ZOJYYNCYQHHA", "name": "\u4e2d\u6b27\u5609\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAYIYINIANCHIYOUQIHUNHEA"}, "011709": {"suoxie": "ZOJYYNCYQHHC", "name": "\u4e2d\u6b27\u5609\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAYIYINIANCHIYOUQIHUNHEC"}, "011710": {"suoxie": "ZORZHHA", "name": "\u4e2d\u6b27\u777f\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOURUIZEHUNHEA"}, "011711": {"suoxie": "ZORZHHC", "name": "\u4e2d\u6b27\u777f\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOURUIZEHUNHEC"}, "011712": {"suoxie": "DMWZCXHHC", "name": "\u5927\u6469\u4e07\u4f17\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DAMOWANZHONGCHUANGXINHUNHEC"}, "011713": {"suoxie": "ZXBCSYYNCYHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u76db\u88d5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINBAOCHENGSHENGYUYINIANCHIYOUHUNHEA"}, "011714": {"suoxie": "ZXBCSYYNCYHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u76db\u88d5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGXINBAOCHENGSHENGYUYINIANCHIYOUHUNHEC"}, "011717": {"suoxie": "PYJHYX6GYCYHHA", "name": "\u6d66\u94f6\u5747\u8861\u4f18\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINJUNHENGYOUXUAN6GEYUECHIYOUHUNHEA"}, "011718": {"suoxie": "PYJHYX6GYCYHHC", "name": "\u6d66\u94f6\u5747\u8861\u4f18\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINJUNHENGYOUXUAN6GEYUECHIYOUHUNHEC"}, "011719": {"suoxie": "PYASSHYNDKZQ", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u534e\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGHUAYINIANDINGKAIZHAIQUAN"}, "011720": {"suoxie": "YFDYXYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXINYINIANCHIYOUHUNHEA"}, "011721": {"suoxie": "YFDYXYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXINYINIANCHIYOUHUNHEC"}, "011722": {"suoxie": "QHKYSZTQJXGPA", "name": "\u524d\u6d77\u5f00\u6e90\u6df1\u5733\u7279\u533a\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANSHENZHENTEQUJINGXUANGUPIAOA"}, "011723": {"suoxie": "QHKYSZTQJXGPC", "name": "\u524d\u6d77\u5f00\u6e90\u6df1\u5733\u7279\u533a\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "QIANHAIKAIYUANSHENZHENTEQUJINGXUANGUPIAOC"}, "011724": {"suoxie": "DFHQHSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u542f\u6052\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHENGSANNIANCHIYOUHUNHEB"}, "011725": {"suoxie": "DFHXYSNCYHHB", "name": "\u4e1c\u65b9\u7ea2\u65b0\u6e90\u4e09\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGXINYUANSANNIANCHIYOUHUNHEB"}, "011726": {"suoxie": "AXXCTGPC", "name": "\u5b89\u4fe1\u65b0\u5e38\u6001\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ANXINXINCHANGTAIGUPIAOC"}, "011727": {"suoxie": "GYJRHHA", "name": "\u5de5\u94f6\u805a\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJURUIHUNHEA"}, "011728": {"suoxie": "GYJRHHC", "name": "\u5de5\u94f6\u805a\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJURUIHUNHEC"}, "011729": {"suoxie": "GYJXHHA", "name": "\u5de5\u94f6\u805a\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUXIANGHUNHEA"}, "011730": {"suoxie": "GYJXHHC", "name": "\u5de5\u94f6\u805a\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUXIANGHUNHEC"}, "011731": {"suoxie": "GTRYARHHA", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u777f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANRUIHUNHEA"}, "011732": {"suoxie": "GTRYARHHC", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u777f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANRUIHUNHEC"}, "011733": {"suoxie": "YHRXYNCYQHH", "name": "\u94f6\u534e\u745e\u7965\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUARUIXIANGYINIANCHIYOUQIHUNHE"}, "011734": {"suoxie": "GSABYFHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOSHOUANBAOYUFENGHUNHEA"}, "011735": {"suoxie": "GSABYFHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOSHOUANBAOYUFENGHUNHEC"}, "011736": {"suoxie": "BYXQ9GYCYHHA", "name": "\u5b9d\u76c8\u7965\u5e869\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGQING9GEYUECHIYOUHUNHEA"}, "011737": {"suoxie": "BYXQ9GYCYHHC", "name": "\u5b9d\u76c8\u7965\u5e869\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BAOYINGXIANGQING9GEYUECHIYOUHUNHEC"}, "011738": {"suoxie": "HAXAYXYNCYHHA", "name": "\u534e\u5b89\u5174\u5b89\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANXINGANYOUXUANYINIANCHIYOUHUNHEA"}, "011739": {"suoxie": "HAXAYXYNCYHHC", "name": "\u534e\u5b89\u5174\u5b89\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANXINGANYOUXUANYINIANCHIYOUHUNHEC"}, "011740": {"suoxie": "BSCZJXHHA", "name": "\u535a\u65f6\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGJINGXUANHUNHEA"}, "011741": {"suoxie": "BSCZJXHHC", "name": "\u535a\u65f6\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGJINGXUANHUNHEC"}, "011742": {"suoxie": "DCHPYNDKZFQS", "name": "\u5927\u6210\u60e0\u5e73\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIPINGYINIANDINGKAIZHAIFAQISHI"}, "011743": {"suoxie": "HXXYWJYNCYHHA", "name": "\u534e\u590f\u5174\u6e90\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAXINGYUANWENJIANYINIANCHIYOUHUNHEA"}, "011744": {"suoxie": "HXXYWJYNCYHHC", "name": "\u534e\u590f\u5174\u6e90\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAXINGYUANWENJIANYINIANCHIYOUHUNHEC"}, "011745": {"suoxie": "HXYL2055WNCYHHFOF", "name": "\u534e\u590f\u517b\u80012055\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYANGLAO2055WUNIANCHIYOUHUNHEFOF"}, "011746": {"suoxie": "NFYPYNCYHHA", "name": "\u5357\u65b9\u8a89\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUPUYINIANCHIYOUHUNHEA"}, "011747": {"suoxie": "NFYPYNCYHHC", "name": "\u5357\u65b9\u8a89\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUPUYINIANCHIYOUHUNHEC"}, "011748": {"suoxie": "HTBRJQCZHHA", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQICHENGZHANGHUNHEA"}, "011749": {"suoxie": "HTBRJQCZHHC", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQICHENGZHANGHUNHEC"}, "011750": {"suoxie": "BSHXYNDKHHA", "name": "\u535a\u65f6\u6052\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXINGYINIANDINGKAIHUNHEA"}, "011751": {"suoxie": "BSHXYNDKHHC", "name": "\u535a\u65f6\u6052\u5174\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXINGYINIANDINGKAIHUNHEC"}, "011752": {"suoxie": "GFHXYXLGYCYHHFOFA", "name": "\u5e7f\u53d1\u6838\u5fc3\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHEXINYOUXUANLIUGEYUECHIYOUHUNHEFOFA"}, "011753": {"suoxie": "GFHXYXLGYCYHHFOFC", "name": "\u5e7f\u53d1\u6838\u5fc3\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHEXINYOUXUANLIUGEYUECHIYOUHUNHEFOFC"}, "011754": {"suoxie": "PYXJTLHBD", "name": "\u9e4f\u626c\u73b0\u91d1\u901a\u5229\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGYANGXIANJINTONGLIHUOBID"}, "011755": {"suoxie": "GFJZYSHHC", "name": "\u5e7f\u53d1\u7ade\u4e89\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAJINGZHENGYOUSHIHUNHEC"}, "011756": {"suoxie": "BSCYYXHHA", "name": "\u535a\u65f6\u4ea7\u4e1a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEYOUXUANHUNHEA"}, "011757": {"suoxie": "BSCYYXHHC", "name": "\u535a\u65f6\u4ea7\u4e1a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICHANYEYOUXUANHUNHEC"}, "011758": {"suoxie": "GFNXCLHHC", "name": "\u5e7f\u53d1\u9006\u5411\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFANIXIANGCELUEHUNHEC"}, "011761": {"suoxie": "PAXRHHA", "name": "\u5e73\u5b89\u946b\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANXINRUIHUNHEA"}, "011762": {"suoxie": "PAXRHHC", "name": "\u5e73\u5b89\u946b\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANXINRUIHUNHEC"}, "011765": {"suoxie": "XYGDZZHHA", "name": "\u5174\u94f6\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINGAODUANZHIZAOHUNHEA"}, "011766": {"suoxie": "XYGDZZHHC", "name": "\u5174\u94f6\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINGAODUANZHIZAOHUNHEC"}, "011767": {"suoxie": "TKHRHHA", "name": "\u6cf0\u5eb7\u5408\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGHERUNHUNHEA"}, "011768": {"suoxie": "TKHRHHC", "name": "\u6cf0\u5eb7\u5408\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGHERUNHUNHEC"}, "011769": {"suoxie": "FGJCHB12GYCYQHHA", "name": "\u5bcc\u56fd\u7cbe\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOJINGCHENGHUIBAO12GEYUECHIYOUQIHUNHEA"}, "011770": {"suoxie": "FGJCHB12GYCYQHHC", "name": "\u5bcc\u56fd\u7cbe\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOJINGCHENGHUIBAO12GEYUECHIYOUQIHUNHEC"}, "011771": {"suoxie": "GSABWLHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u9686\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENLONGHUNHEA"}, "011772": {"suoxie": "GSABWLHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u9686\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENLONGHUNHEC"}, "011773": {"suoxie": "GSABJC6GYCYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u749f\u73f96\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOJINGCHENG6GEYUECHIYOUHUNHEA"}, "011774": {"suoxie": "GSABJC6GYCYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u749f\u73f96\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOJINGCHENG6GEYUECHIYOUHUNHEC"}, "011775": {"suoxie": "GLXYYNCYQHHA", "name": "\u683c\u6797\u946b\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GELINXINYUEYINIANCHIYOUQIHUNHEA"}, "011776": {"suoxie": "GLXYYNCYQHHC", "name": "\u683c\u6797\u946b\u60a6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GELINXINYUEYINIANCHIYOUQIHUNHEC"}, "011777": {"suoxie": "YFDWJZZHHA", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAWENJIANZENGZHANGHUNHEA"}, "011778": {"suoxie": "YFDWJZZHHC", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAWENJIANZENGZHANGHUNHEC"}, "011779": {"suoxie": "YFDWTYNCYHHA", "name": "\u6613\u65b9\u8fbe\u7a33\u6cf0\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAWENTAIYINIANCHIYOUHUNHEA"}, "011780": {"suoxie": "YFDWTYNCYHHC", "name": "\u6613\u65b9\u8fbe\u7a33\u6cf0\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAWENTAIYINIANCHIYOUHUNHEC"}, "011781": {"suoxie": "HDHXHHA", "name": "\u6cd3\u5fb7\u6167\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDEHUIXIANGHUNHEA"}, "011782": {"suoxie": "HDHXHHC", "name": "\u6cd3\u5fb7\u6167\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HONGDEHUIXIANGHUNHEC"}, "011783": {"suoxie": "HDRYSNCYQHH", "name": "\u6cd3\u5fb7\u777f\u6e90\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDERUIYUANSANNIANCHIYOUQIHUNHE"}, "011784": {"suoxie": "THZTLHHA", "name": "\u5929\u5f18\u62db\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGZHAOTIANLIHUNHEA"}, "011785": {"suoxie": "THZTLHHC", "name": "\u5929\u5f18\u62db\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGZHAOTIANLIHUNHEC"}, "011786": {"suoxie": "GYJAHHA", "name": "\u5de5\u94f6\u805a\u5b89\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUANHUNHEA"}, "011787": {"suoxie": "GYJAHHC", "name": "\u5de5\u94f6\u805a\u5b89\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUANHUNHEC"}, "011788": {"suoxie": "GYJYHHA", "name": "\u5de5\u94f6\u805a\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUYIHUNHEA"}, "011789": {"suoxie": "GYJYHHC", "name": "\u5de5\u94f6\u805a\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUYIHUNHEC"}, "011790": {"suoxie": "JXCXQDHH", "name": "\u5efa\u4fe1\u521b\u65b0\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINCHUANGXINQUDONGHUNHE"}, "011791": {"suoxie": "ZSRY9GYCYQHHA", "name": "\u62db\u5546\u745e\u76c89\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIYING9GEYUECHIYOUQIHUNHEA"}, "011792": {"suoxie": "ZSRY9GYCYQHHC", "name": "\u62db\u5546\u745e\u76c89\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIYING9GEYUECHIYOUQIHUNHEC"}, "011793": {"suoxie": "JXZNQCGP", "name": "\u5efa\u4fe1\u667a\u80fd\u6c7d\u8f66\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINZHINENGQICHEGUPIAO"}, "011798": {"suoxie": "HANX6GYHHA", "name": "\u534e\u5b89\u5b81\u4eab6\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANNINGXIANG6GEYUEHUNHEA"}, "011799": {"suoxie": "HANX6GYHHC", "name": "\u534e\u5b89\u5b81\u4eab6\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANNINGXIANG6GEYUEHUNHEC"}, "011800": {"suoxie": "SWLXJZJXHH", "name": "\u7533\u4e07\u83f1\u4fe1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINJIAZHIJINGXUANHUNHE"}, "011801": {"suoxie": "ZYZQYRHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u76c8\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANYINGRUIHUNHEA"}, "011802": {"suoxie": "ZYZQYRHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u76c8\u745e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANYINGRUIHUNHEC"}, "011803": {"suoxie": "JSCCNJ6GYCYHHA", "name": "\u666f\u987a\u957f\u57ce\u5b81\u666f6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGNINGJING6GEYUECHIYOUHUNHEA"}, "011804": {"suoxie": "JSCCNJ6GYCYHHC", "name": "\u666f\u987a\u957f\u57ce\u5b81\u666f6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGNINGJING6GEYUECHIYOUHUNHEC"}, "011805": {"suoxie": "JSYZHXLNCYHHA", "name": "\u5609\u5b9e\u4f18\u8d28\u6838\u5fc3\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUZHIHEXINLIANGNIANCHIYOUHUNHEA"}, "011806": {"suoxie": "JSYZHXLNCYHHC", "name": "\u5609\u5b9e\u4f18\u8d28\u6838\u5fc3\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUZHIHEXINLIANGNIANCHIYOUHUNHEC"}, "011807": {"suoxie": "PAYJJXHHA", "name": "\u5e73\u5b89\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANJIUJINGXUANHUNHEA"}, "011808": {"suoxie": "PAYJJXHHC", "name": "\u5e73\u5b89\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYANJIUJINGXUANHUNHEC"}, "011811": {"suoxie": "CTAHHHFQA", "name": "\u8d22\u901a\u5b89\u534e\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGANHUAHUNHEFAQIA"}, "011812": {"suoxie": "CTAHHHFQC", "name": "\u8d22\u901a\u5b89\u534e\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CAITONGANHUAHUNHEFAQIC"}, "011813": {"suoxie": "RTCXDLHHA", "name": "\u878d\u901a\u521b\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGCHUANGXINDONGLIHUNHEA"}, "011814": {"suoxie": "RTCXDLHHC", "name": "\u878d\u901a\u521b\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGCHUANGXINDONGLIHUNHEC"}, "011815": {"suoxie": "HYYSJXHH", "name": "\u6052\u8d8a\u4f18\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYOUSHIJINGXUANHUNHE"}, "011816": {"suoxie": "RTDYSYYNCYQHH", "name": "\u878d\u901a\u591a\u5143\u6536\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGDUOYUANSHOUYIYINIANCHIYOUQIHUNHE"}, "011817": {"suoxie": "YHAEFHH", "name": "\u94f6\u534e\u963f\u5c14\u6cd5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAAERFAHUNHE"}, "011818": {"suoxie": "PYJYYNCYHHA", "name": "\u9e4f\u626c\u666f\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGYANGYINIANCHIYOUHUNHEA"}, "011819": {"suoxie": "PYJYYNCYHHC", "name": "\u9e4f\u626c\u666f\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGYANGYINIANCHIYOUHUNHEC"}, "011820": {"suoxie": "XYXZYNCYQHHA", "name": "\u5174\u4e1a\u5174\u667a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXINGZHIYINIANCHIYOUQIHUNHEA"}, "011821": {"suoxie": "XYXZYNCYQHHC", "name": "\u5174\u4e1a\u5174\u667a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXINGZHIYINIANCHIYOUQIHUNHEC"}, "011822": {"suoxie": "YFDCYSJYNFBHHA", "name": "\u6613\u65b9\u8fbe\u4ea7\u4e1a\u5347\u7ea7\u4e00\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHANYESHENGJIYINIANFENGBIHUNHEA"}, "011823": {"suoxie": "YFDCYSJYNFBHHC", "name": "\u6613\u65b9\u8fbe\u4ea7\u4e1a\u5347\u7ea7\u4e00\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHANYESHENGJIYINIANFENGBIHUNHEC"}, "011824": {"suoxie": "ZSHJLHZXGPA", "name": "\u6d59\u5546\u6c47\u91d1\u91cf\u5316\u81fb\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGHUIJINLIANGHUAZHENXUANGUPIAOA"}, "011825": {"suoxie": "ZSHJLHZXGPC", "name": "\u6d59\u5546\u6c47\u91d1\u91cf\u5316\u81fb\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHESHANGHUIJINLIANGHUAZHENXUANGUPIAOC"}, "011826": {"suoxie": "HTFJKSHYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u5065\u5eb7\u751f\u6d3b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIANKANGSHENGHUOYINIANCHIYOUHUNHEA"}, "011827": {"suoxie": "HTFJKSHYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u5065\u5eb7\u751f\u6d3b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIANKANGSHENGHUOYINIANCHIYOUHUNHEC"}, "011828": {"suoxie": "PARXCZHHA", "name": "\u5e73\u5b89\u777f\u4eab\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANRUIXIANGCHENGZHANGHUNHEA"}, "011829": {"suoxie": "PARXCZHHC", "name": "\u5e73\u5b89\u777f\u4eab\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANRUIXIANGCHENGZHANGHUNHEC"}, "011830": {"suoxie": "FGTHHHA", "name": "\u5bcc\u56fd\u5929\u6052\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHENGHUNHEA"}, "011831": {"suoxie": "FGTHHHC", "name": "\u5bcc\u56fd\u5929\u6052\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHENGHUNHEC"}, "011832": {"suoxie": "XBLDRGZNZTZSZQA", "name": "\u897f\u90e8\u5229\u5f97\u4eba\u5de5\u667a\u80fd\u4e3b\u9898\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDERENGONGZHINENGZHUTIZHISHUZENGQIANGA"}, "011833": {"suoxie": "XBLDRGZNZTZSZQC", "name": "\u897f\u90e8\u5229\u5f97\u4eba\u5de5\u667a\u80fd\u4e3b\u9898\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDERENGONGZHINENGZHUTIZHISHUZENGQIANGC"}, "011834": {"suoxie": "DCTZYXLYCYHHA", "name": "\u5927\u6210\u6295\u8d44\u4e25\u9009\u516d\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGTOUZIYANXUANLIUYUECHIYOUHUNHEA"}, "011835": {"suoxie": "DCTZYXLYCYHHC", "name": "\u5927\u6210\u6295\u8d44\u4e25\u9009\u516d\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGTOUZIYANXUANLIUYUECHIYOUHUNHEC"}, "011836": {"suoxie": "YHZNJZGPFQS", "name": "\u94f6\u534e\u667a\u80fd\u5efa\u9020\u80a1\u7968\u53d1\u8d77\u5f0f", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAZHINENGJIANZAOGUPIAOFAQISHI"}, "011837": {"suoxie": "PYZGYZCZHHA", "name": "\u9e4f\u626c\u4e2d\u56fd\u4f18\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGZHONGGUOYOUZHICHENGZHANGHUNHEA"}, "011838": {"suoxie": "PYZGYZCZHHC", "name": "\u9e4f\u626c\u4e2d\u56fd\u4f18\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGZHONGGUOYOUZHICHENGZHANGHUNHEC"}, "011839": {"suoxie": "THZZRGZNA", "name": "\u5929\u5f18\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fdA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGRENGONGZHINENGA"}, "011840": {"suoxie": "THZZRGZNC", "name": "\u5929\u5f18\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fdC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGRENGONGZHINENGC"}, "011841": {"suoxie": "JSXRYXYNCYHHA", "name": "\u5609\u5b9e\u5174\u9510\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIXINGRUIYOUXUANYINIANCHIYOUHUNHEA"}, "011842": {"suoxie": "JSXRYXYNCYHHC", "name": "\u5609\u5b9e\u5174\u9510\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIXINGRUIYOUXUANYINIANCHIYOUHUNHEC"}, "011843": {"suoxie": "MSJYNHQDHHA", "name": "\u6c11\u751f\u52a0\u94f6\u5185\u6838\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINNEIHEQUDONGHUNHEA"}, "011844": {"suoxie": "MSJYNHQDHHC", "name": "\u6c11\u751f\u52a0\u94f6\u5185\u6838\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINNEIHEQUDONGHUNHEC"}, "011845": {"suoxie": "BSZQYXHHA", "name": "\u535a\u65f6\u5468\u671f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHOUQIYOUXUANHUNHEA"}, "011846": {"suoxie": "BSZQYXHHC", "name": "\u535a\u65f6\u5468\u671f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHOUQIYOUXUANHUNHEC"}, "011847": {"suoxie": "YFDSYMSYXHHA", "name": "\u6613\u65b9\u8fbe\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDASHANGYEMOSHIYOUXUANHUNHEA"}, "011848": {"suoxie": "YFDSYMSYXHHC", "name": "\u6613\u65b9\u8fbe\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDASHANGYEMOSHIYOUXUANHUNHEC"}, "011850": {"suoxie": "TZTX66GYDKZ", "name": "\u5929\u6cbb\u5929\u4eab66\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANZHITIANXIANG66GEYUEDINGKAIZHAI"}, "011851": {"suoxie": "THXJZZHHA", "name": "\u5929\u5f18\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGXIANJINZHIZAOHUNHEA"}, "011852": {"suoxie": "THXJZZHHC", "name": "\u5929\u5f18\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGXIANJINZHIZAOHUNHEC"}, "011853": {"suoxie": "ZSZZXFLTZSZQA", "name": "\u62db\u5546\u4e2d\u8bc1\u6d88\u8d39\u9f99\u5934\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXIAOFEILONGTOUZHISHUZENGQIANGA"}, "011854": {"suoxie": "ZSZZXFLTZSZQC", "name": "\u62db\u5546\u4e2d\u8bc1\u6d88\u8d39\u9f99\u5934\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXIAOFEILONGTOUZHISHUZENGQIANGC"}, "011855": {"suoxie": "YHCRHH", "name": "\u94f6\u534e\u957f\u8363\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUACHANGRONGHUNHE"}, "011856": {"suoxie": "AXJHCZ18GYCYHHA", "name": "\u5b89\u4fe1\u5747\u8861\u6210\u957f18\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJUNHENGCHENGZHANG18GEYUECHIYOUHUNHEA"}, "011857": {"suoxie": "AXJHCZ18GYCYHHC", "name": "\u5b89\u4fe1\u5747\u8861\u6210\u957f18\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJUNHENGCHENGZHANG18GEYUECHIYOUHUNHEC"}, "011858": {"suoxie": "AXXFSJYNCYHHA", "name": "\u5b89\u4fe1\u6d88\u8d39\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINXIAOFEISHENGJIYINIANCHIYOUHUNHEA"}, "011859": {"suoxie": "AXXFSJYNCYHHC", "name": "\u5b89\u4fe1\u6d88\u8d39\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINXIAOFEISHENGJIYINIANCHIYOUHUNHEC"}, "011860": {"suoxie": "NFZZ1000LJA", "name": "\u5357\u65b9\u4e2d\u8bc11000\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG1000LIANJIEA"}, "011861": {"suoxie": "NFZZ1000LJC", "name": "\u5357\u65b9\u4e2d\u8bc11000\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG1000LIANJIEC"}, "011862": {"suoxie": "NFLCCZHHA", "name": "\u5357\u65b9\u84dd\u7b79\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLANCHOUCHENGZHANGHUNHEA"}, "011863": {"suoxie": "NFLCCZHHC", "name": "\u5357\u65b9\u84dd\u7b79\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLANCHOUCHENGZHANGHUNHEC"}, "011864": {"suoxie": "BSHTZQA", "name": "\u535a\u65f6\u6052\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHENGTAIZHAIQUANA"}, "011865": {"suoxie": "BSHTZQC", "name": "\u535a\u65f6\u6052\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHENGTAIZHAIQUANC"}, "011866": {"suoxie": "GFJZZZHHA", "name": "\u5e7f\u53d1\u4ef7\u503c\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIZENGZHANGHUNHEA"}, "011867": {"suoxie": "GFJZZZHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHIZENGZHANGHUNHEC"}, "011868": {"suoxie": "ZXJTYJHBHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u8fdc\u89c1\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUYUANJIANHUIBAOHUNHEA"}, "011869": {"suoxie": "ZXJTYJHBHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u8fdc\u89c1\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUYUANJIANHUIBAOHUNHEC"}, "011870": {"suoxie": "QHKYGJBJYSHHC", "name": "\u524d\u6d77\u5f00\u6e90\u56fd\u5bb6\u6bd4\u8f83\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANGUOJIABIJIAOYOUSHIHUNHEC"}, "011871": {"suoxie": "QHKYHGSYSJXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u4f18\u52bf\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENYOUSHIJINGXUANHUNHEC"}, "011872": {"suoxie": "ZYYX6GYCYQHHA", "name": "\u4e2d\u90ae\u60a6\u4eab6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOUYUEXIANG6GEYUECHIYOUQIHUNHEA"}, "011873": {"suoxie": "ZYYX6GYCYQHHC", "name": "\u4e2d\u90ae\u60a6\u4eab6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYOUYUEXIANG6GEYUECHIYOUQIHUNHEC"}, "011874": {"suoxie": "BSXJZZHHA", "name": "\u535a\u65f6\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXIANJINZHIZAOHUNHEA"}, "011875": {"suoxie": "BSXJZZHHC", "name": "\u535a\u65f6\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXIANJINZHIZAOHUNHEC"}, "011876": {"suoxie": "JSCCYLJKHHA", "name": "\u666f\u987a\u957f\u57ce\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYILIAOJIANKANGHUNHEA"}, "011877": {"suoxie": "JSCCYLJKHHC", "name": "\u666f\u987a\u957f\u57ce\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYILIAOJIANKANGHUNHEC"}, "011879": {"suoxie": "BSXXXFZTHHC", "name": "\u535a\u65f6\u65b0\u5174\u6d88\u8d39\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINXINGXIAOFEIZHUTIHUNHEC"}, "011880": {"suoxie": "GTZZ15NZJZA", "name": "\u56fd\u6cf0\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIZHONGZHAI15NIANZHENGJINZHAIA"}, "011881": {"suoxie": "GTZZ15NZJZC", "name": "\u56fd\u6cf0\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIZHONGZHAI15NIANZHENGJINZHAIC"}, "011882": {"suoxie": "ZSLCJXGPA", "name": "\u62db\u5546\u84dd\u7b79\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGLANCHOUJINGXUANGUPIAOA"}, "011883": {"suoxie": "ZSLCJXGPC", "name": "\u62db\u5546\u84dd\u7b79\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGLANCHOUJINGXUANGUPIAOC"}, "011884": {"suoxie": "GYJQYXHHA", "name": "\u5de5\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINGQIYOUXUANHUNHEA"}, "011885": {"suoxie": "GYJQYXHHC", "name": "\u5de5\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINJINGQIYOUXUANHUNHEC"}, "011886": {"suoxie": "HYYFGDZZHHA", "name": "\u5f18\u6bc5\u8fdc\u65b9\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGAODUANZHIZAOHUNHEA"}, "011887": {"suoxie": "HYYFGDZZHHC", "name": "\u5f18\u6bc5\u8fdc\u65b9\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGAODUANZHIZAOHUNHEC"}, "011888": {"suoxie": "MSJYZQYXHHA", "name": "\u6c11\u751f\u52a0\u94f6\u5468\u671f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINZHOUQIYOUXUANHUNHEA"}, "011889": {"suoxie": "MSJYZQYXHHC", "name": "\u6c11\u751f\u52a0\u94f6\u5468\u671f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINZHOUQIYOUXUANHUNHEC"}, "011890": {"suoxie": "ZJXJ3GYDQKFZQ", "name": "\u4e2d\u91d1\u65b0\u749f3\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINXINJING3GEYUEDINGQIKAIFANGZHAIQUAN"}, "011891": {"suoxie": "YFDXFCZHHA", "name": "\u6613\u65b9\u8fbe\u5148\u950b\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAXIANFENGCHENGZHANGHUNHEA"}, "011892": {"suoxie": "YFDXFCZHHC", "name": "\u6613\u65b9\u8fbe\u5148\u950b\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAXIANFENGCHENGZHANGHUNHEC"}, "011893": {"suoxie": "YFDCQJZHHA", "name": "\u6613\u65b9\u8fbe\u957f\u671f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHANGQIJIAZHIHUNHEA"}, "011894": {"suoxie": "YFDCQJZHHC", "name": "\u6613\u65b9\u8fbe\u957f\u671f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHANGQIJIAZHIHUNHEC"}, "011895": {"suoxie": "BSYLBJHYHHC", "name": "\u535a\u65f6\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYILIAOBAOJIANHANGYEHUNHEC"}, "011897": {"suoxie": "CCYXHBZQA", "name": "\u957f\u57ce\u60a6\u4eab\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGYUEXIANGHUIBAOZHAIQUANA"}, "011898": {"suoxie": "CCYXHBZQC", "name": "\u957f\u57ce\u60a6\u4eab\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGYUEXIANGHUIBAOZHAIQUANC"}, "011899": {"suoxie": "CAXRKJ6GYDKHHA", "name": "\u957f\u5b89\u946b\u745e\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXINRUIKEJI6GEYUEDINGKAIHUNHEA"}, "011900": {"suoxie": "CAXRKJ6GYDKHHC", "name": "\u957f\u5b89\u946b\u745e\u79d1\u62806\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXINRUIKEJI6GEYUEDINGKAIHUNHEC"}, "011903": {"suoxie": "NFLHYXHHA", "name": "\u5357\u65b9\u9886\u822a\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLINGHANGYOUXUANHUNHEA"}, "011904": {"suoxie": "NFLHYXHHC", "name": "\u5357\u65b9\u9886\u822a\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLINGHANGYOUXUANHUNHEC"}, "011905": {"suoxie": "AXJZQHHHA", "name": "\u5b89\u4fe1\u4ef7\u503c\u542f\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIQIHANGHUNHEA"}, "011906": {"suoxie": "AXJZQHHHC", "name": "\u5b89\u4fe1\u4ef7\u503c\u542f\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIQIHANGHUNHEC"}, "011907": {"suoxie": "GTLHSYLHPZHHC", "name": "\u56fd\u6cf0\u91cf\u5316\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAILIANGHUASHOUYILINGHUOPEIZHIHUNHEC"}, "011908": {"suoxie": "GFHGSJZJXHHA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJIAZHIJINGXUANHUNHEA"}, "011909": {"suoxie": "GFHGSJZJXHHC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJIAZHIJINGXUANHUNHEC"}, "011910": {"suoxie": "NFZL3GYDKZQFQ", "name": "\u5357\u65b9\u81fb\u52293\u4e2a\u6708\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHENLI3GEYUEDINGKAIZHAIQUANFAQI"}, "011911": {"suoxie": "HXXFYXHHA", "name": "\u534e\u590f\u6d88\u8d39\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIAOFEIYOUXUANHUNHEA"}, "011912": {"suoxie": "HXXFYXHHC", "name": "\u534e\u590f\u6d88\u8d39\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIAOFEIYOUXUANHUNHEC"}, "011913": {"suoxie": "HXYHYNCYHHA", "name": "\u534e\u590f\u6c38\u6cd3\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGHONGYINIANCHIYOUHUNHEA"}, "011914": {"suoxie": "HXYHYNCYHHC", "name": "\u534e\u590f\u6c38\u6cd3\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGHONGYINIANCHIYOUHUNHEC"}, "011917": {"suoxie": "SXZQPZSHHHA", "name": "\u5c71\u897f\u8bc1\u5238\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANXIZHENGQUANPINZHISHENGHUOHUNHEA"}, "011918": {"suoxie": "SXZQPZSHHHC", "name": "\u5c71\u897f\u8bc1\u5238\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANXIZHENGQUANPINZHISHENGHUOHUNHEC"}, "011919": {"suoxie": "HYDZZQA", "name": "\u6052\u8d8a\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HENGYUEDUANZHAIZHAIQUANA"}, "011920": {"suoxie": "HYDZZQC", "name": "\u6052\u8d8a\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HENGYUEDUANZHAIZHAIQUANC"}, "011921": {"suoxie": "FGJHCZSNCYQHHA", "name": "\u5bcc\u56fd\u5747\u8861\u6210\u957f\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNHENGCHENGZHANGSANNIANCHIYOUQIHUNHEA"}, "011922": {"suoxie": "FGJHCZSNCYQHHC", "name": "\u5bcc\u56fd\u5747\u8861\u6210\u957f\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJUNHENGCHENGZHANGSANNIANCHIYOUQIHUNHEC"}, "011923": {"suoxie": "DCXFJXGPA", "name": "\u5927\u6210\u6d88\u8d39\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGXIAOFEIJINGXUANGUPIAOA"}, "011924": {"suoxie": "JSGGHLWCYHXZCA", "name": "\u5609\u5b9e\u6e2f\u80a1\u4e92\u8054\u7f51\u4ea7\u4e1a\u6838\u5fc3\u8d44\u4ea7A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIGANGGUHULIANWANGCHANYEHEXINZICHANA"}, "011925": {"suoxie": "JSGGHLWCYHXZCC", "name": "\u5609\u5b9e\u6e2f\u80a1\u4e92\u8054\u7f51\u4ea7\u4e1a\u6838\u5fc3\u8d44\u4ea7C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIGANGGUHULIANWANGCHANYEHEXINZICHANC"}, "011926": {"suoxie": "DCXFJXGPC", "name": "\u5927\u6210\u6d88\u8d39\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGXIAOFEIJINGXUANGUPIAOC"}, "011927": {"suoxie": "BSHYHBHHA", "name": "\u535a\u65f6\u6c47\u8a89\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIYUHUIBAOHUNHEA"}, "011928": {"suoxie": "BSHYHBHHC", "name": "\u535a\u65f6\u6c47\u8a89\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIYUHUIBAOHUNHEC"}, "011929": {"suoxie": "SWATWLCZYNDKZ", "name": "\u7533\u4e07\u5b89\u6cf0\u7a33\u5229\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANANTAIWENLICHUNZHAIYINIANDINGKAIZHAI"}, "011930": {"suoxie": "HXSDQYYNCYHHA", "name": "\u534e\u590f\u65f6\u4ee3\u524d\u6cbf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIASHIDAIQIANYANYINIANCHIYOUHUNHEA"}, "011931": {"suoxie": "HXSDQYYNCYHHC", "name": "\u534e\u590f\u65f6\u4ee3\u524d\u6cbf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIASHIDAIQIANYANYINIANCHIYOUHUNHEC"}, "011932": {"suoxie": "GYZLYJHHA", "name": "\u5de5\u94f6\u6218\u7565\u8fdc\u89c1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHANLUEYUANJIANHUNHEA"}, "011933": {"suoxie": "GYZLYJHHC", "name": "\u5de5\u94f6\u6218\u7565\u8fdc\u89c1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHANLUEYUANJIANHUNHEC"}, "011934": {"suoxie": "ZHLHAEFLGYCYA", "name": "\u4e2d\u822a\u91cf\u5316\u963f\u5c14\u6cd5\u516d\u4e2a\u6708\u6301\u6709A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGHANGLIANGHUAAERFALIUGEYUECHIYOUA"}, "011935": {"suoxie": "ZHLHAEFLGYCYC", "name": "\u4e2d\u822a\u91cf\u5316\u963f\u5c14\u6cd5\u516d\u4e2a\u6708\u6301\u6709C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGHANGLIANGHUAAERFALIUGEYUECHIYOUC"}, "011936": {"suoxie": "HXAEFJXHHA", "name": "\u534e\u590f\u963f\u5c14\u6cd5\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAAERFAJINGXUANHUNHEA"}, "011937": {"suoxie": "HXAEFJXHHC", "name": "\u534e\u590f\u963f\u5c14\u6cd5\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAAERFAJINGXUANHUNHEC"}, "011938": {"suoxie": "BSXNYQCZTHHA", "name": "\u535a\u65f6\u65b0\u80fd\u6e90\u6c7d\u8f66\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINNENGYUANQICHEZHUTIHUNHEA"}, "011939": {"suoxie": "BSXNYQCZTHHC", "name": "\u535a\u65f6\u65b0\u80fd\u6e90\u6c7d\u8f66\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINNENGYUANQICHEZHUTIHUNHEC"}, "011940": {"suoxie": "DCQQMYZQDIIAMY", "name": "\u5927\u6210\u5168\u7403\u7f8e\u5143\u503a(QDII)A\u7f8e\u5143", "style": "QDII", "pinyin": "DACHENGQUANQIUMEIYUANZHAIQDIIAMEIYUAN"}, "011941": {"suoxie": "DCQQMYZQDIICMY", "name": "\u5927\u6210\u5168\u7403\u7f8e\u5143\u503a(QDII)C\u7f8e\u5143", "style": "QDII", "pinyin": "DACHENGQUANQIUMEIYUANZHAIQDIICMEIYUAN"}, "011942": {"suoxie": "JXHLYNCYQZQ", "name": "\u5efa\u4fe1\u6cd3\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINHONGLIYINIANCHIYOUQIZHAIQUAN"}, "011943": {"suoxie": "GYRSYNDKCZZQFQS", "name": "\u5de5\u94f6\u745e\u76db\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUISHENGYINIANDINGKAICHUNZHAIZHAIQUANFAQISHI"}, "011944": {"suoxie": "ZSJRZ3GYDKZ", "name": "\u62db\u5546\u91d1\u878d\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGJINRONGZHAI3GEYUEDINGKAIZHAI"}, "011945": {"suoxie": "HTFWJZYYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u589e\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANZENGYIYINIANCHIYOUHUNHEC"}, "011946": {"suoxie": "JXYFLLZSGYDKZA", "name": "\u5efa\u4fe1\u88d5\u4e30\u5229\u7387\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINYUFENGLILVZHAISANGEYUEDINGKAIZHAIA"}, "011947": {"suoxie": "JXYFLLZSGYDKZC", "name": "\u5efa\u4fe1\u88d5\u4e30\u5229\u7387\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINYUFENGLILVZHAISANGEYUEDINGKAIZHAIC"}, "011948": {"suoxie": "DWZHYLLHHHC", "name": "\u4e1c\u5434\u667a\u6167\u533b\u7597\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUZHIHUIYILIAOLIANGHUAHUNHEC"}, "011949": {"suoxie": "DWDCLHHC", "name": "\u4e1c\u5434\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUDUOCELUEHUNHEC"}, "011950": {"suoxie": "JSWYCZZQC", "name": "\u5609\u5b9e\u7a33\u5143\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENYUANCHUNZHAIZHAIQUANC"}, "011951": {"suoxie": "GSABAHCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u5f18\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANHONGCHUNZHAIYINIANDINGKAIZHAI"}, "011952": {"suoxie": "WJYX3GYDKZQ", "name": "\u4e07\u5bb6\u60a6\u51743\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAYUEXING3GEYUEDINGKAIZHAIQUAN"}, "011953": {"suoxie": "ZSZRCZFQSD", "name": "\u62db\u5546\u62db\u745e\u7eaf\u503a\u53d1\u8d77\u5f0fD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAORUICHUNZHAIFAQISHID"}, "011954": {"suoxie": "GFHRSGYDKZQ", "name": "\u5e7f\u53d1\u6c47\u8363\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIRONGSANGEYUEDINGKAIZHAIQUAN"}, "011955": {"suoxie": "ZSZXCZD", "name": "\u62db\u5546\u62db\u7965\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOXIANGCHUNZHAID"}, "011956": {"suoxie": "PHXNYJXHHA", "name": "\u9e4f\u534e\u65b0\u80fd\u6e90\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINNENGYUANJINGXUANHUNHEA"}, "011957": {"suoxie": "PHXNYJXHHC", "name": "\u9e4f\u534e\u65b0\u80fd\u6e90\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINNENGYUANJINGXUANHUNHEC"}, "011958": {"suoxie": "BYZZ35NGKZZSA", "name": "\u5b9d\u76c8\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGZHONGZHAI35NIANGUOKAIZHAIZHISHUA"}, "011959": {"suoxie": "BYZZ35NGKZZSC", "name": "\u5b9d\u76c8\u4e2d\u503a3-5\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGZHONGZHAI35NIANGUOKAIZHAIZHISHUC"}, "011960": {"suoxie": "XYJFYNDKZQFQS", "name": "\u5174\u4e1a\u5609\u798f\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIAFUYINIANDINGKAIZHAIQUANFAQISHI"}, "011961": {"suoxie": "YFDWX30TGDCYDZA", "name": "\u6613\u65b9\u8fbe\u7a33\u946b30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENXIN30TIANGUNDONGCHIYOUDUANZHAIA"}, "011962": {"suoxie": "YFDWX30TGDCYDZC", "name": "\u6613\u65b9\u8fbe\u7a33\u946b30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENXIN30TIANGUNDONGCHIYOUDUANZHAIC"}, "011963": {"suoxie": "GFWYHHC", "name": "\u5e7f\u53d1\u7a33\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAWENYUHUNHEC"}, "011966": {"suoxie": "ZSZZGFCYZSA", "name": "\u62db\u5546\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGGUANGFUCHANYEZHISHUA"}, "011967": {"suoxie": "ZSZZGFCYZSC", "name": "\u62db\u5546\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGGUANGFUCHANYEZHISHUC"}, "011968": {"suoxie": "NYHLJSZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u76db\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINSHENGZHAIQUAN"}, "011969": {"suoxie": "JXGGTJXHHA", "name": "\u5efa\u4fe1\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINGANGGUTONGJINGXUANHUNHEA"}, "011970": {"suoxie": "JXGGTJXHHC", "name": "\u5efa\u4fe1\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINGANGGUTONGJINGXUANHUNHEC"}, "011971": {"suoxie": "DCYHA", "name": "\u4e1c\u8d22\u94f6\u884cA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYINHANGA"}, "011972": {"suoxie": "DCYHC", "name": "\u4e1c\u8d22\u94f6\u884cC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYINHANGC"}, "011973": {"suoxie": "XHZZ15NNFXA", "name": "\u65b0\u534e\u4e2d\u503a1-5\u5e74\u519c\u53d1\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAZHONGZHAI15NIANNONGFAXINGA"}, "011974": {"suoxie": "XHZZ15NNFXC", "name": "\u65b0\u534e\u4e2d\u503a1-5\u5e74\u519c\u53d1\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAZHONGZHAI15NIANNONGFAXINGC"}, "011975": {"suoxie": "GFJHHBHHA", "name": "\u5e7f\u53d1\u5747\u8861\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUNHENGHUIBAOHUNHEA"}, "011976": {"suoxie": "GFJHHBHHC", "name": "\u5e7f\u53d1\u5747\u8861\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUNHENGHUIBAOHUNHEC"}, "011977": {"suoxie": "GLYJYXHHA", "name": "\u683c\u6797\u7814\u7a76\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINYANJIUYOUXUANHUNHEA"}, "011978": {"suoxie": "GLYJYXHHC", "name": "\u683c\u6797\u7814\u7a76\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINYANJIUYOUXUANHUNHEC"}, "011979": {"suoxie": "ZYZZ15NZJZZSA", "name": "\u4e2d\u90ae\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUZHONGZHAI15NIANZHENGJINZHAIZHISHUA"}, "011980": {"suoxie": "GFJXJXHHA", "name": "\u56fd\u5bcc\u5320\u5fc3\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJIANGXINJINGXUANHUNHEA"}, "011981": {"suoxie": "GFJXJXHHC", "name": "\u56fd\u5bcc\u5320\u5fc3\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUJIANGXINJINGXUANHUNHEC"}, "011982": {"suoxie": "BSNXZZHHC", "name": "\u535a\u65f6\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHINEIXUZENGZHANGHUNHEC"}, "011983": {"suoxie": "YYZZ35NZJZZSA", "name": "\u6c38\u8d62\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGZHAI35NIANZHENGJINZHAIZHISHUA"}, "011984": {"suoxie": "YYZZ35NZJZZSC", "name": "\u6c38\u8d62\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGZHONGZHAI35NIANZHENGJINZHAIZHISHUC"}, "011985": {"suoxie": "SWLXHLCZZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u5408\u5229\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINHELICHUNZHAIZHAIQUANA"}, "011986": {"suoxie": "SWLXHLCZZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u5408\u5229\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINHELICHUNZHAIZHAIQUANC"}, "011987": {"suoxie": "CTZXHXHB6GYHHA", "name": "\u8d22\u901a\u667a\u9009\u6838\u5fc3\u56de\u62a56\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZHIXUANHEXINHUIBAO6GEYUEHUNHEA"}, "011988": {"suoxie": "CTZXHXHB6GYHHC", "name": "\u8d22\u901a\u667a\u9009\u6838\u5fc3\u56de\u62a56\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZHIXUANHEXINHUIBAO6GEYUEHUNHEC"}, "011989": {"suoxie": "HAXZWJYNCYQHHA", "name": "\u6c47\u5b89\u946b\u6cfd\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANXINZEWENJIANYINIANCHIYOUQIHUNHEA"}, "011990": {"suoxie": "HAXZWJYNCYQHHC", "name": "\u6c47\u5b89\u946b\u6cfd\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANXINZEWENJIANYINIANCHIYOUQIHUNHEC"}, "011991": {"suoxie": "HAHLYNCYQHHA", "name": "\u6c47\u5b89\u6cd3\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANHONGLIYINIANCHIYOUQIHUNHEA"}, "011992": {"suoxie": "HAHLYNCYQHHC", "name": "\u6c47\u5b89\u6cd3\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANHONGLIYINIANCHIYOUQIHUNHEC"}, "011993": {"suoxie": "ZYZZ15NZJZZSC", "name": "\u4e2d\u90ae\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUZHONGZHAI15NIANZHENGJINZHAIZHISHUC"}, "011994": {"suoxie": "GLAHXYSHH", "name": "\u56fd\u8054\u5b89\u6838\u5fc3\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHEXINYOUSHIHUNHE"}, "011995": {"suoxie": "GTCYHHA", "name": "\u56fd\u6cf0\u8bda\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAICHENGYIHUNHEA"}, "011996": {"suoxie": "GTCYHHC", "name": "\u56fd\u6cf0\u8bda\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAICHENGYIHUNHEC"}, "011997": {"suoxie": "JSCCAYHBYNCYHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u76c8\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANYINGHUIBAOYINIANCHIYOUHUNHEA"}, "011998": {"suoxie": "JSCCAYHBYNCYHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u76c8\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANYINGHUIBAOYINIANCHIYOUHUNHEC"}, "011999": {"suoxie": "FGAT90TGDCYDZA", "name": "\u5bcc\u56fd\u5b89\u6cf090\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUOANTAI90TIANGUNDONGCHIYOUDUANZHAIA"}, "012000": {"suoxie": "FGAT90TGDCYDZC", "name": "\u5bcc\u56fd\u5b89\u6cf090\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUOANTAI90TIANGUNDONGCHIYOUDUANZHAIC"}, "012001": {"suoxie": "ZTXYYNFBHHA", "name": "\u4e2d\u6cf0\u661f\u5b87\u4e00\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGYUYINIANFENGBIHUNHEA"}, "012002": {"suoxie": "ZTXYYNFBHHC", "name": "\u4e2d\u6cf0\u661f\u5b87\u4e00\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGYUYINIANFENGBIHUNHEC"}, "012003": {"suoxie": "ZSJZCZHHA", "name": "\u62db\u5546\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGJIAZHICHENGZHANGHUNHEA"}, "012004": {"suoxie": "ZSJZCZHHC", "name": "\u62db\u5546\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGJIAZHICHENGZHANGHUNHEC"}, "012005": {"suoxie": "XDAYHSHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6052\u76db\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINDAAOYINHENGSHENGHUNHEA"}, "012006": {"suoxie": "XDAYHSHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6052\u76db\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINDAAOYINHENGSHENGHUNHEC"}, "012007": {"suoxie": "WJRFLHPZHHC", "name": "\u4e07\u5bb6\u745e\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIARUIFULINGHUOPEIZHIHUNHEC"}, "012008": {"suoxie": "YFDWJHBYNHHA", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAWENJIANHUIBAOYINIANHUNHEA"}, "012009": {"suoxie": "YFDWJHBYNHHC", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAWENJIANHUIBAOYINIANHUNHEC"}, "012010": {"suoxie": "FGTXHB6GYCYHHA", "name": "\u5bcc\u56fd\u6cf0\u4eab\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTAIXIANGHUIBAO6GEYUECHIYOUHUNHEA"}, "012011": {"suoxie": "FGTXHB6GYCYHHC", "name": "\u5bcc\u56fd\u6cf0\u4eab\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTAIXIANGHUIBAO6GEYUECHIYOUHUNHEC"}, "012012": {"suoxie": "HFTRX3GYDKZQA", "name": "\u6d77\u5bcc\u901a\u745e\u51743\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIXING3GEYUEDINGKAIZHAIQUANA"}, "012013": {"suoxie": "HFTRX3GYDKZQC", "name": "\u6d77\u5bcc\u901a\u745e\u51743\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIXING3GEYUEDINGKAIZHAIQUANC"}, "012014": {"suoxie": "GYJR6GYCYHHA", "name": "\u5de5\u94f6\u805a\u6da66\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJURUN6GEYUECHIYOUHUNHEA"}, "012015": {"suoxie": "GYJR6GYCYHHC", "name": "\u5de5\u94f6\u805a\u6da66\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJURUN6GEYUECHIYOUHUNHEC"}, "012016": {"suoxie": "GTRYSC3GYDKZ", "name": "\u56fd\u6295\u745e\u94f6\u987a\u62103\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTOURUIYINSHUNCHENG3GEYUEDINGKAIZHAI"}, "012017": {"suoxie": "GTRYHXYNCYZQA", "name": "\u56fd\u6295\u745e\u94f6\u548c\u65ed\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINHEXUYINIANCHIYOUZHAIQUANA"}, "012018": {"suoxie": "GTRYHXYNCYZQC", "name": "\u56fd\u6295\u745e\u94f6\u548c\u65ed\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINHEXUYINIANCHIYOUZHAIQUANC"}, "012019": {"suoxie": "GTRYAZHHA", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u6cfd\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANZEHUNHEA"}, "012020": {"suoxie": "GTRYAZHHC", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u6cfd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANZEHUNHEC"}, "012021": {"suoxie": "GTRYAZHHA", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u667a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANZHIHUNHEA"}, "012022": {"suoxie": "GTRYAZHHC", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u667a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANZHIHUNHEC"}, "012023": {"suoxie": "XYJQHHA", "name": "\u5174\u4e1a\u805a\u4e7e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUQIANHUNHEA"}, "012024": {"suoxie": "XYJQHHC", "name": "\u5174\u4e1a\u805a\u4e7e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUQIANHUNHEC"}, "012025": {"suoxie": "XYJXHHA", "name": "\u5174\u4e1a\u805a\u5174\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUXINGHUNHEA"}, "012026": {"suoxie": "XYJXHHC", "name": "\u5174\u4e1a\u805a\u5174\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUXINGHUNHEC"}, "012027": {"suoxie": "GDAYYNCYQHHA", "name": "\u5149\u5927\u5b89\u9633\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAANYANGYINIANCHIYOUQIHUNHEA"}, "012028": {"suoxie": "GDAYYNCYQHHC", "name": "\u5149\u5927\u5b89\u9633\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDAANYANGYINIANCHIYOUQIHUNHEC"}, "012029": {"suoxie": "GFHXYNCYQHHA", "name": "\u5e7f\u53d1\u6052\u946b\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXINYINIANCHIYOUQIHUNHEA"}, "012030": {"suoxie": "GFHXYNCYQHHC", "name": "\u5e7f\u53d1\u6052\u946b\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXINYINIANCHIYOUQIHUNHEC"}, "012031": {"suoxie": "GDBDXCZZQA", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDABAODEXINCHUNZHAIZHAIQUANA"}, "012032": {"suoxie": "GDBDXCZZQC", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDABAODEXINCHUNZHAIZHAIQUANC"}, "012033": {"suoxie": "GFRSHHA", "name": "\u5e7f\u53d1\u777f\u76db\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUISHENGHUNHEA"}, "012034": {"suoxie": "GFRSHHC", "name": "\u5e7f\u53d1\u777f\u76db\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUISHENGHUNHEC"}, "012035": {"suoxie": "ZXJTWJYNDKFQSZQ", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u9a8f\u4e00\u5e74\u5b9a\u5f00\u53d1\u8d77\u5f0f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENJUNYINIANDINGKAIFAQISHIZHAIQUAN"}, "012036": {"suoxie": "NDXYYXYNCYHH", "name": "\u8bfa\u5fb7\u5174\u8fdc\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEXINGYUANYOUXUANYINIANCHIYOUHUNHE"}, "012037": {"suoxie": "ZSHHYLMB2040SNCYQHHFOF", "name": "\u62db\u5546\u548c\u60e0\u517b\u8001\u76ee\u68072040\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGHEHUIYANGLAOMUBIAO2040SANNIANCHIYOUQIHUNHEFOF"}, "012038": {"suoxie": "YHZYWJYLMBYNCYHHFQSFOF", "name": "\u94f6\u534e\u5c0a\u9890\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAZUNYIWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFAQISHIFOF"}, "012039": {"suoxie": "ZJ15NGKZZS", "name": "\u4e2d\u52a01-5\u5e74\u56fd\u5f00\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIA15NIANGUOKAIZHAIZHISHU"}, "012040": {"suoxie": "PHXXC", "name": "\u9e4f\u534e\u4fe1\u606fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAXINXIC"}, "012041": {"suoxie": "PHGFC", "name": "\u9e4f\u534e\u56fd\u9632C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOFANGC"}, "012042": {"suoxie": "PHYHC", "name": "\u9e4f\u534e\u94f6\u884cC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAYINHANGC"}, "012043": {"suoxie": "PHJC", "name": "\u9e4f\u534e\u9152C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAJIUC"}, "012044": {"suoxie": "PHQSC", "name": "\u9e4f\u534e\u5238\u5546C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAQUANSHANGC"}, "012045": {"suoxie": "DCYYJKGPA", "name": "\u5927\u6210\u533b\u836f\u5065\u5eb7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGYIYAOJIANKANGGUPIAOA"}, "012046": {"suoxie": "DCYYJKGPC", "name": "\u5927\u6210\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGYIYAOJIANKANGGUPIAOC"}, "012047": {"suoxie": "DCZXHHA", "name": "\u5927\u6210\u6d59\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGZHEXINHUNHEA"}, "012048": {"suoxie": "DCZXHHC", "name": "\u5927\u6210\u6d59\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENGZHEXINHUNHEC"}, "012049": {"suoxie": "THAYYNCYA", "name": "\u5929\u5f18\u5b89\u76c8\u4e00\u5e74\u6301\u6709A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGANYINGYINIANCHIYOUA"}, "012050": {"suoxie": "THAYYNCYC", "name": "\u5929\u5f18\u5b89\u76c8\u4e00\u5e74\u6301\u6709C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGANYINGYINIANCHIYOUC"}, "012051": {"suoxie": "SWLXLDSNCYQHH", "name": "\u7533\u4e07\u83f1\u4fe1\u4e50\u9053\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINLEDAOSANNIANCHIYOUQIHUNHE"}, "012052": {"suoxie": "CTZGXJY6GYCYHHFQSA", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u805a\u76ca6\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINJUYI6GEYUECHIYOUHUNHEFAQISHIA"}, "012053": {"suoxie": "CTZGXJY6GYCYHHFQSC", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u805a\u76ca6\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CAITONGZIGUANXINJUYI6GEYUECHIYOUHUNHEFAQISHIC"}, "012054": {"suoxie": "PHAKYNCYQHHA", "name": "\u9e4f\u534e\u5b89\u5eb7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANKANGYINIANCHIYOUQIHUNHEA"}, "012055": {"suoxie": "PHAKYNCYQHHC", "name": "\u9e4f\u534e\u5b89\u5eb7\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANKANGYINIANCHIYOUQIHUNHEC"}, "012056": {"suoxie": "HSJYYLSNCYHHFOF", "name": "\u534e\u5546\u5609\u60a6\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUASHANGJIAYUEYANGLAOSANNIANCHIYOUHUNHEFOF"}, "012057": {"suoxie": "PHPZCZHHA", "name": "\u9e4f\u534e\u54c1\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHICHENGZHANGHUNHEA"}, "012058": {"suoxie": "PHPZCZHHC", "name": "\u9e4f\u534e\u54c1\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHICHENGZHANGHUNHEC"}, "012059": {"suoxie": "PHYY3GYDKZ", "name": "\u9e4f\u534e\u6c38\u76ca3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGYI3GEYUEDINGKAIZHAI"}, "012060": {"suoxie": "FGQQXFJXHHQDIIRMBA", "name": "\u5bcc\u56fd\u5168\u7403\u6d88\u8d39\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "FUGUOQUANQIUXIAOFEIJINGXUANHUNHEQDIIRENMINBIA"}, "012061": {"suoxie": "FGQQXFJXHHQDIIMY", "name": "\u5bcc\u56fd\u5168\u7403\u6d88\u8d39\u7cbe\u9009\u6df7\u5408(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "FUGUOQUANQIUXIAOFEIJINGXUANHUNHEQDIIMEIYUAN"}, "012062": {"suoxie": "FGQQXFJXHHQDIIRMBC", "name": "\u5bcc\u56fd\u5168\u7403\u6d88\u8d39\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "FUGUOQUANQIUXIAOFEIJINGXUANHUNHEQDIIRENMINBIC"}, "012063": {"suoxie": "THZZ15NZJZ", "name": "\u5929\u5f18\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGZHONGZHAI15NIANZHENGJINZHAI"}, "012064": {"suoxie": "YXYFXNYNCYQHH", "name": "\u5706\u4fe1\u6c38\u4e30\u5174\u8bfa\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YUANXINYONGFENGXINGNUOYINIANCHIYOUQIHUNHE"}, "012065": {"suoxie": "JSMATFYNCYQHHA", "name": "\u5609\u5b9e\u6c11\u5b89\u6dfb\u590d\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIMINANTIANFUYINIANCHIYOUQIHUNHEA"}, "012066": {"suoxie": "JSMATFYNCYQHHC", "name": "\u5609\u5b9e\u6c11\u5b89\u6dfb\u590d\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIMINANTIANFUYINIANCHIYOUQIHUNHEC"}, "012067": {"suoxie": "JSLCYSHHA", "name": "\u5609\u5b9e\u84dd\u7b79\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILANCHOUYOUSHIHUNHEA"}, "012068": {"suoxie": "JSLCYSHHC", "name": "\u5609\u5b9e\u84dd\u7b79\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILANCHOUYOUSHIHUNHEC"}, "012069": {"suoxie": "THAKYX12GYCYA", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u4eab12\u4e2a\u6708\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIXIANG12GEYUECHIYOUA"}, "012070": {"suoxie": "THAKYX12GYCYC", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u4eab12\u4e2a\u6708\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIXIANG12GEYUECHIYOUC"}, "012071": {"suoxie": "ZJXLHBYNCYHHA", "name": "\u4e2d\u52a0\u559c\u5229\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXILIHUIBAOYINIANCHIYOUHUNHEA"}, "012072": {"suoxie": "ZJXLHBYNCYHHC", "name": "\u4e2d\u52a0\u559c\u5229\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXILIHUIBAOYINIANCHIYOUHUNHEC"}, "012073": {"suoxie": "HAJHYXHHA", "name": "\u534e\u5b89\u5747\u8861\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUNHENGYOUXUANHUNHEA"}, "012074": {"suoxie": "HAJHYXHHC", "name": "\u534e\u5b89\u5747\u8861\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUNHENGYOUXUANHUNHEC"}, "012075": {"suoxie": "YFDWJTLHHA", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAWENJIANTIANLIHUNHEA"}, "012076": {"suoxie": "YFDWJTLHHC", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAWENJIANTIANLIHUNHEC"}, "012077": {"suoxie": "YFDYXYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u590f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXIAYINIANCHIYOUHUNHEA"}, "012078": {"suoxie": "YFDYXYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u590f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEXIAYINIANCHIYOUHUNHEC"}, "012079": {"suoxie": "XDAYXNYJXHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u65b0\u80fd\u6e90\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINXINNENGYUANJINGXUANHUNHE"}, "012080": {"suoxie": "YFDZZ500ZSLHZQA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500\u6307\u6570\u91cf\u5316\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ZHISHULIANGHUAZENGQIANGA"}, "012081": {"suoxie": "YFDZZ500ZSLHZQC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500\u6307\u6570\u91cf\u5316\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ZHISHULIANGHUAZENGQIANGC"}, "012082": {"suoxie": "BSSZJJ18GYFBA", "name": "\u535a\u65f6\u6570\u5b57\u7ecf\u6d4e18\u4e2a\u6708\u5c01\u95edA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHUZIJINGJI18GEYUEFENGBIA"}, "012083": {"suoxie": "BSSZJJ18GYFBC", "name": "\u535a\u65f6\u6570\u5b57\u7ecf\u6d4e18\u4e2a\u6708\u5c01\u95edC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHUZIJINGJI18GEYUEFENGBIC"}, "012084": {"suoxie": "BSRHYNDKHHA", "name": "\u535a\u65f6\u777f\u5f18\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIRUIHONGYINIANDINGKAIHUNHEA"}, "012085": {"suoxie": "BSRHYNDKHHC", "name": "\u535a\u65f6\u777f\u5f18\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIRUIHONGYINIANDINGKAIHUNHEC"}, "012086": {"suoxie": "BSJKSHHHA", "name": "\u535a\u65f6\u5065\u5eb7\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIANKANGSHENGHUOHUNHEA"}, "012087": {"suoxie": "BSJKSHHHC", "name": "\u535a\u65f6\u5065\u5eb7\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIANKANGSHENGHUOHUNHEC"}, "012088": {"suoxie": "DFHJHZX18GYCYHHA", "name": "\u4e1c\u65b9\u7ea2\u9526\u548c\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINHEZHENXUAN18GEYUECHIYOUHUNHEA"}, "012089": {"suoxie": "DFHJHZX18GYCYHHC", "name": "\u4e1c\u65b9\u7ea2\u9526\u548c\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINHEZHENXUAN18GEYUECHIYOUHUNHEC"}, "012092": {"suoxie": "YHXYJXLNDKZ", "name": "\u94f6\u534e\u4fe1\u7528\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAXINYONGJINGXUANLIANGNIANDINGKAIZHAI"}, "012093": {"suoxie": "PHCXSJHHA", "name": "\u9e4f\u534e\u521b\u65b0\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINSHENGJIHUNHEA"}, "012094": {"suoxie": "PHCXSJHHC", "name": "\u9e4f\u534e\u521b\u65b0\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINSHENGJIHUNHEC"}, "012095": {"suoxie": "CXYHPHYLMBSNCYQHHFOF", "name": "\u957f\u4fe1\u9890\u548c\u5e73\u8861\u517b\u8001\u76ee\u6807\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGXINYIHEPINGHENGYANGLAOMUBIAOSANNIANCHIYOUQIHUNHEFOF"}, "012096": {"suoxie": "XYXDLHHA", "name": "\u946b\u5143\u946b\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANXINDONGLIHUNHEA"}, "012097": {"suoxie": "XYXDLHHC", "name": "\u946b\u5143\u946b\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANXINDONGLIHUNHEC"}, "012098": {"suoxie": "HXCZJHYNCYHH", "name": "\u534e\u590f\u6210\u957f\u673a\u4f1a\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGJIHUIYINIANCHIYOUHUNHE"}, "012099": {"suoxie": "HXWJZLGDCYZA", "name": "\u534e\u590f\u7a33\u5065\u589e\u5229\u6eda\u52a8\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAWENJIANZENGLIGUNDONGCHIYOUZHAIA"}, "012100": {"suoxie": "HXWJZLGDCYZC", "name": "\u534e\u590f\u7a33\u5065\u589e\u5229\u6eda\u52a8\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAWENJIANZENGLIGUNDONGCHIYOUZHAIC"}, "012101": {"suoxie": "ZJJHZQ", "name": "\u4e2d\u91d1\u91d1\u5408\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINHEZHAIQUAN"}, "012104": {"suoxie": "YYHBE", "name": "\u6c38\u8d62\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "YONGYINGHUOBIE"}, "012105": {"suoxie": "YYTTLHBE", "name": "\u6c38\u8d62\u5929\u5929\u5229\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "YONGYINGTIANTIANLIHUOBIE"}, "012106": {"suoxie": "GFATWJYLYNCYHHFOF", "name": "\u5e7f\u53d1\u5b89\u6cf0\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAANTAIWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "012107": {"suoxie": "HDRJSNCYQHHA", "name": "\u6cd3\u5fb7\u745e\u5609\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDERUIJIASANNIANCHIYOUQIHUNHEA"}, "012108": {"suoxie": "HDRJSNCYQHHC", "name": "\u6cd3\u5fb7\u745e\u5609\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDERUIJIASANNIANCHIYOUQIHUNHEC"}, "012109": {"suoxie": "RTWJTRLHPZHHA", "name": "\u878d\u901a\u7a33\u5065\u6dfb\u745e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGWENJIANTIANRUILINGHUOPEIZHIHUNHEA"}, "012110": {"suoxie": "RTWJTRLHPZHHC", "name": "\u878d\u901a\u7a33\u5065\u6dfb\u745e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGWENJIANTIANRUILINGHUOPEIZHIHUNHEC"}, "012111": {"suoxie": "PHAYHHA", "name": "\u9e4f\u534e\u5b89\u9890\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYIHUNHEA"}, "012112": {"suoxie": "PHAYHHC", "name": "\u9e4f\u534e\u5b89\u9890\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYIHUNHEC"}, "012113": {"suoxie": "RTWJZZYNCYQHHA", "name": "\u878d\u901a\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENJIANZENGZHANGYINIANCHIYOUQIHUNHEA"}, "012114": {"suoxie": "RTWJZZYNCYQHHC", "name": "\u878d\u901a\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENJIANZENGZHANGYINIANCHIYOUQIHUNHEC"}, "012115": {"suoxie": "ZSZFCZD", "name": "\u62db\u5546\u62db\u4e30\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOFENGCHUNZHAID"}, "012116": {"suoxie": "ZYCYBETFFQLJA", "name": "\u4e2d\u94f6\u521b\u4e1a\u677fETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINCHUANGYEBANETFFAQILIANJIEA"}, "012117": {"suoxie": "ZYCYBETFFQLJC", "name": "\u4e2d\u94f6\u521b\u4e1a\u677fETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINCHUANGYEBANETFFAQILIANJIEC"}, "012118": {"suoxie": "XQHYZQC", "name": "\u5174\u5168\u6052\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGQUANHENGYUZHAIQUANC"}, "012119": {"suoxie": "GYHXYSHHA", "name": "\u5de5\u94f6\u6838\u5fc3\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHEXINYOUSHIHUNHEA"}, "012120": {"suoxie": "GYHXYSHHC", "name": "\u5de5\u94f6\u6838\u5fc3\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHEXINYOUSHIHUNHEC"}, "012121": {"suoxie": "HXYRLGYCYHHA", "name": "\u534e\u590f\u6c38\u6da6\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGRUNLIUGEYUECHIYOUHUNHEA"}, "012122": {"suoxie": "HXYRLGYCYHHC", "name": "\u534e\u590f\u6c38\u6da6\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGRUNLIUGEYUECHIYOUHUNHEC"}, "012123": {"suoxie": "ZSJACZYX1NFBHH", "name": "\u62db\u5546\u91d1\u5b89\u6210\u957f\u4e25\u90091\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGJINANCHENGZHANGYANXUAN1NIANFENGBIHUNHE"}, "012124": {"suoxie": "BDSYHHA", "name": "\u535a\u9053\u76db\u5f66\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOSHENGYANHUNHEA"}, "012125": {"suoxie": "BDSYHHC", "name": "\u535a\u9053\u76db\u5f66\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOSHENGYANHUNHEC"}, "012126": {"suoxie": "TDHLXNYGPA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u80fd\u6e90\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIXINNENGYUANGUPIAOA"}, "012127": {"suoxie": "TDHLXNYGPC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u80fd\u6e90\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIXINNENGYUANGUPIAOC"}, "012128": {"suoxie": "HTFPBZGZYZ13NA", "name": "\u6c47\u6dfb\u5bcc\u5f6d\u535a\u4e2d\u56fd\u653f\u94f6\u503a1-3\u5e74A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUPENGBOZHONGGUOZHENGYINZHAI13NIANA"}, "012129": {"suoxie": "HTFPBZGZYZ13NC", "name": "\u6c47\u6dfb\u5bcc\u5f6d\u535a\u4e2d\u56fd\u653f\u94f6\u503a1-3\u5e74C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUPENGBOZHONGGUOZHENGYINZHAI13NIANC"}, "012130": {"suoxie": "JSCCXJZZHHA", "name": "\u666f\u987a\u957f\u57ce\u5148\u8fdb\u667a\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGXIANJINZHIZAOHUNHEA"}, "012131": {"suoxie": "JSCCXJZZHHC", "name": "\u666f\u987a\u957f\u57ce\u5148\u8fdb\u667a\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGXIANJINZHIZAOHUNHEC"}, "012132": {"suoxie": "HTBXJZCZA", "name": "\u534e\u6cf0\u4fdd\u5174\u4ef7\u503c\u6210\u957fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJIAZHICHENGZHANGA"}, "012133": {"suoxie": "ZSTHCZD", "name": "\u62db\u5546\u6dfb\u6d69\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANHAOCHUNZHAID"}, "012134": {"suoxie": "ZYJX3GYDKZ", "name": "\u4e2d\u94f6\u5609\u4eab3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINJIAXIANG3GEYUEDINGKAIZHAI"}, "012135": {"suoxie": "PHAY5GYCYQHHC", "name": "\u9e4f\u534e\u5b89\u88d55\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAANYU5GEYUECHIYOUQIHUNHEC"}, "012136": {"suoxie": "JSCCJTXLCZC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u946b\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIXINLICHUNZHAIC"}, "012138": {"suoxie": "JSCCAYHBYNCYHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u76ca\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANYIHUIBAOYINIANCHIYOUHUNHEA"}, "012139": {"suoxie": "JSCCAYHBYNCYHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u76ca\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANYIHUIBAOYINIANCHIYOUHUNHEC"}, "012140": {"suoxie": "TPFTYNDKZQFQS", "name": "\u592a\u5e73\u4e30\u6cf0\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIPINGFENGTAIYINIANDINGKAIZHAIQUANFAQISHI"}, "012143": {"suoxie": "XWNXZZHHA", "name": "\u65b0\u6c83\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINWONEIXUZENGZHANGHUNHEA"}, "012144": {"suoxie": "XWNXZZHHC", "name": "\u65b0\u6c83\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINWONEIXUZENGZHANGHUNHEC"}, "012145": {"suoxie": "ZOWN9GYCYZQA", "name": "\u4e2d\u6b27\u7a33\u5b819\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUWENNING9GEYUECHIYOUZHAIQUANA"}, "012146": {"suoxie": "ZOWN9GYCYZQC", "name": "\u4e2d\u6b27\u7a33\u5b819\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUWENNING9GEYUECHIYOUZHAIQUANC"}, "012147": {"suoxie": "FGDPHXZCHH", "name": "\u5bcc\u56fd\u5927\u76d8\u6838\u5fc3\u8d44\u4ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODAPANHEXINZICHANHUNHE"}, "012148": {"suoxie": "GTRYCYQSHHA", "name": "\u56fd\u6295\u745e\u94f6\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHANYEQUSHIHUNHEA"}, "012149": {"suoxie": "GTRYCYQSHHC", "name": "\u56fd\u6295\u745e\u94f6\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHANYEQUSHIHUNHEC"}, "012150": {"suoxie": "NDJZFXYNCYHH", "name": "\u8bfa\u5fb7\u4ef7\u503c\u53d1\u73b0\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEJIAZHIFAXIANYINIANCHIYOUHUNHE"}, "012151": {"suoxie": "HTBRZZGGT50ETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGGANGGUTONG50ETFLIANJIEA"}, "012152": {"suoxie": "HTBRZZGGT50ETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGGANGGUTONG50ETFLIANJIEC"}, "012153": {"suoxie": "BSYJHXHHA", "name": "\u535a\u65f6\u7814\u7a76\u6167\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUHUIXUANHUNHEA"}, "012154": {"suoxie": "BSYJHXHHC", "name": "\u535a\u65f6\u7814\u7a76\u6167\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUHUIXUANHUNHEC"}, "012155": {"suoxie": "HTFCZXFLGYCYHHA", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u5148\u950b\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGXIANFENGLIUGEYUECHIYOUHUNHEA"}, "012156": {"suoxie": "HTFCZXFLGYCYHHC", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u5148\u950b\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGXIANFENGLIUGEYUECHIYOUHUNHEC"}, "012157": {"suoxie": "HTFSZ50JBMZQZSA", "name": "\u6c47\u6dfb\u5bcc\u4e0a\u8bc150\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUSHANGZHENG50JIBENMIANZENGQIANGZHISHUA"}, "012158": {"suoxie": "HTFSZ50JBMZQZSC", "name": "\u6c47\u6dfb\u5bcc\u4e0a\u8bc150\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUSHANGZHENG50JIBENMIANZENGQIANGZHISHUC"}, "012159": {"suoxie": "CTZGJKCYHHA", "name": "\u8d22\u901a\u8d44\u7ba1\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIANKANGCHANYEHUNHEA"}, "012160": {"suoxie": "CTZGJKCYHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIANKANGCHANYEHUNHEC"}, "012161": {"suoxie": "AXZXYNCYHHA", "name": "\u5b89\u4fe1\u62db\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINZHAOXINYINIANCHIYOUHUNHEA"}, "012162": {"suoxie": "AXZXYNCYHHC", "name": "\u5b89\u4fe1\u62db\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINZHAOXINYINIANCHIYOUHUNHEC"}, "012165": {"suoxie": "GYPBGKZ13NZSE", "name": "\u5de5\u94f6\u5f6d\u535a\u56fd\u5f00\u503a1-3\u5e74\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINPENGBOGUOKAIZHAI13NIANZHISHUE"}, "012166": {"suoxie": "GY13NNFZZSE", "name": "\u5de5\u94f61-3\u5e74\u519c\u53d1\u503a\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANNONGFAZHAIZHISHUE"}, "012167": {"suoxie": "PYASYXWJYLMBYNCYHHFOF", "name": "\u6d66\u94f6\u5b89\u76db\u9890\u4eab\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGYIXIANGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "012168": {"suoxie": "GYZZ15NJCKHE", "name": "\u5de5\u94f6\u4e2d\u503a1-5\u5e74\u8fdb\u51fa\u53e3\u884cE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINZHONGZHAI15NIANJINCHUKOUHANGE"}, "012169": {"suoxie": "GY35NGKZZSE", "name": "\u5de5\u94f63-5\u5e74\u56fd\u5f00\u503a\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN35NIANGUOKAIZHAIZHISHUE"}, "012170": {"suoxie": "HXYSYNCYHHA", "name": "\u534e\u590f\u6c38\u987a\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGSHUNYINIANCHIYOUHUNHEA"}, "012171": {"suoxie": "HXYSYNCYHHC", "name": "\u534e\u590f\u6c38\u987a\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGSHUNYINIANCHIYOUHUNHEC"}, "012172": {"suoxie": "GY13NGKZZSE", "name": "\u5de5\u94f61-3\u5e74\u56fd\u5f00\u503a\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYIN13NIANGUOKAIZHAIZHISHUE"}, "012173": {"suoxie": "GTXZYXYNCYQHHA", "name": "\u56fd\u6cf0\u5174\u6cfd\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIXINGZEYOUXUANYINIANCHIYOUQIHUNHEA"}, "012174": {"suoxie": "GTXZYXYNCYQHHC", "name": "\u56fd\u6cf0\u5174\u6cfd\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIXINGZEYOUXUANYINIANCHIYOUQIHUNHEC"}, "012175": {"suoxie": "YFDWJZLHHA", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAWENJIANZENGLIHUNHEA"}, "012176": {"suoxie": "YFDWJZLHHC", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAWENJIANZENGLIHUNHEC"}, "012177": {"suoxie": "HTBXJZCZC", "name": "\u534e\u6cf0\u4fdd\u5174\u4ef7\u503c\u6210\u957fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJIAZHICHENGZHANGC"}, "012178": {"suoxie": "YHFRJXSNCYQHH", "name": "\u94f6\u534e\u5bcc\u9976\u7cbe\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFURAOJINGXUANSANNIANCHIYOUQIHUNHE"}, "012179": {"suoxie": "PYASCYBETFLJA", "name": "\u6d66\u94f6\u5b89\u76db\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGCHUANGYEBANETFLIANJIEA"}, "012180": {"suoxie": "PYASCYBETFLJC", "name": "\u6d66\u94f6\u5b89\u76db\u521b\u4e1a\u677fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGCHUANGYEBANETFLIANJIEC"}, "012181": {"suoxie": "ZYZNZZGPC", "name": "\u4e2d\u94f6\u667a\u80fd\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINZHINENGZHIZAOGUPIAOC"}, "012182": {"suoxie": "GFHGSJXHHA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJINGXUANHUNHEA"}, "012183": {"suoxie": "GFHGSJXHHC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENJINGXUANHUNHEC"}, "012184": {"suoxie": "DCCXQSHHA", "name": "\u5927\u6210\u521b\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHUANGXINQUSHIHUNHEA"}, "012185": {"suoxie": "DCCXQSHHC", "name": "\u5927\u6210\u521b\u65b0\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHUANGXINQUSHIHUNHEC"}, "012186": {"suoxie": "ZSPZCZHHA", "name": "\u62db\u5546\u54c1\u8d28\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHICHENGZHANGHUNHEA"}, "012187": {"suoxie": "ZSPZCZHHC", "name": "\u62db\u5546\u54c1\u8d28\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHICHENGZHANGHUNHEC"}, "012188": {"suoxie": "HAYSLTHHA", "name": "\u534e\u5b89\u4f18\u52bf\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHILONGTOUHUNHEA"}, "012189": {"suoxie": "HAYSLTHHC", "name": "\u534e\u5b89\u4f18\u52bf\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHILONGTOUHUNHEC"}, "012190": {"suoxie": "HTFTFYHWJYLYNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u798f\u76c8\u548c\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUTIANFUYINGHEWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "012191": {"suoxie": "ZYHT9GYCYQZQA", "name": "\u4e2d\u94f6\u6052\u6cf09\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINHENGTAI9GEYUECHIYOUQIZHAIQUANA"}, "012192": {"suoxie": "ZYHT9GYCYQZQC", "name": "\u4e2d\u94f6\u6052\u6cf09\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINHENGTAI9GEYUECHIYOUQIZHAIQUANC"}, "012193": {"suoxie": "HDRCHHA", "name": "\u6cd3\u5fb7\u777f\u8bda\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDERUICHENGHUNHEA"}, "012194": {"suoxie": "HDRCHHC", "name": "\u6cd3\u5fb7\u777f\u8bda\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGDERUICHENGHUNHEC"}, "012195": {"suoxie": "WJRZHBYNCYHH", "name": "\u4e07\u5bb6\u745e\u6cfd\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIARUIZEHUIBAOYINIANCHIYOUHUNHE"}, "012196": {"suoxie": "ZSPZSHHHA", "name": "\u62db\u5546\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHISHENGHUOHUNHEA"}, "012197": {"suoxie": "ZSPZSHHHC", "name": "\u62db\u5546\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHISHENGHUOHUNHEC"}, "012198": {"suoxie": "GJHXZCYNCYA", "name": "\u56fd\u91d1\u6838\u5fc3\u8d44\u4ea7\u4e00\u5e74\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINHEXINZICHANYINIANCHIYOUA"}, "012199": {"suoxie": "GJHXZCYNCYC", "name": "\u56fd\u91d1\u6838\u5fc3\u8d44\u4ea7\u4e00\u5e74\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINHEXINZICHANYINIANCHIYOUC"}, "012200": {"suoxie": "XHXKJ3GYLHPZHHA", "name": "\u65b0\u534e\u946b\u79d1\u62803\u4e2a\u6708\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINKEJI3GEYUELINGHUOPEIZHIHUNHEA"}, "012201": {"suoxie": "XHXKJ3GYLHPZHHC", "name": "\u65b0\u534e\u946b\u79d1\u62803\u4e2a\u6708\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINKEJI3GEYUELINGHUOPEIZHIHUNHEC"}, "012202": {"suoxie": "ZJXFYXHHA", "name": "\u4e2d\u52a0\u6d88\u8d39\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXIAOFEIYOUXUANHUNHEA"}, "012203": {"suoxie": "ZJXFYXHHC", "name": "\u4e2d\u52a0\u6d88\u8d39\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIAXIAOFEIYOUXUANHUNHEC"}, "012204": {"suoxie": "ZYTLZQA", "name": "\u4e2d\u94f6\u901a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINTONGLIZHAIQUANA"}, "012205": {"suoxie": "ZYTLZQC", "name": "\u4e2d\u94f6\u901a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINTONGLIZHAIQUANC"}, "012206": {"suoxie": "ZTHS300LHYXZQA", "name": "\u4e2d\u6cf0\u6caa\u6df1300\u91cf\u5316\u4f18\u9009\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIHUSHEN300LIANGHUAYOUXUANZENGQIANGA"}, "012207": {"suoxie": "ZTHS300LHYXZQC", "name": "\u4e2d\u6cf0\u6caa\u6df1300\u91cf\u5316\u4f18\u9009\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGTAIHUSHEN300LIANGHUAYOUXUANZENGQIANGC"}, "012208": {"suoxie": "HXGGQYJJHHQDIIA", "name": "\u534e\u590f\u6e2f\u80a1\u524d\u6cbf\u7ecf\u6d4e\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "HUAXIAGANGGUQIANYANJINGJIHUNHEQDIIA"}, "012209": {"suoxie": "HXGGQYJJHHQDIIC", "name": "\u534e\u590f\u6e2f\u80a1\u524d\u6cbf\u7ecf\u6d4e\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "HUAXIAGANGGUQIANYANJINGJIHUNHEQDIIC"}, "012210": {"suoxie": "SWLXZNQCGPA", "name": "\u7533\u4e07\u83f1\u4fe1\u667a\u80fd\u6c7d\u8f66\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINZHINENGQICHEGUPIAOA"}, "012211": {"suoxie": "SWLXZNQCGPC", "name": "\u7533\u4e07\u83f1\u4fe1\u667a\u80fd\u6c7d\u8f66\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINZHINENGQICHEGUPIAOC"}, "012212": {"suoxie": "THZZGDZBZZZQA", "name": "\u5929\u5f18\u4e2d\u8bc1\u9ad8\u7aef\u88c5\u5907\u5236\u9020\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGGAODUANZHUANGBEIZHIZAOZENGQIANGA"}, "012213": {"suoxie": "THZZGDZBZZZQC", "name": "\u5929\u5f18\u4e2d\u8bc1\u9ad8\u7aef\u88c5\u5907\u5236\u9020\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGGAODUANZHUANGBEIZHIZAOZENGQIANGC"}, "012214": {"suoxie": "MSJYHXZCGPA", "name": "\u6c11\u751f\u52a0\u94f6\u6838\u5fc3\u8d44\u4ea7\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINHEXINZICHANGUPIAOA"}, "012215": {"suoxie": "MSJYHXZCGPC", "name": "\u6c11\u751f\u52a0\u94f6\u6838\u5fc3\u8d44\u4ea7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINHEXINZICHANGUPIAOC"}, "012216": {"suoxie": "HTHTSLHHA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGLIHUNHEA"}, "012217": {"suoxie": "HTHTSLHHC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGLIHUNHEC"}, "012218": {"suoxie": "BSLXHHA", "name": "\u535a\u65f6\u4e50\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHILEXIANGHUNHEA"}, "012219": {"suoxie": "BSLXHHC", "name": "\u535a\u65f6\u4e50\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHILEXIANGHUNHEC"}, "012220": {"suoxie": "NFATHHC", "name": "\u5357\u65b9\u5b89\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGANTAIHUNHEC"}, "012221": {"suoxie": "RDHYLDHHA", "name": "\u745e\u8fbe\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIDAHANGYELUNDONGHUNHEA"}, "012222": {"suoxie": "RDHYLDHHC", "name": "\u745e\u8fbe\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIDAHANGYELUNDONGHUNHEC"}, "012223": {"suoxie": "XDAYCZJXHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINCHENGZHANGJINGXUANHUNHEA"}, "012224": {"suoxie": "XDAYCZJXHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINCHENGZHANGJINGXUANHUNHEC"}, "012225": {"suoxie": "JSYSJXHHA", "name": "\u5609\u5b9e\u4f18\u52bf\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUSHIJINGXUANHUNHEA"}, "012226": {"suoxie": "JSYSJXHHC", "name": "\u5609\u5b9e\u4f18\u52bf\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUSHIJINGXUANHUNHEC"}, "012227": {"suoxie": "JSCCGGTQQJZLA", "name": "\u666f\u987a\u957f\u57ce\u6e2f\u80a1\u901a\u5168\u7403\u7ade\u4e89\u529bA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGGANGGUTONGQUANQIUJINGZHENGLIA"}, "012228": {"suoxie": "JSCCGGTQQJZLC", "name": "\u666f\u987a\u957f\u57ce\u6e2f\u80a1\u901a\u5168\u7403\u7ade\u4e89\u529bC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGGANGGUTONGQUANQIUJINGZHENGLIC"}, "012229": {"suoxie": "HAZX90TGDDZA", "name": "\u534e\u5b89\u4f17\u946b90\u5929\u6eda\u52a8\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANZHONGXIN90TIANGUNDONGDUANZHAIA"}, "012230": {"suoxie": "HAZX90TGDDZC", "name": "\u534e\u5b89\u4f17\u946b90\u5929\u6eda\u52a8\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANZHONGXIN90TIANGUNDONGDUANZHAIC"}, "012233": {"suoxie": "ZSAYZQC", "name": "\u62db\u5546\u5b89\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANYINGZHAIQUANC"}, "012234": {"suoxie": "HAJHJXHHA", "name": "\u534e\u5b89\u805a\u5f18\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUHONGJINGXUANHUNHEA"}, "012235": {"suoxie": "HAJHJXHHC", "name": "\u534e\u5b89\u805a\u5f18\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJUHONGJINGXUANHUNHEC"}, "012236": {"suoxie": "ZYCXZZHHC", "name": "\u4e2d\u94f6\u6301\u7eed\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCHIXUZENGZHANGHUNHEC"}, "012237": {"suoxie": "GYXJZLHPZHHC", "name": "\u5de5\u94f6\u65b0\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINXINJIAZHILINGHUOPEIZHIHUNHEC"}, "012238": {"suoxie": "GYYLCYGPC", "name": "\u5de5\u94f6\u517b\u8001\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINYANGLAOCHANYEGUPIAOC"}, "012239": {"suoxie": "HSYSQYYNCYQHH", "name": "\u60e0\u5347\u4f18\u52bf\u4f01\u4e1a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGYOUSHIQIYEYINIANCHIYOUQIHUNHE"}, "012240": {"suoxie": "ZOXYZQ", "name": "\u4e2d\u6b27\u5174\u60a6\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINGYUEZHAIQUAN"}, "012241": {"suoxie": "GYLHCLHHC", "name": "\u5de5\u94f6\u91cf\u5316\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINLIANGHUACELUEHUNHEC"}, "012242": {"suoxie": "HATRZDZ", "name": "\u534e\u5b89\u6dfb\u8363\u4e2d\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANTIANRONGZHONGDUANZHAI"}, "012243": {"suoxie": "DFHNXZZHHB", "name": "\u4e1c\u65b9\u7ea2\u5185\u9700\u589e\u957f\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGNEIXUZENGZHANGHUNHEB"}, "012244": {"suoxie": "GFJRDCJXGPA", "name": "\u5e7f\u53d1\u91d1\u878d\u5730\u4ea7\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAJINRONGDICHANJINGXUANGUPIAOA"}, "012245": {"suoxie": "GFJRDCJXGPC", "name": "\u5e7f\u53d1\u91d1\u878d\u5730\u4ea7\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAJINRONGDICHANJINGXUANGUPIAOC"}, "012246": {"suoxie": "BSYYX30TCYQDZA", "name": "\u535a\u65f6\u6708\u6708\u4eab30\u5929\u6301\u6709\u671f\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BOSHIYUEYUEXIANG30TIANCHIYOUQIDUANZHAIA"}, "012247": {"suoxie": "BSYYX30TCYQDZC", "name": "\u535a\u65f6\u6708\u6708\u4eab30\u5929\u6301\u6709\u671f\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BOSHIYUEYUEXIANG30TIANCHIYOUQIDUANZHAIC"}, "012248": {"suoxie": "DCHXXSYNDKHHA", "name": "\u5927\u6210\u6052\u4eab\u590f\u76db\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHENGXIANGXIASHENGYINIANDINGKAIHUNHEA"}, "012249": {"suoxie": "DCHXXSYNDKHHC", "name": "\u5927\u6210\u6052\u4eab\u590f\u76db\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGHENGXIANGXIASHENGYINIANDINGKAIHUNHEC"}, "012250": {"suoxie": "AXPHZLHHA", "name": "\u5b89\u4fe1\u5e73\u8861\u589e\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINPINGHENGZENGLIHUNHEA"}, "012251": {"suoxie": "AXPHZLHHC", "name": "\u5b89\u4fe1\u5e73\u8861\u589e\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ANXINPINGHENGZENGLIHUNHEC"}, "012252": {"suoxie": "AXHY18GYCYHH", "name": "\u5b89\u4fe1\u5b8f\u76c818\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINHONGYING18GEYUECHIYOUHUNHE"}, "012253": {"suoxie": "PYJRYNCYHHA", "name": "\u9e4f\u626c\u666f\u6da6\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGRUNYINIANCHIYOUHUNHEA"}, "012254": {"suoxie": "PYJRYNCYHHC", "name": "\u9e4f\u626c\u666f\u6da6\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGRUNYINIANCHIYOUHUNHEC"}, "012255": {"suoxie": "FGGZLHH", "name": "\u5bcc\u56fd\u9ad8\u8d28\u91cf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGAOZHILIANGHUNHE"}, "012256": {"suoxie": "AXFSYNCYHHA", "name": "\u5b89\u4fe1\u4e30\u7a57\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINFENGSUIYINIANCHIYOUHUNHEA"}, "012257": {"suoxie": "AXFSYNCYHHC", "name": "\u5b89\u4fe1\u4e30\u7a57\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINFENGSUIYINIANCHIYOUHUNHEC"}, "012258": {"suoxie": "THXYCZHHA", "name": "\u5929\u5f18\u946b\u60a6\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGXINYUECHENGZHANGHUNHEA"}, "012259": {"suoxie": "THXYCZHHC", "name": "\u5929\u5f18\u946b\u60a6\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGXINYUECHENGZHANGHUNHEC"}, "012260": {"suoxie": "GFRMYZQYHHA", "name": "\u5e7f\u53d1\u777f\u660e\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIMINGYOUZHIQIYEHUNHEA"}, "012261": {"suoxie": "GFRMYZQYHHC", "name": "\u5e7f\u53d1\u777f\u660e\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIMINGYOUZHIQIYEHUNHEC"}, "012262": {"suoxie": "HBKCXFZHHA", "name": "\u534e\u5b9d\u53ef\u6301\u7eed\u53d1\u5c55\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOKECHIXUFAZHANHUNHEA"}, "012263": {"suoxie": "HBKCXFZHHC", "name": "\u534e\u5b9d\u53ef\u6301\u7eed\u53d1\u5c55\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOKECHIXUFAZHANHUNHEC"}, "012264": {"suoxie": "ZYYJJXLHPZHHC", "name": "\u4e2d\u94f6\u7814\u7a76\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYANJIUJINGXUANLINGHUOPEIZHIHUNHEC"}, "012265": {"suoxie": "THAY30TGDCYDZ", "name": "\u5929\u5f18\u5b89\u602130\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGANYI30TIANGUNDONGCHIYOUDUANZHAI"}, "012266": {"suoxie": "ZTWGZZG12ZGDZA", "name": "\u4e2d\u6cf0\u7a33\u56fa\u5468\u5468\u8d2d12\u5468\u6eda\u52a8\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGTAIWENGUZHOUZHOUGOU12ZHOUGUNDONGZHAIA"}, "012267": {"suoxie": "ZTWGZZG12ZGDZC", "name": "\u4e2d\u6cf0\u7a33\u56fa\u5468\u5468\u8d2d12\u5468\u6eda\u52a8\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGTAIWENGUZHOUZHOUGOU12ZHOUGUNDONGZHAIC"}, "012268": {"suoxie": "ZSZDXWJHHFQSA", "name": "\u6d59\u5546\u667a\u591a\u4eab\u7a33\u5065\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOXIANGWENJIANHUNHEFAQISHIA"}, "012269": {"suoxie": "ZSZDXWJHHFQSC", "name": "\u6d59\u5546\u667a\u591a\u4eab\u7a33\u5065\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHESHANGZHIDUOXIANGWENJIANHUNHEFAQISHIC"}, "012270": {"suoxie": "FGTXHB6GYGDCYA", "name": "\u5bcc\u56fd\u817e\u4eab\u56de\u62a56\u4e2a\u6708\u6eda\u52a8\u6301\u6709A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTENGXIANGHUIBAO6GEYUEGUNDONGCHIYOUA"}, "012271": {"suoxie": "FGTXHB6GYGDCYC", "name": "\u5bcc\u56fd\u817e\u4eab\u56de\u62a56\u4e2a\u6708\u6eda\u52a8\u6301\u6709C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOTENGXIANGHUIBAO6GEYUEGUNDONGCHIYOUC"}, "012272": {"suoxie": "BHHJCXJZYNCYHH", "name": "\u6e24\u6d77\u6c47\u91d1\u521b\u65b0\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOHAIHUIJINCHUANGXINJIAZHIYINIANCHIYOUHUNHE"}, "012273": {"suoxie": "FGHXJRZSGYDKZA", "name": "\u5bcc\u56fd\u6c47\u946b\u91d1\u878d\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHUIXINJINRONGZHAISANGEYUEDINGKAIZHAIA"}, "012274": {"suoxie": "FGHXJRZSGYDKZC", "name": "\u5bcc\u56fd\u6c47\u946b\u91d1\u878d\u503a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOHUIXINJINRONGZHAISANGEYUEDINGKAIZHAIC"}, "012275": {"suoxie": "FGZZHGS500ETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHUGANGSHEN500ETFLIANJIEA"}, "012276": {"suoxie": "FGZZHGS500ETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHUGANGSHEN500ETFLIANJIEC"}, "012277": {"suoxie": "GTJYHHA", "name": "\u56fd\u6cf0\u4f73\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIJIAYIHUNHEA"}, "012278": {"suoxie": "GTJYHHC", "name": "\u56fd\u6cf0\u4f73\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIJIAYIHUNHEC"}, "012279": {"suoxie": "JSWH6GYCYCZA", "name": "\u5609\u5b9e\u7a33\u548c6\u4e2a\u6708\u6301\u6709\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENHE6GEYUECHIYOUCHUNZHAIA"}, "012280": {"suoxie": "JSWH6GYCYCZC", "name": "\u5609\u5b9e\u7a33\u548c6\u4e2a\u6708\u6301\u6709\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIWENHE6GEYUECHIYOUCHUNZHAIC"}, "012281": {"suoxie": "ZOJYWJYNCYHH", "name": "\u4e2d\u6b27\u7cbe\u76ca\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUJINGYIWENJIANYINIANCHIYOUHUNHE"}, "012282": {"suoxie": "ZORZJXYNHHFOF", "name": "\u4e2d\u6b27\u777f\u667a\u7cbe\u9009\u4e00\u5e74\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOURUIZHIJINGXUANYINIANHUNHEFOF"}, "012283": {"suoxie": "JXPZYLMBRQ2040SNCYQHHFOF", "name": "\u5efa\u4fe1\u666e\u6cfd\u517b\u8001\u76ee\u6807\u65e5\u671f2040\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINPUZEYANGLAOMUBIAORIQI2040SANNIANCHIYOUQIHUNHEFOF"}, "012284": {"suoxie": "GDJKYJHH", "name": "\u5149\u5927\u5065\u5eb7\u4f18\u52a0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAJIANKANGYOUJIAHUNHE"}, "012285": {"suoxie": "ZSWYDZ30TCYZA", "name": "\u62db\u5546\u7a33\u88d5\u77ed\u503a30\u5929\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENYUDUANZHAI30TIANCHIYOUZHAIA"}, "012286": {"suoxie": "ZSWYDZ30TCYZC", "name": "\u62db\u5546\u7a33\u88d5\u77ed\u503a30\u5929\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENYUDUANZHAI30TIANCHIYOUZHAIC"}, "012287": {"suoxie": "DHQH6GYCYHHA", "name": "\u4e1c\u6d77\u542f\u822a6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGHAIQIHANG6GEYUECHIYOUHUNHEA"}, "012288": {"suoxie": "TKHGSCZHHA", "name": "\u6cf0\u5eb7\u6caa\u6e2f\u6df1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGHUGANGSHENCHENGZHANGHUNHEA"}, "012289": {"suoxie": "TKHGSCZHHC", "name": "\u6cf0\u5eb7\u6caa\u6e2f\u6df1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGHUGANGSHENCHENGZHANGHUNHEC"}, "012290": {"suoxie": "ZRHYCZZQA", "name": "\u4e2d\u878d\u6052\u76ca\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYICHUNZHAIZHAIQUANA"}, "012291": {"suoxie": "ZRHYCZZQC", "name": "\u4e2d\u878d\u6052\u76ca\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGYICHUNZHAIZHAIQUANC"}, "012292": {"suoxie": "TKDTYNCYQHHA", "name": "\u6cf0\u5eb7\u9f0e\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGDINGTAIYINIANCHIYOUQIHUNHEA"}, "012293": {"suoxie": "TKDTYNCYQHHC", "name": "\u6cf0\u5eb7\u9f0e\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGDINGTAIYINIANCHIYOUQIHUNHEC"}, "012294": {"suoxie": "TKYSJXSNCYQHH", "name": "\u6cf0\u5eb7\u4f18\u52bf\u7cbe\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGYOUSHIJINGXUANSANNIANCHIYOUQIHUNHE"}, "012295": {"suoxie": "HAJHJRZ3GYDKZFQS", "name": "\u534e\u5b89\u9526\u704f\u91d1\u878d\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANJINHAOJINRONGZHAI3GEYUEDINGKAIZHAIFAQISHI"}, "012296": {"suoxie": "YHXYYNDKZQ", "name": "\u94f6\u6cb3\u5174\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEXINGYIYINIANDINGKAIZHAIQUAN"}, "012297": {"suoxie": "DXCRLHHHA", "name": "\u4e1c\u5174\u5bb8\u745e\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGCHENRUILIANGHUAHUNHEA"}, "012298": {"suoxie": "DXCRLHHHC", "name": "\u4e1c\u5174\u5bb8\u745e\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGCHENRUILIANGHUAHUNHEC"}, "012299": {"suoxie": "PYASAYHBYNCYHHA", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u88d5\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANYUHUIBAOYINIANCHIYOUHUNHEA"}, "012300": {"suoxie": "PYASAYHBYNCYHHC", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u88d5\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANYUHUIBAOYINIANCHIYOUHUNHEC"}, "012301": {"suoxie": "YFDHXZZHH", "name": "\u6613\u65b9\u8fbe\u6838\u5fc3\u667a\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAHEXINZHIZAOHUNHE"}, "012302": {"suoxie": "PYASXFHHA", "name": "\u6d66\u94f6\u5b89\u76db\u946b\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINFUHUNHEA"}, "012303": {"suoxie": "PYASXFHHC", "name": "\u6d66\u94f6\u5b89\u76db\u946b\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINFUHUNHEC"}, "012304": {"suoxie": "PYASXRHHA", "name": "\u6d66\u94f6\u5b89\u76db\u946b\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINRUIHUNHEA"}, "012305": {"suoxie": "PYASXRHHC", "name": "\u6d66\u94f6\u5b89\u76db\u946b\u9510\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINRUIHUNHEC"}, "012308": {"suoxie": "GTJZYJLNFBYZHHA", "name": "\u56fd\u6cf0\u4ef7\u503c\u8fdc\u89c1\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJIAZHIYUANJIANLIANGNIANFENGBIYUNZUOHUNHEA"}, "012309": {"suoxie": "GTJZYJLNFBYZHHC", "name": "\u56fd\u6cf0\u4ef7\u503c\u8fdc\u89c1\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJIAZHIYUANJIANLIANGNIANFENGBIYUNZUOHUNHEC"}, "012310": {"suoxie": "MSJYZZ35NZJZZS", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u503a3-5\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINZHONGZHAI35NIANZHENGJINZHAIZHISHU"}, "012311": {"suoxie": "MSJYKTYL2040SNCYHHFOF", "name": "\u6c11\u751f\u52a0\u94f6\u5eb7\u6cf0\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "MINSHENGJIAYINKANGTAIYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "012312": {"suoxie": "CCXHYXYNDKHHA", "name": "\u957f\u57ce\u5174\u534e\u4f18\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGXINGHUAYOUXUANYINIANDINGKAIHUNHEA"}, "012313": {"suoxie": "CCXHYXYNDKHHC", "name": "\u957f\u57ce\u5174\u534e\u4f18\u9009\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGXINGHUAYOUXUANYINIANDINGKAIHUNHEC"}, "012314": {"suoxie": "NFHYLXHH", "name": "\u5357\u65b9\u884c\u4e1a\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHANGYELINGXIANHUNHE"}, "012315": {"suoxie": "CJHXGGTCZGPA", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u6210\u957f\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGCHENGZHANGGUPIAOA"}, "012316": {"suoxie": "CJHXGGTCZGPC", "name": "\u521b\u91d1\u5408\u4fe1\u6e2f\u80a1\u901a\u6210\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINGANGGUTONGCHENGZHANGGUPIAOC"}, "012317": {"suoxie": "CJHXJXZQA", "name": "\u521b\u91d1\u5408\u4fe1\u805a\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUANGJINHEXINJUXINZHAIQUANA"}, "012318": {"suoxie": "CJHXJXZQC", "name": "\u521b\u91d1\u5408\u4fe1\u805a\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHUANGJINHEXINJUXINZHAIQUANC"}, "012319": {"suoxie": "DCXFDZA", "name": "\u4e1c\u8d22\u6d88\u8d39\u7535\u5b50A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXIAOFEIDIANZIA"}, "012320": {"suoxie": "DCXFDZC", "name": "\u4e1c\u8d22\u6d88\u8d39\u7535\u5b50C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXIAOFEIDIANZIC"}, "012321": {"suoxie": "DCYJSA", "name": "\u4e1c\u8d22\u4e91\u8ba1\u7b97A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYUNJISUANA"}, "012322": {"suoxie": "DCYJSC", "name": "\u4e1c\u8d22\u4e91\u8ba1\u7b97C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIYUNJISUANC"}, "012323": {"suoxie": "HBYLETFLJC", "name": "\u534e\u5b9d\u533b\u7597ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOYILIAOETFLIANJIEC"}, "012324": {"suoxie": "XZQQHH30TCYCDZA", "name": "\u5174\u8bc1\u5168\u7403\u6052\u60e030\u5929\u6301\u6709\u8d85\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGZHENGQUANQIUHENGHUI30TIANCHIYOUCHAODUANZHAIA"}, "012325": {"suoxie": "XZQQHH30TCYCDZC", "name": "\u5174\u8bc1\u5168\u7403\u6052\u60e030\u5929\u6301\u6709\u8d85\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGZHENGQUANQIUHENGHUI30TIANCHIYOUCHAODUANZHAIC"}, "012326": {"suoxie": "THZZQZYLBJSBYFWETFLJA", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETFLIANJIEA"}, "012327": {"suoxie": "THZZQZYLBJSBYFWETFLJC", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETFLIANJIEC"}, "012330": {"suoxie": "GFJY9GYCYQZQA", "name": "\u5e7f\u53d1\u96c6\u4f189\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYOU9GEYUECHIYOUQIZHAIQUANA"}, "012331": {"suoxie": "GFJY9GYCYQZQC", "name": "\u5e7f\u53d1\u96c6\u4f189\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYOU9GEYUECHIYOUQIZHAIQUANC"}, "012332": {"suoxie": "SYXSWJHB6GYCYQHHA", "name": "\u4e0a\u94f6\u946b\u5c1a\u7a33\u5065\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGYINXINSHANGWENJIANHUIBAO6GEYUECHIYOUQIHUNHEA"}, "012333": {"suoxie": "SYXSWJHB6GYCYQHHC", "name": "\u4e0a\u94f6\u946b\u5c1a\u7a33\u5065\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGYINXINSHANGWENJIANHUIBAO6GEYUECHIYOUQIHUNHEC"}, "012334": {"suoxie": "SYHS6GYCYQHHA", "name": "\u4e0a\u94f6\u6167\u5c1a6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINHUISHANG6GEYUECHIYOUQIHUNHEA"}, "012335": {"suoxie": "SYHS6GYCYQHHC", "name": "\u4e0a\u94f6\u6167\u5c1a6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINHUISHANG6GEYUECHIYOUQIHUNHEC"}, "012336": {"suoxie": "GYWJHB60TCYQDZA", "name": "\u5de5\u94f6\u7a33\u5065\u56de\u62a560\u5929\u6301\u6709\u671f\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINWENJIANHUIBAO60TIANCHIYOUQIDUANZHAIA"}, "012337": {"suoxie": "GYWJHB60TCYQDZC", "name": "\u5de5\u94f6\u7a33\u5065\u56de\u62a560\u5929\u6301\u6709\u671f\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINWENJIANHUIBAO60TIANCHIYOUQIDUANZHAIC"}, "012338": {"suoxie": "ZXJTSXZQA", "name": "\u4e2d\u4fe1\u5efa\u6295\u53cc\u946b\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINJIANTOUSHUANGXINZHAIQUANA"}, "012339": {"suoxie": "ZXJTSXZQC", "name": "\u4e2d\u4fe1\u5efa\u6295\u53cc\u946b\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINJIANTOUSHUANGXINZHAIQUANC"}, "012340": {"suoxie": "DCSPYLA", "name": "\u4e1c\u8d22\u98df\u54c1\u996e\u6599A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAISHIPINYINLIAOA"}, "012341": {"suoxie": "DCSPYLC", "name": "\u4e1c\u8d22\u98df\u54c1\u996e\u6599C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAISHIPINYINLIAOC"}, "012342": {"suoxie": "GFRZJXHHA", "name": "\u5e7f\u53d1\u745e\u6cfd\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIZEJINGXUANHUNHEA"}, "012343": {"suoxie": "GFRZJXHHC", "name": "\u5e7f\u53d1\u745e\u6cfd\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIZEJINGXUANHUNHEC"}, "012344": {"suoxie": "JSLXYSHHA", "name": "\u5609\u5b9e\u9886\u5148\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILINGXIANYOUSHIHUNHEA"}, "012345": {"suoxie": "JSLXYSHHC", "name": "\u5609\u5b9e\u9886\u5148\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILINGXIANYOUSHIHUNHEC"}, "012346": {"suoxie": "YFDGGTCZHHA", "name": "\u6613\u65b9\u8fbe\u6e2f\u80a1\u901a\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGANGGUTONGCHENGZHANGHUNHEA"}, "012347": {"suoxie": "YFDGGTCZHHC", "name": "\u6613\u65b9\u8fbe\u6e2f\u80a1\u901a\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAGANGGUTONGCHENGZHANGHUNHEC"}, "012348": {"suoxie": "THHSKJZSQDIIA", "name": "\u5929\u5f18\u6052\u751f\u79d1\u6280\u6307\u6570(QDII)A", "style": "QDII", "pinyin": "TIANHONGHENGSHENGKEJIZHISHUQDIIA"}, "012349": {"suoxie": "THHSKJZSQDIIC", "name": "\u5929\u5f18\u6052\u751f\u79d1\u6280\u6307\u6570(QDII)C", "style": "QDII", "pinyin": "TIANHONGHENGSHENGKEJIZHISHUQDIIC"}, "012352": {"suoxie": "YDTHDLZQA", "name": "\u82f1\u5927\u901a\u60e0\u591a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDATONGHUIDUOLIZHAIQUANA"}, "012353": {"suoxie": "YDTHDLZQC", "name": "\u82f1\u5927\u901a\u60e0\u591a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDATONGHUIDUOLIZHAIQUANC"}, "012354": {"suoxie": "NFXNYCYQSHHA", "name": "\u5357\u65b9\u65b0\u80fd\u6e90\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINNENGYUANCHANYEQUSHIHUNHEA"}, "012355": {"suoxie": "NFXNYCYQSHHC", "name": "\u5357\u65b9\u65b0\u80fd\u6e90\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINNENGYUANCHANYEQUSHIHUNHEC"}, "012356": {"suoxie": "PYASJJX90TGDDZA", "name": "\u6d66\u94f6\u5b89\u76db\u5b63\u5b63\u946b90\u5929\u6eda\u52a8\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGJIJIXIN90TIANGUNDONGDUANZHAIA"}, "012357": {"suoxie": "PYASJJX90TGDDZC", "name": "\u6d66\u94f6\u5b89\u76db\u5b63\u5b63\u946b90\u5929\u6eda\u52a8\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGJIJIXIN90TIANGUNDONGDUANZHAIC"}, "012358": {"suoxie": "HFJXYLXFHHA", "name": "\u6c47\u4e30\u664b\u4fe1\u533b\u7597\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINYILIAOXIANFENGHUNHEA"}, "012359": {"suoxie": "HFJXYLXFHHC", "name": "\u6c47\u4e30\u664b\u4fe1\u533b\u7597\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINYILIAOXIANFENGHUNHEC"}, "012362": {"suoxie": "GTZZQZZQGSETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "012363": {"suoxie": "GTZZQZZQGSETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "012364": {"suoxie": "GFZZGFCYZSA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGGUANGFUCHANYEZHISHUA"}, "012365": {"suoxie": "GFZZGFCYZSC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGGUANGFUCHANYEZHISHUC"}, "012366": {"suoxie": "STMGARHBHHA", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u8363\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGTOUMOGENANRONGHUIBAOHUNHEA"}, "012367": {"suoxie": "STMGARHBHHC", "name": "\u4e0a\u6295\u6469\u6839\u5b89\u8363\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGTOUMOGENANRONGHUIBAOHUNHEC"}, "012368": {"suoxie": "DMYXLGYCYQHHA", "name": "\u5927\u6469\u4f18\u4eab\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOYOUXIANGLIUGEYUECHIYOUQIHUNHEA"}, "012369": {"suoxie": "DMYXLGYCYQHHC", "name": "\u5927\u6469\u4f18\u4eab\u516d\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOYOUXIANGLIUGEYUECHIYOUQIHUNHEC"}, "012370": {"suoxie": "YHXLYNCYQHH", "name": "\u94f6\u534e\u946b\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINLIYINIANCHIYOUQIHUNHE"}, "012371": {"suoxie": "DCHLWA", "name": "\u4e1c\u8d22\u4e92\u8054\u7f51A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIHULIANWANGA"}, "012372": {"suoxie": "DCHLWC", "name": "\u4e1c\u8d22\u4e92\u8054\u7f51C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIHULIANWANGC"}, "012373": {"suoxie": "FGWJHS12GYCYHHA", "name": "\u5bcc\u56fd\u7a33\u5065\u6052\u76db12\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANHENGSHENG12GEYUECHIYOUHUNHEA"}, "012374": {"suoxie": "FGWJHS12GYCYHHC", "name": "\u5bcc\u56fd\u7a33\u5065\u6052\u76db12\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANHENGSHENG12GEYUECHIYOUHUNHEC"}, "012376": {"suoxie": "XBLDXYZQD", "name": "\u897f\u90e8\u5229\u5f97\u7965\u9038\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEXIANGYIZHAIQUAND"}, "012377": {"suoxie": "CSARYNCYHHA", "name": "\u957f\u76db\u5b89\u777f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGANRUIYINIANCHIYOUHUNHEA"}, "012378": {"suoxie": "CSARYNCYHHC", "name": "\u957f\u76db\u5b89\u777f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGSHENGANRUIYINIANCHIYOUHUNHEC"}, "012379": {"suoxie": "CJGGHLW3GYCYHHQDIIA", "name": "\u521b\u91d1\u6e2f\u80a1\u4e92\u8054\u7f513\u4e2a\u6708\u6301\u6709\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "CHUANGJINGANGGUHULIANWANG3GEYUECHIYOUHUNHEQDIIA"}, "012380": {"suoxie": "CJGGHLW3GYCYHHQDIIC", "name": "\u521b\u91d1\u6e2f\u80a1\u4e92\u8054\u7f513\u4e2a\u6708\u6301\u6709\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "CHUANGJINGANGGUHULIANWANG3GEYUECHIYOUHUNHEQDIIC"}, "012381": {"suoxie": "YDTYCZZQE", "name": "\u82f1\u5927\u901a\u76c8\u7eaf\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDATONGYINGCHUNZHAIZHAIQUANE"}, "012382": {"suoxie": "TDHLXXJQLTHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u5174\u666f\u6c14\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXINXINGJINGQILONGTOUHUNHEA"}, "012383": {"suoxie": "TDHLXXJQLTHHC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u65b0\u5174\u666f\u6c14\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIXINXINGJINGQILONGTOUHUNHEC"}, "012384": {"suoxie": "TDHLZDZZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIDAHONGLIZHONGDUANZHAIZHAIQUANA"}, "012385": {"suoxie": "TDHLZDZZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIDAHONGLIZHONGDUANZHAIZHAIQUANC"}, "012386": {"suoxie": "YHYNWJYLYNCYHHFOF", "name": "\u94f6\u6cb3\u9890\u5e74\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHEYINIANWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "012387": {"suoxie": "GJESGCXZZHHA", "name": "\u56fd\u91d1ESG\u6301\u7eed\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINESGCHIXUZENGZHANGHUNHEA"}, "012388": {"suoxie": "GJESGCXZZHHC", "name": "\u56fd\u91d1ESG\u6301\u7eed\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINESGCHIXUZENGZHANGHUNHEC"}, "012389": {"suoxie": "XDAYPZHB6GYCYHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u54c1\u8d28\u56de\u62a56\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINPINZHIHUIBAO6GEYUECHIYOUHUNHE"}, "012390": {"suoxie": "ZOCYQZHHA", "name": "\u4e2d\u6b27\u4ea7\u4e1a\u524d\u77bb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHANYEQIANZHANHUNHEA"}, "012391": {"suoxie": "ZOCYQZHHC", "name": "\u4e2d\u6b27\u4ea7\u4e1a\u524d\u77bb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHANYEQIANZHANHUNHEC"}, "012392": {"suoxie": "XYWA60TGDCYZQA", "name": "\u5174\u94f6\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINWENAN60TIANGUNDONGCHIYOUZHAIQUANA"}, "012393": {"suoxie": "XYWA60TGDCYZQC", "name": "\u5174\u94f6\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINWENAN60TIANGUNDONGCHIYOUZHAIQUANC"}, "012394": {"suoxie": "NYZZXHSMZPPZS", "name": "\u519c\u94f6\u4e2d\u8bc1\u65b0\u534e\u793e\u6c11\u65cf\u54c1\u724c\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NONGYINZHONGZHENGXINHUASHEMINZUPINPAIZHISHU"}, "012395": {"suoxie": "XY60TGDCYDZZQA", "name": "\u5174\u4e1a60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYE60TIANGUNDONGCHIYOUDUANZHAIZHAIQUANA"}, "012396": {"suoxie": "XY60TGDCYDZZQC", "name": "\u5174\u4e1a60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYE60TIANGUNDONGCHIYOUDUANZHAIZHAIQUANC"}, "012397": {"suoxie": "NFJY6GYCYZQA", "name": "\u5357\u65b9\u4f73\u51436\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGJIAYUAN6GEYUECHIYOUZHAIQUANA"}, "012398": {"suoxie": "NFJY6GYCYZQC", "name": "\u5357\u65b9\u4f73\u51436\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGJIAYUAN6GEYUECHIYOUZHAIQUANC"}, "012399": {"suoxie": "NFYYYNCYZQA", "name": "\u5357\u65b9\u6c38\u5143\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGYONGYUANYINIANCHIYOUZHAIQUANA"}, "012400": {"suoxie": "NFYYYNCYZQC", "name": "\u5357\u65b9\u6c38\u5143\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGYONGYUANYINIANCHIYOUZHAIQUANC"}, "012401": {"suoxie": "THZZYYZSZQA", "name": "\u5929\u5f18\u4e2d\u8bc1\u533b\u836f\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYIYAOZHISHUZENGQIANGA"}, "012402": {"suoxie": "THZZYYZSZQC", "name": "\u5929\u5f18\u4e2d\u8bc1\u533b\u836f\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYIYAOZHISHUZENGQIANGC"}, "012403": {"suoxie": "DFZZ15NZCXJRZA", "name": "\u4e1c\u65b9\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIA"}, "012404": {"suoxie": "DFZZ15NZCXJRZC", "name": "\u4e1c\u65b9\u4e2d\u503a1-5\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHONGZHAI15NIANZHENGCEXINGJINRONGZHAIC"}, "012405": {"suoxie": "THGZJZCLZSFQSA", "name": "\u5929\u5f18\u56fd\u8bc1\u5efa\u7b51\u6750\u6599\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGJIANZHUCAILIAOZHISHUFAQISHIA"}, "012406": {"suoxie": "YYCYJZHHA", "name": "\u6c38\u8d62\u957f\u8fdc\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGCHANGYUANJIAZHIHUNHEA"}, "012407": {"suoxie": "YYCYJZHHC", "name": "\u6c38\u8d62\u957f\u8fdc\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGCHANGYUANJIAZHIHUNHEC"}, "012408": {"suoxie": "GFHCYNCYHHA", "name": "\u5e7f\u53d1\u6052\u660c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGCHANGYINIANCHIYOUHUNHEA"}, "012409": {"suoxie": "GFHCYNCYHHC", "name": "\u5e7f\u53d1\u6052\u660c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGCHANGYINIANCHIYOUHUNHEC"}, "012410": {"suoxie": "HFTCZLHHHA", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGLINGHANGHUNHEA"}, "012411": {"suoxie": "HFTCZLHHHC", "name": "\u6d77\u5bcc\u901a\u6210\u957f\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGCHENGZHANGLINGHANGHUNHEC"}, "012412": {"suoxie": "HQCLYXHHA", "name": "\u6c47\u6cc9\u7b56\u7565\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANCELUEYOUXUANHUNHEA"}, "012413": {"suoxie": "JXRYCZC", "name": "\u5efa\u4fe1\u777f\u6021\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRUIYICHUNZHAIC"}, "012414": {"suoxie": "ZSZZBJZSLOFC", "name": "\u62db\u5546\u4e2d\u8bc1\u767d\u9152\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGBAIJIUZHISHULOFC"}, "012415": {"suoxie": "DBSZG60ZZZQA", "name": "\u5fb7\u90a6\u4e0a\u8bc1G60\u7efc\u6307\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DEBANGSHANGZHENGG60ZONGZHIZENGQIANGA"}, "012416": {"suoxie": "DBSZG60ZZZQC", "name": "\u5fb7\u90a6\u4e0a\u8bc1G60\u7efc\u6307\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DEBANGSHANGZHENGG60ZONGZHIZENGQIANGC"}, "012417": {"suoxie": "ZSGZSWYYZSLOFC", "name": "\u62db\u5546\u56fd\u8bc1\u751f\u7269\u533b\u836f\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGGUOZHENGSHENGWUYIYAOZHISHULOFC"}, "012418": {"suoxie": "PAHJ1NDKZFQS", "name": "\u5e73\u5b89\u5408\u8fdb1\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHEJIN1NIANDINGKAIZHAIFAQISHI"}, "012419": {"suoxie": "THGZJZCLZSFQSC", "name": "\u5929\u5f18\u56fd\u8bc1\u5efa\u7b51\u6750\u6599\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGJIANZHUCAILIAOZHISHUFAQISHIC"}, "012420": {"suoxie": "GFJZLXHHC", "name": "\u5e7f\u53d1\u4ef7\u503c\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJIAZHILINGXIANHUNHEC"}, "012421": {"suoxie": "HXYJSHHHA", "name": "\u534e\u590f\u4f18\u52a0\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYOUJIASHENGHUOHUNHEA"}, "012422": {"suoxie": "HXYJSHHHC", "name": "\u534e\u590f\u4f18\u52a0\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYOUJIASHENGHUOHUNHEC"}, "012423": {"suoxie": "SXZQCDZE", "name": "\u5c71\u897f\u8bc1\u5238\u8d85\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANXIZHENGQUANCHAODUANZHAIE"}, "012424": {"suoxie": "HTFXHDKZA", "name": "\u6c47\u6dfb\u5bcc\u946b\u5f18\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINHONGDINGKAIZHAIA"}, "012425": {"suoxie": "HTFXHDKZC", "name": "\u6c47\u6dfb\u5bcc\u946b\u5f18\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINHONGDINGKAIZHAIC"}, "012426": {"suoxie": "NFJZZXHHA", "name": "\u5357\u65b9\u4ef7\u503c\u81fb\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIAZHIZHENXUANHUNHEA"}, "012427": {"suoxie": "NFJZZXHHC", "name": "\u5357\u65b9\u4ef7\u503c\u81fb\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIAZHIZHENXUANHUNHEC"}, "012428": {"suoxie": "HXHXZZHHA", "name": "\u534e\u590f\u6838\u5fc3\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINZHIZAOHUNHEA"}, "012429": {"suoxie": "HXHXZZHHC", "name": "\u534e\u590f\u6838\u5fc3\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINZHIZAOHUNHEC"}, "012430": {"suoxie": "NYRK6GYCYHH", "name": "\u519c\u94f6\u745e\u5eb76\u4e2a\u6708\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NONGYINRUIKANG6GEYUECHIYOUHUNHE"}, "012431": {"suoxie": "GTRYATHHA", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANTAIHUNHEA"}, "012432": {"suoxie": "GTRYATHHC", "name": "\u56fd\u6295\u745e\u94f6\u5b89\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINANTAIHUNHEC"}, "012433": {"suoxie": "HATHYNZQA", "name": "\u534e\u5b89\u6dfb\u548c\u4e00\u5e74\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANTIANHEYINIANZHAIQUANA"}, "012434": {"suoxie": "YHDYHBYNCYQHH", "name": "\u94f6\u534e\u591a\u5143\u56de\u62a5\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUADUOYUANHUIBAOYINIANCHIYOUQIHUNHE"}, "012435": {"suoxie": "WJZRHBYNCYHHA", "name": "\u4e07\u5bb6\u62db\u745e\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAZHAORUIHUIBAOYINIANCHIYOUHUNHEA"}, "012436": {"suoxie": "WJZRHBYNCYHHC", "name": "\u4e07\u5bb6\u62db\u745e\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIAZHAORUIHUIBAOYINIANCHIYOUHUNHEC"}, "012437": {"suoxie": "DBJZYXHHA", "name": "\u5fb7\u90a6\u4ef7\u503c\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGJIAZHIYOUXUANHUNHEA"}, "012438": {"suoxie": "DBJZYXHHC", "name": "\u5fb7\u90a6\u4ef7\u503c\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGJIAZHIYOUXUANHUNHEC"}, "012439": {"suoxie": "DFHRHSNDKHHC", "name": "\u4e1c\u65b9\u7ea2\u777f\u548c\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGRUIHESANNIANDINGKAIHUNHEC"}, "012440": {"suoxie": "PAHX3GYDKZA", "name": "\u5e73\u5b89\u60e0\u4fe13\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXIN3GEYUEDINGKAIZHAIA"}, "012441": {"suoxie": "PAHX3GYDKZC", "name": "\u5e73\u5b89\u60e0\u4fe13\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANHUIXIN3GEYUEDINGKAIZHAIC"}, "012442": {"suoxie": "YYWJZZYNCYHHE", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGWENJIANZENGZHANGYINIANCHIYOUHUNHEE"}, "012443": {"suoxie": "ZSRH6GYCYHHA", "name": "\u62db\u5546\u745e\u9e3f6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHONG6GEYUECHIYOUHUNHEA"}, "012444": {"suoxie": "ZSRH6GYCYHHC", "name": "\u62db\u5546\u745e\u9e3f6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIHONG6GEYUECHIYOUHUNHEC"}, "012445": {"suoxie": "HFXNYGPXFQS", "name": "\u534e\u5bcc\u65b0\u80fd\u6e90\u80a1\u7968\u578b\u53d1\u8d77\u5f0f", "style": "\u80a1\u7968\u578b", "pinyin": "HUAFUXINNENGYUANGUPIAOXINGFAQISHI"}, "012446": {"suoxie": "HATHYNZQC", "name": "\u534e\u5b89\u6dfb\u548c\u4e00\u5e74\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANTIANHEYINIANZHAIQUANC"}, "012447": {"suoxie": "HXHLWLTHHA", "name": "\u534e\u590f\u4e92\u8054\u7f51\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHULIANWANGLONGTOUHUNHEA"}, "012448": {"suoxie": "HXHLWLTHHC", "name": "\u534e\u590f\u4e92\u8054\u7f51\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHULIANWANGLONGTOUHUNHEC"}, "012449": {"suoxie": "GFRYLXHHC", "name": "\u5e7f\u53d1\u777f\u6bc5\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYILINGXIANHUNHEC"}, "012450": {"suoxie": "CXYNYLSNCYHHFOF", "name": "\u957f\u4fe1\u9890\u5e74\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGXINYINIANYANGLAOSANNIANCHIYOUHUNHEFOF"}, "012451": {"suoxie": "GSABAHJRZZQ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u6052\u91d1\u878d\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANHENGJINRONGZHAIZHAIQUAN"}, "012452": {"suoxie": "GTLY30TGDCYDZA", "name": "\u56fd\u6cf0\u5229\u4f1830\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIYOU30TIANGUNDONGCHIYOUDUANZHAIA"}, "012453": {"suoxie": "GTLY30TGDCYDZC", "name": "\u56fd\u6cf0\u5229\u4f1830\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIYOU30TIANGUNDONGCHIYOUDUANZHAIC"}, "012454": {"suoxie": "CHXYSYMSYXHHA", "name": "\u6df3\u539a\u946b\u60a6\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINYUESHANGYEMOSHIYOUXUANHUNHEA"}, "012455": {"suoxie": "CHXYSYMSYXHHC", "name": "\u6df3\u539a\u946b\u60a6\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUXINYUESHANGYEMOSHIYOUXUANHUNHEC"}, "012456": {"suoxie": "PYSZJJXFHHA", "name": "\u9e4f\u626c\u6570\u5b57\u7ecf\u6d4e\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGSHUZIJINGJIXIANFENGHUNHEA"}, "012457": {"suoxie": "PYSZJJXFHHC", "name": "\u9e4f\u626c\u6570\u5b57\u7ecf\u6d4e\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGSHUZIJINGJIXIANFENGHUNHEC"}, "012458": {"suoxie": "TKFAYLYNCYHHFOF", "name": "\u6cf0\u5eb7\u798f\u5b89\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIKANGFUANYANGLAOYINIANCHIYOUHUNHEFOF"}, "012459": {"suoxie": "HTFWJRXYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u777f\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANRUIXIANGYINIANCHIYOUHUNHEA"}, "012460": {"suoxie": "HTFWJRXYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u777f\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANRUIXIANGYINIANCHIYOUHUNHEC"}, "012461": {"suoxie": "DCJDA", "name": "\u4e1c\u8d22\u5bb6\u7535A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIJIADIANA"}, "012462": {"suoxie": "DCJDC", "name": "\u4e1c\u8d22\u5bb6\u7535C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIJIADIANC"}, "012463": {"suoxie": "BSCZYSHHA", "name": "\u535a\u65f6\u6210\u957f\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGYOUSHIHUNHEA"}, "012464": {"suoxie": "BSCZYSHHC", "name": "\u535a\u65f6\u6210\u957f\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGYOUSHIHUNHEC"}, "012465": {"suoxie": "SYHJLZQ", "name": "\u4e0a\u94f6\u6167\u5609\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIJIALIZHAIQUAN"}, "012466": {"suoxie": "JSCLJXHHA", "name": "\u5609\u5b9e\u7b56\u7565\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUEJINGXUANHUNHEA"}, "012467": {"suoxie": "JSCLJXHHC", "name": "\u5609\u5b9e\u7b56\u7565\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUEJINGXUANHUNHEC"}, "012468": {"suoxie": "ZYZQAHZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u704f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANHAOZHAIQUANA"}, "012469": {"suoxie": "ZYZQAHZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u704f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANHAOZHAIQUANC"}, "012470": {"suoxie": "PACFBHBC", "name": "\u5e73\u5b89\u8d22\u5bcc\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANCAIFUBAOHUOBIC"}, "012471": {"suoxie": "ZJYYYNCYHHA", "name": "\u4e2d\u52a0\u90ae\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAYOUYIYINIANCHIYOUHUNHEA"}, "012472": {"suoxie": "ZJYYYNCYHHC", "name": "\u4e2d\u52a0\u90ae\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGJIAYOUYIYINIANCHIYOUHUNHEC"}, "012473": {"suoxie": "DCCZHBLGYCYHHA", "name": "\u5927\u6210\u6210\u957f\u56de\u62a5\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGHUIBAOLIUGEYUECHIYOUHUNHEA"}, "012474": {"suoxie": "DCCZHBLGYCYHHC", "name": "\u5927\u6210\u6210\u957f\u56de\u62a5\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHENGZHANGHUIBAOLIUGEYUECHIYOUHUNHEC"}, "012475": {"suoxie": "PAYZQYHHA", "name": "\u5e73\u5b89\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUZHIQIYEHUNHEA"}, "012476": {"suoxie": "PAYZQYHHC", "name": "\u5e73\u5b89\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUZHIQIYEHUNHEC"}, "012477": {"suoxie": "FGJXJX12GYCYHHA", "name": "\u5bcc\u56fd\u5320\u5fc3\u7cbe\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIANGXINJINGXUAN12GEYUECHIYOUHUNHEA"}, "012478": {"suoxie": "FGJXJX12GYCYHHC", "name": "\u5bcc\u56fd\u5320\u5fc3\u7cbe\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJIANGXINJINGXUAN12GEYUECHIYOUHUNHEC"}, "012479": {"suoxie": "HAXTWJYNCYQHHA", "name": "\u6c47\u5b89\u4fe1\u6cf0\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANXINTAIWENJIANYINIANCHIYOUQIHUNHEA"}, "012480": {"suoxie": "HAXTWJYNCYQHHC", "name": "\u6c47\u5b89\u4fe1\u6cf0\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIANXINTAIWENJIANYINIANCHIYOUQIHUNHEC"}, "012483": {"suoxie": "QHKYYZLT6GYCYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u8d28\u9f99\u59346\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANYOUZHILONGTOU6GEYUECHIYOUHUNHEA"}, "012484": {"suoxie": "QHKYYZLT6GYCYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u4f18\u8d28\u9f99\u59346\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANYOUZHILONGTOU6GEYUECHIYOUHUNHEC"}, "012485": {"suoxie": "JXHYYNCYHHA", "name": "\u5efa\u4fe1\u6c47\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINHUIYIYINIANCHIYOUHUNHEA"}, "012486": {"suoxie": "JXHYYNCYHHC", "name": "\u5efa\u4fe1\u6c47\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINHUIYIYINIANCHIYOUHUNHEC"}, "012487": {"suoxie": "BSHXYNCYQHHA", "name": "\u535a\u65f6\u6052\u73ba\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXIYINIANCHIYOUQIHUNHEA"}, "012488": {"suoxie": "BSHXYNCYQHHC", "name": "\u535a\u65f6\u6052\u73ba\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXIYINIANCHIYOUQIHUNHEC"}, "012489": {"suoxie": "ZSZSCZD", "name": "\u62db\u5546\u62db\u987a\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOSHUNCHUNZHAID"}, "012490": {"suoxie": "ZSZYCZD", "name": "\u62db\u5546\u62db\u6021\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOYICHUNZHAID"}, "012491": {"suoxie": "HSHXYLHHA", "name": "\u534e\u5546\u6838\u5fc3\u5f15\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGHEXINYINLIHUNHEA"}, "012492": {"suoxie": "HSHXYLHHC", "name": "\u534e\u5546\u6838\u5fc3\u5f15\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGHEXINYINLIHUNHEC"}, "012493": {"suoxie": "CXNXJHHHA", "name": "\u957f\u4fe1\u5185\u9700\u5747\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINNEIXUJUNHENGHUNHEA"}, "012494": {"suoxie": "CXNXJHHHC", "name": "\u957f\u4fe1\u5185\u9700\u5747\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINNEIXUJUNHENGHUNHEC"}, "012495": {"suoxie": "MSJYSHDLHH", "name": "\u6c11\u751f\u52a0\u94f6\u53cc\u6838\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINSHUANGHEDONGLIHUNHE"}, "012496": {"suoxie": "TTHYYXGPA", "name": "\u540c\u6cf0\u884c\u4e1a\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAIHANGYEYOUXUANGUPIAOA"}, "012497": {"suoxie": "TTHYYXGPC", "name": "\u540c\u6cf0\u884c\u4e1a\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAIHANGYEYOUXUANGUPIAOC"}, "012498": {"suoxie": "HTFZZ500JBMZQZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1500\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG500JIBENMIANZENGQIANGZHISHUA"}, "012499": {"suoxie": "HTFZZ500JBMZQZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1500\u57fa\u672c\u9762\u589e\u5f3a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG500JIBENMIANZENGQIANGZHISHUC"}, "012500": {"suoxie": "BSHRHBHHA", "name": "\u535a\u65f6\u6c47\u8363\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIRONGHUIBAOHUNHEA"}, "012501": {"suoxie": "BSHRHBHHC", "name": "\u535a\u65f6\u6c47\u8363\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIRONGHUIBAOHUNHEC"}, "012502": {"suoxie": "YHASHH", "name": "\u94f6\u534e\u5b89\u76db\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAANSHENGHUNHE"}, "012503": {"suoxie": "GTZZHBCY50ETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUANBAOCHANYE50ETFLIANJIEA"}, "012504": {"suoxie": "GTZZHBCY50ETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUANBAOCHANYE50ETFLIANJIEC"}, "012505": {"suoxie": "HAMXWJYLMBYNCYHHFQSFOF", "name": "\u534e\u5b89\u6c11\u4eab\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANMINXIANGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFAQISHIFOF"}, "012506": {"suoxie": "DFPZXFYNCYQHHA", "name": "\u4e1c\u65b9\u54c1\u8d28\u6d88\u8d39\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGPINZHIXIAOFEIYINIANCHIYOUQIHUNHEA"}, "012507": {"suoxie": "DFPZXFYNCYQHHC", "name": "\u4e1c\u65b9\u54c1\u8d28\u6d88\u8d39\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGPINZHIXIAOFEIYINIANCHIYOUQIHUNHEC"}, "012508": {"suoxie": "JSAKWJYLMBYNCYQHHFOF", "name": "\u5609\u5b9e\u5b89\u5eb7\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIANKANGWENJIANYANGLAOMUBIAOYINIANCHIYOUQIHUNHEFOF"}, "012509": {"suoxie": "XZQQAYWJYLYNCYHHFOF", "name": "\u5174\u8bc1\u5168\u7403\u5b89\u60a6\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGZHENGQUANQIUANYUEWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "012510": {"suoxie": "GFYZQYYNCYQHHA", "name": "\u56fd\u5bcc\u4f18\u8d28\u4f01\u4e1a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUYOUZHIQIYEYINIANCHIYOUQIHUNHEA"}, "012511": {"suoxie": "GFYZQYYNCYQHHC", "name": "\u56fd\u5bcc\u4f18\u8d28\u4f01\u4e1a\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUYOUZHIQIYEYINIANCHIYOUQIHUNHEC"}, "012512": {"suoxie": "JSYL2045WNCYQHHFOF", "name": "\u5609\u5b9e\u517b\u80012045\u4e94\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYANGLAO2045WUNIANCHIYOUQIHUNHEFOF"}, "012513": {"suoxie": "TKFZYLWNCYHHFOF", "name": "\u6cf0\u5eb7\u798f\u6cfd\u517b\u8001\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIKANGFUZEYANGLAOWUNIANCHIYOUHUNHEFOF"}, "012515": {"suoxie": "NFFRWJYLFOF", "name": "\u5357\u65b9\u5bcc\u745e\u7a33\u5065\u517b\u8001(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGFURUIWENJIANYANGLAOFOF"}, "012516": {"suoxie": "GTZZXFJXSBCYZTETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7ec6\u5206\u673a\u68b0\u8bbe\u5907\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIFENJIXIESHEBEICHANYEZHUTIETFLIANJIEA"}, "012517": {"suoxie": "GTZZXFJXSBCYZTETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7ec6\u5206\u673a\u68b0\u8bbe\u5907\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIFENJIXIESHEBEICHANYEZHUTIETFLIANJIEC"}, "012518": {"suoxie": "JSCCYXYL2040SNCYHHFOF", "name": "\u666f\u987a\u957f\u57ce\u9890\u5fc3\u517b\u80012040\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYIXINYANGLAO2040SANNIANCHIYOUHUNHEFOF"}, "012519": {"suoxie": "DCHXQSHHA", "name": "\u5927\u6210\u6838\u5fc3\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHEXINQUSHIHUNHEA"}, "012520": {"suoxie": "DCHXQSHHC", "name": "\u5927\u6210\u6838\u5fc3\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHEXINQUSHIHUNHEC"}, "012521": {"suoxie": "YDWGZQHXYNCYHHA", "name": "\u82f1\u5927\u7a33\u56fa\u589e\u5f3a\u6838\u5fc3\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINGDAWENGUZENGQIANGHEXINYINIANCHIYOUHUNHEA"}, "012522": {"suoxie": "YDWGZQHXYNCYHHC", "name": "\u82f1\u5927\u7a33\u56fa\u589e\u5f3a\u6838\u5fc3\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINGDAWENGUZENGQIANGHEXINYINIANCHIYOUHUNHEC"}, "012523": {"suoxie": "ZRGZLCZHHA", "name": "\u4e2d\u878d\u9ad8\u8d28\u91cf\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGGAOZHILIANGCHENGZHANGHUNHEA"}, "012524": {"suoxie": "ZRGZLCZHHC", "name": "\u4e2d\u878d\u9ad8\u8d28\u91cf\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGGAOZHILIANGCHENGZHANGHUNHEC"}, "012525": {"suoxie": "RTWXZY6GYCYQHHC", "name": "\u878d\u901a\u7a33\u4fe1\u589e\u76ca6\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENXINZENGYI6GEYUECHIYOUQIHUNHEC"}, "012526": {"suoxie": "GFSJHHA", "name": "\u5e7f\u53d1\u76db\u9526\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGJINHUNHEA"}, "012527": {"suoxie": "GFSJHHC", "name": "\u5e7f\u53d1\u76db\u9526\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGJINHUNHEC"}, "012528": {"suoxie": "GFXRYNCYQHHA", "name": "\u5e7f\u53d1\u946b\u777f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINRUIYINIANCHIYOUQIHUNHEA"}, "012529": {"suoxie": "GFXRYNCYQHHC", "name": "\u5e7f\u53d1\u946b\u777f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINRUIYINIANCHIYOUQIHUNHEC"}, "012530": {"suoxie": "YYHTYYNCYHH", "name": "\u6c38\u8d62\u60e0\u6dfb\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGHUITIANYINGYINIANCHIYOUHUNHE"}, "012533": {"suoxie": "JSJZQDYNCYQHHA", "name": "\u5609\u5b9e\u4ef7\u503c\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHIQUDONGYINIANCHIYOUQIHUNHEA"}, "012534": {"suoxie": "JSJZQDYNCYQHHC", "name": "\u5609\u5b9e\u4ef7\u503c\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHIQUDONGYINIANCHIYOUQIHUNHEC"}, "012535": {"suoxie": "WJQQCZYNCYQHHQDIIA", "name": "\u4e07\u5bb6\u5168\u7403\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "WANJIAQUANQIUCHENGZHANGYINIANCHIYOUQIHUNHEQDIIA"}, "012536": {"suoxie": "WJQQCZYNCYQHHQDIIC", "name": "\u4e07\u5bb6\u5168\u7403\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "WANJIAQUANQIUCHENGZHANGYINIANCHIYOUQIHUNHEQDIIC"}, "012537": {"suoxie": "HBZZXFHGCYZTETFLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEA"}, "012538": {"suoxie": "HBZZXFHGCYZTETFLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEC"}, "012539": {"suoxie": "DFXRZQA", "name": "\u4e1c\u65b9\u5174\u6da6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGXINGRUNZHAIQUANA"}, "012540": {"suoxie": "DFXRZQC", "name": "\u4e1c\u65b9\u5174\u6da6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGXINGRUNZHAIQUANC"}, "012541": {"suoxie": "JYCYSJHHA", "name": "\u91d1\u9e70\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGCHANYESHENGJIHUNHEA"}, "012542": {"suoxie": "JYCYSJHHC", "name": "\u91d1\u9e70\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGCHANYESHENGJIHUNHEC"}, "012543": {"suoxie": "JSZZXNYQCZSA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXINNENGYUANQICHEZHISHUA"}, "012544": {"suoxie": "JSZZXNYQCZSC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXINNENGYUANQICHEZHISHUC"}, "012545": {"suoxie": "FRFYHHA", "name": "\u5bcc\u8363\u798f\u94f6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUYINHUNHEA"}, "012546": {"suoxie": "FRFYHHC", "name": "\u5bcc\u8363\u798f\u94f6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUYINHUNHEC"}, "012547": {"suoxie": "NFYHETFLJE", "name": "\u5357\u65b9\u94f6\u884cETF\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYINHANGETFLIANJIEE"}, "012548": {"suoxie": "HBZZXFSPYLCYZTETFLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIFENSHIPINYINLIAOCHANYEZHUTIETFLIANJIEA"}, "012549": {"suoxie": "HBZZXFSPYLCYZTETFLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIFENSHIPINYINLIAOCHANYEZHUTIETFLIANJIEC"}, "012550": {"suoxie": "HBZZDZ50ETFLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7535\u5b5050ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGDIANZI50ETFLIANJIEA"}, "012551": {"suoxie": "HBZZDZ50ETFLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7535\u5b5050ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGDIANZI50ETFLIANJIEC"}, "012552": {"suoxie": "THZZXPCYZSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINPIANCHANYEZHISHUA"}, "012553": {"suoxie": "THZZXPCYZSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINPIANCHANYEZHISHUC"}, "012554": {"suoxie": "XBLDCYBDPETFLJA", "name": "\u897f\u90e8\u5229\u5f97\u521b\u4e1a\u677f\u5927\u76d8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDECHUANGYEBANDAPANETFLIANJIEA"}, "012555": {"suoxie": "XBLDCYBDPETFLJC", "name": "\u897f\u90e8\u5229\u5f97\u521b\u4e1a\u677f\u5927\u76d8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDECHUANGYEBANDAPANETFLIANJIEC"}, "012556": {"suoxie": "CSJQYXHH", "name": "\u957f\u76db\u666f\u6c14\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGJINGQIYOUXUANHUNHE"}, "012557": {"suoxie": "ZOJQQZYNHHA", "name": "\u4e2d\u6b27\u666f\u6c14\u524d\u77bb\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJINGQIQIANZHANYINIANHUNHEA"}, "012558": {"suoxie": "ZOJQQZYNHHC", "name": "\u4e2d\u6b27\u666f\u6c14\u524d\u77bb\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJINGQIQIANZHANYINIANHUNHEC"}, "012559": {"suoxie": "THZZHGSKJLTZSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHUGANGSHENKEJILONGTOUZHISHUA"}, "012560": {"suoxie": "THZZHGSKJLTZSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHUGANGSHENKEJILONGTOUZHISHUC"}, "012561": {"suoxie": "THZZXCLZTZSFQSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINCAILIAOZHUTIZHISHUFAQISHIA"}, "012562": {"suoxie": "THZZXCLZTZSFQSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINCAILIAOZHUTIZHISHUFAQISHIC"}, "012566": {"suoxie": "CCJWZQC", "name": "\u957f\u57ce\u4e45\u7a33\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIUWENZHAIQUANC"}, "012567": {"suoxie": "CCJWZQD", "name": "\u957f\u57ce\u4e45\u7a33\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGJIUWENZHAIQUAND"}, "012568": {"suoxie": "THGDZZHHA", "name": "\u5929\u5f18\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGGAODUANZHIZAOHUNHEA"}, "012569": {"suoxie": "THGDZZHHC", "name": "\u5929\u5f18\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGGAODUANZHIZAOHUNHEC"}, "012572": {"suoxie": "HYLXTLHHA", "name": "\u6052\u8d8a\u4e50\u4eab\u6dfb\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HENGYUELEXIANGTIANLIHUNHEA"}, "012573": {"suoxie": "HYLXTLHHC", "name": "\u6052\u8d8a\u4e50\u4eab\u6dfb\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HENGYUELEXIANGTIANLIHUNHEC"}, "012574": {"suoxie": "HTFWL60TDZA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u522960\u5929\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENLI60TIANDUANZHAIA"}, "012575": {"suoxie": "HTFWL60TDZC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u522960\u5929\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENLI60TIANDUANZHAIC"}, "012576": {"suoxie": "FGCYHB12GYCYHHA", "name": "\u5bcc\u56fd\u8bda\u76ca\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOCHENGYIHUIBAO12GEYUECHIYOUHUNHEA"}, "012577": {"suoxie": "FGCYHB12GYCYHHC", "name": "\u5bcc\u56fd\u8bda\u76ca\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOCHENGYIHUIBAO12GEYUECHIYOUHUNHEC"}, "012578": {"suoxie": "FGHLHHA", "name": "\u5bcc\u56fd\u7ea2\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHONGLIHUNHEA"}, "012579": {"suoxie": "FGHLHHC", "name": "\u5bcc\u56fd\u7ea2\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHONGLIHUNHEC"}, "012580": {"suoxie": "CTZGHA30TGDZDZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u5b8930\u5929\u6eda\u52a8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGAN30TIANGUNDONGZHONGDUANZHAIA"}, "012581": {"suoxie": "CTZGHA30TGDZDZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u5b8930\u5929\u6eda\u52a8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGAN30TIANGUNDONGZHONGDUANZHAIC"}, "012582": {"suoxie": "JYPZZZYNHHA", "name": "\u4ea4\u94f6\u54c1\u8d28\u589e\u957f\u4e00\u5e74\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINPINZHIZENGZHANGYINIANHUNHEA"}, "012583": {"suoxie": "JYPZZZYNHHC", "name": "\u4ea4\u94f6\u54c1\u8d28\u589e\u957f\u4e00\u5e74\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINPINZHIZENGZHANGYINIANHUNHEC"}, "012584": {"suoxie": "NFZGXXJJ9GYCYQHHQDIIA", "name": "\u5357\u65b9\u4e2d\u56fd\u65b0\u5174\u7ecf\u6d4e9\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408(QDII)A", "style": "QDII", "pinyin": "NANFANGZHONGGUOXINXINGJINGJI9GEYUECHIYOUQIHUNHEQDIIA"}, "012585": {"suoxie": "NFZGXXJJ9GYCYQHHQDIIC", "name": "\u5357\u65b9\u4e2d\u56fd\u65b0\u5174\u7ecf\u6d4e9\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408(QDII)C", "style": "QDII", "pinyin": "NANFANGZHONGGUOXINXINGJINGJI9GEYUECHIYOUQIHUNHEQDIIC"}, "012586": {"suoxie": "NFGGCXSYYNCYHHA", "name": "\u5357\u65b9\u6e2f\u80a1\u521b\u65b0\u89c6\u91ce\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGANGGUCHUANGXINSHIYEYINIANCHIYOUHUNHEA"}, "012587": {"suoxie": "NFGGCXSYYNCYHHC", "name": "\u5357\u65b9\u6e2f\u80a1\u521b\u65b0\u89c6\u91ce\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGANGGUCHUANGXINSHIYEYINIANCHIYOUHUNHEC"}, "012588": {"suoxie": "NFGGTYSQYHHA", "name": "\u5357\u65b9\u6e2f\u80a1\u901a\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGANGGUTONGYOUSHIQIYEHUNHEA"}, "012589": {"suoxie": "NFGGTYSQYHHC", "name": "\u5357\u65b9\u6e2f\u80a1\u901a\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGANGGUTONGYOUSHIQIYEHUNHEC"}, "012590": {"suoxie": "YFDZZQZZQGSETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "012591": {"suoxie": "GFTC180TGDCYZQA", "name": "\u5e7f\u53d1\u6dfb\u8d22180\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFATIANCAI180TIANGUNDONGCHIYOUZHAIQUANA"}, "012592": {"suoxie": "GFTC180TGDCYZQC", "name": "\u5e7f\u53d1\u6dfb\u8d22180\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFATIANCAI180TIANGUNDONGCHIYOUZHAIQUANC"}, "012593": {"suoxie": "GFTC180TGDCYZQE", "name": "\u5e7f\u53d1\u6dfb\u8d22180\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFATIANCAI180TIANGUNDONGCHIYOUZHAIQUANE"}, "012594": {"suoxie": "ZSRX1NCYQHHA", "name": "\u62db\u5546\u745e\u4eab1\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIXIANG1NIANCHIYOUQIHUNHEA"}, "012595": {"suoxie": "ZSRX1NCYQHHC", "name": "\u62db\u5546\u745e\u4eab1\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUIXIANG1NIANCHIYOUQIHUNHEC"}, "012596": {"suoxie": "HTFZZ800ETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1800ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG800ETFLIANJIEA"}, "012597": {"suoxie": "HTFZZ800ETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1800ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG800ETFLIANJIEC"}, "012598": {"suoxie": "HAGZSWYYZSA", "name": "\u534e\u5b89\u56fd\u8bc1\u751f\u7269\u533b\u836f\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANGUOZHENGSHENGWUYIYAOZHISHUA"}, "012599": {"suoxie": "HAGZSWYYZSC", "name": "\u534e\u5b89\u56fd\u8bc1\u751f\u7269\u533b\u836f\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANGUOZHENGSHENGWUYIYAOZHISHUC"}, "012600": {"suoxie": "ZYNHQDGPC", "name": "\u4e2d\u94f6\u5185\u6838\u9a71\u52a8\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINNEIHEQUDONGGUPIAOC"}, "012601": {"suoxie": "CXWHZQA", "name": "\u957f\u4fe1\u7a33\u60e0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENHUIZHAIQUANA"}, "012602": {"suoxie": "CXWHZQC", "name": "\u957f\u4fe1\u7a33\u60e0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENHUIZHAIQUANC"}, "012603": {"suoxie": "FADFLCZC", "name": "\u5bcc\u5b89\u8fbe\u5bcc\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUANDAFULICHUNZHAIC"}, "012604": {"suoxie": "ZSXSYNDKZQXFQS", "name": "\u6d59\u5546\u5174\u76db\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u578b\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGXINGSHENGYINIANDINGKAIZHAIQUANXINGFAQISHI"}, "012605": {"suoxie": "DCZQBXA", "name": "\u4e1c\u8d22\u8bc1\u5238\u4fdd\u9669A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIZHENGQUANBAOXIANA"}, "012606": {"suoxie": "DCZQBXC", "name": "\u4e1c\u8d22\u8bc1\u5238\u4fdd\u9669C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIZHENGQUANBAOXIANC"}, "012607": {"suoxie": "HTFBXLHPZHHC", "name": "\u6c47\u6dfb\u5bcc\u4fdd\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUBAOXINLINGHUOPEIZHIHUNHEC"}, "012608": {"suoxie": "XDAYLXZXHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u9886\u5148\u667a\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLINGXIANZHIXUANHUNHE"}, "012609": {"suoxie": "AXWJHLYNCYHHA", "name": "\u5b89\u4fe1\u7a33\u5065\u6c47\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANHUILIYINIANCHIYOUHUNHEA"}, "012610": {"suoxie": "AXWJHLYNCYHHC", "name": "\u5b89\u4fe1\u7a33\u5065\u6c47\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINWENJIANHUILIYINIANCHIYOUHUNHEC"}, "012611": {"suoxie": "DFZSCZZQA", "name": "\u4e1c\u65b9\u81fb\u5584\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENSHANCHUNZHAIZHAIQUANA"}, "012612": {"suoxie": "DFZSCZZQC", "name": "\u4e1c\u65b9\u81fb\u5584\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGZHENSHANCHUNZHAIZHAIQUANC"}, "012613": {"suoxie": "CJHXCYZXHHA", "name": "\u521b\u91d1\u5408\u4fe1\u4ea7\u4e1a\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINCHANYEZHIXUANHUNHEA"}, "012614": {"suoxie": "CJHXCYZXHHC", "name": "\u521b\u91d1\u5408\u4fe1\u4ea7\u4e1a\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINCHANYEZHIXUANHUNHEC"}, "012615": {"suoxie": "DWGQGGZTLHPZHHC", "name": "\u4e1c\u5434\u56fd\u4f01\u6539\u9769\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUGUOQIGAIGEZHUTILINGHUOPEIZHIHUNHEC"}, "012616": {"suoxie": "JSYHHLHHC", "name": "\u5609\u5b9e\u4f18\u5316\u7ea2\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUHUAHONGLIHUNHEC"}, "012617": {"suoxie": "DWXJJHHC", "name": "\u4e1c\u5434\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXINJINGJIHUNHEC"}, "012618": {"suoxie": "CAHFZDZZQE", "name": "\u957f\u5b89\u6cd3\u6ca3\u4e2d\u77ed\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGANHONGFENGZHONGDUANZHAIZHAIQUANE"}, "012619": {"suoxie": "JSZZRJFWETFLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u8f6f\u4ef6\u670d\u52a1ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGRUANJIANFUWUETFLIANJIEA"}, "012620": {"suoxie": "JSZZRJFWETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u8f6f\u4ef6\u670d\u52a1ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGRUANJIANFUWUETFLIANJIEC"}, "012621": {"suoxie": "NAXFHHC", "name": "\u8bfa\u5b89\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANXIANFENGHUNHEC"}, "012622": {"suoxie": "JYTYCZZQC", "name": "\u91d1\u9e70\u6dfb\u88d5\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANYUCHUNZHAIZHAIQUANC"}, "012623": {"suoxie": "JYTYCZZQC", "name": "\u91d1\u9e70\u6dfb\u76c8\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANYINGCHUNZHAIZHAIQUANC"}, "012624": {"suoxie": "FCFYZQA", "name": "\u8702\u5de2\u4e30\u8fdc\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGYUANZHAIQUANA"}, "012625": {"suoxie": "FCFYZQC", "name": "\u8702\u5de2\u4e30\u8fdc\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGYUANZHAIQUANC"}, "012626": {"suoxie": "SWLXHYBZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u6c47\u5143\u5b9d\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHENWANLINGXINHUIYUANBAOZHAIQUANA"}, "012627": {"suoxie": "SWLXHYBZQC", "name": "\u7533\u4e07\u83f1\u4fe1\u6c47\u5143\u5b9d\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHENWANLINGXINHUIYUANBAOZHAIQUANC"}, "012628": {"suoxie": "HXDPJXHHC", "name": "\u534e\u590f\u5927\u76d8\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIADAPANJINGXUANHUNHEC"}, "012629": {"suoxie": "GFGZBDTXPETFLJA", "name": "\u5e7f\u53d1\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGBANDAOTIXINPIANETFLIANJIEA"}, "012630": {"suoxie": "GFGZBDTXPETFLJC", "name": "\u5e7f\u53d1\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGBANDAOTIXINPIANETFLIANJIEC"}, "012631": {"suoxie": "ZYYXLHPZHHC", "name": "\u4e2d\u94f6\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYOUXUANLINGHUOPEIZHIHUNHEC"}, "012634": {"suoxie": "GTZZYLETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u533b\u7597ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYILIAOETFLIANJIEA"}, "012635": {"suoxie": "GTZZYLETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u533b\u7597ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYILIAOETFLIANJIEC"}, "012636": {"suoxie": "GTZZQZRJETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8f6f\u4ef6ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIRUANJIANETFLIANJIEA"}, "012637": {"suoxie": "GTZZQZRJETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8f6f\u4ef6ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIRUANJIANETFLIANJIEC"}, "012638": {"suoxie": "FGZYJX3GYCYHHFOFA", "name": "\u5bcc\u56fd\u667a\u4f18\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHIYOUJINGXUAN3GEYUECHIYOUHUNHEFOFA"}, "012639": {"suoxie": "FGZYJX3GYCYHHFOFC", "name": "\u5bcc\u56fd\u667a\u4f18\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHIYOUJINGXUAN3GEYUECHIYOUHUNHEFOFC"}, "012640": {"suoxie": "PHWJHLYNCYQHHA", "name": "\u9e4f\u534e\u7a33\u5065\u9e3f\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAWENJIANHONGLIYINIANCHIYOUQIHUNHEA"}, "012641": {"suoxie": "PHWJHLYNCYQHHC", "name": "\u9e4f\u534e\u7a33\u5065\u9e3f\u5229\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAWENJIANHONGLIYINIANCHIYOUQIHUNHEC"}, "012643": {"suoxie": "ZSZZHLETFLJA", "name": "\u62db\u5546\u4e2d\u8bc1\u7ea2\u5229ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGHONGLIETFLIANJIEA"}, "012644": {"suoxie": "ZSZZHLETFLJC", "name": "\u62db\u5546\u4e2d\u8bc1\u7ea2\u5229ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGHONGLIETFLIANJIEC"}, "012645": {"suoxie": "JXZZQZZQGSETFLJA", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEA"}, "012646": {"suoxie": "JXZZQZZQGSETFLJC", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "012647": {"suoxie": "ZODJYNCYHH", "name": "\u4e2d\u6b27\u6d1e\u89c1\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUDONGJIANYINIANCHIYOUHUNHE"}, "012648": {"suoxie": "PHWT30TGDCYZQA", "name": "\u9e4f\u534e\u7a33\u6cf030\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAWENTAI30TIANGUNDONGCHIYOUZHAIQUANA"}, "012649": {"suoxie": "PHWT30TGDCYZQC", "name": "\u9e4f\u534e\u7a33\u6cf030\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAWENTAI30TIANGUNDONGCHIYOUZHAIQUANC"}, "012650": {"suoxie": "BSBDTZTHHA", "name": "\u535a\u65f6\u534a\u5bfc\u4f53\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIBANDAOTIZHUTIHUNHEA"}, "012651": {"suoxie": "BSBDTZTHHC", "name": "\u535a\u65f6\u534a\u5bfc\u4f53\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIBANDAOTIZHUTIHUNHEC"}, "012652": {"suoxie": "YFDYSLHLGYCYHHFOFA", "name": "\u6613\u65b9\u8fbe\u4f18\u52bf\u9886\u822a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUSHILINGHANGLIUGEYUECHIYOUHUNHEFOFA"}, "012653": {"suoxie": "YFDYSLHLGYCYHHFOFC", "name": "\u6613\u65b9\u8fbe\u4f18\u52bf\u9886\u822a\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUSHILINGHANGLIUGEYUECHIYOUHUNHEFOFC"}, "012654": {"suoxie": "XZQQYXPHSGYCYHHFOF", "name": "\u5174\u8bc1\u5168\u7403\u4f18\u9009\u5e73\u8861\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "XINGZHENGQUANQIUYOUXUANPINGHENGSANGEYUECHIYOUHUNHEFOF"}, "012655": {"suoxie": "BS5G50ETFLJA", "name": "\u535a\u65f65G50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHI5G50ETFLIANJIEA"}, "012656": {"suoxie": "JXLXWJ6GYCYHHFOFA", "name": "\u5efa\u4fe1\u9f99\u7965\u7a33\u8fdb6\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINLONGXIANGWENJIN6GEYUECHIYOUHUNHEFOFA"}, "012657": {"suoxie": "JXLXWJ6GYCYHHFOFC", "name": "\u5efa\u4fe1\u9f99\u7965\u7a33\u8fdb6\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIANXINLONGXIANGWENJIN6GEYUECHIYOUHUNHEFOFC"}, "012658": {"suoxie": "BS5G50ETFLJC", "name": "\u535a\u65f65G50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHI5G50ETFLIANJIEC"}, "012659": {"suoxie": "HAAYLHPZHHC", "name": "\u534e\u5b89\u5b89\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANYILINGHUOPEIZHIHUNHEC"}, "012660": {"suoxie": "HAXLXLHPZHHC", "name": "\u534e\u5b89\u65b0\u4e50\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANXINLEXIANGLINGHUOPEIZHIHUNHEC"}, "012661": {"suoxie": "GFHYYNCYQHHA", "name": "\u5e7f\u53d1\u6052\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYIYINIANCHIYOUQIHUNHEA"}, "012662": {"suoxie": "GFHYYNCYQHHC", "name": "\u5e7f\u53d1\u6052\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYIYINIANCHIYOUQIHUNHEC"}, "012663": {"suoxie": "GSABHGS300ETFLJA", "name": "\u56fd\u5bff\u5b89\u4fdd\u6caa\u6e2f\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOHUGANGSHEN300ETFLIANJIEA"}, "012664": {"suoxie": "GSABHGS300ETFLJC", "name": "\u56fd\u5bff\u5b89\u4fdd\u6caa\u6e2f\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOHUGANGSHEN300ETFLIANJIEC"}, "012665": {"suoxie": "GSABYXHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u7965\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOYUXIANGHUNHEA"}, "012666": {"suoxie": "GSABYXHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u88d5\u7965\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOYUXIANGHUNHEC"}, "012667": {"suoxie": "ZRJHYNCYHHA", "name": "\u4e2d\u878d\u666f\u6cd3\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGHONGYINIANCHIYOUHUNHEA"}, "012668": {"suoxie": "ZRJHYNCYHHC", "name": "\u4e2d\u878d\u666f\u6cd3\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGHONGYINIANCHIYOUHUNHEC"}, "012669": {"suoxie": "NFXXCYHHA", "name": "\u5357\u65b9\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINXINGCHANYEHUNHEA"}, "012670": {"suoxie": "NFXXCYHHC", "name": "\u5357\u65b9\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGXINXINGCHANYEHUNHEC"}, "012671": {"suoxie": "JSHXLCHHA", "name": "\u5609\u5b9e\u6838\u5fc3\u84dd\u7b79\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIHEXINLANCHOUHUNHEA"}, "012672": {"suoxie": "JSHXLCHHC", "name": "\u5609\u5b9e\u6838\u5fc3\u84dd\u7b79\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIHEXINLANCHOUHUNHEC"}, "012673": {"suoxie": "HRRX6GYDKHHA", "name": "\u534e\u878d\u878d\u51746\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGRONGXING6GEYUEDINGKAIHUNHEA"}, "012674": {"suoxie": "HRRX6GYDKHHC", "name": "\u534e\u878d\u878d\u51746\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGRONGXING6GEYUEDINGKAIHUNHEC"}, "012675": {"suoxie": "HRRZ6GYDKHHA", "name": "\u534e\u878d\u878d\u6cfd6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGRONGZE6GEYUEDINGKAIHUNHEA"}, "012676": {"suoxie": "HRRZ6GYDKHHC", "name": "\u534e\u878d\u878d\u6cfd6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUARONGRONGZE6GEYUEDINGKAIHUNHEC"}, "012677": {"suoxie": "WJRTHHA", "name": "\u4e07\u5bb6\u745e\u6cf0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIARUITAIHUNHEA"}, "012678": {"suoxie": "WJRTHHC", "name": "\u4e07\u5bb6\u745e\u6cf0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "WANJIARUITAIHUNHEC"}, "012679": {"suoxie": "HTBRGFETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u5149\u4f0fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIGUANGFUETFLIANJIEA"}, "012680": {"suoxie": "HTBRGFETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u5149\u4f0fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIGUANGFUETFLIANJIEC"}, "012681": {"suoxie": "YYXCHHA", "name": "\u6c38\u8d62\u946b\u8fb0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGXINCHENHUNHEA"}, "012682": {"suoxie": "YYXCHHC", "name": "\u6c38\u8d62\u946b\u8fb0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGXINCHENHUNHEC"}, "012683": {"suoxie": "DFHAYZXYNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u5b89\u76c8\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGANYINGZHENXUANYINIANCHIYOUHUNHEA"}, "012684": {"suoxie": "DFHAYZXYNCYHHC", "name": "\u4e1c\u65b9\u7ea2\u5b89\u76c8\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGANYINGZHENXUANYINIANCHIYOUHUNHEC"}, "012685": {"suoxie": "CCYXZYYNCYHHA", "name": "\u957f\u57ce\u4f18\u9009\u62db\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANZHAOYIYINIANCHIYOUHUNHEA"}, "012686": {"suoxie": "CCYXZYYNCYHHC", "name": "\u957f\u57ce\u4f18\u9009\u62db\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGCHENGYOUXUANZHAOYIYINIANCHIYOUHUNHEC"}, "012687": {"suoxie": "HYHYJXSGYHHFOF", "name": "\u6052\u8d8a\u6c47\u4f18\u7cbe\u9009\u4e09\u4e2a\u6708\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEHUIYOUJINGXUANSANGEYUEHUNHEFOF"}, "012688": {"suoxie": "CACZYXHHA", "name": "\u957f\u5b89\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANCHENGZHANGYOUXUANHUNHEA"}, "012689": {"suoxie": "CACZYXHHC", "name": "\u957f\u5b89\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANCHENGZHANGYOUXUANHUNHEC"}, "012690": {"suoxie": "GFXFLXHHA", "name": "\u5e7f\u53d1\u6d88\u8d39\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOFEILINGXIANHUNHEA"}, "012691": {"suoxie": "GFXFLXHHC", "name": "\u5e7f\u53d1\u6d88\u8d39\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOFEILINGXIANHUNHEC"}, "012692": {"suoxie": "BSZZ03NGKHA", "name": "\u535a\u65f6\u4e2d\u503a0-3\u5e74\u56fd\u5f00\u884cA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI03NIANGUOKAIHANGA"}, "012693": {"suoxie": "BSZZ03NGKHC", "name": "\u535a\u65f6\u4e2d\u503a0-3\u5e74\u56fd\u5f00\u884cC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHONGZHAI03NIANGUOKAIHANGC"}, "012694": {"suoxie": "HTFGZSWYYETFLJA", "name": "\u6c47\u6dfb\u5bcc\u56fd\u8bc1\u751f\u7269\u533b\u836fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUGUOZHENGSHENGWUYIYAOETFLIANJIEA"}, "012695": {"suoxie": "HTFGZSWYYETFLJC", "name": "\u6c47\u6dfb\u5bcc\u56fd\u8bc1\u751f\u7269\u533b\u836fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUGUOZHENGSHENGWUYIYAOETFLIANJIEC"}, "012696": {"suoxie": "TTSZJJGPA", "name": "\u540c\u6cf0\u6570\u5b57\u7ecf\u6d4e\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAISHUZIJINGJIGUPIAOA"}, "012697": {"suoxie": "TTSZJJGPC", "name": "\u540c\u6cf0\u6570\u5b57\u7ecf\u6d4e\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAISHUZIJINGJIGUPIAOC"}, "012698": {"suoxie": "PAZZXNYQCETFFQLJA", "name": "\u5e73\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXINNENGYUANQICHEETFFAQILIANJIEA"}, "012699": {"suoxie": "PAZZXNYQCETFFQLJC", "name": "\u5e73\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXINNENGYUANQICHEETFFAQILIANJIEC"}, "012700": {"suoxie": "YFDZZQZZQGSETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIZHENGQUANGONGSIETFLIANJIEC"}, "012701": {"suoxie": "AXMAHBYNCYHHA", "name": "\u5b89\u4fe1\u6c11\u5b89\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINMINANHUIBAOYINIANCHIYOUHUNHEA"}, "012702": {"suoxie": "AXMAHBYNCYHHC", "name": "\u5b89\u4fe1\u6c11\u5b89\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINMINANHUIBAOYINIANCHIYOUHUNHEC"}, "012703": {"suoxie": "HXHXCZHHA", "name": "\u534e\u590f\u6838\u5fc3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINCHENGZHANGHUNHEA"}, "012704": {"suoxie": "ZYXLWJHBYNFBHHA", "name": "\u4e2d\u94f6\u5174\u5229\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINGLIWENJIANHUIBAOYINIANFENGBIHUNHEA"}, "012705": {"suoxie": "ZYXLWJHBYNFBHHC", "name": "\u4e2d\u94f6\u5174\u5229\u7a33\u5065\u56de\u62a5\u4e00\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINXINGLIWENJIANHUIBAOYINIANFENGBIHUNHEC"}, "012706": {"suoxie": "ZYHXJXHHA", "name": "\u4e2d\u94f6\u6838\u5fc3\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINHEXINJINGXUANHUNHEA"}, "012707": {"suoxie": "ZYHXJXHHC", "name": "\u4e2d\u94f6\u6838\u5fc3\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINHEXINJINGXUANHUNHEC"}, "012708": {"suoxie": "DFHZZHLDBDZSA", "name": "\u4e1c\u65b9\u7ea2\u4e2d\u8bc1\u7ea2\u5229\u4f4e\u6ce2\u52a8\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGFANGHONGZHONGZHENGHONGLIDIBODONGZHISHUA"}, "012709": {"suoxie": "DFHZZHLDBDZSC", "name": "\u4e1c\u65b9\u7ea2\u4e2d\u8bc1\u7ea2\u5229\u4f4e\u6ce2\u52a8\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGFANGHONGZHONGZHENGHONGLIDIBODONGZHISHUC"}, "012710": {"suoxie": "HXHXCZHHC", "name": "\u534e\u590f\u6838\u5fc3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHEXINCHENGZHANGHUNHEC"}, "012711": {"suoxie": "QHKYHGSLCJXHHC", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u84dd\u7b79\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENLANCHOUJINGXUANHUNHEC"}, "012712": {"suoxie": "JXHS300HLETFLJA", "name": "\u5efa\u4fe1\u6caa\u6df1300\u7ea2\u5229ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300HONGLIETFLIANJIEA"}, "012713": {"suoxie": "JXHS300HLETFLJC", "name": "\u5efa\u4fe1\u6caa\u6df1300\u7ea2\u5229ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300HONGLIETFLIANJIEC"}, "012715": {"suoxie": "CSCZJZHHC", "name": "\u957f\u76db\u6210\u957f\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGSHENGCHENGZHANGJIAZHIHUNHEC"}, "012716": {"suoxie": "CSCXXFHHC", "name": "\u957f\u76db\u521b\u65b0\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGCHUANGXINXIANFENGHUNHEC"}, "012717": {"suoxie": "YFDZZKJ50ETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u628050ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKEJI50ETFLIANJIEA"}, "012718": {"suoxie": "YFDZZKJ50ETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u628050ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKEJI50ETFLIANJIEC"}, "012719": {"suoxie": "HXXXJJYNCYHHA", "name": "\u534e\u590f\u65b0\u5174\u7ecf\u6d4e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINXINGJINGJIYINIANCHIYOUHUNHEA"}, "012720": {"suoxie": "HXXXJJYNCYHHC", "name": "\u534e\u590f\u65b0\u5174\u7ecf\u6d4e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINXINGJINGJIYINIANCHIYOUHUNHEC"}, "012721": {"suoxie": "ZXBCYLMBRQ2040SNCYQHHFOF", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u517b\u8001\u76ee\u6807\u65e5\u671f2040\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHONGXINBAOCHENGYANGLAOMUBIAORIQI2040SANNIANCHIYOUQIHUNHEFOF"}, "012722": {"suoxie": "PAZZGFCYZSA", "name": "\u5e73\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGGUANGFUCHANYEZHISHUA"}, "012723": {"suoxie": "PAZZGFCYZSC", "name": "\u5e73\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGGUANGFUCHANYEZHISHUC"}, "012724": {"suoxie": "GTZZXMYZETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXUMUYANGZHIETFLIANJIEA"}, "012725": {"suoxie": "GTZZXMYZETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXUMUYANGZHIETFLIANJIEC"}, "012726": {"suoxie": "SWLXRXHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u777f\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINRUIXUANHUNHEA"}, "012727": {"suoxie": "SWLXRXHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u777f\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINRUIXUANHUNHEC"}, "012728": {"suoxie": "GTZZDMYXETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGDONGMANYOUXIETFLIANJIEA"}, "012729": {"suoxie": "GTZZDMYXETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGDONGMANYOUXIETFLIANJIEC"}, "012730": {"suoxie": "GTZZXFHGCYZTETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEA"}, "012731": {"suoxie": "GTZZXFHGCYZTETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEC"}, "012732": {"suoxie": "RTTYYNDKZFQS", "name": "\u878d\u901a\u901a\u8dc3\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGTONGYUEYINIANDINGKAIZHAIFAQISHI"}, "012733": {"suoxie": "YFDZZRGZNZTETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEA"}, "012734": {"suoxie": "YFDZZRGZNZTETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEC"}, "012735": {"suoxie": "CTZGZZ13NGKZA", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANZHONGZHAI13NIANGUOKAIZHAIA"}, "012736": {"suoxie": "CTZGZZ13NGKZC", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGZIGUANZHONGZHAI13NIANGUOKAIZHAIC"}, "012737": {"suoxie": "GFZZCXYCYETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANGXINYAOCHANYEETFLIANJIEA"}, "012738": {"suoxie": "GFZZCXYCYETFLJC", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANGXINYAOCHANYEETFLIANJIEC"}, "012739": {"suoxie": "BYHXZQ", "name": "\u5b9d\u76c8\u9e3f\u7fd4\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAOYINGHONGXIANGZHAIQUAN"}, "012740": {"suoxie": "GYPHHB6GYCYQZQA", "name": "\u5de5\u94f6\u5e73\u8861\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINPINGHENGHUIBAO6GEYUECHIYOUQIZHAIQUANA"}, "012741": {"suoxie": "GYPHHB6GYCYQZQC", "name": "\u5de5\u94f6\u5e73\u8861\u56de\u62a56\u4e2a\u6708\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINPINGHENGHUIBAO6GEYUECHIYOUQIZHAIQUANC"}, "012742": {"suoxie": "GYRFYNDKCZFQS", "name": "\u5de5\u94f6\u745e\u5bcc\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIFUYINIANDINGKAICHUNZHAIFAQISHI"}, "012743": {"suoxie": "HTFTFRXWJYLMBYNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u798f\u777f\u9009\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUTIANFURUIXUANWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "012744": {"suoxie": "GDPZSHHHA", "name": "\u5149\u5927\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAPINZHISHENGHUOHUNHEA"}, "012745": {"suoxie": "HBBRYNDKZ", "name": "\u534e\u5b9d\u5b9d\u745e\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUABAOBAORUIYINIANDINGKAIZHAI"}, "012746": {"suoxie": "FGSLZQZQA", "name": "\u5bcc\u56fd\u53cc\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHUANGLIZENGQIANGZHAIQUANA"}, "012747": {"suoxie": "FGSLZQZQC", "name": "\u5bcc\u56fd\u53cc\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOSHUANGLIZENGQIANGZHAIQUANC"}, "012748": {"suoxie": "HTBRYJZXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u8fdc\u89c1\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYUANJIANZHIXUANHUNHEA"}, "012749": {"suoxie": "HTBRYJZXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u8fdc\u89c1\u667a\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIYUANJIANZHIXUANHUNHEC"}, "012750": {"suoxie": "SYHDLZQ", "name": "\u4e0a\u94f6\u6167\u9f0e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINHUIDINGLIZHAIQUAN"}, "012751": {"suoxie": "JXNSDK100ZSQDIIMYXHA", "name": "\u5efa\u4fe1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570(QDII)\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "JIANXINNASIDAKE100ZHISHUQDIIMEIYUANXIANHUIA"}, "012752": {"suoxie": "JXNSDK100ZSQDIIRMBC", "name": "\u5efa\u4fe1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "JIANXINNASIDAKE100ZHISHUQDIIRENMINBIC"}, "012753": {"suoxie": "JXNSDK100ZSQDIIMYXHC", "name": "\u5efa\u4fe1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570(QDII)\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "JIANXINNASIDAKE100ZHISHUQDIIMEIYUANXIANHUIC"}, "012754": {"suoxie": "PHNDDTLJA", "name": "\u9e4f\u534e\u5185\u5730\u4f4e\u78b3\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUANEIDIDITANLIANJIEA"}, "012755": {"suoxie": "PHNDDTLJC", "name": "\u9e4f\u534e\u5185\u5730\u4f4e\u78b3\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUANEIDIDITANLIANJIEC"}, "012756": {"suoxie": "YFDZZLTQYZSA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u9f99\u5934\u4f01\u4e1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGLONGTOUQIYEZHISHUA"}, "012757": {"suoxie": "YFDZZLTQYZSC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u9f99\u5934\u4f01\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGLONGTOUQIYEZHISHUC"}, "012758": {"suoxie": "GDPZSHHHC", "name": "\u5149\u5927\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAPINZHISHENGHUOHUNHEC"}, "012759": {"suoxie": "GYHGSHLWETFFQSLJA", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUGANGSHENHULIANWANGETFFAQISHILIANJIEA"}, "012760": {"suoxie": "GYHGSHLWETFFQSLJC", "name": "\u5de5\u94f6\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUGANGSHENHULIANWANGETFFAQISHILIANJIEC"}, "012761": {"suoxie": "HTBRSZHLETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u7ea2\u5229ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGHONGLIETFLIANJIEA"}, "012762": {"suoxie": "HTBRSZHLETFLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u7ea2\u5229ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGHONGLIETFLIANJIEC"}, "012763": {"suoxie": "HTZJZZXFSPYLFQA", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIZIJINZHONGZHENGXIFENSHIPINYINLIAOFAQIA"}, "012764": {"suoxie": "HTZJZZXFSPYLFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIZIJINZHONGZHENGXIFENSHIPINYINLIAOFAQIC"}, "012765": {"suoxie": "GFDPJZHHA", "name": "\u5e7f\u53d1\u5927\u76d8\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFADAPANJIAZHIHUNHEA"}, "012766": {"suoxie": "GFDPJZHHC", "name": "\u5e7f\u53d1\u5927\u76d8\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFADAPANJIAZHIHUNHEC"}, "012767": {"suoxie": "CTZGJZFXHHC", "name": "\u8d22\u901a\u8d44\u7ba1\u4ef7\u503c\u53d1\u73b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANJIAZHIFAXIANHUNHEC"}, "012768": {"suoxie": "HXZZDMYXETFLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGDONGMANYOUXIETFLIANJIEA"}, "012769": {"suoxie": "HXZZDMYXETFLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGDONGMANYOUXIETFLIANJIEC"}, "012770": {"suoxie": "GDBDXCXSHHH", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u521b\u65b0\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDABAODEXINCHUANGXINSHENGHUOHUNHE"}, "012771": {"suoxie": "BYYSCYHHC", "name": "\u5b9d\u76c8\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGYOUSHICHANYEHUNHEC"}, "012772": {"suoxie": "XDAYJHPZHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u7cbe\u534e\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINJINGHUAPEIZHIHUNHEC"}, "012773": {"suoxie": "JSCDZZQA", "name": "\u5609\u5b9e\u8d85\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHICHAODUANZHAIZHAIQUANA"}, "012774": {"suoxie": "QHKYFHZQA", "name": "\u524d\u6d77\u5f00\u6e90\u4e30\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANFENGHEZHAIQUANA"}, "012775": {"suoxie": "QHKYFHZQC", "name": "\u524d\u6d77\u5f00\u6e90\u4e30\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "QIANHAIKAIYUANFENGHEZHAIQUANC"}, "012776": {"suoxie": "HXJXLGYCYFOFA", "name": "\u534e\u590f\u805a\u946b\u516d\u4e2a\u6708\u6301\u6709(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUXINLIUGEYUECHIYOUFOFA"}, "012777": {"suoxie": "HXJXLGYCYFOFC", "name": "\u534e\u590f\u805a\u946b\u516d\u4e2a\u6708\u6301\u6709(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAJUXINLIUGEYUECHIYOUFOFC"}, "012778": {"suoxie": "ZOYLCYHHC", "name": "\u4e2d\u6b27\u517b\u8001\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYANGLAOCHANYEHUNHEC"}, "012779": {"suoxie": "BSYDHLZTHHA", "name": "\u535a\u65f6\u79fb\u52a8\u4e92\u8054\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYIDONGHULIANZHUTIHUNHEA"}, "012780": {"suoxie": "BSYDHLZTHHC", "name": "\u535a\u65f6\u79fb\u52a8\u4e92\u8054\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYIDONGHULIANZHUTIHUNHEC"}, "012781": {"suoxie": "YHZZCXYCYETFFQSLJA", "name": "\u94f6\u534e\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGCHUANGXINYAOCHANYEETFFAQISHILIANJIEA"}, "012782": {"suoxie": "YHZZCXYCYETFFQSLJC", "name": "\u94f6\u534e\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGCHUANGXINYAOCHANYEETFFAQISHILIANJIEC"}, "012783": {"suoxie": "PHCZWJYLYNCYQHHFOF", "name": "\u9e4f\u534e\u957f\u6cbb\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUACHANGZHIWENJIANYANGLAOYINIANCHIYOUQIHUNHEFOF"}, "012785": {"suoxie": "PHPZJXHHA", "name": "\u9e4f\u534e\u54c1\u8d28\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHIJINGXUANHUNHEA"}, "012786": {"suoxie": "PHPZJXHHC", "name": "\u9e4f\u534e\u54c1\u8d28\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPINZHIJINGXUANHUNHEC"}, "012787": {"suoxie": "PYASTHPZ6GYCYHHFOFA", "name": "\u6d66\u94f6\u5b89\u76db\u6cf0\u548c\u914d\u7f6e6\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGTAIHEPEIZHI6GEYUECHIYOUHUNHEFOFA"}, "012788": {"suoxie": "PYASTHPZ6GYCYHHFOFC", "name": "\u6d66\u94f6\u5b89\u76db\u6cf0\u548c\u914d\u7f6e6\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGTAIHEPEIZHI6GEYUECHIYOUHUNHEFOFC"}, "012789": {"suoxie": "HTFSXHBZQA", "name": "\u6c47\u6dfb\u5bcc\u53cc\u4eab\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGXIANGHUIBAOZHAIQUANA"}, "012790": {"suoxie": "HTFSXHBZQC", "name": "\u6c47\u6dfb\u5bcc\u53cc\u4eab\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGXIANGHUIBAOZHAIQUANC"}, "012791": {"suoxie": "HTFJJJDYNCYHHFOFA", "name": "\u6c47\u6dfb\u5bcc\u805a\u7126\u7ecf\u5178\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUJIAOJINGDIANYINIANCHIYOUHUNHEFOFA"}, "012792": {"suoxie": "HTFJJJDYNCYHHFOFC", "name": "\u6c47\u6dfb\u5bcc\u805a\u7126\u7ecf\u5178\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUJIAOJINGDIANYINIANCHIYOUHUNHEFOFC"}, "012793": {"suoxie": "CCKCLNDKHHC", "name": "\u957f\u57ce\u79d1\u521b\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGKECHUANGLIANGNIANDINGKAIHUNHEC"}, "012795": {"suoxie": "YFDYX3GYDKZ", "name": "\u6613\u65b9\u8fbe\u88d5\u51743\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAYUXING3GEYUEDINGKAIZHAI"}, "012796": {"suoxie": "HAYX12GYDKCZZQ", "name": "\u6c47\u5b89\u88d5\u517412\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUIANYUXING12GEYUEDINGKAICHUNZHAIZHAIQUAN"}, "012797": {"suoxie": "PHFNZQ", "name": "\u9e4f\u534e\u4e30\u5b81\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGNINGZHAIQUAN"}, "012798": {"suoxie": "HBDSCYHHC", "name": "\u534e\u5b9d\u7b2c\u4e09\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAODISANCHANYEHUNHEC"}, "012799": {"suoxie": "HBLSZTHHC", "name": "\u534e\u5b9d\u7eff\u8272\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOLVSEZHUTIHUNHEC"}, "012800": {"suoxie": "TDZXJYGPC", "name": "\u6cf0\u8fbe\u8f6c\u578b\u673a\u9047\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAZHUANXINGJIYUGUPIAOC"}, "012801": {"suoxie": "FGZZYY50ETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u533b\u836f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIYAO50ETFLIANJIEA"}, "012802": {"suoxie": "FGZZYY50ETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u533b\u836f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIYAO50ETFLIANJIEC"}, "012803": {"suoxie": "ZRJYYNDKZQ", "name": "\u4e2d\u878d\u805a\u4f18\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGJUYOUYINIANDINGKAIZHAIQUAN"}, "012804": {"suoxie": "GFHSKJZSQDIIA", "name": "\u5e7f\u53d1\u6052\u751f\u79d1\u6280\u6307\u6570(QDII)A", "style": "QDII", "pinyin": "GUANGFAHENGSHENGKEJIZHISHUQDIIA"}, "012805": {"suoxie": "GFHSKJZSQDIIC", "name": "\u5e7f\u53d1\u6052\u751f\u79d1\u6280\u6307\u6570(QDII)C", "style": "QDII", "pinyin": "GUANGFAHENGSHENGKEJIZHISHUQDIIC"}, "012806": {"suoxie": "ZSTC1NDKZ", "name": "\u62db\u5546\u6dfb\u54481\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANCHENG1NIANDINGKAIZHAI"}, "012807": {"suoxie": "GLAHX3GYDKZ", "name": "\u56fd\u8054\u5b89\u6052\u946b3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANANHENGXIN3GEYUEDINGKAIZHAI"}, "012808": {"suoxie": "PHZZAGZYCYZSLOFC", "name": "\u9e4f\u534e\u4e2d\u8bc1A\u80a1\u8d44\u6e90\u4ea7\u4e1a\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGAGUZIYUANCHANYEZHISHULOFC"}, "012809": {"suoxie": "PHZZHGSKJLTZSLOFC", "name": "\u9e4f\u534e\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGHUGANGSHENKEJILONGTOUZHISHULOFC"}, "012810": {"suoxie": "PHGZGTHYZSLOFC", "name": "\u9e4f\u534e\u56fd\u8bc1\u94a2\u94c1\u884c\u4e1a\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGGANGTIEHANGYEZHISHULOFC"}, "012811": {"suoxie": "HBMSCIZGAGGJTLOFC", "name": "\u534e\u5b9dMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOMSCIZHONGGUOAGUGUOJITONGLOFC"}, "012812": {"suoxie": "GFXYSYHHA", "name": "\u56fd\u5bcc\u946b\u9890\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOFUXINYISHOUYIHUNHEA"}, "012813": {"suoxie": "GFXYSYHHC", "name": "\u56fd\u5bcc\u946b\u9890\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOFUXINYISHOUYIHUNHEC"}, "012814": {"suoxie": "XHAYYNDKZQFQS", "name": "\u5174\u534e\u5b89\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGHUAANYINGYINIANDINGKAIZHAIQUANFAQISHI"}, "012815": {"suoxie": "BYXXCYHHC", "name": "\u5b9d\u76c8\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGXINXINGCHANYEHUNHEC"}, "012816": {"suoxie": "GTZHLNFBYZHHA", "name": "\u56fd\u6cf0\u81f4\u548c\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIZHIHELIANGNIANFENGBIYUNZUOHUNHEA"}, "012817": {"suoxie": "GTZHLNFBYZHHC", "name": "\u56fd\u6cf0\u81f4\u548c\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIZHIHELIANGNIANFENGBIYUNZUOHUNHEC"}, "012818": {"suoxie": "ZSXCZQZQA", "name": "\u62db\u5546\u4eab\u8bda\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXIANGCHENGZENGQIANGZHAIQUANA"}, "012819": {"suoxie": "ZSXCZQZQC", "name": "\u62db\u5546\u4eab\u8bda\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXIANGCHENGZENGQIANGZHAIQUANC"}, "012820": {"suoxie": "HTFJZLXHH", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHILINGXIANHUNHE"}, "012821": {"suoxie": "YFDYFYNCYQHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u4e30\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEFENGYINIANCHIYOUQIHUNHEA"}, "012822": {"suoxie": "YFDYFYNCYQHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u4e30\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEFENGYINIANCHIYOUQIHUNHEC"}, "012823": {"suoxie": "FGAL90TGDCYZQA", "name": "\u5bcc\u56fd\u5b89\u522990\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOANLI90TIANGUNDONGCHIYOUZHAIQUANA"}, "012824": {"suoxie": "FGAL90TGDCYZQC", "name": "\u5bcc\u56fd\u5b89\u522990\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOANLI90TIANGUNDONGCHIYOUZHAIQUANC"}, "012826": {"suoxie": "GYJN9GYCYQHHA", "name": "\u5de5\u94f6\u805a\u5b819\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUNING9GEYUECHIYOUQIHUNHEA"}, "012827": {"suoxie": "GYJN9GYCYQHHC", "name": "\u5de5\u94f6\u805a\u5b819\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJUNING9GEYUECHIYOUQIHUNHEC"}, "012828": {"suoxie": "FGPCHB12GYCYHHA", "name": "\u5bcc\u56fd\u6d66\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOPUCHENGHUIBAO12GEYUECHIYOUHUNHEA"}, "012829": {"suoxie": "FGPCHB12GYCYHHC", "name": "\u5bcc\u56fd\u6d66\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOPUCHENGHUIBAO12GEYUECHIYOUHUNHEC"}, "012830": {"suoxie": "HTFHBE", "name": "\u6c47\u6dfb\u5bcc\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUOBIE"}, "012831": {"suoxie": "NFZZXNYETFLJA", "name": "\u5357\u65b9\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGXINNENGYUANETFLIANJIEA"}, "012832": {"suoxie": "NFZZXNYETFLJC", "name": "\u5357\u65b9\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGXINNENGYUANETFLIANJIEC"}, "012833": {"suoxie": "JYHXYNCYQHHA", "name": "\u4ea4\u94f6\u9e3f\u4fe1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGXINYINIANCHIYOUQIHUNHEA"}, "012834": {"suoxie": "JYHXYNCYQHHC", "name": "\u4ea4\u94f6\u9e3f\u4fe1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGXINYINIANCHIYOUQIHUNHEC"}, "012835": {"suoxie": "ZSJQJXGPA", "name": "\u62db\u5546\u666f\u6c14\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGJINGQIJINGXUANGUPIAOA"}, "012836": {"suoxie": "ZSJQJXGPC", "name": "\u62db\u5546\u666f\u6c14\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGJINGQIJINGXUANGUPIAOC"}, "012837": {"suoxie": "HACESBDTXPHYZSFQA", "name": "\u534e\u5b89CES\u534a\u5bfc\u4f53\u82af\u7247\u884c\u4e1a\u6307\u6570\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCESBANDAOTIXINPIANHANGYEZHISHUFAQIA"}, "012838": {"suoxie": "HACESBDTXPHYZSFQC", "name": "\u534e\u5b89CES\u534a\u5bfc\u4f53\u82af\u7247\u884c\u4e1a\u6307\u6570\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCESBANDAOTIXINPIANHANGYEZHISHUFAQIC"}, "012839": {"suoxie": "DFHZHSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u667a\u534e\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGZHIHUASANNIANCHIYOUHUNHEA"}, "012840": {"suoxie": "DFHZHSNCYHHC", "name": "\u4e1c\u65b9\u7ea2\u667a\u534e\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGZHIHUASANNIANCHIYOUHUNHEC"}, "012841": {"suoxie": "HTBRJYHBC", "name": "\u534e\u6cf0\u67cf\u745e\u4ea4\u6613\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUIJIAOYIHUOBIC"}, "012842": {"suoxie": "YFDZZJGLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u519b\u5de5(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGJUNGONGLOFC"}, "012843": {"suoxie": "HFTHYYNDKZQFQS", "name": "\u6d77\u5bcc\u901a\u6052\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGHENGYIYINIANDINGKAIZHAIQUANFAQISHI"}, "012844": {"suoxie": "GYRXHX6GYCYHHA", "name": "\u5de5\u94f6\u745e\u4fe1\u6052\u51746\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINRUIXINHENGXING6GEYUECHIYOUHUNHEA"}, "012845": {"suoxie": "GYRXHX6GYCYHHC", "name": "\u5de5\u94f6\u745e\u4fe1\u6052\u51746\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINRUIXINHENGXING6GEYUECHIYOUHUNHEC"}, "012846": {"suoxie": "HYLCJXHH", "name": "\u6052\u8d8a\u84dd\u7b79\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUELANCHOUJINGXUANHUNHE"}, "012847": {"suoxie": "NAJJHBHHC", "name": "\u8bfa\u5b89\u79ef\u6781\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANJIJIHUIBAOHUNHEC"}, "012848": {"suoxie": "DCYXSHHHA", "name": "\u5927\u6210\u60a6\u4eab\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYUEXIANGSHENGHUOHUNHEA"}, "012849": {"suoxie": "DCYXSHHHC", "name": "\u5927\u6210\u60a6\u4eab\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYUEXIANGSHENGHUOHUNHEC"}, "012850": {"suoxie": "ZRDTJJ3GYCYHHA", "name": "\u4e2d\u878d\u4f4e\u78b3\u7ecf\u6d4e3\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGDITANJINGJI3GEYUECHIYOUHUNHEA"}, "012851": {"suoxie": "ZRDTJJ3GYCYHHC", "name": "\u4e2d\u878d\u4f4e\u78b3\u7ecf\u6d4e3\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGDITANJINGJI3GEYUECHIYOUHUNHEC"}, "012852": {"suoxie": "JSYJXFYNCYQHHA", "name": "\u5609\u5b9e\u8fdc\u89c1\u5148\u950b\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYUANJIANXIANFENGYINIANCHIYOUQIHUNHEA"}, "012853": {"suoxie": "JSYJXFYNCYQHHC", "name": "\u5609\u5b9e\u8fdc\u89c1\u5148\u950b\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYUANJIANXIANFENGYINIANCHIYOUQIHUNHEC"}, "012854": {"suoxie": "YDZZESG120CLZSA", "name": "\u82f1\u5927\u4e2d\u8bc1ESG120\u7b56\u7565\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINGDAZHONGZHENGESG120CELUEZHISHUA"}, "012855": {"suoxie": "YDZZESG120CLZSC", "name": "\u82f1\u5927\u4e2d\u8bc1ESG120\u7b56\u7565\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINGDAZHONGZHENGESG120CELUEZHISHUC"}, "012856": {"suoxie": "YHSYYNDKZ", "name": "\u94f6\u534e\u987a\u76ca\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUASHUNYIYINIANDINGKAIZHAI"}, "012857": {"suoxie": "HTFZZZYXFETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHUYAOXIAOFEIETFLIANJIEC"}, "012858": {"suoxie": "THRXLLZFQSA", "name": "\u5929\u5f18\u777f\u9009\u5229\u7387\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGRUIXUANLILVZHAIFAQISHIA"}, "012859": {"suoxie": "THRXLLZFQSC", "name": "\u5929\u5f18\u777f\u9009\u5229\u7387\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGRUIXUANLILVZHAIFAQISHIC"}, "012860": {"suoxie": "YFDBP500ZSRMBC", "name": "\u6613\u65b9\u8fbe\u6807\u666e500\u6307\u6570\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDABIAOPU500ZHISHURENMINBIC"}, "012861": {"suoxie": "YFDBP500ZSMYHC", "name": "\u6613\u65b9\u8fbe\u6807\u666e500\u6307\u6570\u7f8e\u5143\u6c47C", "style": "QDII", "pinyin": "YIFANGDABIAOPU500ZHISHUMEIYUANHUIC"}, "012862": {"suoxie": "HTFZZDCZTZSFQSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u7535\u6c60\u4e3b\u9898\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGDIANCHIZHUTIZHISHUFAQISHIA"}, "012863": {"suoxie": "HTFZZDCZTZSFQSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u7535\u6c60\u4e3b\u9898\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGDIANCHIZHUTIZHISHUFAQISHIC"}, "012864": {"suoxie": "YFDBPYLBJRMBC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u533b\u7597\u4fdd\u5065\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDABIAOPUYILIAOBAOJIANRENMINBIC"}, "012865": {"suoxie": "YFDBPYLBJMYHC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u533b\u7597\u4fdd\u5065\u7f8e\u5143\u6c47C", "style": "QDII", "pinyin": "YIFANGDABIAOPUYILIAOBAOJIANMEIYUANHUIC"}, "012866": {"suoxie": "YFDBPSWKJRMBC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u751f\u7269\u79d1\u6280\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDABIAOPUSHENGWUKEJIRENMINBIC"}, "012867": {"suoxie": "YFDBPSWKJMYHC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u751f\u7269\u79d1\u6280\u7f8e\u5143\u6c47C", "style": "QDII", "pinyin": "YIFANGDABIAOPUSHENGWUKEJIMEIYUANHUIC"}, "012868": {"suoxie": "YFDBPXXKJRMBC", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u4fe1\u606f\u79d1\u6280\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDABIAOPUXINXIKEJIRENMINBIC"}, "012869": {"suoxie": "YBPXXKJMYHC", "name": "\u6613\u6807\u666e\u4fe1\u606f\u79d1\u6280\u7f8e\u5143\u6c47C", "style": "QDII", "pinyin": "YIBIAOPUXINXIKEJIMEIYUANHUIC"}, "012870": {"suoxie": "YFDNSDK100RMBC", "name": "\u6613\u65b9\u8fbe\u7eb3\u65af\u8fbe\u514b100\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDANASIDAKE100RENMINBIC"}, "012871": {"suoxie": "YFDNSDK100MYHC", "name": "\u6613\u65b9\u8fbe\u7eb3\u65af\u8fbe\u514b100\u7f8e\u5143\u6c47C", "style": "QDII", "pinyin": "YIFANGDANASIDAKE100MEIYUANHUIC"}, "012872": {"suoxie": "YFDZXQY100LOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u5c0f\u4f01\u4e1a100(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGXIAOQIYE100LOFC"}, "012873": {"suoxie": "YFDZZGQGGLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u56fd\u4f01\u6539\u9769(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGUOQIGAIGELOFC"}, "012874": {"suoxie": "YFDZZQZZQGSZSLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFC"}, "012875": {"suoxie": "YFDSZ50ZSLOFC", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ZHISHULOFC"}, "012876": {"suoxie": "FRFYHHA", "name": "\u5bcc\u8363\u798f\u8000\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUYAOHUNHEA"}, "012877": {"suoxie": "FRFYHHC", "name": "\u5bcc\u8363\u798f\u8000\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGFUYAOHUNHEC"}, "012878": {"suoxie": "ZXJTLHJX6GYCYHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u91cf\u5316\u7cbe\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULIANGHUAJINGXUAN6GEYUECHIYOUHUNHEA"}, "012879": {"suoxie": "ZXJTLHJX6GYCYHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u91cf\u5316\u7cbe\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOULIANGHUAJINGXUAN6GEYUECHIYOUHUNHEC"}, "012880": {"suoxie": "GTJQYXHHA", "name": "\u56fd\u6cf0\u666f\u6c14\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINGQIYOUXUANHUNHEA"}, "012881": {"suoxie": "GTJQYXHHC", "name": "\u56fd\u6cf0\u666f\u6c14\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINGQIYOUXUANHUNHEC"}, "012882": {"suoxie": "GYKJLTETFFQSLJA", "name": "\u5de5\u94f6\u79d1\u6280\u9f99\u5934ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINKEJILONGTOUETFFAQISHILIANJIEA"}, "012883": {"suoxie": "GYKJLTETFFQSLJC", "name": "\u5de5\u94f6\u79d1\u6280\u9f99\u5934ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINKEJILONGTOUETFFAQISHILIANJIEC"}, "012884": {"suoxie": "HXGGTJXGPFQSLOFC", "name": "\u534e\u590f\u6e2f\u80a1\u901a\u7cbe\u9009\u80a1\u7968\u53d1\u8d77\u5f0f(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAGANGGUTONGJINGXUANGUPIAOFAQISHILOFC"}, "012885": {"suoxie": "HXZZGFCYZSFQSA", "name": "\u534e\u590f\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIA"}, "012886": {"suoxie": "HXZZGFCYZSFQSC", "name": "\u534e\u590f\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIC"}, "012887": {"suoxie": "HXKZZZQZQC", "name": "\u534e\u590f\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAXIAKEZHUANZHAIZENGQIANGZHAIQUANC"}, "012888": {"suoxie": "GYXRYNCYQHHA", "name": "\u5de5\u94f6\u5174\u745e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINGRUIYINIANCHIYOUQIHUNHEA"}, "012889": {"suoxie": "GYXRYNCYQHHC", "name": "\u5de5\u94f6\u5174\u745e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXINGRUIYINIANCHIYOUQIHUNHEC"}, "012890": {"suoxie": "DCJYZQC", "name": "\u5927\u6210\u666f\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGJINGYINGZHAIQUANC"}, "012891": {"suoxie": "AXXFYXHHC", "name": "\u5b89\u4fe1\u946b\u53d1\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINXINFAYOUXUANHUNHEC"}, "012892": {"suoxie": "AXYZQYSNCYHHA", "name": "\u5b89\u4fe1\u4f18\u8d28\u4f01\u4e1a\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINYOUZHIQIYESANNIANCHIYOUHUNHEA"}, "012893": {"suoxie": "AXYZQYSNCYHHC", "name": "\u5b89\u4fe1\u4f18\u8d28\u4f01\u4e1a\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINYOUZHIQIYESANNIANCHIYOUHUNHEC"}, "012894": {"suoxie": "THZZKCCY50ZSA", "name": "\u5929\u5f18\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGKECHUANGCHUANGYE50ZHISHUA"}, "012895": {"suoxie": "THZZKCCY50ZSC", "name": "\u5929\u5f18\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGKECHUANGCHUANGYE50ZHISHUC"}, "012896": {"suoxie": "HAHCZHJX3GYCYHHFOF", "name": "\u534e\u5b89\u6167\u8403\u7ec4\u5408\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHUICUIZUHEJINGXUAN3GEYUECHIYOUHUNHEFOF"}, "012897": {"suoxie": "ZOXLZQC", "name": "\u4e2d\u6b27\u5174\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINGLIZHAIQUANC"}, "012898": {"suoxie": "XYZZKCCY50ZSA", "name": "\u5174\u94f6\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENGKECHUANGCHUANGYE50ZHISHUA"}, "012899": {"suoxie": "XYZZKCCY50ZSC", "name": "\u5174\u94f6\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENGKECHUANGCHUANGYE50ZHISHUC"}, "012900": {"suoxie": "ZSCYBZSZQA", "name": "\u62db\u5546\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGCHUANGYEBANZHISHUZENGQIANGA"}, "012901": {"suoxie": "ZSCYBZSZQC", "name": "\u62db\u5546\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGCHUANGYEBANZHISHUZENGQIANGC"}, "012904": {"suoxie": "STMGXRYXYNCYHH", "name": "\u4e0a\u6295\u6469\u6839\u946b\u777f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENXINRUIYOUXUANYINIANCHIYOUHUNHE"}, "012907": {"suoxie": "PYZZKCCY50ZSA", "name": "\u9e4f\u626c\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENGKECHUANGCHUANGYE50ZHISHUA"}, "012908": {"suoxie": "PYZZKCCY50ZSC", "name": "\u9e4f\u626c\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENGKECHUANGCHUANGYE50ZHISHUC"}, "012909": {"suoxie": "PAYSWJPZSGYCYZQFOFA", "name": "\u5e73\u5b89\u76c8\u76db\u7a33\u5065\u914d\u7f6e\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238(FOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANYINGSHENGWENJIANPEIZHISANGEYUECHIYOUZHAIQUANFOFA"}, "012910": {"suoxie": "PAYSWJPZSGYCYZQFOFC", "name": "\u5e73\u5b89\u76c8\u76db\u7a33\u5065\u914d\u7f6e\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238(FOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANYINGSHENGWENJIANPEIZHISANGEYUECHIYOUZHAIQUANFOFC"}, "012911": {"suoxie": "TTHS300LHZQA", "name": "\u540c\u6cf0\u6caa\u6df1300\u91cf\u5316\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TONGTAIHUSHEN300LIANGHUAZENGQIANGA"}, "012912": {"suoxie": "TTHS300LHZQC", "name": "\u540c\u6cf0\u6caa\u6df1300\u91cf\u5316\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TONGTAIHUSHEN300LIANGHUAZENGQIANGC"}, "012913": {"suoxie": "FZFBQSLHHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u8d8b\u52bf\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGQUSHILINGHANGHUNHEA"}, "012914": {"suoxie": "FZFBQSLHHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u8d8b\u52bf\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGQUSHILINGHANGHUNHEC"}, "012915": {"suoxie": "ZOWL60TGDCYDZA", "name": "\u4e2d\u6b27\u7a33\u522960\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGOUWENLI60TIANGUNDONGCHIYOUDUANZHAIA"}, "012916": {"suoxie": "ZOWL60TGDCYDZC", "name": "\u4e2d\u6b27\u7a33\u522960\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGOUWENLI60TIANGUNDONGCHIYOUDUANZHAIC"}, "012917": {"suoxie": "PAYSLH1NCYHHA", "name": "\u5e73\u5b89\u4f18\u52bf\u9886\u822a1\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUSHILINGHANG1NIANCHIYOUHUNHEA"}, "012918": {"suoxie": "PAYSLH1NCYHHC", "name": "\u5e73\u5b89\u4f18\u52bf\u9886\u822a1\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUSHILINGHANG1NIANCHIYOUHUNHEC"}, "012919": {"suoxie": "HBWLZDHHC", "name": "\u534e\u5b9d\u672a\u6765\u4e3b\u5bfc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOWEILAIZHUDAOHUNHEC"}, "012920": {"suoxie": "YFDQQCZJXHHQDIIRMBA", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u6210\u957f\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDAQUANQIUCHENGZHANGJINGXUANHUNHEQDIIRENMINBIA"}, "012921": {"suoxie": "YFDQQCZJXHHQDIIMYXHA", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u6210\u957f\u7cbe\u9009\u6df7\u5408(QDII)\u7f8e\u5143\u73b0\u6c47A", "style": "QDII", "pinyin": "YIFANGDAQUANQIUCHENGZHANGJINGXUANHUNHEQDIIMEIYUANXIANHUIA"}, "012922": {"suoxie": "YFDQQCZJXHHQDIIRMBC", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u6210\u957f\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "YIFANGDAQUANQIUCHENGZHANGJINGXUANHUNHEQDIIRENMINBIC"}, "012923": {"suoxie": "YFDQQCZJXHHQDIIMYXHC", "name": "\u6613\u65b9\u8fbe\u5168\u7403\u6210\u957f\u7cbe\u9009\u6df7\u5408(QDII)\u7f8e\u5143\u73b0\u6c47C", "style": "QDII", "pinyin": "YIFANGDAQUANQIUCHENGZHANGJINGXUANHUNHEQDIIMEIYUANXIANHUIC"}, "012924": {"suoxie": "HXXSDHHMYXHQDII", "name": "\u534e\u590f\u65b0\u65f6\u4ee3\u6df7\u5408\u7f8e\u5143\u73b0\u6c47(QDII)", "style": "QDII", "pinyin": "HUAXIAXINSHIDAIHUNHEMEIYUANXIANHUIQDII"}, "012925": {"suoxie": "HXXSDHHMYXCQDII", "name": "\u534e\u590f\u65b0\u65f6\u4ee3\u6df7\u5408\u7f8e\u5143\u73b0\u949e(QDII)", "style": "QDII", "pinyin": "HUAXIAXINSHIDAIHUNHEMEIYUANXIANCHAOQDII"}, "012926": {"suoxie": "MSJYZZ500ZSZQA", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG500ZHISHUZENGQIANGA"}, "012927": {"suoxie": "MSJYZZ500ZSZQC", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG500ZHISHUZENGQIANGC"}, "012930": {"suoxie": "ZGJZXFGP", "name": "\u4e2d\u5e9a\u4ef7\u503c\u5148\u950b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGGENGJIAZHIXIANFENGGUPIAO"}, "012931": {"suoxie": "PASJY6GYCYZQA", "name": "\u5e73\u5b89\u53cc\u5b63\u76c86\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANSHUANGJIYING6GEYUECHIYOUZHAIQUANA"}, "012932": {"suoxie": "PASJY6GYCYZQC", "name": "\u5e73\u5b89\u53cc\u5b63\u76c86\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANSHUANGJIYING6GEYUECHIYOUZHAIQUANC"}, "012933": {"suoxie": "YFDWF90TGDCYDZA", "name": "\u6613\u65b9\u8fbe\u7a33\u4e3090\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENFENG90TIANGUNDONGCHIYOUDUANZHAIA"}, "012934": {"suoxie": "YFDWF90TGDCYDZC", "name": "\u6613\u65b9\u8fbe\u7a33\u4e3090\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENFENG90TIANGUNDONGCHIYOUDUANZHAIC"}, "012935": {"suoxie": "WJDXYNDKZFQS", "name": "\u4e07\u5bb6\u9f0e\u946b\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIADINGXINYINIANDINGKAIZHAIFAQISHI"}, "012936": {"suoxie": "MSJYJJPZ6GYCYHHFOF", "name": "\u6c11\u751f\u52a0\u94f6\u79ef\u6781\u914d\u7f6e6\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINJIJIPEIZHI6GEYUECHIYOUHUNHEFOF"}, "012937": {"suoxie": "DCHYYNDKZFQS", "name": "\u5927\u6210\u60e0\u4e1a\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIYEYINIANDINGKAIZHAIFAQISHI"}, "012938": {"suoxie": "CJHXZHZQA", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u6cd3\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNHONGZHAIQUANA"}, "012939": {"suoxie": "CJHXZHZQC", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u6cd3\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNHONGZHAIQUANC"}, "012940": {"suoxie": "ZTXYLHPZHHC", "name": "\u4e2d\u6cf0\u661f\u5143\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGTAIXINGYUANLINGHUOPEIZHIHUNHEC"}, "012941": {"suoxie": "GFTC90TGDCYZQA", "name": "\u5e7f\u53d1\u6dfb\u8d2290\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFATIANCAI90TIANGUNDONGCHIYOUZHAIQUANA"}, "012942": {"suoxie": "GFTC90TGDCYZQC", "name": "\u5e7f\u53d1\u6dfb\u8d2290\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFATIANCAI90TIANGUNDONGCHIYOUZHAIQUANC"}, "012943": {"suoxie": "GFWRLGYCYHHA", "name": "\u5e7f\u53d1\u7a33\u777f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAWENRUILIUGEYUECHIYOUHUNHEA"}, "012944": {"suoxie": "GFWRLGYCYHHC", "name": "\u5e7f\u53d1\u7a33\u777f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAWENRUILIUGEYUECHIYOUHUNHEC"}, "012945": {"suoxie": "NFBYHHA", "name": "\u5357\u65b9\u5b9d\u88d5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOYUHUNHEA"}, "012946": {"suoxie": "NFBYHHC", "name": "\u5357\u65b9\u5b9d\u88d5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGBAOYUHUNHEC"}, "012947": {"suoxie": "BJBLYNDKCZZQFQS", "name": "\u767e\u5609\u767e\u5229\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAIJIABAILIYINIANDINGKAICHUNZHAIZHAIQUANFAQISHI"}, "012948": {"suoxie": "XZQQHLYNDKZQ", "name": "\u5174\u8bc1\u5168\u7403\u6052\u5229\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGZHENGQUANQIUHENGLIYINIANDINGKAIZHAIQUAN"}, "012949": {"suoxie": "DFHZRZX18GYCYHHA", "name": "\u4e1c\u65b9\u7ea2\u62db\u745e\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHAORUIZHENXUAN18GEYUECHIYOUHUNHEA"}, "012950": {"suoxie": "DFHZRZX18GYCYHHC", "name": "\u4e1c\u65b9\u7ea2\u62db\u745e\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGZHAORUIZHENXUAN18GEYUECHIYOUHUNHEC"}, "012951": {"suoxie": "HTFXXTLLGYCYHHA", "name": "\u6c47\u6dfb\u5bcc\u946b\u4eab\u6dfb\u5229\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUXINXIANGTIANLILIUGEYUECHIYOUHUNHEA"}, "012952": {"suoxie": "HTFXXTLLGYCYHHC", "name": "\u6c47\u6dfb\u5bcc\u946b\u4eab\u6dfb\u5229\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUXINXIANGTIANLILIUGEYUECHIYOUHUNHEC"}, "012953": {"suoxie": "HTBRHLHHA", "name": "\u534e\u6cf0\u67cf\u745e\u6052\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAIRUIHENGLIHUNHEA"}, "012954": {"suoxie": "HTBRHLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u6052\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUATAIBAIRUIHENGLIHUNHEC"}, "012955": {"suoxie": "GSABWS6GYCYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u76db6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENSHENG6GEYUECHIYOUHUNHEA"}, "012956": {"suoxie": "GSABWS6GYCYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u7a33\u76db6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOSHOUANBAOWENSHENG6GEYUECHIYOUHUNHEC"}, "012957": {"suoxie": "JS60TGDCYDZA", "name": "\u5609\u5b9e60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHI60TIANGUNDONGCHIYOUDUANZHAIA"}, "012958": {"suoxie": "JS60TGDCYDZC", "name": "\u5609\u5b9e60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHI60TIANGUNDONGCHIYOUDUANZHAIC"}, "012959": {"suoxie": "PAYYWJHB1NCYHHFOFA", "name": "\u5e73\u5b89\u76c8\u60a6\u7a33\u8fdb\u56de\u62a51\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANYINGYUEWENJINHUIBAO1NIANCHIYOUHUNHEFOFA"}, "012960": {"suoxie": "PAYYWJHB1NCYHHFOFC", "name": "\u5e73\u5b89\u76c8\u60a6\u7a33\u8fdb\u56de\u62a51\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANYINGYUEWENJINHUIBAO1NIANCHIYOUHUNHEFOFC"}, "012961": {"suoxie": "ZSCYBDPETFLJA", "name": "\u62db\u5546\u521b\u4e1a\u677f\u5927\u76d8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGCHUANGYEBANDAPANETFLIANJIEA"}, "012962": {"suoxie": "ZSCYBDPETFLJC", "name": "\u62db\u5546\u521b\u4e1a\u677f\u5927\u76d8ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGCHUANGYEBANDAPANETFLIANJIEC"}, "012963": {"suoxie": "ZSWJPHHHA", "name": "\u62db\u5546\u7a33\u5065\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGWENJIANPINGHENGHUNHEA"}, "012964": {"suoxie": "ZSWJPHHHC", "name": "\u62db\u5546\u7a33\u5065\u5e73\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGWENJIANPINGHENGHUNHEC"}, "012965": {"suoxie": "ZSRT1NCYHHA", "name": "\u62db\u5546\u745e\u6cf01\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUITAI1NIANCHIYOUHUNHEA"}, "012966": {"suoxie": "ZSRT1NCYHHC", "name": "\u62db\u5546\u745e\u6cf01\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGRUITAI1NIANCHIYOUHUNHEC"}, "012967": {"suoxie": "GFHYYXSNCYQHHA", "name": "\u5e7f\u53d1\u884c\u4e1a\u4e25\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHANGYEYANXUANSANNIANCHIYOUQIHUNHEA"}, "012968": {"suoxie": "GFHYYXSNCYQHHC", "name": "\u5e7f\u53d1\u884c\u4e1a\u4e25\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHANGYEYANXUANSANNIANCHIYOUQIHUNHEC"}, "012969": {"suoxie": "PHGZBDTXPETFLJA", "name": "\u9e4f\u534e\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGBANDAOTIXINPIANETFLIANJIEA"}, "012970": {"suoxie": "PHGZBDTXPETFLJC", "name": "\u9e4f\u534e\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGBANDAOTIXINPIANETFLIANJIEC"}, "012971": {"suoxie": "DWXFCZHHA", "name": "\u4e1c\u5434\u6d88\u8d39\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXIAOFEICHENGZHANGHUNHEA"}, "012972": {"suoxie": "DWXFCZHHC", "name": "\u4e1c\u5434\u6d88\u8d39\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXIAOFEICHENGZHANGHUNHEC"}, "012973": {"suoxie": "GT800QCYLBJETFLJA", "name": "\u56fd\u6cf0800\u6c7d\u8f66\u4e0e\u96f6\u90e8\u4ef6ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAI800QICHEYULINGBUJIANETFLIANJIEA"}, "012974": {"suoxie": "GT800QCYLBJETFLJC", "name": "\u56fd\u6cf0800\u6c7d\u8f66\u4e0e\u96f6\u90e8\u4ef6ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAI800QICHEYULINGBUJIANETFLIANJIEC"}, "012975": {"suoxie": "XBLDTZHHHFQA", "name": "\u897f\u90e8\u5229\u5f97\u78b3\u4e2d\u548c\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDETANZHONGHEHUNHEFAQIA"}, "012976": {"suoxie": "XBLDTZHHHFQC", "name": "\u897f\u90e8\u5229\u5f97\u78b3\u4e2d\u548c\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDETANZHONGHEHUNHEFAQIC"}, "012977": {"suoxie": "RDXHLH6GYCYHHA", "name": "\u745e\u8fbe\u946b\u7ea2\u91cf\u53166\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIDAXINHONGLIANGHUA6GEYUECHIYOUHUNHEA"}, "012978": {"suoxie": "RDXHLH6GYCYHHC", "name": "\u745e\u8fbe\u946b\u7ea2\u91cf\u53166\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RUIDAXINHONGLIANGHUA6GEYUECHIYOUHUNHEC"}, "012979": {"suoxie": "DCHSKJETFFQSLJA", "name": "\u5927\u6210\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "QDII", "pinyin": "DACHENGHENGSHENGKEJIETFFAQISHILIANJIEA"}, "012980": {"suoxie": "DCHSKJETFFQSLJC", "name": "\u5927\u6210\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "QDII", "pinyin": "DACHENGHENGSHENGKEJIETFFAQISHILIANJIEC"}, "012981": {"suoxie": "HXCXYYLTHHA", "name": "\u534e\u590f\u521b\u65b0\u533b\u836f\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINYIYAOLONGTOUHUNHEA"}, "012982": {"suoxie": "HXCXYYLTHHC", "name": "\u534e\u590f\u521b\u65b0\u533b\u836f\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINYIYAOLONGTOUHUNHEC"}, "012985": {"suoxie": "PAYSHB1NCYHHA", "name": "\u5e73\u5b89\u4f18\u52bf\u56de\u62a51\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUSHIHUIBAO1NIANCHIYOUHUNHEA"}, "012986": {"suoxie": "PAYSHB1NCYHHC", "name": "\u5e73\u5b89\u4f18\u52bf\u56de\u62a51\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANYOUSHIHUIBAO1NIANCHIYOUHUNHEC"}, "012987": {"suoxie": "JHJMHHA", "name": "\u5609\u5408\u9526\u660e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINMINGHUNHEA"}, "012988": {"suoxie": "JHJMHHC", "name": "\u5609\u5408\u9526\u660e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINMINGHUNHEC"}, "012989": {"suoxie": "THGZGGT50ZSA", "name": "\u5929\u5f18\u56fd\u8bc1\u6e2f\u80a1\u901a50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGGANGGUTONG50ZHISHUA"}, "012990": {"suoxie": "THGZGGT50ZSC", "name": "\u5929\u5f18\u56fd\u8bc1\u6e2f\u80a1\u901a50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGGANGGUTONG50ZHISHUC"}, "012991": {"suoxie": "DCMXAYYNCYHHA", "name": "\u5927\u6210\u6c11\u4eab\u5b89\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGMINXIANGANYINGYINIANCHIYOUHUNHEA"}, "012992": {"suoxie": "DCMXAYYNCYHHC", "name": "\u5927\u6210\u6c11\u4eab\u5b89\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DACHENGMINXIANGANYINGYINIANCHIYOUHUNHEC"}, "012993": {"suoxie": "HTFPPLYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u54c1\u724c\u529b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUPINPAILIYINIANCHIYOUHUNHEA"}, "012994": {"suoxie": "HTFPPLYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u54c1\u724c\u529b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUPINPAILIYINIANCHIYOUHUNHEC"}, "012995": {"suoxie": "JSCLSYSNCYQHH", "name": "\u5609\u5b9e\u7b56\u7565\u89c6\u91ce\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUESHIYESANNIANCHIYOUQIHUNHE"}, "012996": {"suoxie": "BSYL2035SNCYHHFQFOF", "name": "\u535a\u65f6\u517b\u80012035\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIYANGLAO2035SANNIANCHIYOUHUNHEFAQIFOF"}, "012997": {"suoxie": "PHYXHBHHC", "name": "\u9e4f\u534e\u4f18\u9009\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAYOUXUANHUIBAOHUNHEC"}, "012998": {"suoxie": "ZSWWHHA", "name": "\u62db\u5546\u7a33\u65fa\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGWENWANGHUNHEA"}, "012999": {"suoxie": "ZSWWHHC", "name": "\u62db\u5546\u7a33\u65fa\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHAOSHANGWENWANGHUNHEC"}, "013000": {"suoxie": "GFSZYNCYHHA", "name": "\u5e7f\u53d1\u76db\u6cfd\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGZEYINIANCHIYOUHUNHEA"}, "013001": {"suoxie": "GFSZYNCYHHC", "name": "\u5e7f\u53d1\u76db\u6cfd\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFASHENGZEYINIANCHIYOUHUNHEC"}, "013002": {"suoxie": "HTFSYKQHBC", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u94b1\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIQIANHUOBIC"}, "013003": {"suoxie": "HTFSYKQHBD", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u94b1\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIQIANHUOBID"}, "013004": {"suoxie": "GTJZLHGPA", "name": "\u56fd\u6cf0\u4ef7\u503c\u9886\u822a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIJIAZHILINGHANGGUPIAOA"}, "013005": {"suoxie": "GTJZLHGPC", "name": "\u56fd\u6cf0\u4ef7\u503c\u9886\u822a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIJIAZHILINGHANGGUPIAOC"}, "013006": {"suoxie": "STMGJQZXHHA", "name": "\u4e0a\u6295\u6469\u6839\u666f\u6c14\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENJINGQIZHENXUANHUNHEA"}, "013007": {"suoxie": "STMGJQZXHHC", "name": "\u4e0a\u6295\u6469\u6839\u666f\u6c14\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENJINGQIZHENXUANHUNHEC"}, "013008": {"suoxie": "CHWN6GYDKZ", "name": "\u6df3\u539a\u7a33\u5b816\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUNHOUWENNING6GEYUEDINGKAIZHAI"}, "013009": {"suoxie": "WJGGTJXHHA", "name": "\u4e07\u5bb6\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAGANGGUTONGJINGXUANHUNHEA"}, "013010": {"suoxie": "WJGGTJXHHC", "name": "\u4e07\u5bb6\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAGANGGUTONGJINGXUANHUNHEC"}, "013011": {"suoxie": "GYZZCXYCYETFFQSLJA", "name": "\u5de5\u94f6\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGCHUANGXINYAOCHANYEETFFAQISHILIANJIEA"}, "013012": {"suoxie": "GYZZCXYCYETFFQSLJC", "name": "\u5de5\u94f6\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGCHUANGXINYAOCHANYEETFFAQISHILIANJIEC"}, "013013": {"suoxie": "HXZZXNYQCETFFQSLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINNENGYUANQICHEETFFAQISHILIANJIEA"}, "013014": {"suoxie": "HXZZXNYQCETFFQSLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINNENGYUANQICHEETFFAQISHILIANJIEC"}, "013015": {"suoxie": "HFTHXHHA", "name": "\u6d77\u5bcc\u901a\u60e0\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGHUIXINHUNHEA"}, "013016": {"suoxie": "HFTHXHHC", "name": "\u6d77\u5bcc\u901a\u60e0\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGHUIXINHUNHEC"}, "013017": {"suoxie": "CJSY6GYCYZQFQSA", "name": "\u957f\u6c5f\u53cc\u76c86\u4e2a\u6708\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGSHUANGYING6GEYUECHIYOUZHAIQUANFAQISHIA"}, "013018": {"suoxie": "CJSY6GYCYZQFQSC", "name": "\u957f\u6c5f\u53cc\u76c86\u4e2a\u6708\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGSHUANGYING6GEYUECHIYOUZHAIQUANFAQISHIC"}, "013019": {"suoxie": "GTZZQZJZCLETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIANZHUCAILIAOETFLIANJIEA"}, "013020": {"suoxie": "GTZZQZJZCLETFLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIANZHUCAILIAOETFLIANJIEC"}, "013021": {"suoxie": "JXXRYNCYHH", "name": "\u5efa\u4fe1\u5174\u6da6\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINXINGRUNYINIANCHIYOUHUNHE"}, "013022": {"suoxie": "BSYL2040WNCYHHFQFOF", "name": "\u535a\u65f6\u517b\u80012040\u4e94\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIYANGLAO2040WUNIANCHIYOUHUNHEFAQIFOF"}, "013023": {"suoxie": "PAJHYX1NCYHHA", "name": "\u5e73\u5b89\u5747\u8861\u4f18\u90091\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJUNHENGYOUXUAN1NIANCHIYOUHUNHEA"}, "013024": {"suoxie": "PAJHYX1NCYHHC", "name": "\u5e73\u5b89\u5747\u8861\u4f18\u90091\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJUNHENGYOUXUAN1NIANCHIYOUHUNHEC"}, "013025": {"suoxie": "FGHGCLLHPZHHC", "name": "\u5bcc\u56fd\u5b8f\u89c2\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOHONGGUANCELUELINGHUOPEIZHIHUNHEC"}, "013026": {"suoxie": "YHLHJZHHC", "name": "\u94f6\u6cb3\u91cf\u5316\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELIANGHUAJIAZHIHUNHEC"}, "013027": {"suoxie": "YHFJSPYLJXHHLOFC", "name": "\u94f6\u534e\u5bcc\u4e45\u98df\u54c1\u996e\u6599\u7cbe\u9009\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFUJIUSHIPINYINLIAOJINGXUANHUNHELOFC"}, "013028": {"suoxie": "HYPZSHHHFQS", "name": "\u6052\u8d8a\u54c1\u8d28\u751f\u6d3b\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEPINZHISHENGHUOHUNHEFAQISHI"}, "013029": {"suoxie": "JSXTYNCYHHA", "name": "\u5609\u5b9e\u946b\u6cf0\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTAIYINIANCHIYOUHUNHEA"}, "013030": {"suoxie": "JSXTYNCYHHC", "name": "\u5609\u5b9e\u946b\u6cf0\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIXINTAIYINIANCHIYOUHUNHEC"}, "013031": {"suoxie": "MSJYZZ800ZSZQFQSA", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1800\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG800ZHISHUZENGQIANGFAQISHIA"}, "013032": {"suoxie": "MSJYZZ800ZSZQFQSC", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1800\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENG800ZHISHUZENGQIANGFAQISHIC"}, "013035": {"suoxie": "FGZZJGZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u519b\u5de5\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJUNGONGZHISHULOFC"}, "013036": {"suoxie": "NFWY60TGDCYZDZC", "name": "\u5357\u65b9\u65fa\u514360\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGWANGYUAN60TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013037": {"suoxie": "CCDJKHHA", "name": "\u957f\u57ce\u5927\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGDAJIANKANGHUNHEA"}, "013038": {"suoxie": "CCDJKHHC", "name": "\u957f\u57ce\u5927\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGDAJIANKANGHUNHEC"}, "013039": {"suoxie": "HFTXRHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGXINRUNHUNHEA"}, "013040": {"suoxie": "HFTXRHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGXINRUNHUNHEC"}, "013041": {"suoxie": "PYJPYNCYHHA", "name": "\u9e4f\u626c\u666f\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGPUYINIANCHIYOUHUNHEA"}, "013042": {"suoxie": "PYJPYNCYHHC", "name": "\u9e4f\u626c\u666f\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGYANGJINGPUYINIANCHIYOUHUNHEC"}, "013043": {"suoxie": "BSYLMB2045WNCYHHFQFOF", "name": "\u535a\u65f6\u517b\u8001\u76ee\u68072045\u4e94\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANGLAOMUBIAO2045WUNIANCHIYOUHUNHEFAQIFOF"}, "013044": {"suoxie": "FGGJAQZTHHC", "name": "\u5bcc\u56fd\u56fd\u5bb6\u5b89\u5168\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGUOJIAANQUANZHUTIHUNHEC"}, "013045": {"suoxie": "FGNXZZHHC", "name": "\u5bcc\u56fd\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUONEIXUZENGZHANGHUNHEC"}, "013046": {"suoxie": "FGCYSJHHC", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHANYESHENGJIHUNHEC"}, "013047": {"suoxie": "FGPZSHHHC", "name": "\u5bcc\u56fd\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOPINZHISHENGHUOHUNHEC"}, "013048": {"suoxie": "FGZZXNYQCZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINNENGYUANQICHEZHISHULOFC"}, "013049": {"suoxie": "XYNYGXGPA", "name": "\u5174\u4e1a\u80fd\u6e90\u9769\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "XINGYENENGYUANGEXINGUPIAOA"}, "013050": {"suoxie": "XYNYGXGPC", "name": "\u5174\u4e1a\u80fd\u6e90\u9769\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "XINGYENENGYUANGEXINGUPIAOC"}, "013051": {"suoxie": "HQZXZYHHA", "name": "\u6c47\u6cc9\u81fb\u5fc3\u81f4\u8fdc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANZHENXINZHIYUANHUNHEA"}, "013052": {"suoxie": "HQZXZYHHC", "name": "\u6c47\u6cc9\u81fb\u5fc3\u81f4\u8fdc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANZHENXINZHIYUANHUNHEC"}, "013053": {"suoxie": "THGZLTJDZSA", "name": "\u5929\u5f18\u56fd\u8bc1\u9f99\u5934\u5bb6\u7535\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGLONGTOUJIADIANZHISHUA"}, "013054": {"suoxie": "THGZLTJDZSC", "name": "\u5929\u5f18\u56fd\u8bc1\u9f99\u5934\u5bb6\u7535\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGLONGTOUJIADIANZHISHUC"}, "013055": {"suoxie": "JSCLJYHHFQSA", "name": "\u5609\u5b9e\u7b56\u7565\u673a\u9047\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUEJIYUHUNHEFAQISHIA"}, "013056": {"suoxie": "JSCLJYHHFQSC", "name": "\u5609\u5b9e\u7b56\u7565\u673a\u9047\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUEJIYUHUNHEFAQISHIC"}, "013057": {"suoxie": "TXHXLLZZQA", "name": "\u6cf0\u4fe1\u6c47\u4eab\u5229\u7387\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINHUIXIANGLILVZHAIZHAIQUANA"}, "013058": {"suoxie": "TXHXLLZZQC", "name": "\u6cf0\u4fe1\u6c47\u4eab\u5229\u7387\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINHUIXIANGLILVZHAIZHAIQUANC"}, "013059": {"suoxie": "GYPHYLSNCYHHFQFOF", "name": "\u5de5\u94f6\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINPINGHENGYANGLAOSANNIANCHIYOUHUNHEFAQIFOF"}, "013060": {"suoxie": "GYYLMB2060WNCYHHFQFOF", "name": "\u5de5\u94f6\u517b\u8001\u76ee\u68072060\u4e94\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYANGLAOMUBIAO2060WUNIANCHIYOUHUNHEFAQIFOF"}, "013061": {"suoxie": "BSYLMB2050WNCYHHFQFOF", "name": "\u535a\u65f6\u517b\u8001\u76ee\u68072050\u4e94\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANGLAOMUBIAO2050WUNIANCHIYOUHUNHEFAQIFOF"}, "013062": {"suoxie": "GSABACCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u8bda\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANCHENGCHUNZHAIYINIANDINGKAIZHAI"}, "013063": {"suoxie": "GFJYYNCYZQA", "name": "\u5e7f\u53d1\u96c6\u76ca\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYIYINIANCHIYOUZHAIQUANA"}, "013064": {"suoxie": "GFJYYNCYZQC", "name": "\u5e7f\u53d1\u96c6\u76ca\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYIYINIANCHIYOUZHAIQUANC"}, "013065": {"suoxie": "GTLZ90TGDCYZDZA", "name": "\u56fd\u6cf0\u5229\u6cfd90\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIZE90TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "013066": {"suoxie": "GTLZ90TGDCYZDZC", "name": "\u56fd\u6cf0\u5229\u6cfd90\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAILIZE90TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013067": {"suoxie": "FADZXPLGYCYHHFQ", "name": "\u5bcc\u5b89\u8fbe\u4e2d\u5c0f\u76d8\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDAZHONGXIAOPANLIUGEYUECHIYOUHUNHEFAQI"}, "013068": {"suoxie": "BSSYX60TGDCYZQA", "name": "\u535a\u65f6\u53cc\u6708\u4eab60\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGYUEXIANG60TIANGUNDONGCHIYOUZHAIQUANA"}, "013069": {"suoxie": "BSSYX60TGDCYZQC", "name": "\u535a\u65f6\u53cc\u6708\u4eab60\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHISHUANGYUEXIANG60TIANGUNDONGCHIYOUZHAIQUANC"}, "013070": {"suoxie": "HXPB15NZJZA", "name": "\u534e\u590f\u5f6d\u535a1-5\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAPENGBO15NIANZHENGJINZHAIA"}, "013071": {"suoxie": "HXPB15NZJZC", "name": "\u534e\u590f\u5f6d\u535a1-5\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIAPENGBO15NIANZHENGJINZHAIC"}, "013072": {"suoxie": "TXYLFWHHFQSA", "name": "\u6cf0\u4fe1\u533b\u7597\u670d\u52a1\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINYILIAOFUWUHUNHEFAQISHIA"}, "013073": {"suoxie": "TXYLFWHHFQSC", "name": "\u6cf0\u4fe1\u533b\u7597\u670d\u52a1\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINYILIAOFUWUHUNHEFAQISHIC"}, "013074": {"suoxie": "YHHS300JZZSC", "name": "\u94f6\u6cb3\u6caa\u6df1300\u4ef7\u503c\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEHUSHEN300JIAZHIZHISHUC"}, "013075": {"suoxie": "JXXY90TGDZDZA", "name": "\u5efa\u4fe1\u946b\u60a690\u5929\u6eda\u52a8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINXINYUE90TIANGUNDONGZHONGDUANZHAIA"}, "013076": {"suoxie": "JXXY90TGDZDZC", "name": "\u5efa\u4fe1\u946b\u60a690\u5929\u6eda\u52a8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINXINYUE90TIANGUNDONGZHONGDUANZHAIC"}, "013077": {"suoxie": "YYQYZQ", "name": "\u6c38\u8d62\u4e7e\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGQIANYIZHAIQUAN"}, "013078": {"suoxie": "FZFBCLLDHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u7b56\u7565\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCELUELUNDONGHUNHEA"}, "013079": {"suoxie": "FZFBCLLDHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u7b56\u7565\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCELUELUNDONGHUNHEC"}, "013080": {"suoxie": "XCZZ800YYZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u533b\u836f\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800YIYAOZHISHULOFC"}, "013081": {"suoxie": "XCZZ800YSZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u6709\u8272\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800YOUSEZHISHULOFC"}, "013082": {"suoxie": "XCZZJJGCZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u57fa\u5efa\u5de5\u7a0b\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGJIJIANGONGCHENGZHISHULOFC"}, "013083": {"suoxie": "XCZZXXAQZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u4fe1\u606f\u5b89\u5168\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGXINXIANQUANZHISHULOFC"}, "013084": {"suoxie": "XCZZZNJJZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u667a\u80fd\u5bb6\u5c45\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGZHINENGJIAJUZHISHULOFC"}, "013085": {"suoxie": "SWLXLTHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e50\u540c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINLETONGHUNHEA"}, "013086": {"suoxie": "SWLXLTHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u4e50\u540c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINLETONGHUNHEC"}, "013087": {"suoxie": "ZJYYYNDKZQ", "name": "\u4e2d\u52a0\u4f18\u60a6\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYOUYUEYINIANDINGKAIZHAIQUAN"}, "013088": {"suoxie": "HSJYYLMB2040SNCYHHFQSFOF", "name": "\u534e\u5546\u5609\u9038\u517b\u8001\u76ee\u68072040\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJIAYIYANGLAOMUBIAO2040SANNIANCHIYOUHUNHEFAQISHIFOF"}, "013089": {"suoxie": "THQJJX3GYCYHHFQSFOFA", "name": "\u5929\u5f18\u65d7\u8230\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGQIJIANJINGXUAN3GEYUECHIYOUHUNHEFAQISHIFOFA"}, "013090": {"suoxie": "THQJJX3GYCYHHFQSFOFC", "name": "\u5929\u5f18\u65d7\u8230\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGQIJIANJINGXUAN3GEYUECHIYOUHUNHEFAQISHIFOFC"}, "013091": {"suoxie": "STMGJHYXHHA", "name": "\u4e0a\u6295\u6469\u6839\u5747\u8861\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENJUNHENGYOUXUANHUNHEA"}, "013092": {"suoxie": "STMGJHYXHHC", "name": "\u4e0a\u6295\u6469\u6839\u5747\u8861\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENJUNHENGYOUXUANHUNHEC"}, "013093": {"suoxie": "HTFKJLTZSA", "name": "\u6c47\u6dfb\u5bcc\u79d1\u6280\u9f99\u5934\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUKEJILONGTOUZHISHUA"}, "013094": {"suoxie": "HTFKJLTZSC", "name": "\u6c47\u6dfb\u5bcc\u79d1\u6280\u9f99\u5934\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUKEJILONGTOUZHISHUC"}, "013095": {"suoxie": "AXYJCZHHA", "name": "\u5b89\u4fe1\u8fdc\u89c1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINYUANJIANCHENGZHANGHUNHEA"}, "013096": {"suoxie": "AXYJCZHHC", "name": "\u5b89\u4fe1\u8fdc\u89c1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINYUANJIANCHENGZHANGHUNHEC"}, "013097": {"suoxie": "CTZGSYZQFQSA", "name": "\u8d22\u901a\u8d44\u7ba1\u53cc\u76c8\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGZIGUANSHUANGYINGZHAIQUANFAQISHIA"}, "013098": {"suoxie": "CTZGSYZQFQSC", "name": "\u8d22\u901a\u8d44\u7ba1\u53cc\u76c8\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGZIGUANSHUANGYINGZHAIQUANFAQISHIC"}, "013099": {"suoxie": "ZSWLZDZ90TCYQZQA", "name": "\u62db\u5546\u7a33\u4e50\u4e2d\u77ed\u503a90\u5929\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENLEZHONGDUANZHAI90TIANCHIYOUQIZHAIQUANA"}, "013100": {"suoxie": "ZSWLZDZ90TCYQZQC", "name": "\u62db\u5546\u7a33\u4e50\u4e2d\u77ed\u503a90\u5929\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENLEZHONGDUANZHAI90TIANCHIYOUQIZHAIQUANC"}, "013101": {"suoxie": "HXWFLGYCYHHA", "name": "\u534e\u590f\u7a33\u798f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAWENFULIUGEYUECHIYOUHUNHEA"}, "013102": {"suoxie": "HXWFLGYCYHHC", "name": "\u534e\u590f\u7a33\u798f\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAWENFULIUGEYUECHIYOUHUNHEC"}, "013103": {"suoxie": "BSXNYZTHHA", "name": "\u535a\u65f6\u65b0\u80fd\u6e90\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINNENGYUANZHUTIHUNHEA"}, "013104": {"suoxie": "BSXNYZTHHC", "name": "\u535a\u65f6\u65b0\u80fd\u6e90\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINNENGYUANZHUTIHUNHEC"}, "013105": {"suoxie": "HAZZGFCYZSFQSA", "name": "\u534e\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIA"}, "013106": {"suoxie": "HAZZGFCYZSFQSC", "name": "\u534e\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIC"}, "013107": {"suoxie": "HXXJZZLTHHA", "name": "\u534e\u590f\u5148\u8fdb\u5236\u9020\u9f99\u5934\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANJINZHIZAOLONGTOUHUNHEA"}, "013108": {"suoxie": "HXXJZZLTHHC", "name": "\u534e\u590f\u5148\u8fdb\u5236\u9020\u9f99\u5934\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANJINZHIZAOLONGTOUHUNHEC"}, "013109": {"suoxie": "HXYSJZYNCYHHA", "name": "\u534e\u590f\u4f18\u52bf\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYOUSHIJIAZHIYINIANCHIYOUHUNHEA"}, "013110": {"suoxie": "HXYSJZYNCYHHC", "name": "\u534e\u590f\u4f18\u52bf\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAYOUSHIJIAZHIYINIANCHIYOUHUNHEC"}, "013111": {"suoxie": "ZJAY30TGDCYDZFQA", "name": "\u4e2d\u91d1\u5b89\u76ca30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGJINANYI30TIANGUNDONGCHIYOUDUANZHAIFAQIA"}, "013112": {"suoxie": "ZJAY30TGDCYDZFQC", "name": "\u4e2d\u91d1\u5b89\u76ca30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGJINANYI30TIANGUNDONGCHIYOUDUANZHAIFAQIC"}, "013113": {"suoxie": "BSBYWJ6GYCYHHA", "name": "\u535a\u65f6\u535a\u76c8\u7a33\u50656\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIBOYINGWENJIAN6GEYUECHIYOUHUNHEA"}, "013114": {"suoxie": "BSBYWJ6GYCYHHC", "name": "\u535a\u65f6\u535a\u76c8\u7a33\u50656\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIBOYINGWENJIAN6GEYUECHIYOUHUNHEC"}, "013115": {"suoxie": "XYJRZ3GYDK", "name": "\u946b\u5143\u91d1\u878d\u503a3\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANJINRONGZHAI3GEYUEDINGKAI"}, "013116": {"suoxie": "HAWTJKHHC", "name": "\u534e\u5b89\u6587\u4f53\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANWENTIJIANKANGHUNHEC"}, "013119": {"suoxie": "ZXBCZZ500ZSLOFC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u8bc1500\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGXINBAOCHENGZHONGZHENG500ZHISHULOFC"}, "013120": {"suoxie": "XCHS300ZSLOFC", "name": "\u4fe1\u8bda\u6caa\u6df1300\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGHUSHEN300ZHISHULOFC"}, "013121": {"suoxie": "XCZZ800JRLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u91d1\u878d(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800JINRONGLOFC"}, "013122": {"suoxie": "XCZZTMTCYZTZSLOFC", "name": "\u4fe1\u8bda\u4e2d\u8bc1TMT\u4ea7\u4e1a\u4e3b\u9898\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGTMTCHANYEZHUTIZHISHULOFC"}, "013123": {"suoxie": "HTFJXHXYSYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u7cbe\u9009\u6838\u5fc3\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJINGXUANHEXINYOUSHIYINIANCHIYOUHUNHEA"}, "013124": {"suoxie": "HTFJXHXYSYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u7cbe\u9009\u6838\u5fc3\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJINGXUANHEXINYOUSHIYINIANCHIYOUHUNHEC"}, "013125": {"suoxie": "HXZZXFSPYLCYZTETFFQSLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4ea7\u4e1a\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXIFENSHIPINYINLIAOCHANYEZHUTIETFFAQISHILIANJIEA"}, "013126": {"suoxie": "HXZZXFSPYLCYZTETFFQSLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4ea7\u4e1a\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXIFENSHIPINYINLIAOCHANYEZHUTIETFFAQISHILIANJIEC"}, "013127": {"suoxie": "HTFHSKJZSFQQDIIA", "name": "\u6c47\u6dfb\u5bcc\u6052\u751f\u79d1\u6280\u6307\u6570\u53d1\u8d77(QDII)A", "style": "QDII", "pinyin": "HUITIANFUHENGSHENGKEJIZHISHUFAQIQDIIA"}, "013128": {"suoxie": "HTFHSKJZSFQQDIIC", "name": "\u6c47\u6dfb\u5bcc\u6052\u751f\u79d1\u6280\u6307\u6570\u53d1\u8d77(QDII)C", "style": "QDII", "pinyin": "HUITIANFUHENGSHENGKEJIZHISHUFAQIQDIIC"}, "013129": {"suoxie": "HTFXFLTZSFQA", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u9f99\u5934\u6307\u6570\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUXIAOFEILONGTOUZHISHUFAQIA"}, "013130": {"suoxie": "HTFXFLTZSFQC", "name": "\u6c47\u6dfb\u5bcc\u6d88\u8d39\u9f99\u5934\u6307\u6570\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUXIAOFEILONGTOUZHISHUFAQIC"}, "013131": {"suoxie": "XBLDFTZQC", "name": "\u897f\u90e8\u5229\u5f97\u6ca3\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEFENGTAIZHAIQUANC"}, "013132": {"suoxie": "CJHXWYMTGPFQA", "name": "\u521b\u91d1\u5408\u4fe1\u6587\u5a31\u5a92\u4f53\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINWENYUMEITIGUPIAOFAQIA"}, "013133": {"suoxie": "CJHXWYMTGPFQC", "name": "\u521b\u91d1\u5408\u4fe1\u6587\u5a31\u5a92\u4f53\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINWENYUMEITIGUPIAOFAQIC"}, "013134": {"suoxie": "NFMSCIZGAGLJE", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA\u80a1\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOAGULIANJIEE"}, "013135": {"suoxie": "YHYNWJYLYNCYFOF", "name": "\u94f6\u6cb3\u60a6\u5b81\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHEYUENINGWENJIANYANGLAOYINIANCHIYOUFOF"}, "013136": {"suoxie": "HSHYYNDKZFQS", "name": "\u60e0\u5347\u548c\u6021\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYIYINIANDINGKAIZHAIFAQISHI"}, "013137": {"suoxie": "STMGDLJXHHC", "name": "\u4e0a\u6295\u6469\u6839\u52a8\u529b\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENDONGLIJINGXUANHUNHEC"}, "013138": {"suoxie": "SYZZ510NGKZZS", "name": "\u4e0a\u94f6\u4e2d\u503a5-10\u5e74\u56fd\u5f00\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGYINZHONGZHAI510NIANGUOKAIZHAIZHISHU"}, "013139": {"suoxie": "SYHTWJYLYNCYFOF", "name": "\u4e0a\u94f6\u6052\u6cf0\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINHENGTAIWENJIANYANGLAOYINIANCHIYOUFOF"}, "013140": {"suoxie": "ZJJXZQ", "name": "\u4e2d\u91d1\u91d1\u4fe1\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINJINXINZHAIQUAN"}, "013141": {"suoxie": "ZXBCHYHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u5f18\u8fdc\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGHONGYUANHUNHE"}, "013142": {"suoxie": "HSLXHLLHPZHHC", "name": "\u534e\u5546\u4e50\u4eab\u4e92\u8054\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGLEXIANGHULIANLINGHUOPEIZHIHUNHEC"}, "013143": {"suoxie": "FGACHB12GYCYQHHA", "name": "\u5bcc\u56fd\u5b89\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOANCHENGHUIBAO12GEYUECHIYOUQIHUNHEA"}, "013144": {"suoxie": "FGACHB12GYCYQHHC", "name": "\u5bcc\u56fd\u5b89\u8bda\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOANCHENGHUIBAO12GEYUECHIYOUQIHUNHEC"}, "013145": {"suoxie": "ZSHJXJZZHH", "name": "\u6d59\u5546\u6c47\u91d1\u5148\u8fdb\u5236\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUIJINXIANJINZHIZAOHUNHE"}, "013146": {"suoxie": "XYHHYNDKZFQ", "name": "\u5174\u94f6\u6c47\u6cd3\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINHUIHONGYINIANDINGKAIZHAIFAQI"}, "013147": {"suoxie": "HTFTZHZTHHA", "name": "\u6c47\u6dfb\u5bcc\u78b3\u4e2d\u548c\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUTANZHONGHEZHUTIHUNHEA"}, "013148": {"suoxie": "HTFTZHZTHHC", "name": "\u6c47\u6dfb\u5bcc\u78b3\u4e2d\u548c\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUTANZHONGHEZHUTIHUNHEC"}, "013149": {"suoxie": "PHSZJLZQC", "name": "\u9e4f\u534e\u53cc\u503a\u52a0\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUASHUANGZHAIJIALIZHAIQUANC"}, "013150": {"suoxie": "HBWJMBFXSGYCYFOF", "name": "\u534e\u5b9d\u7a33\u5065\u76ee\u6807\u98ce\u9669\u4e09\u4e2a\u6708\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUABAOWENJIANMUBIAOFENGXIANSANGEYUECHIYOUFOF"}, "013151": {"suoxie": "CXDTHBHYLHGPC", "name": "\u957f\u4fe1\u4f4e\u78b3\u73af\u4fdd\u884c\u4e1a\u91cf\u5316\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINDITANHUANBAOHANGYELIANGHUAGUPIAOC"}, "013152": {"suoxie": "CXXFJXLHGPC", "name": "\u957f\u4fe1\u6d88\u8d39\u7cbe\u9009\u91cf\u5316\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINXIAOFEIJINGXUANLIANGHUAGUPIAOC"}, "013153": {"suoxie": "CXDZXXLHLHPZHHC", "name": "\u957f\u4fe1\u7535\u5b50\u4fe1\u606f\u91cf\u5316\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINDIANZIXINXILIANGHUALINGHUOPEIZHIHUNHEC"}, "013154": {"suoxie": "CXYLBJHHLOFC", "name": "\u957f\u4fe1\u533b\u7597\u4fdd\u5065\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINYILIAOBAOJIANHUNHELOFC"}, "013155": {"suoxie": "HTFTFHYWJYLMBYNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u798f\u6c47\u76c8\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUTIANFUHUIYINGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "013156": {"suoxie": "XYWA60TGDCYZQE", "name": "\u5174\u94f6\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYINWENAN60TIANGUNDONGCHIYOUZHAIQUANE"}, "013157": {"suoxie": "QHKYXJJHHC", "name": "\u524d\u6d77\u5f00\u6e90\u65b0\u7ecf\u6d4e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANXINJINGJIHUNHEC"}, "013158": {"suoxie": "HXAKWJYLHHFOF", "name": "\u534e\u590f\u5b89\u5eb7\u7a33\u5065\u517b\u8001\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAANKANGWENJIANYANGLAOHUNHEFOF"}, "013159": {"suoxie": "GTRXYNDKZFQS", "name": "\u56fd\u6cf0\u745e\u946b\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUIXINYINIANDINGKAIZHAIFAQISHI"}, "013160": {"suoxie": "CJHXTZHHHA", "name": "\u521b\u91d1\u5408\u4fe1\u78b3\u4e2d\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINTANZHONGHEHUNHEA"}, "013161": {"suoxie": "CJHXTZHHHC", "name": "\u521b\u91d1\u5408\u4fe1\u78b3\u4e2d\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINTANZHONGHEHUNHEC"}, "013162": {"suoxie": "GFHGSKJLTETFLJA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUGANGSHENKEJILONGTOUETFLIANJIEA"}, "013163": {"suoxie": "GFHGSKJLTETFLJC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUGANGSHENKEJILONGTOUETFLIANJIEC"}, "013164": {"suoxie": "DXXYSGYDKZA", "name": "\u4e1c\u5174\u5174\u76c8\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINGYINGSANGEYUEDINGKAIZHAIA"}, "013165": {"suoxie": "DXXYSGYDKZC", "name": "\u4e1c\u5174\u5174\u76c8\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINGYINGSANGEYUEDINGKAIZHAIC"}, "013166": {"suoxie": "DXCXLHHHA", "name": "\u4e1c\u5174\u5bb8\u7965\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGCHENXIANGLIANGHUAHUNHEA"}, "013167": {"suoxie": "DXCXLHHHC", "name": "\u4e1c\u5174\u5bb8\u7965\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGXINGCHENXIANGLIANGHUAHUNHEC"}, "013168": {"suoxie": "DFHWTLCZC", "name": "\u4e1c\u65b9\u7ea2\u7a33\u6dfb\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGHONGWENTIANLICHUNZHAIC"}, "013169": {"suoxie": "JXPB15NZJZA", "name": "\u5efa\u4fe1\u5f6d\u535a1-5\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINPENGBO15NIANZHENGJINZHAIA"}, "013170": {"suoxie": "JXPB15NZJZC", "name": "\u5efa\u4fe1\u5f6d\u535a1-5\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINPENGBO15NIANZHENGJINZHAIC"}, "013171": {"suoxie": "HXHSHLWKJYETFLJQDIIA", "name": "\u534e\u590f\u6052\u751f\u4e92\u8054\u7f51\u79d1\u6280\u4e1aETF\u8054\u63a5(QDII)A", "style": "QDII", "pinyin": "HUAXIAHENGSHENGHULIANWANGKEJIYEETFLIANJIEQDIIA"}, "013172": {"suoxie": "HXHSHLWKJYETFLJQDIIC", "name": "\u534e\u590f\u6052\u751f\u4e92\u8054\u7f51\u79d1\u6280\u4e1aETF\u8054\u63a5(QDII)C", "style": "QDII", "pinyin": "HUAXIAHENGSHENGHULIANWANGKEJIYEETFLIANJIEQDIIC"}, "013173": {"suoxie": "HTCXKJCX3GYDKHHC", "name": "\u7ea2\u571f\u521b\u65b0\u79d1\u6280\u521b\u65b03\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTUCHUANGXINKEJICHUANGXIN3GEYUEDINGKAIHUNHEC"}, "013174": {"suoxie": "YHHZESGLXZS", "name": "\u94f6\u534e\u534e\u8bc1ESG\u9886\u5148\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAHUAZHENGESGLINGXIANZHISHU"}, "013175": {"suoxie": "HFTTZHHHA", "name": "\u6d77\u5bcc\u901a\u78b3\u4e2d\u548c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGTANZHONGHEHUNHEA"}, "013176": {"suoxie": "HFTTZHHHC", "name": "\u6d77\u5bcc\u901a\u78b3\u4e2d\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGTANZHONGHEHUNHEC"}, "013177": {"suoxie": "HBSZCX100ETFFQLJA", "name": "\u534e\u5b9d\u6df1\u8bc1\u521b\u65b0100ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHENZHENGCHUANGXIN100ETFFAQILIANJIEA"}, "013178": {"suoxie": "HBSZCX100ETFFQLJC", "name": "\u534e\u5b9d\u6df1\u8bc1\u521b\u65b0100ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHENZHENGCHUANGXIN100ETFFAQILIANJIEC"}, "013179": {"suoxie": "GFGZXNYCDCETFLJA", "name": "\u5e7f\u53d1\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGXINNENGYUANCHEDIANCHIETFLIANJIEA"}, "013180": {"suoxie": "GFGZXNYCDCETFLJC", "name": "\u5e7f\u53d1\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGXINNENGYUANCHEDIANCHIETFLIANJIEC"}, "013181": {"suoxie": "AXGGTJXHHFQA", "name": "\u5b89\u4fe1\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINGANGGUTONGJINGXUANHUNHEFAQIA"}, "013182": {"suoxie": "AXGGTJXHHFQC", "name": "\u5b89\u4fe1\u6e2f\u80a1\u901a\u7cbe\u9009\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINGANGGUTONGJINGXUANHUNHEFAQIC"}, "013183": {"suoxie": "PYASYLJKHHC", "name": "\u6d66\u94f6\u5b89\u76db\u533b\u7597\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGYILIAOJIANKANGHUNHEC"}, "013184": {"suoxie": "GFHYYNCYHHA", "name": "\u5e7f\u53d1\u6052\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYANGYINIANCHIYOUHUNHEA"}, "013185": {"suoxie": "GFHYYNCYHHC", "name": "\u5e7f\u53d1\u6052\u9633\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGYANGYINIANCHIYOUHUNHEC"}, "013186": {"suoxie": "CCHLZQA", "name": "\u957f\u57ce\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGHENGLIZHAIQUANA"}, "013187": {"suoxie": "CCHLZQC", "name": "\u957f\u57ce\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGHENGLIZHAIQUANC"}, "013188": {"suoxie": "HXNYGXGPC", "name": "\u534e\u590f\u80fd\u6e90\u9769\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIANENGYUANGEXINGUPIAOC"}, "013189": {"suoxie": "JSWJTLYNCYHH", "name": "\u5609\u5b9e\u7a33\u5065\u6dfb\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIWENJIANTIANLIYINIANCHIYOUHUNHE"}, "013190": {"suoxie": "ZRJHHHA", "name": "\u4e2d\u878d\u666f\u60e0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGHUIHUNHEA"}, "013191": {"suoxie": "ZRJHHHC", "name": "\u4e2d\u878d\u666f\u60e0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGRONGJINGHUIHUNHEC"}, "013192": {"suoxie": "HSJYWJYLMBYNCYHHXFQSFOF", "name": "\u534e\u5546\u5609\u60a6\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408\u578b\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUASHANGJIAYUEWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEXINGFAQISHIFOF"}, "013193": {"suoxie": "HSWJTLYNCYHHA", "name": "\u534e\u5546\u7a33\u5065\u6dfb\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUASHANGWENJIANTIANLIYINIANCHIYOUHUNHEA"}, "013194": {"suoxie": "HSWJTLYNCYHHC", "name": "\u534e\u5546\u7a33\u5065\u6dfb\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUASHANGWENJIANTIANLIYINIANCHIYOUHUNHEC"}, "013195": {"suoxie": "ZSZZXNYQCZSA", "name": "\u62db\u5546\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXINNENGYUANQICHEZHISHUA"}, "013196": {"suoxie": "ZSZZXNYQCZSC", "name": "\u62db\u5546\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXINNENGYUANQICHEZHISHUC"}, "013197": {"suoxie": "NFXJLYNDKZ", "name": "\u5357\u65b9\u5174\u9526\u5229\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGXINGJINLIYINIANDINGKAIZHAI"}, "013198": {"suoxie": "NFJJX90TGDCYZQA", "name": "\u5357\u65b9\u5b63\u5b63\u4eab90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJIJIXIANG90TIANGUNDONGCHIYOUZHAIQUANA"}, "013199": {"suoxie": "NFJJX90TGDCYZQC", "name": "\u5357\u65b9\u5b63\u5b63\u4eab90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJIJIXIANG90TIANGUNDONGCHIYOUZHAIQUANC"}, "013200": {"suoxie": "NFJHYXYNCYQHHA", "name": "\u5357\u65b9\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGJUNHENGYOUXUANYINIANCHIYOUQIHUNHEA"}, "013201": {"suoxie": "NFJHYXYNCYQHHC", "name": "\u5357\u65b9\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGJUNHENGYOUXUANYINIANCHIYOUQIHUNHEC"}, "013202": {"suoxie": "HSQHHXCZZQA", "name": "\u6052\u751f\u524d\u6d77\u6052\u7965\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGXIANGCHUNZHAIZHAIQUANA"}, "013203": {"suoxie": "HSQHHXCZZQC", "name": "\u6052\u751f\u524d\u6d77\u6052\u7965\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGXIANGCHUNZHAIZHAIQUANC"}, "013204": {"suoxie": "HSQHHYTLZQA", "name": "\u6052\u751f\u524d\u6d77\u6052\u6e90\u5929\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HENGSHENGQIANHAIHENGYUANTIANLIZHAIQUANA"}, "013205": {"suoxie": "HSQHHYTLZQC", "name": "\u6052\u751f\u524d\u6d77\u6052\u6e90\u5929\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HENGSHENGQIANHAIHENGYUANTIANLIZHAIQUANC"}, "013206": {"suoxie": "GFHYYNDKZ", "name": "\u5e7f\u53d1\u6c47\u5b9c\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAHUIYIYINIANDINGKAIZHAI"}, "013207": {"suoxie": "WJWX30TGDCYDZA", "name": "\u4e07\u5bb6\u7a33\u946b30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "WANJIAWENXIN30TIANGUNDONGCHIYOUDUANZHAIA"}, "013208": {"suoxie": "WJWX30TGDCYDZC", "name": "\u4e07\u5bb6\u7a33\u946b30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "WANJIAWENXIN30TIANGUNDONGCHIYOUDUANZHAIC"}, "013209": {"suoxie": "JYDSYHHA", "name": "\u91d1\u9e70\u5927\u89c6\u91ce\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGDASHIYEHUNHEA"}, "013210": {"suoxie": "JYDSYHHC", "name": "\u91d1\u9e70\u5927\u89c6\u91ce\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGDASHIYEHUNHEC"}, "013211": {"suoxie": "HFAYYNCYQZQA", "name": "\u534e\u5bcc\u5b89\u76c8\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANYINGYINIANCHIYOUQIZHAIQUANA"}, "013212": {"suoxie": "HFAYYNCYQZQC", "name": "\u534e\u5bcc\u5b89\u76c8\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUANYINGYINIANCHIYOUQIZHAIQUANC"}, "013213": {"suoxie": "XYJHYNDKZFQS", "name": "\u5174\u4e1a\u5609\u9e3f\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEJIAHONGYINIANDINGKAIZHAIFAQISHI"}, "013216": {"suoxie": "CTZGHQ90TGDZDZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u542f90\u5929\u6eda\u52a8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGQI90TIANGUNDONGZHONGDUANZHAIA"}, "013217": {"suoxie": "CTZGHQ90TGDZDZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u542f90\u5929\u6eda\u52a8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGQI90TIANGUNDONGZHONGDUANZHAIC"}, "013218": {"suoxie": "GTZZYSJSETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5eETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYOUSEJINSHUETFFAQILIANJIEA"}, "013219": {"suoxie": "GTZZYSJSETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5eETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYOUSEJINSHUETFFAQILIANJIEC"}, "013220": {"suoxie": "ZOXXJZYNCYHHA", "name": "\u4e2d\u6b27\u65b0\u5174\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINXINGJIAZHIYINIANCHIYOUHUNHEA"}, "013221": {"suoxie": "ZOXXJZYNCYHHC", "name": "\u4e2d\u6b27\u65b0\u5174\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINXINGJIAZHIYINIANCHIYOUHUNHEC"}, "013222": {"suoxie": "BYZX3GYDKZQA", "name": "\u535a\u8fdc\u81fb\u4eab3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOYUANZHENXIANG3GEYUEDINGKAIZHAIQUANA"}, "013223": {"suoxie": "BYZX3GYDKZQC", "name": "\u535a\u8fdc\u81fb\u4eab3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOYUANZHENXIANG3GEYUEDINGKAIZHAIQUANC"}, "013224": {"suoxie": "PYASGGTLHHHC", "name": "\u6d66\u94f6\u5b89\u76db\u6e2f\u80a1\u901a\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGGANGGUTONGLIANGHUAHUNHEC"}, "013225": {"suoxie": "JSCCAJYNCYQHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u666f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANJINGYINIANCHIYOUQIHUNHEA"}, "013226": {"suoxie": "JSCCAJYNCYQHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u666f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANJINGYINIANCHIYOUQIHUNHEC"}, "013227": {"suoxie": "ZYXX30TGDCYDZZQA", "name": "\u4e2d\u90ae\u946b\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYOUXINXIANG30TIANGUNDONGCHIYOUDUANZHAIZHAIQUANA"}, "013228": {"suoxie": "ZYXX30TGDCYDZZQC", "name": "\u4e2d\u90ae\u946b\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYOUXINXIANG30TIANGUNDONGCHIYOUDUANZHAIZHAIQUANC"}, "013229": {"suoxie": "ZYRFZQZQA", "name": "\u4e2d\u90ae\u777f\u4e30\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOURUIFENGZENGQIANGZHAIQUANA"}, "013230": {"suoxie": "ZYRFZQZQC", "name": "\u4e2d\u90ae\u777f\u4e30\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYOURUIFENGZENGQIANGZHAIQUANC"}, "013231": {"suoxie": "ZSZDYZQA", "name": "\u6d59\u5546\u667a\u591a\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGZHIDUOYINGZHAIQUANA"}, "013232": {"suoxie": "ZSZDYZQC", "name": "\u6d59\u5546\u667a\u591a\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGZHIDUOYINGZHAIQUANC"}, "013233": {"suoxie": "HXZZ500ZSZXZQA", "name": "\u534e\u590f\u4e2d\u8bc1500\u6307\u6570\u667a\u9009\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ZHISHUZHIXUANZENGQIANGA"}, "013234": {"suoxie": "HXZZ500ZSZXZQC", "name": "\u534e\u590f\u4e2d\u8bc1500\u6307\u6570\u667a\u9009\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ZHISHUZHIXUANZENGQIANGC"}, "013235": {"suoxie": "HFFHYNDKZQFQ", "name": "\u534e\u5bcc\u5bcc\u60e0\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAFUFUHUIYINIANDINGKAIZHAIQUANFAQI"}, "013236": {"suoxie": "CX30TGDCYZQA", "name": "\u957f\u4fe130\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGXIN30TIANGUNDONGCHIYOUZHAIQUANA"}, "013237": {"suoxie": "CX30TGDCYZQC", "name": "\u957f\u4fe130\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGXIN30TIANGUNDONGCHIYOUZHAIQUANC"}, "013238": {"suoxie": "CTJHYXYNCYHHA", "name": "\u8d22\u901a\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGJUNHENGYOUXUANYINIANCHIYOUHUNHEA"}, "013239": {"suoxie": "CTJHYXYNCYHHC", "name": "\u8d22\u901a\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGJUNHENGYOUXUANYINIANCHIYOUHUNHEC"}, "013240": {"suoxie": "CJFY6GYCYZQA", "name": "\u957f\u6c5f\u4e30\u76ca6\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGFENGYI6GEYUECHIYOUZHAIQUANA"}, "013241": {"suoxie": "CJFY6GYCYZQC", "name": "\u957f\u6c5f\u4e30\u76ca6\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGFENGYI6GEYUECHIYOUZHAIQUANC"}, "013242": {"suoxie": "BXRFYSHYGP", "name": "\u5317\u4fe1\u745e\u4e30\u4f18\u52bf\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "BEIXINRUIFENGYOUSHIHANGYEGUPIAO"}, "013243": {"suoxie": "THAKYFYNCYHHA", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u4e30\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIFENGYINIANCHIYOUHUNHEA"}, "013244": {"suoxie": "THAKYFYNCYHHC", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u4e30\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIFENGYINIANCHIYOUHUNHEC"}, "013245": {"suoxie": "TDHLYRYLMBRQ2025YNCYHHFOF", "name": "\u6cf0\u8fbe\u5b8f\u5229\u60a0\u7136\u517b\u8001\u76ee\u6807\u65e5\u671f2025\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIYOURANYANGLAOMUBIAORIQI2025YINIANCHIYOUHUNHEFOF"}, "013247": {"suoxie": "JYRZSNCYQHH", "name": "\u4ea4\u94f6\u745e\u5353\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINRUIZHUOSANNIANCHIYOUQIHUNHE"}, "013248": {"suoxie": "JYHTYNCYQHHA", "name": "\u4ea4\u94f6\u9e3f\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGTAIYINIANCHIYOUQIHUNHEA"}, "013249": {"suoxie": "JYHTYNCYQHHC", "name": "\u4ea4\u94f6\u9e3f\u6cf0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINHONGTAIYINIANCHIYOUQIHUNHEC"}, "013250": {"suoxie": "HTCXZNZZHHFQS", "name": "\u7ea2\u571f\u521b\u65b0\u667a\u80fd\u5236\u9020\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTUCHUANGXINZHINENGZHIZAOHUNHEFAQISHI"}, "013251": {"suoxie": "ZXJTWSZQA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u7855\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENSHUOZHAIQUANA"}, "013252": {"suoxie": "ZXJTWSZQC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u7855\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENSHUOZHAIQUANC"}, "013253": {"suoxie": "HFTYLMBRQ2035FOF", "name": "\u6d77\u5bcc\u901a\u517b\u8001\u76ee\u6807\u65e5\u671f2035(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGYANGLAOMUBIAORIQI2035FOF"}, "013254": {"suoxie": "HAYXWJYLMBYNCYHHFQSFOF", "name": "\u534e\u5b89\u4f18\u4eab\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANYOUXIANGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFAQISHIFOF"}, "013255": {"suoxie": "XQYXJQSGYCYFOFC", "name": "\u5174\u5168\u4f18\u9009\u8fdb\u53d6\u4e09\u4e2a\u6708\u6301\u6709(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANYOUXUANJINQUSANGEYUECHIYOUFOFC"}, "013256": {"suoxie": "JYTRCZZQC", "name": "\u91d1\u9e70\u6dfb\u8363\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGTIANRONGCHUNZHAIZHAIQUANC"}, "013257": {"suoxie": "NFTY6GYCYZQA", "name": "\u5357\u65b9\u901a\u51436\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGTONGYUAN6GEYUECHIYOUZHAIQUANA"}, "013258": {"suoxie": "NFTY6GYCYZQC", "name": "\u5357\u65b9\u901a\u51436\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGTONGYUAN6GEYUECHIYOUZHAIQUANC"}, "013259": {"suoxie": "PYASXJJJGHHC", "name": "\u6d66\u94f6\u5b89\u76db\u65b0\u7ecf\u6d4e\u7ed3\u6784\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXINJINGJIJIEGOUHUNHEC"}, "013260": {"suoxie": "TPRXHHA", "name": "\u592a\u5e73\u777f\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIXIANGHUNHEA"}, "013261": {"suoxie": "TPRXHHC", "name": "\u592a\u5e73\u777f\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIXIANGHUNHEC"}, "013262": {"suoxie": "XBLDGGJXGPC", "name": "\u897f\u90e8\u5229\u5f97\u4e2a\u80a1\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "XIBULIDEGEGUJINGXUANGUPIAOC"}, "013263": {"suoxie": "JYNNYXYNCYZQA", "name": "\u91d1\u9e70\u5e74\u5e74\u90ae\u4eab\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGNIANNIANYOUXIANGYINIANCHIYOUZHAIQUANA"}, "013264": {"suoxie": "JYNNYXYNCYZQC", "name": "\u91d1\u9e70\u5e74\u5e74\u90ae\u4eab\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGNIANNIANYOUXIANGYINIANCHIYOUZHAIQUANC"}, "013265": {"suoxie": "PYCXYNDKZFQS", "name": "\u9e4f\u626c\u6df3\u7199\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNXIYINIANDINGKAIZHAIFAQISHI"}, "013267": {"suoxie": "THAKYLHHA", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYILIHUNHEA"}, "013268": {"suoxie": "THAKYLHHC", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYILIHUNHEC"}, "013269": {"suoxie": "JYRHSNCYQHH", "name": "\u4ea4\u94f6\u745e\u548c\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINRUIHESANNIANCHIYOUQIHUNHE"}, "013270": {"suoxie": "QHKYJLYNCYHHA", "name": "\u524d\u6d77\u5f00\u6e90\u805a\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANJULIYINIANCHIYOUHUNHEA"}, "013271": {"suoxie": "QHKYJLYNCYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u805a\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANJULIYINIANCHIYOUHUNHEC"}, "013272": {"suoxie": "GTJA1NDKZQFQS", "name": "\u56fd\u6cf0\u541b\u5b891\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUNAN1NIANDINGKAIZHAIQUANFAQISHI"}, "013273": {"suoxie": "ZSHS300DCDQZZSC", "name": "\u62db\u5546\u6caa\u6df1300\u5730\u4ea7\u7b49\u6743\u91cd\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300DICHANDENGQUANZHONGZHISHUC"}, "013274": {"suoxie": "CCYHSJHHC", "name": "\u957f\u57ce\u4f18\u5316\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYOUHUASHENGJIHUNHEC"}, "013275": {"suoxie": "FGZZMTZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7164\u70ad\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGMEITANZHISHULOFC"}, "013276": {"suoxie": "FGZZQZZQGSZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFC"}, "013277": {"suoxie": "FGCYBZSLOFC", "name": "\u5bcc\u56fd\u521b\u4e1a\u677f\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOCHUANGYEBANZHISHULOFC"}, "013278": {"suoxie": "FGZZTYCYZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u4f53\u80b2\u4ea7\u4e1a\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGTIYUCHANYEZHISHULOFC"}, "013279": {"suoxie": "GTYXLHYNCYFOF", "name": "\u56fd\u6cf0\u4f18\u9009\u9886\u822a\u4e00\u5e74\u6301\u6709(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIYOUXUANLINGHANGYINIANCHIYOUFOF"}, "013280": {"suoxie": "TDRZWJHHC", "name": "\u6cf0\u8fbe\u777f\u667a\u7a33\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDARUIZHIWENJIANHUNHEC"}, "013281": {"suoxie": "GTJA30TGDCYZDZA", "name": "\u56fd\u6cf0\u541b\u5b8930\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAIJUNAN30TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "013282": {"suoxie": "GTJA30TGDCYZDZC", "name": "\u56fd\u6cf0\u541b\u5b8930\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTAIJUNAN30TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013283": {"suoxie": "ZOXYYNDKZQFQ", "name": "\u4e2d\u6b27\u5174\u76c8\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINGYINGYINIANDINGKAIZHAIQUANFAQI"}, "013284": {"suoxie": "SYJZZZ3GYCYQHHA", "name": "\u4e0a\u94f6\u4ef7\u503c\u589e\u957f3\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGYINJIAZHIZENGZHANG3GEYUECHIYOUQIHUNHEA"}, "013285": {"suoxie": "SYJZZZ3GYCYQHHC", "name": "\u4e0a\u94f6\u4ef7\u503c\u589e\u957f3\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGYINJIAZHIZENGZHANG3GEYUECHIYOUQIHUNHEC"}, "013286": {"suoxie": "FGSZZSETFLJC", "name": "\u5bcc\u56fd\u4e0a\u8bc1\u6307\u6570ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOSHANGZHENGZHISHUETFLIANJIEC"}, "013287": {"suoxie": "YFDYSJZYNCYHHFOFA", "name": "\u6613\u65b9\u8fbe\u4f18\u52bf\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUSHIJIAZHIYINIANCHIYOUHUNHEFOFA"}, "013288": {"suoxie": "YFDYSJZYNCYHHFOFC", "name": "\u6613\u65b9\u8fbe\u4f18\u52bf\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYOUSHIJIAZHIYINIANCHIYOUHUNHEFOFC"}, "013289": {"suoxie": "GYSPYLHHA", "name": "\u5de5\u94f6\u98df\u54c1\u996e\u6599\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINSHIPINYINLIAOHUNHEA"}, "013290": {"suoxie": "GYSPYLHHC", "name": "\u5de5\u94f6\u98df\u54c1\u996e\u6599\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINSHIPINYINLIAOHUNHEC"}, "013291": {"suoxie": "FGHS300ZSZQC", "name": "\u5bcc\u56fd\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOHUSHEN300ZHISHUZENGQIANGC"}, "013292": {"suoxie": "FGZZZNQCLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGZHINENGQICHELOFC"}, "013293": {"suoxie": "CCJKXFHH", "name": "\u957f\u57ce\u5065\u5eb7\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIANKANGXIAOFEIHUNHE"}, "013294": {"suoxie": "DFHZXSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u667a\u9009\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGZHIXUANSANNIANCHIYOUHUNHEA"}, "013295": {"suoxie": "DFHZXSNCYHHC", "name": "\u4e1c\u65b9\u7ea2\u667a\u9009\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGZHIXUANSANNIANCHIYOUHUNHEC"}, "013296": {"suoxie": "MSJYJYJXHH", "name": "\u6c11\u751f\u52a0\u94f6\u805a\u4f18\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINJUYOUJINGXUANHUNHE"}, "013297": {"suoxie": "JHPGYNDKCZZQFQS", "name": "\u5609\u5408\u78d0\u56fa\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANGUYINIANDINGKAICHUNZHAIZHAIQUANFAQISHI"}, "013298": {"suoxie": "NFZZKCCY50ETFLJA", "name": "\u5357\u65b9\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEA"}, "013299": {"suoxie": "NFZZKCCY50ETFLJC", "name": "\u5357\u65b9\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEC"}, "013300": {"suoxie": "GYJZWJ6GYCYHHFOFA", "name": "\u5de5\u94f6\u4ef7\u503c\u7a33\u50656\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJIAZHIWENJIAN6GEYUECHIYOUHUNHEFOFA"}, "013301": {"suoxie": "GYJZWJ6GYCYHHFOFC", "name": "\u5de5\u94f6\u4ef7\u503c\u7a33\u50656\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINJIAZHIWENJIAN6GEYUECHIYOUHUNHEFOFC"}, "013302": {"suoxie": "ZSZZKCCY50ETFLJA", "name": "\u62db\u5546\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEA"}, "013303": {"suoxie": "ZSZZKCCY50ETFLJC", "name": "\u62db\u5546\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEC"}, "013304": {"suoxie": "YFDZZKCCY50ETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEA"}, "013305": {"suoxie": "YFDZZKCCY50ETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEC"}, "013306": {"suoxie": "GTZZKCCY50ETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGKECHUANGCHUANGYE50ETFFAQILIANJIEA"}, "013307": {"suoxie": "GTZZKCCY50ETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGKECHUANGCHUANGYE50ETFFAQILIANJIEC"}, "013308": {"suoxie": "YFDHSKJETFLJQDIIA", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u79d1\u6280ETF\u8054\u63a5(QDII)A", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGKEJIETFLIANJIEQDIIA"}, "013309": {"suoxie": "YFDHSKJETFLJQDIIC", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u79d1\u6280ETF\u8054\u63a5(QDII)C", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGKEJIETFLIANJIEQDIIC"}, "013310": {"suoxie": "HXKCCY50ETFLJA", "name": "\u534e\u590f\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAKECHUANGCHUANGYE50ETFLIANJIEA"}, "013311": {"suoxie": "HXKCCY50ETFLJC", "name": "\u534e\u590f\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAKECHUANGCHUANGYE50ETFLIANJIEC"}, "013312": {"suoxie": "GYZTCLHHC", "name": "\u5de5\u94f6\u4e3b\u9898\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHUTICELUEHUNHEC"}, "013313": {"suoxie": "FGZZKCCY50ETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEA"}, "013314": {"suoxie": "FGZZKCCY50ETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEC"}, "013315": {"suoxie": "JSZZKCCY50ETFFQLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGKECHUANGCHUANGYE50ETFFAQILIANJIEA"}, "013316": {"suoxie": "JSZZKCCY50ETFFQLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGKECHUANGCHUANGYE50ETFFAQILIANJIEC"}, "013317": {"suoxie": "HBZZKCCY50ETFLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEA"}, "013318": {"suoxie": "HBZZKCCY50ETFLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGKECHUANGCHUANGYE50ETFLIANJIEC"}, "013319": {"suoxie": "HAZZXNYQCETFFQSLJA", "name": "\u534e\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXINNENGYUANQICHEETFFAQISHILIANJIEA"}, "013320": {"suoxie": "HAZZXNYQCETFFQSLJC", "name": "\u534e\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXINNENGYUANQICHEETFFAQISHILIANJIEC"}, "013321": {"suoxie": "BSHYWJYNCYQHHA", "name": "\u535a\u65f6\u6052\u76c8\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYINGWENJIANYINIANCHIYOUQIHUNHEA"}, "013322": {"suoxie": "BSHYWJYNCYQHHC", "name": "\u535a\u65f6\u6052\u76c8\u7a33\u5065\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGYINGWENJIANYINIANCHIYOUQIHUNHEC"}, "013323": {"suoxie": "GSABSZSNCYHHA", "name": "\u56fd\u5bff\u5b89\u4fdd\u76db\u6cfd\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOSHENGZESANNIANCHIYOUHUNHEA"}, "013324": {"suoxie": "GSABSZSNCYHHC", "name": "\u56fd\u5bff\u5b89\u4fdd\u76db\u6cfd\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOSHOUANBAOSHENGZESANNIANCHIYOUHUNHEC"}, "013325": {"suoxie": "SWLXHLSGYDQKFZQ", "name": "\u7533\u4e07\u83f1\u4fe1\u6052\u5229\u4e09\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINHENGLISANGEYUEDINGQIKAIFANGZHAIQUAN"}, "013326": {"suoxie": "WJJQQDHHA", "name": "\u4e07\u5bb6\u666f\u6c14\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJINGQIQUDONGHUNHEA"}, "013327": {"suoxie": "WJJQQDHHC", "name": "\u4e07\u5bb6\u666f\u6c14\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJINGQIQUDONGHUNHEC"}, "013328": {"suoxie": "JSQQJZGPQDIIRMB", "name": "\u5609\u5b9e\u5168\u7403\u4ef7\u503c\u80a1\u7968(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "JIASHIQUANQIUJIAZHIGUPIAOQDIIRENMINBI"}, "013329": {"suoxie": "JSQQJZGPQDIIMYXH", "name": "\u5609\u5b9e\u5168\u7403\u4ef7\u503c\u80a1\u7968(QDII)\u7f8e\u5143\u73b0\u6c47", "style": "QDII", "pinyin": "JIASHIQUANQIUJIAZHIGUPIAOQDIIMEIYUANXIANHUI"}, "013330": {"suoxie": "FGZZYHZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYINHANGZHISHULOFC"}, "013331": {"suoxie": "FGZZ1000ZSZQLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG1000ZHISHUZENGQIANGLOFC"}, "013332": {"suoxie": "FGZZ500ZSZQLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG500ZHISHUZENGQIANGLOFC"}, "013333": {"suoxie": "DXXRYNDKC", "name": "\u4e1c\u5174\u5174\u745e\u4e00\u5e74\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINGRUIYINIANDINGKAIC"}, "013334": {"suoxie": "PHJZYH6GYCYHHA", "name": "\u9e4f\u534e\u4ef7\u503c\u8fdc\u822a6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHIYUANHANG6GEYUECHIYOUHUNHEA"}, "013335": {"suoxie": "PHJZYH6GYCYHHC", "name": "\u9e4f\u534e\u4ef7\u503c\u8fdc\u822a6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHIYUANHANG6GEYUECHIYOUHUNHEC"}, "013336": {"suoxie": "THAY90TGDCYDZFQ", "name": "\u5929\u5f18\u5b89\u60a690\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TIANHONGANYUE90TIANGUNDONGCHIYOUDUANZHAIFAQI"}, "013337": {"suoxie": "CJHXYJLF3GYCYHHFQFOFA", "name": "\u521b\u91d1\u5408\u4fe1\u5b9c\u4e45\u6765\u798f3\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINYIJIULAIFU3GEYUECHIYOUHUNHEFAQIFOFA"}, "013338": {"suoxie": "CJHXYJLF3GYCYHHFQFOFC", "name": "\u521b\u91d1\u5408\u4fe1\u5b9c\u4e45\u6765\u798f3\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINYIJIULAIFU3GEYUECHIYOUHUNHEFAQIFOFC"}, "013339": {"suoxie": "CJHXXPCYGPFQA", "name": "\u521b\u91d1\u5408\u4fe1\u82af\u7247\u4ea7\u4e1a\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINPIANCHANYEGUPIAOFAQIA"}, "013340": {"suoxie": "CJHXXPCYGPFQC", "name": "\u521b\u91d1\u5408\u4fe1\u82af\u7247\u4ea7\u4e1a\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINXINPIANCHANYEGUPIAOFAQIC"}, "013341": {"suoxie": "GYHXJYHHA", "name": "\u5de5\u94f6\u6838\u5fc3\u673a\u9047\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHEXINJIYUHUNHEA"}, "013342": {"suoxie": "GYHXJYHHC", "name": "\u5de5\u94f6\u6838\u5fc3\u673a\u9047\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHEXINJIYUHUNHEC"}, "013343": {"suoxie": "PAYXWJ1NCYHHFOFA", "name": "\u5e73\u5b89\u76c8\u6b23\u7a33\u50651\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANYINGXINWENJIAN1NIANCHIYOUHUNHEFOFA"}, "013344": {"suoxie": "PAYXWJ1NCYHHFOFC", "name": "\u5e73\u5b89\u76c8\u6b23\u7a33\u50651\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANYINGXINWENJIAN1NIANCHIYOUHUNHEFOFC"}, "013345": {"suoxie": "FRXXJSHHA", "name": "\u5bcc\u8363\u4fe1\u606f\u6280\u672f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGXINXIJISHUHUNHEA"}, "013346": {"suoxie": "FRXXJSHHC", "name": "\u5bcc\u8363\u4fe1\u606f\u6280\u672f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FURONGXINXIJISHUHUNHEC"}, "013347": {"suoxie": "GYFYHBLHPZHHC", "name": "\u5de5\u94f6\u4e30\u76c8\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINFENGYINGHUIBAOLINGHUOPEIZHIHUNHEC"}, "013348": {"suoxie": "CJHXDJKHHA", "name": "\u521b\u91d1\u5408\u4fe1\u5927\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINDAJIANKANGHUNHEA"}, "013349": {"suoxie": "CJHXDJKHHC", "name": "\u521b\u91d1\u5408\u4fe1\u5927\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUANGJINHEXINDAJIANKANGHUNHEC"}, "013350": {"suoxie": "GDXJFWYHHC", "name": "\u5149\u5927\u5148\u8fdb\u670d\u52a1\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDAXIANJINFUWUYEHUNHEC"}, "013353": {"suoxie": "PHSHYNCYQHHA", "name": "\u9e4f\u534e\u4e0a\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUASHANGHUAYINIANCHIYOUQIHUNHEA"}, "013354": {"suoxie": "PHSHYNCYQHHC", "name": "\u9e4f\u534e\u4e0a\u534e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUASHANGHUAYINIANCHIYOUQIHUNHEC"}, "013355": {"suoxie": "GYXJRGPC", "name": "\u5de5\u94f6\u65b0\u91d1\u878d\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINXINJINRONGGUPIAOC"}, "013356": {"suoxie": "DMHGSJXHHA", "name": "\u5927\u6469\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOHUGANGSHENJINGXUANHUNHEA"}, "013357": {"suoxie": "DMHGSJXHHC", "name": "\u5927\u6469\u6caa\u6e2f\u6df1\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOHUGANGSHENJINGXUANHUNHEC"}, "013358": {"suoxie": "SYGZLYX9GYCYHHA", "name": "\u4e0a\u94f6\u9ad8\u8d28\u91cf\u4f18\u90099\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINGAOZHILIANGYOUXUAN9GEYUECHIYOUHUNHEA"}, "013359": {"suoxie": "SYGZLYX9GYCYHHC", "name": "\u4e0a\u94f6\u9ad8\u8d28\u91cf\u4f18\u90099\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINGAOZHILIANGYOUXUAN9GEYUECHIYOUHUNHEC"}, "013360": {"suoxie": "HXPTHHLOFC", "name": "\u534e\u590f\u78d0\u6cf0\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAPANTAIHUNHELOFC"}, "013365": {"suoxie": "HTFCYSJHHA", "name": "\u6c47\u6dfb\u5bcc\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHANYESHENGJIHUNHEA"}, "013366": {"suoxie": "HTFCYSJHHC", "name": "\u6c47\u6dfb\u5bcc\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHANYESHENGJIHUNHEC"}, "013367": {"suoxie": "HTFDYJZFXHHA", "name": "\u6c47\u6dfb\u5bcc\u591a\u5143\u4ef7\u503c\u53d1\u73b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUDUOYUANJIAZHIFAXIANHUNHEA"}, "013368": {"suoxie": "HTFDYJZFXHHC", "name": "\u6c47\u6dfb\u5bcc\u591a\u5143\u4ef7\u503c\u53d1\u73b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUDUOYUANJIAZHIFAXIANHUNHEC"}, "013369": {"suoxie": "HTFZZHXKJYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u81ea\u4e3b\u6838\u5fc3\u79d1\u6280\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZIZHUHEXINKEJIYINIANCHIYOUHUNHEA"}, "013370": {"suoxie": "HTFZZHXKJYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u81ea\u4e3b\u6838\u5fc3\u79d1\u6280\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZIZHUHEXINKEJIYINIANCHIYOUHUNHEC"}, "013371": {"suoxie": "MSJYXNYZXHHFQA", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u80fd\u6e90\u667a\u9009\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINNENGYUANZHIXUANHUNHEFAQIA"}, "013372": {"suoxie": "MSJYXNYZXHHFQC", "name": "\u6c11\u751f\u52a0\u94f6\u65b0\u80fd\u6e90\u667a\u9009\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINXINNENGYUANZHIXUANHUNHEFAQIC"}, "013373": {"suoxie": "ZYZQAYZQA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u4e1a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYEZHAIQUANA"}, "013374": {"suoxie": "ZYZQAYZQC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5b89\u4e1a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINZHENGQUANANYEZHAIQUANC"}, "013375": {"suoxie": "PAYX120TGDCYZDZA", "name": "\u5e73\u5b89\u5143\u946b120\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANXIN120TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "013376": {"suoxie": "PAYX120TGDCYZDZC", "name": "\u5e73\u5b89\u5143\u946b120\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANXIN120TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013377": {"suoxie": "DHQH6GYCYHHC", "name": "\u4e1c\u6d77\u542f\u822a6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGHAIQIHANG6GEYUECHIYOUHUNHEC"}, "013378": {"suoxie": "FZFBWYCZA", "name": "\u65b9\u6b63\u5bcc\u90a6\u7a33\u88d5\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGWENYUCHUNZHAIA"}, "013379": {"suoxie": "FZFBWYCZC", "name": "\u65b9\u6b63\u5bcc\u90a6\u7a33\u88d5\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGWENYUCHUNZHAIC"}, "013381": {"suoxie": "ZOZX3GYCYHHFOFA", "name": "\u4e2d\u6b27\u7504\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHENXUAN3GEYUECHIYOUHUNHEFOFA"}, "013382": {"suoxie": "ZOZX3GYCYHHFOFC", "name": "\u4e2d\u6b27\u7504\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHENXUAN3GEYUECHIYOUHUNHEFOFC"}, "013385": {"suoxie": "XDAYYSJZHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4f18\u52bf\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYOUSHIJIAZHIHUNHEA"}, "013386": {"suoxie": "XDAYYSJZHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4f18\u52bf\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYOUSHIJIAZHIHUNHEC"}, "013387": {"suoxie": "CCJZLHHHA", "name": "\u957f\u57ce\u4ef7\u503c\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHILINGHANGHUNHEA"}, "013388": {"suoxie": "CCJZLHHHC", "name": "\u957f\u57ce\u4ef7\u503c\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHILINGHANGHUNHEC"}, "013389": {"suoxie": "HXCZXFYNCYHHA", "name": "\u534e\u590f\u6210\u957f\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGXIANFENGYINIANCHIYOUHUNHEA"}, "013390": {"suoxie": "HXCZXFYNCYHHC", "name": "\u534e\u590f\u6210\u957f\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHENGZHANGXIANFENGYINIANCHIYOUHUNHEC"}, "013391": {"suoxie": "ZSATZQD", "name": "\u62db\u5546\u5b89\u6cf0\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGANTAIZHAIQUAND"}, "013392": {"suoxie": "GFGGTYZZZHHC", "name": "\u5e7f\u53d1\u6e2f\u80a1\u901a\u4f18\u8d28\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAGANGGUTONGYOUZHIZENGZHANGHUNHEC"}, "013393": {"suoxie": "XDAYJZJXHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINJIAZHIJINGXUANHUNHEA"}, "013394": {"suoxie": "XDAYJZJXHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINJIAZHIJINGXUANHUNHEC"}, "013395": {"suoxie": "HXXNYCLTHHFQSA", "name": "\u534e\u590f\u65b0\u80fd\u6e90\u8f66\u9f99\u5934\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINNENGYUANCHELONGTOUHUNHEFAQISHIA"}, "013396": {"suoxie": "HXXNYCLTHHFQSC", "name": "\u534e\u590f\u65b0\u80fd\u6e90\u8f66\u9f99\u5934\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINNENGYUANCHELONGTOUHUNHEFAQISHIC"}, "013397": {"suoxie": "SYWJYX12GYCYHHFQSFOFA", "name": "\u4e0a\u94f6\u7a33\u5065\u4f18\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINWENJIANYOUXUAN12GEYUECHIYOUHUNHEFAQISHIFOFA"}, "013398": {"suoxie": "SYWJYX12GYCYHHFQSFOFC", "name": "\u4e0a\u94f6\u7a33\u5065\u4f18\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGYINWENJIANYOUXUAN12GEYUECHIYOUHUNHEFAQISHIFOFC"}, "013399": {"suoxie": "DCWY90TGDCYZQA", "name": "\u5927\u6210\u7a33\u76ca90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENYI90TIANGUNDONGCHIYOUZHAIQUANA"}, "013400": {"suoxie": "DCWY90TGDCYZQC", "name": "\u5927\u6210\u7a33\u76ca90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENYI90TIANGUNDONGCHIYOUZHAIQUANC"}, "013401": {"suoxie": "DCWY90TGDCYZQE", "name": "\u5927\u6210\u7a33\u76ca90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENYI90TIANGUNDONGCHIYOUZHAIQUANE"}, "013402": {"suoxie": "HXHSKJETFFQSLJQDIIA", "name": "\u534e\u590f\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)A", "style": "QDII", "pinyin": "HUAXIAHENGSHENGKEJIETFFAQISHILIANJIEQDIIA"}, "013403": {"suoxie": "HXHSKJETFFQSLJQDIIC", "name": "\u534e\u590f\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)C", "style": "QDII", "pinyin": "HUAXIAHENGSHENGKEJIETFFAQISHILIANJIEQDIIC"}, "013404": {"suoxie": "DCBP500DQZZSQDIIMY", "name": "\u5927\u6210\u6807\u666e500\u7b49\u6743\u91cd\u6307\u6570(QDII)\u7f8e\u5143", "style": "QDII", "pinyin": "DACHENGBIAOPU500DENGQUANZHONGZHISHUQDIIMEIYUAN"}, "013408": {"suoxie": "FCFHZQA", "name": "\u8702\u5de2\u4e30\u548c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOFENGHEZHAIQUANA"}, "013409": {"suoxie": "FCFHZQC", "name": "\u8702\u5de2\u4e30\u548c\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FENGCHAOFENGHEZHAIQUANC"}, "013410": {"suoxie": "BSYLLHPZHHC", "name": "\u535a\u65f6\u88d5\u9686\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYULONGLINGHUOPEIZHIHUNHEC"}, "013411": {"suoxie": "JSFZ6GYGDCYZQFQA", "name": "\u5609\u5b9e\u65b9\u821f6\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIFANGZHOU6GEYUEGUNDONGCHIYOUZHAIQUANFAQIA"}, "013412": {"suoxie": "JSFZ6GYGDCYZQFQC", "name": "\u5609\u5b9e\u65b9\u821f6\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIFANGZHOU6GEYUEGUNDONGCHIYOUZHAIQUANFAQIC"}, "013413": {"suoxie": "JYZZHJZLLOFC", "name": "\u4ea4\u94f6\u4e2d\u8bc1\u73af\u5883\u6cbb\u7406(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINZHONGZHENGHUANJINGZHILILOFC"}, "013414": {"suoxie": "TPZYSGYDKGPFQS", "name": "\u592a\u5e73\u667a\u8fdc\u4e09\u4e2a\u6708\u5b9a\u5f00\u80a1\u7968\u53d1\u8d77\u5f0f", "style": "\u80a1\u7968\u578b", "pinyin": "TAIPINGZHIYUANSANGEYUEDINGKAIGUPIAOFAQISHI"}, "013415": {"suoxie": "YYZZQZYLQXETFFQLJA", "name": "\u6c38\u8d62\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u5668\u68b0ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGZHONGZHENGQUANZHIYILIAOQIXIEETFFAQILIANJIEA"}, "013416": {"suoxie": "YYZZQZYLQXETFFQLJC", "name": "\u6c38\u8d62\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u5668\u68b0ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGZHONGZHENGQUANZHIYILIAOQIXIEETFFAQILIANJIEC"}, "013417": {"suoxie": "BSHXZCJXHHA", "name": "\u535a\u65f6\u6838\u5fc3\u8d44\u4ea7\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHEXINZICHANJINGXUANHUNHEA"}, "013418": {"suoxie": "BSHXZCJXHHC", "name": "\u535a\u65f6\u6838\u5fc3\u8d44\u4ea7\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHEXINZICHANJINGXUANHUNHEC"}, "013419": {"suoxie": "JYYJCZYNDKZ", "name": "\u4ea4\u94f6\u88d5\u666f\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUJINGCHUNZHAIYINIANDINGKAIZHAI"}, "013420": {"suoxie": "FGXHYLMB2045WNCYQHHFQFOF", "name": "\u5bcc\u56fd\u946b\u6c47\u517b\u8001\u76ee\u68072045\u4e94\u5e74\u6301\u6709\u671f\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINHUIYANGLAOMUBIAO2045WUNIANCHIYOUQIHUNHEFAQIFOF"}, "013422": {"suoxie": "TPZXSGYDKHHFQS", "name": "\u592a\u5e73\u667a\u884c\u4e09\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIPINGZHIXINGSANGEYUEDINGKAIHUNHEFAQISHI"}, "013423": {"suoxie": "BYASZDZZQA", "name": "\u5b9d\u76c8\u5b89\u76db\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BAOYINGANSHENGZHONGDUANZHAIZHAIQUANA"}, "013424": {"suoxie": "BYASZDZZQC", "name": "\u5b9d\u76c8\u5b89\u76db\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "BAOYINGANSHENGZHONGDUANZHAIZHAIQUANC"}, "013425": {"suoxie": "BSBP500ETFLJMYHQDIIA", "name": "\u535a\u65f6\u6807\u666e500ETF\u8054\u63a5\u7f8e\u5143\u6c47(QDII)A", "style": "QDII", "pinyin": "BOSHIBIAOPU500ETFLIANJIEMEIYUANHUIQDIIA"}, "013426": {"suoxie": "BLDZGXSYHHA", "name": "\u8d1d\u83b1\u5fb7\u4e2d\u56fd\u65b0\u89c6\u91ce\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BEILAIDEZHONGGUOXINSHIYEHUNHEA"}, "013427": {"suoxie": "BLDZGXSYHHC", "name": "\u8d1d\u83b1\u5fb7\u4e2d\u56fd\u65b0\u89c6\u91ce\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BEILAIDEZHONGGUOXINSHIYEHUNHEC"}, "013428": {"suoxie": "DXXX6GYGDCYZQFQA", "name": "\u4e1c\u5174\u946b\u4eab6\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINXIANG6GEYUEGUNDONGCHIYOUZHAIQUANFAQIA"}, "013429": {"suoxie": "DXXX6GYGDCYZQFQC", "name": "\u4e1c\u5174\u946b\u4eab6\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGXINGXINXIANG6GEYUEGUNDONGCHIYOUZHAIQUANFAQIC"}, "013430": {"suoxie": "JYQSHHC", "name": "\u4ea4\u94f6\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQUSHIHUNHEC"}, "013431": {"suoxie": "HTBRJQHXSNCYQHHA", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u6c47\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIHUIXUANSANNIANCHIYOUQIHUNHEA"}, "013432": {"suoxie": "HTBRJQHXSNCYQHHC", "name": "\u534e\u6cf0\u67cf\u745e\u666f\u6c14\u6c47\u9009\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJINGQIHUIXUANSANNIANCHIYOUQIHUNHEC"}, "013433": {"suoxie": "BSHR6GYCYQHHA", "name": "\u535a\u65f6\u6052\u6da66\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRUN6GEYUECHIYOUQIHUNHEA"}, "013434": {"suoxie": "BSHR6GYCYQHHC", "name": "\u535a\u65f6\u6052\u6da66\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRUN6GEYUECHIYOUQIHUNHEC"}, "013435": {"suoxie": "DCJQJXLGYCYHHA", "name": "\u5927\u6210\u666f\u6c14\u7cbe\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGQIJINGXUANLIUGEYUECHIYOUHUNHEA"}, "013436": {"suoxie": "DCJQJXLGYCYHHC", "name": "\u5927\u6210\u666f\u6c14\u7cbe\u9009\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGQIJINGXUANLIUGEYUECHIYOUHUNHEC"}, "013437": {"suoxie": "CTZGZZYSJSZSFQSA", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5e\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZIGUANZHONGZHENGYOUSEJINSHUZHISHUFAQISHIA"}, "013438": {"suoxie": "CTZGZZYSJSZSFQSC", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5e\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZIGUANZHONGZHENGYOUSEJINSHUZHISHUFAQISHIC"}, "013439": {"suoxie": "JSCYYSHHA", "name": "\u5609\u5b9e\u4ea7\u4e1a\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYEYOUSHIHUNHEA"}, "013440": {"suoxie": "JSCYYSHHC", "name": "\u5609\u5b9e\u4ea7\u4e1a\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYEYOUSHIHUNHEC"}, "013441": {"suoxie": "DCCXYL", "name": "\u4e1c\u8d22\u521b\u65b0\u533b\u7597", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAICHUANGXINYILIAO"}, "013442": {"suoxie": "JXZZ1000ZSZQE", "name": "\u5efa\u4fe1\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aE", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG1000ZHISHUZENGQIANGE"}, "013443": {"suoxie": "JXCYBETFLJE", "name": "\u5efa\u4fe1\u521b\u4e1a\u677fETF\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINCHUANGYEBANETFLIANJIEE"}, "013444": {"suoxie": "JXSZ50ETFFQLJE", "name": "\u5efa\u4fe1\u4e0a\u8bc150ETF\u53d1\u8d77\u8054\u63a5E", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHANGZHENG50ETFFAQILIANJIEE"}, "013445": {"suoxie": "DCXPA", "name": "\u4e1c\u8d22\u82af\u7247A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINPIANA"}, "013446": {"suoxie": "DCXPC", "name": "\u4e1c\u8d22\u82af\u7247C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINPIANC"}, "013447": {"suoxie": "HBZZZNZZZTETFFQSLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u667a\u80fd\u5236\u9020\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGZHINENGZHIZAOZHUTIETFFAQISHILIANJIEA"}, "013448": {"suoxie": "HBZZZNZZZTETFFQSLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u667a\u80fd\u5236\u9020\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGZHINENGZHIZAOZHUTIETFFAQISHILIANJIEC"}, "013449": {"suoxie": "GFJNZQC", "name": "\u5e7f\u53d1\u666f\u5b81\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGNINGZHAIQUANC"}, "013450": {"suoxie": "BSFHLHHHA", "name": "\u535a\u65f6\u51e4\u51f0\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIFENGHUANGLINGHANGHUNHEA"}, "013451": {"suoxie": "BSFHLHHHC", "name": "\u535a\u65f6\u51e4\u51f0\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIFENGHUANGLINGHANGHUNHEC"}, "013453": {"suoxie": "JYGZXNYZSLOFC", "name": "\u4ea4\u94f6\u56fd\u8bc1\u65b0\u80fd\u6e90\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINGUOZHENGXINNENGYUANZHISHULOFC"}, "013454": {"suoxie": "FGWJHYSNCYHHA", "name": "\u5bcc\u56fd\u7a33\u5065\u6052\u8fdc\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANHENGYUANSANNIANCHIYOUHUNHEA"}, "013455": {"suoxie": "FGWJHYSNCYHHC", "name": "\u5bcc\u56fd\u7a33\u5065\u6052\u8fdc\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOWENJIANHENGYUANSANNIANCHIYOUHUNHEC"}, "013456": {"suoxie": "CSAYCZZQD", "name": "\u957f\u76db\u5b89\u9038\u7eaf\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGANYICHUNZHAIZHAIQUAND"}, "013457": {"suoxie": "HXDYSGYDKZQA", "name": "\u534e\u590f\u9f0e\u4e1a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYESANGEYUEDINGKAIZHAIQUANA"}, "013458": {"suoxie": "HXDYSGYDKZQC", "name": "\u534e\u590f\u9f0e\u4e1a\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYESANGEYUEDINGKAIZHAIQUANC"}, "013459": {"suoxie": "HXWXZL80TGDCYZQA", "name": "\u534e\u590f\u7a33\u946b\u589e\u522980\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIAWENXINZENGLI80TIANGUNDONGCHIYOUZHAIQUANA"}, "013460": {"suoxie": "HXWXZL80TGDCYZQC", "name": "\u534e\u590f\u7a33\u946b\u589e\u522980\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIAWENXINZENGLI80TIANGUNDONGCHIYOUZHAIQUANC"}, "013461": {"suoxie": "PYCZXFHHA", "name": "\u9e4f\u626c\u6210\u957f\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGCHENGZHANGXIANFENGHUNHEA"}, "013462": {"suoxie": "PYCZXFHHC", "name": "\u9e4f\u626c\u6210\u957f\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGCHENGZHANGXIANFENGHUNHEC"}, "013463": {"suoxie": "DCZYYSYNCYQHHA", "name": "\u5927\u6210\u81f4\u8fdc\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHIYUANYOUSHIYINIANCHIYOUQIHUNHEA"}, "013464": {"suoxie": "DCZYYSYNCYQHHC", "name": "\u5927\u6210\u81f4\u8fdc\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHIYUANYOUSHIYINIANCHIYOUQIHUNHEC"}, "013465": {"suoxie": "BSZXLHDYZGPA", "name": "\u535a\u65f6\u667a\u9009\u91cf\u5316\u591a\u56e0\u5b50\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIZHIXUANLIANGHUADUOYINZIGUPIAOA"}, "013466": {"suoxie": "BSZXLHDYZGPC", "name": "\u535a\u65f6\u667a\u9009\u91cf\u5316\u591a\u56e0\u5b50\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIZHIXUANLIANGHUADUOYINZIGUPIAOC"}, "013467": {"suoxie": "HXAYWJYLMBYNCYHHFOF", "name": "\u534e\u590f\u5b89\u76c8\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAANYINGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "013468": {"suoxie": "CSSKCZZQD", "name": "\u957f\u76db\u76db\u5eb7\u7eaf\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGSHENGSHENGKANGCHUNZHAIZHAIQUAND"}, "013469": {"suoxie": "TXDTJJHHFQSA", "name": "\u6cf0\u4fe1\u4f4e\u78b3\u7ecf\u6d4e\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINDITANJINGJIHUNHEFAQISHIA"}, "013470": {"suoxie": "TXDTJJHHFQSC", "name": "\u6cf0\u4fe1\u4f4e\u78b3\u7ecf\u6d4e\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINDITANJINGJIHUNHEFAQISHIC"}, "013471": {"suoxie": "HBZZQZNMYZSFQSA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u5168\u6307\u519c\u7267\u6e14\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGQUANZHINONGMUYUZHISHUFAQISHIA"}, "013472": {"suoxie": "HBZZQZNMYZSFQSC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u5168\u6307\u519c\u7267\u6e14\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGQUANZHINONGMUYUZHISHUFAQISHIC"}, "013473": {"suoxie": "HBZZXCLETFFQSLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u65b0\u6750\u6599ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXINCAILIAOETFFAQISHILIANJIEA"}, "013474": {"suoxie": "HBZZXCLETFFQSLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u65b0\u6750\u6599ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXINCAILIAOETFFAQISHILIANJIEC"}, "013475": {"suoxie": "HBZZZNDDQCETFFQSLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGZHINENGDIANDONGQICHEETFFAQISHILIANJIEA"}, "013476": {"suoxie": "HBZZZNDDQCETFFQSLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGZHINENGDIANDONGQICHEETFFAQISHILIANJIEC"}, "013477": {"suoxie": "HBZZJRKJZTETFFQSLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u91d1\u878d\u79d1\u6280\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGJINRONGKEJIZHUTIETFFAQISHILIANJIEA"}, "013478": {"suoxie": "HBZZJRKJZTETFFQSLJC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u91d1\u878d\u79d1\u6280\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGJINRONGKEJIZHUTIETFFAQISHILIANJIEC"}, "013479": {"suoxie": "JYXJZZGPLOFC", "name": "\u91d1\u9e70\u5148\u8fdb\u5236\u9020\u80a1\u7968(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGXIANJINZHIZAOGUPIAOLOFC"}, "013480": {"suoxie": "HBGZZLZSFQA", "name": "\u534e\u5b9d\u56fd\u8bc1\u6cbb\u7406\u6307\u6570\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOGUOZHENGZHILIZHISHUFAQIA"}, "013481": {"suoxie": "HBGZZLZSFQC", "name": "\u534e\u5b9d\u56fd\u8bc1\u6cbb\u7406\u6307\u6570\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOGUOZHENGZHILIZHISHUFAQIC"}, "013483": {"suoxie": "HAYLCXHHC", "name": "\u534e\u5b89\u533b\u7597\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYILIAOCHUANGXINHUNHEC"}, "013484": {"suoxie": "CCHYLDHHC", "name": "\u957f\u57ce\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGHANGYELUNDONGHUNHEC"}, "013485": {"suoxie": "SZJZYSHHFQA", "name": "\u5c1a\u6b63\u7ade\u4e89\u4f18\u52bf\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGZHENGJINGZHENGYOUSHIHUNHEFAQIA"}, "013486": {"suoxie": "SZJZYSHHFQC", "name": "\u5c1a\u6b63\u7ade\u4e89\u4f18\u52bf\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGZHENGJINGZHENGYOUSHIHUNHEFAQIC"}, "013487": {"suoxie": "HALRYNDKZQFQS", "name": "\u534e\u5b89\u9886\u8363\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANLINGRONGYINIANDINGKAIZHAIQUANFAQISHI"}, "013488": {"suoxie": "CXDLHHC", "name": "\u957f\u4fe1\u591a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINDUOLIHUNHEC"}, "013489": {"suoxie": "GFDCDSJJXHHC", "name": "\u5e7f\u53d1\u4e1c\u8d22\u5927\u6570\u636e\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFADONGCAIDASHUJUJINGXUANHUNHEC"}, "013490": {"suoxie": "TTJRJXGPA", "name": "\u540c\u6cf0\u91d1\u878d\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAIJINRONGJINGXUANGUPIAOA"}, "013491": {"suoxie": "TTJRJXGPC", "name": "\u540c\u6cf0\u91d1\u878d\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TONGTAIJINRONGJINGXUANGUPIAOC"}, "013492": {"suoxie": "JSCC30TGDCYDZA", "name": "\u666f\u987a\u957f\u57ce30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINGSHUNCHANGCHENG30TIANGUNDONGCHIYOUDUANZHAIA"}, "013493": {"suoxie": "JSCC30TGDCYDZC", "name": "\u666f\u987a\u957f\u57ce30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JINGSHUNCHANGCHENG30TIANGUNDONGCHIYOUDUANZHAIC"}, "013494": {"suoxie": "HTBRJYZQ", "name": "\u534e\u6cf0\u67cf\u745e\u9526\u5143\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIJINYUANZHAIQUAN"}, "013495": {"suoxie": "XDAYCYYXYNCYHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4ea7\u4e1a\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINCHANYEYOUXUANYINIANCHIYOUHUNHEA"}, "013496": {"suoxie": "XDAYCYYXYNCYHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4ea7\u4e1a\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINCHANYEYOUXUANYINIANCHIYOUHUNHEC"}, "013497": {"suoxie": "YFDYHLLZ3GYDKZ", "name": "\u6613\u65b9\u8fbe\u88d5\u534e\u5229\u7387\u503a3\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAYUHUALILVZHAI3GEYUEDINGKAIZHAI"}, "013498": {"suoxie": "YHYFZQ", "name": "\u94f6\u534e\u6c38\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAYONGFENGZHAIQUAN"}, "013499": {"suoxie": "BSBP500ETFLJMYHQDIIC", "name": "\u535a\u65f6\u6807\u666e500ETF\u8054\u63a5\u7f8e\u5143\u6c47(QDII)C", "style": "QDII", "pinyin": "BOSHIBIAOPU500ETFLIANJIEMEIYUANHUIQDIIC"}, "013500": {"suoxie": "NFJRZTLHPZHHC", "name": "\u5357\u65b9\u91d1\u878d\u4e3b\u9898\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGJINRONGZHUTILINGHUOPEIZHIHUNHEC"}, "013501": {"suoxie": "NFPZYXLHPZHHC", "name": "\u5357\u65b9\u54c1\u8d28\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGPINZHIYOUXUANLINGHUOPEIZHIHUNHEC"}, "013502": {"suoxie": "YFDZZNDDTJJZTETFLJA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGNEIDIDITANJINGJIZHUTIETFLIANJIEA"}, "013503": {"suoxie": "YFDZZNDDTJJZTETFLJC", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGNEIDIDITANJINGJIZHUTIETFLIANJIEC"}, "013504": {"suoxie": "HASHQDHHC", "name": "\u534e\u5b89\u53cc\u6838\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANSHUANGHEQUDONGHUNHEC"}, "013505": {"suoxie": "HAXSLZTGPC", "name": "\u534e\u5b89\u65b0\u4e1d\u8def\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANXINSILUZHUTIGUPIAOC"}, "013506": {"suoxie": "HAYJJXHHC", "name": "\u534e\u5b89\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIUJINGXUANHUNHEC"}, "013507": {"suoxie": "HAZZXFHHC", "name": "\u534e\u5b89\u5236\u9020\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHIZAOXIANFENGHUNHEC"}, "013508": {"suoxie": "GFYTZGSYZQDIIC", "name": "\u5e7f\u53d1\u4e9a\u592a\u4e2d\u9ad8\u6536\u76ca\u503a(QDII)C", "style": "QDII", "pinyin": "GUANGFAYATAIZHONGGAOSHOUYIZHAIQDIIC"}, "013509": {"suoxie": "GFYTZGSYZMYXHQDIIC", "name": "\u5e7f\u53d1\u4e9a\u592a\u4e2d\u9ad8\u6536\u76ca\u503a\u7f8e\u5143\u73b0\u6c47(QDII)C", "style": "QDII", "pinyin": "GUANGFAYATAIZHONGGAOSHOUYIZHAIMEIYUANXIANHUIQDIIC"}, "013510": {"suoxie": "GSABYL2030SNCYHHFOF", "name": "\u56fd\u5bff\u5b89\u4fdd\u517b\u80012030\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOSHOUANBAOYANGLAO2030SANNIANCHIYOUHUNHEFOF"}, "013511": {"suoxie": "HFJXDTXFGPC", "name": "\u6c47\u4e30\u664b\u4fe1\u4f4e\u78b3\u5148\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINDITANXIANFENGGUPIAOC"}, "013512": {"suoxie": "HTFTFZZWJYLMBYNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u798f\u589e\u957f\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUTIANFUZENGZHANGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "013513": {"suoxie": "CAXJZZHHA", "name": "\u957f\u5b89\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXIANJINZHIZAOHUNHEA"}, "013514": {"suoxie": "CAXJZZHHC", "name": "\u957f\u5b89\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANXIANJINZHIZAOHUNHEC"}, "013515": {"suoxie": "HTFLCWJHHC", "name": "\u6c47\u6dfb\u5bcc\u84dd\u7b79\u7a33\u5065\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFULANCHOUWENJIANHUNHEC"}, "013516": {"suoxie": "HTFLCWJHHE", "name": "\u6c47\u6dfb\u5bcc\u84dd\u7b79\u7a33\u5065\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFULANCHOUWENJIANHUNHEE"}, "013517": {"suoxie": "YFDYPYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEPUYINIANCHIYOUHUNHEA"}, "013518": {"suoxie": "YFDYPYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u6d66\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEPUYINIANCHIYOUHUNHEC"}, "013519": {"suoxie": "YFDHZPHYLMBSNCYHHFOF", "name": "\u6613\u65b9\u8fbe\u6c47\u667a\u5e73\u8861\u517b\u8001\u76ee\u6807\u4e09\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAHUIZHIPINGHENGYANGLAOMUBIAOSANNIANCHIYOUHUNHEFOF"}, "013520": {"suoxie": "FRZDZZQA", "name": "\u5bcc\u8363\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGZHONGDUANZHAIZHAIQUANA"}, "013521": {"suoxie": "FRZDZZQC", "name": "\u5bcc\u8363\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FURONGZHONGDUANZHAIZHAIQUANC"}, "013522": {"suoxie": "HFJF60TGDCYZDZA", "name": "\u534e\u5bcc\u5409\u4e3060\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAFUJIFENG60TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "013523": {"suoxie": "HFJF60TGDCYZDZC", "name": "\u534e\u5bcc\u5409\u4e3060\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAFUJIFENG60TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013524": {"suoxie": "FGYXHB12GYCYQHHA", "name": "\u5bcc\u56fd\u60a6\u4eab\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOYUEXIANGHUIBAO12GEYUECHIYOUQIHUNHEA"}, "013525": {"suoxie": "FGYXHB12GYCYQHHC", "name": "\u5bcc\u56fd\u60a6\u4eab\u56de\u62a512\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOYUEXIANGHUIBAO12GEYUECHIYOUQIHUNHEC"}, "013526": {"suoxie": "XCXXCYHHC", "name": "\u4fe1\u8bda\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGXINXINGCHANYEHUNHEC"}, "013529": {"suoxie": "NFFYWJYLYNCYHHFOF", "name": "\u5357\u65b9\u5bcc\u8a89\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGFUYUWENJIANYANGLAOYINIANCHIYOUHUNHEFOF"}, "013530": {"suoxie": "HYYFGQZXSJHHC", "name": "\u5f18\u6bc5\u8fdc\u65b9\u56fd\u4f01\u8f6c\u578b\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGGUOQIZHUANXINGSHENGJIHUNHEC"}, "013531": {"suoxie": "ZSJCCYCZHHC", "name": "\u6d59\u5546\u805a\u6f6e\u4ea7\u4e1a\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGJUCHAOCHANYECHENGZHANGHUNHEC"}, "013532": {"suoxie": "GFAHHBHHE", "name": "\u5e7f\u53d1\u5b89\u5b8f\u56de\u62a5\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAANHONGHUIBAOHUNHEE"}, "013533": {"suoxie": "GFKJCXHHC", "name": "\u5e7f\u53d1\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAKEJICHUANGXINHUNHEC"}, "013534": {"suoxie": "PHWXHHA", "name": "\u9e4f\u534e\u6c83\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAWOXINHUNHEA"}, "013535": {"suoxie": "PHWXHHC", "name": "\u9e4f\u534e\u6c83\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAWOXINHUNHEC"}, "013536": {"suoxie": "PHWH90TGDCYZQA", "name": "\u9e4f\u534e\u7a33\u534e90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUAWENHUA90TIANGUNDONGCHIYOUZHAIQUANA"}, "013537": {"suoxie": "PHWH90TGDCYZQC", "name": "\u9e4f\u534e\u7a33\u534e90\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUAWENHUA90TIANGUNDONGCHIYOUZHAIQUANC"}, "013538": {"suoxie": "PHYN3GYDKZQ", "name": "\u9e4f\u534e\u6c38\u5b813\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAYONGNING3GEYUEDINGKAIZHAIQUAN"}, "013539": {"suoxie": "JSYKYLYNCYHHFOF", "name": "\u5609\u5b9e\u60a6\u5eb7\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIYUEKANGYANGLAOYINIANCHIYOUHUNHEFOF"}, "013540": {"suoxie": "HTFWJRXYNCYHHD", "name": "\u6c47\u6dfb\u5bcc\u7a33\u5065\u777f\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408D", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUWENJIANRUIXIANGYINIANCHIYOUHUNHED"}, "013541": {"suoxie": "YHZZ13NGKZA", "name": "\u94f6\u6cb3\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEZHONGZHAI13NIANGUOKAIZHAIA"}, "013542": {"suoxie": "YHZZ13NGKZC", "name": "\u94f6\u6cb3\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEZHONGZHAI13NIANGUOKAIZHAIC"}, "013543": {"suoxie": "YDAHCZE", "name": "\u82f1\u5927\u5b89\u60e0\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDAANHUICHUNZHAIE"}, "013544": {"suoxie": "JSZY3GYDQCZZQ", "name": "\u5609\u5b9e\u81f4\u8fdc3\u4e2a\u6708\u5b9a\u671f\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIYUAN3GEYUEDINGQICHUNZHAIZHAIQUAN"}, "013545": {"suoxie": "HXZXYNDKZQFQS", "name": "\u534e\u590f\u5353\u4fe1\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAZHUOXINYINIANDINGKAIZHAIQUANFAQISHI"}, "013546": {"suoxie": "CTZGHX30TGDZDZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u4eab30\u5929\u6eda\u52a8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGXIANG30TIANGUNDONGZHONGDUANZHAIA"}, "013547": {"suoxie": "CTZGHX30TGDZDZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u4eab30\u5929\u6eda\u52a8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGXIANG30TIANGUNDONGZHONGDUANZHAIC"}, "013548": {"suoxie": "ZSXLZQZQA", "name": "\u62db\u5546\u4eab\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXIANGLIZENGQIANGZHAIQUANA"}, "013549": {"suoxie": "ZSXLZQZQC", "name": "\u62db\u5546\u4eab\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXIANGLIZENGQIANGZHAIQUANC"}, "013550": {"suoxie": "HTFPPJZYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u54c1\u724c\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUPINPAIJIAZHIYINIANCHIYOUHUNHEA"}, "013551": {"suoxie": "HTFPPJZYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u54c1\u724c\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUPINPAIJIAZHIYINIANCHIYOUHUNHEC"}, "013552": {"suoxie": "HTFCZLXHHA", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGLINGXIANHUNHEA"}, "013553": {"suoxie": "HTFCZLXHHC", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGLINGXIANHUNHEC"}, "013554": {"suoxie": "XDAYYJJZHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYUANJIANJIAZHIHUNHEA"}, "013555": {"suoxie": "XDAYYJJZHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYUANJIANJIAZHIHUNHEC"}, "013558": {"suoxie": "CXLFZQC", "name": "\u957f\u4fe1\u5229\u5bcc\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFUZHAIQUANC"}, "013559": {"suoxie": "ZSJHHBHHA", "name": "\u62db\u5546\u5747\u8861\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGJUNHENGHUIBAOHUNHEA"}, "013560": {"suoxie": "ZSJHHBHHC", "name": "\u62db\u5546\u5747\u8861\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGJUNHENGHUIBAOHUNHEC"}, "013561": {"suoxie": "ZRJXYXHHA", "name": "\u4e2d\u878d\u5320\u5fc3\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJIANGXINYOUXUANHUNHEA"}, "013562": {"suoxie": "ZRJXYXHHC", "name": "\u4e2d\u878d\u5320\u5fc3\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJIANGXINYOUXUANHUNHEC"}, "013563": {"suoxie": "YHJJY3GYGDCYZQA", "name": "\u94f6\u534e\u5b63\u5b63\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAJIJIYING3GEYUEGUNDONGCHIYOUZHAIQUANA"}, "013564": {"suoxie": "YHJJY3GYGDCYZQB", "name": "\u94f6\u534e\u5b63\u5b63\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAJIJIYING3GEYUEGUNDONGCHIYOUZHAIQUANB"}, "013565": {"suoxie": "YHJJY3GYGDCYZQC", "name": "\u94f6\u534e\u5b63\u5b63\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAJIJIYING3GEYUEGUNDONGCHIYOUZHAIQUANC"}, "013566": {"suoxie": "HXJGAQHHC", "name": "\u534e\u590f\u519b\u5de5\u5b89\u5168\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAJUNGONGANQUANHUNHEC"}, "013567": {"suoxie": "SWLXHLHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u5b8f\u91cf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINHONGLIANGHUNHEA"}, "013568": {"suoxie": "SWLXHLHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u5b8f\u91cf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINHONGLIANGHUNHEC"}, "013569": {"suoxie": "THYLYJHHA", "name": "\u5929\u5f18\u6c38\u5229\u4f18\u4f73\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYONGLIYOUJIAHUNHEA"}, "013570": {"suoxie": "THYLYJHHC", "name": "\u5929\u5f18\u6c38\u5229\u4f18\u4f73\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYONGLIYOUJIAHUNHEC"}, "013573": {"suoxie": "ZYXYZDZZQA", "name": "\u4e2d\u90ae\u946b\u6ea2\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUXINYIZHONGDUANZHAIZHAIQUANA"}, "013574": {"suoxie": "ZYXYZDZZQC", "name": "\u4e2d\u90ae\u946b\u6ea2\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUXINYIZHONGDUANZHAIZHAIQUANC"}, "013577": {"suoxie": "HAZY60TGDCYDZA", "name": "\u534e\u5b89\u4f17\u60a660\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANZHONGYUE60TIANGUNDONGCHIYOUDUANZHAIA"}, "013578": {"suoxie": "HAZY60TGDCYDZC", "name": "\u534e\u5b89\u4f17\u60a660\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANZHONGYUE60TIANGUNDONGCHIYOUDUANZHAIC"}, "013581": {"suoxie": "ZHHYHHA", "name": "\u4e2d\u6d77\u6d77\u9890\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHAIHAIYIHUNHEA"}, "013582": {"suoxie": "ZHHYHHC", "name": "\u4e2d\u6d77\u6d77\u9890\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHAIHAIYIHUNHEC"}, "013583": {"suoxie": "ZSPZLHHHA", "name": "\u62db\u5546\u54c1\u8d28\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHILINGHANGHUNHEA"}, "013584": {"suoxie": "ZSPZLHHHC", "name": "\u62db\u5546\u54c1\u8d28\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGPINZHILINGHANGHUNHEC"}, "013585": {"suoxie": "THQXZQFQA", "name": "\u5929\u5f18\u9f50\u4eab\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGQIXIANGZHAIQUANFAQIA"}, "013586": {"suoxie": "THQXZQFQC", "name": "\u5929\u5f18\u9f50\u4eab\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGQIXIANGZHAIQUANFAQIC"}, "013587": {"suoxie": "YDCZZQE", "name": "\u82f1\u5927\u7eaf\u503a\u503a\u5238E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDACHUNZHAIZHAIQUANE"}, "013588": {"suoxie": "GYWJRYYNCYZQA", "name": "\u5de5\u94f6\u7a33\u5065\u745e\u76c8\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINWENJIANRUIYINGYINIANCHIYOUZHAIQUANA"}, "013589": {"suoxie": "GYWJRYYNCYZQC", "name": "\u5de5\u94f6\u7a33\u5065\u745e\u76c8\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINWENJIANRUIYINGYINIANCHIYOUZHAIQUANC"}, "013590": {"suoxie": "NFBJYSHHA", "name": "\u5357\u65b9\u6bd4\u8f83\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGBIJIAOYOUSHIHUNHEA"}, "013591": {"suoxie": "NFBJYSHHC", "name": "\u5357\u65b9\u6bd4\u8f83\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGBIJIAOYOUSHIHUNHEC"}, "013592": {"suoxie": "NFZZ13NGKHZQZSE", "name": "\u5357\u65b9\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u884c\u503a\u5238\u6307\u6570E", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGZHONGZHAI13NIANGUOKAIHANGZHAIQUANZHISHUE"}, "013593": {"suoxie": "NF35NNFZE", "name": "\u5357\u65b93-5\u5e74\u519c\u53d1\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG35NIANNONGFAZHAIE"}, "013594": {"suoxie": "NF710NGKZE", "name": "\u5357\u65b97-10\u5e74\u56fd\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG710NIANGUOKAIZHAIE"}, "013595": {"suoxie": "YYWJZL18GYCYHHE", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u522918\u4e2a\u6708\u6301\u6709\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGWENJIANZENGLI18GEYUECHIYOUHUNHEE"}, "013596": {"suoxie": "ZSZZMTDQZSLOFC", "name": "\u62db\u5546\u4e2d\u8bc1\u7164\u70ad\u7b49\u6743\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGMEITANDENGQUANZHISHULOFC"}, "013597": {"suoxie": "ZSZZQZZQGSZSLOFC", "name": "\u62db\u5546\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFC"}, "013598": {"suoxie": "HRYDZXHBHHA", "name": "\u534e\u6da6\u5143\u5927\u81fb\u9009\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAZHENXUANHUIBAOHUNHEA"}, "013599": {"suoxie": "HRYDZXHBHHC", "name": "\u534e\u6da6\u5143\u5927\u81fb\u9009\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUARUNYUANDAZHENXUANHUIBAOHUNHEC"}, "013600": {"suoxie": "JTRFSJQDHHFQSLOFC", "name": "\u4e5d\u6cf0\u9510\u5bcc\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408\u53d1\u8d77\u5f0f(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAIRUIFUSHIJIANQUDONGHUNHEFAQISHILOFC"}, "013601": {"suoxie": "GTZZGFCYETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGUANGFUCHANYEETFFAQILIANJIEA"}, "013602": {"suoxie": "GTZZGFCYETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGUANGFUCHANYEETFFAQILIANJIEC"}, "013605": {"suoxie": "HXZZNDDTJJZTETFFQSLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGNEIDIDITANJINGJIZHUTIETFFAQISHILIANJIEA"}, "013606": {"suoxie": "HXZZNDDTJJZTETFFQSLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGNEIDIDITANJINGJIZHUTIETFFAQISHILIANJIEC"}, "013607": {"suoxie": "GFRHJQYNCYQHHA", "name": "\u5e7f\u53d1\u777f\u6052\u8fdb\u53d6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIHENGJINQUYINIANCHIYOUQIHUNHEA"}, "013608": {"suoxie": "GFRHJQYNCYQHHC", "name": "\u5e7f\u53d1\u777f\u6052\u8fdb\u53d6\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIHENGJINQUYINIANCHIYOUQIHUNHEC"}, "013609": {"suoxie": "GDBDXZZ15NZJZD", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDABAODEXINZHONGZHAI15NIANZHENGJINZHAID"}, "013610": {"suoxie": "ZXBCQZYSHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u524d\u77bb\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGQIANZHANYOUSHIHUNHE"}, "013611": {"suoxie": "GYMRYNCYHHA", "name": "\u5de5\u94f6\u6c11\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINMINRUIYINIANCHIYOUHUNHEA"}, "013612": {"suoxie": "GYMRYNCYHHC", "name": "\u5de5\u94f6\u6c11\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINMINRUIYINIANCHIYOUHUNHEC"}, "013613": {"suoxie": "BYGJAQHGSGPC", "name": "\u5b9d\u76c8\u56fd\u5bb6\u5b89\u5168\u6caa\u6e2f\u6df1\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BAOYINGGUOJIAANQUANHUGANGSHENGUPIAOC"}, "013614": {"suoxie": "TXXRZQFQSA", "name": "\u6cf0\u4fe1\u946b\u745e\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINXINRUIZHAIQUANFAQISHIA"}, "013615": {"suoxie": "TXXRZQFQSC", "name": "\u6cf0\u4fe1\u946b\u745e\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINXINRUIZHAIQUANFAQISHIC"}, "013618": {"suoxie": "HADAQZTHHC", "name": "\u534e\u5b89\u5927\u5b89\u5168\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANDAANQUANZHUTIHUNHEC"}, "013619": {"suoxie": "HADTLHPZHHC", "name": "\u534e\u5b89\u52a8\u6001\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANDONGTAILINGHUOPEIZHIHUNHEC"}, "013620": {"suoxie": "HAMTHLWHHC", "name": "\u534e\u5b89\u5a92\u4f53\u4e92\u8054\u7f51\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANMEITIHULIANWANGHUNHEC"}, "013621": {"suoxie": "HAZNSHHHC", "name": "\u534e\u5b89\u667a\u80fd\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHINENGSHENGHUOHUNHEC"}, "013622": {"suoxie": "HAZNZBZTGPC", "name": "\u534e\u5b89\u667a\u80fd\u88c5\u5907\u4e3b\u9898\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAANZHINENGZHUANGBEIZHUTIGUPIAOC"}, "013623": {"suoxie": "XCZQLDYNCYHH", "name": "\u6e58\u8d22\u5468\u671f\u8f6e\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIANGCAIZHOUQILUNDONGYINIANCHIYOUHUNHE"}, "013624": {"suoxie": "JSJZCZSNCYQHHA", "name": "\u5609\u5b9e\u4ef7\u503c\u521b\u9020\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHICHUANGZAOSANNIANCHIYOUQIHUNHEA"}, "013625": {"suoxie": "JSJZCZSNCYQHHC", "name": "\u5609\u5b9e\u4ef7\u503c\u521b\u9020\u4e09\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHICHUANGZAOSANNIANCHIYOUQIHUNHEC"}, "013626": {"suoxie": "HXZQQDHHFQSA", "name": "\u534e\u590f\u5468\u671f\u9a71\u52a8\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAZHOUQIQUDONGHUNHEFAQISHIA"}, "013627": {"suoxie": "HXZQQDHHFQSC", "name": "\u534e\u590f\u5468\u671f\u9a71\u52a8\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAZHOUQIQUDONGHUNHEFAQISHIC"}, "013628": {"suoxie": "GFJYZQA", "name": "\u5e7f\u53d1\u96c6\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUEZHAIQUANA"}, "013629": {"suoxie": "GFJYZQC", "name": "\u5e7f\u53d1\u96c6\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJIYUEZHAIQUANC"}, "013630": {"suoxie": "JSJHZXYNCYHHA", "name": "\u5609\u5b9e\u5747\u8861\u81fb\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJUNHENGZHENXUANYINIANCHIYOUHUNHEA"}, "013631": {"suoxie": "JSJHZXYNCYHHC", "name": "\u5609\u5b9e\u5747\u8861\u81fb\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJUNHENGZHENXUANYINIANCHIYOUHUNHEC"}, "013634": {"suoxie": "SWLXSLHHA", "name": "\u7533\u4e07\u83f1\u4fe1\u53cc\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINSHUANGLIHUNHEA"}, "013635": {"suoxie": "SWLXSLHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u53cc\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHENWANLINGXINSHUANGLIHUNHEC"}, "013636": {"suoxie": "GTRYCLHBHHA", "name": "\u56fd\u6295\u745e\u94f6\u7b56\u7565\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCELUEHUIBAOHUNHEA"}, "013637": {"suoxie": "GTRYCLHBHHC", "name": "\u56fd\u6295\u745e\u94f6\u7b56\u7565\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCELUEHUIBAOHUNHEC"}, "013638": {"suoxie": "HANXCLHHC", "name": "\u534e\u5b89\u9006\u5411\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANNIXIANGCELUEHUNHEC"}, "013639": {"suoxie": "GDBDXZZ500ZSZQA", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGDABAODEXINZHONGZHENG500ZHISHUZENGQIANGA"}, "013640": {"suoxie": "GDBDXZZ500ZSZQC", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGDABAODEXINZHONGZHENG500ZHISHUZENGQIANGC"}, "013641": {"suoxie": "BDCZZHGPA", "name": "\u535a\u9053\u6210\u957f\u667a\u822a\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOCHENGZHANGZHIHANGGUPIAOA"}, "013642": {"suoxie": "BDCZZHGPC", "name": "\u535a\u9053\u6210\u957f\u667a\u822a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BODAOCHENGZHANGZHIHANGGUPIAOC"}, "013645": {"suoxie": "JSCCJTDLYNDKCZA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u9f0e\u5229\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIDINGLIYINIANDINGKAICHUNZHAIA"}, "013646": {"suoxie": "JSCCJTDLYNDKCZC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6cf0\u9f0e\u5229\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGJINGTAIDINGLIYINIANDINGKAICHUNZHAIC"}, "013647": {"suoxie": "HTFJDJZCZYNCYHHFOF", "name": "\u6c47\u6dfb\u5bcc\u7ecf\u5178\u4ef7\u503c\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJINGDIANJIAZHICHENGZHANGYINIANCHIYOUHUNHEFOF"}, "013648": {"suoxie": "CXWFZQA", "name": "\u957f\u4fe1\u7a33\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENFENGZHAIQUANA"}, "013649": {"suoxie": "CXWFZQC", "name": "\u957f\u4fe1\u7a33\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINWENFENGZHAIQUANC"}, "013653": {"suoxie": "ZYSQS05NNFHZQZS", "name": "\u4e2d\u94f6\u4e0a\u6e05\u62400-5\u5e74\u519c\u53d1\u884c\u503a\u5238\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINSHANGQINGSUO05NIANNONGFAHANGZHAIQUANZHISHU"}, "013654": {"suoxie": "YYXLTZHZTYNDKZ", "name": "\u6c38\u8d62\u4fe1\u5229\u78b3\u4e2d\u548c\u4e3b\u9898\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGXINLITANZHONGHEZHUTIYINIANDINGKAIZHAI"}, "013655": {"suoxie": "HACLYXHHC", "name": "\u534e\u5b89\u7b56\u7565\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCELUEYOUXUANHUNHEC"}, "013656": {"suoxie": "BSZXCZZQC", "name": "\u535a\u65f6\u81fb\u9009\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIZHENXUANCHUNZHAIZHAIQUANC"}, "013657": {"suoxie": "TTTXHHA", "name": "\u540c\u6cf0\u540c\u6b23\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TONGTAITONGXINHUNHEA"}, "013658": {"suoxie": "TTTXHHC", "name": "\u540c\u6cf0\u540c\u6b23\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TONGTAITONGXINHUNHEC"}, "013659": {"suoxie": "ZRJRXX3GYCYHHA", "name": "\u4e2d\u878d\u91d1\u878d\u946b\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJINRONGXINXUAN3GEYUECHIYOUHUNHEA"}, "013660": {"suoxie": "ZRJRXX3GYCYHHC", "name": "\u4e2d\u878d\u91d1\u878d\u946b\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGJINRONGXINXUAN3GEYUECHIYOUHUNHEC"}, "013661": {"suoxie": "HAYJLHHHA", "name": "\u534e\u5b89\u7814\u7a76\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIULINGHANGHUNHEA"}, "013662": {"suoxie": "HAYJLHHHC", "name": "\u534e\u5b89\u7814\u7a76\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYANJIULINGHANGHUNHEC"}, "013663": {"suoxie": "FGAF30TGDCYDZFQA", "name": "\u5bcc\u56fd\u5b89\u798f30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUOANFU30TIANGUNDONGCHIYOUDUANZHAIFAQIA"}, "013664": {"suoxie": "FGAF30TGDCYDZFQC", "name": "\u5bcc\u56fd\u5b89\u798f30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "FUGUOANFU30TIANGUNDONGCHIYOUDUANZHAIFAQIC"}, "013665": {"suoxie": "YHCZYXYNCYHHA", "name": "\u94f6\u6cb3\u6210\u957f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHENGZHANGYOUXUANYINIANCHIYOUHUNHEA"}, "013666": {"suoxie": "YHCZYXYNCYHHC", "name": "\u94f6\u6cb3\u6210\u957f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHENGZHANGYOUXUANYINIANCHIYOUHUNHEC"}, "013668": {"suoxie": "YYHYYNCYZQFQFOFA", "name": "\u6c38\u8d62\u6167\u76c8\u4e00\u5e74\u6301\u6709\u503a\u5238\u53d1\u8d77(FOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGHUIYINGYINIANCHIYOUZHAIQUANFAQIFOFA"}, "013669": {"suoxie": "YYHYYNCYZQFQFOFC", "name": "\u6c38\u8d62\u6167\u76c8\u4e00\u5e74\u6301\u6709\u503a\u5238\u53d1\u8d77(FOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGHUIYINGYINIANCHIYOUZHAIQUANFAQIFOFC"}, "013674": {"suoxie": "CCJZZXYNCYHHA", "name": "\u957f\u57ce\u4ef7\u503c\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHIZHENXUANYINIANCHIYOUHUNHEA"}, "013675": {"suoxie": "CCJZZXYNCYHHC", "name": "\u957f\u57ce\u4ef7\u503c\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIAZHIZHENXUANYINIANCHIYOUHUNHEC"}, "013676": {"suoxie": "XYXHYNCYHHA", "name": "\u5174\u94f6\u5174\u6167\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYINXINGHUIYINIANCHIYOUHUNHEA"}, "013677": {"suoxie": "XYXHYNCYHHC", "name": "\u5174\u94f6\u5174\u6167\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYINXINGHUIYINIANCHIYOUHUNHEC"}, "013678": {"suoxie": "FGXXHB12GYCYHHA", "name": "\u5bcc\u56fd\u4fe1\u4eab\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOXINXIANGHUIBAO12GEYUECHIYOUHUNHEA"}, "013679": {"suoxie": "FGXXHB12GYCYHHC", "name": "\u5bcc\u56fd\u4fe1\u4eab\u56de\u62a512\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOXINXIANGHUIBAO12GEYUECHIYOUHUNHEC"}, "013682": {"suoxie": "DBRFZQA", "name": "\u5fb7\u90a6\u9510\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIFENGZHAIQUANA"}, "013683": {"suoxie": "DBRFZQC", "name": "\u5fb7\u90a6\u9510\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANGRUIFENGZHAIQUANC"}, "013686": {"suoxie": "HAAXXFHHC", "name": "\u534e\u5b89\u5b89\u4fe1\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANANXINXIAOFEIHUNHEC"}, "013687": {"suoxie": "PACZLT1NCYHHA", "name": "\u5e73\u5b89\u6210\u957f\u9f99\u59341\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANCHENGZHANGLONGTOU1NIANCHIYOUHUNHEA"}, "013688": {"suoxie": "PACZLT1NCYHHC", "name": "\u5e73\u5b89\u6210\u957f\u9f99\u59341\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANCHENGZHANGLONGTOU1NIANCHIYOUHUNHEC"}, "013689": {"suoxie": "XCJS39GYDQKFZQA", "name": "\u6e58\u8d22\u4e45\u76db39\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANGCAIJIUSHENG39GEYUEDINGQIKAIFANGZHAIQUANA"}, "013690": {"suoxie": "XCJS39GYDQKFZQC", "name": "\u6e58\u8d22\u4e45\u76db39\u4e2a\u6708\u5b9a\u671f\u5f00\u653e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIANGCAIJIUSHENG39GEYUEDINGQIKAIFANGZHAIQUANC"}, "013691": {"suoxie": "XHAHCZA", "name": "\u5174\u534e\u5b89\u6052\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGHUAANHENGCHUNZHAIA"}, "013692": {"suoxie": "XHAHCZC", "name": "\u5174\u534e\u5b89\u6052\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGHUAANHENGCHUNZHAIC"}, "013693": {"suoxie": "BDSXYNCYQHH", "name": "\u535a\u9053\u76db\u5174\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BODAOSHENGXINGYINIANCHIYOUQIHUNHE"}, "013696": {"suoxie": "GFAYWJYLMBYNCYHHFOF", "name": "\u5e7f\u53d1\u5b89\u88d5\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAANYUWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "013699": {"suoxie": "YYAY90TGDCYZQFQA", "name": "\u6c38\u8d62\u5b89\u76c890\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGANYING90TIANGUNDONGCHIYOUZHAIQUANFAQIA"}, "013700": {"suoxie": "YYAY90TGDCYZQFQC", "name": "\u6c38\u8d62\u5b89\u76c890\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YONGYINGANYING90TIANGUNDONGCHIYOUZHAIQUANFAQIC"}, "013703": {"suoxie": "ZSTF1NDKZ", "name": "\u62db\u5546\u6dfb\u798f1\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANFU1NIANDINGKAIZHAI"}, "013704": {"suoxie": "DB90TGDCYZDZZQA", "name": "\u5fb7\u90a690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANG90TIANGUNDONGCHIYOUZHONGDUANZHAIZHAIQUANA"}, "013705": {"suoxie": "DB90TGDCYZDZZQC", "name": "\u5fb7\u90a690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DEBANG90TIANGUNDONGCHIYOUZHONGDUANZHAIZHAIQUANC"}, "013706": {"suoxie": "TTTHSGYDKZA", "name": "\u540c\u6cf0\u6cf0\u548c\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAITAIHESANGEYUEDINGKAIZHAIA"}, "013707": {"suoxie": "TTTHSGYDKZC", "name": "\u540c\u6cf0\u6cf0\u548c\u4e09\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TONGTAITAIHESANGEYUEDINGKAIZHAIC"}, "013708": {"suoxie": "YYYLMBRQ2040SNCYHHFQFOF", "name": "\u6c38\u8d62\u517b\u8001\u76ee\u6807\u65e5\u671f2040\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YONGYINGYANGLAOMUBIAORIQI2040SANNIANCHIYOUHUNHEFAQIFOF"}, "013711": {"suoxie": "GFZRZZTLOFC", "name": "\u5e7f\u53d1\u518d\u878d\u8d44\u4e3b\u9898(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAZAIRONGZIZHUTILOFC"}, "013712": {"suoxie": "FZFBXYYNDKHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u946b\u76ca\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGXINYIYINIANDINGKAIHUNHEA"}, "013713": {"suoxie": "FZFBXYYNDKHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u946b\u76ca\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FANGZHENGFUBANGXINYIYINIANDINGKAIHUNHEC"}, "013714": {"suoxie": "FZFBTL12GYCYHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u6cf0\u522912\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FANGZHENGFUBANGTAILI12GEYUECHIYOUHUNHEA"}, "013715": {"suoxie": "FZFBTL12GYCYHHC", "name": "\u65b9\u6b63\u5bcc\u90a6\u6cf0\u522912\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FANGZHENGFUBANGTAILI12GEYUECHIYOUHUNHEC"}, "013716": {"suoxie": "ZRHLCZA", "name": "\u4e2d\u878d\u6052\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGLICHUNZHAIA"}, "013717": {"suoxie": "ZRHLCZC", "name": "\u4e2d\u878d\u6052\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGLICHUNZHAIC"}, "013718": {"suoxie": "XYWY30TCYQZQA", "name": "\u5174\u94f6\u7a33\u76ca30\u5929\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINWENYI30TIANCHIYOUQIZHAIQUANA"}, "013719": {"suoxie": "XYWY30TCYQZQC", "name": "\u5174\u94f6\u7a33\u76ca30\u5929\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINGYINWENYI30TIANCHIYOUQIZHAIQUANC"}, "013720": {"suoxie": "XHZYZQE", "name": "\u65b0\u534e\u589e\u6021\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGYIZHAIQUANE"}, "013721": {"suoxie": "XDAYJQYXHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINJINGQIYOUXUANHUNHEA"}, "013722": {"suoxie": "XDAYJQYXHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u666f\u6c14\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINJINGQIYOUXUANHUNHEC"}, "013724": {"suoxie": "XDAYXYZQA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u946b\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAAOYINXINYIZHAIQUANA"}, "013725": {"suoxie": "XDAYXYZQC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u946b\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAAOYINXINYIZHAIQUANC"}, "013726": {"suoxie": "HSHCWJYNCYHHA", "name": "\u60e0\u5347\u60e0\u8bda\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUICHENGWENJIANYINIANCHIYOUHUNHEA"}, "013727": {"suoxie": "HSHCWJYNCYHHC", "name": "\u60e0\u5347\u60e0\u8bda\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGHUICHENGWENJIANYINIANCHIYOUHUNHEC"}, "013728": {"suoxie": "CJHN30TGDCYDZA", "name": "\u521b\u91d1\u6052\u5b8130\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHENGNING30TIANGUNDONGCHIYOUDUANZHAIA"}, "013729": {"suoxie": "CJHN30TGDCYDZC", "name": "\u521b\u91d1\u6052\u5b8130\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHUANGJINHENGNING30TIANGUNDONGCHIYOUDUANZHAIC"}, "013730": {"suoxie": "FZFBWH3GYDKZ", "name": "\u65b9\u6b63\u5bcc\u90a6\u7a33\u60523\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FANGZHENGFUBANGWENHENG3GEYUEDINGKAIZHAI"}, "013733": {"suoxie": "HTHTSFHHA", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGFENGHUNHEA"}, "013734": {"suoxie": "HTHTSFHHC", "name": "\u7ea2\u5854\u7ea2\u571f\u76db\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGTAHONGTUSHENGFENGHUNHEC"}, "013735": {"suoxie": "CJTFPHYLSNCYHHFQFOFA", "name": "\u521b\u91d1\u6dfb\u798f\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHUANGJINTIANFUPINGHENGYANGLAOSANNIANCHIYOUHUNHEFAQIFOFA"}, "013736": {"suoxie": "CJTFPHYLSNCYHHFQFOFC", "name": "\u521b\u91d1\u6dfb\u798f\u5e73\u8861\u517b\u8001\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHUANGJINTIANFUPINGHENGYANGLAOSANNIANCHIYOUHUNHEFAQIFOFC"}, "013739": {"suoxie": "BJJLHHA", "name": "\u767e\u5609\u805a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAIJIAJULIHUNHEA"}, "013740": {"suoxie": "BJJLHHC", "name": "\u767e\u5609\u805a\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAIJIAJULIHUNHEC"}, "013742": {"suoxie": "XYJYHHC", "name": "\u5174\u4e1a\u805a\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUYUANHUNHEC"}, "013743": {"suoxie": "TXHLSGYDKZQA", "name": "\u6cf0\u4fe1\u6c47\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINHUILISANGEYUEDINGKAIZHAIQUANA"}, "013744": {"suoxie": "TXHLSGYDKZQC", "name": "\u6cf0\u4fe1\u6c47\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIXINHUILISANGEYUEDINGKAIZHAIQUANC"}, "013745": {"suoxie": "PYASSYX60TGDCYDZA", "name": "\u6d66\u94f6\u5b89\u76db\u53cc\u6708\u946b60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGSHUANGYUEXIN60TIANGUNDONGCHIYOUDUANZHAIA"}, "013746": {"suoxie": "PYASSYX60TGDCYDZC", "name": "\u6d66\u94f6\u5b89\u76db\u53cc\u6708\u946b60\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PUYINANSHENGSHUANGYUEXIN60TIANGUNDONGCHIYOUDUANZHAIC"}, "013747": {"suoxie": "XYJFHHC", "name": "\u5174\u4e1a\u805a\u4e30\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUFENGHUNHEC"}, "013748": {"suoxie": "XYJYHHC", "name": "\u5174\u4e1a\u805a\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGYEJUYINGHUNHEC"}, "013751": {"suoxie": "ZXJTWY90TGDCYZDZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u76ca90\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENYI90TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "013752": {"suoxie": "ZXJTWY90TGDCYZDZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u7a33\u76ca90\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINJIANTOUWENYI90TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "013753": {"suoxie": "ZSWFDZ14TGDCYZA", "name": "\u62db\u5546\u7a33\u798f\u77ed\u503a14\u5929\u6eda\u52a8\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENFUDUANZHAI14TIANGUNDONGCHIYOUZHAIA"}, "013754": {"suoxie": "ZSWFDZ14TGDCYZC", "name": "\u62db\u5546\u7a33\u798f\u77ed\u503a14\u5929\u6eda\u52a8\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENFUDUANZHAI14TIANGUNDONGCHIYOUZHAIC"}, "013755": {"suoxie": "ZYZQNXZZHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANNEIXUZENGZHANGHUNHEA"}, "013756": {"suoxie": "ZYZQNXZZHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANNEIXUZENGZHANGHUNHEC"}, "013757": {"suoxie": "TXJHJZHHA", "name": "\u6cf0\u4fe1\u5747\u8861\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINJUNHENGJIAZHIHUNHEA"}, "013758": {"suoxie": "TXJHJZHHC", "name": "\u6cf0\u4fe1\u5747\u8861\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINJUNHENGJIAZHIHUNHEC"}, "013759": {"suoxie": "ZSJXPHHHA", "name": "\u62db\u5546\u7cbe\u9009\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGJINGXUANPINGHENGHUNHEA"}, "013760": {"suoxie": "ZSJXPHHHC", "name": "\u62db\u5546\u7cbe\u9009\u5e73\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGJINGXUANPINGHENGHUNHEC"}, "013761": {"suoxie": "ZOXXYNCYHHFOFA", "name": "\u4e2d\u6b27\u661f\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINGXUANYINIANCHIYOUHUNHEFOFA"}, "013762": {"suoxie": "ZOXXYNCYHHFOFC", "name": "\u4e2d\u6b27\u661f\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINGXUANYINIANCHIYOUHUNHEFOFC"}, "013763": {"suoxie": "ZOXYYX3GYCYHHFOFA", "name": "\u4e2d\u6b27\u661f\u8000\u4f18\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINGYAOYOUXUAN3GEYUECHIYOUHUNHEFOFA"}, "013764": {"suoxie": "ZOXYYX3GYCYHHFOFC", "name": "\u4e2d\u6b27\u661f\u8000\u4f18\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINGYAOYOUXUAN3GEYUECHIYOUHUNHEFOFC"}, "013765": {"suoxie": "PAHT1NCYHHA", "name": "\u5e73\u5b89\u6052\u6cf01\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGTAI1NIANCHIYOUHUNHEA"}, "013766": {"suoxie": "PAHT1NCYHHC", "name": "\u5e73\u5b89\u6052\u6cf01\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANHENGTAI1NIANCHIYOUHUNHEC"}, "013767": {"suoxie": "PAJZHBHHA", "name": "\u5e73\u5b89\u4ef7\u503c\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIAZHIHUIBAOHUNHEA"}, "013768": {"suoxie": "PAJZHBHHC", "name": "\u5e73\u5b89\u4ef7\u503c\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANJIAZHIHUIBAOHUNHEC"}, "013769": {"suoxie": "BSWY9GYCYHHA", "name": "\u535a\u65f6\u7a33\u76ca9\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIWENYI9GEYUECHIYOUHUNHEA"}, "013770": {"suoxie": "BSWY9GYCYHHC", "name": "\u535a\u65f6\u7a33\u76ca9\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIWENYI9GEYUECHIYOUHUNHEC"}, "013771": {"suoxie": "ZJLTJXHHA", "name": "\u4e2d\u52a0\u9f99\u5934\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIALONGTOUJINGXUANHUNHEA"}, "013772": {"suoxie": "ZJLTJXHHC", "name": "\u4e2d\u52a0\u9f99\u5934\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIALONGTOUJINGXUANHUNHEC"}, "013773": {"suoxie": "GTRYYNDQKFZQFQS", "name": "\u56fd\u6cf0\u777f\u5143\u4e00\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUIYUANYINIANDINGQIKAIFANGZHAIQUANFAQISHI"}, "013774": {"suoxie": "YFDQSYXHHA", "name": "\u6613\u65b9\u8fbe\u8d8b\u52bf\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAQUSHIYOUXUANHUNHEA"}, "013775": {"suoxie": "YFDQSYXHHC", "name": "\u6613\u65b9\u8fbe\u8d8b\u52bf\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAQUSHIYOUXUANHUNHEC"}, "013776": {"suoxie": "ZTXWJZJXHHA", "name": "\u4e2d\u6cf0\u5174\u4e3a\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGWEIJIAZHIJINGXUANHUNHEA"}, "013777": {"suoxie": "ZTXWJZJXHHC", "name": "\u4e2d\u6cf0\u5174\u4e3a\u4ef7\u503c\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGTAIXINGWEIJIAZHIJINGXUANHUNHEC"}, "013778": {"suoxie": "JYXXYNCYQHHFOFA", "name": "\u4ea4\u94f6\u5174\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINXINGXIANGYINIANCHIYOUQIHUNHEFOFA"}, "013779": {"suoxie": "JYXXYNCYQHHFOFC", "name": "\u4ea4\u94f6\u5174\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINXINGXIANGYINIANCHIYOUQIHUNHEFOFC"}, "013780": {"suoxie": "HXDFZQ", "name": "\u534e\u590f\u9f0e\u4e30\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGFENGZHAIQUAN"}, "013781": {"suoxie": "ZSHJZYPZYNCYHHFOFA", "name": "\u6d59\u5546\u6c47\u91d1\u5353\u8d8a\u914d\u7f6e\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUIJINZHUOYUEPEIZHIYINIANCHIYOUHUNHEFOFA"}, "013782": {"suoxie": "ZSHJZYPZYNCYHHFOFB", "name": "\u6d59\u5546\u6c47\u91d1\u5353\u8d8a\u914d\u7f6e\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGHUIJINZHUOYUEPEIZHIYINIANCHIYOUHUNHEFOFB"}, "013785": {"suoxie": "DFHYZZXYNCYHHC", "name": "\u4e1c\u65b9\u7ea2\u4f18\u8d28\u7504\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGYOUZHIZHENXUANYINIANCHIYOUHUNHEC"}, "013786": {"suoxie": "XZQQJJPZSNFBHHFOFLOFC", "name": "\u5174\u8bc1\u5168\u7403\u79ef\u6781\u914d\u7f6e\u4e09\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGQUANQIUJIJIPEIZHISANNIANFENGBIHUNHEFOFLOFC"}, "013787": {"suoxie": "JYZXXGYNFBYZHHFOFLOFC", "name": "\u4ea4\u94f6\u667a\u9009\u661f\u5149\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408(FOF-LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINZHIXUANXINGGUANGYINIANFENGBIYUNZUOHUNHEFOFLOFC"}, "013788": {"suoxie": "HTBRXYZLZLOFB", "name": "\u534e\u6cf0\u67cf\u745e\u4fe1\u7528\u589e\u5229\u503a(LOF)B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIXINYONGZENGLIZHAILOFB"}, "013790": {"suoxie": "DCWA60TGDCYZQA", "name": "\u5927\u6210\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENAN60TIANGUNDONGCHIYOUZHAIQUANA"}, "013791": {"suoxie": "DCWA60TGDCYZQC", "name": "\u5927\u6210\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENAN60TIANGUNDONGCHIYOUZHAIQUANC"}, "013792": {"suoxie": "DCWA60TGDCYZQE", "name": "\u5927\u6210\u7a33\u5b8960\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGWENAN60TIANGUNDONGCHIYOUZHAIQUANE"}, "013793": {"suoxie": "FGZPJX12GYCYHHFOFA", "name": "\u5bcc\u56fd\u667a\u6d66\u7cbe\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHIPUJINGXUAN12GEYUECHIYOUHUNHEFOFA"}, "013794": {"suoxie": "FGZPJX12GYCYHHFOFC", "name": "\u5bcc\u56fd\u667a\u6d66\u7cbe\u900912\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHIPUJINGXUAN12GEYUECHIYOUHUNHEFOFC"}, "013795": {"suoxie": "FGZSJX3GYCYHHFOFA", "name": "\u5bcc\u56fd\u667a\u7533\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHISHENJINGXUAN3GEYUECHIYOUHUNHEFOFA"}, "013796": {"suoxie": "FGZSJX3GYCYHHFOFE", "name": "\u5bcc\u56fd\u667a\u7533\u7cbe\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOZHISHENJINGXUAN3GEYUECHIYOUHUNHEFOFE"}, "013797": {"suoxie": "BSYZXXYNCYQHHA", "name": "\u535a\u65f6\u4f18\u8d28\u946b\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYOUZHIXINXUANYINIANCHIYOUQIHUNHEA"}, "013798": {"suoxie": "BSYZXXYNCYQHHC", "name": "\u535a\u65f6\u4f18\u8d28\u946b\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYOUZHIXINXUANYINIANCHIYOUQIHUNHEC"}, "013799": {"suoxie": "CTAY30TCYQZDZA", "name": "\u8d22\u901a\u5b89\u88d530\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGANYU30TIANCHIYOUQIZHONGDUANZHAIA"}, "013800": {"suoxie": "CTAY30TCYQZDZC", "name": "\u8d22\u901a\u5b89\u88d530\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGANYU30TIANCHIYOUQIZHONGDUANZHAIC"}, "013801": {"suoxie": "CTAY30TCYQZDZE", "name": "\u8d22\u901a\u5b89\u88d530\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGANYU30TIANCHIYOUQIZHONGDUANZHAIE"}, "013802": {"suoxie": "CTZGZZGTZSFQSA", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u8bc1\u94a2\u94c1\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZIGUANZHONGZHENGGANGTIEZHISHUFAQISHIA"}, "013803": {"suoxie": "CTZGZZGTZSFQSC", "name": "\u8d22\u901a\u8d44\u7ba1\u4e2d\u8bc1\u94a2\u94c1\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CAITONGZIGUANZHONGZHENGGANGTIEZHISHUFAQISHIC"}, "013804": {"suoxie": "CTZGHY3GYGDCYZQA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8d8a3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGYUE3GEYUEGUNDONGCHIYOUZHAIQUANA"}, "013805": {"suoxie": "CTZGHY3GYGDCYZQB", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8d8a3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGYUE3GEYUEGUNDONGCHIYOUZHAIQUANB"}, "013806": {"suoxie": "CTZGHY3GYGDCYZQC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8d8a3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGYUE3GEYUEGUNDONGCHIYOUZHAIQUANC"}, "013807": {"suoxie": "CTZGHY3GYGDCYZQE", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u8d8a3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238E", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGYUE3GEYUEGUNDONGCHIYOUZHAIQUANE"}, "013808": {"suoxie": "YFDWY120TGDCYDZZQA", "name": "\u6613\u65b9\u8fbe\u7a33\u60a6120\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENYUE120TIANGUNDONGCHIYOUDUANZHAIZHAIQUANA"}, "013809": {"suoxie": "YFDWY120TGDCYDZZQC", "name": "\u6613\u65b9\u8fbe\u7a33\u60a6120\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAWENYUE120TIANGUNDONGCHIYOUDUANZHAIZHAIQUANC"}, "013810": {"suoxie": "GFKCB50ETFFQLJA", "name": "\u5e7f\u53d1\u79d1\u521b\u677f50ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAKECHUANGBAN50ETFFAQILIANJIEA"}, "013811": {"suoxie": "GFKCB50ETFFQLJC", "name": "\u5e7f\u53d1\u79d1\u521b\u677f50ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAKECHUANGBAN50ETFFAQILIANJIEC"}, "013812": {"suoxie": "JSCCJQJQHHA", "name": "\u666f\u987a\u957f\u57ce\u666f\u6c14\u8fdb\u53d6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGQIJINQUHUNHEA"}, "013813": {"suoxie": "JSCCJQJQHHC", "name": "\u666f\u987a\u957f\u57ce\u666f\u6c14\u8fdb\u53d6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGQIJINQUHUNHEC"}, "013814": {"suoxie": "HTFWX120TGDCYZQA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u946b120\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENXIN120TIANGUNDONGCHIYOUZHAIQUANA"}, "013815": {"suoxie": "HTFWX120TGDCYZQC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u946b120\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENXIN120TIANGUNDONGCHIYOUZHAIQUANC"}, "013816": {"suoxie": "HTFZZGFCYZSZQFQSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGUANGFUCHANYEZHISHUZENGQIANGFAQISHIA"}, "013817": {"suoxie": "HTFZZGFCYZSZQFQSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u589e\u5f3a\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGUANGFUCHANYEZHISHUZENGQIANGFAQISHIC"}, "013820": {"suoxie": "HASSZQ", "name": "\u534e\u5b89\u987a\u7a57\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANSHUNSUIZHAIQUAN"}, "013821": {"suoxie": "NFDLYNDKZQ", "name": "\u5357\u65b9\u5b9a\u5229\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGDINGLIYINIANDINGKAIZHAIQUAN"}, "013822": {"suoxie": "NFYYX30TGDCYZQFQA", "name": "\u5357\u65b9\u6708\u6708\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGYUEYUEXIANG30TIANGUNDONGCHIYOUZHAIQUANFAQIA"}, "013823": {"suoxie": "NFYYX30TGDCYZQFQC", "name": "\u5357\u65b9\u6708\u6708\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGYUEYUEXIANG30TIANGUNDONGCHIYOUZHAIQUANFAQIC"}, "013824": {"suoxie": "HFJXHYHH", "name": "\u6c47\u4e30\u664b\u4fe1\u6167\u60a6\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIFENGJINXINHUIYUEHUNHE"}, "013825": {"suoxie": "GFYXPZLNFBHHFOFLOFC", "name": "\u5e7f\u53d1\u4f18\u9009\u914d\u7f6e\u4e24\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAYOUXUANPEIZHILIANGNIANFENGBIHUNHEFOFLOFC"}, "013826": {"suoxie": "THYFWJYLMBYNCYHHFOF", "name": "\u5929\u5f18\u6c38\u4e30\u7a33\u5065\u517b\u8001\u76ee\u6807\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGYONGFENGWENJIANYANGLAOMUBIAOYINIANCHIYOUHUNHEFOF"}, "013827": {"suoxie": "HTBRHY30TGDCYDZA", "name": "\u534e\u6cf0\u67cf\u745e\u9e3f\u76ca30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAIRUIHONGYI30TIANGUNDONGCHIYOUDUANZHAIA"}, "013828": {"suoxie": "HTBRHY30TGDCYDZC", "name": "\u534e\u6cf0\u67cf\u745e\u9e3f\u76ca30\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUATAIBAIRUIHONGYI30TIANGUNDONGCHIYOUDUANZHAIC"}, "013830": {"suoxie": "ZOJSHHA", "name": "\u4e2d\u6b27\u747e\u5c1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUJINSHANGHUNHEA"}, "013831": {"suoxie": "ZOJSHHC", "name": "\u4e2d\u6b27\u747e\u5c1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUJINSHANGHUNHEC"}, "013832": {"suoxie": "ZOHXYNFBHHFOFLOFC", "name": "\u4e2d\u6b27\u6c47\u9009\u4e00\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHUIXUANYINIANFENGBIHUNHEFOFLOFC"}, "013833": {"suoxie": "GTZZ500ETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1500ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG500ETFFAQILIANJIEA"}, "013834": {"suoxie": "GTZZ500ETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1500ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG500ETFFAQILIANJIEC"}, "013835": {"suoxie": "ZJYXCZZQC", "name": "\u4e2d\u52a0\u4f18\u4eab\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAYOUXIANGCHUNZHAIZHAIQUANC"}, "013836": {"suoxie": "BSSDXFHHA", "name": "\u535a\u65f6\u65f6\u4ee3\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHIDAIXIAOFEIHUNHEA"}, "013837": {"suoxie": "BSSDXFHHC", "name": "\u535a\u65f6\u65f6\u4ee3\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHIDAIXIAOFEIHUNHEC"}, "013838": {"suoxie": "ZYHJ60TGDCYDZA", "name": "\u4e2d\u94f6\u6052\u560960\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYINHENGJIA60TIANGUNDONGCHIYOUDUANZHAIA"}, "013839": {"suoxie": "ZYHJ60TGDCYDZC", "name": "\u4e2d\u94f6\u6052\u560960\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHONGYINHENGJIA60TIANGUNDONGCHIYOUDUANZHAIC"}, "013840": {"suoxie": "YHJCDLHHA", "name": "\u94f6\u534e\u96c6\u6210\u7535\u8def\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJICHENGDIANLUHUNHEA"}, "013841": {"suoxie": "YHJCDLHHC", "name": "\u94f6\u534e\u96c6\u6210\u7535\u8def\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJICHENGDIANLUHUNHEC"}, "013844": {"suoxie": "ZXJTRX6GYCYHHFOFA", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOURUIXUAN6GEYUECHIYOUHUNHEFOFA"}, "013845": {"suoxie": "ZXJTRX6GYCYHHFOFC", "name": "\u4e2d\u4fe1\u5efa\u6295\u777f\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOURUIXUAN6GEYUECHIYOUHUNHEFOFC"}, "013846": {"suoxie": "SYXHHHC", "name": "\u4e0a\u94f6\u946b\u6052\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINXINHENGHUNHEC"}, "013847": {"suoxie": "HTBRJXHXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u5320\u5fc3\u6c47\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIANGXINHUIXUANHUNHEA"}, "013848": {"suoxie": "HTBRJXHXHHC", "name": "\u534e\u6cf0\u67cf\u745e\u5320\u5fc3\u6c47\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIANGXINHUIXUANHUNHEC"}, "013849": {"suoxie": "TTYXPZ3GYCYHHFOFA", "name": "\u540c\u6cf0\u4f18\u9009\u914d\u7f6e3\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIYOUXUANPEIZHI3GEYUECHIYOUHUNHEFOFA"}, "013850": {"suoxie": "TTYXPZ3GYCYHHFOFC", "name": "\u540c\u6cf0\u4f18\u9009\u914d\u7f6e3\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIYOUXUANPEIZHI3GEYUECHIYOUHUNHEFOFC"}, "013851": {"suoxie": "ZXJTDTCZHHA", "name": "\u4e2d\u4fe1\u5efa\u6295\u4f4e\u78b3\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUDITANCHENGZHANGHUNHEA"}, "013852": {"suoxie": "ZXJTDTCZHHC", "name": "\u4e2d\u4fe1\u5efa\u6295\u4f4e\u78b3\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUDITANCHENGZHANGHUNHEC"}, "013857": {"suoxie": "XDAYYXZQA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4f18\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINDAAOYINYOUXIANGZHAIQUANA"}, "013858": {"suoxie": "XDAYYXZQC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4f18\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "XINDAAOYINYOUXIANGZHAIQUANC"}, "013859": {"suoxie": "BYPZZXHHA", "name": "\u5b9d\u76c8\u54c1\u8d28\u7504\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGPINZHIZHENXUANHUNHEA"}, "013860": {"suoxie": "BYPZZXHHC", "name": "\u5b9d\u76c8\u54c1\u8d28\u7504\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGPINZHIZHENXUANHUNHEC"}, "013861": {"suoxie": "HDCYSJHHA", "name": "\u6cd3\u5fb7\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDECHANYESHENGJIHUNHEA"}, "013862": {"suoxie": "HDCYSJHHC", "name": "\u6cd3\u5fb7\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDECHANYESHENGJIHUNHEC"}, "013863": {"suoxie": "CTDLCZC", "name": "\u8d22\u901a\u591a\u5229\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CAITONGDUOLICHUNZHAIC"}, "013864": {"suoxie": "PAYH30TGDCYDZA", "name": "\u5e73\u5b89\u5143\u6cd330\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANHONG30TIANGUNDONGCHIYOUDUANZHAIA"}, "013865": {"suoxie": "PAYH30TGDCYDZC", "name": "\u5e73\u5b89\u5143\u6cd330\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PINGANYUANHONG30TIANGUNDONGCHIYOUDUANZHAIC"}, "013867": {"suoxie": "HAYSQYJXHHA", "name": "\u6c47\u5b89\u4f18\u52bf\u4f01\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANYOUSHIQIYEJINGXUANHUNHEA"}, "013868": {"suoxie": "HAYSQYJXHHC", "name": "\u6c47\u5b89\u4f18\u52bf\u4f01\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANYOUSHIQIYEJINGXUANHUNHEC"}, "013869": {"suoxie": "CJHXWLWZTGPFQA", "name": "\u521b\u91d1\u5408\u4fe1\u7269\u8054\u7f51\u4e3b\u9898\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINWULIANWANGZHUTIGUPIAOFAQIA"}, "013870": {"suoxie": "CJHXWLWZTGPFQC", "name": "\u521b\u91d1\u5408\u4fe1\u7269\u8054\u7f51\u4e3b\u9898\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINWULIANWANGZHUTIGUPIAOFAQIC"}, "013871": {"suoxie": "ZSNYZXHHA", "name": "\u62db\u5546\u80fd\u6e90\u8f6c\u578b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGNENGYUANZHUANXINGHUNHEA"}, "013872": {"suoxie": "ZSNYZXHHC", "name": "\u62db\u5546\u80fd\u6e90\u8f6c\u578b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGNENGYUANZHUANXINGHUNHEC"}, "013875": {"suoxie": "XYHXCZD", "name": "\u946b\u5143\u5408\u4eab\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHEXIANGCHUNZHAID"}, "013876": {"suoxie": "CTZGXNYQCHHFQSA", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANXINNENGYUANQICHEHUNHEFAQISHIA"}, "013877": {"suoxie": "CTZGXNYQCHHFQSC", "name": "\u8d22\u901a\u8d44\u7ba1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGZIGUANXINNENGYUANQICHEHUNHEFAQISHIC"}, "013878": {"suoxie": "YXYFZZ500ZSZQFQA", "name": "\u5706\u4fe1\u6c38\u4e30\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YUANXINYONGFENGZHONGZHENG500ZHISHUZENGQIANGFAQIA"}, "013879": {"suoxie": "YXYFZZ500ZSZQFQC", "name": "\u5706\u4fe1\u6c38\u4e30\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YUANXINYONGFENGZHONGZHENG500ZHISHUZENGQIANGFAQIC"}, "013880": {"suoxie": "GFZXHHC", "name": "\u5e7f\u53d1\u62db\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAZHAOXIANGHUNHEC"}, "013881": {"suoxie": "CXZZ500ZSZQC", "name": "\u957f\u4fe1\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGXINZHONGZHENG500ZHISHUZENGQIANGC"}, "013882": {"suoxie": "JYPZSJHHC", "name": "\u4ea4\u94f6\u54c1\u8d28\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINPINZHISHENGJIHUNHEC"}, "013883": {"suoxie": "JYQMHHC", "name": "\u4ea4\u94f6\u542f\u660e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQIMINGHUNHEC"}, "013884": {"suoxie": "JYZTYXHHC", "name": "\u4ea4\u94f6\u4e3b\u9898\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHUTIYOUXUANHUNHEC"}, "013885": {"suoxie": "JYAEFHXHHC", "name": "\u4ea4\u94f6\u963f\u5c14\u6cd5\u6838\u5fc3\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINAERFAHEXINHUNHEC"}, "013886": {"suoxie": "HSXNYQCHHA", "name": "\u534e\u5546\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGXINNENGYUANQICHEHUNHEA"}, "013887": {"suoxie": "HSXNYQCHHC", "name": "\u534e\u5546\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGXINNENGYUANQICHEHUNHEC"}, "013892": {"suoxie": "SYKJQDSZDQKSHHHC", "name": "\u4e0a\u94f6\u79d1\u6280\u9a71\u52a8\u53cc\u5468\u5b9a\u671f\u53ef\u8d4e\u56de\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGYINKEJIQUDONGSHUANGZHOUDINGQIKESHUHUIHUNHEC"}, "013895": {"suoxie": "BYCZJXHHA", "name": "\u5b9d\u76c8\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGCHENGZHANGJINGXUANHUNHEA"}, "013896": {"suoxie": "BYCZJXHHC", "name": "\u5b9d\u76c8\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGCHENGZHANGJINGXUANHUNHEC"}, "013897": {"suoxie": "DBGGTCZJXHHA", "name": "\u5fb7\u90a6\u6e2f\u80a1\u901a\u6210\u957f\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGGANGGUTONGCHENGZHANGJINGXUANHUNHEA"}, "013898": {"suoxie": "DBGGTCZJXHHC", "name": "\u5fb7\u90a6\u6e2f\u80a1\u901a\u6210\u957f\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGGANGGUTONGCHENGZHANGJINGXUANHUNHEC"}, "013899": {"suoxie": "STMGQJYSGPA", "name": "\u4e0a\u6295\u6469\u6839\u5168\u666f\u4f18\u52bf\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENQUANJINGYOUSHIGUPIAOA"}, "013900": {"suoxie": "STMGQJYSGPC", "name": "\u4e0a\u6295\u6469\u6839\u5168\u666f\u4f18\u52bf\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENQUANJINGYOUSHIGUPIAOC"}, "013901": {"suoxie": "HAZX180TCYQZDZA", "name": "\u534e\u5b89\u4f17\u4eab180\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGXIANG180TIANCHIYOUQIZHONGDUANZHAIA"}, "013902": {"suoxie": "HAZX180TCYQZDZC", "name": "\u534e\u5b89\u4f17\u4eab180\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANZHONGXIANG180TIANCHIYOUQIZHONGDUANZHAIC"}, "013903": {"suoxie": "GTJAXXHYHHFQ", "name": "\u56fd\u6cf0\u541b\u5b89\u4fe1\u606f\u884c\u4e1a\u6df7\u5408\u53d1\u8d77", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJUNANXINXIHANGYEHUNHEFAQI"}, "013904": {"suoxie": "JSCCYLMBRQ2035SNCYHHFQFOF", "name": "\u666f\u987a\u957f\u57ce\u517b\u8001\u76ee\u6807\u65e5\u671f2035\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGYANGLAOMUBIAORIQI2035SANNIANCHIYOUHUNHEFAQIFOF"}, "013907": {"suoxie": "YYSC100ETFFQSLJA", "name": "\u6c38\u8d62\u6df1\u521b100ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGSHENCHUANG100ETFFAQISHILIANJIEA"}, "013908": {"suoxie": "YYSC100ETFFQSLJC", "name": "\u6c38\u8d62\u6df1\u521b100ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGSHENCHUANG100ETFFAQISHILIANJIEC"}, "013909": {"suoxie": "YHZXWJYLYNCYHHFQSFOF", "name": "\u94f6\u534e\u5c0a\u79a7\u7a33\u5065\u517b\u8001\u4e00\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHUAZUNXIWENJIANYANGLAOYINIANCHIYOUHUNHEFAQISHIFOF"}, "013910": {"suoxie": "XYXRLNCYHHA", "name": "\u5174\u4e1a\u5174\u777f\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXINGRUILIANGNIANCHIYOUHUNHEA"}, "013911": {"suoxie": "XYXRLNCYHHC", "name": "\u5174\u4e1a\u5174\u777f\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYEXINGRUILIANGNIANCHIYOUHUNHEC"}, "013912": {"suoxie": "ZOZYWJYNCYHHA", "name": "\u4e2d\u6b27\u62db\u76ca\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUZHAOYIWENJIANYINIANCHIYOUHUNHEA"}, "013913": {"suoxie": "ZOZYWJYNCYHHC", "name": "\u4e2d\u6b27\u62db\u76ca\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUZHAOYIWENJIANYINIANCHIYOUHUNHEC"}, "013918": {"suoxie": "SWLXLHXPGPLOFC", "name": "\u7533\u4e07\u83f1\u4fe1\u91cf\u5316\u5c0f\u76d8\u80a1\u7968(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINLIANGHUAXIAOPANGUPIAOLOFC"}, "013919": {"suoxie": "JXZXPXFGPC", "name": "\u5efa\u4fe1\u4e2d\u5c0f\u76d8\u5148\u950b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINZHONGXIAOPANXIANFENGGUPIAOC"}, "013920": {"suoxie": "XHCXYL6GYCYHHFQA", "name": "\u5174\u534e\u521b\u65b0\u533b\u75976\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUACHUANGXINYILIAO6GEYUECHIYOUHUNHEFAQIA"}, "013921": {"suoxie": "XHCXYL6GYCYHHFQC", "name": "\u5174\u534e\u521b\u65b0\u533b\u75976\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUACHUANGXINYILIAO6GEYUECHIYOUHUNHEFAQIC"}, "013922": {"suoxie": "HXZZ1000ETFFQSLJA", "name": "\u534e\u590f\u4e2d\u8bc11000ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG1000ETFFAQISHILIANJIEA"}, "013923": {"suoxie": "HXZZ1000ETFFQSLJC", "name": "\u534e\u590f\u4e2d\u8bc11000ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG1000ETFFAQISHILIANJIEC"}, "013924": {"suoxie": "TTTXHHA", "name": "\u540c\u6cf0\u540c\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TONGTAITONGXIANGHUNHEA"}, "013925": {"suoxie": "TTTXHHC", "name": "\u540c\u6cf0\u540c\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TONGTAITONGXIANGHUNHEC"}, "013926": {"suoxie": "YMGZXF100ZSZQFQA", "name": "\u6613\u7c73\u56fd\u8bc1\u6d88\u8d39100\u6307\u6570\u589e\u5f3a\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIMIGUOZHENGXIAOFEI100ZHISHUZENGQIANGFAQIA"}, "013927": {"suoxie": "YMGZXF100ZSZQFQC", "name": "\u6613\u7c73\u56fd\u8bc1\u6d88\u8d39100\u6307\u6570\u589e\u5f3a\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIMIGUOZHENGXIAOFEI100ZHISHUZENGQIANGFAQIC"}, "013929": {"suoxie": "ZYZQHR9GYCYHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u6052\u745e9\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANHENGRUI9GEYUECHIYOUHUNHEA"}, "013930": {"suoxie": "ZYZQHR9GYCYHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u6052\u745e9\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGYINZHENGQUANHENGRUI9GEYUECHIYOUHUNHEC"}, "013931": {"suoxie": "BSFHYNDKZFQS", "name": "\u535a\u65f6\u5bcc\u6052\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUHENGYINIANDINGKAIZHAIFAQISHI"}, "013932": {"suoxie": "FGZXHYJXYNFBGPFOFLOFC", "name": "\u5bcc\u56fd\u667a\u946b\u884c\u4e1a\u7cbe\u9009\u4e00\u5e74\u5c01\u95ed\u80a1\u7968(FOF-LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOZHIXINHANGYEJINGXUANYINIANFENGBIGUPIAOFOFLOFC"}, "013933": {"suoxie": "GYRZJQYNFBYZGPC", "name": "\u5de5\u94f6\u777f\u667a\u8fdb\u53d6\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINRUIZHIJINQUYINIANFENGBIYUNZUOGUPIAOC"}, "013934": {"suoxie": "CJHLHBHHFQSA", "name": "\u957f\u6c5f\u7ea2\u5229\u56de\u62a5\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGHONGLIHUIBAOHUNHEFAQISHIA"}, "013935": {"suoxie": "CJHLHBHHFQSC", "name": "\u957f\u6c5f\u7ea2\u5229\u56de\u62a5\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGHONGLIHUIBAOHUNHEFAQISHIC"}, "013936": {"suoxie": "GFRSHHA", "name": "\u5e7f\u53d1\u777f\u5347\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUISHENGHUNHEA"}, "013937": {"suoxie": "GFRSHHC", "name": "\u5e7f\u53d1\u777f\u5347\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUISHENGHUNHEC"}, "013938": {"suoxie": "THAKYYHHE", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u517b\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIYANGHUNHEE"}, "013939": {"suoxie": "HQCLYXHHC", "name": "\u6c47\u6cc9\u7b56\u7565\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANCELUEYOUXUANHUNHEC"}, "013940": {"suoxie": "DWYLFWGPA", "name": "\u4e1c\u5434\u533b\u7597\u670d\u52a1\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUYILIAOFUWUGUPIAOA"}, "013941": {"suoxie": "DWYLFWGPC", "name": "\u4e1c\u5434\u533b\u7597\u670d\u52a1\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUYILIAOFUWUGUPIAOC"}, "013942": {"suoxie": "HBZZXYJSZSZQFQA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u6307\u6570\u589e\u5f3a\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIYOUJINSHUZHISHUZENGQIANGFAQIA"}, "013943": {"suoxie": "HBZZXYJSZSZQFQC", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u6307\u6570\u589e\u5f3a\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIYOUJINSHUZHISHUZENGQIANGFAQIC"}, "013946": {"suoxie": "CJHXZZCZZQC", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u667a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNZHICHUNZHAIZHAIQUANC"}, "013949": {"suoxie": "JYKRKJCXHHC", "name": "\u4ea4\u94f6\u79d1\u9510\u79d1\u6280\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINKERUIKEJICHUANGXINHUNHEC"}, "013950": {"suoxie": "JYXFHHC", "name": "\u4ea4\u94f6\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANFENGHUNHEC"}, "013951": {"suoxie": "CJHXZLCZZQC", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u9686\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNLONGCHUNZHAIZHAIQUANC"}, "013952": {"suoxie": "GYRH3GYDKZQA", "name": "\u5de5\u94f6\u745e\u548c3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIHE3GEYUEDINGKAIZHAIQUANA"}, "013953": {"suoxie": "GYRH3GYDKZQC", "name": "\u5de5\u94f6\u745e\u548c3\u4e2a\u6708\u5b9a\u5f00\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIHE3GEYUEDINGKAIZHAIQUANC"}, "013955": {"suoxie": "GFZXPJXHHC", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZHONGXIAOPANJINGXUANHUNHEC"}, "013956": {"suoxie": "HSYYXFJXHHA", "name": "\u534e\u5546\u533b\u836f\u6d88\u8d39\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGYIYAOXIAOFEIJINGXUANHUNHEA"}, "013957": {"suoxie": "HSYYXFJXHHC", "name": "\u534e\u5546\u533b\u836f\u6d88\u8d39\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGYIYAOXIAOFEIJINGXUANHUNHEC"}, "013958": {"suoxie": "HSXXHBYNCYHHA", "name": "\u534e\u5546\u946b\u9009\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGXINXUANHUIBAOYINIANCHIYOUHUNHEA"}, "013959": {"suoxie": "HSXXHBYNCYHHC", "name": "\u534e\u5546\u946b\u9009\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGXINXUANHUIBAOYINIANCHIYOUHUNHEC"}, "013960": {"suoxie": "WJXJYCZYNCYQHHFQSA", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAXINJIYUCHENGZHANGYINIANCHIYOUQIHUNHEFAQISHIA"}, "013961": {"suoxie": "WJXJYCZYNCYQHHFQSC", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAXINJIYUCHENGZHANGYINIANCHIYOUQIHUNHEFAQISHIC"}, "013962": {"suoxie": "HXCXSYYNCYHHA", "name": "\u534e\u590f\u521b\u65b0\u89c6\u91ce\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINSHIYEYINIANCHIYOUHUNHEA"}, "013963": {"suoxie": "HXCXSYYNCYHHC", "name": "\u534e\u590f\u521b\u65b0\u89c6\u91ce\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINSHIYEYINIANCHIYOUHUNHEC"}, "013966": {"suoxie": "XBLDSRYNDKZQFQ", "name": "\u897f\u90e8\u5229\u5f97\u53cc\u745e\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDESHUANGRUIYINIANDINGKAIZHAIQUANFAQI"}, "013967": {"suoxie": "GFHXYNCYQHHA", "name": "\u5e7f\u53d1\u6052\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXIANGYINIANCHIYOUQIHUNHEA"}, "013968": {"suoxie": "GFHXYNCYQHHC", "name": "\u5e7f\u53d1\u6052\u4eab\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAHENGXIANGYINIANCHIYOUQIHUNHEC"}, "013969": {"suoxie": "HXYLYNCYHHA", "name": "\u534e\u590f\u6c38\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGLIYINIANCHIYOUHUNHEA"}, "013970": {"suoxie": "HXYLYNCYHHC", "name": "\u534e\u590f\u6c38\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAYONGLIYINIANCHIYOUHUNHEC"}, "013971": {"suoxie": "CJZLZQC", "name": "\u957f\u6c5f\u5c0a\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGZUNLIZHAIQUANC"}, "013972": {"suoxie": "CJZH30TGDCYDZFQA", "name": "\u957f\u6c5f\u81f4\u60e030\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGJIANGZHIHUI30TIANGUNDONGCHIYOUDUANZHAIFAQIA"}, "013973": {"suoxie": "CJZH30TGDCYDZFQC", "name": "\u957f\u6c5f\u81f4\u60e030\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGJIANGZHIHUI30TIANGUNDONGCHIYOUDUANZHAIFAQIC"}, "013974": {"suoxie": "GTRYHY90TCYQZDZA", "name": "\u56fd\u6295\u745e\u94f6\u6052\u8a8990\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTOURUIYINHENGYU90TIANCHIYOUQIZHONGDUANZHAIA"}, "013975": {"suoxie": "GTRYHY90TCYQZDZC", "name": "\u56fd\u6295\u745e\u94f6\u6052\u8a8990\u5929\u6301\u6709\u671f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUOTOURUIYINHENGYU90TIANCHIYOUQIZHONGDUANZHAIC"}, "013976": {"suoxie": "CTZGHJ60TGDZDZA", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u4f7360\u5929\u6eda\u52a8\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGJIA60TIANGUNDONGZHONGDUANZHAIA"}, "013977": {"suoxie": "CTZGHJ60TGDZDZC", "name": "\u8d22\u901a\u8d44\u7ba1\u9e3f\u4f7360\u5929\u6eda\u52a8\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CAITONGZIGUANHONGJIA60TIANGUNDONGZHONGDUANZHAIC"}, "013978": {"suoxie": "HSHYCZ3GYDKA", "name": "\u60e0\u5347\u548c\u8d62\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYINGCHUNZHAI3GEYUEDINGKAIA"}, "013979": {"suoxie": "HSHYCZ3GYDKC", "name": "\u60e0\u5347\u548c\u8d62\u7eaf\u503a3\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUISHENGHEYINGCHUNZHAI3GEYUEDINGKAIC"}, "013980": {"suoxie": "GDBDXHXHHA", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u6052\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDABAODEXINHENGXINHUNHEA"}, "013981": {"suoxie": "GDBDXHXHHC", "name": "\u5149\u5927\u4fdd\u5fb7\u4fe1\u6052\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGDABAODEXINHENGXINHUNHEC"}, "013982": {"suoxie": "JHPLYNDKCZZQFQS", "name": "\u5609\u5408\u78d0\u7acb\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAHEPANLIYINIANDINGKAICHUNZHAIZHAIQUANFAQISHI"}, "013985": {"suoxie": "RTWJZL6GYCYHHA", "name": "\u878d\u901a\u7a33\u5065\u589e\u52296\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENJIANZENGLI6GEYUECHIYOUHUNHEA"}, "013986": {"suoxie": "RTWJZL6GYCYHHC", "name": "\u878d\u901a\u7a33\u5065\u589e\u52296\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RONGTONGWENJIANZENGLI6GEYUECHIYOUHUNHEC"}, "013987": {"suoxie": "PYASCFETS05NQYQZQZSFQSA", "name": "\u6d66\u94f6\u5b89\u76dbCFETS0-5\u5e74\u671f\u592e\u4f01\u503a\u5238\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGCFETS05NIANQIYANGQIZHAIQUANZHISHUFAQISHIA"}, "013988": {"suoxie": "PYASCFETS05NQYQZQZSFQSC", "name": "\u6d66\u94f6\u5b89\u76dbCFETS0-5\u5e74\u671f\u592e\u4f01\u503a\u5238\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGCFETS05NIANQIYANGQIZHAIQUANZHISHUFAQISHIC"}, "013991": {"suoxie": "ZOGGTJXYNCYHHA", "name": "\u4e2d\u6b27\u6e2f\u80a1\u901a\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUGANGGUTONGJINGXUANYINIANCHIYOUHUNHEA"}, "013992": {"suoxie": "ZOGGTJXYNCYHHC", "name": "\u4e2d\u6b27\u6e2f\u80a1\u901a\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUGANGGUTONGJINGXUANYINIANCHIYOUHUNHEC"}, "013993": {"suoxie": "ZOGYYNCYHHA", "name": "\u4e2d\u6b27\u5149\u71a0\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUGUANGYIYINIANCHIYOUHUNHEA"}, "013994": {"suoxie": "ZOGYYNCYHHC", "name": "\u4e2d\u6b27\u5149\u71a0\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUGUANGYIYINIANCHIYOUHUNHEC"}, "013995": {"suoxie": "JSRHHHA", "name": "\u5609\u5b9e\u878d\u60e0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIRONGHUIHUNHEA"}, "013996": {"suoxie": "JSRHHHC", "name": "\u5609\u5b9e\u878d\u60e0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHIRONGHUIHUNHEC"}, "013997": {"suoxie": "GFZQZQA", "name": "\u5e7f\u53d1\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZENGQIANGZHAIQUANA"}, "013998": {"suoxie": "ZOJTHHA", "name": "\u4e2d\u6b27\u747e\u6dfb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUJINTIANHUNHEA"}, "013999": {"suoxie": "ZOJTHHC", "name": "\u4e2d\u6b27\u747e\u6dfb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUJINTIANHUNHEC"}, "014000": {"suoxie": "ZOFLZQA", "name": "\u4e2d\u6b27\u4e30\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUFENGLIZHAIQUANA"}, "014001": {"suoxie": "ZOFLZQC", "name": "\u4e2d\u6b27\u4e30\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUFENGLIZHAIQUANC"}, "014002": {"suoxie": "PYASQQZNKJQDIIC", "name": "\u6d66\u94f6\u5b89\u76db\u5168\u7403\u667a\u80fd\u79d1\u6280(QDII)C", "style": "QDII", "pinyin": "PUYINANSHENGQUANQIUZHINENGKEJIQDIIC"}, "014003": {"suoxie": "PYASZZDLHHC", "name": "\u6d66\u94f6\u5b89\u76db\u589e\u957f\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGZENGZHANGDONGLIHUNHEC"}, "014004": {"suoxie": "BSFJCZYNDKZ", "name": "\u535a\u65f6\u5bcc\u749f\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIFUJINGCHUNZHAIYINIANDINGKAIZHAI"}, "014005": {"suoxie": "XYHLZQD", "name": "\u946b\u5143\u9e3f\u5229\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINYUANHONGLIZHAIQUAND"}, "014006": {"suoxie": "ZYYL2050WNCYHHFOF", "name": "\u4e2d\u94f6\u517b\u80012050\u4e94\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINYANGLAO2050WUNIANCHIYOUHUNHEFOF"}, "014007": {"suoxie": "HAZZSJYNCYHHA", "name": "\u534e\u5b89\u5236\u9020\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHIZAOSHENGJIYINIANCHIYOUHUNHEA"}, "014008": {"suoxie": "HAZZSJYNCYHHC", "name": "\u534e\u5b89\u5236\u9020\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHIZAOSHENGJIYINIANCHIYOUHUNHEC"}, "014011": {"suoxie": "PYASJZCZHHC", "name": "\u6d66\u94f6\u5b89\u76db\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGJIAZHICHENGZHANGHUNHEC"}, "014012": {"suoxie": "FCFJCZA", "name": "\u8702\u5de2\u4e30\u5409\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGJICHUNZHAIA"}, "014013": {"suoxie": "FCFJCZC", "name": "\u8702\u5de2\u4e30\u5409\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FENGCHAOFENGJICHUNZHAIC"}, "014014": {"suoxie": "ZSZXPHHHA", "name": "\u62db\u5546\u81fb\u9009\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGZHENXUANPINGHENGHUNHEA"}, "014015": {"suoxie": "ZSZXPHHHC", "name": "\u62db\u5546\u81fb\u9009\u5e73\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGZHENXUANPINGHENGHUNHEC"}, "014018": {"suoxie": "HTFZZHGS500ETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHEN500ETFLIANJIEA"}, "014019": {"suoxie": "HTFZZHGS500ETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHEN500ETFLIANJIEC"}, "014020": {"suoxie": "NDLHXFYNCYHHA", "name": "\u8bfa\u5fb7\u91cf\u5316\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODELIANGHUAXIANFENGYINIANCHIYOUHUNHEA"}, "014021": {"suoxie": "NDLHXFYNCYHHC", "name": "\u8bfa\u5fb7\u91cf\u5316\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODELIANGHUAXIANFENGYINIANCHIYOUHUNHEC"}, "014022": {"suoxie": "DMYLMB2040SNCYHHFQFOF", "name": "\u5927\u6469\u517b\u8001\u76ee\u68072040\u4e09\u5e74\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DAMOYANGLAOMUBIAO2040SANNIANCHIYOUHUNHEFAQIFOF"}, "014023": {"suoxie": "TDHLJQLHLNCYHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u666f\u6c14\u9886\u822a\u4e24\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIJINGQILINGHANGLIANGNIANCHIYOUHUNHE"}, "014024": {"suoxie": "HFZYCZYNCYQHHA", "name": "\u534e\u5bcc\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUZHUOYUECHENGZHANGYINIANCHIYOUQIHUNHEA"}, "014025": {"suoxie": "HFZYCZYNCYQHHC", "name": "\u534e\u5bcc\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUZHUOYUECHENGZHANGYINIANCHIYOUQIHUNHEC"}, "014028": {"suoxie": "ZSZZYHZSC", "name": "\u62db\u5546\u4e2d\u8bc1\u94f6\u884c\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGYINHANGZHISHUC"}, "014029": {"suoxie": "PYASHLJXHHC", "name": "\u6d66\u94f6\u5b89\u76db\u7ea2\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGHONGLIJINGXUANHUNHEC"}, "014030": {"suoxie": "DMJKCYHHC", "name": "\u5927\u6469\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOJIANKANGCHANYEHUNHEC"}, "014031": {"suoxie": "NFFZJYYNCYHHA", "name": "\u5357\u65b9\u53d1\u5c55\u673a\u9047\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGFAZHANJIYUYINIANCHIYOUHUNHEA"}, "014032": {"suoxie": "NFFZJYYNCYHHC", "name": "\u5357\u65b9\u53d1\u5c55\u673a\u9047\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGFAZHANJIYUYINIANCHIYOUHUNHEC"}, "014035": {"suoxie": "CCYXZLZQC", "name": "\u957f\u57ce\u60a6\u4eab\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGYUEXIANGZENGLIZHAIQUANC"}, "014036": {"suoxie": "BSCZHBHHA", "name": "\u535a\u65f6\u6210\u957f\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGHUIBAOHUNHEA"}, "014037": {"suoxie": "BSCZHBHHC", "name": "\u535a\u65f6\u6210\u957f\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGHUIBAOHUNHEC"}, "014038": {"suoxie": "JYQCHHA", "name": "\u4ea4\u94f6\u542f\u8bda\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQICHENGHUNHEA"}, "014039": {"suoxie": "JYQCHHC", "name": "\u4ea4\u94f6\u542f\u8bda\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQICHENGHUNHEC"}, "014042": {"suoxie": "YHXCLHPZHHC", "name": "\u94f6\u534e\u5fc3\u8bda\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINCHENGLINGHUOPEIZHIHUNHEC"}, "014043": {"suoxie": "YHXYLHPZHHC", "name": "\u94f6\u534e\u5fc3\u6021\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINYILINGHUOPEIZHIHUNHEC"}, "014044": {"suoxie": "YHFLJXHHC", "name": "\u94f6\u534e\u5bcc\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFULIJINGXUANHUNHEC"}, "014045": {"suoxie": "YHJJCZHHC", "name": "\u94f6\u534e\u79ef\u6781\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJIJICHENGZHANGHUNHEC"}, "014046": {"suoxie": "JYYYCXGPC", "name": "\u4ea4\u94f6\u533b\u836f\u521b\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIAOYINYIYAOCHUANGXINGUPIAOC"}, "014047": {"suoxie": "YHSSJXLHPZHHFQSC", "name": "\u94f6\u534e\u76db\u4e16\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUASHENGSHIJINGXUANLINGHUOPEIZHIHUNHEFAQISHIC"}, "014048": {"suoxie": "YHXSLHPZHHLOFC", "name": "\u94f6\u534e\u946b\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINSHENGLINGHUOPEIZHIHUNHELOFC"}, "014051": {"suoxie": "PAAYLHPZHHC", "name": "\u5e73\u5b89\u5b89\u76c8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANANYINGLINGHUOPEIZHIHUNHEC"}, "014052": {"suoxie": "YHGGTJXGPFQSC", "name": "\u94f6\u534e\u6e2f\u80a1\u901a\u7cbe\u9009\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAGANGGUTONGJINGXUANGUPIAOFAQISHIC"}, "014053": {"suoxie": "TPRQHHA", "name": "\u592a\u5e73\u777f\u5e86\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIQINGHUNHEA"}, "014054": {"suoxie": "TPRQHHC", "name": "\u592a\u5e73\u777f\u5e86\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIPINGRUIQINGHUNHEC"}, "014055": {"suoxie": "TPHXCZ", "name": "\u592a\u5e73\u6052\u5174\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGHENGXINGCHUNZHAI"}, "014056": {"suoxie": "TPFRYNDKZFQS", "name": "\u592a\u5e73\u4e30\u6da6\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGFENGRUNYINIANDINGKAIZHAIFAQISHI"}, "014057": {"suoxie": "FGJAJHJXHHA", "name": "\u5bcc\u56fd\u91d1\u5b89\u5747\u8861\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJINANJUNHENGJINGXUANHUNHEA"}, "014058": {"suoxie": "FGJAJHJXHHC", "name": "\u5bcc\u56fd\u91d1\u5b89\u5747\u8861\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOJINANJUNHENGJINGXUANHUNHEC"}, "014061": {"suoxie": "PYASXXCYHHC", "name": "\u6d66\u94f6\u5b89\u76db\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXINXINGCHANYEHUNHEC"}, "014062": {"suoxie": "JSCCZJTXLHYXGPA", "name": "\u666f\u987a\u957f\u57ce\u4e13\u7cbe\u7279\u65b0\u91cf\u5316\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGZHUANJINGTEXINLIANGHUAYOUXUANGUPIAOA"}, "014063": {"suoxie": "JSCCZJTXLHYXGPC", "name": "\u666f\u987a\u957f\u57ce\u4e13\u7cbe\u7279\u65b0\u91cf\u5316\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JINGSHUNCHANGCHENGZHUANJINGTEXINLIANGHUAYOUXUANGUPIAOC"}, "014064": {"suoxie": "YHNYCYGPFQSC", "name": "\u94f6\u534e\u519c\u4e1a\u4ea7\u4e1a\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUANONGYECHANYEGUPIAOFAQISHIC"}, "014068": {"suoxie": "GYRXYXHHA", "name": "\u5de5\u94f6\u745e\u4fe1\u60a6\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINRUIXINYUEXIANGHUNHEA"}, "014069": {"suoxie": "GYRXYXHHC", "name": "\u5de5\u94f6\u745e\u4fe1\u60a6\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINRUIXINYUEXIANGHUNHEC"}, "014074": {"suoxie": "JSNXJXHHA", "name": "\u5609\u5b9e\u5185\u9700\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHINEIXUJINGXUANHUNHEA"}, "014075": {"suoxie": "JSNXJXHHC", "name": "\u5609\u5b9e\u5185\u9700\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHINEIXUJINGXUANHUNHEC"}, "014076": {"suoxie": "HSHYSGYDKCZZQ", "name": "\u534e\u5546\u9e3f\u6e90\u4e09\u4e2a\u6708\u5b9a\u5f00\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUASHANGHONGYUANSANGEYUEDINGKAICHUNZHAIZHAIQUAN"}, "014077": {"suoxie": "HSWJHLYNCYHHA", "name": "\u534e\u5546\u7a33\u5065\u6c47\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUASHANGWENJIANHUILIYINIANCHIYOUHUNHEA"}, "014078": {"suoxie": "HSWJHLYNCYHHC", "name": "\u534e\u5546\u7a33\u5065\u6c47\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUASHANGWENJIANHUILIYINIANCHIYOUHUNHEC"}, "014079": {"suoxie": "HXHYPZYNFBGPFOFLOFC", "name": "\u534e\u590f\u884c\u4e1a\u914d\u7f6e\u4e00\u5e74\u5c01\u95ed\u80a1\u7968(FOF-LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAHANGYEPEIZHIYINIANFENGBIGUPIAOFOFLOFC"}, "014080": {"suoxie": "JYQHHHC", "name": "\u4ea4\u94f6\u542f\u6c47\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQIHUIHUNHEC"}, "014081": {"suoxie": "PAZZ13NGKZZSA", "name": "\u5e73\u5b89\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAI13NIANGUOKAIZHAIZHISHUA"}, "014082": {"suoxie": "PAZZ13NGKZZSC", "name": "\u5e73\u5b89\u4e2d\u503a1-3\u5e74\u56fd\u5f00\u503a\u6307\u6570C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAI13NIANGUOKAIZHAIZHISHUC"}, "014083": {"suoxie": "ZSHJYX30TGDCYZDZA", "name": "\u6d59\u5546\u6c47\u91d1\u6708\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINYUEXIANG30TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "014084": {"suoxie": "ZSHJYX30TGDCYZDZC", "name": "\u6d59\u5546\u6c47\u91d1\u6708\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHESHANGHUIJINYUEXIANG30TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "014085": {"suoxie": "ZSJCXSWHHC", "name": "\u6d59\u5546\u805a\u6f6e\u65b0\u601d\u7ef4\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGJUCHAOXINSIWEIHUNHEC"}, "014086": {"suoxie": "XZQQHY180TCYZQA", "name": "\u5174\u8bc1\u5168\u7403\u6052\u60a6180\u5929\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGZHENGQUANQIUHENGYUE180TIANCHIYOUZHAIQUANA"}, "014087": {"suoxie": "XZQQHY180TCYZQC", "name": "\u5174\u8bc1\u5168\u7403\u6052\u60a6180\u5929\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGZHENGQUANQIUHENGYUE180TIANCHIYOUZHAIQUANC"}, "014088": {"suoxie": "YYWJZQZQA", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGWENJIANZENGQIANGZHAIQUANA"}, "014089": {"suoxie": "YYWJZQZQC", "name": "\u6c38\u8d62\u7a33\u5065\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YONGYINGWENJIANZENGQIANGZHAIQUANC"}, "014090": {"suoxie": "ZRTYJQ3GYCYHHFQFOFA", "name": "\u4e2d\u878d\u6dfb\u76ca\u8fdb\u53d63\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGTIANYIJINQU3GEYUECHIYOUHUNHEFAQIFOFA"}, "014091": {"suoxie": "ZRTYJQ3GYCYHHFQFOFC", "name": "\u4e2d\u878d\u6dfb\u76ca\u8fdb\u53d63\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGTIANYIJINQU3GEYUECHIYOUHUNHEFAQIFOFC"}, "014092": {"suoxie": "HXYXPZYNFBYZGPC", "name": "\u534e\u590f\u4f18\u9009\u914d\u7f6e\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAYOUXUANPEIZHIYINIANFENGBIYUNZUOGUPIAOC"}, "014094": {"suoxie": "NFYYYNCYHHA", "name": "\u5357\u65b9\u8a89\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUYINGYINIANCHIYOUHUNHEA"}, "014095": {"suoxie": "NFYYYNCYHHC", "name": "\u5357\u65b9\u8a89\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGYUYINGYINIANCHIYOUHUNHEC"}, "014096": {"suoxie": "JYJJXDLHHC", "name": "\u4ea4\u94f6\u7ecf\u6d4e\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJINGJIXINDONGLIHUNHEC"}, "014097": {"suoxie": "PYLX60TGDCYZQA", "name": "\u9e4f\u626c\u5229\u946b60\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGLIXIN60TIANGUNDONGCHIYOUZHAIQUANA"}, "014098": {"suoxie": "PYLX60TGDCYZQC", "name": "\u9e4f\u626c\u5229\u946b60\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGYANGLIXIN60TIANGUNDONGCHIYOUZHAIQUANC"}, "014099": {"suoxie": "BSZZKCCY50ETFFQSLJA", "name": "\u535a\u65f6\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGKECHUANGCHUANGYE50ETFFAQISHILIANJIEA"}, "014100": {"suoxie": "BSZZKCCY50ETFFQSLJC", "name": "\u535a\u65f6\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGKECHUANGCHUANGYE50ETFFAQISHILIANJIEC"}, "014103": {"suoxie": "FADCZJZYNCYQHHA", "name": "\u5bcc\u5b89\u8fbe\u6210\u957f\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDACHENGZHANGJIAZHIYINIANCHIYOUQIHUNHEA"}, "014104": {"suoxie": "FADCZJZYNCYQHHC", "name": "\u5bcc\u5b89\u8fbe\u6210\u957f\u4ef7\u503c\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDACHENGZHANGJIAZHIYINIANCHIYOUQIHUNHEC"}, "014105": {"suoxie": "CCXLYNDKZQFQS", "name": "\u957f\u57ce\u4fe1\u5229\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGXINLIYINIANDINGKAIZHAIQUANFAQISHI"}, "014106": {"suoxie": "RTCZ30LHPZHHC", "name": "\u878d\u901a\u6210\u957f30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "RONGTONGCHENGZHANG30LINGHUOPEIZHIHUNHEC"}, "014107": {"suoxie": "BSPZSHHHA", "name": "\u535a\u65f6\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIPINZHISHENGHUOHUNHEA"}, "014108": {"suoxie": "BSPZSHHHC", "name": "\u535a\u65f6\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIPINZHISHENGHUOHUNHEC"}, "014109": {"suoxie": "RTNXQDHHC", "name": "\u878d\u901a\u5185\u9700\u9a71\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGNEIXUQUDONGHUNHEC"}, "014110": {"suoxie": "JSZZXYJSZTETFFQLJA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXIYOUJINSHUZHUTIETFFAQILIANJIEA"}, "014111": {"suoxie": "JSZZXYJSZTETFFQLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXIYOUJINSHUZHUTIETFFAQILIANJIEC"}, "014112": {"suoxie": "JSDCTLDQHHC", "name": "\u5609\u5b9e\u5bf9\u51b2\u5957\u5229\u5b9a\u671f\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "JIASHIDUICHONGTAOLIDINGQIHUNHEC"}, "014113": {"suoxie": "SYWLSHLHPZHHC", "name": "\u4e0a\u94f6\u672a\u6765\u751f\u6d3b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGYINWEILAISHENGHUOLINGHUOPEIZHIHUNHEC"}, "014114": {"suoxie": "GFHGSYYHHA", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u533b\u836f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENYIYAOHUNHEA"}, "014115": {"suoxie": "GFHGSYYHHC", "name": "\u5e7f\u53d1\u6caa\u6e2f\u6df1\u533b\u836f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHUGANGSHENYIYAOHUNHEC"}, "014116": {"suoxie": "SYHHSYZQZQC", "name": "\u4e0a\u94f6\u6167\u6052\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGYINHUIHENGSHOUYIZENGQIANGZHAIQUANC"}, "014117": {"suoxie": "GTZZHGSCXYCYETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUGANGSHENCHUANGXINYAOCHANYEETFFAQILIANJIEA"}, "014118": {"suoxie": "GTZZHGSCXYCYETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u521b\u65b0\u836f\u4ea7\u4e1aETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUGANGSHENCHUANGXINYAOCHANYEETFFAQILIANJIEC"}, "014119": {"suoxie": "JYSDXFHHA", "name": "\u91d1\u9e70\u65f6\u4ee3\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGSHIDAIXIANFENGHUNHEA"}, "014120": {"suoxie": "JYSDXFHHC", "name": "\u91d1\u9e70\u65f6\u4ee3\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGSHIDAIXIANFENGHUNHEC"}, "014125": {"suoxie": "HXZZ1000ZSZQA", "name": "\u534e\u590f\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG1000ZHISHUZENGQIANGA"}, "014126": {"suoxie": "HXZZ1000ZSZQC", "name": "\u534e\u590f\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG1000ZHISHUZENGQIANGC"}, "014127": {"suoxie": "RTHXJZHHC", "name": "\u878d\u901a\u6838\u5fc3\u4ef7\u503c\u6df7\u5408C", "style": "QDII", "pinyin": "RONGTONGHEXINJIAZHIHUNHEC"}, "014128": {"suoxie": "DCCXYA", "name": "\u4e1c\u8d22\u521b\u65b0\u836fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAICHUANGXINYAOA"}, "014129": {"suoxie": "DCCXYC", "name": "\u4e1c\u8d22\u521b\u65b0\u836fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAICHUANGXINYAOC"}, "014130": {"suoxie": "RTZZYJSYDSJZTZSLOFC", "name": "\u878d\u901a\u4e2d\u8bc1\u4e91\u8ba1\u7b97\u4e0e\u5927\u6570\u636e\u4e3b\u9898\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGZHONGZHENGYUNJISUANYUDASHUJUZHUTIZHISHULOFC"}, "014131": {"suoxie": "HTBRJYZXYNCYQHHA", "name": "\u534e\u6cf0\u67cf\u745e\u805a\u4f18\u667a\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJUYOUZHIXUANYINIANCHIYOUQIHUNHEA"}, "014132": {"suoxie": "HTBRJYZXYNCYQHHC", "name": "\u534e\u6cf0\u67cf\u745e\u805a\u4f18\u667a\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJUYOUZHIXUANYINIANCHIYOUQIHUNHEC"}, "014133": {"suoxie": "GYZZ500LGYCYZSZQA", "name": "\u5de5\u94f6\u4e2d\u8bc1500\u516d\u4e2a\u6708\u6301\u6709\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENG500LIUGEYUECHIYOUZHISHUZENGQIANGA"}, "014134": {"suoxie": "GYZZ500LGYCYZSZQC", "name": "\u5de5\u94f6\u4e2d\u8bc1500\u516d\u4e2a\u6708\u6301\u6709\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENG500LIUGEYUECHIYOUZHISHUZENGQIANGC"}, "014135": {"suoxie": "ZOJALHHHA", "name": "\u4e2d\u6b27\u91d1\u5b89\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJINANLIANGHUAHUNHEA"}, "014136": {"suoxie": "ZOJALHHHC", "name": "\u4e2d\u6b27\u91d1\u5b89\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJINANLIANGHUAHUNHEC"}, "014137": {"suoxie": "ZTARZQA", "name": "\u4e2d\u6cf0\u5b89\u777f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGTAIANRUIZHAIQUANA"}, "014138": {"suoxie": "ZTARZQC", "name": "\u4e2d\u6cf0\u5b89\u777f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGTAIANRUIZHAIQUANC"}, "014141": {"suoxie": "DCXNYHHFQSA", "name": "\u5927\u6210\u65b0\u80fd\u6e90\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGXINNENGYUANHUNHEFAQISHIA"}, "014142": {"suoxie": "DCXNYHHFQSC", "name": "\u5927\u6210\u65b0\u80fd\u6e90\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGXINNENGYUANHUNHEFAQISHIC"}, "014143": {"suoxie": "YHCXCZHHC", "name": "\u94f6\u6cb3\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHUANGXINCHENGZHANGHUNHEC"}, "014144": {"suoxie": "CXXJZBSGYCYHHA", "name": "\u957f\u4fe1\u5148\u8fdb\u88c5\u5907\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINXIANJINZHUANGBEISANGEYUECHIYOUHUNHEA"}, "014145": {"suoxie": "CXXJZBSGYCYHHC", "name": "\u957f\u4fe1\u5148\u8fdb\u88c5\u5907\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINXIANJINZHUANGBEISANGEYUECHIYOUHUNHEC"}, "014148": {"suoxie": "JSCCADYNCYQHHA", "name": "\u666f\u987a\u957f\u57ce\u5b89\u9f0e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANDINGYINIANCHIYOUQIHUNHEA"}, "014149": {"suoxie": "JSCCADYNCYQHHC", "name": "\u666f\u987a\u957f\u57ce\u5b89\u9f0e\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGANDINGYINIANCHIYOUQIHUNHEC"}, "014150": {"suoxie": "XHXYLHPZHHA", "name": "\u65b0\u534e\u946b\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINYILINGHUOPEIZHIHUNHEA"}, "014151": {"suoxie": "GFXXJZYNFBHHA", "name": "\u56fd\u5bcc\u946b\u4eab\u4ef7\u503c\u4e00\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUXINXIANGJIAZHIYINIANFENGBIHUNHEA"}, "014152": {"suoxie": "GFXXJZYNFBHHC", "name": "\u56fd\u5bcc\u946b\u4eab\u4ef7\u503c\u4e00\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUXINXIANGJIAZHIYINIANFENGBIHUNHEC"}, "014153": {"suoxie": "THHZHSGCQJZLZSA", "name": "\u5929\u5f18\u534e\u8bc1\u6caa\u6df1\u6e2f\u957f\u671f\u7ade\u4e89\u529b\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUAZHENGHUSHENGANGCHANGQIJINGZHENGLIZHISHUA"}, "014154": {"suoxie": "THHZHSGCQJZLZSC", "name": "\u5929\u5f18\u534e\u8bc1\u6caa\u6df1\u6e2f\u957f\u671f\u7ade\u4e89\u529b\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUAZHENGHUSHENGANGCHANGQIJINGZHENGLIZHISHUC"}, "014155": {"suoxie": "GTJAZZ500ZSZQA", "name": "\u56fd\u6cf0\u541b\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIJUNANZHONGZHENG500ZHISHUZENGQIANGA"}, "014156": {"suoxie": "GTJAZZ500ZSZQC", "name": "\u56fd\u6cf0\u541b\u5b89\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIJUNANZHONGZHENG500ZHISHUZENGQIANGC"}, "014157": {"suoxie": "GTJACXYYHHFQ", "name": "\u56fd\u6cf0\u541b\u5b89\u521b\u65b0\u533b\u836f\u6df7\u5408\u53d1\u8d77", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJUNANCHUANGXINYIYAOHUNHEFAQI"}, "014158": {"suoxie": "BSPHYNCYQHHA", "name": "\u535a\u65f6\u6d66\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIPUHUIYINIANCHIYOUQIHUNHEA"}, "014159": {"suoxie": "BSPHYNCYQHHC", "name": "\u535a\u65f6\u6d66\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIPUHUIYINIANCHIYOUQIHUNHEC"}, "014160": {"suoxie": "YFDYRYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u878d\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUERONGYINIANCHIYOUHUNHEA"}, "014161": {"suoxie": "YFDYRYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u878d\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUERONGYINIANCHIYOUHUNHEC"}, "014162": {"suoxie": "WJRGZNHHC", "name": "\u4e07\u5bb6\u4eba\u5de5\u667a\u80fd\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIARENGONGZHINENGHUNHEC"}, "014163": {"suoxie": "FGGGTLHJXGPC", "name": "\u5bcc\u56fd\u6e2f\u80a1\u901a\u91cf\u5316\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOGANGGUTONGLIANGHUAJINGXUANGUPIAOC"}, "014164": {"suoxie": "FGRX18GYDKHHC", "name": "\u5bcc\u56fd\u878d\u4eab18\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUORONGXIANG18GEYUEDINGKAIHUNHEC"}, "014167": {"suoxie": "YYHJXYZC", "name": "\u6c38\u8d62\u534e\u5609\u4fe1\u7528\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGHUAJIAXINYONGZHAIC"}, "014171": {"suoxie": "FGXXCZLHJXHHLOFC", "name": "\u5bcc\u56fd\u65b0\u5174\u6210\u957f\u91cf\u5316\u7cbe\u9009\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINXINGCHENGZHANGLIANGHUAJINGXUANHUNHELOFC"}, "014172": {"suoxie": "FGZZGY40ZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5de5\u4e1a4.0\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGONGYE40ZHISHULOFC"}, "014173": {"suoxie": "FGZZYDHLWZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79fb\u52a8\u4e92\u8054\u7f51\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIDONGHULIANWANGZHISHULOFC"}, "014174": {"suoxie": "FGZZGYQYGGZSLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u56fd\u6709\u4f01\u4e1a\u6539\u9769\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGUOYOUQIYEGAIGEZHISHULOFC"}, "014177": {"suoxie": "HAJQQDYNCYHHA", "name": "\u534e\u5b89\u666f\u6c14\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJINGQIQUDONGYINIANCHIYOUHUNHEA"}, "014178": {"suoxie": "HAJQQDYNCYHHC", "name": "\u534e\u5b89\u666f\u6c14\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJINGQIQUDONGYINIANCHIYOUHUNHEC"}, "014179": {"suoxie": "ZYZQYJJZHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANYUANJIANJIAZHIHUNHEA"}, "014180": {"suoxie": "ZYZQYJJZHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u8fdc\u89c1\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANYUANJIANJIAZHIHUNHEC"}, "014181": {"suoxie": "FGDPJZLHJXHHC", "name": "\u5bcc\u56fd\u5927\u76d8\u4ef7\u503c\u91cf\u5316\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODAPANJIAZHILIANGHUAJINGXUANHUNHEC"}, "014185": {"suoxie": "ZSZJTXGPA", "name": "\u62db\u5546\u4e13\u7cbe\u7279\u65b0\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGZHUANJINGTEXINGUPIAOA"}, "014186": {"suoxie": "ZSZJTXGPC", "name": "\u62db\u5546\u4e13\u7cbe\u7279\u65b0\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHAOSHANGZHUANJINGTEXINGUPIAOC"}, "014187": {"suoxie": "HXLHYXGPA", "name": "\u534e\u590f\u91cf\u5316\u4f18\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIALIANGHUAYOUXUANGUPIAOA"}, "014188": {"suoxie": "HXLHYXGPC", "name": "\u534e\u590f\u91cf\u5316\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIALIANGHUAYOUXUANGUPIAOC"}, "014189": {"suoxie": "NFZJTXHHA", "name": "\u5357\u65b9\u4e13\u7cbe\u7279\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHUANJINGTEXINHUNHEA"}, "014190": {"suoxie": "NFZJTXHHC", "name": "\u5357\u65b9\u4e13\u7cbe\u7279\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGZHUANJINGTEXINHUNHEC"}, "014191": {"suoxie": "GFXJZZGPFQSA", "name": "\u5e7f\u53d1\u5148\u8fdb\u5236\u9020\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAXIANJINZHIZAOGUPIAOFAQISHIA"}, "014192": {"suoxie": "GFXJZZGPFQSC", "name": "\u5e7f\u53d1\u5148\u8fdb\u5236\u9020\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGFAXIANJINZHIZAOGUPIAOFAQISHIC"}, "014193": {"suoxie": "HTFZZXPCYZSZQFQA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1a\u6307\u6570\u589e\u5f3a\u53d1\u8d77A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINPIANCHANYEZHISHUZENGQIANGFAQIA"}, "014194": {"suoxie": "HTFZZXPCYZSZQFQC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1a\u6307\u6570\u589e\u5f3a\u53d1\u8d77C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINPIANCHANYEZHISHUZENGQIANGFAQIC"}, "014195": {"suoxie": "TXTL30TCYZQFQSA", "name": "\u6cf0\u4fe1\u6dfb\u522930\u5929\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIXINTIANLI30TIANCHIYOUZHAIQUANFAQISHIA"}, "014196": {"suoxie": "TXTL30TCYZQFQSC", "name": "\u6cf0\u4fe1\u6dfb\u522930\u5929\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "TAIXINTIANLI30TIANCHIYOUZHAIQUANFAQISHIC"}, "014198": {"suoxie": "HXZSXFGPLOFC", "name": "\u534e\u590f\u667a\u80dc\u5148\u950b\u80a1\u7968(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAZHISHENGXIANFENGGUPIAOLOFC"}, "014199": {"suoxie": "JXWXYNCYHHA", "name": "\u5efa\u4fe1\u6c83\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINWOXINYINIANCHIYOUHUNHEA"}, "014200": {"suoxie": "JXWXYNCYHHC", "name": "\u5efa\u4fe1\u6c83\u4fe1\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINWOXINYINIANCHIYOUHUNHEC"}, "014201": {"suoxie": "THZZ1000ZSZQA", "name": "\u5929\u5f18\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG1000ZHISHUZENGQIANGA"}, "014202": {"suoxie": "THZZ1000ZSZQC", "name": "\u5929\u5f18\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG1000ZHISHUZENGQIANGC"}, "014203": {"suoxie": "PYCYQSYNCYHHA", "name": "\u9e4f\u626c\u4ea7\u4e1a\u8d8b\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGCHANYEQUSHIYINIANCHIYOUHUNHEA"}, "014204": {"suoxie": "PYCYQSYNCYHHC", "name": "\u9e4f\u626c\u4ea7\u4e1a\u8d8b\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGCHANYEQUSHIYINIANCHIYOUHUNHEC"}, "014207": {"suoxie": "HACYJXHHA", "name": "\u534e\u5b89\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHANYEJINGXUANHUNHEA"}, "014208": {"suoxie": "HACYJXHHC", "name": "\u534e\u5b89\u4ea7\u4e1a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHANYEJINGXUANHUNHEC"}, "014209": {"suoxie": "MSJYHXZQ", "name": "\u6c11\u751f\u52a0\u94f6\u6052\u7965\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINHENGXIANGZHAIQUAN"}, "014210": {"suoxie": "GTRYJZYSHHA", "name": "\u56fd\u6295\u745e\u94f6\u7ade\u4e89\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINJINGZHENGYOUSHIHUNHEA"}, "014211": {"suoxie": "GTRYJZYSHHC", "name": "\u56fd\u6295\u745e\u94f6\u7ade\u4e89\u4f18\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINJINGZHENGYOUSHIHUNHEC"}, "014212": {"suoxie": "BSYJYXHHA", "name": "\u535a\u65f6\u7814\u7a76\u4f18\u4eab\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUYOUXIANGHUNHEA"}, "014213": {"suoxie": "BSYJYXHHC", "name": "\u535a\u65f6\u7814\u7a76\u4f18\u4eab\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUYOUXIANGHUNHEC"}, "014216": {"suoxie": "JSJDSYCLDQHHC", "name": "\u5609\u5b9e\u7edd\u5bf9\u6536\u76ca\u7b56\u7565\u5b9a\u671f\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "JIASHIJUEDUISHOUYICELUEDINGQIHUNHEC"}, "014220": {"suoxie": "HYYLJKJXHHA", "name": "\u6052\u8d8a\u533b\u7597\u5065\u5eb7\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYILIAOJIANKANGJINGXUANHUNHEA"}, "014221": {"suoxie": "HYYLJKJXHHC", "name": "\u6052\u8d8a\u533b\u7597\u5065\u5eb7\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HENGYUEYILIAOJIANKANGJINGXUANHUNHEC"}, "014224": {"suoxie": "DCJYCZHHA", "name": "\u5927\u6210\u805a\u4f18\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJUYOUCHENGZHANGHUNHEA"}, "014225": {"suoxie": "DCJYCZHHC", "name": "\u5927\u6210\u805a\u4f18\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJUYOUCHENGZHANGHUNHEC"}, "014226": {"suoxie": "ZYZZ800ETFFQSLJA", "name": "\u4e2d\u94f6\u4e2d\u8bc1800ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG800ETFFAQISHILIANJIEA"}, "014227": {"suoxie": "ZYZZ800ETFFQSLJC", "name": "\u4e2d\u94f6\u4e2d\u8bc1800ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG800ETFFAQISHILIANJIEC"}, "014228": {"suoxie": "PYASPZYXHHA", "name": "\u6d66\u94f6\u5b89\u76db\u54c1\u8d28\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGPINZHIYOUXUANHUNHEA"}, "014229": {"suoxie": "PYASPZYXHHC", "name": "\u6d66\u94f6\u5b89\u76db\u54c1\u8d28\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGPINZHIYOUXUANHUNHEC"}, "014230": {"suoxie": "GTRFCZZQ", "name": "\u56fd\u6cf0\u745e\u4e30\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIRUIFENGCHUNZHAIZHAIQUAN"}, "014231": {"suoxie": "GSABAJCZYNDKZ", "name": "\u56fd\u5bff\u5b89\u4fdd\u5b89\u9526\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOSHOUANBAOANJINCHUNZHAIYINIANDINGKAIZHAI"}, "014232": {"suoxie": "BSZJTXZTHHA", "name": "\u535a\u65f6\u4e13\u7cbe\u7279\u65b0\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHUANJINGTEXINZHUTIHUNHEA"}, "014233": {"suoxie": "BSZJTXZTHHC", "name": "\u535a\u65f6\u4e13\u7cbe\u7279\u65b0\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHUANJINGTEXINZHUTIHUNHEC"}, "014234": {"suoxie": "YYXYZQ", "name": "\u6c38\u8d62\u8f69\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YONGYINGXUANYIZHAIQUAN"}, "014235": {"suoxie": "CHSDYXHHA", "name": "\u6df3\u539a\u65f6\u4ee3\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUSHIDAIYOUXUANHUNHEA"}, "014236": {"suoxie": "CHSDYXHHC", "name": "\u6df3\u539a\u65f6\u4ee3\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHUNHOUSHIDAIYOUXUANHUNHEC"}, "014237": {"suoxie": "DCXNYA", "name": "\u4e1c\u8d22\u65b0\u80fd\u6e90A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINNENGYUANA"}, "014238": {"suoxie": "DCXNYC", "name": "\u4e1c\u8d22\u65b0\u80fd\u6e90C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGCAIXINNENGYUANC"}, "014241": {"suoxie": "NYJHSYHH", "name": "\u519c\u94f6\u5747\u8861\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINJUNHENGSHOUYIHUNHE"}, "014242": {"suoxie": "NYHLYLZQ", "name": "\u519c\u94f6\u6c47\u7406\u60a6\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIYUELIZHAIQUAN"}, "014243": {"suoxie": "FGXCLXNYHHC", "name": "\u5bcc\u56fd\u65b0\u6750\u6599\u65b0\u80fd\u6e90\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINCAILIAOXINNENGYUANHUNHEC"}, "014244": {"suoxie": "PYJZLXFYNCYHHA", "name": "\u9e4f\u626c\u7ade\u4e89\u529b\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGJINGZHENGLIXIANFENGYINIANCHIYOUHUNHEA"}, "014245": {"suoxie": "PYJZLXFYNCYHHC", "name": "\u9e4f\u626c\u7ade\u4e89\u529b\u5148\u950b\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGYANGJINGZHENGLIXIANFENGYINIANCHIYOUHUNHEC"}, "014246": {"suoxie": "DMXDFWYHHA", "name": "\u5927\u6469\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOXIANDAIFUWUYEHUNHEA"}, "014247": {"suoxie": "DMXDFWYHHC", "name": "\u5927\u6469\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOXIANDAIFUWUYEHUNHEC"}, "014248": {"suoxie": "XYYNCYZQA", "name": "\u5174\u4e1a\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYINIANCHIYOUZHAIQUANA"}, "014249": {"suoxie": "XYYNCYZQC", "name": "\u5174\u4e1a\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINGYEYINIANCHIYOUZHAIQUANC"}, "014250": {"suoxie": "JXXY90TGDCYZDZZQA", "name": "\u5efa\u4fe1\u946b\u602190\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINXINYI90TIANGUNDONGCHIYOUZHONGDUANZHAIZHAIQUANA"}, "014251": {"suoxie": "JXXY90TGDCYZDZZQC", "name": "\u5efa\u4fe1\u946b\u602190\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINXINYI90TIANGUNDONGCHIYOUZHONGDUANZHAIZHAIQUANC"}, "014254": {"suoxie": "XDAYZYSNCYHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u667a\u8fdc\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHIYUANSANNIANCHIYOUHUNHEA"}, "014255": {"suoxie": "XDAYZYSNCYHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u667a\u8fdc\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHIYUANSANNIANCHIYOUHUNHEC"}, "014256": {"suoxie": "FGZZYLZTZSZQLOFC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5a31\u4e50\u4e3b\u9898\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYULEZHUTIZHISHUZENGQIANGLOFC"}, "014257": {"suoxie": "ZRHZCZA", "name": "\u4e2d\u878d\u6052\u6cfd\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGZECHUNZHAIA"}, "014258": {"suoxie": "ZRHZCZC", "name": "\u4e2d\u878d\u6052\u6cfd\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGRONGHENGZECHUNZHAIC"}, "014259": {"suoxie": "BJBXCZ", "name": "\u767e\u5609\u767e\u5174\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BAIJIABAIXINGCHUNZHAI"}, "014260": {"suoxie": "WJXJYLTQYHHC", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u9f99\u5934\u4f01\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINJIYULONGTOUQIYEHUNHEC"}, "014261": {"suoxie": "STMGWXYJYNCYQHHA", "name": "\u4e0a\u6295\u6469\u6839\u6c83\u4eab\u8fdc\u89c1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENWOXIANGYUANJIANYINIANCHIYOUQIHUNHEA"}, "014262": {"suoxie": "STMGWXYJYNCYQHHC", "name": "\u4e0a\u6295\u6469\u6839\u6c83\u4eab\u8fdc\u89c1\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENWOXIANGYUANJIANYINIANCHIYOUQIHUNHEC"}, "014265": {"suoxie": "XHDLZQE", "name": "\u65b0\u534e\u9f0e\u5229\u503a\u5238E", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUADINGLIZHAIQUANE"}, "014267": {"suoxie": "HSJZLYXHHA", "name": "\u534e\u5546\u7ade\u4e89\u529b\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJINGZHENGLIYOUXUANHUNHEA"}, "014268": {"suoxie": "HSJZLYXHHC", "name": "\u534e\u5546\u7ade\u4e89\u529b\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJINGZHENGLIYOUXUANHUNHEC"}, "014269": {"suoxie": "JSBJSJXLNDQHHA", "name": "\u5609\u5b9e\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIBEIJIAOSUOJINGXUANLIANGNIANDINGQIHUNHEA"}, "014270": {"suoxie": "JSBJSJXLNDQHHC", "name": "\u5609\u5b9e\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIBEIJIAOSUOJINGXUANLIANGNIANDINGQIHUNHEC"}, "014271": {"suoxie": "DCBJSLNDKHHA", "name": "\u5927\u6210\u5317\u4ea4\u6240\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGBEIJIAOSUOLIANGNIANDINGKAIHUNHEA"}, "014272": {"suoxie": "DCBJSLNDKHHC", "name": "\u5927\u6210\u5317\u4ea4\u6240\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGBEIJIAOSUOLIANGNIANDINGKAIHUNHEC"}, "014273": {"suoxie": "GFBJSJXLNDKHHA", "name": "\u5e7f\u53d1\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFABEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEA"}, "014274": {"suoxie": "GFBJSJXLNDKHHC", "name": "\u5e7f\u53d1\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFABEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEC"}, "014275": {"suoxie": "YFDBJSJXLNDKHHA", "name": "\u6613\u65b9\u8fbe\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDABEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEA"}, "014276": {"suoxie": "YFDBJSJXLNDKHHC", "name": "\u6613\u65b9\u8fbe\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDABEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEC"}, "014277": {"suoxie": "WJBJSHXLNDKHHA", "name": "\u4e07\u5bb6\u5317\u4ea4\u6240\u6167\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIABEIJIAOSUOHUIXUANLIANGNIANDINGKAIHUNHEA"}, "014278": {"suoxie": "WJBJSHXLNDKHHC", "name": "\u4e07\u5bb6\u5317\u4ea4\u6240\u6167\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIABEIJIAOSUOHUIXUANLIANGNIANDINGKAIHUNHEC"}, "014279": {"suoxie": "HTFBJSCXJXLNDKHHA", "name": "\u6c47\u6dfb\u5bcc\u5317\u4ea4\u6240\u521b\u65b0\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUBEIJIAOSUOCHUANGXINJINGXUANLIANGNIANDINGKAIHUNHEA"}, "014280": {"suoxie": "HTFBJSCXJXLNDKHHC", "name": "\u6c47\u6dfb\u5bcc\u5317\u4ea4\u6240\u521b\u65b0\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUBEIJIAOSUOCHUANGXINJINGXUANLIANGNIANDINGKAIHUNHEC"}, "014282": {"suoxie": "ZXBCCZDLHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u6210\u957f\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGCHENGZHANGDONGLIHUNHEC"}, "014283": {"suoxie": "HXBJSJXLNDKHHFQS", "name": "\u534e\u590f\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIABEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEFAQISHI"}, "014285": {"suoxie": "XYJKCYHHFQSA", "name": "\u946b\u5143\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANJIANKANGCHANYEHUNHEFAQISHIA"}, "014286": {"suoxie": "XYJKCYHHFQSC", "name": "\u946b\u5143\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANJIANKANGCHANYEHUNHEFAQISHIC"}, "014292": {"suoxie": "JSCYLXHHA", "name": "\u5609\u5b9e\u4ea7\u4e1a\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYELINGXIANHUNHEA"}, "014293": {"suoxie": "JSCYLXHHC", "name": "\u5609\u5b9e\u4ea7\u4e1a\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHANYELINGXIANHUNHEC"}, "014294": {"suoxie": "NFBJSJXLNDKHHFQ", "name": "\u5357\u65b9\u5317\u4ea4\u6240\u7cbe\u9009\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408\u53d1\u8d77", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGBEIJIAOSUOJINGXUANLIANGNIANDINGKAIHUNHEFAQI"}, "014295": {"suoxie": "NYHLJSYX6GYCYQHHFOFA", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u7a57\u4f18\u90096\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NONGYINHUILIJINSUIYOUXUAN6GEYUECHIYOUQIHUNHEFOFA"}, "014296": {"suoxie": "NYHLJSYX6GYCYQHHFOFC", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u7a57\u4f18\u90096\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NONGYINHUILIJINSUIYOUXUAN6GEYUECHIYOUQIHUNHEFOFC"}, "014297": {"suoxie": "STMGYYY30TGDCYFQSDZZQA", "name": "\u4e0a\u6295\u6469\u6839\u6708\u6708\u76c830\u5929\u6eda\u52a8\u6301\u6709\u53d1\u8d77\u5f0f\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANGTOUMOGENYUEYUEYING30TIANGUNDONGCHIYOUFAQISHIDUANZHAIZHAIQUANA"}, "014298": {"suoxie": "STMGYYY30TGDCYFQSDZZQC", "name": "\u4e0a\u6295\u6469\u6839\u6708\u6708\u76c830\u5929\u6eda\u52a8\u6301\u6709\u53d1\u8d77\u5f0f\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANGTOUMOGENYUEYUEYING30TIANGUNDONGCHIYOUFAQISHIDUANZHAIZHAIQUANC"}, "014299": {"suoxie": "TDHLXJZZGPA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5148\u8fdb\u5236\u9020\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIXIANJINZHIZAOGUPIAOA"}, "014300": {"suoxie": "TDHLXJZZGPC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5148\u8fdb\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLIXIANJINZHIZAOGUPIAOC"}, "014301": {"suoxie": "CJJQHY3GYCYGPFOFA", "name": "\u521b\u91d1\u666f\u6c14\u884c\u4e1a3\u4e2a\u6708\u6301\u6709\u80a1\u7968(FOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINJINGQIHANGYE3GEYUECHIYOUGUPIAOFOFA"}, "014302": {"suoxie": "CJJQHY3GYCYGPFOFC", "name": "\u521b\u91d1\u666f\u6c14\u884c\u4e1a3\u4e2a\u6708\u6301\u6709\u80a1\u7968(FOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINJINGQIHANGYE3GEYUECHIYOUGUPIAOFOFC"}, "014303": {"suoxie": "HAZZNDXNYZTETFFQSLJA", "name": "\u534e\u5b89\u4e2d\u8bc1\u5185\u5730\u65b0\u80fd\u6e90\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGNEIDIXINNENGYUANZHUTIETFFAQISHILIANJIEA"}, "014304": {"suoxie": "HAZZNDXNYZTETFFQSLJC", "name": "\u534e\u5b89\u4e2d\u8bc1\u5185\u5730\u65b0\u80fd\u6e90\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGNEIDIXINNENGYUANZHUTIETFFAQISHILIANJIEC"}, "014307": {"suoxie": "JSDYDLHHA", "name": "\u5609\u5b9e\u591a\u5143\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIDUOYUANDONGLIHUNHEA"}, "014308": {"suoxie": "JSDYDLHHC", "name": "\u5609\u5b9e\u591a\u5143\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIDUOYUANDONGLIHUNHEC"}, "014319": {"suoxie": "DBBDTCYHHFQSA", "name": "\u5fb7\u90a6\u534a\u5bfc\u4f53\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGBANDAOTICHANYEHUNHEFAQISHIA"}, "014320": {"suoxie": "DBBDTCYHHFQSC", "name": "\u5fb7\u90a6\u534a\u5bfc\u4f53\u4ea7\u4e1a\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGBANDAOTICHANYEHUNHEFAQISHIC"}, "014321": {"suoxie": "DBZQJXHHFQSA", "name": "\u5fb7\u90a6\u5468\u671f\u7cbe\u9009\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGZHOUQIJINGXUANHUNHEFAQISHIA"}, "014322": {"suoxie": "DBZQJXHHFQSC", "name": "\u5fb7\u90a6\u5468\u671f\u7cbe\u9009\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DEBANGZHOUQIJINGXUANHUNHEFAQISHIC"}, "014323": {"suoxie": "CJXX3GYCYHHFQSFOFA", "name": "\u957f\u6c5f\u946b\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGJIANGXINXUAN3GEYUECHIYOUHUNHEFAQISHIFOFA"}, "014324": {"suoxie": "CJXX3GYCYHHFQSFOFC", "name": "\u957f\u6c5f\u946b\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77\u5f0f(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGJIANGXINXUAN3GEYUECHIYOUHUNHEFAQISHIFOFC"}, "014325": {"suoxie": "GLAHXQSYNCYHHA", "name": "\u56fd\u8054\u5b89\u6838\u5fc3\u8d8b\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHEXINQUSHIYINIANCHIYOUHUNHEA"}, "014326": {"suoxie": "GLAHXQSYNCYHHC", "name": "\u56fd\u8054\u5b89\u6838\u5fc3\u8d8b\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHEXINQUSHIYINIANCHIYOUHUNHEC"}, "014327": {"suoxie": "GLXXCYHHA", "name": "\u683c\u6797\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINXINXINGCHANYEHUNHEA"}, "014328": {"suoxie": "GLXXCYHHC", "name": "\u683c\u6797\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GELINXINXINGCHANYEHUNHEC"}, "014329": {"suoxie": "ZRYSCYHHA", "name": "\u4e2d\u878d\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYOUSHICHANYEHUNHEA"}, "014330": {"suoxie": "ZRYSCYHHC", "name": "\u4e2d\u878d\u4f18\u52bf\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYOUSHICHANYEHUNHEC"}, "014331": {"suoxie": "HTBRZZXTCYETFFQSLJA", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGXITUCHANYEETFFAQISHILIANJIEA"}, "014332": {"suoxie": "HTBRZZXTCYETFFQSLJC", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGXITUCHANYEETFFAQISHILIANJIEC"}, "014333": {"suoxie": "GYYSLHHHA", "name": "\u5de5\u94f6\u4f18\u52bf\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYOUSHILINGHANGHUNHEA"}, "014334": {"suoxie": "GYYSLHHHC", "name": "\u5de5\u94f6\u4f18\u52bf\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINYOUSHILINGHANGHUNHEC"}, "014335": {"suoxie": "XCZQLDHHLOFC", "name": "\u4fe1\u8bda\u5468\u671f\u8f6e\u52a8\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGZHOUQILUNDONGHUNHELOFC"}, "014336": {"suoxie": "JYYFZQC", "name": "\u91d1\u9e70\u5143\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYINGYUANFENGZHAIQUANC"}, "014339": {"suoxie": "CJZNZZHHFQSA", "name": "\u957f\u6c5f\u667a\u80fd\u5236\u9020\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGZHINENGZHIZAOHUNHEFAQISHIA"}, "014340": {"suoxie": "CJZNZZHHFQSC", "name": "\u957f\u6c5f\u667a\u80fd\u5236\u9020\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGZHINENGZHIZAOHUNHEFAQISHIC"}, "014344": {"suoxie": "PHZZ500ZSZQA", "name": "\u9e4f\u534e\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ZHISHUZENGQIANGA"}, "014345": {"suoxie": "PHZZ500ZSZQC", "name": "\u9e4f\u534e\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ZHISHUZENGQIANGC"}, "014346": {"suoxie": "YHXFZTHHC", "name": "\u94f6\u534e\u6d88\u8d39\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXIAOFEIZHUTIHUNHEC"}, "014347": {"suoxie": "FGQSYXHHA", "name": "\u5bcc\u56fd\u8d8b\u52bf\u4f18\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOQUSHIYOUXIANHUNHEA"}, "014348": {"suoxie": "FGQSYXHHC", "name": "\u5bcc\u56fd\u8d8b\u52bf\u4f18\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOQUSHIYOUXIANHUNHEC"}, "014349": {"suoxie": "YHXRLHPZHHLOFC", "name": "\u94f6\u534e\u946b\u9510\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINRUILINGHUOPEIZHIHUNHELOFC"}, "014350": {"suoxie": "HSZYCZYNCYHHA", "name": "\u534e\u5546\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGZHUOYUECHENGZHANGYINIANCHIYOUHUNHEA"}, "014351": {"suoxie": "HSZYCZYNCYHHC", "name": "\u534e\u5546\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGZHUOYUECHENGZHANGYINIANCHIYOUHUNHEC"}, "014362": {"suoxie": "RYWJPZLNCYHHA", "name": "\u777f\u8fdc\u7a33\u8fdb\u914d\u7f6e\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RUIYUANWENJINPEIZHILIANGNIANCHIYOUHUNHEA"}, "014363": {"suoxie": "RYWJPZLNCYHHC", "name": "\u777f\u8fdc\u7a33\u8fdb\u914d\u7f6e\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "RUIYUANWENJINPEIZHILIANGNIANCHIYOUHUNHEC"}, "014364": {"suoxie": "YHHGSZZGPC", "name": "\u94f6\u534e\u6caa\u6e2f\u6df1\u589e\u957f\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "YINHUAHUGANGSHENZENGZHANGGUPIAOC"}, "014367": {"suoxie": "ZSTYCZD", "name": "\u62db\u5546\u6dfb\u60a6\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGTIANYUECHUNZHAID"}, "014372": {"suoxie": "ZSHS300ZSZQLOFC", "name": "\u6d59\u5546\u6caa\u6df1300\u6307\u6570\u589e\u5f3a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGHUSHEN300ZHISHUZENGQIANGLOFC"}, "014373": {"suoxie": "ZSQJXFHHC", "name": "\u6d59\u5546\u5168\u666f\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGQUANJINGXIAOFEIHUNHEC"}, "014376": {"suoxie": "DWXNYQCGPA", "name": "\u4e1c\u5434\u65b0\u80fd\u6e90\u6c7d\u8f66\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUXINNENGYUANQICHEGUPIAOA"}, "014377": {"suoxie": "DWXNYQCGPC", "name": "\u4e1c\u5434\u65b0\u80fd\u6e90\u6c7d\u8f66\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUXINNENGYUANQICHEGUPIAOC"}, "014378": {"suoxie": "CJHXZRZQA", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u777f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNRUIZHAIQUANA"}, "014379": {"suoxie": "CJHXZRZQC", "name": "\u521b\u91d1\u5408\u4fe1\u5c0a\u777f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHUANGJINHEXINZUNRUIZHAIQUANC"}, "014380": {"suoxie": "JXZGZZ2025GPC", "name": "\u5efa\u4fe1\u4e2d\u56fd\u5236\u90202025\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINZHONGGUOZHIZAO2025GUPIAOC"}, "014381": {"suoxie": "CCJYLHPZHHC", "name": "\u957f\u57ce\u4e45\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUYUANLINGHUOPEIZHIHUNHEC"}, "014382": {"suoxie": "BSGQGGGPC", "name": "\u535a\u65f6\u56fd\u4f01\u6539\u9769\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "BOSHIGUOQIGAIGEGUPIAOC"}, "014383": {"suoxie": "SWLXJLSGYDKZ", "name": "\u7533\u4e07\u83f1\u4fe1\u96c6\u5229\u4e09\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHENWANLINGXINJILISANGEYUEDINGKAIZHAI"}, "014389": {"suoxie": "HACYDL6GYCYHHA", "name": "\u534e\u5b89\u4ea7\u4e1a\u52a8\u529b6\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHANYEDONGLI6GEYUECHIYOUHUNHEA"}, "014390": {"suoxie": "HACYDL6GYCYHHC", "name": "\u534e\u5b89\u4ea7\u4e1a\u52a8\u529b6\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHANYEDONGLI6GEYUECHIYOUHUNHEC"}, "014392": {"suoxie": "JSZQCZZQ", "name": "\u5609\u5b9e\u81f4\u4e7e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHIQIANCHUNZHAIZHAIQUAN"}, "014393": {"suoxie": "CJHXZXHBHHA", "name": "\u521b\u91d1\u5408\u4fe1\u7504\u9009\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINZHENXUANHUIBAOHUNHEA"}, "014394": {"suoxie": "CJHXZXHBHHC", "name": "\u521b\u91d1\u5408\u4fe1\u7504\u9009\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHUANGJINHEXINZHENXUANHUIBAOHUNHEC"}, "014395": {"suoxie": "ZYZQCZLHHHA", "name": "\u4e2d\u94f6\u8bc1\u5238\u6210\u957f\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANCHENGZHANGLINGHANGHUNHEA"}, "014396": {"suoxie": "ZYZQCZLHHHC", "name": "\u4e2d\u94f6\u8bc1\u5238\u6210\u957f\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHENGQUANCHENGZHANGLINGHANGHUNHEC"}, "014397": {"suoxie": "ZYHY180TCYZQA", "name": "\u4e2d\u94f6\u6052\u60a6180\u5929\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINHENGYUE180TIANCHIYOUZHAIQUANA"}, "014398": {"suoxie": "ZYHY180TCYZQC", "name": "\u4e2d\u94f6\u6052\u60a6180\u5929\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINHENGYUE180TIANCHIYOUZHAIQUANC"}, "014399": {"suoxie": "ZYMLYNCYQZQA", "name": "\u4e2d\u94f6\u6c11\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINMINLIYINIANCHIYOUQIZHAIQUANA"}, "014400": {"suoxie": "ZYMLYNCYQZQC", "name": "\u4e2d\u94f6\u6c11\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINMINLIYINIANCHIYOUQIZHAIQUANC"}, "014404": {"suoxie": "ZODYJZSNCYHHA", "name": "\u4e2d\u6b27\u591a\u5143\u4ef7\u503c\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUDUOYUANJIAZHISANNIANCHIYOUHUNHEA"}, "014405": {"suoxie": "ZODYJZSNCYHHC", "name": "\u4e2d\u6b27\u591a\u5143\u4ef7\u503c\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUDUOYUANJIAZHISANNIANCHIYOUHUNHEC"}, "014406": {"suoxie": "FGZZXHSMZPPGCETFLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65b0\u534e\u793e\u6c11\u65cf\u54c1\u724c\u5de5\u7a0bETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINHUASHEMINZUPINPAIGONGCHENGETFLIANJIEA"}, "014407": {"suoxie": "FGZZXHSMZPPGCETFLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65b0\u534e\u793e\u6c11\u65cf\u54c1\u724c\u5de5\u7a0bETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINHUASHEMINZUPINPAIGONGCHENGETFLIANJIEC"}, "014416": {"suoxie": "TKYJJXGPFQA", "name": "\u6cf0\u5eb7\u7814\u7a76\u7cbe\u9009\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "TAIKANGYANJIUJINGXUANGUPIAOFAQIA"}, "014417": {"suoxie": "TKYJJXGPFQC", "name": "\u6cf0\u5eb7\u7814\u7a76\u7cbe\u9009\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "TAIKANGYANJIUJINGXUANGUPIAOFAQIC"}, "014418": {"suoxie": "XBLDCESXPZSZQA", "name": "\u897f\u90e8\u5229\u5f97CES\u82af\u7247\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDECESXINPIANZHISHUZENGQIANGA"}, "014419": {"suoxie": "XBLDCESXPZSZQC", "name": "\u897f\u90e8\u5229\u5f97CES\u82af\u7247\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDECESXINPIANZHISHUZENGQIANGC"}, "014420": {"suoxie": "ZOCZLHYNCYHHA", "name": "\u4e2d\u6b27\u6210\u957f\u9886\u822a\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHENGZHANGLINGHANGYINIANCHIYOUHUNHEA"}, "014421": {"suoxie": "ZOCZLHYNCYHHC", "name": "\u4e2d\u6b27\u6210\u957f\u9886\u822a\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHENGZHANGLINGHANGYINIANCHIYOUHUNHEC"}, "014422": {"suoxie": "HYYFXFSJHHC", "name": "\u5f18\u6bc5\u8fdc\u65b9\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGYIYUANFANGXIAOFEISHENGJIHUNHEC"}, "014423": {"suoxie": "HFJXYJJXHH", "name": "\u6c47\u4e30\u664b\u4fe1\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINYANJIUJINGXUANHUNHE"}, "014424": {"suoxie": "BSHSYLBJETFFQSLJQDIIA", "name": "\u535a\u65f6\u6052\u751f\u533b\u7597\u4fdd\u5065ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)A", "style": "QDII", "pinyin": "BOSHIHENGSHENGYILIAOBAOJIANETFFAQISHILIANJIEQDIIA"}, "014425": {"suoxie": "BSHSYLBJETFFQSLJQDIIC", "name": "\u535a\u65f6\u6052\u751f\u533b\u7597\u4fdd\u5065ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)C", "style": "QDII", "pinyin": "BOSHIHENGSHENGYILIAOBAOJIANETFFAQISHILIANJIEQDIIC"}, "014426": {"suoxie": "HSZZTYCDAAAZS7TCY", "name": "\u60e0\u5347\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUISHENGZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014427": {"suoxie": "FGZZTYCDAAAZS7TCY", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014428": {"suoxie": "ZHZZTYCDAAAZS7TCY", "name": "\u4e2d\u822a\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGHANGZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014429": {"suoxie": "HFZZTYCDAAAZS7TCY", "name": "\u534e\u5bcc\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAFUZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014430": {"suoxie": "NFZZTYCDAAAZS7TCY", "name": "\u5357\u65b9\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "NANFANGZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014431": {"suoxie": "HXZZXCLZTETFFQSLJA", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINCAILIAOZHUTIETFFAQISHILIANJIEA"}, "014432": {"suoxie": "HXZZXCLZTETFFQSLJC", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINCAILIAOZHUTIETFFAQISHILIANJIEC"}, "014435": {"suoxie": "ZHRXYNDKZFQA", "name": "\u4e2d\u822a\u745e\u590f\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIXIAYINIANDINGKAIZHAIFAQIA"}, "014436": {"suoxie": "ZHRXYNDKZFQC", "name": "\u4e2d\u822a\u745e\u590f\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIXIAYINIANDINGKAIZHAIFAQIC"}, "014437": {"suoxie": "PHZZTYCDAAAZS7TCY", "name": "\u9e4f\u534e\u4e2d\u8bc1\u540c\u4e1a\u5b58\u5355AAA\u6307\u65707\u5929\u6301\u6709", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAZHONGZHENGTONGYECUNDANAAAZHISHU7TIANCHIYOU"}, "014438": {"suoxie": "BSHSKJETFFQSLJQDIIA", "name": "\u535a\u65f6\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)A", "style": "QDII", "pinyin": "BOSHIHENGSHENGKEJIETFFAQISHILIANJIEQDIIA"}, "014439": {"suoxie": "BSHSKJETFFQSLJQDIIC", "name": "\u535a\u65f6\u6052\u751f\u79d1\u6280ETF\u53d1\u8d77\u5f0f\u8054\u63a5(QDII)C", "style": "QDII", "pinyin": "BOSHIHENGSHENGKEJIETFFAQISHILIANJIEQDIIC"}, "014440": {"suoxie": "BSHXWJYNCYHHA", "name": "\u535a\u65f6\u6052\u946b\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXINWENJIANYINIANCHIYOUHUNHEA"}, "014441": {"suoxie": "BSHXWJYNCYHHC", "name": "\u535a\u65f6\u6052\u946b\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGXINWENJIANYINIANCHIYOUHUNHEC"}, "014442": {"suoxie": "ZJZZ15NZJZZS", "name": "\u4e2d\u52a0\u4e2d\u503a1-5\u5e74\u653f\u91d1\u503a\u6307\u6570", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJIAZHONGZHAI15NIANZHENGJINZHAIZHISHU"}, "014446": {"suoxie": "PHWRZDZ", "name": "\u9e4f\u534e\u7a33\u745e\u4e2d\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "PENGHUAWENRUIZHONGDUANZHAI"}, "014447": {"suoxie": "DCHYYNDKZFQS", "name": "\u5927\u6210\u60e0\u6e90\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIYUANYINIANDINGKAIZHAIFAQISHI"}, "014448": {"suoxie": "AXYNYNDKZFQS", "name": "\u5b89\u4fe1\u6c38\u5b81\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINYONGNINGYINIANDINGKAIZHAIFAQISHI"}, "014451": {"suoxie": "THXXYNDKZQFQ", "name": "\u5929\u5f18\u65b0\u4eab\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGXINXIANGYINIANDINGKAIZHAIQUANFAQI"}, "014453": {"suoxie": "ZYXDLGPC", "name": "\u4e2d\u94f6\u65b0\u52a8\u529b\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINXINDONGLIGUPIAOC"}, "014454": {"suoxie": "ZYSXHBHHC", "name": "\u4e2d\u94f6\u53cc\u606f\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINSHUANGXIHUIBAOHUNHEC"}, "014455": {"suoxie": "ZYCZYXGPC", "name": "\u4e2d\u94f6\u6210\u957f\u4f18\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGYINCHENGZHANGYOUXUANGUPIAOC"}, "014456": {"suoxie": "ZSWHZDZ60TCYZQA", "name": "\u62db\u5546\u7a33\u6052\u4e2d\u77ed\u503a60\u5929\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENHENGZHONGDUANZHAI60TIANCHIYOUZHAIQUANA"}, "014457": {"suoxie": "ZSWHZDZ60TCYZQC", "name": "\u62db\u5546\u7a33\u6052\u4e2d\u77ed\u503a60\u5929\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGWENHENGZHONGDUANZHAI60TIANCHIYOUZHAIQUANC"}, "014458": {"suoxie": "NF02NGKZE", "name": "\u5357\u65b90-2\u5e74\u56fd\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG02NIANGUOKAIZHAIE"}, "014459": {"suoxie": "NF15NGKZE", "name": "\u5357\u65b91-5\u5e74\u56fd\u5f00\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANG15NIANGUOKAIZHAIE"}, "014460": {"suoxie": "PAPZYXHHA", "name": "\u5e73\u5b89\u54c1\u8d28\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANPINZHIYOUXUANHUNHEA"}, "014461": {"suoxie": "PAPZYXHHC", "name": "\u5e73\u5b89\u54c1\u8d28\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANPINZHIYOUXUANHUNHEC"}, "014465": {"suoxie": "CSAXZDZD", "name": "\u957f\u76db\u5b89\u946b\u4e2d\u77ed\u503aD", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "CHANGSHENGANXINZHONGDUANZHAID"}, "014470": {"suoxie": "FADJKRSHHC", "name": "\u5bcc\u5b89\u8fbe\u5065\u5eb7\u4eba\u751f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAJIANKANGRENSHENGHUNHEC"}, "014471": {"suoxie": "FADXXCZHHC", "name": "\u5bcc\u5b89\u8fbe\u65b0\u5174\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDAXINXINGCHENGZHANGHUNHEC"}, "014472": {"suoxie": "JSCCYJCZHHA", "name": "\u666f\u987a\u957f\u57ce\u8fdc\u89c1\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYUANJIANCHENGZHANGHUNHEA"}, "014473": {"suoxie": "JSCCYJCZHHC", "name": "\u666f\u987a\u957f\u57ce\u8fdc\u89c1\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYUANJIANCHENGZHANGHUNHEC"}, "014476": {"suoxie": "SXZQ90TGDCYDZA", "name": "\u5c71\u897f\u8bc1\u523890\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANXIZHENGQUAN90TIANGUNDONGCHIYOUDUANZHAIA"}, "014477": {"suoxie": "SXZQ90TGDCYDZC", "name": "\u5c71\u897f\u8bc1\u523890\u5929\u6eda\u52a8\u6301\u6709\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHANXIZHENGQUAN90TIANGUNDONGCHIYOUDUANZHAIC"}, "014480": {"suoxie": "HXDYZQA", "name": "\u534e\u590f\u9f0e\u4f18\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYOUZHAIQUANA"}, "014481": {"suoxie": "HXDYZQC", "name": "\u534e\u590f\u9f0e\u4f18\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXIADINGYOUZHAIQUANC"}, "014484": {"suoxie": "HTFZZ13NYHPJAAZSFQSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u9690\u542b\u8bc4\u7ea7AA+\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANYINHANPINGJIAAZHISHUFAQISHIA"}, "014485": {"suoxie": "HTFZZ13NYHPJAAZSFQSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u503a1-3\u5e74\u9690\u542b\u8bc4\u7ea7AA+\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUZHONGZHAI13NIANYINHANPINGJIAAZHISHUFAQISHIC"}, "014488": {"suoxie": "GTRYCYSJLNCYHHA", "name": "\u56fd\u6295\u745e\u94f6\u4ea7\u4e1a\u5347\u7ea7\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHANYESHENGJILIANGNIANCHIYOUHUNHEA"}, "014489": {"suoxie": "GTRYCYSJLNCYHHC", "name": "\u56fd\u6295\u745e\u94f6\u4ea7\u4e1a\u5347\u7ea7\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHANYESHENGJILIANGNIANCHIYOUHUNHEC"}, "014492": {"suoxie": "ZSHJXLZQZQA", "name": "\u6d59\u5546\u6c47\u91d1\u5174\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGHUIJINXINGLIZENGQIANGZHAIQUANA"}, "014493": {"suoxie": "ZSHJXLZQZQC", "name": "\u6d59\u5546\u6c47\u91d1\u5174\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHESHANGHUIJINXINGLIZENGQIANGZHAIQUANC"}, "014494": {"suoxie": "WJXFCZE", "name": "\u4e07\u5bb6\u946b\u4e30\u7eaf\u503aE", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINFENGCHUNZHAIE"}, "014497": {"suoxie": "NAYJYXHHC", "name": "\u8bfa\u5b89\u7814\u7a76\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANYANJIUYOUXUANHUNHEC"}, "014498": {"suoxie": "NAHXHHC", "name": "\u8bfa\u5b89\u9e3f\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANHONGXINHUNHEC"}, "014499": {"suoxie": "NFZXZZHHC", "name": "\u5357\u65b9\u8f6c\u578b\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGZHUANXINGZENGZHANGHUNHEC"}, "014500": {"suoxie": "HCWLWJTYZQA", "name": "\u534e\u5bb8\u672a\u6765\u7a33\u5065\u6dfb\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUACHENWEILAIWENJIANTIANYINGZHAIQUANA"}, "014501": {"suoxie": "HCWLWJTYZQC", "name": "\u534e\u5bb8\u672a\u6765\u7a33\u5065\u6dfb\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUACHENWEILAIWENJIANTIANYINGZHAIQUANC"}, "014504": {"suoxie": "PYCKZQD", "name": "\u9e4f\u626c\u6df3\u5f00\u503a\u5238D", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGYANGCHUNKAIZHAIQUAND"}, "014505": {"suoxie": "ZYSYHHC", "name": "\u4e2d\u94f6\u6536\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINSHOUYIHUNHEC"}, "014506": {"suoxie": "BSCZZXHHA", "name": "\u535a\u65f6\u6210\u957f\u81fb\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGZHENXUANHUNHEA"}, "014507": {"suoxie": "BSCZZXHHC", "name": "\u535a\u65f6\u6210\u957f\u81fb\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHENGZHANGZHENXUANHUNHEC"}, "014508": {"suoxie": "HTFXJZZHHA", "name": "\u6c47\u6dfb\u5bcc\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUXIANJINZHIZAOHUNHEA"}, "014509": {"suoxie": "HTFXJZZHHC", "name": "\u6c47\u6dfb\u5bcc\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUXIANJINZHIZAOHUNHEC"}, "014511": {"suoxie": "YDAY30TGDCYZQFQSA", "name": "\u82f1\u5927\u5b89\u76c830\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINGDAANYING30TIANGUNDONGCHIYOUZHAIQUANFAQISHIA"}, "014512": {"suoxie": "YDAY30TGDCYZQFQSC", "name": "\u82f1\u5927\u5b89\u76c830\u5929\u6eda\u52a8\u6301\u6709\u503a\u5238\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINGDAANYING30TIANGUNDONGCHIYOUZHAIQUANFAQISHIC"}, "014517": {"suoxie": "HX30TGDDZFQSA", "name": "\u534e\u590f30\u5929\u6eda\u52a8\u77ed\u503a\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIA30TIANGUNDONGDUANZHAIFAQISHIA"}, "014518": {"suoxie": "HX30TGDDZFQSC", "name": "\u534e\u590f30\u5929\u6eda\u52a8\u77ed\u503a\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAXIA30TIANGUNDONGDUANZHAIFAQISHIC"}, "014519": {"suoxie": "BSHSGGXLETFFQSLJA", "name": "\u535a\u65f6\u6052\u751f\u9ad8\u80a1\u606f\u7387ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHENGSHENGGAOGUXILVETFFAQISHILIANJIEA"}, "014520": {"suoxie": "BSHSGGXLETFFQSLJC", "name": "\u535a\u65f6\u6052\u751f\u9ad8\u80a1\u606f\u7387ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHENGSHENGGAOGUXILVETFFAQISHILIANJIEC"}, "014521": {"suoxie": "NALXLHPZHHC", "name": "\u8bfa\u5b89\u5229\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANLIXINLINGHUOPEIZHIHUNHEC"}, "014522": {"suoxie": "HTFDTTZYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u4f4e\u78b3\u6295\u8d44\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUDITANTOUZIYINIANCHIYOUHUNHEA"}, "014523": {"suoxie": "HTFDTTZYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u4f4e\u78b3\u6295\u8d44\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUDITANTOUZIYINIANCHIYOUHUNHEC"}, "014526": {"suoxie": "HTFZPQLZZYNCYHHA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u6f5c\u529b\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANQIANLIZENGZHANGYINIANCHIYOUHUNHEA"}, "014527": {"suoxie": "HTFZPQLZZYNCYHHC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u76d8\u6f5c\u529b\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUZHONGPANQIANLIZENGZHANGYINIANCHIYOUHUNHEC"}, "014528": {"suoxie": "HTFMSCIZGA50HLHTETFLJA", "name": "\u6c47\u6dfb\u5bccMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEA"}, "014529": {"suoxie": "HTFMSCIZGA50HLHTETFLJC", "name": "\u6c47\u6dfb\u5bccMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEC"}, "014530": {"suoxie": "HXMSCIZGA50HLHTETFFQSLJA", "name": "\u534e\u590fMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAMSCIZHONGGUOA50HULIANHUTONGETFFAQISHILIANJIEA"}, "014531": {"suoxie": "HXMSCIZGA50HLHTETFFQSLJC", "name": "\u534e\u590fMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAMSCIZHONGGUOA50HULIANHUTONGETFFAQISHILIANJIEC"}, "014532": {"suoxie": "YFDMSCIZGA50HLHTETFLJA", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEA"}, "014533": {"suoxie": "YFDMSCIZGA50HLHTETFLJC", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEC"}, "014534": {"suoxie": "NFMSCIZGA50HLHTETFLJA", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEA"}, "014535": {"suoxie": "NFMSCIZGA50HLHTETFLJC", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOA50HULIANHUTONGETFLIANJIEC"}, "014536": {"suoxie": "NAGDZZGPC", "name": "\u8bfa\u5b89\u9ad8\u7aef\u5236\u9020\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANGAODUANZHIZAOGUPIAOC"}, "014537": {"suoxie": "ZYZGHHLOFC", "name": "\u4e2d\u94f6\u4e2d\u56fd\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHONGGUOHUNHELOFC"}, "014538": {"suoxie": "CCYJJXHHC", "name": "\u957f\u57ce\u7814\u7a76\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYANJIUJINGXUANHUNHEC"}, "014539": {"suoxie": "HAYSJXHHA", "name": "\u534e\u5b89\u4f18\u52bf\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHIJINGXUANHUNHEA"}, "014540": {"suoxie": "HAYSJXHHC", "name": "\u534e\u5b89\u4f18\u52bf\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANYOUSHIJINGXUANHUNHEC"}, "014543": {"suoxie": "TFZZHGSYJSCYA", "name": "\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e91\u8ba1\u7b97\u4ea7\u4e1aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUZHONGZHENGHUGANGSHENYUNJISUANCHANYEA"}, "014544": {"suoxie": "TFZZHGSYJSCYC", "name": "\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e91\u8ba1\u7b97\u4ea7\u4e1aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUZHONGZHENGHUGANGSHENYUNJISUANCHANYEC"}, "014549": {"suoxie": "JYSJCYLHPZHHC", "name": "\u4ea4\u94f6\u6570\u636e\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINSHUJUCHANYELINGHUOPEIZHIHUNHEC"}, "014550": {"suoxie": "NAYXLHPZHHC", "name": "\u8bfa\u5b89\u76ca\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANYIXINLINGHUOPEIZHIHUNHEC"}, "014551": {"suoxie": "NAXDLLHPZHHC", "name": "\u8bfa\u5b89\u65b0\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANXINDONGLILINGHUOPEIZHIHUNHEC"}, "014552": {"suoxie": "ZHRHESGYNDKZFQA", "name": "\u4e2d\u822a\u745e\u534eESG\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIHUAESGYINIANDINGKAIZHAIFAQIA"}, "014553": {"suoxie": "ZHRHESGYNDKZFQC", "name": "\u4e2d\u822a\u745e\u534eESG\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHANGRUIHUAESGYINIANDINGKAIZHAIFAQIC"}, "014554": {"suoxie": "BSHRYNFBHHA", "name": "\u535a\u65f6\u6052\u745e\u4e00\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRUIYINIANFENGBIHUNHEA"}, "014555": {"suoxie": "BSHRYNFBHHC", "name": "\u535a\u65f6\u6052\u745e\u4e00\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHENGRUIYINIANFENGBIHUNHEC"}, "014558": {"suoxie": "HSPZHXHHA", "name": "\u534e\u5546\u54c1\u8d28\u6167\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGPINZHIHUIXUANHUNHEA"}, "014559": {"suoxie": "HSPZHXHHC", "name": "\u534e\u5546\u54c1\u8d28\u6167\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGPINZHIHUIXUANHUNHEC"}, "014560": {"suoxie": "DFQCCYQSHHA", "name": "\u4e1c\u65b9\u6c7d\u8f66\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGQICHECHANYEQUSHIHUNHEA"}, "014561": {"suoxie": "DFQCCYQSHHC", "name": "\u4e1c\u65b9\u6c7d\u8f66\u4ea7\u4e1a\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGQICHECHANYEQUSHIHUNHEC"}, "014566": {"suoxie": "GTJASRWJYNCYHHFOFA", "name": "\u56fd\u6cf0\u541b\u5b89\u5584\u878d\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIJUNANSHANRONGWENJIANYINIANCHIYOUHUNHEFOFA"}, "014567": {"suoxie": "GTJASRWJYNCYHHFOFC", "name": "\u56fd\u6cf0\u541b\u5b89\u5584\u878d\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTAIJUNANSHANRONGWENJIANYINIANCHIYOUHUNHEFOFC"}, "014568": {"suoxie": "HXJSYXYNCYHHFOFA", "name": "\u534e\u590f\u805a\u76db\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAJUSHENGYOUXUANYINIANCHIYOUHUNHEFOFA"}, "014569": {"suoxie": "HXJSYXYNCYHHFOFC", "name": "\u534e\u590f\u805a\u76db\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAJUSHENGYOUXUANYINIANCHIYOUHUNHEFOFC"}, "014570": {"suoxie": "DWDTCZZQC", "name": "\u4e1c\u5434\u9f0e\u6cf0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGWUDINGTAICHUNZHAIZHAIQUANC"}, "014571": {"suoxie": "DWAXLHHHC", "name": "\u4e1c\u5434\u5b89\u4eab\u91cf\u5316\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUANXIANGLIANGHUAHUNHEC"}, "014572": {"suoxie": "CXYLJXHHC", "name": "\u957f\u4fe1\u94f6\u5229\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINYINLIJINGXUANHUNHEC"}, "014573": {"suoxie": "DFHJHZXLNCYHH", "name": "\u4e1c\u65b9\u7ea2\u9526\u5f18\u7504\u9009\u4e24\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINHONGZHENXUANLIANGNIANCHIYOUHUNHE"}, "014574": {"suoxie": "XYQJNYHHFQSA", "name": "\u946b\u5143\u6e05\u6d01\u80fd\u6e90\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANQINGJIENENGYUANHUNHEFAQISHIA"}, "014575": {"suoxie": "XYQJNYHHFQSC", "name": "\u946b\u5143\u6e05\u6d01\u80fd\u6e90\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINYUANQINGJIENENGYUANHUNHEFAQISHIC"}, "014581": {"suoxie": "DWAEFLHPZHHC", "name": "\u4e1c\u5434\u963f\u5c14\u6cd5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUAERFALINGHUOPEIZHIHUNHEC"}, "014582": {"suoxie": "PYASXRWJYNCYHHFOFA", "name": "\u6d66\u94f6\u5b89\u76db\u5174\u8363\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINGRONGWENJIANYINIANCHIYOUHUNHEFOFA"}, "014583": {"suoxie": "PYASXRWJYNCYHHFOFC", "name": "\u6d66\u94f6\u5b89\u76db\u5174\u8363\u7a33\u5065\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGXINGRONGWENJIANYINIANCHIYOUHUNHEFOFC"}, "014584": {"suoxie": "SWLXJZYLHHC", "name": "\u7533\u4e07\u83f1\u4fe1\u4ef7\u503c\u4f18\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINJIAZHIYOULIHUNHEC"}, "014585": {"suoxie": "YHXXSNCYHHA", "name": "\u94f6\u534e\u5fc3\u5174\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINXINGSANNIANCHIYOUHUNHEA"}, "014586": {"suoxie": "YHXXSNCYHHC", "name": "\u94f6\u534e\u5fc3\u5174\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINXINGSANNIANCHIYOUHUNHEC"}, "014591": {"suoxie": "GFRYYNCYQHHA", "name": "\u5e7f\u53d1\u745e\u8a89\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYUYINIANCHIYOUQIHUNHEA"}, "014592": {"suoxie": "GFRYYNCYQHHC", "name": "\u5e7f\u53d1\u745e\u8a89\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYUYINIANCHIYOUQIHUNHEC"}, "014594": {"suoxie": "HTFWF60TGDCYZDZA", "name": "\u6c47\u6dfb\u5bcc\u7a33\u798f60\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENFU60TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "014595": {"suoxie": "HTFWF60TGDCYZDZC", "name": "\u6c47\u6dfb\u5bcc\u7a33\u798f60\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENFU60TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "014596": {"suoxie": "HTFWF60TGDCYZDZE", "name": "\u6c47\u6dfb\u5bcc\u7a33\u798f60\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aE", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFUWENFU60TIANGUNDONGCHIYOUZHONGDUANZHAIE"}, "014597": {"suoxie": "HTBRFLHHC", "name": "\u534e\u6cf0\u67cf\u745e\u5bcc\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIBAIRUIFULIHUNHEC"}, "014598": {"suoxie": "YYHXHHFQA", "name": "\u6c38\u8d62\u5408\u4eab\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGHEXIANGHUNHEFAQIA"}, "014599": {"suoxie": "YYHXHHFQC", "name": "\u6c38\u8d62\u5408\u4eab\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YONGYINGHEXIANGHUNHEFAQIC"}, "014600": {"suoxie": "BSHBYXHHA", "name": "\u535a\u65f6\u56de\u62a5\u4e25\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUIBAOYANXUANHUNHEA"}, "014601": {"suoxie": "BSHBYXHHC", "name": "\u535a\u65f6\u56de\u62a5\u4e25\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHUIBAOYANXUANHUNHEC"}, "014602": {"suoxie": "JSZZYLZSFQSA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u533b\u7597\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGYILIAOZHISHUFAQISHIA"}, "014603": {"suoxie": "JSZZYLZSFQSC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u533b\u7597\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGYILIAOZHISHUFAQISHIC"}, "014604": {"suoxie": "JSZZGFCYZSFQSA", "name": "\u5609\u5b9e\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIA"}, "014605": {"suoxie": "JSZZGFCYZSFQSC", "name": "\u5609\u5b9e\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1a\u6307\u6570\u53d1\u8d77\u5f0fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGGUANGFUCHANYEZHISHUFAQISHIC"}, "014606": {"suoxie": "ZSGDZBHHA", "name": "\u62db\u5546\u9ad8\u7aef\u88c5\u5907\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGGAODUANZHUANGBEIHUNHEA"}, "014607": {"suoxie": "ZSGDZBHHC", "name": "\u62db\u5546\u9ad8\u7aef\u88c5\u5907\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGGAODUANZHUANGBEIHUNHEC"}, "014608": {"suoxie": "ZOZQJQHHFQA", "name": "\u4e2d\u6b27\u5468\u671f\u666f\u6c14\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHOUQIJINGQIHUNHEFAQIA"}, "014609": {"suoxie": "ZOZQJQHHFQC", "name": "\u4e2d\u6b27\u5468\u671f\u666f\u6c14\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHOUQIJINGQIHUNHEFAQIC"}, "014610": {"suoxie": "PHXXBHBC", "name": "\u9e4f\u534e\u5174\u946b\u5b9d\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAXINGXINBAOHUOBIC"}, "014611": {"suoxie": "FGHXKJ12GYCYHHA", "name": "\u5bcc\u56fd\u6838\u5fc3\u79d1\u628012\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHEXINKEJI12GEYUECHIYOUHUNHEA"}, "014612": {"suoxie": "FGHXKJ12GYCYHHC", "name": "\u5bcc\u56fd\u6838\u5fc3\u79d1\u628012\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOHEXINKEJI12GEYUECHIYOUHUNHEC"}, "014615": {"suoxie": "SZZXHHFQA", "name": "\u5c1a\u6b63\u6b63\u946b\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGZHENGZHENGXINHUNHEFAQIA"}, "014616": {"suoxie": "SZZXHHFQC", "name": "\u5c1a\u6b63\u6b63\u946b\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "SHANGZHENGZHENGXINHUNHEFAQIC"}, "014617": {"suoxie": "YFDRYAHYNCYHHFOFA", "name": "\u6613\u65b9\u8fbe\u5982\u610f\u5b89\u548c\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDARUYIANHEYINIANCHIYOUHUNHEFOFA"}, "014618": {"suoxie": "YFDRYAHYNCYHHFOFC", "name": "\u6613\u65b9\u8fbe\u5982\u610f\u5b89\u548c\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDARUYIANHEYINIANCHIYOUHUNHEFOFC"}, "014621": {"suoxie": "AXCYYNCYHHA", "name": "\u5b89\u4fe1\u695a\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINCHUYINGYINIANCHIYOUHUNHEA"}, "014622": {"suoxie": "AXCYYNCYHHC", "name": "\u5b89\u4fe1\u695a\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ANXINCHUYINGYINIANCHIYOUHUNHEC"}, "014627": {"suoxie": "CTDCLFRHHFQSLOFC", "name": "\u8d22\u901a\u591a\u7b56\u7565\u798f\u745e\u6df7\u5408\u53d1\u8d77\u5f0f(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGDUOCELUEFURUIHUNHEFAQISHILOFC"}, "014628": {"suoxie": "CTFSHHFQLOFC", "name": "\u8d22\u901a\u798f\u76db\u6df7\u5408\u53d1\u8d77(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGFUSHENGHUNHEFAQILOFC"}, "014629": {"suoxie": "HTZJZNLHGPFQC", "name": "\u534e\u6cf0\u7d2b\u91d1\u667a\u80fd\u91cf\u5316\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "HUATAIZIJINZHINENGLIANGHUAGUPIAOFAQIC"}, "014630": {"suoxie": "HTFZZRGZNZTETFLJA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEA"}, "014631": {"suoxie": "HTFZZRGZNZTETFLJC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGRENGONGZHINENGZHUTIETFLIANJIEC"}, "014638": {"suoxie": "YHAXDZZQD", "name": "\u94f6\u534e\u5b89\u946b\u77ed\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANXINDUANZHAIZHAIQUAND"}, "014639": {"suoxie": "XZQQHHSNCYHHA", "name": "\u5174\u8bc1\u5168\u7403\u5408\u8861\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGQUANQIUHEHENGSANNIANCHIYOUHUNHEA"}, "014640": {"suoxie": "XZQQHHSNCYHHC", "name": "\u5174\u8bc1\u5168\u7403\u5408\u8861\u4e09\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGQUANQIUHEHENGSANNIANCHIYOUHUNHEC"}, "014641": {"suoxie": "STMGHYLDHHC", "name": "\u4e0a\u6295\u6469\u6839\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENHANGYELUNDONGHUNHEC"}, "014642": {"suoxie": "STMGXXDLHHC", "name": "\u4e0a\u6295\u6469\u6839\u65b0\u5174\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENXINXINGDONGLIHUNHEC"}, "014643": {"suoxie": "PYASSRCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u745e\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGRUICHUNZHAIZHAIQUANA"}, "014644": {"suoxie": "PYASSRCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u745e\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGRUICHUNZHAIZHAIQUANC"}, "014645": {"suoxie": "PAYXJHPZ1NCYHHFOFA", "name": "\u5e73\u5b89\u76c8\u79a7\u5747\u8861\u914d\u7f6e1\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PINGANYINGXIJUNHENGPEIZHI1NIANCHIYOUHUNHEFOFA"}, "014646": {"suoxie": "PAYXJHPZ1NCYHHFOFC", "name": "\u5e73\u5b89\u76c8\u79a7\u5747\u8861\u914d\u7f6e1\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "PINGANYINGXIJUNHENGPEIZHI1NIANCHIYOUHUNHEFOFC"}, "014647": {"suoxie": "RTXJZZHHA", "name": "\u878d\u901a\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGXIANJINZHIZAOHUNHEA"}, "014648": {"suoxie": "RTXJZZHHC", "name": "\u878d\u901a\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGXIANJINZHIZAOHUNHEC"}, "014649": {"suoxie": "YYYZJXHHFQA", "name": "\u6c38\u8d62\u4f18\u8d28\u7cbe\u9009\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGYOUZHIJINGXUANHUNHEFAQIA"}, "014650": {"suoxie": "YYYZJXHHFQC", "name": "\u6c38\u8d62\u4f18\u8d28\u7cbe\u9009\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YONGYINGYOUZHIJINGXUANHUNHEFAQIC"}, "014651": {"suoxie": "DCZJTXHHA", "name": "\u5927\u6210\u4e13\u7cbe\u7279\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHUANJINGTEXINHUNHEA"}, "014652": {"suoxie": "DCZJTXHHC", "name": "\u5927\u6210\u4e13\u7cbe\u7279\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHUANJINGTEXINHUNHEC"}, "014653": {"suoxie": "JXZYCZYNCYHHA", "name": "\u5efa\u4fe1\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINZHUOYUECHENGZHANGYINIANCHIYOUHUNHEA"}, "014654": {"suoxie": "JXZYCZYNCYHHC", "name": "\u5efa\u4fe1\u5353\u8d8a\u6210\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINZHUOYUECHENGZHANGYINIANCHIYOUHUNHEC"}, "014657": {"suoxie": "ZORXZYYNCYQHHA", "name": "\u4e2d\u6b27\u878d\u4eab\u589e\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURONGXIANGZENGYIYINIANCHIYOUQIHUNHEA"}, "014658": {"suoxie": "ZORXZYYNCYQHHC", "name": "\u4e2d\u6b27\u878d\u4eab\u589e\u76ca\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOURONGXIANGZENGYIYINIANCHIYOUQIHUNHEC"}, "014661": {"suoxie": "THSHJETFFQLJA", "name": "\u5929\u5f18\u4e0a\u6d77\u91d1ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGSHANGHAIJINETFFAQILIANJIEA"}, "014662": {"suoxie": "THSHJETFFQLJC", "name": "\u5929\u5f18\u4e0a\u6d77\u91d1ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGSHANGHAIJINETFFAQILIANJIEC"}, "014663": {"suoxie": "FGCXFZLNDKHHA", "name": "\u5bcc\u56fd\u521b\u65b0\u53d1\u5c55\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHUANGXINFAZHANLIANGNIANDINGKAIHUNHEA"}, "014664": {"suoxie": "FGCXFZLNDKHHC", "name": "\u5bcc\u56fd\u521b\u65b0\u53d1\u5c55\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHUANGXINFAZHANLIANGNIANDINGKAIHUNHEC"}, "014665": {"suoxie": "GFYXYNCYHHFOF", "name": "\u5e7f\u53d1\u60a6\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUANGFAYUEXIANGYINIANCHIYOUHUNHEFOF"}, "014670": {"suoxie": "YHAYDZZQD", "name": "\u94f6\u534e\u5b89\u76c8\u77ed\u503a\u503a\u5238D", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YINHUAANYINGDUANZHAIZHAIQUAND"}, "014671": {"suoxie": "FGYLZQA", "name": "\u5bcc\u56fd\u88d5\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOYULIZHAIQUANA"}, "014672": {"suoxie": "FGYLZQC", "name": "\u5bcc\u56fd\u88d5\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOYULIZHAIQUANC"}, "014673": {"suoxie": "FGZZGGTHLWETFFQSLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6e2f\u80a1\u901a\u4e92\u8054\u7f51ETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGANGGUTONGHULIANWANGETFFAQISHILIANJIEA"}, "014674": {"suoxie": "FGZZGGTHLWETFFQSLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6e2f\u80a1\u901a\u4e92\u8054\u7f51ETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGANGGUTONGHULIANWANGETFFAQISHILIANJIEC"}, "014677": {"suoxie": "ZXBCZYDLHHE", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u8fdc\u52a8\u529b\u6df7\u5408E", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGZHIYUANDONGLIHUNHEE"}, "014680": {"suoxie": "JYYXYNCYHHFOFA", "name": "\u4ea4\u94f6\u4f18\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINYOUXIANGYINIANCHIYOUHUNHEFOFA"}, "014681": {"suoxie": "JYYXYNCYHHFOFC", "name": "\u4ea4\u94f6\u4f18\u4eab\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIAOYINYOUXIANGYINIANCHIYOUHUNHEFOFC"}, "014682": {"suoxie": "FGZPWJ12GYCYHHFOFA", "name": "\u5bcc\u56fd\u667a\u6d66\u7a33\u8fdb12\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOZHIPUWENJIN12GEYUECHIYOUHUNHEFOFA"}, "014683": {"suoxie": "FGZPWJ12GYCYHHFOFC", "name": "\u5bcc\u56fd\u667a\u6d66\u7a33\u8fdb12\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "FUGUOZHIPUWENJIN12GEYUECHIYOUHUNHEFOFC"}, "014688": {"suoxie": "ZSZJCZD", "name": "\u62db\u5546\u62db\u666f\u7eaf\u503aD", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGZHAOJINGCHUNZHAID"}, "014689": {"suoxie": "GTZZGGT50ETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGGUTONG50ETFFAQILIANJIEA"}, "014690": {"suoxie": "GTZZGGT50ETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGGUTONG50ETFFAQILIANJIEC"}, "014691": {"suoxie": "ZJLHYXHHA", "name": "\u4e2d\u52a0\u91cf\u5316\u7814\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIALIANGHUAYANXUANHUNHEA"}, "014692": {"suoxie": "ZJLHYXHHC", "name": "\u4e2d\u52a0\u91cf\u5316\u7814\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGJIALIANGHUAYANXUANHUNHEC"}, "014695": {"suoxie": "CSXJZZLGYCYHHA", "name": "\u957f\u76db\u5148\u8fdb\u5236\u9020\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGXIANJINZHIZAOLIUGEYUECHIYOUHUNHEA"}, "014696": {"suoxie": "CSXJZZLGYCYHHC", "name": "\u957f\u76db\u5148\u8fdb\u5236\u9020\u516d\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGXIANJINZHIZAOLIUGEYUECHIYOUHUNHEC"}, "014699": {"suoxie": "DFGDZZHHA", "name": "\u4e1c\u65b9\u9ad8\u7aef\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGGAODUANZHIZAOHUNHEA"}, "014700": {"suoxie": "DFGDZZHHC", "name": "\u4e1c\u65b9\u9ad8\u7aef\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGGAODUANZHIZAOHUNHEC"}, "014703": {"suoxie": "BSSDLHHHA", "name": "\u535a\u65f6\u65f6\u4ee3\u9886\u822a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHIDAILINGHANGHUNHEA"}, "014704": {"suoxie": "BSSDLHHHC", "name": "\u535a\u65f6\u65f6\u4ee3\u9886\u822a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHISHIDAILINGHANGHUNHEC"}, "014708": {"suoxie": "THZXJKHHA", "name": "\u5929\u5f18\u81fb\u9009\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGZHENXUANJIANKANGHUNHEA"}, "014709": {"suoxie": "THZXJKHHC", "name": "\u5929\u5f18\u81fb\u9009\u5065\u5eb7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGZHENXUANJIANKANGHUNHEC"}, "014712": {"suoxie": "HSQHHYZQA", "name": "\u6052\u751f\u524d\u6d77\u6052\u88d5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYUZHAIQUANA"}, "014713": {"suoxie": "HSQHHYZQC", "name": "\u6052\u751f\u524d\u6d77\u6052\u88d5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HENGSHENGQIANHAIHENGYUZHAIQUANC"}, "014714": {"suoxie": "GYRXYNDKCZZQFQS", "name": "\u5de5\u94f6\u745e\u5174\u4e00\u5e74\u5b9a\u5f00\u7eaf\u503a\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINRUIXINGYINIANDINGKAICHUNZHAIZHAIQUANFAQISHI"}, "014716": {"suoxie": "DXXYZQA", "name": "\u4e1c\u5174\u5174\u6e90\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGXINGXINGYUANZHAIQUANA"}, "014717": {"suoxie": "DXXYZQC", "name": "\u4e1c\u5174\u5174\u6e90\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGXINGXINGYUANZHAIQUANC"}, "014718": {"suoxie": "FGTXJHHHA", "name": "\u5bcc\u56fd\u5929\u65ed\u5747\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANXUJUNHENGHUNHEA"}, "014719": {"suoxie": "FGTXJHHHC", "name": "\u5bcc\u56fd\u5929\u65ed\u5747\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANXUJUNHENGHUNHEC"}, "014720": {"suoxie": "CJJLZQXC", "name": "\u957f\u6c5f\u805a\u5229\u503a\u5238\u578bC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGJULIZHAIQUANXINGC"}, "014724": {"suoxie": "DFLHDCLHHC", "name": "\u4e1c\u65b9\u91cf\u5316\u591a\u7b56\u7565\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGLIANGHUADUOCELUEHUNHEC"}, "014727": {"suoxie": "YFDCZDLHHA", "name": "\u6613\u65b9\u8fbe\u6210\u957f\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHENGZHANGDONGLIHUNHEA"}, "014728": {"suoxie": "YFDCZDLHHC", "name": "\u6613\u65b9\u8fbe\u6210\u957f\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHENGZHANGDONGLIHUNHEC"}, "014729": {"suoxie": "QHKYXXCYHHC", "name": "\u524d\u6d77\u5f00\u6e90\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "QIANHAIKAIYUANXINXINGCHANYEHUNHEC"}, "014736": {"suoxie": "CJHXZJTXGPFQA", "name": "\u521b\u91d1\u5408\u4fe1\u4e13\u7cbe\u7279\u65b0\u80a1\u7968\u53d1\u8d77A", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINZHUANJINGTEXINGUPIAOFAQIA"}, "014737": {"suoxie": "CJHXZJTXGPFQC", "name": "\u521b\u91d1\u5408\u4fe1\u4e13\u7cbe\u7279\u65b0\u80a1\u7968\u53d1\u8d77C", "style": "\u80a1\u7968\u578b", "pinyin": "CHUANGJINHEXINZHUANJINGTEXINGUPIAOFAQIC"}, "014746": {"suoxie": "BLDGGTYJSYHHA", "name": "\u8d1d\u83b1\u5fb7\u6e2f\u80a1\u901a\u8fdc\u666f\u89c6\u91ce\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BEILAIDEGANGGUTONGYUANJINGSHIYEHUNHEA"}, "014747": {"suoxie": "BLDGGTYJSYHHC", "name": "\u8d1d\u83b1\u5fb7\u6e2f\u80a1\u901a\u8fdc\u666f\u89c6\u91ce\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BEILAIDEGANGGUTONGYUANJINGSHIYEHUNHEC"}, "014750": {"suoxie": "XHXFJX6GYCYHHFQA", "name": "\u5174\u534e\u6d88\u8d39\u7cbe\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUAXIAOFEIJINGXUAN6GEYUECHIYOUHUNHEFAQIA"}, "014751": {"suoxie": "XHXFJX6GYCYHHFQC", "name": "\u5174\u534e\u6d88\u8d39\u7cbe\u90096\u4e2a\u6708\u6301\u6709\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGHUAXIAOFEIJINGXUAN6GEYUECHIYOUHUNHEFAQIC"}, "014752": {"suoxie": "CXWJZZYNCYHHA", "name": "\u957f\u4fe1\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANZENGZHANGYINIANCHIYOUHUNHEA"}, "014753": {"suoxie": "CXWJZZYNCYHHC", "name": "\u957f\u4fe1\u7a33\u5065\u589e\u957f\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINWENJIANZENGZHANGYINIANCHIYOUHUNHEC"}, "014756": {"suoxie": "PHCZLHLNCYQHHA", "name": "\u9e4f\u534e\u6210\u957f\u9886\u822a\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGLINGHANGLIANGNIANCHIYOUQIHUNHEA"}, "014757": {"suoxie": "PHCZLHLNCYQHHC", "name": "\u9e4f\u534e\u6210\u957f\u9886\u822a\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHENGZHANGLINGHANGLIANGNIANCHIYOUQIHUNHEC"}, "014758": {"suoxie": "MSJYYYJKGPC", "name": "\u6c11\u751f\u52a0\u94f6\u533b\u836f\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "MINSHENGJIAYINYIYAOJIANKANGGUPIAOC"}, "014759": {"suoxie": "ZOQFHHA", "name": "\u4e2d\u6b27\u742a\u798f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUQIFUHUNHEA"}, "014760": {"suoxie": "ZOQFHHC", "name": "\u4e2d\u6b27\u742a\u798f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUQIFUHUNHEC"}, "014765": {"suoxie": "ZOTZHHHFQA", "name": "\u4e2d\u6b27\u78b3\u4e2d\u548c\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUTANZHONGHEHUNHEFAQIA"}, "014766": {"suoxie": "ZOTZHHHFQC", "name": "\u4e2d\u6b27\u78b3\u4e2d\u548c\u6df7\u5408\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUTANZHONGHEHUNHEFAQIC"}, "014767": {"suoxie": "JSCCHCWJ6GYCYQHHA", "name": "\u666f\u987a\u957f\u57ce\u534e\u57ce\u7a33\u50656\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGHUACHENGWENJIAN6GEYUECHIYOUQIHUNHEA"}, "014768": {"suoxie": "JSCCHCWJ6GYCYQHHC", "name": "\u666f\u987a\u957f\u57ce\u534e\u57ce\u7a33\u50656\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINGSHUNCHANGCHENGHUACHENGWENJIAN6GEYUECHIYOUQIHUNHEC"}, "014773": {"suoxie": "HAPZLXHHA", "name": "\u534e\u5b89\u54c1\u8d28\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANPINZHILINGXIANHUNHEA"}, "014774": {"suoxie": "HAPZLXHHC", "name": "\u534e\u5b89\u54c1\u8d28\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANPINZHILINGXIANHUNHEC"}, "014775": {"suoxie": "ZSABZLZQA", "name": "\u62db\u5546\u5b89\u672c\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANBENZENGLIZHAIQUANA"}, "014776": {"suoxie": "FGZZXPCYETFFQSLJA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINPIANCHANYEETFFAQISHILIANJIEA"}, "014777": {"suoxie": "FGZZXPCYETFFQSLJC", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1aETF\u53d1\u8d77\u5f0f\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINPIANCHANYEETFFAQISHILIANJIEC"}, "014786": {"suoxie": "HSPZYXHHA", "name": "\u60e0\u5347\u54c1\u8d28\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUISHENGPINZHIYOUXUANHUNHEA"}, "014787": {"suoxie": "HSPZYXHHC", "name": "\u60e0\u5347\u54c1\u8d28\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUISHENGPINZHIYOUXUANHUNHEC"}, "014794": {"suoxie": "FGYJYXHHA", "name": "\u5bcc\u56fd\u8fdc\u89c1\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYUANJIANYOUXUANHUNHEA"}, "014795": {"suoxie": "FGYJYXHHC", "name": "\u5bcc\u56fd\u8fdc\u89c1\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOYUANJIANYOUXUANHUNHEC"}, "014799": {"suoxie": "GYZRYNCYHHA", "name": "\u5de5\u94f6\u62db\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINZHAORUIYINIANCHIYOUHUNHEA"}, "014800": {"suoxie": "GYZRYNCYHHC", "name": "\u5de5\u94f6\u62db\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GONGYINZHAORUIYINIANCHIYOUHUNHEC"}, "014801": {"suoxie": "HTCXFYZDZA", "name": "\u7ea2\u571f\u521b\u65b0\u4e30\u6e90\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HONGTUCHUANGXINFENGYUANZHONGDUANZHAIA"}, "014802": {"suoxie": "HTCXFYZDZC", "name": "\u7ea2\u571f\u521b\u65b0\u4e30\u6e90\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HONGTUCHUANGXINFENGYUANZHONGDUANZHAIC"}, "014805": {"suoxie": "GJLHJXHHA", "name": "\u56fd\u91d1\u91cf\u5316\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINLIANGHUAJINGXUANHUNHEA"}, "014806": {"suoxie": "GJLHJXHHC", "name": "\u56fd\u91d1\u91cf\u5316\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINLIANGHUAJINGXUANHUNHEC"}, "014807": {"suoxie": "TDHLJQZX18GYCYHHA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u666f\u6c14\u667a\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIJINGQIZHIXUAN18GEYUECHIYOUHUNHEA"}, "014808": {"suoxie": "TDHLJQZX18GYCYHHC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u666f\u6c14\u667a\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIJINGQIZHIXUAN18GEYUECHIYOUHUNHEC"}, "014809": {"suoxie": "HAFRYNCYHHA", "name": "\u534e\u5b89\u6ca3\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANFENGRUIYINIANCHIYOUHUNHEA"}, "014810": {"suoxie": "HAFRYNCYHHC", "name": "\u534e\u5b89\u6ca3\u745e\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAANFENGRUIYINIANCHIYOUHUNHEC"}, "014811": {"suoxie": "PAXYCZ1NCYHHA", "name": "\u5e73\u5b89\u5174\u5955\u6210\u957f1\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANXINGYICHENGZHANG1NIANCHIYOUHUNHEA"}, "014812": {"suoxie": "PAXYCZ1NCYHHC", "name": "\u5e73\u5b89\u5174\u5955\u6210\u957f1\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANXINGYICHENGZHANG1NIANCHIYOUHUNHEC"}, "014814": {"suoxie": "GLHHCZ", "name": "\u683c\u6797\u6cd3\u7693\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GELINHONGHAOCHUNZHAI"}, "014818": {"suoxie": "GJXXJZHHA", "name": "\u56fd\u91d1\u65b0\u5174\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINXINGJIAZHIHUNHEA"}, "014819": {"suoxie": "GJXXJZHHC", "name": "\u56fd\u91d1\u65b0\u5174\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJINXINXINGJIAZHIHUNHEC"}, "014825": {"suoxie": "HQXZWLYNCYHHA", "name": "\u6c47\u6cc9\u5174\u81f3\u672a\u6765\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANXINGZHIWEILAIYINIANCHIYOUHUNHEA"}, "014826": {"suoxie": "HQXZWLYNCYHHC", "name": "\u6c47\u6cc9\u5174\u81f3\u672a\u6765\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIQUANXINGZHIWEILAIYINIANCHIYOUHUNHEC"}, "014829": {"suoxie": "NDXNYQCHHA", "name": "\u8bfa\u5fb7\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEXINNENGYUANQICHEHUNHEA"}, "014830": {"suoxie": "NDXNYQCHHC", "name": "\u8bfa\u5fb7\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEXINNENGYUANQICHEHUNHEC"}, "014831": {"suoxie": "XYZZ1000ZSZQA", "name": "\u5174\u94f6\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENG1000ZHISHUZENGQIANGA"}, "014832": {"suoxie": "XYZZ1000ZSZQC", "name": "\u5174\u94f6\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINZHONGZHENG1000ZHISHUZENGQIANGC"}, "014833": {"suoxie": "HTFYXHHC", "name": "\u6c47\u6dfb\u5bcc\u76c8\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGXINHUNHEC"}, "014834": {"suoxie": "HTFYXHHD", "name": "\u6c47\u6dfb\u5bcc\u76c8\u946b\u6df7\u5408D", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGXINHUNHED"}, "014836": {"suoxie": "HTFYALHPZHHC", "name": "\u6c47\u6dfb\u5bcc\u76c8\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGANLINGHUOPEIZHIHUNHEC"}, "014837": {"suoxie": "HTFYALHPZHHD", "name": "\u6c47\u6dfb\u5bcc\u76c8\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408D", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYINGANLINGHUOPEIZHIHUNHED"}, "014838": {"suoxie": "XYTZHZTHHA", "name": "\u5174\u94f6\u78b3\u4e2d\u548c\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINTANZHONGHEZHUTIHUNHEA"}, "014839": {"suoxie": "XYTZHZTHHC", "name": "\u5174\u94f6\u78b3\u4e2d\u548c\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGYINTANZHONGHEZHUTIHUNHEC"}, "014840": {"suoxie": "ZSYHHH", "name": "\u62db\u5546\u88d5\u534e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGYUHUAHUNHE"}, "014848": {"suoxie": "TDHLMLYNDKZQFQS", "name": "\u6cf0\u8fbe\u5b8f\u5229\u95fd\u5229\u4e00\u5e74\u5b9a\u5f00\u503a\u5238\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIDAHONGLIMINLIYINIANDINGKAIZHAIQUANFAQISHI"}, "014849": {"suoxie": "JXJKMSHHC", "name": "\u5efa\u4fe1\u5065\u5eb7\u6c11\u751f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINJIANKANGMINSHENGHUNHEC"}, "014852": {"suoxie": "JSTHYNCYQHHA", "name": "\u5609\u5b9e\u6dfb\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHITIANHUIYINIANCHIYOUQIHUNHEA"}, "014853": {"suoxie": "JSTHYNCYQHHC", "name": "\u5609\u5b9e\u6dfb\u60e0\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JIASHITIANHUIYINIANCHIYOUQIHUNHEC"}, "014861": {"suoxie": "SWLXSXHHXFQSA", "name": "\u7533\u4e07\u83f1\u4fe1\u53cc\u79a7\u6df7\u5408\u578b\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINSHUANGXIHUNHEXINGFAQISHIA"}, "014862": {"suoxie": "SWLXSXHHXFQSC", "name": "\u7533\u4e07\u83f1\u4fe1\u53cc\u79a7\u6df7\u5408\u578b\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINSHUANGXIHUNHEXINGFAQISHIC"}, "014863": {"suoxie": "JXXXCYGPC", "name": "\u5efa\u4fe1\u4fe1\u606f\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINXINXICHANYEGUPIAOC"}, "014864": {"suoxie": "JXSPYLHYGPC", "name": "\u5efa\u4fe1\u98df\u54c1\u996e\u6599\u884c\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINSHIPINYINLIAOHANGYEGUPIAOC"}, "014866": {"suoxie": "DMMSCIZGAGZQC", "name": "\u5927\u6469MSCI\u4e2d\u56fdA\u80a1\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DAMOMSCIZHONGGUOAGUZENGQIANGC"}, "014867": {"suoxie": "DMYYAHHHC", "name": "\u5927\u6469\u4f18\u60a6\u5b89\u548c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOYOUYUEANHEHUNHEC"}, "014868": {"suoxie": "DMLDYXZQC", "name": "\u5927\u6469\u7075\u52a8\u4f18\u9009\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DAMOLINGDONGYOUXUANZHAIQUANC"}, "014869": {"suoxie": "DMNXZZHHC", "name": "\u5927\u6469\u5185\u9700\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMONEIXUZENGZHANGHUNHEC"}, "014871": {"suoxie": "DMKJLXHHC", "name": "\u5927\u6469\u79d1\u6280\u9886\u5148\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DAMOKEJILINGXIANHUNHEC"}, "014872": {"suoxie": "JSPZLCYNCYHHA", "name": "\u5609\u5b9e\u54c1\u8d28\u84dd\u7b79\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIPINZHILANCHOUYINIANCHIYOUHUNHEA"}, "014873": {"suoxie": "JSPZLCYNCYHHC", "name": "\u5609\u5b9e\u54c1\u8d28\u84dd\u7b79\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIPINZHILANCHOUYINIANCHIYOUHUNHEC"}, "014874": {"suoxie": "HSHYHBHHA", "name": "\u60e0\u5347\u60e0\u8fdc\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIYUANHUIBAOHUNHEA"}, "014875": {"suoxie": "HSHYHBHHC", "name": "\u60e0\u5347\u60e0\u8fdc\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUISHENGHUIYUANHUIBAOHUNHEC"}, "014878": {"suoxie": "HAJZQDYNCYHHA", "name": "\u534e\u5b89\u4ef7\u503c\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJIAZHIQUDONGYINIANCHIYOUHUNHEA"}, "014879": {"suoxie": "HAJZQDYNCYHHC", "name": "\u534e\u5b89\u4ef7\u503c\u9a71\u52a8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANJIAZHIQUDONGYINIANCHIYOUHUNHEC"}, "014887": {"suoxie": "ZSAF1NDKZFQS", "name": "\u62db\u5546\u5b89\u798f1\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANFU1NIANDINGKAIZHAIFAQISHI"}, "014888": {"suoxie": "DFHJRZX18GYCYHHA", "name": "\u4e1c\u65b9\u7ea2\u9526\u878d\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINRONGZHENXUAN18GEYUECHIYOUHUNHEA"}, "014889": {"suoxie": "DFHJRZX18GYCYHHC", "name": "\u4e1c\u65b9\u7ea2\u9526\u878d\u7504\u900918\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJINRONGZHENXUAN18GEYUECHIYOUHUNHEC"}, "014892": {"suoxie": "YYTTX12GYCYHHA", "name": "\u6c38\u8d62\u6dfb\u6dfb\u6b2312\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGTIANTIANXIN12GEYUECHIYOUHUNHEA"}, "014893": {"suoxie": "YYTTX12GYCYHHC", "name": "\u6c38\u8d62\u6dfb\u6dfb\u6b2312\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YONGYINGTIANTIANXIN12GEYUECHIYOUHUNHEC"}, "014894": {"suoxie": "DWZZ13NZCXJRZA", "name": "\u4e1c\u5434\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGWUZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIA"}, "014895": {"suoxie": "DWZZ13NZCXJRZC", "name": "\u4e1c\u5434\u4e2d\u503a1-3\u5e74\u653f\u7b56\u6027\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGWUZHONGZHAI13NIANZHENGCEXINGJINRONGZHAIC"}, "014902": {"suoxie": "CCXNYGPFQSA", "name": "\u957f\u57ce\u65b0\u80fd\u6e90\u80a1\u7968\u53d1\u8d77\u5f0fA", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGXINNENGYUANGUPIAOFAQISHIA"}, "014903": {"suoxie": "CCXNYGPFQSC", "name": "\u957f\u57ce\u65b0\u80fd\u6e90\u80a1\u7968\u53d1\u8d77\u5f0fC", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGCHENGXINNENGYUANGUPIAOFAQISHIC"}, "014904": {"suoxie": "YFDYWYNCYHHA", "name": "\u6613\u65b9\u8fbe\u60a6\u7a33\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEWENYINIANCHIYOUHUNHEA"}, "014905": {"suoxie": "YFDYWYNCYHHC", "name": "\u6613\u65b9\u8fbe\u60a6\u7a33\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YIFANGDAYUEWENYINIANCHIYOUHUNHEC"}, "014906": {"suoxie": "GTZZXFDZZTETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIAOFEIDIANZIZHUTIETFFAQILIANJIEA"}, "014907": {"suoxie": "GTZZXFDZZTETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIAOFEIDIANZIZHUTIETFFAQILIANJIEC"}, "014908": {"suoxie": "GTZZXCLZTETFFQLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINCAILIAOZHUTIETFFAQILIANJIEA"}, "014909": {"suoxie": "GTZZXCLZTETFFQLJC", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF\u53d1\u8d77\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINCAILIAOZHUTIETFFAQILIANJIEC"}, "014912": {"suoxie": "NFXYZQ", "name": "\u5357\u65b9\u4fe1\u5143\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGXINYUANZHAIQUAN"}, "014913": {"suoxie": "BSYJHBHHA", "name": "\u535a\u65f6\u7814\u7a76\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUHUIBAOHUNHEA"}, "014914": {"suoxie": "BSYJHBHHC", "name": "\u535a\u65f6\u7814\u7a76\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYANJIUHUIBAOHUNHEC"}, "014919": {"suoxie": "YHXXYNCYQHHA", "name": "\u94f6\u534e\u5fc3\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINXUANYINIANCHIYOUQIHUNHEA"}, "014920": {"suoxie": "YHXXYNCYQHHC", "name": "\u94f6\u534e\u5fc3\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXINXUANYINIANCHIYOUQIHUNHEC"}, "014922": {"suoxie": "HXESGKCXTZYNCYHHA", "name": "\u534e\u590fESG\u53ef\u6301\u7eed\u6295\u8d44\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAESGKECHIXUTOUZIYINIANCHIYOUHUNHEA"}, "014923": {"suoxie": "HXESGKCXTZYNCYHHC", "name": "\u534e\u590fESG\u53ef\u6301\u7eed\u6295\u8d44\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAESGKECHIXUTOUZIYINIANCHIYOUHUNHEC"}, "014929": {"suoxie": "MSJYCXCZHHC", "name": "\u6c11\u751f\u52a0\u94f6\u521b\u65b0\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINCHUANGXINCHENGZHANGHUNHEC"}, "014930": {"suoxie": "FGGDZZHYGPC", "name": "\u5bcc\u56fd\u9ad8\u7aef\u5236\u9020\u884c\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOGAODUANZHIZAOHANGYEGUPIAOC"}, "014931": {"suoxie": "FGTYHGSPHHHC", "name": "\u5bcc\u56fd\u5929\u6e90\u6caa\u6e2f\u6df1\u5e73\u8861\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOTIANYUANHUGANGSHENPINGHENGHUNHEC"}, "014932": {"suoxie": "STMGYLJKGPC", "name": "\u4e0a\u6295\u6469\u6839\u533b\u7597\u5065\u5eb7\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENYILIAOJIANKANGGUPIAOC"}, "014933": {"suoxie": "NFYYBJLHPZHHC", "name": "\u5357\u65b9\u533b\u836f\u4fdd\u5065\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYIYAOBAOJIANLINGHUOPEIZHIHUNHEC"}, "014936": {"suoxie": "CJZX3GYCYHHFOFC", "name": "\u957f\u6c5f\u667a\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGZHIXUAN3GEYUECHIYOUHUNHEFOFC"}, "014937": {"suoxie": "STMGHXJXGPC", "name": "\u4e0a\u6295\u6469\u6839\u6838\u5fc3\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENHEXINJINGXUANGUPIAOC"}, "014938": {"suoxie": "TTCYSJHHA", "name": "\u540c\u6cf0\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAICHANYESHENGJIHUNHEA"}, "014939": {"suoxie": "TTCYSJHHC", "name": "\u540c\u6cf0\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAICHANYESHENGJIHUNHEC"}, "014940": {"suoxie": "TTZZCXHHA", "name": "\u540c\u6cf0\u81ea\u4e3b\u521b\u65b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIZIZHUCHUANGXINHUNHEA"}, "014941": {"suoxie": "TTZZCXHHC", "name": "\u540c\u6cf0\u81ea\u4e3b\u521b\u65b0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TONGTAIZIZHUCHUANGXINHUNHEC"}, "014942": {"suoxie": "PHZZXFHGCYZTETFLJA", "name": "\u9e4f\u534e\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEA"}, "014943": {"suoxie": "PHZZXFHGCYZTETFLJC", "name": "\u9e4f\u534e\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGXIFENHUAGONGCHANYEZHUTIETFLIANJIEC"}, "014949": {"suoxie": "JYHYLHPZHHC", "name": "\u4ea4\u94f6\u6052\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINHENGYILINGHUOPEIZHIHUNHEC"}, "014953": {"suoxie": "XDAYZCJXHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u81f3\u8bda\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHICHENGJINGXUANHUNHEC"}, "014954": {"suoxie": "XDAYYJYXHHC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u7814\u7a76\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINYANJIUYOUXUANHUNHEC"}, "014960": {"suoxie": "GTRADCLLHPZHHC", "name": "\u56fd\u6cf0\u878d\u5b89\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRONGANDUOCELUELINGHUOPEIZHIHUNHEC"}, "014963": {"suoxie": "JYXJZZHHC", "name": "\u4ea4\u94f6\u5148\u8fdb\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANJINZHIZAOHUNHEC"}, "014964": {"suoxie": "STMGKJQYLHPZHHC", "name": "\u4e0a\u6295\u6469\u6839\u79d1\u6280\u524d\u6cbf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHANGTOUMOGENKEJIQIANYANLINGHUOPEIZHIHUNHEC"}, "014967": {"suoxie": "JXQLXLCGPC", "name": "\u5efa\u4fe1\u6f5c\u529b\u65b0\u84dd\u7b79\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINQIANLIXINLANCHOUGUPIAOC"}, "014986": {"suoxie": "DFHXDLHHC", "name": "\u4e1c\u65b9\u6838\u5fc3\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHEXINDONGLIHUNHEC"}, "014989": {"suoxie": "GTXJJLHPZHHC", "name": "\u56fd\u6cf0\u65b0\u7ecf\u6d4e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIXINJINGJILINGHUOPEIZHIHUNHEC"}, "014994": {"suoxie": "GTSZ180JRETFLJC", "name": "\u56fd\u6cf0\u4e0a\u8bc1180\u91d1\u878dETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENG180JINRONGETFLIANJIEC"}, "014995": {"suoxie": "ZOLHXFHHA", "name": "\u4e2d\u6b27\u91cf\u5316\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOULIANGHUAXIANFENGHUNHEA"}, "014996": {"suoxie": "ZOLHXFHHC", "name": "\u4e2d\u6b27\u91cf\u5316\u5148\u950b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOULIANGHUAXIANFENGHUNHEC"}, "014997": {"suoxie": "GTGZXNYQCZSC", "name": "\u56fd\u6cf0\u56fd\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGXINNENGYUANQICHEZHISHUC"}, "014998": {"suoxie": "GTMFCLJZHHC", "name": "\u56fd\u6cf0\u6c11\u798f\u7b56\u7565\u4ef7\u503c\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINFUCELUEJIAZHIHUNHEC"}, "014999": {"suoxie": "HTBXJNYHHA", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJINIANYINGHUNHEA"}, "015000": {"suoxie": "HTBXJNYHHC", "name": "\u534e\u6cf0\u4fdd\u5174\u5409\u5e74\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAOXINGJINIANYINGHUNHEC"}, "015001": {"suoxie": "GYWLCYGPC", "name": "\u5de5\u94f6\u7269\u6d41\u4ea7\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINWULIUCHANYEGUPIAOC"}, "015002": {"suoxie": "GYSTHJGPC", "name": "\u5de5\u94f6\u751f\u6001\u73af\u5883\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINSHENGTAIHUANJINGGUPIAOC"}, "015004": {"suoxie": "ZYNYGXHHXFQA", "name": "\u4e2d\u90ae\u80fd\u6e90\u9769\u65b0\u6df7\u5408\u578b\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUNENGYUANGEXINHUNHEXINGFAQIA"}, "015005": {"suoxie": "ZYNYGXHHXFQC", "name": "\u4e2d\u90ae\u80fd\u6e90\u9769\u65b0\u6df7\u5408\u578b\u53d1\u8d77C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUNENGYUANGEXINHUNHEXINGFAQIC"}, "015006": {"suoxie": "ZOLHDLHHA", "name": "\u4e2d\u6b27\u91cf\u5316\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOULIANGHUADONGLIHUNHEA"}, "015007": {"suoxie": "ZOLHDLHHC", "name": "\u4e2d\u6b27\u91cf\u5316\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOULIANGHUADONGLIHUNHEC"}, "015008": {"suoxie": "HAYL30TCYQDZA", "name": "\u6c47\u5b89\u6c38\u522930\u5929\u6301\u6709\u671f\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANYONGLI30TIANCHIYOUQIDUANZHAIA"}, "015009": {"suoxie": "HAYL30TCYQDZC", "name": "\u6c47\u5b89\u6c38\u522930\u5929\u6301\u6709\u671f\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIANYONGLI30TIANCHIYOUQIDUANZHAIC"}, "015010": {"suoxie": "JHJXHHA", "name": "\u5609\u5408\u9526\u946b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINXINHUNHEA"}, "015011": {"suoxie": "JHJXHHC", "name": "\u5609\u5408\u9526\u946b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAHEJINXINHUNHEC"}, "015012": {"suoxie": "PYASAHHBYNCYHHA", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u5f18\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANHONGHUIBAOYINIANCHIYOUHUNHEA"}, "015013": {"suoxie": "PYASAHHBYNCYHHC", "name": "\u6d66\u94f6\u5b89\u76db\u5b89\u5f18\u56de\u62a5\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PUYINANSHENGANHONGHUIBAOYINIANCHIYOUHUNHEC"}, "015017": {"suoxie": "GTRFWYZZHHLOFC", "name": "\u56fd\u6cf0\u878d\u4e30\u5916\u5ef6\u589e\u957f\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRONGFENGWAIYANZENGZHANGHUNHELOFC"}, "015021": {"suoxie": "PAJYXHBC", "name": "\u5e73\u5b89\u4ea4\u6613\u578b\u8d27\u5e01C", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANJIAOYIXINGHUOBIC"}, "015022": {"suoxie": "WJAHCZ3GYCYZQXA", "name": "\u4e07\u5bb6\u5b89\u6052\u7eaf\u503a3\u4e2a\u6708\u6301\u6709\u503a\u5238\u578bA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAANHENGCHUNZHAI3GEYUECHIYOUZHAIQUANXINGA"}, "015023": {"suoxie": "WJAHCZ3GYCYZQXC", "name": "\u4e07\u5bb6\u5b89\u6052\u7eaf\u503a3\u4e2a\u6708\u6301\u6709\u503a\u5238\u578bC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAANHENGCHUNZHAI3GEYUECHIYOUZHAIQUANXINGC"}, "015030": {"suoxie": "BSYJHBHHA", "name": "\u535a\u65f6\u8fdc\u89c1\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYUANJIANHUIBAOHUNHEA"}, "015031": {"suoxie": "BSYJHBHHC", "name": "\u535a\u65f6\u8fdc\u89c1\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYUANJIANHUIBAOHUNHEC"}, "015032": {"suoxie": "ZRYYXFHHA", "name": "\u4e2d\u878d\u533b\u836f\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYIYAOXIAOFEIHUNHEA"}, "015033": {"suoxie": "ZRYYXFHHC", "name": "\u4e2d\u878d\u533b\u836f\u6d88\u8d39\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGYIYAOXIAOFEIHUNHEC"}, "015037": {"suoxie": "THMSCIZGA50HLHTZSA", "name": "\u5929\u5f18MSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGMSCIZHONGGUOA50HULIANHUTONGZHISHUA"}, "015038": {"suoxie": "THMSCIZGA50HLHTZSC", "name": "\u5929\u5f18MSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGMSCIZHONGGUOA50HULIANHUTONGZHISHUC"}, "015039": {"suoxie": "CXJLQSHHC", "name": "\u957f\u4fe1\u91d1\u5229\u8d8b\u52bf\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINJINLIQUSHIHUNHEC"}, "015040": {"suoxie": "GTGZSPYLHYLOFC", "name": "\u56fd\u6cf0\u56fd\u8bc1\u98df\u54c1\u996e\u6599\u884c\u4e1a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGSHIPINYINLIAOHANGYELOFC"}, "015041": {"suoxie": "XBLDXFHHC", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u5bcc\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINFUHUNHEC"}, "015042": {"suoxie": "GTGZFDCHYZSC", "name": "\u56fd\u6cf0\u56fd\u8bc1\u623f\u5730\u4ea7\u884c\u4e1a\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGFANGDICHANHANGYEZHISHUC"}, "015045": {"suoxie": "DCHXYNDKZFQS", "name": "\u5927\u6210\u60e0\u4fe1\u4e00\u5e74\u5b9a\u5f00\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGHUIXINYINIANDINGKAIZHAIFAQISHI"}, "015046": {"suoxie": "QHKYZHJGZSC", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u822a\u519b\u5de5\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGHANGJUNGONGZHISHUC"}, "015048": {"suoxie": "JXXNYHYGPC", "name": "\u5efa\u4fe1\u65b0\u80fd\u6e90\u884c\u4e1a\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "JIANXINXINNENGYUANHANGYEGUPIAOC"}, "015054": {"suoxie": "STMGSRHHFOFC", "name": "\u4e0a\u6295\u6469\u6839\u5c1a\u777f\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENSHANGRUIHUNHEFOFC"}, "015055": {"suoxie": "STAXHBYNCYZC", "name": "\u4e0a\u6295\u5b89\u4eab\u56de\u62a5\u4e00\u5e74\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUANXIANGHUIBAOYINIANCHIYOUZHAIC"}, "015057": {"suoxie": "STMGHXYXHHC", "name": "\u4e0a\u6295\u6469\u6839\u6838\u5fc3\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENHEXINYOUXUANHUNHEC"}, "015078": {"suoxie": "PALHPZHHC", "name": "\u5e73\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANLINGHUOPEIZHIHUNHEC"}, "015096": {"suoxie": "DCSZJJHHFQSA", "name": "\u4e1c\u8d22\u6570\u5b57\u7ecf\u6d4e\u6df7\u5408\u53d1\u8d77\u5f0fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAISHUZIJINGJIHUNHEFAQISHIA"}, "015097": {"suoxie": "DCSZJJHHFQSC", "name": "\u4e1c\u8d22\u6570\u5b57\u7ecf\u6d4e\u6df7\u5408\u53d1\u8d77\u5f0fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGCAISHUZIJINGJIHUNHEFAQISHIC"}, "015098": {"suoxie": "ZOXXDYYNCYHHA", "name": "\u4e2d\u6b27\u946b\u4eab\u9f0e\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUXINXIANGDINGYIYINIANCHIYOUHUNHEA"}, "015099": {"suoxie": "ZOXXDYYNCYHHC", "name": "\u4e2d\u6b27\u946b\u4eab\u9f0e\u76ca\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "ZHONGOUXINXIANGDINGYIYINIANCHIYOUHUNHEC"}, "015102": {"suoxie": "DFHESGKCXTZHHA", "name": "\u4e1c\u65b9\u7ea2ESG\u53ef\u6301\u7eed\u6295\u8d44\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGESGKECHIXUTOUZIHUNHEA"}, "015103": {"suoxie": "DFHESGKCXTZHHC", "name": "\u4e1c\u65b9\u7ea2ESG\u53ef\u6301\u7eed\u6295\u8d44\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGESGKECHIXUTOUZIHUNHEC"}, "015143": {"suoxie": "ZOZNZZHHA", "name": "\u4e2d\u6b27\u667a\u80fd\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHINENGZHIZAOHUNHEA"}, "015144": {"suoxie": "ZOZNZZHHC", "name": "\u4e2d\u6b27\u667a\u80fd\u5236\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUZHINENGZHIZAOHUNHEC"}, "020001": {"suoxie": "GTJYZZHH", "name": "\u56fd\u6cf0\u91d1\u9e70\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJINYINGZENGZHANGHUNHE"}, "020002": {"suoxie": "GTJLZQA", "name": "\u56fd\u6cf0\u91d1\u9f99\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIJINLONGZHAIQUANA"}, "020003": {"suoxie": "GTJLHYHH", "name": "\u56fd\u6cf0\u91d1\u9f99\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINLONGHANGYEHUNHE"}, "020005": {"suoxie": "GTJMWJHH", "name": "\u56fd\u6cf0\u91d1\u9a6c\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINMAWENJIANHUNHE"}, "020007": {"suoxie": "GTHBA", "name": "\u56fd\u6cf0\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAIHUOBIA"}, "020009": {"suoxie": "GTJPLCHH", "name": "\u56fd\u6cf0\u91d1\u9e4f\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINPENGLANCHOUHUNHE"}, "020010": {"suoxie": "GTJNCXCZHH", "name": "\u56fd\u6cf0\u91d1\u725b\u521b\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINNIUCHUANGXINCHENGZHANGHUNHE"}, "020011": {"suoxie": "GTHS300ZSA", "name": "\u56fd\u6cf0\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHUSHEN300ZHISHUA"}, "020012": {"suoxie": "GTJLZQC", "name": "\u56fd\u6cf0\u91d1\u9f99\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIJINLONGZHAIQUANC"}, "020015": {"suoxie": "GTQWYSHH", "name": "\u56fd\u6cf0\u533a\u4f4d\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIQUWEIYOUSHIHUNHE"}, "020018": {"suoxie": "GTJLHH", "name": "\u56fd\u6cf0\u91d1\u9e7f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINLUHUNHE"}, "020019": {"suoxie": "GTSLZQA", "name": "\u56fd\u6cf0\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAISHUANGLIZHAIQUANA"}, "020020": {"suoxie": "GTSLZQC", "name": "\u56fd\u6cf0\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAISHUANGLIZHAIQUANC"}, "020021": {"suoxie": "GTSZ180JRETFLJ", "name": "\u56fd\u6cf0\u4e0a\u8bc1180\u91d1\u878dETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENG180JINRONGETFLIANJIE"}, "020022": {"suoxie": "GTCLJZLHPZHH", "name": "\u56fd\u6cf0\u7b56\u7565\u4ef7\u503c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAICELUEJIAZHILINGHUOPEIZHIHUNHE"}, "020023": {"suoxie": "GTSJQDCLHH", "name": "\u56fd\u6cf0\u4e8b\u4ef6\u9a71\u52a8\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAISHIJIANQUDONGCELUEHUNHE"}, "020026": {"suoxie": "GTCZYXHH", "name": "\u56fd\u6cf0\u6210\u957f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAICHENGZHANGYOUXUANHUNHE"}, "020031": {"suoxie": "GTXJGLHBA", "name": "\u56fd\u6cf0\u73b0\u91d1\u7ba1\u7406\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAIXIANJINGUANLIHUOBIA"}, "020032": {"suoxie": "GTXJGLHBB", "name": "\u56fd\u6cf0\u73b0\u91d1\u7ba1\u7406\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAIXIANJINGUANLIHUOBIB"}, "020033": {"suoxie": "GTMAZLZQA", "name": "\u56fd\u6cf0\u6c11\u5b89\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIMINANZENGLIZHAIQUANA"}, "020034": {"suoxie": "GTMAZLZQC", "name": "\u56fd\u6cf0\u6c11\u5b89\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIMINANZENGLIZHAIQUANC"}, "040001": {"suoxie": "HACXHH", "name": "\u534e\u5b89\u521b\u65b0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANCHUANGXINHUNHE"}, "040002": {"suoxie": "HAZGAGZQZS", "name": "\u534e\u5b89\u4e2d\u56fdA\u80a1\u589e\u5f3a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGGUOAGUZENGQIANGZHISHU"}, "040003": {"suoxie": "HAXJFLHBA", "name": "\u534e\u5b89\u73b0\u91d1\u5bcc\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANXIANJINFULIHUOBIA"}, "040004": {"suoxie": "HABLPZHH", "name": "\u534e\u5b89\u5b9d\u5229\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANBAOLIPEIZHIHUNHE"}, "040005": {"suoxie": "HAHLHH", "name": "\u534e\u5b89\u5b8f\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHONGLIHUNHE"}, "040007": {"suoxie": "HAZXPCZHH", "name": "\u534e\u5b89\u4e2d\u5c0f\u76d8\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHONGXIAOPANCHENGZHANGHUNHE"}, "040008": {"suoxie": "HACLYXHHA", "name": "\u534e\u5b89\u7b56\u7565\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCELUEYOUXUANHUNHEA"}, "040009": {"suoxie": "HAWDSYZQA", "name": "\u534e\u5b89\u7a33\u5b9a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANWENDINGSHOUYIZHAIQUANA"}, "040010": {"suoxie": "HAWDSYZQB", "name": "\u534e\u5b89\u7a33\u5b9a\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANWENDINGSHOUYIZHAIQUANB"}, "040011": {"suoxie": "HAHXYXHH", "name": "\u534e\u5b89\u6838\u5fc3\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHEXINYOUXUANHUNHE"}, "040012": {"suoxie": "HAQHSYZQA", "name": "\u534e\u5b89\u5f3a\u5316\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANQIANGHUASHOUYIZHAIQUANA"}, "040013": {"suoxie": "HAQHSYZQB", "name": "\u534e\u5b89\u5f3a\u5316\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANQIANGHUASHOUYIZHAIQUANB"}, "040015": {"suoxie": "HADTLHPZHHA", "name": "\u534e\u5b89\u52a8\u6001\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANDONGTAILINGHUOPEIZHIHUNHEA"}, "040016": {"suoxie": "HAHYLDHH", "name": "\u534e\u5b89\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHANGYELUNDONGHUNHE"}, "040018": {"suoxie": "HAXGJXGPQDII", "name": "\u534e\u5b89\u9999\u6e2f\u7cbe\u9009\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "HUAANXIANGGANGJINGXUANGUPIAOQDII"}, "040019": {"suoxie": "HAWGSYZQC", "name": "\u534e\u5b89\u7a33\u56fa\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANWENGUSHOUYIZHAIQUANC"}, "040020": {"suoxie": "HASJZTHH", "name": "\u534e\u5b89\u5347\u7ea7\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANSHENGJIZHUTIHUNHE"}, "040021": {"suoxie": "HADZHSJGP", "name": "\u534e\u5b89\u5927\u4e2d\u534e\u5347\u7ea7\u80a1\u7968", "style": "QDII", "pinyin": "HUAANDAZHONGHUASHENGJIGUPIAO"}, "040022": {"suoxie": "HAKZZZQA", "name": "\u534e\u5b89\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAANKEZHUANZHAIZHAIQUANA"}, "040023": {"suoxie": "HAKZZZQB", "name": "\u534e\u5b89\u53ef\u8f6c\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUAANKEZHUANZHAIZHAIQUANB"}, "040025": {"suoxie": "HAKJDLHH", "name": "\u534e\u5b89\u79d1\u6280\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANKEJIDONGLIHUNHE"}, "040026": {"suoxie": "HAXYSJHZQA", "name": "\u534e\u5b89\u4fe1\u7528\u56db\u5b63\u7ea2\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANXINYONGSIJIHONGZHAIQUANA"}, "040035": {"suoxie": "HANXCLHHA", "name": "\u534e\u5b89\u9006\u5411\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANNIXIANGCELUEHUNHEA"}, "040036": {"suoxie": "HAAXSYZQA", "name": "\u534e\u5b89\u5b89\u5fc3\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANANXINSHOUYIZHAIQUANA"}, "040037": {"suoxie": "HAAXSYZQB", "name": "\u534e\u5b89\u5b89\u5fc3\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANANXINSHOUYIZHAIQUANB"}, "040038": {"suoxie": "HARRXHBA", "name": "\u534e\u5b89\u65e5\u65e5\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANRIRIXINHUOBIA"}, "040039": {"suoxie": "HARRXHBB", "name": "\u534e\u5b89\u65e5\u65e5\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANRIRIXINHUOBIB"}, "040040": {"suoxie": "HACZZQA", "name": "\u534e\u5b89\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANCHUNZHAIZHAIQUANA"}, "040041": {"suoxie": "HACZZQC", "name": "\u534e\u5b89\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAANCHUNZHAIZHAIQUANC"}, "040045": {"suoxie": "HATXZDZA", "name": "\u534e\u5b89\u6dfb\u946b\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUAANTIANXINZHONGDUANZHAIA"}, "040046": {"suoxie": "HANSDK100ZS", "name": "\u534e\u5b89\u7eb3\u65af\u8fbe\u514b100\u6307\u6570", "style": "QDII", "pinyin": "HUAANNASIDAKE100ZHISHU"}, "040047": {"suoxie": "HANSDK100ZSXC", "name": "\u534e\u5b89\u7eb3\u65af\u8fbe\u514b100\u6307\u6570\u73b0\u949e", "style": "QDII", "pinyin": "HUAANNASIDAKE100ZHISHUXIANCHAO"}, "040048": {"suoxie": "HANSDK100ZSXH", "name": "\u534e\u5b89\u7eb3\u65af\u8fbe\u514b100\u6307\u6570\u73b0\u6c47", "style": "QDII", "pinyin": "HUAANNASIDAKE100ZHISHUXIANHUI"}, "040180": {"suoxie": "HASZ180ETFLJ", "name": "\u534e\u5b89\u4e0a\u8bc1180ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHANGZHENG180ETFLIANJIE"}, "040190": {"suoxie": "HASZLTETFLJ", "name": "\u534e\u5b89\u4e0a\u8bc1\u9f99\u5934ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHANGZHENGLONGTOUETFLIANJIE"}, "041001": {"suoxie": "HACXHH", "name": "\u534e\u5b89\u521b\u65b0\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAANCHUANGXINHUNHE"}, "041002": {"suoxie": "HAZGAGZQZS", "name": "\u534e\u5b89\u4e2d\u56fdA\u80a1\u589e\u5f3a\u6307\u6570(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGGUOAGUZENGQIANGZHISHU"}, "041003": {"suoxie": "HAXJFLHBB", "name": "\u534e\u5b89\u73b0\u91d1\u5bcc\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANXIANJINFULIHUOBIB"}, "041004": {"suoxie": "HABLPZHH", "name": "\u534e\u5b89\u5b9d\u5229\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANBAOLIPEIZHIHUNHE"}, "041005": {"suoxie": "HAHLHH", "name": "\u534e\u5b89\u5b8f\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHONGLIHUNHE"}, "041007": {"suoxie": "HAZXPCZHH", "name": "\u534e\u5b89\u4e2d\u5c0f\u76d8\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANZHONGXIAOPANCHENGZHANGHUNHE"}, "041008": {"suoxie": "HACLYXHHA", "name": "\u534e\u5b89\u7b56\u7565\u4f18\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCELUEYOUXUANHUNHEA"}, "041009": {"suoxie": "HAWDSYZQA", "name": "\u534e\u5b89\u7a33\u5b9a\u6536\u76ca\u503a\u5238A(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANWENDINGSHOUYIZHAIQUANA"}, "041011": {"suoxie": "HAHXYXHH", "name": "\u534e\u5b89\u6838\u5fc3\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANHEXINYOUXUANHUNHE"}, "050001": {"suoxie": "BSJZZZHH", "name": "\u535a\u65f6\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIAZHIZENGZHANGHUNHE"}, "050002": {"suoxie": "BSHS300ZSA", "name": "\u535a\u65f6\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUSHEN300ZHISHUA"}, "050003": {"suoxie": "BSXJSYHBA", "name": "\u535a\u65f6\u73b0\u91d1\u6536\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIXIANJINSHOUYIHUOBIA"}, "050004": {"suoxie": "BSJXHHA", "name": "\u535a\u65f6\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJINGXUANHUNHEA"}, "050006": {"suoxie": "BSWDJZZQB", "name": "\u535a\u65f6\u7a33\u5b9a\u4ef7\u503c\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIWENDINGJIAZHIZHAIQUANB"}, "050007": {"suoxie": "BSPHPZHH", "name": "\u535a\u65f6\u5e73\u8861\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "BOSHIPINGHENGPEIZHIHUNHE"}, "050008": {"suoxie": "BSDSCYHH", "name": "\u535a\u65f6\u7b2c\u4e09\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIDISANCHANYEHUNHE"}, "050009": {"suoxie": "BSXXCZHH", "name": "\u535a\u65f6\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIXINXINGCHENGZHANGHUNHE"}, "050010": {"suoxie": "BSTXJZHHA", "name": "\u535a\u65f6\u7279\u8bb8\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHITEXUJIAZHIHUNHEA"}, "050011": {"suoxie": "BSXYZQAB", "name": "\u535a\u65f6\u4fe1\u7528\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIXINYONGZHAIQUANAB"}, "050012": {"suoxie": "BSCLHH", "name": "\u535a\u65f6\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHICELUEHUNHE"}, "050013": {"suoxie": "BSSZCDPETFLJ", "name": "\u535a\u65f6\u4e0a\u8bc1\u8d85\u5927\u76d8ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENGCHAODAPANETFLIANJIE"}, "050014": {"suoxie": "BSCYCZHHA", "name": "\u535a\u65f6\u521b\u4e1a\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGYECHENGZHANGHUNHEA"}, "050015": {"suoxie": "BSDZHYTJX", "name": "\u535a\u65f6\u5927\u4e2d\u534e\u4e9a\u592a\u7cbe\u9009", "style": "QDII", "pinyin": "BOSHIDAZHONGHUAYATAIJINGXUAN"}, "050016": {"suoxie": "BSHGHBZQAB", "name": "\u535a\u65f6\u5b8f\u89c2\u56de\u62a5\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHONGGUANHUIBAOZHAIQUANAB"}, "050018": {"suoxie": "BSHYLDHH", "name": "\u535a\u65f6\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIHANGYELUNDONGHUNHE"}, "050019": {"suoxie": "BSZZZQZQA", "name": "\u535a\u65f6\u8f6c\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "BOSHIZHUANZHAIZENGQIANGZHAIQUANA"}, "050020": {"suoxie": "BSKTZZQHB", "name": "\u535a\u65f6\u6297\u901a\u80c0\u589e\u5f3a\u56de\u62a5", "style": "QDII", "pinyin": "BOSHIKANGTONGZHANGZENGQIANGHUIBAO"}, "050021": {"suoxie": "BSCYBETFLJA", "name": "\u535a\u65f6\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHICHUANGYEBANETFLIANJIEA"}, "050022": {"suoxie": "BSHBHH", "name": "\u535a\u65f6\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIHUIBAOHUNHE"}, "050023": {"suoxie": "BSTYZQA", "name": "\u535a\u65f6\u5929\u9890\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHITIANYIZHAIQUANA"}, "050024": {"suoxie": "BSSZZRZYETFLJ", "name": "\u535a\u65f6\u4e0a\u8bc1\u81ea\u7136\u8d44\u6e90ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENGZIRANZIYUANETFLIANJIE"}, "050025": {"suoxie": "BSBP500ETFLJA", "name": "\u535a\u65f6\u6807\u666e500ETF\u8054\u63a5A", "style": "QDII", "pinyin": "BOSHIBIAOPU500ETFLIANJIEA"}, "050026": {"suoxie": "BSYLBJHYHHA", "name": "\u535a\u65f6\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIYILIAOBAOJIANHANGYEHUNHEA"}, "050027": {"suoxie": "BSXYZCZZQA", "name": "\u535a\u65f6\u4fe1\u7528\u503a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIXINYONGZHAICHUNZHAIZHAIQUANA"}, "050030": {"suoxie": "BSYZPXSYZQ", "name": "\u535a\u65f6\u4e9a\u6d32\u7968\u606f\u6536\u76ca\u503a\u5238", "style": "QDII", "pinyin": "BOSHIYAZHOUPIAOXISHOUYIZHAIQUAN"}, "050106": {"suoxie": "BSWDJZZQA", "name": "\u535a\u65f6\u7a33\u5b9a\u4ef7\u503c\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIWENDINGJIAZHIZHAIQUANA"}, "050111": {"suoxie": "BSXYZQC", "name": "\u535a\u65f6\u4fe1\u7528\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIXINYONGZHAIQUANC"}, "050116": {"suoxie": "BSHGHBZQC", "name": "\u535a\u65f6\u5b8f\u89c2\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHONGGUANHUIBAOZHAIQUANC"}, "050119": {"suoxie": "BSZZZQZQC", "name": "\u535a\u65f6\u8f6c\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "BOSHIZHUANZHAIZENGQIANGZHAIQUANC"}, "050123": {"suoxie": "BSTYZQC", "name": "\u535a\u65f6\u5929\u9890\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHITIANYIZHAIQUANC"}, "050201": {"suoxie": "BSJZZZEHHH", "name": "\u535a\u65f6\u4ef7\u503c\u589e\u957f\u8d30\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIAZHIZENGZHANGERHAOHUNHE"}, "050202": {"suoxie": "BSYZPXSYZQXH", "name": "\u535a\u65f6\u4e9a\u6d32\u7968\u606f\u6536\u76ca\u503a\u5238\u73b0\u6c47", "style": "QDII", "pinyin": "BOSHIYAZHOUPIAOXISHOUYIZHAIQUANXIANHUI"}, "050203": {"suoxie": "BSYZPXSYZQXC", "name": "\u535a\u65f6\u4e9a\u6d32\u7968\u606f\u6536\u76ca\u503a\u5238\u73b0\u949e", "style": "QDII", "pinyin": "BOSHIYAZHOUPIAOXISHOUYIZHAIQUANXIANCHAO"}, "051001": {"suoxie": "BSJZZZHH", "name": "\u535a\u65f6\u4ef7\u503c\u589e\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIAZHIZENGZHANGHUNHE"}, "051010": {"suoxie": "BSTXJZHHA", "name": "\u535a\u65f6\u7279\u8bb8\u4ef7\u503c\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHITEXUJIAZHIHUNHEA"}, "051011": {"suoxie": "BSXYZQAB", "name": "\u535a\u65f6\u4fe1\u7528\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIXINYONGZHAIQUANAB"}, "051014": {"suoxie": "BSCYCZHHA", "name": "\u535a\u65f6\u521b\u4e1a\u6210\u957f\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGYECHENGZHANGHUNHEA"}, "051016": {"suoxie": "BSHGHBZQAB", "name": "\u535a\u65f6\u5b8f\u89c2\u56de\u62a5\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIHONGGUANHUIBAOZHAIQUANAB"}, "051106": {"suoxie": "BSWDJZZQA", "name": "\u535a\u65f6\u7a33\u5b9a\u4ef7\u503c\u503a\u5238A(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIWENDINGJIAZHIZHAIQUANA"}, "051201": {"suoxie": "BSJZZZEHHH", "name": "\u535a\u65f6\u4ef7\u503c\u589e\u957f\u8d30\u53f7\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIJIAZHIZENGZHANGERHAOHUNHE"}, "070001": {"suoxie": "JSCZSYHHA", "name": "\u5609\u5b9e\u6210\u957f\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHICHENGZHANGSHOUYIHUNHEA"}, "070002": {"suoxie": "JSZZHH", "name": "\u5609\u5b9e\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIZENGZHANGHUNHE"}, "070003": {"suoxie": "JSWJHH", "name": "\u5609\u5b9e\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIASHIWENJIANHUNHE"}, "070005": {"suoxie": "JSZQ", "name": "\u5609\u5b9e\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIZHAIQUAN"}, "070006": {"suoxie": "JSFWZZHYHH", "name": "\u5609\u5b9e\u670d\u52a1\u589e\u503c\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIFUWUZENGZHIHANGYEHUNHE"}, "070008": {"suoxie": "JSHBA", "name": "\u5609\u5b9e\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOBIA"}, "070009": {"suoxie": "JSCDZZQC", "name": "\u5609\u5b9e\u8d85\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIASHICHAODUANZHAIZHAIQUANC"}, "070010": {"suoxie": "JSZTHH", "name": "\u5609\u5b9e\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIZHUTIHUNHE"}, "070011": {"suoxie": "JSCLHH", "name": "\u5609\u5b9e\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICELUEHUNHE"}, "070012": {"suoxie": "JSHWZGGPHH", "name": "\u5609\u5b9e\u6d77\u5916\u4e2d\u56fd\u80a1\u7968\u6df7\u5408", "style": "QDII", "pinyin": "JIASHIHAIWAIZHONGGUOGUPIAOHUNHE"}, "070013": {"suoxie": "JSYJJXHHA", "name": "\u5609\u5b9e\u7814\u7a76\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYANJIUJINGXUANHUNHEA"}, "070015": {"suoxie": "JSDYZQA", "name": "\u5609\u5b9e\u591a\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIDUOYUANZHAIQUANA"}, "070016": {"suoxie": "JSDYZQB", "name": "\u5609\u5b9e\u591a\u5143\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIDUOYUANZHAIQUANB"}, "070017": {"suoxie": "JSLHAEFHH", "name": "\u5609\u5b9e\u91cf\u5316\u963f\u5c14\u6cd5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILIANGHUAAERFAHUNHE"}, "070018": {"suoxie": "JSHBHH", "name": "\u5609\u5b9e\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIHUIBAOHUNHE"}, "070019": {"suoxie": "JSJZYSHH", "name": "\u5609\u5b9e\u4ef7\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIJIAZHIYOUSHIHUNHE"}, "070020": {"suoxie": "JSWGSYZQC", "name": "\u5609\u5b9e\u7a33\u56fa\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIWENGUSHOUYIZHAIQUANC"}, "070021": {"suoxie": "JSZTXDLHH", "name": "\u5609\u5b9e\u4e3b\u9898\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIZHUTIXINDONGLIHUNHE"}, "070022": {"suoxie": "JSLXCZHH", "name": "\u5609\u5b9e\u9886\u5148\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHILINGXIANCHENGZHANGHUNHE"}, "070023": {"suoxie": "JSSZJBM120LJA", "name": "\u5609\u5b9e\u6df1\u8bc1\u57fa\u672c\u9762120\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHISHENZHENGJIBENMIAN120LIANJIEA"}, "070025": {"suoxie": "JSXYZQA", "name": "\u5609\u5b9e\u4fe1\u7528\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIXINYONGZHAIQUANA"}, "070026": {"suoxie": "JSXYZQC", "name": "\u5609\u5b9e\u4fe1\u7528\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIXINYONGZHAIQUANC"}, "070027": {"suoxie": "JSZQYXHH", "name": "\u5609\u5b9e\u5468\u671f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIZHOUQIYOUXUANHUNHE"}, "070028": {"suoxie": "JSAXHBA", "name": "\u5609\u5b9e\u5b89\u5fc3\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIANXINHUOBIA"}, "070029": {"suoxie": "JSAXHBB", "name": "\u5609\u5b9e\u5b89\u5fc3\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIANXINHUOBIB"}, "070030": {"suoxie": "JSZC400ETFLJA", "name": "\u5609\u5b9e\u4e2d\u521b400ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGCHUANG400ETFLIANJIEA"}, "070031": {"suoxie": "JSQQFDCQDII", "name": "\u5609\u5b9e\u5168\u7403\u623f\u5730\u4ea7(QDII)", "style": "QDII", "pinyin": "JIASHIQUANQIUFANGDICHANQDII"}, "070032": {"suoxie": "JSYHHLHHA", "name": "\u5609\u5b9e\u4f18\u5316\u7ea2\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUHUAHONGLIHUNHEA"}, "070037": {"suoxie": "JSCZZQA", "name": "\u5609\u5b9e\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHICHUNZHAIZHAIQUANA"}, "070038": {"suoxie": "JSCZZQC", "name": "\u5609\u5b9e\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHICHUNZHAIZHAIQUANC"}, "070039": {"suoxie": "JSZZ500ETFLJC", "name": "\u5609\u5b9e\u4e2d\u8bc1500ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500ETFLIANJIEC"}, "070088": {"suoxie": "JSHBB", "name": "\u5609\u5b9e\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIHUOBIB"}, "070099": {"suoxie": "JSYZQYHH", "name": "\u5609\u5b9e\u4f18\u8d28\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIYOUZHIQIYEHUNHE"}, "080001": {"suoxie": "CSCZJZHHA", "name": "\u957f\u76db\u6210\u957f\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "CHANGSHENGCHENGZHANGJIAZHIHUNHEA"}, "080002": {"suoxie": "CSCXXFHHA", "name": "\u957f\u76db\u521b\u65b0\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGCHUANGXINXIANFENGHUNHEA"}, "080003": {"suoxie": "CSJJPZZQ", "name": "\u957f\u76db\u79ef\u6781\u914d\u7f6e\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGSHENGJIJIPEIZHIZHAIQUAN"}, "080005": {"suoxie": "CSLHHLHH", "name": "\u957f\u76db\u91cf\u5316\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGLIANGHUAHONGLIHUNHE"}, "080006": {"suoxie": "CSHQHYHHQDII", "name": "\u957f\u76db\u73af\u7403\u884c\u4e1a\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "CHANGSHENGHUANQIUHANGYEHUNHEQDII"}, "080007": {"suoxie": "CSTXHYPZHHA", "name": "\u957f\u76db\u540c\u946b\u884c\u4e1a\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGTONGXINHANGYEPEIZHIHUNHEA"}, "080008": {"suoxie": "CSZLXXCYHHA", "name": "\u957f\u76db\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGZHANLUEXINXINGCHANYEHUNHEA"}, "080011": {"suoxie": "CSHBA", "name": "\u957f\u76db\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGSHENGHUOBIA"}, "080012": {"suoxie": "CSDZXXCYHH", "name": "\u957f\u76db\u7535\u5b50\u4fe1\u606f\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGDIANZIXINXICHANYEHUNHE"}, "080015": {"suoxie": "CSZXPJXHH", "name": "\u957f\u76db\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGZHONGXIAOPANJINGXUANHUNHE"}, "090001": {"suoxie": "DCJZZZHH", "name": "\u5927\u6210\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJIAZHIZENGZHANGHUNHE"}, "090002": {"suoxie": "DCZQAB", "name": "\u5927\u6210\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGZHAIQUANAB"}, "090003": {"suoxie": "DCLCWJHH", "name": "\u5927\u6210\u84dd\u7b79\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGLANCHOUWENJIANHUNHE"}, "090004": {"suoxie": "DCJXZZHH", "name": "\u5927\u6210\u7cbe\u9009\u589e\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGXUANZENGZHIHUNHE"}, "090005": {"suoxie": "DCHBA", "name": "\u5927\u6210\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHUOBIA"}, "090006": {"suoxie": "DC2020SMZQHH", "name": "\u5927\u62102020\u751f\u547d\u5468\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENG2020SHENGMINGZHOUQIHUNHE"}, "090007": {"suoxie": "DCCLHBHH", "name": "\u5927\u6210\u7b56\u7565\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCELUEHUIBAOHUNHE"}, "090009": {"suoxie": "DCHYLDHH", "name": "\u5927\u6210\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHANGYELUNDONGHUNHE"}, "090010": {"suoxie": "DCZZHLZSA", "name": "\u5927\u6210\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENGHONGLIZHISHUA"}, "090011": {"suoxie": "DCHXSDLHH", "name": "\u5927\u6210\u6838\u5fc3\u53cc\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGHEXINSHUANGDONGLIHUNHE"}, "090012": {"suoxie": "DCSZCZ40ETFLJ", "name": "\u5927\u6210\u6df1\u8bc1\u6210\u957f40ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGSHENZHENGCHENGZHANG40ETFLIANJIE"}, "090013": {"suoxie": "DCJZYSHH", "name": "\u5927\u6210\u7ade\u4e89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGZHENGYOUSHIHUNHE"}, "090015": {"suoxie": "DCNXZZHHA", "name": "\u5927\u6210\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGNEIXUZENGZHANGHUNHEA"}, "090016": {"suoxie": "DCXFZTHH", "name": "\u5927\u6210\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGXIAOFEIZHUTIHUNHE"}, "090017": {"suoxie": "DCKZZZQZQ", "name": "\u5927\u6210\u53ef\u8f6c\u503a\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "DACHENGKEZHUANZHAIZENGQIANGZHAIQUAN"}, "090018": {"suoxie": "DCXRCYHH", "name": "\u5927\u6210\u65b0\u9510\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGXINRUICHANYEHUNHE"}, "090019": {"suoxie": "DCJHHHA", "name": "\u5927\u6210\u666f\u6052\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGHENGHUNHEA"}, "090020": {"suoxie": "DCJKCYHH", "name": "\u5927\u6210\u5065\u5eb7\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJIANKANGCHANYEHUNHE"}, "090021": {"suoxie": "DCYTLYGYGDCYZDZA", "name": "\u5927\u6210\u6708\u6dfb\u5229\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGYUETIANLIYIGEYUEGUNDONGCHIYOUZHONGDUANZHAIA"}, "090022": {"suoxie": "DCXJZLHBA", "name": "\u5927\u6210\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGXIANJINZENGLIHUOBIA"}, "090023": {"suoxie": "DCAHJRZC", "name": "\u5927\u6210\u5b89\u6c47\u91d1\u878d\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGANHUIJINRONGZHAIC"}, "091001": {"suoxie": "DCJZZZHH", "name": "\u5927\u6210\u4ef7\u503c\u589e\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJIAZHIZENGZHANGHUNHE"}, "091002": {"suoxie": "DCZQAB", "name": "\u5927\u6210\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGZHAIQUANAB"}, "091003": {"suoxie": "DCLCWJHH", "name": "\u5927\u6210\u84dd\u7b79\u7a33\u5065\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGLANCHOUWENJIANHUNHE"}, "091005": {"suoxie": "DCHBB", "name": "\u5927\u6210\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGHUOBIB"}, "091006": {"suoxie": "DC2020SMZQHH", "name": "\u5927\u62102020\u751f\u547d\u5468\u671f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DACHENG2020SHENGMINGZHOUQIHUNHE"}, "091021": {"suoxie": "DCYTLYGYGDCYZDZB", "name": "\u5927\u6210\u6708\u6dfb\u5229\u4e00\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DACHENGYUETIANLIYIGEYUEGUNDONGCHIYOUZHONGDUANZHAIB"}, "091022": {"suoxie": "DCXJZLHBB", "name": "\u5927\u6210\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGXIANJINZENGLIHUOBIB"}, "091023": {"suoxie": "DCAHJRZA", "name": "\u5927\u6210\u5b89\u6c47\u91d1\u878d\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DACHENGANHUIJINRONGZHAIA"}, "092002": {"suoxie": "DCZQC", "name": "\u5927\u6210\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DACHENGZHAIQUANC"}, "096001": {"suoxie": "DCBP500DQZZS", "name": "\u5927\u6210\u6807\u666e500\u7b49\u6743\u91cd\u6307\u6570", "style": "QDII", "pinyin": "DACHENGBIAOPU500DENGQUANZHONGZHISHU"}, "100016": {"suoxie": "FGTYHGSPHHHA", "name": "\u5bcc\u56fd\u5929\u6e90\u6caa\u6e2f\u6df1\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOTIANYUANHUGANGSHENPINGHENGHUNHEA"}, "100017": {"suoxie": "FGTYHGSPHHHA", "name": "\u5bcc\u56fd\u5929\u6e90\u6caa\u6e2f\u6df1\u5e73\u8861\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "FUGUOTIANYUANHUGANGSHENPINGHENGHUNHEA"}, "100018": {"suoxie": "FGTLZZZQ", "name": "\u5bcc\u56fd\u5929\u5229\u589e\u957f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANLIZENGZHANGZHAIQUAN"}, "100019": {"suoxie": "FGTLZZZQ", "name": "\u5bcc\u56fd\u5929\u5229\u589e\u957f\u503a\u5238(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANLIZENGZHANGZHAIQUAN"}, "100020": {"suoxie": "FGTYJZHHA", "name": "\u5bcc\u56fd\u5929\u76ca\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANYIJIAZHIHUNHEA"}, "100021": {"suoxie": "FGTYJZHHA", "name": "\u5bcc\u56fd\u5929\u76ca\u4ef7\u503c\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANYIJIAZHIHUNHEA"}, "100022": {"suoxie": "FGTRQSHH", "name": "\u5bcc\u56fd\u5929\u745e\u5f3a\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANRUIQIANGSHIHUNHE"}, "100023": {"suoxie": "FGTRQSHH", "name": "\u5bcc\u56fd\u5929\u745e\u5f3a\u52bf\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANRUIQIANGSHIHUNHE"}, "100025": {"suoxie": "FGTSHBA", "name": "\u5bcc\u56fd\u5929\u65f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOTIANSHIHUOBIA"}, "100026": {"suoxie": "FGTHWJYXHH", "name": "\u5bcc\u56fd\u5929\u5408\u7a33\u5065\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHEWENJIANYOUXUANHUNHE"}, "100027": {"suoxie": "FGTHWJYXHH", "name": "\u5bcc\u56fd\u5929\u5408\u7a33\u5065\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHEWENJIANYOUXUANHUNHE"}, "100028": {"suoxie": "FGTSHBB", "name": "\u5bcc\u56fd\u5929\u65f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOTIANSHIHUOBIB"}, "100029": {"suoxie": "FGTCHLHH", "name": "\u5bcc\u56fd\u5929\u6210\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOTIANCHENGHONGLIHUNHE"}, "100030": {"suoxie": "FGTCHLHH", "name": "\u5bcc\u56fd\u5929\u6210\u7ea2\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOTIANCHENGHONGLIHUNHE"}, "100032": {"suoxie": "FGZZHLZSZQA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHONGLIZHISHUZENGQIANGA"}, "100033": {"suoxie": "FGZZHLZSZQA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570\u589e\u5f3aA(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHONGLIZHISHUZENGQIANGA"}, "100035": {"suoxie": "FGYHZQZQAB", "name": "\u5bcc\u56fd\u4f18\u5316\u589e\u5f3a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOYOUHUAZENGQIANGZHAIQUANAB"}, "100036": {"suoxie": "FGYHZQZQAB", "name": "\u5bcc\u56fd\u4f18\u5316\u589e\u5f3a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOYOUHUAZENGQIANGZHAIQUANAB"}, "100037": {"suoxie": "FGYHZQZQC", "name": "\u5bcc\u56fd\u4f18\u5316\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOYOUHUAZENGQIANGZHAIQUANC"}, "100038": {"suoxie": "FGHS300ZSZQA", "name": "\u5bcc\u56fd\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOHUSHEN300ZHISHUZENGQIANGA"}, "100039": {"suoxie": "FGTZTSZTLDHH", "name": "\u5bcc\u56fd\u901a\u80c0\u901a\u7f29\u4e3b\u9898\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTONGZHANGTONGSUOZHUTILUNDONGHUNHE"}, "100050": {"suoxie": "FGQQZQQDII", "name": "\u5bcc\u56fd\u5168\u7403\u503a\u5238(QDII)", "style": "QDII", "pinyin": "FUGUOQUANQIUZHAIQUANQDII"}, "100051": {"suoxie": "FGKZZA", "name": "\u5bcc\u56fd\u53ef\u8f6c\u503aA", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "FUGUOKEZHUANZHAIA"}, "100052": {"suoxie": "FGKZZA", "name": "\u5bcc\u56fd\u53ef\u8f6c\u503aA(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "FUGUOKEZHUANZHAIA"}, "100053": {"suoxie": "FGSZZSETFLJA", "name": "\u5bcc\u56fd\u4e0a\u8bc1\u6307\u6570ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOSHANGZHENGZHISHUETFLIANJIEA"}, "100055": {"suoxie": "FGQQKJHLWQDII", "name": "\u5bcc\u56fd\u5168\u7403\u79d1\u6280\u4e92\u8054\u7f51(QDII)", "style": "QDII", "pinyin": "FUGUOQUANQIUKEJIHULIANWANGQDII"}, "100056": {"suoxie": "FGDTHBHH", "name": "\u5bcc\u56fd\u4f4e\u78b3\u73af\u4fdd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUODITANHUANBAOHUNHE"}, "100058": {"suoxie": "FGCYZQA", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHANYEZHAIQUANA"}, "100059": {"suoxie": "FGCYZQA", "name": "\u5bcc\u56fd\u4ea7\u4e1a\u503a\u5238A(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHANYEZHAIQUANA"}, "100060": {"suoxie": "FGGXJSCYHH", "name": "\u5bcc\u56fd\u9ad8\u65b0\u6280\u672f\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOGAOXINJISHUCHANYEHUNHE"}, "100061": {"suoxie": "FGZGZXPHHQDIIRMB", "name": "\u5bcc\u56fd\u4e2d\u56fd\u4e2d\u5c0f\u76d8\u6df7\u5408(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "FUGUOZHONGGUOZHONGXIAOPANHUNHEQDIIRENMINBI"}, "100066": {"suoxie": "FGCZZQFQAB", "name": "\u5bcc\u56fd\u7eaf\u503a\u503a\u5238\u53d1\u8d77A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHUNZHAIZHAIQUANFAQIAB"}, "100067": {"suoxie": "FGCZZQFQAB", "name": "\u5bcc\u56fd\u7eaf\u503a\u503a\u5238\u53d1\u8d77A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHUNZHAIZHAIQUANFAQIAB"}, "100068": {"suoxie": "FGCZZQFQC", "name": "\u5bcc\u56fd\u7eaf\u503a\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOCHUNZHAIZHAIQUANFAQIC"}, "100072": {"suoxie": "FGQHBDKZAB", "name": "\u5bcc\u56fd\u5f3a\u56de\u62a5\u5b9a\u5f00\u503aA/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOQIANGHUIBAODINGKAIZHAIAB"}, "100073": {"suoxie": "FGQHBDKZC", "name": "\u5bcc\u56fd\u5f3a\u56de\u62a5\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOQIANGHUIBAODINGKAIZHAIC"}, "110001": {"suoxie": "YFDPWZZHH", "name": "\u6613\u65b9\u8fbe\u5e73\u7a33\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YIFANGDAPINGWENZENGZHANGHUNHE"}, "110002": {"suoxie": "YFDCLCZHH", "name": "\u6613\u65b9\u8fbe\u7b56\u7565\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACELUECHENGZHANGHUNHE"}, "110003": {"suoxie": "YFDSZ50ZQA", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ZENGQIANGA"}, "110005": {"suoxie": "YFDJJCZHH", "name": "\u6613\u65b9\u8fbe\u79ef\u6781\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAJIJICHENGZHANGHUNHE"}, "110006": {"suoxie": "YFDHBA", "name": "\u6613\u65b9\u8fbe\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAHUOBIA"}, "110007": {"suoxie": "YFDWJSYZQA", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAWENJIANSHOUYIZHAIQUANA"}, "110008": {"suoxie": "YFDWJSYZQB", "name": "\u6613\u65b9\u8fbe\u7a33\u5065\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAWENJIANSHOUYIZHAIQUANB"}, "110009": {"suoxie": "YFDJZJXHH", "name": "\u6613\u65b9\u8fbe\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAJIAZHIJINGXUANHUNHE"}, "110010": {"suoxie": "YFDJZCZHH", "name": "\u6613\u65b9\u8fbe\u4ef7\u503c\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAJIAZHICHENGZHANGHUNHE"}, "110011": {"suoxie": "YFDYZJXHHQDII", "name": "\u6613\u65b9\u8fbe\u4f18\u8d28\u7cbe\u9009\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "YIFANGDAYOUZHIJINGXUANHUNHEQDII"}, "110012": {"suoxie": "YFDKHLHPZHH", "name": "\u6613\u65b9\u8fbe\u79d1\u6c47\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAKEHUILINGHUOPEIZHIHUNHE"}, "110013": {"suoxie": "YFDKXHH", "name": "\u6613\u65b9\u8fbe\u79d1\u7fd4\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKEXIANGHUNHE"}, "110015": {"suoxie": "YFDHYLXHH", "name": "\u6613\u65b9\u8fbe\u884c\u4e1a\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAHANGYELINGXIANHUNHE"}, "110016": {"suoxie": "YFDHBB", "name": "\u6613\u65b9\u8fbe\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAHUOBIB"}, "110017": {"suoxie": "YFDZQHBZQA", "name": "\u6613\u65b9\u8fbe\u589e\u5f3a\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAZENGQIANGHUIBAOZHAIQUANA"}, "110018": {"suoxie": "YFDZQHBZQB", "name": "\u6613\u65b9\u8fbe\u589e\u5f3a\u56de\u62a5\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAZENGQIANGHUIBAOZHAIQUANB"}, "110019": {"suoxie": "YFDSZ100ETFLJA", "name": "\u6613\u65b9\u8fbe\u6df1\u8bc1100ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHENZHENG100ETFLIANJIEA"}, "110020": {"suoxie": "YFDHS300ETFLJA", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300ETFLIANJIEA"}, "110021": {"suoxie": "YFDSZZPETFLJA", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc1\u4e2d\u76d8ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENGZHONGPANETFLIANJIEA"}, "110022": {"suoxie": "YFDXFHYGP", "name": "\u6613\u65b9\u8fbe\u6d88\u8d39\u884c\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YIFANGDAXIAOFEIHANGYEGUPIAO"}, "110023": {"suoxie": "YFDYLBJHYHH", "name": "\u6613\u65b9\u8fbe\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAYILIAOBAOJIANHANGYEHUNHE"}, "110025": {"suoxie": "YFDZYHYHH", "name": "\u6613\u65b9\u8fbe\u8d44\u6e90\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAZIYUANHANGYEHUNHE"}, "110026": {"suoxie": "YFDCYBETFLJA", "name": "\u6613\u65b9\u8fbe\u521b\u4e1a\u677fETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDACHUANGYEBANETFLIANJIEA"}, "110027": {"suoxie": "YFDAXHBZQA", "name": "\u6613\u65b9\u8fbe\u5b89\u5fc3\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAANXINHUIBAOZHAIQUANA"}, "110028": {"suoxie": "YFDAXHBZQB", "name": "\u6613\u65b9\u8fbe\u5b89\u5fc3\u56de\u62a5\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDAANXINHUIBAOZHAIQUANB"}, "110029": {"suoxie": "YFDKXHH", "name": "\u6613\u65b9\u8fbe\u79d1\u8baf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKEXUNHUNHE"}, "110030": {"suoxie": "YFDHS300LHZQ", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u91cf\u5316\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300LIANGHUAZENGQIANG"}, "110031": {"suoxie": "YFDHSGQETFLJA", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u56fd\u4f01ETF\u8054\u63a5A", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGGUOQIETFLIANJIEA"}, "110032": {"suoxie": "YFDHSGQETFLJXHA", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u56fd\u4f01ETF\u8054\u63a5\u73b0\u6c47A", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGGUOQIETFLIANJIEXIANHUIA"}, "110033": {"suoxie": "YFDHSGQETFLJXCA", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u56fd\u4f01ETF\u8054\u63a5\u73b0\u949eA", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGGUOQIETFLIANJIEXIANCHAOA"}, "110035": {"suoxie": "YFDSZZQZQA", "name": "\u6613\u65b9\u8fbe\u53cc\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDASHUANGZHAIZENGQIANGZHAIQUANA"}, "110036": {"suoxie": "YFDSZZQZQC", "name": "\u6613\u65b9\u8fbe\u53cc\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDASHUANGZHAIZENGQIANGZHAIQUANC"}, "110037": {"suoxie": "YFDCZZQA", "name": "\u6613\u65b9\u8fbe\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDACHUNZHAIZHAIQUANA"}, "110038": {"suoxie": "YFDCZZQC", "name": "\u6613\u65b9\u8fbe\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDACHUNZHAIZHAIQUANC"}, "110050": {"suoxie": "YFDAHZDZC", "name": "\u6613\u65b9\u8fbe\u5b89\u548c\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANHEZHONGDUANZHAIC"}, "110051": {"suoxie": "YFDAHZDZA", "name": "\u6613\u65b9\u8fbe\u5b89\u548c\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANHEZHONGDUANZHAIA"}, "110052": {"suoxie": "YFDAYZDZZQC", "name": "\u6613\u65b9\u8fbe\u5b89\u6e90\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANYUANZHONGDUANZHAIZHAIQUANC"}, "110053": {"suoxie": "YFDAYZDZZQA", "name": "\u6613\u65b9\u8fbe\u5b89\u6e90\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "YIFANGDAANYUANZHONGDUANZHAIZHAIQUANA"}, "112002": {"suoxie": "YFDCLCZEHHH", "name": "\u6613\u65b9\u8fbe\u7b56\u7565\u6210\u957f\u4e8c\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACELUECHENGZHANGERHAOHUNHE"}, "118001": {"suoxie": "YFDYZJXGP", "name": "\u6613\u65b9\u8fbe\u4e9a\u6d32\u7cbe\u9009\u80a1\u7968", "style": "QDII", "pinyin": "YIFANGDAYAZHOUJINGXUANGUPIAO"}, "118002": {"suoxie": "YFDBPXFPZSA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u6d88\u8d39\u54c1\u6307\u6570A", "style": "QDII", "pinyin": "YIFANGDABIAOPUXIAOFEIPINZHISHUA"}, "121001": {"suoxie": "GTRYRHZQ", "name": "\u56fd\u6295\u745e\u94f6\u878d\u534e\u503a\u5238", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINRONGHUAZHAIQUAN"}, "121002": {"suoxie": "GTRYJQHYHH", "name": "\u56fd\u6295\u745e\u94f6\u666f\u6c14\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOTOURUIYINJINGQIHANGYEHUNHE"}, "121003": {"suoxie": "GTRYHXQYHH", "name": "\u56fd\u6295\u745e\u94f6\u6838\u5fc3\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINHEXINQIYEHUNHE"}, "121005": {"suoxie": "GTRYCXDLHH", "name": "\u56fd\u6295\u745e\u94f6\u521b\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHUANGXINDONGLIHUNHE"}, "121006": {"suoxie": "GTRYWJZZHH", "name": "\u56fd\u6295\u745e\u94f6\u7a33\u5065\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINWENJIANZENGZHANGHUNHE"}, "121008": {"suoxie": "GTRYCZYXHH", "name": "\u56fd\u6295\u745e\u94f6\u6210\u957f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHENGZHANGYOUXUANHUNHE"}, "121009": {"suoxie": "GTRYWDZLZQ", "name": "\u56fd\u6295\u745e\u94f6\u7a33\u5b9a\u589e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINWENDINGZENGLIZHAIQUAN"}, "121010": {"suoxie": "GTRYRYLHPZHH", "name": "\u56fd\u6295\u745e\u94f6\u745e\u6e90\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUIYUANLINGHUOPEIZHIHUNHE"}, "121011": {"suoxie": "GTRYHBA", "name": "\u56fd\u6295\u745e\u94f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINHUOBIA"}, "121012": {"suoxie": "GTRYYHZQZQAB", "name": "\u56fd\u6295\u745e\u94f6\u4f18\u5316\u589e\u5f3a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINYOUHUAZENGQIANGZHAIQUANAB"}, "128001": {"suoxie": "GTRYRHZQ", "name": "\u56fd\u6295\u745e\u94f6\u878d\u534e\u503a\u5238(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOTOURUIYINRONGHUAZHAIQUAN"}, "128002": {"suoxie": "GTRYJQHYHH", "name": "\u56fd\u6295\u745e\u94f6\u666f\u6c14\u884c\u4e1a\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOTOURUIYINJINGQIHANGYEHUNHE"}, "128005": {"suoxie": "GTRYCXDLHH", "name": "\u56fd\u6295\u745e\u94f6\u521b\u65b0\u52a8\u529b\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHUANGXINDONGLIHUNHE"}, "128006": {"suoxie": "GTRYWJZZHH", "name": "\u56fd\u6295\u745e\u94f6\u7a33\u5065\u589e\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINWENJIANZENGZHANGHUNHE"}, "128008": {"suoxie": "GTRYCZYXHH", "name": "\u56fd\u6295\u745e\u94f6\u6210\u957f\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINCHENGZHANGYOUXUANHUNHE"}, "128011": {"suoxie": "GTRYHBB", "name": "\u56fd\u6295\u745e\u94f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTOURUIYINHUOBIB"}, "128012": {"suoxie": "GTRYYHZQZQAB", "name": "\u56fd\u6295\u745e\u94f6\u4f18\u5316\u589e\u5f3a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINYOUHUAZENGQIANGZHAIQUANAB"}, "128112": {"suoxie": "GTRYYHZQZQC", "name": "\u56fd\u6295\u745e\u94f6\u4f18\u5316\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINYOUHUAZENGQIANGZHAIQUANC"}, "150005": {"suoxie": "YHYFHBA", "name": "\u94f6\u6cb3\u94f6\u5bcc\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHEYINFUHUOBIA"}, "150015": {"suoxie": "YHYFHBB", "name": "\u94f6\u6cb3\u94f6\u5bcc\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHEYINFUHUOBIB"}, "150103": {"suoxie": "YHYTHH", "name": "\u94f6\u6cb3\u94f6\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEYINTAIHUNHE"}, "150968": {"suoxie": "YHYJJXHH", "name": "\u94f6\u6cb3\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEYANJIUJINGXUANHUNHE"}, "150988": {"suoxie": "YHQBHBA", "name": "\u94f6\u6cb3\u94b1\u5305\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHEQIANBAOHUOBIA"}, "150998": {"suoxie": "YHQBHBB", "name": "\u94f6\u6cb3\u94b1\u5305\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHEQIANBAOHUOBIB"}, "151001": {"suoxie": "YHWJHH", "name": "\u94f6\u6cb3\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YINHEWENJIANHUNHE"}, "151002": {"suoxie": "YHSYHH", "name": "\u94f6\u6cb3\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHESHOUYIHUNHE"}, "159001": {"suoxie": "YFDBZJHBA", "name": "\u6613\u65b9\u8fbe\u4fdd\u8bc1\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDABAOZHENGJINHUOBIA"}, "159002": {"suoxie": "YFDBZJHBB", "name": "\u6613\u65b9\u8fbe\u4fdd\u8bc1\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDABAOZHENGJINHUOBIB"}, "159003": {"suoxie": "ZSBZJKXA", "name": "\u62db\u5546\u4fdd\u8bc1\u91d1\u5feb\u7ebfA", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGBAOZHENGJINKUAIXIANA"}, "159004": {"suoxie": "ZSBZJKXB", "name": "\u62db\u5546\u4fdd\u8bc1\u91d1\u5feb\u7ebfB", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGBAOZHENGJINKUAIXIANB"}, "159005": {"suoxie": "HTFSYKQHBA", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u94b1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIQIANHUOBIA"}, "159006": {"suoxie": "HTFSYKQHBB", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u94b1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIQIANHUOBIB"}, "159601": {"suoxie": "HXMSCIZGA50HLHTETF", "name": "\u534e\u590fMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAMSCIZHONGGUOA50HULIANHUTONGETF"}, "159602": {"suoxie": "NFMSCIZGA50HLHTETF", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOA50HULIANHUTONGETF"}, "159605": {"suoxie": "GFHWZGHLW30QDIIETF", "name": "\u5e7f\u53d1\u6d77\u5916\u4e2d\u56fd\u4e92\u8054\u7f5130(QDII-ETF)", "style": "QDII", "pinyin": "GUANGFAHAIWAIZHONGGUOHULIANWANG30QDIIETF"}, "159606": {"suoxie": "YFDZZ500ZLCZETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500\u8d28\u91cf\u6210\u957fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ZHILIANGCHENGZHANGETF"}, "159607": {"suoxie": "JSZZHWZGHLW30ETFQDII", "name": "\u5609\u5b9e\u4e2d\u8bc1\u6d77\u5916\u4e2d\u56fd\u4e92\u8054\u7f5130ETF(QDII)", "style": "QDII", "pinyin": "JIASHIZHONGZHENGHAIWAIZHONGGUOHULIANWANG30ETFQDII"}, "159608": {"suoxie": "GFZZXYJSZTETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGXIYOUJINSHUZHUTIETF"}, "159610": {"suoxie": "JSZZ500ZQCLETF", "name": "\u666f\u987a\u4e2d\u8bc1500\u589e\u5f3a\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENG500ZENGQIANGCELUEETF"}, "159611": {"suoxie": "GFZZQZDLGYSYETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u7535\u529b\u516c\u7528\u4e8b\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIDIANLIGONGYONGSHIYEETF"}, "159613": {"suoxie": "JSZZXXAQZTETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u4fe1\u606f\u5b89\u5168\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXINXIANQUANZHUTIETF"}, "159618": {"suoxie": "HAZZGFCYETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGGUANGFUCHANYEETF"}, "159619": {"suoxie": "GTZZJJETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u57fa\u5efaETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGJIJIANETF"}, "159701": {"suoxie": "ZSZZWLWZTETF", "name": "\u62db\u5546\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGWULIANWANGZHUTIETF"}, "159702": {"suoxie": "HTFZZRGZNZTETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGRENGONGZHINENGZHUTIETF"}, "159703": {"suoxie": "THZZXCLZTETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGXINCAILIAOZHUTIETF"}, "159706": {"suoxie": "HASZ100ETF", "name": "\u534e\u5b89\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHENZHENG100ETF"}, "159707": {"suoxie": "HBZZ800DCETF", "name": "\u534e\u5b9d\u4e2d\u8bc1800\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG800DICHANETF"}, "159708": {"suoxie": "XBLDSZHLETF", "name": "\u897f\u90e8\u5229\u5f97\u6df1\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDESHENZHENGHONGLIETF"}, "159709": {"suoxie": "GYRXSZWLW50ETF", "name": "\u5de5\u94f6\u745e\u4fe1\u6df1\u8bc1\u7269\u8054\u7f5150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINSHENZHENGWULIANWANG50ETF"}, "159710": {"suoxie": "JXZZZNDDQCETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGZHINENGDIANDONGQICHEETF"}, "159711": {"suoxie": "HXZZGGT50ETF", "name": "\u534e\u590f\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGGANGGUTONG50ETF"}, "159712": {"suoxie": "GTZZGGT50ETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGGUTONG50ETF"}, "159713": {"suoxie": "FGZZXTCYETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXITUCHANYEETF"}, "159715": {"suoxie": "YFDZZXTCYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXITUCHANYEETF"}, "159716": {"suoxie": "HBSC100ETF", "name": "\u534e\u5b9d\u6df1\u521b100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHENCHUANG100ETF"}, "159717": {"suoxie": "PHGZESG300ETF", "name": "\u9e4f\u534e\u56fd\u8bc1ESG300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGESG300ETF"}, "159718": {"suoxie": "PAZZGGTYYWSZHETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u6e2f\u80a1\u901a\u533b\u836f\u536b\u751f\u7efc\u5408ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGGANGGUTONGYIYAOWEISHENGZONGHEETF"}, "159719": {"suoxie": "PAFSZGGQKFGYETF", "name": "\u5e73\u5b89\u5bcc\u65f6\u4e2d\u56fd\u56fd\u4f01\u5f00\u653e\u5171\u8d62ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANFUSHIZHONGGUOGUOQIKAIFANGGONGYINGETF"}, "159720": {"suoxie": "TKZZZNDDQCETF", "name": "\u6cf0\u5eb7\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGZHONGZHENGZHINENGDIANDONGQICHEETF"}, "159721": {"suoxie": "YYSZCX100ETF", "name": "\u6c38\u8d62\u6df1\u8bc1\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGSHENZHENGCHUANGXIN100ETF"}, "159723": {"suoxie": "HTFZZHGSKJLTETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHENKEJILONGTOUETF"}, "159725": {"suoxie": "GYRXZZXSXFETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1\u7ebf\u4e0a\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENGXIANSHANGXIAOFEIETF"}, "159726": {"suoxie": "HXHSZGNDQYGGXLETF", "name": "\u534e\u590f\u6052\u751f\u4e2d\u56fd\u5185\u5730\u4f01\u4e1a\u9ad8\u80a1\u606f\u7387ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHENGSHENGZHONGGUONEIDIQIYEGAOGUXILVETF"}, "159728": {"suoxie": "NFGZZXXFETF", "name": "\u5357\u65b9\u56fd\u8bc1\u5728\u7ebf\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGGUOZHENGZAIXIANXIAOFEIETF"}, "159729": {"suoxie": "HTFZZHGSHLWETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHENHULIANWANGETF"}, "159730": {"suoxie": "BSGZLTJDETF", "name": "\u535a\u65f6\u56fd\u8bc1\u9f99\u5934\u5bb6\u7535ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIGUOZHENGLONGTOUJIADIANETF"}, "159731": {"suoxie": "HXZZSHCYETF", "name": "\u534e\u590f\u4e2d\u8bc1\u77f3\u5316\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGSHIHUACHANYEETF"}, "159732": {"suoxie": "HXGZXFDZZTETF", "name": "\u534e\u590f\u56fd\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAGUOZHENGXIAOFEIDIANZIZHUTIETF"}, "159733": {"suoxie": "JSZZXFDZETF", "name": "\u666f\u987a\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENGXIAOFEIDIANZIETF"}, "159735": {"suoxie": "YHZZGGTXFZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "159736": {"suoxie": "THZZSPYLETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u98df\u54c1\u996e\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGSHIPINYINLIAOETF"}, "159738": {"suoxie": "HTBRZZHGSYJSCYETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e91\u8ba1\u7b97\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGHUGANGSHENYUNJISUANCHANYEETF"}, "159739": {"suoxie": "PHZZYJSYDSJZTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u4e91\u8ba1\u7b97\u4e0e\u5927\u6570\u636e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYUNJISUANYUDASHUJUZHUTIETF"}, "159740": {"suoxie": "DCHSKJETFQDII", "name": "\u5927\u6210\u6052\u751f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "DACHENGHENGSHENGKEJIETFQDII"}, "159741": {"suoxie": "JSHSKJETFQDII", "name": "\u5609\u5b9e\u6052\u751f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "JIASHIHENGSHENGKEJIETFQDII"}, "159742": {"suoxie": "BSHSKJETFQDII", "name": "\u535a\u65f6\u6052\u751f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "BOSHIHENGSHENGKEJIETFQDII"}, "159743": {"suoxie": "BSHBXJDNZHETF", "name": "\u535a\u65f6\u6e56\u5317\u65b0\u65e7\u52a8\u80fd\u8f6c\u6362ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUBEIXINJIUDONGNENGZHUANHUANETF"}, "159745": {"suoxie": "GTZZQZJZCLETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIANZHUCAILIAOETF"}, "159747": {"suoxie": "NFZZXGKJETFQDII", "name": "\u5357\u65b9\u4e2d\u8bc1\u9999\u6e2f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "NANFANGZHONGZHENGXIANGGANGKEJIETFQDII"}, "159748": {"suoxie": "FGHGSCXYCYETF", "name": "\u5bcc\u56fd\u6caa\u6e2f\u6df1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOHUGANGSHENCHUANGXINYAOCHANYEETF"}, "159750": {"suoxie": "ZSZZXGKJETFQDII", "name": "\u62db\u5546\u4e2d\u8bc1\u9999\u6e2f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "ZHAOSHANGZHONGZHENGXIANGGANGKEJIETFQDII"}, "159751": {"suoxie": "PHZZGGTKJETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u6e2f\u80a1\u901a\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGANGGUTONGKEJIETF"}, "159752": {"suoxie": "SWLXZZNDXNYZTETF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u5185\u5730\u65b0\u80fd\u6e90\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGNEIDIXINNENGYUANZHUTIETF"}, "159755": {"suoxie": "GFGZXNYCDCETF", "name": "\u5e7f\u53d1\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGXINNENGYUANCHEDIANCHIETF"}, "159757": {"suoxie": "JSCCGZXNYCDCETF", "name": "\u666f\u987a\u957f\u57ce\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGGUOZHENGXINNENGYUANCHEDIANCHIETF"}, "159758": {"suoxie": "HXZZHLZLETF", "name": "\u534e\u590f\u4e2d\u8bc1\u7ea2\u5229\u8d28\u91cfETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGHONGLIZHILIANGETF"}, "159760": {"suoxie": "TKGZGGWSYJKETF", "name": "\u6cf0\u5eb7\u56fd\u8bc1\u516c\u5171\u536b\u751f\u4e0e\u5065\u5eb7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGGUOZHENGGONGGONGWEISHENGYUJIANKANGETF"}, "159761": {"suoxie": "GTZZXCLZTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINCAILIAOZHUTIETF"}, "159763": {"suoxie": "JXZZXCLZTETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGXINCAILIAOZHUTIETF"}, "159766": {"suoxie": "FGZZLYZTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65c5\u6e38\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGLVYOUZHUTIETF"}, "159767": {"suoxie": "XYGZXNYCDCETF", "name": "\u5174\u94f6\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYINGUOZHENGXINNENGYUANCHEDIANCHIETF"}, "159768": {"suoxie": "YHZZNDDCZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u5185\u5730\u5730\u4ea7\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGNEIDIDICHANZHUTIETF"}, "159769": {"suoxie": "YHZZXFDZZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "159770": {"suoxie": "THZZJQRETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u673a\u5668\u4ebaETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGJIQIRENETF"}, "159773": {"suoxie": "HTBRCYBKJETF", "name": "\u534e\u6cf0\u67cf\u745e\u521b\u4e1a\u677f\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUICHUANGYEBANKEJIETF"}, "159775": {"suoxie": "JXGZXNYCDCETF", "name": "\u5efa\u4fe1\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINGUOZHENGXINNENGYUANCHEDIANCHIETF"}, "159776": {"suoxie": "YHZZGGTYYWSZHETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u6e2f\u80a1\u901a\u533b\u836f\u536b\u751f\u7efc\u5408ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGGANGGUTONGYIYAOWEISHENGZONGHEETF"}, "159777": {"suoxie": "GLACYBKJETF", "name": "\u56fd\u8054\u5b89\u521b\u4e1a\u677f\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANCHUANGYEBANKEJIETF"}, "159778": {"suoxie": "PHZZGYHLWZTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u5de5\u4e1a\u4e92\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGONGYEHULIANWANGZHUTIETF"}, "159779": {"suoxie": "ZSZZXFDZZTETF", "name": "\u62db\u5546\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "159780": {"suoxie": "NFZZKCCY50ETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGKECHUANGCHUANGYE50ETF"}, "159781": {"suoxie": "YFDZZKCCY50ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKECHUANGCHUANGYE50ETF"}, "159782": {"suoxie": "YHZZKCCY50ETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGKECHUANGCHUANGYE50ETF"}, "159783": {"suoxie": "HXZZKCCY50ETF", "name": "\u534e\u590f\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGKECHUANGCHUANGYE50ETF"}, "159786": {"suoxie": "YHZZXNXSZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u865a\u62df\u73b0\u5b9e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGXUNIXIANSHIZHUTIETF"}, "159787": {"suoxie": "YFDZZQZJZCLETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIJIANZHUCAILIAOETF"}, "159788": {"suoxie": "YFDZZGGTZG100ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6e2f\u80a1\u901a\u4e2d\u56fd100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGANGGUTONGZHONGGUO100ETF"}, "159789": {"suoxie": "JXZZYLZTETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u996e\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGYINLIAOZHUTIETF"}, "159790": {"suoxie": "HXZZNDDTJJZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGNEIDIDITANJINGJIZHUTIETF"}, "159791": {"suoxie": "HXHS300ESGJZETF", "name": "\u534e\u590f\u6caa\u6df1300ESG\u57fa\u51c6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ESGJIZHUNETF"}, "159792": {"suoxie": "FGZZGGTHLWETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6e2f\u80a1\u901a\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGANGGUTONGHULIANWANGETF"}, "159793": {"suoxie": "PAZZHGSXSXFZTETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u7ebf\u4e0a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGHUGANGSHENXIANSHANGXIAOFEIZHUTIETF"}, "159795": {"suoxie": "HTFZZZNQCZTETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHINENGQICHEZHUTIETF"}, "159796": {"suoxie": "HTFZZDCZTETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u7535\u6c60\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGDIANCHIZHUTIETF"}, "159797": {"suoxie": "HTFZZQZYLQXETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u5668\u68b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGQUANZHIYILIAOQIXIEETF"}, "159799": {"suoxie": "PAZZGGTXFZTETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "159801": {"suoxie": "GFGZBDTXPETF", "name": "\u5e7f\u53d1\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAGUOZHENGBANDAOTIXINPIANETF"}, "159804": {"suoxie": "GSABCJX88ETF", "name": "\u56fd\u5bff\u5b89\u4fdd\u521b\u7cbe\u900988ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOCHUANGJINGXUAN88ETF"}, "159805": {"suoxie": "PHZZCMETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u4f20\u5a92ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGCHUANMEIETF"}, "159806": {"suoxie": "GTZZXNYQCETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXINNENGYUANQICHEETF"}, "159807": {"suoxie": "YFDZZKJ50ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u79d1\u628050ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGKEJI50ETF"}, "159808": {"suoxie": "RTCYBETF", "name": "\u878d\u901a\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGCHUANGYEBANETF"}, "159810": {"suoxie": "PYASCYBETF", "name": "\u6d66\u94f6\u5b89\u76db\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGCHUANGYEBANETF"}, "159811": {"suoxie": "BSZZ5GCY50ETF", "name": "\u535a\u65f6\u4e2d\u8bc15G\u4ea7\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG5GCHANYE50ETF"}, "159812": {"suoxie": "QHKYHJETF", "name": "\u524d\u6d77\u5f00\u6e90\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "QIANHAIKAIYUANHUANGJINETF"}, "159813": {"suoxie": "PHGZBDTXPETF", "name": "\u9e4f\u534e\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGBANDAOTIXINPIANETF"}, "159814": {"suoxie": "XBLDCYBDPETF", "name": "\u897f\u90e8\u5229\u5f97\u521b\u4e1a\u677f\u5927\u76d8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDECHUANGYEBANDAPANETF"}, "159815": {"suoxie": "ZSZZZJ100ETF", "name": "\u62db\u5546\u4e2d\u8bc1\u6d59\u6c5f100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGZHEJIANG100ETF"}, "159816": {"suoxie": "PH04NDFZFZETF", "name": "\u9e4f\u534e0-4\u5e74\u5730\u65b9\u653f\u5e9c\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUA04NIANDIFANGZHENGFUZHAIETF"}, "159819": {"suoxie": "YFDZZRGZNZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGRENGONGZHINENGZHUTIETF"}, "159820": {"suoxie": "THZZ500ETF", "name": "\u5929\u5f18\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENG500ETF"}, "159821": {"suoxie": "ZYZQCYBETF", "name": "\u4e2d\u94f6\u8bc1\u5238\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHENGQUANCHUANGYEBANETF"}, "159822": {"suoxie": "YHGYNFDYBPZGXJJETFQDII", "name": "\u94f6\u534e\u5de5\u94f6\u5357\u65b9\u4e1c\u82f1\u6807\u666e\u4e2d\u56fd\u65b0\u7ecf\u6d4eETF(QDII)", "style": "QDII", "pinyin": "YINHUAGONGYINNANFANGDONGYINGBIAOPUZHONGGUOXINJINGJIETFQDII"}, "159823": {"suoxie": "JSHSZGQYETFQDII", "name": "\u5609\u5b9e\u6052\u751f\u4e2d\u56fd\u4f01\u4e1aETF(QDII)", "style": "QDII", "pinyin": "JIASHIHENGSHENGZHONGGUOQIYEETFQDII"}, "159824": {"suoxie": "BSXNYQCETF", "name": "\u535a\u65f6\u65b0\u80fd\u6e90\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIXINNENGYUANQICHEETF"}, "159825": {"suoxie": "FGZZNYZTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u519c\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGNONGYEZHUTIETF"}, "159827": {"suoxie": "YHZZNYZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u519c\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGNONGYEZHUTIETF"}, "159828": {"suoxie": "GTZZYLETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u533b\u7597ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYILIAOETF"}, "159830": {"suoxie": "THSHJETF", "name": "\u5929\u5f18\u4e0a\u6d77\u91d1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGSHANGHAIJINETF"}, "159831": {"suoxie": "JSSHJETF", "name": "\u5609\u5b9e\u4e0a\u6d77\u91d1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHISHANGHAIJINETF"}, "159832": {"suoxie": "PASHJETF", "name": "\u5e73\u5b89\u4e0a\u6d77\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "PINGANSHANGHAIJINETF"}, "159833": {"suoxie": "DCSHJETF", "name": "\u5927\u6210\u4e0a\u6d77\u91d1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGSHANGHAIJINETF"}, "159834": {"suoxie": "NFSHJETF", "name": "\u5357\u65b9\u4e0a\u6d77\u91d1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGHAIJINETF"}, "159835": {"suoxie": "JXZZCXYCYETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGCHUANGXINYAOCHANYEETF"}, "159836": {"suoxie": "THCYB300ETF", "name": "\u5929\u5f18\u521b\u4e1a\u677f300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBAN300ETF"}, "159837": {"suoxie": "YFDZZSWKJZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGSHENGWUKEJIZHUTIETF"}, "159838": {"suoxie": "BSYY50ETF", "name": "\u535a\u65f6\u533b\u836f50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYIYAO50ETF"}, "159839": {"suoxie": "HTFGZSWYYETF", "name": "\u6c47\u6dfb\u5bcc\u56fd\u8bc1\u751f\u7269\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUGUOZHENGSHENGWUYIYAOETF"}, "159840": {"suoxie": "GYRXGZXNYCDCETF", "name": "\u5de5\u94f6\u745e\u4fe1\u56fd\u8bc1\u65b0\u80fd\u6e90\u8f66\u7535\u6c60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINGUOZHENGXINNENGYUANCHEDIANCHIETF"}, "159841": {"suoxie": "THZZQZZQGSETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "159842": {"suoxie": "YHZZQZZQGSETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "159843": {"suoxie": "ZSGZSPYLETF", "name": "\u62db\u5546\u56fd\u8bc1\u98df\u54c1\u996e\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGGUOZHENGSHIPINYINLIAOETF"}, "159845": {"suoxie": "HXZZ1000ETF", "name": "\u534e\u590f\u4e2d\u8bc11000ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG1000ETF"}, "159846": {"suoxie": "JXZZ1000ETF", "name": "\u5efa\u4fe1\u4e2d\u8bc11000ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENG1000ETF"}, "159847": {"suoxie": "YFDZZYLETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u533b\u7597ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGYILIAOETF"}, "159848": {"suoxie": "GLAZZQZZQGSETF", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "159849": {"suoxie": "ZSZZSWKJZTETF", "name": "\u62db\u5546\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGSHENGWUKEJIZHUTIETF"}, "159850": {"suoxie": "HXHSZGQYETFQDII", "name": "\u534e\u590f\u6052\u751f\u4e2d\u56fd\u4f01\u4e1aETF(QDII)", "style": "QDII", "pinyin": "HUAXIAHENGSHENGZHONGGUOQIYEETFQDII"}, "159851": {"suoxie": "HBZZJRKJZTETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u91d1\u878d\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGJINRONGKEJIZHUTIETF"}, "159852": {"suoxie": "JSZZRJFWETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u8f6f\u4ef6\u670d\u52a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGRUANJIANFUWUETF"}, "159853": {"suoxie": "NFZZKJ100ETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u79d1\u6280100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGKEJI100ETF"}, "159855": {"suoxie": "YHZZYSZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u5f71\u89c6\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGYINGSHIZHUTIETF"}, "159856": {"suoxie": "GYRXZZHGSHLWETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENGHUGANGSHENHULIANWANGETF"}, "159857": {"suoxie": "THZZGFCYETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGGUANGFUCHANYEETF"}, "159858": {"suoxie": "NFZZCXYCYETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGCHUANGXINYAOCHANYEETF"}, "159859": {"suoxie": "THGZSWYYETF", "name": "\u5929\u5f18\u56fd\u8bc1\u751f\u7269\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGGUOZHENGSHENGWUYIYAOETF"}, "159861": {"suoxie": "GTZZHBCY50ETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUANBAOCHANYE50ETF"}, "159862": {"suoxie": "YHZZXFSPYLCYZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGXIFENSHIPINYINLIAOCHANYEZHUTIETF"}, "159863": {"suoxie": "PHZZGFCYETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGUANGFUCHANYEETF"}, "159864": {"suoxie": "GTZZGFCYETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGUANGFUCHANYEETF"}, "159865": {"suoxie": "GTZZXMYZETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXUMUYANGZHIETF"}, "159866": {"suoxie": "GYRXDHRJ225ETFQDII", "name": "\u5de5\u94f6\u745e\u4fe1\u5927\u548c\u65e5\u7ecf225ETF(QDII)", "style": "QDII", "pinyin": "GONGYINRUIXINDAHERIJING225ETFQDII"}, "159867": {"suoxie": "PHZZXMYZETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGXUMUYANGZHIETF"}, "159869": {"suoxie": "HXZZDMYXETF", "name": "\u534e\u590f\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGDONGMANYOUXIETF"}, "159870": {"suoxie": "PHZZXFHGCYETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGXIFENHUAGONGCHANYEETF"}, "159871": {"suoxie": "YHZZYSJSETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGYOUSEJINSHUETF"}, "159872": {"suoxie": "PHZZCLWZTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u8f66\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGCHELIANWANGZHUTIETF"}, "159873": {"suoxie": "THZZQZYLBJSBYFWETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETF"}, "159875": {"suoxie": "JSZZXNYETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXINNENGYUANETF"}, "159876": {"suoxie": "HBYSJSETF", "name": "\u534e\u5b9d\u6709\u8272\u91d1\u5c5eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOYOUSEJINSHUETF"}, "159877": {"suoxie": "NFZZQZYLBJSBYFWETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETF"}, "159880": {"suoxie": "PHGZYSJSHYETF", "name": "\u9e4f\u534e\u56fd\u8bc1\u6709\u8272\u91d1\u5c5e\u884c\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGYOUSEJINSHUHANGYEETF"}, "159881": {"suoxie": "GTZZYSJSETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6709\u8272\u91d1\u5c5eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYOUSEJINSHUETF"}, "159883": {"suoxie": "YYZZQZYLQXETF", "name": "\u6c38\u8d62\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u5668\u68b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGZHONGZHENGQUANZHIYILIAOQIXIEETF"}, "159885": {"suoxie": "PHZZNDDTJJZTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGNEIDIDITANJINGJIZHUTIETF"}, "159886": {"suoxie": "FGXFJXSBCYZTETF", "name": "\u5bcc\u56fd\u7ec6\u5206\u673a\u68b0\u8bbe\u5907\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOXIFENJIXIESHEBEICHANYEZHUTIETF"}, "159887": {"suoxie": "FGZZ800YHETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1800\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG800YINHANGETF"}, "159888": {"suoxie": "HXZZZNQCZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHINENGQICHEZHUTIETF"}, "159889": {"suoxie": "GTZZZNQCZTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGZHINENGQICHEZHUTIETF"}, "159890": {"suoxie": "ZSZZYJSETF", "name": "\u62db\u5546\u4e2d\u8bc1\u4e91\u8ba1\u7b97ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGYUNJISUANETF"}, "159891": {"suoxie": "JXZZQZYLBJSBYFWETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETF"}, "159892": {"suoxie": "HXHSXGSSSWKJETFQDII", "name": "\u534e\u590f\u6052\u751f\u9999\u6e2f\u4e0a\u5e02\u751f\u7269\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "HUAXIAHENGSHENGXIANGGANGSHANGSHISHENGWUKEJIETFQDII"}, "159895": {"suoxie": "YFDZZWLWZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGWULIANWANGZHUTIETF"}, "159896": {"suoxie": "NFZZWLWZTETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGWULIANWANGZHUTIETF"}, "159897": {"suoxie": "JXZZWLWZTETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGWULIANWANGZHUTIETF"}, "159898": {"suoxie": "ZSZZQZYLQXETF", "name": "\u62db\u5546\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u5668\u68b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGQUANZHIYILIAOQIXIEETF"}, "159899": {"suoxie": "ZSZZQZRJETF", "name": "\u62db\u5546\u4e2d\u8bc1\u5168\u6307\u8f6f\u4ef6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGQUANZHIRUANJIANETF"}, "159901": {"suoxie": "YFDSZ100ETF", "name": "\u6613\u65b9\u8fbe\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHENZHENG100ETF"}, "159902": {"suoxie": "HXZXQY100ETF", "name": "\u534e\u590f\u4e2d\u5c0f\u4f01\u4e1a100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGXIAOQIYE100ETF"}, "159903": {"suoxie": "NFSZCFETF", "name": "\u5357\u65b9\u6df1\u8bc1\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHENZHENGCHENGFENETF"}, "159905": {"suoxie": "GYSZHLETF", "name": "\u5de5\u94f6\u6df1\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHENZHENGHONGLIETF"}, "159906": {"suoxie": "DCSZCZ40ETF", "name": "\u5927\u6210\u6df1\u8bc1\u6210\u957f40ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGSHENZHENGCHENGZHANG40ETF"}, "159907": {"suoxie": "GFZXQY300ETF", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u4f01\u4e1a300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGXIAOQIYE300ETF"}, "159908": {"suoxie": "BSCYBETF", "name": "\u535a\u65f6\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHICHUANGYEBANETF"}, "159909": {"suoxie": "ZSSZTMT50ETF", "name": "\u62db\u5546\u6df1\u8bc1TMT50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENGTMT50ETF"}, "159910": {"suoxie": "JSSZJBM120ETF", "name": "\u5609\u5b9e\u6df1\u8bc1\u57fa\u672c\u9762120ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHISHENZHENGJIBENMIAN120ETF"}, "159912": {"suoxie": "HTFSZ300ETF", "name": "\u6c47\u6dfb\u5bcc\u6df1\u8bc1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUSHENZHENG300ETF"}, "159913": {"suoxie": "JYSZ300JZETF", "name": "\u4ea4\u94f6\u6df1\u8bc1300\u4ef7\u503cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHENZHENG300JIAZHIETF"}, "159915": {"suoxie": "YFDCYBETF", "name": "\u6613\u65b9\u8fbe\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDACHUANGYEBANETF"}, "159916": {"suoxie": "SF60ETF", "name": "\u6df1F60ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENF60ETF"}, "159918": {"suoxie": "JSZC400ETF", "name": "\u5609\u5b9e\u4e2d\u521b400ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGCHUANG400ETF"}, "159919": {"suoxie": "JSHS300ETF", "name": "\u5609\u5b9e\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300ETF"}, "159920": {"suoxie": "HXHSETFQDII", "name": "\u534e\u590f\u6052\u751fETF(QDII)", "style": "QDII", "pinyin": "HUAXIAHENGSHENGETFQDII"}, "159922": {"suoxie": "JSZZ500ETF", "name": "\u5609\u5b9e\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500ETF"}, "159923": {"suoxie": "DCZZ100ETF", "name": "\u5927\u6210\u4e2d\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENG100ETF"}, "159925": {"suoxie": "NFHS300ETF", "name": "\u5357\u65b9\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300ETF"}, "159926": {"suoxie": "JSZZZQGZETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u4e2d\u671f\u56fd\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIASHIZHONGZHENGZHONGQIGUOZHAIETF"}, "159928": {"suoxie": "HTFZZZYXFETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHUYAOXIAOFEIETF"}, "159929": {"suoxie": "HTFZZYYWSETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u533b\u836f\u536b\u751fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYIYAOWEISHENGETF"}, "159930": {"suoxie": "HTFZZNYETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGNENGYUANETF"}, "159931": {"suoxie": "HTFZZJRDCETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u91d1\u878d\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGJINRONGDICHANETF"}, "159932": {"suoxie": "DCZZ500SSETF", "name": "\u5927\u6210\u4e2d\u8bc1500\u6df1\u5e02ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENG500SHENSHIETF"}, "159933": {"suoxie": "GTRYJRDCETF", "name": "\u56fd\u6295\u745e\u94f6\u91d1\u878d\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINJINRONGDICHANETF"}, "159934": {"suoxie": "YFDHJETF", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "YIFANGDAHUANGJINETF"}, "159935": {"suoxie": "JSCCZZ500ETF", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGZHONGZHENG500ETF"}, "159936": {"suoxie": "GFZZQZKXXFETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u53ef\u9009\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIKEXUANXIAOFEIETF"}, "159937": {"suoxie": "BSHJETF", "name": "\u535a\u65f6\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "BOSHIHUANGJINETF"}, "159938": {"suoxie": "GFZZQZYYWSETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u533b\u836f\u536b\u751fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIYIYAOWEISHENGETF"}, "159939": {"suoxie": "GFZZQZXXJSETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u4fe1\u606f\u6280\u672fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIXINXIJISHUETF"}, "159940": {"suoxie": "GFZZQZJRDCETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u91d1\u878d\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIJINRONGDICHANETF"}, "159941": {"suoxie": "GFNSDK100ETF", "name": "\u5e7f\u53d1\u7eb3\u65af\u8fbe\u514b100ETF", "style": "QDII", "pinyin": "GUANGFANASIDAKE100ETF"}, "159943": {"suoxie": "DCSZCFETF", "name": "\u5927\u6210\u6df1\u8bc1\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGSHENZHENGCHENGFENETF"}, "159944": {"suoxie": "GFZZQZYCLETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u539f\u6750\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHIYUANCAILIAOETF"}, "159945": {"suoxie": "GFZZQZNYETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u5168\u6307\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGQUANZHINENGYUANETF"}, "159948": {"suoxie": "NFCYBETF", "name": "\u5357\u65b9\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGCHUANGYEBANETF"}, "159949": {"suoxie": "HACYB50ETF", "name": "\u534e\u5b89\u521b\u4e1a\u677f50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCHUANGYEBAN50ETF"}, "159951": {"suoxie": "JSZGCAGETF", "name": "\u5609\u5b9e\u4e2d\u5173\u6751A\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGGUANCUNAGUETF"}, "159952": {"suoxie": "GFCYBETF", "name": "\u5e7f\u53d1\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFACHUANGYEBANETF"}, "159954": {"suoxie": "NFHSZGQYETF", "name": "\u5357\u65b9\u6052\u751f\u4e2d\u56fd\u4f01\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHENGSHENGZHONGGUOQIYEETF"}, "159955": {"suoxie": "JSCYBETF", "name": "\u5609\u5b9e\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHICHUANGYEBANETF"}, "159956": {"suoxie": "JXCYBETF", "name": "\u5efa\u4fe1\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINCHUANGYEBANETF"}, "159957": {"suoxie": "HXCYBETF", "name": "\u534e\u590f\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGYEBANETF"}, "159958": {"suoxie": "GYRXCYBETF", "name": "\u5de5\u94f6\u745e\u4fe1\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINCHUANGYEBANETF"}, "159959": {"suoxie": "YHZZYQJGTZETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u592e\u4f01\u7ed3\u6784\u8c03\u6574ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGYANGQIJIEGOUTIAOZHENGETF"}, "159960": {"suoxie": "PAGGTHSZGQYETF", "name": "\u5e73\u5b89\u6e2f\u80a1\u901a\u6052\u751f\u4e2d\u56fd\u4f01\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANGANGGUTONGHENGSHENGZHONGGUOQIYEETF"}, "159961": {"suoxie": "FZFBSZ100ETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGSHENZHENG100ETF"}, "159962": {"suoxie": "HXZZSCGGETF", "name": "\u534e\u590f\u4e2d\u8bc1\u56db\u5ddd\u56fd\u6539ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGSICHUANGUOGAIETF"}, "159964": {"suoxie": "PACYBETF", "name": "\u5e73\u5b89\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANCHUANGYEBANETF"}, "159965": {"suoxie": "ZRYSCJ50ETF", "name": "\u4e2d\u878d\u592e\u89c6\u8d22\u7ecf50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGYANGSHICAIJING50ETF"}, "159966": {"suoxie": "HXCLCETF", "name": "\u534e\u590f\u521b\u84dd\u7b79ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGLANCHOUETF"}, "159967": {"suoxie": "HXCCZETF", "name": "\u534e\u590f\u521b\u6210\u957fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIACHUANGCHENGZHANGETF"}, "159968": {"suoxie": "BSZZ500ETF", "name": "\u535a\u65f6\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENG500ETF"}, "159969": {"suoxie": "YHSZ100ETF", "name": "\u94f6\u534e\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUASHENZHENG100ETF"}, "159970": {"suoxie": "GYRXSZ100ETF", "name": "\u5de5\u94f6\u745e\u4fe1\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINSHENZHENG100ETF"}, "159971": {"suoxie": "FGCYBETF", "name": "\u5bcc\u56fd\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOCHUANGYEBANETF"}, "159972": {"suoxie": "PHZZ5NDZETF", "name": "\u9e4f\u534e\u4e2d\u8bc15\u5e74\u5730\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAZHONGZHENG5NIANDIZHAIETF"}, "159973": {"suoxie": "HYYFMQLX100ETF", "name": "\u5f18\u6bc5\u8fdc\u65b9\u6c11\u4f01\u9886\u5148100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGYIYUANFANGMINQILINGXIAN100ETF"}, "159974": {"suoxie": "FGYQCXETF", "name": "\u5bcc\u56fd\u592e\u4f01\u521b\u65b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOYANGQICHUANGXINETF"}, "159975": {"suoxie": "ZSSZ100ETF", "name": "\u62db\u5546\u6df1\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENG100ETF"}, "159976": {"suoxie": "GYYGADWQCX100ETF", "name": "\u5de5\u94f6\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533a\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINYUEGANGAODAWANQUCHUANGXIN100ETF"}, "159977": {"suoxie": "THCYBETF", "name": "\u5929\u5f18\u521b\u4e1a\u677fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGCHUANGYEBANETF"}, "159978": {"suoxie": "JXZZHGSYGADWQFZZTETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533a\u53d1\u5c55\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGHUGANGSHENYUEGANGAODAWANQUFAZHANZHUTIETF"}, "159979": {"suoxie": "GFYGADWQCX100ETF", "name": "\u5e7f\u53d1\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533a\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYUEGANGAODAWANQUCHUANGXIN100ETF"}, "159980": {"suoxie": "DCYSJSQHETF", "name": "\u5927\u6210\u6709\u8272\u91d1\u5c5e\u671f\u8d27ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "DACHENGYOUSEJINSHUQIHUOETF"}, "159981": {"suoxie": "JXYSNYHGQHETF", "name": "\u5efa\u4fe1\u6613\u76db\u80fd\u6e90\u5316\u5de5\u671f\u8d27ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINYISHENGNENGYUANHUAGONGQIHUOETF"}, "159982": {"suoxie": "PHZZ500ETF", "name": "\u9e4f\u534e\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ETF"}, "159983": {"suoxie": "HXYGADWQCX100ETF", "name": "\u534e\u590f\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533a\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAYUEGANGAODAWANQUCHUANGXIN100ETF"}, "159984": {"suoxie": "NFYGADWQETF", "name": "\u5357\u65b9\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGYUEGANGAODAWANQUETF"}, "159985": {"suoxie": "HXSLDPQHETF", "name": "\u534e\u590f\u9972\u6599\u8c46\u7c95\u671f\u8d27ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIASILIAODOUPOQIHUOETF"}, "159986": {"suoxie": "HYYFGZXF100ETF", "name": "\u5f18\u6bc5\u8fdc\u65b9\u56fd\u8bc1\u6d88\u8d39100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HONGYIYUANFANGGUOZHENGXIAOFEI100ETF"}, "159987": {"suoxie": "YHZZYFCX100ETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u7814\u53d1\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGYANFACHUANGXIN100ETF"}, "159988": {"suoxie": "PAZZ05NDFZETF", "name": "\u5e73\u5b89\u4e2d\u503a-0-5\u5e74\u5730\u65b9\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAI05NIANDIFANGZHAIETF"}, "159990": {"suoxie": "YHJCXPJZETF", "name": "\u94f6\u534e\u5de8\u6f6e\u5c0f\u76d8\u4ef7\u503cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAJUCHAOXIAOPANJIAZHIETF"}, "159991": {"suoxie": "ZSCYBDPETF", "name": "\u62db\u5546\u521b\u4e1a\u677f\u5927\u76d8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGCHUANGYEBANDAPANETF"}, "159992": {"suoxie": "YHZZCXYCYETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGCHUANGXINYAOCHANYEETF"}, "159993": {"suoxie": "PHGZZQLTETF", "name": "\u9e4f\u534e\u56fd\u8bc1\u8bc1\u5238\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGZHENGQUANLONGTOUETF"}, "159994": {"suoxie": "YHZZ5GTXZTETF", "name": "\u94f6\u534e\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENG5GTONGXINZHUTIETF"}, "159995": {"suoxie": "HXGZBDTXPETF", "name": "\u534e\u590f\u56fd\u8bc1\u534a\u5bfc\u4f53\u82af\u7247ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAGUOZHENGBANDAOTIXINPIANETF"}, "159996": {"suoxie": "GTZZQZJDETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u5bb6\u7535ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIJIADIANETF"}, "159997": {"suoxie": "THZZDZETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u7535\u5b50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGDIANZIETF"}, "159998": {"suoxie": "THZZJSJETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u8ba1\u7b97\u673aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGJISUANJIETF"}, "159999": {"suoxie": "YYZZ500ETF", "name": "\u6c38\u8d62\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGZHONGZHENG500ETF"}, "160105": {"suoxie": "NFJJPZHHLOF", "name": "\u5357\u65b9\u79ef\u6781\u914d\u7f6e\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIJIPEIZHIHUNHELOF"}, "160106": {"suoxie": "NFGZZHHLOF", "name": "\u5357\u65b9\u9ad8\u589e\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGAOZENGZHANGHUNHELOF"}, "160107": {"suoxie": "NFGZZHHLOF", "name": "\u5357\u65b9\u9ad8\u589e\u957f\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGGAOZENGZHANGHUNHELOF"}, "160119": {"suoxie": "NFZZ500ETFLJA", "name": "\u5357\u65b9\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500ETFLIANJIEA"}, "160120": {"suoxie": "NFZZ500ETFLJA", "name": "\u5357\u65b9\u4e2d\u8bc1500ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500ETFLIANJIEA"}, "160125": {"suoxie": "NFXGYXGP", "name": "\u5357\u65b9\u9999\u6e2f\u4f18\u9009\u80a1\u7968", "style": "QDII", "pinyin": "NANFANGXIANGGANGYOUXUANGUPIAO"}, "160126": {"suoxie": "NFXGYXGP", "name": "\u5357\u65b9\u9999\u6e2f\u4f18\u9009\u80a1\u7968(\u540e\u7aef)", "style": "QDII", "pinyin": "NANFANGXIANGGANGYOUXUANGUPIAO"}, "160127": {"suoxie": "NFXXXFZZGPLOFA", "name": "\u5357\u65b9\u65b0\u5174\u6d88\u8d39\u589e\u957f\u80a1\u7968(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGXINXINGXIAOFEIZENGZHANGGUPIAOLOFA"}, "160128": {"suoxie": "NFJLDKZLOFA", "name": "\u5357\u65b9\u91d1\u5229\u5b9a\u5f00\u503a(LOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJINLIDINGKAIZHAILOFA"}, "160129": {"suoxie": "NFJLDKZLOFC", "name": "\u5357\u65b9\u91d1\u5229\u5b9a\u5f00\u503a(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGJINLIDINGKAIZHAILOFC"}, "160133": {"suoxie": "NFTYXCYGPLOF", "name": "\u5357\u65b9\u5929\u5143\u65b0\u4ea7\u4e1a\u80a1\u7968(LOF)", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGTIANYUANXINCHANYEGUPIAOLOF"}, "160135": {"suoxie": "NFZZGTCYZSLOF", "name": "\u5357\u65b9\u4e2d\u8bc1\u9ad8\u94c1\u4ea7\u4e1a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGGAOTIECHANYEZHISHULOF"}, "160137": {"suoxie": "NFZZHLWZSLOF", "name": "\u5357\u65b9\u4e2d\u8bc1\u4e92\u8054\u7f51\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGHULIANWANGZHISHULOF"}, "160140": {"suoxie": "NFDQSMGJXA", "name": "\u5357\u65b9\u9053\u743c\u65af\u7f8e\u56fd\u7cbe\u9009A", "style": "QDII", "pinyin": "NANFANGDAOQIONGSIMEIGUOJINGXUANA"}, "160141": {"suoxie": "NFDQSMGJXC", "name": "\u5357\u65b9\u9053\u743c\u65af\u7f8e\u56fd\u7cbe\u9009C", "style": "QDII", "pinyin": "NANFANGDAOQIONGSIMEIGUOJINGXUANC"}, "160142": {"suoxie": "NFYSCYLOF", "name": "\u5357\u65b9\u4f18\u52bf\u4ea7\u4e1a(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGYOUSHICHANYELOF"}, "160143": {"suoxie": "NFCYB2NDQKFHH", "name": "\u5357\u65b9\u521b\u4e1a\u677f2\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHUANGYEBAN2NIANDINGQIKAIFANGHUNHE"}, "160144": {"suoxie": "NFXXXFZZGPLOFC", "name": "\u5357\u65b9\u65b0\u5174\u6d88\u8d39\u589e\u957f\u80a1\u7968(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "NANFANGXINXINGXIAOFEIZENGZHANGGUPIAOLOFC"}, "160211": {"suoxie": "GTZXPCZHHLOF", "name": "\u56fd\u6cf0\u4e2d\u5c0f\u76d8\u6210\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIZHONGXIAOPANCHENGZHANGHUNHELOF"}, "160212": {"suoxie": "GTGZYSHHLOF", "name": "\u56fd\u6cf0\u4f30\u503c\u4f18\u52bf\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIGUZHIYOUSHIHUNHELOF"}, "160213": {"suoxie": "GTNSDK100ZS", "name": "\u56fd\u6cf0\u7eb3\u65af\u8fbe\u514b100\u6307\u6570", "style": "QDII", "pinyin": "GUOTAINASIDAKE100ZHISHU"}, "160215": {"suoxie": "GTJZJDHHLOF", "name": "\u56fd\u6cf0\u4ef7\u503c\u7ecf\u5178\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIAZHIJINGDIANHUNHELOF"}, "160216": {"suoxie": "GTDZSP", "name": "\u56fd\u6cf0\u5927\u5b97\u5546\u54c1", "style": "QDII", "pinyin": "GUOTAIDAZONGSHANGPIN"}, "160217": {"suoxie": "GTXYHLZQA", "name": "\u56fd\u6cf0\u4fe1\u7528\u4e92\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIXINYONGHULIZHAIQUANA"}, "160218": {"suoxie": "GTGZFDCHYZS", "name": "\u56fd\u6cf0\u56fd\u8bc1\u623f\u5730\u4ea7\u884c\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGFANGDICHANHANGYEZHISHU"}, "160219": {"suoxie": "GTGZYYWSHYZS", "name": "\u56fd\u6cf0\u56fd\u8bc1\u533b\u836f\u536b\u751f\u884c\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGYIYAOWEISHENGHANGYEZHISHU"}, "160220": {"suoxie": "GTMYHHLOFA", "name": "\u56fd\u6cf0\u6c11\u76ca\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINYIHUNHELOFA"}, "160221": {"suoxie": "GTGZYSJSHYZSLOF", "name": "\u56fd\u6cf0\u56fd\u8bc1\u6709\u8272\u91d1\u5c5e\u884c\u4e1a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGYOUSEJINSHUHANGYEZHISHULOF"}, "160222": {"suoxie": "GTGZSPYLHYLOF", "name": "\u56fd\u6cf0\u56fd\u8bc1\u98df\u54c1\u996e\u6599\u884c\u4e1a(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGSHIPINYINLIAOHANGYELOF"}, "160223": {"suoxie": "GTCYBZSLOF", "name": "\u56fd\u6cf0\u521b\u4e1a\u677f\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAICHUANGYEBANZHISHULOF"}, "160224": {"suoxie": "GTZZJSJZTETFLJA", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u8ba1\u7b97\u673a\u4e3b\u9898ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGJISUANJIZHUTIETFLIANJIEA"}, "160225": {"suoxie": "GTGZXNYQCZS", "name": "\u56fd\u6cf0\u56fd\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGXINNENGYUANQICHEZHISHU"}, "160226": {"suoxie": "GTMYHHLOFC", "name": "\u56fd\u6cf0\u6c11\u76ca\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIMINYIHUNHELOFC"}, "160311": {"suoxie": "HXLCHHLOF", "name": "\u534e\u590f\u84dd\u7b79\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIALANCHOUHUNHELOF"}, "160314": {"suoxie": "HXHYHHLOF", "name": "\u534e\u590f\u884c\u4e1a\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAHANGYEHUNHELOF"}, "160322": {"suoxie": "HXGGTJXGPFQSLOFA", "name": "\u534e\u590f\u6e2f\u80a1\u901a\u7cbe\u9009\u80a1\u7968\u53d1\u8d77\u5f0f(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAGANGGUTONGJINGXUANGUPIAOFAQISHILOFA"}, "160323": {"suoxie": "HXPTHHLOFA", "name": "\u534e\u590f\u78d0\u6cf0\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUAXIAPANTAIHUNHELOFA"}, "160324": {"suoxie": "HXPSHHLOF", "name": "\u534e\u590f\u78d0\u665f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAPANSHENGHUNHELOF"}, "160325": {"suoxie": "HXCYBLNDKHH", "name": "\u534e\u590f\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "160326": {"suoxie": "HXYXPZYNFBYZGPA", "name": "\u534e\u590f\u4f18\u9009\u914d\u7f6e\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAYOUXUANPEIZHIYINIANFENGBIYUNZUOGUPIAOA"}, "160415": {"suoxie": "HALHDYZHHLOF", "name": "\u534e\u5b89\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANLIANGHUADUOYINZIHUNHELOF"}, "160416": {"suoxie": "HABPQQSYZS", "name": "\u534e\u5b89\u6807\u666e\u5168\u7403\u77f3\u6cb9\u6307\u6570", "style": "QDII", "pinyin": "HUAANBIAOPUQUANQIUSHIYOUZHISHU"}, "160418": {"suoxie": "HAZZYHZS", "name": "\u534e\u5b89\u4e2d\u8bc1\u94f6\u884c\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGYINHANGZHISHU"}, "160419": {"suoxie": "HAZZQZZQGSZS", "name": "\u534e\u5b89\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHU"}, "160420": {"suoxie": "HACYB50ZS", "name": "\u534e\u5b89\u521b\u4e1a\u677f50\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCHUANGYEBAN50ZHISHU"}, "160421": {"suoxie": "HAZZJXHH", "name": "\u534e\u5b89\u667a\u589e\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANZHIZENGJINGXUANHUNHE"}, "160422": {"suoxie": "HACYB50ETFLJA", "name": "\u534e\u5b89\u521b\u4e1a\u677f50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCHUANGYEBAN50ETFLIANJIEA"}, "160424": {"suoxie": "HACYB50ETFLJC", "name": "\u534e\u5b89\u521b\u4e1a\u677f50ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCHUANGYEBAN50ETFLIANJIEC"}, "160425": {"suoxie": "HACYBLNDKHH", "name": "\u534e\u5b89\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANCHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "160505": {"suoxie": "BSZTHYHHLOF", "name": "\u535a\u65f6\u4e3b\u9898\u884c\u4e1a\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHUTIHANGYEHUNHELOF"}, "160512": {"suoxie": "BSZYPPHHLOF", "name": "\u535a\u65f6\u5353\u8d8a\u54c1\u724c\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIZHUOYUEPINPAIHUNHELOF"}, "160513": {"suoxie": "BSWJHBZQLOFA", "name": "\u535a\u65f6\u7a33\u5065\u56de\u62a5\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIWENJIANHUIBAOZHAIQUANLOFA"}, "160514": {"suoxie": "BSWJHBZQLOFC", "name": "\u535a\u65f6\u7a33\u5065\u56de\u62a5\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BOSHIWENJIANHUIBAOZHAIQUANLOFC"}, "160515": {"suoxie": "BSAF18GYDKZA", "name": "\u535a\u65f6\u5b89\u4e3018\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANFENG18GEYUEDINGKAIZHAIA"}, "160516": {"suoxie": "BSZZQZZQGSZSLOF", "name": "\u535a\u65f6\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOF"}, "160517": {"suoxie": "BSZZYHZSLOF", "name": "\u535a\u65f6\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGYINHANGZHISHULOF"}, "160518": {"suoxie": "BSRY", "name": "\u535a\u65f6\u777f\u8fdc", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRUIYUAN"}, "160519": {"suoxie": "BSRLSJQDLOF", "name": "\u535a\u65f6\u777f\u5229\u4e8b\u4ef6\u9a71\u52a8(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIRUILISHIJIANQUDONGLOF"}, "160523": {"suoxie": "BSAF18GYDKZC", "name": "\u535a\u65f6\u5b89\u4e3018\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANFENG18GEYUEDINGKAIZHAIC"}, "160524": {"suoxie": "BSHTHH", "name": "\u535a\u65f6\u5f18\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "BOSHIHONGTAIHUNHE"}, "160526": {"suoxie": "BSYSQYLOFA", "name": "\u535a\u65f6\u4f18\u52bf\u4f01\u4e1a(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYOUSHIQIYELOFA"}, "160527": {"suoxie": "BSYJYX3NFBHHA", "name": "\u535a\u65f6\u7814\u7a76\u4f18\u90093\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUYOUXUAN3NIANFENGBIHUNHEA"}, "160528": {"suoxie": "BSYJYX3NFBHHC", "name": "\u535a\u65f6\u7814\u7a76\u4f18\u90093\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIYANJIUYOUXUAN3NIANFENGBIHUNHEC"}, "160529": {"suoxie": "BSCYBLNDKHH", "name": "\u535a\u65f6\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHICHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "160602": {"suoxie": "PHPTZQA", "name": "\u9e4f\u534e\u666e\u5929\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAPUTIANZHAIQUANA"}, "160603": {"suoxie": "PHPTSYHH", "name": "\u9e4f\u534e\u666e\u5929\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAPUTIANSHOUYIHUNHE"}, "160605": {"suoxie": "PHZG50HH", "name": "\u9e4f\u534e\u4e2d\u56fd50\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAZHONGGUO50HUNHE"}, "160606": {"suoxie": "PHHBA", "name": "\u9e4f\u534e\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAHUOBIA"}, "160607": {"suoxie": "PHJZYSHHLOF", "name": "\u9e4f\u534e\u4ef7\u503c\u4f18\u52bf\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJIAZHIYOUSHIHUNHELOF"}, "160608": {"suoxie": "PHPTZQB", "name": "\u9e4f\u534e\u666e\u5929\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAPUTIANZHAIQUANB"}, "160609": {"suoxie": "PHHBB", "name": "\u9e4f\u534e\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUAHUOBIB"}, "160610": {"suoxie": "PHDLZZHHLOF", "name": "\u9e4f\u534e\u52a8\u529b\u589e\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUADONGLIZENGZHANGHUNHELOF"}, "160611": {"suoxie": "PHYZZLHHLOF", "name": "\u9e4f\u534e\u4f18\u8d28\u6cbb\u7406\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAYOUZHIZHILIHUNHELOF"}, "160612": {"suoxie": "PHFSZQ", "name": "\u9e4f\u534e\u4e30\u6536\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAFENGSHOUZHAIQUAN"}, "160613": {"suoxie": "PHSSCXHHLOF", "name": "\u9e4f\u534e\u76db\u4e16\u521b\u65b0\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUASHENGSHICHUANGXINHUNHELOF"}, "160615": {"suoxie": "PHHS300ZSLOFA", "name": "\u9e4f\u534e\u6caa\u6df1300\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAHUSHEN300ZHISHULOFA"}, "160616": {"suoxie": "PHZZ500ZSLOFA", "name": "\u9e4f\u534e\u4e2d\u8bc1500\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG500ZHISHULOFA"}, "160617": {"suoxie": "PHFRZQLOF", "name": "\u9e4f\u534e\u4e30\u6da6\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAFENGRUNZHAIQUANLOF"}, "160618": {"suoxie": "PHFZZQLOF", "name": "\u9e4f\u534e\u4e30\u6cfd\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGZEZHAIQUANLOF"}, "160620": {"suoxie": "PHZZAGZYCYZSLOFA", "name": "\u9e4f\u534e\u4e2d\u8bc1A\u80a1\u8d44\u6e90\u4ea7\u4e1a\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGAGUZIYUANCHANYEZHISHULOFA"}, "160621": {"suoxie": "PHFHZQLOFA", "name": "\u9e4f\u534e\u4e30\u548c\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAFENGHEZHAIQUANLOFA"}, "160622": {"suoxie": "PHFLZQLOF", "name": "\u9e4f\u534e\u4e30\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUAFENGLIZHAIQUANLOF"}, "160624": {"suoxie": "PHXFLXHH", "name": "\u9e4f\u534e\u6d88\u8d39\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAXIAOFEILINGXIANHUNHE"}, "160625": {"suoxie": "PHZQBXZSLOF", "name": "\u9e4f\u534e\u8bc1\u5238\u4fdd\u9669\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHENGQUANBAOXIANZHISHULOF"}, "160626": {"suoxie": "PHXXA", "name": "\u9e4f\u534e\u4fe1\u606fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAXINXIA"}, "160627": {"suoxie": "PHCLYXLHPZHH", "name": "\u9e4f\u534e\u7b56\u7565\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUACELUEYOUXUANLINGHUOPEIZHIHUNHE"}, "160628": {"suoxie": "PHZZ800DCZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1800\u5730\u4ea7\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG800DICHANZHISHULOF"}, "160629": {"suoxie": "PHZZCMZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u4f20\u5a92\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGCHUANMEIZHISHULOF"}, "160630": {"suoxie": "PHGFA", "name": "\u9e4f\u534e\u56fd\u9632A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOFANGA"}, "160631": {"suoxie": "PHYHA", "name": "\u9e4f\u534e\u94f6\u884cA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAYINHANGA"}, "160632": {"suoxie": "PHJA", "name": "\u9e4f\u534e\u9152A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAJIUA"}, "160633": {"suoxie": "PHQSA", "name": "\u9e4f\u534e\u5238\u5546A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAQUANSHANGA"}, "160634": {"suoxie": "PHZZHBCYZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGHUANBAOCHANYEZHISHULOF"}, "160635": {"suoxie": "PHZZYYWSLOFA", "name": "\u9e4f\u534e\u4e2d\u8bc1\u533b\u836f\u536b\u751f(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYIYAOWEISHENGLOFA"}, "160636": {"suoxie": "PHZZYDHLWZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u79fb\u52a8\u4e92\u8054\u7f51\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYIDONGHULIANWANGZHISHULOF"}, "160637": {"suoxie": "PHCYBZSLOF", "name": "\u9e4f\u534e\u521b\u4e1a\u677f\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUACHUANGYEBANZHISHULOF"}, "160638": {"suoxie": "PHZZYDYLZTZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u4e00\u5e26\u4e00\u8def\u4e3b\u9898\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYIDAIYILUZHUTIZHISHULOF"}, "160639": {"suoxie": "PHZZGTCYZSLOF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u9ad8\u94c1\u4ea7\u4e1a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGAOTIECHANYEZHISHULOF"}, "160642": {"suoxie": "PHZRHHLOF", "name": "\u9e4f\u534e\u589e\u745e\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAZENGRUIHUNHELOF"}, "160643": {"suoxie": "PHKTJGZSLOFA", "name": "\u9e4f\u534e\u7a7a\u5929\u519b\u5de5\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAKONGTIANJUNGONGZHISHULOFA"}, "160644": {"suoxie": "PHGMHLGPRMB", "name": "\u9e4f\u534e\u6e2f\u7f8e\u4e92\u8054\u80a1\u7968\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "PENGHUAGANGMEIHULIANGUPIAORENMINBI"}, "160645": {"suoxie": "PHJXHBSNDKHH", "name": "\u9e4f\u534e\u7cbe\u9009\u56de\u62a5\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJINGXUANHUIBAOSANNIANDINGKAIHUNHE"}, "160646": {"suoxie": "PHZZHGSKJLTZSLOFA", "name": "\u9e4f\u534e\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGHUGANGSHENKEJILONGTOUZHISHULOFA"}, "160706": {"suoxie": "JSHS300ETFLJA", "name": "\u5609\u5b9e\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300ETFLIANJIEA"}, "160716": {"suoxie": "JSJBM50ZSLOFA", "name": "\u5609\u5b9e\u57fa\u672c\u976250\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIJIBENMIAN50ZHISHULOFA"}, "160717": {"suoxie": "JSHSZGQY", "name": "\u5609\u5b9e\u6052\u751f\u4e2d\u56fd\u4f01\u4e1a", "style": "QDII", "pinyin": "JIASHIHENGSHENGZHONGGUOQIYE"}, "160718": {"suoxie": "JSDLSYZQ", "name": "\u5609\u5b9e\u591a\u5229\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIASHIDUOLISHOUYIZHAIQUAN"}, "160719": {"suoxie": "JSHJ", "name": "\u5609\u5b9e\u9ec4\u91d1", "style": "QDII", "pinyin": "JIASHIHUANGJIN"}, "160722": {"suoxie": "JSHZHHLOF", "name": "\u5609\u5b9e\u60e0\u6cfd\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIHUIZEHUNHELOF"}, "160723": {"suoxie": "JSYYQDIILOF", "name": "\u5609\u5b9e\u539f\u6cb9(QDII-LOF)", "style": "QDII", "pinyin": "JIASHIYUANYOUQDIILOF"}, "160724": {"suoxie": "JSHS300ETFLJC", "name": "\u5609\u5b9e\u6caa\u6df1300ETF\u8054\u63a5C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300ETFLIANJIEC"}, "160725": {"suoxie": "JSJBM50ZSLOFC", "name": "\u5609\u5b9e\u57fa\u672c\u976250\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIJIBENMIAN50ZHISHULOFC"}, "160726": {"suoxie": "JSRXDQHH", "name": "\u5609\u5b9e\u745e\u4eab\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHIRUIXIANGDINGQIHUNHE"}, "160727": {"suoxie": "JSCYBLNDQHH", "name": "\u5609\u5b9e\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHICHUANGYEBANLIANGNIANDINGQIHUNHE"}, "160805": {"suoxie": "CSTZYSHHLOF", "name": "\u957f\u76db\u540c\u667a\u4f18\u52bf\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGTONGZHIYOUSHIHUNHELOF"}, "160806": {"suoxie": "CSTQZZ800LOF", "name": "\u957f\u76db\u540c\u5e86\u4e2d\u8bc1800(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGTONGQINGZHONGZHENG800LOF"}, "160807": {"suoxie": "CSHS300ZSLOF", "name": "\u957f\u76db\u6caa\u6df1300\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGHUSHEN300ZHISHULOF"}, "160812": {"suoxie": "CSTYCZHBLOF", "name": "\u957f\u76db\u540c\u76ca\u6210\u957f\u56de\u62a5(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGTONGYICHENGZHANGHUIBAOLOF"}, "160813": {"suoxie": "CSTSCZYXLOF", "name": "\u957f\u76db\u540c\u76db\u6210\u957f\u4f18\u9009(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGSHENGTONGSHENGCHENGZHANGYOUXUANLOF"}, "160814": {"suoxie": "CSZZJRDCZSLOF", "name": "\u957f\u76db\u4e2d\u8bc1\u91d1\u878d\u5730\u4ea7\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGZHONGZHENGJINRONGDICHANZHISHULOF"}, "160910": {"suoxie": "DCCXCZHHLOF", "name": "\u5927\u6210\u521b\u65b0\u6210\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHUANGXINCHENGZHANGHUNHELOF"}, "160916": {"suoxie": "DCYXHHLOF", "name": "\u5927\u6210\u4f18\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGYOUXUANHUNHELOF"}, "160918": {"suoxie": "DCZXPHHLOFA", "name": "\u5927\u6210\u4e2d\u5c0f\u76d8\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGZHONGXIAOPANHUNHELOFA"}, "160919": {"suoxie": "DCCYSJGPLOF", "name": "\u5927\u6210\u4ea7\u4e1a\u5347\u7ea7\u80a1\u7968(LOF)", "style": "\u80a1\u7968\u578b", "pinyin": "DACHENGCHANYESHENGJIGUPIAOLOF"}, "160921": {"suoxie": "DCDCLHHLOF", "name": "\u5927\u6210\u591a\u7b56\u7565\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGDUOCELUEHUNHELOF"}, "160922": {"suoxie": "DCHSZHZXXGZSA", "name": "\u5927\u6210\u6052\u751f\u7efc\u5408\u4e2d\u5c0f\u578b\u80a1\u6307\u6570A", "style": "QDII", "pinyin": "DACHENGHENGSHENGZONGHEZHONGXIAOXINGGUZHISHUA"}, "160924": {"suoxie": "DCHSZS", "name": "\u5927\u6210\u6052\u751f\u6307\u6570", "style": "QDII", "pinyin": "DACHENGHENGSHENGZHISHU"}, "160925": {"suoxie": "DCZHHSG300ZSLOFA", "name": "\u5927\u6210\u4e2d\u534e\u6caa\u6df1\u6e2f300\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGHUAHUSHENGANG300ZHISHULOFA"}, "160926": {"suoxie": "DCCYBLNDKHHA", "name": "\u5927\u6210\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGCHUANGYEBANLIANGNIANDINGKAIHUNHEA"}, "161005": {"suoxie": "FGTHCZHHABLOF", "name": "\u5bcc\u56fd\u5929\u60e0\u6210\u957f\u6df7\u5408A/B(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHUICHENGZHANGHUNHEABLOF"}, "161006": {"suoxie": "FGTHCZHHABLOF", "name": "\u5bcc\u56fd\u5929\u60e0\u6210\u957f\u6df7\u5408A/B(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANHUICHENGZHANGHUNHEABLOF"}, "161010": {"suoxie": "FGTFQHZQLOF", "name": "\u5bcc\u56fd\u5929\u4e30\u5f3a\u5316\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANFENGQIANGHUAZHAIQUANLOF"}, "161014": {"suoxie": "FGHLHBLNDQKFZQ", "name": "\u5bcc\u56fd\u6c47\u5229\u56de\u62a5\u4e24\u5e74\u5b9a\u671f\u5f00\u653e\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOHUILIHUIBAOLIANGNIANDINGQIKAIFANGZHAIQUAN"}, "161015": {"suoxie": "FGTYZQLOFC", "name": "\u5bcc\u56fd\u5929\u76c8\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANYINGZHAIQUANLOFC"}, "161017": {"suoxie": "FGZZ500ZSZQLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG500ZHISHUZENGQIANGLOFA"}, "161018": {"suoxie": "FGTFQHZQLOF", "name": "\u5bcc\u56fd\u5929\u4e30\u5f3a\u5316\u503a\u5238(LOF)(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUGUOTIANFENGQIANGHUAZHAIQUANLOF"}, "161019": {"suoxie": "FGXTFZQLOF", "name": "\u5bcc\u56fd\u65b0\u5929\u950b\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXINTIANFENGZHAIQUANLOF"}, "161020": {"suoxie": "FGXTFZQLOF", "name": "\u5bcc\u56fd\u65b0\u5929\u950b\u503a\u5238(LOF)(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOXINTIANFENGZHAIQUANLOF"}, "161021": {"suoxie": "FGZZ500ZSZQLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a(LOF)A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG500ZHISHUZENGQIANGLOFA"}, "161022": {"suoxie": "FGCYBZSLOFA", "name": "\u5bcc\u56fd\u521b\u4e1a\u677f\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOCHUANGYEBANZHISHULOFA"}, "161024": {"suoxie": "FGZZJGZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u519b\u5de5\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJUNGONGZHISHULOFA"}, "161025": {"suoxie": "FGZZYDHLWZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79fb\u52a8\u4e92\u8054\u7f51\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIDONGHULIANWANGZHISHULOFA"}, "161026": {"suoxie": "FGZZGYQYGGZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u56fd\u6709\u4f01\u4e1a\u6539\u9769\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGUOYOUQIYEGAIGEZHISHULOFA"}, "161027": {"suoxie": "FGZZQZZQGSZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFA"}, "161028": {"suoxie": "FGZZXNYQCZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINNENGYUANQICHEZHISHULOFA"}, "161029": {"suoxie": "FGZZYHZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYINHANGZHISHULOFA"}, "161030": {"suoxie": "FGZZTYCYZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u4f53\u80b2\u4ea7\u4e1a\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGTIYUCHANYEZHISHULOFA"}, "161031": {"suoxie": "FGZZGY40ZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5de5\u4e1a4.0\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGONGYE40ZHISHULOFA"}, "161032": {"suoxie": "FGZZMTZSLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7164\u70ad\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGMEITANZHISHULOFA"}, "161033": {"suoxie": "FGZZZNQCLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGZHINENGQICHELOFA"}, "161035": {"suoxie": "FGZZYYZTZSZQA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u533b\u836f\u4e3b\u9898\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIYAOZHUTIZHISHUZENGQIANGA"}, "161036": {"suoxie": "FGZZYLZTZSZQLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5a31\u4e50\u4e3b\u9898\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYULEZHUTIZHISHUZENGQIANGLOFA"}, "161037": {"suoxie": "FGZZGDZZZSZQXLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u9ad8\u7aef\u5236\u9020\u6307\u6570\u589e\u5f3a\u578b(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGAODUANZHIZAOZHISHUZENGQIANGXINGLOFA"}, "161038": {"suoxie": "FGXXCZLHJXHHLOFA", "name": "\u5bcc\u56fd\u65b0\u5174\u6210\u957f\u91cf\u5316\u7cbe\u9009\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXINXINGCHENGZHANGLIANGHUAJINGXUANHUNHELOFA"}, "161039": {"suoxie": "FGZZ1000ZSZQLOFA", "name": "\u5bcc\u56fd\u4e2d\u8bc11000\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG1000ZHISHUZENGQIANGLOFA"}, "161040": {"suoxie": "FGCYBLNDKHH", "name": "\u5bcc\u56fd\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOCHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "161115": {"suoxie": "YFDSFTLZQLOF", "name": "\u6613\u65b9\u8fbe\u5c81\u4e30\u6dfb\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YIFANGDASUIFENGTIANLIZHAIQUANLOF"}, "161116": {"suoxie": "YFDHJZTRMBA", "name": "\u6613\u65b9\u8fbe\u9ec4\u91d1\u4e3b\u9898\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDAHUANGJINZHUTIRENMINBIA"}, "161117": {"suoxie": "YFDYXDKZ", "name": "\u6613\u65b9\u8fbe\u6c38\u65ed\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAYONGXUDINGKAIZHAI"}, "161118": {"suoxie": "YFDZXQY100LOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u5c0f\u4f01\u4e1a100(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGXIAOQIYE100LOFA"}, "161119": {"suoxie": "YFDZZXZZLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a\u65b0\u7efc\u6307(LOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAIXINZONGZHILOFA"}, "161120": {"suoxie": "YFDZZXZZLOFC", "name": "\u6613\u65b9\u8fbe\u4e2d\u503a\u65b0\u7efc\u6307(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YIFANGDAZHONGZHAIXINZONGZHILOFC"}, "161121": {"suoxie": "YFDZZYHZSLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGYINHANGZHISHULOFA"}, "161122": {"suoxie": "YFDZZWDSWKJZSLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4e07\u5f97\u751f\u7269\u79d1\u6280\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGWANDESHENGWUKEJIZHISHULOFA"}, "161123": {"suoxie": "YFDZZWDBGCZLOF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4e07\u5f97\u5e76\u8d2d\u91cd\u7ec4(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGWANDEBINGGOUCHONGZULOF"}, "161124": {"suoxie": "YFDXGXXGZSA", "name": "\u6613\u65b9\u8fbe\u9999\u6e2f\u5c0f\u578b\u80a1\u6307\u6570A", "style": "QDII", "pinyin": "YIFANGDAXIANGGANGXIAOXINGGUZHISHUA"}, "161125": {"suoxie": "YFDBP500ZSRMBA", "name": "\u6613\u65b9\u8fbe\u6807\u666e500\u6307\u6570\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDABIAOPU500ZHISHURENMINBIA"}, "161126": {"suoxie": "YFDBPYLBJRMBA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u533b\u7597\u4fdd\u5065\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDABIAOPUYILIAOBAOJIANRENMINBIA"}, "161127": {"suoxie": "YFDBPSWKJRMBA", "name": "\u6613\u65b9\u8fbe\u6807\u666e\u751f\u7269\u79d1\u6280\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDABIAOPUSHENGWUKEJIRENMINBIA"}, "161128": {"suoxie": "YBPXXKJRMBA", "name": "\u6613\u6807\u666e\u4fe1\u606f\u79d1\u6280\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIBIAOPUXINXIKEJIRENMINBIA"}, "161129": {"suoxie": "YFDYYALRMB", "name": "\u6613\u65b9\u8fbe\u539f\u6cb9A\u7c7b\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "YIFANGDAYUANYOUALEIRENMINBI"}, "161130": {"suoxie": "YFDNSDK100RMBA", "name": "\u6613\u65b9\u8fbe\u7eb3\u65af\u8fbe\u514b100\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "YIFANGDANASIDAKE100RENMINBIA"}, "161131": {"suoxie": "YFDKRHHLOF", "name": "\u6613\u65b9\u8fbe\u79d1\u6da6\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAKERUNHUNHELOF"}, "161132": {"suoxie": "YFDKSDKHH", "name": "\u6613\u65b9\u8fbe\u79d1\u987a\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIFANGDAKESHUNDINGKAIHUNHE"}, "161211": {"suoxie": "GTHS300JRDCLJ", "name": "\u56fd\u6295\u6caa\u6df1300\u91d1\u878d\u5730\u4ea7\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOUHUSHEN300JINRONGDICHANLIANJIE"}, "161213": {"suoxie": "GTRYZZXFFWZS", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u8bc1\u6d88\u8d39\u670d\u52a1\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINZHONGZHENGXIAOFEIFUWUZHISHU"}, "161215": {"suoxie": "GTRYZZXFFWZS", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u8bc1\u6d88\u8d39\u670d\u52a1\u6307\u6570(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINZHONGZHENGXIAOFEIFUWUZHISHU"}, "161216": {"suoxie": "GTRYSZZQLOFA", "name": "\u56fd\u6295\u745e\u94f6\u53cc\u503a\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINSHUANGZHAIZHAIQUANLOFA"}, "161217": {"suoxie": "GTRYZZZYZS", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u8bc1\u8d44\u6e90\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINZHONGZHENGZIYUANZHISHU"}, "161218": {"suoxie": "GTRYZZZYZS", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u8bc1\u8d44\u6e90\u6307\u6570(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINZHONGZHENGZIYUANZHISHU"}, "161219": {"suoxie": "GTRYXXCYHH", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINXINXINGCHANYEHUNHE"}, "161220": {"suoxie": "GTRYXXCYHH", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTOURUIYINXINXINGCHANYEHUNHE"}, "161221": {"suoxie": "GTRYSZZQC", "name": "\u56fd\u6295\u745e\u94f6\u53cc\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTOURUIYINSHUANGZHAIZHAIQUANC"}, "161222": {"suoxie": "GTRYRLHHLOF", "name": "\u56fd\u6295\u745e\u94f6\u745e\u5229\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUILIHUNHELOF"}, "161224": {"suoxie": "GTRYXSLHHLOF", "name": "\u56fd\u6295\u745e\u94f6\u65b0\u4e1d\u8def\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINXINSILUHUNHELOF"}, "161225": {"suoxie": "GTRYRYHHLOF", "name": "\u56fd\u6295\u745e\u94f6\u745e\u76c8\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUIYINGHUNHELOF"}, "161226": {"suoxie": "GTRYBYQHLOF", "name": "\u56fd\u6295\u745e\u94f6\u767d\u94f6\u671f\u8d27(LOF)", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUOTOURUIYINBAIYINQIHUOLOF"}, "161227": {"suoxie": "GTRYSZ100ZS", "name": "\u56fd\u6295\u745e\u94f6\u6df1\u8bc1100\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINSHENZHENG100ZHISHU"}, "161228": {"suoxie": "GTRYSZ100ZS", "name": "\u56fd\u6295\u745e\u94f6\u6df1\u8bc1100\u6307\u6570(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTOURUIYINSHENZHENG100ZHISHU"}, "161229": {"suoxie": "GTRYZGJZFXGP", "name": "\u56fd\u6295\u745e\u94f6\u4e2d\u56fd\u4ef7\u503c\u53d1\u73b0\u80a1\u7968", "style": "QDII", "pinyin": "GUOTOURUIYINZHONGGUOJIAZHIFAXIANGUPIAO"}, "161232": {"suoxie": "GTRYRSHHLOF", "name": "\u56fd\u6295\u745e\u94f6\u745e\u76db\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUISHENGHUNHELOF"}, "161233": {"suoxie": "GTRYRTDCLHHA", "name": "\u56fd\u6295\u745e\u94f6\u745e\u6cf0\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTOURUIYINRUITAIDUOCELUEHUNHEA"}, "161505": {"suoxie": "YHTLZQLOFA", "name": "\u94f6\u6cb3\u901a\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHETONGLIZHAIQUANLOFA"}, "161506": {"suoxie": "YHTLZQLOFC", "name": "\u94f6\u6cb3\u901a\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHETONGLIZHAIQUANLOFC"}, "161601": {"suoxie": "RTXLCHH", "name": "\u878d\u901a\u65b0\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGXINLANCHOUHUNHE"}, "161602": {"suoxie": "RTXLCHH", "name": "\u878d\u901a\u65b0\u84dd\u7b79\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGXINLANCHOUHUNHE"}, "161603": {"suoxie": "RTZQAB", "name": "\u878d\u901a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZHAIQUANAB"}, "161604": {"suoxie": "RTSZ100ZSA", "name": "\u878d\u901a\u6df1\u8bc1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENG100ZHISHUA"}, "161605": {"suoxie": "RTLCCZHH", "name": "\u878d\u901a\u84dd\u7b79\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGLANCHOUCHENGZHANGHUNHE"}, "161606": {"suoxie": "RTHYJQHHA", "name": "\u878d\u901a\u884c\u4e1a\u666f\u6c14\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGHANGYEJINGQIHUNHEA"}, "161607": {"suoxie": "RTJC100ZSALOF", "name": "\u878d\u901a\u5de8\u6f6e100\u6307\u6570A(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGJUCHAO100ZHISHUALOF"}, "161608": {"suoxie": "RTYZFHBA", "name": "\u878d\u901a\u6613\u652f\u4ed8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGYIZHIFUHUOBIA"}, "161609": {"suoxie": "RTDLXFHH", "name": "\u878d\u901a\u52a8\u529b\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGDONGLIXIANFENGHUNHE"}, "161610": {"suoxie": "RTLXCZHHLOFA", "name": "\u878d\u901a\u9886\u5148\u6210\u957f\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGLINGXIANCHENGZHANGHUNHELOFA"}, "161611": {"suoxie": "RTNXQDHHA", "name": "\u878d\u901a\u5185\u9700\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGNEIXUQUDONGHUNHEA"}, "161612": {"suoxie": "RTSZCFZSA", "name": "\u878d\u901a\u6df1\u8bc1\u6210\u4efd\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENGCHENGFENZHISHUA"}, "161613": {"suoxie": "RTCYBZSA", "name": "\u878d\u901a\u521b\u4e1a\u677f\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGCHUANGYEBANZHISHUA"}, "161614": {"suoxie": "RTSJTLZQLOFA", "name": "\u878d\u901a\u56db\u5b63\u6dfb\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGSIJITIANLIZHAIQUANLOFA"}, "161615": {"suoxie": "RTYZFHBB", "name": "\u878d\u901a\u6613\u652f\u4ed8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGYIZHIFUHUOBIB"}, "161616": {"suoxie": "RTYLBJHYHHAB", "name": "\u878d\u901a\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408A/B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGYILIAOBAOJIANHANGYEHUNHEAB"}, "161617": {"suoxie": "RTYLBJHYHHAB", "name": "\u878d\u901a\u533b\u7597\u4fdd\u5065\u884c\u4e1a\u6df7\u5408A/B(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGYILIAOBAOJIANHANGYEHUNHEAB"}, "161618": {"suoxie": "RTSSTLDKZA", "name": "\u878d\u901a\u5c81\u5c81\u6dfb\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGSUISUITIANLIDINGKAIZHAIA"}, "161619": {"suoxie": "RTSSTLDKZB", "name": "\u878d\u901a\u5c81\u5c81\u6dfb\u5229\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGSUISUITIANLIDINGKAIZHAIB"}, "161620": {"suoxie": "RTHXJZHHA", "name": "\u878d\u901a\u6838\u5fc3\u4ef7\u503c\u6df7\u5408A", "style": "QDII", "pinyin": "RONGTONGHEXINJIAZHIHUNHEA"}, "161622": {"suoxie": "RTHCBHBA", "name": "\u878d\u901a\u6c47\u8d22\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGHUICAIBAOHUOBIA"}, "161623": {"suoxie": "RTHCBHBB", "name": "\u878d\u901a\u6c47\u8d22\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGHUICAIBAOHUOBIB"}, "161624": {"suoxie": "RTKZZZQA", "name": "\u878d\u901a\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "RONGTONGKEZHUANZHAIZHAIQUANA"}, "161625": {"suoxie": "RTKZZZQC", "name": "\u878d\u901a\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "RONGTONGKEZHUANZHAIZHAIQUANC"}, "161626": {"suoxie": "RTTFZQLOFA", "name": "\u878d\u901a\u901a\u798f\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGFUZHAIQUANLOFA"}, "161627": {"suoxie": "RTTFZQLOFC", "name": "\u878d\u901a\u901a\u798f\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "RONGTONGTONGFUZHAIQUANLOFC"}, "161628": {"suoxie": "RTZZYJSYDSJZTZSLOF", "name": "\u878d\u901a\u4e2d\u8bc1\u4e91\u8ba1\u7b97\u4e0e\u5927\u6570\u636e\u4e3b\u9898\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGZHONGZHENGYUNJISUANYUDASHUJUZHUTIZHISHULOF"}, "161631": {"suoxie": "RTRGZNZSLOFA", "name": "\u878d\u901a\u4eba\u5de5\u667a\u80fd\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGRENGONGZHINENGZHISHULOFA"}, "161653": {"suoxie": "RTZQAB", "name": "\u878d\u901a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZHAIQUANAB"}, "161654": {"suoxie": "RTSZ100ZSA", "name": "\u878d\u901a\u6df1\u8bc1100\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENG100ZHISHUA"}, "161655": {"suoxie": "RTLCCZHH", "name": "\u878d\u901a\u84dd\u7b79\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "RONGTONGLANCHOUCHENGZHANGHUNHE"}, "161656": {"suoxie": "RTHYJQHHA", "name": "\u878d\u901a\u884c\u4e1a\u666f\u6c14\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGHANGYEJINGQIHUNHEA"}, "161657": {"suoxie": "RTJC100ZSALOF", "name": "\u878d\u901a\u5de8\u6f6e100\u6307\u6570A(LOF)(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGJUCHAO100ZHISHUALOF"}, "161659": {"suoxie": "RTDLXFHH", "name": "\u878d\u901a\u52a8\u529b\u5148\u950b\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGDONGLIXIANFENGHUNHE"}, "161660": {"suoxie": "RTLXCZHHLOFA", "name": "\u878d\u901a\u9886\u5148\u6210\u957f\u6df7\u5408(LOF)A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGLINGXIANCHENGZHANGHUNHELOFA"}, "161661": {"suoxie": "RTNXQDHHA", "name": "\u878d\u901a\u5185\u9700\u9a71\u52a8\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "RONGTONGNEIXUQUDONGHUNHEA"}, "161662": {"suoxie": "RTSZCFZSA", "name": "\u878d\u901a\u6df1\u8bc1\u6210\u4efd\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGSHENZHENGCHENGFENZHISHUA"}, "161663": {"suoxie": "RTCYBZSA", "name": "\u878d\u901a\u521b\u4e1a\u677f\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "RONGTONGCHUANGYEBANZHISHUA"}, "161693": {"suoxie": "RTZQC", "name": "\u878d\u901a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "RONGTONGZHAIQUANC"}, "161706": {"suoxie": "ZSYZCZHHLOF", "name": "\u62db\u5546\u4f18\u8d28\u6210\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGYOUZHICHENGZHANGHUNHELOF"}, "161707": {"suoxie": "ZSYZCZHHLOF", "name": "\u62db\u5546\u4f18\u8d28\u6210\u957f\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGYOUZHICHENGZHANGHUNHELOF"}, "161713": {"suoxie": "ZSXYTLZQLOFA", "name": "\u62db\u5546\u4fe1\u7528\u6dfb\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXINYONGTIANLIZHAIQUANLOFA"}, "161715": {"suoxie": "ZSDZSPLOF", "name": "\u62db\u5546\u5927\u5b97\u5546\u54c1(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGDAZONGSHANGPINLOF"}, "161716": {"suoxie": "ZSSZZQZQLOFC", "name": "\u62db\u5546\u53cc\u503a\u589e\u5f3a\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGSHUANGZHAIZENGQIANGZHAIQUANLOFC"}, "161718": {"suoxie": "ZSHS300GBT", "name": "\u62db\u5546\u6caa\u6df1300\u9ad8\u8d1d\u5854", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300GAOBEITA"}, "161720": {"suoxie": "ZSZZQZZQGSZSLOFA", "name": "\u62db\u5546\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFA"}, "161721": {"suoxie": "ZSHS300DCDQZZSA", "name": "\u62db\u5546\u6caa\u6df1300\u5730\u4ea7\u7b49\u6743\u91cd\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300DICHANDENGQUANZHONGZHISHUA"}, "161722": {"suoxie": "ZSFTHHLOF", "name": "\u62db\u5546\u4e30\u6cf0\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGFENGTAIHUNHELOF"}, "161723": {"suoxie": "ZSZZYHZSA", "name": "\u62db\u5546\u4e2d\u8bc1\u94f6\u884c\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGYINHANGZHISHUA"}, "161724": {"suoxie": "ZSZZMTDQZSLOFA", "name": "\u62db\u5546\u4e2d\u8bc1\u7164\u70ad\u7b49\u6743\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGMEITANDENGQUANZHISHULOFA"}, "161725": {"suoxie": "ZSZZBJZSLOFA", "name": "\u62db\u5546\u4e2d\u8bc1\u767d\u9152\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGBAIJIUZHISHULOFA"}, "161726": {"suoxie": "ZSGZSWYYZSLOFA", "name": "\u62db\u5546\u56fd\u8bc1\u751f\u7269\u533b\u836f\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGGUOZHENGSHENGWUYIYAOZHISHULOFA"}, "161727": {"suoxie": "ZSZRHHLOF", "name": "\u62db\u5546\u589e\u8363\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGZENGRONGHUNHELOF"}, "161728": {"suoxie": "ZSRZYXHHLOF", "name": "\u62db\u5546\u745e\u667a\u4f18\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGRUIZHIYOUXUANHUNHELOF"}, "161729": {"suoxie": "ZS3NFBRLHH", "name": "\u62db\u55463\u5e74\u5c01\u95ed\u745e\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANG3NIANFENGBIRUILIHUNHE"}, "161810": {"suoxie": "YHNXJXHHLOF", "name": "\u94f6\u534e\u5185\u9700\u7cbe\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUANEIXUJINGXUANHUNHELOF"}, "161811": {"suoxie": "YHHS300ZSLOF", "name": "\u94f6\u534e\u6caa\u6df1300\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAHUSHEN300ZHISHULOF"}, "161812": {"suoxie": "YHSZ100ZSLOF", "name": "\u94f6\u534e\u6df1\u8bc1100\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUASHENZHENG100ZHISHULOF"}, "161815": {"suoxie": "YHKTZZT", "name": "\u94f6\u534e\u6297\u901a\u80c0\u4e3b\u9898", "style": "QDII", "pinyin": "YINHUAKANGTONGZHANGZHUTI"}, "161816": {"suoxie": "YHZZDQZ90ZSLOF", "name": "\u94f6\u534e\u4e2d\u8bc1\u7b49\u6743\u91cd90\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGDENGQUANZHONG90ZHISHULOF"}, "161818": {"suoxie": "YHXFZTHHA", "name": "\u94f6\u534e\u6d88\u8d39\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAXIAOFEIZHUTIHUNHEA"}, "161820": {"suoxie": "YHCZXYZQLOF", "name": "\u94f6\u534e\u7eaf\u503a\u4fe1\u7528\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUACHUNZHAIXINYONGZHAIQUANLOF"}, "161823": {"suoxie": "YHYXCZZQLOFA", "name": "\u94f6\u534e\u6c38\u5174\u7eaf\u503a\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAYONGXINGCHUNZHAIZHAIQUANLOFA"}, "161824": {"suoxie": "YHYXCZZQLOFC", "name": "\u94f6\u534e\u6c38\u5174\u7eaf\u503a\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHUAYONGXINGCHUNZHAIZHAIQUANLOFC"}, "161831": {"suoxie": "YHHSZGQYZS", "name": "\u94f6\u534e\u6052\u751f\u4e2d\u56fd\u4f01\u4e1a\u6307\u6570", "style": "QDII", "pinyin": "YINHUAHENGSHENGZHONGGUOQIYEZHISHU"}, "161834": {"suoxie": "YHXRLHPZHHLOFA", "name": "\u94f6\u534e\u946b\u9510\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINRUILINGHUOPEIZHIHUNHELOFA"}, "161837": {"suoxie": "YHDPLNDKHH", "name": "\u94f6\u534e\u5927\u76d8\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUADAPANLIANGNIANDINGKAIHUNHE"}, "161838": {"suoxie": "YHCYBLNDQKFHH", "name": "\u94f6\u534e\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUACHUANGYEBANLIANGNIANDINGQIKAIFANGHUNHE"}, "161902": {"suoxie": "WJZQSYZQ", "name": "\u4e07\u5bb6\u589e\u5f3a\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIAZENGQIANGSHOUYIZHAIQUAN"}, "161903": {"suoxie": "WJHYYXHHLOF", "name": "\u4e07\u5bb6\u884c\u4e1a\u4f18\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHANGYEYOUXUANHUNHELOF"}, "161907": {"suoxie": "WJZZHLZSLOF", "name": "\u4e07\u5bb6\u4e2d\u8bc1\u7ea2\u5229\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIAZHONGZHENGHONGLIZHISHULOF"}, "161908": {"suoxie": "WJTLZQLOF", "name": "\u4e07\u5bb6\u6dfb\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIATIANLIZHAIQUANLOF"}, "161910": {"suoxie": "WJXJYJZQDA", "name": "\u4e07\u5bb6\u65b0\u673a\u9047\u4ef7\u503c\u9a71\u52a8A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINJIYUJIAZHIQUDONGA"}, "161911": {"suoxie": "WJQHSYDKZ", "name": "\u4e07\u5bb6\u5f3a\u5316\u6536\u76ca\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAQIANGHUASHOUYIDINGKAIZHAI"}, "161912": {"suoxie": "WJSHZR18GYDKA", "name": "\u4e07\u5bb6\u793e\u4f1a\u8d23\u4efb18\u4e2a\u6708\u5b9a\u5f00A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIASHEHUIZEREN18GEYUEDINGKAIA"}, "161913": {"suoxie": "WJSHZR18GYDKC", "name": "\u4e07\u5bb6\u793e\u4f1a\u8d23\u4efb18\u4e2a\u6708\u5b9a\u5f00C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIASHEHUIZEREN18GEYUEDINGKAIC"}, "161914": {"suoxie": "WJCYB2NDQKFHHA", "name": "\u4e07\u5bb6\u521b\u4e1a\u677f2\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIACHUANGYEBAN2NIANDINGQIKAIFANGHUNHEA"}, "161915": {"suoxie": "WJCYB2NDQKFHHC", "name": "\u4e07\u5bb6\u521b\u4e1a\u677f2\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIACHUANGYEBAN2NIANDINGQIKAIFANGHUNHEC"}, "162006": {"suoxie": "CCJFHHLOF", "name": "\u957f\u57ce\u4e45\u5bcc\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGJIUFUHUNHELOF"}, "162102": {"suoxie": "JYZXPJXHH", "name": "\u91d1\u9e70\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINYINGZHONGXIAOPANJINGXUANHUNHE"}, "162105": {"suoxie": "JYCJZLZQLOFC", "name": "\u91d1\u9e70\u6301\u4e45\u589e\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYINGCHIJIUZENGLIZHAIQUANLOFC"}, "162107": {"suoxie": "JYXJZZGPLOFA", "name": "\u91d1\u9e70\u5148\u8fdb\u5236\u9020\u80a1\u7968(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "JINYINGXIANJINZHIZAOGUPIAOLOFA"}, "162108": {"suoxie": "JYYSZQXFQSC", "name": "\u91d1\u9e70\u5143\u76db\u503a\u5238\u578b\u53d1\u8d77\u5f0fC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYINGYUANSHENGZHAIQUANXINGFAQISHIC"}, "162201": {"suoxie": "TDHLCZHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLICHENGZHANGHUNHE"}, "162202": {"suoxie": "TDHLZQHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5468\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIZHOUQIHUNHE"}, "162203": {"suoxie": "TDHLWDHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u7a33\u5b9a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIWENDINGHUNHE"}, "162204": {"suoxie": "TDHLHYHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIHANGYEHUNHE"}, "162205": {"suoxie": "TDHLFXYSHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u98ce\u9669\u9884\u7b97\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TAIDAHONGLIFENGXIANYUSUANHUNHE"}, "162206": {"suoxie": "TDHLHBA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIDAHONGLIHUOBIA"}, "162207": {"suoxie": "TDHLXLYXHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6548\u7387\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TAIDAHONGLIXIAOLVYOUXUANHUNHE"}, "162208": {"suoxie": "TDHLSXQYGP", "name": "\u6cf0\u8fbe\u5b8f\u5229\u9996\u9009\u4f01\u4e1a\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "TAIDAHONGLISHOUXUANQIYEGUPIAO"}, "162209": {"suoxie": "TDHLSZYXHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u5e02\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLISHIZHIYOUXUANHUNHE"}, "162210": {"suoxie": "TDHLJLZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u96c6\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIDAHONGLIJILIZHAIQUANA"}, "162211": {"suoxie": "TDHLPZSHHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u54c1\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIDAHONGLIPINZHISHENGHUOHUNHE"}, "162212": {"suoxie": "TDHLHLXFHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u7ea2\u5229\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLIHONGLIXIANFENGHUNHE"}, "162213": {"suoxie": "TDHLHS300ZSZQA", "name": "\u6cf0\u8fbe\u5b8f\u5229\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAHONGLIHUSHEN300ZHISHUZENGQIANGA"}, "162214": {"suoxie": "TDHLLXZXPHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u9886\u5148\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLILINGXIANZHONGXIAOPANHUNHE"}, "162215": {"suoxie": "TDHLJLZQLOF", "name": "\u6cf0\u8fbe\u5b8f\u5229\u805a\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIDAHONGLIJULIZHAIQUANLOF"}, "162216": {"suoxie": "TDZZ500ZSZQLOF", "name": "\u6cf0\u8fbe\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3a(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIDAZHONGZHENG500ZHISHUZENGQIANGLOF"}, "162299": {"suoxie": "TDHLJLZQC", "name": "\u6cf0\u8fbe\u5b8f\u5229\u96c6\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIDAHONGLIJILIZHAIQUANC"}, "162307": {"suoxie": "HFTZZ100ZSLOFA", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1100\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENG100ZHISHULOFA"}, "162411": {"suoxie": "HBBPYQSYGPRMBA", "name": "\u534e\u5b9d\u6807\u666e\u6cb9\u6c14\u4e0a\u6e38\u80a1\u7968\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUABAOBIAOPUYOUQISHANGYOUGUPIAORENMINBIA"}, "162412": {"suoxie": "HBYLETFLJA", "name": "\u534e\u5b9d\u533b\u7597ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOYILIAOETFLIANJIEA"}, "162413": {"suoxie": "HBZZ1000ZS", "name": "\u534e\u5b9d\u4e2d\u8bc11000\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG1000ZHISHU"}, "162414": {"suoxie": "HBXJYHHLOFA", "name": "\u534e\u5b9d\u65b0\u673a\u9047\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOXINJIYUHUNHELOFA"}, "162415": {"suoxie": "HBBPMGXFRMBA", "name": "\u534e\u5b9d\u6807\u666e\u7f8e\u56fd\u6d88\u8d39\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "HUABAOBIAOPUMEIGUOXIAOFEIRENMINBIA"}, "162416": {"suoxie": "HBGGTHSXG35", "name": "\u534e\u5b9d\u6e2f\u80a1\u901a\u6052\u751f\u9999\u6e2f35", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOGANGGUTONGHENGSHENGXIANGGANG35"}, "162509": {"suoxie": "GLAZZ100ZSLOF", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1100\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENG100ZHISHULOF"}, "162510": {"suoxie": "GLAZXZZLOF", "name": "\u56fd\u8054\u5b89\u4e2d\u5c0f\u7efc\u6307(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGXIAOZONGZHILOF"}, "162511": {"suoxie": "GLASJXYZQLOF", "name": "\u56fd\u8054\u5b89\u53cc\u4f73\u4fe1\u7528\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANSHUANGJIAXINYONGZHAIQUANLOF"}, "162605": {"suoxie": "JSCCDYHHLOF", "name": "\u666f\u987a\u957f\u57ce\u9f0e\u76ca\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGDINGYIHUNHELOF"}, "162606": {"suoxie": "JSCCDYHHLOF", "name": "\u666f\u987a\u957f\u57ce\u9f0e\u76ca\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGDINGYIHUNHELOF"}, "162607": {"suoxie": "JSCCZYLDHH", "name": "\u666f\u987a\u957f\u57ce\u8d44\u6e90\u5784\u65ad\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGZIYUANLONGDUANHUNHE"}, "162703": {"suoxie": "GFXPCZHHLOFA", "name": "\u5e7f\u53d1\u5c0f\u76d8\u6210\u957f\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOPANCHENGZHANGHUNHELOFA"}, "162711": {"suoxie": "GFZZ500ETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ETFLIANJIEA"}, "162712": {"suoxie": "GFJLZQLOFA", "name": "\u5e7f\u53d1\u805a\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJULIZHAIQUANLOFA"}, "162713": {"suoxie": "GFZZ500ETFLJA", "name": "\u5e7f\u53d1\u4e2d\u8bc1500ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ETFLIANJIEA"}, "162714": {"suoxie": "GFSZ100ZSLOFA", "name": "\u5e7f\u53d1\u6df1\u8bc1100\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFASHENZHENG100ZHISHULOFA"}, "162715": {"suoxie": "GFJYZQLOFA", "name": "\u5e7f\u53d1\u805a\u6e90\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJUYUANZHAIQUANLOFA"}, "162716": {"suoxie": "GFJYZQLOFC", "name": "\u5e7f\u53d1\u805a\u6e90\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJUYUANZHAIQUANLOFC"}, "162717": {"suoxie": "GFZRZZTLOFA", "name": "\u5e7f\u53d1\u518d\u878d\u8d44\u4e3b\u9898(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAZAIRONGZIZHUTILOFA"}, "162719": {"suoxie": "GFDQSSYZSRMBA", "name": "\u5e7f\u53d1\u9053\u743c\u65af\u77f3\u6cb9\u6307\u6570\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "GUANGFADAOQIONGSISHIYOUZHISHURENMINBIA"}, "162720": {"suoxie": "GFCYBLNDKHH", "name": "\u5e7f\u53d1\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "162907": {"suoxie": "TXZZRLJBM400ZSLOF", "name": "\u6cf0\u4fe1\u4e2d\u8bc1\u9510\u8054\u57fa\u672c\u9762400\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIXINZHONGZHENGRUILIANJIBENMIAN400ZHISHULOF"}, "163001": {"suoxie": "CXYLBJHHLOFA", "name": "\u957f\u4fe1\u533b\u7597\u4fdd\u5065\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINYILIAOBAOJIANHUNHELOFA"}, "163003": {"suoxie": "CXLXZQLOFC", "name": "\u957f\u4fe1\u5229\u946b\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIXINZHAIQUANLOFC"}, "163005": {"suoxie": "CXLZZQLOFC", "name": "\u957f\u4fe1\u5229\u4f17\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIZHONGZHAIQUANLOFC"}, "163007": {"suoxie": "CXLZZQLOFA", "name": "\u957f\u4fe1\u5229\u4f17\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIZHONGZHAIQUANLOFA"}, "163008": {"suoxie": "CXLXZQLOFA", "name": "\u957f\u4fe1\u5229\u946b\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIXINZHAIQUANLOFA"}, "163109": {"suoxie": "SWLXSZCFZSLOF", "name": "\u7533\u4e07\u83f1\u4fe1\u6df1\u8bc1\u6210\u4efd\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINSHENZHENGCHENGFENZHISHULOF"}, "163110": {"suoxie": "SWLXLHXPGPLOFA", "name": "\u7533\u4e07\u83f1\u4fe1\u91cf\u5316\u5c0f\u76d8\u80a1\u7968(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "SHENWANLINGXINLIANGHUAXIAOPANGUPIAOLOFA"}, "163111": {"suoxie": "SWLXZXQY100ZSLOFA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u5c0f\u4f01\u4e1a100\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGXIAOQIYE100ZHISHULOFA"}, "163113": {"suoxie": "SWLXZZSWZQHYZS", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u7533\u4e07\u8bc1\u5238\u884c\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGSHENWANZHENGQUANHANGYEZHISHU"}, "163114": {"suoxie": "SWLXZZHBCYZSLOFA", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGHUANBAOCHANYEZHISHULOFA"}, "163115": {"suoxie": "SWLXZZJGZSLOF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u519b\u5de5\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGJUNGONGZHISHULOF"}, "163116": {"suoxie": "SWZZSWDZHYTZZSLOFA", "name": "\u7533\u4e07\u4e2d\u8bc1\u7533\u4e07\u7535\u5b50\u884c\u4e1a\u6295\u8d44\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANZHONGZHENGSHENWANDIANZIHANGYETOUZIZHISHULOFA"}, "163118": {"suoxie": "SWLXZZSWYYSWZSLOF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u7533\u4e07\u533b\u836f\u751f\u7269\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGSHENWANYIYAOSHENGWUZHISHULOF"}, "163208": {"suoxie": "NAYQNY", "name": "\u8bfa\u5b89\u6cb9\u6c14\u80fd\u6e90", "style": "QDII", "pinyin": "NUOANYOUQINENGYUAN"}, "163209": {"suoxie": "NACYBZSZQLOFA", "name": "\u8bfa\u5b89\u521b\u4e1a\u677f\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANCHUANGYEBANZHISHUZENGQIANGLOFA"}, "163210": {"suoxie": "NACZDKZA", "name": "\u8bfa\u5b89\u7eaf\u503a\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANCHUNZHAIDINGKAIZHAIA"}, "163211": {"suoxie": "NACZDKZC", "name": "\u8bfa\u5b89\u7eaf\u503a\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NUOANCHUNZHAIDINGKAIZHAIC"}, "163302": {"suoxie": "DMZYYXHHLOF", "name": "\u5927\u6469\u8d44\u6e90\u4f18\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOZIYUANYOUXUANHUNHELOF"}, "163402": {"suoxie": "XQQSTZHHLOF", "name": "\u5174\u5168\u8d8b\u52bf\u6295\u8d44\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANQUSHITOUZIHUNHELOF"}, "163403": {"suoxie": "XQQSTZHHLOF", "name": "\u5174\u5168\u8d8b\u52bf\u6295\u8d44\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANQUSHITOUZIHUNHELOF"}, "163406": {"suoxie": "XQHRHHLOF", "name": "\u5174\u5168\u5408\u6da6\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHERUNHUNHELOF"}, "163407": {"suoxie": "XQHS300ZSLOFA", "name": "\u5174\u5168\u6caa\u6df1300\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGQUANHUSHEN300ZHISHULOFA"}, "163408": {"suoxie": "XQHS300ZSLOFA", "name": "\u5174\u5168\u6caa\u6df1300\u6307\u6570(LOF)A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGQUANHUSHEN300ZHISHULOFA"}, "163409": {"suoxie": "XQLSTZHHLOF", "name": "\u5174\u5168\u7eff\u8272\u6295\u8d44\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANLVSETOUZIHUNHELOF"}, "163410": {"suoxie": "XQLSTZHHLOF", "name": "\u5174\u5168\u7eff\u8272\u6295\u8d44\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANLVSETOUZIHUNHELOF"}, "163411": {"suoxie": "XQJXHH", "name": "\u5174\u5168\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANJINGXUANHUNHE"}, "163412": {"suoxie": "XQQZCHHLOF", "name": "\u5174\u5168\u8f7b\u8d44\u4ea7\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANQINGZICHANHUNHELOF"}, "163413": {"suoxie": "XQQZCHHLOF", "name": "\u5174\u5168\u8f7b\u8d44\u4ea7\u6df7\u5408(LOF)(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANQINGZICHANHUNHELOF"}, "163415": {"suoxie": "XQSYMSYXHH", "name": "\u5174\u5168\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANSHANGYEMOSHIYOUXUANHUNHE"}, "163416": {"suoxie": "XQSYMSYXHH", "name": "\u5174\u5168\u5546\u4e1a\u6a21\u5f0f\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANSHANGYEMOSHIYOUXUANHUNHE"}, "163417": {"suoxie": "XQHYHHLOFA", "name": "\u5174\u5168\u5408\u5b9c\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANHEYIHUNHELOFA"}, "163418": {"suoxie": "XQHXLNFBYZHHLOFA", "name": "\u5174\u5168\u5408\u5174\u4e24\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANHEXINGLIANGNIANFENGBIYUNZUOHUNHELOFA"}, "163503": {"suoxie": "TZHXCZHHLOF", "name": "\u5929\u6cbb\u6838\u5fc3\u6210\u957f\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANZHIHEXINCHENGZHANGHUNHELOF"}, "163801": {"suoxie": "ZYZGHHLOFA", "name": "\u4e2d\u94f6\u4e2d\u56fd\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHONGGUOHUNHELOFA"}, "163802": {"suoxie": "ZYHBA", "name": "\u4e2d\u94f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINHUOBIA"}, "163803": {"suoxie": "ZYCXZZHHA", "name": "\u4e2d\u94f6\u6301\u7eed\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCHIXUZENGZHANGHUNHEA"}, "163804": {"suoxie": "ZYSYHHA", "name": "\u4e2d\u94f6\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINSHOUYIHUNHEA"}, "163805": {"suoxie": "ZYCLHH", "name": "\u4e2d\u94f6\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINCELUEHUNHE"}, "163806": {"suoxie": "ZYZLZQ", "name": "\u4e2d\u94f6\u589e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINZENGLIZHAIQUAN"}, "163807": {"suoxie": "ZYYXLHPZHHA", "name": "\u4e2d\u94f6\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINYOUXUANLINGHUOPEIZHIHUNHEA"}, "163808": {"suoxie": "ZYZZ100ZSZQ", "name": "\u4e2d\u94f6\u4e2d\u8bc1100\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG100ZHISHUZENGQIANG"}, "163809": {"suoxie": "ZYLCHH", "name": "\u4e2d\u94f6\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINLANCHOUHUNHE"}, "163810": {"suoxie": "ZYJZHH", "name": "\u4e2d\u94f6\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINJIAZHIHUNHE"}, "163811": {"suoxie": "ZYSLZQA", "name": "\u4e2d\u94f6\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINSHUANGLIZHAIQUANA"}, "163812": {"suoxie": "ZYSLZQB", "name": "\u4e2d\u94f6\u53cc\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINSHUANGLIZHAIQUANB"}, "163813": {"suoxie": "ZYQQCL", "name": "\u4e2d\u94f6\u5168\u7403\u7b56\u7565", "style": "QDII", "pinyin": "ZHONGYINQUANQIUCELUE"}, "163816": {"suoxie": "ZYZZZQZQA", "name": "\u4e2d\u94f6\u8f6c\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGYINZHUANZHAIZENGQIANGZHAIQUANA"}, "163817": {"suoxie": "ZYZZZQZQB", "name": "\u4e2d\u94f6\u8f6c\u503a\u589e\u5f3a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "ZHONGYINZHUANZHAIZENGQIANGZHAIQUANB"}, "163818": {"suoxie": "ZYZXPCZHH", "name": "\u4e2d\u94f6\u4e2d\u5c0f\u76d8\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHONGXIAOPANCHENGZHANGHUNHE"}, "163819": {"suoxie": "ZYXYZLZQLOFA", "name": "\u4e2d\u94f6\u4fe1\u7528\u589e\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINXINYONGZENGLIZHAIQUANLOFA"}, "163820": {"suoxie": "ZYHBB", "name": "\u4e2d\u94f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGYINHUOBIB"}, "163821": {"suoxie": "ZYHS300DQZZS", "name": "\u4e2d\u94f6\u6caa\u6df1300\u7b49\u6743\u91cd\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINHUSHEN300DENGQUANZHONGZHISHU"}, "163822": {"suoxie": "ZYZTCLHH", "name": "\u4e2d\u94f6\u4e3b\u9898\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYINZHUTICELUEHUNHE"}, "163823": {"suoxie": "ZYWJCLHH", "name": "\u4e2d\u94f6\u7a33\u5065\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINWENJIANCELUEHUNHE"}, "163825": {"suoxie": "ZYHLBNDKZ", "name": "\u4e2d\u94f6\u4e92\u5229\u534a\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINHULIBANNIANDINGKAIZHAI"}, "163827": {"suoxie": "ZYCYZZQA", "name": "\u4e2d\u94f6\u4ea7\u4e1a\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINCHANYEZHAIZHAIQUANA"}, "163907": {"suoxie": "ZHHYCZFQS", "name": "\u4e2d\u6d77\u60e0\u88d5\u7eaf\u503a\u53d1\u8d77\u5f0f", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGHAIHUIYUCHUNZHAIFAQISHI"}, "164105": {"suoxie": "HFQHHBZQLOF", "name": "\u534e\u5bcc\u5f3a\u5316\u56de\u62a5\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUQIANGHUAHUIBAOZHAIQUANLOF"}, "164205": {"suoxie": "THWHXXCY", "name": "\u5929\u5f18\u6587\u5316\u65b0\u5174\u4ea7\u4e1a", "style": "\u80a1\u7968\u578b", "pinyin": "TIANHONGWENHUAXINXINGCHANYE"}, "164206": {"suoxie": "THTLZQLOFC", "name": "\u5929\u5f18\u6dfb\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGTIANLIZHAIQUANLOFC"}, "164208": {"suoxie": "THFLZQLOF", "name": "\u5929\u5f18\u4e30\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGFENGLIZHAIQUANLOF"}, "164210": {"suoxie": "THTLZQLOFC", "name": "\u5929\u5f18\u540c\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANHONGTONGLIZHAIQUANLOFC"}, "164304": {"suoxie": "XHZZHBCYZS", "name": "\u65b0\u534e\u4e2d\u8bc1\u73af\u4fdd\u4ea7\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAZHONGZHENGHUANBAOCHANYEZHISHU"}, "164401": {"suoxie": "QHKYZZJKCYZS", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u8bc1\u5065\u5eb7\u4ea7\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGZHENGJIANKANGCHANYEZHISHU"}, "164402": {"suoxie": "QHKYZHJGZSA", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u822a\u519b\u5de5\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGHANGJUNGONGZHISHUA"}, "164403": {"suoxie": "QHKYHGSNYHH", "name": "\u524d\u6d77\u5f00\u6e90\u6caa\u6e2f\u6df1\u519c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "QIANHAIKAIYUANHUGANGSHENNONGYEHUNHE"}, "164508": {"suoxie": "GFZZ100ZSZQLOF", "name": "\u56fd\u5bcc\u4e2d\u8bc1100\u6307\u6570\u589e\u5f3a(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOFUZHONGZHENG100ZHISHUZENGQIANGLOF"}, "164509": {"suoxie": "GFHLZQLOFA", "name": "\u56fd\u5bcc\u6052\u5229\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOFUHENGLIZHAIQUANLOFA"}, "164510": {"suoxie": "GFHLZQLOFC", "name": "\u56fd\u5bcc\u6052\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOFUHENGLIZHAIQUANLOFC"}, "164606": {"suoxie": "HTBRXYZLZLOFA", "name": "\u534e\u6cf0\u67cf\u745e\u4fe1\u7528\u589e\u5229\u503a(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIXINYONGZENGLIZHAILOFA"}, "164701": {"suoxie": "HTFHJJGJS", "name": "\u6c47\u6dfb\u5bcc\u9ec4\u91d1\u53ca\u8d35\u91d1\u5c5e", "style": "QDII", "pinyin": "HUITIANFUHUANGJINJIGUIJINSHU"}, "164702": {"suoxie": "HTFJJHDKZ", "name": "\u6c47\u6dfb\u5bcc\u5b63\u5b63\u7ea2\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUJIJIHONGDINGKAIZHAI"}, "164703": {"suoxie": "HTFCZZQLOF", "name": "\u6c47\u6dfb\u5bcc\u7eaf\u503a\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUCHUNZHAIZHAIQUANLOF"}, "164705": {"suoxie": "HTFHSZSA", "name": "\u6c47\u6dfb\u5bcc\u6052\u751f\u6307\u6570A", "style": "QDII", "pinyin": "HUITIANFUHENGSHENGZHISHUA"}, "164808": {"suoxie": "GYSJSYZQLOF", "name": "\u5de5\u94f6\u56db\u5b63\u6536\u76ca\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINSIJISHOUYIZHAIQUANLOF"}, "164809": {"suoxie": "GYZZ500ETFLJA", "name": "\u5de5\u94f6\u4e2d\u8bc1500ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENG500ETFLIANJIEA"}, "164810": {"suoxie": "GYCZDKZ", "name": "\u5de5\u94f6\u7eaf\u503a\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINCHUNZHAIDINGKAIZHAI"}, "164811": {"suoxie": "GYZZJJJXTFZZSA", "name": "\u5de5\u94f6\u4e2d\u8bc1\u4eac\u6d25\u5180\u534f\u540c\u53d1\u5c55\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGJINGJINJIXIETONGFAZHANZHISHUA"}, "164814": {"suoxie": "GYSZZQZQ", "name": "\u5de5\u94f6\u53cc\u503a\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINSHUANGZHAIZENGQIANGZHAIQUAN"}, "164818": {"suoxie": "GYZZCMZSLOFA", "name": "\u5de5\u94f6\u4e2d\u8bc1\u4f20\u5a92\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGCHUANMEIZHISHULOFA"}, "164824": {"suoxie": "GYYDJJRMB", "name": "\u5de5\u94f6\u5370\u5ea6\u57fa\u91d1\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GONGYINYINDUJIJINRENMINBI"}, "164825": {"suoxie": "GYZZJJJXTFZZSC", "name": "\u5de5\u94f6\u4e2d\u8bc1\u4eac\u6d25\u5180\u534f\u540c\u53d1\u5c55\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGJINGJINJIXIETONGFAZHANZHISHUC"}, "164826": {"suoxie": "GYCYBLNDKHHA", "name": "\u5de5\u94f6\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINCHUANGYEBANLIANGNIANDINGKAIHUNHEA"}, "164902": {"suoxie": "JYXYTLZQLOF", "name": "\u4ea4\u94f6\u4fe1\u7528\u6dfb\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINXINYONGTIANLIZHAIQUANLOF"}, "164903": {"suoxie": "JYXYTLZQLOF", "name": "\u4ea4\u94f6\u4fe1\u7528\u6dfb\u5229\u503a\u5238(LOF)(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINXINYONGTIANLIZHAIQUANLOF"}, "164905": {"suoxie": "JYGZXNYZSLOFA", "name": "\u4ea4\u94f6\u56fd\u8bc1\u65b0\u80fd\u6e90\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINGUOZHENGXINNENGYUANZHISHULOFA"}, "164906": {"suoxie": "JYZZHWZGHLWZS", "name": "\u4ea4\u94f6\u4e2d\u8bc1\u6d77\u5916\u4e2d\u56fd\u4e92\u8054\u7f51\u6307\u6570", "style": "QDII", "pinyin": "JIAOYINZHONGZHENGHAIWAIZHONGGUOHULIANWANGZHISHU"}, "164908": {"suoxie": "JYZZHJZLLOFA", "name": "\u4ea4\u94f6\u4e2d\u8bc1\u73af\u5883\u6cbb\u7406(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINZHONGZHENGHUANJINGZHILILOFA"}, "165309": {"suoxie": "JXHS300ZSLOF", "name": "\u5efa\u4fe1\u6caa\u6df1300\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300ZHISHULOF"}, "165310": {"suoxie": "JXHS300ZSZQLOFA", "name": "\u5efa\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300ZHISHUZENGQIANGLOFA"}, "165311": {"suoxie": "JXXYZQZQLOFA", "name": "\u5efa\u4fe1\u4fe1\u7528\u589e\u5f3a\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINXINYONGZENGQIANGZHAIQUANLOFA"}, "165312": {"suoxie": "JXYSCJ50ZSLOF", "name": "\u5efa\u4fe1\u592e\u89c6\u8d22\u7ecf50\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINYANGSHICAIJING50ZHISHULOF"}, "165313": {"suoxie": "JXYSDLHHLOF", "name": "\u5efa\u4fe1\u4f18\u52bf\u52a8\u529b\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINYOUSHIDONGLIHUNHELOF"}, "165314": {"suoxie": "JXXYZQZQLOFC", "name": "\u5efa\u4fe1\u4fe1\u7528\u589e\u5f3a\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINXINYONGZENGQIANGZHAIQUANLOFC"}, "165317": {"suoxie": "JXFYDCLHHLOF", "name": "\u5efa\u4fe1\u4e30\u88d5\u591a\u7b56\u7565\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINFENGYUDUOCELUEHUNHELOF"}, "165508": {"suoxie": "XCSDJZHHLOF", "name": "\u4fe1\u8bda\u6df1\u5ea6\u4ef7\u503c\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGSHENDUJIAZHIHUNHELOF"}, "165509": {"suoxie": "XCZQSYZQLOF", "name": "\u4fe1\u8bda\u589e\u5f3a\u6536\u76ca\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINCHENGZENGQIANGSHOUYIZHAIQUANLOF"}, "165510": {"suoxie": "XCSGPZ", "name": "\u4fe1\u8bda\u56db\u56fd\u914d\u7f6e", "style": "QDII", "pinyin": "XINCHENGSIGUOPEIZHI"}, "165511": {"suoxie": "ZXBCZZ500ZSLOFA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u4e2d\u8bc1500\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGXINBAOCHENGZHONGZHENG500ZHISHULOFA"}, "165512": {"suoxie": "XCXJYHHLOF", "name": "\u4fe1\u8bda\u65b0\u673a\u9047\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGXINJIYUHUNHELOF"}, "165513": {"suoxie": "XCQQSPZT", "name": "\u4fe1\u8bda\u5168\u7403\u5546\u54c1\u4e3b\u9898", "style": "QDII", "pinyin": "XINCHENGQUANQIUSHANGPINZHUTI"}, "165515": {"suoxie": "XCHS300ZSLOFA", "name": "\u4fe1\u8bda\u6caa\u6df1300\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGHUSHEN300ZHISHULOFA"}, "165516": {"suoxie": "XCZQLDHHLOFA", "name": "\u4fe1\u8bda\u5468\u671f\u8f6e\u52a8\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGZHOUQILUNDONGHUNHELOFA"}, "165517": {"suoxie": "XCSYZQLOF", "name": "\u4fe1\u8bda\u53cc\u76c8\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINCHENGSHUANGYINGZHAIQUANLOF"}, "165519": {"suoxie": "XCZZ800YYZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u533b\u836f\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800YIYAOZHISHULOFA"}, "165520": {"suoxie": "XCZZ800YSZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u6709\u8272\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800YOUSEZHISHULOFA"}, "165521": {"suoxie": "XCZZ800JRLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1800\u91d1\u878d(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENG800JINRONGLOFA"}, "165522": {"suoxie": "XCZZTMTCYZTZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1TMT\u4ea7\u4e1a\u4e3b\u9898\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGTMTCHANYEZHUTIZHISHULOFA"}, "165523": {"suoxie": "XCZZXXAQZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u4fe1\u606f\u5b89\u5168\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGXINXIANQUANZHISHULOFA"}, "165524": {"suoxie": "XCZZZNJJZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u667a\u80fd\u5bb6\u5c45\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGZHINENGJIAJUZHISHULOFA"}, "165525": {"suoxie": "XCZZJJGCZSLOFA", "name": "\u4fe1\u8bda\u4e2d\u8bc1\u57fa\u5efa\u5de5\u7a0b\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINCHENGZHONGZHENGJIJIANGONGCHENGZHISHULOFA"}, "165526": {"suoxie": "XCXWHHLOFA", "name": "\u4fe1\u8bda\u65b0\u65fa\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINWANGHUNHELOFA"}, "165527": {"suoxie": "XCXWHHLOFC", "name": "\u4fe1\u8bda\u65b0\u65fa\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGXINWANGHUNHELOFC"}, "165528": {"suoxie": "XCDL", "name": "\u4fe1\u8bda\u9f0e\u5229", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGDINGLI"}, "165530": {"suoxie": "ZXBCHZ18GYDKZQ", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u60e0\u6cfd18\u4e2a\u6708\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINBAOCHENGHUIZE18GEYUEDINGKAIZHAIQUAN"}, "165531": {"suoxie": "XCDCLLHPZHH", "name": "\u4fe1\u8bda\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINCHENGDUOCELUELINGHUOPEIZHIHUNHE"}, "165806": {"suoxie": "DWHS300ZSA", "name": "\u4e1c\u5434\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGWUHUSHEN300ZHISHUA"}, "165810": {"suoxie": "DWHS300ZSC", "name": "\u4e1c\u5434\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGWUHUSHEN300ZHISHUC"}, "166001": {"suoxie": "ZOXQSHHA", "name": "\u4e2d\u6b27\u65b0\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINQUSHIHUNHEA"}, "166002": {"suoxie": "ZOXLCHHA", "name": "\u4e2d\u6b27\u65b0\u84dd\u7b79\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUXINLANCHOUHUNHEA"}, "166005": {"suoxie": "ZOJZFXHHA", "name": "\u4e2d\u6b27\u4ef7\u503c\u53d1\u73b0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUJIAZHIFAXIANHUNHEA"}, "166006": {"suoxie": "ZOHYCZHHLOFA", "name": "\u4e2d\u6b27\u884c\u4e1a\u6210\u957f\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHANGYECHENGZHANGHUNHELOFA"}, "166007": {"suoxie": "ZOHTJXHHA", "name": "\u4e2d\u6b27\u4e92\u901a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHUTONGJINGXUANHUNHEA"}, "166008": {"suoxie": "ZOZQHBZQLOFA", "name": "\u4e2d\u6b27\u589e\u5f3a\u56de\u62a5\u503a\u5238(LOF)A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUZENGQIANGHUIBAOZHAIQUANLOFA"}, "166009": {"suoxie": "ZOXDLHHLOFA", "name": "\u4e2d\u6b27\u65b0\u52a8\u529b\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUXINDONGLIHUNHELOFA"}, "166010": {"suoxie": "ZODLZQA", "name": "\u4e2d\u6b27\u9f0e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGOUDINGLIZHAIQUANA"}, "166011": {"suoxie": "ZOSSCZHHLOFA", "name": "\u4e2d\u6b27\u76db\u4e16\u6210\u957f\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUSHENGSHICHENGZHANGHUNHELOFA"}, "166012": {"suoxie": "ZOXYZLZQLOFC", "name": "\u4e2d\u6b27\u4fe1\u7528\u589e\u5229\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUXINYONGZENGLIZHAIQUANLOFC"}, "166014": {"suoxie": "ZOHBA", "name": "\u4e2d\u6b27\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUHUOBIA"}, "166015": {"suoxie": "ZOHBB", "name": "\u4e2d\u6b27\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGOUHUOBIB"}, "166016": {"suoxie": "ZOCZZQLOFC", "name": "\u4e2d\u6b27\u7eaf\u503a\u503a\u5238(LOF)C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGOUCHUNZHAIZHAIQUANLOFC"}, "166019": {"suoxie": "ZOJZZXHHA", "name": "\u4e2d\u6b27\u4ef7\u503c\u667a\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUJIAZHIZHIXUANHUNHEA"}, "166020": {"suoxie": "ZOCZYXHHA", "name": "\u4e2d\u6b27\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUCHENGZHANGYOUXUANHUNHEA"}, "166023": {"suoxie": "ZORFLHPZHHLOFA", "name": "\u4e2d\u6b27\u745e\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOURUIFENGLINGHUOPEIZHIHUNHELOFA"}, "166024": {"suoxie": "ZOHLSNDQKFHH", "name": "\u4e2d\u6b27\u6052\u5229\u4e09\u5e74\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHENGLISANNIANDINGQIKAIFANGHUNHE"}, "166025": {"suoxie": "ZOYJLNDKHHA", "name": "\u4e2d\u6b27\u8fdc\u89c1\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUYUANJIANLIANGNIANDINGKAIHUNHEA"}, "166027": {"suoxie": "ZOCYBLNDKHHA", "name": "\u4e2d\u6b27\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHUANGYEBANLIANGNIANDINGKAIHUNHEA"}, "166105": {"suoxie": "XDAYXAZQ", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u946b\u5b89\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAAOYINXINANZHAIQUAN"}, "166107": {"suoxie": "XDAYLHDYZHHLOFA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLIANGHUADUOYINZIHUNHELOFA"}, "166108": {"suoxie": "XDAYLHDYZHHLOFC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u91cf\u5316\u591a\u56e0\u5b50\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLIANGHUADUOYINZIHUNHELOFC"}, "166109": {"suoxie": "XDAYLHXFLOFA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u91cf\u5316\u5148\u950b(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLIANGHUAXIANFENGLOFA"}, "166110": {"suoxie": "XDAYLHXFLOFC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u91cf\u5316\u5148\u950b(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLIANGHUAXIANFENGLOFC"}, "166301": {"suoxie": "HSXQSYXLHPZHH", "name": "\u534e\u5546\u65b0\u8d8b\u52bf\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGXINQUSHIYOUXUANLINGHUOPEIZHIHUNHE"}, "166401": {"suoxie": "PYASWJZLZQC", "name": "\u6d66\u94f6\u5b89\u76db\u7a33\u5065\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGWENJIANZENGLIZHAIQUANC"}, "166402": {"suoxie": "PYASHGSJBM100", "name": "\u6d66\u94f6\u5b89\u76db\u6caa\u6e2f\u6df1\u57fa\u672c\u9762100", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGHUGANGSHENJIBENMIAN100"}, "166801": {"suoxie": "ZSJCXSWHHA", "name": "\u6d59\u5546\u805a\u6f6e\u65b0\u601d\u7ef4\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGJUCHAOXINSIWEIHUNHEA"}, "166802": {"suoxie": "ZSHS300ZSZQLOFA", "name": "\u6d59\u5546\u6caa\u6df1300\u6307\u6570\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGHUSHEN300ZHISHUZENGQIANGLOFA"}, "166803": {"suoxie": "ZSHMCZA", "name": "\u6d59\u5546\u60e0\u6c11\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGHUIMINCHUNZHAIA"}, "166902": {"suoxie": "MSJYPWZLA", "name": "\u6c11\u751f\u52a0\u94f6\u5e73\u7a33\u589e\u5229A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINPINGWENZENGLIA"}, "166903": {"suoxie": "MSJYPWZLC", "name": "\u6c11\u751f\u52a0\u94f6\u5e73\u7a33\u589e\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINPINGWENZENGLIC"}, "166904": {"suoxie": "MSJYPWTLZQA", "name": "\u6c11\u751f\u52a0\u94f6\u5e73\u7a33\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINPINGWENTIANLIZHAIQUANA"}, "166905": {"suoxie": "MSJYPWTLZQC", "name": "\u6c11\u751f\u52a0\u94f6\u5e73\u7a33\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINPINGWENTIANLIZHAIQUANC"}, "167001": {"suoxie": "PADTHHLOF", "name": "\u5e73\u5b89\u9f0e\u6cf0\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANDINGTAIHUNHELOF"}, "167002": {"suoxie": "PADYHHLOF", "name": "\u5e73\u5b89\u9f0e\u8d8a\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANDINGYUEHUNHELOF"}, "167003": {"suoxie": "PADHHHLOFA", "name": "\u5e73\u5b89\u9f0e\u5f18\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PINGANDINGHONGHUNHELOFA"}, "167301": {"suoxie": "FZFBBXZTZS", "name": "\u65b9\u6b63\u5bcc\u90a6\u4fdd\u9669\u4e3b\u9898\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGBAOXIANZHUTIZHISHU"}, "167302": {"suoxie": "FZFBDWQZZLOF", "name": "\u65b9\u6b63\u5bcc\u90a6\u5927\u6e7e\u533a\u7efc\u6307(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGDAWANQUZONGZHILOF"}, "167501": {"suoxie": "AXBLZQLOF", "name": "\u5b89\u4fe1\u5b9d\u5229\u503a\u5238(LOF)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINBAOLIZHAIQUANLOF"}, "167503": {"suoxie": "AXZZYDYLZTZS", "name": "\u5b89\u4fe1\u4e2d\u8bc1\u4e00\u5e26\u4e00\u8def\u4e3b\u9898\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINZHONGZHENGYIDAIYILUZHUTIZHISHU"}, "167504": {"suoxie": "AXZDLLZLOFA", "name": "\u5b89\u4fe1\u4e2d\u77ed\u5229\u7387\u503a(LOF)A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINZHONGDUANLILVZHAILOFA"}, "167505": {"suoxie": "AXZDLLZLOFC", "name": "\u5b89\u4fe1\u4e2d\u77ed\u5229\u7387\u503a(LOF)C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINZHONGDUANLILVZHAILOFC"}, "167506": {"suoxie": "AXSZKJZSLOFA", "name": "\u5b89\u4fe1\u6df1\u5733\u79d1\u6280\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINSHENZHENKEJIZHISHULOFA"}, "167507": {"suoxie": "AXSZKJZSLOFC", "name": "\u5b89\u4fe1\u6df1\u5733\u79d1\u6280\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ANXINSHENZHENKEJIZHISHULOFC"}, "167508": {"suoxie": "AXJZFXLNDKHHLOF", "name": "\u5b89\u4fe1\u4ef7\u503c\u53d1\u73b0\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ANXINJIAZHIFAXIANLIANGNIANDINGKAIHUNHELOF"}, "167601": {"suoxie": "GJHS300ZSZQ", "name": "\u56fd\u91d1\u6caa\u6df1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOJINHUSHEN300ZHISHUZENGQIANG"}, "167702": {"suoxie": "DBLHYXGPLOFA", "name": "\u5fb7\u90a6\u91cf\u5316\u4f18\u9009\u80a1\u7968(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "DEBANGLIANGHUAYOUXUANGUPIAOLOFA"}, "167703": {"suoxie": "DBLHYXGPLOFC", "name": "\u5fb7\u90a6\u91cf\u5316\u4f18\u9009\u80a1\u7968(LOF)C", "style": "\u80a1\u7968\u578b", "pinyin": "DEBANGLIANGHUAYOUXUANGUPIAOLOFC"}, "168002": {"suoxie": "GSABCLJXHHLOF", "name": "\u56fd\u5bff\u5b89\u4fdd\u7b56\u7565\u7cbe\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOCELUEJINGXUANHUNHELOF"}, "168101": {"suoxie": "JTRZSJQDHHLOF", "name": "\u4e5d\u6cf0\u9510\u667a\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIRUIZHISHIJIANQUDONGHUNHELOF"}, "168102": {"suoxie": "JTRFSJQDHHFQSLOFA", "name": "\u4e5d\u6cf0\u9510\u5bcc\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408\u53d1\u8d77\u5f0f(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIUTAIRUIFUSHIJIANQUDONGHUNHEFAQISHILOFA"}, "168103": {"suoxie": "JTRYHHLOF", "name": "\u4e5d\u6cf0\u9510\u76ca\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIRUIYIHUNHELOF"}, "168104": {"suoxie": "JTRFLHPZHHLOFA", "name": "\u4e5d\u6cf0\u9510\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIRUIFENGLINGHUOPEIZHIHUNHELOFA"}, "168105": {"suoxie": "JTTFLHPZHHLOF", "name": "\u4e5d\u6cf0\u6cf0\u5bcc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAITAIFULINGHUOPEIZHIHUNHELOF"}, "168111": {"suoxie": "JTRFLHPZHHLOFC", "name": "\u4e5d\u6cf0\u9510\u4e30\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIUTAIRUIFENGLINGHUOPEIZHIHUNHELOFC"}, "168203": {"suoxie": "ZRGZGTHYZSLOF", "name": "\u4e2d\u878d\u56fd\u8bc1\u94a2\u94c1\u884c\u4e1a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGGUOZHENGGANGTIEHANGYEZHISHULOF"}, "168204": {"suoxie": "ZRZZMTZSLOF", "name": "\u4e2d\u878d\u4e2d\u8bc1\u7164\u70ad\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGZHONGZHENGMEITANZHISHULOF"}, "168205": {"suoxie": "ZRZZYHZSLOF", "name": "\u4e2d\u878d\u4e2d\u8bc1\u94f6\u884c\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGZHONGZHENGYINHANGZHISHULOF"}, "168207": {"suoxie": "ZRCYBLNDKHH", "name": "\u4e2d\u878d\u521b\u4e1a\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGRONGCHUANGYEBANLIANGNIANDINGKAIHUNHE"}, "168301": {"suoxie": "DHXLHHLOF", "name": "\u4e1c\u6d77\u7965\u9f99\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIXIANGLONGHUNHELOF"}, "168401": {"suoxie": "HTZXJXHHLOF", "name": "\u7ea2\u571f\u8f6c\u578b\u7cbe\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTUZHUANXINGJINGXUANHUNHELOF"}, "168501": {"suoxie": "BXRFCYSJ", "name": "\u5317\u4fe1\u745e\u4e30\u4ea7\u4e1a\u5347\u7ea7", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BEIXINRUIFENGCHANYESHENGJI"}, "168601": {"suoxie": "HAYYDKHH", "name": "\u6c47\u5b89\u88d5\u9633\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIANYUYANGDINGKAIHUNHE"}, "168701": {"suoxie": "HXZYJRKJZSLOFA", "name": "\u5408\u7166\u667a\u8fdc\u91d1\u878d\u79d1\u6280\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HEXUZHIYUANJINRONGKEJIZHISHULOFA"}, "168702": {"suoxie": "HXZYJRKJZSLOFC", "name": "\u5408\u7166\u667a\u8fdc\u91d1\u878d\u79d1\u6280\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HEXUZHIYUANJINRONGKEJIZHISHULOFC"}, "169101": {"suoxie": "DFHRFHH", "name": "\u4e1c\u65b9\u7ea2\u777f\u4e30\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIFENGHUNHE"}, "169102": {"suoxie": "DFHRYSNHH", "name": "\u4e1c\u65b9\u7ea2\u777f\u9633\u4e09\u5e74\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIYANGSANNIANHUNHE"}, "169103": {"suoxie": "DFHRXSNDKHH", "name": "\u4e1c\u65b9\u7ea2\u777f\u8f69\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIXUANSANNIANDINGKAIHUNHE"}, "169104": {"suoxie": "DFHRMHGSHHLOF", "name": "\u4e1c\u65b9\u7ea2\u777f\u6ee1\u6caa\u6e2f\u6df1\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIMANHUGANGSHENHUNHELOF"}, "169105": {"suoxie": "DFHRHHGSHHLOF", "name": "\u4e1c\u65b9\u7ea2\u777f\u534e\u6caa\u6e2f\u6df1\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIHUAHUGANGSHENHUNHELOF"}, "169106": {"suoxie": "DFHCXYXDKHH", "name": "\u4e1c\u65b9\u7ea2\u521b\u65b0\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGCHUANGXINYOUXUANDINGKAIHUNHE"}, "169107": {"suoxie": "DFHHYWNDKHH", "name": "\u4e1c\u65b9\u7ea2\u6052\u9633\u4e94\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGHENGYANGWUNIANDINGKAIHUNHE"}, "169108": {"suoxie": "DFHJHYXDKHH", "name": "\u4e1c\u65b9\u7ea2\u5747\u8861\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGJUNHENGYOUXUANDINGKAIHUNHE"}, "169109": {"suoxie": "DFHRHSNDKHHA", "name": "\u4e1c\u65b9\u7ea2\u777f\u548c\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGRUIHESANNIANDINGKAIHUNHEA"}, "169201": {"suoxie": "ZSDYSJQDHH", "name": "\u6d59\u5546\u9f0e\u76c8\u4e8b\u4ef6\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHESHANGDINGYINGSHIJIANQUDONGHUNHE"}, "180001": {"suoxie": "YHYSQYHH", "name": "\u94f6\u534e\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YINHUAYOUSHIQIYEHUNHE"}, "180003": {"suoxie": "YHDQS88ZS", "name": "\u94f6\u534e-\u9053\u743c\u65af88\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUADAOQIONGSI88ZHISHU"}, "180008": {"suoxie": "YHHBA", "name": "\u94f6\u534e\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOBIA"}, "180009": {"suoxie": "YHHBB", "name": "\u94f6\u534e\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUAHUOBIB"}, "180010": {"suoxie": "YHYZZZHH", "name": "\u94f6\u534e\u4f18\u8d28\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAYOUZHIZENGZHANGHUNHE"}, "180011": {"suoxie": "YHYSQYHH", "name": "\u94f6\u534e\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "YINHUAYOUSHIQIYEHUNHE"}, "180012": {"suoxie": "YHFYZTHH", "name": "\u94f6\u534e\u5bcc\u88d5\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFUYUZHUTIHUNHE"}, "180013": {"suoxie": "YHLXCLHH", "name": "\u94f6\u534e\u9886\u5148\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUALINGXIANCELUEHUNHE"}, "180015": {"suoxie": "YHZQSYZQ", "name": "\u94f6\u534e\u589e\u5f3a\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHUAZENGQIANGSHOUYIZHAIQUAN"}, "180018": {"suoxie": "YHHXZTHH", "name": "\u94f6\u534e\u548c\u8c10\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAHEXIEZHUTIHUNHE"}, "180020": {"suoxie": "YHCZXFHH", "name": "\u94f6\u534e\u6210\u957f\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUACHENGZHANGXIANFENGHUNHE"}, "180025": {"suoxie": "YHXYSLZQA", "name": "\u94f6\u534e\u4fe1\u7528\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHUAXINYONGSHUANGLIZHAIQUANA"}, "180026": {"suoxie": "YHXYSLZQC", "name": "\u94f6\u534e\u4fe1\u7528\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHUAXINYONGSHUANGLIZHAIQUANC"}, "180028": {"suoxie": "YHYXLHPZHH", "name": "\u94f6\u534e\u6c38\u7965\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAYONGXIANGLINGHUOPEIZHIHUNHE"}, "180031": {"suoxie": "YHZXPHH", "name": "\u94f6\u534e\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAZHONGXIAOPANHUNHE"}, "180101": {"suoxie": "BSSKCYREIT", "name": "\u535a\u65f6\u86c7\u53e3\u4ea7\u56edREIT", "style": "Reits", "pinyin": ""}, "180201": {"suoxie": "PAGZGHREIT", "name": "\u5e73\u5b89\u5e7f\u5dde\u5e7f\u6cb3REIT", "style": "Reits", "pinyin": ""}, "180202": {"suoxie": "HXYXGSREIT", "name": "\u534e\u590f\u8d8a\u79c0\u9ad8\u901fREIT", "style": "Reits", "pinyin": ""}, "180301": {"suoxie": "HTYTGREIT", "name": "\u7ea2\u571f\u76d0\u7530\u6e2fREIT", "style": "Reits", "pinyin": ""}, "180801": {"suoxie": "ZHSGLNREIT", "name": "\u4e2d\u822a\u9996\u94a2\u7eff\u80fdREIT", "style": "Reits", "pinyin": ""}, "183001": {"suoxie": "YHQQYX", "name": "\u94f6\u534e\u5168\u7403\u4f18\u9009", "style": "QDII", "pinyin": "YINHUAQUANQIUYOUXUAN"}, "184801": {"suoxie": "PHQHWKREITS", "name": "\u9e4f\u534e\u524d\u6d77\u4e07\u79d1REITS", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "PENGHUAQIANHAIWANKEREITS"}, "200001": {"suoxie": "CCJHLHPZHH", "name": "\u957f\u57ce\u4e45\u6052\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUHENGLINGHUOPEIZHIHUNHE"}, "200002": {"suoxie": "CCJTHS300ZSA", "name": "\u957f\u57ce\u4e45\u6cf0\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGJIUTAIHUSHEN300ZHISHUA"}, "200003": {"suoxie": "CCHBA", "name": "\u957f\u57ce\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGHUOBIA"}, "200006": {"suoxie": "CCXFZZHH", "name": "\u957f\u57ce\u6d88\u8d39\u589e\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGXIAOFEIZENGZHIHUNHE"}, "200007": {"suoxie": "CCAXHBHH", "name": "\u957f\u57ce\u5b89\u5fc3\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGANXINHUIBAOHUNHE"}, "200008": {"suoxie": "CCPPYXHH", "name": "\u957f\u57ce\u54c1\u724c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGPINPAIYOUXUANHUNHE"}, "200009": {"suoxie": "CCWJZLZQA", "name": "\u957f\u57ce\u7a33\u5065\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGWENJIANZENGLIZHAIQUANA"}, "200010": {"suoxie": "CCSDLHH", "name": "\u957f\u57ce\u53cc\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGSHUANGDONGLIHUNHE"}, "200012": {"suoxie": "CCZXPCZHH", "name": "\u957f\u57ce\u4e2d\u5c0f\u76d8\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGZHONGXIAOPANCHENGZHANGHUNHE"}, "200013": {"suoxie": "CCJJZLZQA", "name": "\u957f\u57ce\u79ef\u6781\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGJIJIZENGLIZHAIQUANA"}, "200015": {"suoxie": "CCYHSJHHA", "name": "\u957f\u57ce\u4f18\u5316\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGYOUHUASHENGJIHUNHEA"}, "200016": {"suoxie": "CCWJCZLHPZHH", "name": "\u957f\u57ce\u7a33\u5065\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGWENJIANCHENGZHANGLINGHUOPEIZHIHUNHE"}, "200103": {"suoxie": "CCHBB", "name": "\u957f\u57ce\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGCHENGHUOBIB"}, "200113": {"suoxie": "CCJJZLZQC", "name": "\u957f\u57ce\u79ef\u6781\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGCHENGJIJIZENGLIZHAIQUANC"}, "201001": {"suoxie": "CCJHLHPZHH", "name": "\u957f\u57ce\u4e45\u6052\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGCHENGJIUHENGLINGHUOPEIZHIHUNHE"}, "201002": {"suoxie": "CCJTHS300ZSA", "name": "\u957f\u57ce\u4e45\u6cf0\u6caa\u6df1300\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGCHENGJIUTAIHUSHEN300ZHISHUA"}, "202001": {"suoxie": "NFWJCZHH", "name": "\u5357\u65b9\u7a33\u5065\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGWENJIANCHENGZHANGHUNHE"}, "202002": {"suoxie": "NFWJCZEHHH", "name": "\u5357\u65b9\u7a33\u5065\u6210\u957f\u8d30\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGWENJIANCHENGZHANGERHAOHUNHE"}, "202003": {"suoxie": "NFJYCZHHA", "name": "\u5357\u65b9\u7ee9\u4f18\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIYOUCHENGZHANGHUNHEA"}, "202004": {"suoxie": "NFJYCZHHA", "name": "\u5357\u65b9\u7ee9\u4f18\u6210\u957f\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGJIYOUCHENGZHANGHUNHEA"}, "202005": {"suoxie": "NFCFJXHHA", "name": "\u5357\u65b9\u6210\u4efd\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHENGFENJINGXUANHUNHEA"}, "202006": {"suoxie": "NFCFJXHHA", "name": "\u5357\u65b9\u6210\u4efd\u7cbe\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCHENGFENJINGXUANHUNHEA"}, "202007": {"suoxie": "NFLYCYZTHH", "name": "\u5357\u65b9\u9686\u5143\u4ea7\u4e1a\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLONGYUANCHANYEZHUTIHUNHE"}, "202008": {"suoxie": "NFLYCYZTHH", "name": "\u5357\u65b9\u9686\u5143\u4ea7\u4e1a\u4e3b\u9898\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGLONGYUANCHANYEZHUTIHUNHE"}, "202009": {"suoxie": "NFSYHLHH", "name": "\u5357\u65b9\u76db\u5143\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGSHENGYUANHONGLIHUNHE"}, "202010": {"suoxie": "NFSYHLHH", "name": "\u5357\u65b9\u76db\u5143\u7ea2\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGSHENGYUANHONGLIHUNHE"}, "202011": {"suoxie": "NFYXJZHHA", "name": "\u5357\u65b9\u4f18\u9009\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYOUXUANJIAZHIHUNHEA"}, "202012": {"suoxie": "NFYXJZHHA", "name": "\u5357\u65b9\u4f18\u9009\u4ef7\u503c\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGYOUXUANJIAZHIHUNHEA"}, "202015": {"suoxie": "NFHS300LJJJA", "name": "\u5357\u65b9\u6caa\u6df1300\u8054\u63a5\u57fa\u91d1A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300LIANJIEJIJINA"}, "202016": {"suoxie": "NFHS300LJJJA", "name": "\u5357\u65b9\u6caa\u6df1300\u8054\u63a5\u57fa\u91d1A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHUSHEN300LIANJIEJIJINA"}, "202017": {"suoxie": "NFSZCFETFLJA", "name": "\u5357\u65b9\u6df1\u8bc1\u6210\u4efdETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHENZHENGCHENGFENETFLIANJIEA"}, "202018": {"suoxie": "NFSZCFETFLJA", "name": "\u5357\u65b9\u6df1\u8bc1\u6210\u4efdETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHENZHENGCHENGFENETFLIANJIEA"}, "202019": {"suoxie": "NFCLYHHH", "name": "\u5357\u65b9\u7b56\u7565\u4f18\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCELUEYOUHUAHUNHE"}, "202020": {"suoxie": "NFCLYHHH", "name": "\u5357\u65b9\u7b56\u7565\u4f18\u5316\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGCELUEYOUHUAHUNHE"}, "202021": {"suoxie": "NFXKETFLJA", "name": "\u5357\u65b9\u5c0f\u5eb7ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGXIAOKANGETFLIANJIEA"}, "202022": {"suoxie": "NFXKETFLJA", "name": "\u5357\u65b9\u5c0f\u5eb7ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGXIAOKANGETFLIANJIEA"}, "202023": {"suoxie": "NFYXCZHHA", "name": "\u5357\u65b9\u4f18\u9009\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYOUXUANCHENGZHANGHUNHEA"}, "202024": {"suoxie": "NFYXCZHHA", "name": "\u5357\u65b9\u4f18\u9009\u6210\u957f\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGYOUXUANCHENGZHANGHUNHEA"}, "202025": {"suoxie": "NFSZ380ETFLJA", "name": "\u5357\u65b9\u4e0a\u8bc1380ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG380ETFLIANJIEA"}, "202026": {"suoxie": "NFSZ380ETFLJA", "name": "\u5357\u65b9\u4e0a\u8bc1380ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG380ETFLIANJIEA"}, "202027": {"suoxie": "NFGDZBLHPZHHA", "name": "\u5357\u65b9\u9ad8\u7aef\u88c5\u5907\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGGAODUANZHUANGBEILINGHUOPEIZHIHUNHEA"}, "202028": {"suoxie": "NFGDZBLHPZHHA", "name": "\u5357\u65b9\u9ad8\u7aef\u88c5\u5907\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NANFANGGAODUANZHUANGBEILINGHUOPEIZHIHUNHEA"}, "202101": {"suoxie": "NFBYZQA", "name": "\u5357\u65b9\u5b9d\u5143\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGBAOYUANZHAIQUANA"}, "202102": {"suoxie": "NFDLZQZQC", "name": "\u5357\u65b9\u591a\u5229\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGDUOLIZENGQIANGZHAIQUANC"}, "202103": {"suoxie": "NFDLZQZQA", "name": "\u5357\u65b9\u591a\u5229\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGDUOLIZENGQIANGZHAIQUANA"}, "202105": {"suoxie": "NFGLHBZQAB", "name": "\u5357\u65b9\u5e7f\u5229\u56de\u62a5\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGGUANGLIHUIBAOZHAIQUANAB"}, "202106": {"suoxie": "NFGLHBZQAB", "name": "\u5357\u65b9\u5e7f\u5229\u56de\u62a5\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGGUANGLIHUIBAOZHAIQUANAB"}, "202107": {"suoxie": "NFGLHBZQC", "name": "\u5357\u65b9\u5e7f\u5229\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NANFANGGUANGLIHUIBAOZHAIQUANC"}, "202108": {"suoxie": "NFRYCZZQAB", "name": "\u5357\u65b9\u6da6\u5143\u7eaf\u503a\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGRUNYUANCHUNZHAIZHAIQUANAB"}, "202109": {"suoxie": "NFRYCZZQAB", "name": "\u5357\u65b9\u6da6\u5143\u7eaf\u503a\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGRUNYUANCHUNZHAIZHAIQUANAB"}, "202110": {"suoxie": "NFRYCZZQC", "name": "\u5357\u65b9\u6da6\u5143\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NANFANGRUNYUANCHUNZHAIZHAIQUANC"}, "202211": {"suoxie": "NFZZ100ZSA", "name": "\u5357\u65b9\u4e2d\u8bc1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG100ZHISHUA"}, "202212": {"suoxie": "NFPHPZHH", "name": "\u5357\u65b9\u5e73\u8861\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NANFANGPINGHENGPEIZHIHUNHE"}, "202213": {"suoxie": "NFHXJZHH", "name": "\u5357\u65b9\u6838\u5fc3\u7ade\u4e89\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGHEXINJINGZHENGHUNHE"}, "202301": {"suoxie": "NFXJZLHBA", "name": "\u5357\u65b9\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINZENGLIHUOBIA"}, "202302": {"suoxie": "NFXJZLHBB", "name": "\u5357\u65b9\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGXIANJINZENGLIHUOBIB"}, "202305": {"suoxie": "NFWY60TGDCYZDZA", "name": "\u5357\u65b9\u65fa\u514360\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "NANFANGWANGYUAN60TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "202307": {"suoxie": "NFSYBHBA", "name": "\u5357\u65b9\u6536\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGSHOUYIBAOHUOBIA"}, "202308": {"suoxie": "NFSYBHBB", "name": "\u5357\u65b9\u6536\u76ca\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGSHOUYIBAOHUOBIB"}, "202801": {"suoxie": "NFQQJXPZ", "name": "\u5357\u65b9\u5168\u7403\u7cbe\u9009\u914d\u7f6e", "style": "QDII", "pinyin": "NANFANGQUANQIUJINGXUANPEIZHI"}, "206001": {"suoxie": "PHHTA", "name": "\u9e4f\u534e\u5f18\u6cf0A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGTAIA"}, "206002": {"suoxie": "PHJXCZHH", "name": "\u9e4f\u534e\u7cbe\u9009\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAJINGXUANCHENGZHANGHUNHE"}, "206003": {"suoxie": "PHXYZLZQA", "name": "\u9e4f\u534e\u4fe1\u7528\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAXINYONGZENGLIZHAIQUANA"}, "206004": {"suoxie": "PHXYZLZQB", "name": "\u9e4f\u534e\u4fe1\u7528\u589e\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAXINYONGZENGLIZHAIQUANB"}, "206006": {"suoxie": "PHQQZDZQDIIRMBA", "name": "\u9e4f\u534e\u5168\u7403\u4e2d\u77ed\u503a(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "PENGHUAQUANQIUZHONGDUANZHAIQDIIRENMINBIA"}, "206007": {"suoxie": "PHXFYXHH", "name": "\u9e4f\u534e\u6d88\u8d39\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXIAOFEIYOUXUANHUNHE"}, "206008": {"suoxie": "PHFSZQ", "name": "\u9e4f\u534e\u4e30\u76db\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PENGHUAFENGSHENGZHAIQUAN"}, "206009": {"suoxie": "PHXXCYHH", "name": "\u9e4f\u534e\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUAXINXINGCHANYEHUNHE"}, "206011": {"suoxie": "PHMGFDCQDII", "name": "\u9e4f\u534e\u7f8e\u56fd\u623f\u5730\u4ea7(QDII)", "style": "QDII", "pinyin": "PENGHUAMEIGUOFANGDICHANQDII"}, "206012": {"suoxie": "PHJZJXGP", "name": "\u9e4f\u534e\u4ef7\u503c\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "PENGHUAJIAZHIJINGXUANGUPIAO"}, "206013": {"suoxie": "PHHGLHPZHH", "name": "\u9e4f\u534e\u5b8f\u89c2\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAHONGGUANLINGHUOPEIZHIHUNHE"}, "206015": {"suoxie": "PHCZZQ", "name": "\u9e4f\u534e\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUACHUNZHAIZHAIQUAN"}, "206018": {"suoxie": "PHCYZZQ", "name": "\u9e4f\u534e\u4ea7\u4e1a\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PENGHUACHANYEZHAIZHAIQUAN"}, "210001": {"suoxie": "JYCFYXHH", "name": "\u91d1\u9e70\u6210\u4efd\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGCHENGFENYOUXUANHUNHE"}, "210002": {"suoxie": "JYHLJZHH", "name": "\u91d1\u9e70\u7ea2\u5229\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGHONGLIJIAZHIHUNHE"}, "210003": {"suoxie": "JYHYYSHH", "name": "\u91d1\u9e70\u884c\u4e1a\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGHANGYEYOUSHIHUNHE"}, "210004": {"suoxie": "JYWJCZHH", "name": "\u91d1\u9e70\u7a33\u5065\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGWENJIANCHENGZHANGHUNHE"}, "210005": {"suoxie": "JYZTYSHH", "name": "\u91d1\u9e70\u4e3b\u9898\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGZHUTIYOUSHIHUNHE"}, "210006": {"suoxie": "JYYXHHA", "name": "\u91d1\u9e70\u5143\u79a7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "JINYINGYUANXIHUNHEA"}, "210007": {"suoxie": "JYJSLXLHPZHHA", "name": "\u91d1\u9e70\u6280\u672f\u9886\u5148\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGJISHULINGXIANLINGHUOPEIZHIHUNHEA"}, "210008": {"suoxie": "JYCLPZHH", "name": "\u91d1\u9e70\u7b56\u7565\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGCELUEPEIZHIHUNHE"}, "210009": {"suoxie": "JYHXZYHH", "name": "\u91d1\u9e70\u6838\u5fc3\u8d44\u6e90\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYINGHEXINZIYUANHUNHE"}, "210010": {"suoxie": "JYLHPZHHA", "name": "\u91d1\u9e70\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGLINGHUOPEIZHIHUNHEA"}, "210011": {"suoxie": "JYLHPZHHC", "name": "\u91d1\u9e70\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYINGLINGHUOPEIZHIHUNHEC"}, "210012": {"suoxie": "JYHBA", "name": "\u91d1\u9e70\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYINGHUOBIA"}, "210013": {"suoxie": "JYHBB", "name": "\u91d1\u9e70\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYINGHUOBIB"}, "210014": {"suoxie": "JYYFZQA", "name": "\u91d1\u9e70\u5143\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYINGYUANFENGZHAIQUANA"}, "213001": {"suoxie": "BYHLSYLHPZHHA", "name": "\u5b9d\u76c8\u9e3f\u5229\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGHONGLISHOUYILINGHUOPEIZHIHUNHEA"}, "213002": {"suoxie": "BYFYHZZHH", "name": "\u5b9d\u76c8\u6cdb\u6cbf\u6d77\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGFANYANHAIZENGZHANGHUNHE"}, "213003": {"suoxie": "BYCLZZHH", "name": "\u5b9d\u76c8\u7b56\u7565\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGCELUEZENGZHANGHUNHE"}, "213006": {"suoxie": "BYHXYSHHA", "name": "\u5b9d\u76c8\u6838\u5fc3\u4f18\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BAOYINGHEXINYOUSHIHUNHEA"}, "213007": {"suoxie": "BYZQSYZQAB", "name": "\u5b9d\u76c8\u589e\u5f3a\u6536\u76ca\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BAOYINGZENGQIANGSHOUYIZHAIQUANAB"}, "213008": {"suoxie": "BYZYYXHH", "name": "\u5b9d\u76c8\u8d44\u6e90\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGZIYUANYOUXUANHUNHE"}, "213009": {"suoxie": "BYHBA", "name": "\u5b9d\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "BAOYINGHUOBIA"}, "213010": {"suoxie": "BYZZ100ZSZQA", "name": "\u5b9d\u76c8\u4e2d\u8bc1100\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BAOYINGZHONGZHENG100ZHISHUZENGQIANGA"}, "213907": {"suoxie": "BYZQSYZQAB", "name": "\u5b9d\u76c8\u589e\u5f3a\u6536\u76ca\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BAOYINGZENGQIANGSHOUYIZHAIQUANAB"}, "213908": {"suoxie": "BYZYYXHH", "name": "\u5b9d\u76c8\u8d44\u6e90\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BAOYINGZIYUANYOUXUANHUNHE"}, "213909": {"suoxie": "BYHBB", "name": "\u5b9d\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "BAOYINGHUOBIB"}, "213917": {"suoxie": "BYZQSYZQC", "name": "\u5b9d\u76c8\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "BAOYINGZENGQIANGSHOUYIZHAIQUANC"}, "217001": {"suoxie": "ZSATHH", "name": "\u62db\u5546\u5b89\u6cf0\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGANTAIHUNHE"}, "217002": {"suoxie": "ZSATPHHH", "name": "\u62db\u5546\u5b89\u6cf0\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGANTAIPINGHENGHUNHE"}, "217003": {"suoxie": "ZSATZQA", "name": "\u62db\u5546\u5b89\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGANTAIZHAIQUANA"}, "217004": {"suoxie": "ZSXJZZHBA", "name": "\u62db\u5546\u73b0\u91d1\u589e\u503c\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGXIANJINZENGZHIHUOBIA"}, "217005": {"suoxie": "ZSXFHH", "name": "\u62db\u5546\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "ZHAOSHANGXIANFENGHUNHE"}, "217008": {"suoxie": "ZSABZLZQC", "name": "\u62db\u5546\u5b89\u672c\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANBENZENGLIZHAIQUANC"}, "217009": {"suoxie": "ZSHXJZHH", "name": "\u62db\u5546\u6838\u5fc3\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGHEXINJIAZHIHUNHE"}, "217010": {"suoxie": "ZSDPLCHH", "name": "\u62db\u5546\u5927\u76d8\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGDAPANLANCHOUHUNHE"}, "217011": {"suoxie": "ZSAXSYZQC", "name": "\u62db\u5546\u5b89\u5fc3\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANXINSHOUYIZHAIQUANC"}, "217012": {"suoxie": "ZSHYLXHHA", "name": "\u62db\u5546\u884c\u4e1a\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGHANGYELINGXIANHUNHEA"}, "217013": {"suoxie": "ZSZXPHH", "name": "\u62db\u5546\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHAOSHANGZHONGXIAOPANHUNHE"}, "217014": {"suoxie": "ZSXJZZHBB", "name": "\u62db\u5546\u73b0\u91d1\u589e\u503c\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGXIANJINZENGZHIHUOBIB"}, "217016": {"suoxie": "ZSSZ100ZSA", "name": "\u62db\u5546\u6df1\u8bc1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENG100ZHISHUA"}, "217017": {"suoxie": "ZSSZXF80ETFLJA", "name": "\u62db\u5546\u4e0a\u8bc1\u6d88\u8d3980ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHANGZHENGXIAOFEI80ETFLIANJIEA"}, "217018": {"suoxie": "ZSARJQZQ", "name": "\u62db\u5546\u5b89\u745e\u8fdb\u53d6\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANRUIJINQUZHAIQUAN"}, "217019": {"suoxie": "ZSSZTMT50ETFLJA", "name": "\u62db\u5546\u6df1\u8bc1TMT50ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHENZHENGTMT50ETFLIANJIEA"}, "217020": {"suoxie": "ZSADLHPZHH", "name": "\u62db\u5546\u5b89\u8fbe\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGANDALINGHUOPEIZHIHUNHE"}, "217021": {"suoxie": "ZSYSQYHH", "name": "\u62db\u5546\u4f18\u52bf\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGYOUSHIQIYEHUNHE"}, "217022": {"suoxie": "ZSCYZQA", "name": "\u62db\u5546\u4ea7\u4e1a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGCHANYEZHAIQUANA"}, "217023": {"suoxie": "ZSXYZQZQA", "name": "\u62db\u5546\u4fe1\u7528\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGXINYONGZENGQIANGZHAIQUANA"}, "217024": {"suoxie": "ZSAYZQA", "name": "\u62db\u5546\u5b89\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGANYINGZHAIQUANA"}, "217025": {"suoxie": "ZSLC7TZQA", "name": "\u62db\u5546\u7406\u8d227\u5929\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGLICAI7TIANZHAIQUANA"}, "217026": {"suoxie": "ZSLC7TZQB", "name": "\u62db\u5546\u7406\u8d227\u5929\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ZHAOSHANGLICAI7TIANZHAIQUANB"}, "217027": {"suoxie": "ZSYSCJ50ZSA", "name": "\u62db\u5546\u592e\u89c6\u8d22\u7ecf50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGYANGSHICAIJING50ZHISHUA"}, "217203": {"suoxie": "ZSATZQB", "name": "\u62db\u5546\u5b89\u6cf0\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHAOSHANGANTAIZHAIQUANB"}, "229002": {"suoxie": "TDHLNXCLHH", "name": "\u6cf0\u8fbe\u5b8f\u5229\u9006\u5411\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIDAHONGLINIXIANGCELUEHUNHE"}, "233001": {"suoxie": "DMJCHYHH", "name": "\u5927\u6469\u57fa\u7840\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOJICHUHANGYEHUNHE"}, "233005": {"suoxie": "DMQSYZQ", "name": "\u5927\u6469\u5f3a\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DAMOQIANGSHOUYIZHAIQUAN"}, "233006": {"suoxie": "DMLXYSHH", "name": "\u5927\u6469\u9886\u5148\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOLINGXIANYOUSHIHUNHE"}, "233007": {"suoxie": "DMZYCZHH", "name": "\u5927\u6469\u5353\u8d8a\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOZHUOYUECHENGZHANGHUNHE"}, "233008": {"suoxie": "DMXFLHHH", "name": "\u5927\u6469\u6d88\u8d39\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOXIAOFEILINGHANGHUNHE"}, "233009": {"suoxie": "DMDYZCLHH", "name": "\u5927\u6469\u591a\u56e0\u5b50\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMODUOYINZICELUEHUNHE"}, "233010": {"suoxie": "DMSZ300ZSZQ", "name": "\u5927\u6469\u6df1\u8bc1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DAMOSHENZHENG300ZHISHUZENGQIANG"}, "233011": {"suoxie": "DMZTYXHH", "name": "\u5927\u6469\u4e3b\u9898\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOZHUTIYOUXUANHUNHE"}, "233012": {"suoxie": "DMDYSYZQA", "name": "\u5927\u6469\u591a\u5143\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DAMODUOYUANSHOUYIZHAIQUANA"}, "233013": {"suoxie": "DMDYSYZQC", "name": "\u5927\u6469\u591a\u5143\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DAMODUOYUANSHOUYIZHAIQUANC"}, "233015": {"suoxie": "DMLHPZHHA", "name": "\u5927\u6469\u91cf\u5316\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DAMOLIANGHUAPEIZHIHUNHEA"}, "240001": {"suoxie": "HBBKXFP", "name": "\u534e\u5b9d\u5b9d\u5eb7\u6d88\u8d39\u54c1", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUABAOBAOKANGXIAOFEIPIN"}, "240002": {"suoxie": "HBBKPZHH", "name": "\u534e\u5b9d\u5b9d\u5eb7\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUABAOBAOKANGPEIZHIHUNHE"}, "240003": {"suoxie": "HBBKZQA", "name": "\u534e\u5b9d\u5b9d\u5eb7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOBAOKANGZHAIQUANA"}, "240004": {"suoxie": "HBDLZH", "name": "\u534e\u5b9d\u52a8\u529b\u7ec4\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAODONGLIZUHE"}, "240005": {"suoxie": "HBDCLZZKF", "name": "\u534e\u5b9d\u591a\u7b56\u7565\u589e\u957f\u5f00\u653e", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAODUOCELUEZENGZHANGKAIFANG"}, "240006": {"suoxie": "HBXJBHBA", "name": "\u534e\u5b9d\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOXIANJINBAOHUOBIA"}, "240007": {"suoxie": "HBXJBHBB", "name": "\u534e\u5b9d\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOXIANJINBAOHUOBIB"}, "240008": {"suoxie": "HBSYZZHH", "name": "\u534e\u5b9d\u6536\u76ca\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOSHOUYIZENGZHANGHUNHE"}, "240009": {"suoxie": "HBXJCZHH", "name": "\u534e\u5b9d\u5148\u8fdb\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXIANJINCHENGZHANGHUNHE"}, "240010": {"suoxie": "HBHYJXHH", "name": "\u534e\u5b9d\u884c\u4e1a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOHANGYEJINGXUANHUNHE"}, "240011": {"suoxie": "HBDPJXHH", "name": "\u534e\u5b9d\u5927\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAODAPANJINGXUANHUNHE"}, "240012": {"suoxie": "HBZQSYZQA", "name": "\u534e\u5b9d\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOZENGQIANGSHOUYIZHAIQUANA"}, "240013": {"suoxie": "HBZQSYZQB", "name": "\u534e\u5b9d\u589e\u5f3a\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUABAOZENGQIANGSHOUYIZHAIQUANB"}, "240014": {"suoxie": "HBZZ100ZSA", "name": "\u534e\u5b9d\u4e2d\u8bc1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG100ZHISHUA"}, "240015": {"suoxie": "HBZZ100ZSA", "name": "\u534e\u5b9d\u4e2d\u8bc1100\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENG100ZHISHUA"}, "240016": {"suoxie": "HBSZ180JZLJ", "name": "\u534e\u5b9d\u4e0a\u8bc1180\u4ef7\u503c\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHANGZHENG180JIAZHILIANJIE"}, "240017": {"suoxie": "HBXXCYHH", "name": "\u534e\u5b9d\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOXINXINGCHANYEHUNHE"}, "240018": {"suoxie": "HBKZZZQA", "name": "\u534e\u5b9d\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUABAOKEZHUANZHAIZHAIQUANA"}, "240019": {"suoxie": "HBZZYHETFLJA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u94f6\u884cETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGYINHANGETFLIANJIEA"}, "240020": {"suoxie": "HBYYSWHH", "name": "\u534e\u5b9d\u533b\u836f\u751f\u7269\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOYIYAOSHENGWUHUNHE"}, "240022": {"suoxie": "HBZYYXHHA", "name": "\u534e\u5b9d\u8d44\u6e90\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUABAOZIYUANYOUXUANHUNHEA"}, "241001": {"suoxie": "HBHWZGCZHH", "name": "\u534e\u5b9d\u6d77\u5916\u4e2d\u56fd\u6210\u957f\u6df7\u5408", "style": "QDII", "pinyin": "HUABAOHAIWAIZHONGGUOCHENGZHANGHUNHE"}, "253010": {"suoxie": "GLAAXCZHH", "name": "\u56fd\u8054\u5b89\u5b89\u5fc3\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "GUOLIANANANXINCHENGZHANGHUNHE"}, "253020": {"suoxie": "GLAZLZQA", "name": "\u56fd\u8054\u5b89\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANZENGLIZHAIQUANA"}, "253021": {"suoxie": "GLAZLZQB", "name": "\u56fd\u8054\u5b89\u589e\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANZENGLIZHAIQUANB"}, "253030": {"suoxie": "GLAXXZYZQ", "name": "\u56fd\u8054\u5b89\u4fe1\u5fc3\u589e\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANXINXINZENGYIZHAIQUAN"}, "253050": {"suoxie": "GLAHBA", "name": "\u56fd\u8054\u5b89\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOLIANANHUOBIA"}, "253051": {"suoxie": "GLAHBB", "name": "\u56fd\u8054\u5b89\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOLIANANHUOBIB"}, "253060": {"suoxie": "GLAXXZZZQA", "name": "\u56fd\u8054\u5b89\u4fe1\u5fc3\u589e\u957f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANXINXINZENGZHANGZHAIQUANA"}, "253061": {"suoxie": "GLAXXZZZQB", "name": "\u56fd\u8054\u5b89\u4fe1\u5fc3\u589e\u957f\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANANXINXINZENGZHANGZHAIQUANB"}, "255010": {"suoxie": "GLAWJHH", "name": "\u56fd\u8054\u5b89\u7a33\u5065\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOLIANANWENJIANHUNHE"}, "257010": {"suoxie": "GLAXPJXHH", "name": "\u56fd\u8054\u5b89\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANXIAOPANJINGXUANHUNHE"}, "257020": {"suoxie": "GLAJXHH", "name": "\u56fd\u8054\u5b89\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANJINGXUANHUNHE"}, "257021": {"suoxie": "GLAJXHH", "name": "\u56fd\u8054\u5b89\u7cbe\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANJINGXUANHUNHE"}, "257030": {"suoxie": "GLAYSHH", "name": "\u56fd\u8054\u5b89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANYOUSHIHUNHE"}, "257031": {"suoxie": "GLAYSHH", "name": "\u56fd\u8054\u5b89\u4f18\u52bf\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANYOUSHIHUNHE"}, "257040": {"suoxie": "GLAHLHH", "name": "\u56fd\u8054\u5b89\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHONGLIHUNHE"}, "257041": {"suoxie": "GLAHLHH", "name": "\u56fd\u8054\u5b89\u7ea2\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANHONGLIHUNHE"}, "257050": {"suoxie": "GLAZTQDHH", "name": "\u56fd\u8054\u5b89\u4e3b\u9898\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANZHUTIQUDONGHUNHE"}, "257060": {"suoxie": "GLASZSPETFLJ", "name": "\u56fd\u8054\u5b89\u4e0a\u8bc1\u5546\u54c1ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANSHANGZHENGSHANGPINETFLIANJIE"}, "257070": {"suoxie": "GLAYXHYHH", "name": "\u56fd\u8054\u5b89\u4f18\u9009\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOLIANANYOUXUANHANGYEHUNHE"}, "260101": {"suoxie": "JSCCYXHH", "name": "\u666f\u987a\u957f\u57ce\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGYOUXUANHUNHE"}, "260102": {"suoxie": "JSHBA", "name": "\u666f\u987a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNHUOBIA"}, "260103": {"suoxie": "JSCCDLPHHH", "name": "\u666f\u987a\u957f\u57ce\u52a8\u529b\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINGSHUNCHANGCHENGDONGLIPINGHENGHUNHE"}, "260104": {"suoxie": "JSCCNXZZHH", "name": "\u666f\u987a\u957f\u57ce\u5185\u9700\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGNEIXUZENGZHANGHUNHE"}, "260108": {"suoxie": "JSCCXXCZHH", "name": "\u666f\u987a\u957f\u57ce\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGXINXINGCHENGZHANGHUNHE"}, "260109": {"suoxie": "JSCCNXEHHH", "name": "\u666f\u987a\u957f\u57ce\u5185\u9700\u8d30\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGNEIXUERHAOHUNHE"}, "260110": {"suoxie": "JSCCJXLCHH", "name": "\u666f\u987a\u957f\u57ce\u7cbe\u9009\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGJINGXUANLANCHOUHUNHE"}, "260111": {"suoxie": "JSCCGSZLHH", "name": "\u666f\u987a\u957f\u57ce\u516c\u53f8\u6cbb\u7406\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGGONGSIZHILIHUNHE"}, "260112": {"suoxie": "JSCCNYJJHH", "name": "\u666f\u987a\u957f\u57ce\u80fd\u6e90\u57fa\u5efa\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGNENGYUANJIJIANHUNHE"}, "260115": {"suoxie": "JSCCZXPHH", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGZHONGXIAOPANHUNHE"}, "260116": {"suoxie": "JSCCHXJZLHHA", "name": "\u666f\u987a\u957f\u57ce\u6838\u5fc3\u7ade\u4e89\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGHEXINJINGZHENGLIHUNHEA"}, "260117": {"suoxie": "JSCCZZCYHH", "name": "\u666f\u987a\u957f\u57ce\u652f\u67f1\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINGSHUNCHANGCHENGZHIZHUCHANYEHUNHE"}, "260202": {"suoxie": "JSHBB", "name": "\u666f\u987a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINGSHUNHUOBIB"}, "261001": {"suoxie": "JSCCWDSYZQA", "name": "\u666f\u987a\u957f\u57ce\u7a33\u5b9a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGWENDINGSHOUYIZHAIQUANA"}, "261002": {"suoxie": "JSCCYXZLZQA", "name": "\u666f\u987a\u957f\u57ce\u4f18\u4fe1\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGYOUXINZENGLIZHAIQUANA"}, "261101": {"suoxie": "JSCCWDSYZQC", "name": "\u666f\u987a\u957f\u57ce\u7a33\u5b9a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINGSHUNCHANGCHENGWENDINGSHOUYIZHAIQUANC"}, "261102": {"suoxie": "JSCCYXZLZQC", "name": "\u666f\u987a\u957f\u57ce\u4f18\u4fe1\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINGSHUNCHANGCHENGYOUXINZENGLIZHAIQUANC"}, "262001": {"suoxie": "JSCCDZHHHQDIIRMB", "name": "\u666f\u987a\u957f\u57ce\u5927\u4e2d\u534e\u6df7\u5408(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "JINGSHUNCHANGCHENGDAZHONGHUAHUNHEQDIIRENMINBI"}, "270001": {"suoxie": "GFJFHH", "name": "\u5e7f\u53d1\u805a\u5bcc\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAJUFUHUNHE"}, "270002": {"suoxie": "GFWJZZHHA", "name": "\u5e7f\u53d1\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAWENJIANZENGZHANGHUNHEA"}, "270004": {"suoxie": "GFHBA", "name": "\u5e7f\u53d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOBIA"}, "270005": {"suoxie": "GFJFHHA", "name": "\u5e7f\u53d1\u805a\u4e30\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUFENGHUNHEA"}, "270006": {"suoxie": "GFCLYXHH", "name": "\u5e7f\u53d1\u7b56\u7565\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACELUEYOUXUANHUNHE"}, "270007": {"suoxie": "GFDPCZHH", "name": "\u5e7f\u53d1\u5927\u76d8\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFADAPANCHENGZHANGHUNHE"}, "270008": {"suoxie": "GFHXJXHH", "name": "\u5e7f\u53d1\u6838\u5fc3\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHEXINJINGXUANHUNHE"}, "270009": {"suoxie": "GFZQZQC", "name": "\u5e7f\u53d1\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZENGQIANGZHAIQUANC"}, "270010": {"suoxie": "GFHS300ETFLJA", "name": "\u5e7f\u53d1\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ETFLIANJIEA"}, "270011": {"suoxie": "GFJFHH", "name": "\u5e7f\u53d1\u805a\u5bcc\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAJUFUHUNHE"}, "270012": {"suoxie": "GFWJZZHHA", "name": "\u5e7f\u53d1\u7a33\u5065\u589e\u957f\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUANGFAWENJIANZENGZHANGHUNHEA"}, "270014": {"suoxie": "GFHBB", "name": "\u5e7f\u53d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOBIB"}, "270015": {"suoxie": "GFJFHHA", "name": "\u5e7f\u53d1\u805a\u4e30\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJUFENGHUNHEA"}, "270016": {"suoxie": "GFCLYXHH", "name": "\u5e7f\u53d1\u7b56\u7565\u4f18\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFACELUEYOUXUANHUNHE"}, "270017": {"suoxie": "GFDPCZHH", "name": "\u5e7f\u53d1\u5927\u76d8\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFADAPANCHENGZHANGHUNHE"}, "270020": {"suoxie": "GFHS300ETFLJA", "name": "\u5e7f\u53d1\u6caa\u6df1300ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ETFLIANJIEA"}, "270021": {"suoxie": "GFJRHHA", "name": "\u5e7f\u53d1\u805a\u745e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAJURUIHUNHEA"}, "270022": {"suoxie": "GFNXZZHHA", "name": "\u5e7f\u53d1\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFANEIXUZENGZHANGHUNHEA"}, "270023": {"suoxie": "GFQQJXGPQDII", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANGUPIAOQDII"}, "270025": {"suoxie": "GFHYLXHHA", "name": "\u5e7f\u53d1\u884c\u4e1a\u9886\u5148\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAHANGYELINGXIANHUNHEA"}, "270026": {"suoxie": "GFZXQY300ETFLJA", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u4f01\u4e1a300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGXIAOQIYE300ETFLIANJIEA"}, "270028": {"suoxie": "GFZZYJXHHA", "name": "\u5e7f\u53d1\u5236\u9020\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZHIZAOYEJINGXUANHUNHEA"}, "270029": {"suoxie": "GFJCXYZQA", "name": "\u5e7f\u53d1\u805a\u8d22\u4fe1\u7528\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJUCAIXINYONGZHAIQUANA"}, "270030": {"suoxie": "GFJCXYZQB", "name": "\u5e7f\u53d1\u805a\u8d22\u4fe1\u7528\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAJUCAIXINYONGZHAIQUANB"}, "270036": {"suoxie": "GFZXQY300ETFLJA", "name": "\u5e7f\u53d1\u4e2d\u5c0f\u4f01\u4e1a300ETF\u8054\u63a5A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGXIAOQIYE300ETFLIANJIEA"}, "270041": {"suoxie": "GFXFPJXHHA", "name": "\u5e7f\u53d1\u6d88\u8d39\u54c1\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXIAOFEIPINJINGXUANHUNHEA"}, "270042": {"suoxie": "GFNSDK100ZSA", "name": "\u5e7f\u53d1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570A", "style": "QDII", "pinyin": "GUANGFANASIDAKE100ZHISHUA"}, "270043": {"suoxie": "GFLCNNHZQ", "name": "\u5e7f\u53d1\u7406\u8d22\u5e74\u5e74\u7ea2\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFALICAINIANNIANHONGZHAIQUAN"}, "270044": {"suoxie": "GFSZTLZQA", "name": "\u5e7f\u53d1\u53cc\u503a\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFASHUANGZHAITIANLIZHAIQUANA"}, "270045": {"suoxie": "GFSZTLZQC", "name": "\u5e7f\u53d1\u53cc\u503a\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFASHUANGZHAITIANLIZHAIQUANC"}, "270046": {"suoxie": "GFJRCZ", "name": "\u5e7f\u53d1\u666f\u8363\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFAJINGRONGCHUNZHAI"}, "270048": {"suoxie": "GFCZZQA", "name": "\u5e7f\u53d1\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFACHUNZHAIZHAIQUANA"}, "270049": {"suoxie": "GFCZZQC", "name": "\u5e7f\u53d1\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGFACHUNZHAIZHAIQUANC"}, "270050": {"suoxie": "GFXJJHHA", "name": "\u5e7f\u53d1\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAXINJINGJIHUNHEA"}, "288001": {"suoxie": "HXJDHH", "name": "\u534e\u590f\u7ecf\u5178\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HUAXIAJINGDIANHUNHE"}, "288002": {"suoxie": "HXSRHH", "name": "\u534e\u590f\u6536\u5165\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIASHOURUHUNHE"}, "288101": {"suoxie": "HXHBA", "name": "\u534e\u590f\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAHUOBIA"}, "288102": {"suoxie": "HXWDSLZQC", "name": "\u534e\u590f\u7a33\u5b9a\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAXIAWENDINGSHUANGLIZHAIQUANC"}, "288201": {"suoxie": "HXHBB", "name": "\u534e\u590f\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAHUOBIB"}, "290001": {"suoxie": "TXTTSYHBA", "name": "\u6cf0\u4fe1\u5929\u5929\u6536\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TAIXINTIANTIANSHOUYIHUOBIA"}, "290002": {"suoxie": "TXXXCLHH", "name": "\u6cf0\u4fe1\u5148\u884c\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINXIANXINGCELUEHUNHE"}, "290003": {"suoxie": "TXSXSLZQ", "name": "\u6cf0\u4fe1\u53cc\u606f\u53cc\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINSHUANGXISHUANGLIZHAIQUAN"}, "290004": {"suoxie": "TXYZSHHH", "name": "\u6cf0\u4fe1\u4f18\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINYOUZHISHENGHUOHUNHE"}, "290005": {"suoxie": "TXYSZZHH", "name": "\u6cf0\u4fe1\u4f18\u52bf\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINYOUSHIZENGZHANGHUNHE"}, "290006": {"suoxie": "TXLCJXHH", "name": "\u6cf0\u4fe1\u84dd\u7b79\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINLANCHOUJINGXUANHUNHE"}, "290007": {"suoxie": "TXZQZQSYA", "name": "\u6cf0\u4fe1\u503a\u5238\u589e\u5f3a\u6536\u76caA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINZHAIQUANZENGQIANGSHOUYIA"}, "290008": {"suoxie": "TXFZZTHH", "name": "\u6cf0\u4fe1\u53d1\u5c55\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINFAZHANZHUTIHUNHE"}, "290009": {"suoxie": "TXZQHBZQ", "name": "\u6cf0\u4fe1\u5468\u671f\u56de\u62a5\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINZHOUQIHUIBAOZHAIQUAN"}, "290010": {"suoxie": "TXZZ200ZS", "name": "\u6cf0\u4fe1\u4e2d\u8bc1200\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIXINZHONGZHENG200ZHISHU"}, "290011": {"suoxie": "TXZXPJXHH", "name": "\u6cf0\u4fe1\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINZHONGXIAOPANJINGXUANHUNHE"}, "290012": {"suoxie": "TXHYJXHHA", "name": "\u6cf0\u4fe1\u884c\u4e1a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TAIXINHANGYEJINGXUANHUNHEA"}, "290014": {"suoxie": "TXXDFWYHH", "name": "\u6cf0\u4fe1\u73b0\u4ee3\u670d\u52a1\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TAIXINXIANDAIFUWUYEHUNHE"}, "291007": {"suoxie": "TXZQZQSYC", "name": "\u6cf0\u4fe1\u503a\u5238\u589e\u5f3a\u6536\u76caC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TAIXINZHAIQUANZENGQIANGSHOUYIC"}, "310308": {"suoxie": "SWLXSLJXHH", "name": "\u7533\u4e07\u83f1\u4fe1\u76db\u5229\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHENWANLINGXINSHENGLIJINGXUANHUNHE"}, "310318": {"suoxie": "SWLXHS300ZSZQA", "name": "\u7533\u4e07\u83f1\u4fe1\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINHUSHEN300ZHISHUZENGQIANGA"}, "310328": {"suoxie": "SWLXXDLHH", "name": "\u7533\u4e07\u83f1\u4fe1\u65b0\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINXINDONGLIHUNHE"}, "310338": {"suoxie": "SWLXSYBHBA", "name": "\u7533\u4e07\u83f1\u4fe1\u6536\u76ca\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHENWANLINGXINSHOUYIBAOHUOBIA"}, "310339": {"suoxie": "SWLXSYBHBB", "name": "\u7533\u4e07\u83f1\u4fe1\u6536\u76ca\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHENWANLINGXINSHOUYIBAOHUOBIB"}, "310358": {"suoxie": "SWLXXJJHH", "name": "\u7533\u4e07\u83f1\u4fe1\u65b0\u7ecf\u6d4e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINXINJINGJIHUNHE"}, "310368": {"suoxie": "SWLXJZYSHH", "name": "\u7533\u4e07\u83f1\u4fe1\u7ade\u4e89\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINJINGZHENGYOUSHIHUNHE"}, "310388": {"suoxie": "SWLXXFZZHH", "name": "\u7533\u4e07\u83f1\u4fe1\u6d88\u8d39\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHENWANLINGXINXIAOFEIZENGZHANGHUNHE"}, "310398": {"suoxie": "SWLXHS300JZZSA", "name": "\u7533\u4e07\u83f1\u4fe1\u6caa\u6df1300\u4ef7\u503c\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINHUSHEN300JIAZHIZHISHUA"}, "310508": {"suoxie": "SWLXWYBZQ", "name": "\u7533\u4e07\u83f1\u4fe1\u7a33\u76ca\u5b9d\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHENWANLINGXINWENYIBAOZHAIQUAN"}, "310518": {"suoxie": "SWLXKZZZQ", "name": "\u7533\u4e07\u83f1\u4fe1\u53ef\u8f6c\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "SHENWANLINGXINKEZHUANZHAIZHAIQUAN"}, "320001": {"suoxie": "NAPHHH", "name": "\u8bfa\u5b89\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NUOANPINGHENGHUNHE"}, "320002": {"suoxie": "NAHBA", "name": "\u8bfa\u5b89\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANHUOBIA"}, "320003": {"suoxie": "NAXFHHA", "name": "\u8bfa\u5b89\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANXIANFENGHUNHEA"}, "320004": {"suoxie": "NAYHSYZQ", "name": "\u8bfa\u5b89\u4f18\u5316\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NUOANYOUHUASHOUYIZHAIQUAN"}, "320005": {"suoxie": "NAJZZZHH", "name": "\u8bfa\u5b89\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANJIAZHIZENGZHANGHUNHE"}, "320006": {"suoxie": "NALHPZHH", "name": "\u8bfa\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANLINGHUOPEIZHIHUNHE"}, "320007": {"suoxie": "NACZHH", "name": "\u8bfa\u5b89\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANCHENGZHANGHUNHE"}, "320008": {"suoxie": "NAZLZQA", "name": "\u8bfa\u5b89\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NUOANZENGLIZHAIQUANA"}, "320009": {"suoxie": "NAZLZQB", "name": "\u8bfa\u5b89\u589e\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NUOANZENGLIZHAIQUANB"}, "320010": {"suoxie": "NAZZ100ZSA", "name": "\u8bfa\u5b89\u4e2d\u8bc1100\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANZHONGZHENG100ZHISHUA"}, "320011": {"suoxie": "NAZXPJXHH", "name": "\u8bfa\u5b89\u4e2d\u5c0f\u76d8\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANZHONGXIAOPANJINGXUANHUNHE"}, "320012": {"suoxie": "NAZTJXHH", "name": "\u8bfa\u5b89\u4e3b\u9898\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANZHUTIJINGXUANHUNHE"}, "320013": {"suoxie": "NAQQHJ", "name": "\u8bfa\u5b89\u5168\u7403\u9ec4\u91d1", "style": "QDII", "pinyin": "NUOANQUANQIUHUANGJIN"}, "320014": {"suoxie": "NAHS300ZSZQA", "name": "\u8bfa\u5b89\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NUOANHUSHEN300ZHISHUZENGQIANGA"}, "320015": {"suoxie": "NAHYLDHH", "name": "\u8bfa\u5b89\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANHANGYELUNDONGHUNHE"}, "320016": {"suoxie": "NADCLHH", "name": "\u8bfa\u5b89\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUOANDUOCELUEHUNHE"}, "320017": {"suoxie": "NAQQSYBDC", "name": "\u8bfa\u5b89\u5168\u7403\u6536\u76ca\u4e0d\u52a8\u4ea7", "style": "QDII", "pinyin": "NUOANQUANQIUSHOUYIBUDONGCHAN"}, "320018": {"suoxie": "NAXDLLHPZHHA", "name": "\u8bfa\u5b89\u65b0\u52a8\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUOANXINDONGLILINGHUOPEIZHIHUNHEA"}, "320019": {"suoxie": "NAHBB", "name": "\u8bfa\u5b89\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NUOANHUOBIB"}, "320020": {"suoxie": "NACLJXGP", "name": "\u8bfa\u5b89\u7b56\u7565\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANCELUEJINGXUANGUPIAO"}, "320021": {"suoxie": "NASLZQFQ", "name": "\u8bfa\u5b89\u53cc\u5229\u503a\u5238\u53d1\u8d77", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NUOANSHUANGLIZHAIQUANFAQI"}, "320022": {"suoxie": "NAYJJXGP", "name": "\u8bfa\u5b89\u7814\u7a76\u7cbe\u9009\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "NUOANYANJIUJINGXUANGUPIAO"}, "340001": {"suoxie": "XQKZZHH", "name": "\u5174\u5168\u53ef\u8f6c\u503a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGQUANKEZHUANZHAIHUNHE"}, "340005": {"suoxie": "XQHBA", "name": "\u5174\u5168\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINGQUANHUOBIA"}, "340006": {"suoxie": "XQQQSYGP", "name": "\u5174\u5168\u5168\u7403\u89c6\u91ce\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XINGQUANQUANQIUSHIYEGUPIAO"}, "340007": {"suoxie": "XQSHZRHH", "name": "\u5174\u5168\u793e\u4f1a\u8d23\u4efb\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGQUANSHEHUIZERENHUNHE"}, "340008": {"suoxie": "XQYJZZHH", "name": "\u5174\u5168\u6709\u673a\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINGQUANYOUJIZENGZHANGHUNHE"}, "340009": {"suoxie": "XQPWZLZQA", "name": "\u5174\u5168\u78d0\u7a33\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINGQUANPANWENZENGLIZHAIQUANA"}, "350001": {"suoxie": "TZCFZZHH", "name": "\u5929\u6cbb\u8d22\u5bcc\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TIANZHICAIFUZENGZHANGHUNHE"}, "350002": {"suoxie": "TZDTJJHH", "name": "\u5929\u6cbb\u4f4e\u78b3\u7ecf\u6d4e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIDITANJINGJIHUNHE"}, "350004": {"suoxie": "TZTDLHBA", "name": "\u5929\u6cbb\u5929\u5f97\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANZHITIANDELIHUOBIA"}, "350005": {"suoxie": "TZZGZZ2025", "name": "\u5929\u6cbb\u4e2d\u56fd\u5236\u90202025", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIZHONGGUOZHIZAO2025"}, "350006": {"suoxie": "TZWJSYZQ", "name": "\u5929\u6cbb\u7a33\u5065\u53cc\u76c8\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANZHIWENJIANSHUANGYINGZHAIQUAN"}, "350007": {"suoxie": "TZQSJXHH", "name": "\u5929\u6cbb\u8d8b\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIQUSHIJINGXUANHUNHE"}, "350008": {"suoxie": "TZXXFHH", "name": "\u5929\u6cbb\u65b0\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIXINXIAOFEIHUNHE"}, "350009": {"suoxie": "TZYJQDHHA", "name": "\u5929\u6cbb\u7814\u7a76\u9a71\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANZHIYANJIUQUDONGHUNHEA"}, "360001": {"suoxie": "GDLHGP", "name": "\u5149\u5927\u91cf\u5316\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUANGDALIANGHUAGUPIAO"}, "360003": {"suoxie": "GDHBA", "name": "\u5149\u5927\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAHUOBIA"}, "360005": {"suoxie": "GDHLHH", "name": "\u5149\u5927\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAHONGLIHUNHE"}, "360006": {"suoxie": "GDXZZHH", "name": "\u5149\u5927\u65b0\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAXINZENGZHANGHUNHE"}, "360007": {"suoxie": "GDYSPZHH", "name": "\u5149\u5927\u4f18\u52bf\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYOUSHIPEIZHIHUNHE"}, "360008": {"suoxie": "GDZLSYZQA", "name": "\u5149\u5927\u589e\u5229\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAZENGLISHOUYIZHAIQUANA"}, "360009": {"suoxie": "GDZLSYZQC", "name": "\u5149\u5927\u589e\u5229\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAZENGLISHOUYIZHAIQUANC"}, "360010": {"suoxie": "GDJHJXHH", "name": "\u5149\u5927\u5747\u8861\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAJUNHENGJINGXUANHUNHE"}, "360011": {"suoxie": "GDDTYXHH", "name": "\u5149\u5927\u52a8\u6001\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGDADONGTAIYOUXUANHUNHE"}, "360012": {"suoxie": "GDZXPHH", "name": "\u5149\u5927\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAZHONGXIAOPANHUNHE"}, "360013": {"suoxie": "GDXYTYZQA", "name": "\u5149\u5927\u4fe1\u7528\u6dfb\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAXINYONGTIANYIZHAIQUANA"}, "360014": {"suoxie": "GDXYTYZQC", "name": "\u5149\u5927\u4fe1\u7528\u6dfb\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAXINYONGTIANYIZHAIQUANC"}, "360016": {"suoxie": "GDHYLDHH", "name": "\u5149\u5927\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAHANGYELUNDONGHUNHE"}, "360019": {"suoxie": "GDTTYWNDKZ", "name": "\u5149\u5927\u6dfb\u5929\u76c8\u4e94\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDATIANTIANYINGWUNIANDINGKAIZHAI"}, "370010": {"suoxie": "STMGHBA", "name": "\u4e0a\u6295\u6469\u6839\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENHUOBIA"}, "370011": {"suoxie": "STMGHBB", "name": "\u4e0a\u6295\u6469\u6839\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "SHANGTOUMOGENHUOBIB"}, "370023": {"suoxie": "STMGZZXFFWZS", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u8bc1\u6d88\u8d39\u670d\u52a1\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUMOGENZHONGZHENGXIAOFEIFUWUZHISHU"}, "370024": {"suoxie": "STMGHXYXHHA", "name": "\u4e0a\u6295\u6469\u6839\u6838\u5fc3\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENHEXINYOUXUANHUNHEA"}, "370027": {"suoxie": "STMGZX30HH", "name": "\u4e0a\u6295\u6469\u6839\u667a\u900930\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENZHIXUAN30HUNHE"}, "371020": {"suoxie": "STMGCZZQA", "name": "\u4e0a\u6295\u6469\u6839\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENCHUNZHAIZHAIQUANA"}, "371120": {"suoxie": "STMGCZZQB", "name": "\u4e0a\u6295\u6469\u6839\u7eaf\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGTOUMOGENCHUNZHAIZHAIQUANB"}, "372010": {"suoxie": "STMGQHHBZQA", "name": "\u4e0a\u6295\u6469\u6839\u5f3a\u5316\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUMOGENQIANGHUAHUIBAOZHAIQUANA"}, "372110": {"suoxie": "STMGQHHBZQB", "name": "\u4e0a\u6295\u6469\u6839\u5f3a\u5316\u56de\u62a5\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "SHANGTOUMOGENQIANGHUAHUIBAOZHAIQUANB"}, "373010": {"suoxie": "STMGSXPHHHA", "name": "\u4e0a\u6295\u6469\u6839\u53cc\u606f\u5e73\u8861\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGTOUMOGENSHUANGXIPINGHENGHUNHEA"}, "373020": {"suoxie": "STMGSHPHHH", "name": "\u4e0a\u6295\u6469\u6839\u53cc\u6838\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "SHANGTOUMOGENSHUANGHEPINGHENGHUNHE"}, "375010": {"suoxie": "STMGZGYSHH", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u56fd\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENZHONGGUOYOUSHIHUNHE"}, "376510": {"suoxie": "STMGDPLCGP", "name": "\u4e0a\u6295\u6469\u6839\u5927\u76d8\u84dd\u7b79\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "SHANGTOUMOGENDAPANLANCHOUGUPIAO"}, "377010": {"suoxie": "STMGAEFHH", "name": "\u4e0a\u6295\u6469\u6839\u963f\u5c14\u6cd5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENAERFAHUNHE"}, "377016": {"suoxie": "STMGYTYSHH", "name": "\u4e0a\u6295\u6469\u6839\u4e9a\u592a\u4f18\u52bf\u6df7\u5408", "style": "QDII", "pinyin": "SHANGTOUMOGENYATAIYOUSHIHUNHE"}, "377020": {"suoxie": "STMGNXDLHH", "name": "\u4e0a\u6295\u6469\u6839\u5185\u9700\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENNEIXUDONGLIHUNHE"}, "377150": {"suoxie": "STMGJKPZSHHH", "name": "\u4e0a\u6295\u6469\u6839\u5065\u5eb7\u54c1\u8d28\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENJIANKANGPINZHISHENGHUOHUNHE"}, "377240": {"suoxie": "STMGXXDLHHA", "name": "\u4e0a\u6295\u6469\u6839\u65b0\u5174\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENXINXINGDONGLIHUNHEA"}, "377530": {"suoxie": "STMGHYLDHHA", "name": "\u4e0a\u6295\u6469\u6839\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENHANGYELUNDONGHUNHEA"}, "378006": {"suoxie": "STMGQQXXSCHH", "name": "\u4e0a\u6295\u6469\u6839\u5168\u7403\u65b0\u5174\u5e02\u573a\u6df7\u5408", "style": "QDII", "pinyin": "SHANGTOUMOGENQUANQIUXINXINGSHICHANGHUNHE"}, "378010": {"suoxie": "STMGCZXFHH", "name": "\u4e0a\u6295\u6469\u6839\u6210\u957f\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENCHENGZHANGXIANFENGHUNHE"}, "378546": {"suoxie": "STMGQQTRZYHH", "name": "\u4e0a\u6295\u6469\u6839\u5168\u7403\u5929\u7136\u8d44\u6e90\u6df7\u5408", "style": "QDII", "pinyin": "SHANGTOUMOGENQUANQIUTIANRANZIYUANHUNHE"}, "379010": {"suoxie": "STMGZXPHH", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "SHANGTOUMOGENZHONGXIAOPANHUNHE"}, "380005": {"suoxie": "ZYCZZQA", "name": "\u4e2d\u94f6\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINCHUNZHAIZHAIQUANA"}, "380006": {"suoxie": "ZYCZZQC", "name": "\u4e2d\u94f6\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINCHUNZHAIZHAIQUANC"}, "380009": {"suoxie": "ZYTLZQFQA", "name": "\u4e2d\u94f6\u6dfb\u5229\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGYINTIANLIZHAIQUANFAQIA"}, "380010": {"suoxie": "ZYJXZQA", "name": "\u4e2d\u94f6\u805a\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINJUXIANGZHAIQUANA"}, "380011": {"suoxie": "ZYJXZQB", "name": "\u4e2d\u94f6\u805a\u4eab\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYINJUXIANGZHAIQUANB"}, "392001": {"suoxie": "ZHHBA", "name": "\u4e2d\u6d77\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGHAIHUOBIA"}, "392002": {"suoxie": "ZHHBB", "name": "\u4e2d\u6d77\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGHAIHUOBIB"}, "393001": {"suoxie": "ZHYSJXLHPZHH", "name": "\u4e2d\u6d77\u4f18\u52bf\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIYOUSHIJINGXUANLINGHUOPEIZHIHUNHE"}, "395001": {"suoxie": "ZHWJSYZQ", "name": "\u4e2d\u6d77\u7a33\u5065\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHAIWENJIANSHOUYIZHAIQUAN"}, "395011": {"suoxie": "ZHZQSYZQA", "name": "\u4e2d\u6d77\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHAIZENGQIANGSHOUYIZHAIQUANA"}, "395012": {"suoxie": "ZHZQSYZQC", "name": "\u4e2d\u6d77\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGHAIZENGQIANGSHOUYIZHAIQUANC"}, "398001": {"suoxie": "ZHYZCZHH", "name": "\u4e2d\u6d77\u4f18\u8d28\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIYOUZHICHENGZHANGHUNHE"}, "398002": {"suoxie": "ZHYZCZHH", "name": "\u4e2d\u6d77\u4f18\u8d28\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIYOUZHICHENGZHANGHUNHE"}, "398011": {"suoxie": "ZHFHZLHH", "name": "\u4e2d\u6d77\u5206\u7ea2\u589e\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIFENHONGZENGLIHUNHE"}, "398012": {"suoxie": "ZHFHZLHH", "name": "\u4e2d\u6d77\u5206\u7ea2\u589e\u5229\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIFENHONGZENGLIHUNHE"}, "398021": {"suoxie": "ZHNYCLHH", "name": "\u4e2d\u6d77\u80fd\u6e90\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAINENGYUANCELUEHUNHE"}, "398031": {"suoxie": "ZHLCHH", "name": "\u4e2d\u6d77\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAILANCHOUHUNHE"}, "398041": {"suoxie": "ZHLHCLHH", "name": "\u4e2d\u6d77\u91cf\u5316\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAILIANGHUACELUEHUNHE"}, "398051": {"suoxie": "ZHHBXNYHH", "name": "\u4e2d\u6d77\u73af\u4fdd\u65b0\u80fd\u6e90\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGHAIHUANBAOXINNENGYUANHUNHE"}, "398061": {"suoxie": "ZHXFHH", "name": "\u4e2d\u6d77\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGHAIXIAOFEIHUNHE"}, "399001": {"suoxie": "ZHSZ50ZSZQ", "name": "\u4e2d\u6d77\u4e0a\u8bc150\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGHAISHANGZHENG50ZHISHUZENGQIANG"}, "399011": {"suoxie": "ZHYLBJZTGP", "name": "\u4e2d\u6d77\u533b\u7597\u4fdd\u5065\u4e3b\u9898\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGHAIYILIAOBAOJIANZHUTIGUPIAO"}, "400001": {"suoxie": "DFLHH", "name": "\u4e1c\u65b9\u9f99\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGLONGHUNHE"}, "400003": {"suoxie": "DFJXHH", "name": "\u4e1c\u65b9\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGJINGXUANHUNHE"}, "400004": {"suoxie": "DFJXHH", "name": "\u4e1c\u65b9\u7cbe\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGJINGXUANHUNHE"}, "400005": {"suoxie": "DFJZBHBA", "name": "\u4e1c\u65b9\u91d1\u8d26\u7c3f\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGJINZHANGBUHUOBIA"}, "400006": {"suoxie": "DFJZBHBB", "name": "\u4e1c\u65b9\u91d1\u8d26\u7c3f\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGFANGJINZHANGBUHUOBIB"}, "400007": {"suoxie": "DFCLCZHH", "name": "\u4e1c\u65b9\u7b56\u7565\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGCELUECHENGZHANGHUNHE"}, "400008": {"suoxie": "DFCLCZHH", "name": "\u4e1c\u65b9\u7b56\u7565\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGCELUECHENGZHANGHUNHE"}, "400009": {"suoxie": "DFWJHBZQA", "name": "\u4e1c\u65b9\u7a33\u5065\u56de\u62a5\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGWENJIANHUIBAOZHAIQUANA"}, "400011": {"suoxie": "DFHXDLHHA", "name": "\u4e1c\u65b9\u6838\u5fc3\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHEXINDONGLIHUNHEA"}, "400012": {"suoxie": "DFHXDLHHA", "name": "\u4e1c\u65b9\u6838\u5fc3\u52a8\u529b\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHEXINDONGLIHUNHEA"}, "400013": {"suoxie": "DFCZSYLHPZHHA", "name": "\u4e1c\u65b9\u6210\u957f\u6536\u76ca\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGCHENGZHANGSHOUYILINGHUOPEIZHIHUNHEA"}, "400015": {"suoxie": "DFXNYQCHH", "name": "\u4e1c\u65b9\u65b0\u80fd\u6e90\u6c7d\u8f66\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGXINNENGYUANQICHEHUNHE"}, "400016": {"suoxie": "DFQHSYZQ", "name": "\u4e1c\u65b9\u5f3a\u5316\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGQIANGHUASHOUYIZHAIQUAN"}, "400020": {"suoxie": "DFCZHBPHHH", "name": "\u4e1c\u65b9\u6210\u957f\u56de\u62a5\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "DONGFANGCHENGZHANGHUIBAOPINGHENGHUNHE"}, "400023": {"suoxie": "DFDCLLHPZHHA", "name": "\u4e1c\u65b9\u591a\u7b56\u7565\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGDUOCELUELINGHUOPEIZHIHUNHEA"}, "400025": {"suoxie": "DFXXCZHH", "name": "\u4e1c\u65b9\u65b0\u5174\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGXINXINGCHENGZHANGHUNHE"}, "400027": {"suoxie": "DFSZTLZQA", "name": "\u4e1c\u65b9\u53cc\u503a\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGSHUANGZHAITIANLIZHAIQUANA"}, "400029": {"suoxie": "DFSZTLZQC", "name": "\u4e1c\u65b9\u53cc\u503a\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGFANGSHUANGZHAITIANLIZHAIQUANC"}, "400030": {"suoxie": "DFTYZQ", "name": "\u4e1c\u65b9\u6dfb\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGFANGTIANYIZHAIQUAN"}, "400032": {"suoxie": "DFZTJXHH", "name": "\u4e1c\u65b9\u4e3b\u9898\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGZHUTIJINGXUANHUNHE"}, "410001": {"suoxie": "HFJZLYXHH", "name": "\u534e\u5bcc\u7ade\u4e89\u529b\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUJINGZHENGLIYOUXUANHUNHE"}, "410002": {"suoxie": "HFHBA", "name": "\u534e\u5bcc\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAFUHUOBIA"}, "410003": {"suoxie": "HFCZQSHH", "name": "\u534e\u5bcc\u6210\u957f\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFUCHENGZHANGQUSHIHUNHE"}, "410004": {"suoxie": "HFSYZQZQA", "name": "\u534e\u5bcc\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUSHOUYIZENGQIANGZHAIQUANA"}, "410005": {"suoxie": "HFSYZQZQB", "name": "\u534e\u5bcc\u6536\u76ca\u589e\u5f3a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAFUSHOUYIZENGQIANGZHAIQUANB"}, "410006": {"suoxie": "HFCLJXHH", "name": "\u534e\u5bcc\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUCELUEJINGXUANHUNHE"}, "410007": {"suoxie": "HFJZZZHH", "name": "\u534e\u5bcc\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAFUJIAZHIZENGZHANGHUNHE"}, "410008": {"suoxie": "HFZZ100ZS", "name": "\u534e\u5bcc\u4e2d\u8bc1100\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENG100ZHISHU"}, "410009": {"suoxie": "HFLZSMLHH", "name": "\u534e\u5bcc\u91cf\u5b50\u751f\u547d\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAFULIANGZISHENGMINGLIHUNHE"}, "410010": {"suoxie": "HFZXQY100ZSZQ", "name": "\u534e\u5bcc\u4e2d\u5c0f\u4f01\u4e1a100\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGXIAOQIYE100ZHISHUZENGQIANG"}, "420001": {"suoxie": "THJXHH", "name": "\u5929\u5f18\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "TIANHONGJINGXUANHUNHE"}, "420002": {"suoxie": "THYLZQA", "name": "\u5929\u5f18\u6c38\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGYONGLIZHAIQUANA"}, "420003": {"suoxie": "THYDJZCZHH", "name": "\u5929\u5f18\u6c38\u5b9a\u4ef7\u503c\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGYONGDINGJIAZHICHENGZHANGHUNHE"}, "420005": {"suoxie": "THZQCLHH", "name": "\u5929\u5f18\u5468\u671f\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "TIANHONGZHOUQICELUEHUNHE"}, "420006": {"suoxie": "THXJGJHBA", "name": "\u5929\u5f18\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGXIANJINGUANJIAHUOBIA"}, "420008": {"suoxie": "THZQFQSA", "name": "\u5929\u5f18\u503a\u5238\u53d1\u8d77\u5f0fA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGZHAIQUANFAQISHIA"}, "420009": {"suoxie": "THAKYYHHA", "name": "\u5929\u5f18\u5b89\u5eb7\u9890\u517b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "TIANHONGANKANGYIYANGHUNHEA"}, "420102": {"suoxie": "THYLZQB", "name": "\u5929\u5f18\u6c38\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGYONGLIZHAIQUANB"}, "420106": {"suoxie": "THXJGJHBB", "name": "\u5929\u5f18\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "TIANHONGXIANJINGUANJIAHUOBIB"}, "420108": {"suoxie": "THZQFQSB", "name": "\u5929\u5f18\u503a\u5238\u53d1\u8d77\u5f0fB", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "TIANHONGZHAIQUANFAQISHIB"}, "450001": {"suoxie": "GFZGSYHH", "name": "\u56fd\u5bcc\u4e2d\u56fd\u6536\u76ca\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GUOFUZHONGGUOSHOUYIHUNHE"}, "450002": {"suoxie": "GFTXSZHH", "name": "\u56fd\u5bcc\u5f39\u6027\u5e02\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUTANXINGSHIZHIHUNHE"}, "450003": {"suoxie": "GFQLZHHHA", "name": "\u56fd\u5bcc\u6f5c\u529b\u7ec4\u5408\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUQIANLIZUHEHUNHEA"}, "450004": {"suoxie": "GFSHJZHH", "name": "\u56fd\u5bcc\u6df1\u5316\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUSHENHUAJIAZHIHUNHE"}, "450005": {"suoxie": "GFQHSYZQA", "name": "\u56fd\u5bcc\u5f3a\u5316\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUQIANGHUASHOUYIZHAIQUANA"}, "450006": {"suoxie": "GFQHSYZQC", "name": "\u56fd\u5bcc\u5f3a\u5316\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOFUQIANGHUASHOUYIZHAIQUANC"}, "450007": {"suoxie": "GFCZDLHH", "name": "\u56fd\u5bcc\u6210\u957f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUCHENGZHANGDONGLIHUNHE"}, "450008": {"suoxie": "GFHS300ZSZQ", "name": "\u56fd\u5bcc\u6caa\u6df1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOFUHUSHEN300ZHISHUZENGQIANG"}, "450009": {"suoxie": "GFZXPGP", "name": "\u56fd\u5bcc\u4e2d\u5c0f\u76d8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOFUZHONGXIAOPANGUPIAO"}, "450010": {"suoxie": "GFCLHBHH", "name": "\u56fd\u5bcc\u7b56\u7565\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOFUCELUEHUIBAOHUNHE"}, "450011": {"suoxie": "GFYJJXHH", "name": "\u56fd\u5bcc\u7814\u7a76\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOFUYANJIUJINGXUANHUNHE"}, "450018": {"suoxie": "GFHJXYZQA", "name": "\u56fd\u5bcc\u6052\u4e45\u4fe1\u7528\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOFUHENGJIUXINYONGZHAIQUANA"}, "450019": {"suoxie": "GFHJXYZQC", "name": "\u56fd\u5bcc\u6052\u4e45\u4fe1\u7528\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOFUHENGJIUXINYONGZHAIQUANC"}, "457001": {"suoxie": "GFYZJHGPQDII", "name": "\u56fd\u5bcc\u4e9a\u6d32\u673a\u4f1a\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "GUOFUYAZHOUJIHUIGUPIAOQDII"}, "460001": {"suoxie": "HTBRSSZGHH", "name": "\u534e\u6cf0\u67cf\u745e\u76db\u4e16\u4e2d\u56fd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUISHENGSHIZHONGGUOHUNHE"}, "460002": {"suoxie": "HTBRJJCZHHA", "name": "\u534e\u6cf0\u67cf\u745e\u79ef\u6781\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIJICHENGZHANGHUNHEA"}, "460003": {"suoxie": "HTBRWBZLZQB", "name": "\u534e\u6cf0\u67cf\u745e\u7a33\u672c\u589e\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIWENBENZENGLIZHAIQUANB"}, "460005": {"suoxie": "HTBRJZZZHHA", "name": "\u534e\u6cf0\u67cf\u745e\u4ef7\u503c\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIJIAZHIZENGZHANGHUNHEA"}, "460006": {"suoxie": "HTBRHBA", "name": "\u534e\u6cf0\u67cf\u745e\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUIHUOBIA"}, "460007": {"suoxie": "HTBRHYLXHH", "name": "\u534e\u6cf0\u67cf\u745e\u884c\u4e1a\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUIHANGYELINGXIANHUNHE"}, "460008": {"suoxie": "HTBRWJSYZQA", "name": "\u534e\u6cf0\u67cf\u745e\u7a33\u5065\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIWENJIANSHOUYIZHAIQUANA"}, "460009": {"suoxie": "HTBRLHXXHHA", "name": "\u534e\u6cf0\u67cf\u745e\u91cf\u5316\u5148\u884c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUATAIBAIRUILIANGHUAXIANXINGHUNHEA"}, "460010": {"suoxie": "HTBRYZLDQYHH", "name": "\u534e\u6cf0\u67cf\u745e\u4e9a\u6d32\u9886\u5bfc\u4f01\u4e1a\u6df7\u5408", "style": "QDII", "pinyin": "HUATAIBAIRUIYAZHOULINGDAOQIYEHUNHE"}, "460106": {"suoxie": "HTBRHBB", "name": "\u534e\u6cf0\u67cf\u745e\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUIHUOBIB"}, "460108": {"suoxie": "HTBRWJSYZQC", "name": "\u534e\u6cf0\u67cf\u745e\u7a33\u5065\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUATAIBAIRUIWENJIANSHOUYIZHAIQUANC"}, "460220": {"suoxie": "HTBRSZZXPETFLJ", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u4e2d\u5c0f\u76d8ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGZHONGXIAOPANETFLIANJIE"}, "460300": {"suoxie": "HTBRHS300ETFLJA", "name": "\u534e\u6cf0\u67cf\u745e\u6caa\u6df1300ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIHUSHEN300ETFLIANJIEA"}, "470006": {"suoxie": "HTFYYBJHHA", "name": "\u6c47\u6dfb\u5bcc\u533b\u836f\u4fdd\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYIYAOBAOJIANHUNHEA"}, "470007": {"suoxie": "HTFSZZHZS", "name": "\u6c47\u6dfb\u5bcc\u4e0a\u8bc1\u7efc\u5408\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUSHANGZHENGZONGHEZHISHU"}, "470008": {"suoxie": "HTFCLHBHH", "name": "\u6c47\u6dfb\u5bcc\u7b56\u7565\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCELUEHUIBAOHUNHE"}, "470009": {"suoxie": "HTFMYHLHHA", "name": "\u6c47\u6dfb\u5bcc\u6c11\u8425\u6d3b\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUMINYINGHUOLIHUNHEA"}, "470010": {"suoxie": "HTFDYSYZQA", "name": "\u6c47\u6dfb\u5bcc\u591a\u5143\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUDUOYUANSHOUYIZHAIQUANA"}, "470011": {"suoxie": "HTFDYSYZQC", "name": "\u6c47\u6dfb\u5bcc\u591a\u5143\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUDUOYUANSHOUYIZHAIQUANC"}, "470014": {"suoxie": "HTFLC14TZQA", "name": "\u6c47\u6dfb\u5bcc\u7406\u8d2214\u5929\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFULICAI14TIANZHAIQUANA"}, "470018": {"suoxie": "HTFSLZQA", "name": "\u6c47\u6dfb\u5bcc\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUSHUANGLIZHAIQUANA"}, "470021": {"suoxie": "HTFYXHBLHPZHHA", "name": "\u6c47\u6dfb\u5bcc\u4f18\u9009\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUYOUXUANHUIBAOLINGHUOPEIZHIHUNHEA"}, "470028": {"suoxie": "HTFSHZRHH", "name": "\u6c47\u6dfb\u5bcc\u793e\u4f1a\u8d23\u4efb\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUSHEHUIZERENHUNHE"}, "470030": {"suoxie": "HTFXXZQ", "name": "\u6c47\u6dfb\u5bcc\u946b\u79a7\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFUXINXIZHAIQUAN"}, "470058": {"suoxie": "HTFKZHZQA", "name": "\u6c47\u6dfb\u5bcc\u53ef\u8f6c\u6362\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUITIANFUKEZHUANHUANZHAIQUANA"}, "470059": {"suoxie": "HTFKZHZQC", "name": "\u6c47\u6dfb\u5bcc\u53ef\u8f6c\u6362\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HUITIANFUKEZHUANHUANZHAIQUANC"}, "470060": {"suoxie": "HTFLC60TZQA", "name": "\u6c47\u6dfb\u5bcc\u7406\u8d2260\u5929\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFULICAI60TIANZHAIQUANA"}, "470068": {"suoxie": "HTFSZ300ETFLJ", "name": "\u6c47\u6dfb\u5bcc\u6df1\u8bc1300ETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUSHENZHENG300ETFLIANJIE"}, "470078": {"suoxie": "HTFZQSYZQC", "name": "\u6c47\u6dfb\u5bcc\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUZENGQIANGSHOUYIZHAIQUANC"}, "470088": {"suoxie": "HTF6YHTLDKZA", "name": "\u6c47\u6dfb\u5bcc6\u6708\u7ea2\u6dfb\u5229\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFU6YUEHONGTIANLIDINGKAIZHAIA"}, "470089": {"suoxie": "HTF6YHTLDKZC", "name": "\u6c47\u6dfb\u5bcc6\u6708\u7ea2\u6dfb\u5229\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFU6YUEHONGTIANLIDINGKAIZHAIC"}, "470098": {"suoxie": "HTFNXTZHH", "name": "\u6c47\u6dfb\u5bcc\u9006\u5411\u6295\u8d44\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUNIXIANGTOUZIHUNHE"}, "470888": {"suoxie": "HTFXGHHQDII", "name": "\u6c47\u6dfb\u5bcc\u9999\u6e2f\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "HUITIANFUXIANGGANGHUNHEQDII"}, "471014": {"suoxie": "HTFLC14TZQB", "name": "\u6c47\u6dfb\u5bcc\u7406\u8d2214\u5929\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFULICAI14TIANZHAIQUANB"}, "471060": {"suoxie": "HTFLC60TZQB", "name": "\u6c47\u6dfb\u5bcc\u7406\u8d2260\u5929\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUITIANFULICAI60TIANZHAIQUANB"}, "472007": {"suoxie": "HTFLLZ", "name": "\u6c47\u6dfb\u5bcc\u5229\u7387\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUITIANFULILVZHAI"}, "481001": {"suoxie": "GYHXJZHHA", "name": "\u5de5\u94f6\u6838\u5fc3\u4ef7\u503c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHEXINJIAZHIHUNHEA"}, "481004": {"suoxie": "GYWJCZHHA", "name": "\u5de5\u94f6\u7a33\u5065\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINWENJIANCHENGZHANGHUNHEA"}, "481006": {"suoxie": "GYHLHH", "name": "\u5de5\u94f6\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINHONGLIHUNHE"}, "481008": {"suoxie": "GYDPLCHH", "name": "\u5de5\u94f6\u5927\u76d8\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINDAPANLANCHOUHUNHE"}, "481009": {"suoxie": "GYHS300ZSA", "name": "\u5de5\u94f6\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINHUSHEN300ZHISHUA"}, "481010": {"suoxie": "GYZXPHH", "name": "\u5de5\u94f6\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHONGXIAOPANHUNHE"}, "481012": {"suoxie": "GYSZHLETFLJA", "name": "\u5de5\u94f6\u6df1\u8bc1\u7ea2\u5229ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHENZHENGHONGLIETFLIANJIEA"}, "481013": {"suoxie": "GYXFFWHHA", "name": "\u5de5\u94f6\u6d88\u8d39\u670d\u52a1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINXIAOFEIFUWUHUNHEA"}, "481015": {"suoxie": "GYZTCLHHA", "name": "\u5de5\u94f6\u4e3b\u9898\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINZHUTICELUEHUNHEA"}, "481017": {"suoxie": "GYLHCLHHA", "name": "\u5de5\u94f6\u91cf\u5316\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GONGYINLIANGHUACELUEHUNHEA"}, "482002": {"suoxie": "GYHB", "name": "\u5de5\u94f6\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GONGYINHUOBI"}, "483003": {"suoxie": "GYJXPHHH", "name": "\u5de5\u94f6\u7cbe\u9009\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "GONGYINJINGXUANPINGHENGHUNHE"}, "485005": {"suoxie": "GYZQSYZQB", "name": "\u5de5\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINZENGQIANGSHOUYIZHAIQUANB"}, "485007": {"suoxie": "GYTLZQB", "name": "\u5de5\u94f6\u6dfb\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANLIZHAIQUANB"}, "485011": {"suoxie": "GYRXSLZQB", "name": "\u5de5\u94f6\u745e\u4fe1\u53cc\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINRUIXINSHUANGLIZHAIQUANB"}, "485014": {"suoxie": "GYTYZQB", "name": "\u5de5\u94f6\u6dfb\u9890\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANYIZHAIQUANB"}, "485018": {"suoxie": "GY7TLCZQB", "name": "\u5de5\u94f67\u5929\u7406\u8d22\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN7TIANLICAIZHAIQUANB"}, "485019": {"suoxie": "GYXYCZZQB", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAIZHAIQUANB"}, "485020": {"suoxie": "GY14TLCZQFQB", "name": "\u5de5\u94f614\u5929\u7406\u8d22\u503a\u5238\u53d1\u8d77B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN14TIANLICAIZHAIQUANFAQIB"}, "485022": {"suoxie": "GYZYZDZF", "name": "\u5de5\u94f6\u5c0a\u76ca\u4e2d\u77ed\u503aF", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNYIZHONGDUANZHAIF"}, "485105": {"suoxie": "GYZQSYZQA", "name": "\u5de5\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINZENGQIANGSHOUYIZHAIQUANA"}, "485107": {"suoxie": "GYTLZQA", "name": "\u5de5\u94f6\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANLIZHAIQUANA"}, "485111": {"suoxie": "GYRXSLZQA", "name": "\u5de5\u94f6\u745e\u4fe1\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINRUIXINSHUANGLIZHAIQUANA"}, "485114": {"suoxie": "GYTYZQA", "name": "\u5de5\u94f6\u6dfb\u9890\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINTIANYIZHAIQUANA"}, "485118": {"suoxie": "GY7TLCZQA", "name": "\u5de5\u94f67\u5929\u7406\u8d22\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN7TIANLICAIZHAIQUANA"}, "485119": {"suoxie": "GYXYCZZQA", "name": "\u5de5\u94f6\u4fe1\u7528\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GONGYINXINYONGCHUNZHAIZHAIQUANA"}, "485120": {"suoxie": "GY14TLCZQFQA", "name": "\u5de5\u94f614\u5929\u7406\u8d22\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYIN14TIANLICAIZHAIQUANFAQIA"}, "485122": {"suoxie": "GYZYZDZC", "name": "\u5de5\u94f6\u5c0a\u76ca\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GONGYINZUNYIZHONGDUANZHAIC"}, "485205": {"suoxie": "GYZQSYZQA", "name": "\u5de5\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238A(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GONGYINZENGQIANGSHOUYIZHAIQUANA"}, "486001": {"suoxie": "GYQQGPQDIIRMB", "name": "\u5de5\u94f6\u5168\u7403\u80a1\u7968(QDII)\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "GONGYINQUANQIUGUPIAOQDIIRENMINBI"}, "486002": {"suoxie": "GYQQJXGPQDII", "name": "\u5de5\u94f6\u5168\u7403\u7cbe\u9009\u80a1\u7968(QDII)", "style": "QDII", "pinyin": "GONGYINQUANQIUJINGXUANGUPIAOQDII"}, "487016": {"suoxie": "GYRXLHPZHHA", "name": "\u5de5\u94f6\u745e\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINRUIXINLINGHUOPEIZHIHUNHEA"}, "487021": {"suoxie": "GYYZJXHH", "name": "\u5de5\u94f6\u4f18\u8d28\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GONGYINYOUZHIJINGXUANHUNHE"}, "501000": {"suoxie": "GJXXLHPZLOF", "name": "\u56fd\u91d1\u946b\u65b0\u7075\u6d3b\u914d\u7f6e(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOJINXINXINLINGHUOPEIZHILOF"}, "501001": {"suoxie": "CTDCLJXHHLOF", "name": "\u8d22\u901a\u591a\u7b56\u7565\u7cbe\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGDUOCELUEJINGXUANHUNHELOF"}, "501002": {"suoxie": "CXJZYXHH", "name": "\u957f\u4fe1\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINJIAZHIYOUXUANHUNHE"}, "501005": {"suoxie": "HTFZZJZYLZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u7cbe\u51c6\u533b\u7597\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGJINGZHUNYILIAOZHISHUA"}, "501006": {"suoxie": "HTFZZJZYLZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u7cbe\u51c6\u533b\u7597\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGJINGZHUNYILIAOZHISHUC"}, "501007": {"suoxie": "HTFZZHLWYLZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e92\u8054\u7f51\u533b\u7597\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHULIANWANGYILIAOZHISHUA"}, "501008": {"suoxie": "HTFZZHLWYLZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e92\u8054\u7f51\u533b\u7597\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHULIANWANGYILIAOZHISHUC"}, "501009": {"suoxie": "HTFZZSWKJZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGSHENGWUKEJIZHISHUA"}, "501010": {"suoxie": "HTFZZSWKJZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGSHENGWUKEJIZHISHUC"}, "501011": {"suoxie": "HTFZZZYZSLOFA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e2d\u836f\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHONGYAOZHISHULOFA"}, "501012": {"suoxie": "HTFZZZYZSLOFC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e2d\u836f\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGZHONGYAOZHISHULOFC"}, "501015": {"suoxie": "CTDCLSJHHLOF", "name": "\u8d22\u901a\u591a\u7b56\u7565\u5347\u7ea7\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGDUOCELUESHENGJIHUNHELOF"}, "501016": {"suoxie": "GTZZSWZQHYZS", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7533\u4e07\u8bc1\u5238\u884c\u4e1a\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGSHENWANZHENGQUANHANGYEZHISHU"}, "501017": {"suoxie": "GTRFWYZZHHLOFA", "name": "\u56fd\u6cf0\u878d\u4e30\u5916\u5ef6\u589e\u957f\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRONGFENGWAIYANZENGZHANGHUNHELOFA"}, "501018": {"suoxie": "NFYYA", "name": "\u5357\u65b9\u539f\u6cb9A", "style": "QDII", "pinyin": "NANFANGYUANYOUA"}, "501019": {"suoxie": "GTGZHTJGZS", "name": "\u56fd\u6cf0\u56fd\u8bc1\u822a\u5929\u519b\u5de5\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIGUOZHENGHANGTIANJUNGONGZHISHU"}, "501021": {"suoxie": "HBXGZXQDIILOFA", "name": "\u534e\u5b9d\u9999\u6e2f\u4e2d\u5c0f(QDII-LOF)A", "style": "QDII", "pinyin": "HUABAOXIANGGANGZHONGXIAOQDIILOFA"}, "501022": {"suoxie": "YHXSLHPZHHLOFA", "name": "\u94f6\u534e\u946b\u76db\u7075\u6d3b\u914d\u7f6e\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAXINSHENGLINGHUOPEIZHIHUNHELOFA"}, "501025": {"suoxie": "PHXGYHZSLOFA", "name": "\u9e4f\u534e\u9999\u6e2f\u94f6\u884c\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAXIANGGANGYINHANGZHISHULOFA"}, "501026": {"suoxie": "CTDCLFXHHLOF", "name": "\u8d22\u901a\u591a\u7b56\u7565\u798f\u4eab\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGDUOCELUEFUXIANGHUNHELOF"}, "501027": {"suoxie": "GTRXLOF", "name": "\u56fd\u6cf0\u878d\u4fe1(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIRONGXINLOF"}, "501028": {"suoxie": "CTDCLFRHHFQSLOFA", "name": "\u8d22\u901a\u591a\u7b56\u7565\u798f\u745e\u6df7\u5408\u53d1\u8d77\u5f0f(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGDUOCELUEFURUIHUNHEFAQISHILOFA"}, "501029": {"suoxie": "HBBPZGAGHLJHZSA", "name": "\u534e\u5b9d\u6807\u666e\u4e2d\u56fdA\u80a1\u7ea2\u5229\u673a\u4f1a\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOBIAOPUZHONGGUOAGUHONGLIJIHUIZHISHUA"}, "501030": {"suoxie": "HTFZZHJZLZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u73af\u5883\u6cbb\u7406\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUANJINGZHILIZHISHUA"}, "501031": {"suoxie": "HTFZZHJZLZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u73af\u5883\u6cbb\u7406\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUANJINGZHILIZHISHUC"}, "501032": {"suoxie": "CTFSHHFQLOFA", "name": "\u8d22\u901a\u798f\u76db\u6df7\u5408\u53d1\u8d77(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGFUSHENGHUNHEFAQILOFA"}, "501036": {"suoxie": "HTFZZ500A", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1500A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG500A"}, "501037": {"suoxie": "HTFZZ500C", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1500C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENG500C"}, "501038": {"suoxie": "YHMZDCLDQKFHH", "name": "\u94f6\u534e\u660e\u62e9\u591a\u7b56\u7565\u5b9a\u671f\u5f00\u653e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAMINGZEDUOCELUEDINGQIKAIFANGHUNHE"}, "501039": {"suoxie": "HTFRFHHLOFA", "name": "\u6c47\u6dfb\u5bcc\u777f\u4e30\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFURUIFENGHUNHELOFA"}, "501040": {"suoxie": "HTFRFHHLOFC", "name": "\u6c47\u6dfb\u5bcc\u777f\u4e30\u6df7\u5408(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFURUIFENGHUNHELOFC"}, "501041": {"suoxie": "HTFHALOFA", "name": "\u6c47\u6dfb\u5bcc\u5f18\u5b89(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUHONGANLOFA"}, "501042": {"suoxie": "HTFHALOFC", "name": "\u6c47\u6dfb\u5bcc\u5f18\u5b89(LOF)C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUITIANFUHONGANLOFC"}, "501043": {"suoxie": "HTFHS300ZSA", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300ZHISHUA"}, "501045": {"suoxie": "HTFHS300ZSC", "name": "\u6c47\u6dfb\u5bcc\u6caa\u6df1300\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUHUSHEN300ZHISHUC"}, "501046": {"suoxie": "CTDCLFXDKHH", "name": "\u8d22\u901a\u591a\u7b56\u7565\u798f\u946b\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGDUOCELUEFUXINDINGKAIHUNHE"}, "501047": {"suoxie": "HTFZZQZZQGSZSA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHUA"}, "501048": {"suoxie": "HTFZZQZZQGSZSC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHUC"}, "501049": {"suoxie": "DFHRXSNDKHHA", "name": "\u4e1c\u65b9\u7ea2\u777f\u73ba\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIXISANNIANDINGKAIHUNHEA"}, "501050": {"suoxie": "HXSZ50AHYXZSA", "name": "\u534e\u590f\u4e0a\u8bc150AH\u4f18\u9009\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENG50AHYOUXUANZHISHUA"}, "501051": {"suoxie": "YXYFHLHHLOF", "name": "\u5706\u4fe1\u6c38\u4e30\u6c47\u5229\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YUANXINYONGFENGHUILIHUNHELOF"}, "501053": {"suoxie": "DFHMBYXDKHH", "name": "\u4e1c\u65b9\u7ea2\u76ee\u6807\u4f18\u9009\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGFANGHONGMUBIAOYOUXUANDINGKAIHUNHE"}, "501054": {"suoxie": "DFHRZSNDKHHA", "name": "\u4e1c\u65b9\u7ea2\u777f\u6cfd\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGRUIZESANNIANDINGKAIHUNHEA"}, "501057": {"suoxie": "HTFZZXNYQCA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINNENGYUANQICHEA"}, "501058": {"suoxie": "HTFZZXNYQCC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINNENGYUANQICHEC"}, "501059": {"suoxie": "XBLDGQHLLOFA", "name": "\u897f\u90e8\u5229\u5f97\u56fd\u4f01\u7ea2\u5229(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEGUOQIHONGLILOFA"}, "501060": {"suoxie": "ZJYX300ZSLOFA", "name": "\u4e2d\u91d1\u4f18\u9009300\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINYOUXUAN300ZHISHULOFA"}, "501061": {"suoxie": "ZJYX300ZSLOFC", "name": "\u4e2d\u91d1\u4f18\u9009300\u6307\u6570(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINYOUXUAN300ZHISHULOFC"}, "501062": {"suoxie": "NFRHDKHHLOF", "name": "\u5357\u65b9\u745e\u5408\u5b9a\u5f00\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGRUIHEDINGKAIHUNHELOF"}, "501063": {"suoxie": "HTFYXDKHH", "name": "\u6c47\u6dfb\u5bcc\u60a6\u4eab\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYUEXIANGDINGKAIHUNHE"}, "501064": {"suoxie": "GTJZYXLHPZHH", "name": "\u56fd\u6cf0\u4ef7\u503c\u4f18\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJIAZHIYOUXUANLINGHUOPEIZHIHUNHE"}, "501065": {"suoxie": "HTFJDCZDKHH", "name": "\u6c47\u6dfb\u5bcc\u7ecf\u5178\u6210\u957f\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJINGDIANCHENGZHANGDINGKAIHUNHE"}, "501066": {"suoxie": "DFHHYWNDKHH", "name": "\u4e1c\u65b9\u7ea2\u6052\u5143\u4e94\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGFANGHONGHENGYUANWUNIANDINGKAIHUNHE"}, "501067": {"suoxie": "ZSFSAH50ZSA", "name": "\u62db\u5546\u5bcc\u65f6A-H50\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGFUSHIAH50ZHISHUA"}, "501068": {"suoxie": "ZSFSAH50ZSC", "name": "\u62db\u5546\u5bcc\u65f6A-H50\u6307\u6570C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGFUSHIAH50ZHISHUC"}, "501069": {"suoxie": "HBBPZGAGZLJZ", "name": "\u534e\u5b9d\u6807\u666e\u4e2d\u56fdA\u80a1\u8d28\u91cf\u4ef7\u503c", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOBIAOPUZHONGGUOAGUZHILIANGJIAZHI"}, "501070": {"suoxie": "GFRYSNDKHH", "name": "\u5e7f\u53d1\u777f\u9633\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFARUIYANGSANNIANDINGKAIHUNHE"}, "501071": {"suoxie": "HDSNFBFZHH", "name": "\u6cd3\u5fb7\u4e09\u5e74\u5c01\u95ed\u4e30\u6cfd\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HONGDESANNIANFENGBIFENGZEHUNHE"}, "501073": {"suoxie": "HAKCZTHH", "name": "\u534e\u5b89\u79d1\u521b\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANKECHUANGZHUTIHUNHE"}, "501075": {"suoxie": "WJKC3NFBHHA", "name": "\u4e07\u5bb6\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAKECHUANG3NIANFENGBIHUNHEA"}, "501076": {"suoxie": "PHKC3NFBHH", "name": "\u9e4f\u534e\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PENGHUAKECHUANG3NIANFENGBIHUNHE"}, "501077": {"suoxie": "FGKCZTHH", "name": "\u5bcc\u56fd\u79d1\u521b\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUGUOKECHUANGZHUTIHUNHE"}, "501078": {"suoxie": "GFKCZT3NFBHH", "name": "\u5e7f\u53d1\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501079": {"suoxie": "DCKCZT3NFBHH", "name": "\u5927\u6210\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DACHENGKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501080": {"suoxie": "ZJKCZT3NFBHH", "name": "\u4e2d\u91d1\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501081": {"suoxie": "ZOKCZT3NFBHH", "name": "\u4e2d\u6b27\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGOUKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501082": {"suoxie": "BSKCZT3NFBHH", "name": "\u535a\u65f6\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "BOSHIKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501083": {"suoxie": "YHKCZT3NFBHH", "name": "\u94f6\u534e\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHUAKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501085": {"suoxie": "CTKCZT3NFBHH", "name": "\u8d22\u901a\u79d1\u521b\u4e3b\u98983\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CAITONGKECHUANGZHUTI3NIANFENGBIHUNHE"}, "501086": {"suoxie": "HBMSCIZGAGGJTLOFA", "name": "\u534e\u5b9dMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOMSCIZHONGGUOAGUGUOJITONGLOFA"}, "501087": {"suoxie": "JYRFSNFBHH", "name": "\u4ea4\u94f6\u745e\u4e30\u4e09\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINRUIFENGSANNIANFENGBIHUNHE"}, "501088": {"suoxie": "JSRHSNDQHH", "name": "\u5609\u5b9e\u745e\u8679\u4e09\u5e74\u5b9a\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIRUIHONGSANNIANDINGQIHUNHE"}, "501089": {"suoxie": "FZFBXFHLZSZQLOF", "name": "\u65b9\u6b63\u5bcc\u90a6\u6d88\u8d39\u7ea2\u5229\u6307\u6570\u589e\u5f3a(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGXIAOFEIHONGLIZHISHUZENGQIANGLOF"}, "501090": {"suoxie": "HBZZXFLTZSLOFA", "name": "\u534e\u5b9d\u4e2d\u8bc1\u6d88\u8d39\u9f99\u5934\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIAOFEILONGTOUZHISHULOFA"}, "501091": {"suoxie": "JSRXSNFBYZHHA", "name": "\u5609\u5b9e\u745e\u7199\u4e09\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIASHIRUIXISANNIANFENGBIYUNZUOHUNHEA"}, "501092": {"suoxie": "JYRSSNFBYZHH", "name": "\u4ea4\u94f6\u745e\u601d\u4e09\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINRUISISANNIANFENGBIYUNZUOHUNHE"}, "501093": {"suoxie": "HXXYLNDKHH", "name": "\u534e\u590f\u7fd4\u9633\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXIANGYANGLIANGNIANDINGKAIHUNHE"}, "501095": {"suoxie": "ZYZQKC3NFBHH", "name": "\u4e2d\u94f6\u8bc1\u5238\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYINZHENGQUANKECHUANG3NIANFENGBIHUNHE"}, "501096": {"suoxie": "GLAKC3NFBHH", "name": "\u56fd\u8054\u5b89\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOLIANANKECHUANG3NIANFENGBIHUNHE"}, "501097": {"suoxie": "GSABKC3NFBHH", "name": "\u56fd\u5bff\u5b89\u4fdd\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOSHOUANBAOKECHUANG3NIANFENGBIHUNHE"}, "501098": {"suoxie": "JXKJCX3NFBHH", "name": "\u5efa\u4fe1\u79d1\u6280\u521b\u65b03\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIANXINKEJICHUANGXIN3NIANFENGBIHUNHE"}, "501099": {"suoxie": "PAKJCX3NFBHH", "name": "\u5e73\u5b89\u79d1\u6280\u521b\u65b03\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANKEJICHUANGXIN3NIANFENGBIHUNHE"}, "501100": {"suoxie": "BSAKDKZLOF", "name": "\u535a\u65f6\u5b89\u5eb7\u5b9a\u5f00\u503a(LOF)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "BOSHIANKANGDINGKAIZHAILOF"}, "501186": {"suoxie": "HXXRHHLOF", "name": "\u534e\u590f\u5174\u878d\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAXIAXINGRONGHUNHELOF"}, "501188": {"suoxie": "HTFHXJXHHLOF", "name": "\u6c47\u6dfb\u5bcc\u6838\u5fc3\u7cbe\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFUHEXINJINGXUANHUNHELOF"}, "501189": {"suoxie": "JSCYYXHHLOF", "name": "\u5609\u5b9e\u4ea7\u4e1a\u4f18\u9009\u6df7\u5408(LOF)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIASHICHANYEYOUXUANHUNHELOF"}, "501200": {"suoxie": "MSJYKC3NFBHH", "name": "\u6c11\u751f\u52a0\u94f6\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINKECHUANG3NIANFENGBIHUNHE"}, "501201": {"suoxie": "HTKJCX3NFBHH", "name": "\u7ea2\u571f\u79d1\u6280\u521b\u65b03\u5e74\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HONGTUKEJICHUANGXIN3NIANFENGBIHUNHE"}, "501202": {"suoxie": "HTZJKC3NFBHHC", "name": "\u534e\u6cf0\u7d2b\u91d1\u79d1\u521b3\u5e74\u5c01\u95ed\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUATAIZIJINKECHUANG3NIANFENGBIHUNHEC"}, "501203": {"suoxie": "YFDCXWL18GYFBHH", "name": "\u6613\u65b9\u8fbe\u521b\u65b0\u672a\u676518\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDACHUANGXINWEILAI18GEYUEFENGBIHUNHE"}, "501205": {"suoxie": "PHCXWL18GYFBHH", "name": "\u9e4f\u534e\u521b\u65b0\u672a\u676518\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PENGHUACHUANGXINWEILAI18GEYUEFENGBIHUNHE"}, "501206": {"suoxie": "HTFCXWL18GYFBHH", "name": "\u6c47\u6dfb\u5bcc\u521b\u65b0\u672a\u676518\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHUANGXINWEILAI18GEYUEFENGBIHUNHE"}, "501207": {"suoxie": "HXCXWL18GYFBHH", "name": "\u534e\u590f\u521b\u65b0\u672a\u676518\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIACHUANGXINWEILAI18GEYUEFENGBIHUNHE"}, "501208": {"suoxie": "ZOCXWL18GYFBHH", "name": "\u4e2d\u6b27\u521b\u65b0\u672a\u676518\u4e2a\u6708\u5c01\u95ed\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUCHUANGXINWEILAI18GEYUEFENGBIHUNHE"}, "501209": {"suoxie": "YHFJSPYLJXHHLOFA", "name": "\u94f6\u534e\u5bcc\u4e45\u98df\u54c1\u996e\u6599\u7cbe\u9009\u6df7\u5408(LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAFUJIUSHIPINYINLIAOJINGXUANHUNHELOFA"}, "501210": {"suoxie": "JYZXXGYNFBYZHHFOFLOFA", "name": "\u4ea4\u94f6\u667a\u9009\u661f\u5149\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u6df7\u5408(FOF-LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINZHIXUANXINGGUANGYINIANFENGBIYUNZUOHUNHEFOFLOFA"}, "501211": {"suoxie": "MSJYYX6GYDKHHFOFLOF", "name": "\u6c11\u751f\u52a0\u94f6\u4f18\u4eab6\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408(FOF-LOF)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "MINSHENGJIAYINYOUXIANG6GEYUEDINGKAIHUNHEFOFLOF"}, "501212": {"suoxie": "GFYXPZLNFBHHFOFLOFA", "name": "\u5e7f\u53d1\u4f18\u9009\u914d\u7f6e\u4e24\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAYOUXUANPEIZHILIANGNIANFENGBIHUNHEFOFLOFA"}, "501213": {"suoxie": "ZOHXYNFBHHFOFLOFA", "name": "\u4e2d\u6b27\u6c47\u9009\u4e00\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGOUHUIXUANYINIANFENGBIHUNHEFOFLOFA"}, "501215": {"suoxie": "XZQQJJPZSNFBHHFOFLOFA", "name": "\u5174\u8bc1\u5168\u7403\u79ef\u6781\u914d\u7f6e\u4e09\u5e74\u5c01\u95ed\u6df7\u5408(FOF-LOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGQUANQIUJIJIPEIZHISANNIANFENGBIHUNHEFOFLOFA"}, "501216": {"suoxie": "FGZXHYJXYNFBGPFOFLOFA", "name": "\u5bcc\u56fd\u667a\u946b\u884c\u4e1a\u7cbe\u9009\u4e00\u5e74\u5c01\u95ed\u80a1\u7968(FOF-LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "FUGUOZHIXINHANGYEJINGXUANYINIANFENGBIGUPIAOFOFLOFA"}, "501217": {"suoxie": "HXHYPZYNFBGPFOFLOFA", "name": "\u534e\u590f\u884c\u4e1a\u914d\u7f6e\u4e00\u5e74\u5c01\u95ed\u80a1\u7968(FOF-LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAHANGYEPEIZHIYINIANFENGBIGUPIAOFOFLOFA"}, "501218": {"suoxie": "GYRZJQYNFBYZGPA", "name": "\u5de5\u94f6\u777f\u667a\u8fdb\u53d6\u4e00\u5e74\u5c01\u95ed\u8fd0\u4f5c\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GONGYINRUIZHIJINQUYINIANFENGBIYUNZUOGUPIAOA"}, "501219": {"suoxie": "HXZSXFGPLOFA", "name": "\u534e\u590f\u667a\u80dc\u5148\u950b\u80a1\u7968(LOF)A", "style": "\u80a1\u7968\u578b", "pinyin": "HUAXIAZHISHENGXIANFENGGUPIAOLOFA"}, "501300": {"suoxie": "HFTQQSYZQRMB", "name": "\u6d77\u5bcc\u901a\u5168\u7403\u6536\u76ca\u503a\u5238\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "HAIFUTONGQUANQIUSHOUYIZHAIQUANRENMINBI"}, "501301": {"suoxie": "HBXGDPA", "name": "\u534e\u5b9d\u9999\u6e2f\u5927\u76d8A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOXIANGGANGDAPANA"}, "501302": {"suoxie": "NFHZETFLJLOFA", "name": "\u5357\u65b9\u6052\u6307ETF\u8054\u63a5(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGHENGZHIETFLIANJIELOFA"}, "501303": {"suoxie": "GFHSZXGZSA", "name": "\u5e7f\u53d1\u6052\u751f\u4e2d\u578b\u80a1\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHENGSHENGZHONGXINGGUZHISHUA"}, "501305": {"suoxie": "HTFZZGGTLOFA", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6e2f\u80a1\u901a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGANGGUTONGLOFA"}, "501306": {"suoxie": "HTFZZGGTLOFC", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6e2f\u80a1\u901a(LOF)C", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGANGGUTONGLOFC"}, "501307": {"suoxie": "YHZZHGSGGXA", "name": "\u94f6\u6cb3\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u9ad8\u80a1\u606fA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEZHONGZHENGHUGANGSHENGAOGUXIA"}, "501308": {"suoxie": "YHZZHGSGGXC", "name": "\u94f6\u6cb3\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u9ad8\u80a1\u606fC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEZHONGZHENGHUGANGSHENGAOGUXIC"}, "501309": {"suoxie": "GTHSGGTZSLOF", "name": "\u56fd\u6cf0\u6052\u751f\u6e2f\u80a1\u901a\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHENGSHENGGANGGUTONGZHISHULOF"}, "501310": {"suoxie": "HBHGSZGZQLOFA", "name": "\u534e\u5b9d\u6caa\u6e2f\u6df1\u4e2d\u56fd\u589e\u5f3a(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOHUGANGSHENZHONGGUOZENGQIANGLOFA"}, "501311": {"suoxie": "JSGGTXJJZSA", "name": "\u5609\u5b9e\u6e2f\u80a1\u901a\u65b0\u7ecf\u6d4e\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIGANGGUTONGXINJINGJIZHISHUA"}, "502000": {"suoxie": "XBLDZZ500ZSZQA", "name": "\u897f\u90e8\u5229\u5f97\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEZHONGZHENG500ZHISHUZENGQIANGA"}, "502003": {"suoxie": "YFDZZJGLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u519b\u5de5(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGJUNGONGLOFA"}, "502006": {"suoxie": "YFDZZGQGGLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u56fd\u4f01\u6539\u9769(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGUOQIGAIGELOFA"}, "502010": {"suoxie": "YFDZZQZZQGSZSLOFA", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIZHENGQUANGONGSIZHISHULOFA"}, "502013": {"suoxie": "CSZZSWYDYLZSLOF", "name": "\u957f\u76db\u4e2d\u8bc1\u7533\u4e07\u4e00\u5e26\u4e00\u8def\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGZHONGZHENGSHENWANYIDAIYILUZHISHULOF"}, "502020": {"suoxie": "GJSZ50ZSZQLOF", "name": "\u56fd\u91d1\u4e0a\u8bc150\u6307\u6570\u589e\u5f3a(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOJINSHANGZHENG50ZHISHUZENGQIANGLOF"}, "502023": {"suoxie": "PHGZGTHYZSLOFA", "name": "\u9e4f\u534e\u56fd\u8bc1\u94a2\u94c1\u884c\u4e1a\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAGUOZHENGGANGTIEHANGYEZHISHULOFA"}, "502040": {"suoxie": "CSSZ50ZSLOF", "name": "\u957f\u76db\u4e0a\u8bc150\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGSHANGZHENG50ZHISHULOF"}, "502048": {"suoxie": "YFDSZ50ZSLOFA", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ZHISHULOFA"}, "502053": {"suoxie": "CSZZQZZQZSLOF", "name": "\u957f\u76db\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u6307\u6570(LOF)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGZHONGZHENGQUANZHIZHENGQUANZHISHULOF"}, "502056": {"suoxie": "GFZZYLZSLOFA", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u533b\u7597\u6307\u6570(LOF)A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGYILIAOZHISHULOFA"}, "506000": {"suoxie": "NFKCB3NDKHH", "name": "\u5357\u65b9\u79d1\u521b\u677f3\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NANFANGKECHUANGBAN3NIANDINGKAIHUNHE"}, "506001": {"suoxie": "WJKCB2NDKHH", "name": "\u4e07\u5bb6\u79d1\u521b\u677f2\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAKECHUANGBAN2NIANDINGKAIHUNHE"}, "506002": {"suoxie": "YFDKCBLNDKHH", "name": "\u6613\u65b9\u8fbe\u79d1\u521b\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIFANGDAKECHUANGBANLIANGNIANDINGKAIHUNHE"}, "506003": {"suoxie": "FGKCBLNDKHH", "name": "\u5bcc\u56fd\u79d1\u521b\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOKECHUANGBANLIANGNIANDINGKAIHUNHE"}, "506005": {"suoxie": "BSKCBSNDKHH", "name": "\u535a\u65f6\u79d1\u521b\u677f\u4e09\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "BOSHIKECHUANGBANSANNIANDINGKAIHUNHE"}, "506006": {"suoxie": "HTFKCB2NDKHH", "name": "\u6c47\u6dfb\u5bcc\u79d1\u521b\u677f2\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUKECHUANGBAN2NIANDINGKAIHUNHE"}, "506007": {"suoxie": "GFKCBLNDKHH", "name": "\u5e7f\u53d1\u79d1\u521b\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAKECHUANGBANLIANGNIANDINGKAIHUNHE"}, "506008": {"suoxie": "CCKCLNDKHHA", "name": "\u957f\u57ce\u79d1\u521b\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGCHENGKECHUANGLIANGNIANDINGKAIHUNHEA"}, "506009": {"suoxie": "GTKCBLNDKHH", "name": "\u56fd\u6cf0\u79d1\u521b\u677f\u4e24\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIKECHUANGBANLIANGNIANDINGKAIHUNHE"}, "508000": {"suoxie": "ZJGDYREIT", "name": "\u5f20\u6c5f\u5149\u5927\u56edREIT", "style": "Reits", "pinyin": ""}, "508001": {"suoxie": "HHYHHREIT", "name": "\u6caa\u676d\u752c\u676d\u5fbdREIT", "style": "Reits", "pinyin": ""}, "508006": {"suoxie": "FGSCSWREIT", "name": "\u5bcc\u56fd\u9996\u521b\u6c34\u52a1REIT", "style": "Reits", "pinyin": ""}, "508027": {"suoxie": "DWSYCYREIT", "name": "\u4e1c\u5434\u82cf\u56ed\u4ea7\u4e1aREIT", "style": "Reits", "pinyin": ""}, "508056": {"suoxie": "ZJPLSREIT", "name": "\u4e2d\u91d1\u666e\u6d1b\u65afREIT", "style": "Reits", "pinyin": ""}, "508099": {"suoxie": "JXZGCREIT", "name": "\u5efa\u4fe1\u4e2d\u5173\u6751REIT", "style": "Reits", "pinyin": ""}, "510010": {"suoxie": "JYSZ180GSZLETF", "name": "\u4ea4\u94f6\u4e0a\u8bc1180\u516c\u53f8\u6cbb\u7406ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHANGZHENG180GONGSIZHILIETF"}, "510020": {"suoxie": "BSSZCDPETF", "name": "\u535a\u65f6\u4e0a\u8bc1\u8d85\u5927\u76d8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENGCHAODAPANETF"}, "510030": {"suoxie": "HBSZ180JZETF", "name": "\u534e\u5b9d\u4e0a\u8bc1180\u4ef7\u503cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHANGZHENG180JIAZHIETF"}, "510050": {"suoxie": "HXSZ50ETF", "name": "\u534e\u590f\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENG50ETF"}, "510060": {"suoxie": "GYSZYQETF", "name": "\u5de5\u94f6\u4e0a\u8bc1\u592e\u4f01ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHANGZHENGYANGQIETF"}, "510080": {"suoxie": "CSQZZSZQZQ", "name": "\u957f\u76db\u5168\u503a\u6307\u6570\u589e\u5f3a\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGSHENGQUANZHAIZHISHUZENGQIANGZHAIQUAN"}, "510081": {"suoxie": "CSDTJXHH", "name": "\u957f\u76db\u52a8\u6001\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGDONGTAIJINGXUANHUNHE"}, "510090": {"suoxie": "JXZRETF", "name": "\u5efa\u4fe1\u8d23\u4efbETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZERENETF"}, "510100": {"suoxie": "YFDSZ50ETF", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENG50ETF"}, "510110": {"suoxie": "HFTSZZQETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u5468\u671fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGSHANGZHENGZHOUQIETF"}, "510120": {"suoxie": "HFTSZFZQETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u975e\u5468\u671fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGSHANGZHENGFEIZHOUQIETF"}, "510130": {"suoxie": "ZPETF", "name": "\u4e2d\u76d8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGPANETF"}, "510150": {"suoxie": "ZSSZXF80ETF", "name": "\u62db\u5546\u4e0a\u8bc1\u6d88\u8d3980ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHANGZHENGXIAOFEI80ETF"}, "510160": {"suoxie": "ZZNFXKCYZSETF", "name": "\u4e2d\u8bc1\u5357\u65b9\u5c0f\u5eb7\u4ea7\u4e1a\u6307\u6570ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGZHENGNANFANGXIAOKANGCHANYEZHISHUETF"}, "510170": {"suoxie": "GLASZSPETF", "name": "\u56fd\u8054\u5b89\u4e0a\u8bc1\u5546\u54c1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANSHANGZHENGSHANGPINETF"}, "510180": {"suoxie": "HASZ180ETF", "name": "\u534e\u5b89\u4e0a\u8bc1180ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHANGZHENG180ETF"}, "510190": {"suoxie": "HASZLTETF", "name": "\u534e\u5b89\u4e0a\u8bc1\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHANGZHENGLONGTOUETF"}, "510200": {"suoxie": "HASZZQETF", "name": "\u6c47\u5b89\u4e0a\u8bc1\u8bc1\u5238ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANSHANGZHENGZHENGQUANETF"}, "510210": {"suoxie": "SZZZETF", "name": "\u4e0a\u8bc1\u7efc\u6307ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGZHENGZONGZHIETF"}, "510220": {"suoxie": "HTBRSZZXPETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u4e2d\u5c0f\u76d8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGZHONGXIAOPANETF"}, "510230": {"suoxie": "GTSZ180JRETF", "name": "\u56fd\u6cf0\u4e0a\u8bc1180\u91d1\u878dETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENG180JINRONGETF"}, "510270": {"suoxie": "ZYSZGQ100ETF", "name": "\u4e2d\u94f6\u4e0a\u8bc1\u56fd\u4f01100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINSHANGZHENGGUOQI100ETF"}, "510290": {"suoxie": "NFSZ380ETF", "name": "\u5357\u65b9\u4e0a\u8bc1380ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENG380ETF"}, "510300": {"suoxie": "HTBRHS300ETF", "name": "\u534e\u6cf0\u67cf\u745e\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIHUSHEN300ETF"}, "510310": {"suoxie": "YFDHS300FQSETF", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u53d1\u8d77\u5f0fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300FAQISHIETF"}, "510330": {"suoxie": "HXHS300ETF", "name": "\u534e\u590f\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUSHEN300ETF"}, "510350": {"suoxie": "GYRXHS300ETF", "name": "\u5de5\u94f6\u745e\u4fe1\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINHUSHEN300ETF"}, "510360": {"suoxie": "GFHS300ETF", "name": "\u5e7f\u53d1\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAHUSHEN300ETF"}, "510370": {"suoxie": "XYHS300ETF", "name": "\u5174\u4e1a\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYEHUSHEN300ETF"}, "510380": {"suoxie": "GSABHS300ETF", "name": "\u56fd\u5bff\u5b89\u4fdd\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOHUSHEN300ETF"}, "510390": {"suoxie": "PAHS300ETF", "name": "\u5e73\u5b89\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANHUSHEN300ETF"}, "510410": {"suoxie": "BSSZZRZYETF", "name": "\u535a\u65f6\u4e0a\u8bc1\u81ea\u7136\u8d44\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENGZIRANZIYUANETF"}, "510440": {"suoxie": "DCZZ500HSETF", "name": "\u5927\u6210\u4e2d\u8bc1500\u6caa\u5e02ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENG500HUSHIETF"}, "510500": {"suoxie": "NFZZ500ETF", "name": "\u5357\u65b9\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500ETF"}, "510510": {"suoxie": "GFZZ500ETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG500ETF"}, "510530": {"suoxie": "GYZZ500ETF", "name": "\u5de5\u94f6\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENG500ETF"}, "510550": {"suoxie": "FZFBZZ500ETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENG500ETF"}, "510560": {"suoxie": "GSABZZ500ETF", "name": "\u56fd\u5bff\u5b89\u4fdd\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOZHONGZHENG500ETF"}, "510570": {"suoxie": "XYZZ500ETF", "name": "\u5174\u4e1a\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYEZHONGZHENG500ETF"}, "510580": {"suoxie": "YFDZZ500ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG500ETF"}, "510590": {"suoxie": "PAZZ500ETF", "name": "\u5e73\u5b89\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENG500ETF"}, "510600": {"suoxie": "SWLXSZ50ETF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINSHANGZHENG50ETF"}, "510630": {"suoxie": "HXXFETF", "name": "\u534e\u590f\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAXIAOFEIETF"}, "510650": {"suoxie": "SZJRDCFQSETF", "name": "\u4e0a\u8bc1\u91d1\u878d\u5730\u4ea7\u53d1\u8d77\u5f0fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGZHENGJINRONGDICHANFAQISHIETF"}, "510660": {"suoxie": "HXYYETF", "name": "\u534e\u590f\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAYIYAOETF"}, "510680": {"suoxie": "WJSZ50ETF", "name": "\u4e07\u5bb6\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIASHANGZHENG50ETF"}, "510690": {"suoxie": "XYSZ180JRETF", "name": "\u5174\u4e1a\u4e0a\u8bc1180\u91d1\u878dETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYESHANGZHENG180JINRONGETF"}, "510710": {"suoxie": "BSSZ50ETF", "name": "\u535a\u65f6\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHISHANGZHENG50ETF"}, "510760": {"suoxie": "GTSZZHJYETF", "name": "\u56fd\u6cf0\u4e0a\u8bc1\u7efc\u5408\u4ea4\u6613ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAISHANGZHENGZONGHEJIAOYIETF"}, "510770": {"suoxie": "SWLXSZG60CXETF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e0a\u8bc1G60\u521b\u65b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINSHANGZHENGG60CHUANGXINETF"}, "510800": {"suoxie": "JXSZ50ETF", "name": "\u5efa\u4fe1\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHANGZHENG50ETF"}, "510810": {"suoxie": "HTFZZSHGQETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u4e0a\u6d77\u56fd\u4f01ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGSHANGHAIGUOQIETF"}, "510850": {"suoxie": "GYRXSZ50ETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINSHANGZHENG50ETF"}, "510860": {"suoxie": "XYSZ50ETF", "name": "\u5174\u4e1a\u4e0a\u8bc150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYESHANGZHENG50ETF"}, "510880": {"suoxie": "HTBRSZHLETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGHONGLIETF"}, "510900": {"suoxie": "YFDHSGQETF", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u56fd\u4f01ETF", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGGUOQIETF"}, "510990": {"suoxie": "GYRXZZ180ESGETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1180ESGETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENG180ESGETF"}, "511010": {"suoxie": "GTSZ5NQGZETF", "name": "\u56fd\u6cf0\u4e0a\u8bc15\u5e74\u671f\u56fd\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAISHANGZHENG5NIANQIGUOZHAIETF"}, "511020": {"suoxie": "PAZZ510NGZHYQETF", "name": "\u5e73\u5b89\u4e2d\u8bc15-10\u5e74\u56fd\u503a\u6d3b\u8dc3\u5238ETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHENG510NIANGUOZHAIHUOYUEQUANETF"}, "511030": {"suoxie": "PAZZZLCYZETF", "name": "\u5e73\u5b89\u4e2d\u503a\u503a\u5229\u5dee\u56e0\u5b50ETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANZHONGZHAIZHAILICHAYINZIETF"}, "511060": {"suoxie": "HFTSZ5NQETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc15\u5e74\u671fETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGSHANGZHENG5NIANQIETF"}, "511080": {"suoxie": "CSQZZSZQZQ", "name": "\u957f\u76db\u5168\u503a\u6307\u6570\u589e\u5f3a\u503a\u5238(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGSHENGQUANZHAIZHISHUZENGQIANGZHAIQUAN"}, "511081": {"suoxie": "CSDTJXHH", "name": "\u957f\u76db\u52a8\u6001\u7cbe\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGDONGTAIJINGXUANHUNHE"}, "511180": {"suoxie": "HFTSZTZJKZZETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u6295\u8d44\u7ea7\u53ef\u8f6c\u503aETF", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "HAIFUTONGSHANGZHENGTOUZIJIKEZHUANZHAIETF"}, "511220": {"suoxie": "HFTSZCTZETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u57ce\u6295\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGSHANGZHENGCHENGTOUZHAIETF"}, "511260": {"suoxie": "GTSZ10NQGZETF", "name": "\u56fd\u6cf0\u4e0a\u8bc110\u5e74\u671f\u56fd\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAISHANGZHENG10NIANQIGUOZHAIETF"}, "511270": {"suoxie": "HFTSZ10NQETF", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc110\u5e74\u671fETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGSHANGZHENG10NIANQIETF"}, "511310": {"suoxie": "FGZZ10NQGZETF", "name": "\u5bcc\u56fd\u4e2d\u8bc110\u5e74\u671f\u56fd\u503aETF", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "FUGUOZHONGZHENG10NIANQIGUOZHAIETF"}, "511360": {"suoxie": "HFTZZDRETF", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1\u77ed\u878dETF", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAIFUTONGZHONGZHENGDUANRONGETF"}, "511380": {"suoxie": "BSKZZETF", "name": "\u535a\u65f6\u53ef\u8f6c\u503aETF", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "BOSHIKEZHUANZHAIETF"}, "511600": {"suoxie": "HARRXHBH", "name": "\u534e\u5b89\u65e5\u65e5\u946b\u8d27\u5e01H", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAANRIRIXINHUOBIH"}, "511620": {"suoxie": "GTSLHBETF", "name": "\u56fd\u6cf0\u77ac\u5229\u8d27\u5e01ETF", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAISHUNLIHUOBIETF"}, "511650": {"suoxie": "HXKXJYXHB", "name": "\u534e\u590f\u5feb\u7ebf\u4ea4\u6613\u578b\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIAKUAIXIANJIAOYIXINGHUOBI"}, "511660": {"suoxie": "JXXJTYHBH", "name": "\u5efa\u4fe1\u73b0\u91d1\u6dfb\u76ca\u8d27\u5e01H", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINXIANJINTIANYIHUOBIH"}, "511670": {"suoxie": "HTZJTTJJYXHBA", "name": "\u534e\u6cf0\u7d2b\u91d1\u5929\u5929\u91d1\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIZIJINTIANTIANJINJIAOYIXINGHUOBIA"}, "511690": {"suoxie": "DCTYJYXHBE", "name": "\u5927\u6210\u6dfb\u76ca\u4ea4\u6613\u578b\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGTIANYIJIAOYIXINGHUOBIE"}, "511700": {"suoxie": "PAJYXHBE", "name": "\u5e73\u5b89\u4ea4\u6613\u578b\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "PINGANJIAOYIXINGHUOBIE"}, "511770": {"suoxie": "JYZYHBE", "name": "\u91d1\u9e70\u589e\u76ca\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYINGZENGYIHUOBIE"}, "511800": {"suoxie": "YFDHBE", "name": "\u6613\u65b9\u8fbe\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "YIFANGDAHUOBIE"}, "511810": {"suoxie": "NFLCJJYXHBH", "name": "\u5357\u65b9\u7406\u8d22\u91d1\u4ea4\u6613\u578b\u8d27\u5e01H", "style": "\u8d27\u5e01\u578b", "pinyin": "NANFANGLICAIJINJIAOYIXINGHUOBIH"}, "511820": {"suoxie": "PHTLJYXHBB", "name": "\u9e4f\u534e\u6dfb\u5229\u4ea4\u6613\u578b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PENGHUATIANLIJIAOYIXINGHUOBIB"}, "511830": {"suoxie": "HTBRJYHBA", "name": "\u534e\u6cf0\u67cf\u745e\u4ea4\u6613\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIBAIRUIJIAOYIHUOBIA"}, "511850": {"suoxie": "ZSCFBJYXHBE", "name": "\u62db\u5546\u8d22\u5bcc\u5b9d\u4ea4\u6613\u578b\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHAOSHANGCAIFUBAOJIAOYIXINGHUOBIE"}, "511860": {"suoxie": "BSBZJHBETF", "name": "\u535a\u65f6\u4fdd\u8bc1\u91d1\u8d27\u5e01ETF", "style": "\u8d27\u5e01\u578b", "pinyin": "BOSHIBAOZHENGJINHUOBIETF"}, "511880": {"suoxie": "YHRLA", "name": "\u94f6\u534e\u65e5\u5229A", "style": "\u8d27\u5e01\u578b", "pinyin": "YINHUARILIA"}, "511900": {"suoxie": "FGSYBJYXHBH", "name": "\u5bcc\u56fd\u6536\u76ca\u5b9d\u4ea4\u6613\u578b\u8d27\u5e01H", "style": "\u8d27\u5e01\u578b", "pinyin": "FUGUOSHOUYIBAOJIAOYIXINGHUOBIH"}, "511910": {"suoxie": "RTYZFHBE", "name": "\u878d\u901a\u6613\u652f\u4ed8\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "RONGTONGYIZHIFUHUOBIE"}, "511920": {"suoxie": "GFHBE", "name": "\u5e7f\u53d1\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAHUOBIE"}, "511930": {"suoxie": "ZRRRYJYXHBA", "name": "\u4e2d\u878d\u65e5\u65e5\u76c8\u4ea4\u6613\u578b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ZHONGRONGRIRIYINGJIAOYIXINGHUOBIA"}, "511950": {"suoxie": "GFTLHBETFA", "name": "\u5e7f\u53d1\u6dfb\u5229\u8d27\u5e01ETFA", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFATIANLIHUOBIETFA"}, "511960": {"suoxie": "JSKXHBH", "name": "\u5609\u5b9e\u5feb\u7ebf\u8d27\u5e01H", "style": "\u8d27\u5e01\u578b", "pinyin": "JIASHIKUAIXIANHUOBIH"}, "511970": {"suoxie": "GSABHBE", "name": "\u56fd\u5bff\u5b89\u4fdd\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOHUOBIE"}, "511980": {"suoxie": "HTFTFTHBE", "name": "\u6c47\u6dfb\u5bcc\u6dfb\u5bcc\u901a\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUTIANFUTONGHUOBIE"}, "511990": {"suoxie": "HBXJTYA", "name": "\u534e\u5b9d\u73b0\u91d1\u6dfb\u76caA", "style": "\u8d27\u5e01\u578b", "pinyin": "HUABAOXIANJINTIANYIA"}, "512000": {"suoxie": "HBQSETF", "name": "\u534e\u5b9d\u5238\u5546ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOQUANSHANGETF"}, "512010": {"suoxie": "YFDHS300YYETF", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300YIYAOETF"}, "512040": {"suoxie": "FGZZJZETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u4ef7\u503cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJIAZHIETF"}, "512070": {"suoxie": "YFDHS300FYETF", "name": "\u6613\u65b9\u8fbe\u6caa\u6df1300\u975e\u94f6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHUSHEN300FEIYINETF"}, "512090": {"suoxie": "YFDMSCIZGAGETF", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOAGUETF"}, "512100": {"suoxie": "NFZZ1000ETF", "name": "\u5357\u65b9\u4e2d\u8bc11000ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG1000ETF"}, "512120": {"suoxie": "HAZZXFYYETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u7ec6\u5206\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXIFENYIYAOETF"}, "512150": {"suoxie": "HAFSZGA50ETF", "name": "\u6c47\u5b89\u5bcc\u65f6\u4e2d\u56fdA50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIANFUSHIZHONGGUOA50ETF"}, "512160": {"suoxie": "NFMSCIZGAGGJTETF", "name": "\u5357\u65b9MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGMSCIZHONGGUOAGUGUOJITONGETF"}, "512170": {"suoxie": "HBZZYLETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u533b\u7597ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGYILIAOETF"}, "512180": {"suoxie": "JXMSCIZGAGGJTETF", "name": "\u5efa\u4fe1MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINMSCIZHONGGUOAGUGUOJITONGETF"}, "512190": {"suoxie": "ZSZJFHETF", "name": "\u6d59\u5546\u4e4b\u6c5f\u51e4\u51f0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHESHANGZHIJIANGFENGHUANGETF"}, "512200": {"suoxie": "NFZZFDCETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u623f\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGFANGDICHANETF"}, "512220": {"suoxie": "JSZZKJCMTX150ETF", "name": "\u666f\u987a\u4e2d\u8bc1\u79d1\u6280\u4f20\u5a92\u901a\u4fe1150ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNZHONGZHENGKEJICHUANMEITONGXIN150ETF"}, "512260": {"suoxie": "HAZZDBDETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u4f4e\u6ce2\u52a8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGDIBODONGETF"}, "512280": {"suoxie": "JSMSCIZGAGETF", "name": "\u666f\u987aMSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNMSCIZHONGGUOAGUETF"}, "512290": {"suoxie": "GTZZSWYYETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u751f\u7269\u533b\u836fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGSHENGWUYIYAOETF"}, "512320": {"suoxie": "GYMSCIZGETF", "name": "\u5de5\u94f6MSCI\u4e2d\u56fdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINMSCIZHONGGUOETF"}, "512330": {"suoxie": "NFZZ500XXJSETF", "name": "\u5357\u65b9\u4e2d\u8bc1500\u4fe1\u606f\u6280\u672fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500XINXIJISHUETF"}, "512350": {"suoxie": "XYZZFJ50ETF", "name": "\u5174\u4e1a\u4e2d\u8bc1\u798f\u5efa50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINGYEZHONGZHENGFUJIAN50ETF"}, "512360": {"suoxie": "PAMSCIZGAGGJETF", "name": "\u5e73\u5b89MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANMSCIZHONGGUOAGUGUOJIETF"}, "512380": {"suoxie": "YHMSCIZGAGETF", "name": "\u94f6\u534eMSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAMSCIZHONGGUOAGUETF"}, "512390": {"suoxie": "PAMSCIZGAGETF", "name": "\u5e73\u5b89MSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANMSCIZHONGGUOAGUETF"}, "512400": {"suoxie": "NFZZSWYSJSETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u7533\u4e07\u6709\u8272\u91d1\u5c5eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGSHENWANYOUSEJINSHUETF"}, "512480": {"suoxie": "GLAZZBDTETF", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u534a\u5bfc\u4f53ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGBANDAOTIETF"}, "512500": {"suoxie": "HXZZ500ETF", "name": "\u534e\u590f\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG500ETF"}, "512510": {"suoxie": "HTBRZZ500ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENG500ETF"}, "512520": {"suoxie": "HTMSCIZGAGGJTETF", "name": "\u534e\u6cf0MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIMSCIZHONGGUOAGUGUOJITONGETF"}, "512530": {"suoxie": "JXHS300HLETF", "name": "\u5efa\u4fe1\u6caa\u6df1300\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINHUSHEN300HONGLIETF"}, "512550": {"suoxie": "JSFSZGA50ETF", "name": "\u5609\u5b9e\u5bcc\u65f6\u4e2d\u56fdA50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIFUSHIZHONGGUOA50ETF"}, "512560": {"suoxie": "YFDZZJGETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u519b\u5de5ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGJUNGONGETF"}, "512570": {"suoxie": "YFDZZQZZQGSETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "512580": {"suoxie": "GFZZHBETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u73af\u4fddETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGHUANBAOETF"}, "512590": {"suoxie": "PYASZZGGXETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u9ad8\u80a1\u606fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGGAOGUXIETF"}, "512600": {"suoxie": "JSZZZYXFETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u4e3b\u8981\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGZHUYAOXIAOFEIETF"}, "512610": {"suoxie": "JSZZYYWSETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u533b\u836f\u536b\u751fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGYIYAOWEISHENGETF"}, "512640": {"suoxie": "JSZZJRDCETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u91d1\u878d\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGJINRONGDICHANETF"}, "512650": {"suoxie": "TFZZCSJETF", "name": "\u6dfb\u5bcc\u4e2d\u8bc1\u957f\u4e09\u89d2ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUZHONGZHENGCHANGSANJIAOETF"}, "512660": {"suoxie": "GTZZJGETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u519b\u5de5ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGJUNGONGETF"}, "512670": {"suoxie": "PHZZGFETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u56fd\u9632ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGUOFANGETF"}, "512680": {"suoxie": "GFZZJGETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u519b\u5de5ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJUNGONGETF"}, "512690": {"suoxie": "PHZZJETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u9152ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGJIUETF"}, "512700": {"suoxie": "NFZZYHETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGYINHANGETF"}, "512710": {"suoxie": "FGZZJGLTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u519b\u5de5\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGJUNGONGLONGTOUETF"}, "512720": {"suoxie": "GTZZJSJETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u8ba1\u7b97\u673aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGJISUANJIETF"}, "512730": {"suoxie": "PHZZYHETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGYINHANGETF"}, "512750": {"suoxie": "JSZZRLJBMETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u9510\u8054\u57fa\u672c\u9762ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGRUILIANJIBENMIANETF"}, "512760": {"suoxie": "GTCESBDTXPETF", "name": "\u56fd\u6cf0CES\u534a\u5bfc\u4f53\u82af\u7247ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAICESBANDAOTIXINPIANETF"}, "512770": {"suoxie": "HXZLXXCZETF", "name": "\u534e\u590f\u6218\u7565\u65b0\u5174\u6210\u6307ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHANLUEXINXINGCHENGZHIETF"}, "512780": {"suoxie": "GFZZJJJFZETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u4eac\u6d25\u5180\u53d1\u5c55ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJINGJINJIFAZHANETF"}, "512800": {"suoxie": "HBZZYHETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGYINHANGETF"}, "512810": {"suoxie": "HBZZJGETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u519b\u5de5ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGJUNGONGETF"}, "512820": {"suoxie": "HTFZZYHETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGYINHANGETF"}, "512870": {"suoxie": "NHZZHZWQETF", "name": "\u5357\u534e\u4e2d\u8bc1\u676d\u5dde\u6e7e\u533aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANHUAZHONGZHENGHANGZHOUWANQUETF"}, "512880": {"suoxie": "GTZZQZZQGSETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "512890": {"suoxie": "HTBRZZDBDETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u4f4e\u6ce2\u52a8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGDIBODONGETF"}, "512900": {"suoxie": "NFZZQZZQGSETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "512910": {"suoxie": "GFZZ100ETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENG100ETF"}, "512920": {"suoxie": "XHMSCIZGAGGJETF", "name": "\u65b0\u534eMSCI\u4e2d\u56fdA\u80a1\u56fd\u9645ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAMSCIZHONGGUOAGUGUOJIETF"}, "512930": {"suoxie": "PARGZNETF", "name": "\u5e73\u5b89\u4eba\u5de5\u667a\u80fdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANRENGONGZHINENGETF"}, "512950": {"suoxie": "HXZZYQETF", "name": "\u534e\u590f\u4e2d\u8bc1\u592e\u4f01ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYANGQIETF"}, "512960": {"suoxie": "BSYQJGTZETF", "name": "\u535a\u65f6\u592e\u4f01\u7ed3\u6784\u8c03\u6574ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGQIJIEGOUTIAOZHENGETF"}, "512970": {"suoxie": "PAYGADWQETF", "name": "\u5e73\u5b89\u7ca4\u6e2f\u6fb3\u5927\u6e7e\u533aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANYUEGANGAODAWANQUETF"}, "512980": {"suoxie": "GFZZCMETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u4f20\u5a92ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANMEIETF"}, "512990": {"suoxie": "HXMSCIAGGJTETF", "name": "\u534e\u590fMSCIA\u80a1\u56fd\u9645\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAMSCIAGUGUOJITONGETF"}, "513000": {"suoxie": "YFDRJ225ETF", "name": "\u6613\u65b9\u8fbe\u65e5\u7ecf225ETF", "style": "QDII", "pinyin": "YIFANGDARIJING225ETF"}, "513010": {"suoxie": "YFDHSKJQDIIETF", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u79d1\u6280(QDII-ETF)", "style": "QDII", "pinyin": "YIFANGDAHENGSHENGKEJIQDIIETF"}, "513020": {"suoxie": "GTZZGGTKJETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6e2f\u80a1\u901a\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGGUTONGKEJIETF"}, "513030": {"suoxie": "HADGDAXETFQDII", "name": "\u534e\u5b89\u5fb7\u56fd(DAX)ETF(QDII)", "style": "QDII", "pinyin": "HUAANDEGUODAXETFQDII"}, "513050": {"suoxie": "YFDZGHL50ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u6982\u4e92\u805450ETF", "style": "QDII", "pinyin": "YIFANGDAZHONGGAIHULIAN50ETF"}, "513060": {"suoxie": "BSHSYLBJQDIIETF", "name": "\u535a\u65f6\u6052\u751f\u533b\u7597\u4fdd\u5065(QDII-ETF)", "style": "QDII", "pinyin": "BOSHIHENGSHENGYILIAOBAOJIANQDIIETF"}, "513070": {"suoxie": "YFDZZGGTXFZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "513080": {"suoxie": "HAFGCAC40ETFQDII", "name": "\u534e\u5b89\u6cd5\u56fdCAC40ETF(QDII)", "style": "QDII", "pinyin": "HUAANFAGUOCAC40ETFQDII"}, "513090": {"suoxie": "YFDZZXGZQTZETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u9999\u6e2f\u8bc1\u5238\u6295\u8d44ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXIANGGANGZHENGQUANTOUZIETF"}, "513100": {"suoxie": "GTNSDK100ETF", "name": "\u56fd\u6cf0\u7eb3\u65af\u8fbe\u514b100ETF", "style": "QDII", "pinyin": "GUOTAINASIDAKE100ETF"}, "513130": {"suoxie": "HTBRNFDYHSKJQDIIETF", "name": "\u534e\u6cf0\u67cf\u745e\u5357\u65b9\u4e1c\u82f1\u6052\u751f\u79d1\u6280(QDII-ETF)", "style": "QDII", "pinyin": "HUATAIBAIRUINANFANGDONGYINGHENGSHENGKEJIQDIIETF"}, "513150": {"suoxie": "HTBRGGTKJETF", "name": "\u534e\u6cf0\u67cf\u745e\u6e2f\u80a1\u901a\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIGANGGUTONGKEJIETF"}, "513160": {"suoxie": "YHHSGGTZGKJETF", "name": "\u94f6\u534e\u6052\u751f\u6e2f\u80a1\u901a\u4e2d\u56fd\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAHENGSHENGGANGGUTONGZHONGGUOKEJIETF"}, "513180": {"suoxie": "HXHSKJETFQDII", "name": "\u534e\u590f\u6052\u751f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "HUAXIAHENGSHENGKEJIETFQDII"}, "513200": {"suoxie": "YFDZZGGTYYWSZHETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6e2f\u80a1\u901a\u533b\u836f\u536b\u751f\u7efc\u5408ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGANGGUTONGYIYAOWEISHENGZONGHEETF"}, "513230": {"suoxie": "HXZZGGTXFZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "513300": {"suoxie": "HXNSDK100ETFQDII", "name": "\u534e\u590f\u7eb3\u65af\u8fbe\u514b100ETF(QDII)", "style": "QDII", "pinyin": "HUAXIANASIDAKE100ETFQDII"}, "513320": {"suoxie": "YFDHSGGTXJJETF", "name": "\u6613\u65b9\u8fbe\u6052\u751f\u6e2f\u80a1\u901a\u65b0\u7ecf\u6d4eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAHENGSHENGGANGGUTONGXINJINGJIETF"}, "513330": {"suoxie": "HXHSHLWKJYETFQDII", "name": "\u534e\u590f\u6052\u751f\u4e92\u8054\u7f51\u79d1\u6280\u4e1aETF(QDII)", "style": "QDII", "pinyin": "HUAXIAHENGSHENGHULIANWANGKEJIYEETFQDII"}, "513360": {"suoxie": "BSQQZGJYQDIIETF", "name": "\u535a\u65f6\u5168\u7403\u4e2d\u56fd\u6559\u80b2(QDII-ETF)", "style": "QDII", "pinyin": "BOSHIQUANQIUZHONGGUOJIAOYUQDIIETF"}, "513500": {"suoxie": "BSBP500ETF", "name": "\u535a\u65f6\u6807\u666e500ETF", "style": "QDII", "pinyin": "BOSHIBIAOPU500ETF"}, "513520": {"suoxie": "HXYCRJ225ETF", "name": "\u534e\u590f\u91ce\u6751\u65e5\u7ecf225ETF", "style": "QDII", "pinyin": "HUAXIAYECUNRIJING225ETF"}, "513550": {"suoxie": "HTBRZZGGT50ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u6e2f\u80a1\u901a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGGANGGUTONG50ETF"}, "513580": {"suoxie": "HAHSKJQDIIETF", "name": "\u534e\u5b89\u6052\u751f\u79d1\u6280(QDII-ETF)", "style": "QDII", "pinyin": "HUAANHENGSHENGKEJIQDIIETF"}, "513590": {"suoxie": "PHZZGGTXFZTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "513600": {"suoxie": "NFHZETF", "name": "\u5357\u65b9\u6052\u6307ETF", "style": "QDII", "pinyin": "NANFANGHENGZHIETF"}, "513660": {"suoxie": "HXHGTHSETF", "name": "\u534e\u590f\u6caa\u6e2f\u901a\u6052\u751fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAHUGANGTONGHENGSHENGETF"}, "513680": {"suoxie": "JXGGTHSZGETF", "name": "\u5efa\u4fe1\u6e2f\u80a1\u901a\u6052\u751f\u4e2d\u56fdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINGANGGUTONGHENGSHENGZHONGGUOETF"}, "513690": {"suoxie": "BSHSGGXETF", "name": "\u535a\u65f6\u6052\u751f\u9ad8\u80a1\u606fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHENGSHENGGAOGUXIETF"}, "513700": {"suoxie": "PHZZGGTYYWSZHJYETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u6e2f\u80a1\u901a\u533b\u836f\u536b\u751f\u7efc\u5408\u4ea4\u6613ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGANGGUTONGYIYAOWEISHENGZONGHEJIAOYIETF"}, "513770": {"suoxie": "HBZZGGTHLWETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u6e2f\u80a1\u901a\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGGANGGUTONGHULIANWANGETF"}, "513800": {"suoxie": "NFDFTOPIXETF", "name": "\u5357\u65b9\u9876\u5cf0TOPIX-ETF", "style": "QDII", "pinyin": "NANFANGDINGFENGTOPIXETF"}, "513860": {"suoxie": "HFTZZGGTKJETF", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1\u6e2f\u80a1\u901a\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENGGANGGUTONGKEJIETF"}, "513880": {"suoxie": "HARJ225ETF", "name": "\u534e\u5b89\u65e5\u7ecf225ETF", "style": "QDII", "pinyin": "HUAANRIJING225ETF"}, "513890": {"suoxie": "STMGHSKJETFQDII", "name": "\u4e0a\u6295\u6469\u6839\u6052\u751f\u79d1\u6280ETF(QDII)", "style": "QDII", "pinyin": "SHANGTOUMOGENHENGSHENGKEJIETFQDII"}, "513900": {"suoxie": "HACESGGTETF", "name": "\u534e\u5b89CES\u6e2f\u80a1\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANCESGANGGUTONGETF"}, "513960": {"suoxie": "BSZZGGTXFZTETF", "name": "\u535a\u65f6\u4e2d\u8bc1\u6e2f\u80a1\u901a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGGANGGUTONGXIAOFEIZHUTIETF"}, "513980": {"suoxie": "JSCCZZGGTKJETF", "name": "\u666f\u987a\u957f\u57ce\u4e2d\u8bc1\u6e2f\u80a1\u901a\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGZHONGZHENGGANGGUTONGKEJIETF"}, "513990": {"suoxie": "ZSSZGGTETF", "name": "\u62db\u5546\u4e0a\u8bc1\u6e2f\u80a1\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGSHANGZHENGGANGGUTONGETF"}, "515000": {"suoxie": "HBZZKJLTETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u79d1\u6280\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGKEJILONGTOUETF"}, "515010": {"suoxie": "HXZZQZZQGSETF", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "515020": {"suoxie": "HXZZYHETF", "name": "\u534e\u590f\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYINHANGETF"}, "515030": {"suoxie": "HXZZXNYQCETF", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINNENGYUANQICHEETF"}, "515050": {"suoxie": "HXZZ5GTXZTETF", "name": "\u534e\u590f\u4e2d\u8bc15G\u901a\u4fe1\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENG5GTONGXINZHUTIETF"}, "515060": {"suoxie": "HXZZQZFDCETF", "name": "\u534e\u590f\u4e2d\u8bc1\u5168\u6307\u623f\u5730\u4ea7ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGQUANZHIFANGDICHANETF"}, "515070": {"suoxie": "HXZZRGZNZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGRENGONGZHINENGZHUTIETF"}, "515080": {"suoxie": "ZSZZHLETF", "name": "\u62db\u5546\u4e2d\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGHONGLIETF"}, "515090": {"suoxie": "BSKCXFZ100ETF", "name": "\u535a\u65f6\u53ef\u6301\u7eed\u53d1\u5c55100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIKECHIXUFAZHAN100ETF"}, "515100": {"suoxie": "JSCCHLDBD100ETF", "name": "\u666f\u987a\u957f\u57ce\u7ea2\u5229\u4f4e\u6ce2\u52a8100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JINGSHUNCHANGCHENGHONGLIDIBODONG100ETF"}, "515110": {"suoxie": "YFDZZGQYDYLETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGGUOQIYIDAIYILUETF"}, "515120": {"suoxie": "GFZZCXYCYETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGCHUANGXINYAOCHANYEETF"}, "515130": {"suoxie": "BSHS300ETF", "name": "\u535a\u65f6\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHUSHEN300ETF"}, "515150": {"suoxie": "FGZZGQYDYLETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGGUOQIYIDAIYILUETF"}, "515160": {"suoxie": "ZSMSCIZGAGGJTETF", "name": "\u62db\u5546MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGMSCIZHONGGUOAGUGUOJITONGETF"}, "515170": {"suoxie": "HXZZXFSPYLZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXIFENSHIPINYINLIAOZHUTIETF"}, "515180": {"suoxie": "YFDZZHLETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGHONGLIETF"}, "515190": {"suoxie": "ZYZQZZ500ETF", "name": "\u4e2d\u94f6\u8bc1\u5238\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHENGQUANZHONGZHENG500ETF"}, "515200": {"suoxie": "SWLXZZYFCX100ETF", "name": "\u7533\u4e07\u83f1\u4fe1\u4e2d\u8bc1\u7814\u53d1\u521b\u65b0100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHENWANLINGXINZHONGZHENGYANFACHUANGXIN100ETF"}, "515210": {"suoxie": "GTZZGTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u94a2\u94c1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGGANGTIEETF"}, "515220": {"suoxie": "GTZZMTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7164\u70adETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGMEITANETF"}, "515230": {"suoxie": "GTZZQZRJETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u8f6f\u4ef6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHIRUANJIANETF"}, "515250": {"suoxie": "FGZZZNQCZTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGZHINENGQICHEZHUTIETF"}, "515260": {"suoxie": "HBZZDZ50ETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7535\u5b5050ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGDIANZI50ETF"}, "515280": {"suoxie": "FGZZYHETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYINHANGETF"}, "515290": {"suoxie": "THZZYHETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYINHANGETF"}, "515293": {"suoxie": "THZZYHETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGYINHANGETF"}, "515300": {"suoxie": "JSHS300HLDBDETF", "name": "\u5609\u5b9e\u6caa\u6df1300\u7ea2\u5229\u4f4e\u6ce2\u52a8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIHUSHEN300HONGLIDIBODONGETF"}, "515310": {"suoxie": "TFHS300ETF", "name": "\u6dfb\u5bcc\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUHUSHEN300ETF"}, "515320": {"suoxie": "HAZZDZ50ETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u7535\u5b5050ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGDIANZI50ETF"}, "515330": {"suoxie": "THHS300ETF", "name": "\u5929\u5f18\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHUSHEN300ETF"}, "515350": {"suoxie": "MSJYHS300ETF", "name": "\u6c11\u751f\u52a0\u94f6\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINHUSHEN300ETF"}, "515360": {"suoxie": "FZFBHS300ETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGHUSHEN300ETF"}, "515380": {"suoxie": "TKHS300ETF", "name": "\u6cf0\u5eb7\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGHUSHEN300ETF"}, "515390": {"suoxie": "HAHS300ETF", "name": "\u534e\u5b89\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANHUSHEN300ETF"}, "515400": {"suoxie": "FGZZDSJCYETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5927\u6570\u636e\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGDASHUJUCHANYEETF"}, "515450": {"suoxie": "NFZGAGDPHLDB50ETF", "name": "\u5357\u65b9\u4e2d\u56fdA\u80a1\u5927\u76d8\u7ea2\u5229\u4f4e\u6ce250ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGGUOAGUDAPANHONGLIDIBO50ETF"}, "515500": {"suoxie": "HFTZZCSJLXETF", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1\u957f\u4e09\u89d2\u9886\u5148ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENGCHANGSANJIAOLINGXIANETF"}, "515510": {"suoxie": "JSZZ500CZGZETF", "name": "\u5609\u5b9e\u4e2d\u8bc1500\u6210\u957f\u4f30\u503cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENG500CHENGZHANGGUZHIETF"}, "515520": {"suoxie": "DCMSCIZGAGZYJZ100ETF", "name": "\u5927\u6210MSCI\u4e2d\u56fdA\u80a1\u8d28\u4f18\u4ef7\u503c100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGMSCIZHONGGUOAGUZHIYOUJIAZHI100ETF"}, "515530": {"suoxie": "TKZZ500ETF", "name": "\u6cf0\u5eb7\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGZHONGZHENG500ETF"}, "515550": {"suoxie": "ZRZZ500ETF", "name": "\u4e2d\u878d\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGRONGZHONGZHENG500ETF"}, "515560": {"suoxie": "JXZZQZZQGSETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "515570": {"suoxie": "SXZZHLETF", "name": "\u5c71\u897f\u4e2d\u8bc1\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANXIZHONGZHENGHONGLIETF"}, "515580": {"suoxie": "HTBRZZKJ100ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u79d1\u6280100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGKEJI100ETF"}, "515590": {"suoxie": "QHKYZZ500DQETF", "name": "\u524d\u6d77\u5f00\u6e90\u4e2d\u8bc1500\u7b49\u6743ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "QIANHAIKAIYUANZHONGZHENG500DENGQUANETF"}, "515600": {"suoxie": "GFYQCXETF", "name": "\u5e7f\u53d1\u592e\u4f01\u521b\u65b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAYANGQICHUANGXINETF"}, "515610": {"suoxie": "ZYZZ800ETF", "name": "\u4e2d\u94f6\u4e2d\u8bc1800ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG800ETF"}, "515630": {"suoxie": "PHZZ800ZBETF", "name": "\u9e4f\u534e\u4e2d\u8bc1800\u8bc1\u4fddETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENG800ZHENGBAOETF"}, "515650": {"suoxie": "FGZZXF50ETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6d88\u8d3950ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIAOFEI50ETF"}, "515660": {"suoxie": "GLAHS300ETF", "name": "\u56fd\u8054\u5b89\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANHUSHEN300ETF"}, "515670": {"suoxie": "ZYZZ100ETF", "name": "\u4e2d\u94f6\u4e2d\u8bc1100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYINZHONGZHENG100ETF"}, "515680": {"suoxie": "JSZZYQCXQDETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGYANGQICHUANGXINQUDONGETF"}, "515690": {"suoxie": "PHZZGGXLTETF", "name": "\u9e4f\u534e\u4e2d\u8bc1\u9ad8\u80a1\u606f\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGHUAZHONGZHENGGAOGUXILONGTOUETF"}, "515700": {"suoxie": "PAZZXNYQCCYETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXINNENGYUANQICHECHANYEETF"}, "515710": {"suoxie": "HBZZXFSPYLZTETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u7ec6\u5206\u98df\u54c1\u996e\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIFENSHIPINYINLIAOZHUTIETF"}, "515750": {"suoxie": "FGZZKJ50CLETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u628050\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKEJI50CELUEETF"}, "515760": {"suoxie": "HXZZZJGZCXFZETF", "name": "\u534e\u590f\u4e2d\u8bc1\u6d59\u6c5f\u56fd\u8d44\u521b\u65b0\u53d1\u5c55ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHEJIANGGUOZICHUANGXINFAZHANETF"}, "515770": {"suoxie": "STMGMSCIZGAGETF", "name": "\u4e0a\u6295\u6469\u6839MSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUMOGENMSCIZHONGGUOAGUETF"}, "515780": {"suoxie": "PYASMSCIZGAGETF", "name": "\u6d66\u94f6\u5b89\u76dbMSCI\u4e2d\u56fdA\u80a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGMSCIZHONGGUOAGUETF"}, "515790": {"suoxie": "HTBRZZGFCYETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGGUANGFUCHANYEETF"}, "515800": {"suoxie": "TFZZ800ETF", "name": "\u6dfb\u5bcc\u4e2d\u8bc1800ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUZHONGZHENG800ETF"}, "515810": {"suoxie": "YFDZZ800ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1800ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENG800ETF"}, "515820": {"suoxie": "FGZZ800ETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1800ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENG800ETF"}, "515830": {"suoxie": "GYRXZZ800ETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1800ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENG800ETF"}, "515850": {"suoxie": "FGZZQZZQGSETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "515860": {"suoxie": "JSXXKJ100ETF", "name": "\u5609\u5b9e\u65b0\u5174\u79d1\u6280100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIXINXINGKEJI100ETF"}, "515870": {"suoxie": "JSXJZZ100ETF", "name": "\u5609\u5b9e\u5148\u8fdb\u5236\u9020100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIXIANJINZHIZAO100ETF"}, "515880": {"suoxie": "GTZZQZTXSBETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5168\u6307\u901a\u4fe1\u8bbe\u5907ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGQUANZHITONGXINSHEBEIETF"}, "515890": {"suoxie": "BSHLETF", "name": "\u535a\u65f6\u7ea2\u5229ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIHONGLIETF"}, "515900": {"suoxie": "BSYQCXQDETF", "name": "\u535a\u65f6\u592e\u4f01\u521b\u65b0\u9a71\u52a8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIYANGQICHUANGXINQUDONGETF"}, "515910": {"suoxie": "ZJMSCIZGAGGJZLETF", "name": "\u4e2d\u91d1MSCI\u4e2d\u56fdA\u80a1\u56fd\u9645\u8d28\u91cfETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGJINMSCIZHONGGUOAGUGUOJIZHILIANGETF"}, "515920": {"suoxie": "BSZNXFETF", "name": "\u535a\u65f6\u667a\u80fd\u6d88\u8d39ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHINENGXIAOFEIETF"}, "515930": {"suoxie": "YYHS300ETF", "name": "\u6c38\u8d62\u6caa\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YONGYINGHUSHEN300ETF"}, "515950": {"suoxie": "FGZZYY50ETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u533b\u836f50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGYIYAO50ETF"}, "515960": {"suoxie": "JSYYJK100ETF", "name": "\u5609\u5b9e\u533b\u836f\u5065\u5eb7100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIYIYAOJIANKANG100ETF"}, "515980": {"suoxie": "HFZZRGZNCYETF", "name": "\u534e\u5bcc\u4e2d\u8bc1\u4eba\u5de5\u667a\u80fd\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGRENGONGZHINENGCHANYEETF"}, "515990": {"suoxie": "TFZZGQYDYLETF", "name": "\u6dfb\u5bcc\u4e2d\u8bc1\u56fd\u4f01\u4e00\u5e26\u4e00\u8defETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANFUZHONGZHENGGUOQIYIDAIYILUETF"}, "516000": {"suoxie": "HXZZDSJCYETF", "name": "\u534e\u590f\u4e2d\u8bc1\u5927\u6570\u636e\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGDASHUJUCHANYEETF"}, "516010": {"suoxie": "GTZZDMYXETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGDONGMANYOUXIETF"}, "516020": {"suoxie": "HBHGETF", "name": "\u534e\u5b9d\u5316\u5de5ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOHUAGONGETF"}, "516050": {"suoxie": "GYZZKJLTETF", "name": "\u5de5\u94f6\u4e2d\u8bc1\u79d1\u6280\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINZHONGZHENGKEJILONGTOUETF"}, "516060": {"suoxie": "GYRXZZCXYCYETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENGCHUANGXINYAOCHANYEETF"}, "516070": {"suoxie": "YFDZZNDDTJJETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGNEIDIDITANJINGJIETF"}, "516080": {"suoxie": "YFDZZCXYCYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGCHUANGXINYAOCHANYEETF"}, "516090": {"suoxie": "YFDZZXNYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXINNENGYUANETF"}, "516100": {"suoxie": "HXZZJRKJZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u91d1\u878d\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGJINRONGKEJIZHUTIETF"}, "516110": {"suoxie": "GTZZ800QCYLBJETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1800\u6c7d\u8f66\u4e0e\u96f6\u90e8\u4ef6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG800QICHEYULINGBUJIANETF"}, "516120": {"suoxie": "FGZZXFHGCYZTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIFENHUAGONGCHANYEZHUTIETF"}, "516130": {"suoxie": "HBZZXFLTETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u6d88\u8d39\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGXIAOFEILONGTOUETF"}, "516150": {"suoxie": "JSZZXTCYETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXITUCHANYEETF"}, "516160": {"suoxie": "NFZZXNYETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGXINNENGYUANETF"}, "516163": {"suoxie": "NFZZXNYETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGXINNENGYUANETF"}, "516180": {"suoxie": "PAZZGFCYETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGGUANGFUCHANYEETF"}, "516190": {"suoxie": "HXZZWYCMETF", "name": "\u534e\u590f\u4e2d\u8bc1\u6587\u5a31\u4f20\u5a92ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGWENYUCHUANMEIETF"}, "516200": {"suoxie": "HAZZQZZQGSETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u5168\u6307\u8bc1\u5238\u516c\u53f8ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGQUANZHIZHENGQUANGONGSIETF"}, "516210": {"suoxie": "HAZZYHETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGYINHANGETF"}, "516220": {"suoxie": "GTZZXFHGCYZTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIFENHUAGONGCHANYEZHUTIETF"}, "516260": {"suoxie": "HXZZWLWZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGWULIANWANGZHUTIETF"}, "516270": {"suoxie": "HAZZNDXNYZTETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u5185\u5730\u65b0\u80fd\u6e90\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGNEIDIXINNENGYUANZHUTIETF"}, "516290": {"suoxie": "HTFZZGFCYETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGGUANGFUCHANYEETF"}, "516300": {"suoxie": "HTBRZZ1000ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc11000ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENG1000ETF"}, "516310": {"suoxie": "YFDZZYHETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u94f6\u884cETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGYINHANGETF"}, "516320": {"suoxie": "HXZZZBCYETF", "name": "\u534e\u590f\u4e2d\u8bc1\u88c5\u5907\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHUANGBEICHANYEETF"}, "516330": {"suoxie": "HTBRZZWLWZTETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGWULIANWANGZHUTIETF"}, "516350": {"suoxie": "YFDZZXPCYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXINPIANCHANYEETF"}, "516360": {"suoxie": "HBXCLETF", "name": "\u534e\u5b9d\u65b0\u6750\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOXINCAILIAOETF"}, "516380": {"suoxie": "HBZNDDQCETF", "name": "\u534e\u5b9d\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHINENGDIANDONGQICHEETF"}, "516390": {"suoxie": "HTFZZXNYQCCYETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINNENGYUANQICHECHANYEETF"}, "516480": {"suoxie": "GLAZZXCLZTETF", "name": "\u56fd\u8054\u5b89\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANZHONGZHENGXINCAILIAOZHUTIETF"}, "516500": {"suoxie": "HXZZSWKJZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGSHENGWUKEJIZHUTIETF"}, "516510": {"suoxie": "YFDZZYJSETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u4e91\u8ba1\u7b97ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGYUNJISUANETF"}, "516520": {"suoxie": "HTBRZZZNQCZTETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u667a\u80fd\u6c7d\u8f66\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGZHINENGQICHEZHUTIETF"}, "516530": {"suoxie": "YHZZXDWLETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u73b0\u4ee3\u7269\u6d41ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGXIANDAIWULIUETF"}, "516550": {"suoxie": "JSZZDNYETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u5927\u519c\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGDANONGYEETF"}, "516560": {"suoxie": "HBZZYLCYETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u517b\u8001\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGYANGLAOCHANYEETF"}, "516570": {"suoxie": "YFDZZSHCYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u77f3\u5316\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGSHIHUACHANYEETF"}, "516580": {"suoxie": "BSZZXNYETF", "name": "\u535a\u65f6\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGXINNENGYUANETF"}, "516590": {"suoxie": "YFDZZZNDDQCETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGZHINENGDIANDONGQICHEETF"}, "516600": {"suoxie": "GYRXZZXFFWLXETF", "name": "\u5de5\u94f6\u745e\u4fe1\u4e2d\u8bc1\u6d88\u8d39\u670d\u52a1\u9886\u5148ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINRUIXINZHONGZHENGXIAOFEIFUWULINGXIANETF"}, "516610": {"suoxie": "DCZZQZYLBJSBYFWETF", "name": "\u5927\u6210\u4e2d\u8bc1\u5168\u6307\u533b\u7597\u4fdd\u5065\u8bbe\u5907\u4e0e\u670d\u52a1ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGZHONGZHENGQUANZHIYILIAOBAOJIANSHEBEIYUFUWUETF"}, "516620": {"suoxie": "GTZZYSZTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u5f71\u89c6\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGYINGSHIZHUTIETF"}, "516630": {"suoxie": "HXZZYJSETF", "name": "\u534e\u590f\u4e2d\u8bc1\u4e91\u8ba1\u7b97ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGYUNJISUANETF"}, "516640": {"suoxie": "FGZZXPCYETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINPIANCHANYEETF"}, "516650": {"suoxie": "HXXFYSJSCYZTETF", "name": "\u534e\u590f\u7ec6\u5206\u6709\u8272\u91d1\u5c5e\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAXIFENYOUSEJINSHUCHANYEZHUTIETF"}, "516660": {"suoxie": "HAZZXNYQCETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u65b0\u80fd\u6e90\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGXINNENGYUANQICHEETF"}, "516670": {"suoxie": "ZSZZXMYZETF", "name": "\u62db\u5546\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGXUMUYANGZHIETF"}, "516680": {"suoxie": "JXZZXFYSJSCYETF", "name": "\u5efa\u4fe1\u4e2d\u8bc1\u7ec6\u5206\u6709\u8272\u91d1\u5c5e\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINZHONGZHENGXIFENYOUSEJINSHUCHANYEETF"}, "516690": {"suoxie": "YHZZXFHGCYZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u7ec6\u5206\u5316\u5de5\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGXIFENHUAGONGCHANYEZHUTIETF"}, "516700": {"suoxie": "HBDSJETF", "name": "\u534e\u5b9d\u5927\u6570\u636eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAODASHUJUETF"}, "516710": {"suoxie": "HXZZXCLZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINCAILIAOZHUTIETF"}, "516720": {"suoxie": "PYASZZESG120CLETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1ESG120\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGESG120CELUEETF"}, "516730": {"suoxie": "PYASZZZQGS30ETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u8bc1\u5238\u516c\u53f830ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGZHENGQUANGONGSI30ETF"}, "516750": {"suoxie": "FGZZQZJZCLETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5168\u6307\u5efa\u7b51\u6750\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGQUANZHIJIANZHUCAILIAOETF"}, "516760": {"suoxie": "PAZZXMYZETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u755c\u7267\u517b\u6b96ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXUMUYANGZHIETF"}, "516770": {"suoxie": "HTBRZZDMYXETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u52a8\u6f2b\u6e38\u620fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGDONGMANYOUXIETF"}, "516780": {"suoxie": "HTBRZZXTCYETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u7a00\u571f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGXITUCHANYEETF"}, "516790": {"suoxie": "HTBRZZYLBJETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u533b\u7597\u4fdd\u5065ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGYILIAOBAOJIANETF"}, "516800": {"suoxie": "HBZZZNZZETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u667a\u80fd\u5236\u9020ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGZHINENGZHIZAOETF"}, "516810": {"suoxie": "HXZZNYZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u519c\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGNONGYEZHUTIETF"}, "516820": {"suoxie": "PAYYJYLQXCXETF", "name": "\u5e73\u5b89\u533b\u836f\u53ca\u533b\u7597\u5668\u68b0\u521b\u65b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANYIYAOJIYILIAOQIXIECHUANGXINETF"}, "516830": {"suoxie": "FGHS300ESGJZETF", "name": "\u5bcc\u56fd\u6caa\u6df1300ESG\u57fa\u51c6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOHUSHEN300ESGJIZHUNETF"}, "516850": {"suoxie": "HXZZXNYETF", "name": "\u534e\u590f\u4e2d\u8bc1\u65b0\u80fd\u6e90ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGXINNENGYUANETF"}, "516860": {"suoxie": "BSJRKJETF", "name": "\u535a\u65f6\u91d1\u878d\u79d1\u6280ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIJINRONGKEJIETF"}, "516880": {"suoxie": "YHZZGFCYETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u5149\u4f0f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGGUANGFUCHANYEETF"}, "516890": {"suoxie": "PAZZXCLZTETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u65b0\u6750\u6599\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXINCAILIAOZHUTIETF"}, "516900": {"suoxie": "HAZZSWSPYLETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u7533\u4e07\u98df\u54c1\u996e\u6599ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGSHENWANSHIPINYINLIAOETF"}, "516910": {"suoxie": "FGZZXDWLETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u73b0\u4ee3\u7269\u6d41ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIANDAIWULIUETF"}, "516920": {"suoxie": "HTFZZXPCYETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u82af\u7247\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGXINPIANCHANYEETF"}, "516930": {"suoxie": "MSJYZZSWKJZTETF", "name": "\u6c11\u751f\u52a0\u94f6\u4e2d\u8bc1\u751f\u7269\u79d1\u6280\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGJIAYINZHONGZHENGSHENGWUKEJIZHUTIETF"}, "516950": {"suoxie": "YHZZJJETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u57fa\u5efaETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGJIJIANETF"}, "516960": {"suoxie": "GTXFJXSBCYZTETF", "name": "\u56fd\u6cf0\u7ec6\u5206\u673a\u68b0\u8bbe\u5907\u4ea7\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIXIFENJIXIESHEBEICHANYEZHUTIETF"}, "516970": {"suoxie": "GFZZJJGCETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u57fa\u5efa\u5de5\u7a0bETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGJIJIANGONGCHENGETF"}, "516980": {"suoxie": "HFZZZQGSXFCLETF", "name": "\u534e\u5bcc\u4e2d\u8bc1\u8bc1\u5238\u516c\u53f8\u5148\u950b\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGZHENGQUANGONGSIXIANFENGCELUEETF"}, "517000": {"suoxie": "YHZZHGS500ETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGHUGANGSHEN500ETF"}, "517010": {"suoxie": "YFDZZHGS500ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGHUGANGSHEN500ETF"}, "517030": {"suoxie": "YFDZZHGS300ETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6caa\u6e2f\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGHUGANGSHEN300ETF"}, "517050": {"suoxie": "HTBRZZHGSHLWETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGHUGANGSHENHULIANWANGETF"}, "517060": {"suoxie": "HBZZHGS500ETF", "name": "\u534e\u5b9d\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOZHONGZHENGHUGANGSHEN500ETF"}, "517080": {"suoxie": "HTFZZHGS500ETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHEN500ETF"}, "517090": {"suoxie": "GTFSZGGQKFGYETF", "name": "\u56fd\u6cf0\u5bcc\u65f6\u4e2d\u56fd\u56fd\u4f01\u5f00\u653e\u5171\u8d62ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIFUSHIZHONGGUOGUOQIKAIFANGGONGYINGETF"}, "517100": {"suoxie": "FGZZHGS500ETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGHUGANGSHEN500ETF"}, "517110": {"suoxie": "GTZZHGSCXYCYETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUGANGSHENCHUANGXINYAOCHANYEETF"}, "517120": {"suoxie": "HTBRZZCXYCYETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u521b\u65b0\u836f\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGCHUANGXINYAOCHANYEETF"}, "517160": {"suoxie": "NFZZCJBHZTETF", "name": "\u5357\u65b9\u4e2d\u8bc1\u957f\u6c5f\u4fdd\u62a4\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENGCHANGJIANGBAOHUZHUTIETF"}, "517170": {"suoxie": "HXZZHGS500ETF", "name": "\u534e\u590f\u4e2d\u8bc1\u6caa\u6e2f\u6df1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGHUGANGSHEN500ETF"}, "517180": {"suoxie": "NFFSZGGQKFGYETF", "name": "\u5357\u65b9\u5bcc\u65f6\u4e2d\u56fd\u56fd\u4f01\u5f00\u653e\u5171\u8d62ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGFUSHIZHONGGUOGUOQIKAIFANGGONGYINGETF"}, "517200": {"suoxie": "JSZZHGSHLWETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e92\u8054\u7f51ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGHUGANGSHENHULIANWANGETF"}, "517270": {"suoxie": "PYASZZHGSKJLTETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGHUGANGSHENKEJILONGTOUETF"}, "517280": {"suoxie": "THZZHGSXSXFZTETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u7ebf\u4e0a\u6d88\u8d39\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHUGANGSHENXIANSHANGXIAOFEIZHUTIETF"}, "517300": {"suoxie": "GSABZZHGS300ETF", "name": "\u56fd\u5bff\u5b89\u4fdd\u4e2d\u8bc1\u6caa\u6e2f\u6df1300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOSHOUANBAOZHONGZHENGHUGANGSHEN300ETF"}, "517330": {"suoxie": "YFDZZCJBHZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u957f\u6c5f\u4fdd\u62a4\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGCHANGJIANGBAOHUZHUTIETF"}, "517350": {"suoxie": "GFZZHGSKJLTETF", "name": "\u5e7f\u53d1\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFAZHONGZHENGHUGANGSHENKEJILONGTOUETF"}, "517360": {"suoxie": "HAZZHGSKJ100ETF", "name": "\u534e\u5b89\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANZHONGZHENGHUGANGSHENKEJI100ETF"}, "517380": {"suoxie": "THHSHSGCXYJX50ETF", "name": "\u5929\u5f18\u6052\u751f\u6caa\u6df1\u6e2f\u521b\u65b0\u836f\u7cbe\u900950ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGHENGSHENGHUSHENGANGCHUANGXINYAOJINGXUAN50ETF"}, "517390": {"suoxie": "THZZHGSYJSCYETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u4e91\u8ba1\u7b97\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHUGANGSHENYUNJISUANCHANYEETF"}, "517500": {"suoxie": "GTZZHGSDMYXETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u52a8\u6f2b\u6e38\u620fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGHUGANGSHENDONGMANYOUXIETF"}, "517550": {"suoxie": "ZSZZHGSXFLTETF", "name": "\u62db\u5546\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u6d88\u8d39\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGHUGANGSHENXIAOFEILONGTOUETF"}, "517660": {"suoxie": "THZZHGSWLWZTETF", "name": "\u5929\u5f18\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u7269\u8054\u7f51\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TIANHONGZHONGZHENGHUGANGSHENWULIANWANGZHUTIETF"}, "517760": {"suoxie": "PYASZZHGSXFLTETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u6d88\u8d39\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGHUGANGSHENXIAOFEILONGTOUETF"}, "517770": {"suoxie": "PYASZZHGSYXJWHCMETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u6e38\u620f\u53ca\u6587\u5316\u4f20\u5a92ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGHUGANGSHENYOUXIJIWENHUACHUANMEIETF"}, "517780": {"suoxie": "PYASZHJYFWHSG300ETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u534e\u4ea4\u6613\u670d\u52a1\u6caa\u6df1\u6e2f300ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGHUAJIAOYIFUWUHUSHENGANG300ETF"}, "517800": {"suoxie": "FZFBHGSRGZN50ETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u6caa\u6e2f\u6df1\u4eba\u5de5\u667a\u80fd50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGHUGANGSHENRENGONGZHINENG50ETF"}, "517850": {"suoxie": "HTFZZHGSZJZZCX50ETF", "name": "\u6c47\u6dfb\u5bcc\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u5f20\u6c5f\u81ea\u4e3b\u521b\u65b050ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUZHONGZHENGHUGANGSHENZHANGJIANGZIZHUCHUANGXIN50ETF"}, "517880": {"suoxie": "HTBRZZPPXF50ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u54c1\u724c\u6d88\u8d3950ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGPINPAIXIAOFEI50ETF"}, "517900": {"suoxie": "ZSZZYHAHJGYXETF", "name": "\u62db\u5546\u4e2d\u8bc1\u94f6\u884cAH\u4ef7\u683c\u4f18\u9009ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGYINHANGAHJIAGEYOUXUANETF"}, "517960": {"suoxie": "STMGZZHGSKJ100ETF", "name": "\u4e0a\u6295\u6469\u6839\u4e2d\u8bc1\u6caa\u6e2f\u6df1\u79d1\u6280100ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "SHANGTOUMOGENZHONGZHENGHUGANGSHENKEJI100ETF"}, "517980": {"suoxie": "HFZZYHAHJGYXETF", "name": "\u534e\u5bcc\u4e2d\u8bc1\u94f6\u884cAH\u4ef7\u683c\u4f18\u9009ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGYINHANGAHJIAGEYOUXUANETF"}, "517990": {"suoxie": "ZSZZHGS500YYWSETF", "name": "\u62db\u5546\u4e2d\u8bc1\u6caa\u6e2f\u6df1500\u533b\u836f\u536b\u751fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGHUGANGSHEN500YIYAOWEISHENGETF"}, "518600": {"suoxie": "GFSHJETF", "name": "\u5e7f\u53d1\u4e0a\u6d77\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUANGFASHANGHAIJINETF"}, "518660": {"suoxie": "GYRXHJETF", "name": "\u5de5\u94f6\u745e\u4fe1\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GONGYINRUIXINHUANGJINETF"}, "518680": {"suoxie": "FGSHJETF", "name": "\u5bcc\u56fd\u4e0a\u6d77\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "FUGUOSHANGHAIJINETF"}, "518800": {"suoxie": "GTHJETF", "name": "\u56fd\u6cf0\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "GUOTAIHUANGJINETF"}, "518850": {"suoxie": "HXHJETF", "name": "\u534e\u590f\u9ec4\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAXIAHUANGJINETF"}, "518860": {"suoxie": "JXSHJETF", "name": "\u5efa\u4fe1\u4e0a\u6d77\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "JIANXINSHANGHAIJINETF"}, "518880": {"suoxie": "HAHJYETF", "name": "\u534e\u5b89\u9ec4\u91d1\u6613ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "HUAANHUANGJINYIETF"}, "518890": {"suoxie": "ZYSHJETF", "name": "\u4e2d\u94f6\u4e0a\u6d77\u91d1ETF", "style": "\u5546\u54c1\uff08\u4e0d\u542bQDII\uff09", "pinyin": "ZHONGYINSHANGHAIJINETF"}, "519001": {"suoxie": "YHJZYXHH", "name": "\u94f6\u534e\u4ef7\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHUAJIAZHIYOUXUANHUNHE"}, "519002": {"suoxie": "HAAXXFHHA", "name": "\u534e\u5b89\u5b89\u4fe1\u6d88\u8d39\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAANANXINXIAOFEIHUNHEA"}, "519003": {"suoxie": "HFTSYZZHH", "name": "\u6d77\u5bcc\u901a\u6536\u76ca\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGSHOUYIZENGZHANGHUNHE"}, "519005": {"suoxie": "HFTGPHH", "name": "\u6d77\u5bcc\u901a\u80a1\u7968\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGGUPIAOHUNHE"}, "519007": {"suoxie": "HFTQHHBHH", "name": "\u6d77\u5bcc\u901a\u5f3a\u5316\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGQIANGHUAHUIBAOHUNHE"}, "519008": {"suoxie": "HTFYSJXHH", "name": "\u6c47\u6dfb\u5bcc\u4f18\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUSHIJINGXUANHUNHE"}, "519009": {"suoxie": "HTFYSJXHH", "name": "\u6c47\u6dfb\u5bcc\u4f18\u52bf\u7cbe\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUYOUSHIJINGXUANHUNHE"}, "519010": {"suoxie": "HTFJHZZHH", "name": "\u6c47\u6dfb\u5bcc\u5747\u8861\u589e\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUNHENGZENGZHANGHUNHE"}, "519011": {"suoxie": "HFTJXHH", "name": "\u6d77\u5bcc\u901a\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGJINGXUANHUNHE"}, "519013": {"suoxie": "HFTFGYSHH", "name": "\u6d77\u5bcc\u901a\u98ce\u683c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGFENGGEYOUSHIHUNHE"}, "519015": {"suoxie": "HFTJXEHHH", "name": "\u6d77\u5bcc\u901a\u7cbe\u9009\u8d30\u53f7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "HAIFUTONGJINGXUANERHAOHUNHE"}, "519017": {"suoxie": "DCJJCZHH", "name": "\u5927\u6210\u79ef\u6781\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJIJICHENGZHANGHUNHE"}, "519018": {"suoxie": "HTFJHZZHH", "name": "\u6c47\u6dfb\u5bcc\u5747\u8861\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJUNHENGZENGZHANGHUNHE"}, "519019": {"suoxie": "DCJYLXHH", "name": "\u5927\u6210\u666f\u9633\u9886\u5148\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DACHENGJINGYANGLINGXIANHUNHE"}, "519020": {"suoxie": "GTJTLHPZHHA", "name": "\u56fd\u6cf0\u91d1\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJINTAILINGHUOPEIZHIHUNHEA"}, "519021": {"suoxie": "GTJDJZHH", "name": "\u56fd\u6cf0\u91d1\u9f0e\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJINDINGJIAZHIHUNHE"}, "519022": {"suoxie": "GTJTLHPZHHC", "name": "\u56fd\u6cf0\u91d1\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOTAIJINTAILINGHUOPEIZHIHUNHEC"}, "519023": {"suoxie": "HFTWJTLZQC", "name": "\u6d77\u5bcc\u901a\u7a33\u5065\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGWENJIANTIANLIZHAIQUANC"}, "519024": {"suoxie": "HFTWJTLZQA", "name": "\u6d77\u5bcc\u901a\u7a33\u5065\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGWENJIANTIANLIZHAIQUANA"}, "519025": {"suoxie": "HFTLXCZHH", "name": "\u6d77\u5bcc\u901a\u9886\u5148\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGLINGXIANCHENGZHANGHUNHE"}, "519026": {"suoxie": "HFTZXPHH", "name": "\u6d77\u5bcc\u901a\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGZHONGXIAOPANHUNHE"}, "519027": {"suoxie": "HFTSZZQETFLJ", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u5468\u671fETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGSHANGZHENGZHOUQIETFLIANJIE"}, "519029": {"suoxie": "HXWZHH", "name": "\u534e\u590f\u7a33\u589e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAWENZENGHUNHE"}, "519030": {"suoxie": "HFTWGSYZQ", "name": "\u6d77\u5bcc\u901a\u7a33\u56fa\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGWENGUSHOUYIZHAIQUAN"}, "519032": {"suoxie": "HFTSZFZQETFLJ", "name": "\u6d77\u5bcc\u901a\u4e0a\u8bc1\u975e\u5468\u671fETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGSHANGZHENGFEIZHOUQIETFLIANJIE"}, "519033": {"suoxie": "HFTGCDXHH", "name": "\u6d77\u5bcc\u901a\u56fd\u7b56\u5bfc\u5411\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGGUOCEDAOXIANGHUNHE"}, "519034": {"suoxie": "HFTZZ500ZQA", "name": "\u6d77\u5bcc\u901a\u4e2d\u8bc1500\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HAIFUTONGZHONGZHENG500ZENGQIANGA"}, "519035": {"suoxie": "FGTBCXZTHH", "name": "\u5bcc\u56fd\u5929\u535a\u521b\u65b0\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANBOCHUANGXINZHUTIHUNHE"}, "519036": {"suoxie": "FGTBCXZTHH", "name": "\u5bcc\u56fd\u5929\u535a\u521b\u65b0\u4e3b\u9898\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOTIANBOCHUANGXINZHUTIHUNHE"}, "519039": {"suoxie": "CSTDZTHH", "name": "\u957f\u76db\u540c\u5fb7\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGSHENGTONGDEZHUTIHUNHE"}, "519050": {"suoxie": "HFTAYSYHHA", "name": "\u6d77\u5bcc\u901a\u5b89\u9890\u6536\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGANYISHOUYIHUNHEA"}, "519051": {"suoxie": "HFTYNDKZA", "name": "\u6d77\u5bcc\u901a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGYINIANDINGKAIZHAIA"}, "519056": {"suoxie": "HFTNXRDHH", "name": "\u6d77\u5bcc\u901a\u5185\u9700\u70ed\u70b9\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAIFUTONGNEIXUREDIANHUNHE"}, "519060": {"suoxie": "HFTCZZQC", "name": "\u6d77\u5bcc\u901a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGCHUNZHAIZHAIQUANC"}, "519061": {"suoxie": "HFTCZZQA", "name": "\u6d77\u5bcc\u901a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGCHUNZHAIZHAIQUANA"}, "519062": {"suoxie": "HFTAEFDCHHA", "name": "\u6d77\u5bcc\u901a\u963f\u5c14\u6cd5\u5bf9\u51b2\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "HAIFUTONGAERFADUICHONGHUNHEA"}, "519066": {"suoxie": "HTFLCWJHHA", "name": "\u6c47\u6dfb\u5bcc\u84dd\u7b79\u7a33\u5065\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFULANCHOUWENJIANHUNHEA"}, "519067": {"suoxie": "HTFLCWJHHA", "name": "\u6c47\u6dfb\u5bcc\u84dd\u7b79\u7a33\u5065\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUITIANFULANCHOUWENJIANHUNHEA"}, "519068": {"suoxie": "HTFCZJDHH", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u7126\u70b9\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGJIAODIANHUNHE"}, "519069": {"suoxie": "HTFJZJXHHA", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHIJINGXUANHUNHEA"}, "519070": {"suoxie": "HTFJZJXHHA", "name": "\u6c47\u6dfb\u5bcc\u4ef7\u503c\u7cbe\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUJIAZHIJINGXUANHUNHEA"}, "519071": {"suoxie": "HTFCZJDHH", "name": "\u6c47\u6dfb\u5bcc\u6210\u957f\u7126\u70b9\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUITIANFUCHENGZHANGJIAODIANHUNHE"}, "519078": {"suoxie": "HTFZQSYZQA", "name": "\u6c47\u6dfb\u5bcc\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUITIANFUZENGQIANGSHOUYIZHAIQUANA"}, "519087": {"suoxie": "XHYXFHHH", "name": "\u65b0\u534e\u4f18\u9009\u5206\u7ea2\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAYOUXUANFENHONGHUNHE"}, "519088": {"suoxie": "XHYXFHHH", "name": "\u65b0\u534e\u4f18\u9009\u5206\u7ea2\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAYOUXUANFENHONGHUNHE"}, "519089": {"suoxie": "XHYXCZHH", "name": "\u65b0\u534e\u4f18\u9009\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAYOUXUANCHENGZHANGHUNHE"}, "519091": {"suoxie": "XHFZYYSHH", "name": "\u65b0\u534e\u6cdb\u8d44\u6e90\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAFANZIYUANYOUSHIHUNHE"}, "519093": {"suoxie": "XHZSPZQYHH", "name": "\u65b0\u534e\u94bb\u77f3\u54c1\u8d28\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAZUANSHIPINZHIQIYEHUNHE"}, "519095": {"suoxie": "XHHYZQLHHH", "name": "\u65b0\u534e\u884c\u4e1a\u5468\u671f\u8f6e\u6362\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAHANGYEZHOUQILUNHUANHUNHE"}, "519097": {"suoxie": "XHZXSZYXHH", "name": "\u65b0\u534e\u4e2d\u5c0f\u5e02\u503c\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAZHONGXIAOSHIZHIYOUXUANHUNHE"}, "519099": {"suoxie": "XHLHZTHH", "name": "\u65b0\u534e\u7075\u6d3b\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUALINGHUOZHUTIHUNHE"}, "519100": {"suoxie": "CSZZ100ZS", "name": "\u957f\u76db\u4e2d\u8bc1100\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGSHENGZHONGZHENG100ZHISHU"}, "519110": {"suoxie": "PYASJZCZHHA", "name": "\u6d66\u94f6\u5b89\u76db\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGJIAZHICHENGZHANGHUNHEA"}, "519111": {"suoxie": "PYASYHSYZQA", "name": "\u6d66\u94f6\u5b89\u76db\u4f18\u5316\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGYOUHUASHOUYIZHAIQUANA"}, "519112": {"suoxie": "PYASYHSYZQC", "name": "\u6d66\u94f6\u5b89\u76db\u4f18\u5316\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PUYINANSHENGYOUHUASHOUYIZHAIQUANC"}, "519113": {"suoxie": "PYASJZSHHH", "name": "\u6d66\u94f6\u5b89\u76db\u7cbe\u81f4\u751f\u6d3b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGJINGZHISHENGHUOHUNHE"}, "519115": {"suoxie": "PYASHLJXHHA", "name": "\u6d66\u94f6\u5b89\u76db\u7ea2\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PUYINANSHENGHONGLIJINGXUANHUNHEA"}, "519116": {"suoxie": "PYASHS300ZSZQ", "name": "\u6d66\u94f6\u5b89\u76db\u6caa\u6df1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGHUSHEN300ZHISHUZENGQIANG"}, "519117": {"suoxie": "PYASJBM400ZS", "name": "\u6d66\u94f6\u5b89\u76db\u57fa\u672c\u9762400\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGJIBENMIAN400ZHISHU"}, "519118": {"suoxie": "PYASXFHBDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u5e78\u798f\u56de\u62a5\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGXINGFUHUIBAODINGKAIZHAIA"}, "519119": {"suoxie": "PYASXFHBDKZB", "name": "\u6d66\u94f6\u5b89\u76db\u5e78\u798f\u56de\u62a5\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGXINGFUHUIBAODINGKAIZHAIB"}, "519120": {"suoxie": "PYASXXCYHHA", "name": "\u6d66\u94f6\u5b89\u76db\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXINXINGCHANYEHUNHEA"}, "519121": {"suoxie": "PYAS6GYDQZQA", "name": "\u6d66\u94f6\u5b89\u76db6\u4e2a\u6708\u5b9a\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENG6GEYUEDINGQIZHAIQUANA"}, "519122": {"suoxie": "PYAS6GYDQZQC", "name": "\u6d66\u94f6\u5b89\u76db6\u4e2a\u6708\u5b9a\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENG6GEYUEDINGQIZHAIQUANC"}, "519125": {"suoxie": "PYASXFSJHHA", "name": "\u6d66\u94f6\u5b89\u76db\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXIAOFEISHENGJIHUNHEA"}, "519126": {"suoxie": "PYASXJJJGHHA", "name": "\u6d66\u94f6\u5b89\u76db\u65b0\u7ecf\u6d4e\u7ed3\u6784\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXINJINGJIJIEGOUHUNHEA"}, "519127": {"suoxie": "PYASSSJXHHA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u4e16\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGSHENGSHIJINGXUANHUNHEA"}, "519130": {"suoxie": "HFTXNXHHA", "name": "\u6d77\u5bcc\u901a\u65b0\u5185\u9700\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINNEIXUHUNHEA"}, "519133": {"suoxie": "HFTGGQDHH", "name": "\u6d77\u5bcc\u901a\u6539\u9769\u9a71\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGGAIGEQUDONGHUNHE"}, "519134": {"suoxie": "HFTFXHH", "name": "\u6d77\u5bcc\u901a\u5bcc\u7965\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HAIFUTONGFUXIANGHUNHE"}, "519136": {"suoxie": "HFTRFZQX", "name": "\u6d77\u5bcc\u901a\u745e\u4e30\u503a\u5238\u578b", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAIFUTONGRUIFENGZHAIQUANXING"}, "519137": {"suoxie": "HFTRFZQ", "name": "\u6d77\u5bcc\u901a\u745e\u798f\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIFUZHAIQUAN"}, "519138": {"suoxie": "HFTRXYNDKZQ", "name": "\u6d77\u5bcc\u901a\u745e\u7965\u4e00\u5e74\u5b9a\u5f00\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUIXIANGYINIANDINGKAIZHAIQUAN"}, "519139": {"suoxie": "HFTHGSLHPZHH", "name": "\u6d77\u5bcc\u901a\u6caa\u6e2f\u6df1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGHUGANGSHENLINGHUOPEIZHIHUNHE"}, "519150": {"suoxie": "XHYXXFHH", "name": "\u65b0\u534e\u4f18\u9009\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAYOUXUANXIAOFEIHUNHE"}, "519152": {"suoxie": "XHCZTLZQFQA", "name": "\u65b0\u534e\u7eaf\u503a\u6dfb\u5229\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUACHUNZHAITIANLIZHAIQUANFAQIA"}, "519153": {"suoxie": "XHCZTLZQFQC", "name": "\u65b0\u534e\u7eaf\u503a\u6dfb\u5229\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUACHUNZHAITIANLIZHAIQUANFAQIC"}, "519156": {"suoxie": "XHHYLHPZHHA", "name": "\u65b0\u534e\u884c\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAHANGYELINGHUOPEIZHIHUNHEA"}, "519157": {"suoxie": "XHHYLHPZHHC", "name": "\u65b0\u534e\u884c\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAHANGYELINGHUOPEIZHIHUNHEC"}, "519158": {"suoxie": "XHQSLHHH", "name": "\u65b0\u534e\u8d8b\u52bf\u9886\u822a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINHUAQUSHILINGHANGHUNHE"}, "519160": {"suoxie": "XHAXHJDQZQA", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u91d1\u5b9a\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIJINDINGQIZHAIQUANA"}, "519161": {"suoxie": "XHAXHJDQZQC", "name": "\u65b0\u534e\u5b89\u4eab\u60e0\u91d1\u5b9a\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINHUAANXIANGHUIJINDINGQIZHAIQUANC"}, "519162": {"suoxie": "XHZYZQA", "name": "\u65b0\u534e\u589e\u6021\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGYIZHAIQUANA"}, "519163": {"suoxie": "XHZYZQC", "name": "\u65b0\u534e\u589e\u6021\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINHUAZENGYIZHAIQUANC"}, "519165": {"suoxie": "XHXLLHPZHH", "name": "\u65b0\u534e\u946b\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINHUAXINLILINGHUOPEIZHIHUNHE"}, "519170": {"suoxie": "PYASZZDLHHA", "name": "\u6d66\u94f6\u5b89\u76db\u589e\u957f\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGZENGZHANGDONGLIHUNHEA"}, "519171": {"suoxie": "PYASYLJKHHA", "name": "\u6d66\u94f6\u5b89\u76db\u533b\u7597\u5065\u5eb7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGYILIAOJIANKANGHUNHEA"}, "519172": {"suoxie": "PYASRZJXHHA", "name": "\u6d66\u94f6\u5b89\u76db\u777f\u667a\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGRUIZHIJINGXUANHUNHEA"}, "519173": {"suoxie": "PYASRZJXHHC", "name": "\u6d66\u94f6\u5b89\u76db\u777f\u667a\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGRUIZHIJINGXUANHUNHEC"}, "519175": {"suoxie": "PYASJJDJQHH", "name": "\u6d66\u94f6\u5b89\u76db\u7ecf\u6d4e\u5e26\u5d1b\u8d77\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGJINGJIDAIJUEQIHUNHE"}, "519176": {"suoxie": "PYASXFSJHHC", "name": "\u6d66\u94f6\u5b89\u76db\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGXIAOFEISHENGJIHUNHEC"}, "519177": {"suoxie": "PYASSSJXHHC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u4e16\u7cbe\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PUYINANSHENGSHENGSHIJINGXUANHUNHEC"}, "519180": {"suoxie": "WJ180ZS", "name": "\u4e07\u5bb6180\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "WANJIA180ZHISHU"}, "519181": {"suoxie": "WJHXZZHH", "name": "\u4e07\u5bb6\u548c\u8c10\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHEXIEZENGZHANGHUNHE"}, "519182": {"suoxie": "WJHXZZHH", "name": "\u4e07\u5bb6\u548c\u8c10\u589e\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAHEXIEZENGZHANGHUNHE"}, "519183": {"suoxie": "WJSYQLHPZHH", "name": "\u4e07\u5bb6\u53cc\u5f15\u64ce\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIASHUANGYINQINGLINGHUOPEIZHIHUNHE"}, "519185": {"suoxie": "WJJXHH", "name": "\u4e07\u5bb6\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "WANJIAJINGXUANHUNHE"}, "519186": {"suoxie": "WJWJZLZQA", "name": "\u4e07\u5bb6\u7a33\u5065\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIAWENJIANZENGLIZHAIQUANA"}, "519187": {"suoxie": "WJWJZLZQC", "name": "\u4e07\u5bb6\u7a33\u5065\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIAWENJIANZENGLIZHAIQUANC"}, "519188": {"suoxie": "WJXYHLZQA", "name": "\u4e07\u5bb6\u4fe1\u7528\u6052\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINYONGHENGLIZHAIQUANA"}, "519189": {"suoxie": "WJXYHLZQC", "name": "\u4e07\u5bb6\u4fe1\u7528\u6052\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIAXINYONGHENGLIZHAIQUANC"}, "519190": {"suoxie": "WJSLZQ", "name": "\u4e07\u5bb6\u53cc\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "WANJIASHUANGLIZHAIQUAN"}, "519191": {"suoxie": "WJXLLHPZHH", "name": "\u4e07\u5bb6\u65b0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINLILINGHUOPEIZHIHUNHE"}, "519193": {"suoxie": "WJXFCZ", "name": "\u4e07\u5bb6\u6d88\u8d39\u6210\u957f", "style": "\u80a1\u7968\u578b", "pinyin": "WANJIAXIAOFEICHENGZHANG"}, "519195": {"suoxie": "WJPZ", "name": "\u4e07\u5bb6\u54c1\u8d28", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAPINZHI"}, "519196": {"suoxie": "WJXXLCLHPZHH", "name": "\u4e07\u5bb6\u65b0\u5174\u84dd\u7b79\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAXINXINGLANCHOULINGHUOPEIZHIHUNHE"}, "519197": {"suoxie": "WJYDLHPZHH", "name": "\u4e07\u5bb6\u9890\u8fbe\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAYIDALINGHUOPEIZHIHUNHE"}, "519198": {"suoxie": "WJYHLHPZHH", "name": "\u4e07\u5bb6\u9890\u548c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAYIHELINGHUOPEIZHIHUNHE"}, "519199": {"suoxie": "WJJXZDZA", "name": "\u4e07\u5bb6\u5bb6\u4eab\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "WANJIAJIAXIANGZHONGDUANZHAIA"}, "519206": {"suoxie": "WJNNHRDKZA", "name": "\u4e07\u5bb6\u5e74\u5e74\u6052\u8363\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIANIANNIANHENGRONGDINGKAIZHAIA"}, "519207": {"suoxie": "WJNNHRDKZC", "name": "\u4e07\u5bb6\u5e74\u5e74\u6052\u8363\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIANIANNIANHENGRONGDINGKAIZHAIC"}, "519208": {"suoxie": "WJ35NZJZCZA", "name": "\u4e07\u5bb63-5\u5e74\u653f\u91d1\u503a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIA35NIANZHENGJINZHAICHUNZHAIA"}, "519209": {"suoxie": "WJ35NZJZCZC", "name": "\u4e07\u5bb63-5\u5e74\u653f\u91d1\u503a\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "WANJIA35NIANZHENGJINZHAICHUNZHAIC"}, "519212": {"suoxie": "WJHGZSDCLHH", "name": "\u4e07\u5bb6\u5b8f\u89c2\u62e9\u65f6\u591a\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "WANJIAHONGGUANZESHIDUOCELUEHUNHE"}, "519220": {"suoxie": "HFTJLZQ", "name": "\u6d77\u5bcc\u901a\u805a\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGJULIZHAIQUAN"}, "519221": {"suoxie": "HFTXYHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u76ca\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINYIHUNHEC"}, "519222": {"suoxie": "HFTXYHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u76ca\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINYIHUNHEA"}, "519223": {"suoxie": "HFTXRHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u8363\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINRONGHUNHEC"}, "519224": {"suoxie": "HFTXRHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u8363\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINRONGHUNHEA"}, "519225": {"suoxie": "HFTJLZQ", "name": "\u6d77\u5bcc\u901a\u96c6\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGJILIZHAIQUAN"}, "519226": {"suoxie": "HFTRLZQ", "name": "\u6d77\u5bcc\u901a\u745e\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAIFUTONGRUILIZHAIQUAN"}, "519228": {"suoxie": "HFTXXLHPZHHC", "name": "\u6d77\u5bcc\u901a\u6b23\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINXIANGLINGHUOPEIZHIHUNHEC"}, "519229": {"suoxie": "HFTXXLHPZHHA", "name": "\u6d77\u5bcc\u901a\u6b23\u4eab\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HAIFUTONGXINXIANGLINGHUOPEIZHIHUNHEA"}, "519300": {"suoxie": "DCHS300ZSA", "name": "\u5927\u6210\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGHUSHEN300ZHISHUA"}, "519301": {"suoxie": "DCHS300ZSA", "name": "\u5927\u6210\u6caa\u6df1300\u6307\u6570A(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DACHENGHUSHEN300ZHISHUA"}, "519322": {"suoxie": "PYASSYDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u5143\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGYUANDINGKAIZHAIA"}, "519323": {"suoxie": "PYASSYDKZC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u5143\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGYUANDINGKAIZHAIC"}, "519324": {"suoxie": "PYASSXDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u946b\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGXINDINGKAIZHAIA"}, "519325": {"suoxie": "PYASSXDKZC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u946b\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGXINDINGKAIZHAIC"}, "519328": {"suoxie": "PYASSTCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u6cf0\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGTAICHUNZHAIZHAIQUANA"}, "519329": {"suoxie": "PYASSTCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u6cf0\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGTAICHUNZHAIZHAIQUANC"}, "519330": {"suoxie": "PYASSYCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u8dc3\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGYUECHUNZHAIZHAIQUANA"}, "519331": {"suoxie": "PYASSYCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u8dc3\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGYUECHUNZHAIZHAIQUANC"}, "519332": {"suoxie": "PYASSDCZZQA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u8fbe\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGDACHUNZHAIZHAIQUANA"}, "519333": {"suoxie": "PYASSDCZZQC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u8fbe\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGDACHUNZHAIZHAIQUANC"}, "519334": {"suoxie": "PYASSQ3GYDKZA", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u52e43\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGQIN3GEYUEDINGKAIZHAIA"}, "519335": {"suoxie": "PYASSQ3GYDKZC", "name": "\u6d66\u94f6\u5b89\u76db\u76db\u52e43\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PUYINANSHENGSHENGQIN3GEYUEDINGKAIZHAIC"}, "519501": {"suoxie": "WJHBR", "name": "\u4e07\u5bb6\u8d27\u5e01R", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAHUOBIR"}, "519505": {"suoxie": "HFTHBA", "name": "\u6d77\u5bcc\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGHUOBIA"}, "519506": {"suoxie": "HFTHBB", "name": "\u6d77\u5bcc\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HAIFUTONGHUOBIB"}, "519507": {"suoxie": "WJHBB", "name": "\u4e07\u5bb6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAHUOBIB"}, "519508": {"suoxie": "WJHBA", "name": "\u4e07\u5bb6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIAHUOBIA"}, "519509": {"suoxie": "PYASHBA", "name": "\u6d66\u94f6\u5b89\u76db\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGHUOBIA"}, "519510": {"suoxie": "PYASHBB", "name": "\u6d66\u94f6\u5b89\u76db\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGHUOBIB"}, "519511": {"suoxie": "WJRRXHBA", "name": "\u4e07\u5bb6\u65e5\u65e5\u85aa\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIARIRIXINHUOBIA"}, "519512": {"suoxie": "WJRRXHBB", "name": "\u4e07\u5bb6\u65e5\u65e5\u85aa\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIARIRIXINHUOBIB"}, "519513": {"suoxie": "WJRRXHBR", "name": "\u4e07\u5bb6\u65e5\u65e5\u85aa\u8d27\u5e01R", "style": "\u8d27\u5e01\u578b", "pinyin": "WANJIARIRIXINHUOBIR"}, "519516": {"suoxie": "PYASHBE", "name": "\u6d66\u94f6\u5b89\u76db\u8d27\u5e01E", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGHUOBIE"}, "519517": {"suoxie": "HTFHBB", "name": "\u6c47\u6dfb\u5bcc\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUOBIB"}, "519518": {"suoxie": "HTFHBA", "name": "\u6c47\u6dfb\u5bcc\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUHUOBIA"}, "519519": {"suoxie": "HTBRWBZLZQA", "name": "\u534e\u6cf0\u67cf\u745e\u7a33\u672c\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUATAIBAIRUIWENBENZENGLIZHAIQUANA"}, "519566": {"suoxie": "PYASRRYHBA", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u76c8\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIYINGHUOBIA"}, "519567": {"suoxie": "PYASRRYHBB", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u76c8\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIYINGHUOBIB"}, "519568": {"suoxie": "PYASRRYHBD", "name": "\u6d66\u94f6\u5b89\u76db\u65e5\u65e5\u76c8\u8d27\u5e01D", "style": "\u8d27\u5e01\u578b", "pinyin": "PUYINANSHENGRIRIYINGHUOBID"}, "519588": {"suoxie": "JYHBA", "name": "\u4ea4\u94f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINHUOBIA"}, "519589": {"suoxie": "JYHBB", "name": "\u4ea4\u94f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JIAOYINHUOBIB"}, "519601": {"suoxie": "HFTZGHWHH", "name": "\u6d77\u5bcc\u901a\u4e2d\u56fd\u6d77\u5916\u6df7\u5408", "style": "QDII", "pinyin": "HAIFUTONGZHONGGUOHAIWAIHUNHE"}, "519602": {"suoxie": "HFTDZHHHQDII", "name": "\u6d77\u5bcc\u901a\u5927\u4e2d\u534e\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "HAIFUTONGDAZHONGHUAHUNHEQDII"}, "519606": {"suoxie": "GTJXGP", "name": "\u56fd\u6cf0\u91d1\u946b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "GUOTAIJINXINGUPIAO"}, "519610": {"suoxie": "YHWLHHA", "name": "\u94f6\u6cb3\u65fa\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEWANGLIHUNHEA"}, "519611": {"suoxie": "YHWLHHC", "name": "\u94f6\u6cb3\u65fa\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEWANGLIHUNHEC"}, "519612": {"suoxie": "YHWLHHI", "name": "\u94f6\u6cb3\u65fa\u5229\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEWANGLIHUNHEI"}, "519613": {"suoxie": "YHJSHHA", "name": "\u94f6\u6cb3\u541b\u5c1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNSHANGHUNHEA"}, "519614": {"suoxie": "YHJSHHC", "name": "\u94f6\u6cb3\u541b\u5c1a\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNSHANGHUNHEC"}, "519615": {"suoxie": "YHJSHHI", "name": "\u94f6\u6cb3\u541b\u5c1a\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNSHANGHUNHEI"}, "519616": {"suoxie": "YHJXHHA", "name": "\u94f6\u6cb3\u541b\u4fe1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNXINHUNHEA"}, "519617": {"suoxie": "YHJXHHC", "name": "\u94f6\u6cb3\u541b\u4fe1\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNXINHUNHEC"}, "519618": {"suoxie": "YHJXHHI", "name": "\u94f6\u6cb3\u541b\u4fe1\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNXINHUNHEI"}, "519619": {"suoxie": "YHJRLHPZHHA", "name": "\u94f6\u6cb3\u541b\u8363\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNRONGLINGHUOPEIZHIHUNHEA"}, "519620": {"suoxie": "YHJRLHPZHHC", "name": "\u94f6\u6cb3\u541b\u8363\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNRONGLINGHUOPEIZHIHUNHEC"}, "519621": {"suoxie": "YHJRLHPZHHI", "name": "\u94f6\u6cb3\u541b\u8363\u7075\u6d3b\u914d\u7f6e\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNRONGLINGHUOPEIZHIHUNHEI"}, "519622": {"suoxie": "YHJYZQ", "name": "\u94f6\u6cb3\u541b\u6021\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJUNYIZHAIQUAN"}, "519623": {"suoxie": "YHJYHHA", "name": "\u94f6\u6cb3\u541b\u8000\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNYAOHUNHEA"}, "519624": {"suoxie": "YHJYHHC", "name": "\u94f6\u6cb3\u541b\u8000\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNYAOHUNHEC"}, "519625": {"suoxie": "YHJSHHA", "name": "\u94f6\u6cb3\u541b\u76db\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNSHENGHUNHEA"}, "519626": {"suoxie": "YHJSHHC", "name": "\u94f6\u6cb3\u541b\u76db\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNSHENGHUNHEC"}, "519627": {"suoxie": "YHJRHHA", "name": "\u94f6\u6cb3\u541b\u6da6\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNRUNHUNHEA"}, "519628": {"suoxie": "YHJRHHC", "name": "\u94f6\u6cb3\u541b\u6da6\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEJUNRUNHUNHEC"}, "519629": {"suoxie": "YHRLHHA", "name": "\u94f6\u6cb3\u777f\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHERUILIHUNHEA"}, "519630": {"suoxie": "YHRLHHC", "name": "\u94f6\u6cb3\u777f\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHERUILIHUNHEC"}, "519632": {"suoxie": "YHJH3GYDKZ", "name": "\u94f6\u6cb3\u541b\u8f893\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJUNHUI3GEYUEDINGKAIZHAI"}, "519640": {"suoxie": "YHHLHHA", "name": "\u94f6\u6cb3\u9e3f\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEHONGLIHUNHEA"}, "519641": {"suoxie": "YHHLHHC", "name": "\u94f6\u6cb3\u9e3f\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEHONGLIHUNHEC"}, "519642": {"suoxie": "YHZZHH", "name": "\u94f6\u6cb3\u667a\u9020\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEZHIZAOHUNHE"}, "519644": {"suoxie": "YHZLHH", "name": "\u94f6\u6cb3\u667a\u8054\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEZHILIANHUNHE"}, "519646": {"suoxie": "YHXLHHI", "name": "\u94f6\u6cb3\u946b\u5229\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEXINLIHUNHEI"}, "519647": {"suoxie": "YHHLHHI", "name": "\u94f6\u6cb3\u9e3f\u5229\u6df7\u5408I", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEHONGLIHUNHEI"}, "519648": {"suoxie": "YHTLCZI", "name": "\u94f6\u6cb3\u6cf0\u5229\u7eaf\u503aI", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHETAILICHUNZHAII"}, "519651": {"suoxie": "YHZXHH", "name": "\u94f6\u6cb3\u8f6c\u578b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEZHUANXINGHUNHE"}, "519652": {"suoxie": "YHXLHHA", "name": "\u94f6\u6cb3\u946b\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEXINLIHUNHEA"}, "519653": {"suoxie": "YHXLHHC", "name": "\u94f6\u6cb3\u946b\u5229\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEXINLIHUNHEC"}, "519654": {"suoxie": "YHFLCZZQ", "name": "\u94f6\u6cb3\u4e30\u5229\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEFENGLICHUNZHAIZHAIQUAN"}, "519655": {"suoxie": "YHFWHH", "name": "\u94f6\u6cb3\u670d\u52a1\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHEFUWUHUNHE"}, "519656": {"suoxie": "YHLHPZHHA", "name": "\u94f6\u6cb3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHELINGHUOPEIZHIHUNHEA"}, "519657": {"suoxie": "YHLHPZHHC", "name": "\u94f6\u6cb3\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YINHELINGHUOPEIZHIHUNHEC"}, "519660": {"suoxie": "YHZLZQA", "name": "\u94f6\u6cb3\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEZENGLIZHAIQUANA"}, "519661": {"suoxie": "YHZLZQC", "name": "\u94f6\u6cb3\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEZENGLIZHAIQUANC"}, "519662": {"suoxie": "YHJYHB6GYDKA", "name": "\u94f6\u6cb3\u4e45\u76ca\u56de\u62a56\u4e2a\u6708\u5b9a\u5f00A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJIUYIHUIBAO6GEYUEDINGKAIA"}, "519663": {"suoxie": "YHJYHB6GYDKC", "name": "\u94f6\u6cb3\u4e45\u76ca\u56de\u62a56\u4e2a\u6708\u5b9a\u5f00C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHEJIUYIHUIBAO6GEYUEDINGKAIC"}, "519664": {"suoxie": "YHMLHHA", "name": "\u94f6\u6cb3\u7f8e\u4e3d\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEMEILIHUNHEA"}, "519665": {"suoxie": "YHMLHHC", "name": "\u94f6\u6cb3\u7f8e\u4e3d\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEMEILIHUNHEC"}, "519666": {"suoxie": "YHYXTLZQB", "name": "\u94f6\u6cb3\u94f6\u4fe1\u6dfb\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEYINXINTIANLIZHAIQUANB"}, "519667": {"suoxie": "YHYXTLZQA", "name": "\u94f6\u6cb3\u94f6\u4fe1\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEYINXINTIANLIZHAIQUANA"}, "519668": {"suoxie": "YHCZHH", "name": "\u94f6\u6cb3\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHENGZHANGHUNHE"}, "519669": {"suoxie": "YHLXZQ", "name": "\u94f6\u6cb3\u9886\u5148\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHELINGXIANZHAIQUAN"}, "519670": {"suoxie": "YHHYHH", "name": "\u94f6\u6cb3\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEHANGYEHUNHE"}, "519671": {"suoxie": "YHHS300JZZSA", "name": "\u94f6\u6cb3\u6caa\u6df1300\u4ef7\u503c\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEHUSHEN300JIAZHIZHISHUA"}, "519672": {"suoxie": "YHLCHH", "name": "\u94f6\u6cb3\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHELANCHOUHUNHE"}, "519673": {"suoxie": "YHKLGP", "name": "\u94f6\u6cb3\u5eb7\u4e50\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "YINHEKANGLEGUPIAO"}, "519674": {"suoxie": "YHCXCZHHA", "name": "\u94f6\u6cb3\u521b\u65b0\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHECHUANGXINCHENGZHANGHUNHEA"}, "519675": {"suoxie": "YHTLCZA", "name": "\u94f6\u6cb3\u6cf0\u5229\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINHETAILICHUNZHAIA"}, "519676": {"suoxie": "YHQHZQ", "name": "\u94f6\u6cb3\u5f3a\u5316\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEQIANGHUAZHAIQUAN"}, "519677": {"suoxie": "YHDTBTXJAZS", "name": "\u94f6\u6cb3\u5b9a\u6295\u5b9d\u817e\u8baf\u6d4e\u5b89\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHEDINGTOUBAOTENGXUNJIANZHISHU"}, "519678": {"suoxie": "YHXFHH", "name": "\u94f6\u6cb3\u6d88\u8d39\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEXIAOFEIHUNHE"}, "519679": {"suoxie": "YHZTHH", "name": "\u94f6\u6cb3\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YINHEZHUTIHUNHE"}, "519680": {"suoxie": "JYZLZQAB", "name": "\u4ea4\u94f6\u589e\u5229\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGLIZHAIQUANAB"}, "519681": {"suoxie": "JYZLZQAB", "name": "\u4ea4\u94f6\u589e\u5229\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGLIZHAIQUANAB"}, "519682": {"suoxie": "JYZLZQC", "name": "\u4ea4\u94f6\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGLIZHAIQUANC"}, "519683": {"suoxie": "JYSLZQAB", "name": "\u4ea4\u94f6\u53cc\u5229\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINSHUANGLIZHAIQUANAB"}, "519684": {"suoxie": "JYSLZQAB", "name": "\u4ea4\u94f6\u53cc\u5229\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINSHUANGLIZHAIQUANAB"}, "519685": {"suoxie": "JYSLZQC", "name": "\u4ea4\u94f6\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINSHUANGLIZHAIQUANC"}, "519686": {"suoxie": "JYSZ180GSZLLJ", "name": "\u4ea4\u94f6\u4e0a\u8bc1180\u516c\u53f8\u6cbb\u7406\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHANGZHENG180GONGSIZHILILIANJIE"}, "519687": {"suoxie": "JYSZ180GSZLLJ", "name": "\u4ea4\u94f6\u4e0a\u8bc1180\u516c\u53f8\u6cbb\u7406\u8054\u63a5(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHANGZHENG180GONGSIZHILILIANJIE"}, "519688": {"suoxie": "JYJXHH", "name": "\u4ea4\u94f6\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJINGXUANHUNHE"}, "519689": {"suoxie": "JYJXHH", "name": "\u4ea4\u94f6\u7cbe\u9009\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJINGXUANHUNHE"}, "519690": {"suoxie": "JYWJPZHHA", "name": "\u4ea4\u94f6\u7a33\u5065\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINWENJIANPEIZHIHUNHEA"}, "519691": {"suoxie": "JYWJPZHHA", "name": "\u4ea4\u94f6\u7a33\u5065\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINWENJIANPEIZHIHUNHEA"}, "519692": {"suoxie": "JYCZHHA", "name": "\u4ea4\u94f6\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANGHUNHEA"}, "519693": {"suoxie": "JYCZHHA", "name": "\u4ea4\u94f6\u6210\u957f\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANGHUNHEA"}, "519694": {"suoxie": "JYLCHH", "name": "\u4ea4\u94f6\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINLANCHOUHUNHE"}, "519695": {"suoxie": "JYLCHH", "name": "\u4ea4\u94f6\u84dd\u7b79\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINLANCHOUHUNHE"}, "519696": {"suoxie": "JYHQJXHHQDII", "name": "\u4ea4\u94f6\u73af\u7403\u7cbe\u9009\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "JIAOYINHUANQIUJINGXUANHUNHEQDII"}, "519697": {"suoxie": "JYYSHYHH", "name": "\u4ea4\u94f6\u4f18\u52bf\u884c\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINYOUSHIHANGYEHUNHE"}, "519698": {"suoxie": "JYXFHHA", "name": "\u4ea4\u94f6\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANFENGHUNHEA"}, "519699": {"suoxie": "JYXFHHA", "name": "\u4ea4\u94f6\u5148\u950b\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANFENGHUNHEA"}, "519700": {"suoxie": "JYZTYXHHA", "name": "\u4ea4\u94f6\u4e3b\u9898\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHUTIYOUXUANHUNHEA"}, "519701": {"suoxie": "JYZTYXHHA", "name": "\u4ea4\u94f6\u4e3b\u9898\u4f18\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHUTIYOUXUANHUNHEA"}, "519702": {"suoxie": "JYQSHHA", "name": "\u4ea4\u94f6\u8d8b\u52bf\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQUSHIHUNHEA"}, "519703": {"suoxie": "JYQSHHA", "name": "\u4ea4\u94f6\u8d8b\u52bf\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINQUSHIHUNHEA"}, "519704": {"suoxie": "JYXJZZHHA", "name": "\u4ea4\u94f6\u5148\u8fdb\u5236\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANJINZHIZAOHUNHEA"}, "519705": {"suoxie": "JYXJZZHHA", "name": "\u4ea4\u94f6\u5148\u8fdb\u5236\u9020\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXIANJINZHIZAOHUNHEA"}, "519706": {"suoxie": "JYSZ300JZETFLJ", "name": "\u4ea4\u94f6\u6df1\u8bc1300\u4ef7\u503cETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHENZHENG300JIAZHIETFLIANJIE"}, "519707": {"suoxie": "JYSZ300JZETFLJ", "name": "\u4ea4\u94f6\u6df1\u8bc1300\u4ef7\u503cETF\u8054\u63a5(\u540e\u7aef)", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIAOYINSHENZHENG300JIAZHIETFLIANJIE"}, "519710": {"suoxie": "JYCLHBLHPZHH", "name": "\u4ea4\u94f6\u7b56\u7565\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINCELUEHUIBAOLINGHUOPEIZHIHUNHE"}, "519712": {"suoxie": "JYAEFHXHHA", "name": "\u4ea4\u94f6\u963f\u5c14\u6cd5\u6838\u5fc3\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINAERFAHEXINHUNHEA"}, "519713": {"suoxie": "JYAEFHXHHA", "name": "\u4ea4\u94f6\u963f\u5c14\u6cd5\u6838\u5fc3\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINAERFAHEXINHUNHEA"}, "519714": {"suoxie": "JYXFXQDGP", "name": "\u4ea4\u94f6\u6d88\u8d39\u65b0\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "JIAOYINXIAOFEIXINQUDONGGUPIAO"}, "519715": {"suoxie": "JYXFXQDGP", "name": "\u4ea4\u94f6\u6d88\u8d39\u65b0\u9a71\u52a8\u80a1\u7968(\u540e\u7aef)", "style": "\u80a1\u7968\u578b", "pinyin": "JIAOYINXIAOFEIXINQUDONGGUPIAO"}, "519717": {"suoxie": "JYSLDZGDJXYZ", "name": "\u4ea4\u94f6\u65bd\u7f57\u5fb7\u4e2d\u9ad8\u7b49\u7ea7\u4fe1\u7528\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINSHILUODEZHONGGAODENGJIXINYONGZHAI"}, "519718": {"suoxie": "JYCZZQFQA", "name": "\u4ea4\u94f6\u7eaf\u503a\u503a\u5238\u53d1\u8d77A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINCHUNZHAIZHAIQUANFAQIA"}, "519719": {"suoxie": "JYCZZQFQA", "name": "\u4ea4\u94f6\u7eaf\u503a\u503a\u5238\u53d1\u8d77A(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINCHUNZHAIZHAIQUANFAQIA"}, "519720": {"suoxie": "JYCZZQFQC", "name": "\u4ea4\u94f6\u7eaf\u503a\u503a\u5238\u53d1\u8d77C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINCHUNZHAIZHAIQUANFAQIC"}, "519722": {"suoxie": "JYYHCZZQ", "name": "\u4ea4\u94f6\u88d5\u60e0\u7eaf\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUHUICHUNZHAIZHAIQUAN"}, "519723": {"suoxie": "JYSLDZQAB", "name": "\u4ea4\u94f6\u53cc\u8f6e\u52a8\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINSHUANGLUNDONGZHAIQUANAB"}, "519724": {"suoxie": "JYSLDZQAB", "name": "\u4ea4\u94f6\u53cc\u8f6e\u52a8\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINSHUANGLUNDONGZHAIQUANAB"}, "519725": {"suoxie": "JYSLDZQC", "name": "\u4ea4\u94f6\u53cc\u8f6e\u52a8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINSHUANGLUNDONGZHAIQUANC"}, "519726": {"suoxie": "JYWGSYZQ", "name": "\u4ea4\u94f6\u7a33\u56fa\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINWENGUSHOUYIZHAIQUAN"}, "519727": {"suoxie": "JYCZ30HH", "name": "\u4ea4\u94f6\u6210\u957f30\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANG30HUNHE"}, "519728": {"suoxie": "JYCZ30HH", "name": "\u4ea4\u94f6\u6210\u957f30\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINCHENGZHANG30HUNHE"}, "519729": {"suoxie": "JYZQSYZQ", "name": "\u4ea4\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINZENGQIANGSHOUYIZHAIQUAN"}, "519730": {"suoxie": "JYDQZFYYFZQA", "name": "\u4ea4\u94f6\u5b9a\u671f\u652f\u4ed8\u6708\u6708\u4e30\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINDINGQIZHIFUYUEYUEFENGZHAIQUANA"}, "519731": {"suoxie": "JYDQZFYYFZQC", "name": "\u4ea4\u94f6\u5b9a\u671f\u652f\u4ed8\u6708\u6708\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINDINGQIZHIFUYUEYUEFENGZHAIQUANC"}, "519732": {"suoxie": "JYDQZFSXPHHH", "name": "\u4ea4\u94f6\u5b9a\u671f\u652f\u4ed8\u53cc\u606f\u5e73\u8861\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIAOYINDINGQIZHIFUSHUANGXIPINGHENGHUNHE"}, "519733": {"suoxie": "JYQHHBZQAB", "name": "\u4ea4\u94f6\u5f3a\u5316\u56de\u62a5\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINQIANGHUAHUIBAOZHAIQUANAB"}, "519734": {"suoxie": "JYQHHBZQAB", "name": "\u4ea4\u94f6\u5f3a\u5316\u56de\u62a5\u503a\u5238A/B(\u540e\u7aef)", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINQIANGHUAHUIBAOZHAIQUANAB"}, "519735": {"suoxie": "JYQHHBZQC", "name": "\u4ea4\u94f6\u5f3a\u5316\u56de\u62a5\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINQIANGHUAHUIBAOZHAIQUANC"}, "519736": {"suoxie": "JYXCZHH", "name": "\u4ea4\u94f6\u65b0\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXINCHENGZHANGHUNHE"}, "519737": {"suoxie": "JYXCZHH", "name": "\u4ea4\u94f6\u65b0\u6210\u957f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINXINCHENGZHANGHUNHE"}, "519738": {"suoxie": "JYZQHBLHPZHHA", "name": "\u4ea4\u94f6\u5468\u671f\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHOUQIHUIBAOLINGHUOPEIZHIHUNHEA"}, "519739": {"suoxie": "JYZQHBLHPZHHA", "name": "\u4ea4\u94f6\u5468\u671f\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHOUQIHUIBAOLINGHUOPEIZHIHUNHEA"}, "519740": {"suoxie": "JYFYSYZQA", "name": "\u4ea4\u94f6\u4e30\u76c8\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGYINGSHOUYIZHAIQUANA"}, "519743": {"suoxie": "JYFRSYZQAB", "name": "\u4ea4\u94f6\u4e30\u6da6\u6536\u76ca\u503a\u5238A/B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGRUNSHOUYIZHAIQUANAB"}, "519745": {"suoxie": "JYFRSYZQC", "name": "\u4ea4\u94f6\u4e30\u6da6\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGRUNSHOUYIZHAIQUANC"}, "519746": {"suoxie": "JYFXSYZQA", "name": "\u4ea4\u94f6\u4e30\u4eab\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGXIANGSHOUYIZHAIQUANA"}, "519748": {"suoxie": "JYFXSYZQC", "name": "\u4ea4\u94f6\u4e30\u4eab\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINFENGXIANGSHOUYIZHAIQUANC"}, "519752": {"suoxie": "JYXHBLHPZHHA", "name": "\u4ea4\u94f6\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINXINHUIBAOLINGHUOPEIZHIHUNHEA"}, "519753": {"suoxie": "JYAXSYZQ", "name": "\u4ea4\u94f6\u5b89\u5fc3\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIAOYINANXINSHOUYIZHAIQUAN"}, "519755": {"suoxie": "JYDCLHBLHPZHHA", "name": "\u4ea4\u94f6\u591a\u7b56\u7565\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINDUOCELUEHUIBAOLINGHUOPEIZHIHUNHEA"}, "519756": {"suoxie": "JYGQGGLHPZHH", "name": "\u4ea4\u94f6\u56fd\u4f01\u6539\u9769\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINGUOQIGAIGELINGHUOPEIZHIHUNHE"}, "519759": {"suoxie": "JYZQHBLHPZHHC", "name": "\u4ea4\u94f6\u5468\u671f\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINZHOUQIHUIBAOLINGHUOPEIZHIHUNHEC"}, "519760": {"suoxie": "JYXHBLHPZHHC", "name": "\u4ea4\u94f6\u65b0\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINXINHUIBAOLINGHUOPEIZHIHUNHEC"}, "519761": {"suoxie": "JYDCLHBLHPZHHC", "name": "\u4ea4\u94f6\u591a\u7b56\u7565\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINDUOCELUEHUIBAOLINGHUOPEIZHIHUNHEC"}, "519762": {"suoxie": "JYYTCZZQA", "name": "\u4ea4\u94f6\u88d5\u901a\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUTONGCHUNZHAIZHAIQUANA"}, "519763": {"suoxie": "JYYTCZZQC", "name": "\u4ea4\u94f6\u88d5\u901a\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUTONGCHUNZHAIZHAIQUANC"}, "519766": {"suoxie": "JYRXLHPZHH", "name": "\u4ea4\u94f6\u8363\u946b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINRONGXINLINGHUOPEIZHIHUNHE"}, "519767": {"suoxie": "JYKJCXLHPZHH", "name": "\u4ea4\u94f6\u79d1\u6280\u521b\u65b0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINKEJICHUANGXINLINGHUOPEIZHIHUNHE"}, "519768": {"suoxie": "JYYXHBLHPZHHA", "name": "\u4ea4\u94f6\u4f18\u9009\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINYOUXUANHUIBAOLINGHUOPEIZHIHUNHEA"}, "519769": {"suoxie": "JYYXHBLHPZHHC", "name": "\u4ea4\u94f6\u4f18\u9009\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINYOUXUANHUIBAOLINGHUOPEIZHIHUNHEC"}, "519770": {"suoxie": "JYYZHBLHPZHHA", "name": "\u4ea4\u94f6\u4f18\u62e9\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINYOUZEHUIBAOLINGHUOPEIZHIHUNHEA"}, "519771": {"suoxie": "JYYZHBLHPZHHC", "name": "\u4ea4\u94f6\u4f18\u62e9\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINYOUZEHUIBAOLINGHUOPEIZHIHUNHEC"}, "519772": {"suoxie": "JYXSHLLHPZHH", "name": "\u4ea4\u94f6\u65b0\u751f\u6d3b\u529b\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINXINSHENGHUOLILINGHUOPEIZHIHUNHE"}, "519773": {"suoxie": "JYSJCYLHPZHHA", "name": "\u4ea4\u94f6\u6570\u636e\u4ea7\u4e1a\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINSHUJUCHANYELINGHUOPEIZHIHUNHEA"}, "519776": {"suoxie": "JYYYCZZQA", "name": "\u4ea4\u94f6\u88d5\u76c8\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUYINGCHUNZHAIZHAIQUANA"}, "519777": {"suoxie": "JYYYCZZQC", "name": "\u4ea4\u94f6\u88d5\u76c8\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYUYINGCHUNZHAIZHAIQUANC"}, "519778": {"suoxie": "JYJJXDLHHA", "name": "\u4ea4\u94f6\u7ecf\u6d4e\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIAOYINJINGJIXINDONGLIHUNHEA"}, "519779": {"suoxie": "JYHGSJZJXHH", "name": "\u4ea4\u94f6\u6caa\u6e2f\u6df1\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JIAOYINHUGANGSHENJIAZHIJINGXUANHUNHE"}, "519782": {"suoxie": "JYYLCZZQA", "name": "\u4ea4\u94f6\u88d5\u9686\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYULONGCHUNZHAIZHAIQUANA"}, "519783": {"suoxie": "JYYLCZZQC", "name": "\u4ea4\u94f6\u88d5\u9686\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYULONGCHUNZHAIZHAIQUANC"}, "519784": {"suoxie": "JYJSSYZQA", "name": "\u4ea4\u94f6\u5883\u5c1a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINJINGSHANGSHOUYIZHAIQUANA"}, "519785": {"suoxie": "JYJSSYZQC", "name": "\u4ea4\u94f6\u5883\u5c1a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINJINGSHANGSHOUYIZHAIQUANC"}, "519786": {"suoxie": "JYYLCZZQA", "name": "\u4ea4\u94f6\u88d5\u5229\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYULICHUNZHAIZHAIQUANA"}, "519787": {"suoxie": "JYYLCZZQC", "name": "\u4ea4\u94f6\u88d5\u5229\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIAOYINYULICHUNZHAIZHAIQUANC"}, "519800": {"suoxie": "HXBZJHBA", "name": "\u534e\u590f\u4fdd\u8bc1\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIABAOZHENGJINHUOBIA"}, "519801": {"suoxie": "HXBZJHBB", "name": "\u534e\u590f\u4fdd\u8bc1\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUAXIABAOZHENGJINHUOBIB"}, "519858": {"suoxie": "GFXJBCNHBA", "name": "\u5e7f\u53d1\u73b0\u91d1\u5b9d\u573a\u5185\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAXIANJINBAOCHANGNEIHUOBIA"}, "519859": {"suoxie": "GFXJBCNHBB", "name": "\u5e7f\u53d1\u73b0\u91d1\u5b9d\u573a\u5185\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGFAXIANJINBAOCHANGNEIHUOBIB"}, "519878": {"suoxie": "GSABCNSSSSHBA", "name": "\u56fd\u5bff\u5b89\u4fdd\u573a\u5185\u5b9e\u65f6\u7533\u8d4e\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOCHANGNEISHISHISHENSHUHUOBIA"}, "519879": {"suoxie": "GSABCNSSSSHBB", "name": "\u56fd\u5bff\u5b89\u4fdd\u573a\u5185\u5b9e\u65f6\u7533\u8d4e\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOSHOUANBAOCHANGNEISHISHISHENSHUHUOBIB"}, "519888": {"suoxie": "HTFSYKXHBA", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u7ebf\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIXIANHUOBIA"}, "519889": {"suoxie": "HTFSYKXHBB", "name": "\u6c47\u6dfb\u5bcc\u6536\u76ca\u5feb\u7ebf\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUITIANFUSHOUYIKUAIXIANHUOBIB"}, "519898": {"suoxie": "DCXJBHBA", "name": "\u5927\u6210\u73b0\u91d1\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGXIANJINBAOHUOBIA"}, "519899": {"suoxie": "DCXJBHBB", "name": "\u5927\u6210\u73b0\u91d1\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DACHENGXIANJINBAOHUOBIB"}, "519908": {"suoxie": "HXXHHHA", "name": "\u534e\u590f\u5174\u534e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINGHUAHUNHEA"}, "519909": {"suoxie": "HAASHH", "name": "\u534e\u5b89\u5b89\u987a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANANSHUNHUNHE"}, "519915": {"suoxie": "FGXFZTHHA", "name": "\u5bcc\u56fd\u6d88\u8d39\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUGUOXIAOFEIZHUTIHUNHEA"}, "519918": {"suoxie": "HXXHHH", "name": "\u534e\u590f\u5174\u548c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUAXIAXINGHEHUNHE"}, "519929": {"suoxie": "CXDZXXLHLHPZHHA", "name": "\u957f\u4fe1\u7535\u5b50\u4fe1\u606f\u91cf\u5316\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINDIANZIXINXILIANGHUALINGHUOPEIZHIHUNHEA"}, "519933": {"suoxie": "CXLFZQ", "name": "\u957f\u4fe1\u5229\u53d1\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFAZHAIQUAN"}, "519935": {"suoxie": "CXCXQDGP", "name": "\u957f\u4fe1\u521b\u65b0\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINCHUANGXINQUDONGGUPIAO"}, "519937": {"suoxie": "CXXRHHA", "name": "\u957f\u4fe1\u5148\u9510\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "CHANGXINXIANRUIHUNHEA"}, "519940": {"suoxie": "CXFQCZYNDKZC", "name": "\u957f\u4fe1\u5bcc\u5168\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUQUANCHUNZHAIYINIANDINGKAIZHAIC"}, "519941": {"suoxie": "CXFQCZYNDKZA", "name": "\u957f\u4fe1\u5bcc\u5168\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUQUANCHUNZHAIYINIANDINGKAIZHAIA"}, "519942": {"suoxie": "CXLLC", "name": "\u957f\u4fe1\u5229\u7387C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINLILVC"}, "519943": {"suoxie": "CXLLA", "name": "\u957f\u4fe1\u5229\u7387A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINLILVA"}, "519944": {"suoxie": "CXFACZBNDKZC", "name": "\u957f\u4fe1\u5bcc\u5b89\u7eaf\u503a\u534a\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUANCHUNZHAIBANNIANDINGKAIZHAIC"}, "519945": {"suoxie": "CXFACZBNDKZA", "name": "\u957f\u4fe1\u5bcc\u5b89\u7eaf\u503a\u534a\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUANCHUNZHAIBANNIANDINGKAIZHAIA"}, "519947": {"suoxie": "CXLBZQA", "name": "\u957f\u4fe1\u5229\u4fdd\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIBAOZHAIQUANA"}, "519949": {"suoxie": "CXLXHHA", "name": "\u957f\u4fe1\u5229\u4fe1\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIXINHUNHEA"}, "519951": {"suoxie": "CXLTLHPZHHA", "name": "\u957f\u4fe1\u5229\u6cf0\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLITAILINGHUOPEIZHIHUNHEA"}, "519953": {"suoxie": "CXFHCZYNDKZC", "name": "\u957f\u4fe1\u5bcc\u6d77\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUHAICHUNZHAIYINIANDINGKAIZHAIC"}, "519955": {"suoxie": "CXFMCZYNDKZC", "name": "\u957f\u4fe1\u5bcc\u6c11\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINFUMINCHUNZHAIYINIANDINGKAIZHAIC"}, "519956": {"suoxie": "CXRJHHC", "name": "\u957f\u4fe1\u777f\u8fdb\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINRUIJINHUNHEC"}, "519957": {"suoxie": "CXRJHHA", "name": "\u957f\u4fe1\u777f\u8fdb\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINRUIJINHUNHEA"}, "519959": {"suoxie": "CXDLHHA", "name": "\u957f\u4fe1\u591a\u5229\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINDUOLIHUNHEA"}, "519960": {"suoxie": "CXLGHHC", "name": "\u957f\u4fe1\u5229\u5e7f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIGUANGHUNHEC"}, "519961": {"suoxie": "CXLGHHA", "name": "\u957f\u4fe1\u5229\u5e7f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIGUANGHUNHEA"}, "519962": {"suoxie": "CXLYHHC", "name": "\u957f\u4fe1\u5229\u76c8\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIYINGHUNHEC"}, "519963": {"suoxie": "CXLYHHA", "name": "\u957f\u4fe1\u5229\u76c8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINLIYINGHUNHEA"}, "519965": {"suoxie": "CXLHDCLGPA", "name": "\u957f\u4fe1\u91cf\u5316\u591a\u7b56\u7565\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINLIANGHUADUOCELUEGUPIAOA"}, "519967": {"suoxie": "CXLFZQA", "name": "\u957f\u4fe1\u5229\u5bcc\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFUZHAIQUANA"}, "519969": {"suoxie": "CXXLLHPZHH", "name": "\u957f\u4fe1\u65b0\u5229\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINXINLILINGHUOPEIZHIHUNHE"}, "519971": {"suoxie": "CXGGHLHH", "name": "\u957f\u4fe1\u6539\u9769\u7ea2\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "CHANGXINGAIGEHONGLIHUNHE"}, "519972": {"suoxie": "CXCZYNDKZC", "name": "\u957f\u4fe1\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINCHUNZHAIYINIANDINGKAIZHAIC"}, "519973": {"suoxie": "CXCZYNDKZA", "name": "\u957f\u4fe1\u7eaf\u503a\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINCHUNZHAIYINIANDINGKAIZHAIA"}, "519975": {"suoxie": "CXLHZXPGP", "name": "\u957f\u4fe1\u91cf\u5316\u4e2d\u5c0f\u76d8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "CHANGXINLIANGHUAZHONGXIAOPANGUPIAO"}, "519976": {"suoxie": "CXKZZZQC", "name": "\u957f\u4fe1\u53ef\u8f6c\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGXINKEZHUANZHAIZHAIQUANC"}, "519977": {"suoxie": "CXKZZZQA", "name": "\u957f\u4fe1\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CHANGXINKEZHUANZHAIZHAIQUANA"}, "519979": {"suoxie": "CXNXCZHHA", "name": "\u957f\u4fe1\u5185\u9700\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINNEIXUCHENGZHANGHUNHEA"}, "519981": {"suoxie": "CXBP100DQZZSRMB", "name": "\u957f\u4fe1\u6807\u666e100\u7b49\u6743\u91cd\u6307\u6570\u4eba\u6c11\u5e01", "style": "QDII", "pinyin": "CHANGXINBIAOPU100DENGQUANZHONGZHISHURENMINBI"}, "519983": {"suoxie": "CXLHXFHHA", "name": "\u957f\u4fe1\u91cf\u5316\u5148\u950b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINLIANGHUAXIANFENGHUNHEA"}, "519985": {"suoxie": "CXCZYHZQA", "name": "\u957f\u4fe1\u7eaf\u503a\u58f9\u53f7\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGXINCHUNZHAIYIHAOZHAIQUANA"}, "519986": {"suoxie": "CXHLYSHH", "name": "\u957f\u4fe1\u6052\u5229\u4f18\u52bf\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINHENGLIYOUSHIHUNHE"}, "519987": {"suoxie": "CXHLYSHH", "name": "\u957f\u4fe1\u6052\u5229\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINHENGLIYOUSHIHUNHE"}, "519989": {"suoxie": "CXLFZQC", "name": "\u957f\u4fe1\u5229\u4e30\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGXINLIFENGZHAIQUANC"}, "519990": {"suoxie": "CXSLYXHHA", "name": "\u957f\u4fe1\u53cc\u5229\u4f18\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINSHUANGLIYOUXUANHUNHEA"}, "519991": {"suoxie": "CXSLYXHHA", "name": "\u957f\u4fe1\u53cc\u5229\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINSHUANGLIYOUXUANHUNHEA"}, "519992": {"suoxie": "CXZLDTCLHH", "name": "\u957f\u4fe1\u589e\u5229\u52a8\u6001\u7b56\u7565\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINZENGLIDONGTAICELUEHUNHE"}, "519993": {"suoxie": "CXZLDTCLHH", "name": "\u957f\u4fe1\u589e\u5229\u52a8\u6001\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINZENGLIDONGTAICELUEHUNHE"}, "519994": {"suoxie": "CXJLQSHH", "name": "\u957f\u4fe1\u91d1\u5229\u8d8b\u52bf\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINJINLIQUSHIHUNHE"}, "519995": {"suoxie": "CXJLQSHH", "name": "\u957f\u4fe1\u91d1\u5229\u8d8b\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINJINLIQUSHIHUNHE"}, "519996": {"suoxie": "CXYLJXHHA", "name": "\u957f\u4fe1\u94f6\u5229\u7cbe\u9009\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINYINLIJINGXUANHUNHEA"}, "519997": {"suoxie": "CXYLJXHHA", "name": "\u957f\u4fe1\u94f6\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGXINYINLIJINGXUANHUNHEA"}, "519998": {"suoxie": "CXLXSYHBB", "name": "\u957f\u4fe1\u5229\u606f\u6536\u76ca\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGXINLIXISHOUYIHUOBIB"}, "519999": {"suoxie": "CXLXSYHBA", "name": "\u957f\u4fe1\u5229\u606f\u6536\u76ca\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGXINLIXISHOUYIHUOBIA"}, "530001": {"suoxie": "JXHJJZHH", "name": "\u5efa\u4fe1\u6052\u4e45\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINHENGJIUJIAZHIHUNHE"}, "530002": {"suoxie": "JXHBA", "name": "\u5efa\u4fe1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JIANXINHUOBIA"}, "530003": {"suoxie": "JXYXCZHHA", "name": "\u5efa\u4fe1\u4f18\u9009\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINYOUXUANCHENGZHANGHUNHEA"}, "530005": {"suoxie": "JXYHPZHH", "name": "\u5efa\u4fe1\u4f18\u5316\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINYOUHUAPEIZHIHUNHE"}, "530006": {"suoxie": "JXHXJXHH", "name": "\u5efa\u4fe1\u6838\u5fc3\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINHEXINJINGXUANHUNHE"}, "530008": {"suoxie": "JXWDZLZQC", "name": "\u5efa\u4fe1\u7a33\u5b9a\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINWENDINGZENGLIZHAIQUANC"}, "530009": {"suoxie": "JXSYZQZQA", "name": "\u5efa\u4fe1\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHOUYIZENGQIANGZHAIQUANA"}, "530010": {"suoxie": "JXSZSHZRETFLJ", "name": "\u5efa\u4fe1\u4e0a\u8bc1\u793e\u4f1a\u8d23\u4efbETF\u8054\u63a5", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHANGZHENGSHEHUIZERENETFLIANJIE"}, "530011": {"suoxie": "JXNSDLHH", "name": "\u5efa\u4fe1\u5185\u751f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINNEISHENGDONGLIHUNHE"}, "530012": {"suoxie": "JXJJPZHH", "name": "\u5efa\u4fe1\u79ef\u6781\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIANXINJIJIPEIZHIHUNHE"}, "530014": {"suoxie": "JXLLZZQ", "name": "\u5efa\u4fe1\u5229\u7387\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINLILVZHAIZHAIQUAN"}, "530015": {"suoxie": "JXSZJBM60ETFLJA", "name": "\u5efa\u4fe1\u6df1\u8bc1\u57fa\u672c\u976260ETF\u8054\u63a5A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHENZHENGJIBENMIAN60ETFLIANJIEA"}, "530016": {"suoxie": "JXHWJZHH", "name": "\u5efa\u4fe1\u6052\u7a33\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JIANXINHENGWENJIAZHIHUNHE"}, "530017": {"suoxie": "JXSXHLZQA", "name": "\u5efa\u4fe1\u53cc\u606f\u7ea2\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHUANGXIHONGLIZHAIQUANA"}, "530018": {"suoxie": "JXSZ100ZSZQ", "name": "\u5efa\u4fe1\u6df1\u8bc1100\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIANXINSHENZHENG100ZHISHUZENGQIANG"}, "530019": {"suoxie": "JXSHZRHH", "name": "\u5efa\u4fe1\u793e\u4f1a\u8d23\u4efb\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JIANXINSHEHUIZERENHUNHE"}, "530020": {"suoxie": "JXZZZQZQA", "name": "\u5efa\u4fe1\u8f6c\u503a\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "JIANXINZHUANZHAIZENGQIANGZHAIQUANA"}, "530021": {"suoxie": "JXCZZQA", "name": "\u5efa\u4fe1\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINCHUNZHAIZHAIQUANA"}, "530028": {"suoxie": "JXDZZQC", "name": "\u5efa\u4fe1\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINDUANZHAIZHAIQUANC"}, "530029": {"suoxie": "JXRYYNDKZ", "name": "\u5efa\u4fe1\u8363\u5143\u4e00\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINRONGYUANYINIANDINGKAIZHAI"}, "530030": {"suoxie": "JXZYAXLCZQA", "name": "\u5efa\u4fe1\u5468\u76c8\u5b89\u5fc3\u7406\u8d22\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINZHOUYINGANXINLICAIZHAIQUANA"}, "531008": {"suoxie": "JXWDZLZQA", "name": "\u5efa\u4fe1\u7a33\u5b9a\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINWENDINGZENGLIZHAIQUANA"}, "531009": {"suoxie": "JXSYZQZQC", "name": "\u5efa\u4fe1\u6536\u76ca\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHOUYIZENGQIANGZHAIQUANC"}, "531017": {"suoxie": "JXSXHLZQC", "name": "\u5efa\u4fe1\u53cc\u606f\u7ea2\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JIANXINSHUANGXIHONGLIZHAIQUANC"}, "531020": {"suoxie": "JXZZZQZQC", "name": "\u5efa\u4fe1\u8f6c\u503a\u589e\u5f3a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "JIANXINZHUANZHAIZENGQIANGZHAIQUANC"}, "531021": {"suoxie": "JXCZZQC", "name": "\u5efa\u4fe1\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JIANXINCHUNZHAIZHAIQUANC"}, "531028": {"suoxie": "JXDZZQA", "name": "\u5efa\u4fe1\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINDUANZHAIZHAIQUANA"}, "531030": {"suoxie": "JXZYAXLCZQB", "name": "\u5efa\u4fe1\u5468\u76c8\u5b89\u5fc3\u7406\u8d22\u503a\u5238B", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "JIANXINZHOUYINGANXINLICAIZHAIQUANB"}, "539001": {"suoxie": "JXNSDK100ZSQDIIRMBA", "name": "\u5efa\u4fe1\u7eb3\u65af\u8fbe\u514b100\u6307\u6570(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "JIANXINNASIDAKE100ZHISHUQDIIRENMINBIA"}, "539002": {"suoxie": "JXXXSCHHQDII", "name": "\u5efa\u4fe1\u65b0\u5174\u5e02\u573a\u6df7\u5408(QDII)", "style": "QDII", "pinyin": "JIANXINXINXINGSHICHANGHUNHEQDII"}, "539003": {"suoxie": "JXFS100ZSQDIIRMBA", "name": "\u5efa\u4fe1\u5bcc\u65f6100\u6307\u6570(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "JIANXINFUSHI100ZHISHUQDIIRENMINBIA"}, "540001": {"suoxie": "HFJX2016ZQHH", "name": "\u6c47\u4e30\u664b\u4fe12016\u5468\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIFENGJINXIN2016ZHOUQIHUNHE"}, "540002": {"suoxie": "HFJXLTHH", "name": "\u6c47\u4e30\u664b\u4fe1\u9f99\u817e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINLONGTENGHUNHE"}, "540003": {"suoxie": "HFJXDTCLHHA", "name": "\u6c47\u4e30\u664b\u4fe1\u52a8\u6001\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINDONGTAICELUEHUNHEA"}, "540004": {"suoxie": "HFJX2026ZQHH", "name": "\u6c47\u4e30\u664b\u4fe12026\u5468\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIFENGJINXIN2026ZHOUQIHUNHE"}, "540005": {"suoxie": "HFJXPWZLZDZA", "name": "\u6c47\u4e30\u664b\u4fe1\u5e73\u7a33\u589e\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIFENGJINXINPINGWENZENGLIZHONGDUANZHAIA"}, "540006": {"suoxie": "HFJXDPGPA", "name": "\u6c47\u4e30\u664b\u4fe1\u5927\u76d8\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINDAPANGUPIAOA"}, "540007": {"suoxie": "HFJXZXPGP", "name": "\u6c47\u4e30\u664b\u4fe1\u4e2d\u5c0f\u76d8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINZHONGXIAOPANGUPIAO"}, "540008": {"suoxie": "HFJXDTXFGPA", "name": "\u6c47\u4e30\u664b\u4fe1\u4f4e\u78b3\u5148\u950b\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINDITANXIANFENGGUPIAOA"}, "540009": {"suoxie": "HFJXXFHLGP", "name": "\u6c47\u4e30\u664b\u4fe1\u6d88\u8d39\u7ea2\u5229\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINXIAOFEIHONGLIGUPIAO"}, "540010": {"suoxie": "HFJXKJXFGP", "name": "\u6c47\u4e30\u664b\u4fe1\u79d1\u6280\u5148\u950b\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "HUIFENGJINXINKEJIXIANFENGGUPIAO"}, "540011": {"suoxie": "HFJXHBA", "name": "\u6c47\u4e30\u664b\u4fe1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUIFENGJINXINHUOBIA"}, "540012": {"suoxie": "HFJXHSLTZSA", "name": "\u6c47\u4e30\u664b\u4fe1\u6052\u751f\u9f99\u5934\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUIFENGJINXINHENGSHENGLONGTOUZHISHUA"}, "541001": {"suoxie": "HFJX2016ZQHH", "name": "\u6c47\u4e30\u664b\u4fe12016\u5468\u671f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIFENGJINXIN2016ZHOUQIHUNHE"}, "541002": {"suoxie": "HFJXLTHH", "name": "\u6c47\u4e30\u664b\u4fe1\u9f99\u817e\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUIFENGJINXINLONGTENGHUNHE"}, "541004": {"suoxie": "HFJX2026ZQHH", "name": "\u6c47\u4e30\u664b\u4fe12026\u5468\u671f\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "HUIFENGJINXIN2026ZHOUQIHUNHE"}, "541005": {"suoxie": "HFJXPWZLZDZC", "name": "\u6c47\u4e30\u664b\u4fe1\u5e73\u7a33\u589e\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HUIFENGJINXINPINGWENZENGLIZHONGDUANZHAIC"}, "541011": {"suoxie": "HFJXHBB", "name": "\u6c47\u4e30\u664b\u4fe1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUIFENGJINXINHUOBIB"}, "550001": {"suoxie": "XCSJHHH", "name": "\u4fe1\u8bda\u56db\u5b63\u7ea2\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGSIJIHONGHUNHE"}, "550002": {"suoxie": "ZXBCJCCZHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u7cbe\u8403\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGJINGCUICHENGZHANGHUNHE"}, "550003": {"suoxie": "ZXBCSSLCHH", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u76db\u4e16\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGSHENGSHILANCHOUHUNHE"}, "550004": {"suoxie": "XCSDYZQA", "name": "\u4fe1\u8bda\u4e09\u5f97\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINCHENGSANDEYIZHAIQUANA"}, "550005": {"suoxie": "XCSDYZQB", "name": "\u4fe1\u8bda\u4e09\u5f97\u76ca\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINCHENGSANDEYIZHAIQUANB"}, "550008": {"suoxie": "XCYSJXHH", "name": "\u4fe1\u8bda\u4f18\u80dc\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGYOUSHENGJINGXUANHUNHE"}, "550009": {"suoxie": "XCZXPHH", "name": "\u4fe1\u8bda\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGZHONGXIAOPANHUNHE"}, "550010": {"suoxie": "XCHBA", "name": "\u4fe1\u8bda\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGHUOBIA"}, "550011": {"suoxie": "XCHBB", "name": "\u4fe1\u8bda\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XINCHENGHUOBIB"}, "550012": {"suoxie": "ZXBCJHZQA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u534e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGHUAZHAIQUANA"}, "550013": {"suoxie": "ZXBCJHZQC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u666f\u534e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINBAOCHENGJINGHUAZHAIQUANC"}, "550015": {"suoxie": "ZXBCZYDLHHA", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u8fdc\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGZHIYUANDONGLIHUNHEA"}, "550016": {"suoxie": "ZXBCZYDLHHC", "name": "\u4e2d\u4fe1\u4fdd\u8bda\u81f3\u8fdc\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINBAOCHENGZHIYUANDONGLIHUNHEC"}, "550018": {"suoxie": "XCYZCZZQA", "name": "\u4fe1\u8bda\u4f18\u8d28\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGYOUZHICHUNZHAIZHAIQUANA"}, "550019": {"suoxie": "XCYZCZZQB", "name": "\u4fe1\u8bda\u4f18\u8d28\u7eaf\u503a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINCHENGYOUZHICHUNZHAIZHAIQUANB"}, "551001": {"suoxie": "XCSJHHH", "name": "\u4fe1\u8bda\u56db\u5b63\u7ea2\u6df7\u5408(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINCHENGSIJIHONGHUNHE"}, "560000": {"suoxie": "PYASZZZNDDQCETF", "name": "\u6d66\u94f6\u5b89\u76db\u4e2d\u8bc1\u667a\u80fd\u7535\u52a8\u6c7d\u8f66ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PUYINANSHENGZHONGZHENGZHINENGDIANDONGQICHEETF"}, "560002": {"suoxie": "YMHLCZHH", "name": "\u76ca\u6c11\u7ea2\u5229\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIMINHONGLICHENGZHANGHUNHE"}, "560003": {"suoxie": "YMCXYSHH", "name": "\u76ca\u6c11\u521b\u65b0\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "YIMINCHUANGXINYOUSHIHUNHE"}, "560006": {"suoxie": "YMHXZZHH", "name": "\u76ca\u6c11\u6838\u5fc3\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "YIMINHEXINZENGZHANGHUNHE"}, "560050": {"suoxie": "HTFMSCIZGA50HLHTETF", "name": "\u6c47\u6dfb\u5bccMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUITIANFUMSCIZHONGGUOA50HULIANHUTONGETF"}, "560100": {"suoxie": "NFZZ500ZQCLETF", "name": "\u5357\u65b9\u4e2d\u8bc1500\u589e\u5f3a\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGZHONGZHENG500ZENGQIANGCELUEETF"}, "560500": {"suoxie": "PYZZ500ZLCZETF", "name": "\u9e4f\u626c\u4e2d\u8bc1500\u8d28\u91cf\u6210\u957fETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENG500ZHILIANGCHENGZHANGETF"}, "560560": {"suoxie": "TKZZNDDTJJETF", "name": "\u6cf0\u5eb7\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4eETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "TAIKANGZHONGZHENGNEIDIDITANJINGJIETF"}, "560600": {"suoxie": "FZFBZZYYJYLQXCXETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u4e2d\u8bc1\u533b\u836f\u53ca\u533b\u7597\u5668\u68b0\u521b\u65b0ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGZHONGZHENGYIYAOJIYILIAOQIXIECHUANGXINETF"}, "560660": {"suoxie": "XHZZYJS50ETF", "name": "\u65b0\u534e\u4e2d\u8bc1\u4e91\u8ba1\u7b9750ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XINHUAZHONGZHENGYUNJISUAN50ETF"}, "560800": {"suoxie": "PYZZSZJJZTETF", "name": "\u9e4f\u626c\u4e2d\u8bc1\u6570\u5b57\u7ecf\u6d4e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PENGYANGZHONGZHENGSHUZIJINGJIZHUTIETF"}, "561100": {"suoxie": "FGZZXFDZZTETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "561120": {"suoxie": "FGZZQZJYDQETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u5168\u6307\u5bb6\u7528\u7535\u5668ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGQUANZHIJIAYONGDIANQIETF"}, "561130": {"suoxie": "FGZZXHSMZPPGCETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u65b0\u534e\u793e\u6c11\u65cf\u54c1\u724c\u5de5\u7a0bETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGXINHUASHEMINZUPINPAIGONGCHENGETF"}, "561300": {"suoxie": "GTHS300ZQCLETF", "name": "\u56fd\u6cf0\u6caa\u6df1300\u589e\u5f3a\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIHUSHEN300ZENGQIANGCELUEETF"}, "561310": {"suoxie": "GTZZXFDZZTETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "561350": {"suoxie": "GTZZ500ETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1500ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENG500ETF"}, "561500": {"suoxie": "HTBRZZQYHXJZL50ETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1\u4f01\u4e1a\u6838\u5fc3\u7ade\u4e89\u529b50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENGQIYEHEXINJINGZHENGLI50ETF"}, "561550": {"suoxie": "HTBRZZ500ZQCLETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e2d\u8bc1500\u589e\u5f3a\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUIZHONGZHENG500ZENGQIANGCELUEETF"}, "561600": {"suoxie": "PAZZXFDZZTETF", "name": "\u5e73\u5b89\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "561800": {"suoxie": "HFZZXYJSZTETF", "name": "\u534e\u5bcc\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAFUZHONGZHENGXIYOUJINSHUZHUTIETF"}, "561900": {"suoxie": "ZSHS300ESGJZETF", "name": "\u62db\u5546\u6caa\u6df1300ESG\u57fa\u51c6ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300ESGJIZHUNETF"}, "561910": {"suoxie": "ZSZZDCZTETF", "name": "\u62db\u5546\u4e2d\u8bc1\u7535\u6c60\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGDIANCHIZHUTIETF"}, "561990": {"suoxie": "ZSHS300ZQCLETF", "name": "\u62db\u5546\u6caa\u6df1300\u589e\u5f3a\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGHUSHEN300ZENGQIANGCELUEETF"}, "562300": {"suoxie": "YHZZNDDTJJZTETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u5185\u5730\u4f4e\u78b3\u7ecf\u6d4e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGNEIDIDITANJINGJIZHUTIETF"}, "562360": {"suoxie": "YHZZJQRETF", "name": "\u94f6\u534e\u4e2d\u8bc1\u673a\u5668\u4ebaETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YINHUAZHONGZHENGJIQIRENETF"}, "562500": {"suoxie": "HXZZJQRETF", "name": "\u534e\u590f\u4e2d\u8bc1\u673a\u5668\u4ebaETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGJIQIRENETF"}, "562510": {"suoxie": "HXZZLYZTETF", "name": "\u534e\u590f\u4e2d\u8bc1\u65c5\u6e38\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGLVYOUZHUTIETF"}, "562520": {"suoxie": "HXZZZX1000CZCXCLETF", "name": "\u534e\u590f\u4e2d\u8bc1\u667a\u90091000\u6210\u957f\u521b\u65b0\u7b56\u7565ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIAZHONGZHENGZHIXUAN1000CHENGZHANGCHUANGXINCELUEETF"}, "562800": {"suoxie": "JSZZXYJSZTETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7a00\u6709\u91d1\u5c5e\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGXIYOUJINSHUZHUTIETF"}, "562880": {"suoxie": "JSZZDCZTETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u7535\u6c60\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGDIANCHIZHUTIETF"}, "562900": {"suoxie": "YFDZZXDNYZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u73b0\u4ee3\u519c\u4e1a\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXIANDAINONGYEZHUTIETF"}, "562910": {"suoxie": "YFDZZZBCYETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u88c5\u5907\u4ea7\u4e1aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGZHUANGBEICHANYEETF"}, "562950": {"suoxie": "YFDZZXFDZZTETF", "name": "\u6613\u65b9\u8fbe\u4e2d\u8bc1\u6d88\u8d39\u7535\u5b50\u4e3b\u9898ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAZHONGZHENGXIAOFEIDIANZIZHUTIETF"}, "563000": {"suoxie": "YFDMSCIZGA50HLHTETF", "name": "\u6613\u65b9\u8fbeMSCI\u4e2d\u56fdA50\u4e92\u8054\u4e92\u901aETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDAMSCIZHONGGUOA50HULIANHUTONGETF"}, "570001": {"suoxie": "NDJZYSHH", "name": "\u8bfa\u5fb7\u4ef7\u503c\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEJIAZHIYOUSHIHUNHE"}, "570005": {"suoxie": "NDCZYSHH", "name": "\u8bfa\u5fb7\u6210\u957f\u4f18\u52bf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODECHENGZHANGYOUSHIHUNHE"}, "570006": {"suoxie": "NDZXPHH", "name": "\u8bfa\u5fb7\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEZHONGXIAOPANHUNHE"}, "570007": {"suoxie": "NDYX30HH", "name": "\u8bfa\u5fb7\u4f18\u900930\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEYOUXUAN30HUNHE"}, "570008": {"suoxie": "NDZQCLHH", "name": "\u8bfa\u5fb7\u5468\u671f\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NUODEZHOUQICELUEHUNHE"}, "571002": {"suoxie": "NDLHPZHH", "name": "\u8bfa\u5fb7\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "NUODELINGHUOPEIZHIHUNHE"}, "573003": {"suoxie": "NDZQSYZQ", "name": "\u8bfa\u5fb7\u589e\u5f3a\u6536\u76ca\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NUODEZENGQIANGSHOUYIZHAIQUAN"}, "580001": {"suoxie": "DWJHYSJXHH", "name": "\u4e1c\u5434\u5609\u79be\u4f18\u52bf\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUJIAHEYOUSHIJINGXUANHUNHE"}, "580002": {"suoxie": "DWSDLHHA", "name": "\u4e1c\u5434\u53cc\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUSHUANGDONGLIHUNHEA"}, "580003": {"suoxie": "DWHYLDHHA", "name": "\u4e1c\u5434\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUHANGYELUNDONGHUNHEA"}, "580005": {"suoxie": "DWJQCLHHA", "name": "\u4e1c\u5434\u8fdb\u53d6\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUJINQUCELUEHUNHEA"}, "580006": {"suoxie": "DWXJJHHA", "name": "\u4e1c\u5434\u65b0\u7ecf\u6d4e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUXINJINGJIHUNHEA"}, "580007": {"suoxie": "DWAXLHHHA", "name": "\u4e1c\u5434\u5b89\u4eab\u91cf\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUANXIANGLIANGHUAHUNHEA"}, "580008": {"suoxie": "DWXCYJXGPA", "name": "\u4e1c\u5434\u65b0\u4ea7\u4e1a\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "DONGWUXINCHANYEJINGXUANGUPIAOA"}, "580009": {"suoxie": "DWDCLHHA", "name": "\u4e1c\u5434\u591a\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUDUOCELUEHUNHEA"}, "581002": {"suoxie": "DWSDLHHA", "name": "\u4e1c\u5434\u53cc\u52a8\u529b\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGWUSHUANGDONGLIHUNHEA"}, "582003": {"suoxie": "DWPZYHHHA", "name": "\u4e1c\u5434\u914d\u7f6e\u4f18\u5316\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUPEIZHIYOUHUAHUNHEA"}, "583001": {"suoxie": "DWHBA", "name": "\u4e1c\u5434\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGWUHUOBIA"}, "583101": {"suoxie": "DWHBB", "name": "\u4e1c\u5434\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "DONGWUHUOBIB"}, "585001": {"suoxie": "DWZZXXZS", "name": "\u4e1c\u5434\u4e2d\u8bc1\u65b0\u5174\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "DONGWUZHONGZHENGXINXINGZHISHU"}, "588000": {"suoxie": "HXSZKCB50CFETF", "name": "\u534e\u590f\u4e0a\u8bc1\u79d1\u521b\u677f50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAXIASHANGZHENGKECHUANGBAN50CHENGFENETF"}, "588050": {"suoxie": "GYSZKC50CFETF", "name": "\u5de5\u94f6\u4e0a\u8bc1\u79d1\u521b50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GONGYINSHANGZHENGKECHUANG50CHENGFENETF"}, "588060": {"suoxie": "GFSZKCB50CFETF", "name": "\u5e7f\u53d1\u4e0a\u8bc1\u79d1\u521b\u677f50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUANGFASHANGZHENGKECHUANGBAN50CHENGFENETF"}, "588080": {"suoxie": "YFDSZKCB50CFETF", "name": "\u6613\u65b9\u8fbe\u4e0a\u8bc1\u79d1\u521b\u677f50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "YIFANGDASHANGZHENGKECHUANGBAN50CHENGFENETF"}, "588090": {"suoxie": "HTBRSZKCB50CFETF", "name": "\u534e\u6cf0\u67cf\u745e\u4e0a\u8bc1\u79d1\u521b\u677f50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUATAIBAIRUISHANGZHENGKECHUANGBAN50CHENGFENETF"}, "588150": {"suoxie": "NFSZKCB50ETF", "name": "\u5357\u65b9\u4e0a\u8bc1\u79d1\u521b\u677f50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NANFANGSHANGZHENGKECHUANGBAN50ETF"}, "588180": {"suoxie": "GLASZKCB50CFETF", "name": "\u56fd\u8054\u5b89\u4e0a\u8bc1\u79d1\u521b\u677f50\u6210\u4efdETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOLIANANSHANGZHENGKECHUANGBAN50CHENGFENETF"}, "588280": {"suoxie": "HASZKCB50ETF", "name": "\u534e\u5b89\u4e0a\u8bc1\u79d1\u521b\u677f50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUAANSHANGZHENGKECHUANGBAN50ETF"}, "588300": {"suoxie": "ZSZZKCCY50ETF", "name": "\u62db\u5546\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHAOSHANGZHONGZHENGKECHUANGCHUANGYE50ETF"}, "588310": {"suoxie": "FZFBKCCY50ETF", "name": "\u65b9\u6b63\u5bcc\u90a6\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FANGZHENGFUBANGKECHUANGCHUANGYE50ETF"}, "588330": {"suoxie": "HBSCLTETF", "name": "\u534e\u5b9d\u53cc\u521b\u9f99\u5934ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "HUABAOSHUANGCHUANGLONGTOUETF"}, "588360": {"suoxie": "GTZZKCCY50ETF", "name": "\u56fd\u6cf0\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "GUOTAIZHONGZHENGKECHUANGCHUANGYE50ETF"}, "588380": {"suoxie": "FGZZKCCY50ETF", "name": "\u5bcc\u56fd\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "FUGUOZHONGZHENGKECHUANGCHUANGYE50ETF"}, "588390": {"suoxie": "BSZZKCCY50ETF", "name": "\u535a\u65f6\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "BOSHIZHONGZHENGKECHUANGCHUANGYE50ETF"}, "588400": {"suoxie": "JSZZKCCY50ETF", "name": "\u5609\u5b9e\u4e2d\u8bc1\u79d1\u521b\u521b\u4e1a50ETF", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "JIASHIZHONGZHENGKECHUANGCHUANGYE50ETF"}, "590001": {"suoxie": "ZYHXYXHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUHEXINYOUXUANHUNHE"}, "590002": {"suoxie": "ZYHXCZHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUHEXINCHENGZHANGHUNHE"}, "590003": {"suoxie": "ZYHXYSLHPZHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u4f18\u52bf\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUHEXINYOUSHILINGHUOPEIZHIHUNHE"}, "590005": {"suoxie": "ZYHXZTHH", "name": "\u4e2d\u90ae\u6838\u5fc3\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUHEXINZHUTIHUNHE"}, "590006": {"suoxie": "ZYZXPLHPZHH", "name": "\u4e2d\u90ae\u4e2d\u5c0f\u76d8\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGYOUZHONGXIAOPANLINGHUOPEIZHIHUNHE"}, "590007": {"suoxie": "ZYZZ500ZSZQA", "name": "\u4e2d\u90ae\u4e2d\u8bc1500\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "ZHONGYOUZHONGZHENG500ZHISHUZENGQIANGA"}, "590008": {"suoxie": "ZYZLXXCYHH", "name": "\u4e2d\u90ae\u6218\u7565\u65b0\u5174\u4ea7\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGYOUZHANLUEXINXINGCHANYEHUNHE"}, "590009": {"suoxie": "ZYWDSYZQA", "name": "\u4e2d\u90ae\u7a33\u5b9a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUWENDINGSHOUYIZHAIQUANA"}, "590010": {"suoxie": "ZYWDSYZQC", "name": "\u4e2d\u90ae\u7a33\u5b9a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGYOUWENDINGSHOUYIZHAIQUANC"}, "610001": {"suoxie": "XDAYLXZZHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u9886\u5148\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINLINGXIANZENGZHANGHUNHE"}, "610002": {"suoxie": "XDAYJHPZHHA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u7cbe\u534e\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAAOYINJINGHUAPEIZHIHUNHEA"}, "610004": {"suoxie": "XDAYZXPHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINZHONGXIAOPANHUNHE"}, "610005": {"suoxie": "XDAYHLHBHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u7ea2\u5229\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINHONGLIHUIBAOHUNHE"}, "610006": {"suoxie": "XDAYCYSJHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINCHANYESHENGJIHUNHE"}, "610007": {"suoxie": "XDAYXFYXHH", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u6d88\u8d39\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINDAAOYINXIAOFEIYOUXUANHUNHE"}, "610008": {"suoxie": "XDAYXYZZQA", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4fe1\u7528\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAAOYINXINYONGZHAIZHAIQUANA"}, "610108": {"suoxie": "XDAYXYZZQC", "name": "\u4fe1\u8fbe\u6fb3\u94f6\u4fe1\u7528\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAAOYINXINYONGZHAIZHAIQUANC"}, "620001": {"suoxie": "JYSABSDLHH", "name": "\u91d1\u5143\u987a\u5b89\u5b9d\u77f3\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "JINYUANSHUNANBAOSHIDONGLIHUNHE"}, "620002": {"suoxie": "JYSACZDLHH", "name": "\u91d1\u5143\u987a\u5b89\u6210\u957f\u52a8\u529b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYUANSHUNANCHENGZHANGDONGLIHUNHE"}, "620003": {"suoxie": "JYSAFLZQ", "name": "\u91d1\u5143\u987a\u5b89\u4e30\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "JINYUANSHUNANFENGLIZHAIQUAN"}, "620004": {"suoxie": "JYSAJZZZHH", "name": "\u91d1\u5143\u987a\u5b89\u4ef7\u503c\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYUANSHUNANJIAZHIZENGZHANGHUNHE"}, "620006": {"suoxie": "JYSAXFZTHH", "name": "\u91d1\u5143\u987a\u5b89\u6d88\u8d39\u4e3b\u9898\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "JINYUANSHUNANXIAOFEIZHUTIHUNHE"}, "620007": {"suoxie": "JYSAYZJXHHA", "name": "\u91d1\u5143\u987a\u5b89\u4f18\u8d28\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "JINYUANSHUNANYOUZHIJINGXUANHUNHEA"}, "620009": {"suoxie": "JYSAFXZQ", "name": "\u91d1\u5143\u987a\u5b89\u4e30\u7965\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "JINYUANSHUNANFENGXIANGZHAIQUAN"}, "620010": {"suoxie": "JYSAJYBHBA", "name": "\u91d1\u5143\u987a\u5b89\u91d1\u5143\u5b9d\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYUANSHUNANJINYUANBAOHUOBIA"}, "620011": {"suoxie": "JYSAJYBHBB", "name": "\u91d1\u5143\u987a\u5b89\u91d1\u5143\u5b9d\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "JINYUANSHUNANJINYUANBAOHUOBIB"}, "630001": {"suoxie": "HSLXQYHH", "name": "\u534e\u5546\u9886\u5148\u4f01\u4e1a\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGLINGXIANQIYEHUNHE"}, "630002": {"suoxie": "HSSSCZHH", "name": "\u534e\u5546\u76db\u4e16\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGSHENGSHICHENGZHANGHUNHE"}, "630003": {"suoxie": "HSSYZQZQA", "name": "\u534e\u5546\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGSHOUYIZENGQIANGZHAIQUANA"}, "630005": {"suoxie": "HSDTAEFHH", "name": "\u534e\u5546\u52a8\u6001\u963f\u5c14\u6cd5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGDONGTAIAERFAHUNHE"}, "630006": {"suoxie": "HSCYSJHH", "name": "\u534e\u5546\u4ea7\u4e1a\u5347\u7ea7\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGCHANYESHENGJIHUNHE"}, "630007": {"suoxie": "HSWJSLZQA", "name": "\u534e\u5546\u7a33\u5065\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGWENJIANSHUANGLIZHAIQUANA"}, "630008": {"suoxie": "HSCLJXHH", "name": "\u534e\u5546\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGCELUEJINGXUANHUNHE"}, "630009": {"suoxie": "HSWDZLZQA", "name": "\u534e\u5546\u7a33\u5b9a\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGWENDINGZENGLIZHAIQUANA"}, "630010": {"suoxie": "HSJZJXHH", "name": "\u534e\u5546\u4ef7\u503c\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGJIAZHIJINGXUANHUNHE"}, "630011": {"suoxie": "HSZTJXHH", "name": "\u534e\u5546\u4e3b\u9898\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HUASHANGZHUTIJINGXUANHUNHE"}, "630012": {"suoxie": "HSXJZLHBA", "name": "\u534e\u5546\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "HUASHANGXIANJINZENGLIHUOBIA"}, "630015": {"suoxie": "HSDPLHJXHH", "name": "\u534e\u5546\u5927\u76d8\u91cf\u5316\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGDAPANLIANGHUAJINGXUANHUNHE"}, "630016": {"suoxie": "HSJZGXHHFQS", "name": "\u534e\u5546\u4ef7\u503c\u5171\u4eab\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUASHANGJIAZHIGONGXIANGHUNHEFAQISHI"}, "630103": {"suoxie": "HSSYZQZQB", "name": "\u534e\u5546\u6536\u76ca\u589e\u5f3a\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGSHOUYIZENGQIANGZHAIQUANB"}, "630107": {"suoxie": "HSWJSLZQB", "name": "\u534e\u5546\u7a33\u5065\u53cc\u5229\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGWENJIANSHUANGLIZHAIQUANB"}, "630109": {"suoxie": "HSWDZLZQC", "name": "\u534e\u5546\u7a33\u5b9a\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUASHANGWENDINGZENGLIZHAIQUANC"}, "630112": {"suoxie": "HSXJZLHBB", "name": "\u534e\u5546\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "HUASHANGXIANJINZENGLIHUOBIB"}, "650001": {"suoxie": "YDCZZQA", "name": "\u82f1\u5927\u7eaf\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDACHUNZHAIZHAIQUANA"}, "650002": {"suoxie": "YDCZZQC", "name": "\u82f1\u5927\u7eaf\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "YINGDACHUNZHAIZHAIQUANC"}, "660001": {"suoxie": "NYHYCZHH", "name": "\u519c\u94f6\u884c\u4e1a\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHANGYECHENGZHANGHUNHE"}, "660002": {"suoxie": "NYHJZLZQA", "name": "\u519c\u94f6\u6052\u4e45\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NONGYINHENGJIUZENGLIZHAIQUANA"}, "660003": {"suoxie": "NYPHSLHH", "name": "\u519c\u94f6\u5e73\u8861\u53cc\u5229\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "NONGYINPINGHENGSHUANGLIHUNHE"}, "660004": {"suoxie": "NYCLJZHH", "name": "\u519c\u94f6\u7b56\u7565\u4ef7\u503c\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINCELUEJIAZHIHUNHE"}, "660005": {"suoxie": "NYZXPHH", "name": "\u519c\u94f6\u4e2d\u5c0f\u76d8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINZHONGXIAOPANHUNHE"}, "660006": {"suoxie": "NYDPLCHH", "name": "\u519c\u94f6\u5927\u76d8\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINDAPANLANCHOUHUNHE"}, "660007": {"suoxie": "NYHBA", "name": "\u519c\u94f6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHUOBIA"}, "660008": {"suoxie": "NYHS300ZSA", "name": "\u519c\u94f6\u6caa\u6df1300\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NONGYINHUSHEN300ZHISHUA"}, "660009": {"suoxie": "NYZQSYZQA", "name": "\u519c\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NONGYINZENGQIANGSHOUYIZHAIQUANA"}, "660010": {"suoxie": "NYCLJXHH", "name": "\u519c\u94f6\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINCELUEJINGXUANHUNHE"}, "660011": {"suoxie": "NYZZ500ZS", "name": "\u519c\u94f6\u4e2d\u8bc1500\u6307\u6570", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "NONGYINZHONGZHENG500ZHISHU"}, "660012": {"suoxie": "NYXFZTHHA", "name": "\u519c\u94f6\u6d88\u8d39\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINXIAOFEIZHUTIHUNHEA"}, "660015": {"suoxie": "NYHYLDHH", "name": "\u519c\u94f6\u884c\u4e1a\u8f6e\u52a8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "NONGYINHANGYELUNDONGHUNHE"}, "660016": {"suoxie": "NYHLJJGDJZQ", "name": "\u519c\u94f6\u6c47\u7406\u91d1\u805a\u9ad8\u7b49\u7ea7\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "NONGYINHUILIJINJUGAODENGJIZHAIQUAN"}, "660102": {"suoxie": "NYHJZLZQC", "name": "\u519c\u94f6\u6052\u4e45\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NONGYINHENGJIUZENGLIZHAIQUANC"}, "660107": {"suoxie": "NYHBB", "name": "\u519c\u94f6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "NONGYINHUOBIB"}, "660109": {"suoxie": "NYZQSYZQC", "name": "\u519c\u94f6\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "NONGYINZENGQIANGSHOUYIZHAIQUANC"}, "671010": {"suoxie": "XBLDCLYXHHA", "name": "\u897f\u90e8\u5229\u5f97\u7b56\u7565\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XIBULIDECELUEYOUXUANHUNHEA"}, "671030": {"suoxie": "XBLDSJQDGP", "name": "\u897f\u90e8\u5229\u5f97\u4e8b\u4ef6\u9a71\u52a8\u80a1\u7968", "style": "\u80a1\u7968\u578b", "pinyin": "XIBULIDESHIJIANQUDONGGUPIAO"}, "673010": {"suoxie": "XBLDXDXHH", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u52a8\u5411\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINDONGXIANGHUNHE"}, "673020": {"suoxie": "XBLDCZJXHH", "name": "\u897f\u90e8\u5229\u5f97\u6210\u957f\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDECHENGZHANGJINGXUANHUNHE"}, "673030": {"suoxie": "XBLDDCLYXHH", "name": "\u897f\u90e8\u5229\u5f97\u591a\u7b56\u7565\u4f18\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEDUOCELUEYOUXUANHUNHE"}, "673040": {"suoxie": "XBLDHYZTYXHHA", "name": "\u897f\u90e8\u5229\u5f97\u884c\u4e1a\u4e3b\u9898\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEHANGYEZHUTIYOUXUANHUNHEA"}, "673043": {"suoxie": "XBLDHYZTYXHHC", "name": "\u897f\u90e8\u5229\u5f97\u884c\u4e1a\u4e3b\u9898\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEHANGYEZHUTIYOUXUANHUNHEC"}, "673050": {"suoxie": "XBLDXYHH", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u76c8\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINYINGHUNHE"}, "673060": {"suoxie": "XBLDJRLHPZHHA", "name": "\u897f\u90e8\u5229\u5f97\u666f\u745e\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJINGRUILINGHUOPEIZHIHUNHEA"}, "673071": {"suoxie": "XBLDXDLHHA", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINDONGLIHUNHEA"}, "673073": {"suoxie": "XBLDXDLHHC", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINDONGLIHUNHEC"}, "673081": {"suoxie": "XBLDXYHHA", "name": "\u897f\u90e8\u5229\u5f97\u7965\u8fd0\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXIANGYUNHUNHEA"}, "673083": {"suoxie": "XBLDXYHHC", "name": "\u897f\u90e8\u5229\u5f97\u7965\u8fd0\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXIANGYUNHUNHEC"}, "673090": {"suoxie": "XBLDGGJXGPA", "name": "\u897f\u90e8\u5229\u5f97\u4e2a\u80a1\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "XIBULIDEGEGUJINGXUANGUPIAOA"}, "673100": {"suoxie": "XBLDHS300ZSZQA", "name": "\u897f\u90e8\u5229\u5f97\u6caa\u6df1300\u6307\u6570\u589e\u5f3aA", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEHUSHEN300ZHISHUZENGQIANGA"}, "673101": {"suoxie": "XBLDHS300ZSZQC", "name": "\u897f\u90e8\u5229\u5f97\u6caa\u6df1300\u6307\u6570\u589e\u5f3aC", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "XIBULIDEHUSHEN300ZHISHUZENGQIANGC"}, "673110": {"suoxie": "XBLDXRHH", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u6da6\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINRUNHUNHE"}, "673120": {"suoxie": "XBLDXFHHA", "name": "\u897f\u90e8\u5229\u5f97\u65b0\u5bcc\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEXINFUHUNHEA"}, "673141": {"suoxie": "XBLDJCHHA", "name": "\u897f\u90e8\u5229\u5f97\u666f\u7a0b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJINGCHENGHUNHEA"}, "673143": {"suoxie": "XBLDJCHHC", "name": "\u897f\u90e8\u5229\u5f97\u666f\u7a0b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XIBULIDEJINGCHENGHUNHEC"}, "675011": {"suoxie": "XBLDWJSLZQA", "name": "\u897f\u90e8\u5229\u5f97\u7a33\u5065\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEWENJIANSHUANGLIZHAIQUANA"}, "675013": {"suoxie": "XBLDWJSLZQC", "name": "\u897f\u90e8\u5229\u5f97\u7a33\u5065\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEWENJIANSHUANGLIZHAIQUANC"}, "675031": {"suoxie": "XBLDTTXHBA", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u946b\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANXINHUOBIA"}, "675032": {"suoxie": "XBLDTTXHBB", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u946b\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANXINHUOBIB"}, "675041": {"suoxie": "XBLDHXA", "name": "\u897f\u90e8\u5229\u5f97\u5408\u4eabA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHEXIANGA"}, "675043": {"suoxie": "XBLDHXC", "name": "\u897f\u90e8\u5229\u5f97\u5408\u4eabC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHEXIANGC"}, "675051": {"suoxie": "XBLDHYZQA", "name": "\u897f\u90e8\u5229\u5f97\u5408\u8d62\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHEYINGZHAIQUANA"}, "675053": {"suoxie": "XBLDHYZQC", "name": "\u897f\u90e8\u5229\u5f97\u5408\u8d62\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHEYINGZHAIQUANC"}, "675061": {"suoxie": "XBLDTTFHBA", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u5bcc\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANFUHUOBIA"}, "675062": {"suoxie": "XBLDTTFHBB", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u5bcc\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANFUHUOBIB"}, "675071": {"suoxie": "XBLDTTJHBA", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u91d1\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANJINHUOBIA"}, "675072": {"suoxie": "XBLDTTJHBB", "name": "\u897f\u90e8\u5229\u5f97\u5929\u6dfb\u91d1\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "XIBULIDETIANTIANJINHUOBIB"}, "675081": {"suoxie": "XBLDXYZQA", "name": "\u897f\u90e8\u5229\u5f97\u7965\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEXIANGYINGZHAIQUANA"}, "675083": {"suoxie": "XBLDXYZQC", "name": "\u897f\u90e8\u5229\u5f97\u7965\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBULIDEXIANGYINGZHAIQUANC"}, "675091": {"suoxie": "XBLDXYZQA", "name": "\u897f\u90e8\u5229\u5f97\u7965\u9038\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEXIANGYIZHAIQUANA"}, "675093": {"suoxie": "XBLDXYZQC", "name": "\u897f\u90e8\u5229\u5f97\u7965\u9038\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEXIANGYIZHAIQUANC"}, "675100": {"suoxie": "XBLDDZCZA", "name": "\u897f\u90e8\u5229\u5f97\u5f97\u5c0a\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEDEZUNCHUNZHAIA"}, "675111": {"suoxie": "XBLDHXZQA", "name": "\u897f\u90e8\u5229\u5f97\u6c47\u4eab\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHUIXIANGZHAIQUANA"}, "675113": {"suoxie": "XBLDHXZQC", "name": "\u897f\u90e8\u5229\u5f97\u6c47\u4eab\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHUIXIANGZHAIQUANC"}, "675121": {"suoxie": "XBHYZQA", "name": "\u897f\u90e8\u6c47\u9038\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBUHUIYIZHAIQUANA"}, "675123": {"suoxie": "XBHYZQC", "name": "\u897f\u90e8\u6c47\u9038\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XIBUHUIYIZHAIQUANC"}, "675161": {"suoxie": "XBLDHYZQA", "name": "\u897f\u90e8\u5229\u5f97\u6c47\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHUIYINGZHAIQUANA"}, "675163": {"suoxie": "XBLDHYZQC", "name": "\u897f\u90e8\u5229\u5f97\u6c47\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XIBULIDEHUIYINGZHAIQUANC"}, "686868": {"suoxie": "ZSJYCZA", "name": "\u6d59\u5546\u805a\u76c8\u7eaf\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGJUYINGCHUNZHAIA"}, "686869": {"suoxie": "ZSJYCZC", "name": "\u6d59\u5546\u805a\u76c8\u7eaf\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHESHANGJUYINGCHUNZHAIC"}, "688888": {"suoxie": "ZSJCCYCZHHA", "name": "\u6d59\u5546\u805a\u6f6e\u4ea7\u4e1a\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHESHANGJUCHAOCHANYECHENGZHANGHUNHEA"}, "690001": {"suoxie": "MSJYPPLCHH", "name": "\u6c11\u751f\u52a0\u94f6\u54c1\u724c\u84dd\u7b79\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINPINPAILANCHOUHUNHE"}, "690002": {"suoxie": "MSZQSYZQA", "name": "\u6c11\u751f\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGZENGQIANGSHOUYIZHAIQUANA"}, "690003": {"suoxie": "MSJYJXHH", "name": "\u6c11\u751f\u52a0\u94f6\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINJINGXUANHUNHE"}, "690004": {"suoxie": "MSJYWJCZHH", "name": "\u6c11\u751f\u52a0\u94f6\u7a33\u5065\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINWENJIANCHENGZHANGHUNHE"}, "690005": {"suoxie": "MSJYNXZZHH", "name": "\u6c11\u751f\u52a0\u94f6\u5185\u9700\u589e\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINNEIXUZENGZHANGHUNHE"}, "690006": {"suoxie": "MSJYXYSLZQA", "name": "\u6c11\u751f\u52a0\u94f6\u4fe1\u7528\u53cc\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGJIAYINXINYONGSHUANGLIZHAIQUANA"}, "690007": {"suoxie": "MSJYJQHYHHA", "name": "\u6c11\u751f\u52a0\u94f6\u666f\u6c14\u884c\u4e1a\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "MINSHENGJIAYINJINGQIHANGYEHUNHEA"}, "690008": {"suoxie": "MSZZNDZYZTZSA", "name": "\u6c11\u751f\u4e2d\u8bc1\u5185\u5730\u8d44\u6e90\u4e3b\u9898\u6307\u6570A", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "MINSHENGZHONGZHENGNEIDIZIYUANZHUTIZHISHUA"}, "690009": {"suoxie": "MSJYHLHBHH", "name": "\u6c11\u751f\u52a0\u94f6\u7ea2\u5229\u56de\u62a5\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINHONGLIHUIBAOHUNHE"}, "690010": {"suoxie": "MSJYXJZLHBA", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u589e\u5229\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINZENGLIHUOBIA"}, "690011": {"suoxie": "MSJYJJCZHHFQS", "name": "\u6c11\u751f\u52a0\u94f6\u79ef\u6781\u6210\u957f\u6df7\u5408\u53d1\u8d77\u5f0f", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "MINSHENGJIAYINJIJICHENGZHANGHUNHEFAQISHI"}, "690012": {"suoxie": "MSJYFXZQ", "name": "\u6c11\u751f\u52a0\u94f6\u4e30\u946b\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "MINSHENGJIAYINFENGXINZHAIQUAN"}, "690202": {"suoxie": "MSZQSYZQC", "name": "\u6c11\u751f\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGZENGQIANGSHOUYIZHAIQUANC"}, "690206": {"suoxie": "MSJYXYSLZQC", "name": "\u6c11\u751f\u52a0\u94f6\u4fe1\u7528\u53cc\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "MINSHENGJIAYINXINYONGSHUANGLIZHAIQUANC"}, "690210": {"suoxie": "MSJYXJZLHBB", "name": "\u6c11\u751f\u52a0\u94f6\u73b0\u91d1\u589e\u5229\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "MINSHENGJIAYINXIANJINZENGLIHUOBIB"}, "700001": {"suoxie": "PAHYXFHH", "name": "\u5e73\u5b89\u884c\u4e1a\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "PINGANHANGYEXIANFENGHUNHE"}, "700002": {"suoxie": "PASZ300ZSZQ", "name": "\u5e73\u5b89\u6df1\u8bc1300\u6307\u6570\u589e\u5f3a", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "PINGANSHENZHENG300ZHISHUZENGQIANG"}, "700003": {"suoxie": "PACLXFHH", "name": "\u5e73\u5b89\u7b56\u7565\u5148\u950b\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANCELUEXIANFENGHUNHE"}, "700004": {"suoxie": "PALHPZHHA", "name": "\u5e73\u5b89\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "PINGANLINGHUOPEIZHIHUNHEA"}, "700005": {"suoxie": "PATLZQA", "name": "\u5e73\u5b89\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANTIANLIZHAIQUANA"}, "700006": {"suoxie": "PATLZQC", "name": "\u5e73\u5b89\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "PINGANTIANLIZHAIQUANC"}, "710001": {"suoxie": "FADYSCZHH", "name": "\u5bcc\u5b89\u8fbe\u4f18\u52bf\u6210\u957f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FUANDAYOUSHICHENGZHANGHUNHE"}, "710002": {"suoxie": "FADCLJXHH", "name": "\u5bcc\u5b89\u8fbe\u7b56\u7565\u7cbe\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "FUANDACELUEJINGXUANHUNHE"}, "710301": {"suoxie": "FADZQSYZQA", "name": "\u5bcc\u5b89\u8fbe\u589e\u5f3a\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUANDAZENGQIANGSHOUYIZHAIQUANA"}, "710302": {"suoxie": "FADZQSYZQC", "name": "\u5bcc\u5b89\u8fbe\u589e\u5f3a\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FUANDAZENGQIANGSHOUYIZHAIQUANC"}, "710501": {"suoxie": "FADXJTHBA", "name": "\u5bcc\u5b89\u8fbe\u73b0\u91d1\u901a\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FUANDAXIANJINTONGHUOBIA"}, "710502": {"suoxie": "FADXJTHBB", "name": "\u5bcc\u5b89\u8fbe\u73b0\u91d1\u901a\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FUANDAXIANJINTONGHUOBIB"}, "720001": {"suoxie": "CTJZDLHH", "name": "\u8d22\u901a\u4ef7\u503c\u52a8\u91cf\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CAITONGJIAZHIDONGLIANGHUNHE"}, "720002": {"suoxie": "CTKZZZQA", "name": "\u8d22\u901a\u53ef\u8f6c\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u53ef\u8f6c\u503a", "pinyin": "CAITONGKEZHUANZHAIZHAIQUANA"}, "720003": {"suoxie": "CTSYZQZQA", "name": "\u8d22\u901a\u6536\u76ca\u589e\u5f3a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CAITONGSHOUYIZENGQIANGZHAIQUANA"}, "730001": {"suoxie": "FZFBCXDLHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u521b\u65b0\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGCHUANGXINDONGLIHUNHEA"}, "730002": {"suoxie": "FZFBHLJXHHA", "name": "\u65b9\u6b63\u5bcc\u90a6\u7ea2\u5229\u7cbe\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGFUBANGHONGLIJINGXUANHUNHEA"}, "730003": {"suoxie": "FZFBHBA", "name": "\u65b9\u6b63\u5bcc\u90a6\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "FANGZHENGFUBANGHUOBIA"}, "730103": {"suoxie": "FZFBHBB", "name": "\u65b9\u6b63\u5bcc\u90a6\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "FANGZHENGFUBANGHUOBIB"}, "740001": {"suoxie": "CAHGCLHH", "name": "\u957f\u5b89\u5b8f\u89c2\u7b56\u7565\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGANHONGGUANCELUEHUNHE"}, "740101": {"suoxie": "CAHS300FZQ", "name": "\u957f\u5b89\u6caa\u6df1300\u975e\u5468\u671f", "style": "\u6307\u6570\u578b-\u80a1\u7968", "pinyin": "CHANGANHUSHEN300FEIZHOUQI"}, "740601": {"suoxie": "CAHBA", "name": "\u957f\u5b89\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGANHUOBIA"}, "740602": {"suoxie": "CAHBB", "name": "\u957f\u5b89\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "CHANGANHUOBIB"}, "750001": {"suoxie": "AXLHPZHH", "name": "\u5b89\u4fe1\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINLINGHUOPEIZHIHUNHE"}, "750002": {"suoxie": "AXMBSYZQA", "name": "\u5b89\u4fe1\u76ee\u6807\u6536\u76ca\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINMUBIAOSHOUYIZHAIQUANA"}, "750003": {"suoxie": "AXMBSYZQC", "name": "\u5b89\u4fe1\u76ee\u6807\u6536\u76ca\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINMUBIAOSHOUYIZHAIQUANC"}, "750005": {"suoxie": "AXPWZZHHFQA", "name": "\u5b89\u4fe1\u5e73\u7a33\u589e\u957f\u6df7\u5408\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ANXINPINGWENZENGZHANGHUNHEFAQIA"}, "750006": {"suoxie": "AXXJGLHBA", "name": "\u5b89\u4fe1\u73b0\u91d1\u7ba1\u7406\u8d27\u5e01A", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINXIANJINGUANLIHUOBIA"}, "750007": {"suoxie": "AXXJGLHBB", "name": "\u5b89\u4fe1\u73b0\u91d1\u7ba1\u7406\u8d27\u5e01B", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINXIANJINGUANLIHUOBIB"}, "762001": {"suoxie": "GJGXFQA", "name": "\u56fd\u91d1\u56fd\u946b\u53d1\u8d77A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUOJINGUOXINFAQIA"}, "770001": {"suoxie": "DBYHHH", "name": "\u5fb7\u90a6\u4f18\u5316\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DEBANGYOUHUAHUNHE"}, "850003": {"suoxie": "HTHSLGYCYZQA", "name": "\u6d77\u901a\u6d77\u5347\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAITONGHAISHENGLIUGEYUECHIYOUZHAIQUANA"}, "850004": {"suoxie": "HTLHJZJXYNCYHHB", "name": "\u6d77\u901a\u91cf\u5316\u4ef7\u503c\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUAJIAZHIJINGXUANYINIANCHIYOUHUNHEB"}, "850005": {"suoxie": "HTHXYSYNCYHHB", "name": "\u6d77\u901a\u6838\u5fc3\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHEXINYOUSHIYINIANCHIYOUHUNHEB"}, "850006": {"suoxie": "HTHLYXYNCYHHB", "name": "\u6d77\u901a\u7ea2\u5229\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHONGLIYOUXUANYINIANCHIYOUHUNHEB"}, "850010": {"suoxie": "HTLHCZJXYNCYHHB", "name": "\u6d77\u901a\u91cf\u5316\u6210\u957f\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUACHENGZHANGJINGXUANYINIANCHIYOUHUNHEB"}, "850013": {"suoxie": "HTPZSJYNCYHHA", "name": "\u6d77\u901a\u54c1\u8d28\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGPINZHISHENGJIYINIANCHIYOUHUNHEA"}, "850088": {"suoxie": "HTLHJZJXYNCYHHA", "name": "\u6d77\u901a\u91cf\u5316\u4ef7\u503c\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUAJIAZHIJINGXUANYINIANCHIYOUHUNHEA"}, "850099": {"suoxie": "HTLHJZJXYNCYHHC", "name": "\u6d77\u901a\u91cf\u5316\u4ef7\u503c\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUAJIAZHIJINGXUANYINIANCHIYOUHUNHEC"}, "850588": {"suoxie": "HTHXYSYNCYHHA", "name": "\u6d77\u901a\u6838\u5fc3\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHEXINYOUSHIYINIANCHIYOUHUNHEA"}, "850599": {"suoxie": "HTHXYSYNCYHHC", "name": "\u6d77\u901a\u6838\u5fc3\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHEXINYOUSHIYINIANCHIYOUHUNHEC"}, "850688": {"suoxie": "HTHLYXYNCYHHA", "name": "\u6d77\u901a\u7ea2\u5229\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHONGLIYOUXUANYINIANCHIYOUHUNHEA"}, "850699": {"suoxie": "HTHLYXYNCYHHC", "name": "\u6d77\u901a\u7ea2\u5229\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGHONGLIYOUXUANYINIANCHIYOUHUNHEC"}, "851088": {"suoxie": "HTLHCZJXYNCYHHA", "name": "\u6d77\u901a\u91cf\u5316\u6210\u957f\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUACHENGZHANGJINGXUANYINIANCHIYOUHUNHEA"}, "851099": {"suoxie": "HTLHCZJXYNCYHHC", "name": "\u6d77\u901a\u91cf\u5316\u6210\u957f\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGLIANGHUACHENGZHANGJINGXUANYINIANCHIYOUHUNHEC"}, "851399": {"suoxie": "HTPZSJYNCYHHC", "name": "\u6d77\u901a\u54c1\u8d28\u5347\u7ea7\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "HAITONGPINZHISHENGJIYINIANCHIYOUHUNHEC"}, "851810": {"suoxie": "HTXXSGYCYZQA", "name": "\u6d77\u901a\u946b\u9009\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINXUANSANGEYUECHIYOUZHAIQUANA"}, "851816": {"suoxie": "HTXXSGYCYZQC", "name": "\u6d77\u901a\u946b\u9009\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINXUANSANGEYUECHIYOUZHAIQUANC"}, "851830": {"suoxie": "HTAYZDZA", "name": "\u6d77\u901a\u5b89\u88d5\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAITONGANYUZHONGDUANZHAIA"}, "851836": {"suoxie": "HTAYZDZC", "name": "\u6d77\u901a\u5b89\u88d5\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "HAITONGANYUZHONGDUANZHAIC"}, "851860": {"suoxie": "HTXYZQA", "name": "\u6d77\u901a\u946b\u9038\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINYIZHAIQUANA"}, "851880": {"suoxie": "HTXYZQC", "name": "\u6d77\u901a\u946b\u9038\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINYIZHAIQUANC"}, "851890": {"suoxie": "HTATZQA", "name": "\u6d77\u901a\u5b89\u6cf0\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANTAIZHAIQUANA"}, "851896": {"suoxie": "HTATZQC", "name": "\u6d77\u901a\u5b89\u6cf0\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANTAIZHAIQUANC"}, "851900": {"suoxie": "HTAYZQA", "name": "\u6d77\u901a\u5b89\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANYUEZHAIQUANA"}, "851986": {"suoxie": "HTAYZQC", "name": "\u6d77\u901a\u5b89\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANYUEZHAIQUANC"}, "852300": {"suoxie": "HTXYZQC", "name": "\u6d77\u901a\u946b\u60a6\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINYUEZHAIQUANC"}, "852389": {"suoxie": "HTXYZQA", "name": "\u6d77\u901a\u946b\u60a6\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGXINYUEZHAIQUANA"}, "855001": {"suoxie": "HTHSLGYCYZQC", "name": "\u6d77\u901a\u6d77\u5347\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HAITONGHAISHENGLIUGEYUECHIYOUZHAIQUANC"}, "860001": {"suoxie": "GDYGHHA", "name": "\u5149\u5927\u9633\u5149\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGHUNHEA"}, "860005": {"suoxie": "GDYGTLZQA", "name": "\u5149\u5927\u9633\u5149\u6dfb\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAYANGGUANGTIANLIZHAIQUANA"}, "860006": {"suoxie": "GDYGYXYNCYHHA", "name": "\u5149\u5927\u9633\u5149\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGYOUXUANYINIANCHIYOUHUNHEA"}, "860007": {"suoxie": "GDYGJZ30GYHHA", "name": "\u5149\u5927\u9633\u5149\u4ef7\u503c30\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGJIAZHI30GEYUEHUNHEA"}, "860008": {"suoxie": "GDYGSH18GYHHA", "name": "\u5149\u5927\u9633\u5149\u751f\u6d3b18\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGSHENGHUO18GEYUEHUNHEA"}, "860009": {"suoxie": "GDYGWJZZHHA", "name": "\u5149\u5927\u9633\u5149\u7a33\u5065\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGWENJIANZENGZHANGHUNHEA"}, "860010": {"suoxie": "GDYGDC6GYHHA", "name": "\u5149\u5927\u9633\u5149\u5bf9\u51b26\u4e2a\u6708\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GUANGDAYANGGUANGDUICHONG6GEYUEHUNHEA"}, "860011": {"suoxie": "GDYGDC6GYHHA", "name": "\u5149\u5927\u9633\u5149\u5bf9\u51b26\u4e2a\u6708\u6df7\u5408A(\u540e\u7aef)", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GUANGDAYANGGUANGDUICHONG6GEYUEHUNHEA"}, "860012": {"suoxie": "GDYGWZSY12GYCYZQA", "name": "\u5149\u5927\u9633\u5149\u7a33\u503a\u6536\u76ca12\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAYANGGUANGWENZHAISHOUYI12GEYUECHIYOUZHAIQUANA"}, "860016": {"suoxie": "GDYGQMXCXQDZTHHA", "name": "\u5149\u5927\u9633\u5149\u542f\u660e\u661f\u521b\u65b0\u9a71\u52a8\u4e3b\u9898\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGQIMINGXINGCHUANGXINQUDONGZHUTIHUNHEA"}, "860018": {"suoxie": "GDYGZZHHA", "name": "\u5149\u5927\u9633\u5149\u667a\u9020\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGZHIZAOHUNHEA"}, "860022": {"suoxie": "GDYG3GYCYFOFA", "name": "\u5149\u5927\u9633\u51493\u4e2a\u6708\u6301\u6709(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANG3GEYUECHIYOUFOFA"}, "860027": {"suoxie": "GDYGJZ30GYHHB", "name": "\u5149\u5927\u9633\u5149\u4ef7\u503c30\u4e2a\u6708\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGJIAZHI30GEYUEHUNHEB"}, "860028": {"suoxie": "GDYGDC6GYHHB", "name": "\u5149\u5927\u9633\u5149\u5bf9\u51b26\u4e2a\u6708\u6df7\u5408B", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GUANGDAYANGGUANGDUICHONG6GEYUEHUNHEB"}, "860029": {"suoxie": "GDYGDC6GYHHC", "name": "\u5149\u5927\u9633\u5149\u5bf9\u51b26\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "GUANGDAYANGGUANGDUICHONG6GEYUEHUNHEC"}, "860030": {"suoxie": "GDYGTLZQC", "name": "\u5149\u5927\u9633\u5149\u6dfb\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAYANGGUANGTIANLIZHAIQUANC"}, "860033": {"suoxie": "GDYGWZSY12GYCYZQC", "name": "\u5149\u5927\u9633\u5149\u7a33\u503a\u6536\u76ca12\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUANGDAYANGGUANGWENZHAISHOUYI12GEYUECHIYOUZHAIQUANC"}, "860035": {"suoxie": "GDYGWZZDZZQA", "name": "\u5149\u5927\u9633\u5149\u7a33\u503a\u4e2d\u77ed\u503a\u503a\u5238A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGDAYANGGUANGWENZHAIZHONGDUANZHAIZHAIQUANA"}, "860036": {"suoxie": "GDYGHHB", "name": "\u5149\u5927\u9633\u5149\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGHUNHEB"}, "860037": {"suoxie": "GDYGHHC", "name": "\u5149\u5927\u9633\u5149\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGHUNHEC"}, "860038": {"suoxie": "GDYGZZHHB", "name": "\u5149\u5927\u9633\u5149\u667a\u9020\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGZHIZAOHUNHEB"}, "860039": {"suoxie": "GDYGZZHHC", "name": "\u5149\u5927\u9633\u5149\u667a\u9020\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGZHIZAOHUNHEC"}, "860050": {"suoxie": "GDYGWZZDZZQC", "name": "\u5149\u5927\u9633\u5149\u7a33\u503a\u4e2d\u77ed\u503a\u503a\u5238C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGDAYANGGUANGWENZHAIZHONGDUANZHAIZHAIQUANC"}, "860051": {"suoxie": "GDYGBDX180TGDC", "name": "\u5149\u5927\u9633\u5149\u5317\u6597\u661f180\u5929\u6eda\u52a8C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAYANGGUANGBEIDOUXING180TIANGUNDONGC"}, "860052": {"suoxie": "GDYGQMXCXQDZTHHB", "name": "\u5149\u5927\u9633\u5149\u542f\u660e\u661f\u521b\u65b0\u9a71\u52a8\u4e3b\u9898\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGQIMINGXINGCHUANGXINQUDONGZHUTIHUNHEB"}, "860053": {"suoxie": "GDYGQMXCXQDZTHHC", "name": "\u5149\u5927\u9633\u5149\u542f\u660e\u661f\u521b\u65b0\u9a71\u52a8\u4e3b\u9898\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGQIMINGXINGCHUANGXINQUDONGZHUTIHUNHEC"}, "860055": {"suoxie": "GDYGYXYNCYHHB", "name": "\u5149\u5927\u9633\u5149\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGYOUXUANYINIANCHIYOUHUNHEB"}, "860056": {"suoxie": "GDYGYXYNCYHHC", "name": "\u5149\u5927\u9633\u5149\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGYOUXUANYINIANCHIYOUHUNHEC"}, "860058": {"suoxie": "GDYGWJZZHHC", "name": "\u5149\u5927\u9633\u5149\u7a33\u5065\u589e\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGWENJIANZENGZHANGHUNHEC"}, "860060": {"suoxie": "GDYGSH18GYHHB", "name": "\u5149\u5927\u9633\u5149\u751f\u6d3b18\u4e2a\u6708\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGSHENGHUO18GEYUEHUNHEB"}, "860061": {"suoxie": "GDYGSH18GYHHC", "name": "\u5149\u5927\u9633\u5149\u751f\u6d3b18\u4e2a\u6708\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANGSHENGHUO18GEYUEHUNHEC"}, "860063": {"suoxie": "GDYG3GYCYFOFC", "name": "\u5149\u5927\u9633\u51493\u4e2a\u6708\u6301\u6709(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGDAYANGGUANG3GEYUECHIYOUFOFC"}, "862001": {"suoxie": "GDYGXGJXHHQDIIRMBA", "name": "\u5149\u5927\u9633\u5149\u9999\u6e2f\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "GUANGDAYANGGUANGXIANGGANGJINGXUANHUNHEQDIIRENMINBIA"}, "862011": {"suoxie": "GDYGXGJXHHQDIIMYA", "name": "\u5149\u5927\u9633\u5149\u9999\u6e2f\u7cbe\u9009\u6df7\u5408(QDII)\u7f8e\u5143A", "style": "QDII", "pinyin": "GUANGDAYANGGUANGXIANGGANGJINGXUANHUNHEQDIIMEIYUANA"}, "862012": {"suoxie": "GDYGXGJXHHQDIIRMBC", "name": "\u5149\u5927\u9633\u5149\u9999\u6e2f\u7cbe\u9009\u6df7\u5408(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "GUANGDAYANGGUANGXIANGGANGJINGXUANHUNHEQDIIRENMINBIC"}, "863001": {"suoxie": "GDYGXJBHB", "name": "\u5149\u5927\u9633\u5149\u73b0\u91d1\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUANGDAYANGGUANGXIANJINBAOHUOBI"}, "865040": {"suoxie": "GDYGBDX180TGDA", "name": "\u5149\u5927\u9633\u5149\u5317\u6597\u661f180\u5929\u6eda\u52a8A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGDAYANGGUANGBEIDOUXING180TIANGUNDONGA"}, "870005": {"suoxie": "GFZGHXJXYNCYHHA", "name": "\u5e7f\u53d1\u8d44\u7ba1\u6838\u5fc3\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZIGUANHEXINJINGXUANYINIANCHIYOUHUNHEA"}, "870008": {"suoxie": "GFQLYNCYQZQA", "name": "\u5e7f\u53d1\u4e7e\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAQIANLIYINIANCHIYOUQIZHAIQUANA"}, "870009": {"suoxie": "GFPHJXYNCYHHA", "name": "\u5e7f\u53d1\u5e73\u8861\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAPINGHENGJINGXUANYINIANCHIYOUHUNHEA"}, "870012": {"suoxie": "GFZGRL3NDKZ", "name": "\u5e7f\u53d1\u8d44\u7ba1\u777f\u52293\u5e74\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZIGUANRUILI3NIANDINGKAIZHAI"}, "870015": {"suoxie": "GFZGZLZDZ", "name": "\u5e7f\u53d1\u8d44\u7ba1\u662d\u5229\u4e2d\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZIGUANZHAOLIZHONGDUANZHAI"}, "870017": {"suoxie": "GFZGXFJXLHPZHH", "name": "\u5e7f\u53d1\u8d44\u7ba1\u6d88\u8d39\u7cbe\u9009\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAZIGUANXIAOFEIJINGXUANLINGHUOPEIZHIHUNHE"}, "871003": {"suoxie": "GFZGJZZZLHPZHH", "name": "\u5e7f\u53d1\u8d44\u7ba1\u4ef7\u503c\u589e\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "GUANGFAZIGUANJIAZHIZENGZHANGLINGHUOPEIZHIHUNHE"}, "872003": {"suoxie": "GFQLYNCYQZQ", "name": "\u5e7f\u53d1\u4e7e\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAQIANLIYINIANCHIYOUQIZHAIQUAN"}, "872014": {"suoxie": "GFQLYNCYQZQC", "name": "\u5e7f\u53d1\u4e7e\u5229\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAQIANLIYINIANCHIYOUQIZHAIQUANC"}, "872015": {"suoxie": "GFZGZLZDZA", "name": "\u5e7f\u53d1\u8d44\u7ba1\u662d\u5229\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZIGUANZHAOLIZHONGDUANZHAIA"}, "872016": {"suoxie": "GFZGZLZDZB", "name": "\u5e7f\u53d1\u8d44\u7ba1\u662d\u5229\u4e2d\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZIGUANZHAOLIZHONGDUANZHAIB"}, "872017": {"suoxie": "GFZGZLZDZC", "name": "\u5e7f\u53d1\u8d44\u7ba1\u662d\u5229\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "GUANGFAZIGUANZHAOLIZHONGDUANZHAIC"}, "872018": {"suoxie": "GFPHJXYNCYHH", "name": "\u5e7f\u53d1\u5e73\u8861\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAPINGHENGJINGXUANYINIANCHIYOUHUNHE"}, "872019": {"suoxie": "GFPHJXYNCYHHC", "name": "\u5e7f\u53d1\u5e73\u8861\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAPINGHENGJINGXUANYINIANCHIYOUHUNHEC"}, "872020": {"suoxie": "GFZGHXJXYNCYHH", "name": "\u5e7f\u53d1\u8d44\u7ba1\u6838\u5fc3\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZIGUANHEXINJINGXUANYINIANCHIYOUHUNHE"}, "872021": {"suoxie": "GFZGHXJXYNCYHHC", "name": "\u5e7f\u53d1\u8d44\u7ba1\u6838\u5fc3\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUANGFAZIGUANHEXINJINGXUANYINIANCHIYOUHUNHEC"}, "872022": {"suoxie": "GFZGRL3NDKZA", "name": "\u5e7f\u53d1\u8d44\u7ba1\u777f\u52293\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZIGUANRUILI3NIANDINGKAIZHAIA"}, "872023": {"suoxie": "GFZGRL3NDKZB", "name": "\u5e7f\u53d1\u8d44\u7ba1\u777f\u52293\u5e74\u5b9a\u5f00\u503aB", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZIGUANRUILI3NIANDINGKAIZHAIB"}, "872024": {"suoxie": "GFZGRL3NDKZC", "name": "\u5e7f\u53d1\u8d44\u7ba1\u777f\u52293\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUANGFAZIGUANRUILI3NIANDINGKAIZHAIC"}, "873002": {"suoxie": "GFZGQQJXYNCYQZQQDII", "name": "\u5e7f\u53d1\u8d44\u7ba1\u5168\u7403\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238(QDII)", "style": "QDII", "pinyin": "GUANGFAZIGUANQUANQIUJINGXUANYINIANCHIYOUQIZHAIQUANQDII"}, "873013": {"suoxie": "GFQQJXYNCYZQQDIIRMBA", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u503a\u5238(QDII)\u4eba\u6c11\u5e01A", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANYINIANCHIYOUZHAIQUANQDIIRENMINBIA"}, "873017": {"suoxie": "GFQQJXYNCYZQQDIIRMBC", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u503a\u5238(QDII)\u4eba\u6c11\u5e01C", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANYINIANCHIYOUZHAIQUANQDIIRENMINBIC"}, "873018": {"suoxie": "GFQQJXYNCYZQQDIIMYA", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u503a\u5238(QDII)\u7f8e\u5143A", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANYINIANCHIYOUZHAIQUANQDIIMEIYUANA"}, "873021": {"suoxie": "GFQQJXYNCYZQQDIIMYC", "name": "\u5e7f\u53d1\u5168\u7403\u7cbe\u9009\u4e00\u5e74\u6301\u6709\u503a\u5238(QDII)\u7f8e\u5143C", "style": "QDII", "pinyin": "GUANGFAQUANQIUJINGXUANYINIANCHIYOUZHAIQUANQDIIMEIYUANC"}, "880007": {"suoxie": "ZSZGZYCZLHPZHHA", "name": "\u62db\u5546\u8d44\u7ba1\u667a\u8fdc\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGZIGUANZHIYUANCHENGZHANGLINGHUOPEIZHIHUNHEA"}, "880009": {"suoxie": "ZSZGRFSGYCYZD", "name": "\u62db\u5546\u8d44\u7ba1\u777f\u4e30\u4e09\u4e2a\u6708\u6301\u6709\u503aD", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANRUIFENGSANGEYUECHIYOUZHAID"}, "880011": {"suoxie": "ZSZGZYZLZQD", "name": "\u62db\u5546\u8d44\u7ba1\u667a\u8fdc\u589e\u5229\u503a\u5238D", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANZHIYUANZENGLIZHAIQUAND"}, "881007": {"suoxie": "ZSZGZYCZLHPZHHC", "name": "\u62db\u5546\u8d44\u7ba1\u667a\u8fdc\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHAOSHANGZIGUANZHIYUANCHENGZHANGLINGHUOPEIZHIHUNHEC"}, "881010": {"suoxie": "ZSZGRFSGYCYZA", "name": "\u62db\u5546\u8d44\u7ba1\u777f\u4e30\u4e09\u4e2a\u6708\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANRUIFENGSANGEYUECHIYOUZHAIA"}, "881011": {"suoxie": "ZSZGRFSGYCYZC", "name": "\u62db\u5546\u8d44\u7ba1\u777f\u4e30\u4e09\u4e2a\u6708\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANRUIFENGSANGEYUECHIYOUZHAIC"}, "881012": {"suoxie": "ZSZGZYZLZQA", "name": "\u62db\u5546\u8d44\u7ba1\u667a\u8fdc\u589e\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANZHIYUANZENGLIZHAIQUANA"}, "881013": {"suoxie": "ZSZGZYZLZQC", "name": "\u62db\u5546\u8d44\u7ba1\u667a\u8fdc\u589e\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHAOSHANGZIGUANZHIYUANZENGLIZHAIQUANC"}, "890005": {"suoxie": "CJZLZQA", "name": "\u957f\u6c5f\u5c0a\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGZUNLIZHAIQUANA"}, "890008": {"suoxie": "CJZX3GYCYHHFOFA", "name": "\u957f\u6c5f\u667a\u90093\u4e2a\u6708\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "CHANGJIANGZHIXUAN3GEYUECHIYOUHUNHEFOFA"}, "890011": {"suoxie": "CJJLZQXA", "name": "\u957f\u6c5f\u805a\u5229\u503a\u5238\u578bA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "CHANGJIANGJULIZHAIQUANXINGA"}, "900003": {"suoxie": "ZXZQZXJZCZHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u81fb\u9009\u4ef7\u503c\u6210\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANZHENXUANJIAZHICHENGZHANGHUNHEA"}, "900007": {"suoxie": "ZXZQYHYNCYZA", "name": "\u4e2d\u4fe1\u503a\u5238\u4f18\u5316\u4e00\u5e74\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZHAIQUANYOUHUAYINIANCHIYOUZHAIA"}, "900008": {"suoxie": "ZXZQWJHBHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u7a33\u5065\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANWENJIANHUIBAOHUNHEA"}, "900009": {"suoxie": "ZXZQCZDLHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u6210\u957f\u52a8\u529b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANCHENGZHANGDONGLIHUNHEA"}, "900010": {"suoxie": "ZXZYCZLNCYHHA", "name": "\u4e2d\u4fe1\u5353\u8d8a\u6210\u957f\u4e24\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHUOYUECHENGZHANGLIANGNIANCHIYOUHUNHEA"}, "900011": {"suoxie": "ZXZQHLJZA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u7ea2\u5229\u4ef7\u503cA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANHONGLIJIAZHIA"}, "900012": {"suoxie": "ZXZQCFYXYNCYHHFOFA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u8d22\u5bcc\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANCAIFUYOUXUANYINIANCHIYOUHUNHEFOFA"}, "900013": {"suoxie": "ZXZQPZSHHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u54c1\u8d28\u751f\u6d3b\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANPINZHISHENGHUOHUNHEA"}, "900015": {"suoxie": "ZXZQZQZQLGYCYA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u503a\u5238\u589e\u5f3a\u516d\u4e2a\u6708\u6301\u6709A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZHENGQUANZHAIQUANZENGQIANGLIUGEYUECHIYOUA"}, "900017": {"suoxie": "ZXZYSBGYCYZQA", "name": "\u4e2d\u4fe1\u589e\u76ca\u5341\u516b\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZENGYISHIBAGEYUECHIYOUZHAIQUANA"}, "900018": {"suoxie": "ZXZQZLYNDKZA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u589e\u5229\u4e00\u5e74\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINZHENGQUANZENGLIYINIANDINGKAIZHAIA"}, "900019": {"suoxie": "ZXZQLGYZQA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u516d\u4e2a\u6708\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINZHENGQUANLIUGEYUEZHAIQUANA"}, "900022": {"suoxie": "ZXZQZXHBHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u81fb\u9009\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANZHENXUANHUIBAOHUNHEA"}, "900026": {"suoxie": "ZXZQXYYNCYZQ", "name": "\u4e2d\u4fe1\u8bc1\u5238\u4fe1\u76c8\u4e00\u5e74\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINZHENGQUANXINYINGYINIANCHIYOUZHAIQUAN"}, "900027": {"suoxie": "ZXZQXYYNCYHHA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u4fe1\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANXINYUANYINIANCHIYOUHUNHEA"}, "900029": {"suoxie": "ZXZQLHYXA", "name": "\u4e2d\u4fe1\u8bc1\u5238\u91cf\u5316\u4f18\u9009A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGXINZHENGQUANLIANGHUAYOUXUANA"}, "900030": {"suoxie": "ZXZQLHYXC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u91cf\u5316\u4f18\u9009C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGXINZHENGQUANLIANGHUAYOUXUANC"}, "900039": {"suoxie": "ZXZQLGYZQC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u516d\u4e2a\u6708\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINZHENGQUANLIUGEYUEZHAIQUANC"}, "900052": {"suoxie": "ZXZQZXHBHHB", "name": "\u4e2d\u4fe1\u8bc1\u5238\u81fb\u9009\u56de\u62a5\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANZHENXUANHUIBAOHUNHEB"}, "900057": {"suoxie": "ZXZYSBGYCYZQC", "name": "\u4e2d\u4fe1\u589e\u76ca\u5341\u516b\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZENGYISHIBAGEYUECHIYOUZHAIQUANC"}, "900059": {"suoxie": "ZXZQCZDLHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u6210\u957f\u52a8\u529b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANCHENGZHANGDONGLIHUNHEC"}, "900077": {"suoxie": "ZXZQXYYNCYHHB", "name": "\u4e2d\u4fe1\u8bc1\u5238\u4fe1\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANXINYUANYINIANCHIYOUHUNHEB"}, "900078": {"suoxie": "ZXZQWJHBHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u7a33\u5065\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANWENJIANHUIBAOHUNHEC"}, "900079": {"suoxie": "ZXZQZXJZCZHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u81fb\u9009\u4ef7\u503c\u6210\u957f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANZHENXUANJIAZHICHENGZHANGHUNHEC"}, "900087": {"suoxie": "ZXZQXYYNCYHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u4fe1\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANXINYUANYINIANCHIYOUHUNHEC"}, "900089": {"suoxie": "ZXZQHLJZC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u7ea2\u5229\u4ef7\u503cC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANHONGLIJIAZHIC"}, "900090": {"suoxie": "ZXZYCZLNCYHHB", "name": "\u4e2d\u4fe1\u5353\u8d8a\u6210\u957f\u4e24\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHUOYUECHENGZHANGLIANGNIANCHIYOUHUNHEB"}, "900097": {"suoxie": "ZXZQYHYNCYZC", "name": "\u4e2d\u4fe1\u503a\u5238\u4f18\u5316\u4e00\u5e74\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZHAIQUANYOUHUAYINIANCHIYOUZHAIC"}, "900099": {"suoxie": "ZXZQHLJZB", "name": "\u4e2d\u4fe1\u8bc1\u5238\u7ea2\u5229\u4ef7\u503cB", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANHONGLIJIAZHIB"}, "900100": {"suoxie": "ZXZYCZLNCYHHC", "name": "\u4e2d\u4fe1\u5353\u8d8a\u6210\u957f\u4e24\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHUOYUECHENGZHANGLIANGNIANCHIYOUHUNHEC"}, "900112": {"suoxie": "ZXZQCFYXYNCYHHFOFC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u8d22\u5bcc\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408(FOF)C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANCAIFUYOUXUANYINIANCHIYOUHUNHEFOFC"}, "900133": {"suoxie": "ZXZQPZSHHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u54c1\u8d28\u751f\u6d3b\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANPINZHISHENGHUOHUNHEC"}, "900152": {"suoxie": "ZXZQZXHBHHC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u81fb\u9009\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINZHENGQUANZHENXUANHUIBAOHUNHEC"}, "900155": {"suoxie": "ZXZQZQZQLGYCYC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u503a\u5238\u589e\u5f3a\u516d\u4e2a\u6708\u6301\u6709C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ZHONGXINZHENGQUANZHAIQUANZENGQIANGLIUGEYUECHIYOUC"}, "900188": {"suoxie": "ZXZQZLYNDKZC", "name": "\u4e2d\u4fe1\u8bc1\u5238\u589e\u5229\u4e00\u5e74\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGXINZHENGQUANZENGLIYINIANDINGKAIZHAIC"}, "910004": {"suoxie": "DFHQHSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u6052\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHENGSANNIANCHIYOUHUNHEA"}, "910005": {"suoxie": "DFHQXSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u5174\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIXINGSANNIANCHIYOUHUNHEA"}, "910006": {"suoxie": "DFHQSSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u76db\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQISHENGSANNIANCHIYOUHUNHEA"}, "910007": {"suoxie": "DFHQYSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u5143\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIYUANSANNIANCHIYOUHUNHEA"}, "910009": {"suoxie": "DFHQCSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u7a0b\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQICHENGSANNIANCHIYOUHUNHEA"}, "910010": {"suoxie": "DFHXHHHA", "name": "\u4e1c\u65b9\u7ea2\u65b0\u6d77\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGXINHAIHUNHEA"}, "910011": {"suoxie": "DFHQRSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u745e\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIRUISANNIANCHIYOUHUNHEA"}, "910017": {"suoxie": "DFHDYCLHHA", "name": "\u4e1c\u65b9\u7ea2\u591a\u5143\u7b56\u7565\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGDUOYUANCELUEHUNHEA"}, "910021": {"suoxie": "DFHQHSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u534e\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHUASANNIANCHIYOUHUNHEA"}, "910022": {"suoxie": "DFHQHSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u822a\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIHANGSANNIANCHIYOUHUNHEA"}, "910024": {"suoxie": "DFHQYSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u542f\u9633\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGQIYANGSANNIANCHIYOUHUNHEA"}, "910026": {"suoxie": "DFHXYSNCYHHA", "name": "\u4e1c\u65b9\u7ea2\u65b0\u6e90\u4e09\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGXINYUANSANNIANCHIYOUHUNHEA"}, "910028": {"suoxie": "DFHNXZZHHA", "name": "\u4e1c\u65b9\u7ea2\u5185\u9700\u589e\u957f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGFANGHONGNEIXUZENGZHANGHUNHEA"}, "920002": {"suoxie": "ZJJXGPA", "name": "\u4e2d\u91d1\u7cbe\u9009\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINJINGXUANGUPIAOA"}, "920003": {"suoxie": "ZJXRGPA", "name": "\u4e2d\u91d1\u65b0\u9510\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINXINRUIGUPIAOA"}, "920007": {"suoxie": "ZJHRZQA", "name": "\u4e2d\u91d1\u6052\u745e\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINHENGRUIZHAIQUANA"}, "920008": {"suoxie": "ZJJQHBHHA", "name": "\u4e2d\u91d1\u8fdb\u53d6\u56de\u62a5\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINJINQUHUIBAOHUNHEA"}, "920011": {"suoxie": "ZJAXHBLHPZHHA", "name": "\u4e2d\u91d1\u5b89\u5fc3\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINANXINHUIBAOLINGHUOPEIZHIHUNHEA"}, "920012": {"suoxie": "ZJHYLH3GYDKHHA", "name": "\u4e2d\u91d1\u6c47\u8d8a\u91cf\u53163\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408A", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ZHONGJINHUIYUELIANGHUA3GEYUEDINGKAIHUNHEA"}, "920921": {"suoxie": "ZJAXHBLHPZHHC", "name": "\u4e2d\u91d1\u5b89\u5fc3\u56de\u62a5\u7075\u6d3b\u914d\u7f6e\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINANXINHUIBAOLINGHUOPEIZHIHUNHEC"}, "920922": {"suoxie": "ZJJXGPC", "name": "\u4e2d\u91d1\u7cbe\u9009\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINJINGXUANGUPIAOC"}, "920923": {"suoxie": "ZJXRGPC", "name": "\u4e2d\u91d1\u65b0\u9510\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "ZHONGJINXINRUIGUPIAOC"}, "920926": {"suoxie": "ZJHYLH3GYDKHHC", "name": "\u4e2d\u91d1\u6c47\u8d8a\u91cf\u53163\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408C", "style": "\u6df7\u5408-\u7edd\u5bf9\u6536\u76ca", "pinyin": "ZHONGJINHUIYUELIANGHUA3GEYUEDINGKAIHUNHEC"}, "920927": {"suoxie": "ZJHRZQC", "name": "\u4e2d\u91d1\u6052\u745e\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ZHONGJINHENGRUIZHAIQUANC"}, "920928": {"suoxie": "ZJJQHBHHC", "name": "\u4e2d\u91d1\u8fdb\u53d6\u56de\u62a5\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "ZHONGJINJINQUHUIBAOHUNHEC"}, "930602": {"suoxie": "GXJZZXHH", "name": "\u56fd\u4fe1\u4ef7\u503c\u667a\u9009\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOXINJIAZHIZHIXUANHUNHE"}, "940018": {"suoxie": "HTZJTTFHB", "name": "\u534e\u6cf0\u7d2b\u91d1\u5929\u5929\u53d1\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "HUATAIZIJINTIANTIANFAHUOBI"}, "952003": {"suoxie": "GTJAZZ13NZJZA", "name": "\u56fd\u6cf0\u541b\u5b89\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUNANZHONGZHAI13NIANZHENGJINZHAIA"}, "952004": {"suoxie": "GJZGJDMHH", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u660e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJUNZIGUANJUNDEMINGHUNHE"}, "952009": {"suoxie": "GJZGJDXLNCYQHHA", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u946b\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJUNZIGUANJUNDEXINLIANGNIANCHIYOUQIHUNHEA"}, "952013": {"suoxie": "GJZGJDYSGYCYHHA", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u76ca\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJUNZIGUANJUNDEYISANGEYUECHIYOUHUNHEA"}, "952020": {"suoxie": "GJZGJDYZQA", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u76c8\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOJUNZIGUANJUNDEYINGZHAIQUANA"}, "952024": {"suoxie": "GTJAJDSZQ", "name": "\u56fd\u6cf0\u541b\u5b89\u541b\u5f97\u76db\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOTAIJUNANJUNDESHENGZHAIQUAN"}, "952035": {"suoxie": "GTJAJDCHH", "name": "\u56fd\u6cf0\u541b\u5b89\u541b\u5f97\u8bda\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOTAIJUNANJUNDECHENGHUNHE"}, "952099": {"suoxie": "GJZGJDXLNCYQHHC", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u946b\u4e24\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJUNZIGUANJUNDEXINLIANGNIANCHIYOUQIHUNHEC"}, "952100": {"suoxie": "GTJAXJGJHB", "name": "\u56fd\u6cf0\u541b\u5b89\u73b0\u91d1\u7ba1\u5bb6\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "GUOTAIJUNANXIANJINGUANJIAHUOBI"}, "952303": {"suoxie": "GTJAZZ13NZJZC", "name": "\u56fd\u6cf0\u541b\u5b89\u4e2d\u503a1-3\u5e74\u653f\u91d1\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOTAIJUNANZHONGZHAI13NIANZHENGJINZHAIC"}, "952313": {"suoxie": "GJZGJDYSGYCYHHC", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u76ca\u4e09\u4e2a\u6708\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "GUOJUNZIGUANJUNDEYISANGEYUECHIYOUHUNHEC"}, "952320": {"suoxie": "GJZGJDYZQC", "name": "\u56fd\u541b\u8d44\u7ba1\u541b\u5f97\u76c8\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOJUNZIGUANJUNDEYINGZHAIQUANC"}, "959991": {"suoxie": "XZZGJQLLXYSYNCYHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u9886\u5148\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINLINGXIANYOUSHIYINIANCHIYOUHUNHEA"}, "959993": {"suoxie": "XZZGJQLLXYSYNCYHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u9886\u5148\u4f18\u52bf\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINLINGXIANYOUSHIYINIANCHIYOUHUNHEC"}, "970001": {"suoxie": "TZLGYDKZA", "name": "\u5929\u6cfd\u516d\u4e2a\u6708\u5b9a\u5f00\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANZELIUGEYUEDINGKAIZHAIA"}, "970002": {"suoxie": "TZLGYDKZC", "name": "\u5929\u6cfd\u516d\u4e2a\u6708\u5b9a\u5f00\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANZELIUGEYUEDINGKAIZHAIC"}, "970003": {"suoxie": "AXRHZDZA", "name": "\u5b89\u4fe1\u745e\u9e3f\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINRUIHONGZHONGDUANZHAIA"}, "970004": {"suoxie": "AXRHZDZB", "name": "\u5b89\u4fe1\u745e\u9e3f\u4e2d\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINRUIHONGZHONGDUANZHAIB"}, "970005": {"suoxie": "AXRHZDZC", "name": "\u5b89\u4fe1\u745e\u9e3f\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "ANXINRUIHONGZHONGDUANZHAIC"}, "970006": {"suoxie": "HAZQHYZLYNCYHHA", "name": "\u534e\u5b89\u8bc1\u5238\u6c47\u8d62\u589e\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANZHENGQUANHUIYINGZENGLIYINIANCHIYOUHUNHEA"}, "970007": {"suoxie": "HAZQHYZLYNCYHHB", "name": "\u534e\u5b89\u8bc1\u5238\u6c47\u8d62\u589e\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANZHENGQUANHUIYINGZENGLIYINIANCHIYOUHUNHEB"}, "970008": {"suoxie": "HAZQHYZLYNCYHHC", "name": "\u534e\u5b89\u8bc1\u5238\u6c47\u8d62\u589e\u5229\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "HUAANZHENGQUANHUIYINGZENGLIYINIANCHIYOUHUNHEC"}, "970009": {"suoxie": "FZZQJLFYNCYQHHA", "name": "\u65b9\u6b63\u8bc1\u5238\u91d1\u7acb\u65b9\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGZHENGQUANJINLIFANGYINIANCHIYOUQIHUNHEA"}, "970010": {"suoxie": "FZZQJLFYNCYQHHC", "name": "\u65b9\u6b63\u8bc1\u5238\u91d1\u7acb\u65b9\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "FANGZHENGZHENGQUANJINLIFANGYINIANCHIYOUQIHUNHEC"}, "970011": {"suoxie": "PAAYTLBNGDCYA", "name": "\u5e73\u5b89\u5b89\u8d62\u6dfb\u5229\u534a\u5e74\u6eda\u52a8\u6301\u6709A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANANYINGTIANLIBANNIANGUNDONGCHIYOUA"}, "970012": {"suoxie": "PAAYTLBNGDCYB", "name": "\u5e73\u5b89\u5b89\u8d62\u6dfb\u5229\u534a\u5e74\u6eda\u52a8\u6301\u6709B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANANYINGTIANLIBANNIANGUNDONGCHIYOUB"}, "970013": {"suoxie": "PAAYTLBNGDCYC", "name": "\u5e73\u5b89\u5b89\u8d62\u6dfb\u5229\u534a\u5e74\u6eda\u52a8\u6301\u6709C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANANYINGTIANLIBANNIANGUNDONGCHIYOUC"}, "970014": {"suoxie": "PAAYTLBNGDCYD", "name": "\u5e73\u5b89\u5b89\u8d62\u6dfb\u5229\u534a\u5e74\u6eda\u52a8\u6301\u6709D", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "PINGANANYINGTIANLIBANNIANGUNDONGCHIYOUD"}, "970015": {"suoxie": "SWHLCZLHPZHH", "name": "\u7533\u4e07\u7ea2\u5229\u6210\u957f\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "SHENWANHONGLICHENGZHANGLINGHUOPEIZHIHUNHE"}, "970016": {"suoxie": "ZXJTJZZZA", "name": "\u4e2d\u4fe1\u5efa\u6295\u4ef7\u503c\u589e\u957fA", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUJIAZHIZENGZHANGA"}, "970017": {"suoxie": "ZXJTJZZZC", "name": "\u4e2d\u4fe1\u5efa\u6295\u4ef7\u503c\u589e\u957fC", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "ZHONGXINJIANTOUJIAZHIZENGZHANGC"}, "970018": {"suoxie": "FZZQJGWLGYCYZQA", "name": "\u65b9\u6b63\u8bc1\u5238\u91d1\u6e2f\u6e7e\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGZHENGQUANJINGANGWANLIUGEYUECHIYOUZHAIQUANA"}, "970019": {"suoxie": "FZZQJGWLGYCYZQC", "name": "\u65b9\u6b63\u8bc1\u5238\u91d1\u6e2f\u6e7e\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGZHENGQUANJINGANGWANLIUGEYUECHIYOUZHAIQUANC"}, "970020": {"suoxie": "XDJZJXA", "name": "\u4fe1\u8fbe\u4ef7\u503c\u7cbe\u9009A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAJIAZHIJINGXUANA"}, "970021": {"suoxie": "XDJZJXB", "name": "\u4fe1\u8fbe\u4ef7\u503c\u7cbe\u9009B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "XINDAJIAZHIJINGXUANB"}, "970022": {"suoxie": "XDFRLGYCY", "name": "\u4fe1\u8fbe\u4e30\u777f\u516d\u4e2a\u6708\u6301\u6709", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "XINDAFENGRUILIUGEYUECHIYOU"}, "970023": {"suoxie": "TFTYYNDKHH", "name": "\u5929\u98ce\u5929\u76c8\u4e00\u5e74\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u5e73\u8861", "pinyin": "TIANFENGTIANYINGYINIANDINGKAIHUNHE"}, "970024": {"suoxie": "HAZQJYYNCYA", "name": "\u534e\u5b89\u8bc1\u5238\u805a\u8d62\u4e00\u5e74\u6301\u6709A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANJUYINGYINIANCHIYOUA"}, "970025": {"suoxie": "HAZQJYYNCYB", "name": "\u534e\u5b89\u8bc1\u5238\u805a\u8d62\u4e00\u5e74\u6301\u6709B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANJUYINGYINIANCHIYOUB"}, "970026": {"suoxie": "SWHYLTKLDZZQ", "name": "\u7533\u4e07\u5b8f\u6e90\u7075\u901a\u5feb\u5229\u77ed\u503a\u503a\u5238", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "SHENWANHONGYUANLINGTONGKUAILIDUANZHAIZHAIQUAN"}, "970027": {"suoxie": "GHLGYGDCYZQA", "name": "\u56fd\u6d77\u516d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOHAILIUGEYUEGUNDONGCHIYOUZHAIQUANA"}, "970028": {"suoxie": "GHLGYGDCYZQC", "name": "\u56fd\u6d77\u516d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOHAILIUGEYUEGUNDONGCHIYOUZHAIQUANC"}, "970029": {"suoxie": "AXZGRYTLA", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u5143\u6dfb\u5229A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZIGUANRUIYUANTIANLIA"}, "970030": {"suoxie": "AXZGRYTLB", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u5143\u6dfb\u5229B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZIGUANRUIYUANTIANLIB"}, "970031": {"suoxie": "AXZGRYTLC", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u5143\u6dfb\u5229C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINZIGUANRUIYUANTIANLIC"}, "970032": {"suoxie": "DHHRJQLHPZHHA", "name": "\u4e1c\u6d77\u6d77\u777f\u8fdb\u53d6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIJINQULINGHUOPEIZHIHUNHEA"}, "970033": {"suoxie": "DHHRJQLHPZHHB", "name": "\u4e1c\u6d77\u6d77\u777f\u8fdb\u53d6\u7075\u6d3b\u914d\u7f6e\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIJINQULINGHUOPEIZHIHUNHEB"}, "970034": {"suoxie": "HAZQHYSGYDK", "name": "\u534e\u5b89\u8bc1\u5238\u5408\u8d62\u4e09\u4e2a\u6708\u5b9a\u5f00", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANHEYINGSANGEYUEDINGKAI"}, "970035": {"suoxie": "HAZQHYJGYCY", "name": "\u534e\u5b89\u8bc1\u5238\u5408\u8d62\u4e5d\u4e2a\u6708\u6301\u6709", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANHEYINGJIUGEYUECHIYOU"}, "970036": {"suoxie": "HAZQRYYNCYZQA", "name": "\u534e\u5b89\u8bc1\u5238\u777f\u8d62\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANRUIYINGYINIANCHIYOUZHAIQUANA"}, "970037": {"suoxie": "HAZQRYYNCYZQB", "name": "\u534e\u5b89\u8bc1\u5238\u777f\u8d62\u4e00\u5e74\u6301\u6709\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANRUIYINGYINIANCHIYOUZHAIQUANB"}, "970038": {"suoxie": "DGDY6GYCYZ", "name": "\u4e1c\u839e\u5fb7\u76ca6\u4e2a\u6708\u6301\u6709\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DONGGUANDEYI6GEYUECHIYOUZHAI"}, "970039": {"suoxie": "TFLGYGDCYZQA", "name": "\u5929\u98ce\u516d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANFENGLIUGEYUEGUNDONGCHIYOUZHAIQUANA"}, "970040": {"suoxie": "TFLGYGDCYZQC", "name": "\u5929\u98ce\u516d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TIANFENGLIUGEYUEGUNDONGCHIYOUZHAIQUANC"}, "970041": {"suoxie": "GHLHYXYNCYGPA", "name": "\u56fd\u6d77\u91cf\u5316\u4f18\u9009\u4e00\u5e74\u6301\u6709\u80a1\u7968A", "style": "\u80a1\u7968\u578b", "pinyin": "GUOHAILIANGHUAYOUXUANYINIANCHIYOUGUPIAOA"}, "970042": {"suoxie": "GHLHYXYNCYGPC", "name": "\u56fd\u6d77\u91cf\u5316\u4f18\u9009\u4e00\u5e74\u6301\u6709\u80a1\u7968C", "style": "\u80a1\u7968\u578b", "pinyin": "GUOHAILIANGHUAYOUXUANYINIANCHIYOUGUPIAOC"}, "970043": {"suoxie": "DWYYYNCYHHA", "name": "\u4e1c\u5434\u88d5\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUYUYINGYINIANCHIYOUHUNHEA"}, "970044": {"suoxie": "DWYYYNCYHHB", "name": "\u4e1c\u5434\u88d5\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUYUYINGYINIANCHIYOUHUNHEB"}, "970045": {"suoxie": "DWYYYNCYHHC", "name": "\u4e1c\u5434\u88d5\u76c8\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGWUYUYINGYINIANCHIYOUHUNHEC"}, "970046": {"suoxie": "DHHRJXLHPZHHA", "name": "\u4e1c\u6d77\u6d77\u777f\u5065\u884c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIJIANXINGLINGHUOPEIZHIHUNHEA"}, "970047": {"suoxie": "DHHRJXLHPZHHB", "name": "\u4e1c\u6d77\u6d77\u777f\u5065\u884c\u7075\u6d3b\u914d\u7f6e\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIJIANXINGLINGHUOPEIZHIHUNHEB"}, "970048": {"suoxie": "DHHRZYLHPZHH", "name": "\u4e1c\u6d77\u6d77\u777f\u81f4\u8fdc\u7075\u6d3b\u914d\u7f6e\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIZHIYUANLINGHUOPEIZHIHUNHE"}, "970049": {"suoxie": "DHZGHXZL3GYDKZ", "name": "\u4e1c\u6d77\u8d44\u7ba1\u6d77\u946b\u589e\u52293\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGHAIZIGUANHAIXINZENGLI3GEYUEDINGKAIZHAI"}, "970050": {"suoxie": "DHHRRY3GYDKHH", "name": "\u4e1c\u6d77\u6d77\u777f\u9510\u610f3\u4e2a\u6708\u5b9a\u5f00\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u7075\u6d3b", "pinyin": "DONGHAIHAIRUIRUIYI3GEYUEDINGKAIHUNHE"}, "970051": {"suoxie": "YHAFJGYGDCYHH", "name": "\u94f6\u6cb3\u5b89\u4e30\u4e5d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "YINHEANFENGJIUGEYUEGUNDONGCHIYOUHUNHE"}, "970052": {"suoxie": "FZXYYNCYZQA", "name": "\u65b9\u6b63\u946b\u60a6\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGXINYUEYINIANCHIYOUZHAIQUANA"}, "970053": {"suoxie": "FZXYYNCYZQC", "name": "\u65b9\u6b63\u946b\u60a6\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "FANGZHENGXINYUEYINIANCHIYOUZHAIQUANC"}, "970054": {"suoxie": "XDTLSGYCYZQ", "name": "\u4fe1\u8fbe\u6dfb\u5229\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDATIANLISANGEYUECHIYOUZHAIQUAN"}, "970055": {"suoxie": "XDXLLGYCYZQ", "name": "\u4fe1\u8fbe\u4fe1\u5229\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAXINLILIUGEYUECHIYOUZHAIQUAN"}, "970056": {"suoxie": "GLJRY3GYGDCYZA", "name": "\u56fd\u8054\u91d1\u5982\u610f3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANJINRUYI3GEYUEGUNDONGCHIYOUZHAIA"}, "970057": {"suoxie": "GLJRY3GYGDCYZC", "name": "\u56fd\u8054\u91d1\u5982\u610f3\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANJINRUYI3GEYUEGUNDONGCHIYOUZHAIC"}, "970058": {"suoxie": "AXRY3GYGDCYZA", "name": "\u5b89\u4fe1\u745e\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINRUIYING3GEYUEGUNDONGCHIYOUZHAIA"}, "970059": {"suoxie": "AXRY3GYGDCYZB", "name": "\u5b89\u4fe1\u745e\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503aB", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINRUIYING3GEYUEGUNDONGCHIYOUZHAIB"}, "970060": {"suoxie": "AXRY3GYGDCYZC", "name": "\u5b89\u4fe1\u745e\u76c83\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "ANXINRUIYING3GEYUEGUNDONGCHIYOUZHAIC"}, "970061": {"suoxie": "HXLXZZGSGYCYA", "name": "\u534e\u946b\u4e50\u4eab\u5468\u5468\u8d2d\u4e09\u4e2a\u6708\u6301\u6709A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXINLEXIANGZHOUZHOUGOUSANGEYUECHIYOUA"}, "970062": {"suoxie": "HXLXZZGSGYCYC", "name": "\u534e\u946b\u4e50\u4eab\u5468\u5468\u8d2d\u4e09\u4e2a\u6708\u6301\u6709C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "HUAXINLEXIANGZHOUZHOUGOUSANGEYUECHIYOUC"}, "970063": {"suoxie": "HAZQHYLGYCYZQ", "name": "\u534e\u5b89\u8bc1\u5238\u5408\u8d62\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANHEYINGLIUGEYUECHIYOUZHAIQUAN"}, "970064": {"suoxie": "CCZQSGYGDCYA", "name": "\u957f\u57ce\u8bc1\u5238\u4e09\u4e2a\u6708\u6eda\u52a8\u6301\u6709A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHENGQUANSANGEYUEGUNDONGCHIYOUA"}, "970065": {"suoxie": "CCZQSGYGDCYB", "name": "\u957f\u57ce\u8bc1\u5238\u4e09\u4e2a\u6708\u6eda\u52a8\u6301\u6709B", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHENGQUANSANGEYUEGUNDONGCHIYOUB"}, "970066": {"suoxie": "CCZQSGYGDCYC", "name": "\u957f\u57ce\u8bc1\u5238\u4e09\u4e2a\u6708\u6eda\u52a8\u6301\u6709C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHENGQUANSANGEYUEGUNDONGCHIYOUC"}, "970067": {"suoxie": "XZZGJQLXFSJHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6d88\u8d39\u5347\u7ea7\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXIAOFEISHENGJIHUNHEA"}, "970068": {"suoxie": "XZZGJQLXFSJHHB", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6d88\u8d39\u5347\u7ea7\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXIAOFEISHENGJIHUNHEB"}, "970069": {"suoxie": "XZZGJQLXFSJHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6d88\u8d39\u5347\u7ea7\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXIAOFEISHENGJIHUNHEC"}, "970070": {"suoxie": "GYYYSGYDKZ", "name": "\u56fd\u5143\u5143\u8d62\u56db\u4e2a\u6708\u5b9a\u5f00\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOYUANYUANYINGSIGEYUEDINGKAIZHAI"}, "970071": {"suoxie": "DYCYCXCZ", "name": "\u7b2c\u4e00\u521b\u4e1a\u521b\u4eab\u7eaf\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DIYICHUANGYECHUANGXIANGCHUNZHAI"}, "970072": {"suoxie": "HAZQHYTLZQ", "name": "\u534e\u5b89\u8bc1\u5238\u5408\u8d62\u6dfb\u5229\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANHEYINGTIANLIZHAIQUAN"}, "970073": {"suoxie": "DZRHCZYXHHA", "name": "\u4e1c\u8bc1\u878d\u6c47\u6210\u957f\u4f18\u9009\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGZHENGRONGHUICHENGZHANGYOUXUANHUNHEA"}, "970074": {"suoxie": "DZRHCZYXHHC", "name": "\u4e1c\u8bc1\u878d\u6c47\u6210\u957f\u4f18\u9009\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "DONGZHENGRONGHUICHENGZHANGYOUXUANHUNHEC"}, "970075": {"suoxie": "CCZQZDZA", "name": "\u957f\u57ce\u8bc1\u5238\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHENGQUANZHONGDUANZHAIA"}, "970076": {"suoxie": "CCZQZDZC", "name": "\u957f\u57ce\u8bc1\u5238\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "CHANGCHENGZHENGQUANZHONGDUANZHAIC"}, "970077": {"suoxie": "AXZGRXYNCYQZQA", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u946b\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINZIGUANRUIXINYINIANCHIYOUQIZHAIQUANA"}, "970078": {"suoxie": "AXZGRXYNCYQZQB", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u946b\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINZIGUANRUIXINYINIANCHIYOUQIZHAIQUANB"}, "970079": {"suoxie": "AXZGRXYNCYQZQC", "name": "\u5b89\u4fe1\u8d44\u7ba1\u745e\u946b\u4e00\u5e74\u6301\u6709\u671f\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "ANXINZIGUANRUIXINYINIANCHIYOUQIZHAIQUANC"}, "970080": {"suoxie": "DHZQHY3GYCYQ", "name": "\u4e1c\u6d77\u8bc1\u5238\u6d77\u76c83\u4e2a\u6708\u6301\u6709\u671f", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGHAIZHENGQUANHAIYING3GEYUECHIYOUQI"}, "970081": {"suoxie": "DHZQHXTLDZ", "name": "\u4e1c\u6d77\u8bc1\u5238\u6d77\u946b\u6dfb\u5229\u77ed\u503a", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGHAIZHENGQUANHAIXINTIANLIDUANZHAI"}, "970082": {"suoxie": "DHZQHXZL", "name": "\u4e1c\u6d77\u8bc1\u5238\u6d77\u946b\u5c0a\u5229", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGHAIZHENGQUANHAIXINZUNLI"}, "970083": {"suoxie": "DHZQHY6GYCYQHH", "name": "\u4e1c\u6d77\u8bc1\u5238\u6d77\u76c86\u4e2a\u6708\u6301\u6709\u671f\u6df7\u5408", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "DONGHAIZHENGQUANHAIYING6GEYUECHIYOUQIHUNHE"}, "970084": {"suoxie": "GLHFZQA", "name": "\u56fd\u8054\u6c47\u5bcc\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANHUIFUZHAIQUANA"}, "970085": {"suoxie": "GLHFZQC", "name": "\u56fd\u8054\u6c47\u5bcc\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "GUOLIANHUIFUZHAIQUANC"}, "970086": {"suoxie": "HAZQHYSGYCYZQ", "name": "\u534e\u5b89\u8bc1\u5238\u5408\u8d62\u4e09\u4e2a\u6708\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUAANZHENGQUANHEYINGSANGEYUECHIYOUZHAIQUAN"}, "970087": {"suoxie": "DWAXZDZA", "name": "\u4e1c\u5434\u5b89\u946b\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGWUANXINZHONGDUANZHAIA"}, "970088": {"suoxie": "DWAXZDZB", "name": "\u4e1c\u5434\u5b89\u946b\u4e2d\u77ed\u503aB", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGWUANXINZHONGDUANZHAIB"}, "970089": {"suoxie": "DWAXZDZC", "name": "\u4e1c\u5434\u5b89\u946b\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGWUANXINZHONGDUANZHAIC"}, "970093": {"suoxie": "XZZGJQLJHYXYNCYHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINJUNHENGYOUXUANYINIANCHIYOUHUNHEA"}, "970094": {"suoxie": "XZZGJQLJHYXYNCYHHB", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINJUNHENGYOUXUANYINIANCHIYOUHUNHEB"}, "970095": {"suoxie": "XZZGJQLJHYXYNCYHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5747\u8861\u4f18\u9009\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINJUNHENGYOUXUANYINIANCHIYOUHUNHEC"}, "970096": {"suoxie": "DZRHXY90TGDCYZDZA", "name": "\u4e1c\u8bc1\u878d\u6c47\u79a7\u60a690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUIXIYUE90TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "970097": {"suoxie": "DZRHXY90TGDCYZDZC", "name": "\u4e1c\u8bc1\u878d\u6c47\u79a7\u60a690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUIXIYUE90TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "970098": {"suoxie": "DZRHXX30TGDCYZDZA", "name": "\u4e1c\u8bc1\u878d\u6c47\u946b\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUIXINXIANG30TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "970099": {"suoxie": "DZRHXX30TGDCYZDZC", "name": "\u4e1c\u8bc1\u878d\u6c47\u946b\u4eab30\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUIXINXIANG30TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "970100": {"suoxie": "XZZGJQLXXYXYNCYQHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u4eab\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGXIANGYOUXUANYINIANCHIYOUQIHUNHEA"}, "970101": {"suoxie": "XZZGJQLXXYXYNCYQHHB", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u4eab\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGXIANGYOUXUANYINIANCHIYOUQIHUNHEB"}, "970102": {"suoxie": "XZZGJQLXXYXYNCYQHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u4eab\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGXIANGYOUXUANYINIANCHIYOUQIHUNHEC"}, "970103": {"suoxie": "HCZQCXYNCYZQA", "name": "\u534e\u521b\u8bc1\u5238\u521b\u4eab\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUACHUANGZHENGQUANCHUANGXIANGYINIANCHIYOUZHAIQUANA"}, "970104": {"suoxie": "HCZQCXYNCYZQB", "name": "\u534e\u521b\u8bc1\u5238\u521b\u4eab\u4e00\u5e74\u6301\u6709\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HUACHUANGZHENGQUANCHUANGXIANGYINIANCHIYOUZHAIQUANB"}, "970105": {"suoxie": "AXZGTLBHB", "name": "\u5b89\u4fe1\u8d44\u7ba1\u5929\u5229\u5b9d\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "ANXINZIGUANTIANLIBAOHUOBI"}, "970106": {"suoxie": "DYCYCHYGYCYZ", "name": "\u7b2c\u4e00\u521b\u4e1a\u521b\u548c\u4e00\u4e2a\u6708\u6301\u6709\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "DIYICHUANGYECHUANGHEYIGEYUECHIYOUZHAI"}, "970107": {"suoxie": "DHHXSY3GYCYA", "name": "\u4e1c\u6d77\u6d77\u946b\u53cc\u60a63\u4e2a\u6708\u6301\u6709A", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGHAIHAIXINSHUANGYUE3GEYUECHIYOUA"}, "970108": {"suoxie": "DHHXSY3GYCYC", "name": "\u4e1c\u6d77\u6d77\u946b\u53cc\u60a63\u4e2a\u6708\u6301\u6709C", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGHAIHAIXINSHUANGYUE3GEYUECHIYOUC"}, "970109": {"suoxie": "GLJRYSLYNCYZQA", "name": "\u56fd\u8054\u91d1\u5982\u610f\u53cc\u5229\u4e00\u5e74\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANJINRUYISHUANGLIYINIANCHIYOUZHAIQUANA"}, "970110": {"suoxie": "GLJRYSLYNCYZQB", "name": "\u56fd\u8054\u91d1\u5982\u610f\u53cc\u5229\u4e00\u5e74\u6301\u6709\u503a\u5238B", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANJINRUYISHUANGLIYINIANCHIYOUZHAIQUANB"}, "970111": {"suoxie": "GLJRYSLYNCYZQC", "name": "\u56fd\u8054\u91d1\u5982\u610f\u53cc\u5229\u4e00\u5e74\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "GUOLIANJINRUYISHUANGLIYINIANCHIYOUZHAIQUANC"}, "970112": {"suoxie": "XZZGJQLXRYXYNCYQHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u777f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGRUIYOUXUANYINIANCHIYOUQIHUNHEA"}, "970113": {"suoxie": "XZZGJQLXRYXYNCYQHHB", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u777f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGRUIYOUXUANYINIANCHIYOUQIHUNHEB"}, "970114": {"suoxie": "XZZGJQLXRYXYNCYQHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u5174\u777f\u4f18\u9009\u4e00\u5e74\u6301\u6709\u671f\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u80a1", "pinyin": "XINGZHENGZIGUANJINQILINXINGRUIYOUXUANYINIANCHIYOUQIHUNHEC"}, "970116": {"suoxie": "FZZQXJGHB", "name": "\u65b9\u6b63\u8bc1\u5238\u73b0\u91d1\u6e2f\u8d27\u5e01", "style": "\u8d27\u5e01\u578b", "pinyin": "FANGZHENGZHENGQUANXIANJINGANGHUOBI"}, "970117": {"suoxie": "DWYF6GYCYZQA", "name": "\u4e1c\u5434\u88d5\u4e306\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGWUYUFENG6GEYUECHIYOUZHAIQUANA"}, "970118": {"suoxie": "DWYF6GYCYZQC", "name": "\u4e1c\u5434\u88d5\u4e306\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DONGWUYUFENG6GEYUECHIYOUZHAIQUANC"}, "970119": {"suoxie": "XZZGJQLHRZYYNCYHHA", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6052\u777f\u81f4\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408A", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGZHENGZIGUANJINQILINHENGRUIZHIYUANYINIANCHIYOUHUNHEA"}, "970120": {"suoxie": "XZZGJQLHRZYYNCYHHB", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6052\u777f\u81f4\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408B", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGZHENGZIGUANJINQILINHENGRUIZHIYUANYINIANCHIYOUHUNHEB"}, "970121": {"suoxie": "XZZGJQLHRZYYNCYHHC", "name": "\u5174\u8bc1\u8d44\u7ba1\u91d1\u9e92\u9e9f\u6052\u777f\u81f4\u8fdc\u4e00\u5e74\u6301\u6709\u6df7\u5408C", "style": "\u6df7\u5408\u578b-\u504f\u503a", "pinyin": "XINGZHENGZIGUANJINQILINHENGRUIZHIYUANYINIANCHIYOUHUNHEC"}, "970122": {"suoxie": "SZHLZQA", "name": "\u4e0a\u8bc1\u5f18\u5229\u503a\u5238A", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGZHENGHONGLIZHAIQUANA"}, "970123": {"suoxie": "SZHLZQC", "name": "\u4e0a\u8bc1\u5f18\u5229\u503a\u5238C", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "SHANGZHENGHONGLIZHAIQUANC"}, "970125": {"suoxie": "YHYXLGYCYZQA", "name": "\u94f6\u6cb3\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238A", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEYOUXUANLIUGEYUECHIYOUZHAIQUANA"}, "970126": {"suoxie": "YHYXLGYCYZQC", "name": "\u94f6\u6cb3\u4f18\u9009\u516d\u4e2a\u6708\u6301\u6709\u503a\u5238C", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "YINHEYOUXUANLIUGEYUECHIYOUZHAIQUANC"}, "970127": {"suoxie": "DBZGYYX30TGDZA", "name": "\u5fb7\u90a6\u8d44\u7ba1\u6708\u6708\u946b30\u5929\u6eda\u52a8\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGZIGUANYUEYUEXIN30TIANGUNDONGZHAIA"}, "970128": {"suoxie": "DBZGYYX30TGDZC", "name": "\u5fb7\u90a6\u8d44\u7ba1\u6708\u6708\u946b30\u5929\u6eda\u52a8\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "DEBANGZIGUANYUEYUEXIN30TIANGUNDONGZHAIC"}, "970129": {"suoxie": "XDYYY30TCYZQ", "name": "\u4fe1\u8fbe\u6708\u6708\u76c830\u5929\u6301\u6709\u503a\u5238", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "XINDAYUEYUEYING30TIANCHIYOUZHAIQUAN"}, "970132": {"suoxie": "DZRHTTYZDZA", "name": "\u4e1c\u8bc1\u878d\u6c47\u6dfb\u6dfb\u76ca\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUITIANTIANYIZHONGDUANZHAIA"}, "970133": {"suoxie": "DZRHTTYZDZC", "name": "\u4e1c\u8bc1\u878d\u6c47\u6dfb\u6dfb\u76ca\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u4e2d\u77ed\u503a", "pinyin": "DONGZHENGRONGHUITIANTIANYIZHONGDUANZHAIC"}, "970134": {"suoxie": "HTAR90TGDCYZDZA", "name": "\u6d77\u901a\u5b89\u6da690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aA", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANRUN90TIANGUNDONGCHIYOUZHONGDUANZHAIA"}, "970135": {"suoxie": "HTAR90TGDCYZDZC", "name": "\u6d77\u901a\u5b89\u6da690\u5929\u6eda\u52a8\u6301\u6709\u4e2d\u77ed\u503aC", "style": "\u503a\u5238\u578b-\u6df7\u5408\u503a", "pinyin": "HAITONGANRUN90TIANGUNDONGCHIYOUZHONGDUANZHAIC"}, "980003": {"suoxie": "TPYLGYGDCYZ", "name": "\u592a\u5e73\u6d0b\u516d\u4e2a\u6708\u6eda\u52a8\u6301\u6709\u503a", "style": "\u503a\u5238\u578b-\u957f\u503a", "pinyin": "TAIPINGYANGLIUGEYUEGUNDONGCHIYOUZHAI"

In [11]:
def get_fund_info_per1_list(url_fund):
    # 每个基金一页一天数据，per=1，返回list
    
    list_info = [] # 存储基金信息
    
    # re.findall,xpath 匹配信息
    response = requests.get(url_fund)
    text = response.text
    text_html = re.findall('content:"(.*?)",records:',text)[0]
    html = etree.HTML(text_html)
    html_data = html.xpath('//tr/td')
    for info in html_data:
        list_info.append(info.text)  # 每个字段一行：净值日期 单位净值 累计净值 日增长率 申购状态 赎回状态 分红送配
    
    return list_info

In [12]:
def get_timestamp(date_str):
    # 输入'2012-02-19' 得到浮点数时间戳
    return time.mktime(time.strptime(date_str,"%Y-%m-%d"))

In [13]:
def get_cnt_days(sdate,edate):  # 计算时间段间天数，截止时间是否大于起止时间
    stime = time.mktime(time.strptime(sdate,"%Y-%m-%d"))
    etime = time.mktime(time.strptime(edate,"%Y-%m-%d"))
    cnt_days = count_days = int((edate - sdate)/(24*60*60))+1
    return (cnt_days, False if cnt_days<0 else True)

In [14]:
def get_fund_data_url(fund_id='512000',page=1,sdate='2021-09-16',edate='2021-09-16',per=1):
    return 'https://fundf10.eastmoney.com/F10DataApi.aspx?type=lsjz&code={}&page={}&sdate={}&edate={}&per={}'.format(fund_id,page,sdate,edate,per)

In [15]:
def get_fund_data_list(fund_id,pages,start_date,end_date,per):
    fund_data_list = []
    for page in range(pages):
        fund_id_url = get_fund_data_url(fund_id,page+1,start_date,end_date,per)
        #print("fund_id_url:{}".format(fund_id_url))
        tmp_list = get_fund_info_per1_list(fund_id_url)
        if tmp_list == ['暂无数据']:
            break
        else:
            fund_data_list.append(tmp_list)
        #print("current data:{}".format(tmp_list))
    return fund_data_list

In [16]:
def get_cnt_pages(tmp_fund_data_url):
    # 输入基金数据网址，反悔此条件下一共有多少页数据
    response = requests.get(tmp_fund_data_url)
    text = response.text
    #print(text)
    text_html = re.findall(',records:(.*?),pages:',text)
    #print(text_html)
    return int(text_html[0])

In [17]:
def save2excel(fund_id_list,results,file_path='../fund_res.xlsx',use_name=False):
    writer = pd.ExcelWriter(file_path)
    if use_name:
        if os.path.exists('fund_info.json'):
            fund_json = get_fund_json()
        else:
            fund_json = make_fund_json()
    for idx,fund_id in enumerate(fund_id_list):
        df_fund = pd.DataFrame(results[idx][::-1])
        df_fund.columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态','分红送配']
        if use_name:
            df_fund.to_excel(writer,sheet_name=str(fund_id+fund_json[fund_id]['name']))
        else:
            df_fund.to_excel(writer,sheet_name=str(fund_id))
        print("Finished {}".format(fund_id))
    writer.save()
    print("Finished [{}] .".format(file_path))

In [ ]:
def save2basevalue(fund_id_list,results,file_path='../fund_res.xlsx',use_name=False):
    writer = pd.ExcelWriter(file_path)
    res_sheet = []
    if use_name:
        if os.path.exists('fund_info.json'):
            fund_json = get_fund_json()
        else:
            fund_json = make_fund_json()
    for idx,fund_id in enumerate(fund_id_list):
        df_fund = pd.DataFrame(results[idx][::-1])
        df_fund.columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态','分红送配']
        if use_name:
            df_fund.to_excel(writer,sheet_name=str(fund_id+fund_json[fund_id]['name']))
        else:
            df_fund.to_excel(writer,sheet_name=str(fund_id))
        print("Finished {}".format(fund_id))
    writer.save()
    print("Finished [{}] .".format(file_path))  

In [18]:
fund_id_list=['164906',
'000071',
'004424',
'004752',
'001552',
'006327',
'001469',
'162411',
'000942',
'000216',
'000968',
'001051',
'110027',
'100032',
'000614',
'002708',
'001052',
'000478',
'050025',
'502010',
'001180',
'100038',
'161017',
'001064']
basevalue=[1.4569,
1.4047,
2.4457,
0.7726,
0.9568,
1.0963,
1.0594,
0.4606,
1.2307,
1.1303,
0.9458,
0.9152,
1.5391,
0.8898,
1.0683,
2.4707,
0.5583,
2.1436,
2.2822,
0.8603,
0.7741,
1.3438,
1.4342,
0.688]

In [19]:
#fund_id_list = ['512000','512880','512070']  # 基金代码
#fund_id_list = ['513050','513130','159920','159740']
fund_id_list = ['159938','512980','159920','512000','512880','510050'] 
# 每个基金的时间区间
time_interval = [['2021-01-01','2021-10-13'],
                 ['2021-01-01','2021-10-13'],
                 ['2021-01-01','2021-10-13'],
                 ['2021-01-01','2021-10-13'],
                 ['2021-01-01','2021-10-13'],
                 ['2021-01-01','2021-10-13'],
                ]
#fund_id = 512000
#start_date = '2021-09-13'
#end_date = '2021-09-16'
per = 1

In [20]:
get_results = []
for idx,fund_id in enumerate(fund_id_list):
    
    # 获取时间节点并检查
    # todo：检查结束时间
    start_date,end_date = time_interval[idx]  # 获取时间区间
    build_date = get_start_date(fund_id)
    build_time_stamp = get_timestamp(build_date)
    start_time_stamp = get_timestamp(start_date)
    if start_time_stamp < build_time_stamp:
        start_date = build_date  # 最早开始时间是建立时间
        print("{}的最早开始时间更正为创建时间:{}".format(fund_id,start_date))
    
    # 获取当前条件下一共多少页数据
    pages = get_cnt_pages(get_fund_data_url(fund_id,1,start_date,end_date,per))
    
    print("fund_id:{},[{}-{}],per:{},pages:{}".format(fund_id,start_date,end_date,per,pages))
    # 提取基金数据
    fund_data = get_fund_data_list(fund_id,pages,start_date,end_date,per)
    get_results.append(fund_data) # 此时每个基金的结果list为倒排需要用get_results[?][::-1]

fund_id:159938,[2021-01-01-2021-10-13],per:1,pages:186
fund_id:512980,[2021-01-01-2021-10-13],per:1,pages:186
fund_id:159920,[2021-01-01-2021-10-13],per:1,pages:186
fund_id:512000,[2021-01-01-2021-10-13],per:1,pages:186
fund_id:512880,[2021-01-01-2021-10-13],per:1,pages:186
fund_id:510050,[2021-01-01-2021-10-13],per:1,pages:186


In [19]:
save2excel(fund_id_list,get_results,file_path='../fund_res_all.xlsx',use_name=True)

Finished 159938
Finished 512980
Finished 159920
Finished 512000
Finished 512880
Finished 510050
Finished [../fund_res_all.xlsx] .


In [36]:
df_found_i = pd.DataFrame(get_results[0])
########为DataFrame数据设置columns
df_found_i.columns=['净值日期','单位净值','累计净值','日增长率','申购状态','赎回状态','分红送配']
#########为了数据便于自己使用，文件命名，依据基金的中文名称 。
df_found_i.to_excel('../%s.xlsx'%(512000))


In [54]:
get_results[0][::-1]

[['2016-08-30', '1.0000', '1.0000', None, '场内买入', '场内卖出', None],
 ['2016-08-31', '1.0011', '1.0011', '0.11%', '场内买入', '场内卖出', None],
 ['2016-09-01', '0.9988', '0.9988', '-0.23%', '场内买入', '场内卖出', None],
 ['2016-09-02', '1.0000', '1.0000', '0.12%', '场内买入', '场内卖出', None],
 ['2016-09-07', '1.0005', '1.0005', None, '场内买入', '场内卖出', None],
 ['2016-09-09', '0.9979', '0.9979', None, '场内买入', '场内卖出', None],
 ['2016-09-13', '0.9893', '0.9893', None, '场内买入', '场内卖出', None],
 ['2016-09-14', '0.9804', '0.9804', '-0.90%', '场内买入', '场内卖出', None],
 ['2016-09-19', '0.9839', '0.9839', '0.36%', '场内买入', '场内卖出', None],
 ['2016-09-20', '0.9822', '0.9822', '-0.17%', '场内买入', '场内卖出', None],
 ['2016-09-21', '0.9889', '0.9889', '0.68%', '场内买入', '场内卖出', None],
 ['2016-09-22', '0.9897', '0.9897', '0.08%', '场内买入', '场内卖出', None],
 ['2016-09-23', '0.9836', '0.9836', '-0.62%', '场内买入', '场内卖出', None],
 ['2016-09-26', '0.9671', '0.9671', '-1.68%', '场内买入', '场内卖出', None],
 ['2016-09-27', '0.9767', '0.9767', '0.99%', '场内买入', '场